In [118]:
from collections import OrderedDict
import numpy as np
import spacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        word_s =["‘", 'chahie|', "\\", '/', 'tha|', 'haim', 'haim|', '`', "'","’", '-', 'jaenge|', 'haim|', '"', 'hai)|', 'paenge|', '\n', '(', ')' ,'-', 'hai|','?', '!', ',', '|', '.', ';', ':', 'andar', 'ata', 'adi', 'apa', 'apana', 'apani', 'apani', 'apane', 'abhi', 'abhi',  'adi', 'apa', 'inhim', 'inhem', 'inhom', 'itayadi', 'ityadi', 'ina', 'inaka', 'inhim', 'inhem', 'inhom', 'isa', 'isaka', 'isaki', 'isaki', 'isake', 'isamem', 'isi', 'isi', 'ise', 'unhim', 'unhem', 'unhom', 'una', 'unaka', 'unaki', 'unaki', 'unake', 'unako', 'unhim', 'unhem', 'unhom', 'usa', 'usake', 'usi', 'usi', 'use', 'eka', 'evam', 'esa', 'ese', 'aise', 'ora', 'aura', 'kai', 'kai', 'kara', 'karata', 'karate', 'karana', 'karane', 'karem', 'kahate', 'kaha', 'ka', 'kaphi', 'kai', 'ki', 'kinhem', 'kinhom', 'kitana', 'kinhem', 'kinhom', 'kiya', 'kira', 'kisa', 'kisi', 'kisi', 'kise', 'ki', 'kuchha', 'kula', 'ke', 'ko', 'koi', 'koi', 'kona', 'konasa', 'kauna', 'kaunasa', 'gaya', 'ghara', 'jaba', 'jaham', 'jaham', 'ja', 'jinhem', 'jinhom', 'jitana', 'jidhara', 'jina', 'jinhem', 'jinhom', 'jisa', 'jise', 'jidhara', 'jesa', 'jese', 'jaisa', 'jaise', 'jo', 'taka', 'taba', 'taraha', 'tinhem', 'tinhom', 'tina', 'tinhem', 'tinhom', 'tisa', 'tise', 'to', 'tha', 'thi', 'thi', 'the', 'dabara', 'davara', 'diya', 'dusara', 'dusare', 'dusare', 'do', 'dvara', 'na', 'nahim', 'nahim', 'na', 'niche', 'nihayata', 'niche', 'ne', 'para', 'pahale', 'pura', 'pura', 'pe', 'phira', 'bani', 'bani', 'bahi', 'bahi', 'bahuta', 'bada', 'bala', 'bilakula', 'bhi', 'bhitara', 'bhi', 'bhitara', 'magara', 'mano', 'me', 'mem', 'yadi', 'yaha', 'yaham', 'yaham', 'yahi', 'yahi', 'ya', 'yiha', 'ye', 'rakhem', 'ravasa', 'raha', 'rahe', 'vasa', 'lie', 'liye', 'lekina', 'va', 'vageraha', 'varaga', 'varga', 'vaha', 'vaham', 'vaham', 'vahim', 'vahim', 'vale', 'vuha', 've', 'va', 'airaha', 'sanga', 'sakata', 'sakate', 'sabase', 'sabhi', 'sabhi', 'satha', 'sabuta', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone''sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone']
       
        for word in word_s:
           lexeme = nlp.vocab[word]
           lexeme.is_stop = True 
            
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                #if token.pos_ in candidate_pos and token.is_stop is False:
                if token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
            
       # print(sentences)    
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        token_pairs_freq = []
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])    
                    token_pairs.append(pair)
        #for word in token_pairs:
         #   print(word)
        unique_values = []
        unique_values = pd.Series(token_pairs).unique()
        #for i in unique_values:
            #print(i)
            
        arr = pd.DataFrame([])
       
        ls = []
        for i in unique_values:
            count = 0
            for j in token_pairs:
                if i == j:
                    count = count + 1
                else:
                    continue
            #print(i, count)
            lm = []
            lm = [i,count]
            ls.append(lm)
            #arr = arr.append(pd.DataFrame({'Pairs': pd.Series(data = [i]), 'Freq': pd.Series(data = [count])}))
        
        #for word in ls:
         #   print(word)
            
        data = pd.DataFrame(ls,columns=['Pairs','Freq'])
        data = data.sort_values('Freq', ascending = False)
        for i in data['Freq']:
            print(i)
        #for i in data['Pairs']:
         #   print(i)
        print(data)
         
      
        
        #print(token_pairs_freq)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight
        
        
        
text = '''prabhavita sandhi ke charom ora ki mansapeshiyom ko rogi ko sandhi ko gati dete hue ichchhapurvaka sikoda़na tatha shithila karana chahie, ise sthira vyayama kaha jata hai, aura hara ghante mem 2-3 minata taka kiya jata hai| jaise-jaise rogi ki sthiti mem sudhara hota hai, vyayama ko badha़ana chahie, para bahuta jaldi-jaldi nahim, nahim to bimari ka punaravartana ho sakata hai| aba dhire-dhire vyayama ko badha़ana chahie kintu atyasakti se bimari ka punaravartana ho sakata hai| rogi apane arama ke samaya ko kama kara sakata hai| khapachchi ko lambe samaya ke lie hataya ja sakata hai ya kevala rata ke samaya rakha ja sakata hai| dhire-dhire, sandhiyom ke vyayama se, gatishilata badha़ jaegi| rogi ko apane apa vyayama karana chahie aura kuchha vyayamom ko chikitsaka ki sahayata se kara sakata hai| vaha eka vakara ki sahayata se ya kisi ke sahare se chala sakata hai| vaha baisakhi se chala sakata hai| baisakhi pakshaghata se bachane ke lie kuhani baisakhi ki apeksha kankha ki baisakhi ka upayoga karem| baisakhi ke bada rogi chhada़i ke sahare se chala sakata hai| dhire-dhire rogi apani mansapeshiyom ke niyantrana ko badha़ana prarambha karata hai|
yaha saba usaki ayu, rogi ke prayatna tatha vaha isase shighra hi chhutakara pane ke lie kitana kritasankalpa hai, isa para nirbhara karega| jirna avastha mem vyayama atyavashyaka haim, nahim to vikriti nishchita hai| yadi sandhiyom ka uchita rupa se vyayama nahim kiya gaya, to ve muda़ne ki avastha mem bhi sthira ho jati haim, mansapeshiyam patali tatha vyartha ho jati haim aura uttejaka utpada sandhiyom ke charom ora jama ho jate haim| amavati sandhishotha mem kai anya vikritiyam hoti haim| yadi rogi narama bistara para sota raha hai, to graiva merudanda, thoda़i ke vakshasthala ke samipa pahunchane taka jukati hai| rogi aisa lagata hai jaise vaha eka gole mem guda़i-muda़i hokara leta hai| haddiyam asthisushira ho jati haim aura prayah thoda़e se sharirika prashikshana se tuta jati haim| ina rogiyom ko eka suniyojita aura paryavekshita vyayama upachara ki avashyakata hoti hai| dhire-dhire unaki gatishilata tatha mansapeshiyom ki kshamata badha़ti jaegi| vyayama se purva sandhiyom ko kuchha ushma upachara bhi die ja sakate haim| yaha mansapeshiyom ko shithila karane mem sahayata karata hai tatha rakta apurti ko badha़ata hai aura isilie yaha vyayama adhika prabhavi ho jata hai| yaha dhire-dhira sandhi ke kada़epana tatha sankuchana ko sudharata hai aura isamem chikitsaka tatha rogi donom ki ora se bahuta adhika dhairya tatha driढ़ta ki avashyakata hoti hai| prabhavita sandhi ke lie pahala vyayama chikitsaka dvara eka lambe satra mem sikhaya jana chahie| chikitsaka dvara vyayama ka kama se kama saptaha mem do bara nirikshana hona chahie tatha sudhara para dhyana diya jana chahie| vyayama tala sarvottama sahayaka haim, jaham rogi apani sandhiyom ko svatantratapurvaka sabhi dishaom mem chala sakata hai| vishesha sandhi vyayamom ke satha-satha samanya vyayama bhi diya jana chahie| mansapeshiyom ki kshamata ko badha़ane ke lie pratirodhaka vyayama die jane chahie jo sharira ki gatividhi mem tatha kama karate samaya sahara deti haim| yadi apaka vajana 70 kilo hai, to pani mem apake pairom ko kevala 7 kilo ko sahara dena hoga| yadi apa kamaja़ora pairom ke karana jaise adharangaghata mem ya kashtakara ghutanom ke karana jaise sandhishotha mem, khada़e nahim ho sakate, to sambhavatah apa santulana karane mem sakshama honge; pani apako aisa karane mem sahayata karega| pani sanstambhi mansapeshiyom ko shithila karata hai taki apa pani mem adhika asani se vyayama kara sakem| pani tanava kama karata hai aura yaha tanavapurna rogiyom ke lie mahattvapurna sahayata hai| yadi tala ke pani ka tapamana 37 digri sentigre़da hai to yaha apako behatara arama dega tatha darda aura pida़a ko kama karega| ghutanom ko sidhe rakhate hue pairom ko eka ke bada eka chalaem| kuchha minata chalane ke bada, rogi ke, pindaliyom ki mansapeshiyom mem tivra pida़a hoti hai aura vaha kuchha minatom ke arama se kama ho jati hai| yaha rakta vahikaom ki sankirnata ke karana hoti hai|
chalate samaya rakta apurti mem vriddhi honi hi chahie| jaba rogi chalata hai to rakta ki avashyakata badha़ jati hai aura yadi, rakta vahikaom ki sankirnata ke karana, rakta ki apurti mem vriddhi nahim hoti hai, to vaha eka araktatajanya sthiti mem pahuncha jati hai| ina rogiyom ko dhunrapana banda kara dena chahie, kyonki aise rogiyom mem sigareta pine ka hanikaraka prabhava hota hai| unhem pratidina eka ghante taka teja़i se chalane ke lie kaha jata hai| jaba pida़a ki punaravritti hoti hai to unhem kuchha minatom ke lie arama karane aura taba teja़i se chalana punah arambha karane ki anumati di jati hai| dhamanikakathinya abhilopa ke lakshanom vale rogiyo ko jo bistara ya kursi taka simita rahate haim, chikitsaka ya parivara ke kisi sadasya ki sahayata se kie jane vale kramika pratibandhatmaka gatividhiyom se vyayama karana chahie| vyayama karane ke lie ayu rukavata nahim hai| eka 67 sala ka vyakti savirama langada़epana ke satha kevala 5 blaka chala saka| unhem prarambha mem 1/2 se 1 kimi. taka chalana chahie aura jaise-jaise svasthya anumati deta hai vaise-vaise pha़asale ko dhire-dhire badha़ana chahie| ise sarala mukta hasta vyayamom ya si़dha़i vale vyayamom, mitara vale saikila ya treda mila ke satha sammilita kiya ja sakata hai|
unhem variyata se yadi duri kama ho to banisbata kara se jane ke chalana chahie| isa sudhara ke satha, unhem vyayama ke samaya hridaya gati mem giravata anubhava hoga| yaham taka ve bhi jinhem roga ke gambhira prakara haim, apane vyayama starom aura aksijana antargrahana ko bisa pratishata badha़a sakate haim| shvasana ke ‘punarprashikshana’ mem shvasana vyayama bahuta adhika sahayaka haim| hama dama, shvasanali-shotha aura vatasphiti ke lie inhim vyayamom ka prayoga karate haim| hamanem ina vyayamom mem se kai ke sabhi achchhe sarom ko sammilita karane ki koshisha ki hai| rogi ko chikitsaka para vishvasa hona chahie| unhem vyaktigata rupa se sikhaya jana chahie aura bada mem ve kisi samuha mem sammilita ho sakate haim| jaba vaha ina vyayamom ko samuha mem karata hai, to use dusarom se protsahana milega aura vaha adhika vishvasapurna anubhava karega tatha behatara prayasa karega| arama karana dama tatha vatasphiti ke rogi ke lie sabase mahattvapurna hai| ye rogi hamesha tanavapurna, ghabaraye hue tatha thake hue hote haim kyonki hara samaya ye shvasarodha se bachane ke lie lagatara prayasa karate rahate haim| yadi apa ina rogiyom ko dhyana se dekhenge to apa tanava ke kai lakshana paenge| unake hatha mem kampakampi ho sakati hai, jaba unaki ankhem banda ho to phada़ka sakati haim, aura unake vakshasthala ke kshetra mem alpa gatividhi ke paitarna ho sakate haim kyonki ve hara samaya apane shvasana ke bare mem sacheta rahate haim| ve upari vakshasthala, kandhem, gale tatha merudanda mem adhika gatividhi dikha sakate haim, visheshakara jaba unhem koi dekha raha ho| yaha tanava tatha ye vyartha ki gatividhiyam durbhagyavasha bahuta adhika urja khatma karate haim| yadi ve arama se haim, kevala taba hi ve adhika nipunata se aura kuchha sahi tarike se shvasana karana sikha sakate haim| rogi ko ina vyartha tividhiyom ke bare mem bataya jana chahie aura udara shvasana ki gudha़ta ko bhi sikhana chahie|
use yaha bataya jana chahie ki uchita shvasana kama kashtakara hoga tatha shvasahinata ke akramana ko bhi kama kara dega| ye nai pranaliyam alpavadhi pranaliyam nahim haim, aura sthayi rupa se prayoga kie jane chahie| akramana ke samaya use ina nai pranaliyom ki ora adhika dhyana dena chahie aura use isa nae jnana se labha hoga| ina rogiyom ko ghara mem anandadayaka aura anukula vatavarana pradana karana chahie| unhem eka nishchinta pravriti bhi apanani chahie| jaba rogi ko tanava mukta hona sikhaya jata hai, to eka nihshabdata ka vatavarana hona chahie| use usake sira aura ghutanom ko takiye se sahara dekara tatha banhom ko arama se vishrama karate hue arama se lita dena chahie| use tanavapurna aura tanavamukta mansapeshiyom ke bicha antara ke bare mem batana chahie| yaha taba taka jari rakhana chahie jaba taka ki vaha kaise unhem puri taraha se tanavamukta karana hai yaha nahim jana jata| ise banha, kandhe, gale, chehare, vakshasthala tatha udara ki mansapeshiyom mem karana chahie| mansapeshiyom ke ina tanavom ko mapa ja sakata hai, aura hamem unaki saphalata ke bare mem janakari ho jaegi| jaba rogi isa tanavamukta hone ki prakriya ko sikha leta hai, to vaha vyayama ke lie taiyara hai| vaha kandhe tatha upari vakshasthala ki chhoti mansapeshiyom ko tanane aura tanavamukta karane se prarambha karata hai| shvasana ke satha-satha isi uddeshya se yojanabaddha kie gae vividha layabaddha aura dolana vyayama karae jate haim| shvasana vyayamom ko subaha ya shama mem jaba peta khali hota hai, taba karem| khada़i avastha mansapeshi ki gatividhiyom ke lie svatantrata ka labha pradana karata hai| sidhe khada़e hom, banhe apake parshva mem hom, dhire se aura gahare shvasa chhoda़em, gale ko jukaem, apane kandhom ko age lae tatha peta ko sikoda़em| apa paenge ki apa aura adhika hava chhoda़ sakate haim| aba dhire se shvasa lena arambha karem, vakshasthala, peta ka vistara karem tatha gale ko sidha karem| thoda़a aura adhika shvasa lene ke lie, apane vakshasthala ko uncha uthaem, peta ko thoda़a khincha le tatha gale ko badha़aem| kuchha kshanom ke lie shvasa ko rokem aura dhire se shvasa chhoda़na prarambha karem tatha banhom aura eda़iyom ki gatividhi ko ulata dem| kuchha kshanom ke lie shvasa ko rokem aura ise punah prarambha karem| jaba pamva ke chapa samatala hom, to rogi ko chalate samaya pida़a ka anubhava hota hai|
ghutane bhi tirachhe sanketa karate haim, isaka artha hai ki pura pamva nitamba sandhi para ghuma gaya hai| adhikatara vikritiyam pahale chara mahinom mem utpanna hote haim, jaba bachcha palatane ke lie bahuta chhota hota hai| kisi bhi sthiti mem bachche ko rakha jaya, janma ke thika bada se hi, vaha apani aramadeha avastha janata hai| tani hui mansapeshiyam kamaja़ora ho jati haim tatha sikuda़i hui mansapeshiyam shaktishali ho jati haim| 
jaba pida़a tivra aura sandhi uttejita hoti hai to eka tivra avastha ko pahuncha jati hai| ina rogiyom ka atmavishvasa ke satha upachara hona chahie, rogi taka sandesha svatah hi pahuncha jaega tatha unhem janakari ho jaegi ki ve sahi hathom mem haim| eka patale suti gadde sahita eka lakada़i ka bencha paryapta hona chahie| padapata se bachane ke lie paira ko gaddidara gatte para rakha ja sakata hai| rata mem kevala eka patala takiya dena chahie| muda़i hui kohani ke satha, bahom ko sharira se juda़a hua rakhana chahie| 
bahom ki sthiti mem nirantara parivartana hote rahana chahie| sujana ghatati jati hai, aura marija ko darda se bahuta hi jalda arama mila jata hai| niche ki ora sira karake pharsha para leta jaem, hatha ko niche ki ora va hatheliyom ko pharsha para rakhem|
'''
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=2, lower=False)
tr4w.get_keywords(25)

5
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [119]:
from collections import OrderedDict
import numpy as np
import spacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        word_s =["‘", 'chahie|', "\\", '/', 'tha|', 'haim', 'haim|', '`', "'","’", '-', 'jaenge|', 'haim|', '"', 'hai)|', 'paenge|', '\n', '(', ')' ,'-', 'hai|','?', '!', ',', '|', '.', ';', ':', 'andar', 'ata', 'adi', 'apa', 'apana', 'apani', 'apani', 'apane', 'abhi', 'abhi',  'adi', 'apa', 'inhim', 'inhem', 'inhom', 'itayadi', 'ityadi', 'ina', 'inaka', 'inhim', 'inhem', 'inhom', 'isa', 'isaka', 'isaki', 'isaki', 'isake', 'isamem', 'isi', 'isi', 'ise', 'unhim', 'unhem', 'unhom', 'una', 'unaka', 'unaki', 'unaki', 'unake', 'unako', 'unhim', 'unhem', 'unhom', 'usa', 'usake', 'usi', 'usi', 'use', 'eka', 'evam', 'esa', 'ese', 'aise', 'ora', 'aura', 'kai', 'kai', 'kara', 'karata', 'karate', 'karana', 'karane', 'karem', 'kahate', 'kaha', 'ka', 'kaphi', 'kai', 'ki', 'kinhem', 'kinhom', 'kitana', 'kinhem', 'kinhom', 'kiya', 'kira', 'kisa', 'kisi', 'kisi', 'kise', 'ki', 'kuchha', 'kula', 'ke', 'ko', 'koi', 'koi', 'kona', 'konasa', 'kauna', 'kaunasa', 'gaya', 'ghara', 'jaba', 'jaham', 'jaham', 'ja', 'jinhem', 'jinhom', 'jitana', 'jidhara', 'jina', 'jinhem', 'jinhom', 'jisa', 'jise', 'jidhara', 'jesa', 'jese', 'jaisa', 'jaise', 'jo', 'taka', 'taba', 'taraha', 'tinhem', 'tinhom', 'tina', 'tinhem', 'tinhom', 'tisa', 'tise', 'to', 'tha', 'thi', 'thi', 'the', 'dabara', 'davara', 'diya', 'dusara', 'dusare', 'dusare', 'do', 'dvara', 'na', 'nahim', 'nahim', 'na', 'niche', 'nihayata', 'niche', 'ne', 'para', 'pahale', 'pura', 'pura', 'pe', 'phira', 'bani', 'bani', 'bahi', 'bahi', 'bahuta', 'bada', 'bala', 'bilakula', 'bhi', 'bhitara', 'bhi', 'bhitara', 'magara', 'mano', 'me', 'mem', 'yadi', 'yaha', 'yaham', 'yaham', 'yahi', 'yahi', 'ya', 'yiha', 'ye', 'rakhem', 'ravasa', 'raha', 'rahe', 'vasa', 'lie', 'liye', 'lekina', 'va', 'vageraha', 'varaga', 'varga', 'vaha', 'vaham', 'vaham', 'vahim', 'vahim', 'vale', 'vuha', 've', 'va', 'airaha', 'sanga', 'sakata', 'sakate', 'sabase', 'sabhi', 'sabhi', 'satha', 'sabuta', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone''sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone']
       
        for word in word_s:
           lexeme = nlp.vocab[word]
           lexeme.is_stop = True 
            
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                #if token.pos_ in candidate_pos and token.is_stop is False:
                if token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
            
       # print(sentences)    
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        token_pairs_freq = []
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])    
                    token_pairs.append(pair)
        #for word in token_pairs:
         #   print(word)
        unique_values = []
        unique_values = pd.Series(token_pairs).unique()
        #for i in unique_values:
            #print(i)
            
        arr = pd.DataFrame([])
       
        ls = []
        for i in unique_values:
            count = 0
            for j in token_pairs:
                if i == j:
                    count = count + 1
                else:
                    continue
            #print(i, count)
            lm = []
            lm = [i,count]
            ls.append(lm)
            #arr = arr.append(pd.DataFrame({'Pairs': pd.Series(data = [i]), 'Freq': pd.Series(data = [count])}))
        
        #for word in ls:
         #   print(word)
            
        data = pd.DataFrame(ls,columns=['Pairs','Freq'])
        data = data.sort_values('Freq', ascending = False)
        #for i in data['Freq']:
         #   print(i)
        for i in data['Pairs']:
            print(i)
        print(data)
         
      
        
        #print(token_pairs_freq)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight
        
        
        
text = '''dama eka aisi avastha hai, jo shvasa marga ko prabhavita karati hai |chhoti-chhoti nalikayem jo hava ko phephaड़om mem andara-bahara le jati haim | 
jaba koi dame ka marija dama ke karaka ke samparka mem ata hai taba shvasa marga ki nali ki divara ki peshiyam kasa jati haim, jisase ki vayu-marga aura sankara ho jata hai| 
vayumarga ki sataha lala ho jati hai, aura phulana shuru kara deti hai| 
aksara chipachipa shleshma ya balagama utpanna hota hai| 
ye sare prabhava vayumarga ko aura bhi sankara aura ugra bana dete haim jisase dama ke lakshana dikhane lagate haim| 
dama ke samanya lakshanom mem khansi, gharagharahata ya chhati mem ghara-ghara ki dhvani, sansa ka phulana, chhati mem jakaड़na ka mahasusa hona shamila haim| 
dama kisi bhi unra mem ho sakata hai| 
dama ka karana kuchha bhi ho sakata hai, jo shvasa nali ko avaruddha kara dama ke lakshanom ko prakata hone para badhya karata hai| 
hara kisi ka dama bhinna hota hai, aura sambhavatah apake pasa usake prakata hone ke aneka sakriya karana ho sakate haim | 
samanya karana jaise sardi-jukama ya sardi ke satha bukhara (phlu), tambaku ka dhuam, kasarata aura kai padarthom se elarji jaise paraga ke kanom se , royendara kapaड़om ya janavarom ki khala se ya phira ghara ke dhula ke kanom se, ho sakata hai| 
bigaड़te huai dama ke lakshanom ki vajaha se kuchha logom ko apani jivana shaili ko badalana paड़ sakata hai | 
yaha pahachanana bahuta hi mushkila ho sakata hai ki kauna se karana apake dame ko badha़ate haim| 
kabhi-kabhi sambandha suspashta hota hai, udaharana ke taura para jaba apa kisi billi ya paraga kanom ke samparka mem ate haim to kuchha hi minitom mem apake dame ke lakshana ubharane lagate haim| 
kuchha logom mem dera se pratikriya prarambha hoti hai| 
aise karaka jo apake dama ko aura bigaड़ dem unase bachakara aura sahi samaya para dava lekara apa anavashyaka lakshanom ko kama kara sakate haim aura samanya jivanacharya ke annada ko ka़ayama rakha sakate haim| 
kuchha shreshtha upachara upalabdha haim ,jo apake dama ko niyantrita karane mem sahayaka ho sakate haim| 
dama ke adhikatara upacharom ke liye sabase adhika prabhavakari tarika hai, dava ko sansa ke jariye lena, jo ki sidhe phephaड़om taka pahunchati hai| 
bahuta prakara ke shvasayantra upalabdha haim aura sabase mahatvapurna yaha hai ki apa jo bhi yantra upayoga karate haim, vaha apake liye suvidhajanaka ho aura jise achchhi taraha se prayoga mem laya ja sake | 
apaka daॉktara ya narsa apako salaha denge ki kauna sa yantra sabase uttama hai, aura kisa prakara se isaka sahi dhanga se prayoga karana hai, yaha batayenge| 
dama ki mukhya do prakara ki aushadhiyam hoti hai -rahata dene ya piड़a kama karane vali evam, rokane vali ya nivarana karane vali| 
piड़a kama karane vale shvasayantra sadharanataya nile ranga ke hote haim, jinake lene se lakshanom ko rahata milati hai | 
sankare shvasamargom ko ghere rahane vali mansa peshiyom ko arama dekara, ye shighra kama karate haim. 
dame ke daure ke upachara mem ye jaruri hote haim| 
yadi apako apane piड़ahari shvasayantra ka saptaha mem tina ya chara bara prayoga karana paड़e, to apako daॉktara ya narsa ke pasa jana chahie jo lakshanom ka punaravalokana karake aura unako niyantrana mem la sakenge | 
nirodhaka sadharanataya bhure, lala ya narangi ranga ke shvasayantra hote haim| 
ye shvasamarga mem sujana aura jalana ko niyantrita kara aura atisamvedanashila hone se roka kara karya karate haim, aura tivra daure ke khatare ko kama karate haim| 
eka antarala ke bada kshamata baढ़ti jati hai, isaliye isako pratidina lene ki avashyakata hai| samanyatah subaha aura shama aura jaba apa svastha mahasusa kara rahe hom taba bhi | 
adhikansha nirodhakom mem stiraॉyada aushadhi rahati hai| 
yaha samajana mahatvapurna hoga ki yaha usa enabolika stiraॉyada ke samana nahim hota, jisaka upayoga khilaड़i apane pradarshana ko behatara banane ke liye karate haim| 
dusari aura bhi prakara ki davaiyam jo apake piड़ahari aura nirodhaka shvasayantra ke satha yadi jarurata ho to li ja sakati haim, jaise nirodhaka goliyam aura lambe samaya taka sakriya rahane vali piड़ahari goliyam| 
nayi davaom ki janakari ke liye jo shayada apake liye adhika asaradara hongi, apane daॉktara ya narsa se punchhiye | 
bahuta se dame ke marija aise nirodhaka ka prayoga karate haim, jisamem kama matra ki stiraॉyada sansa dvara khichi ja sakati hai | 
dama upachara mem jo stiraॉyada prayoga mem lai jati hai, vaha kaॉrtikostiraॉyada hai-jo sharira ke andara banane vali prakritika stiraॉyada ka pratirupa hoti hai| 
ye enabolika stiraॉyada se purnataya bhinna hoti hai, jisaka pahalavana aura khilada़i upayoga karate haim| 
shvasita stiraॉyada sidhe shvasamarga mem jati hai, atah bahuta thoड़i matra sharira ke shesha bhagom dvara shoshita ki jati hai | 
apake daॉktara apake liye sabase kama sambhavita matra likhenge | 
bachchom ke vikasa ke liye karibi se jancha karate rahana chahiai | 
kabhi-kabhi, yadi apake dame ke lakshana tivra ho jae taba shayada apake daॉktara apako stiraॉyada goliyom ka laghu korsa denge | 
ye stiraॉyada, shighra aura shaktishali ढ़nga se shvasamarga ki jalana ko shanta karane ka karya karate haim| 
3 se 14 dinom taka ka dava ka laghu korsa, koi bhi lambe samaya ka bura asara nahim hone dete | 
stiraॉyada ki goliyam sharira ki chechaka ke liye pratirodha ko kama karati hai yadi apa stiraॉyada ki goliyam le rahe haim , aura apako chechaka ho jaai to apa apane daॉktara se samparka kariye | 
adhikansha nirodhaka upacharom mem stiraॉyada hota hai aura unako shvasayantra dvara lene ka tatparya yaha hai ki bahuta kama matra mem stiraॉyada ka upayoga hona chahiye kyonki sansa dvara khinchi hui dava sidhe shvasamarga mem jati hai, jaham isaki jarurata hoti hai, aura bahuta thoड़i matra hi sharira ke shesha bhagom mem shoshita ho pati hai| 
jaba stiraॉyada goliyom ke rupa mem liya jata taba isaki matra adhika hoti hai (eka stiraॉyada goli mem, eka papha se 50 guna adhika matra mem hoti hai jo eka manaka stiraॉyada shvasayantra se milati hai|) aura isaki adhikasham matra na sirpha phephaड़om dvara balki sharira ke dusare bhagom dvara bhi shoshita ho jati hai | 
lambe samaya (mahinom ya varshom taka ) niyamita rupa se stiraॉyada goliyom ke prayoga se gambhira bura prabhava ho sakate haim, jaise bhangura asthiyam (asthi-sushirata ), kharoncha lagana, madhumeha, motiya binda, bhukha baढ़na, sine mem jalana, badahajami ityadi | 
ye davaiyam shayada apako udasina bana sakati haim ya manasika santulana mem bigaड़ ya phira chehare para sujana la sakati haim | 
apake dame ke lakshanom ko niyantrita karane ke liye, apake daॉktara ya narsa ko apa se lakshanom ko niyantrita karane ke sabase achchha tarike para charcha karani chahiye aura isi ke sandarbha mem apako dama ki davaom ka aura isake adhika bigaड़ne para apa kya karem isaka likhita lekha jokha dena chahiye| 
apake nirikshana ke daurana unase dame ki davaiyom ka karda aura dame ke daure mem niji karya yojana ke vishaya mem puchhiye| 
apake piड़ahari (nile) shvasayantra se do papha lijiye, evam baithakara apane kase hue kapaड़e ढ़ile kariye | 
yadi dame ke daure ke daurana turanta koi sudhara mahasusa na ho, to lagatara hara eka minita ke antarala para pancha minita taka shvasayantra se eka eka papha lete rahana chahiye ya phira lakshanom mem sudhara a jaye taba taka | 
hamare dama visheshajnom aura narsom se dubhashiyom dvara asthama yu.ke. salaha laina para gopaniya aura svatantra dama ki salaha ke liye phona para bata kariye| 
kainsara ke vishvasom mem aise tathya shamila haim, ki kainsara ka karana eka karta ya aijenta haim, (usa karta ke svabhava ko nashta karake usako roka ja sakata hai ) kainsara usa avastha mem pahachana ja sakata jaba isaka prathama aghata ho, taba upachara ka ashvasana ho sakega | kainsara ke svarupa ke vishaya mem yahi anivarya vastuem haim, jinake bare mem hamem chinta karani chahiye| 
manushya jati ke itihasa mem dushkara dhana sambandhi aura insani kimata para itane kama logom dvara itane jyada logom se itani jaldi-jaldi itana jyada jutha itane lambe samaya taka kabhi nahim bola gaya| 
kainsarolaॉji ke sampurna kshetra ke vishvasa ka nichoda़ hai, brihat anabhijnata ke satha ati karani, ati dave aura ati ashajanaka adi| 
kainsara ki paribhasha dene ki asana-si dikhane vali samasya se hi hamari anabhijnata ki shuruata hoti hai| 
kainsara visheshajna kainsara ke karana ke baढ़e-chaढ़e dave kara jnana ka bhramajala paida karate haim | 
visheshajnom dvara prastuta eka bhi karana aisa nahim hai (dhunrapana sahita), jo kainsara ki avriti ke liye uttaradayi ho evam kainsara ki anivarya sharta ho, isake bavajuda bhi, kainsarolaॉji sabhi para niyantrana rakhati hai, jisamem svayam surya bhi kainsarojanika ki taraha shamila hai| 
ina sabaka parinama kainsara bhaya hai, eka aisa roga (jisaka pratishthita nyu janarala ऑpha medisina ke divangata sampadaka ijela phingara dvara spashta rupa se varnana kiya gaya) jo kainsara jitana hi gambhira hai aura naitika rupa atyadhika vidhvansakari hai| 
kainsaravisheshajna roga ke vishaya mem ati karate haim- ve isaka ati nidana aura ati upachara karate haim -kyonki ve isa chetavani ko manane se inkara karate haim ki koi bhi kainsara arambhika sthiti mem lakshana dekhakara nahim bataya ja sakata ya rogamukta sima taka usaka upachara kiya ja sakata hai, kainsara chikitsaka jo dekhata hai, usaka hi upachara karata hai| 
upachara anukula paddhati ka bhrama mukhya rupa se marija mem hota hai jo kainsara ki upasthiti ka anubhava nahim karata ya chikitsaka use khoja pane mem hi samartha nahim hote | 
sambhavatah anumanita jivana-kala ke sandarbha mem, upachara ke satha bachane ki sambhavana, bina upachara ki tulana mem achchhi nahim hoti hai, aura isa bata ki bhi sambhavana hai ki shayada upachara se kainsara ke rogi ka jivanakala kama ho jaye | 
yaha kainsara tvacha ke thika niche hota hai| 
isake svabhavika itihasa ka shatabdiyom se adhyayana hota aya hai| 
yaha tyumara, svatah parikshana, roga vishayaka parikshana, shrenikarana, harmona rogopachara aura jo bhi apake pasa hai,usake liye eka sabase adhika suvashya hai| 
eka udaharana ke rupa mem stana kainsara, kainsirolaॉji ki sampurna viphalata ka pratika hai| 
kainsaravisheshajnom para yadi chhoड़ dem, yaha kabhi nahim hoga| 
kainsara sansthaem samvedi tarike se dimaga ko chalati haim| 
barneta sankshepa kahate hai, ki kainsarolaॉji ki sampurna loka chhavi eka manavatavadi hai na ki jiva-vijnana sambandhi vidvatta hai| 
kainsara ki dustara sachchai yaha hai ki yaha vijnana va manava ke adhina nahim hai, vainabarga ne ise vijnana se pare kaha hai | 
kainsara ke sare sulabha aura vishleshaniya pahalu - koshika, tyumara, upachara, karana aura rokathama, anuvanshakiya, isaka sabase mahatvapurna karana - kisi na kisi vajaha se svayam ko, isa purvanumana ke sahare nahim chhoda़te haim ki hama kya janate ya kara sakate haim| 
upara diye gaye pratyeka pahaluom ki tulana mem vibhinna tathyom / vishamataom ka pratirupa niche nirdharita kiya gaya hai| 
sheksapiyara ki shaili mem kainsara koshika ka ashaya hai : eka kainsara koshika, manava ke liye, eka bhayankara vyadhi hai, sara (shrimana shodha kartta ), parantu yaha mera apana mata hai| 
isiliye ,yaha dusari koshika ki taraha hi paida hokara baढ़ti hai, yaha sharira ki dusari anya samanya koshikaom ki taraha hi bina adhika svechchha se saitotaॉksika aijenta- davaom, eksa kiranom ka shikara ho jati hai | 
apani isi svayam samanata ke lakshana dvara, kainsara koshika svayam ko vikirana chikitsa ya rasayana chikitsa dvara kisi chayanashila kriya se eka bara aura sadaiva ke liye roga mukta kara leti hai| 
eka kainsara koshika apane apa mem anokhi hoti hai -jo na to kisi vishishta dava ke prati samvedanashila hoti hai aura na hi kisi tike ke dvara nivarya hai| 
kainsara koshikaem, manushya ya jiva-jantu ke sharira mem ananta hoti hai |
isiliye yaha koi ashcharya ki bata nahim hai, ki gambhira rakta kainsara (lyukemiya) ka marija yadyapi jaba nirdharita samaya ke liye koshika-visha ki bhari matra se bauchharita hota hai taba bhi rakta kainsara koshaom se kabhi mukta nahim ho pata hai | 
yaha kainsara ke yatharthavadi tathya ka utkrishta udaharana hai ki kainsara ki kuchha koshikaom ko nashta kiya ja sakata hai,jabaki svatah kainsara ko nahim |
kainsara koshikaom ka apani utpatti ki jagaha chhoड़kara dusari jagaha jane ka guna purva nirdharita, vyaktivadi, aura apurvanumeya hota hai | 
eka kainsara koshika ki anishchitata aura usake vyaktitva ke karana hi kainsara koshika madala banane ka prashna hi nahim uthata| 
padartha ke samana hi kainsara koshika svayam ko paribhashita karane ki chunauti deti hai| 
hama basa itana kara sakate haim ki vishaya ki saiddhantika shabdadambara se vyakhya karem| 
kainsara koshika, yaha kya hai, jo karati hai usake ke liye jimmedara hai, jo use karana hai vaha karati hi hai , jo vaha hai usake liye vaha kya karati hai, aura vaha kya hai| 
upari panktiyam smitharsa ke vyapakikarana ko vistrita karati haim, ki kainsara koshika ka koi alaga sanrachanatmaka tatva nahim hai,parantu yaha eka grahi anga hai| 
kainsara ki sampurna prakriya ke bare mem bhi yahi kaha ja sakata hai| 
kipalinga ke anusara tyumara shodha ka uddeshya (roga vishayaka kainsara shodha) eka kainsara ke marija mem tyumara banane ke bare mem, kya, kyom, kaba, kaise, kaham, aura kauna, janana hai| 
ghatana ke pashchata hi eka chikitsaka apane sabhi upakaranom sahita jnani banata hai | 
usake liye yaha janana mushkila hai ki tyumara kaba lailaja hai, chahe marija pahali bara mile ya upachara hone ke bada mile| 
sabhi anubhavi roga visheshajna janate haim, ki pratyeka tyumara apani sukshma sanrachana ki vishishtata ko pradarshita karata hai| 
kyom aura kaise akaranavada , kainsara-ke-prabhava aura tyumara ke banane se sambandhita hai, jo jivana ke anivarya anga haim| 
jisa taraha eka samanya koshika aura usaka pratyakshikarana hamare liye eka rahasya hai,usi taraha eka kainsara koshika aura usaka pratyakshikarana bhi| 
samuhika stara para kya, kaham, aura kauna purvanumeya nishchitataem haim, parantu vyaktigata stara para kevala sambhavanaem haim| 
gambhira limphoblastika lyukemiya rakta kainsara ke eka svarupa dvara mahamariya dharana ki sambhavana ka achchhi taraha se vyakhyana kiya ja sakata hai |
vishvavyapi rupa se, thoड़i-si vividhata ke satha eka desha se dusare desha mem, prativarsha 100,000 ki jana-sankhaya mem do ya tina mamalom mem yaha hota hai | 
isa silasile mem, yaha jaruri nahim ki agali ghatana dvara pichhali ghatana ka anusarana ho| 
kainsara sansthaom dvara prastuta daravane akaड़e, samanyatah upara di gayi kalpita sada eka-si vriddhi ke satha ghatanaom ki shrrinkhala para adharita haim| 
aisi apariharya vikasa ki kami, vastava mem, prayogashala mem upayoga kiye jane vale jantu-tyumara-madala ko amanya kara deti hai| 
aisa mata ki, kainsara ka upachara jaruri hai| isaka mukhya dosha yaha hai ki , upachara ke bada marija jarura bachega ya achchha mahasusa karata hai, na ki usake upachara ke bavajuda| 
eka shalya chikitsaka, jisane isa siddhanta ke liye bhari kimata di, usane karsinoma ke adhunika upachara ke uttaraprabhavom ka varnana apane anubhavom ke eka niji spashtikarana mem kiya| 
1938 mem likhe isa lekha ki prasangikata 1994 mem bhi dhumila nahim hui | 
kainsara upachara vidhi ki koi bhi pranali kainsara ko thika nahim kara sakati hai, sabhi isake pata lagane yogya pratyakshikarana para hi dhava bolate haim| 
halanki gupta rupa se phailane vala kainsara eka nirvivada tathya hai phira bhi sare rogopacharom ka prayoga kiya hi jata hai| 
shalyachikitsa kainsara ke phailava ko baढ़ava de sakati hai jabaki baki rogopachara paddhatiyam usaki upasthiti ka baढ़ava deti haim| 
kainsara rogopachara jivana ko asana ज़rura bana sakata hai parantu jivana nahim baढ़a sakata usase koi pha़raka nahim paड़ta ki vaha kaba aura kaise diya gaya hai| 
prachanda lasika ke adhishveta raktata (limphaitika lyukemiya) evam chhati ke kainsara (braista kainsara) mem bimari ki avadhi ka nrityu dara se koi sambandha nahim hota| 
atah yaha asha karana uchita hoga ki eka prabhavashali rasayanika upacharatmaka ejenta bhi inamem se kisi bhi bimari ke marija़ ke jivana ko nahim badha़a sakata, nishchaya hi jivana para rasayanopachara ke prabhava ko dekhana abhi taka sambhava nahim ho paya hai| 
hala hi mem vayaskom mem gambhira meloblastika lyukomiya ke lie isa vishaya para ज़ora dala gaya hai| 
unako manushya ya pashuom ke kainsara se koi lena dena nahim hai| 
isa alagava ko isa tathya se anubhava kiya ja sakata hai ki prakritika ya otochithaॉnasa kainsara una davaom ke prati 100% pratirodhi hai jo tathakathita pratiropita kainsara ke viruddha 100% prabhavakari haim| 
kainsaroloji mem manushya ya pashu sthiti se sambandha rakhane vala koi kainsara-therapi-maॉdala nahim hai| 
parikshana naliyom mem prapta saphalata parikshana naliyom taka hi simita raha gayi haim| 
yaha isa bata ko spashta kara deta hai ki kainsaroloji usa karana ko dhundha rahi hai jo kabhi tha hi nahim| 
jisaka koi karana nahim hai usaka koi upachara bhi nahim ho sakata| 
****kainsara eka vishishtha kasheru ki lakshana hai jo samuhika stara para karya karata hai, evam yaha bahugunanakhandiya vanshanugata dvara bhi phailata hai| 
anuvanshikata ka nirmula nirdharana gausiyana kahavata mem sankshiptikarana kiya gaya hai- daimanausa hairaiditsa| 
vishva bhara mem chini janasankhya mem naisophrayainjiyala karasinoma evam yahudi logom mem lyukemiya kisa varga evam vibhinna ayu ke vyaktiyom mem kisa taraha ka kainsara milega yaha samuha ka milajula jina bhandara nirdharita karata hai| 
kainsara hone ki sambhavana vyaktigata rupa se hara vyakti mem usaki jina ke dvara nirdharita nahim hoti balki samuhika rupa se jina samuha ke satha pratikriya hoti hai, yaha vaha anubhuti jo batati hai ki kainsara ka karana anuvanshikata nahim,balki eka (junda-lakshana,harda-iti) hai,aura nashchita rupa se eka vaha tathya jo anuvanshikata janane se pare - spashta evam transa-jenitiksa hai | 
adhunika anuvanshiki ne apani eka-jina-eka-lakshana ki dharana ko chhoड़kara yaha svikara kara liya hai ki kai jinom dvara ekala lakshana niyantrita kiye jate haim | 
manava ke 100.000 jinom mem se kauna-si koshikaem isa prakara ki koshikaom ko kainsara koshikaom mem badalane se rokati haim, isaka pata nahim lagaya ja saka hai, aisa shayada isaliye ki abhi taka jina ki koi spashta paribhasha bhi nahim pata hai| 
kainsara ke jinsa ka pata lagane ki koi bhi koshisha pareshaniyom se bhari hui hai jo ki anuvanshiki vijnana se pare haim| 
baranata ne hala hi mem isa bhrama ki ora sanketa kiya hai ki i.kolai ko jo ho sakata hai vo eka hathi mem bhi phailaya sakata hai| 
jina prakalana dvara kainsara ka upachara utana hi kathina hai jitana samaja jata hai| 
svargiya pha़auldsa ne kainsara para vijaya pane ke bajaya usako aura achchhe se sasaja kara usaki upasthiti ka karana janane ke lie vicharashila anusandhana ki ja़rurata para ja़ora diya tha| 
satya to yaha kisi aura ki apeksha hamare pasa ja़yada tathya haim para yaha nahim pata ki unaka karana kya hai|
aura jo nirvivada tathya hamare pasa haim vaha kainsara ko samajane ke lie kaफ़i haim| 
atah kainsara ke karana ya upachara ke spashtikarana para ki gayi yaha samasta anusandhana prakriya nishchaya hi parinama ki drishti se bekara rahi hai| 
vicharashila kainsara anusandhana mem tina visheshataom ki avashyakata hoti haim- shalinata se yaha vichara karana ki insana janavarom se behatara hai, eka samanyajna ka vyapaka drishtikona aura ina donom ke dvara yaha svikara karana ki kainsara jivana evam marana ka abhinna anga hai| 
kainsara anusandhaniya hai parantu kevala jnana ke stara para| 
paryavarana ki drishti se unaki paristhiti samana pratita hoti hai| 
sabase jyada rochaka bata yaha hai ki lagabhaga eka jaisi paristhiti mem raha rahe anya dalom ki apeksha kuchha vishesha dala ke logom mem hama kuchha kainsara bahuta samanya pate haim| 
sankshepa mem kaha ja sakata hai ki kainsara manaviya stara para bhi eka sarvajnata samanya lakshana hai jo ki purva anumanita kainsara ke jinsa se svatantra aura yaha apane sarvabhaumika prabhava mem tatastha hota hai| 
bhale hi apa prakriti ko ज़ora laga kara bahara kara dem parantu apaki murkhata ka majaka uड़ati hui prakriti rahasyamaya tarike se apani vijaya prapta karake rahegi| 
kainsara ki mulabhutata ka matalaba hota hai ki yaha vyaktiyom ka vikasatmaka karyakrama hai jo nirdharita karata hai ki kainsara hoga ya nahim| 
yadi yaha usake karyakrama ka bhaga nahim hai, to koi ise utpanna bhi nahim kara sakata ; yadi hai to koi ise bacha nahim sakata hai|
kainsara apane khuda ke sharira va rakta se paida hota hai| 
thika yahi tathya horaitiyana ki uparyukta manushya se sambandhita kainsara ki kahavata ko sabita karata hai| 
kula milakara sare rogopachara prakriti ko bahara nahim nikala sakate, jo apane apa ko eka manushya ke kainsara ke rupa mem pradarshita karate haim| 
manava evam pashu chaturyami ikaiyam haim, jinaka chautha ayama samaya hai| 
portamaina ke shabdom mem, pashu apane shuruata se hi yane virya evam dimba ke milapa se yugmanaja ke nirmana se hi yaha samaya ke samanurupa hai|
amataura para sabhi sharirika vikasa ya kshaya mem parivartana purva nirdharita yojana ke satha-satha hote haim yaha yojana samaya ke satha-satha prakata hoti hai| 
yaha kainsara ki samaya niyantritata hai jo eka 14 varsha ke laड़ke ya 94 varsha ke adami ke bhojana nali ke kainsara ki upasthiti ko nirdharita karata hai| 
isa prakara ki upasthiti ka samaya prayah vitarita hota hai|
portamaina kita ke badalava ki bata karate hue eka vishesha suchana ke bare mem batate haim ki halanki hama eka tarike se abhi taka yaha nahim janate haim ki paripakva jivadhari ande mem pahale se hi rupa le leta hai| 
phauldsa pashu jivana ke dvara aisi purva-yojana ka karya se pahale nirnaya ke rupa mem ullekha karate haim| 
kainsara eka asthayi ankaड़a adhara hai, isaka karyakrama pahale se hi vyakti mem antahsthapita hota hai aura vyakti ki ayu ke satha apane apa ko prakata karata hai| 
purva-yojana phira se eka bara phauldsa ke dvara varnita karya se pahale ke nirnaya ki samanya sakaratmaka va nakaratmaka donom hi paristhitiyom ke samana hai| 
udaharana ke taura para pahala atinaitikatavadi dhunrapana na karate hue bhi phephaड़om ke kainsara se mara jata hai aura dusara lagatara dhunrapana karane vala bina kisi phephaड़om ke kainsara aura kisi aura kainsara ke bina apani nabbe varsha ki ayu mem bhi khushahala jivana bitata hai| 
eka vyakti ke samayika vikasa ke bhaga ke rupa mem kainsara ka upa siddhanta ekadishi samaya ke samana hai, yaha aparivartaniya hai| 
samanyatah hara jati mem vriddhavastha ka rupa eka samana hi hota hai, chahem kolijana mem pha़iज़iko-kaimikala parivartana dvara jancha karai jae , rakta-nalikaom mem apakarshaka parivartana ka prabhava ya atyanta asadhya bimari ke prabhava ka karana ho| 
nishchita rupa se sara yaha hai ki para hara jati ke liye anuvanshika samaya para karyakrama nirdharita hai| 
eka aisi jaivika ghaड़i aura sadhana jarura hona chahie jisake dvara prakriya shreni ko vikasa mulaka uttarajivi jaruratom ke anusara pahale ya bada mem ghatita kiya ja sake | 
kainsara ki paintrajektoriyala utpatti ne , manushya ke antarika garbhavastha se vriddhavastha taka, jiryamana kahalane vali prakriya se isaki rokathama ki hai ,vyutpatti vishayaka karanom ke liye prakriya apane apa mem kevala jaragrasta vyakti mem utpanna hone ki sambhavana rakhati hai| 
vriddhavastha ki paribhasha mulabhuta prakriya se ki gai hai jo eka vyakti mem nrityu ki sambhavata ko baढ़ati hai| 
kainsara jisa kisi bhi unra mem ghatita hota hai yaha eka antarika prakriya hai jo nrityu ki sambhavata ko baढ़ati hai chahe yaha do varsha ke bachche mem ho ya assi varsha ke vyakti mem ho | 
vastava mem isaka nrityu ki sambhavana ko baढ़ane ka karya adhika gambhira taba hota hai jaba yaha kisi yuva vyakti ko prabhavita karata hai| 
eka vriddha manushya mem vividha, halke ya ausata darje ki vriddhavasthiya prakriyaem eka prabhavashali pranaghataka kula yoga utpanna karati hai| 
strehalara ne vastavikata ,pragatishilata aura vinashakata ko vriddhavastha ki kasauti bataya hai| 
kainsara se marane vala eka bachcha vriddhavasthiya prakriya se marata hai| 
vaha eka aisi prakriya se marata hai jo antarika, vinashakari aura pragatishila thi aura bachcha jaba jinda tha taba usake marane ki sambhavana ko baढ़ane mem sahayoga diya tha | 
sankshepa mem, yaha parikalpana ki kainsara eka antarika, samaya ke adhina va vriddhavasthiya prakriya hai, yaha kainsara ki prakriti para eka purnakara drishtikona hai| 
antarikata kainsarojana ko svikara nahim karati hai| 
samayika prakriti ke karana graiva kainsara ka eka yuva laड़ki aura eka sattara varshiya mahila mem hona samaja ja sakata hai|samayikata kainsara ko pratigamana ki anusati nahim deti hai, eka aisa parinama jo kainsara vijnana ke dvara pramanita hota hai|kainsarojana ki samayika prakriti ise sahi arthom mem kainsara-priponara banati hai|
kainsara ki jiryamana prakriti ke karana yaha kai nrityu purva shaktiyom mem se eka hai; jiryamana prakriti ki vajaha se kainsara avashyaka rupa se pranaghataka prakriya se alaga rakhata hai| 
pholdasa ne kainsara ke liye sena ki lokapriya shabdavali jaise hatyara, katila ya khuni, dushmana ityadi ka upayoga karane para bhartsana ki| 
kainsara ko kisi manushya ki mauta ke liye jimmevara nahim thaharaya ja sakata hai|isaliye kainsara ke liye isa prakara ki shabdavali ka upayoga karana sahi nahim hai|
upacharita ya ana upacharita kainsara ke hone para bhi jivita rahane ke brihada ankaड़om se prerita hokara vatarahausa ne salaha di hai ki kainsara ke lakshana hone para bhi eka manushya ko jivana bima ke phayadom se vanchita nahim karana chahie | 
bina isa para - tuma pranaghataka ho ,kahane ka aropa lagate hue yaha kainsara ko jivana ke samagra hisse ka darja deta hai| 
kainsara ke marija ki nrityu jarura hogi ,yadi kainsara se nahim , to bhi (kisi anya karana se) kainsara ko satha lekara to marenge hi| 
bahuta sare vriddha vyaktiyom ki dirghakalika adhishveta raktata (lyukemiya) roga ke satha nrityu hoti hai. 
jonsa ne prasangavasha aspashta sharira vijnana sambandhi tantra jo marija ki nrityu ka karana banati hai, aura usake satha kainsara ka bhi, ullekha kiya hai| 
kainsara vaijnaniki se pare hai| 
yakrita koshanu adhikatara eka sadharana koshanu ki taraha hai, jisamem akriti sambandhi koi guna nahim hote haim jo usako asadharana banaye | 
phira bhi yaha eka atyadhika adhunika audyogika rasayanika karakhana hai | 
smitharsa ne kainsara koshika ke bare mem jo bataya usa para jora dene ke liye yaham yakrita koshika ka ullekha kiya gaya hai- donom mem hi spashta karane yogya sanrachanatmaka astitva nahim hai, kevala vyavaharika anga hai| 
kainsara ki koshika eka kadama age jati hai |
alaga-alaga janavarom mem yakrita koshikaem samana dikhati aura vyavahara karati hai jabaki kainsara koshikaem aisa nahim karati hai| 
pratyeka samaya eka kainsara banata hai,jati prakata hoti hai-eka nayi jati banati hai,jaise yaha abhutapurva adbhuta aura doharayi nahim ja sakati | 
kainsara vijnana ki utkrishta sima usake vaidha mulabhava -kainsara koshika ki ajnanata para nirbhara hai | 
vainabarga ne eka prashna ko vijnana se pare kaha hai jaba isako vijnana se puchha sakem parantu vijnana dvara isa para koi uttara na diya ja sake .isa taraha ka eka prashna vijnana se pare hai. 
karanata /chikitsyata kainsara ka eka aisa hi prashna hai| 
isake viparita davom ke bavajuda kainsara vijnana eka gaira vijnana hai| 
jaba koi jiva pratirodha karane mem asamartha hota hai, jaise kainsara mem , taba usake apane svayam ke lakshanom dvara nirdharita anukrama para aura vishishta ढ़nga se nashta hone lagata hai| 
roga eka vyaktigata ghatana hai| 
yaha svayam vyakti vishesha ko bhi shamila karata hai| 
aleksisa karela ke isa kathana ne eka aura vyaktivadi tippani ko kainsara ki vaijnanika se pare prakriti mem joड़a | 
dhyana rahe ki karela jiva vinasha para charcha karate haim parantu ढ़nga aura chala ka nirdharana jiva ke apane gunom se hota hai| 
kainsara vijnana isiliye vyakti ki dohari vishishtata ka samana karata hai ,jisaka jaivika prakshepa patha purva nirdharita hai, aura kainsara jo apurvanumeya hai| 
kainsara ke karanom ka adhyayana karane se pata chalata hai ki kainsara vijnana kabhi bhi kainsara shuru nahim kara sakata jaba koshikaem ya jiva aisa nahim chahate haim|
jaba kisi se kainsara ke hone ka dava kiya jaye to yaha bhranti rahi hai ki isake bada kainsara hota hai, isaliye yaha kainsara ka karana hai| 
pangu kara dene vale ina sabhi pratibandhom ke karana ,yaha samajana asana hai ki kainsara vijnana kainsara ke khilapha kabhi bhi kuchha nahim kara paya hai aura na kara sakega sivaya isake ki isaka jaivika ghatana ke rupa mem adhyayana kare| 
kainsara apane bahuta se tathyom mem bodhaganya hai aura samuhika stara para isaka bartava purvanumeya hai| 
vijnana ka vyutpattika taura para matalaba hai janana na ki karana | 
yadi hama use samajane ka prayasa kara le to kainsara vijnana se pare nahim hai| 
yaha janakari ki kainsara avashya rupa se gaira nidanatmaka aura lailaja hai, aisi dharana hamem kainsara para kama na karane ki tarapha age dhakela sakati hai| 
munsipha , mumbai ke eka prakhyata shalya chikitsaka , jinaka takiya kalama tha, ki eka achchha shalya chikitsaka vo hai jo janata hai ki kaba use shalya chikitsa nahim karani chahie | 
aja ke takaniki daura mem eka ayurvijnani daॉktara ko yaha janana jaruri hai ki kaba use shalya chikitsa nahim karani chahie | 
chikitsa ke kshetra mem nishkriyata ki disha mem yaha andolana teji se gati pakaड़ raha hai| 
larda platasa ki niji aura vivadaspada atmakatha mem bahuta kuchha likha hai ki kaise adhunika davaiyom se bachana chahie | 
barabara kulitana ne abhi hala mem vijnana mem stana kainsara pata lagane ke vishaya para rapata di hai neshanala kainsara instityuta aura amerikana kainsara sosaiti ke dvara sanyukta rupa se maimographi (stana kainsara ki jancha ),bayopasi ,aura shalya chikitsa ke dvara spashtikarana yojana ko nirupita kiya gaya hai| 
upara diye gaye kathana ki pushti ke liye purahsthasambandhi (prastetika kainsara) kainsara ke adhyayana se kulitana yaha darshate hai ki bahuta se protestika kainsara apane samvahaka ko pareshana nahim karate hai| 
tatparya yaha hai ki eka aise roga ka upachara karake una para koi kripa nahim karate hai ,jaba roga unake kisi pareshani ka karana nahim bana raha hai| 
kainsara ke bare mem lekhaka ki eka pustaka, jo 1973 mem prakashita hui, se eka anuchchheda yaham duharae jane yogya hai|
kuchha bhi na karana – jaba taka kainsara ke marija ke marज़ se badhya na ho jaem taba taka na nidana karana aura na upachara - yaha ananubhutachikitsa, svechchhacharita, eka prakara ke mera-adara-karo-tuma-(marija)-tatha-prakriti mem vishvasa ki parakashtha ka rupa hai|
yaha asvikriti ko eka suyogya ashvasana ya socha-samaja bhari parityaga-bhavana ka sahara dekara hastakshepa karane se mana karana hai|
mana ki kabhi bhi kuchha na karane ki naubata nahim ati, parantu jaba kabhi jaruri ho, yaha kabhi kuchha nahim jepharsana ka pavitra chhala hona chahie|
isa para adhika ja़ora dene ki avashyakata ho hi nahim sakati ki koi bhi chikitsaka pahale eka salahakara hota hai, karta sirpha tabhi jaba paristhiti badhya kare|
yadi koi marija usase puchhe ki kahim vaha chikitsakiya anveshana ki duniya mem yulesija़ to nahim hai, to bhale hi vaha rasayana chikitsa se mara dala jae, ya 'goldena geta brija' se niche gira paड़e, chikitsaka ka paramarsha to eka joradara nahim hi hona chahie, jisake lie marija ko na to usaka shulka chukane se mana karana chahie aura na hi use kanuna ki kachahari mem ghasitana chahie|
kainsara vijnana taka ke kshetra mem meri ka marga (chintana aura akriyata ka marga) martha ke marga se adhika varenya hai, aisa samajane ka tatparya ho sakata hai manasika/kayika manava lagata ke apavyaya para, janavarom para aura isake pratidana ke rupa mem kainsara ki samagra samasya para kathina shrama se uparjita dhana ki atyanta bachata|
eka hridaya roga visheshajna ne tippani ki hai ki i ke epha (i si ji) mashina ne paramanu bama se kahim jyada tabahi machai hai|
yaha tabahi isa mashina dvara janita hridaya vikshipti ke rupa mem hai|
krishchiyana barnarda ikeji (isiji) ke vidyutiya arekha ki bata karate haim jo uddeshyapurna jivana jine vale khushadila insanom ko nirasha ke bhayabhita, dukhi praniyom mem badala dalate haim|
hairisana ne tippani ki hai ki chikitsaka loga ikejiaitisa roga se grasta haim aura unake marijom ka bhagavana hi bhala karem|
krausa ne sahi hi kaha hai ki nidana sarvadhika samanya rogom mem se eka hai|
abataka hama loga yaha bodha vikasita karane mem sakshama ho gae haim ki yadi kainsara takaliphadeha na ho to kuchha bhi karane ki jarurata nahim hai, yaham taka ki nidana bhi nahim, aura isa prakara sabhi anusandhanom para hone vala kharcha bachaya ja sakata hai jo anyatha hota hai|
ekabara yadi nidana ho jane para upachara apariharya nahim hai|
yadi kainsara takaliphadeha hai, to alpatama hi kiya jana chahie|
ajakala, amula shalya-kriya ke bada stana kainsara ke sandarbha mem jo parinama prapta hote haim lagabhaga vaise hi parinama mahaja stana-arbuda ke uchchhedana ke bada samane ate haim|
aisa nyunatama upachara anya prakara ke kaisarom mem bhi vyavaharya hai - purusha-granthi, peta, agnyashaya, ityadi|
kainsara visheshajnom ko aura adhika dhana kamane ke lie logom ke satha samanom ki taraha vyavahara karane ki chhuta nahim hai|
salom se kainsara ke ऑpareshana marija ke labha ke bajaya vyaktigata labha ke lie kie jate rahe; abhi bhi kula milakara sthiti mem koi antara nahim aya hai|
anavashyaka shalya-kriya' ki vartamana dara shalya chikitsakom ke ‘arthika’ labha para shaॉ dvara kie gae prahara ko nyayasangata thaharati hai|
arle ke shabdom mem kaha jae to abhi taka kainsara sambandhi shodha ka artha yaha nikala hai ki jyadatara loga kainsara se marane ke bajaya kainsara ke para jite haim| 
phira bhi, mera vishvasa hai ki amariki aushadhi ke sara-sankshepa ko koi vyakti isa ukti ke adhara para uchita rupa se tarkasangata karara de sakata hai ki jo kisi roga-dasha ka chuhe ya kutte mem upachara sambhava bana sakata hai use, utane hi dhana, prayasa aura buddhi ke satha, manushya ki aushadhi para bhi lagu kiya ja sakata hai|
barneta ne jo bata amerika ke bare mem kahi hai, kisi bhi anya desha para lagu ki ja sakati hai|
bambai mem 1952 mem sansthapita kainsara shodha sansthana bharata sarakara ke paramanu urja sansthapana ke adhina eka vitta-poshita sanstha hai|
isa pancha-manjili imarata mem ve adhikatara chijem haim jo kisi kainsara sansthana mem ho sakati haim|
yaha apani eka bahurangi, janasamanya ke upayoga ke lie uddishta, pustika mem tamama chatukaritapurna varnana deta hai jisamem samanya, kainsara-purva evam kainsarakaraka utakom ke adhyayana ke lie 200,000 guna baड़a avardhana dene vale philipa iema 300 ilektraॉna sukshmadarshi ki janakari bhi shamila hai|
vijnana ko manushya ke lie satya ki khoja hona chahie|
bajata anuprayoga ka ekamatra ansha hai jisamem ja़ra bhi sara hai| 
vittiya koshom ke upayoga ke sandarbha mem rashtriya svasthya sansthana (amarika) ke adhyayana anubhaga ke eka sadasya ki uparyukta tippani kainsara vijnana ke bare mem hiksana ka eka drishtanta hai ki jaba kya kiya jae yaha samaja mem na ae to aura adhika arthika sahayata ki manga karo|
sankaya arjita vittiya kosha ke prashasana mem nimagna haim, aura unaka prachalita sahityika kaushala hai vittiya kosha ke lie yachana-prastava likhana|
sankshepa mem, akadamika jivana vichara-chintana ke lie na hokara, sakriyata ka bana gaya hai|
aja vijnana mem kisi vyakti ki pahachana isa bata se hoti hai ki vaha kitana kosha la raha hai|
amerika mem, kainsara ke shodha ke lie varshika vyaya shighra hi 1 kharaba daॉlara ke stara para pahuncha jaega, aura sirpha़ mudra-sphiti se kadama milate hue ise prati varsha kama se kama 1 karoड़ daॉlara ki dara se baढ़ana hoga|
kainsara ke pasa pitja़a, madhyanha bhojana, thietara partiyam, kosha ekatrita karane ke luau aura chamaka-damaka haim, aura 'vastavika daॉlara tatha pratishtha mem', hridaya/mana: roga bhi isaki barabari nahim kara sakate haim|
unhem chinta karane ki jarurata nahim; kainsara unhem nirasha nahim karega; kainsara ke bare mem logom ki samaja aisa kara sakati hai|
kainsara vijnana ka yaha sansara hai jonsom ke satha-satha na chala pane ki ati samanya manaviya asaphalata|
kisi sanstha dvara arjita raikhika gativardhaka kisi anya (sanstha) ke jyada baड़e gativardhaka ke samane lajjita ho uthata hai|
barmaina yaha vivarana dete haim ki kisa prakara jaba kisi namachina hasti ko kuchha ho jata hai to sansthaem isase apani punji jutane mem bhiड़ jati haim|
bharata ke rashtrapati, sanjiva reddi, ko phephaड़e ke arbuda se grasta paya gaya|
hamesha ki taraha ke dhumadhama ke satha unhem vimana se kainsara shodha ke makka- slona-ketaringa instityuta, nyuyaॉrka, le jaya gaya|
kisi prakara, bharata sarakara ko yaha samaja diya gaya ki aisa to karana hi tha, kyonki bharata ke pasa raikhika gativardhaka nahim hai|
kisi ne virodha jataya, parantu sansthapana ke kolahala mem usaki avaja duba kara raha gai|
rashtrapati ki bimari se apeksha ki jati hai ki vaha bharata ke pramukha kainsara kendrom ke lie pratishthita raikhika gativardhaka ki virasata chhoड़ jaegi|
varsha 1968 ke lie amerika ne, pratyaksha ya apratyaksha rupa se, kainsara para 15 biliyana daॉlara kharcha kie|
1994 mem svasthya avadhana para kula vyaya pratidina biliyana daॉlara se bhi adhika jane ka anumana hai|
nissandeha, isaka eka kapha़i baड़a hissa avashya hi kainsara ke nidana, punarnidana, upachara tatha punaruupachara ke lie hoga|
bahuta sare bharatiya jo svayam apane evam samaja ke lie eka upayogi jivana ji sakane mem sakshama haim, dhana ke abhava mem upachara ki yachana karane jate haim - chahe vaha tyubarkyulosisa, kushtha, ya jirna poliyonyelaitisa ke lie ho - jabaki apani tamama parishkrita suvidhaom ke satha kainsara shodha evam upachara bambai, dilli, kalakatta aura madrasa ke mukhya kendrom mem chhae hue haim jaham ve sanriddha pashchimi jagata ka andhanukarana karate haim|
svayam bharatiya kainsara sosayati, apane janma se hi, 'amerikana kainsara sosayati ki mahatvapurna seva se utprerita' thi|
ashcharya ki koi bata nahim ki hama raikhika gativardhakom ki kharidari ke lie nikala pada़e haim|
visheshikrita sansthanom mem 'hama roga ke bare mem saba janate haim' ka mahaula chhaya hua hai|
hamari chinta yaham bada vale mudde ko lekara hai|
rogom mem sabase daravane nama, kainsara, ke satha yaha sahaja hi hai ki loga visheshajnata kendrom ki talasha karate haim|
vigata varsha mem hamane kainsara para prashansaniya aura vistrita chikitsopachara karya gaira-visheshajnata vale sadharana aspatalom mem dekha hai|
bambai ka visheshajnata-prapta kainsara kendra, tata memoriyala haॉspitala, mastishka kainsara para 'sadharana' aspatalom (jinase lekhaka juड़e hue haim) ke mukabale kama karya karata hai jabaki usake pasa eka tantrika shalya vibhaga hai joki mastishka kainsara ke mamalom ke lie bambai se bahara ke marijom ke lie bhi eka sandarbha kendra bana chuka hai|
sadharana aspatalom mem kainsara ke bare mem kie gae nidana, utakavikriti tatha shava-pariksha sambandhi adhyayana bhi mahatvapurna hote haim|
phira bhi jo sarvadhika mahatvapurna bata hai vaha yaha tathya hai ki kisi sadharana aspatala mem upacharita kainsara ka mamala jaba kisi visheshajnata-prapta kendra mem upacharita hota hai, to jyada badatara parinama nahim deta hai - eka aisi sachchai jo vaishvika pushti ki anumati deti hai|
kainsara ke upacharakarta ka mukhya kama hai gathana dekhana, shalya-kriya dvara use katakara hata dena aura/ya eksa-re, davaom aura haॉrmonom ke jariye use kama kara dena|
kainsara ka roga-nidana, arthata kainsarakaraka gathana/gathanom ka pata lagana naidanika parikshana aura janchom para adharita hota hai jo ki kisi bhi sadharana aspatala ki prakaryatmaka kshamata se bahara ki bata nahim hai |
kainsara ke upachara ka mukhya adhara hai shalya-kriya aura ise jyadatara susajjita sadharana aspatalom mem puri taraha se sampadita kiya ja sakata hai|
kainsara ke marija ko jo chahie vaha hai avashyaka 'roga-nidana' aura bina samaya gamvae 'upachara'|
kainsara ko avisheshikrita kara dene se ise prapta karane mem madada milegi|
logom ko yaha sachchai bataie ki kauna upachara kara raha hai aura kaham, yaha mahatvapurna nahim hai balki yaha ki kisa vyakti ka aura kisa chija ke lie ilaja kiya ja raha hai|
kainsara ko svikara karana, vastu-sthiti jaisi hai usa para vichara karana, bina kisi hila-havale ke, bina galati ya ulajana ke, apane apa mem avishkarom ki samasta paidavara se adhika bhadra vastu hai|
phransisa bekana ka ahvana kainsara chikitsaka aura kainsara ke marija donom hi ke lie prasangika hai|
yaha eka shabda ‘vichara’ apane apa mem vinanrata, dhairya aura niyantrana ki avashyakata ka sandesha lie hue hai|
kainsara ko samajane ka artha hai kainsara ko svikara karana, garima ke satha|
thika vaise hi jaise upayukta vyakti ke lie 'roga-nidana' apariharya hai, kainsara se grasta marija ka 'upachara' bhi avashyaka hai|
jori grahama ka yaha kahana bilkula sahi hai ki kainsara anya kai rogom ki tulana mem adhika sadhya hai|
parantu isa bata ko pracharita kie jane se purva grahama dvara sanketita upachara ko samaje jane ki jarurata hai|
amerika, kanada aura greta britena mem apane 32 varshom ke anubhava mem mainne antarika kainsara ya stana kainsara se grasta kisi bhi marija ko usa rupa mem thika hote nahim dekha hai jaisaki ama loga thika hone shabda ka artha samajate haim - arthata roga-prakriya ko dura bhaga dena aura use kabhi lautane na dena|
aura dekhabhala kie jane ki isa yogyata mem eka ora to roga ki tivrata kama karana shamila hai aura dusari ora jivana ka sammana karane ke upaya|
kainsara para eka sammanita alekha, jisaka shirshaka hai 'kainsara medisina', isaka sachcha aura bhavya uddeshya prakata kara raha hota hai jaba vaha eka jagaha para yaha kahata hai ki kainsara ka lakshanika upachara 'sarvottama naidanika prabandhana' ka anga hai aura vaha 'kisi bhi vishishta kainsara upachara ke merudanda' ka nirmana karata hai|
chahe jitana bhi kudha़a jae, yaha agami parivartana ka eka sanketa hai ki paramparagata anga-sanrakshi prakriyaem aba angachchhedaka kainsara chikitsa ki jagaha le rahi hai|
pashchima mem hala ka 'haॉsapisa muvamenta' isa tathya ki vyapaka hoti svikaryata jalakata hai ki antakari kainsara se pida़ita marija ko bhi, anya sabhi batom se pahale, sharirika aura manasika donom hi rupom mem apane astitva ke sammana ki jarurata hai|
kainsara ke bare mem tathyom ka athaha sagara – kai salom se puri duniya mem aneka vaijnanikom dvara kie gae samarpita karyom ke parinama - eka upayogi, vyavaharya sanshleshana taka pahunchane dene ke lie varjaniya rupa se vishala laga sakate haim|
isa pustaka mem prastuta avadharanaem aura sachchaiyam kuchha aura hi kahati haim - naidanika evam anubhavika shodha ke parinamom ko shodhakartaom, chikitsakom, ama logom, aura sabase baढ़kara, kainsara ke marijom ko rochaka evam vyapaka lagane vale eka pariprekshyatmaka rupa mem ekikrita kara sakana sambhava hai|
kainsara kiya nahim ja sakata, ise roka nahim ja sakata|
yada rakhie ki yugom se kainsara manavajati ke sanga raha hai aura isaka ghatita hona na to prakriti ka kopa hai aura na hi usaki ora se danda|
pratyeka kainsara apake apane hi sva ka eka anga hai|
yadi apa ise pyara nahim kara sakate to apako isase ghrina karane ki bhi jarurata nahim hai|
hara kainsara apako, ya apake chikitsaka ko, tanga karane se pahale chira kala se apake sanga raha hai|
kainsara ka jalda nidana/upachara eka mithaka hai jise daphana diya jana chahie|
upara uddhrita kie gae karanom se, apako kainsara ke lie apani jancha karana bilkula bhi jaruri nahim hai|
kainsara ke lie khuda ko tabhi pareshana kijie jaba, aura sirpha jaba, vaha vakai apako pareshana karane lage|
kainsara hamesha jana nahim leta aura na hi yaha hamesha nidana ya upachara ke bada eka alpakalina jivana ki ora sanketa karata hai|
apane kainsara ke satha jina taya kara lijie jaba taka ki kainsara apake satha marana pasanda kare|
samajie ki yaha jaruri nahim ki kainsara apake peshe ya apake jivana ke utsaha ko bhanga kare|
chunki kainsara ke ilaja jaisi koi chija nahim hai, ata: lakshanom ke upachara para ja़ora dijie aura isase age kuchha nahim|
chikitsopachara ka koi bhi amula parivartanavada dava se ki gai ghrinaspada ati-hatya hai|
yadi apako upachara karana hi hai to shalya-kriya ki talasha kijie; yadi apako vikirana chikitsa ya rasayanopachara diya jae to kama se kama ka agraha kijie aura phira usa koshikiya ugahi ko chukane ke lie taiyara rahie jise sira se paira taka apake sharira ko ada karana hoga|
eka sammanajanaka mauta ke rupa mem apa apane sharira aura apani atma ke prati kartavya se bandhe haim|
svayam ko nrityu ki garima se vanchita mata kijie|
kainsara eka prajati hai, eka varga hai, eka kramagata charitra hai|
na to apa ise uttaradhikara mem pa sakate haim aura na hi apani santati ko virasata mem de sakate haim|
mahaja shabdakosha mem nihita artha se pare jane ki cheshta ki gai hai jisase adhika vyapaka pariprekshya prastuta kiya ja sake|
vyakhya ke anta mem sambandhita shabda koshthaka mem ullikhita kie gae haim|
yaha hippokretasa ke vichara-mantra praimama naॉna nokeyara para adharita eka samiti hai, jisaka artha hai ki koi bhi upachara kisi marija ke lie jo nyunatama karya kara sakata hai vaha hai use hani na pahunchana|
eka satva, jise sharira mem pravishta karate hi vaha pratirakshi (eka prakara ka protina) aura/ya koshikaom (lasakoshikaom) ke rupa mem eka atyuchcha vishishta pratikriya ko janma deta hai|
pratirakshi/koshikaom ke satha kisi pratijana ki vishishta pratikriyatmakata prayogashala parikshana mem rakta ya utaka mem isaki upasthiti ka jnana karati hai|
kuchha kainsara apani koshikaom ka samvahana karate haim aura/ya rakta pratijanom mem unaka utsarjana kara dete haim, jisaka eka udaharana hai karkatabhruniya (karsinoembraॉyanika) pratijana (siie)|
sabase adhika samanya rupa se udarantra nali ke kainsara ke satha paya jane vala siie, ka prayogashala mem kisi vishishta pratirakshi ke satha isaki pratisamvedanashilata pramanita karake pata lagaya jata hai|
isamem mukha ke pradaha sahita chhote, pida़adayaka chhale hote haim|
dhamaniyom ki sthulata evam kathorata vriddhavastha ke satha eka ama bata hai|
yaha kisi vyakti ke svayam ke utakom se utpanna hota hai; pratiropita nahim kiya jata|
isaka artha hai, ki ye svasthya ya jivana ke lie ashankaprada nahim hai; asadhya ke viparita, gaira kainsaragrasta hai|
eka susadhya arbuda/apavriddhi mem eka samanya utaka ke sadrishya sukshmadarshiya lakshana hote haim|
shvasanali (vayuvahika) ke age shvasa pranala vayu-margom ka eka pravibhajana hai|
phephada़e ka kainsara ama taura para baड़e shvasa pranala mem prarambha hota hai, tatha isalie ise prayah shvasani kainsara ya shvasani karsinoma kaha jata hai|
yaha abhilakshaka sukshmadarshiya chitra vala eka lasika kainsara hai, jo yuvavastha mem jyada ama hai|
ise barkita ka lasika kainsara bhi kaha jata hai|
yaha eka samanya koshika ki kainsaragrasta karane ki kshamata hai tatha isilie yaha kainsara koshika mem badala jati hai|
yaha aise padartha ki ora sanketa karata hai jo kathita rupa se kainsara paida karata hai|
vaha padartha, jo kisi kainsarakari ki sahayata karata hai, sahakainsarakari kahalata hai, tatha vaha jo isaki kriya-vidhi ka virodha karata hai, pratikainsarakari kahalata hai|
isaka artha hai ki aisi bimari jo lambi avadhi dvara abhilakshita ho, ya eka lambe samaya taka usaki barambara punaravritti ho, tatha prayah usaki gambhirata dhire-dhire badha़ rahi ho; ugrata ke viruddha|
yaha lasikakoshikaom ke sarvatra atyadhika prasarana se abhilakshita, dhire-dhire badha़ne vali adhishvetaraktata ka eka prakara hai|
yaha mukhyatah adheda़ tatha vriddhavastha ki bimari hai|
chirakalika asthimajja adhishvetaraktata, prayah siemaela ke rupa mem sankshipta ki hui, dhire-dhire badha़ne vali adhishvetaraktata ka eka prakara hai, jo asthi-majja mem prarambha hoti tatha bada mem rakta mem tatha kahim para bhi prakata hoti, danedara shveta rakta koshikaom ke atyadhika prasarana se abhilakshita hoti hai|??
yakrita ke kramika vinasha evam kathorata se abhilakshita yaha eka chirakalika bimari hai|
reshedara protina jo prani sharira ko dhancha pradana karata hai, koshikaom evam utakom ko eka satha bandhane vale mukhya kankaliya padarthom mem se eka hai|
yaha kolposkopa kahalane vale eka upakarana se kiya jane vala yoni evam garbhashaya griva ka eka parikshana hai jo prakasha evam avardhana deta hai|
yaha eka prakriya hai jisase eka koshika, dusare prakara ki koshika mem parivartita hone ke lie apana lakshana badalati hai|
koshika vibhajana evam prasarana ki prakriya, jisaka vijnana saitokainetiksa kahalata hai|
jo amataura para madhumeha ke rupa mem jana jata hai use chikitsakiya shabdavali mem daibitisa melitasa (mitha daibitisa) kaha jata hai kyonki rogi ke mutra mem se shakkara nikalati hai|
isa prakara ke madhumeha ko svadahina madhumeha se alaga karana chahie, jisamem rogi baड़i matra mem ‘svadahina’ mutra visarjita karata hai|
yaha shabda, prachina phransisi shabda desaese (desa-ki kami, tatha ese- sahajata) se vyutpanna hua hai, tatha jisaka vastava mem artha disa-ija़ ya sahajata ki kami hai|
chikitsa vijnana mem isaka vyutpatti-mulaka mahatva lupta ho gaya hai isalie dija़ija़ shabda ka mukta rupa se prayoga hota hai tababhi jabaki tathakathita bimari - purahstha granthi ya stana mem eka kainsaragrasta punja - bimari ke malika ko kisi bhi prakara se asahaja nahim karata|
vartamana lokachara mem, isaka artha hai garbhashaya ke garbhashaya griva ki upakala ki parata ki koshikiya asamanyata, jisaki eka rogavijnani asamanyata ke rupa mem pahachana karata hai, lekina usa koti taka nahim jise vaha kainsara kahane ke lie taiyara ho|
harmona samanyatah apani vishesha granthi se sravita hota hai|
jaba yaha anyatra kisi dusare utaka se bhi sravita hota hai, to ise asthanika harmona kaha jata hai|
yaha eka shabda hai, jo vidyuta-hridlekha (isiji), kabhi-kabhi ikeji anuvarti karyavahi ke rupa mem sankshipta kiya hua, ki naidanika evam purvanumanika upayogita mem chikitsaka ya rogi ke atyadhika vishvasa ka varnana karata hai|
yaha roga nidana tatha/ya upachara ka anusarana kara rahe rogi ki sthiti ka avadhika punarmulyankana tatha abhilekhana ki chikitsa prakriya hai|
yaha eka saiddhantika avritti vitarana darshata hai jo ghantakara, samamitiya tatha aparimita vistara ka hai|
svasthya ya bimari se sambandhita, aneka jaivika lakshana gaॉsiya vitarana pradarshita karate haim|
yaha vriddhavastha ka, tatha vriddhom ki samasyaom ka vijnana hai| 
yaha shreni 1 ke rupa mem shrenibaddha karate hue eka kainsara ki gambhirata varnana karane ka tarika hai|
kisi kainsara ki gambhirata usaki koshikaom ki samanyata se prasthana karane ke sidhe anupati mana liya jata hai jaba inhem kisi sukshmadarshi se dekha jata hai|
koi kainsara shreni 1 ka taba hota hai, jaba isaki adhikansha koshikaem akriti evam vinyasa mem samanya ke samakaksha hoti haim, tatha shreni 4 ka, jaba adhikansha koshikaem akriti evam vinyasa mem apasamanya dikhai deti haim|
yaha sukshmadarshi se - samanya ya kainsaragrasta- utakom ke adhyayana se sambandhita hai|
kainsara vijnana mem, kainsara ke lie eka samanarthaka ke rupa mem isaka prayoga kiya jata haih isalie, asadhya arbuda ya asadhya ghava|
samanyatah chikitsashastra mem, asadhya kisi bimari ki bhari gambhirata ko suchita karata haih jaise, asadhya bukhara, asadhya uchcharaktachapa, tatha asadhya maleriya|
tvacha ki ranjita koshikaom se utpanna hone vala tvacha ka koi kainsara; ankha, shleshma jilli, tatha anya utakom se bhi utpanna ho sakata hai|
yaha na to bimari (kainsara, sankramana) ke isake udgama sthala se kisi dusare sthala para phailane ya sthanantarita hone se sambandha rakhata hai na hi isase pratyakshatah sambaddha hai|
arbuda, jisaka aksharashah artha nae sire se srijita utaka hai, kainsara ka sanketa karata hai, yadyapi aisi prakriya pradaha, ghava ke bharane mem bhi ghatita hoti hai|
kisi vyakti mem kainsara ki upasthiti kai na pahachane jane vale jinom dvara nirdharita hoti hai (isalie paॉlijenika/bahukramagunita vanshanukrama kahalati hai) jo sampurna junda ki jinom ke samanvayana se yaha nirdharita karati hai ki kainsara hoga ya nahim?
tatha kisi vyakti vishesha mem isa prakara ka anuvanshika sanchalana parimanatmaka hai aura gunatmaka nahim|
sabhi manavom mem kainsara vikasita ho sakata hai; kevala kuchha loga hi haim, jinamem parimanatmaka jina prabhava unhem eka nishchita anuvanshika dahalija ke pare le jane ke lie paryapta hai|
janmajata kurachanaom jaise bhanga oshtha se lekara ama bimariyom jaise pachana tantra ke chhale, hridayaghata, madhumeha, ya uchcharaktachapa taka vistrita kismom ki bimariyom ki upasthiti ki vyakhya karane ke lie paॉlijenika vanshanukrama ka sahara liya gaya hai|
kainsara vijnana mem, yaha usa sthana ki ora sanketa karata hai, jaham pahale kainsara utpanna hota hai; isalie, prarambhika sthana, prarambhika kainsara, prarambhika vikasa adi|
jaba koi kainsara, prarambhika sthana se nikalakara, anya atirikta sthana/sthanom para prarambhika sthana se sasharira vichhinna hokara sthapita ho jata hai, to ise paravarti ya vikshepi kainsara ho jana kahate haim|
paravarti sthana se, vikshepana ki sampurna prakriya ki punaravritti ki ja sakati hai|
jnanamimansa ki drishti se, yaha jnana ki usa avastha ki ora sanketa karata hai, jo nishchitata se kama hai lekina ajnana se adhika hai|
janapadika-rogavijnana ki drishti se isaka artha, junda ke stara para nishchitata hai, jo, junda banane vali sankhya ki tulana mem sankhyatmaka rupa se kama hai, jaba ise vyaktigata stara para abhivyakta kiya jata hai to avashyakata ki anivaryata eka sanyoga, sambhavana ya sambhavyata ka mamala hai|
anishchitata ka isa prakara ka mapana ya parimanana, sambhavyata kahalata hai|
sambhavyata ki janapadika-rogavaijnanika sankalpana ko, tivra lasikakoshikaprasuka adhishvetarakkata, rakta kainsara ke eka prakara, dvara sarvottama riti se pravardhita kiya ja sakata hai|
vaishvika rupa se, yaha prativarsha prati 100,000 ki janasankhya mem 2 se 3 mamalom ki dara se eka desha se dusare desha mem thoda़e-se parivartana ke satha hota hai|
samanyatah yaha shalya-chikitsa ke lie lagu hota hai, lekina rogopachara ke anya tarikom tatha chikitsiya sanyojanom ke lie bhi lagu hota hai|
adhiativadi rogopachara, chikitsiya ativada ki parakashtha ka pratinidhitva karata hai|
vikirana-chikitsa, ksha-kiranom dvara kainsara (ya kisi anya roga) ka upachara hai|
yaha anuvanshika abhiyantriki ki hala hi mem vikasita takanikom se prayogashala mem ekatrita dienae ke eka nae sanyojana vale jivanu ka eka prakara hai|
jaba eka samanya koshika pahale se hi vidyamana kainsara mem sammilita hone ke lie kainsaramaya banati hai, to use bharti ya navakainsarikarana kahate haim|
yaha kisi bimari ke lakshanom ki asthayi ghatava hai|
yaha sanyoji utakom-visheshatah asthi, upasthi, mansapeshi, tatha pravarani mem utpanna hone vale kainsara ki ora sanketa karata hai|
sarkarbuda karsinoma se isa prakara bhinna hota hai ki karsinoma tvacha aura antarika angom ko avritta karane vali koshikaom se, ya yakrita, avatu, piyusha adi jaisi granthiyom ko banane vali koshikaom se utpanna hota hai|
yaha paniri, vasamaya padartha ki ora sanketa karata hai jo purushom mem granthiyom, shishna tatha shishnagrachchhada mem, ya mahilaom mem bhaga-shishna, bhagoshtha tatha laghuoshtha ke asapasa ekatrita hota hai|
ye kisi kainsara ki koshikaem haim jo kisi samaya eka bara kisi pashu mem utpanna hui thim tatha kramika rupa se parakhanaliyom tatha pashuom mem utpanna karate hue, aneka varshom se samvardhita kiya gaya hai|
aisi koshikaom ko jaba vishesha rupa se taiyara kie gae pashuom mem sancharita kiya jata hai to ye punjom ka nirmana karati haim jinhem pratiropita kainsara kahate haim|
aise ‘kainsara’ aburda pranaliyom ka nirmana karate haim jina para aushadhiyom ke parikshana kie jate haim|
yaha eka ja़eraksa mashina mem parichalita takanika ki taraha eka takanika se pratibimba prapta karake, kiya jane vala stana ka adhyayana hai| 
isalie inaka unmulana karana utana hi anuchita pratita hota hai jitana ki inako bardashta karane ke lie rahane dena, jaba inaka vikshubdha hona prarambha hota hai tatha puri taraha se nikala ja sakata hai|
teraha varshom taka sarvadhika jivita rahane vala rogi eka yureshiyai tha, jisane apani puri ‘bimari’ bhara upachara lena asvikara kara diya|
teraha varshom taka apane kainsara ke satha jine ke bada, vaha athahattara varsha ki ayu mem hridayaghata se mara|
purani adhishvetarakkata rogi ke aneka varshom taka jivita rahane ke mamalom ke barambara hone vale udaharana prastuta karati hai, jaba ina mamalom mem upachara nahim kiya gaya ya kevala taba upachara kiya gaya jaba rogi ne asahajata anubhava ki|
vriddha rogiyom mem se aneka isa bimari ke satha marate haim banisbata isa bimari se|
adhishvetarakkata ki koshikaom se asthi-majja ke ‘vyapaka antahsyandana’ ke bavajuda usaka svasthya nirantara achchha raha|
alakshanatmaka hote hue bhi, use apane jivana ke antima pancha varshom mem, kainsara rasayana-chikitsa dvara apani ganana ka upachara karane ke lie phansaya gaya; bada mem usamem vividha prakara ke sankramana vikasita hue, tatha antatah atyadhika avarti phupphusapradaha ke karana usaki mauta hui|
unhonne yadrichchhika rupa se, chhati mem parisimita ashalyakaraniya shvasani kainsara ke 249 rogiyom ko, pratyeka ka alaga-alaga upachara karate hue chara alaga-alaga samuhom mem vibhajita kiya gaya|
eka samuha ko ‘mahatvapurna’ lakshana dikhai dene taka upachara nahim diya gaya|
anya tina samuhom ko upachara diya gaya, bhale hi parikshana mem pravesha karane ke samaya unamem lakshana the ya nahim, tatha unhem vikirana-chikitsa, rasayana-chikitsa ya ina donom ka sanyojana diya gaya|
pratiksha karo-evam-dekho samuha mem ausata uttarajivita 8.4 maha thi, jabaki vikirana-chikitsa, rasayana-chikitsa ya unake sanyojana se upachara kie gae samuhom mem yaha kramashah 8.3, 8.7 tatha 8.8 maha thi|
rogiyom ke usa samuha ne bhi, jinaka arbuda-rodhi upachara, taba taka vilambita kiya gaya tha, jaba taka lakshana prakata nahim hue, utana hi achchha prashamana prapta kiya, jitana ki unhonne jinaka tatkala upachara kiya gaya tha|
isa vivarana ke lekhakom ne yaha anubhava kiya ki unake parinama isa bata ka koi sakshya prastuta nahim karate ki ashalyakaraniya shvasani kainsara ke rogiyom mem, vikirana-chikitsa tatha/ya rasayana-chikitsa dvara tatkala upachara karane se uttarajivita pravardhita hui hai ya asamartha karane vale lakshanom se bacha ja saka hai|
‘isake viparita, yadi koi isa sahitya ke avalokana ka kashta karata hai, to isi vishaya para isamem prachura lekha die gae haim’ jo ashalyakrita tta-pathari, anupacharita stana kainsara, tatha jathariya/grahani chhale, dvikapardi sankirnata tatha grasa-nalika, udara, vrihadantra, malashaya, yakrita, pittashaya, tatha agnyashaya ke kainsara ki prakritika pragati’ darshate haim|
eka mahatvapurna kainsara vaijnaniki vastavikata yaha hai ki sabhi kainsara apane prarambha ke samaya se lekara, pancha se pandraha varsha ki avadhi mem, roganidana ke samaya taka ‘bhanli-bhanti prachchhanna’ tatha pida़arahita hote haim| 
adhika mahatvapurna rupa se, ki aneka kainsara vikshepi sthala ki prarambhika avastha para roga nidana ho jane ke bada bhi pida़arahita bane rahate haim|
isa kainsara ki yahi pida़arahitata thi, jisane kisi meyo, kisi vilki, ya kisi daॉrna ko vyapaka rupa se phaile tatha ashalyakaraniya kainsara ka roga nidana hone taka shantipurvaka karya jari rakhane diya tatha khuli evam banda prakriya ke paryapta bada nrityu ai|
lainseta ne mukha-kainsara ko eka duhsadhya naidanika (roga vishayaka) samasya ke rupa varnita kiya, tatha isa bata para shoka prakata kiya ki inglainda tatha velsa ke prarambhika mukha-kainsara ke sabhi rogiyom ke adhe se adhika ne, roga ki antima avastha mem apane apako prastuta kiya|
satya yaha hai ki rogi ke sharira mem apani upasthiti ke pramukha bhaga ke daurana pida़arahita rahana kainsara ki asali prakriti hai|
prakriti ki taraha, kainsara krura hai lekina kainsara dayalu hai|
tatha kainsara pida़arahita hai, kyonki yaha saprayojana rupa se kahana ho to aisa hi hai|
koi rogi, jo kisi kainsara ki ‘avahelana’ karata hai, aisa isalie karata hai kyonki kainsara use lambe samaya taka, asahaja nahim karata|
aie aba hama nishkriya stana apavriddhi vali eka mahila ki samasya para vichara karate haim : yadi yaha gaira-kainsaragrasta hai, to kuchha bhi karane ki avashyakata nahim hai|
yadi yaha kainsaragrasta hai, to kuchha bhi karane ke lie apako bahuta vilamba ho chuka hai|
eka tarkasangata nishkarsha yaha hai ki isa rogi ke lie, roga naidanika ya chikitsiya, kuchha bhi nahim kiya jana chahie|
yaha prastava ashcharyajanaka laga sakata hai, ise pratishthita koshikiya tatha arbudiya kainsara vastavikataom ka puri taraha se samarthana prapta hai|
eka vriddha vyakti mem ‘nitya jancha’ ke daurana eka kathora lekina nishkriya purahsthika pinda paya jae to use kainsara hone ke sanjnana mem sadbhavapurnata se ghasitane ki avashyakata nahim hai|
alikhita rupa se tatha sahi tarike se, isaka roganidana eka stana apavriddhi, eka purahsthika pinda tatha aisa hi kuchha hai|
tarkasangata chikitsashastra ke lie yaha eka mahana dina hoga, jaba chikitsaka usa kainsara ka roga nidana na karane, tatha isalie usaka upachara na karane ka adhikara prapta karega, jo apane svami ke satha shanti se hai|
jaisa ki yaha prastava sakhta laga sakata hai, yaha nivedana karata hai ki rogi ki chetanaom mem kainsara ke badhaka hone se purva rogi ko manasika nrityu se bachaya jae|
yaha tarka diya ja sakata hai ki jaba taka ki rogi ko pahale se hi savadhana na kiya jae, vaha roga se bekhabara raha sakata hai|
yaha chetavani avanchhaniya hai kyonki yaha rogi athava chikitsaka-rogi ke lie bhi, nikrishtatama shashvata pratyasha ke satha eka prakara ke maranoparanta astitva ko asamaya le ati hai|
jaba chetavani ko chikitsaka dvara spashta rupa se asvikara kiya jata hai to yaha eka rogi ko akasmika kainsarayukta ya hridiya rogom ya nrityu se nahim bachata hai|
ye lekhaka, svasthya ki sapha tippani diye jane ke matra pandraha dina bada hi hridayaghata se achanaka mari, eka samanya chikitsaka ki patni ko janate haim, jo apane chalisa ke dashaka mem, bambai ke kuchha prasiddha hridayaroga visheshajnom dvara kisi bhi hridaya samasya se mukta ghoshita ki gai thi|
yaha eka asamanya ghatana nahim hai, bada़e hridayi klinika vale bada़e shaharom mem, jaham eka adami purnatah samanya ikeji(isiji) ke satha hridayaroga visheshajna ke paramarshi kaksha se praphullatapurvaka bahara nikalata hai, aura klinika se matra kuchha gaja ki duri para, sirpha़ hridayaghata se marane ke lie girata hai|
kainsara, kisi vyakti ko asahaja karane se purva eka lambi avadhi taka dakshatapurvaka chhipa rahata hai|)
yaha janate hue ki anidanya aura alakshanatmaka kainsara ki avadhi kapha़i lambi hoti hai, upara ke do chikitsaka-rogiyom mem kainsara ‘lakshanatmaka mem badalane se purva kai varshom taka bina dikhavata ke chhupe’ avashya rahe hogem; aura jitane samaya taka kainsara anidanya (evam anupacharita) raha gaya tha, utane samaya taka saubhagya se donom shalyachikitsakom ko kitsiya maranoparanta astitva se bachaya gaya|
chikitsaka, jinhonne isa mata para vichara kiya ki kainsara ka hamesha upachara nahim kiya ja sakata hai yaha bata dimaga़ mem avashya rakhani hogi, phira bhi, isa adhunika mukadamebaja़ samaja mem jisake lie unhonne ka़imata ada ki hai ya ka़imata ada karane ke lie taiyara haim, ke na milane para chikitsakom ke viruddha vadi mem parivartita hone ke lie pravritta rogi bhi samavishta haim|
jaise ki chija़e aja haim, chikitsiya, nyayika, vidhika tatha sarvajanika janamata eka ‘surakshita chalane vale’ vyakti ko nirdosha thaharata hai jo hara kainsara mamale ka upachara karata hai, kintu jaba taka ki use bilkula aisa karana hi pada़e taba taka chikitsa karane se mana kara rahe eka chikitsaka ko kshama nahim karega|
upachara ka abhava bhi upachara ka eka tarika़a hai - isa ruchihina para sahayaka goli ko nigalane ke lie chikitsakom aura anabhijna, vidhika, nyayika janata ko taiyara karake hi aise eka ‘khatara mola lene vale’ ki sthiti ko uttarottara surakshita rupa mem badali ja sakati hai|
isa asambhava dikhane vale lakshya ko prapta karane mem sahayata ke lie, prabuddha chikitsaka eka enosi-esosieshana ऑpha kainsara therapista(eesiti) prarambha kara sakate haim jisaka adarsha vakya ‘pahale hani mata pahunchao’ hona chahie|
isa eesiti ko kainsara ki amanya aura akirtita susadhyata, kainsara ki prakkathaniyata, kainsara upachara ke sabhi prakarom ki khataranaka prakriti, sabhi kainsara chikitsakom ka ‘yadi apa karo to bhi shapita’ aura ‘yadi apa na karo to bhi shapita’ ka anubhava, tatha, inase sarvopari, ki kainsara bhi rogi ko akele chhoda़ dene ki anumati deta hai, ka avashya prachara karana chahie|
eesiti antatah sarakara ya anya sansthaom se yaha dikhakara vittiya sahayata pane ka prabandha kara sakata hai ki eesiti ka tatparya puri duniya mem, prativarsha kainsara para pratyaksha ya apratyaksha rupa se kharcha hone vali vishala dhanarashi ki bahuta adhika bachata karana ho sakata hai|
yadi pramanikata ko siddha kiya ja sake, to chikitsakom ka purvanumana karane ka adhikara uchita hai; yadi nahim, to chikitsaka se bhavishyavakta ka svanga karana banda karane ki manga ki ja sakati hai| 
vyapaka rupa se pracharita nyayalaya ki sunavai mem, chikitsaka ka purvanumana tha ki yadi jivana banae rakhane vale yantrom ko hata liya jaya to karena ena kvinalana, eka taruna amariki balika jo sammurchha mem thi, mara jaegi|
usake mata-pita ne plaga ko nikalane ke lie nyayalaya se anunaya kiya taki karena mara sake| 
bhavishyasuchaka kala ki avastha mem bahuta kuchha janana baki hai|
roga ke nidana, purvanumana tatha upachara ke traya mem se, purvanumana mulyankana karane ke lie sabase kathina hota hai|
chija़om ki sahi bhavishyavani ana prayah sabase adhika chakkara mem dalane vala, pechida, aura sandehaspada hota hai|
yada rakhem ki hamem do ankhem aura do kana parantu kevala eka jibha pradana kie gae haim|
saba kaha ja chuka hai, ki chikitsiya purvanumana eka vijnana ki apeksha kala adhika hai|
pahala upaya use usake paropakara ke adharom para kshama karane ki anumati deta hai; dusara upaya usake lie sukshma raksha pradana karata hai, jisamem rogi kabhi-kabhi hi apekshita se lamba jivita rahata hai, usamem apane purvanumanaka mem dosha dekhane ka jigara hota hai|
aise chikitsaka bhi haim jo, apana mahattva dikhane aura eka bahana banane ke lie ashvasta hote hue, kharaba ko bahuta kharaba pratita hote banate haim aura bahuta kharaba ko eka bhishana durghatana banate haim|
kyom eka kainsara purvanumanaka apane apa ko itana ashvasta mahasusa karata hai, isake kai karana haim|
usake naidanika aura upacharatmaka kaushala prati dina vikasa prakata karate haim|
eka kainsara mamale ka usaka ankalana - roga-vishayaka parikshana, guhantadarshana, eka shastragara ke parikshanom, aura shalyachikitsa ke jancha parinamom sahita prakata rupa se sampurna hai|
abhi bhi purvanumana ke sadhanom ki isa prabhavashali shrinkhala ke bavaja़uda, purvanumanaka hara avastha mem, avishvasaniyata ka samana karata hai|
jise usane hamesha prarambhika kainsara hai socha vaisa kabhi-kabhara hi hua|
kainsara koshika-vijnana bahuta adhika svechchhachari hai isilie avishvasaniya bhi hai|
utaka-vijnana isase adhika behatara saphala nahim hai|
kuchha karmachariyom ke anubhava ke viparita, hamane yaha nahim paya hai ki namuno ke jivoti parikshana (bayopsi) ka utaka vijnana, purvanumana ya prabandhana mem koi upayogi margadarshana pradana karata hai|
1970 mem ho dvara kiya gaya yaha sadharanikarana – jisaki shreni mem adhikatara kainsarom ko avibhedita kiya jata hai – 1960 mem sudaralainda dvara kie gae sadharanikarana ke samana hai ki vartamana mem, purvanumananusara bhinna hone vale kainsara prayah akriti vijnananusara avibhedya hote haim|
kainsara ka vargikarana tatha prastutikarana, purvanumanakom ke anusara spashta rupa se bahari manadanda para nirbhara karate hue, bhavishyavakta ki bhumika nibhane mem sahasi prayasa prastuta karate haim; parantu die gae karsinoma ko jatharantra patha ke rogavijnani ya vayudabi dabava ke vyavaharika svasthya para nirbhara karate hue, eka dina II agale dina III , ya viparita krama se, vargikrita kiya ja sakata hai|
brihadantra tatha malashaya ke granthikarsinoma ke vargikarana para likhate hue, boyada tippani karate haim ki jabaki ankada़om ki drishti se varga, lasika granthi ki sahabhagita, purvanumana, ityadi, ke bicha kuchha tala mela sthapita karana sambhava hai, isaka yaha artha nahim hai ki eka rogi mem isaka bhavishyasuchaka mahattva hai|
purvanumana kainsara hai, anya vibhagom ki taraha, paristhitika pramanom ke adhara para yaha eka dharana hai, para koi bhi dharana sammanita nahim ki ja sakati jaba pramana vyapaka rupa se sandigdha ho|
phira bhi, unake eka ya adhika jagahom para kainsara ka pata lagane se yaha garanti nahim hota hai ki kainsara isake atirikta anyatra upasthita nahim hai|
aisi ajnanata ke bavajuda, purva upachara ke adhara para anukula purvanumana karane ki prerana sabhi mem eka adhika samanya kathinai hai|
yaha 1936 jitane pahale hua tha, ki naithansana aura velcha ne apane stana karsinoma ke shreni mem bataya ki, upachara mem sabase kama vilamba kie jane vale rogiyom mem saba se kharaba purvanumana hote haim|
kabhi-kabhara hi nahim, arthi ka eka purvanumana galata siddha hota hai, aura rogi yaha dikhate hue jivita ho jata hai, ki purvanumanaka ne rogi ke lie jo sanchita tha usase adhika bhishana durghatana dekhi thi|
unase chikitsaka vayu ki disha pahachanata hai; vaha vayu ki gati, ya mausama ki avastha jaise tapamana, ardrata athava drishyata ke bare mem kuchha bhi nahim janata hai|
purvanumana karane vale chikitsaka, samanyatah, sankhyiki ke vayu-disha darshaka yantra-prakriti se anabhijna hote haim jo unake pasa madhumeha, koronari hridaya roga, uchcha raktachapa, ya vibhinna kainsarom para likhe adhikarika lekhanom mem spashta, vishvasaniya, pidha़i-dara-pidha़i se sahi pramanita ankada़om ke rupa mem ate haim| 
purvanumanaka ke pasa usake kisi vyaktigata mamale mem margadarshana karane ke lie kuchha nahim hai –jisake lie vaha, ja़yada se ja़yada, retrognostiketa ya ghatana ke bada buddhimana ho sakata hai|
adhi sankhyakom ke satya mem kisi antarnihita vishvasa dvara samarthita, purvanumanaka kisi vyakti ke sune akada़om ke bahirveshana karane ko suvidhajanaka pata hai, isa tathya ka najara-andaja karate hue ki yaha bahirveshana hisenabarga ki sandigdhata se bhara hua hai|
jaivika samasyaom mem, mahattvapurna jatilata ke parivartanashila karaka prayah upasthita hote haim, jisaka avashyaka mahattva sankhyiki-vijnana se bayomitri ko alaga karata hai|
aisi chetavaniyam purvanumanaka ki ankhom se bacha jate haim aura isilie vaha hisenabarga ya mekdaॉnalda ke bavajuda purvanumana karana jari rakhata hai|
jaba vaha sankhyikiya purvanumana ka prayoga karata hai taba bhi, yaha purvanumanaka sambhavatah mulagrantha ke prayah atyanta arambha mem upasthita sachetaka vivaranom ki upeksha karata hai|
anya jirna rogom ki taraha yaha madhumeha mem bhi satya hai ki rogi ke lie purvanumana ashcharyajanaka rupa se vyaktigata hota hai|
purvanumana se sambandhita sadharanikarana shayada vishesha samuha mem aura vishesha jatilataom ke ausatom ke adhara para hota hai; phira bhi, pratyeka samuha mem jivana ki avadhi aura madhumeha ke rogottara vikara ki upasthiti ya anupasthiti mem vistrita vibhinnataem pai gai haim|
bruke, ka kainsara para lekha, purvanumanana ko chikitsa mem sabase mahattvapurna kriya ke rupa mem varnita karata hai| 
aura sambhavatah, yaha saba milakara vibhinna karanom se satya hai, jaise ki rogi tatha usake nikata sambandhiyom se bantane ke lie ama taura para kainsara para asvikrita chikitsiya ajnanata aura rogi ko yaha janane dena ki kainsara utana hi udara ho sakata hai jitana ki vaha nirdayi ho sakata hai|
purvanumanana ka samaya rogi ke satha samasyaom para batachita karane ka samaya hai|
yaha use roga ka samana karane, aura usake satha jine ki karmashakti pradana karake rogi ke mitra ki taraha kama karane ka samaya hai|
purvanumana mem una kshetrom ka jaham se ashvasana liya ja sakata hai ki jancha karana aura unhem prakata karana tatha una kshetrom ko jaham se niradhara bhaya utpanna hote haim ko nashta karana shamila hai|
kainsara ke rogi prayah avasada mem jite haim aura yaha purvanumana karane vale chikitsaka ka kama hai ki unhem eka sala jine ke lie vapasa bulae – yaha kahate hue ki jivana mem hara anapurya satha sekanda vale minata ko bharane ke lie kipalinjiyana prerana se mulakata hoti hai jo lambi dauda़ mem mulyavana hota hai|
purvanumanana mem rogi ko sauddeshya bhaya prayoga karane mem sanriddha samanya jana sambandhi aura chikitsiya lekha ke atyachara se bachana bhi shamila hai|
kisi dhunrapana karane vale mem phephada़e ka karsinoma, ya kisi svadalolupa mem amashaya ka karsinoma, athava kisi mahila jisane jine ke lie sambhoga kiya mem garbhashaya griva ka karsinoma, dostovaski ki aparadha aura danda ki kahani nahim hai|
dakshineshvara, ramakrishna paramahansa mem- jise vilsana ishvara se sakshata darshana ka abhilashi, eka bhagavata-madahosha mahana santa kahate haim,- eka pranaghataka svarayantriya kainsara ki usa avastha mem upasthiti, nishchita rupa se kupita bhagavana se kathora danda nahim tha|
jo maim nahim janata hum, vaha ajneya hai|
jo maim janata hum, vaha batane yogya hai|
junda-yatharthavada, samanya vibhajana, gompertja़ kriya, roga-vishishta nrityu sankhya ke vakra, adi isa satya ke sukshma aura anavarata sanketaka haim ki eka vyakti, apane sabhi purva anirnita, asamanantarita tatha nahim doharaye jane yogya vilakshanata ke bavajuda, kai lakshanom ke sandarbha mem junda para nirbhara karata hai| 
vilisa apane ‘staitistikala stadi ऑpha tyumarsa’(arbuda ke sankhyikiya adhyayana) adhyaya mem, isa bata para bala dete haim ki kisi janasankhya mem, kainsara nritakom ki eka paryapta lambi shrinkhala ka unra vibhajana, samanya vibhajana ka ‘eka adarsha pravahi vakra’ pradana karata hai|
vibhajana ki yaha samanyata eka junda- kriya hai, aura, eka vyaktigata stara para, attharaha, ya atthanabe ki unra para kainsara se marana, yaha vakra ke usa bindu para nirbhara karata hai jisa para vaha pada़ta hai|
attharaha ki unra mem kainsara se marane ke lie taiyara vyakti ke lie, (satha-satha usake karibi logom ke lie), yaha ‘gada़bada़’ hai ki use prakriti dvara isa prakara ‘shikara banaya’ jana chahie|
para yadi vaha aura dusare yaha samajem ki usaki ‘‘gada़bada़i’’ kramabaddha ‘adarsha’ vakra ka eka bhaga hai, to kashta ke isa bodha ke kama hone ki sambhavana hai|
kainsara ke purvanumana mem rogi ko yaha samajana ki usaka kainsara, jo abhi usako asahaja kara raha hai, pancha se pandraha salom se usake satha raha raha hai, se yukta retrognostikeshana bhi sammilita hona chahie| 
isake atirikta, ki kisi kainsara ki prarambhikata ya vilambata chikitsaka ke dimaga़ mem hota hai, na ki usa kainsara mem|
rogi ka kainsara yatharthavada ki vaha eka koshikiya tathya ko ashraya de raha hai, jisamem yaham taka purvanumanaka bhi kevala utana hi buddhimana hai jitana ki vaha svayam, usako roga ke viruddha lada़ne mem samana rupa se mahattvapurna sahabhagi bana sakata hai|
aise yatharthavada ki anupasthiti mem, rogi nirasha ke karana ki aushadhi tatha chikitsaka use usaka deya nahim pradana kara rahe haim, ajiba atyachara ke bodha se pida़ita hota hai|
purvanumanana mem jancha sambandhi pratibandha aura upacharatmaka mahattva ko svikara karana sammilita hai|
munhasa shayada sabase samanya dirghakalika tvacha roga hai|
yaha tvagvasiya (jo vasa hai) granthiyom (sibesiyasa glainda) aura chehare , gardana, sine aura kandhom para samanyatah pai jane vali heyara phaॉlikalsa ki eka suji hui avastha hai|
lagabhaga, 12 aura 24 sala ke bicha ki unra vale dasa mem se atha navayuvaka loga munhase ki kuchha matra se pida़ita rahate haim|
yaha vayahsandhi para anubhava kie jane vale harmono ki gada़bada़iyom se ghanishtha rupa se sambandhita hai|
adhikatara rogi 20 aura 30 sala ki unra ke bicha achchhe ho jate haim|
para yaha 30 sala se adhika unra vale purushom mem phira bhi samanya hai|
auratom mem, yaha paintisa ke age kabhi-kabhara hi ka़ayama rahata hai aura samanyatah hara masika avadhi se pahale badatara ho jata hai|
yaha roga usa unra mem bahuta sharmindagi paida karata hai jaba loga vaiyaktika bahyakriti ke bare mem samvedanashila hone ki ora jukava rakhate haim|
sampurna sharira ko dhakane vali yaha tvacha, adbhuta aura gudha़ tantra hai|
yaha tina mukhya uddeshyom ko pura karati hai, jaise ki antarika sharirika gathana ki suraksha, sharira ke tapamana ka niyantrana aura koshika mala tatha niyamita avashishta ka nishkasana|
tvacha sampurna sharira ki prakriya se sidhi juda़i hui aura ghanishtata se lipati hui hai|
munhase sahita, sabhi tvacha roga, pure sharira ke asanyakakaryata ke parinama haim|
munhasa kaॉmedonsa ya kale masse, phunsiyom, chhote satahi tvagvasiya putaka aura ghava ke chihnom se visheshita hota hai|
munhase ke adhe darjana se adhika prakara haim|
ye sabhi tvagvasiya granthiyom ya heyara phaॉlikalsa se juda़i hui granthiyom se juda़i hui haim|
kale masse munhase ke sabase samanya prakara haim|
matha, kanapatiyam, gala, thoda़i, sina tatha pitha mukhya rupa se prabhavita kshetra haim|
durlabha mamalom mem, bahuta bada़e chihnom sahita kale masse se lagabhaga pura sharira achchhadita ho sakata hai| 
munhase ke sabhi prakarom mem unaki utpatti ka karana galata ahara adatem haim, jaise ki aniyamita samaya para khana, anupayukta bhojana, starcha, sharkara tatha vasa yukta bhojanom ki adhikata|
jirna kabja munhase ka dusara mukhya karana hai|
yadi ante thika se nahim chalati haim, to avashishta padartha ka nishkasana jitani jaldi hona chahie utani jaldi nahim hota hai aura rakta pravaha vishakta padarthom se bhara jata hai|
isa adhishesha mala ka nishkasana karane ke lie kie gae tvacha ke ye atirikta prayatna munhasom tatha tvacha roga ke anya prakarom mem parinita ho jate haim|
aura munhase ka dusara mahattvapurna karana hai asvasthyakara rahana-sahana ke parinamasvarupa utpanna tvacha ki pranashakti se vanchita eka avastha|
isa vikara ke anya sahayaka karaka haim, chaya, kaॉphi, madira tatha tambaku ka atyadhika prayoga, shramasadhya adhyayana aura nishkriya adatem jo apacha tatha samanya kamaja़ori ki ora le jati haim| 
aushadhiyom ya marahama ke prayoga dvara munhase ke upachara se koi uddeshya pura nahim hota|
ye kevala tvagvasiya granthiyom ke karya ko asthayi rupa se roka dete haim|
prakritika upachara mem, ahara tatha kuchha jala anuprayoga para mukhya rupa se ja़ora diya jata hai|
shuru-shuru mem, rogi ko lagabhaga eka haphte taka sarva phalahara ka ashraya lena chahie|
isa pathyapathya niyama mem eka dina mem tina bara taje rasadara phalom jaise seba, nashapati, angura, chhote chakotare, anannasa aura chiku se yukta bhojana sammilita hona chahie|
nibu vansha ke phalom, kele, sukhe, dera taka pakae gae ya dibbabanda phalom ko nahim lena chahie|
mitha na kiya hua nibu ka ya sada pani, garma ya thanda donom mem se eka ko pina chahie aura kuchha bhi nahim|
isa avadhi ke daurana, antom ko sapha़ karane ke lie garama pani ka enima roja lena chahie aura kabja़ ka unmulana karane ke lie anya sabhi upaya apanane chahie|
eka saptaha ke sarva phalahara ke pashchat, rogi dhire-dhire mukhyatah eka kshariya susantulita ahara apana sakata hai|
jyada jora kachche khadya padarthom, visheshatah taje phalom aura sabjiyom, ankurita bijom, kachrrache giridara phalom aura hara taraha ke anajom, visheshatah jvara-banjara, kodom tatha bhure chavala para hona chahie| 
munhasom ki sabhi sthitiyom mem garma pani mem epsama namaka dalakara haphte mem do bara snana karana , atyadhika labhadayaka hota hai|
srravasrrathrrayalabha ke lie ahara para punarrra dhrrayana dena jaruri hai| 
shvetasarayukta, protina aura vasa yukrrata ahara ka nishedha hona chahie| 
jigara mem lagabhaga 90 pratishata sharaba ka ऑksikarana dhire dhire hota hai aura shesha 10 pratishata sansa lene aura peshaba ke madhyama se bahara nikala jati hai|
munhasom ko thika karane ke lie, niyasina aura vitamina e ina do vitaminom ka saphalatapuvarrraka upayoga kiya jata hai| 
munhasom ke karana hone valem dagom se bachane ke lie tatha purane dagom hatane ke lie vitamina i bhi utana hi jaruri hai| 
simita upachara ke liye, chhidrom ko kholane aura phira kharaba padatharrra ko bahara nikalane ke lie gamarrra sikai karani chahie| 
phira thamड़e pani se saphai karem| 
dhupa va hava mem apane sharira ko udbhasita kara, dhupa va hava ka srranana karana, bahuta hi labhadayaka hai| 
sharaba ko eka khadya padartha nahim kaha sakate kyonki bhojanapranali mem pravesha karane para yaha kisi bhi taraha se badalati nahim ya pachati bhi nahim|
ghisi hui kakani, dudha mem paki hui jai evam achrrachhi gajara ka upacharatma lepa lagana prabhavakari paya gaya hai| 
lepa ko adhe ghante ke lie lagana chahie, phira chahare ko thamड़e pani se sapha kara lena chahie| 
haphrrate mem do bara, ipsama namaka ka gamarrra srranana, hara taraha ke munhasom ke lie labhadayaka hoga| 
tina paunda namaka ko lagabhaga 100 phairanhaita ke tapamana vale, 60 litara pani mem milakara isa srranana ki taiyari ki jati hai|marija ko 25 se 35 minata taka isa taraha ka srranana karana chahie, jaba taka ki usake sharira se pasina na nikalane lagem| 
srranana ke bada, marija ko dhire-dhire samanya tapamana para le ana chahie| 
vishva svasthya sangathana ne madatrrayaya ko 20 vim sadi ki tina pramukha khataranaka bimariyom mem se eka suchibaddha kiya hai| 
madatrrayaya eka gambhira samajika samasya bhi hai| 
yaha akrrasara garibi va kuchha hada taka gunahom ko bhi apane satha lata hai, aura prayah shadishuda jivana mem duhkha aura gharom ke tutane ka karana bhi banata hai| 
yaha bahuta se yatayata niyamom ke ullanghanom va dugharrratanaom ka bhi karana banata hai| 
sharaba koi prakriti mem paya jane vala padatharrra nahim hai| 
yaha apaghatana ke parinama srravarupa banati hai, aura isi vajaha se yaha visha varga ke antargata ati hai| 
ithaila sharaba, vaina va biyara mem paya jane vala pramukha nashila ghataka hai, aura asatu sharaba eka aisa jaharila nashila padartha hai, jo mastishka aura tantrika tantra ko manda kara deta hai| 
sharaba ko khadya padatharrra nahim kaha ja sakata, kyonki yaha poshana-nali mem to pravesha karati hai, phira bhi yaha na kisi mem parivatirrrata hoti hai aura na hajama hoti hai | 
yaha rakrrata dhara ke dvara teज़i se sokha li jati hai, aura phira yaha sharira ke pratrrayeka anga mem pahuncha jati hai, tatha atyavashyaka angom jaise dimaga aura yakrita para bura prabhava dalati hai| 
jaba eka bahya protina (eka elarji karaka) aura sharira ke samvedanashila utakom ke bicha samparka hota hai to usase elarji pratikriya ho sakati hai|
eka sharabi ka chehara suja hua, raktaranjita ankhe, kakarrrasha avaja़ aura naड़i spandana bhi teja़ hota hai| 
shishuom ko kama se kama atha mahine ke lie stanapana karaya jana chahie kyonki isa avadhi ke daurana sabhi avashyaka poshaka tatva pradana karane ka prakriti ka yaha eka tarika hai|
ulti, unmada, hanikaraka ninarrraya aura ashanta ninda, kuchha samanrraya lakshana haim| 
eka sharabi, jo ahara se jrrayada nashile padatharrra ka sevana karata hai, vaha payarrraprrata vitamina grahana nahim kara pata hai| 
kuchha vitamina jo vaha grahana karata hai, usake sharira mem sharaba ke dahana atharrrata pachana ke daurana bahara nikala jate haim| 
vitamina ki kami ke karana, kamponmada, ainthana, tantrika-shotha, ankhom ki vyadhiyam aura kamajora yadadashrrata ho sakati hai| 
atrrayadhika madyapana se hone vali vitamina ki kami akrrasara balom ke asamaya pakane ka karana banati hai| 
dirghakalika madyapana, sharira mem khanijom, visheshatah maiganishiyama, ke kami ka karana banata hai, jo hatha, pamva va jibha mem kampana, aithana, dimagi asrritharata aura pasina ana jaise lakshanom ko utpanna karata hai| 
sharaba, logom ko apana adi bana leti hai| 
apa jitani jrrayada piyenge, utani hi jyada aura lene ki ichrrachha prabala hogi| 
apa jitana jrrayada madyapana karenge, utana hi kama khana khayenge| 
upavasa rakhate samaya , yaha sunishchita kiya jana chahie ki sharira dvara sthapita atma - saphai ki prakriya ke dvara bahara nikale jane vale vishakta utpadom se antada़iyam pratyeka dina shuddha honi chahie |
atrrayadhika madyapana, yakrita para adhika bhara dalata hai| 
yaha dhire-dhire isake karya karane ki kshamata ko nashrrata kara deta hai, aura akrrasara yakrita ke sutranaroga (sirosisa) ka karana bhi banata hai| 
agara vaha sharirika aura manasika rupa se sharaba para nirbhara ho jata hai ,to vaha eka sharabi bana jata hai|
yaha dimaka ki kshati ka karana ho sakata hai kyonki dimaga ki koshikaem akrrasara isase prabhavita hoti haim | 
sharaba ka prabhava hridaya para bhi paड़ta hai, jisase yaha kamajora va shithila ho jata hai| 
sharaba ke atrrayadhika sevana ka parinama madatyaya hai| 
kabhi- kabhi koi vyakti jalda hi sharaba ka adi ho jata hai aura kabhi-kabhi kisi vyakti ko sharabi banane mem kai sala laga sakate haim| 
amataura para koi bhi vrrayakrrita isaka sevana samajika jivana ko aura anandamaya banane ke lie, chinta se ubharane ke lie ya achrrachhi ninda pane ke lie, karane lagata hai| 
yadi vrrayakrrita sharirika aura manasika rupa se sharaba para nibharrrara ho jata hai, to vaha eka sharabi bana jata hai| 
jivana ke tanava va thakana se jujane ki apani akshamata ke karana, loga atrrayadhika madyapana ka sahara lete hai| 
atrrayadhika madyapana karane vale ko, sabase pahale madyapana chhoड़ne ka dridha sankalrrapa karana chahie| 
use tatkala hi sharaba pini banda karani chahie, kyonki sharaba ki lata aisi nahim hai jise ki dhire-dhire chhoड़a ja sake| 
yaha mana jata hai ki elarji ka pramukha karana 10 se 12 mahine ki unra taka pahunchane se pahale hi bachchom ko anaja, mansa, makka, vasayukta dudha jaise khadya padartha khilana hai |
shuruata mem kama se kama 10 dinom ke lie, marija ko parimarjana rasa para rakhana chahie| 
isa daurana, use subaha 8 baje se shama 8 baje ke bicha, pratrrayeka do ghante bada, santare ke rasa ka sevana karana chahie| 
yadi ichchha ho to rasa ko gamarrra pani ke satha milakara patala kara sakate haim| 
aura kuchha bhi nahim lena chahie, anrrayatha vrata (sanyama) ka vanchhita parinama prapta nahim hoga| 
elarji kai prakara ki hoti haim aura kai prakara ke elarji karakom se ho sakati haim|
vrata (sanyama) ke daurana pratrrayeka dina, sharira ki srravayam shuddha karane ki prakiya ke dvara hi antom ko jaharile padathorrram ko bahara nishrrakasita karana chahie| 
aura kuchha bhi nahim liya jana chahie anyatha rasa upavasa ki upayogita puri taraha khatma ho jaegi|
rasa vrata (sanyama) ke daurana, marija ko shayada hi sharaba pine ki ichchha mahasusa hogi| 
yaha 10 dina madyapana ki adata se chhutakare pane ke liye eka achchhi shuruata honge aura ye na kevala sharirika balrrika manasika nibharrrarata se bhi mukrrita dilane mem madada karenge| 
rasom para adharita vrata (sanyama) ki shuruata ke bada, atyavashyaka paushaka tatvom ka eka santulita ahara avashrrayaka hai| 
isa prakara ke ahara mem hara taraha ke anaja, giridara phala, bija aura ankurita anaja, taje phala aura sabrrajiyam shamila honi chahie| 
nashrrate mem taje phala aura dudha le sakate haim| 
dopahara ke bhojana mem ubali hui sabrrajiyam, gehum ke ate ki chapatiyam aura chhachha le sakate haim| 
ratri ke bhojana mem salada aura ankurita anaja ko kaphi matra mem shamila kara sakate haim| 
upachara ki shuruata mem, yadi marija ko kabhi sharaba pine ki ichchha ho, to use rahata pahunchane ke lie upayukrrata anukalrrapa pradana karana uchita hoga| 
shahada mila hua, yadi ichrrachha ho to, eka grralasa taje phalom ka rasa sabase uttama peya vikalrrapa hai| 
isake badale mem, paushtika chakaleta kha sakate haim| 
marija ke pasa tina vakta ke bhojana ke alava yadi use uttejaka padartha lene ki ichchha mahasusa ho rahi ho, to bicha -bicha mem khane ke liye hamesha asani se upalabrradha hone vale rasa,chakaleta ya anrraya koi bhi khane ki chiज़e honi chahie| 
sabhi parishrrakrita khadya padatharrra jaise shakrrakara,chavala, maikaroni utrrapadom aura maida tatha mansa se paraheja karana chahie| 
marija ko do ya tina bara adhika ahara lene ki apeksha, dina mem kai bara halrraka ahara lena chahie, aura teja़ masale jaise kali micharrra, sarasom aura micharrra ka paraheja karana chahie| 
use dhrumapana nahim karana chahie, krrayonki yaha sharaba pine ki ichchha ko aura adhika prabala karata hai| 
eka bahari protina- eka elajirrrakaraka- aura sharira ke utaka, jo bahuta adhika samvedanashila hote haim, jaba ina donom ka sampakarrra hota hai, taba unamem elajirrraka kriya utrrapanrrana ho sakati hai| 
elajirrrakaraka, utakom taka trravacha ya shrraleshmala jilliyom se pratrrayaksha sampakarrra, ya avashoshana ke bada rakrrata pravaha ke madhyama se pahuncha sakate haim| 
marija ke elajirrrakarakom ke sampakarrra mem ane ke kuchha minatom mem, ya kuchha ghantombada, ya kuchha dinom ke bada bhi, elajirrra ka prabhava dikha sakata hai| 
sharira ka koi bhi anga elajirrra se prabhavita ho sakata hai| 
sharira ka vaha bhaga jo elajirrra se prabhavita hota hai, praghata anga kahalata hai| 
naka aura ankhe, trravacha, sina, antaड़iyam aura kana samanrraya anga hai| 
elarji ke dushprabhava kai prakara ki chija़om va paristhitiyom ke karana utrrapanrrana hote haim| 
isamem shamila haim, paraga, dhula, saundayarrra prasadhana, aura pashuom ke bala; jaharile paudhem, sirama, tika aura davaiyam; bhautika karaka jaise gamirrra, sardi aura dhupa; isake satha-satha kai taraha ke khadya padatharrra| 
bhojana se hone vali elajirrra mem, ahara mem nihita protina ko isaka karana mana jata hai, halanki jo ahara elajirrra ke dushparinamom ka karana hote haim, unamem protina ki matra bhinna-bhinna ho sakati hai| 
satha hi kisi ahara se hone vale elajirrra ke dushparinama tatkala ho sakate haim ya kuchha samayantarala para bhi ho sakate haim| 
khadya padathorrram ke ankhya elajirrrakarakom mem sabase sadharana haim- santare, dudha, anrrade, gehum, machhali, chakaleta, gobhi, alu, tamatara aura srratraberi| 
yogaja aura parimajarrraka bhi elajirrra ko baढ़ane ke lie uttaradayi haim| 
kharabi kevala khadya padatharrrom mem nahim hoti hai, balrrika unaka krraya kiya ja raha hai, isa bata se bhi hoti hai- una para gaisom ka chhidakava hota hai, unrrahem ranga jata hai, unrrahem lambe samaya ke liye surakshita rakha jata hai, aura samanrrayatah unako dushita kiya jata hai| 
jisa taraha se elajirrra ke karaka bhinna-bhinna hote haim, usi taraha isake lakshana bhi bhinna-bhinna hote haim| 
inamem shamila haim, bara-bara siradadarrra hona, adhasisi, chakrrakara, chiड़chiड़apana, ghabarana, udasi, tantrika-shula, chhinka, netra-shrraleshma, khaja, anrralashula, paragaja jrravara, apachana, kabrraja, dasrrata, jathariya ghava, dama, mutapa, uchrracharakrratachapa, sine mem dadarrra, dila ka daura, banda ya bahati naka, sansa lene mem pareshani, chehare va ankhom para sujana adi| 
eka hi ahara alaga-alaga logom mem bhinna-bhinna lakshana utpanna kara sakata hai| 
kai elarjiyam vividha prakara ki hoti haim, aura bahuta se elajirrrakarakom ke karana ho sakati haim| 
pratirodhakashakti mem kami va ahara mem gaड़baड़i va galata jivana shaili ke karana utrrapanrrana hoi antarika asangati ka elajirrra eka sanketa hoti hai| 
aisa mana jata hai ki jaba taka shishu 10 se 12 mahine ka na ho jae, taba taka use anaja, mansa, makai, aura gaढ़a dudha adi khilana, elajirrra ka mukhrraya karana hota hai| 
shishuom mem isa ayu se pahale ina aharom ko pachana ke lie jina uchita kinvakom ki jarurata hoti hai, unaki kami ke karana ye ahara elarji utpanna karate haim
shishuom ko kama se kama atha mahine taka mam ka dudha pilana chahie, krrayonki isa daurana shishu ko jina avashrrayaka poshaka tatrravom ki jarurata hoti hai, unhem pradana karane ka yaha eka prakritika tarika hai| 
elajirrra ka eka anrraya mahatrravapunarrra karana hai, ajakala upalabrradha hone vale, bahuta se rasayanika sanyojakom se yukrrata sansadhita ahara, jinamem se kaiyom se bhayankara elarjiyam hoti haim| 
ahara ki asantulitata ke parinamasrravarupa bhi elajirrra utrrapanna ho sakati hai| 
parishrrakrita chini ke atrrayadhika sevana aura rakrrata mem chini ki aniyamitataom ke chalate, ya kharaba ahariya tarikom se hone vale khanija va vitamina ke asantulana se sharira mem chini ko niyantrita rakhane ki kshamata kama ho sakati hai| 
bhavanatrramaka evam manasika tanava bhi, elajirrra utrrapanna kara sakate haim| 
tanava se sabandhita vishayom ke, vishrrava pramukha anusandhanakatarrra, chikitrrasaka, hainsa salrraye ke anusara, akrrasara elajirrra ke lakshana aura kuchha nahim balki tanava ke viruddha sharira ki pratikriya hoti hai| 
dirghakalika tanava ke karana eka vyakti samanya aharom ya sthanom ki chiज़om jaise petrola ke dhuai ke prati bhi samvedanashila ho sakata hai| 
bahuta-se elajirrraka asantulanom se nipatane ke kai tarike haim| 

yaha kathina hai, kintu asambhava kayarrra nahim|
dusare krama mem, jaba unaki pahachana ho jae, taba unase paraheja karana chahie| 
tisare aura sabase pramukha kadama mem, samanrraya srravasrrathrraya aura pratirodhaka kshamata ko badhana, jisase isase laड़ne ki pratirodhaka shakti utpanna ki ja sake| 
asantulana paida karane vale aharom ko dhundhane ke do tarike haim|
pahala tarika hai, parakha़ aura chuka vidhi se asantulita ahara ko bahara nikalana| 
yaha svayam hi bahuta se khatarom aura khadya padathorrram ka nishrrakasana karata hai|
jaham taka sambhava ho jaivika, asansadhita va aparivartita khadya padathorrra ka sevana karem, aura isase apa kuchha anrraya jokhimom jaise-jiva nashaka, bahuta se chhidakavom aura anrraya jaharile tatrravom se bacha sakegem|
jitana sambhava ho, utana asantulana phailane vale tatrravom ka nishrrakasana kara, eka srravayam-parikshana prakriya karani chahie, jisase khadya padatharrra ke pratrrayeka sandehatrramaka lakshana ko jana ja sakem| 
aisi salaha di jati hai, ki apa, do saprrataha ke lie vilopana ahara lem arthata jaba taka karana ka pata na chale,taba taka sandigdha khadya padatharrra ko varjita karem |
kabhi kabhi, branda ya prakara ko badala kara, apa usa khadya padatharrra ke badale mem kuchha aura le sakate haim, jo apako hani nahim pahunchaega| 
udaharana ke lie, jaise chakaleta apako hani pahunchata hai, to apa karoba (eka sadabahara peda़ jisaka tana khane yogrraya hota hai) kha sakate haim, jo usi taraha ke srravada ka hota hai, kintu nukasanadayaka nahim hota hai|
elarji ko janane ka dusara tarika hai, daॉ. koka ka nasa jancha tarika|
apane ahara ko eka hi khadya padatharrra taka simita rakhem, aura khane ke bada adha ghanta intajara karem aura phira se apani nabrraja ki jancha karem|
eka halrraki badha़otari, yaham taka ki 16 atirikrrata dhaड़kanem samanrraya mani jati haim|
yadi apaka naड़i spandana 84 ke upara nahim jata hai, to apa elajirrra se mukrrata mane ja sakate haim|
kintu yadi apaka naड़i spandana isa bindu se upara jata hai, aura ahara grahana karane ke eka ghante bada bhi jrrayada hi rahata hai, to apane apana elajirrrakaraka khadya padatharrra dhumढ़ liya hai|
tathapi, elajirrra se suraksha ya mukrrita ka sabase uttama tarika hai, sampunarrra sharirika pratirodhaka kshamata ko majabuta karana, taki apa apane sampakarrra mem ane vale elajirrrakarakom se asani se prabhavita na ho sakem|
arambha ke chara-pancha dinom taka marija ko, taje phalom ka rasa lena chahie| 
thoda़e samaya ke lie, rasom para adharita sanyama ko apanane se purani elarjiyom ki sahanashilata mem baढ़otari ho sakati hai| 
taje phalom ke rasa para adharita sanyama ke bada, marija eka haphrrate taka, sabrrajiyom aura phalom jaise- gajara, angura ya seba ka ekala ahara le sakata hai| 
usake bada eka aura khadya padatharrra ko apane ahara mem shamila kara sakata hai|
eka haphrrate bada, tisara ahara aura isa taraha aura bhi ahara shamila hote jaegem|
chara saprrataha bada, protina yukrrata khadya padatharrra ko eka eka karake shamila kara sakate haim|
yadi kisi nae khadya padatharrra ko khane se elajirrraka prabhava dikhe, to usa khadya padatharrra ko khana nahim chahie, aura usaki jagaha kisi anrraya nae khadya padatharrra ko ajamana chahie|
isa taraha se apa antatah, apane ahara se, sabhi elajirrrakarakom se chhutakara pa sakate haim| 
sharira ko apani nitrraya ki prakriyaom ke lie, baड़e paimane para kshariya sanchaya ki avashrrayakata hoti hai| 
dinabhara lie jane vale galata khadya padarthom, thakavata, manasika tanava aura ninda ki kami ke karana, utrrapanrrana hone vali anrrala ki samasrraya, kshariya sangrahom ki prachurata se puri ki ja sakati hai|
kshariya tatva ka nimarrrana karane vale, khadya padathorrram ka prachura matra mem sevana kara, sharira ke kshariya sangraha ko badhana, elajirrra se pida़ita logom ke lie avashrrayaka hai|
khadya padatharrra jinrrahem ahara se varjita karana chahie, ve ninrrana haim-chaya, kaphi, chakaleta, shitala peya, sharaba, chini, mithaiyam aura aise khadya padatharrra jinamem shakrrakara ho, parishrrakrita anaja, mansa, machhali, mugarrra, tambakhu, dudha, panira, makrrakhana, dhunrapana, namakina va achara vale khadya padatharrra shamila haim, aura aise khadya padatharrra jinamem kisi bhi taraha ka rasayanika avayava jaise yogaja, parirakshaka aura sungadhatrramaka tatrrava shamila haim|
ye khadya padatharrra sharira mem jaharile sanchayana, adhivrikrraka granthiyom ko adhika utrratejita karane, aura agrranyashayi kinvaka utrrapadana ko vikrita karane ya rakta shakarrrara santulana mem bigaड़ adi ka karana hote haim|
nivarana ke uddeshyom se, sampunarrra si kaॉmprralekrrasa vitamina- jo bayophlevonaॉyadsa ke nama se jane jate haim, inako lene ki salaha di jati hai|
ve dhire-dhire, sharira ko bahuta si elajirrrayom, visheshatah paragaja jrravara ke sankramana se pratirakshana karane ke lie, koshikaom ki paraganrrayata ko majabuta banate haim|
akrrasara bi5 ya paintothenika anrrala ka sanyojana, elajirrra ki bimari mem bahuta rahata pahunchata hai|
bahuta si elajirrrayam shayada, adhivrikrraka granthi ke kayarrra karane ki kshamata mem ane vali kharabi ke parinama srravarupa, utrrapanrrana ho sakati hai| 
isa taraha ke rogiyom mem painatothenika anrrala ki prachura matra, unhem rogamukta hone mem madada karati hai, yadyapi srravasrrathrrayalabha mem kai haphrrate lagenge|
jaisa ki kuchha adhrrayayanom mem bataya gaya hai ki, vitamina i mem prabhavashali elajirrra-virodhi tatrrava pae jate haim, atah prachura matra mem vitamina i ka sevana bhi phayademanda hota hai| 
yaha akrrasara parishrrakrita khadya padathorrram ke sevana ke parinamasvarupa hoti hai, aura yaha manava jati ko prabhavita karane vali sabase ama bimariyom mem se eka hai|
hamari dhamaniyom va shiraom mem pravahita hone vale lagabhaga adhe rakrrata mem, lala rakrrata kanikaem hoti haim, jo utakom taka prana vayu le jati haim|
lala kanikaem lagabhaga 120 dina taka jivita rahati haim, aura phira nashrrata ho jati hai tatha pratidina nai kanikaem inaka sthana leti haim|
pratyeka vrrayakrrita mem 100 pratishata rudhira-varnika (himogrralobina) ya lagabhaga 15 grama se 100 si si rakrrata hona chahie aura rakrrata mem prati milimitara mem pancha lakha lala kanikaem honi chahie|
rudhira-varnika (himogrralina) ki matra mem kami ke parinamasvarupa rakrrata kshinata hoti hai|
marija akrrasara kamajori, thakavata, shakrrita ki kami aura chakrrakara ki shikayata karata hai|
anrraya lakshanom mem shamila haim, mariyala sa dikhana, samaya se pahale juriyom ka ana, nirasa va thaki hui ankhem, kamajora srramarana shakrrita, thoड़i si hi mehanata karane para hamphane lagana, siradadarrra, ghava ke thika hone mem samaya lagana, hridaya ka teji se dhaड़kana aura manasika udasi |
trravacha aura shrralema jilliyom ka pila dikhana, nakhunom ko bhangura dikhana aura age ki ora kinare para unamem dadarrra bhi ho sakata hai|
asthi majja mem dhimi gati se lala rakrrata kanikaom ka nimarrrana, yaha ya to asthi majja mem dosha se, ya phira lauha, vitaminom aura protinom ke apayarrraprrata matra mem sevana se hota hai, jo ki rakrratahinata ka eka mukhrraya karana hai| 
anrraya mukhrraya karanom mem, kisi dugharrratana ke karana atrrayadhika khuna ka baha jana, stravi bavasira aura mahilaom mem atrrayadhika masika dharma ka hona bhi shamila hai| 
isake alava, pachana anrrala ya haidrokrralorika anrrala ki kami, jo lauha va protinom ke pachana ke lie avashrrayaka hai, ya bhavanatrramaka tanava, chinta aura pareshani, jo sharira mem haidrokrralorika anrrala ke nimarrrana mem badha dalati hai, arakrratata ka bhi karana bana sakati hai|
anrratra parajivi ya kida़e bhi, arakrratata ke anrraya karana haim|
ankada़a-kida़e, pina-kida़e, kenchuem aura tepa (lamba)-kida़e, rakrrata ke satha-satha vitaminom ko kha kara jivita rahate haim| 
anrratra parajivi ke lakshana haim, malashaya mem khujali, sote samaya bure sapanom ka ana va bechaini, dasrrata, dugarrrandhadara shrravasa, ankhom ke niche kale ghere aura lagatara khane ki ichrrachha ka bani rahana|
lahasuna, taja papita aura ghisi hui kachrrachi gajara, kuchha prakara ke anrratra parajiviyom ko nashrrata karane mem sahayaka ho sakate haim|
arakrratata ke upachara mem ahara pramukha rupa se mahatrravapunarrra hai|
parishrrakrita khadya padatharrra jaise-sapheda dabalaroti, parishrrakrita chavala, shakrrakara aura mithaiyam, sharira ko avashrrayaka lauha se vanchita kara dete haim|
lauha ko hamesha, bhojana mem isake prakritika jaivika rupa mem grahana karana chahie, krrayonki ajaivika lauha ka prayoga khataranaka sabita ho sakata hai| 
yaha sanrakshi vitaminom aura asantriprrata vasa anrralom ke nashrrata hone, va gambhira yakrita kshati, gabharrradharana ke daurana gabharrrapata aura dera se ya samaya se pahale hi janrrama, ka karana bana sakata hai|
ahara ko mukhyatah kshariya hona chahie|
vishesha dhrrayana kachrrache phala aura sabrrajiyom ki ora hona chahie, jinamem lauha prachura matra mem hota hai|
lauha ki matra se bharapura sabrrajiyam haim, palaka, hari prrayaja, kunhaड़a, gajara, muli, chukandara, ajamoda, aravi, tamatara aura alu (chhilake ke satha)|
phala jinamem lauha prachura matra mem paya jata haim, ninrranalikhita hai- kele, seva, kale angura, khubani, alubukhara, kishamisha aura srratraberi| 
kele vishesharupa se phayademanda haim, krrayonki inamem asani se pachane vale lauha ke alava, pholika anrrala aura bi 12 bhi paya jata hai, aura ye donom hi arakrratata ke upachara mem atrrayadhika upayogi haim|
anrraya lauha mem prachurata vale khadya padatharrra haim- gehum, kodu, sema, soya sema, suyarrramukhi ke bija, ashodhita guड़rasa, anrrade aura shahada|
shahada mem kaॉpara bhi prachura matra mem paya jata hai, jo lauha ke avashoshana mem sahayaka hota hai|
ahara mem uchrracha jaiva mulrrayata ke, protina ki prachurata vale khadya padatharrra, jaise - dudha, ghara mem banayi gayi panira aura anrrade, bhi shamila hone chahie| 
vitamina bi-12 arakrratata se raksha ya upachara ke lie atyavashrrayaka hai|
yaha vitamina akrrasara jantu protina aura visheshatah gurade aura yakrita jaise jaivika mansa mem paya jata hai|
atrrayadhika uttama ahara ko akrrasara, adhika rudhira-varnika (himogrralina) aura adhika lala kanikaom ke satha joda़a jata hai, kintu isake apane nukasana bhi hai| 
arakrratata ka eka aura karana anrratra sadana hai, jo mukhrraya rupa se mansa ka adhika sevana karane ke karana hota hai|
isake alava, pashuom mem vrrayapta bimariyom ke karana dhire-dhire sabhi taraha ke mansa khataranaka hote ja rahe haim|
phira bhi, vitamina bi 12 ke kuchha aura bhi achrrachhe srota upalabrradha haim, jaise-deri ke utrrapada, dudha, anrrade, panira aura mungaphali|
gehum ke ankura aura soya sema mem bhi, kuchha matra mem bi 12 paya jata hai| 
shakahariyom ko apane ahara mem, payarrraprrata matra mem dudha, dudha ke utrrapada aura anrradom ko shamila karana chahie|
arakrratata se bachava ke lie, jaruri hai ki sabhi taraha ke bi kaॉnrrapalekrrasa, jinamem bi 12 aura usake satha-satha uparokrrata prakritika khadya padatharrrom ka sevana kiya jana chahiye|
dudha se bane utrrapadom ka sevana karem, jisamem sabhi protinom ko samavishrrata karane vala vitamina bi 12 hai, jo isa bimari ke viruddha eka achrrachha kavacha hai| 
esrrakaॉrabika anrrala ka prachura matra mem sevana, lauha ke avashoshana ki kriya ko sarala banane ke lie avashrrayaka hai|
nimbu-vansha ke phalom ke kama se kama do bhojanansha aura anrraya esakorabika anrrala ki prachurata vale phalom ka sevana pratidina karana chahie| 
chukandara ka jikra yaham avashyaka hai, kyonki arakrratata ke upachara mem yaha atimahatrravapunarrra hai|
chukandara ke rasa mem, adhika lauha ansha ke satha, potesiyama, phasaphorasa, kailashiyama, gandhaka, ayodina, lauha, tamba, kaborrrahaidretasa, protina, vasa, vitamina bi1, bi2, niyasina, bi6, si aura vitamina pi bhi paya jata hai, tatha chukandara ka rasa lala rakrrata kanikaom ko punanirrramirrrata va punahsakriya karata hai, tatha sharira ko taji prana vayu ki aputirrra karata hai|
arakrratata vale vrrayakrrita ko apana ahara upachara taje uttama phalom se arambha karana chahie, lagabhaga pancha dinom ke lie |
isa daurana use, pancha-ghantom ke antarala para, dina mem tina bara taje rasadara phalom ka ahara lena chahie|
isake bada lagabhaga 15 dinom ke lie phala aura dudha ka ahara liya ja sakata hai| 
isa pathrrayapathrraya niyama mem, ahara sabhi phala aharom jaisa hi hai, lekina yaham pratyeka ahara ke satha dudha bhi shamila ho gaya hai|
marija pahale dina do pinta (dedha़ pava) se shuru kara sakata hai, aura pratidina adha pinta badha़akara, ise eka dina mem chara-pancha pinta taka kara sakata hai|
phala aura dudha ke ahara ke bada, marija dhire-dhire achrrachhe-santulita ahara, jisamem tina mukhrraya khadya samuha ho-bija, giridara phala aura anaja, sabrrajiyam aura phala shamila hom, inaka sevana arambha kara sakata hai|
yaha krimirupi unrradukapuchrrachha mem jalana va sujana ora sanketa karata hai|
yaha apane apako tikshrrana aura gambhira rupa mem prasrratuta karata hai, aura donom hi lingom ko samana rupa se prabhavita karata hai|
yaha bimari aba 10 se 30 vasharrra ki unra ke bicha mem hue, tikshrrana, udariya akasmika darda ka karana hai|yaha alpavikasita deshom ki apeksha vikasita deshom mem jrrayada payi jati hai|
unrrakupuchrrachha eka chhoti si nalika hai, jo baड़i anta ke pahale hisrrase, anrradhranta, ke anta mem hoti hai|
yaha samanyatah tina se chara incha lambi hoti hai|
isaki sanrachana usi sakhrrata tantumaya bahari avarana se nimirrrata hoti hai, jo sampunarrra poshana-nala ki suraksha karata hai|
bahari avarana ke niche peshiya utakom ki eka parata hoti hai, aura usake bada lasika uti ki eka parata hoti hai| 
unrrakupuchrratha ka karya, jo isa lasika uti ke dvara kiya jata hai, vaha hai, sharira mem banane vale, takalipha़deha vrrayatharrra padathorrram ya trravacha athava jilrrali ke madhrrayama se ane vale jaivika visha, ko nishrraprabhavita karana|
unadukapuchrrachhashotha akrrasara, udara ke madhya mem hone vale akasrramika dadarrra se shuru hota hai, jo dhire-dhire niche dahini ora badha़ta jata hai| 
dadarrra se pahale, udara mem pareshani, apachana, dasrrata ya kabrraja ho sakata hai|
marija ko akrrasara, 100 se 102 digri pheranahaita ka halrraka bukhara rahata hai|
michali ana to ama bata hai, aura marija do ya tina bara ulati kara sakata hai|
peta ke danyi ora ki peshiyam tani hui va sakhrrata ho jati haim|
marija ko danyi tanga khichanem se kuchha rahata mahasusa hoti hai|
udara ke banyi ora dabane se, danyi ora ka dadarrra badha़ta jata hai|
khansane aura chhinkane se dadarrra, aura bhi badatara ho jata hai|
yadi sujana aura jalana lagatara badha़ti rahi to, ho sakata hai, ki undukapuchchha phata jae aura udara guhika mem pasa ka pravaha kara dem| 
isaka parinama, antom ki jilli ki sujana namaka gambhira avastha ho sakati hai| 
tapamana badha़ jata hai, arthata bukhara ho jata hai, aura marija pila tatha chipachipa (pasine se) sa ho jata hai| 
isa sthita mem turanta shalya-chikitsa karani pada़ sakati hai| 
unduka-shotha ki lambi avastha mem, marija kabja, bhukha na lagane aura halki michali ane ke satha, danyi ora ke nichale udara mem lagatara tivra darda se pida़ita ho sakata hai| 
unduka-shotha, vishakta antom ki sthiti ke karana hota hai| 
jaba adhika matra mem jaharila mala, unduka mem ekatrita ho jata hai, to undukapuchchha ugra, adhika-kriyashila tatha lala ho jata hai| 
yaha prakriti ka apani ora se, visha ko simita rakhane va use nashta kara dene ka eka prayasa hai| 
yaha sthita galata ahara ki adata va tantra ki durbalata ke karana utpanna hoti hai| 
antra nali mem kuchha kitanu jo aksara hote haim, unake karana aura adhika sujana va sankramana utpanna ho jata hai| 
jaise hi tivra darda, ulti aura bukhara ke prathamika lakshana dikhem, marija ko turanta arama ke lie lita dena chahie| 
isa bimari ke upachara mem arama bahuta hi jaruri hota hai| 
marija ko upavasa karana chahie, jo unduka-shotha ka ekamatra sachcha upachara hai| 
purnatah khana dena banda kara dena chahie| 
jala ke alava tantra mem kisi bhi chija़ ka pravesha varja hona chahie| 
nichali anta ko sapha karane ke lie, shuru ke tina dinom taka, pratyeka dina, lagabhaga eka pinta (1/2 litara) garma pani ka, kama enima dena chahie| 
pratidina, kai bara darda vali jagaha para garma patti rakhi ja sakati hai| 
phalalena ke sukhe kapada़e se avaranita, gili pattiyom se bane udara paika ko, udara ke charom ora kasakara bandhana chahie, aisa lagatara taba taka karana chahie, jaba taka sabhi gambhira lakshana ghata na jae| 
jaba tisare dina sabhi gambhira lakshana ghata jae, taba marija ko lagabhaga tina pinta garma pani ka, eka purna enima dena chahie, tatha ise pratidina taba taka doharana chahie, jaba taka ki jalana, sujana aura darda ghata na jae| 
marija ko tisare dina se hi phalom ka rasa diya ja sakata hai| 
isa sadharana upachara ko samajadari ke satha apanane (lagu karane) se, unduka-shotha se chhutakara milega| 
tina dinom taka phalom ke rasa para rahane ke bada, marija age ke chara ya pancha dinom ke lie sabhi phalom ke ahara ko apana sakata hai| 
isa daurana, use eka dina mem tina bara taje rasadara phalom ka ahara lena chahie| 
isake bada hi use, eka achchhe-santulita ahara ko apanana chahie, jisamem tina mulabhuta khadya padartha - bija, kashthadara phala aura anaja, tatha sabjiyam aura phala shamila hom| 
lambi unduka-shotha ki avastha mem, do se tina saptaha taka ke, alpa samaya ke lie upavasa ke satha, kevala dudha ka ahara lena chahie| 
isa pathyapathya niyama mem, pahale dina subaha 8 baje se shama 8 baje taka ke bicha, pratyeka do ghante mem , agale dina prati dedha ghante mem , tatha tisare dina prati eka ghante mem eka glasa dudha pina chahie|
usake bada, dudha ki matra badha़ai ja sakati hai, atah yadi prati adhe ghante mem eka glasa dudha apa arama se pi sakate haim, to isa matra mem dudha ka sevana karana chahie| 
sampurna dudha ahara ke bada, marija ko taje phalom aura hari pattedara sabjiyom para vishesha dhyana dete hue, dhire-dhire eka achchhe-santulita ahara ko lena arambha karana chahie| 
kuchha sabjiyom ke rasa, visheshatah gajara ka rasa, vo bhi chukandara va kakada़i ke rasa ke satha, lena unduka-shotha ke upachara ke lie atyadhika labhadayaka mana gaya hai| 
methi ke danom se bani chaya ka niyamita rupa se sevana karane se bhi, unrradukapuchchha ko adhika shlema aura anta ke avashesha ka kshepana kshetra banane se bachane mem sahayata milati hai| 
unduka-shotha ke marija ko, yadi lagatara kabja ki shikayata hoti hai, to use isake unmulana ke hara sambhava upaya apanane chahie| 
garma sikai aura udariya lepom ka subaha shama prayoga karane se bhi, marija ko kaphi arama mila sakata hai| 
udariya malisha bhi kaphi labhadayaka hoti hai| 
eka bara mala, unrraduka se malashaya mem a jae aura vaham se bahara nikala jae, to undukapuchchha mem pareshani va jalana kama ho jaegi, tatha undukapuchchha ko shalya-chikitsa dvara nikalane ki jarurata bhi nahim hogi| 
shalya-chikitsa ka, kevala kuchha durlabha sthitiyom mem hi sahara lena chahie, jaba undukapuchchha putika ho jae| 
yaha kailshiyama ya chune ke karana, dhamaniyom ki bhittiyom ki saghanata ki ora sanketa karata hai| 
yaha adhunika samaya ki eka samanya bimari bana chuki hai jisake karana vriddha vyaktiyom mem akshamata aura nrityudara badha़ gaya hai| 
dhamani kathinya aksara dhamani ki bhittiyom mem saghanata ke satha hota hai, rakta nalika ki bhittiyom mem eka prakara ka vikara ya unake bhitari parata ka nridukarana | 
isa prakara ki vikriti ke lie, hridaya ki chakriya nalika ya mastishka taka jane vali dhamaniyam sabase khataranaka jagaha hai | 
dhamani kathinya ke parinama srravarupa, rakta ke svachchhanda pravaha mem hasrratakshepa karane vali, chhoti dhamaniyom ki sankirnata ke satha, raktanalika ka lachilapana khatma ho jata hai| 
ye parivartana dhire-dhire koshikaom aura shiraom taka bhi pahuncha sakate haim| 
dhamani kathinya ki pravritti mahilaom ki apeksha purushom mem, visheshatah navayuvakom mem jyada payi jati hai| 
aisa anumana lagaya hai ki 40 sala se adhika ayu vale, 40 pratishata vyaktiyom mem, unaki chakriya dhamani mem avarodhana ki ullekhaniya matra payi jati hai, aura yaha kisi bhi samaya hridayaghata ki ora agrasara ho sakati hai|
antargrasta dhamani ke adhara para, dhamani kathinya ke lakshana badalate rahate haim| 
aparyapta rakta apurti ke lakshana, samanyatah sarvaprathama pairom mem najara ate haim| 
halke vyayama ke bada bhi pairom mem sunnapana aura thithurana aura ainthana va darda ho sakata hai | 
yadi chakriya dhamaniyam shamila haim, to marija ko teja darda ho sakata hai, jo hritshula ka lakshana hai| 
jaba mastishka ki ora jane vali dhamaniyam shamila hoti haim, taba shiraem phata sakati haim, jisase ki mastishka ke utakom mem rakta srava hota hai | 
yadi rakta ke thakke ke satha avarodhana hai, to sharira ke eka tarapha anshatah ya purnatah lakave ke satha, pramastishkiya nadiaghata ke rupa mem parinamita ho sakata hai| 
isake karana smarana shakti bhi kho sakati hai aura vriddha logom ke mastishka mem sanshaya ki sthita utpanna ho sakati hai| 
yadi gurde ki ora jane vali dhamaniyam shamila haim, to marija uchcha raktachapa aura gurde ke roga se pidita ho sakata hai| 
dhamani kathinya ka sabase mahatvapurna karana parishkrita chini, parishkrita khadya padarthom aura adhika vasa yukta ahara tatha kolestrola ki adhikata vale aharom ka adhika sevana karana hai| 
dhamaniyom ke kathorikarana ka karana, kuchha anya bimariyam, jaise- uchcha raktachapa, motapa, madhumeha, gathiya, maleriya aura garami adi bhi ho sakati haim| 
bhavanatmaka tanava bhi mahatvapurna bhumika ada karata hai, aura visheshatah sthanabaddha karya se june logom ki, manasika tatha bhavanatmaka avyavastha ke daurana, hridayaghata ki ashanka aura bhi samanya ho jati hai| 
anuvanshikata bhi apani bhumika nibhati hai, aura yaha bimari vanshom se parivarom mem chali ati hai| 
yadi dhamani kathinya ka karana jnata ho jae, to ise khatma karane ke lie, pancha se sata dinom ke lie, phalom ka rasa pina arambha karake, chikitsakiya karyavahi shighra ki jani chahiye | 
mausama mem upalabdha sabhi taji kachchi sabjiyom aura taje phalom ke rasa ka sevana kiya ja sakata hai| 
angura ka rasa, anannasa ka rasa, nibu ka rasa aura hari sabjiyom ka rasa visheshataura para phayademanda hota hai| 
upavasa ke daurana, antom ki saphai ke lie, pratidina garama pani ka enima lena chahie| 
phalom ke rasa para adharita upavasa ke bada, marija ko kachche khadya padarthom para vishesha dhyana dete hue ninna tina mukhya khadyannom, kramashah bija, kashthadara phala aura anaja, sabjiyam aura phala ka sevana karana chahie| 
bahuta se kachche aura ankurita bija, tatha giridara phalom ka sevana karana chahie| 
thande pere hue vanaspati telom, visheshakara suryamukhi ka tela, alasi ke bijom ka tela aura jaituna ke tela ka niyamita prayoga karana chahie| 
age bhi svasthyalabha ke adhara para, tina mahine ya aise hi anya samayantarala para, phalom para adharita laghu upavasa ko apanaya ja sakata hai| 
marija ko eka ya do bara adhika ahara lene ki apeksha, kai bara thoda़a-thoda़a ahara lena chahie| 
use vanaspati vasa aura santripta vasa ki adhikata vale khadya padartha, jaise-makkhana, malai, ghi aura mansa ke sevana se paraheja karana chahie| 
use mansa, namaka aura sabhi parishkrita va sansadhita khadya padarthom, masalom, sasa, achara, kada़ka chaya, kaॉphi, parishkrita chini, maida aura unase bane sabhi padarthom ke sevana se paraheja karana chahie| 
elyuminiyama aura tambe ke bartanom mem pakae gae khadya padarthom ka sevana nahim karana chahie, kyonki isase vishakta padartha sharira mem pravesha kara, mahadhamani aura dhamani ki bhittiyom mem ekatrita ho jate haim| 
dhunrapana yadi adatana haim, to isaka parityaga kara dena chahie, kyonki dhunrapana dhamaniyom ko sankuchita kara deta hai aura sthiti ko gambhira banata hai| 
vartamana khojom se pata chala hai ki, lahasuna aura pyaja, dhamani kathinya ke vikasa mem badhaka ka kama karate haim| 
kolestrola ko pitta anlom mem parivartita karane mem sahayata karane ke karana vitamina si bhi labhadayaka siddha hua hai| 
marija ko bahuta se bahya vyayamom ko apana kara, sabhi manasika tanavom va chintaom se dura rahana chahie| 
garma snana ya savadhani se apanaya gaya shitala snana, bhi sahayaka hota hai| 
dirghakala taka ya eka dina ke samayantarala mem, sote samaya kiya jane vala halka snana bhi labhadayaka hota hai| 
gathiya shabda ka ashaya hai, 'joda़om mem sujana va jalana hona' | 
yaha do grika shabdom ko milakara bana hai, athrona ka ashaya hai-joda़ aura itisa ka ashaya hai-jalana/sujana/darda| 
prarambhika avastha mem, aksara sampurna sharira shamila rahata hai, tatha eka ya do joda़ purnatah vikrita ho jate haim, jo ki marija ko apanga aura bahuta hi kamajora bana deta hai| 
joda़o ki do shreniyam hoti haim, kramashah pahali hai, achala sandhi ya jo bahuta jyada hilati nahim hai aura jinamem kotara nahim hota haim aura dusari hai, chala sandhi ya svatantratapurvaka hilati hai aura jinamem sanyukta kotara bhi hota hai| 
pahale prakara ke joda़, sira va merudanda़ mem paye jate haim| 
dusari taraha ka joda़, jo jyadatara gathiya se prabhavita hote haim, bahuta jyada hote haim, aura ye kandhom, kohiniyom, kalaiyom, ungiliyom, ghutanom, takhanom aura paira ke anguthe mem paye jate haim| 
gathiya kai prakara ka hota hai, unamem se jyadatara hone vala gathiya haim-asthi-sandhi-shotha aura amavati gathiya| 
gathiya ka mukhya lakshana hai, sujana/darda, jo joda़om ke utakom mem hui, kisi taraha ki gada़bada़i ya chota ki pratikriya hoti hai| 
asthi sandhi shotha joda़om ki eka apakarshaka bimari hai, jo aksara vriddha logom mem hoti hai| 
yaha purushom ki apeksha, mahilaom mem jyada payi jati hai| 
yaha bimari, aise joda jo visheshatah adhika vajana sahate haim, jaise-ridha ki haddi aura ghutanom ki sandhi-upasthi mem hue sanrachanatmaka parivartana ka parinama hai| 
asthi sandhi shotha ke mukhya lakshana, joda़om mem darda aura akadana hona hai| 
aksara vyayama ke bada darda aura bhi badha़ jata hai| 
anya lakshanom mem shamila haim-ankhom mem pani hona, pairom mem ainthana hona, elarji, asthi sandhi shotha, pittashaya ke kriya-kalapa mem kshati aura yakrita mem gada़bada़i | 
isake sambhavita karanom mem shamila haim, kuposhana, lagatara sharirika thakavata, motapa, granthiya karya kshamata mem kami, kailshiyama ki kami aura haidroklorika anla ki kami| 
amavati gathiya eka gambhira bimari hai, jo na kevala ungaliyom, kalaiyom, kulhom, ghutanom aura pairom ko prabhavita karati hai, balki mansa-peshiyom, nasom aura sharira ke anya utakom ko bhi prabhavita karati hai| 
yaha bachapana se lekara vriddhavastha taka, kabhi bhi ho sakata hai, kintu aksara 25 se 50 varsha ki ayu ke bicha prakata hota hai| 
yaha purushom ki apeksha, mahilaom mem tina guna adhika payi jati hai| 
yaha bimari sandhi-snehika mem sujana ke karana ya joda़om ki bhitari parata mem sujana va sambhavita vikriti ke dvara hoti hai| 
amavati gathiya ko aksara kukada phuda disija़ kaha jata hai| 
aksara yaha kai mahinom mem, eka ya eka se adhika joda़om mem, sthayi darda aura akada़na ke satha, dhire-dhire vikasita hota hai| 
antatah sampurna sharira prabhavita hota hai| 
isake lakshanom mem -araktata, vrihadantra shotha, kabja, pittashaya mem gada़bada़i, ninna raktachapa, vikrita hatha aura pamva adi shamila hai| 
yaha sthiti nyasargom (haramona) mem asantulana, sharirika aura bhavanatmaka tanava, sankramana, tivra bhaya, aghata aura chota adi ke karana utpanna ho sakati hai| 
anuvanshika karana bhi isa bimari ke lie uttaradayi ho sakate haim| 
adhunika chikitsiya peshe ke anusara, gathiya ka koi bhi ilaja nahim hai, aura marija ko isaki ke satha jina sikha lena chahie| 
yadyapi, prakritika chikitsa, bimari ke ahara-sambandhi upachara para bharosa karati hai| 
adhikansha chirakalika gathiya rogi khuba khane vale hote haim aura aksara 3,500 se 4,000 kailori dene vale ahara liya karate haim|
chunki ve grahana kie gae isa bhojana ke samasta shvetasariya tatvom ka upayoga nahim kara sakate, ata: jivavisha sangraha hone lagate haim aura atyadhika anla apashishta ka parinama hota hai sandhiyom ki maujuda sthiti mem vriddhi |
alpatama karbohaidreta ghataka ke satha lagabhaga 2,000 kailori vale alpa-kailori ahara ka sujava diya jata hai|
tathapi, isa ahara mem vitaminom, kailshiyama, phaॉsphorasa aura lauha ki paryapta matra ka samavesha hona chahie|
gathiya ke marija ka ahara kshariya prakriti ka hona chahie tatha *usamem pratiraksha ke lie phalom aura sabjiyom tatha urja ke lie protina aura karbohaidreta ka samavesha hona chahie|
isamem saladom ke rupa mem eka-do taji kachchi sabjiyam aura kama se kama do paki hui sabjiyam ho sakati haim|
pattagobhi, gajara, ajamoda, khira, kasani, kahu, pyaja, muli, tamatara aura jalakumbhi kachche salada ke lie prayukta kie ja sakate haim|
paki hui sabjiyom mem shatavari, chukandara, phulagobhi, pattagobhi, gajara, ajamoda, baingana, masharuma, pyaja, matara, binsa, palaka, skvasha, tamatara aura shalagama shamila ho sakate haim|
gambhira sthitiyom mem, yaha uchita hoga ki marija ko lagabhaga eka saptaha ke lie vanaspatika rasopachara para rakha jae|
gajara, ajamoda tatha lala chukandara ke rasa mem milakara kisi bhi hari pattedara sabji se nikala hua hara rasa gathiya ke lie vishesha rupa se sahayaka hota hai|
kachche rasom ki kshariya kriya sandhiyom ke asa-pasa tatha anya utakom mem ekatrita sangrahom ka vilaya karati hai|
taja ananasa bhi upayogi hai kyonki taje ananasa rasa ka kinvaka, asthisandhishotha tatha amavati gathiya mem jalana aura shotha ko kama karata hai| 
hara do mahine ke antaralom para punaravritta rasa upavasa ki salaha di jati hai|
amavata tatha gathiya ki dashaom mem kachche alu ka rasopachara atyadhika saphala jaivika upacharom mem se eka mana jata hai|
yaha sadiyom se loka aushadhi mem vyavahrita hota aya hai|
alu ka rasa taiyara karane ki purani vidhi thi bina chhilaka utare alu ko patale-patale katarom mem katana aura ratabhara thande pani se bhare eka baड़e gilasa mem chhoड़ dena|
yaha pani subaha-subaha khali peta piya jana chahie|
aluom se taja rasa bhi nikala ja sakata hai aura 50:50 ke anupata mem pani ke satha milakara, subaha-subaha sabase pahale piya ja sakata hai|
kuchha kha़asa khadya padartha gathiya ke marijom ke lie hanikaraka haim aura unhem bhojana se nikala diya jana chahie|
inamem shamila haim kisi bhi prakara ke vatita jala, panira ko chhoड़kara sabhi chija़, bekana, haima, saॉseja tatha sanrakshita mansa, pestri, keka, pai, mithe bana aura sapheda dabalaroti, sabhi salada dresinga, mansa rasa se bane sabhi supa, chavala tatha maide ke utpada|
kaindi, madhuraka, shakkara, aiskrima, masalom, chaya aura kaॉpha़i se bhi paraheja kiya jana chahie|
gathiya mem jina phalom ki anumati di gai hai ve haim seba, nimbu, santare, kele, nasapatiyam, vividha beriyam, khubani, ananasa, alubukhare aura melana|
vitamina e aura di sankramana se bachava karane, phalasvarupa gathiya ko rokane, mem mahatvapurna bhumika nibhate haim|
santare, papite, gajara, shuddha dudha aura makkhana, sabhi hari pattedara sabjiyam, tamatara aura kachche kele vitamina e se sanriddha haim|
vitamina di mukhyata: tvacha ko dhupa dikhane se prapta hota hai|
gathiya ke nivarana mem suryaprakasha eka pramukha karaka hai|
kabja se bachana chahie kyonki yaha pranali ko vishakta karata hai tatha joड़om ki jalana aura sujana baढ़ata hai|
tahalane jaise halke vyayama labhadayaka hote haim|
gathiya rokane mem sharira ka samanya vajana banae rakhana bhi eka mahatvapurna karaka hai|
motapa vajana sahane vale joड़om para atyadhika davaba dalata hai aura kandaraom, asthibandhom aura mansapeshiyom ke nirbadha karya mem hastakshepa karata hai|
sharira ko hara samaya garma rakha jana chahie|
joड़om para kasakara patti nahim bandhi jani chahie kyonki yaha anga-sanchalana ko parisimita karata hai tatha rakta ke mukta paribhramana mem dakhala deta hai|
shayana kaksha mem prachura samvatana honi chahie|
gathiya se grasita logom ke lie arama baड़a mahatvapurna hai(arama baड़a mahatvapurna hai unake lie jo gathiya se grasita haim), jinhem apana karya ya vyayama bahuta nahim karana chahie ya na hi manoranjanatmaka karyakalapom mem lipta hona chahie|
shvasanali-shotha phephaड़e ke shvasa-nali athava vayu marga ke sujana ki ora sanketa karata hai|
yaha ni:shvasana-prakriya ko prabhavita karane vala eka shvasana vikara hai|
jyadatara mamalom mem, kuchha sankramana naka aura gale mem bhi hote haim|
shvasanali-shotha tivra ya chirakalika ho sakata hai|
chirakalika mamalom mem, roga lambi avadhi se rahata hai|
yaha tivra prakara se jyada gambhira kisma ka hai kyonki ho sakata hai phephaड़om mem sthayi parivartana ho chuka ho, aura isa prakara unake samanya karya-sanchalana mem badha pahuncha raha ho|
chirakalika shvasanali-shotha ayu-vriddhi ki eka dasha aura sambhavatah shvasa-nali ka vikara hai joki vatavaraniya pradushana, shvasa mem lie gae jaivikom tatha dhunrapana karane valom mem, kai varshom se sansom mem svayam lie ja rahe kshobhakari tatvom se prapta jalana ke kai varshom ke adhina hai|
yaha mahilaom ki tulana mem purushom mem jyada raha hai aura nrityu-dara bhi purushom mem kahim adhika hai|
shvasanali-shotha ke adhikansha mamalom mem, kanthanali, shvasa-pranala tatha shvasa-nalikaem tivra rupa se shothagrasta ho jati haim|
jalana ke karana utaka suja jate haim|
kaphi matra mem shleshma bahara nikalati hai aura shothagrasta shleshma-jilliyom ki raksha ke lie vayu pravaha-nali mem uda़ela di jati hai|
samanyata: thoड़a bukhara, aura sansa lene mem kathinai ke satha-satha chhati mem gahara balagama rahata hai|
anya lakshana haim gala baithana, chhati mem darda, takata ki kami aura khinnata|
shvasana ki yaha kathinai taba taka bani rahati hai jaba taka sujana kama nahim ho jati aura shleshma nikala nahim jati|
chirakalika mamalom mem, shvasa-nali mem atyadhika shleshma paida hone ke atirikta, sankramana aura phephaड़e ka kshaya bhi hota hai|
tivra shvasanali-shotha bahudha samanya sardi jukama ke pratiphala ke rupa mem ya imphluenja ke hamale ke parinamasvarupa ho sakate haim|
roga ki gambhirata sankramana ke vistara ki adhikata para nirbhara karati hai jo sardi, ardrata aura dhuem ki avasthiti se ugra ho jata hai|
tivra shvasanali-shotha ke bara-bara akramana aura doshapurna ahara adatom ke karana roga chirakalika bana jata hai|
adatana parishkrita aharom, jaise shveta sharkara, parishodhita anajom aura maide ke utpadom ka prayoga karane ke phalasvarupa pranali mem vishakta padartha jama hone lagata hai aura yaha vishakta apashishta shvasa-nalika mem ikattha hone lagata hai|
shvasanali-shotha ke anya karana haim atyadhika dhunrapana, damaghontu vatavarana mem rahana ya kama karana tatha purva rogom ke shamana ke lie davaom ka upayoga|
vatavaraniya dasha aura jalavayu roga ke akramana ke mahatvapurna karaka tatva haim|
pradushana jitana hi adhika hoga, chirakalika shvasanali-shotha ki dara bhi utani hi adhika hogi|
shvasanali-shotha ke tivra mamalom mem, lakshanom ke shanta hone taka marija ko santare ke rasa aura jala para upavasa karana chahie|
prakriya yaha hai ki 8 baje subaha se 8 baje rata taka eka santare ka rasa eka glasa garma pani mem pratyeka do ghante para liya jae| isa avadhi mem, ushna jala ki enima se antom ki rojana saphai ki jani chahie| 
rasa-upavasa ke bada, do ya tina dinom ke lie marija ko purna phalahara apanana chahie|
chirakalika shvasanali-shotha ke mamale mem, upachara ke arambha mem marija pancha se sata dinom ke lie purna phalahara apana sakata hai|
isa pathyahara mem, use hara roja tina bara taje phalom jaise sebom, nasapatiyom, angurom, chhote chakotarom, santarom, ananasom, ada़uom aura melanom ka ahara grahana karana chahie|
kele aura sukhe athava damapukha़ta ya tinabanda phala nahim khane chahie|
peya ke rupa mem, mithasa-rahita nimbu pani athava thanda ya garma sada pani liya ja sakata hai|
purna phalahara ke bada, marija ko tina mulabhuta suvargom, yatha, bijom, kashthaphalom aura anajom, sabjiyom tatha phalom, para adharita eka susantulita ahara ka palana karana chahie, jaisaki svasthya aura roga mem ahara ke adhyaya 1 mem rekhankita kiya gaya hai|
marija ko mansom, shakkara, chaya, kaॉpha़i, masalom, acharom, parishkrita evam sansadhita bhojya padarthom, nridula peyom, kaindiyom, aiskrima tatha shakkara aura maide se bane utpadom se paraheja karana chahie|
shvasanali-shotha ke upachara mem kuchha kha़asa sabjiyom ke rasa bahumulya pae gae haim|
gajara aura palaka ka sammishrita rasa vishesha rupa se labhadayaka hai|
isa sammishrana mem nuskha anupata upayogi paya gaya hai, vaha hai 16 aunsa ya 1 pinta rasa taiyara karane ke lie 10 aunsa gajara aura 6 aunsa palaka|
akramana ki tivra avastha mem hara ya hara dusari rata ko garma ipsama namaka snana atyanta labhadayaka rahega|
marija ko lagabhaga 20 minata taka snana mem nimajjita rahana chahie|
chirakalika shvasanali-shotha ke mamale mem, yaha snana saptaha mem do bara liya ja sakata hai|
garma nichoड़e hue taulie ka chhati ke upari bhaga para prayoga karana bhi upayogi siddha hota hai|
shvasanali-shotha ke upachara mem taji hava aura ghara se bahara kie jane vale vyayama bhi avashyaka haim aura marija ko pratidina lambi duri taka tahalana chahie|
isa shabda ka shabdika artha hai jalaprapata|
yaha ankha ke lensa mem aparadarshita ki ora ingita karata hai jisase drishti dhundhali ho jati hai|
jaba prakasha kiranem lensa ki kshamata se gujara kara ankhom mem pravesha nahim kara sakatim to parinamasvarupa andhapana a jata hai|
motiyabinda ka prathama lakshana hai drishti ka dhundhalapana|
marija vastuom ko kendrita rupa se dekha pane mem kathinai anubhava karata hai|
jaise-jaise motiyabinda baढ़ta jata hai, marija ko dohari drishti ya dhabbe athava donom ka anubhava ho sakata hai|
andhapana dhire-dhire baढ़ta jata hai|
shuru mem, dina ke prachura prakasha ki tulana mem suryasta ke samaya mem drishti behatara ho sakati hai kyonki andhere mem adhika vyapaka phaili putaliyom se hokara prakasha pravesha karata hai|
baढ़i hui avastha mem, vastuem aura loga mahaja prakasha ke dhabbe dikha sakate haim|
antima avastha mem, putaliyom mem dhusara-sapheda rangahinata a jati hai|
motiyabinda aksara ankha ke anya doshom ke sanga-sanga paya jata hai|
chara karaka haim jo lensa ki paradarshita ke kshaya mem yogadana karate haim|
jaisaki bahutere rogom mem hota hai, ahara sambandhi gaड़baड़iyom tatha doshapurna jivana-shaili ke karana rakta-pravaha mem upasthita visha, motiyabinda ka asali karana haim|
rakta-pravaha mem upasthita visha-tatva kinhim upalabdha durbala sthalom para ashraya pane ke lie pure sharira mem phaila jata hai|
yaha lensa ko prabhavita karata hai yadi yaha bhaga tanava, ankhom ke atishaya prayoga ya sthaniya jalana ke karana kamajora ho chuka hai|
samaya gujarate-gujarate sthiti badatara ho jati hai aura taba motiyabinda baढ़na shuru ho jata hai|
kuchha visheshajnom ki manyata hai ki aneka motiyabindom ka sabase mahatvapurna karana hai kuposhana|
yaha bata motiyabinda ki usa koti ke bare mem bhi sacha ho sakata hai jise ama taura para buढ़ape ya baढ़ti unra ka motiyabinda kaha jata hai|
karana jivana bhara ka kuposhana ho sakata hai|
netra visheshajna daॉ. maॉrgana raiphoda, jinhonne kai varshom taka motiyabinda ke bare mem adhyayana kiya hai, doshapurna poshana ko motiyabinda ka eka adharabhuta karaka manate haim|
apane anubhava se unhonne yaha paya hai ki motiyabinda se bachava karane ka arambha poshana mem sudhara karane se hota hai|
samajauta karane ke lie motiyabinda sarvadhika aड़iyala sthiti hai|
agara isane apani gahari jaड़em jama li haim to sankata ke nivarana ke lie shalya-karya se kama kuchha bhi sahayaka nahim hoga|
parantu, yadi, motiyabinda arambhika avasthaom mem hai, to isa vyadhi para prakritika sadhanom se niyantrana kara pane ke suavasara hote haim|
yaham taka ki baढ़e hue mamale bhi badatara hone se roke ja sakate haim|
pranali se vishakta padartha ka saphaya karane ki sampurna prakriya atyavashyaka hai|
arambha karane ke lie, santare ke rasa aura jala ke satha tina se chara dinom ka upavasa rakhana labhadayaka rahega|
isa avadhi mem ushma jala ki enima li ja sakati hai|
isa arambhika upavasa ke bada, do saptahom ke lie bahuta hi sanyamita prakriti ke ahara ka anugamana kiya jana chahie|
isa pathyapathya niyama mem, subaha ke nashte ke antargata santare ya angura athava anya koi bhi rasila mausami phala shamila ho sakata hai|
shama ke bhojana mem apane hi rasa mem bhapa se paripakita palaka, pattagobhi, phulagobhi gajara ya shalagama jaisi sabja़iyam tatha kuchha giriyam athava seba, nashapati aura angura jaise katipaya phala shamila ho sakate haim|
isa suchi mem roti ya anya kisi bhi bhojya vastu ka samavesha nahim kiya jana chahie|
eka chhote-se upavasa ke bada sanyamita pathyahara upachara arambha hone ke tina mahine bada duharaya jana chahie, aura yadi avashyaka ho to tina mahine bada eka bara phira|
upavasa ke daurana, aura avashyakata paड़ne para bada mem bhi, hara roja ushma jala ki enima se antom ki saphai ki jani chahie|
marija ko bina chokara ki roti, shakkara, malai, parishodhita anajom, chavala, ubale hue alu, pudinga, tatha kachauड़i, teja chaya athava kaॉphi, surasara vale peya padarthom, masalom, acharom, saॉsom ya anya tathakathita pachana-sahayaka tatvom se paraheja karana chahie|
aise adhikadhika pramana samane a rahe haim jo yaha darshate haim ki kai mamalom mem samuchita poshaka upachara se motiyabinda ko vastuta: thika kiya gaya hai|
tathapi, aise upachara ke lie chha: mahine se tina varsha taka ka samaya laga sakata hai|
amerika ki eka sukhyata poshana visheshajna, ऍdila devisa, ne dhyana dilaya hai ki yadi janavarom ko triptophana ke atmikarana ke lie avashyaka paintothinika esida evam amino esida tatha triptophana aura vitamina bi-6 se vanchita kara diya jae to unhem motiyabinda ho jata hai|
ve batati haim ki motiyabinda ke marija ke ahara mem bi2, bi6 aura satha hi pure bi kaॉmpleksa, paintothinika esida, vitamina si, di, i evam anya poshaka tatva adhika hone chahie|
pathyahara upachara ke satha-satha, marija ko ankhom ko tanavamukta aura majabuta banane ki vibhinna paddhatiyam apanani chahie |
inake antargata ankhom ko halke se upara-niche, eka ora se dusari ora tatha eka golai mem, ghaड़i ki sui ki disha mem aura phira ghaड़i ki sui ki ulti disha mem ghumana; gardana ko vrittakara evam arddha-vrittakara rupa se ghumana tatha kandhom ko ghaड़i ki sui ki disha aura phira viparita disha mem phurti se ghumana shamila hai|
hatheliyom se dabana tanava hatane aura ankhom tatha unake asa-pasa ke utakom ko arama pahunchane mem bahuta hi adhika labhadayaka hai|
ankhom ko hatheliyom se dhankie, dahini hatheli dahini ankha para aura baim hatheli baim ankha para|
khuda ankhom para unaka davaba mata dalie|
phira apani kohani apane ghutanom para girane dijie, jo apasa mem eka-dusare se najadika hom|
kalima ki kalpana karane ki koshisha kijie, jo kali aura adhika kali hoti ja rahi ho|
taji hava aura ghara se bahara kie jane vale halke vyayama, jaise tahalana, isa upachara ke anya anivarya pahalu haim|
samanya sardi, jise tivra jukama se bhi jana jata hai, vishanu ke sankramana ke karana upari shvasa-nalika ki eka sujana hai |
yaha anya sabhi rogom ki tulana mem kahim jyada hoti hai|
ausata rupa se, eka vyakti sala mem tina bara isa roga ka shikara hota hai| 
sardi samanyata: tina se dasa dinom taka rahati hai|
marija pahale tina dina ya adhika dayaniya anubhava karata hai|
sardi ke prathama lakshana haim gale mem kharasha ka anubhava tatha nasika marga ka sankuchana|
halanki ama taura para yaha bimari naka aura gale mem shuru hoti hai, parantu yaha sharira ke sabhi angom ko prabhavita karati hai|
isake samanya lakshana haim naka ka bahana, chhinkana, tapamana mem vriddhi, siradarda, gale mem kharasha, kampakapi, sharira mem darda aura takalipha tatha bhukha ki kami|
nasarandhrom ke pasa ki tvacha suja sakati hai|
sadharana sardi vishanu ke samparka mem ane ke parinamasvarupa hoti hai|
parantu, isaki tivrata vyakti ke svasthya evam paryavaraniya karakom para nirbhara karegi|
ninna jaiva shakti, naka aura gale mem pratyurjata sambandhi gaड़baड़iyam, sharira mem kampakampi, ninda na ana, avasada, thakana tatha tapamana mem akasmika parivartana, dhula tatha chiड़chiड़apana utpanna karane vali anya chijom ka antah shvasana jaise karaka sardi ki bimari ko baढ़ane mem yogadana dene vale pramukha tatva haim|
tathapi, sardi ka vastavika karana hai khane-pine ki doshapurna adatom dvara sharira mem utpanna vishakta sthiti , jaiseki shvetasara, karbohaidreta, protina evam anya anla-janaka khadya bahuta jyada matra mem grahana karana|
ata:, sardi manaviya pranali ke andara se vishakta apashishta ko bahara nikalane ka prakriti ka saralatama marga hai|
sardi ki avadhi sharira ke andara sangrahita visha-tatva ki matra aura unhem bahara nikalane ki tvaritata para nirbhara karegi | 
sardi ka upachara ऍspirina, kodina, adi jaisi paramparika shamanakari aushadhiyom se karana bhavishya mem aura bhi adhika gambhira prakriti ki samasya ka marga prashasta karana hoga kyonki aisa upachara usa samaya jari nirmulanakari prakriya para sahasa purnavirama laga deta hai aura vishakta padartha ko dhakiyakara puna: utakom mem pahuncha deta hai|
isake atirikta, aushadhiyom ka asara sardi ki miyada para nahim paड़ta|
bilkula sahi kaha gaya hai ki sardi dava le kara eka haphte mem thika ki ja sakati hai, anyatha yaha sata dinom mem kama ho jaegi|
sardi ka ekamatra vastavika upachara hai sahi ahara|
upachara arambha karane ka sarvottama tarika hai marija ko do dinom ka upavasa karane ke lie chhoड़ dena| 
isa avadhi mem nimbu ke rasa aura shahada mile hue ushma jala ya phalom ke rasa aura sushma jala ke siva aura kuchha bhi nahim liya jana chahie|
rakta ki anliyata ko udasina karane ke lie phalom ke rasa ka tarala ahara adhika matra mem jaruri hai tatha gurde ki saphai ke lie garma peya avashyaka haim |
khasa taura para ananasa ka rasa bahuta hi labhakari hai|
isa avadhi mem antom ki saphai ke lie ushma jala ki enima ka hara roja prayoga kiya jana chahie|
isa alpakalika rasa-upavasa ke bada tina dinom taka vishuddha rupa se taje phalom ka ahara lena chahie|
isa pathyahara mem, marija ko pratidina seba, nashapati, angura, santare, ananasa, satalu, tarabuje jaise taja़e aura rasile ya anya kisi rasile mausami phala ka dina mem tina bara ahara lena chahie|
kele, sukhe ya sije hue ya tinabanda phala nahim lene chahie| 
ahara mem anya koi bhi khadya-vastu shamila nahim ki jani chahie kyonki isase upachara ki sari upayogita samapta ho jaegi|
vishishta phalahara ke bada, marija kramika rupa se tina adharabhuta khadya shreniyom, yatha bija, gariyom, tatha anajom, sabjiyom aura phalom para utara sakata hai|
salaha di jati hai ki kuchha dinom ke lie mansa, machhali, ande, panira tatha shvetasara-yukta bhojya padarthom se paraheja kiya jae|
sharira ke lie vanchhita sabhi vitaminom aura khanija tatvom ki apurti karane vala ahara lekara marija ko apani sampurna pranali majabuta bana leni chahie| 
tathapi, ina poshaka tatvom ki suchi mem sarvopari hai vitamina si|
yaha vyakti ko sankramana se bachata hai tatha eka hanirahita pratijaiviki ka kama karata hai|
yaha nimbu prajati ke phalom, hari pattedara sabjiyom, ankurita bangala chane aura hare chane mem paya jata hai|
eka nobela puraskara vijeta vaijnanika, daॉ. lainasa paॉlinga, ke anusara, isa vitamina ka ishtatama dainika matra mem prayoga samanya sardi se bachaega aura yadi sardi pahale hi ho chuki hai, to isa vitamina ki baड़i khurakem lakshanom se rahata dengi aura isaki miyada kama karengi|
vo anumana lagate haim ki pratidina eka se do grama athava 1000 migra. se lekara 2000 migra. taka ki matra isa vitamina ki lagabhaga ishtatama matra hai|
unaka sujava hai ki sardi-jukama ka prathama lakshana ubharate hi vitamina si ke eka ya do 500 migra. vale taibaleta nigale lie jaem aura pratyeka ghante eka atirikta taibaleta lete hue upachara jari rakha jae|
kai adhyayanom mem yaha paya gaya hai ki pani mem patala karake aura dina mem kai bara piya gaya pyaja ke rasa ke satha lahasuna ka tela sadharana sardi ke upachara mem atyanta hi prabhavi hoga|
bhapa ka antahshvasana naka ke utakom ke sankuchana se rahata pane mem madada karega|
namaka milakara garma jala se garara karana gale ki kharasha ke lie labhakari hai|
sardi mem prayukta 'chesta paiksa' dinabhara mem do-tina bara lagae jane chahie kyonki ve phephaड़om ki jakaड़na dura karenge aura jami hui shleshma ko hatane mem madada karenge|
samanya sardi ke upachara mem anya upayogi upaya haim halki dhupa senkana, taji hava aura gahara shvasana, teja kadamom se tahalana, gahari ninda, mausama ki avashyakataom ke hisaba se apane kapaड़om aura adatom ka samanjasya bithana, taki mausama ke badalava ko nishprabhavita kara diya jae|
atyanta gambhira mamalom mem, ghava rakta-srava ka karana bana jata hai aura marija mavada evam shleshma ke satha khuni mala-tyaga karane lagata hai|
yaha roga baड़i anta ki bhittiyom se sati najuka jilliyom mem lambe samaya se upasthita kshobha ke karana hota hai|
yaha bahuta chhote bachchom se lekara baड़e logom taka sabhi ayu-vargom ko prabhavita karata hai|
samanyata:, baड़i anta ka prakarya yaha hai ki vaha apashishta padartha ko tabataka jama karati rahati hai jabataka adhikansha tarala padartha na hata die jaem taki nirgata kie jane yogya ashoshya khadya padarthom se bana sugathita narma mala bana sake |
jo loga antra-kshubdhata ke shikara hote haim unamem aniyamita aura doshapurna sankuchana pae jate haim jo baim ora khasa taura para drishtigochara hote haim|
purane ghava vale vrihadantrashotha ama taura para antaड़iyom ke nichale hisse mem shuru hota hai aura upara ki ora phailane lagata hai| 
pahala lakshana hai mala-tyaga ki pahale se jyada tvaritata aura usake bada peta mem maroड़ ke satha darda aura mala mem rakta-sanita shleshma ka ana|
jaise-jaise roga upara ki ora phailata hai, mala panila aura jyada bara ata hai, tatha malashaya mem khinchava ke lakshana ate haim|
antom se rakta aura drava ki isa kshati ke parinamasvarupa kamajori, bukhara, chakkara ana, ulti, bhukha ki kami aura raktalpata ke lakshana ubharate haim|
marija ko bada mem peta phulane jaisa bhi mahasusa ho sakata hai kyonki vata ka avashoshana ya nishkramana samanya dhanga se nahim hota hai|
kuchha marija eka ke bada eka kabja aura dasta ke shikara ho jate haim|
kuchha anya marija eka satha kai salom taka dirghakalika pechisha ke shikara ho sakate haim|
marija ama taura para kuposhita ho jata hai aura usake vajana mem gambhira rupa se kami a sakati hai|
aksara vaha anidra roga se bhi grasita ho ja sakata hai|
apane gambhira rupa mem ghava vala vrihadantrashotha poshana sambandhi samasyaom ko bhi janma de sakata hai|
sujana ki dashaom ke karana bhojana ka thika se atmasata na hona abhava-janita rogom ko utpanna kara sakata hai|
isaka parinama kramasha: snayavika chiड़chiड़apana, thakana evam avasada ho sakata hai |
atyanta gambhira mamalom mem marija atmahatya ki pravritti ka vikasa bhi kara sakata hai|
vrihadantrashotha ka mukhya karana hai dirgha kalika kabja evam dastavara davaom ka sevana|
kabja kathora maliya padarthom ke jama ho jane ka karana banata hai jo kabhi bhi thika prakara se bahara nahim nikala pate|
dastavara davaom ka sevana kshubdhata baढ़ata hai| 
aksara, vrihadantrashotha khasa taura para kupachita rukhaड़e anajom evam karbohaidreta ke karana hota hai jo antom mem kshobha utpanna karate haim| 
yaha katipaya bhojya padarthom, khasa taura para dudha, gehum aura andom ke prati pratyurja-janita samvedanashilata ke karana bhi ho sakata hai|
aksara, pratijaiviki davaem lene se antom mem jivanuom ka pushpana bigaड़ sakata hai tatha ye samuchita pachana-kriya mem hastakshepa kara sakati haim|
atishaya tanava bhi ghava-janita vrihadantrashotha utpanna kara sakata hai|
kisi bhi prakara ke atishaya tanava ke daurana, edrinala haॉrmona ka srava sharira ke protina ki itani kshati karata hai ki kai bara to antom ke kinare ki bhitti-rekhaem manom sachamucha chata kara di jati haim|
aisa tanava sharira ko paintothinika anla se bhi rikta kara deta hai|
janavarom para kie gae prayoga yaha darsha chuke haim ki jaba unhem paintothinika anla se rahita ahara diya jata hai to ve ghava-janita vrihadantrashotha se grasta ho sakate haim|
vrihadantrashotha ke upachara mem ahara eka mahatvapurna bhumika nibhata hai|
salaha di jati hai ki vrihadantrashotha ke jyadatara mamalom mem pancha- dinom ke lie kevala rasom ke satha upavasa rakha jae|
rasom mem thoड़a ubala hua pani milaya ja sakata hai|
papite ka rasa aura kachchi pattagobhi athava gajara ka rasa vishesha rupa se labhadayaka hai|
nimbu ke rasa se paraheja karana chahie|
ushma jala ki enima se antom ki roja saphai ki jani chahie|
isa rasa-upavasa ke bada, marija ko kramasha: mulayama rupa se paki ya bhapa para sijai hui sabjiyom, chavala, daliya (gehum ke rukhaड़e danom vali) tatha achchhi taraha pake hue phala jaise kela aura papita, dahi aura ghara mem bane kutira panira ke samavesha vala halka kintu adhika bara liya jane vala ahara apanana chahie|
10 dinom ke bada dhire-dhire isa ahara mem ankurita bija evam anaja, purnakara danom se bani roti tatha kachchi sabjiyam shamila ki ja sakati haim|
nridula nariyala pani atyanta labhadayaka hai kyonki yaha baड़i anta ki mulayama shlaishmika jilli ko arama pahunchata hai|
lauha evam phaॉsphorasa ke apane prachura sankendrana ke karana pakae hue seva bhi ghava ki dashaom mem arogya mem sahayaka hote haim|
sabhi khadya padathorrram ko dhime-dhime aura khuba chaba kara khana chahie|
aise khadya padatharrra jinrrahem ahara se varjita karana chahie, ye haim- chini, maide ki roti aura maide ke anrraya utrrapadana, bahuta adhika masaledara bhojana, adhika namaka vale khadya padatharrra, kada़ka chaya, kaॉphi aura madaka peya tatha alyuminiyama ke batarrrana mem pakaya bhojana| 
marija ke malotsarga ka samaya pratidina eka hi hona chahie, aura isake prayasa mem use 10 se 15 minata vyatita karane chahie| 
shaucha ke samaya pura ज़ora lagane se bachana chahie| 
subaha uthate hi sarvaprathama do grralasa pani pine se apako malotsarga mem asani hogi | 
yadi malotsarga na ho raha ho , taba enima ka prayoga kara sakate haim, | 
haphrrate mem do bara chhachha rupi enima ka prayoga karana upashamaka‎ va malashaya mem jivanvika vanasrrapati ko punah srrathapita karane mem bhi sahayaka hota hai| 
punarrra vishrama aura atyadhika tarala padarthom ka sevana atyavashyaka hai| 
marija ko tanava ke sabhi karanom ko dura kara dena chahie, aura akshamata se samajauta kara dhiraja ke satha apane kashrrata se samana karana chahie| 
netra-shotha netrashrraleshmala, ankhom ke agrabhaga ko achchhadita karane vali paradasharrraka jilrrali para hone vali jalana se sambandha rakhata hai| 
ise ama bhasha mem ankhe ana bhi kahate haim aura yaha ankhom ki vyadhiyom mem sabase ama vyadhi hai| 
pratrrayaksha sampakarrra se yaha eka vrrayakrrita se dusare vrrayakrrita mem phailata hai| 
atrrayadhika bhida़-bhada़, ganda parivesha aura asrravasrrathrrayakara rahana-sahana ke halatom ke karana yaha roga mahamari ka rupa le sakata hai| 
ankha ki putali aura palakom ka bhitari bhaga lala ho jata hai| 
sabase pahale ankhe lala va shushrraka hokara jalane lagati haim| 
bada mem, ho sakata hai ki ankhom se pani bahane lage| 
atrrayadhika gambhira sthita mem, mavada ka nimarrrana hota hai| 
sote samaya yaha padatharrra sukha jata hai, aura jisake karana donom palakem apasa mem chipaka jati hai| 
chikitrrasakiya vijnana ka manana hai, ki netra-shotha, jivanu sankramana, vishanuom ya ankhom ke tanava ke karana hota hai| 
lambe samaya taka kritrima prakasha mem kama karana aura eka hi taraha ke kama mem ankhom ka atrrayadhi upayoga karana, nisandehatmaka rupa se isa bimari ka karana banata hai| 
ahara mem gada़bada़i ya jivana jine ke galata tarikom ke karana, utrrapanrrana hone vali samanrraya visharakrratata ke parinamasrravarupa, jukama ho jata hai , jo isaka vasrratavika karana ho sakata hai| 
marija samanrrayatah jukama ya jukama sambandhi samanrraya anya lakshanom se pida़ita hota hai| 
kisi lepa ya marahama ke dvara, netra-shotha ka upachara nahim kiya ja sakata | 
upachara prabhavashila ho, isaliye mulabhuta upachara jaruri hai| 
apake sharira ke bhitara se sampunarrra saphai aura ahara ke prakritika niyamom ka palana tatha samanrraya jivana jina, netra-shotha ko thika karane mem sahayaka ho sakate haim| 
upachara arambha karane ka sabase uttama tarika hai, lagabhaga sata dinom taka, kevala taje phalom ka ahara lena| 
ahara mem mausami taje aura rasadara phala jaise- seva, santara, nashapati, angura, ananrranasa aura anya chhote angura phala shamila kara sakate haim| 
jaham taka ho sake, kele ka sevana nahim karana chahie| 
isa ahara mem khane ki kisi bhi anrraya vastu ko shamila nahim karani chahie| 
aise loga jinrrahem atrrayadhika kashrrata hai, unrrahem tina ya chara dinom ke lie rasa para adharita upavasa ko apanana chahie| 
tarika yaha hai ki subaha 8 baje se shama 8 baje ke bicha prati do ghantem bada, santare ke rasa ka sevana karem, yadi ichchha ho, to gamarrra pani satha le sakate haim | 
yadi santare ka rasa pasanda na ho to isake badale mem gajara ka rasa le sakate haim| 
upavasa ke daurana, pratidina gamarrra pani ka enima lena chahie| 
kuchha dinom taka phalom ke rasa ka sevana karane ke bada, agale sata dinom ke lie sabhi phalom ka sevana kiya ja sakata hai| 
usake bada, motiyabinda para adharita adhyaya 12 mem ulrralekhita yojana ke tahada, marija samanrraya ahara grahana kara sakata hai| 
ankhom ke arama va shakti ke lie ulrralekhita ankhom ki peshi ka vyayama bhi, netra shotha ke upachara mem labhadayaka siddha hoga hai| 
marija ko atyadhika shnetasara va shakrrakara yukrrata khadya padatharrra jaise maide ki roti, parishrrakrita anaja, alu, puninga, paisa, pesrratri, shakrrakara, jaima aura mishrratannom ke sevana se paraheja karana chahie, jo ki samanrraya jukama ke satha-satha netra shotha ka karana hote haim| 
use atrrayadhika matra mem mansa aura anrraya protina tatha vasayukrrata ahara, kada़ka chaya va kaphi, atrrayadhika namaka, masalem aura parishrrakrita chataniyom ke sevana se bhi paraheja karana chahie| 
kachchi sabrrajiyom ka rasa, visheshatah gajara aura palaka ka rasa, netra shotha ke upachara mem labhadayaka paya gaya hai| 
ina donom sabrrajiyom ka mishrita rasa, atrrayadhika labhadayaka mana jata hai| 
isa mishrana mem chhah aunrrasa palaka ke rasa mem, dasa aunrrasa gajara ka rasa milana chahie| 
netra shotha ke upachara mem vitamina e aura vitamina bi 2 bhi labhadayaka paye gaye haim| 
marija ko ina do vitaminom ki prachurata vale khadya padathorrram ka sevana atrrayadhika matra mem karana chahie| 
vitamina e ke mukhrraya srota haim- dudha, dahi, makrrakhana, gajara, kaddu, hari pattedara sabrrajiyam, tamatara, ama aura papita| 
vitamina bi2 ki prachurata vale khadya padatharrra haim- hari pattedara sabrrajiyam, dudha, badama, nimbu ke vansha ke phala, kele aura tamatara| 
ankhom ke sthaniya upachara ke rupa mem ankhom para thandi sikai karake atikriya sthaniya rakrrata pravaha ko bhaga kara tatkala arama paya ja sakata hai| 
ise thamड़e pani se tara-batara karem| 
atirikta pani ko nichoda़ dem aura arama se taulie ko donom ankhom para rakhem| 
prati dina vrrayayama mem shvasana vrrayayama aura taji hava atrrayadhika labhadayaka hoti hai| 
ankhom ki dekhabhala savadhani puvarrraka kari jani chahie aura kritrima prakasha mem atrrayadhika padha़ne evam sukshma karya karane se bhi bachana chahie| 
kabrraja, pachana tantra ki eka samanrraya vyadhi hai| 
isa sthita mem malotsarga niyamita nahim hota hai, ya malashaya puri taraha se khali nahim hota hai| 
kabrraja kai anrraya bimariyom ka pramukha karana hai, krrayonki aisi sthiti visharakrrata padatharrrom ko paida karati hai aura phira ye rakrrata pravaha mem sanrramilita hokara, sharira ke anrraya bhagom taka pahuncha jata hai| 
isake parinamasrravarupa pramukha anga kamajora ho jate haim, aura sampunarrra sharira ki pratirodhaka kshamata ghata jati hai| 
unrraduka shotha, sandhivata, gathiya, uchrracha rakrratachapa, motiyabinda aura kainsara, kuchha aisi bimariyam haim, jinamem dirdhakalika kabrraja eka mahatrravapunarrra purvavritta karaka hota hai| 
samanrraya srravasrrathrraya ke lie kitane bara shaucha avashrrayaka hai,yaha eka vrrayakita se dusare vrrayakrrita mem bhinrrana hota hai| 
jrrayadatara loga dina mem eka bara hi shaucha jate haim, kuchha dina mem do bara jate haim, jabaki kuchha aura do dina mem eka bara jate haim| 
phira bhi, arama aura achrrachhe srravasrrathrraya ke lie, kama se kama dina mem eka bara malashaya ko puri taraha sapha karana avashyaka hai, aura ise samanrraya bhi mana jata hai| 
kabrraja ke sabase samanrraya lakshana haim- mala ke sakhrrata ho jane ke karana aprayikata, aniyamitata ya malatyaga mem kathinai hona| 
anrraya lakshanom mem haim- gandi jibha, badabudara shvasa , bhukha na lagana, siradadarrra, chakrrakara, ankhom ke niche kale ghere, udasi, michali, chehare para muhanse, munha mem chhale, udara ka lagatara bhari bane rahana, kabrraja ke satha dasrrata ki shikayata bhi hona, nasom mem ganthe va sujana, kati kshetra mem dadarrra, anrralata, sine mem jalana aura anidra| 
dirghakalina kabrraja ke pramukha karana haim-anuchita ahara aura galata jivana shaili | 
apani prakritika avasrratha mem sabhi khadya padatharrrom mem achrrachhi 'rukshansha' ki matra hoti hai, jo khadya padatharrra ke prakritika santulana ko banae rakhane ke lie atyavashyaka, aura jo kramakunchana mem bhi sahayaka hota hai-yaha eka aisi prakritika prakriya hai, jisake dvara ahara,poshana nala mem jata hai, 
aja hama jo khadya padatharrra khate haim usamem prakritika kanom ya 'rukshansha' ki kami hoti hai, aura isake parinama srravarupa dirghakalika kabrraja ki shikayata ho jati hai| 
parishrrakrita tatha vitamina va khanijom ki kami vale ahara ka sevana, apayarrrapta matra mem pani pina, adhika matra mem mansa ka sevana, kada़ka chaya va kaॉphi ka adhika sevana karana, apayarrrapata matra mem chabana, adhika khana aura ahara ka galata chunava karana, khane va pine ki aniyamita adatem, sabhi milakara kabja ka karana banate haim| 
anrraya karanom mem shamila haim- malotsarga ki galata aura aniyamita adatem, barambara rechakom ka prayoga, adhika baithane ke karana udara ki peshiyom mem kamajori, sharirika gatividhiyom ki kami evam bhavanatrramaka tanava va pareshani| 
bimariyam jaise-abarrruda ya sujana, akriyashila yakrita, vrihadanrratra shotha, antom ki sanstabhi avasrratha, atisakriyata, malashaya aura vrihadanrratra ki bimari, gande danta, gabharrrashaya mem bimari, madhumeha, anrraya rogom ke upachara ke lie upayoga mem lai davaiyam, ridha ke nichale hisrrase ki asamanrraya sthita aura purastha granthi ka badha jana bhi, dirghakalika kabrraja ka karana ho sakata hai| 
vriddha marija peshiyom ke shithila pada़ jane para, paushanika avashrrayakataom aura ghatati hui gatividhiom ke karana apayarrrapta ahara ke sevana se kabrraja ke shikara ho sakate haim| 
rechaka aura virechaka kevala asrrathayi arama pradana karate haim| 
ye udara aura antom ko anavashrrayaka rupa se atisamvedanashila bana dete haim, aura malashaya ko kamajora karate haim| 
virechaka ke durupayoga se, rakrrata mem poteshiyama ki kami aura srravasrrathrraya mem digharrrakalika kharabi a sakati hai| 
kabrraja ke upachara ka prakritika tarika hai- sabhi kritrima davaom ko trrayaga kara, antom ko punah prashikshita karana| 
niyamita samaya para ahara, malatyaga aura nidra ka anupalana, santulita ahara, payarrrapta vrrayayama, aura mansa-peshiyom ko majabuta karane ke satha samanrraya srravasrrathrraya ka uchrracha srratara kabrraja ke upachara ke lie avashrrayaka hai| 
kabrraja ke upachara ke lie sabase pramukha tatva prakritika aura sadharana ahara hai| 
sabhi bhavanatrramaka vikarom mem udasi sabase samanrraya vikara hai| 
yaha thoड़i-si nirasha ki bhavana se lekara atrrayanta dukha aura udasi ki sthita taka ho sakati hai| 
yaha kai taraha ke sharirika aura manovaijnanika lakshanom ke karana utrrapanrrana hota hai, jo eka satha milakara sanlakshana banate haim|
udasi ke daura se gujarana kisi bhi vrrayakrrita ke lie sabase apriya anubhava hota hai| 
isase nipatana sharirika bimariyom se nipatane se jyada kathina hota hai| 
adhunika jivana ki dina pratidina ki badha़ti hui kathinaiyam aura usake parinami sankatom ke satha-satha manasika tanava aura vikriti, akrrasara isa gada़bada़i ko utrrapanrrana karati hai| 
udasi ke daura mem atrramahatrraya ka khatara pramukha hota hai| 
nidana ki drishti se udasi ka upachara hamesha asana nahim hota| 
udasi ke sabase asadharana lakshana hai- nukasana ki tivra bhavana, duruha udasi, shakrrita aura ruchi donom ka kshaya hona| 
akrrasara marija thaka hua sa mahasusa karata hai aura apane asa-pasa ki chiज़om mem usaki ruchi ghatane lagati hai| 
akrrasara, marija subaha 4 se 5 ke bicha avasada-grasta jaga jata hai aura punah sone mem asamatharrra hota hai| 
nidra ke anrraya ashanta svarupa hai-gahari ninda ane mem kathinai, duhsrravapana aura madhya-ratri ke bada bara-bara ninda khulana| 
marija akrrasara aparadha, kashrratakari ehasasa aura atmalinata se pida़ita rahata hai| 
kuchha logom mem bhojana ke prati ruchi samaprrata ho jati hai aura unaka vajana nirantara ghatane lagata hai, jabaki kuchha anrraya loga barambara khane lagate haim aura isake parinamasrravarupa unake vajana mem vriddhi hone lagati hai| 
gambhira udasi ke mamalom mem, sharira ka ninrrana tapamana, ninrrana rakrratachapa, gamarrra tamatamahata aura kampakampi jaise aneka lakshana ubharane lagate haim| 
udasa vrrayakrritayom ke utrrapida़ita mastishka se, bahya abhivrrayakrrita ke rupa mem, sahayata ke lie chitkara nikalati hai| 
gambhira rupa se udasa vrrayakrrita apane apa ko ayogya mahasusa karata hai aura antatah apane savarrranasha aura vatarrramana nairashrraya ke lie apane apa ko utrrataradayi manane lagata hai| 
adhivrikrraka granthi ke kayarrra mem kami ana bhi manasika udasi ka eka mukhrraya karana hota hai| 
aniyamita ahara ki adatem pachana mem samasya ka karana banati hai, aura isase charbi ka atmasatkarana bhi hone lagata hai| 
anaja,chini, kaॉphi,chaya,chakaleta adi ke rupa mem kaborrrahaidreta ka adhika upayoga aura ahara mem sabrrajiyom aura phalom ka apekshakrita kama matra mem prayoga karane se badahajami ho sakati hai| 
apachana ke karana pachana nali mem gaisa ka nimarrrana hota hai, jo hridaya aura phephada़om ke madhyapata para dababa ka karana banati hai| 
isake parinamasrravarupa, utakom mem akrrasijana ki aputirrra ghata jati hai, jisase kabarrranadaiakrrasaida ka srratara badha़ jata hai, aura yaha samanrraya udasi ka karana banata hai| 
davaom ka adhika aura andhadhundha prayoga bhi sharira mem vitaminom aura khanijom ka galata atmasatkarana karata hai, aura yaha antatah udasi ka karana banata hai| 
aispirina ke upayoga se vitamina si ki kami aura anrralarodhiyom ke upayoga se kailashiyama aura vitamina bi ki kami ho sakati hai| 
parishrrakrita ya sansadhita khadya padathorrram, tale hue khadya padatharrrom aura vasa ke atrrayadhika prayoga ke parinamasrravarupa madhumeha, ninrrana rakrrata shakarrrara (haipoglisimiya) aura yakrita mem kamajori utrrapanrrana ho jati hai, aura vaha hamem udasi ki ora le ja sakati hai| 
adhunika chikitrrasakiya pranali udasi ka upachara prati udasi davaom ke prayoga se karati hai, jisase asrrathayi arama to milata hai, kintu isake apane hanikaraka prabhava bhi hote haim, aura yaha isake karanom ko nashrrata nahim karati ya isaki punaravritti ko nahim rokati hai| 
hanikaraka prabhavom mem shamila haim- yakrita ki bhari kshati, atisamvedanashilata, anidra, matibhrama, aspashta avasrratha, ainthana aura rakrratachapa mem giravata ana|
prakritika upachara mem, aise ahara para vishesha dhyana diya jata hai, jo vyakti ke manasika svasthya para prabhava dalata hai| 

kailirphoniya vishrvavidyalaya ke sahayogi chikitsakiya prophesara, daॉ. prisila mastishka ke rasadravyom ke nirmana ke lie, paushanika chikitsa ka nuskha batalate haim, jaise- sirotonina aura norepainephrina, jo kisi ki manodasha ko prabhavita karata hai aura aksara udasa logom mem isaki kami hoti hai| 
vaha vitamina bi ki prachurata vale khadya padarthom, jaise-sabuta anaja, hari sabjiyam, ande aura machhali ke sevana ki salaha deti hai| 
udasi se pidita vyakti ke ahara mem ninnalikhita chijem nahim honi chahie jaise- chaya, kaॉphi, sharaba, chakaleta, kola, maide ke bane sabhi padartha, chini, kritrima rangom se yukta ahara, rasayana yukta ahara, sapheda chavala aura teja masale adi| 
ahara kevala tina bara ke bhojana taka hi simita hona chahie| 
subaha ke nashte mem, dudha ke satha phala va thoda़e se giridara phala aura bijom ka sevana kiya ja sakata hai| 
dopahara ke bhojana mem, ubali hui sabjiyom, gehum ke ate se bani chapatiyom aura eka glasa chhachha ka sevana kiya ja sakata hai| 
ratri ke bhojana mem, hari sabjiyom ka salada aura sabhi upalabdha ankurita anaja jaise- garari ke bija, munga, ghara ka panira ya eka glasa chhachha lena uttama hoga| 
udasa mijaja, karya karane se thika ho sakata hai| 
aise loga jo udasa haim, ve kisi karya mem lina hokara apani chinta ko bhula sakate haim| 
udasi ke upachara mem, vyayama bhi mahatvapurna bhumika ada karata hai| 
yaha na kevala sharira ko sharirika aura manasika rupa se tandarusta karata hai, balki manoranjana aura arama bhi pradana karata hai| 
yaha upalabdhi ki bhavana ka sanchara karata hai, aura jisase asahaya hone ki bhavana kama hoti hai| 
marija ko arama aura dhyana karane ki vaijnanika kala, jo udasi ke ilaja mem chirakalika rupa se sahayaka hoti hai, isako bhi avashyaka rupa se sikhana chahie| 
madhumeha madhupradata eka paushanika gada़bada़i hai, jo rakta mem draksha-sharkara ke asamanya unnata stara aura peshaba se atirikta draksha-sharkara ke utsarjana dvara lakshanita hoti hai| 
yaha insulina ki anshika athava sampurna kami ka parinama hai, jisase karbohaidreta chapayachaya ke satha-satha protina aura vasa ke chapayachaya mem asamanyata utpanna hoti hai| 
ati prachina samaya se hi chikitsa vijnana mem madhumeha ka ullekha hai| 
isaka prabhava atita ki apeksha vartamana mem adhika hai| 
bahutayata aura prachurata mem khadya padartha ki apurti ke karana duniya ke atyadhika adhunika shaharom mem jane para, yaha visheshataura para satya pratita hota hai| 
rakta mem badha़te hue draksha sharkara ke stara ke parikshanom mem sabase samanya hai khane se pahale aura khane ke do ghante bada rakta mem draksha sharkara ka stara ka parikshana| 
rakta draksha sharkara avyava ki samanya matra hai-pratyeka 100 mi.li. rakta mem 80 se 120 mi.gra., aura khane ke do ghante bada, yaha pratyeka 100 mi.li. mem 180 mi.gra. taka badha़ sakati hai| 
isa kasauti se thoda़i bhi jyada matra hone para, ise madhumeha stara kaha ja sakata hai| 
apake shishu se lekara vriddha avastha taka kisi bhi unra mem madhumeha ho sakata hai| 
jyadatara yaha madhyama ya vriddhavastha mem dekha jata hai| 
yaha anumana lagaya gaya hai ki madhumeha se pida़ita 80 se 85 pratishata loga 45 varsha ya usase adhika unra ke hote haim| 
madhumeha shabda grika bhasha ke shabda se liya gaya haim, jisaka ashaya haim, bahana; hokara gujarana, aura madhupradayata letina shabda shahada se liya gaya hai| 
isa prakara do lakshana, kramashah atyadhika mutra tyaga aura mutra mem draksha sharkara ka paya jana, isa bimari se parichita karate haim| 
samanya dashaom mem lagabhaga tina pinta mutra tyaga hota hai, kintu madhumeha ki dasha mem yaha atha se chalisa pinta taka ho sakata hai| 
peshaba pile ranga ki, anliya pratikriya aura mithi gandha vali hoti hai| 
isamem vidyamana shakkara ki matra, prati aunsa mem, do se lekara chalisa grama taka ho sakati hai, bahuta si sthitiyom mem yaha pratidina, 30 pinta peshaba mem jyada se jyada do liba taka ho jati hai| 
madhumeha rogi adhikatara samaya mem bhukha va pyasa ka anubhava karate haim, unaka vajana nahim badhata hai, jabaki ve aksara khate rahate haim, aura ve jaldi hi sharirika va manasika rupa se thaka bhi jate haim| 
vaha pila sa dikhata hai, aura araktata se pida़ita ho sakata hai, najuka angom ke charom ora atyadhika khujali hoti hai, atispandana aura samanya kamajori hoti hai| 
vaha nindralu sa mahasusa karata hai, aura usamem samanya vyakti ki apeksha kama bhavana kama hoti hai| 
na kevala shakkara aura parishkrita karbohaidreta, balki protina aura vasa ka bhi atyadhika sevana hanikaraka hai, jo sharkara mem parivartita ho jata hai, aura madhumeha ke rupa mem parinamita hota hai| 
adhika ahara ka sevana agnashaya para bhara dalata hai aura antatogattva isaki samanya gatividhi ko thapa kara deta hai| 
yaha anumanita kiya gaya hai, ki samanya motape vale vyakti mem madhumeha ki shikayata chara guna badha़ jati hai, aura atyadhika mote vyakti mem yaha shikayata 30 guna badha़ jati hai| 
shoka, chinta aura pareshani bhi upachaya para gahara prabhava dalati hai aura yaha peshaba mem sharkara ke prakata hone ka karana bana sakati hai| 
yaha bimari kuchha anya gambhira angika asantulanom, jaise kainsara, tapedika aura pramastishkiya bimariyom se sambandhita ho sakati hai| 
isa bimari ke vikasa mem anuvanshikata bhi eka mahattavapurna karaka hai| 
yaha purnatah sahi kaha gaya hai, ki anuvanshikata eka topa ke samana hai aura motapa usa topa ko chalane ka ya prerita karane ka kama karata hai| 
madhumeha ke upachara ki koi bhi saphala pranali, bimari ke vastavika karana ko khatma karane aura marija ke sampurna svasthya stara banane ki ora lakshita hoti hai| 
isa prakara ke upachara mem ahara chikitsa mukhya adhara hoti hai| 
madhumeha rogi ke lie prathamika ahariya dhyana dene yogya bata yaha hai, ki use punarrra rupa se dugdha-shakahara ko apanana chahie aura kama kailauri, kama-vasa, uchcha gunavatta vale anliya ahara, va prakritika khadya padathorrram ka sevana karana chahie| 
madhumeha rogi ke lie phala, giridara phala aura sabjiyam, gehum ke ate ki rotiyam aura dugdha utpada eka achchha ahara hai| 
pake hue stacharrra yukta khadya padartha ke sevana se paraheja karana chahie, kyonki pakane ki kriya mem starcha kanikaom ka sailulasa avarana tuta jata hai aura phalasvarupa pranali dvara starcha ka avashoshana asani se ho jata hai|
atyadhika avashoshana gurde ke dvara chhoda़ diya jata hai aura peshaba mem sharkara ke rupa mem prakata hota hai| 
lara aura chhoti antom ke pachana rasa sharira ki avashyakata ke liye jina parimanom ko sharkara mem parivartita karane ki avashyakata hoti hai, unako kachche starcha yukta khadya padarthom se niyantrita karata hai| 
kachche starcha yukta khadya padartha ka aprayogita aura apachita hissa tantra ke lie hanikaraka nahim hota hai, kyonki isamem asani se khamira nahim uthata hai| 
madhumeha rogi ko shakkara aura starcha ki matra vale taje phala aura sabjiyom ke sevana mem darana nahim chahie| 
shakkara (phala sharkara) vale taje phala, jinhem apane upachaya ke lie madhusudani ki avashyakata nahim hoti hai aura ye madhumeha rogi ke dvara asani se pachae ja sakate haim| 
vasa aura telom ka prayoga kama karana chahie, kyonki ye protina aura starcha ko sokhane ki apaki shakti ko kama karate haim| 
kachche khadya padartho ke sevana para vishesha dhyana dena chahie, kyonki ye prerita kara madhusudani ke utpadana ko badha़ate haim| 
protina ke lie, ghara para bana panira, dudha ke kai prakara aura giridara phala sarvottama hote haim| 
marija ko adhika ahara ka parityaga karana chahie, aura dina mem tina bara adhika ahara lene ke sthana para chara se pancha bara thoda़a-thoda़a ahara lena chahie| 
madhumeha rogi ko sakaratmaka parinamom ke lie, apane ahara ke daurana, tina se chara karelom ka rasa lena chahie| 
marija ko pachana tantra para viparita prabhava dalane ke karana chaya, kaॉphi aura koka ke sevana ka parityaga karana chahie| 
anya khadya padartha jinaka sevana nahim karana chahie, ye haim- maide ki bani sapheda roti, maide ke anya utpada, chini, dibba banda phala, mithaiyam, chakaleta, pestri, paisa, pakavana, parishkrita anaja aura sharaba yukta peya|
madhumeha ke upachara ka pramukha poshaka mainganija hai, jo prakritika madusudani ke utpadana mem pramukha bhumika nibhata hai| 
yaha nimbu vansha ke phalom, giridara phalom ke bahari bhaga, anajom aura khae jane vale paudhom ki hari pattiyom mem paya jata hai| 
anya vishesha yogyata vale poshaka tatva haim - jinka, bi-kampalaiksa aura bahu-asantripta vasiya anla| 
barambara ya pani jaise mala tyaga ko atisara kahate haim| 
ante samanya rupa se pratidina 10 litara se jyada dravya avashoshita karati haim, jo ahara se aura udara, yakrita va agnashaya ke sravana se milata hai| 
atisara ki dasha mem jala ya to shoshita nahim hota hai ya sharira ke angom se adhika matra mem sravita hota hai| 
phira yaha malashaya ki ora chala jata hai, jaham pani ko rakhane ki kshamata simita hoti hai| 
atisara ke kai aura vibhinna prakara ke karana hote haim| 
mukhya karana haim-adhika ahara lena ya galata khadya padarthom ka sevana karana, anta ki nali mem sada़ava hona, karbohaidreta ka purna pachana na hone ke karana kinvana hona, tantrakiya atisamvedanashilata aura rechakom ka adhika prayoga karana| 
anya karanom mem shamila haim- parajiviyom, roganuom, vishanuom, jivanuom ke dvara sankramana ya khadya padarntho, pani ya hava ke madhyama se sharira mem pravesha karane vale, kisi padartha ki elarji se ya samanya khadya padartha jaise- dudha, gehum, ande aura samudri khadya padarthom aura bhavanatmaka tanava ya yuvakom mem tanava aura bachchom mem dara | 
yaha pratijaiviki davaom ke sevana ka bhi parinama ho sakata hai| 
yaha antom mem aise rogajanaka jivanuom, jina para pratijaiviki upachara ho raha ho, isake satha-satha labhadayaka jivanuom ke nasha ke karana bhi hota hai| 
atisara chhoti ya bada़i antom ko prabhavita karane vali, angika bimari ka mukhya lakshana ho sakata hai, jaise-sangrani sanlakshana, asadhya bimari aura vranakari brihadantra shotha| 
yaha amashaya-antiya nali mem shalya kriya ke parinama se bhi ho sakata hai| 
atisara kabja ke satha bari-bari ho sakata hai| 
yaha antarghatta kathora mala ke karana shleshma jilli mem jalana ke parinasvarupa ho sakata hai| 
adhika samaya taka atisara ki shikayata, kuchha anya vishishta kathinaiyom ki ora agrasara kara sakati hai| 
inamem poshaka tatvom ke sharira mem shoshita hue bina hi sharira se bahara ho jane ke karana, vitamina e, di, i va ke aura anya poshaka tatvom ki kami ke karana samanya kamajori ka ana, sharira mem taralata ki kami ke karana nirjalikarana aura sharira se khanija ka prakshalana, aura kamajora sthiti shamila ho sakati hai| 
bahuta si kathinaiyom ke bicha, nirjalikarana eka gambhira samasya hota hai, vishesha rupa se jaba dasta ke satha ulati bhi ho rahi ho| 
nirjalikarana mem udara ke upara garma aura shushka tvacha, dhansi hui ankhe, sukha munha, adhika pyasa aura peshaba ki gati kama ho jati hai| 
ise samanya rupa se roka ja sakata hai, yadi ulti ke satha athava ulti ke bina hi atisara se pida़ita marija ko prachura matra mem tarala dravyom ka sevana karaya jae| 
marija ko subaha 6 baje se rata 10 baje ke bicha prati ghante mem lagabhaga 150 se 200 mili litara tarala chijom ka sevana karana chahie| 
atisara ki gambhira dasha mem- yaha sujava diya jata hai, ki amashaya-antiya nali ko purna arama dene ke uddeshya se, do dina ka purna upavasa kiya jae| 
isa samaya ke daurana taralata ki kami ko pura karane ke lie marija kevala garama pani ka sevana kara sakata hai| 
isa daurana pratidina 95 pheranahaita ke tapamana ke pani ke satha enima liya ja sakata hai| 
gambhira lakshanom ki samapti ke bada, santare ya anara jaise phalom ka rasa, chhachha ya nariyala pani liya ja sakata hai| 
jau ke pani ko dudha ki samana matra ke satha milakara, usamem shakkara milakara bhi liya ja sakata hai| 
madhyama sthita mem, achchhi taraha se pake hue chavala ya khichada़i (chavala aura munga dala, thoda़e se namaka ke satha pakayi hui) ko dahi aura pake hue kele ke satha khaya ja sakata hai| 
atisara ki dasha mem dudha aura dudha se bane peya, sabuta anaja ki rotiyam aura anaja, panira, kele ke alava sabhi phalom, giridara phala, mansa, vasa yukta supa, mithaiyam aura tamatara ke rasa ke alava sabhi sabjiyom ke sevana se paraheja karana chahie| 
sthita mem sudhara ane ke bada, papite ke rasa, nimbu ke rasa aura taje ananasa ke rasa, paki hui sabjiyam, chavala, dahi aura chhachha jaise dudha ke rupa ko shamila karate hue, dhire-dhire ahara ko badha़aya ja sakata hai| 
kachche khadya padarthom ka sevana, marija ke purnarupa se thika ho jane ke bada hi kiya ja sakata hai| 
atisara ki dasha mem kuchha prakritika upachara bhi prabhavashila pae jate haim aura ye haim-gajara ka rasa, kele, haldi ka paudara, samvardhita ya anliya dudha (dahi) aura lahasuna| 
ye hanikaraka antiya jivanuom ki vriddhi mem madada karate haim aura ulti hone se rokate haim| 
eka paunda gajara ko pancha aunsa pani mem narama hone taka pakaya ja sakata hai| 
gude ko chhanana chahie aura isamem ubale hue pani ko milakara, ise eka chauthai banana chahie| 
tina chauthai chhote chammacha namaka lekara mila sakate haim| 
pratyeka adhe ghante mem, marija ko supa ki thoni-thoni matra deni chahie| 
kele mem pekatina hota hai aura yaha labhadayaka jivanuom ki vriddhi ko protsahita karata hai| 
masale ki taraha prayoga kiya jane vala eka pila paudara, haldi labhadayaka siddha hota hai| 
anliya dudha jaise dahi ya chhachha, hanikaraka antiya vanaspati ko nashta karane mem madada karate haim, aura hitakara ya anukula vanaspati ko punahsthapita karate haim| 
samvardhita dudha ke anla roganuom aura jivanuom se bhi lada़te haim| 
lahasuna bhi eka prakritika upachara hai, jo atisara se lada़ta hai aura parajiviyom ko nashta karata hai|
yaha eka shaktishali, prabhavi tatha ahanikara pratijivanu hai tatha pachana mem sahayaka hota hai|
disapepsiya yunani mula ka eka shabda hai, jisaka artha apachana ya pachana mem kathinai hai|
bhojana ko lene se sambaddha koi bhi udarantriya lakshana ajirna kahalata hai|
aja yaha sarvadhika ama bimariyom mem se eka hai tatha ahara sambandhi galatiyom ke parinamasvarupa hota hai|
amashaya, jo sharira ka sarvadhika prayukta anga hai, eka nashapati ke akara ki thaili ke samana hota hai|
yaha pachana kshetra ka eka anga banati hai jo lagabhaga 28 phita lambi, chhallom mem kundalita eka nali hoti hai|
isamem kitana bhojana hai isake adhara para isake akara evam sthiti mem antara ata hai|
atibharita amashaya madhyapata ko uchita dhanga se kama karane se rokata hai|
udariya pida़a, khane ke bada ati-purnata ki anubhuti, amlashula, bhukha ki kami, michali ya ulati tatha udara-vayu ya gaisa ajirna ke sadharana lakshana haim|
ulati karane se prayah rahata milati hai|
anya lakshana haim mukha mem ganda svada, alepita jibha tatha badabudara shvasa |
kabhi-kabhi gale mem ghutana ki samvedana ka anubhava kiya jata hai|
apachana ke adhikansha mamalom mem, rogi koshthabaddhata se piड़ita hota hai jo pranali ki amlata ko badha़ati hai|
ajirna ke mukhya karana haim, adhika khana, galata bhojana sanyojanom ko khana, bahuta adhika teji se khana tatha bhojana ke uchita charvana tatha larasravana ki avahelana karana |
adhika khane ya barabara khane se pranali mem jvaragrasta sthiti utpanna hoti hai tatha pachaka angom para adhika bhara pada़ta hai|
yaha atyadhika amla utpanna karata hai tatha isake karana jathariya shleshma jilli sankulita ho jati hai|
adhika khana amashaya, yakrita, guradom tatha antom ke karya ko adhika kathina bana deta hai|
jaba yaha khadya-padartha sada़ta hai, to isake visha vapasa rakta mem avashoshita ho jate haim tatha isake parinamasvarupa, puri pranali vishakta ho jati hai|
aneka loga, jo apana khana tanava mem tatha shighrata se nigalate haim, isa bimari se piड़ita hote haim|
jaba khadya-padartha ko baड़e tukada़om mem nigala jata hai, to amashaya ko adhika kathinai se kama karana pada़ta hai tatha adhika haidroklorika anla sravita hota hai|
atyadhika shighrata se khana bhi kisi ke lie vayu nigalane ka karana banata hai|
ye buri adatem dahana, jalana ki anubhuti ya khatta svada utpanna karati hui, kuchha pachaka tarala padartha ko grasanali mem dhakelati haim, jo amashaya amla ka bhrama deta hai|
kuchha khadya-padartha, kha़asakara yadi unhem uchita tarike se na pakaya gaya ho, to ajirna paida karate haim|
kuchha loga, kuchha khadya-padarthom jaise phalliyom, pattagobhi, pyaja, kakada़i, muli tatha samudri-khadya-padarthom ke prati nakaratmaka dhanga se pratikriya karate haim|
tale hue khadya-padarthom ke alava garishtha evam masaledara khadya-padarthom se bahudha udariya baicheni tatha gaisa paida hoti hai ya vidyamana sthiti bigada़ jati hai|
atyadhika dhunrapana evam sharaba ke sevana se bhi amashaya mem gada़bada़i paida ho sakati hai|
baddhakoshthata udarantra kshetra ke madhyama se samanya pravaha mem badha utpanna kara sakati hai, jisake parinamasvarupa gaisa tatha udariya pida़a hoti hai|
khane ke satha-satha pine ki adata apachana ka eka anya karana hai, kyonki khane ke satha tarala padartha lene se pachaka rasa patale ho jate haim tatha unaki shakti kama ho jati hai|
anidra roga, irshya, bhaya tatha krodha jaise manobhava tatha vyayama ki kami isa bimari ke anya karanom mem se haim|
ajirna ka ekamatra prabhavi upachara pachana kshetra ki sampurna saphai tatha usake bada jivana-shaili mem parivartana ke satha-satha, ahara-sambandhi uchita adatom ko apanana hai|
isa upachara ko prarambha karane ka sarvottama tarika lagabhaga pancha dinom taka kevala phalom ke ahara ko angikara karana hai|
isa pathyapathya-niyama mem, rogi ko seba, nashapati, angura, santare, chhote chakotare, anannasa, ada़u tatha melansa jaise taje rasile phalom ka dina mem tina bara bhojana lena chahie|
tathapi, sukhe, damapukhta ya dibbabanda phalom ko nahim lena chahie|
phalahara ke satha koi anya khadya-padartha nahim lena chahie, anyatha isa upachara ka mahatva nashta ho jaega|
gambhira ajirna ke mamale mem, kevala phalahara ko apanane se pahale do ya tina dina taka upavasa rakhana upayukta hoga|
kevala phalahara ke bada, rogi lagabhaga dasa dinom taka halki-si pakai gai sabjiyam, rasile phala tatha chhachha yukta asani se pachane vale khadya-padarthom ka pratibandhita ahara le sakata hai|
kali mircha, sarasom, siraka ya achara jaise masalom evam bagharom, jo bhojana ko adhika svadishta banate haim tatha adhika khane ki ora pravritta karate haim, se avashya dura rahana chahie|
sharaba, tambaku, kada़ka chaya tatha kaॉphi, atyadhika masaledara mansom, atyadhika ubala hua dudha, dalom, alu, chavala, chija़, parishkrita, sansadhita, basi tatha dibbabanda khadya-padarthom, ina sabhi se dura rahana chahie|
dahi tatha panira ka mukta rupa se prayoga kiya ja sakata hai|
bhojana ke pahale khanija namaka ke churna ke satha lagabhaga eka grama adaraka chabana, jirna ajirna ka eka gharelu upachara hai|
udara-vayu tatha gaisa ke lie, lahasuna eka utkrishta upachara hai|
>yaha sada़iyala jiva-vishom ko nishprabhavita karata hai tatha asvasthyakara jivanu ko marata hai|
yaha gaisa ko bhi dura karata hai tatha pachana mem sahayaka hota hai|
ajirna ke mamale mem bi vitaminsa bhi labhadayaka hote haim|
shvetasarom ke pachana ke lie bi1 ya thaimina vishesha rupa se upayogi hai|
lekina yaha sunishchita kiya jana chahie ki, jo yadi kevala eka bi ghataka dene se ho sakata hai aise asantulana se bachane ke lie, kisi rupa mem sampurna bi mishrita samuha shamila kiya jaya|
bhojana ke adha ghanta purva evam eka ghante bada pani ya anya tarala padartha lie jane chahie|
‘ekjima’ shabda eka grika shabda se vyutpanna hua hai jisaka artha hai ‘phoda़a’|
yaha tvacha ke eka pradaha ki ora sanketa karata hai jisake parinamasvarupa phapholom evam phunsiyom ka nirmana hota hai|
yaha tvacha ki sabhi bimariyom mem sarvadhika ama evam sarvadhika kashtaprada hai|
pranali ki vishakta sthiti ke parinamasvarupa hua, khaja anivarya rupa se eka svabhavika roga hai|
isa roga mem rupom ke vistrita prakara samahita hai, jinamem se adhikansha, jirna kisma ke haim|
tvacha sharira ke utsarjana angom mem se eka hai jisake madhyama se vishakta dravyom ki baड़i matra asani se nikala jati haim|
apani vistrita sataha ke karana, yaha apashishta dravya ko prabhavi rupa se nikala sakata hai, jise sambhalane ki kshamata nishkasana ke anya angom-arthat gurdom, antom tatha phephada़om-mem nahim hai|
khaja,apane tivra prakara mem, tvacha ki lali evam sujana, sukshma phaphalom ke nirmana evam prachanda tapa dvara prakata hoti hai|
yadi phaphole phuta jate haim, to eka aparipakva, gili sataha taiyara hoti hai|
isase, rangahina srava risata hai, jo tvacha ki papaड़iyam banati haim jaba yaha sanchita hota hai|
samanyatah yaha roga rata mem badatara ho jata hai jaba bichhavana dvara sharira ka tapa bana rahata hai|
sabhi avasthaom mem tvacha mem khujali hoti hai|
gili avastha mem, yaha jivanu se sankramita ho sakati hai|
jalana ki pratikriya mem kharonchane se sthiti ka achchha hona prabhavita hota hai|
kharonchane se na kevala sankramana phailata hai balki isase sukhane evam papada़i utarane ki avastha bhi badha़ jati hai|
khaja hone mem pratyurjataem eka mahatvapurna bhumika nibhati haim|
kapada़e ya baratana dhone ke lie upayoga kie jane vale sabuna ya prakshalakom ke prati pratyurjata hone ke karana kuchha mahilaom ke hathom para khaja ho jati hai|
yaha kuchha logom ki ungaliyom ke asapasa jaba ve anguthi pahanate haim to dhatuom ke prati pratyurjata hone ke karana vikasita ho jati hai 
saina entoniyo mem teksasa vishvavidyalaya svasthya vijnana kendra ke anusandhanakartaom ne, apakshaya khaja vale bachchom ke hala hi mem kie gae eka adhyayana mem, yaha paya ki 75 pratishata aneka khadya padarthom ke prati pratyurja the|
samvedanashila logom ke lie ande, mungaphali, chaॉkaleta, gehum, gaya ka dudha, murgi tatha alu sarvadhika ama karana haim|
khaja ka vastavika karana sharira ke vibhinna chhidrom se vishom ko utsarjita karane mem manava pranali ki viphalata hai|
malashaya se mala ke madhyama se, mutrashaya se mutra ke madhyama se, phephada़om se shvasa ke madhyama se tatha tvacha ke romakupom se pasine ke madhyama se apashishta dravya utsarjita hote haim|
kabhi-kabhi tvacha ke romakupom ko atyadhika kama karana pada़ta hai kyonki anya chhidrom se apashishta dravya uchita dhanga se nishkasita nahim hota hai|
yadi romakupom ko unaka samanya karya nishpadita karane ka avasara nahim diya jata hai, to pasina vishakta dravya se bharapura hoga tatha yaha khaja, munhase, phoda़e tatha anya udbhedanom jaise tvacha rogom ki badha़otari kara dega|
khaja, tanava ke karana bhi ho sakati hai|
nirmita tanava ke lie tvacha ko nirmukti anga ke rupa mem mana jata hai|
varishtha prabandhaka jinhem prayah shramika samasyaom ka samana karana pada़ta hai ya samayabaddha niyatakaryom ko karana pada़ta hai, tanava ya dabava khaja ke rupa mem jane jane vale tvacha vikarom ke shikara ho jate haim|
yaha apane apako hathom tatha pairom aura garadana ke pichhe vrittakara risane vale ghavom ke madhyama se prakata karata hai|
yaha sthiti eka tanga karane vale pati /vali patni, irshya, kuntha tatha anya manovegom ke samuha dvara bhi utpanna ho sakati hai|
anya karanom mem doshapurna upapachaya, baddhakoshthata tatha paushanika dosha samahita haim|
aneka mamalom mem purane rogom ka shamanatmaka aushadhopachara bhi eka sarvadhika shaktishali sahayaka karyotpadaka ghataka hai|
yadi nihsrava daba jata hai, to kuchha anya adhika gambhira roga vikasita ho sakate haim|
khaja ko thika karane ka sarvottama upaya rakta pravaha evam sharira ki saphai karana hai|
samasya ki gambhirata evam kalavadhi ke adhara para, pancha se dasa dinom taka, santare ke rasa tatha pani para upavasa ke satha upachara prarambha karana chahie|
rasa upavasa se sharira se vishakta apashishta ke nishkasana mem sahayata milegi tatha santoshajanaka sudhara mem agrasara hogi|
kuchha mamalom mem, upavasa ke prarambha mem tvacha ke madhyama se apashishta dravya ke badha़e hue nishkasana ke karana sthiti aura adhika kharaba ho sakati hai|
lekina jaisa-jaise upavasa chalata rahata hai, sudhara apane apa prakata hoga|
rasa upavasa ke pashchata sampurnahara breda ya chapatiyom ke satha parose gae phala, kachchi ya bina namaka ki vashpita sabjiyam li ja sakati haim|
ghi ke sthana para nariyala ke tela ka upayoga kiya ja sakata hai|
kuchha dinom ke pashchata, ahara mem dahi tatha dudha shamila kiya ja sakata hai|
gajara tatha kharabuja vishesha rupa se labhadayaka haim|
isake pashchata rogi bijom, giridara phalom tatha anajom, sabjiyom tatha phalom jaise tina buniyadi khadya-padartha samuhom ke susantulita ahara ko dhire-dhire prarambha kara sakata hai|
ahara ke eka baड़e ansha mem kachche khadya-padartha samahita hone chahie|
garari, munga tatha soyabina jaise bijom tatha phalliyom ko ankurita kiya ja sakata hai|
isa ahara mem kachchi ghani mem pirae gae vanaspatika tela, shahada tatha khamira milae ja sakate haim|
ki ja rahi pragati ke adhara para do maha ya ese hi antaralom para rasa upavasa ko doharaya ja sakata hai|
khaja ke jirna tatha adhika kathina mamalom mem, rogi ko rogamukta hone taka saptaha mem kama se kama eka dina upavasa karana chahie|
isa rogi ko chaya, kaॉphi, madaka peyom tatha sabhi masalom aura atyadhika masaledara vyanjanom se dura rahana chahie|
use shakkara, maide ke utpadom, parimarjita chavala tatha chhilakarahita jau jaise vikrita anajom tatha dibbabanda ya botalabanda khadya padarthom se bhi dura rahana chahie|
use kevala vishuddha evam paushtika khadya padartha hi khane chahie|
kachchi sabjiyom ke rasa, vishesha rupa se palaka ke rasa ke satha gajara ke rasa ka sammishrana, khaja ke upachara mem atyadhika labhadayaka siddha hue haim|
isa sammishrana mem labhadayaka mana gaya anupata, solaha aunsa ya eka pinta rasa banane ke lie, gajara dasa aunsa evam palaka chhah aunsa hai|
rogi ko yathasambhava adhika taji hava milani chahie|
pratibandhaka kapada़e nahim pahane jane chahie|
do ya tina litara pani pratidina piya jana chahie tatha rogi ko eka dina mem do ya tina bara avashya nahana chahie|
chhajana se prabhavita angom ko chhoda़kara, shesha tvacha, nahane se pahale hatha ki hatheliyom se prabalata se ragada़i jani chahie|
chhajana vale bhaga para nariyala ka tela lagaya ja sakata hai|
yaha tvacha ko komala bane rahane mem sahayata karega|
antom ko kriyashila banane ke lie tahalana ya dhire-dhire dauda़na prarambha kiya jana chahie|
suryasnana bhi labhadayaka hai, kyonki isase hanikaraka jivanu mara jate haim tatha pratah jaldi hi ushakala ki prathama kirana ke samaya prarambha kiya jana chahie|
chhajana ke sthanom para mitti ka halka lepa lagana bhi upayogi hai|
yaha lepa eka bara mem eka ghante ke lie lagaya jana chahie|
eka dina mem do ya tina bara doharaem|
ajirna, gauta ya madhumeha ke parinamasvarupa utpanna chhajana, kevala tabhi thika ho sakati hai, jaba ina rogom ka saphalatapurvaka upachara kiya gaya ho|
asvasthyakara botala se pine ke karana chhajana se piड़ita bachchom ko yadi chaya ke eka chammacha shahada mile hue phalom ke rasom ya pani para rakha jae to unhem rahata mila sakati hai|
miragi eka bahuta prachina roga hai jisane nepoliyana, elekjendara tatha juliyasa sijara sahita vishva ke bahuta se mahanatama vyaktiyom ko piड़ita kiya |
prachina logom ka yaha vishvasa tha ki piड़ita vyaktiyom ke sharira mem dushtatmaem pravesha kara gai haim, unaki atma para kabja kara liya hai tatha unake sharira ko ainthana se bhara diya hai|
yunaniyom ka yaha vishvasa tha ki panya vastuom se yaha roga utpanna hota hai|
prachina isaiyom ne ina ainthanom ke lie shaitana ko dosha diya tha|
miragi aniyamita antaralom para, avarti akasmika daurom se pahachani jati hai|
ina daurom ke samaya rogi maroda़ ke satha aintha jata hai tatha acheta hokara jamina para gira jata hai, jisake karana jabaradasta tantrikiya uthala-puthala hoti hai|
miragi ke do mukhya prakara haim, jinhem laghu apasmara tatha guru apasmara ke rupa mem jana jata hai|
pratyeka apane svayam ke vishishta pratimana ka anukarana karati hai|
laghu apasmara mem, jo miragi ka eka kama gambhira rupa hai, eka daura pada़ta hai tatha kuchha sekanda ke andara chala jata hai|
isake rogi mem bina ainthana ke chetana kshanika rupa se chali jati hai, kevala kabhi-kabhi thoda़i kathorata ati hai, ya chetana ki kisi pratyaksha hani ke bina jataka ya ankhom, sira, dhada़ ya agrangom ke sanchalana jaisi ainthana ke satha halaka jataka laga sakata hai|
ho sakata hai ki rogi na gire|
jo kuchha vaha kara raha hai, use vaha achanaka roka sakata hai tatha jaba daura samapta ho jata hai use phira se karane lagata hai, bina yaha janate hue ki kya ho chuka hai|
laghu apasmara ke daure jivana mem kisi bhi samaya ghatita ho sakate haim, lekina bachchom mem ye sarvadhika nirantara haim|
guru apasmara ke mamale mem yaha daura eka natakiya prabhava ke satha ata hai|
isa daure mem achanaka chetana chali jane ke satha-satha banhom, pairom tatha sharira ka tivra sankunchana hota hai|
isa daure ke prarambha hone se pahale, kuchha rogiyom ko koi dhara ya hava ya pani ka koi pravaha sharira para se pravahita hone, shora, gandha ya prakasha ki kaundha jaisi anuthi anubhutiyom ke rupa mem eka chetavani ya purvabhasa hota hai|
eka pratikatmaka daure mem, rogi chikhata-chillata hai, jamina para gira jata hai, chetana kho deta hai tatha usamem ainthana vikasita ho jati hai|
ainthanom ke satha munha mem jaga a sakati hai, mansapeshiyom mem ainthana ho sakati hai, jibha kata sakati hai, angom ka bhranta sthirikarana ho sakata hai, sira ka ghurnana ho sakata hai tatha ankhom ka apasarana ho sakata hai|
rogi apane mutrashaya evam malashaya ki mansapeshiyom para se niyantrana kho sakata hai|
yaha daura kai minatom taka bana raha sakata hai tatha samanyatah isake bada gahari ninda ati hai|
jagane para, rogi ko vaha kuchha bhi yada nahim raha sakata hai, jo usake satha hua hai|
loga, jo miragi se piड़ita haim, kisi bhi anya prakara se apasamanya nahim hote|
samanyatah ve janate haim ki vishishta uddipaka dvara daure prerita ho sakate haim|
miragi ke daurom ke bicha, unaka mastishka samanya rupa se kama karata hai|
miragi ka artha hai mastishka ki koshikaom ki kshati ya kuchha vanshagata apasamanyataom ke karana, mastishka ke andara vidyutiya apakriya|
pachaka gada़badiyam, antra vishaktata tatha eka vikrita tantrikiya sthiti prayah laghu apasmara ke mukhya karana hote haim|
guru apasmara, samanyatah anuvanshika prabhavom, mastishka ya tantrikiya pranali ke gambhira sadama ya chota ke parinamasvarupa hota hai|
dirghakalika uchcha tapamana ki vidyamanata ke satha, tanika-shotha, antrajvara tatha anya roga bhi guru apasmara prarambha kara sakate haim|
yaha, kuchha khadya padarthom ke kuchha vishishta prakara ke prati pratyurjatapravana pratikriya ke parinamasvarupa ho sakata hai, visheshatah protina ke kuchha vishishta rupa jo mansa ke mukhya ghataka hote haim |
parisanchari vikarom jaise mastishka ki ora agrasara dhamaniyom ke kathorikarana ke karana bhi miragi ke daure a sakate haim|
yaha prakara virala hai tatha kevala bahuta vayovriddha vyaktiyom mem ghatita hota hai|
purana madyavyasana, sisa vishaktikarana, kokena tatha isi prakara ki anya adatem bhi isa roga ki ora le jati haim|
miragi ke daure ke anya karanom mem, manasika dvanda, apurna khanija paripachana, vishesha rupa se maigneshiyama tatha kailshiyama ke, tatha galata vitamina upapachaya shamila haim| kuchha anusandhanakartaom ke anusara, haipoglaisimiya ya nimna rakta sharkara bhi miragi ke sarvadhika mamalom mem shamila hoti hai|
prakritika upachara mem, miragi ke rogi ko eka atiniyamanishtha ahara, sampurna vishrama tatha khuli hava mem anukulatama vyayama vale eka kathora pathyapathya-niyama ka anupalana karana pada़ta hai |
use eka sadharana evam sahi prakritika jivana ka palana avashya karana chahie|
use eka pramudita, ashavadi manovritti avashya apanani chahie, manasika evam sharirika atyadhika parishrama tatha chinta se avashya dura rahana chahie|
isa upachara ka sarvadhika mahatvapurna pahalu hai ahara |
sabase pahale, rogi ko prathama kuchha dinom taka kevala phalahara para rakha jana chahie|
isa avadhi ke daurana, use santare, seba, angura, chhota chakotara, adu, nashapati, anannasa tatha kharabuja, tarabuja jaise taje rasile phalom ka dina mem tina bara bhojana karana chahie|
usake pashchata, vaha lasunaghasa bijom tatha munga phaliyom jaise ankurita bijom, kachchi sabjiyom tatha phalom para jora dete hue, bijom, giriyom tatha anajom, sabjiyom tatha phalom jaise tina buniyadi khadya samuhom ka santulita ahara dhire-dhire apana sakata hai|
isa ahara mem kachcha dudha, vishesha rupa se bakari ka kachcha dudha, tatha kachcha makkhana tatha ghara ka bana hua panira jaise dugdha utpadom ki eka santulita matra shamila honi chahie|
isa ahara mem se, dudha ko chhoda़kara, shesha sabhi pashu protinom ko puri taraha se nikala dena chahie, kyonki unamem na kevala maigneshiyama ki kami hoti hai balki ve sharira ko usake apane maigneshiyama ke bhandara satha hi satha vitamina bi 6 se bhi vanchita karate haim| miragi ke rogiyom ko ina donom padarthom ki baड़i matra mem avashyakata hoti hai|
maigneshiyama ke sarvottama khadya srota, kachchi giriyam, bija, soya ki phaliyam, palaka, karamasaga tatha chukandara ke pattom jaisi hari pattedara sabjiyam haim|
isa rogi ko samasta parishkrita khadya-padarthom, tale evam chikanai yukta khadya-padarthom, shakkara evam usase nirmita utpadom, kada़ka chaya, kaॉphi, madaka peyom, masalom tatha achara se dura rahana chahie|
use atyadhika khane se bachana chahie tatha kuchha baड़i matra mem lie jane vale bhojanom ke sthana para barabara thoda़i-thoda़i matra mem bhojana karana chahie|
yadi aise rogi ne pahale aneka varshom taka shaktishali davaiyam li haim, to use achanaka puri taraha se una davaiyom ko banda nahim karana chahie|
sabase pahale, isa khuraka ko kama karake adha kara diya jae tatha usake bada ise taba taka dhire-dhire aura kama kiya jae jaba taka ki ise puri taraha se chhoda़a ja sake|
miragi ke rogiyom ko achchhe svasthya ke samasta prakritika niyamom ka kada़ai se palana karana chahie tatha samanya svasthya ka sarvochcha stara banana chahie tatha use banae rakhana chahie|
unhem manasika rupa se sakriya bane rahana chahie lekina samasta manasika tatha sharirika tanavom se bachana chahie|
shranti, thakana ya thakavata ki anubhuti ki ora sanketa karati hai|
yaha asthayi ya chirakalika ho sakati hai|
lagabhaga pratyeka vyakti ko kuchha avasarom para vishrama evam ninda ka tyaga karate hue, adhika samaya taka kama karana pada़ta hai, jisake karana asthayi shranti ho sakati hai|
isa sthiti ko paryapta vishrama dvara thika kiya ja sakata hai|
tathapi, chirakalika ya nirantara shranti eka gambhira samasya hai, jisake lie upachara ki eka vyapaka yojana avashyaka hai|
chirakalika shranti, vibhinna prakara ke ghatakom ke parinamasvarupa ho sakati hai|
eka vishishta acharana visheshata, badhyakarita, nirantara shranti ki ora agrasara ho sakati hai|
aneka vyakti lagatara yaha anubhava karate rahate haim ki ve taba taka vishrama nahim kara sakate, jaba taka ki ve vaha saba kuchha samapta nahim kara lete, jise eka bara mem pura karane ki avashyakata hai|
ye vyakti samanyatah purnatavadi, vyagra hote haim tatha taba taka vishrama nahim kara sakate, jaba taka ki ve pura kama samapta nahim kara lete, isaki paravaha kie bina ki ve kitane thake ho sakate haim|
shranti ka mukhya karana, bhojana ki galata adatom ke karana, kama jivana-shakti ya urja ki kami hai|
shranti isa bata ka sanketa hai ki sharira ki koshikaom ko avashyaka urja ka satata pravaha dene ke lie, unhem bhojana mem paryapta kriyashila paramanu prapta nahim ho rahe haim|
sapheda shakkara jaise parishkrita khadya-padarthom, parishkrita anajom tatha sapheda ate ke utpadom, ke alava sansadhita, dibbabanda tatha parirakshita khadya-padarthom ke abhyasika upayoga se samanyatah hamari pranali para bahuta bura prabhava pada़ta hai|
isa prakara se ‘vikrita’ khadya-padartha, eka bahuta baड़i sima taka, apane amulya vitaminom tatha khanijom se vanchita ho jate haim|
isa prakara ke khadya-padartha, vyagrata, thakana, sthulata tatha aja prachalita aneka shikayatom ke samuha ki ora agrasara karate haim|
kuchha sharirika sthitiyam shranti utpanna kara sakati haim|
raktalpata thakana ki ora agrasara hone ka eka bahuta ama roga hai|
ise ‘suspashta thaki hui’ vyakulata ke rupa mem jana jata hai|
raktalpata mem, utakom ko bahuta kama ऑksijana pahunchati hai, jisake parinamasvarupa samanya rupa se urja utpanna nahim ho sakati|
isake karana nirantara thakana tatha manasika avasada hota hai|
raktalpata samanyatah, lauha tatha vitamina bi12 ki kamiyom ke parinamasvarupa hoti hai|
kabhi-kabhi vitamina bi6 tatha phaॉlika amla ki kamiyam bhi shamila hoti haim|
anidra roga ya ninda ki kami, santapana shranti ka eka karana ho sakati hai|
ninda ki goliyom tatha anya davaiyom dvara utprerita ninda se shranti nahim nikalati hai|
vastutah, jitani lambi avadhi taka ninda ki goliyom ka prayoga kiya jata hai, shranti utani hi adhika pareshana karane vali tatha chirakalika ho jati hai|
shranti ka karana banane ke alava, ninda ki goliyam, tantrikiya tanava ki ora agrasara karati haim, joki shranti ko aura badha़a deti hai|
unnidrata ke hanikaraka prabhava ninda ki goliyom ke prabhavom ki tulana mem vastava mem kama haim|
ahara mem sanshodhana karake tatha vishrama karane ki ichchhashakti ka vikasa karake anidra roga ka upachara kiya ja sakata hai|
antra parajivi bhi shranti ki ora agrasara kara sakate haim, kyonki ve sharira ko achchhe poshana se vanchita kara dete haim tatha apane apako paushtika lala rakta se bhara lete haim|
anya roga, jinake karana shranti ho sakati hai, nimna raktachapa, nimna raktasharkara, sharira mem kisi bhi prakara ka sankramana, yakrita ki kshati, eka manda thairaॉida tatha kritrima suvasom, rangom tatha parirakshakom sahita yojakom ke karana kuchha khadya-padarthom tatha davaiyom ke prati pratyurjata haim|
manasika tanava, shranti ke pramukha karanom mem se eka hai|
vyakti jo tanavagrasta hota hai tatha vishrama nahim kara sakata hai usake sharira ki samasta mansapeshiyam kuchha sankunchita ho jati hai |
isase urja ki baड़i matra asamanya rupa se anavashyaka apavyaya ki ora agrasara hoti hai|
khadya-padarthom ka nirantara dahana hota rahata hai, dugdha amla, sharira starcha mem parivartana ke lie yakrita mem pahunchane ki tulana mem, adhika teji se sanchita hota hai|
vyakti, jo atisamvedanashila, vyagra tatha chida़chida़e hote haim, samanyatah isa prakara ki shranti se piड़ita hote haim|
anasulaji bhavanatmaka samasyaom ke parinamasvarupa bhi shranti ho sakati hai|
kisi sthiti, paryavarana tatha kisi vyakti ke prati bhi aprasannata, thakana ki anubhuti mem baड़a yogadana de sakati hai|
apane manobhavom ko sanchita karane ki tulana mem unhem abhivyakta karana behatara hai|
chinta, uba ki koi anubhuti tatha eka asaphalatavadi manovritti jaise anya bhavanatmaka ghatakom ke parinamasvarupa shranti ho sakati hai|
paushanika mapadanda shranti ke upachara mem sarvadhika anivarya haim|
adhyayana batate haim ki loga jo thoda़i matra mem bicha-bicha mem bhojana karate haim ve, jo dina mem kevala tina bara bhojana karate haim ki tulana mem, thakana tatha ghabarahata se kama pida़ita hote haim, adhika spashta rupa se sochate haim evam adhika sakshama hote haim|
isa bicha ke bhojana mem taja़e ya sukhe phala, taja़e phala ya sabjiyom ke rasa, kachchi sabjiyam athava sampurna anaja ke breda ki eka chhoti saindavicha sammilita honi chahie|
yaha bicha ka bhojana thoda़i matra mem hona chahie tatha niyamita ahara mem kama bhojana khana chahie| 
inhem nirdishta samaya jaise subaha 11 baje, shama ko 4 baje aura sone jane se purva lena chahie| 
rogi ko svasthyavardhaka bhojana khana chahie jo ki sharira mem urja ki apurti karata hai|
landana ke charlsa di koti marsha ne apani pustaka ‘priskripshana phaॉra enarji’ mem thakana dura karane tatha urja badha़ane ke lie bhojana nirdharita kie haim|
unhem unaki svabhavika avastha mem lena chahie|
ye makai danom ke siriyala haim|
inhem taja़a pisana chahie|
hamare pasa anapake anajom mem uttama svasthya ke lie eka uchita bhojana hai jisamem avashyaka vitamina tatha urja sarjaka hote haim|
anaja danom ke atirikta, marsha chhilakom se sidhe li gai taji kachchi giriyom aura kanda-mulom ki bhi salaha dete haim|
ve kahate haim ki koi bhi bija ya kanda-mula jo phira se uga jaenge, ve manava jivanashakti ko navina karenge|
rogi ko bijom, giriyom tatha anajom, sabjiyom evam phalom se nirmita anukulatama ahara lena chahie|
lagabhaga, hara khadya-padartha samuha ko tina bara ke bhojanom mem se eka ke adhikansha ki apurti karani chahie|
ankurita karana bijom, binom tatha anajom ko kachche rupa mem khane ka eka achchha tarika hai, ankurana khadya-padarthom ke paushanika mahattva ko badha़a deta hai aura ankurana ke samaya bijom mem kai nae vitamina banate ya vriddhi karate haim|
rogi ko vishesha rakshaka khadya-padarthom jaise dudha, uchcha koti ke bina garama kie tilahanom ya giriyom ko dabane se prapta aparishkrita vanaspatika tela aura shahada sahita tina svasthyakaraka bhojana samuha mem vriddhi karana karana chahie|
rogi ko paushanika kamiyom ke prati eka prabhavakari driढ़ pramana ke rupa mem prakritika vitamina aura khanija mem vriddhi karana chahie kyonki aisa paya gaya hai ki ye kamiyam thakana mem eka karaka haim|
paintothenika anla ki kami, e bi vitamina, vishesha rupa se charama thakana ki ora agrasara karati hai kyonki isa vitamina ki kami adhivrikka granthiyom ke kshaya se sambandhita hai|
phira bhi, alaga se pentothenika anla lete samaya, yaha yada rakhana chahie ki, anya bi vitaminom ke asantulana se bachane ke lie isake satha sampurna bi kampleksa bhi liya jana chahie|
vastava mem, sampurna bi kampleksa granthiyom ke poshana tatha niyantrana mem sahayata karake tantrikaom ki raksha karata hai aura urja badha़ata hai|
vitamina bi se sanriddha shakahari bhojana gehum aura anya sampurna anaja siriyala, hari pattidara sabjiyam, bina paॉlisha ka chavala, dudha, giriyam, kela, yista, dalem tatha matara haim|
anya vitamina vividha rogom se sambandhita thakana mem vishishta bhumika nibhate haim|
kamiyom ko sudharane se thakana gayaba ho jaegi|
potaishiyama thakana ke viruddha sanrakshana ke lie vishesha rupa se avashyaka hai|
kachchi hari sabjiyam isa khanija se sanriddha hoti haim|
kailshiyama vishrama ke lie anivarya hai aura anidra tatha tanava donom ke mamalom mem jo ki thakana ki ora agrasara kara sakate haim, labhaprada hai|
sodiyama aura jinka bhi thakana ke upachara mem labhaprada haim|
kachchi sabjiyom ke rasa, visheshakara alaga se ya chukandara aura khire ke rasa ke satha milakara liya gaya gajara ka rasa, thakana se ubarane mem atyadhika mulyavana haim|
solaha aunsa ke rasa ke sammishrana mem dasa aunsa gajara, tatha chukandara aura khire pratyeka ke tina-tina aunsa ke anupata ko upakari mana jata hai|
rogi ko urja badha़ane ke lie espirina, prashantaka tatha anya aushadhiyom ko lene, kaॉphi ya madira pine, dhunrapana karane, kuchha sharkara ya mithai khane jaise saharom para nirbhara karane se bachana chahie|
ye kevala asthayi vriddhi dete haim aura isake shighra bada hi yaha vyakti ko pahale se bhi badatara sthiti mem chhoda़te hue, urja stara ko niche gira deta hai|
aise sahare antatah bahuta hanikaraka hote haim aura alpaglukoja़raktata ya ninna rakta sharkara tatha madatyaya jaisi bimariyom mem parinita ho sakate haim|
pramukha samasyaem jo pittashaya ko pida़ita karate haim, ve kolisistaitisa aura pittapathariyom ke rupa mem jani jane vali eka shotha avastha hai|
shotha avastha prayah kolilithiesisa ya pitta-pathari ki ora agrasara hoti hai, jo samanyatah pitta ke sammishrana mem gada़bada़i ke karana hoti hai|
kolestraॉla aura pitta lavanom ke anupata mem koi parivartana, jamava ke nirmana mem parinita ho sakate haim|
pittashaya dasa sentimitara lamba aura tina sem pancha sentimitara chauda़a, eka nashapati ke akara ka anga hai|
yaha yakrita ki bhitari-sataha para dayim ora pasali ke nichale kinare ke thika niche hota hai| 
pittashaya ka karya yakrita dvara sravita pitta ko pachana ki prakriya mem sahayata dene ke lie ise uchita samaya para dyodinama mem pahunchane ke lie sanchita evam sandra karana hai|
pitta mukhyatah pitta lavanom evam anlom, ranga dravyom tatha kolestraॉla ka bana eka utsarga hai|
pitta vasa ke pachana evam avashoshana mem sahayata karata hai aura vasa mem ghulanashila vitaminom e, di, i tatha ke, khanijom tatha kailshiyama ke avashoshana mem sahayata karata hai|
pittashaya samanyatah bhara hua aura bhojanom ke madhya shithila hota hai|
pachana ki prakriya ke daurana, jaba bhojana dyodinama mem pahunchata hai, to kolisistokainina namaka eka harmona ka antarika shleshmika jilli mem utpadana hona prarambha hota hai|
jaba yaha harmona rakta pravaha dvara pittashaya mem pahunchata hai, to yaha pittashaya ko sankuchita hone ke lie prerita karata hai, phalasvarupa pitta sravita hota hai jo eka ubhaya vahini ke dvara dyodinama mem ekatrita hota hai|
shuru mem, pittapathariyam mahina bajari ke rupa mem ho sakati haim|
parantu ye mahina kana anta mem adhika bada़e pathariyom ke nirmana ki agrasara hote hue aura adhika jamavom ke lie nabhi banate haim|
pittapathariyom ki ghatana vishesha rupa se sthulakaya aura 40 varsha se adhika unra vali mahilaom mem, purushom ki apeksha pancha guna adhika hota hai|
pittashaya ke rogom ka mukhya lakshana hai tivra ya bicha-bicha mem hone vali adhijathara ki pida़a| 
apacha, gaisa, bhojana ke bada purnata ka anubhava, koshthabaddhata, michali aura drishti dosha anya samanya lakshana haim|
vasa ke prati asahishnuta, chakkara, piliya, raktalpata, munhasa aura anya ghava bhi ho sakate haim| 
apasphita shiraem, bavasira aura keshikaom ka nashta hona bhi pittashaya ke rogom se sambandhita gada़bada़iyam haim|
motape se pittashaya ke rogom ka sambandha sade ahara para jivita, adima logom mem unaki suchita durlabhata se milakara ye batate haim ki ina rogom ke vikasa mem ahariya karaka mukhya bhumika nibhate haim| 
pittashaya ke vikarom ka mukhya karana parishkrita karbohaidretom, vishesha rupa se sharkara se hua adhika-poshana hai kyonki reshom ki hani urja antargrahana ke eka prakritika avarodha ko hata deta hai|
adhika-poshana bhi kolestraॉla ke srava mem vriddhi karane ki ora agrasara karata hai|
vasa se sanriddha bhojana pittayasha pida़a ya pittapathari udarashula ka daura utpanna kara sakata hai| 
jirna kabja़ pahale se hi pravritta karane vala eka sabase mahattvapurna karaka hai|
kamaja़ora svasthya, anuvanshika ghataka, tanava, merudandiya visthapana, galata mudra tatha mansapeshiya tanava se bhi pittashaya ke vikara utpanna ho sakate haim|
chini pittashaya ke vikarom ko krodha ki bhavana se joda़te haim|
yadi pittapathariyam bahuta bada़i hom ya una mamalom mem jinamem ve lambe samaya se upasthita hom to shalyakriya avashyaka ho jati hai|
chhote pittapathariyom ko, phira bhi, ahariya upacharom dvara hataya ja sakata hai|
pittashaya ke sujana va jalana ke tivra mamalom mem, rogi ko do ya tina dinom ke lie upavasa karana chahie, jaba taka ki tivra avastha samapta nahim ho jati|
upavasa ke daurana pani ke atirikta kuchha bhi nahim lena chahie|
upavasa ke bada, rogi ko kuchha dinom ke lie phala tatha sabji ke rasom ka sevana karana chahie|
gajara, chukandara, chhote chakotare, nashapati, nibu ya angura ke rasom ka upayoga kiya ja sakata hai| 
rasom ke pashchat, rogi ko eka aise susantulita bhojana ko apanana chahie jisamem sabhi anivarya poshaka-tatvom ki paryapta matra honi chahie|
adarshatah, bhojana kachchi aura paki sabjiyom, phala evam sabji ke rasom, tatha madhyama matra mem phala va bijom para ja़ora dene ke satha, dugdha-shakahari hona chahie|
nashapati udaratapurvaka khaya jana chahie kyonki inase pittashaya para eka vishesha upacharatmaka prabhava hota hai|
bhojana mem dahi, panira tatha eka bada़a chammacha jaituna ka tela bhi dina mem do bara sammalita karana chahie|
tela vasa ko pachane vale enja़aimom, pitta aura laipesa ke utpadana mem eka uttejaka padartha ki taraha kama karata hai|
bhojana mem uchcha gunavatta vala vanaspatika tela pitta-pathari ke nirmana ko bhi rokata hai|
rogi ko tina bada़e bhojanom ki apeksha bara-bara thoda़e-thoda़e bhojana karana chahie|
pitta pathariyom ko dura karane ke lie kuchha prakritika upachara karane vale chikitsakom dvara tela upachara ki salaha di jati hai|
kachcha, prakritika, aparishkrita vanaspatika telom, udaharana ke lie, jaituna ya surajamukhi tela, ka upayoga kiya jata hai|
eka aunsa vanaspatika tela, variyata se jaituna tela, subaha mem sabase pahale lena aura isake turanta bada chara aunsa chhote chakotare ya nibu ka rasa pina, yaha prakriya hai|
yaha upachara kai dinom taka, yadi avashyaka ho to, yaham taka ki haphtom taka, hara subaha kiya jana chahie|
peta ke upari kshetra mem garama paika ya sikaim ke prayoga se pitta-pathari ki udara pida़a kama ki ja sakati hai|
yadi rogi ko kabja़ hai to sharira ke tapamana para garama pani ka enima maliya sanchayana ko nikalane mem sahayata karega|
gaistraitisa amashaya ki sataha ka sujana va jalana hai|
yaha eka kashtadayaka avastha hai jo yadi samaya para upachara nahim kiya gaya to vrana(alsara) sahita kai jatilataom ki ora agrasara kara sakata hai|
kabja़ isa avastha ko kisi anya vikara ki apeksha aura adhika bigada़ta hai|
jvalanashila vikshati (lija़na) tivra aparadanakari amashaya shotha ya jirna apakshayi amashaya shotha mem se koi eka ho sakata hai|
dusara prakara lauha nyuna raktalpata se pida़ita adhe rogiyom mem upasthita paya gaya hai|
amashaya shotha ke pramukha lakshana bhukha ki kami, michali, ulati, siradarda aura chakkara haim|
amashaya ke kshetra mem pida़a aura vikalata ka anubhava bhi hota hai|
adhika jirna mamalom mem, peta mem purnata ka anubhava hota hai, kha़asakara bhojana ke bada|
lambi bimari aksara vaja़na ki kami, raktalpata aura kabhi-kabhi amashaya ke rakta srava mem parinita ho jati hai|
tivra akramana ke daurana aura jirna amashaya shotha ke adhikatara mamalom mem bhi haidroklorika anla ke srava mem kami aura shleshma ka atyadhika nissarana ho sakata hai|
amashaya shotha ke sabase adhika niyamita karana hai ahariya asavadhani jaise ki adatana adhika khana; anuchita rupa se mishrita ya anuchita dhanga se pakae bhojana ko khana, kada़i chaya, kaॉphi ya madaka peyom ka atyadhika sevana, masale, saॉsa ka adhika matra mem adatana upayoga karana adi|
kabhi-kabhi yaha kuchha rogom jaise khasara, rohini(dipha़thiriya), inpha़luenja़a, vishanujanya nimoniya adi ka anusarana kara sakata hai|
prayah, yaha chinta, vyagrata, dukha, aura dirghakalina tanava ke bhi phalasvarupa hota hai|
kuchha aushadhiyom, teja anlom aura ksharaka padarthom ke prayoga bhi amashaya shotha ko badha़a sakate haim| 
rogi ko tivra tatha jirna donom mamalom mem upavasa prarambha karana chahie|
tivra mamale mem, rogi samanyatah do ya tina dinom ke alpakalika upavasa ke pashchat achchha ho jata hai|
jirna avastha mem, upavasa ko sata dina ya aise hi lambi avadhi ke lie ja़ari rakhana pada़ sakata hai| 
vikalpa ke rupa mem, ho rahi pragati para nirbhara karate hue, alpakalika upavasa ki eka ya do mahine ke antara mem punaravritti ki ja sakati|
yaha upavasa phala ke rasom para rakha ja sakata hai|
upavasa se, takalipha़ dene vali vastuom ka sevana turanta prabhavapurna dhanga se banda ho jata hai, amashaya ko arama milata hai aura jalana utpanna karane vali, vishakta avastha, ko ghatane diya jata hai|
upavasa se nishkasana mem vriddhi hoti hai aura sharira mem ekatrita atirikta vishakta padartha ko phenka diya jata hai|
tivra lakshanom ke ghatane ke bada, rogi ko agale tina dinom ke lie kevala phalahara apanana chahie| 
rasile phalom jaise seba, nashapati, angura, chhote chakotare, santare, anannasa, chiku aura melanom ko isa avadhi ke daurana pancha ghante ke antara para liya ja sakata hai|
rogi ko madira, nikotina, masale aura bagharom, mansahari bhojanom, mirchiyom, khatti chija़om, acharom, kada़i chaya va kaॉphi ke upayoga se paraheja karana chahie|
use mithaiyom, pestriyom, kekom aura vatita jala ka bhi paraheja karana chahie| 
dahi aura ghara ka bana panira liya ja sakata hai| 
gajara ke rasa ka palaka ke rasa ke satha sevana karana, jathara-shotha ke lie kaphi labhadayaka upachara mana gaya hai| 
isa mishrana mem chhah aunsa palaka ke rasa mem dasa aunsa gajara ka rasa milana chahie| 
bahuta se vividha khadya padarthom ko eka hi ahara mem nahim milana chahie| 
rata ko sone ke kama se kama do ghante pahale bhojana karana chahie| 
pratidina atha se dasa glasa pani ka sevana kiya jana chahie, kintu pani ko bhojana ke satha nahim lena chahie, kyonki yaha pachaka rasom ko patala kara deta hai aura jisase pachana ki kriya mem vilamba hota hai| 
aura ina saba mem vishesha mahatvapurna bata hai, ki hamem khane ke daurana jaldabaji se paraheja karana chahie tatha khane ko shanta va khushanuma mahaula mem parosa jana chahie| 
upachara ke arambha se hi, antom ki svachchhata hetu lagabhaga eka saptaha ke lie pratidina, garma pani ka enima kiya jana chahie| 
yadi kabja adatana hai, to isake upachara ya unmulana ke sabhi upaya kiye jane chahie| 
marija ke pratidina shushka ragada़ dekara spanja se sapha़ karana chahie| 
pratidina dina mem do bara, garma sampida़ka athava garma botala ka prayoga kara, khali peta ya khane ke do ghante bada, garmi ka prayoga karana bhi labhadayaka hoga| 
marija ko koi bhi kathina sharirika aura manasika karya nahim karana chahie| 
use shvasana sambandhi vyayama aura kuchha anya sarala vyayama jaise-tahalana, tairana aura golpha khelana adi karane chahie| 
use chintaom aura manasika tanavom se bhi paraheja karana chahie| 
kanchabindu ankhom ki eka gambhira bimari hai, jo netra golaka mem bada़te hue dabava se lakshanita hoti hai, jise intaऑkyulara (netra ke andara ka)dababa kahate hai| 
yaha sharira ke uchcha raktachapa jaisa hi hai| 
isalie isa sthita ko ankhom ka uchcha raktachapa ke nama se bhi jana jata hai| 
eka nishchita sima taka ke intaऑkyulara (netra ke andara ka) dabava ko avashyaka mana jata hai, lekina isake badha़ne se ankhom ko nukasana pahunchata hai, aura jisake parinama svarupa ankhom ki roshani bhi ja sakati hai| 
aja kanchabindu vayaskom mem andhepana ka mukhya karana hai| 
pratyeka atha mem se eka andha vyakti kanchabindu ka shikara hai| 
nikata drishtidosha vale vyaktiyom ki apeksha, dura-drishtidosha vale vyaktiyom mem isa bimari ke hone ki sambhavana adhika hoti hai| 
kanchabindu ka prathama lakshana hai, rata mem dura sthita vastuom ke charom ora tejomandala ya rangina golom ka dikhana| 
isa sthita mem ankha ki putali aksara age ki ora ho jati hai, aura marija bahudha mastaka, kanapati aura galom ke pasa lagatara darda ki shikayata karata hai| 
sira darda asamanya nahim hai| 
jaise hi kanchabindu vikasita hota hai, vaise hi drishti mem dhire-dhire kamajori ane lagati hai, aura yadi bimari ke prathamika charana mem hi uchita upayom ko nahim apanaya jata hai, to anta mem isaka parinama andhepana hota hai| 
chikitsa vijnana ke anusara ankhom para adhika tanava aura lambe samaya taka kharaba roshani mem karya karana hi, kanchabindu ka mukhya karana hai| 
kintu galata ahara, galata jivana shaili aura kisi anya bimari ke upachara ke lie lambe samaya taka nishedhatmaka davaom ka prayoga hi vastava mem kanchabindu ke mukhya karana haim | 
ankhom para tanava kevala eka sahayaka lakshana hai| 
kanchabindu lambe tanava ke karana bhi ho sakata hai aura yaha aksara adhivrikka thakana ki pratikriya hota hai| 
adhivrikka granthi ke etdosterona ke utpadana kara pane ki akshamata ke parinama svarupa sharira mem namaka ki atyadhika kami ho jati hai aura phala svarupa utakom mem tarala padartha ka sanchayana ho jata hai| 
ankhom mem pani ki adhikata ke karana, netra golaka kada़a ho jata hai, tatha usaki narami aura lachilapana nashta ho jata hai| 
kanchabindu ko chakkara, shiranala (sainasa), elarji, madhumeha, rakta mem honevali sharkara ki bhari kami(haipogsimiya), dhamani kathinya aura svachalita tantrakiya pranali mem asantulana, se susambandha bhi kiya jata hai| 
kanchabindu ka purina chikitsakiya upachara, shalya kriya ke madhyama se kiya jata hai, jo pani ke karana ankhom mem hone vale antarika dabava se chhutakara dilata hai| 
halanki, yaha adhika pani maujudagi ke karana ko nahim hatata hai| 
phalasvarupa, shalyakriya ke bada bhi, yaha nishchita nahim rahata hai, ki jo bhi kashta hai, vaha dubara nahim hoga, ya yaha dusari ankha para koi prabhava nahim dalega| 
kanchabindu ka prakritika upachara, thika vaisa hi hai, jaisa atyadhika vishaktata se juda़i hui kisi bhi anya sthiti ka hota hai aura yaha bachi hui drishti ki suraksha ki ora dhyana deta hai| 
yadi isaki prathamika avastha mem hi isaka upachara ho jata hai, to usake parinama anandadayaka hote haim| 
yadyapi vikasita kanchabindu ki sthiiti ka ilaja nahim ho sakata hai, phira bhi, kuchha paushtika aura anya jaivika prayasa, sthita ke niyantrana aura bachi hui drishti ki suraksha ke lie karagara siddha ho sakate haim| 
kanchabindu ke marijom ke dvara, kuchha khadya padarthom ka imanadari se paraheja kiya jana chahie| 
visheshatah kaphi se, usake uchcha kaiphina avyava ke karana, purnataya dura rahana chahie| 
kaiphina, shira-sankirnana ke uttejana, raktachapa ko badhane tatha ankhom mem rakta pravaha ko badhane ka karana hota hai| 
biyara aura tambaku se bhi paraheja karana chahie, kyonki ye rakta nalikaom ke sankuchana ka karana ho sakati haim|
chaya ko kevala niyantrita rupa mem hi liya jana chahie| 
marija ko kisi bhi samaya, tarala chijom ka adhika sevana nahim karana chahie, chahe vaha rasa ho, dudha ho ya pani hi kyom na ho| 
vaha kama se kama eka ghante ke antarala mem, eka dina mem kai bara pani ki thoda़i-thoda़i matra ka sevana kara sakata hai| 
kanchabindu se pida़ita marija ka ahara mukhya rupa se tina adharabhuta khadya samagri para nirbhara hona chahie, jaise - dane, giridara phala aura anaja, sabjiyam aura phala, taje phalom, aura sabjiyom para vishesha dhyana dete hue, vitamina si ki prachurata vale kachche padartha adi| 
motiyabinda ke marija ke lie sujavita ahara ka sevana, kanchabindu se pida़ita marija ke dvara bhi kiya ja sakata hai| 
adhyaya mem varnita, ankhom ko arama pahunchane vale aura ankhom ke sashaktikarana ke bahuta se nuskhe bhi kanchabindu ke upachara mem karagara sabita hogem| 
kuchha poshaka karaka bhi kanchabindu ke upachara mem sahayaka hote haim| 
aisa paya gaya hai ki kanchabindu ke marijom mem, aksara vitamina e, bi, si, protina, kailashiyama aura anya khanijom ki kami payi jati hai| 
kailashiyama aura vitamina bi-kampalaiksa jaise poshaka tatva bhi intaऑkyulara (netra ke andara ka) sthita mem arama pahunchane mem sahayaka siddha hote haim| 
bahuta se daktarom ka manana hai ki kanchabindu mem intaऑkyulara (netra ke andara ka) dababa ko vitamina si upachara ke madhyama se kama kiya ja sakata hai| 
daॉ. maikala varano aura unake sahakarmiyom ne hala hi mem roma, itali mem pradarshita kiya hai, ki 150 ponda vajana vale vyakti ko dina mem pancha bara 7000 miligrama vitamina si (eskarbika anla) dekara, 45 dinom ke andara vachhanniya intaऑkyulara (netra ke andara ka) dababa ko paya ja sakata hai| 
peta mem halki pareshani aura atisara jaise lakshana, vitamina si ke adhika sevana ka parinama hote haim, jo kshanika hote haim va shighra hi khatma ho jate haim| 
aisa sujava diya jata hai ki adhika matra ke viparita prabhavom ko kama karane ke lie, vitamina si ki pratyeka matra ke satha, hamensha kailashiyama ki kuchha matra ka sevana kiya jana chahie| 
chunki kanchabindu ko eka 'tanava bimari' ki taraha mana jata hai, atah marija ko isa bimari ke daurana bhavanatmaka tanava se paraheja karana chahie aura jivana jine ke shanta tatha arama dayaka tarike ko apanana chahie| 
use adhika duradarshana aura philmem dekhane tatha atyadhika padha़ne jaisi adate jo ankhom para dababa dalati haim, usa se bhi paraheja karana chahie | 
gathiya, joda़o mem eka taraha ki jalana aura avarti sujana ki ora sanketa karati hai| 
yadyapi svabhava se dirghakalika, yaha tikshna aghata ke rupa mem prakata hoti hai| 
yaha eka aisi bimari hai, jo mukhya rupa se madhyama-avastha ke purushom ko prabhavita karati hai| 
kabhi-kabhi rajonivritti ke bada auratem bhi isa bimari se grasita ho jati haim| 
prachina grisa aura roma ke chikitsakom ko gathiya ke bare mem pata tha| 
yaha 18vim sadi mem jana gaya ki adhika anandadayaka ahara aura sharabayukta peya, aksara gathiya ke aghata ka karana banate haim| 
isa bimari ne purva mem elekjendara mahana, luthara, nyutana, milatana, daॉ. jonasana, phrenkalina aura luisa XIV jaise kai prasiddha vyaktiyom ko prabhavita kiya hai| 
gathiya ka aghata aksara, anguthe mem tikshna darda ke satha shuru hota hai, jisamem angutha kuchha hi ghanto mem narma, garma aura phula hua ho jata hai| 
aksara gambhira avastha mem, prabhavita paira mem thoda़a bhi vajana rakhana asambhava sa hota hai| 
yaha anya joda़ jaise ghutane aura kalaiyom ko bhi usi taraha se prabhavita kara sakata hai| 
kabhi-kabhi eka hi samaya mem eka se adhika joda़ prabhavita ho jate haim| 
aghata aksara madhya ratri ya ekadama subaha ke samaya mem hota hai, jaba marija achanaka jagata hai| 
gambhira aghata samanyatah eka saptaha ya aise hi samaya taka hota hai| 
isa samaya ke daurana marija ko halka bukhara raha sakata hai, aura bhojana ke prati usaki anichchha ho sakati hai| 
usaka samanya svasthya aksara aprabhavita rahata hai| 
kai haphtom ya mahinom ke bada aghata, phira se ho sakata hai| 
yadi bimari ka upachara uchita dhanga se nahim hua ho, to yaha samayanntarala kama bhi ho sakata hai| 
joda़ dhire-dhire gathiya se grasita ho jate haim| 
purane gathiya mem, mutranla sphataka ke khada़iyamaya pinda़ jonom mem raha jate haim aura tvacha ke bhitara utpanna ho jate haim| 
gathiya ki anya gambhira samasya hai, mutranla ke satha gurde mem patthara ka hona, jo peta mem gambhira shula ka karana hota hai| 
kuchha sthitiyom mem gurdem kharaba ho jate haim, aura niyamita rupa se karya karana banda kara deta hai| 
yaha eka gambhira sthita hai, kyonki isamem vishakta vyartha padartha jo samanyatah gurde ke dvara bahara nikala diya jata hai, vaha rakta mem sanchayita ho jata hai| 
gathiya ka mukhya karana hai-joda़om, tvacha aura gurdom mem mutranla sphataka ka nirmita ho jana| 
mutranla, sharira ki rasayanika prakriya ka antima utpada hai| 
gathiya se prabhavita logom mem samanya vyakti ki apeksha, uchcha matra ka mutranla paya jata hai, jo ya to adhika matra mem anla ke nirmana ke karana hota hai ya gurde ke dvara mutra ke madhyama se nishkasita hone vali anla ki ghati hui matra ke karana hota hai| 
yaha mutranla aksara rakta mem hi vilina hota hai| 
kintu jaba rakta mem isaki matra atyadhika ho jati hai, taba mutranla joda़om mem sui ke akara ke sphataka ka nirmana karata hai, jisase gathiya ka aghata hota hai| 
anuvanshikata, isa bimari ke hone ka eka pramukha karana hai aura kuchha nishchita jatiyom mem gathiya ki pravriti payi jati hai| 
anya karanom mem shamila haim, sharabayukta peyom ka adhika sevana, protina aura karbohaidreta ki prachurata vale khadya padarthom ka niyamita rupa se sevana tatha uchita vyayama ki kami | 
tanava ko bhi gathiya ka pramukha karana mana jata hai| 
sacheta pratikriya ke daurana, sharira ki lakhom koshikaem nashta ho jati haim aura ina koshikaom se nikalane vali mutranla ki bada़i matra, utakom mem pravesha kara jati hai| 
marija ko pancha se sata dinom taka, santare ke rasa ya pani para nirbhara sanyama (vrata) ko apanana chahie| 
kabhi-kabhi, sanyama/upavasa ke arambhika charana mem sthiti aura bhi kharaba ho jati hai, jaba rasom ke dvara vighatita mutranla, nishkasana ke lie rakta pravahikaom me chhoda़ diya jata hai| 
nirantara upavasa karane se yaha aksara suvyavasthita ho jata hai| 
gambhira sthiti mem, yaha sujava diya jata hai, ki eka lambe samaya ke upavasa ke bajaya tina dinom ke chhote-chhote upavasom ki shrrinkhala ko apanaya jaye | 
isa samaya ke daurana, anto ki svachchhata ke lie, pratidina garama pani se gudavasti ki prakriya ko apanana chahie| 
gathiya ke gambhira lakshanom ke ghata jane para, marija agale tina ya chara dinom ke lie, sabhi taraha ke phala ke ahara ko grahana kara sakata hai| 
isa pranali mem, use angura, seva, nashapati, ada़u, santare aura ananasa jaise rasile phalom ke tina ahara lene chahie| 
marija ko sabhi protina aura mutranla utpadaka khadya padarthom jaise- sabhi taraha ke mansa, ande aura machhali se paraheja karana chahie| 
granthiya mansa visheshakara hanikaraka hote haim| 
use sabhi vishakta dravyom [sharabom], chaya, kaphi, chini, maida aura usake utpada, aura sabhi dibba banda tatha parishkrita khadya padartho se paraheja karana chahie| 
masale aura namaka ka prayoga sambhavatah kama se kama karana chahie| 
potaishiyama ki prachurata vale khadya padartha jaise alu, kele, hari pattidara sabjiyam, sema aura kachchi sabjiyom ke rasa, gathiya (gauta) se suraksha pradana karate hai| 
chunkadara aura kakada़i ke rasa ke satha gajara ke rasa ka sevana karana bhi vishishta rupa se labhadayaka hota hai| 
eka pinta ya chholaha aunsa ki matra vala mishrita rasa banane ke liye tina aunsa chunkadara aura kakada़i ke rasa ko dasa aunsa gajara ke rasa mem milana chahie| 
kachche alu aura taje ananasa ka rasa bhi labhadayaka hota hai| 
mutra marga mem pattharom ki badha़ti hui ashanka ke karana, marija ko sharabarahita tarala chijom ka adhika rupa se prayoga karana chahie| 
use kama se kama atha gilasa thande ya garma pani ka sevana karana chahie| 
pairom ko dina mem do bara, epsama namaka se dhona chahie| 
pairom ko dhone ke lie, adhe ya eka paunda namaka ko garma pani mem mila sakate hai| 
saptaha mem tina bara sampurna epsama namaka snana bhi lena chahie| 
bada mem snana ko dhatakara, saptaha mem do bara bhi kiya ja sakata hai| 
rata ke samaya mem, prabhavita joda़om para thanda़e lepom ka prayoga karana bhi labhadayaka hoga| 
taji hava aura kuchha bahya vyayama bhi avashyaka haim| 
marija ko jaham taka sambhava ho apane jivana se tanava ko dura karana chahie| 
chakriya hridaya bimari ke antargata hridaya ko paryapta matra mem rakta ki apurti na kara pane ke karana hridaya-dhamani ke viphala hone se hone vale roga shamila haim| 
unamem hritshula, hrida-dhanasta ya hridayadhata aura bina kisi rodhagalana ke achanaka nrityu shamila hai| 
vartamana samaya mem, hridaya ki bimariyom mem dhyana dene yogya ijapha hua hai| 
pashrchimi deshom mem, hridayadhata nrityu ke karako mem pahala sthana rakhata hai| 
bharata mem tapedika aura sankramana ke bada, isaka tisara sthana hai| 
yaha bimari hara unra ke logom ko aura donom hi linga ke logom ko prabhavita karati hai, yadyapi visheshatah 40 se 60 varsha ki unra mem yaha mahilaom ki apeksha purushom mem adhika samanya hai| 
hridaya jo ki sharira ka atyavashyaka anga hai, lagabhaga eka bandhi hui mutthi ke akara ka hota hai| 
yaha garbhashaya mem janma ke pahale se hi karya karane lagata hai| 
isaka vajana lagabhaga 300 grama hota hai, yaha pratidina sharira se lagabhaga 4300 gailana rakta nikalata hai aura sabhi angom mem ऑksijana tatha poshana ki apurti karata hai| 
yaha eka dina mem 100000 bara dhada़kata hai, aura 60,000 mila ki chhoti-chhoti rakta nalikaom ke madhyama se ninratara rakta nikalata hai| 
isake badalem mem, hridaya ko poshana ke lie rakta ki avashyakata hoti hai, jo hrida-dhamani ke dvara apurtita kiya jata hai| 
vasa yukta padartha ke jama jane ke karana, dhamaniyom ka sankuchana aura kathorikarana ho jata hai, jisase rakta ka pravaha ruka jata hai| 
taba hridaya ko paryapta matra mem ऑksijana nahim mila pata hai| 
yaha sthita hridaya mem rakta ki paryapta purti na hone ya hridashula (ikimiya) ki taraha jani jati hai, jo ki eka letina shabda hai, jisaka ashaya hai- sine mem darda| 
yaha vastava mem hridaya mem aura adhika rakta ki avashyakata ka eka sanketa hai| 
isa sthita mem, parishrama ya atiutsaha sine mem darda ka karana bana sakata hai, atah marija, sharirika kriyaem simita rupa mem hi karem| 
yaha eka chetavani ki taraha kama karata hai, ki karya mem eka virama lae aura aise surakshatmaka upayom ko apanaye jo apako hridayadhata se bachayenge|
yadi dhamani ke bhitari tarapha kisi thakke ke jamane ya shiravarodha ke karana, dhamani ka sankuchana ho jata hai, to yaha hridaya ke usa hisse ke nrita hone ka karana bana sakata hai, jo ki ruddha dhamani para nirbhara hai, yaha hridayadhata ya hrida-dhanasrata kahalata hai| 
yaha daga chhoda़te huye nrityu ya svasthata ki ora agrasara ho sakata hai| 
marija thika hone vale dhava ke satha, gambhira rupa se akshama ho sakata hai ya apani sharirika kriyaom mem pratibandhom ke satha samanya jivana ji sakata hai| 
achanaka nrityu ki adhikatara dhatanaem, hridashula ya hrida-dhanasrata se pida़ita vyaktiyom mem hoti hai| 
hrida-dhamani usaki bhitari parata mem bahuta se rasayanom ke jamane ke karana sankuchita ho jati hai| 
yaha bahuta se tatvom ke grahana karane aura avashoshana ke paripekshya mem, sharira ki upapachaya prakriya mem vanshanukramita ya uparjita gada़bada़i ke karana hota hai| 
vasa ki prachurata vale khadya padartha visheshatah pashu-vasa se yukta ahara, hrida dhamani mem vasiya tatvom ke jamane ka karana banate haim, aura use ruddha va sankuchita karate haim| 
dhamani mem kisi chija ke bharane ki prakriya dhamani kathinya kahalati hai aura yaha parisancharana tantra ko prabhavita karane vala eka bada़a apakarshaka parivartana hai| 
hridaya ki bimari ka eka sadharana lakshana, shvasa lene mem pareshani ana ya kama ke daurana jaldi hamphane lagana, jo rakta mem paryapta matra mem ऑksijana na hone ke karana hota hai| 
anya sadharana lakshana hai, sine mem darda ka hona ya kisi bhi anga (bhuja) mem darda hona, anya lakshana haim-atispandana, ashaktata, bhavanatmaka asthirata, hatha va pamva ka thanda़a hona, adhika pasina ana aura thakavata| 
ye sabhi lakshana kisi anya asantulana ke karana bhi ho sakate haim| 
atah ina lakshanom ki vastavika prakriti janane ke lie, inaka uchita parikshana aura adhyayana avashyaka hai| 
hridaya ki bimariyom ke vastavika karana haim- khane ki galata adatem, jivana jine ka galata tarika aura bahuta tanava ka hona| 
naishanala harta aura langsa inastityuta ke prasiddha phreramingama harta adhyayana ne, hrida-dhamani bimariyom ke sata pramukha lakshanom ki pahachana ki hai| 
inamem se koi bhi eka ya mishrita khatare hridaya ki bimari ke karana ho sakate haim| 
unamem se bahuta se ahara se utpanna hote haim| 
ina khataranaka lakshanom ko apani jivana shaili mem parivartana karake ya ahara ko punah nirdharita karake niyantrita kiya ja sakata hai| 
lagatara chinta aura pareshani, adhivrikka granthi ko adhivrikka majjakka se sravita hone va tanava ko badhane vale harmonom aura............ke adhika utpadana hetu uttejita karata hai|......... 
yaha dhamaniyom ko ruddha karata hai, raktachapa ko badhata hai aura hridaya ke lie karya ko badhata hai| 
hridaya sambandhi bimariyom ka mulabhuta karana ahara hai| 
eka yathochita ahara jo sharira ke rasayana mem parivartana aura grahana kie jane vale samanya poshakom ki gunavatta mem sudhara ke lie banaya gaya, bahuta si sthitiyom mem, hridaya aura rakta nalikaom mem hone vale apakarshaka parivartana mem viparita prabhava bhi dala sakata hai| 
ahara dugdha-shakahari, aura kama sodiyama aura kailori vala hona chahie| 
usamem uchcha gunavatta, sabuta anajom, bijom, taje phalom aura sabjiyom para vishesha dhyana dete hue, prakritika jaivika khadya padarthom ka sammishrana hona chahie| 
khadya padartha jinaka parityaga kiya jana chahie-maide se nirmita utpada, mithaiyam, chakaleta, dibbabanda tarala khadya padartha, shitala peya, sharabata, ajaivika vasiya padartha jaise- makkhana, malai aura vasiya mansa| 
dhire-dhire namaka aura shakkara ki matra kama karani chahie| 
adhika gude vale khadya padarthom mem sodiyama ki uchcha matra payi jati hai aura kuchha mansa bhi adhika vasa yukta hote haim| 
ye adhika matra mem anla ka nirmana karate haim aura pranali mem uchcha stara ki vishakta ka nirmana bhi karate haim| 
inase hridaya rogiyom ko bachana chahiye| 
marija ko chaya, kaॉphi, sharaba aura tambaku se bhi paraheja karana chahie| 
chaya aura kaॉphi mem kaiphina namaka tatva paya jata hai, jo hridaya aura tantrakiyatantra para vishakta prabhava dalata hai| 
kaiphina eka teja hridayiya uttejaka hai, jise niyamita rupa se grahana karane para, dhada़kana mem tivrata ya hridaya ke dhada़kanom ki gati mem gada़bada़i a sakati hai| 
sharaba yakrita ko nashta karata hai aura hridaya ko atyadhika uttejita karata hai| 
yaha rakta- sharkara ke stara mem parivartana karata hai aura sharira se vitamina bi ke sangraha ko khatma karata hai| 
nikotina, hridaya ki peshiyom para vishakta va uttejaka prabhava dalata hai aura rakta-sharkara stara mem gada़bada़i lata hai| 
hridaya ki bimari se pida़ita vyakti ke ahara mem, jaham taka sambhava ho una sabhi poshaka tatvom ko shamila karana chahie, jo avashyaka vitaminom, khanijom aura anajom (jute hue avayava) ki prachura matra upalabdha ho|
ahara ka eka bada़a hissa phala va sabjiyam honi chahie aura jaba bhi sambhava ho unhem unake taje rupa mem hi grahana karana chahie| 
angura aura seva vishesha rupa se labhadayaka haim| 
sabhi avashyaka vasiya anla jo sirama kolestrola ke stara ko ninna karane aura dhamani kathinya ke khatare ko kama karane ke lie avashyaka hai, suryamukhi ke bija ke tela, makke ke tela ya kusuma ke tela se prapta kie ja sakate haim| 
bahuta se shodhom se pata chala hai ki aise vyakti jinaka sharira samanyataura para samanjita (thika ya vyavasthita ) nahim kara sakata hai, unamem lahasuna kolestrola ke stara ko dhatata hai| 
kolestrola ko dhatane vali eka anya mahatvapurna aushadhi garari hai| 
pila-bhura yaugika jo bahuta se paudhom aura pashuom mem, visheshatah ande mem paya jata hai, dhamani mem vasa ke jamane se suraksha pradana karata hai| 
khadya padartha ke uttama srota hai-aparishkrita, kachcha, ashodhita jaivika tela, bija aura anaja haim| 
hridaya ki bimari se pida़ita vyakti ko vitamina i ki prachurata vale khadya padarthom ka sevana karana chahie, kyonki ye vitamina koshikaom mem aksijana ki matra badha़akara, hridaya ki karyapranali mem sudhara late haim| 
ye parisancharana aura peshiya takata mem bhi ijapha karate haim| 
bahuta se sampurna ahara utpada aura hari sabjiyam, visheshatah gobhi ki bahari pattiyam, vitamina i ki achchhi srota haim| 
vitamina bi ka samuha hridaya aura parisanchari gada़bada़i ke daurana mahatvapurna hai| 
vitamina bi ka sarvottama srota sabuta anaja hai| 
vitamina si bhi avashyaka hai, kyonki yaha keshika- bhitti mem hone vali anayasa rukavata, jo hridayadhata ki ora agrasara kara sakati hai, usase suraksha pradana karata hai| 
yaha uchcha rakta kolestrola ke virodha mem bhi suraksha pradana karata hai| 
gussa, dara, nirasha aura isi taraha ke bhavom se hone vale tanava ke karana rakta mem vasa ki matra aura kolestrola ka stara turanta badha़ sakata hai, kintu yadi ahara mem vitamina si aura painthothenika anla ki paryapta matra ho to tanava se hone vali kshati kaphi kama ho jati hai| 
vitamina si ka sabase vipula srota nibu vansha ke phala haim| 
marija ko achchhe svasthya ke nirmana ke lie, prakriti ke kuchha nirdharita niyamom para bhi dhyana dena chahie, jaise- thoda़a vyayama karana, paryapta arama karana aura ninda lena, sahi manasika manovritti apanana aura svachchha hava lena tatha shuddha jala ka sevana karana| 
sadharanatah parichita atiraktachapa ya uchcha raktachapa, dhire-dhire marane vale ki taraha samaje jate haim| 
yaha madhyama unra mem hone vali bimari hai| 
jivana ki teja gati aura audyaugika va mahanagariya vatavarana ke karana upaje manasika tatha sharirika dabava se manovaijnanika chinta ka udaya hota hai| 
chinta aura manasika phikra, rakta ke pravaha mem adhivrikka granthi se nikalane vale harmona ko badhate haim, jisake phalasvarupa, rakta para dabava badha़ne lagata hai| 
sharira mem dhamaniyom ke madhyama se pravahita hone vala rakta, pratyeka koshika mem poshana va aksijana ki apurti karata hai| 
jaise hi hridaya dhada़kane ki kriya dvara, bada़i dhamaniyom taka rakta ko pahunchata hai, isake dvara lagaya jane vala bala una para eka dabava nirmita karata hai, aisi kriya raktachapa kahalati hai| 
atah eka nishrchita stara ka raktachapa, sharira mem rakta ke parisamvahana ke lie avashyaka hai| 
kintu jaba yaha dabava bahuta adhika badha़ jata hai, taba yaha atiraktachapa mem parivartita ho jata hai, jisake karana \\ keshika kahalani vali sampurna sharira ki chhoti rakta vahikaem ainthita ya atisankuchita ho jati hai| 
yaha sankuchana, rakta vahikaom ke madhyama se hridaya para rakta ke pravaha ke liye adhika dabava dalata hai| 
atah hridaya mem badha़te hue dabava ke karana rakta para dabava bhi badhata jata hai| 
raktachapa, rakta daba mapi namaka yantra se pare para milimitara mem mapa jata hai| 
jaba dabava sabase adhika ho jata hai, taba hridaya ki dhada़kana prakunchaka dabava kahalati hai, aura jaba do dhada़kanom ke bicha ka dabava sabase kama ho jata hai, taba yaha anushithilita dabava kahalata hai| 
jaise ki hridaya sampurna sharira mem rakta ka parisamvahana karata hai aura hridaya ki kriyaom ki ora sanketa karata hai, atah sabase pahala hissa (raktadabamapi ka) hridaya ke sankuchana ke dabava ko darshata hai| 
jaba hridaya shanta hota hai aura rakta vahikaom ki sthita ko darshata hai, taba dusara hissa (rakta daba mapi ka) dhamaniyom mem upasthita dabava ko darshata hai| 
120/70 ko rakta chapa ka samanya stara mana jata hai, kintu isake 140/90 hone para bhi ise samanya hi mana jata hai| 
rakta chapa jo ki 140/90 aura 160/95 ke bicha ke stara ka hota hai, use simarekha ki taraha mana jata hai| 
160/96 se lekara 180/144 taka ka raktachapa, madhyama atiraktachapa ki taraha vargikrita kiya jata hai, jabaki 180/115 ko gambhirata ki drishti se dekha jata hai| 
badhe hue anushithilita dabava ko, badhe hue prakunchaka dabava ki apeksha adhika gambhira mana jata hai, kyonki isaka gambhira dirghakalika prabhava hota hai| 
halke aura madhyama darje ka uchcha raktachapa ho sakata hai varshom taka koi lakshana na dikhaye| 
pahala lakshana subaha ke samaya tahalane ke daurana, sira aura gardana ke pichhale hisse mem darda ke rupa mem ubhara sakata hai, kintu yaha bhi jaldi hi gayaba ho jata hai| 
uchcha raktachapa ke kuchha aura anya samanya lakshana haim- chakkara ana, bahom, kandhom mem darda, paira aura pitha ke hisse mem pida़a aura darda hona, dhada़kana ka teja hona, sine mem darda hona, jaldi-jaldi mutratyaga karana, naka se rakta ka bahana, tantrakiya chinta aura thakavata ka hona, chida़chida़apana, bhavanatmaka asthirata, thakavata aura anidra ka hona| 
eka vyakti jo uchcha raktachapa se pidita hai, vaha koi bhi kathina karya nahim kara sakata hai va thaka hua sa mahasusa karata hai, aura hamesha hi asvasthya ya khinna sa rahata hai| 
vaha shvasa lene mem dikkata mahasusa kara sakata hai, aura apachana se bhi pida़ita ho sakata hai|
yadi uchcha raktachapa ka upachara nahim kiya jae, to yaha hridayaghata ya daura aura kisi anya taraha ki vikalangata, jaise retina ki nishkriyata, ka bhi karana bana sakata hai| 
tanava aura jivana jine ki galata shaili, uchcha raktachapa ka sabase pramukha karana hai| 
loga jo aksara tanava mem rahate haim, uchcha raktachapa se pidita hote haim, visheshatah jaba ve tanava mem hote haim| 
yadi tanava eka lambe samaya ke lie raha jata hai, to ho sakata hai ki tanava ke ghatane ke bada bhi, raktachapa nahimm ghate aura yaha bhi ho sakata hai ki vaha sthayi ho jae| 
eka aniyamita jivana shaili, dhunrapana aura vishaktata vale padarthom, chaya, kaॉphi, kola peya aura parishkrita khadya padarthom ka atyadhika sevana, jivana ki svabhavika gati ko nashta kara sakate haim| 
sharira se vyartha aura vishakta padarthom ke bahishkarana mem rukavata ane se dhamani aura shiraem dhimi ya susta pada jati haim| 
dhamaniyom ka kathora hona, motapa, madhumeha aura gambhira kabja bhi uchcha raktachapa ka karana bana sakate haim| 
uchcha raktachapa ke anya karana haim-darda nivaraka goliyom ka atyadhika prayoga karana, adhika namaka ka sevana, khana na khana aura adhika vasa yukta ahara ka sevana karana, tatvom ki kami vale ahara ka sevana karana aura avashyaka poshaka tatvom ki kami vale parishkrita khadya padarthom ka sevana karana| 
davaiyam karana ko dura nahim karati haim, aura na hi sthiti mem sudhara lati haim| 
bina kisi apavada ke, uchcha raktachapa ki pratyeka dava, vishakta hoti haim aura duhkhada prabhava vali hoti haim| 
uchcha raktachapa ko thika karane ka sabase surakshita upaya hai, sharira se visha ko nashta karana, jo ki isaka karana hai| 
uchcharaktachapa se pida़ita vyakti ko hamesha, niyamita ahara, vyayama aura vishrama ke niyama ka palana karana chahie| 
ahara ka sarvopari mahatva hota hai| 
kisi bhi anya khadya padartha ki apeksha mansa aura ande rakta chapa ko adhika badha़ate haim| 
eka paramparika ahara ki apeksha taje phalom aura sabjiyom se yukta eka prakritika ahara sharira se visha ko bahara nikalane mem kahim adhika karagara siddha hota hai| 
uchcha raktachapa se pida़ita vyakti ko svasthya hone ki prakriya mem kama se kama eka haphte taka purnarupa se phalom para adharita ahara ko apanana chahie aura eka dina mem tina bara pratyeka pancha ghante ke antarala mem phalom ka sevana karana chahie| 
use rasadara phala jaise-santare, seva, nashapati, ama, amaruda, anannasa aura angura ka sevana karana chahie| 
eka haphte taka phalom para adharita ahara ko grahana karane ke bada dudha ka sevana kiya ja sakata hai| 
dudha taja va kevala eka bara ubala hona chahie| 
do haphte ke bada marija ko anaja khane ki svikriti di ja sakati hai| 
uchcha raktachapa ke marija ke lie sabjiyam bhi labhadayaka sabita hoti haim| 
sabjiyom ko jyadatara kachcha khana chahie| 
yadi unhem pakaya ja raha haim, to isa bata ka dhyana rakhana chahie ki pakane ki prakriya ke daurana unake prakritika guna nashta na ho| 
sabjiyam jaise-khira, gajara, tamatara, pyaja, muli, gobhi aura palaka ko kachcha khana hi uttama hota hai| 
unhem chhote tukada़om mem kata ja sakata hai, aura use aura bhi svadishta banane ke lie usamem thoda़a sa namaka aura nibu ka rasa milaya ja sakata hai| 
namaka ka prayoga varjita hona chahie; kisi bhi halata mem, ise eka dina mem chara grama ya adhe chhote chammacha se jyada nahimm lena chahie| 
sodiyama karboneta vale benkinga paudara se bhi paraheja karana chahie| 
lahasuna ko raktachapa ghatane vali eka pramukha prabhavashali aushadhi mana gaya hai| 
isaki, chhoti dhamaniyom ki ainthana mem arama pahunchane ki shakti ke karana dabava aura chinta ghata jati haim| 
chakkara ke lakshanom, thithurahata aura shvasa lene mem takalipha tatha pachana tantra mem banane vali gaisa se arama pahunchane ke alava lahasuna dhada़kanom ko dhima kara, hridaya ki gati mem parivartana bhi karati hai|
badha़e hue rakta chapa kama karane ke lie dina mem do se tina kaipasula ki ausatana khuraka leni chahie| 
adhunika shodhom ne ahariya kailashiyama, poteshiyama aura uchcha raktachapa ke bicha eka mahatvapurna sambandha pradarshita kiya hai| 
shodhakartaom ne paya hai, ki aise loga jo poteshiyama-prachura/sampurna ahara ka sevana karate haim, unamem grahana kie jane vale namaka ki matra mem niyantrana na karane ke bavajuda bhi, uchcha raktachapa hone ke kama asara pae jate haim| 
unhonem yaha bhi paya hai, ki uchcha raktachapa se pida़ita vyakti, deri utpadom ke rupa mem kailashiyama ka adhika sevana nahimm karate haim| 
ye do avashyaka poshaka tatva sharira se adhika sodiyama ko bahara nikala phenkane mem sahayata karate haim aura aisi mahatvapurna kriyaom mem shamila hote haim, jo vahikiya pranali ki prakriya ka niyantrana karati hai| poteshiyama phala aura sabjiyom mem prachura matra mem paya jata hai aura kailashiyama dudha ke utpadom mem paya jata haim| 
uchcha raktachapa se pida़ita marija ko eka achchhe santulita ahara ka anupalana karana chahie, jisamem khadya padarthom ke avayava lagabhaga ninnalikhita anupatom mem ho: karbohaidreta bisa pratishata, protina dasa se pandraha pratishata, vasa pancha pratishata, aura phala tatha sabjiyam satha se painsatha pratishata| 
isa yojana ke tahada, eka mukhya ahara ko kachche khadya padarthom ka hona chahie, jaba ki dusare mukhya ahara mem pake hue khadya padartha shamila ho sakate haim| 
ahara ko dhire-dhire aura eka aramadayaka vatavarana mem lena chahie| 
pachana kriya arambha hone ke samaya se hi, khadya padartha ko munha mem achchhi taraha chabana chahie| 
rata ka bhojana samanyatah dera se nahim grahana karana chahie| 
ninna rakta-sharkara ya haipoglaikemiya, ise agara eka vyavasthita chikitsakiya tathya mem kaha jae, to yaha rakta sharkara upapachaya***** mem hone vali gada़bada़i hai| 
yaha eka aisi sthiti hai, jisamem agnashaya adhika matra mem madhusudani utpadita karane lagata hai, jo rakta sharkara ke kama hone ka karana banata hai| 
yaha bada mem kabhi madhumeha ke rupa mem parinamita ho sakata hai| 
kabhi-kabhi svasthya logom mem bhi karbohaidreta ki prachurata vale ahara ke sevana ke kuchha ghante bada, visheshatah peshiya thakavata ke bada,hapoglaekemiya dikhane lagata hai| 
yaha jivana ke kuchha shuruati dinom mem jyada paya jata hai, visheshatah samaya se pahale hue shishuom mem| 
hapoglaekemiya eka gambhira gada़bada़i hai, kyonki isamem raktasharkara ke stara ke bahuta adhika ghata jane para, dimaka achchhi taraha se karya nahim kara pata hai| 
sharira ke anya angom ki taraha hi, dimaka bhi apana indhana ahara se hi prapta karata hai| 
kintu yaha kevala sharira ke dvara karbohaidreta se utpadita sharkara ka hi prayoga kara pata hai| 
sharira ke anya utakom ki taraha yaha bhi apana indhana sangrahita nahimm kara sakata hai| 
atah isake lie rakta vahikaom ke madhyama se sharkara ki niyamita apurti avashyaka hai| 
ninna rakta sharkara stara ke karana upaji manasika gada़bada़i, kisi bhi vyakti ke jivana ko gambhira rupa se prabhavita kara sakati hai, aura hapoglaekemiya ka bara bara praghata, vyakti ki manasika sthiti mem sthayi parivartana la sakata hai| 
mithaiyom ke lie lalasa aura ahara ke bicha mem adhika matra mem starcha ka sevana karana, rakta sharkara ke ninna stara ka pahala lakshana hai| 
jaba rakta sharkara ka stara samanya se kahi kama ho jata hai, taba atispandana, adhika pasine ka ana, ghabarahata hona, pareshani mahasusa karana, thakavata hona, udasi hona, drishti dosha aura sira mem darda adi jaise lakshana prakata hote haim| 
anya lakshanom mem shamila haim-kampana, thithurahata, asthira manasika sthiti, chakkara aura kisi prakara ki yauna gada़bada़i | 
jyadatara marija bhukha mahasusa karate haim, aura kamajori va bhavanatmaka chida़chida़ahata se nijata pane ke lie jaldi-jaldi ahara ka sevana karate haim| 
yadi ve kai ghanto taka ahara grahana nahimm kara pate haim to ve chintita mahasusa karate haim| 
isaka akramana ya arambha achanaka hota hai, aura marija jaldi hi bhramita va phuhaड़ ho sakata hai| 
yadi sthiti ka upachara uchita dhanga se na kiya jae, to marija behosha ya bahuta nidralu ho sakata hai| 
rakta sharkara ke stara ke ghatane ka mukhya karana, parishkrita khadya padarthom, chini aura chini yukta khadya padarthom ke satha-satha tarala peya, kola peya aura kaॉphi ka adhika sevana karana hai| 
ye tatva agnashaya, adhivrikka granthiyom aura yakrita ki chini ka uchita nipatara karane ki kshamata mem ayogyata late haim| 
inasulina ya isi taraha ki kisi anya dava ke prayoga se madhumeha ke upachara ke samaya karbohaidreta ki paryapta matra mem apurti ke bina hi haipoglaikemiya ka aghata avakshepita ho sakata hai| 
rakta sharkara ke kama hone ke anya karana haim-insulina sravita karane vali agnashaya ki dvipaka koshika mem arbuda ka hona, yakrita, piyusha granthi ya adhivrikka granthi ki kriya pranali mem gada़bada़i hona| 
bane hue tanava ki sthiti, adhivrikka granthi ko kamajora banati hai aura draksha-sharkara ke avashoshana ke galata dhanga ko arambha karati hai| 
uchcha jaiva protina vala ahara jo samanyatah haipoglaekemiya ke lie sujaya jata hai, isa taraha ki gada़bada़i mem anukula nahim hai| 
ho sakata hai ki yaha asthayi rupa se sthiti para niyantrana pa lem, kintu yaha kai anya lihaja mem hanikaraka hai| 
uchcha protina vale ahara ka lagatara sevana karane se kai anya bimariyam, jaise-hridaya mem pareshani, gathiya, gurde ki samasya aura kainsara bhi janma le sakati hai| 
haipoglaekemiya ke lie nirdharita kiya gaya ahara mukhya rupa se tina mulabhuta khadya samuhom para nirbhara haim-kramashah anaja, bija aura giridara phala, sabjiyam aura phala jo dudha ke anupuraka ho, dugdha-utpada aura vanaspati tela| 
bija, giridara phala aura anaja isa ahara ke mukhya sanghataka haim| 
bija aura giridara phalom ko kachcha hi khana chahie| 
danom ko anaja ke rupa mem pakana chahie| 
paka hua anaja dhire-dhire pachata hai, aura yaha ahara lene ke chhah se atha ghantom ke bada dhire-dhire poshaka tatva aura sharkara ko rakta pravaha mem karata hai| 
yaha rakta sharkara ke stara ko samanya aura lambe samaya ke lie sthayi rakhega| 
ninna rakta sharkara se pida़ita vyakti ko, dina mem do ya tina bara eka satha jyada ahara lene ke bajae, chhah se atha bara thoda़a-thoda़a ahara lena chahie| 
ahara ke bicha mem kachche phalom aura bijom jaise kaddu ya suryamukhi ke bija athava dudha, chhachha ya phalom ke rasa ka sevana, bahuta hi jyada labhadayaka hoga| 
sansadhita aura parishkrita khadya padartha, shakkara, maida aura unake saha-utpada, sabhi ko ahara se purnarupa se hata dena chahie| 
kaॉphi, sharaba aura shitala peyom se bhi paraheja karana chahie| 
namaka ke prayoga mem kami lani chahie, kyonki namaka ka adhika prayoga, rakta mem poteshiyama ki kami lata hai, jo rakta sharkara ke kama hone ka karana banata hai| 
phlu ke nama se bhi jana jane vali eka roga vishayaka sthiti , jo imphluenaja vishanu ke sankramana ke karana upajati hai| 
imphluenja vishanu ka mukhya prabhava upari shvasana nali, naka aura gale taka hota hai| jisame sambhavatah phephada़e aura shvasana nalika bhi shamila hoti hai| 
yaha bahuta adhika sankramaka hai aura kisi bhi samaya para, janasankhya ke eka bada़e hisse para mahamariya prabhava dalane ki kshamata rakhata hai| 
yadyapi yaha bimari aksara thanda़ ke mahine mem najara ati hai, phira bhi yaha kisi bhi samaya upaja sakati hai| 
yaha aksara thanda़, bukhara, sira darda aura gambhira peshiya darda ke satha arambha hota hai| 
marija dukhi aura kamajora sa mahasusa karata hai| 
naka aura gale mem jalana va sujana hoti hai, jo shvasa nali se hote hue phephada़om taka bhi phaila sakati hai, aura jisake parinamasvarupa gale mem darda va kapha ho sakata hai tatha naka aura ankha bahati hui ho sakati hai| 
imphluenja ki madhyama sthiti mem, tapamana 102 pheranahaita taka ho sakata hai aura do se tina ghante taka rahata hai| 
gambhira sthiti mem yaha 104 pherenahaita taka pahuncha sakata hai aura chara se pancha dina taka rahata hai| 
kamajori aura thakavata kai haphtom taka raha sakati hai| 
isamem shvasa nali mem takalipha hone ke karana, sine mem atyadhika kapha ho sakata hai| 
bachchom mem yaha bimari maroda़na se shuru ho sakati hai, aura bukhara mem 105 phe. se lekara 106 phe. taka ki vriddhi ho sakati hai| 
marija atyadhika kamajora mahasusa karata hai| 
imphluenja, eka jivanu bimari ki taraha jani jati hai| 
samanyatah aisa mana jata hai, ki yaha prathamika taura para jivanuom ki kriyaom ke karana utpanna hota hai, kintu prabhavita vyakti mem yaha tantra ki vishaili va kharaba dasha ke karana vikasita hota hai| 
yaha sthiti ahara mem honem vali gada़bada़iyom aura jivana jine ki galata shaili, jaise chinta, adhika kama, uchita vyayama ki kami, dama-ghontu kamare mem rahana aura dera rata taka jagana jaise karanom se utpanna hoti hai| 
sahi mayanom mem svasthya rahane vale vyakti ki sharira mem, koi bhi jivanu na hi apana dera jama sakata hai aura na hi vikasita ho sakata hai| 
imphluenja se pida़ita vyakti se yaha aise vyaktiyom mem jinaki sharirika sthiti kamajora ho asani se pahuncha jata hai| 
aise hi eka mahamari ki shuruata hoti hai|
inpha़luenja़a, anya sabhi tivra rogom ki taraha, sva-shuddhi ka eka svabhavika prayasa hai aura, yadi prakritika upayom se sahi tarike se upachara kiya jaya, to jaham taka ki rogi ke bhavi svasthya ka sambandha hai, bahuta achchhe parinama ho sakate haim|
adhunika chikitsiya paddhatiyom se roga ka upachara nimoniya, vrikka vikarom, kana tatha sine ki pareshaniyom adi jaise lagabhaga kai jatilataom ko la sakata hai|
yaha upachara ki shamanatmaka prakriti ke karana hota hai jo shuddhi prakriya ko rokati hai aura vishaile dravyom ko sharira mem punah adhika gaharai taka dhakelati hai|
inpha़luenja़a ki tivra avastha mem, rogi ko sabhi thosa aharom se paraheja karana chahie aura kevala phala tatha sabji ke rasom ko roga ki gambhirata para nirbhara karate hue, pahale tina se pancha dinom taka, 50:50 ke anupata mem pani ke satha milakara pina chahie|
isa rasa upavasa ko tapamana ke utarakara samanya hone taka jari rakhana chahie|
isa avadhi mem anta sapha karane ke lie gunagune pani ka enima pratidina lena chahie|
jvara utarane ke bada, yaha rogi do ya tina dinom ke lie kevala phalahara apana sakata hai|
isa ahara niyama mem, rogi ko seba, nashapati, angura, santare, anannasa, ada़u tatha melana jaise taje rasadara phalom ka pancha ghante ke antarala para dina mem tina bara bhojana karana chahie|
kele, ya sukhe, damapukha़ta ya dibbabanda phala, phira bhi, nahim lie jane chahie|
isa phalahara mem anya khadya padartha nahim milana chahie; nahim to upachara ka mahattva nashta ho jaega|
isaka anusarana agale do ya tina dinom taka phala tatha dugdhahara se kiya ja sakata hai|
isake bada, rogi svasthya evam roga mem ahara ke 1 adhyaya mem darshae anusara tina mula bhojana vargom, jaise ki, danom, kashthaphalom tatha dhanyom, sabjiyom aura phalom ka eka susantulita ahara apana sakata hai|
masale aura baghara, tatha achara, jo bhojana ko aura adhika svadishta banate haim tatha adhika khane ke lie badhya karate haim, se avashya dura rahana chahie|
salada dresinga mem nibu ke rasa ka prayoga kiya ja sakata hai|
inpha़luenja़a ke upachara mem gajara evam palaka ke rasa vishesha rupa se labhaprada pae gae haim|
ina donom sabjiyom ke rasom ko alaga-alaga ya chhah aunsa palaka ke rasa ko dasa aunsa gajara ke rasa ke satha milakara sanyukta rupa se liya ja sakata hai|
sabjiyom ko, phira bhi, rasa nikalane se purva, kitanashaka ke chhida़kavom adi ke avashesha ko hatane ke lie bahuta sare thande, bahate pani se achchhi taraha se dhona chahie|
jvara ke krama ke daurana, tapamana ghatane ka prakritika upaya hai shitala paikom ke sadhana|
yaha uchita hai ki pradahayukta kantha hone para gale mem eka paika ke satha, eka dina mem kai bara sharirika paika ka prayoga karem|
yaha paika chadara ya anya linena kapada़om ke chaukora tukada़e ko thande pani mem nichoda़kara, ise rogi ke sharira tatha pairom ke thika charom tarapha़ lapetakara aura taba eka kambala se purnatah dhankakara banaya jata hai|
gale ke paika ke mamale mem, linena ko eka pha़lalena ke tukada़e se dhanka ja sakata hai|
isa paika ko ekadha ghante ke lie rakha ja sakata hai|
isa paika ko hatane ke bada sharira ko gunagune pani ke satha spanja se sapha़ karana chahie|
isa rogi ko bistara para hi rakhana chahie aura use vahim rahana chahie jaba taka ki vaha phira se achchha nahim ho jata|
insonniya shabda nidra ke purna abhava ka yathashabda dyotaka hai|
yaha, halanki, nidra ki sapeksha asamarthata ko suchita karane ke lie prayukta ki jati hai jisamem ninda ane mem kathinai, ninda lage rahane mem kathinai, jaldi jagane ya ina takalipha़om mem se kisi ka bhi sanyoga shamila hota hai|
anidra roga vyakti ko manasika vishrama se vanchita karata hai aura phalasvarupa dina ke samaya mem usake kriya-kalapom mem badha dalata hai|
jaba yaha eka adata bana jati hai to yaha eka gambhira svasthya sankata khada़a karati hai|
nidra sharira ke lie vishrama ki eka niyatakalika avastha hai jo ki isake sakshama karya ke lie purnatah avashyaka hai|
karya-shakti ke svasthyalabha ke lie yaha anivarya sthiti hai|
yaha tanava se mukti deti hai, mastishka aura sharira ko vishrama deti hai tatha vyakti subaha mem taja़a aura tanavarahita hokara jagata hai|
nidra ki yaha matra eka vyakti se dusare vyakti ke bicha bahuta vistrita sima mem badalati rahati hai|
ama taura para, hara rata sata se atha ghante ki nidra adhikatara logom ke lie paryapta hoti hai|
kuchha, halanki, chara se pancha ghante ke satha bhi achchha karate haim kyonki unaki nidra adhika gahari aura adhika sphurtidayaka hoti hai|
anidra roga vibhinna karanom se vayovriddha ke bicha samanya hai|
vayovriddha ki nidra rata ke samaya alpakalina anidra dvara prayah pratibadhita hoti hai|
aise mamalom mem yaha matra ki apeksha gunavatta hai jo sabase adhika prabhavita hoti hai|
unra ke satha, gahari nidra ka avadhi mem dhire-dhire kami hoti hai|
unra badha़ne ke satha nidra ki avashyakata bhi kama ho jati hai|
12 varsha ki ayu mem hara rata nau ghante ki nidra se lekara nidra ki ausata avashyakataem 20 varsha ki ayu mem atha ghante, 40 mem sata ghante, 60 mem 6 1/2 ghante aura 80 mem 6 ghante taka ghata jati haim|
yadyapi ninda lage rahane mem kathinai sabase samanya prakara ka anidra roga hai, tathapi ekamatra lakshana jo aksara anidra roga ke akramana ko nishchita karata hai vaha hai ninda ane mem kathinai|
anya lakshana haim bhavatmaka asthirata, samanvayana ki kami, bhranti aura udasinata ki eka sthira bhavana|
anidra ka sabase samanya karana vyagrata, chinta, atishrama tatha atiuttejana dvara utpanna manasika tanava hai|
vidvesha, krodha aura katuta ki dabi bhavanaom se bhi anidra roga ho sakata hai|
koshthabaddhata, apacha, ratri mem adhika khana, shvetasarom, mithaiyom, chaya ya kaॉpha़i ka atyadhika sevana aura bhukhepeta sona inake anya karanom mem se haim|
dhunrapana anidra roga ka eka anya asandehaspada karana hai kyonki yaha tantrika tantra mem jalana paida karati hai, visheshakara pachana tantra ki tantrikaom mem|
paryavarana karaka jaise ati sankulata, atyadhika shora aura ghatiya bistara ya ghara ki paristhitiyam bhi asthayi nidra kathinaiyom ki ora le ja sakate haim|
aksara, ninda na ane ke bare mem chinta hi kisi ko jagrita rakhane ke lie paryapta hai|
ninda ki goliyam anidra ka upachara nahim haim|
ye jaba lagatara li jati haim to adata banane vali aura kama karagara hoti haim|
ninda ki goliyom ke parshva prabhavom mem apacha, tvacha para phunsi, sankramaka roga ke prati ghati hui pratirodhakshamata, parisanchari aura shvasa-sambandhi samasyaem, bhukha ki kami, uchcha raktachapa, vrikka tatha yakrita ki samasyaem aura manasika bhranti sammilita haim|
anidra roga ke upachara mem ahara eka mahattvapurna karaka hai|
shodhakarya darshata hai ki jirna anidra roga se pida़ita logom mem lagabhaga nirapavada rupa se bi-kampleksa vitaminom aura vitamina si tatha bi, aura kailshiyama, maignishiyama, maiganija़, poteshiyama tatha jinka jaise pramukha poshaka tatvom ki ullekhaniya kami hoti hai|
nidra prakriya nipunata se karya karane mem asamartha hoti hai jaba taka ki ahara mem ina poshaka tatvom mem se pratyeka paryapta matra mem upasthita na hom|
vibhinna khadya tatvom mem se, thaimina ya vitamina bi1 ka vishesha mahattva hai|
yaha maja़buta, svastha tantrikaom ke lie atyavashyaka hai|
kisi sharira ko eka lambe samaya taka thaimina nahim diya gaya ho to vaha svabhavika nidra lane ke lie paryapta vishrama karane mem asamartha hoga|
isa vitamina ke bahumulya srota haim sampurna anaja ke siriyala, dala aura kashthaphala|
anidra roga ke upachara mem khana-pana ke paitarna mem samanya sudhara ke satha eka santulita ahara bahuta hada taka asaradara hoga|
aise ahara mem maide ke utpada, shakkara aura isake utpada, chaya, kaॉpha़i, chaॉkaleta, kola peya, madira, charbidara khadya, tale hue khadya, sanyojaka, parirakshaka, ranjaka, svadishtakara, lavana ka atyadhika prayoga aura teja baghara yukta khadya nahim hone chahie|
rogi upachara ke prarambha mem tina ya chara dinom ke lie phalahara apana sakata hai|
isa ahara niyama mem, use santare, angura, seba, nashapati, ada़u aura anannasa jaise taje rasadara phalom ka dina mem tina bara bhojana karana chahie|
yaha rakta pravaha ko sapha़ karane aura sambhavya pachaka tatha antra-sambandhi gada़bada़iyom ko dura karane mem sahayata karega| 
isa kala mem antom ko gunagune pani ke enima se pratidina sapha़ karani chahie|
phalahara ke pashchat, rogi eka parivartita khana-pana ke paitarna ka anusarana kara sakata hai jisamem subaha ke nashte mem taje aura sukhe phala, dane tatha dahi hone chahie|
do pramukha bhojanom mem se, eka mem eka bada़i mishrita salada sammilita honi chahie aura dusara protina adharita hona chahie|
sote samaya shahada se mitha kiya gaya eka kapa dudha labhadayaka hota hai kyonki dudha mem sammilita amino esida triptophena nidra lata hai|
anidra roga ke upachara mem ajavaina ko bhi labhaprada mana jata hai|
rata mem dera se bhojana karana aksara anidra lati hai|
atah, anidra roga se pida़ita ko, sone se kama se kama tina ghante purva apana antima bhojana khana chahie|
jaba koi bhavukatavasha ashanta ya thakana athava tivra avasada se pida़ita ho to use kabhi bhi bhojana nahim karana chahie kyonki yaha jathariya piड़a mem parinita ho jaega|
ve jo anidra roga se pida़ita haim unhem hamesha eka aramadeha vatavarana mem bhojana karana chahie|
yaha dina ke kisi bhi samaya para mahattvapurna hai, kintu adhika vishesha rupa se rata mem|
samasya para ka़abu pane ke lie ahariya upachara ke satha-satha, anya prabhavakari upayom ko bhi apanana chahie| 
inamem ridha़ ki haddi para garama senka ka anuprayoga, garama pani mem paira snana ya bari-bari se garama aura thande pani mem paira snana tatha sone ke samaya lamba tatastha nimajjana snana (92o se 96o pheranahaita) sammilita haim|
jitana sambhava ho sake utane tanava karakom ko dura karane ke lie sabhi prayasa bhi karane chahie|
isa disha mem uthaye gae kadamom mem tanava mukti ke kisi bhi upaya ka niyamita abhyasa ya dhyana takanika, dhire se kama karane ki kala vikasita karana, kha़asakara khane, chalane aura bolane jaisi kriya-kalapom ki, dina ka karya samaya nau ya dasa ghante taka simita karana, tatha saptaha mem 5 1/2 dina, srijanatmaka shauka vikasita karana aura haradina kuchha samaya isa para vyatita karana shamila hona chahie|
sabhi yakrita vikarom mem piliya sabase samanya hai|
yaha eka avastha hai jisamem tvacha ka pila vivarnana ho jata hai aura rakta mem pitta rangadravya, bilirubina, ki vriddhi ke karana shleshma jilliyam payi jati haim|
yaha pitta, yakrita dvara utpanna kiya jata hai, aura eka atyavashyaka pachaka rasa hai jo uchita poshana ke lie anivarya hai|
yaha pachana ki samanya prakriya para sabase anukula prabhava dalata hai|
yaha bhojana mem hone vale vikara parivartanom se bhi bachata hai|
yadi pitta ko anta mem pravesha karane se roka diya jata hai to gaisa aura anya utpadom mem vriddhi ho jati hai|
ama taura para, pitta ka utpadana aura isaka pravaha anavarata hota hai|
piliya ke lakshana haim atyadhika durbalata, siradarda, jvara, bhukha ki kami, anavashyaka thakana, gambhira koshthabaddhata, michali aura ankhom, jibha, tvacha tatha mutra ki pili rangata|
rogi yakrita kshetra mem halki pida़a bhi anubhava kara sakata hai|
pratirodhatmaka piliya atyadhika khujalahata se sambaddha ho sakata hai|
piliya yakrita ki kharabi ka parichayaka hai|
yaha pitta vahikaom ke avarodha se ho sakata hai jo pitta lavanom tatha rangadravyom ko anto mem nissarita karati haim| 
pitta taba rakta ke satha mila jata hai aura yaha tvacha ko pili rangata deta hai|
pitta vahikaom ke ye avarodha yakrita ke pitta-pathariyom ya sujana aura jalana ke karana ho sakate haim, jo eka vishanu se hone vale yakrita-shotha (hipetaitisa) ke rupa mem jana jata hai|
bada ke mamale mem, yaha vishanu phailata hai aura ati sankulata, gande pasa-paड़osa, asvasthyakara paristhitiyam tatha bhojana aura pani ke dushana ke karana mahamari ki ora le ja sakata hai|
piliya ke anya karana haim ghataka raktalpata aura yakrita ko prabhavita karane vale vishesha roga jaise taiphaida, maleriya, pita-jvara tatha kshayaroga|
sadharana prakara ke piliya ka ahara upachara tatha vyayamom dvara teja़i se upachara kiya ja sakata hai|
gambhira mamalom mem, jo pitta vahikaom mem avarodha ya dabava ke karana hue haim, svathya labha, phira bhi, dhimi gati se hoga|
rogi ko roga ke tivra lakshanom ke ghatane taka vishrama karana chahie|
rogi ko eka saptaha ke lie rasa upavasa para rakhana chahie|
santare, nibu, angura, nashapati, gajara, chukandara tatha ganne ke rasom ka sevana kiya ja sakata hai|
isa avadhi mem, niyamita antra malatyaga ko sunishchita karane ke lie, tatha phalasvarupa rakta pravaha mem sada़e hui, vishakta dravyom ke avashoshana ko rokane ke lie pratidina garama pani ka enima lena chahie|
tivra lakshanom ke ghatate taka rasa upavasa jari rakhana chahie|
rasa upavasa ke bada, rogi agale tina se pancha dinom taka phalahara apana sakata hai|
isa ahara niyama mem, use kele ko chhoda़kara seba, nashapati, angura, santare tatha anannasa jaise taje rasadara phalom ka dina mem tina bara bhojana karana chahie|
sabhi vasa jaise ghi, makkhana, malai tatha telom se kama se kama do saptaha ke lie avashya paraheja karana chahie, tatha isake bada ahara mem makkhana aura jaituna ka tela sammilita kiya ja sakata hai parantu unaka kama se kama upabhoga karana chahie|
sabjiyom tatha phalom se prapta sarvashreshtha, vasarahita, eka halka karbohaidreta ahara lena chahie|
rogi ko taji sabjiyom tatha phalom ke rasom ka prachura matra mem sevana karana chahie|
pratidina ki kachchi shakahari salada mem dugdhatikta ki pattiyom, pattiyom sahita muli aura endaiva ko milana chahie| 
kachche seba tatha nashapati vishesha rupa se labhaprada hote haim|
dina ke samaya kai bara piya gaya, jau ka pani, piliya ke lie achchha upachara mana jata hai|
eka kapa jau ko sava do litara pani mem ubalana chahie aura dhimi ancha para tina ghante taka pakana chahie|
pachana gada़bada़iyom se avashya bachana chahie|
ahara mem dala aura phaliyom ki taraha nichali antom mem uttejita hone ya sada़ne ki pravriti vale khadyom ko sammilita nahim karana chahie|
nibu rasa ke satha bahuta sara pani pine se kshatigrasta yakrita koshikaem surakshita rahengi|
piliya ka rogi upara ke ahara niyama dvara bahuta asani se paristhitiyom para kabu pa sakata hai aura apane bimara yakrita ko svastha bana sakata hai jaba taka ki vaha eka bara phira samanya rupa se kama na karane lage|
ahara tatha jivana shaili ki uchita dekhabhala, niyamita, niyantrita vyayama tatha dhupa, taja़i hava ki barambara anashrayata aura paryapta vishrama ke satha yakrita pida़a ki punaravritti se bacha ja sakata hai|
shodhakarya darshate haim ki yakrita mem svayam ko punarutpanna karane ki utkrishta kshamata hoti hai basharte ki sabhi avashyaka poshaka tatvom ki paryapta matra mem apurti ki gai ho|
protina, vitamina si, aura bi vitamina, vishesha rupa se kolina, tatha vitamina i se sampurna uchcha ahara isake punarutpadana ko jaldi kara sakati hai|
svathya labha ke bada bhi, pida़a ki punaravritti ko rokane ke lie eka lambi avadhi taka isa ahara ko banae rakhana avashyaka hai|
yakrita ka sirosisa koshikaom ki ullekhaniya kami dvara visheshita yakrita rogom ke sabhi prakarom ki ora ishara karate haim|
yaha yakrita sambandhi sabase gambhira rogom mem se eka hai|
yakrita dhire-dhire akara mem sankuchita hota jata hai aura kada़a tatha chamada़e ke samana ho jata hai|
yakrita sharira ke sabase mahatvapurna granthiya angom mem se eka hai|
yaha madhyapata ke thika niche udara ke dahine bhaga mem upara ki ora sthita hota hai|
yaha eka vishala rasayanika prayogashala hai jo kai mahatvapurna karya karata hai|
yaha pitta, kolestraॉla, lesithina, rakta elbyumina jo ki utaka avasheshom ko hatane ke lie atyavashyaka hai, prothraॉmbina jo rakta ke thakka banane ke lie anivarya hai aura bahuta se enja़aimom ka nirmana karata hai|
yaha una harmonom ko nishkriya bana deta hai jinaki aba aura avashyakata nahim hoti hai, kai amino anlom ka sanshleshana karata hai jo utaka nirmana mem prayukta hote haim aura protinom ko jaba inaki urja ke lie jarurata hoti hai to sharkara tatha vasa mem vibhajita karata hai|
yaha vitaminom tatha khanija padarthom ka sanchaya karata hai|
yaha hanikaraka padarthom ko nashta karata hai aura dragom, vishom, rasayanom tatha jivanvika sankramanom se utpanna jivavishom ka vishaharana karata hai|
yakrita ki kshati ina sabhi karyom mem badha utpanna karati hai|
yakrita ke sirosisa mem, halanki punarutpadana ki kriya jari rahati hai, yakrita koshikaom ki kami koshika pratisthapana se adhika hoti hai|
vahikiya tantra mem bhi vikriti ata hai jo yakrita se hokara bahane vale portala rakta pravaha mem badha dalata hai|
teji se phailane vala yakrita ki sanrachana tatha karya ka yaha vikara anta mem yakrita sambandhi viphalata aura nrityu ki ora le ja sakata hai|
sirosisa ke kai prakarom mem sabase samanya hai portala sirosisa, jo henoka ke sirosisa ke rupa mem bhi jana jata hai|
isa roga ki prarambhika avastha mem, kabhi-kabhara michali tatha ulati sahita, gaisa aura apacha ke bara-bara akramana ke atirikta shayada kuchha aura adhika na ho|
thoda़i udara pida़a ya vajana mem kami ho sakati hai|
unnata avastha mem, rogi ko ninna shreni ka jvara ata hai|
usaki badabudara sansa, piliyagrasta tvacha aura udara mem phuli hui shiraem hoti haim|
chehare, gale, hathom tatha dhada़ para bala ki taraha, chhoti makada़iyom ke samana, lala chihna prakata ho sakate haim|
udara phula tatha suja jata hai, dimaga़ dhundhala ho jata hai aura peta se bahuta adhika raktastravana ho sakata hai| 
lambe samaya taka madira ka atyadhika prayoga yakrita ke sirosisa ka sabase prabala karana hai|
isaka akalana kiya gaya hai ki yunaiteda steta mem 12 mem se 1 dirghakalika madyavyasaniyom ko sirosisa hota hai|
yadi yaha vyakti madira se paraheja nahim karata hai, to yaha roga yakrita sambandhi viphalata ki antima-avastha ki ora agrasara ho sakata hai|
sirosisa madira grahana karane ki avadhi tatha usaki pratidina upabhukta matra se sambaddha pratita hoti hai|
hala ka shodhakarya yaha sanketa karata hai ki sirosisa utpanna karane ke lie madira grahana karane ki ausata avadhi 10 varsha hai aura madira ki matra pratidina 16 aunsa se adhika hona akalita ki gai hai|
sirosisa ke vikasa mem aparyapta poshana dusara utpadaka karaka ho sakata hai aura eka dirghakalika madyavyasani prayah gambhira kuposhana se bhi pida़ita rahata hai, kyonki vaha kabhi-kabhi hi bhojana karata hai|
sirosisa ke anya karana atyadhika masaledara bhojana ka bahuta adhika antargrahana, ushnadeshiya vatavarana mem lambi avadhi taka kvinaina ka adatana sevana aura upadansha, jvara tatha anya rogom ke lie aushadhi upachara haim|
sadharanatah yaha sharira ki uchcha vishakta sthiti ka bhi parinama ho sakata hai|
vastava mem, jo kuchha bhi yakrita ki koshikaom para lagatara atyadhika boja dalata hai aura unake antima vikara ki ora le jata hai, vaha sirosisa ka eka sahayaka karana ho sakata hai|
rogi ko bistara para hi rakhana chahie|
use kisi bhi prakara ke madira se purnatah paraheja karana chahie|
use sata dinom ke rasa upavasa ke satha prarambhika yakrita svachchhata karyakrama se gujarana chahie|
isa avadhi mem lala chukandara, nibu, papite aura angura se taje-nikale rasom ka sevana kara sakata hai|
do ya tina saptaha taka phala tatha dugdhahara dvara isaka anusarana kara sakata hai|
isa ahara niyama mem, rogi ko taje rasadara phalom aura dudha ka eka dina mem tina bara bhojana karana chahie|
phalom mem seba, nashapati, angura, chhote chakotare, santare, anannasa aura ada़u sammilita ho sakate haim|
pahale dina tina pava dudha le sakata hai|
ise pratidina pauna pava se dedha़ ya paune do kilo taka badha़ana chahie|
ise bahuta dhire-dhire ghutakana chahie|
phala aura dugdhahara ke pashchat, rogi ko jaivika rupa se ugae gae kachche khadyom para bala dete hue tina mulabhuta bhojanom ka susantulita ahara dhire-dhire arambha karana chahie|
sirosisa mem paryapta uchcha koti ki protina avashyaka hai|
yakrita ke rogiyom ke lie sabase utkrishta protina bakari ke kachche dudha, griha-nirmita aparishkrita panira, ankurita dane aura anaja tatha kachche kashthaphalom, visheshakara badama se prapta hote haim|
isa sthiti mem chukandara, skvasha, karela, bhata, tamatara, gajara, muli aura papite jaisi sabjiyam labhadayaka hoti haim|
sabhi vasa aura telom ko kai saptaha taka ahara se nikala dena chahie|
rogi ko sabhi parishkrita, sansadhita tatha dibbabanda khadyom, kisi bhi rupa mem shakkara, masale tatha baghara, kada़i chaya aura kaॉpha़i, tale khadya, ghi mem pakae sabhi vyanjanom, tela ya makkhana tatha sabhi charbi sanriddha mansom se paraheja karana chahie|
namaka ka prayoga simita hona chahie|
rogi ko bhojana mem sabhi rasayanika sanyojiyom aura jaham taka ho sake hava, pani tatha vatavarana ke vishom se bhi dura rahana chahie|
upachara kala mem antom ki saphai karane ke lie gunagune pani ke enima ka prayoga karana chahie|
yadi koshthabaddhata jirna hai to isake unmulana ke lie sabhi upaya karane chahie|
bari-bari se yakrita kshetra ko dabane ke bada sadharana gili chadara se ragada़ne ka prayoga labhaprada hoga|
subaha mem shushka gharshana aura shvasana tatha anya vyayama, upachara ki niyamita dainika visheshata banani chahie|
adhakapari eka prachina aura vikata roga hai|
isane sija़ra, darvina tatha phrayada jaise pratishthita vyaktiyom ko pareshana kiya|
yaha rahana-sahana ki adhunika paristhitiyom mem bhayanaka anupata mem hua hai aura aba vishva ki janasankhya ke lagabhaga 10 pratishata ko piड़ita karati hai aisa mana jata hai|
adhakapari ko bahuta adhika siradarda ke satha, aksara sira ke eka tarapha aura pachana, yakrita tatha drishti ki gada़bada़iyom se sambandhita avegi pida़a ke rupa mem bataya ja sakata hai|
yaha prayah taba hota hai jaba koi vyakti atyadhika manasika tanava mem hota hai ya achanaka usa avastha se chhutakara pa jata hai|
unake lie, sabakuchha usi samaya hona chahie aura jaba ve khatma karate haim; achanaka ve atyadhika manasika tanava ki avastha se bahuta adhika rahata ke ehasasa mem utara jate haim|
sira tatha gale ki mansapeshiyam, lagatara tanava ke prati pratikriya karati huim atishramita ho jati haim|
kasi hui mansapeshiyam dhamaniyom ko sankuchita kara deti haim aura rakta pravaha ko kama kara deti haim|
taba, jaba vaha vyakti achanaka vishrama karata hai, sankuchita mansapeshiyam, rakta-vahika ki divara ka vistara karati hui phaila jati haim|
hridaya ke hara dhada़kana ke satha, rakta ina vahikaom se hokara age badha़ta hai tatha tivra pida़a utpanna karate hue unhem aura adhika phailata hai|
adhakapari ke siradarda adhivrikka ke harmona stravana ke dainika avartana se sambandhita haim aisa mana jata hai|
adhivrikka ke harmona ka stara pratahkala mem apane shikhara para hota hai aura lagabhaga dopahara 3 baje eka ninna stara para pahunchate hue, sandhyakala mem dhire-dhire kama ho jata hai| ina harmonom ka vahikiya mansapeshi ke tana ko niyantrita karane aura vahikavisphara se bachane ka eka mahattvapurna karya hai|
jaba adhivrikka harmonom ka utpadana kama hota hai, to rakta vahikaem phailava ki ora adhika pravritta hoti haim|
isalie adhakapari ka siradarda prayah sandhyakala mem aura kabhi-kabhara pratahkala mem shuru hote haim|
adhakapari ka eka nishchita rupa hota hai|
sira ke eka tarapha़ hi isa pida़a ka prakopa hota hai aura prayah ankha se phailati hai|
eka akramana mem sira ka dahini tarapha prabhavita ho sakata hai aura dusare mem shayada banyi tarapha|
adhakapari ke akramana se pahale ama taura para thoda़e samaya ke lie udasi, chida़chida़apana aura bhukha ki kami hoti hai|
kuchha logom ko yaha daura roja़ana, anyom ko hara mahine athava hara dusare ya tisare mahine aura kaiyom ko kai salom mem sirpha़ eka ya do bara pada़ta hai|
adhakapari ko ubau siradarda ke rupa mem bhi jana jata hai|
isake mukhya lakshana dhamakadara pida़a, michali aura ulati haim|
sira ke prabhavita bhaga ki rakta vahikaem bahara nikali hui aura dhada़kati hui hongi|
adhakapari akramana karane se purva uchita chetavani deti hai|
yaha rogi apane samane prakasha ki chamaka ya kale dhabbe ya vastuom ke kevala bhaga dekhata hai|
vaha eka bhuja ya paira athava chehare ke eka bhaga mem samvedanashunyata ya durbalata anubhava kara sakata hai|
kabhi-kabhi yaha samvedanashunyata chehare ke donom tarapha aura jibha tatha vani ko bigada़ti aura kathina banati hui pure mukha ko prabhavita kara sakati hai|
jaise-jaise siradarda vikasita hota hai, pachana ki gada़bada़i eka spashta lakshana hota hai|
adhakapari ka eka tivra akramana bahuta thakau hota hai aura rogi ko bistara para arama karana pada़ sakata hai taki use jaham taka ho sake prakasha tatha shoragula se bahara nikala ja sake, kyonki donom hi pida़adayaka uttejana utpanna karati haim|
pura sira atisamvedanashila ho jata hai aura yaham taka ki thoda़i si bhi gatividhi atyadhika pida़a badha़ati hai| 
hara eka vyakti ke satha akramana ki avadhi bhinna hoti hai|
kuchha loga kevala kuchha ghantom mem hi gambhira lakshanom se nijata pa sakate haim| 
anya logom ko samanya sthiti mem ane ke lie eka dina ke arama ki jarurata hoti hai| 
aura bhi anya logom ko jo ki kama khushanasiba hai, unhem thika hone mem do, tina ya yaham taka ki chara dina ka samaya bhi laga sakata hai| 
adhasisi ke hone ke kai karana ho sakate haim, jaise-ninna rakta sharkara, elarji, sankramana, kuchha davaiyam jaise-vitamina e ka atyadhika prayoga karana, kamajora sharirika gathana, shakti ka abhava, poshaka tatvom ki kami, atyadhika karya, aniyamita nidra aura arama, atyadhika dhunrapana, madirapana aura kama kriyaem| 
rajodharma mem hone vale rasayanika harmona parivartanom ke prabhava ke karana, yaha bhi mahilaom mem adhasisi ka pramukha karana ho sakata hai| 
isa taraha se hone vala adhasisi, rajonivritti ke satha hi samapta ho jata hai| 
adhasisi ko utpanna karane ke bahuta se karana hote haim| 
sabase mahatvapurna hai, marija ke dvara aise khadya padarthom ka sevana karana jisase ho sakata hai, ki use elarji ho| 
una khadya padarthom mem shamila haim-chakaleta, panira aura anya deri utpada, sabhi taraha tale hue ahara, pyaja, tamatara, nibu vansha ke phalom ke satha-satha kaॉphi, chaya, nikotina aura sharabayukta peya, jo nasom ko sankuchita ya udasina karate haim aura rakta vahikaom ke akara mem parivartana late haim| 
anya prerakom mem shamila haim-atyadhika teja prakasha, ankhom mem tanava, utsaha, bhaya, jaldabaji, gussa, kathina parishrama ke bada aprasannata aura udasi| 
darda nivaraka davaiyam asthayi arama de sakati haim, kintu ye karana ki samapti nahim karati haim| 
marija jaldi hi teja davaom ka adi ho jata hai, jo samaya ke satha apana asara kho deti haim| 
adhasisi ka sabase uttama upachara, ise na hone dena hi hai| 
kama urja stara bhi adhasisi ka eka pramukha lakshana hai| 
atah isa bimari ke kisi bhi saphala upachara ka udyeshya purna svasthya prapta karana va sampurna sharira mem nava-prana dalana hona chahiye|ise prapta karane hetu sharira ke sampurna shuddhikarana ke satha prana shakti ka sanchara karane vale upayom ko apanana atyavashyaka hai |
arambha mem do ya tina dinom ke lie marija santare ke rasa aura pani para adharita vrata ki sahayata lem | 
rasa lene ki vidhi ke anusara, subaha 8 baje se shama 8 baje ke bicha, pratyeka do ghante ke antarala mem,garma pani ke satha santare ke rasa lena chahiye, lekina isake badale mem gajara, kakada़i evam ajamoda jaisi sabjiyom ke rasa ka sevana bhi kiya ja sakata hai| 
antom ki saphai ke lie, vrata ke dinom mem bhi, garma pani se gudavasti ki prakriya ko apanana chahie| 
thoda़e dinom ke rasa para adharita vrata ke bada marija lagabhaga pancha dinom ke lie dina mem tina bara taje rasadara phala jaise-seva, nashapati, angura, chhota chakotara, santare, annanasa, adu aura kharabuja adi sabhi phalom ke ahara ko apana sakata hai|
isa samaya ke daurana kisi bhi anya khadya padartha ka sevana nahim karana chahie; anyatha sampurna upachara ki upayogita nashta ho sakati hai| 
sabhi phalom ke ahara ka sevana karane ke bada, marija tina mukhya khadya samuhom-kramashah bija, giridara phala aura anaja, sabjiyam aura phalom para adharita, eka santulita ahara ko grahana kara sakata hai| 
yaha ahara dudha, dahi, chhachha, vanaspati tela aura shahada ke satha lena chahie| 
marija ki sthiti ke anusara eka ya do mahine ke antarala para alpakalina vrata ya sabhi phalom ke sevana ke ahara ki avashyakata ho sakata hai| 
yadi kabja adatana ho gaya ho, to ise nashta karane ke sabhi upayom ko apanana chahie| 
khadya padartha jinaka bhavishya mem paraheja karana chahie, ve haim- maide ke khadya padartha, chini, mishthanna, keka, pestri, mithaiyam, parishkrita anaja, mansiya khadya padartha, adhika kailori vale charbidara khadya padartha, dibba banda ya parirakshita khadya padartha, achara, masale aura saॉsa| 
atyadhika anaja ke sevana se bhi paraheja karana chahie| 
marija ko eka ya do bara mem jyada ahara grahana karane ki apeksha kai bara thoda़a-thoda़a ahara grahana karana chahie| 
atyadhika ahara ke sevana se bhi paraheja karana chahie| 
adhasisi ke upachara mem, nikotina anla vishesha rupa se sahayaka hai| 
atah vitamina bi ke kisi bhi eka prakara ka sevana karane ke samaya, yaha jaruri hai ki hama sampurna vitamina bi-kampalaiksa ko kisi na kisi rupa mem grahana kare, kyonki kisi bhi eka tatva ki adhikata ke karana dusara tatva asantulita ho sakata hai| 
yaha kisi bhi anya avanchhaniya samasya ke rupa mem prakata ho sakata hai| 
vastava mem, bi kampalaiksa ke sabhi prakara, adhasisi se bachava ke lie labhadayaka hai| 
adhika pani pina, pairom ko garma pani se nahalana, peta aura pita para sikaim karana, sira para (40 phem. se 60 phem.) ka thanda़a sampida़na karana aura jyada garma pani mem taulie ko bhigakara use gale para lagana, adhasisi ke sira darda mem kaphi arama pahunchata hai| 
marija ko kaphi vyayama karana chahie aura taji svachchha hava mem tahalana bhi chahie| 
niphraitisa gurde mem sujana athava jalana ki ora sanketa karata hai| 
yaha eka gambhira sthiti hai, aura yaha vikata ya dirghakalika bhi ho sakati hai| 
yaha bimari mukhyataura para bachapana ya kishoravastha mem prakata hoti hai| 
yadi isaka upachara arambhika avastha mem hi na kiya jae, to yaha dhire-dhire battara ho jati hai, aura nrityu ka karana bana jati hai| 
vaikalpika rupa mem yaha, eka dirghakalika avastha mem parivartita ho jati hai, jaham marija behatara to ho sakata hai kintu thika nahimm ho sakata| 
aksara peshaba mem rakta va shvetaka ho sakata hai, tatha lala aura shveta koshikaom ke dale ke satha ulti ho sakati hai, jo kshatipurna gurde se nikalata hai| 
marija chehare para sujana aura paira va talavom para sujana se pida़ita ho sakata hai| 
kai varshom se chali ane vali, niphraitisa ki dirghakalika avastha mem, marija ki peshaba se bhari matra mem shvetaka ka nishkasana hota hai| 
bada mem raktachapa mem vriddhi ho sakati hai aura marija mem mutra-raktadosha (yurimiya) ka vikasa ho sakata hai| 
isamem kai bara visheshatah ratri mem mutra ka nishkasana ho sakata hai| 
niphraitisa mem aksara gale mem kuchha golanu sankramana ya lala bukhara ya amavati bukhara hota hai|
samanyatah niphraitisa hone ke karana vahi haim, jo gurde ki bimariyom ke hone ke karana haim, jaise- khane ki galata adatem, atyadhika sharaba pina, kisi bimari ke upachara mem nishedhatmaka davaom ka prayoga karana, apachana ya peta ki gada़bada़i ke lie hamesha rasayanika davaom ka prayoga karana aura aispirina ya aisi hi anya darda nivaraka davaom ka prayoga karana| 
poshaka tatvom ki kami bhi niphraitisa ka karana bana sakati hai| 
yaha bimari janavarom ki bahuta si prajatiyom mem, ahara mem vitamina bi kolaina (janavarom ke utakom mem paya jane vala eka rangahina ghulanashila kshara) ki kami ke karana ho jati hai| 
aise janavara jinamem avashyaka vasiya anlom aura maignishiyama ki kami hoti hai, unamem bhi niphraitisa ho jata hai| 
jaba vitamina bi6 aura maignishiyama ki apurti kama hoti hai, taba kailashiyama ke satha akselika anla ke tikshna sphatakom ke karana , gurde mem kharabi a jati hai| 
vitamina i ki kami hone para bhi niphraitisa ho sakata hai| 
niphraitisa ka sabase surakshita upachara upavasa karana hai| 
jivavisha va daihika ashuddhata jo gurde ki sujana ka karana haim, unhem vrata ke dvara shighra hi samapta kiya jata hai| 
jaba taka ki gambhira lakshana ghata na jae, taba taka marija ko lagabhaga sata se dasa dinom ke lie rasa para adharita vrata karana chahie| 
isa samaya para jyadatara gajara, ajavaina aura kakada़i jaisi sabjiyom ke rasa ka sevana karana chahie| 
vrata ke daurana, vrata se upaji svayam svachchhata pranali dvara, antom ki saphai ke udyeshya se, pratidina garma pani se gudavastita ki kriya ko apanakara vishakta padarthom ka nishkasana karana chahie| 
rasom para adharita vrata ke bada, marija chara ya pancha dinom taka sabhi phalom ke ahara ko grahana kara sakata hai| 
isa daurana pancha ghanto ke antarala para, rasadara phala jaise-seva, santare, nashapati aura annanasa ka sevana kiya ja sakata hai| 
sabhi phalom ke ahara ke bada marija phala aura dudha ke ahara ka sevana kara sakata hai| 
isa prakriya pathyapathya niyama mem, uparokta phalom ke ahara ke satha ho sake bakari ka kachcha dudha shamila kara grahana kiya ja sakata hai| 
usake bada marija dhire-dhire, taje phalom aura kachchi va paki hui sabjiyom para vishishta dhyana dete hue, ninna protina vale santulita shakahari ahara ko arambha kara sakata hai| 
dirghakalika niphraitisa ki dasha mem tina dinom ke alpa samaya ke phalom ke rasa para adharita vrata ko apanaya ja sakata hai| 
usake bada eka saptaha ya dasa dinom ke lie niyantrita ahara ka sevana kiya ja sakata hai| 
isa pathyapathya niyama ke daurana, nashte mem santare ya santare ke rasa ka sevana kiya ja sakata hai| 
dopahara ke bhojana mem jaituna ke tela aura nibu ke rasa rupi masale ke satha, mausama ki kachchi sabjiyom ke salada ko shamila kiya ja sakata hai, tatha ratri ke bhojana mem eka ya do sabjiyam, jo apane hi rasa mem paki hui ho aura kuchha giridara phalom ko shamila kiya ja sakata hai| 
usake bada dhire-dhire marija, protina ki ninna matra vale santulita shakahari ahara ka sevana kara sakata hai| 
eka saptaha ke alpa samaya ke lie rasom para adharita vrata ke bada, do ya tina mahine ke samayantarala para, jaba taka ki gurde ki dasha mem sudhara na najara ane lage, marija niyantrita ahara ko grahana kara sakata hai| 
marija ko ऑkselika anla ki adhikata vali sabjiyam jaise palaka aura revatachini se paraheja karana chahie| 
chakaleta aura koko mem bhi ऑkselika anla ki matra payi jati hai, atah use bhi khana nahimm chahie| 
lahasuna, shatavari, ajamoda, jalakumbhi, kakada़i aura ajavaina uttama sabjiyam haim| 
papita aura kela sarvottama phala haim| 
donom ki hi tasira gurde ke lie svasthyavardhaka hai| 
dahi aura ghara para nirmita panira ki thoda़i si matra ko ahara mem shamila kiya ja sakata hai| 
sabhi taraha ke namaka ko ahara se prithaka kara dena chahie| 
eka ya do bara adhika ahara lene ki apeksha, pancha se chhah bara thoda़a-thoda़a ahara grahana karana chahie| 
ahara mem sabhi poshaka-tatvom ki paryapta matra honi chahie| 
bhojana se na prapta ho pane vale poshaka-tatvom ki kami ko pura karane ke lie sampurakom ko dena chahie| 
sampurakom mem shamila kie ja sakata hai-eka chhota chammacha pila-bhura yaugika samuha, jo bahuta se pashuom va paudhom ke utakom mem paya jata hai, visheshatah ande mem, vitamina bi6 ki 30 migra. matra, vitamina e ke 25 gra. iyusa, aura 300 se 600 iyusa vitamina i| 
ghunrapana aura sharaba pine ki adatom ka purnarupa se parityaga karana chahie| 
adhyayanom se pata chala hai ki dhunrapana, gurde ko purnarupa se kharaba kara deta hai| 
marija ko sapheda dabala roti, chini, keka, pestri, puda़inga, parishkrita anaja, charbidara aura tale hue khadya padarthom se paraheja karana chahie| 
use chaya, kaॉphi, sabhi mansala khadya padarthom, masalom, achara aura saॉsa se bhi paraheja karana chahie|
anya tarikom ke madhyama se (vyartha padarthom ke ) nishkasana mem vriddhi kara, gurde ko arama pahunchane vale sabhi tarikom ko apanana chahie| 
tvacha ke madhyama se nishkasana ke lie, jaham taka sambhava ho, pratyeka dusare dina ipsama namaka ke garma snana ko apanana chahie| 
niphraitisa ki dasha mem, taji hava aura bahya vyayama ko apanana bahuta hi labhadayaka hoga, tatha jaham taka sambhava ho marija ko dina mem eka ya do bara lagabhaga tina kilomitara ki duri taka tahalana chahie| 
use jaldabaji aura atiutsaha ka parityaga karana chahie| 
use dera ratri taka jagane se bhi bachana chahie| 
yadi ina upacharom ko imanadari ke satha apanaya jae, to gambhira niphraitisa ke marija mem jalda hi sudhara najara a jaege| 
yaham taka ki niphraitisa ki vikasita dasha mem bhi, uparokta ullekhita upacharom ko apanane se, grasita vyakti ki dasha mem kaphi sudhara laye ja sakate haim| 
niphraitisa ati gambhira tantrikiya gada़bada़iyom mem se eka hai| 
isamem tantrika mem sujana ya jalana hoti hai jisamem eka tantrika ya tantrika ka samuha shamila hota hai|
kabhi-kabhi, sharira ke vibhinna bhagom ke vividha prithaka tantrika samuha isamem shamila ho sakate haim| 
yaha dasha bahu-niphraitisa kahalati hai|
yaha bahutantrika roga ki taraha bhi jani jati hai, yadi sapha-sapha kaha jae, to yaha sthiti sujana ya jalana nahim hai, balki tantrika ki dasha mem aya eka parivartana hai, jisake parinamasvarupa pratikriya mem kamajori ati hai aura samvedana mem parivartana ata hai| 
niphraitisa ka mukhya lakshana hai, prabhavita tantrika mem junajuni aura jalana tatha darda ka hona| 
gambhira dasha mem jada़vatata aura samvedana ki samapti tatha sanndhita peshiyom mem lakave ki shikayata ho sakati hai| 
isalie, chehare para aya asthayi pakshaghata, prabhavita ora ki chehare ki tantrikaom mem ae parivartana ka parinama ho sakata hai| 
isa avastha ki gambhira dasha mem, ho sakata hai ki samanya svasthya ke abhava aura chehare ke prabhavita tarapha ki peshiyom mem shakti ke abhava ke karana, marija apani ankhom ko bhi banda na kara pae | 
niphraitisa, ridha ki tantrika mem hui ghataka araktata ke karana bhi hota hai| 
isa avastha mem marija ko andhere mem chalane mem kaphi dikkata ho sakati hai| 
tantrika-shotha ka pramukha karana hai, dirghakalika anlaraktata, arthata, rakta aura sharira ke anya taralom mem anla ki atyadhika matra ho jane ki sthiti | 
sharira ke sabhi taralom ki prakriti, unaki pratikriya mem kshariya honi chahie, kintu jaba lambe samaya taka galata ahara lene ke karana, anliya vyartha padartha nirantara utakom mem utpanna hota rahata hai, taba isake parinamasvarupa anlaraktata ki dasha ka udaya hota hai| 
jivana jine ki galata shaili, atyadhika karya karana adi snayu tamntra ke svasthya mem giravata late hai aura tantrika-shotha ke karana banate haim| 
kai taraha ke poshaka tatvom ki kami aura upapachayi gada़bada़iyam jaise-doshapurna kailashiyama upapachaya, vibhinna taraha ke bi vitamina jaise-bi12, bi6,bi1, painathothenika anla aura bi2 tatha samanya vishaktata ke parinamasvarupa bhi isa bimari ka udaya hota hai|
tantrika-shotha ke anya karanom mem shamila haim-aghata, gahara prabhava dalane vali chota, eka buri chota ya pramukha tantrika nali mem atyadhika dabava ka hona aura haddi ki sthiti mem parivartana ya tuta jana| 
kyonki chota se tantra ko aghata pahuncha sakata hai, atah peshiyom ki koi bhi prabala prakriya ya joda़om ka atyadhika khinchava, ye sabhi tantrika-shotha ka karana bana sakate haim| 
kai taraha ke sankramana ke karana bhi yaha sthiti utpanna ho sakati hai, ye sankramana haim-kshayaroga, kantharoga, dhanusha-tankara, kodha़ aura madhumeha, kitanashakom, pare, sise, sankhiya aura sharaba ke karana hui vishaktata| 
dardanivaraka davaom ka prayoga kara, tantrika-shotha ka upachara karane para, asthayi arama to milata hai, kintu ye davaem kashta ko prabhavapurnarupa se nahim khatma kara pati haim| 
isase sharira ke kuchha anya angom, visheshatah hridaya aura gurde ke svasthya ke jokhima ki kimata para, darda se kuchha samaya ke lie to arama milata hai, kintu tantrika-shotha mem koi bhi vishesha sudhara najara nahim ata hai| 
tantrika-shotha ka sarvottama upachara hai, marija sabhi poshaka tatvom ka prachura matra ka sevana kare, aura sabhi vitaminom aura anya poshakom ko purnarupa se pacha le| 
sabuta anajom, visheshatah sabuta gehum, bhure chavala, kachche va ankurita bija, vishishta rupa se chhachha ya ghara ke bane hue panira ke rupa mem dudha para vishishta dhyana dete hue, kramashah bija, giridara phala aura anaja, sabjiyom aura phalom ke rupa mem ahara mem tina adharabhuta khadya samuha shamila hone chahie| 
isa pathyapathya niyama mem, nashte mem taje phala, thoda़e se kachche giridara phala athava eka ya do chammacha suryamukhi va kaddu ke bijom ko shamila kiya ja sakata hai| 
ubali hui sabjiyam, gehum ki rotiyam aura eka gilasa chhachha ka sevana dopahara ke bhojana mem kiya ja sakata hai| 
ratri ke bhojana mem eka katora bhara ke taji-hari sabjiyom ke salada, ghara ke bane hue taje panira, taje makkhana aura eka gilasa chhachha ko shamila kiya ja sakata haim| 
gambhira dasha mem upayukta ahara dene se pahale, marija ko chara se pancha dinom ke lie, phalom ke rasa para adharita vrata ko karana chahie| 
rasa ke lie gajara, chukandara, nibu vansha ke phala, seva aura annanasa ka prayoga kiya ja sakata hai| 
tantrika-shotha ke nivarana va upachara mem sabhi prakara ke vitamina bi atyadhika labhadayaka hote haim| 
sapheda roti, chini, parishkrita anajom, mansa, machhali, dibba banda khadya padarthom, chaya, kaॉphi aura masale jaise khadya padartha, jinaka lagatara sevana karane se utakom mem anliya ashuddhata utpanna hoti hai; marija ko unaka sevana nahim karana chahie| 
marija ko saptaha mem do ya tina bara, ipsama namaka ke garma snana ko apanana chahie, aura yaha snana 25 se 30 minata ka hona chahie| 
dina mem kai bara prabhavita hisse ko, ipsama namaka vale garma pani,( jisamem thoda़e se pani mem eka chhota chammacha ipsama namaka mila ho), se snana karana chahie| 
marija ko tahalane ke satha halke vyayama bhi karane chahie| 
sthulata ya motape ko sharira ki aisi dasha ke rupa mem lakshanita kiya ja sakata hai, jisamem vasamaya utakom mem atyadhika matra mem charbi ekatrita ho jati hai| 
yaha (motapa) aksara sharirika jaruratom ke karana atyadhika ahara ke upabhoga ka parinama hota hai| 
pashchimi deshom ke logom aura bharata ke adhika amadani vale samuhom tatha anya vikasashila deshom ke bicha motapa bahuta hi samanya hai| 
motapa kisi bhi unra mem, kisi bhi linga ke vyakti mem ubhara sakata hai| 
yaha una vyaktiyom mem jyada paya jata hai, jo adhika ahara ka sevana karate haim aura aramadayaka (baithe rahane vala) jivana vyatita karate hai| 
mahilaom mem motapa, garbhadharana va rajonivritti ke bada prakata hota hai| 
garbhadharana ke daurana, eka mahila ke vajana mem, lagabhaga 12 kigra. ki vriddhi hoti hai| 
stanya kala mem apurti ke lie bhanda़arana ke rupa mem karya karane ke liye isaka eka bhaga vasa utaka mem hui vriddhi ke rupa mem prakata hota hai|
bahuta-si mahilaom mem adhika vriddhi hoti hai aura isa vriddhi ka kuchha ansha bana rahata hai| 
pratyeka agale bachche ke satha, ve dhire-dhire moti hoti jati haim| 
samanyataura para unra, linga aura unchai ko, vajana ki adarsha suchi ke satha susambandhita karake, usake anusara motape ko nirdharita kiya jata hai aura phira use 10, 20 ya 30 pratishata adhika vajana ke rupa mem shrenibaddha kiya jata hai| 
phira bhi samanya vajana sharirika rachana para nirbhara karata hai aura kuchha logom mem, jo anuvanshika rupa se vishala banavata aura prabala peshiyom ke malika hote haim, unaka vajana bina motape ke bhi adarsha vajana se kahim jyada hota hai| 
atah aura bhi gahana vaijnanika shodha ke anurupa, motape ka nirdharana sampurna sharirika vajana ke anupata mem, vasa ki matra para nirbhara hota hai| 
samanya kishora vyakti ke lie sharira mem vasa ki ausata matra 12 pratishata hai, aura navayuvati ke lie yaha matra 26 pratishata hai| 
eka vyakti jisamem vasa ki matra sampurna sharika ke vajana ke anupata mem 20 pratishata se jyada hoti hai, use sthula ke rupa mem dekha ja sakata hai aura mahilaom mem 30 pratishata se jyada ki matra motape ko prakata karati hai| 
motapa eka gambhira jokhima hai, kyonki atyadhika charbi hridaya, gurdom aura yaham taka ki yakrita para bhi dabava dalati hai, aura joda़om, kulhom, ghutanom aura pairom para jami hui adhika charbi jivanakala ko ghata deti hai| 
yaha bilkula satya kaha gaya hai ki, 'motapa jitana jyada hoga, jivana utana hi chhota hoga|'
adhika vajana vale vyakti mem sambhavitataura para ninnalikhita bimariyam hoti haim jaise- dhamani kathinya, hridayaghata, uchcharaktachapa, madhumeha, gathiya, thakka jamana aura yakrita va pittashaya mem gada़bada़i ka hona| 
aksara motape ka pramukha karana adhika ahara ka sevana karana hota hai-arthata sharira ke lie avashyaka shakti ki apeksha, adhika matra mem kailori ko grahana karana| 
kuchha logom ko atyadhika khane ki adata hoti hai, jabaki kuchha anya logom ko uchcha kailori vale khadya padarthom ke sevana ki adata hoti haim| 
aise logom ka vajana lagatara badhata hai, kyonki ve shakti ki avashyakata ko kama karane ke paripekshya mem apani bhukha ki pravritti mem samanjasya baitha pane mem asaphala hote haim| 
vartamana samaya mem, sharira ke badha़te hue motape ke sambandha mem logom mem jagarukata ayi hai| 
aise loga jo samanyatah udasa, duhkhi, akele ya pyara se vihina rahate haim, aura jinhem apane parivara ya samajika va vittiya stara se asantosha rahata hai, aksara unamem atibhojana ki pravriti utpanna karata hai, kyonki khana unhem ananda aura rahata deta hai| 
kabhi-kabhi motapa avatu ya piyusha granthi mem ai gada़bada़i ke karana bhi hota hai| 
kintu granthiya gada़bada़i, motape ke sampurna prasanga mem kevala do pratishata ki hi bhagidari rakhati hai| 
isa taraha ke logom mem adhabhuta upapachaya dara ninna hota hai aura yadi ve kama kailori vale ahara ka sevana nahim karate haim, to unaka vajana lagatara badha़ta hi jata hai| 
eka behatara suniyojita ahara yojana jo vyayama va anya sudharatmaka upayom se susambaddha ho, isako apanana hi motape se nipatane ka sabase kushala tarika hai| 
isa upachara mem sabase adhika dhyana dene vali bata yaha honi chahie, ki eka aise santulita ahara ka chayana, jisamem sabhi avashyaka poshaka tatvom ki paryapta matra ho aura kailori ki kama se kama matra ho| 
arambha mem marija ko sata se dasa dinom ke lie phalom ke rasa para adharita vrata ko karana chahie, isake daurana nibu, chhota chakotara, santara, annanasa, gobhi aura ajavayana ke rasom ka sevana kiya ja sakata hai| 
chalisa dinom ke dirgha samaya ke lie bhi, rasom para adharita vrata ko kiya ja sakata hai, kintu aisa kevala nipuna nirdesha aura nirikshana ke antargata hi karana chahie| 
vaikalpika rupa mem, do mahine ke niyamita samayantarala para athava jaba taka vajana mem chahi gayi kami na a jae, taba taka phalom ke rasa para adharita laghukalina vrata ko doharaya ja sakata hai| 
upachara ke kuchha shuruati dinom mem aura yadi avashyaka ho to bada mem bhi, antom ke svachchhikarana ke lie, garma pani se gudavasti ki kriya ko apanaya ja sakata hai| 
rasom para adharita vrata ke bada, chara se pancha dinom ke lie marija ko sabhi phalom para nirbhara rahana chahie| 
isa pathyapathya niyama mem marija ko dina mem tina bara, taje rasadara phalom jaise-santare, chhota chakotara, annanasa aura papite ke rasa ko lena chahie| 
usake bada vaha - kachche phalom, sabjiyom aura taje rasom para vishishta dhyana dete hue, dhire-dhire ninna kailori vale eka achchhe santulita ahara, jisamem ninnalikhita tina adharabhuta khadya samuha-kramashah bija, giridara phala aura anaja, sabjiyam aura phala shamila ho, usako grahana kara sakata hai| 
khadya padartha jinhem sakhtata se ghatana ya purnarupa se nakarana chahie, ye haim-adhika vasa vale padartha jaise-makkhana, panira, chakaleta, malai, aisakrima, mansala vasa, tale hue khadya padartha, karbohaidreta ki adhikata vale khadya padartha, jaise-breda, kaindi, keka, kukisa, anaja ke utpada, phaliyam, alu, shahada, chini, sharabata aura adhika kailori vali puda़iga; peya jaise- sabhi shitala peya aura sharaba yukta peya| 
khadya padarthom ke kama sevana ke karana, upaji bhukha ki lalasa (huka) se ubarane (nijata pane) ke lie, marija ko kramika taura para pani ka sevana ya kama kailori vale khadya padarthom aura pani ki adhikata vale khadya padarthom jaise-sabjiyam aura kuchha phalom ka sevana karate rahana chahie| 
malai nikala hua dudha, chhachha, bajare ka pani, phalom ke rasa, hari ubali hui sabjiyam aura kachcha salada, vajana ghatane ke lie achchhe haim| 
hamesha chhah se atha gilasa pani pina chahie, kintu bhojana ke satha nahim| 
phleterisama - vajana ghatane ka eka vishvasta tarika hai-phleterisama |
yaha 1898 mem sanyukta rashtra amerika ke horesa phreletara ke dvara khoja gaya tha| 
ve 50 paunda़ ke atirikta bhara ke shikara the, unhem hara chhah mahine mem phlu hota tha aura ve barabara apachana va thakavata ke ahasasa ki shikayata karate the| 
phlecheta ne purna charvana ke sabase mahatvapurna niyama ko svikara kiya| 
prarambha mem unhone chabane ki eka adarsha sankhya nirdharita karane ka prayasa kiya, kintu unhone paya ki eka nivale bhojana ko, tisa bara se kama chabana aparyapta hai| 
phalatah, unhonne taya kiya ki thosa va tarala khadya padarthom ko chabane ki eka adarsha sankhya nirdharita honi chahie| 
horesa phlechara ne ina niyamom ka palana pancha mahine taka kiya| 
isake parinamasvarupa usane 60 paunda se bhi jyada vajana ghataya aura agale 20 varshom taka dubala raha| 
phlechara ke anusara banayi gayi vajana ghatane ki yaha yojana ashcharyajanaka karya karati hai, aura yaha eka bara karake dekhane yogya hai| 
yaha mahatvapurna hai ki mota vyakti, vajana ghatane ke lie, apane ahara ko isa prakara se abhikalpita kare ki- vaha eka saptaha ya dasa dinom mem eka pamuda vajana ghata sake| 
yaha use ahara ki nayi adatom ke prayoga ke yogya banaega| 
ahariya upachara ke satha-satha, marija ko vajana ghatane ke sabhi anya upayom ko bhi apanana chahie| 
vyayama, vajana ghatane ki yojana ka eka pramukha hissa hai| 
yaha sharira ki peshiyom ko majabuta karane ke alava, sharira ke vasa mem lipta kailori ke prayoga va chinta ke nivarana mem sahayata karata hai| 
yoga ke kuchha asana jaise-sarvangasana, bhujangasana, dhanurasana, chakrasana, vajrasana aura yogamudra bhi kaphi labhadayaka haim| 
ye jame hue vasa ko samapta kara ya punahvitarita kara, dubale hone mem madada karate haim aura kamajora angom ko shaktishali banate haim| 
marija ko aise tarikom ko apanana chahie, jo atyadhika pasina utpanna karate hom, jaise-sona bailta, bhapa snana aura adhika malisha| 
ye sabhi vajana ghatane mem sahayaka hote haim| 
aura ina saba se pramukha hai ki, mote vyakti ko nakaratmaka bhavom, jaise-chinta, dara, shatrutapurna bhava aura asuraksha ki bhavana ko samapta karane va jivana ko dekhane ke sakaratmaka najarie ko badha़ane ka prayasa karana chahie| 
apa mem jivana jine ke prati utsaha ki bhavana hona chahie| 
aise bahuta hi kama loga hai, jo apane jivana ke anta taka apane svasthya ko sakriya banae rakhana chahate haim| 
aise loga samaya se pahale hi budhe aura durbala ho jate haim| 
dina pratidina, varsha dara varsha, jaise-jaise hama paripakva hote jae, hamem apane svasthya adhikosha mem apani sehata ki sampatti ko badha़ane ke bare mem avashyaka rupa se sochana chahie| 
kisi bhi nipuna nirdeshaka ke sannidhya mem, ina koshom ka prayoga bephikra rupa se kiya ja sakata hai| 
sehata ke eka yojanabaddha drishtikona ke dvara, hama apani shakti ko labhaprada rupa se kharcha (prayoga) kara sakate haim| 
hama eka manava ke rupa mem sashakta tatha aura bhi purna honge| 
hama apani sehata ki shakti ko kharcha kara sakate haim aura phira usake bada bhi, hamane jo khoya tha, use ya yaham taka ki aura bhi mulyavana shakti ko, hama punah prapta kara sakate haim| 
yaha saba kevala janakara nirdesha ke dvara kiya ja sakata hai| 
sharira ko hrishta-pushta, utsahi aura susvastha banae rakhane ke lie avashyaka jnana ko, achchhi taraha samajana aura apanana chahie| 
vyartha ki adatom ka nirakarana karana chahie| 
apavyaya se jivana ke vastavika ananda ko pana bada़a hi durlabha hai| 
achchhi sehata ke siddhantom para phijulakharcha hona, apako purnatah parasta/nashta kara sakata hai| 
apane jivana ko savadhanipurvaka jie| 
jisa taraha apa apane paisom ka sanchaya adhikosha mem rakha dete haim, usa taraha apa ise adhikosha mem nahim rakha sakate| 
achchhi sehata eka aisi chija़ hai, jise apa apane satha rakha sakate haim| 
yaha apaki chhavi ko arunima aura apake hridaya ko eka laya pradana karata hai| 
yaha apako, mitra va pyara pane ke avasarom ko upalabdha karata hai| 
apako vriddhavastha mem bhi kishora dikhane va kishoravastha ko mahasusa karane ki ora dhyeyita hona chahie| 
ullasita aura kriyashila svasthya eka bada़i sampatti hai| 
isa prakara apa apani sampatti aura achchhi vittiya, samajika, naitika, manasika aura dharmika jivana shaili ko nirmita kara sakate haim| 
jaba apa bimara ya asvastha anubhava karem to chinta na karem|
aisa na sochem ki achchhe svasthya ka yaha khajana apake pahuncha se bahara hai|
kathina parishrama ke lie taiyara rahie|
dhairya rakhem aura nirantara karate rahem|
apa shighra hi dekhenge ki apa apane roga se mukta ho sakate haim; apaki shakti badha़ti hi jaegi, apaka jivana ke lie utsaha dhire-dhire apako tripta kara dega, aura apa punah ashavadi hone ka anubhava karane lagenge|
ja़ldabaja़i mata kijie|
varshom ki vyartha aura laparavaha jindagi ke bada hi koi bimara pada़ta hai|
sudhara bhi dhimi tatha sthayi gati se ata hai|
sabase mahattvapurna bata yaha hai ki apa ko ashapurna hona chahie|
yadi apa sabhi ashaem chhoda़ dete haim, to apa chinta karate rahenge aura apa yaha samajenge ki apake karmom se apa mem sabhi asaphalataom tatha pida़aom ki dhera laga gai hai|
apako aisa lagega ki yaha sansara apaka shatru bana gaya hai|
apaka sabase bada़a shatru apako utana hani nahim pahuncha sakata jitana ki apa svayam ko aise nirashavada se pahuncha sakate haim|
vaha sabase nirbala hota hai jo kisi sangharsha mem harata hai|
apako sangharsha karane aura jitane ka prayasa avashya karana chahie|
yaha apako aura adhika bala dega|
yaha pratishtha aura prerana ka eka srota hoga tatha aura bada़i upalabdhiyom ki ora le jane mem sahayata karega|
prakriti ne manushya ko mahattvakanksha aura use purna karane ke lie prerana bhi di hai|
usane use ashaom tatha svapnom sahita, sangharsha karane ke lie hathiyarom se laisa kiya hai|
bhutakala ki samaja se, vaha bhavishya ke karya ke lie sakriya tatha atyavashyaka mana ka prayoga karake labha uthata hai|
purna karane ki yaha kshamata usamem hai|
use phalane-phulane ki avashyakata hai aura use saphalata avashya prapta karani chahie|
jivana usase kuchha kama svikara nahim karega aura use shanti tatha prasannata se rahane bhi nahim dega|
hara vyakti apane svayam ke kshetra mem saphala ho sakata hai|
koi shakti ya paristhitiyam use puri taraha se nirasha nahim kara sakatim|
koi bhagya hatasha hone ke lie usaka vinasha nahim kara sakata|
asha, sankalpana, svapna, vaha ina sabhi ko purna kara sakata hai|
phira bhi, saphalata kevala mana ke dvara prapta nahim ho sakati, kyonki mana sharira ke bina raha ya karya nahim kara sakata hai|
aisa sochana murkhata hai ki mana ko sharira ki kimata para vikasita kiya ja sakata hai|
yaha momabatti ke prakasha ko bachane ke lie momabatti ko nashta karane ki taraha hai|
adhunika jivana mem sharirika sanskriti ke mata ka chalana atyavashyaka rupa se anivarya ho gaya hai|
yaha puchhana murkhata hai ki kauna sa adhika mahattvapurna hai – sharira ya mana|
donom ghanishtha rupa se eka dusare se sambaddha aura apasa mem guthe hue haim|
eka ko dusare ki kimata para vikasita karane ka koi bhi prayasa sva-parajaya hoga|
yunani pashchimi sabhyata ke sansthapaka ke rupa mem jane jate haim|
unhonne apani sabhyata ko jimanastika aura sangita para adharita kiya tha|
unake lie jimanastika ka uddeshya tha nagna hokara kie gae niyamita vyayama dvara sudaula sharira ka nirmana|
sangita jisamem kavita aura nataka sammilita the, ka uddeshya tha mana ka prashikshana tatha bhavana se mukti|
yadi koi sharirika tatha manasika rupa se vikasita hota tha, to unake siddhanta ke anusara, vaha shikshita tha|
bharata isase age chala aura yoga tatha dhyana ka avishkara kiya, aura mana ki kshamata ki vistirnata ki khoja ki|
yoga asana ke vyayamom ne sharira ko tandurusta tatha svasthya banaya, aura taba dhyana dvara sharira ko niyantrita karane ki isaki vistrita kshamata ko janane ke lie mana ke ajnata ekanta sthana mem adhika gaharai mem gae|
aba yaha bina kisi sanshaya ke pramanita ho chuka hai ki kai rogom mem mana doshi hota hai ya phira yaha mukhya bhumika nibhata hai|
yadi mana ko niyantrita kiya ja sake to roga ko bhi niyantrita kiya ja sakata hai|
kisi manushya ko bhayabhita kijie aura usaka hridaya adhika tivrata se dhada़kane lagega|
usako prasanna kijie to apa usaki ankhom mem chamaka aura usake galom para lali dekhenge|
use sochane-vicharane ke achchhe upaya sikhaem aura usaki saphalatapurvaka karya karane ki kshamata badha़ jati hai|
usake sharira ki jivana shakti mem sudhara hoti hai aura usaka mana kriyashila tatha tivra ho jata hai|
mana ka eka unnata samanvaya uttamatara manasika shuddhata ki ora le jata hai|
eka prabhavakari sharira ka artha hai prabhavakari vichara|
nishchita rupa se aura achchhe se karya karane ki yogyata nishchita rupa se tatha tivrata se kama karane ki kshamata deti hai|
unhonne mansapeshiyom ka samanvaya kara liya hai jisaka artha hai ki ve purna rupa se apane mana ke niyantrana mem haim| 
loga jinake pasa sabase uttama buddhi hoti hai ve achchhe vyayami bhi hote haim|
jaॉrja vashingatana lambi kuda mem 22 phuta taka kuda sakate the|
aisa kuchha jo ki kevala lagabhaga dasa hajara mem se eka hi kara pane mem samartha hota hai|
aisa jana jata tha ki laॉrda bayarana ke mudgarapada the|
ve apani kavita mem utane hi achchhe the|
naipoliyana eka chhote kada ka manushya tha|
mutapa aura ja़rurata se jyada khane se usaka jivana kama ho gaya, phira bhi usamem bahuta adhika urja thi, vaha adatana eka dina mem 18 ghante kama karata tha|
tenisana ko achchhi dikhane vali ke rupa mem varnita kiya jata hai|
vardasvartha sundara tatha hiro ke samanupatika sharira vala tha|
vaॉltara skaॉta ka harkyulina ke samanupatika diladaula tha|
taॉlstaॉya paisatha sala mem bhi eka bahuta badha़iya tairaka the|
unhonne chhaisatha sala ki unra mem sayakala chalana sikha|
jaba ve pachasa ke the to ve tina dinom mem 130 mila chale the|
inamem se kuchha logom ko vishesha roga the aura phira bhi ve durbala nahim the|
harbarta spensara vikrita pachana ke satha-satha anidra roga se bechaina the, phira bhi ve assi sala ke hone taka jivita rahe| 
unaki vastavika samasya yaha thi ki kyonki ve sochana banda nahim pate the atah unhem sone mem kathinai hoti thi| 
gandhiji atyanta dubale-patale the|
ve lambe samaya taka kama kara sakate the aura niyamita paidala chalane vale the|
buddhi kshamata ka sira ke akara aura banavata se thoda़a hi lena-dena hai|
sabase bada़e mastishka ka bhara 3 paunda tha|
pratibhashaliyom mem se kai logom ke mastishka asamanya rupa se bada़e the, kintu unamem se kuchha ve bhi the jinake mastishka ke akara vastava mem samanya se kama the|
ve loga jo apana pura jivana sharirika vikasa ke lie samarpita karate haim, unaka sambhavatah adhika achchha sharirika- gathana hoga banisbata unake jo nishkriya jivana jite haim, jaise vaijnanika, vakila ya chikitsaka|
jivana ke khela mem sharira aura mana donom ka suvyavasthita vikasa sabase adhika bara jitata hai|
kuchha logom ko isa bata ka ehasasa nahim hota hai ki sharirika jivana apane apa mem hamare sabhi prayasom ke yogya hai|
kaiyom ka drishtikona hai ki dhana ke sangraha ka pramukha mahattva hai|
isaki khoja mem ve sharirika tandarusti ki avashyakata ko anadekha karane ki ora jukava rakha sakate haim|
unhem samaya bitane ke satha pachhatava ho sakata hai|
jo atma hatya karata hai usa adami ka dava hota hai ki yaha jivana jine layaka nahim hai|
jo apane svasthya ki dekhabhala nahim karate haim, ve bhi dhire-dhire aura kashtapurvaka atma hatya karate haim|
aise loga bhi haim jo apane sharirika kriya-kalapa ko dhana ki khoja ke lie samaya mem vriddhi karane ki asha se, ninnatama mem ghata dete haim|
yaha dharana svayam ko mata deti hai|
vaha manushya jo apane sharira ki upeksha karata hai, asthayi rupa se, anya karyom ke lie samaya mem vriddhi kara sakata hai|
yadi tatkalika koi bura prabhava nahim hota hai, to use lagata hai ki vaha usa manushya se badha़kara hai jo apane sharira ko vikasita karane ki sochata hai aura usake lie samaya deta hai|
vaha manushya jo apane sharira ki upeksha karata hai, asthayi rupa se, anya kriya-kalapom ke lie samaya mem vriddhi avashya karata hai|
alasya ki adata pahale jindagi ki ga़aira-sharirika ja़rurata se banai jati hai|
taba upeksha ki yaha adata, sharirika rupa se kriyashila jivana ki ora vapasi se nirutsahita karane ke lie apane apa mem eka guna bana jati hai|
isa prakara sharira ki upeksha ke dushchakra ki nimva pada़ti hai|
aksara, yaha sthiti bada se badatara ho jati hai, jaba taka svasthya para koi spashta virama nahim laga jata hai aura usake manasika ya arthika upalabdhiyom mem hastakshepa karane lagata hai|
kevala taba hi pida़a aura bhaya, sharira ki tandurusti ko anadekha karake usane jo bhula ki hai usaka bodha balapurvaka karate haim|
taba achchhe svasthya ki talasha mem, vilambita tatha andha bhagadauda़ hota hai|
manushya ko rogi banana prakriti ki chahata nahim hai|
yaha durbhagyavasha taba hota hai jaba prakritika kriya-kalapa ka damana aura sahaja pravritiyom ki vikriti hoti hai|
eka bara jaba svasthya bigada़ jata hai, to kaise bhi ho, prakriti vaise hi isa kshati ko sudharane ka prayatna karati hai jaise ki vaha sharira ki sataha para ke ghava ko bharati hai|
isamem, hama sharira ki sahayata karane ke lie prakriti ki sahayata kara sakate haim|
roka-thama upachara se behatara hai|
ve jinake svasthya mem kami hai aura taba ise punah prapta karate haim unhem isaka ehasasa hota hai ki roka-thama upachara se behatara hai|
yadi svasthya ki kami ko roka ja sakata hai to yaha behatara hai aura aise vyakti ke pasa prakriti ke prati kritajna hone ke behatara karana haim|
rogom ki roka-thama tatha svasthya nirmana pradhana abhipraya hote haim, upachara isake bada mem ata hai|
jo sampanna haim ve uttarajivita ki avashyakata ko samajate haim kyonki isa sampatti ka ananda lene ke lie unake svasthya nirmana ki apeksha unake vittiya sanrajya ka nirmana kama mahattvapurna hai|
kevala achchhe svasthya se hi ve ananda le sakate haim aura jivana ke rasa tatha prasannata ko pa sakate haim|
yaha sharira bhagavana ki sabase sundara rachana hai|
isake pasa rogom se lada़ne ki apani khuda ki prakriya hai|
yaha eka svayam ka anurakshana karane vala yantra hai tatha yaha svayam ko jivita rakhane vala bhi hai|
yaha svayam ko nipunata se chala sakata hai aura jaba avashyakata ho to apani marammata bhi kara sakata hai|
yadi apa apani galatiyom ko janate haim aura unhem karane se bachate haim to apa apane svasthya mem sudhara laenge- apa eka shaktishali vyakti haim| 
yadi apa apani galatiyom ko janate haim aura unhem karate rahate haim to apa asvastha ho jaenge kintu apani galatiyom ko sudharakara apa isase bahara nikala sakate haim|
apa eka samanya vyakti haim|
yadi apa galatiyam karate haim aura yadi apa nahim janate ki apa unhem kara rahe haim, taba to apa unase bachane mem asamartha haim tatha apa rogi ho jaenge aura isase bahara nahim nikala sakenge- apa eka durbala vyakti haim|
svasthya manushya ka janmasiddha adhikara hai|
svastha rahana utana hi svabhavika hai jitana ki janma lena|
sabhi rogatmaka paristhitiyam, sabhi roga tatha roga ki sabhi pravrittiyam, sharirika aura svasthyakara niyamom ke atikramana ke parinama haim|
yaha svasthya ke isa vijnana ko ati sankshepa mem kahane ka eka dhanga hai|
samaya bitane ke satha hamem jnata hua hai ki yaha aisa nahim hai|
penisilina ki apani sva simaem haim|
hara 2-3 sala mem hama maisina ke eka nae utpadana ka vikasa karate haim aura 2-3 sala mem ve pratirodhi jivanu ki vriddhi ke karana prabhavahina ho jate haim|
ve sankramana rokane mem sahayata karate haim, isamem sanshaya nahim hai, kintu ve parshva prabhava bhi utpanna karate haim|
kainsara aura edsa ke agamana ke satha hama aushadhiyom ki simaom ko samaja pae haim|
hama janate haim ki hama jo bhojana karate haim aura paryavarika khatare kainsara ke lie jimmedara haim|
ahara mem adhika vasa aura tantuom ka abhava kainsara utpanna karata hai|
naya shodha sharira ke bhitara hi - roga-pratikaraka pratikriya ke madhyama se, sharira ke pratirakshi tantra para nirbhara karate hue upachara vikasita karane para hai|
yadi hama apane bhojana, vyayama tatha vishrama se satarka nahim haim, to hamare sarvottama vaijnanika jnana ke bada bhi, nae roga utpanna hote rahenge|
jaba apako jvara ata hai to chikitsaka ke pasa mata dauda़ie, 2-3 dinom taka pratiksha kijie, ho sakata hai yaha apane apa utara jae|
jaba apako jvara hota hai, to 400 sentigreta (1020 pheranahaita) para roga-pratikaraka ka nirmana 20 gune se adhika badha़ jata hai|
isaka artha hai ki sharira svayam se lada़ne ke lie sakshama bana raha hai|
sardi ya siradarda jaise mamuli rogom ke lie aushadhiyam na lem, pratiksha karem aura dekhem, adhikatara bara ye kuchha dinom ke andara ojala ho jaenge|
sardi eka vishanujanya sankramana hai, yaha apane apa ko kama karane ke lie 5-7 dina lega hi, chahe apa aushadhiyam lem ya na lem|
vastava mem, vishanujanya sankramana ke lie koi bhi aushadhi prabhavakari nahim hoti hai|
sabase ahanikara dikhane vali aushadhiyam vastava mem ahanikara nahim haim, unake bhi parshva prabhava hote haim|
eka achchha chikitsaka chunie jo kama se kama aushadhiyam batalata hai|
roga ki sahi pahachana usaki visheshajnata aura kaushala para nirbhara karata hai|
isilie yaha behatara hai ki 2-3 dinom taka pratiksha karem aura upachara se purva roga ka uchita nirnaya karavaem, sharira ko svayam ka upachara karane ka eka avasara pradana karem|
yoga chikitsa vijnana ka uddhara karane ke lie kai prakara se a raha hai|
vastuom para mana ki pakada़ adhunika yuga ka chamatkara bana gai hai|
sadiyom se bharata ke yogi sva-niyantrana ka abhyasa karate rahe haim|
ve una rogom ka upachara karane mem samartha hue haim jise chikitsaka karane mem asamartha the|
ve jaiva pratipushti mem vishvasa karate haim aura unaka mukhya vakyansha hai rogi svayam se achchha hota hai|
prayoga darshate haim ki samvedanashila yantrom ki sahayata se, rogi kuchha vishesha sharirika karyom ko niyantrita kara sakata hai, jise pahale aniyantraniya samaja jata tha|
vaijnanikom ne paya hai ki hridaya, mastishka, mansapeshi samudayom aura parisanchari tantrom se sambandhita rogom para, jaiva pratipushti se yadi unhem purna rupa se rogamukta nahim kiya ja sakata to, ka़abu avashya paya ja sakata hai, jo prantastha (kaॉrteksa) ke thika niche sthita limbika tantra ke alpha mastishka tarangom aura vidyuta avegom ko niyantrita karane ki hamari kshamata se juda़i hui hai|
1935 mem vaijnanikom ne kaha ki jaba hama dura ke sthana mem ekataka dekhate haim aura taba hamari ankhe kuchha samaya ke lie banda hoti haim to samanyatah alpha tarangem prakata hoti haim|
mastishka samanyatah kevala bita tarangem utpanna karata hai aura yaha ama taura para sharira tatha tantrika tantra ko dhimi aura shithila karati hai|
1910 mem, jarmana sammohaka johana shultja़ ne, rogiyom ko vishrama ki avastha mem svayam se batem karana sikhaya| 
unhem ashcharyajanaka parinama mile|
udaharana ke lie, jaba ve mere hatha garama haim vakyansha ko duharae to unake hatha vastava mem adhika garama ho gae the| 
1964 mem amariki chikitsakom ne shultja़ ke prayogom ko upakaranom, jisase tvacha ke tapamana ko maॉnitara kiya gaya tha, ke madhyama se pramanita kie|
svayansevakom ko vishrama karane aura apane hathom ko adhika garama hote hue anubhava karane ke lie kaha gaya tha|
upakarana ne unake hathom ke tapamana mem vriddhi ankita kiya|
isamem sirpha़ roga ke abhava ke atirikta bahuta kuchha shamila hota hai|
vastava mem, eka svastha vyakti kevala achchha hi anubhava nahim karata hai balki, vaha anya logom ke satha achchhe se milajula kara rahata hai aura jivana ke prati yatharthavadi drishtikona rakhata hai|
achchha svasthya logom ko apane lakshyom ko prapta karane aura jivana ka bharapura ananda lene mem sahayata karata hai| 
atah logom ko manava sharira ke bare mem maulika jnana avashya hona chahie, ki yaha kaise karya karata hai, aura isaki jatila suraksha prakriya kya haim|
achchhe svasthya ke bare mem jnana ko hara eka ki shiksha ka bhaga banana chahie taki chhoti unra se hi ve jana jaem ki kya unake sharira ke lie achchha hoga aura kya ise hani pahunchaega|
sharirika svasthya ko banae rakhane ke lie, sharira ke sabhi bhagom ko milajula kara kama karana chahie|
eka achchhi sharirika evam manasika sthiti mem kisi vyakti mem sakriya jivana ka ananda lene aura dainika jivana nirvaha ke tanava ko sahane ke lie urja aura shakti hoti hai|
achchhe jivana nirvaha ke lie poshana, vyayama, sustana, vishrama, nidra, svachchhata aura chikitsiya tatha danta sambandhi dekha-bhala anivarya shartem haim|
eka santulita ahara svasthya, uchita vriddhi tatha vikasa ke lie sabhi bhojana pradana karata hai|
ahara vijnom ne poshaka tatvom ko pancha pramukha vargom mem vargikrita kiya hai: karbohaidreta, vasa, protina, vitamina tatha khanija|
jala jivana ke lie anivarya hai parantu ise poshaka tatvom se alaga mana jata hai|
eka santulita ahara mem vistrita prakara ke khadya padartha sammilita hote haim|
phala aura sabjiyam vitamina tatha khanija pradana karate haim|
mansa, kukkuta, machhali, ande, dugdha utpada tatha kashthaphala protina ke sanriddha srota haim|
dabalaroti, siriyala aura alu karbohaidreta ke satha-satha vitamina tatha khanija pradana karate haim|
achchhe poshana mem prati dina bhojana ko uchita matra mem khana shamila hai, adhika khana mutapa ki ora agrasara karata hai|
avashyakata se adhika vajana hridaya para atirikta tanava dalata hai aura kisi vyakti mem madhumeha tatha anya rogom ke hone ki sambhavana ko badha़ata hai|
vajana kama karane ka sabase achchha tarika, ahara tatha upachara ke eka niyamita svasthya utpadaka karyakrama ka anusarana karana hai|
vyayama mansapeshiyom ko maja़buta banata hai aura parisanchari tatha shvasana tantra ke karya ko behatara banata hai|
yaha sharira ko tanava sahane ke lie sakshama banata hai anyatha yaha sharirika aura bhavatmaka samasyaem utpanna kara sakata hai|
vishrama tatha nidra thakana para ka़abu pane aura urja ki punah prapti mem sahayata karate haim|
svachchhata jivanuom tatha kitanuom ki vriddhi ko niyantrita karati hai|
dainika snana sharira ko maila tatha gandha se mukta rakhati hai|
balom ko bhi niyamita rupa se dhona chahie|
danta-sambandhi dainika dekhabhala, dantom ko thika se sapha़ karana aura unhem svachchha rakhana, achchha svasthya banae rakhane ke lie anivarya haim|
hamem apane sharira ke svastha hone ki kshamata ka puri taraha se ehasasa nahim hua hai|
prakriti ne hamare sharira ke rogom se lada़ne ke lie eka suraksha prakriya banaya hai|
isalie svasthya utpadana kheta mem utpadana karane se kuchha adhika bhinna nahim hai|
behatara poshana, surya prakasha, taji hava, vyayama tatha manasika vishrama se apaka svasthya khila jaega|
isa sharira mem svastha hone ki prakritika kshamata hai|
shalyachikitsaka haddi ke tute hue sirom ko milakara rakhenge, palastara ki eka patti se unhem unake sthana para rakhenge aura kuchha saptahom ke bada ye haddiyam eka bara phira juda़ jaengi|
dhire-dhire ghava bharata jata hai, eka nai tvacha ka nirmana hota hai, aura papada़i apane svayam ke bhara se jada़ jati hai|
kahie ki eka durghatana ke bada, jaba eka bahya padartha sharira mem antahsthapita ho jata hai, to ho sakata hai ise shalyachikitsaka se hataya jaya para vaha achchha prakriti ke dvara hi hota hai|
yadi shalyachikitsaka upalabdha nahim hai, aura yaha padartha hataya nahim gaya hai, to pida़a badha़ jaegi, tatha bahya akramaka ke charom ora sujana aura jalana arambha ho jaega|
chikitsiya shabdom mem, bhakshakakoshikaem utpanna hoti haim, pipa bana jata hai aura yaha bahya pinda ya to pacha liya jata hai ya phira pipa ke satha bahara phenka diya jata hai|
jaba hama bimara pada़te haim to prakriti, eka mata ki taraha, hamare sanrakshana ke lie satha hoti hai, aura yadi hama svastha haim to hamare svasthya ko utkrishta banati hai|
hama svasthya utpadana ki tulana khetom mem kie jane vale krishi utpadana se kara sakate haim|
yadi hama achchhe bijom ka upayoga karate haim, uchita urvarana, nami, surya prakasha aura eka sahi tapamana rakhate haim to phasala lahalahaegi| yadi inamem se koi eka bhi anupasthita hai, yadi phasala ko uchita nami ya uchita tapamana ya sahi prakara ka urvaraka nahim milata hai to vriddhi ruka jaegi|
yadi anupasthita karaka ko vapasa de diya jaya to yaha phasala phira se svastha aura hari-bhari ho jaegi|
krishi mem, bija ki utkrishtata ka pramukha mahattva hai|
eka achchhi kisma ka bija eka achchha utpadana dega|
yadi hama eka kharaba kisma ke bija ko urvaraka, sahi nami, surya prakasha tatha garmi denge, to bhi utpadita phasala utani achchhi nahim hogi jitani ki eka achchhe kisma ke bija se hogi|
thika yahi manava sharira ke lie bhi satya hai|
yadi hama achchhe svasthya vale mata-pita pane vale bhagyavana haim, to bahuta hada taka hama bhi achchha svasthya virasata mem paenge|
isa bata ka aba ehasasa ho gaya hai ki dhunrapana ya sharaba athava bahuta adhika chaya ya kaॉpha़i bhi badha़te hue bachche para pratikula prabhava dala sakate haim|
atah, mata ka sharirika svasthya evam manasika svasthya, santana ki vriddhi ko prabhavita karate haim|
yaha mata ke garbha mem ropita bija ke prakara para nirbhara karata hai ki, abhi paida hone vala bachcha eka sannyasi, eka ugra vyakti, eka sangitakara, eka darshanika ya eka aparadhi hoga|
mahabharata mem, yoddha arjuna ki eka kahani hai, jo usaki garbhavati patni (unake pahale putra abhimanyu ka garbha dharana ki hui) se sambandhita hai aura jo sainya nirmanom ki jatilata aura unaka kaise samana karem, ke bare mem hai|
usane usako isake bare mem bataya ki kaise vaha shatru ke shivira (chakravyuha) mem phansa gae the aura kaise ve aura unake sainika bahara nikalane mem saphala hue|
unaki patni, eka sansarika stri hone ke nate samarika ghatanaom ke prati utsuka nahim thi, aura jaba arjuna yuddha ki bhumi mem apani virata ke bare mem vyaktavya ko vistarita kara rahe the to vaha nipata uba vasha so gai|
abhimanyu ka janma hua aura vaha eka achchhe sainika ke rupa mem praudha़ hua|
vaha apane pita ke hi samana paristhiti mem phansa gaya|
vaha shatru ke jala se bahara nahim nikala pa raha tha, aura vaha samana karate hue pakada़a gaya tatha mara gaya|
isaka karana isa sachai se ho sakata hai ki jaba arjuna, shatru ki rachana se bahara kaise nikale, isa guढ़ta ka vivarana kara rahe the to usaki mata so gai|
yaha kahani ujagara karati hai ki eka mam ki manasika avastha ka badha़te hue bachche para suspashta prabhava tha
hindu dharmagrantha paramarsha dete haim ki garbhavastha ke daurana mam ke pasa eka prashanta vatavarana hona chahie, use achchhi pustakem aura dharmika sahitya padha़ne ke lie kahana chahie, aura achchha sangita sunane ke lie kahana chahie, taki badha़te hue bachche ke mastishka mem uchita bija boya ja sake | 
yaham taka ki paudhom ki vriddhi para bhi sangita ka achchha prabhava hai |
vaijnanika rupa se, yaha paya gaya hai ki paudhom ke lie prati dina bajae gae sangita ne unako behatara phalane phulane mem sahayata ki apekshakrita unake jinako sangita ka labha nahim mila |
yaha gayom ke lie bhi satya haih yadi unake lie sangita bajaya jata hai, to ve adhika dudha deti haim | 
muje lagata hai ki garbhavastha ke daurana poshana aura vyayamom ko uchita mahatva avashya dena chahie |

kuchha bimariyom mem anuvanshika karaka ko hama inkara nahim kara sakate haim |
yadi hama eka svastha vyakti ke gale ki eka khurachana lete haim aura sukshmadarshi se parikshana karate haim to hama ise bahuta si bimariyom ke jivanu se chhidrita paenge |
sharira mem hamem bimariyom se mukta rakhane ki shakti hai aura phira bhi jivanu ko ashraya deta hai |
yadi hamare pasa uchcha stara ki jivana shakti hai to jaba hama isake samparka mem ate haim taba bhi hama bimari se aprabhavita rahenge |
jaba hamari pratirodhakata ya jivana shakti kama hoti hai, ye jivanu sankhya mem badha़enge aura hama para akramana karenge aura taba hama isake shikara ho sakate haim | 
hama eka uchita poshana, aramadayaka ninda, vyayama dvara, bahuta sara pani pikara, kabja़ se bachate hue aura apane apa ko tanavamukta rakhate hue jivana shakti ko uchchatara stara ki ora le ja sakate haim|
hamem bimari ke karya-karana sambandha mem manasika sthiti ki bhumika ko nahim bhulana chahie | 
yaha bahuta si bimariyom mem eka mukhya bhumika nibhati hai |
roganu ko marane aura bimari dura karane ke lie adhunika vijnana ka siddhanta hai | 
yadi hama isa siddhanta ko uparyukta tathya ke sandarbha mem dekhate haim ki apa roganu ko ashraya de sakate haim aura phira bhi shayada bimari se pida़ita nahim hom to yaha vastava mem eka tarka sangata mastishka ko pasanda nahim ata hai |
sharira svayam hi bimariyom se chhutakara pane ka adhika utsuka hota hai aura isako yaha kshamata prapta hai |
isake pasa eka svayam raksha yantravinyasa hai |
jaba hamare shatru, bimari utpanna karane vale jivanu se samana hota hai, to usake lie hamare sharira mem eka sannihita raksha yantravinyasa hai | 
sharira ka prathama bachava bahya tvacha aura antarika angom ke charom ora ki jilli hai |
dono kinchita pratirodhi tarala haim |
udariya anla khana pachane ke atirikta bahuta se khataranaka akramakom ko bhi marata hai jo hama logom dvara aksara nigale jate haim |
jaba sharira ki pratirodhakata kama hoti hai aura kitanu samvedanashila kshetrom mem pravesha karate haim, sharira ki sena, rakta ki shveta rakta koshikaem, apani raksha ke lie sainika karyavahi mem utha khada़i hoti haim |
ausata manava sharira mem ve 30000 lakha hoti haim |
jaba kitanu pravesha karate haim, ye koshikaem sankramana ke sthana para ekatra aura samvriddha ho jati haim aura ise avashoshita karake aura pacha kara shatru ko mara deti haim | 
isalie ina bahadura yoddhaom ko phaigosaitsa (khane vali nalika) ka nama diya gaya hai |
bahuta si phaigosaitsa yuddha ke samaya mara jati haim jo sankramita kshetra, jaise eka phoda़a, mem prakopita hoti haim |
ina koshikaom ka nrita sharira mavada ke rupa mem ekatrita hota hai | 
eka dina phoda़a phutata hai, puri mavada bahara ati hai aura phoda़a thika ho jata hai | 
rogapratikaraka rakta mem jivanu ke pratibhedana dvara entijena ya visha ki pratikriya mem bane vishishta padartha hote haim |
ve bimari ke samaya sharira mem raha rahe roganuom se lada़ai karane ke lie bane vishishta padartha hote haim |
yaha rogapratikaraka bimari se lada़ne mem sabase prabhavakari hai aura roganu ina rogapratikarakom ke prati kabhi bhi pratirodhakata nahim vikasita kara sakate haim kyonki ve jivanu dvara utpanna entijena ya visha ki pratikriya ke parinamasvarupa sharira ke tatkalika utpada hote haim |
ve bahuta vishishta hote haim; ve raha rahe jivanu ke sthana para pahunchate haim |
isalie sharira ke ve anga ya bhaga jo bimara nahim haim prabhavita nahim hote haim |
jaham isaki avashyakata hai yaha usi sthana para kriya karega lekina usi samaya yaha sharira ke anya bhagom ya angom para jaham isaki avashyakata nahim hai, kshatikaraka tarike se kama karega |
sabhi aushadhiyom mem abhishta prabhava dene ke atirikta hanikaraka parshva prabhava bhi hote haim |
yadi koi aushadhi lambe samaya taka upayoga ki jati hai, to parshva prabhava badha़ne jari raha sakate haim aura sharira ke anya bhagom ko kshati pahuncha sakate haim |
eka sadharana dardanashaka amashayi shotha utpanna kara sakata hai, pachana kharaba kara sakata hai, aura yaham taka ki hridaya, gurde aura yakrita ko bhi prabhavita kara sakata hai |
lekina adhikatara mamalom mem parshva prabhava bhi ane mem manda hote haim |
ve rogi ko turanta nahim maluma pada़te haim |
kabhi-kabhi ve eka lambe antarala ke bada prakata hote haim aura rogi nayi bimari ke karana ko pahale li gayi aushadhi ke kharaba prabhava se joda़ne mem sakshama nahim hota hai |
koi bhi chikitsaka ka yahi mata hoga ki kisi bimari ka upachara karane ka sabase achchha tarika hai bina aushadhiyom ka upachara |
yaham hama dhyana akarshita kara sakate haim ki jaba yuddha phaigosaitsa aura rogapratikarakom dvara jita ja chuka hai, taba bhi sharira rogapratikarakom ko viyojita nahim karata hai|
yakrita aura lasika tantra mem bane naye rogapratikarakom dvara samaya-samaya para sudridha़ hone ke karana, ve rakta ki dhara mem lambe samaya taka bane rahate haim, isalie yadi vahi roganu lautata hai, to upayukta rogapratikaraka unake akramana ka pratirodha karane ke lie intaja़ara karate haim |
jaba koi bachcha khasara ya chhoti chechaka se pida़ita hota hai, taba isa bimari se ise rodhakshamata prapta hoti hai, dusara akramana samanyatah nahi ata hai |
yaha yogyatama ki uttara jivita hai, juggi-jopada़iyom mem rahane vale bachche samanyatah bahuta si una bimariyom ke lie pratirakshi hote haim jo anyatha susampanna parivarom mem bachchom ko mara sakati haim |
isalie jaba apako jvara ho to pareshana na hom |
lekina isako bahuta adhika badha़ne na dem- jaise 104 digri phaॉrenahaita se upara |
apa ise niche lane ke lie eka barpha ki topi aura eka shitala ponchha ka upayoga kara sakate haim | 
samanyatah yaha 2-3 dinom mem arama aura upavasa ke dvara samanya taka a jaega |
yadi apa dava lekara jvara kama karate haim to apa uchita roga nidana mem badha dalate haim |
rodha kshamata svayam-raksha yantravinyasa hai jisake dvara sharira svayam ko akramana karane vale roganuom aura jivanu se bachata hai |
isamem bi aura ti limphosaitsa, monosaitsa, asthi majja mem koshikaem, lasika granthiyam, peyarsa paichesa aura thaimasa shamila haim |
rakshatmaka rodhakshamata sambhavatah rogotpadaka sukshmajivom ke sagara mem, jinamem hama rahate haim, manavom dvara uttarajivita ke lie mahatvapurna hai |
hama pine, dhunrapana karane, galata prakara ka khana khane, kuchha samaya taka bina vyayama ke rahane mem lipta ho sakate haim aura usi samaya yaham taka ki bahuta sara tanava saha sakate haim, lekina yadi yaha lambe samaya taka jari rahata hai to sharira isake satha kiya gaya anyaya nahim saha sakata hai | 
isake pasa apana samanjana hai aura yaha sharira ko apani uchchatama kshamata taka ina anyayom ke prati dashanukulita karane ki koshisha karata hai |
eka nishchita avastha para isaka samanjana asaphala ho jata hai |
sharira apane para kie gae aura adhika pratada़na ko nahim saha sakata hai | 
yaha tutana shuru ho jata hai aura bimari dikhani shuru ho jati hai |
eka sima hai jisake age sharira isa boja ko nahim sahega, isake bada yaha kahata hai : ‘mainne hara mana liya hai’ | 
isake bada anajane sakriya pratibhagi isa para achanaka akramana karate haim aura apana atidushta karya shuru kara dete haim |
hamare sharira ko svastha rakhane ke do upaya haim |
prathama galata khadya khane mem lipta hone se bachem |
eka uchita poshahara ka palana karem, prati dina vyayama karem, aura tanava se dura rahem | 
sharira ki manga ko dhyana purvaka sunem, jaba apa asvastha mahasusa karem ya kisi bimari se pida़ita hom to chetavani chinhom ke prati dhyana dem aura sharira ke punah svasthya labha mem sahayata karane ke lie apane jine ki shaili ko badalem |
svasthya stara sudhara kara bimari se chhutakara pane ki koshisha karem |
khana aura pine ki adatem sva-niyantrana mem honi chahie, jisase ki jitani jaldi sambhava ho apako bimari se bahara nikalane ke lie sakshama banane ke lie sharira mem hui kshati simita ho | 
kai bara hama sharira ki manga ko nahim sunate aura atidushta adatom ko jari rakhate haim |
uchcha jivana shaili ka tarika jisake hama adi ho chuke haim itana priya hai ki hama sharira ki chhoti mangom aura isake chetavani chinhom ko sunane se bachate haim |
bimari akshunya chalati rahati hai aura eka samaya ata hai jaba yaha yathochita sahayata ke bahara ho jati hai | 
kuchha samaya pahale, eka chikitsa mahavidyalaya ke pradhanacharya apani gardana mem aura kandhe ke joda़ mem darda ki samasyaom ke satha mere chikitsalaya mem ae |
chaubisa varsha pahale, vo mere shalya chikitsa ke prophesara the, aba ve rogagrasta the |
apane adhika yuva dinom mem ve apane anushasana aura sahasa ke lie jane jate the |
eka pratishthita shalya chikitsaka, eka atyuttama adhyapaka, ve eka shrinkhalabaddha dhunrapana karane vale the |
ve apane vidyarthiyom ko hamesha uchita dhanga se taiyara aura kakshaom ke lie samaya para ate dekhana chahate the |
jaba mainne paya ki ve dhunrapana chhoda़ chuke the, to maim unhem dekhakara aura adhika ashcharyachakita tha |
mainne unase puchha kya ho gaya tha |

mainne apane sahakarmiyom mem se eka se paramarsha liya jisane hamase kaha ki shvasanali shotha ka karana dhunrapana tha|
maim dhunrapana nahim chhoda़ saka, isalie shvasanali shotha badatara hota raha |
mainne unase dusari bara paramarsha liya |
unhonne muje eka bara phira dhunrapana na karane ki salaha di |
mainne nahim chhoda़a |
chhah mahine bada maim unake pasa tisari bara gaya |
isa bara ve gambhira the |
eka sampurna parikshana ke bada, unhonne meri sthiti ki gambhirata ke bare mem muje savadhana kiya aura age kaha ki yadi mainne dhunrapana jari rakha to mere pasa jine ke lie lagabhaga chhah mahine honge |
taba isake bada mainne dhunrapana banda karane ka nirnaya kiya | 
mainne 3 mahine taka dhunrapana nahim kiya |
bada mem 6 mahine taka mainne eka sigareta paika aura eka machisa ki dibbi apane jeba mem rakhi| 
dhunrapana chhoda़na asana hai lekina dhunrapana ke pralobhana se bahara nikalana adhika kathina hai |
taba mainne paya ki yadi mere pasa eka sigareta aura eka machisa ki dibbi hoti thi to bhi muje dhunrapana ki chahata nahim hoti thi| 
usa dina muje mahasusa hua ki mainne sigareta hamesha ke lie chhoda़ diya tha |
durbhagyavasha loga vahi galata kama karate rahate haim isa vastavikata ke bavaja़uda ki ve janate haim ki yaha unake lie achchha nahim hai |
yadi apake pasa eka samvedanashila sharira aura eka sashakta dimaga़ hai, to apane sharira ko punah svasthya labha prapta karane mem sahayata karane ke lie, ise eka uchcha svasthya stara taka le jane ke lie, apa apane jine ke tarikom ko parivartita karane mem sakshama ho sakate haim, jisase ki apa eka bara phira, sanyata tarike se, apane ananda ke lakshya mem lipta ho sakate haim |
lekina hama aksara apani svayam ki adatom ke kaidi ho jate haim |
hamara ananda sharira ki avashyakataom se utkrishta ho jata hai |
hama madya, nashili davaom ya sigareta ke adi ho jate haim |
apani praiktisa ke daurana mainne paya ki khane ki lata hamari sabase bada़i bimariyom mem se eka hai |
kevala atyadhika dabava ke antargata loga apane khane ki adata kuchha samaya ke lie parivartita karate haim aura phira ve apane purane tarikom para lauta ate haim |
ve aksara aisi tippani karate haim, ki yadi unhem unaki pasanda ke anusara ahara na mile, to unake jivana ka koi matalaba hi nahim hai| 
ve aise chikitsaka ko pasanda karate haim, jo ahara mem kisi bhi prakara ka parivartana nahim karata ho| 
ve chikitsakom aura aspatalom ko, moti rakama (adhika rashi) vale chikitsakiya bila ada karane aura kitani bhi davaiyam aura khuraka lene ke lie taiyara hote haim| 
kintu yadi ve sharira ki chetavani para dhyana nahim dete haim aura ina davaom va khurakom ke abhashi sahayoga ke anusara karya karate haim, to ye saba bahuta lambe samaya ke lie karagara sabita nahim hoti haim| 
kisi bhi lakhapati se puchhe, ki kya vaha jivana ka ananda utha raha haim? 
kisi kachchi unra mahila se puchhe, ki kya vaha apani khubasurati ka ananda utha rahi haim? 
athaka parishrama mem lipta kisi bhi darshanika se puchhe, ki kya vaha jivana ka ananda le raha hai?
eka akeli budha़i avivahita stri , jisane apani pragati ke lie, pyara aura matritva ko asvikara kara diya ho, usase puchho ki kya vaha apani saphalata ka ananda le pa rahi hai?
aise vyakti apane sharira ko upekshita kara dete haim| 
ve asaphala ho jate haim| 
jalda hi ya kuchha samaya ke bada viphalata hatha lagati hai aura jaba taka apako isa bata ka ehasasa hota hai, taba taka ho sakata hai, ki bahuta dera ho jae| 
ho sakata hai ki taba apa ina sabake lie apane karmom ko doshi thaharae- aura aisa mane ki apa ne apane pichhale janma mem kuchha bure kama kie honge, aura bhagavana una sabake lie apako aba saja de rahe haim| 
achchhi sehata ka ananda lene ke lie, jaruri nahim hai ki apa eka sadhu (santa) ya eka satvika (shakahari aura bina tale hue ahara ka sevana karane vala) banem| 
apako apane sharira ke bishaya mem janakari honi chahie, jo hanikaraka khadya padarthom ke bure prabhava ke khilapha turanta pratikriya karem| 
usake bada apane apa se puchhe ki kya apa paryapta vyayama kara rahe haim aura kya apa chinta se mukta haim?
goliyom aura davaom ka prayoga kie bina, prakritika tarikom se sharirika svasthya mem kaise vriddhi ki jae, isa hetu mulabhuta pathapradarshana ke lie, sehata se sambandhita kitabom ko padha़e| 
jaba apaki sehata achchhi hoti hai, taba apa sahi patha para hote haim, aura jivana ka ananda utha sakate haim tatha chahe gae sabhi maje kara sakate haim| 
kevala adhunika samaya mem hi loga apani sehata ke prati jagaruka hue hai aura unhonne yaha janakari hasila ki hai, ki sehata eka aisi chija hai, jo hamare apane niyantrana mem hai| 
isalie yaha dimaka aura hridaya tatha niji akarshana ki sthiti hai| 
yaha saba kuchha svayam ke andara hi hai| 
samanyataura para loga isa pahalu para thoda़a dhyana dene lage haim| 
sehata aura bimariyam, prakriti ke niyamom dvara sanchalita hoti haim| 
kisi ki bhi sehata ki jimmedari apane svayam para hi nirbhara karati hai| 
ajakala loga aise tarikom ko janane ke prati utsuka hai, jinake madhyama se ve sehata ki suraksha kara sakate haim aura isake khone para, punah ise prapta bhi kara sakate haim| 
ve yaha janane ke lie utsuka haim, ki kaise sharira ka dhyana rakha jae| 
ve isa bata ko samajane lage haim ki khurakom aura davaom ka andhadhundha sevana kuprabhavom ka karana hota hai| 
prakriti ke niyamom ke satha susangata aura saubhagyashali rahem| 
ina niyamom ka kisi bhi prakara se bhanga hona, duhkha ko upajata hai| 
isa prakara ke loga bhitari chinta se grasita hote haim| 
jaba apaki sehata achchhi va sehatamanda hogi, taba apa kriyashila rahenge| 
apako kama karane ki ichchha hogi| 
vaham saphalata ki bhavana ka janma hota hai| 
apa khusha dikhate haim| 
apaki ankhom mem chamaka hoti hai| 
apa apane dimaka mem spashtata mahasusa karate haim| 
ehasasa ki prabalata apako khusha rakhati hai|
thanda़ ke mausama ki apeksha, adhika garmi ke mausama mem thakavata jaldi mahasusa hoti hai| 
dhimi roshani hamem asani se thaka deti hai| 
shoragola vala vatavarana, jaldi thakata hai| 
aisa pesha jisamem bauddhika kama shamila hote haim, vaham kuchha atirikta karaka bhi vyakti ko prabhavita karate haim| 
yadi apa apane karya ko karana pasanda karate haim, to apa use karane mem kama thaka hua mahasusa karenge| 
yadi apaka gharelu jivana apake anukula hai, yadi apa apane charom ora ki chijom para adhika ruchi lete hai, to apa kama thaka hua sa mahasusa karenge| 
ऑksijana ki apurti mem ayi vriddhi, peshiya upapachaya mem uttamata lati hai| 
peshiyom mem draksha-sharkara ki kshinata bahuta kama ho jati hai| 
eka vyakti jo vyayama karata hai, use apane ahara ke prati kaphi niyamita hona chahie| 
use adhika labhakari ahara ka sevana karana chahie; visheshatah yadi vaha kishora aura badha़ti hui unra mem hai| 
yaha peshiyom mem sharkara ke bhanda़ara ko badha़ayega| 
peshiyam dirghakala taka karya kara sakati haim| 
sharkara upapachaya bhi manda ho sakata hai| 
yaha punah peshiyom mem sharkara ki kshinata ki gati ko kama karata hai| 
isa prakara ke logom ke lie karbohaidreta ki prachurata vala ahara anukula nahim hota haim, kyonki isase sharkara ka sampurna riktikarana aura bhi teja gati se hota hai aura isalie ve shighrata se thaka hua mahasusa karate haim| 
bahuta se loga aisa manate haim, ki vyayama kevala bachapana mem hi kiya jana chahie| 
bachchom ke lie, khela sarvottama hai, kyonki ye dilachaspa hote haim aura hara bachche ko khelana pasanda hota hai| 
aise bachche jo padha़ai mem hi vyasta rahate haim aura khela ki upeksha karate haim, unhem khela khelane ke prati protsahita karana chahie| 
yadi yaha (khela) sambhava nahim hai, to unhem vyayama karane ke lie phusalana chahie| 
yadi eka bachcha aise khela khelata hai, jisamem kevala usake pairom ka hi vyayama hota hai, to use sharira ke upari hisse ke bhi vyayama ki shiksha deni chahie| 
jaba eka vyakti apani padha़ai ko pura kara lem, taba use sampurna jivana-kala mem chusta-durusta rahane ke lie vyayama karana arambha kara dena chahie| 
jaba bhi sambhava ho, yaha khelom ya vyayama ke dvara pura kiya ja sakata hai| 
eka vyakti jaba apane chalisavem janma-dina para pahunchata hai, to vaha apane arjana kala ke sarvochcha shikhara para hota hai| 
vaha atyadhika vyasta aura apekshakrita rupa se adhika dhani hota hai| 
vaha vyayama karane ke liye samaya nahim nikala pata aura garishta bhojana karata hai| 
unra ke isa pada़ava para, sharira ke kriyakalapa manda pada़ne lagate haim| 
raktasanchara manda ho jata hai| 
isa samaya motape ki ora jukava hota hai aura joda़ sakhta hone lagate haim| 
isa unra vyayama atyavashyaka hai| 
yaha budha़ape ki prakriya ko dhima kara dega arthata apa mem budha़ape ke lakshana dera se ayenge| 
vaha baidamintana ya golpha khela sakata hai, jo kama shrama-sadhya hote haim| 
kintu yadi apa eka lambe arase ke bada vyayama shuru karane vale haim, to apako ise bahuta hi dhire-dhire arambha karana hoga| 
60 varsha ki unra mem, yaha bahuta kuchha apake sharira ki sthiti para nirbhara karega| 
kevala aise kuchha loga, jo aba bhi niyamita rupa se vyayama karate haim, ve hi adhika parishrama vale vyayama ko kara sakate haim| 
anya logom ke lie, raktasanchara hetu, sphurtikaraka rupa mem tahalana hi paryapta haim| 
apani sharira rupi ghada़i ko, vyayama ki sahayata se ghumavadara banaem rakhem, taki yaha nirvighna rupa se chalati rahem| 
yadi apaki sehata najuka/kamajora hai, to manda gati ko apanaem aura sadharana vyayamom ko hi karem| 
budha़ape mem, sharira eka thaki hui mashina ki taraha ho jata hai| 
ise arambha karana kathina hota hai, kintu dhire-dhire yaha nirvighna rupa se karya karane lagata hai| 
vyayama ko garbhadharana, stanya-kala aura rajodharma ke daurana samapta nahim karana chahie| 
garbhadharana ke daurana vyayama karane se, udara ki peshiyam dridha़ hoti haim aura dardarahita va asana prasava mem madada karati hai| 
garbhadharana ke akhiri haphtom mem, usamem kuchha parivartana kie ja sakate haim| 
prasava ke bada kuchha dinom ka arama jaruri hoga| 
stanya-kala ke daurana vyayama karane se sharira sundara akara mem rahega| 
rajodharma ke daurana bhi vyayama ko nirantara kiya ja sakata hai| 
yadi raktapravaha atyadhika ho raha hai, to isamem parivartana kiya ja sakata hai| 
apa dina mem kisi bhi samaya vyayama kara sakate haim jaba apa khali ho| 
yadi apake pasa vikalpa hai, to dopahara ka samaya sarvottama hai| 
apa use subaha ke samaya mem bhi kara sakate haim| 
samaya ke isa pada़ava para, joda़ thoda़e se sakhta ho jate haim aura ratri ke vishrama ke bada hridaya-gati thoda़i manda ho jati hai| 
kathina parishrama vale vyayama ko arambha karane se pahale, apa thoda़a sa sakriya/saragarmita ho sakate haim|
usase sharira ko koi thakavata nahim honi chahie| 
apa mem dina ke karyom ko sampadita karane ki shakti nahim bachegi|
ise bistara para jane se pahale bhi kiya ja sakata hai| 
yaha manasika karya karane valom ko tarotaja karega| 
aise loga jo dina ke daurana sharirika karya mem vyasta rahate haim, unake lie yaha atyadhika sphurtidayaka hai, kintu ahara ko vyayama karane ke do ghante pahale lena na bhulem| 
rata mem adhika garishta bhojana na karem| 
yadi apaki dinacharya apako dina mem do bara vyayama karane ki svikriti deti hai, to thoda़a vyayama dina ke samaya mem karem aura thona sa atirikta vyayama shama ke samaya mem karem| 
vyayama ko kabhi bhi bhojana lene ke turanta bada mem nahim karana chahie| 
kathora vyayamom ko hamesha, ahara lene ke adhe ghante pahale ya ahara lene ke do ghante bada hi karana chahie| 
jaba ahara ka sevana kiya jata hai, taba rakta pachana angom mem jata hai aura tantrom ki shakti inaki ora ghuma jati hai| 
yadi vyayama ahara lene ke turanta bada mem kiya jae, to rakta aura tantrom ki shakti peshiyom mem chali jaegi aura pachana ashuddha ho jaega| 
sharira ko eka bara mem eka hi karya dena behatara hoga| 
yadi apa halke vyayama jaise-tahalana ya golpha khelana adi karana chahate haim, to apa ise dahi aura salada ya supa aura tosta ya phalom ke halke ahara ko grahana karane ke turanta bada bhi kara sakate haim| 
kintu yadi apa isa taraha ke nape tule (santulita) ahara ke bada bhi,baicheni mahasusa karate hai, to apa 'ahara grahana karane ke adhe ghante bada' ke niyama ka palana kare| 
apako varsha ke 365 dina vyayama karane ki avashyakata nahim hai| 
pratyeka haphte eka dina vishrama rakhe, kintu vishrama vale dinom ki sankhya mem vriddhi na karem| 
yadi apa kisi dina vyayama na kara pae, to agale dina atirikta vyayama na karem| 
yadi apa eka saptaha taka vyayama na kara pae ho, to aisa mata sochie ki vyayama ko punah arambha karane ke lie, kisi vishishta vidhi ki avashyakata hai| 
yadi apa atiutsahi haim, to ho sakata hai, ki eka upayogi pranali ka galata prayoga ho jae| 
jaba apa bimara ho taba vyayama na karem| 
bimari ke daurana, arama karana sabase behatara upachara hai| 
yadi apa vyayama ke daurana ya vyayama ke bada darda mahasusa karate hai athava yadi apake kuchha khasa joda़om mem sujana haim, to apa usi samaya se vyayama banda kara dem| 
vyayama ke agale dina, ho sakata hai ki apa thaka hua mahasusa kare| 
aisa taba ho sakata hai, jaba apane isaki shuruata hi ki ho ya apa ati utsahi ho| 
eka dina ka arama karem, usase apa punah svasthya labha paegem| 
yadi apa kamajora hai ya bahuta thake hue hai, to sakriya vyayama se paraheja karemm| 
apa malisha ke rupa mem nishkriya vyayama kara sakate haim| 
yadi apa tanava grasta haim ya asthibandha phata gaya hai, to vyayama se paraheja karemm| 
apa kuchha dina arama kare, aura phira apa isase mukti mahasusa karegem| 
gatishilata ko banae rakhane ke lie, khinchava vale kuchha vyayama kie jate haim| 
aise vyayamom se arambha karem, jo karane mem asana ho| 
yadi apa jalda parinama ki lalacha mem, adhika shrama-sadhya vyayamom se arambha karate hai, to apako nirasha hatha lagegi| 
jaba apa vyayama arambha karate haim, taba apa mem pida़a, mocha ya joda़om mem khichamva jaisi shikayatem utpanna hogi| 
ye (shikayatem) sthayi nahim hoti haim aura ye dhire-dhire samapta ho jati haim| 
atyadhika vyayama na karem, na hi apani sari atirikta shakti ka prayoga vyayama mem karem, ye vikriti utpanna kara sakata hai| 
sahi vyayamom ka chunava karem aura unhem niyamita tarike se aura eka nidharrrarita samaya ke lie hi karem| 
saptaha mem eka bara, apa apane vyayamom ki sankhya mem, vriddhi kara sakate haim| 
hamesha isa bata ka dhyana rakhem ki vyayama asani se kiya ja sake| 
vyayama ko adhika karana ya kama karana bura hai| 
yadi apa apani mansapeshiyom ki shakti mem ya kasava mem kisi prakara ki koi bhi vriddhi nirantara mahasusa nahim kara rahe hai, to sambhavatah apa vyayama ko thika taraha se nahim kara pa rahe haim| 
apane apa ka parikshana kare, aura apa jana jayegem ki apa kitana kara sakate haim| 
yadi apa nausikhie hai, to 5 minata ke vyayama se arambha karem| 
vyayamom ke bicha ki gati aura rukavatem, nirdharita karengi, ki apa ko kauna se vyayama karane haim| 
apako yaha janakara ashchayarrra hoga ki pancha minata mem apa kitana kara sakate haim| 
apa apane vyayama ke samaya ko hara haphte 2 minata taka badha़a sakate haim, 2 minata se jyada mata badha़ayem| 
apani sharirika jaruratom ke hisaba se apane ko tandurasta rakhane ke lie, samaya ka chunava bada़i savadhani se karana chahie| 
yadi apa niche die gae asanom ke liye pratidina 20 minata bhi kharcha karenge, to apa roga mukta raha sakate haim| 
badha़i hui hdaya dhada़kana ke karana, rakta sanchara badha़ jata hai| 
badhi hui shvasana prakriya ke karana, rakta mem adhika ऑksijana ki apurti hoti hai| 
adhika rakta, ऑksijana ki adhika matra ko, sharira ke sabhi angom mem pahunchata hai| 
isa prakara, rakta sanchara se sabhi anga, adhika ऑksijana va adhika poshana prapta karate haim, tatha sharira ke vibhinna angom dvara nishkasita adhika vyartha padartha isa rakta ke dvara bahara nikala die jate haim| 
isa prakara vyayama ke madhyama se sharira ke samanya svasthya mem sudhara hona arambha ho jata hai| 
vyayama ke daurana, mansapeshiyam sankuchita aura shithila hoti rahati haim| 
sampurna sharira ka lagabhaga adha hissa mansapeshiyom se bana hai| 
jaba ye sankuchita hoti haim, taba unamem nihita sampurna rakta bahara nikala diya jata hai| 
jaba ye phailati haim taba sampurna rakta andara ki ora pravahita hota hai| 
isa prakriya ki punarravriti, mansapeshiyom ko majabuta banati hai| 
isa prakara hradaya, prabhavashali rupa se aura bhi adhika rakta pampa karata hai aura spandamana rakta vahikaem ise sharira ke vibhinna bhagom taka le jati haim| 
rakta sharira ke sabase duravarti kshetrom taka bhi pahuncha jata hai| 
jaba yaha karbanadaiaksaida phephanom taka pahunchati hai, to yaha phephada़om ko uttejita karati hai| 
shvasana ki dara va gahanata badha़ jati hai| 
shvasana mem yaha badha़ta, phephada़om mem karbanadaiaksaida va ऑksijana ki lene va chhoda़ne ki dara ko badha़ata hai| 
isa prakara sharira mem aksijana ki badhi hui manga purna ho jati hai aura sakriya sharira se utpadita hone vali aura adhika karbanadaiaksaida, phephada़om ke madhyama se nishkasita kara di jati hai| 
lasika pravaha badha़ jata hai| 
lasika jo ki pache hue khadya padartho ka vahaka hai, sharira se vyartha padartho ke bahishkarana ka karya karata hai| 
tvacha, gurde, aura antem jaise bahishkarana karane vale sabhi anga, behatara kshamata ke satha adhika karya karane lagate haim| 
tvacha aura bhi kriyashila ho jati hai, romachhidra khula jate hai aura adhika pasina nikalane lagata hai| 
yuriya ke nishkasana ke alava, yaha thandaka pahunchane vale sahayaka ke rupa mem bhi karya karata hai| 
sharira jo vyayama karane se aura adhika garma ho jata hai, ae hue pasine se shitala ho jata hai| 
vyayama ke daurana gurde apane apako isake anukula bana lete hai| 
jala ki manga badha़ jati hai| 
isalie vyayama ke daurana, gurde aura bhi adhika sakriya ho jate haim| 
antada़iyam aura bhi adhika nipunata ke satha svachchha ho jati haim| 
yaha saba udariya bhitti ke sankuchana dvara, chhoti va bada़i antom mem malisha ke prabhava ke karana hota hai| 
upara ki ora se madhyapata(dayaphrama) bhi yahi karya karata hai| 
isa prakara antom ke avayavom ko bahara ki ora le jate hue ,ante kramankuchana kriya ko aura bhi behatara dhanga se karati haim| 
isa prakara bahishkarana ke charom pramukha madhyama- tvacha, gurde , phephada़em aura ante adhika prabhavashali dhanga se karya karate haim, aura sharira ko andara se svachchha karate haim| 
snana dvara bahara se ki jane vali svachchhata ki apeksha yaha saphai karya adhika behatara hai| 
khadya padartho ka pachana va paripachana, vyayama se aura bhi adhika unnata ho jata hai| 
sanchara mem vriddhi, pache hue khadya padartha ke vitarana mem behatara sahayaka hoti hai| 
vyayama ke daurana adhika poshana ki avashyakata hoti hai| 
purani koshikaem nashta ho jati haim aura nayi koshikaom ka nirmana hota hai| 
atah isa prakara, sharira hara samaya apana navinikarana karata rahata hai| 
nayi mansapeshiyom ke nirmana ke lie, koshikaom ke nirmana ki sanragi ki avashyakata hoti hai| 
sharira adhika ahara ki manga karata hai| 
madhyapata(dayaphrama) ke dvara yakrita aura agnashya ki malisha ho jati hai| 
phephada़om ke vistara va sankuchana se madhyapata joshapurna dhanga se ghatata aura badha़ta hai| 
yakrita aura agnashya ka stravana badha़ jata hai aura isa prakara khadya padartho ke pachana ke lie, aura bhi adhika matra mem pachaka rasa upalabdha ho pate haim| 
isa taraha vyayama ke dvara bhukha badha़ti hai aura ahara behatara dhanga se pachata aura paripachita hota hai| 
vyayama ke daurana purani mansapeshiya koshikaem nashta ho jati haim| 
ve bahara nikala di jati haim, aura nayi mansapeshiya koshikaem nirmita ho jati haim| 
ye nayi koshikaem bada़i aura majabuta hoti haim| 
nayi mansapeshiya koshikaem, dugdhanla kahalane vale thakavata-visha ki kama matra utpadita karati haim| 
ye koshikaem prabhavashali hoti haim aura sharira ke nishkasana angom para kama dababa dalati haim| 
sanyojaka utaka snayu ,jo mansapeshiyom ko haddiyom se joda़te haim, ka navinikarana ho jata hai| 
ve mansapeshiyom ki kriyaom ko behatara banane mem sahayaka hote haim| 
majabuta haddiyam aura lochadara joda़, joda़om ke nirvighna kriyanvayana mem madada karate haim| 
vyayama se hradaya ko bhi labha hota hai| 
jaise hi vyayama ke daurana hradaya ki mansapeshiyom ka sankuchana aura shithilikarana sashakta ho jata hai, ye mansapeshiyam vyayamita ho jati haim aura majabuta bana jati haim| 
hradaya janma se lekara nrityu taka lagatara dhada़kata rahata hai| 
yaha vrihada matra mem karya karata hai| 
pachchisa varsha ke vyakti ke hradaya ke dvara, 24 ghanto mem jitana karya kiya jata hai, vaha chhah tana vajana ko eka phuta ki unchai taka khisakane ke samana hota hai| 
hradaya isa karya ko apane apa ko bina thakaye karata hai| 
dhada़kanem jo hoshohavasa mem niyantrita nahim hoti , unhem nasom ke dvara niyantrita kiya jata hai| 
aise bahuta se yogi haim, jo ekagrata ke madhyama se apane hradaya ki pranali ko niyantrita kara sakate haim| 
ve apani ichchha se, apani hradaya ki dhada़kana ko ghata sakate haim, aura yaham taka ki use kuchha kshanom ke lie roka bhi sakate haim| 
vyayamita hradaya se, phephada़e aura bhi adhika rakta prapta karate hai| 
jaba phephada़om mem adhika rakta ata hai, to phephada़e shighrata se usaka shuddhikarana kara, use vapasa hradaya ki ora bheja dete haim| 
isa uddeshya ke lie, shvasa teja va gahari ati hai| 
phephada़e apane apa ko vyayama ke anurupa samayojita kara lete haim| 
sine ka prasara vrihada ho jata haim aura phephada़om ki nayi koshikaem nirmita hoti haim| 
phephada़e aura bhi vrihada va adhika lochadara ho jate haim, jisase phephada़om ki aura adhika vayu (ऑksijana) lene ki kshamata mem vriddhi ho jati haim| 
antaraparshuka mansapeshiyam vikasita ho jati haim aura sansa lene ke daurana pasaliyam adhika phaila jati haim| 
isa prakara phephada़om ki kshamata badha़ jati hai |
sansa andara lene aura bahara chhoda़ne ke bada sine ko mapane se yaha mahasusa bhi ki ja sakati hai |
vyayama karane ke bada bhi sansa lene para chhati ka ghera dhire dhire badha़ jata hai |
hamare margadarshana ke lie hamare pasa eka achchha jancha nuskha hai |
vyayama karate samaya yaha eka suraksha dvara (kapatika) evam eka sima ke rupa mem kama karata hai |
hamem isa bata ka andaja़a laga lena chahie ki behatara parinama ke lie hamem kitana vyayama karana chahie |
usase jyada karane para hama thakana mahasusa karane lagate haim |
jaba hama dauda़te haim ya vyayama karate haim to sansem aura lambi ho jati haim |
jisase phephada़om mem adhika ऑksijana ane lagati hai aura rakta mem avashoshita hoti jati hai |
aura jaba hama sansa bahara chhoda़te haim to karbana daiऑksaida rakta se hava mem bahara nikala di jati hai |
sansa mem isa vriddhi ko atirikta vayu kaha jata hai |
yaham taka ki vyayama banda karane ke bada bhi , hamara hradaya taba taka teji se dhada़kata rahata hai jaba taka ki sabase vishaile tina padartha sharira se bahara nahim nikala jate |
isase atyadhika thakana mahasusa hoti hai aura yaha thakana sharira ke lie kaphi kashtaprada hoti hai |
samanya thakana kuchha hi ghantom mem ya eka rata ki ninda ke bada dura ho jati hai |
apake karya karane ki kshamata aura karya-kushalata badha़ jati hai | 
prashikshana dene ke stara ka andaja़a ऑksijana ki khapata aura ekatrita hue laiktika esida ki matra ko mapane se lagaya ja sakata hai |
isake lie eka sarala tarika taiyara kiya gaya hai |
eka prashikshita vyakti mem , vyayama ke daurana palsa dara mem dhimi vriddhi evam vyayama ke bada turanta aramadayaka sthiti ki vapasi hi prabhavakari rakta pravaha ki visheshata hai |
kathina abhyasa ke daurana palsa dara 140 prati minata taka jati hai |
vyayama rokane ke bada pancha minata mem yaha sthiti samanya ho jani chahie |
palsa dara mana ki bhavanaom se bhi prabhavita hoti hai |
yaha eka bahuta hi sadharana jancha hai aura asani se kisi bhi vyakti dvara istemala ki ja sakati hai |
sharirika tandurusti ki yaha jancha hridaya ki tandurusti ke lie eka vishvasaniya mapadanda hai |
bujurgom mem, prashikshana ke prabhava ko isi prakara pradarshita kiya ja sakata hai |
eka tanavapurna sthiti apako jaldi thaka sakati haim |
phurtilapana ,sahanashilata , nanrata aura takata sharirika tandurusti ke parikshana haim |
ina gunom ke sammishrana se , jo eka vyakti ko sashakta sharirika gatividhiyom ko achchhi taraha se pradarshita karane layaka banate haim ,sharirika tandurusti badha़ti hai |
svastha loga bhi sharirika taura para asvastha ho sakate haim kyonki ve niyamita rupa se vyayama nahim karate |
achchha svasthya evam sharirika tandurusti donom eka hi bata haim |
jo loga sharirika rupa se tandurusta hote haim ,ve adhika patale rahate haim aura bimariyom ke prati unaki pratirodhakata adhika hoti haim aura agara ve bimara pada़ bhi jate haim to jaldi hi thika bhi ho jate haim |
ve aura adhika satarka aura tanava mukta rahate haim |
sharirika tandurusti eka vyaktigata jimmedari hai |
adhikansha loga sharirika rupa se asvastha hote haim kyonki ve ajnanatavasha paryapta vyayama nahim karate |
kishoravastha aura madhyama unra ke bicha ke varsha sharirika tandurusti ke lie vahuta mahatvapurna varsha haim |
paryapta ninda ana , thika se khana , niyamita chikitsiya jancha aura danta chikitsiya dekhabhala prapta karana aura vyaktigata sapha saphai banae rakhana adi adatem sharirika tandurusti banae rakhane mem sahayaka hotim haim |
jyada khane se ya galata prakara ka bhojana karane se , dhunrapana , nashili davaom aura atyadhika sharaba ke sevana se svasthya ko nukasana ho sakata hai |
eka vayaska vyakti ki sharirika tandurusti usake bachapana ki tandurusti se prabhavita rahati hai |
aisa lagata hai ki koi bhi apane svasthya ke lie nahim sochata |
eka mahine ke lie apa apane apa ko pratidina vyayama karane ke lie badhya evam anushasita kijie aura phira yaha apaki eka adata bana jaegi |
apa ke lie yaha eka atirikta adata bana jani chahie |
taba apa ke lie khushi ke sabhi daravaje khula jaengem , apa khula kara jiengem aura jivana ke hara eka pala ka ananda uthaengem |
vyayama karane ka itihasa haja़arom varsha purana hai |
eka prachina pustaka malayuddha mem vyayama ke vistara ka ullekha milata hai aura kathina vyayama karate vakta agara kuchha galata ho jata hai to yaha pustaka dakshata se ilaja karane ka paramarsha bhi deti hai |
hanumana ki virata ko dekhakara kushti ,danda baithaka,mugdara adi jaise bharatiya vyayamom mem dridha़ vishvasa hota hai evam takata prapta karane ke lie hanumana ko puja bhi jata hai |
darda evam anya bimariyom ke nivarana ke lie prachina china ka kunga phu,dharmacharyom dvara nirdharita karmakandi mudraom aura gatiyom ki eka shrrinkhala thi |
prachina yunaniyom ke bhi svasthya ke apane bhagavana the |
asala mem vaham svasthya ke mandira bhi hote the |
pura rashtra apane rajaom sahita,apane sharira ko tandurusta ,chharahara aura sundara rakhane ke lie ina mandirom mem puja kiya karata tha | 
adhunika chikitsa ke pita hippokretsa ne kaha hai ki sthula angom ki kshirnata una logom mem sabase jyada hoti hai jo apane paira upara karake baithate haim aura unase vyayama nahim karate |
jo loga khuba paidala ghumate haim unamem kshirnata bahuta kama hoti hai |
motapa kama karane ke lie hippokretsa teja kadamom se niyamita paidala ghumane ki salaha dete haim |
eka lambe samaya ke vishrama ke bada kathina vyayama shuru karane para vaha eka satarka paddhati apanane ka nirdesha dete haim |
adi manava isaka abhyasa kiya karate the |
aja ke adhunika samaya mem ,udariya bhitti ko majabuta banane ke lie phira se hama purvakalina paidala ghumane-phirane ke vyayamom ke aura janma dene ke turanta pashchat ke vyayamom ke siddhanta ko apanane lage haim |
rusi lekhaka esa. pi. bodakina ,una logom ko pranayama karane ki salaha dete haim jinako dila ka daura pada़ chuka hai aura isako karane se mamuli parishrama karane para ve hamphenge nahim |
kamajora mansapeshiyom ko majabuta banane ke lie vishesha rupa se phalija़ pada़ne ki sthiti mem ,eka sahayaka ke rupa mem eka saralikarana taiyara kiya gaya hai |
svaichchhika mansapeshiyom para phira se niyantrana pane ke lie ye bahuta hi upayogi haim |
loga janate haim ki apa kaise dikhate haim aura apa janate haim ki apa kaisa mahasusa karate haim |
vyayama karane se apaki dikhavata mem sudhara hoga ,apaka lada hua peta pichaka jaega aura apaki mansapeshiyam ubhara jaengi |
apaka vyaktitva aura chala dhala behatara ho jaenge |
apake kadamom mem phurti a jaegi |
apa eka tarotaja़a , yuva vyakti ki taraha mahasusa karenge |
eka vyakti ki palsa dara prati minata 72 hoti hai |
yadi apa thoda़i dura taka hi daunda़ rahe haim ya adhika kathina vyayama kara rahe haim to apa hamphane lagenge ,aura lambe samaya taka dauda़ne ke layaka nahim rahenge |
aisi sthiti mem apaki palsa 140 prati minata taka pahuncha sakati haim |
yadi apa niyamita rupa se vyayama karate haim to apaki palsa dara dhire-dhire kama hone lagegi |
eka saptaha , eka mahine, 3 mahine, eka sala mem apa vishrama ki sthiti mem ,apani palsa dara mem eka ullekhaniya kami mahasusa karenge |
yadi apa kaphi dura taka daunda़ rahe haim ya adhika kathina vyayama kara rahem haim to aba apa hamphane se pahale hi adhika lambe samaya taka dauda़ne ya vyayama karane ke layaka honge |
isaka matalaba yaha hai ki aba apa bina thake hi adhika samaya taka kama karane mem sakshama haim |
dina ki samapti para aba apa nidhala ya thaka hua mahasusa nahim karenge |
aba chunki apa adhika jivanta evam urjavana ho gaye haim aura jindagi ko bharapura jine ke lie taiyara ho gaye haim , taba bhi apa ke andara jivana ka ananda lene ke lie urja ka eka bahuta bada़a bhandara sanchita ho gaya hai |
adhikansha dila ke daure aura sadame bhavanatmaka tanava ki avadhi ke daurana pada़te haim |
yaha vyapara mem nukasana hone ke samaya ho sakata hai ya yaham eka tanavapurna vishva kapa dekhate samaya ho sakata hai |
tanava ke daurana , rakta mem edrinelina namaka eka harmona sravita hota hai |
jisake karana palsa dara badha़ jati hai ,raktachapa badha़ jata hai ,rakta adhika gadha़a ho jata hai aura rakta sharkara badha़ jati hai |
yadi apa niyamita rupa se vyayama karate haim , to apa ina saba tanava sanketom ka samana kara sakate haim |
dila ki kasarata karana apako ,apata sthitiyom ka samana behatara tarike se karane mem madada kara sakata hai |
hridaya ki mansapeshiyom ki koronari dhamani ki kai shakhaem hoti haim jo bahuta adhika matra mem rakta keshikaom se juda़i rahatim haim |
eka apata sthiti ke daurana jaise hi palsa teja़ hoti hai , adhika rakta pampa hone lagata hai , sabhi rakta keshikaem khulane lagati haim aura isilie taba aisa koi bhi mauka nahim hota ki hradaya ko ऑksijana ki kama apurti ho jisase hradaya ke phela hone ka dara bana rahe | 
rakta keshikaem khula jati haim aura raktachapa nahim badha़ta |
jaba apa vyayama karate haim to rakta mem sharkara kharcha ho jati hai aura eka aisi apata sthiti se nipatane ke lie apa hamesha eka behatara tarike se taiyara rahate haim | 
yaham taka ki agara isa taraha ke kisi vyakti ki koronari dhamani asthayi vasa dvara avaruddha hai , to hradaya ऑksijana ki kami se prabhavita nahim hoga |
koronari dhamaniyom ki bahuta adhika shakhaem hone ke karana evam hradaya ki divara ki rakta keshikaom mem rakta ki atyadhika apurti ke karana aisa sambhava hota hai |
ऑksijana ki paryapta apurti bani rahegi evam hradaya bhi upekshita nahim rahega |
jaba apa kisi kara ka petrola tainka bharate haim , to pura bharane para yaha bahane lagata hai |
agara apa pratidina 2800 kailori urja grahana kara rahe haim aura niyamita karya mem kevala 2400 kailori urja ka upayoga karate haim ,to baki bachi 400 kailori urja sangrahita ho jati hai |
bachi hui urja jaba sangrahita hoti rahati haim to apaka vajana badha़na shuru ho jata hai |
jaba apa vyayama karate haim , to vasa mansapeshiyom mem badala jati hai , mansapeshiyam vasa ki tulana mem bhari hoti haim , to vyayama dvara apaka vajana ghatane ki bajaya badha़ne lagata hai |
lekina apa behatara mahasusa karate haim aura behatara dikhate bhi haim |
vajana kama karane ke lie sabase achchha tarika kama kailori ka ahara lene ke satha-satha vyayama karana hai |
insana mem kai shanta , lachile aura majabuta joda़ hote haim |
ve sharira ka sampurna vajana eka jagaha se dusari jagaha bina thake le jate haim |
ridha़ ki haddi mem chhote-chhote joda़om ki eka shrrinkhala una saba mem sabase mahatvapurna haim |
unake lachilepana ke karana hi hama juka sakate haim , muda़ sakate haim ,ghuma sakate haim aura phira bhi hama una para maja़buti aura niyantrana rakha sakate haim |
unra badha़ne ke satha-satha ye joda़ sakhta hote jate haim |
jaba taka apa apane joda़om ko vyayama ki madada se lachila rakha sakate haim , apa utane hi lambe samaya taka javana raha sakate haim | 
ridha़ ki gatishilata ke lie yoga vyayama (asana) sabase achchhe haim |
yoga se tantrika tantra majabuta ho jata hai |
ridha़ apake sharira ke lie eka baitari ki taraha hai |
yadi apa isaka upayoga karate rahenge , to yaha lachili bani rahegi |
yaha apako yuva hone ka ehasasa karata hai aura sharira ki anya sabhi kriyaom ko eka majabuta sthiti mem rakhata hai |
kisi pustaka ki sahayata se asana sikhana kabhi-kabhi khataranaka ho sakata hai |
apa asanom se kevala tabhi labhanvita honge jaba inhem karate samaya asana ya apake baithane ki mudra sahi ho | 
isa karana se eka achchhe adhyapaka se asana sikhana hamesha surakshita hota hai |
maim shirshasana (shirsha para khada़a) ki salaha nahim deta hum |
kevala ve loga jinaka achchha svasthya hai shirshasana se labhanvita ho sakate haim|
kamajora loga sarvanga asana (kandhe para khada़a) karake usi prakara labhanvita honge |
vyayama ki dara ghatai ya badha़ai ja sakati hai |
yadi apa mahasusa karate haim ki apane paryapta vyayama nahim kiya hai to apa gati badha़a sakate haim jaba taka ki apa thaka hua na mahasusa karem |
lambavata sthiti ki apeksha avanata sthiti mem kiye gaye vyayamom ko adhika parishrama ki avashyakata hoti hai |
yadi apako sahare ki avashyakata hai to apa vyayama karane ke lie eka vishishta upakarana chuna sakate haim |
yadi apako mansa-peshiyam pasanda haim, to mansa-peshiyom para tanava dalie, phira pratirodhakata badha़ana jari rakhie | 
eka nishchita bindu taka jukana asana hai, isa bindu ke age apa apani mansa-peshiyom mem tanava mahasusa karenge aura yadi apa phira bhi age jate haim to apa darda mahasusa karenge |
yadi apa vyayama karana jari rakhate haim to joda़om ki gatishilata badha़ti jati hai |
darda ka bindu kuchha dinom mem tanava ka bindu ho jata hai aura dhire-dhire apa phira bhi bina darda ke isa bindu ke age ja sakate haim |
yaha siddhanta susthira joda़om vale rogiyom ko vyayama karane mem upayoga kiya ja chuka hai |
gatishilata aura darda aparoksha rupa se sambandhita haim |
vyayamom ki sahayata se, jaise-jaise vaha apani gatividhi badha़ata hai, darda kama hota jata hai, aura antatah vaha pida़amukta ho jata hai |
yadi yaha gatividhi darda ke bindu taka puri ho jati hai, to mansa-peshiyam sanrakshatmaka pratikriya mem tana jati haim aura abhishta parinama nahim prapta honge |
eka avyavasthita aura laparavaha shaili mem vyayama na karem |
vyayamom ko dhyanapurvaka, achchhi taraha se aura uchita gati se karana chahie |
jaba apa vyayamom ko karate haim, to unhem puri ekagrata ke satha kijie |
pratyeka kshana apake mastishka ko maluma hona chahie ki apaka sharira kya kara raha hai |
apako vyayamom ko eka saja़a ki taraha nahim karana chahie |
apa vyayamom ko sangita ke satha-satha kara sakate haim ya eka vishishta samaya ke lie uchita vyayama chuna sakate haim|
apane vyayamom ko shvasana ke satha sanyukta kijie |
yaha apane achchhe prabhava ko samvriddha karega, isalie unhem taba karem jaba apa tanavamukta hom |
yaham taka ki eka halke tanava se tantrika urja ka apakshaya hota hai |
vyayama eka hi bara mem nahim pure kie jane chahie | 
pratyeka vyayama ke bada, thoda़i dera ke lie arama karem |
yaha apaki vyayama karati mansa-peshiyom ko rakta se purnatah bhara jane mem sahayata karega |
yaha phephada़om ko rakta se sabhi thakane vale visha ko akarshita karane aura sapha karane mem sahayata karega |
ruchi ko jivita rakhane ke lie apane vyayama ka khela-kuda, naukayana ya tairaki ke satha ekantarana kijie |
apa vyayama ka ekantarana yogasana ke satha kara sakate hai |
vyayama eka dina kijie aura yogasana agale dina |
yogasana sharira ko svastha aura tantrikaom ko majabuta banata hai |
yadi vyayama mahatvapurna haim, to arama bhi usi prakara hai|
arama ke bada, apa urjavana mahasusa karenge aura yaha urja vyayamom ko karane mem upayoga ki jani chahie |
yadi apa isaka upayoga nahi karate haim, to ya to yaha vyartha ho jata hai ya sharira isako utpanna karana banda kara deta hai |
arama ke daurana, sharira apashishta padarthom ko nikalata hai, nayi koshikaem banata hai aura purani ki marammata karata hai |
jaba apa thaka hua mahasusa karem to apako arama avashya karana chahie |
yadi apane uchita matra mem vyayamom ko kiya hai to apa eka aramadayaka ninda ke bada taja़a ho jaenge|
ninda ki avashyakata bhinna-bhinna logom ke lie bhinna-bhinna hoti hai |
bahutom ka kama chhah ghantom se chala sakata hai; anyom ko 8 se 10 ghantom ki avashyakata ho sakati hai |
yadi apa achchhi taraha se arama karana janate haim, to apani ninda se pahale arama karem, apaki ninda ki avashyakata kama ho jaegi |
eka majabuta aura sugathita sharira jivana ki eka mahana sampatti hai |
yaha viparita paristhitiyom ka samana karane tatha apake peshevara jivana vritta aura dina-pratidina ki kriyaom mem adhika unchaiyam prapta karane mem sahayata karata hai |
eka prajnavana poshahara ke satha nitya kriyaom ki eka suniyojita kramavyavastha apako aura unchi upalabdhiyom ki ora le ja sakata hai aura manasika tatha bhavanatmaka kathinaiyom se ubarane mem sahayata karata hai aura antarmana se prasannata lata hai |
achchha svasthya aura utkrishta sharirika shakti vyayamom aura vishishta niyamom ke palana dvara uparjita ki ja sakati hai |
jaisai-jaisai apa paripakvata ki ora badha़te haim yaha prakritika rupa se nahim ata hai |
hama jo khana khate haim vaha mahatvapurna hota hai |
khana dainika karya ke lie urja pradana karata hai, vriddhi mem sahayata karata hai aura vitamina tatha khanija pradana karata hai jo hamem achchha svasthya banae rakhane mem sahayata karate haim |
shvasana sharira ki vriddhi ke lie avashyaka sabase mahatvapurna karakom mem se eka hai |
hama bina bhojana ke kai dinom taka raha sakate haim, bina pani ke kuchha dinom taka, lekina bina hava ke hama pancha minata taka bhi nahim raha sakate |
sharira ki vriddhi ke lie shuddha vayu eka mahatvapurna karaka hai |
apane sone ke kamare ki khida़kiyom ko khola dem |
ashvasta hom ki vaham dina aura rata taji hava ki apurti hoti hai |
adhikadhika loga kamare ka tapamana, ardrata aura samvatana ko vatanukulakom ke dvara niyantrita karate haim |
kama se kama, yaha uchita samvatana mem sahayata karata hai |
hama janate haim ki 50% sharira mansa-peshiyom ka bana hai |
unaki thika prakara se vriddhi aura vikasa ke lie upayukta vyayama avashya karana chahie | 
jaba eka purusha apane sharira ka vyayama karata hai, to vaha hatta-katta ho jata hai aura jaba eka stri apane sharira ka vyayama karati hai to yaha gola-matola aura vakriya hone mem sahayata karata hai | 
prakriti ne stri ke sharira ko isa prakara banaya hai ki mansa-peshiyam usi prakara se nahim badha़ti haim jaise ki purusha ki mansa-peshiyam badha़ti haim |
bahuta se loga samajate haim ki achchha svasthya kisi ajnata vastu se ata hai | 
aba yaha suspashta hai ki uttamatara svasthya koi bhi prapta kara sakata hai jo eka prayatna aura eka chhota sa tyaga karane ke lie taiyara hai |
bahuta se loga adhika sharirika urja vale aura dirgha kala taka yuva lagate haim |
yaha taba hota hai jaba ve ise apane janakom se vanshanukrama se prapta karane mem saubhagyashali hote haim |
ve apane sharira ka dushprayoga aura durvyavahara karate dikhate haim aura phira bhi apaneapa ko svastha rakhate haim |
ve, vastava mem, apane purvajom se prapta jivanashakti ka vyaya kara rahe haim |
lekina unaki santati ko isaka mulya chukana pada़ sakata hai |
yadi hama una logom ke jivana ka adhyayana karate haim jinaka lamba jivana raha hai, unaki dirghayu ka rahasya unake poshahara aura dainika vyayamom mem paya gaya hai|
mahilaem purushom se adhika teja़i se budha़i hoti haim, kyonki, purusha apaneapa ko adhika sakriya rakhate haim |
jaba apa vyayama karate haim to apake pasa eka maja़buta hridaya, maja़buta udara hota hai aura isilie apake mastishka ko bhi behatara poshana milata hai |
apa eka vyayama chunie aura ise niyamita rupa se karie, lekina, yada rakhie, eka sundara sharira ka nirmana karane ke lie varshom lagate haim |
pratyeka ke pasa unake ilake mem vyayamashala nahim hoti hai |
phira bhi, yadi koi vyayamashala jana vahana nahim kara sakata hai, to sabase achchha vikalpa hai tahalana, teja़i se chalana ya dhire-dhire dauda़na |
yaha vyakti ki ayu aura takata ke anusara niyantrita kiya ja sakata hai |
yadi apa dukhi ya udasa haim, to tahalem aura soddeshya tahalem, aura apa apani udasi se bahara a jaenge |
yada apako koi samasya ya bhavanatmaka tanava hai, to ghara se bahara nikalem aura tahalane ke lie jaem aura apa ise hala karane mem sakshama honge |
tahalem aura apake pasa sochane ka samaya hoga |
apani samasya ko samajem aura jivana ko achchha banane ke lie apani niti ki yojana banaem |
tahalana eka ruchikara kriya hai |
apa bahuta se naye mitra banaenge aura yaha apako manava kriyaom ko bantane ka ehasasa dilaega |
yaha budha़e aura adheda़ unra ke logom ke lie achchha hai |
yuvaom ko aura adhika kathora vyayama karana chahie ya prabalata se dauda़na chahie | 
unhem krida़aom mem bhaga lena chahie |
striyom ke lie vyayama purushom ki apeksha adhika mahatvapurna haim |
ve takata aura sundarata prapta karati haim |
yadi ve hrishta-pushta haim, to ve apane bachchom mem shakti pradana karengi |
yadi ve kamaja़ora haim, to unake bachche kamaja़ora honge |
asaphala shadiyom ka pata sharira ki kamaja़oriyom aura ninna jivanashakti mem lagaya ja sakata hai |
yadi stri adhika sajiva, adhika sunanya aura svasthya se sampanna hai to shadi mem asaphalataem kama hongi
tairaki eka achchha vyayama hai |
tairaki mem sharira ke pratyeka bhaga ka vyayama hota hai kyonki apa hava mem eka chida़iya ki taraha, sabhi dishaom mem chalane ke lie mukta hote haim |
ainkiloja़inga spaॉndilaitisa mem anga-vikriti samanyatah baithane ya letane ke daurana kharaba mudra ke karana hoti hai |
dhire-dhire ina marija़om ka jukava aura gatishilata ki pratibaddhata badha़ti jati hai | 
unhem eka kathora bistara para eka patale takie ke satha sone ki koshisha karani chahie |
unhem kuchha minatom ke lie bina kisi sahare ke pitha para aura aundhe sone ki koshisha karani chahie |
unhem hamesha eka sidhi prishtha vali kursi para baithana chahie aura vyayama karana jari rakhana chahie yaham taka ki jaba unhem thoda़a sahana karane layaka darda ho taba bhi |
isa prakara ve ane vali anga vikriti ko roka ya kama kara sakate haim, adhika gatishilata pa sakate haim aura darda bahuta kama ho jaega |
ye sandhiyam shighra hi sakha़ta ho sakati haim aura gatishilata kho sakati haim|
yaham taka ki roga ke akramana se jaba pida़a tivra ho, taba bhi eka dhimi sthaniya sandhi gatividhi dina mem eka ya do bara, thoda़e samaya ke lie prarambha karani chahie|
rogi ko svayam se ya chikitsaka dvara sandhiyom ko sahara dekara, vyayama karane ke lie protsahita karana chahie|
vyayama ke bada pida़a mem vriddhi gati ki atyadhika matra ki ora sanketa karata hai|
prabhavita sandhi ke charom ora ki mansapeshiyom ko rogi ko sandhi ko gati dete hue ichchhapurvaka sikoda़na tatha shithila karana chahie, ise sthira vyayama kaha jata hai, aura hara ghante mem 2-3 minata taka kiya jata hai|
jaise-jaise rogi ki sthiti mem sudhara hota hai, vyayama ko badha़ana chahie, para bahuta jaldi-jaldi nahim, nahim to bimari ka punaravartana ho sakata hai|
aba dhire-dhire vyayama ko badha़ana chahie kintu atyasakti se bimari ka punaravartana ho sakata hai| 
rogi apane arama ke samaya ko kama kara sakata hai|
khapachchi ko lambe samaya ke lie hataya ja sakata hai ya kevala rata ke samaya rakha ja sakata hai|
dhire-dhire, sandhiyom ke vyayama se, gatishilata badha़ jaegi|
rogi ko apane apa vyayama karana chahie aura kuchha vyayamom ko chikitsaka ki sahayata se kara sakata hai|
vaha eka vakara ki sahayata se ya kisi ke sahare se chala sakata hai|
vaha baisakhi se chala sakata hai|
baisakhi pakshaghata se bachane ke lie kuhani baisakhi ki apeksha kankha ki baisakhi ka upayoga karem| 
baisakhi ke bada rogi chhada़i ke sahare se chala sakata hai|
dhire-dhire rogi apani mansapeshiyom ke niyantrana ko badha़ana prarambha karata hai|
yaha saba usaki ayu, rogi ke prayatna tatha vaha isase shighra hi chhutakara pane ke lie kitana kritasankalpa hai, isa para nirbhara karega|
jirna avastha mem vyayama atyavashyaka haim, nahim to vikriti nishchita hai|
yadi sandhiyom ka uchita rupa se vyayama nahim kiya gaya, to ve muda़ne ki avastha mem bhi sthira ho jati haim, mansapeshiyam patali tatha vyartha ho jati haim aura uttejaka utpada sandhiyom ke charom ora jama ho jate haim|
amavati sandhishotha mem kai anya vikritiyam hoti haim|
yadi rogi narama bistara para sota raha hai, to graiva merudanda, thoda़i ke vakshasthala ke samipa pahunchane taka jukati hai|
rogi aisa lagata hai jaise vaha eka gole mem guda़i-muda़i hokara leta hai|
haddiyam asthisushira ho jati haim aura prayah thoda़e se sharirika prashikshana se tuta jati haim|
ina rogiyom ko eka suniyojita aura paryavekshita vyayama upachara ki avashyakata hoti hai|
dhire-dhire unaki gatishilata tatha mansapeshiyom ki kshamata badha़ti jaegi|
vyayama se purva sandhiyom ko kuchha ushma upachara bhi die ja sakate haim|
yaha mansapeshiyom ko shithila karane mem sahayata karata hai tatha rakta apurti ko badha़ata hai aura isilie yaha vyayama adhika prabhavi ho jata hai|
yaha dhire-dhira sandhi ke kada़epana tatha sankuchana ko sudharata hai aura isamem chikitsaka tatha rogi donom ki ora se bahuta adhika dhairya tatha driढ़ta ki avashyakata hoti hai|
prabhavita sandhi ke lie pahala vyayama chikitsaka dvara eka lambe satra mem sikhaya jana chahie| 
chikitsaka dvara vyayama ka kama se kama saptaha mem do bara nirikshana hona chahie tatha sudhara para dhyana diya jana chahie|
vyayama tala sarvottama sahayaka haim, jaham rogi apani sandhiyom ko svatantratapurvaka sabhi dishaom mem chala sakata hai|
vishesha sandhi vyayamom ke satha-satha samanya vyayama bhi diya jana chahie|
mansapeshiyom ki kshamata ko badha़ane ke lie pratirodhaka vyayama die jane chahie jo sharira ki gatividhi mem tatha kama karate samaya sahara deti haim|
yadi apaka vajana 70 kilo hai, to pani mem apake pairom ko kevala 7 kilo ko sahara dena hoga| 
yadi apa kamaja़ora pairom ke karana jaise adharangaghata mem ya kashtakara ghutanom ke karana jaise sandhishotha mem, khada़e nahim ho sakate, to sambhavatah apa santulana karane mem sakshama honge; pani apako aisa karane mem sahayata karega|
pani sanstambhi mansapeshiyom ko shithila karata hai taki apa pani mem adhika asani se vyayama kara sakem|
pani tanava kama karata hai aura yaha tanavapurna rogiyom ke lie mahattvapurna sahayata hai|
yadi tala ke pani ka tapamana 37 digri sentigre़da hai to yaha apako behatara arama dega tatha darda aura pida़a ko kama karega|
ghutanom ko sidhe rakhate hue pairom ko eka ke bada eka chalaem|
kuchha minata chalane ke bada, rogi ke, pindaliyom ki mansapeshiyom mem tivra pida़a hoti hai aura vaha kuchha minatom ke arama se kama ho jati hai|
yaha rakta vahikaom ki sankirnata ke karana hoti hai|
chalate samaya rakta apurti mem vriddhi honi hi chahie|
jaba rogi chalata hai to rakta ki avashyakata badha़ jati hai aura yadi, rakta vahikaom ki sankirnata ke karana, rakta ki apurti mem vriddhi nahim hoti hai, to vaha eka araktatajanya sthiti mem pahuncha jati hai|
ina rogiyom ko dhunrapana banda kara dena chahie, kyonki aise rogiyom mem sigareta pine ka hanikaraka prabhava hota hai|
unhem pratidina eka ghante taka teja़i se chalane ke lie kaha jata hai|
jaba pida़a ki punaravritti hoti hai to unhem kuchha minatom ke lie arama karane aura taba teja़i se chalana punah arambha karane ki anumati di jati hai|
dhamanikakathinya abhilopa ke lakshanom vale rogiyo ko jo bistara ya kursi taka simita rahate haim, chikitsaka ya parivara ke kisi sadasya ki sahayata se kie jane vale kramika pratibandhatmaka gatividhiyom se vyayama karana chahie|
vyayama karane ke lie ayu rukavata nahim hai|
eka 67 sala ka vyakti savirama langada़epana ke satha kevala 5 blaka chala saka|
unhem prarambha mem 1/2 se 1 kimi. taka chalana chahie aura jaise-jaise svasthya anumati deta hai vaise-vaise pha़asale ko dhire-dhire badha़ana chahie|
ise sarala mukta hasta vyayamom ya si़dha़i vale vyayamom, mitara vale saikila ya treda mila ke satha sammilita kiya ja sakata hai|
unhem variyata se yadi duri kama ho to banisbata kara se jane ke chalana chahie|
isa sudhara ke satha, unhem vyayama ke samaya hridaya gati mem giravata anubhava hoga|
yaham taka ve bhi jinhem roga ke gambhira prakara haim, apane vyayama starom aura aksijana antargrahana ko bisa pratishata badha़a sakate haim|
shvasana ke ‘punarprashikshana’ mem shvasana vyayama bahuta adhika sahayaka haim|
hama dama, shvasanali-shotha aura vatasphiti ke lie inhim vyayamom ka prayoga karate haim|
hamanem ina vyayamom mem se kai ke sabhi achchhe sarom ko sammilita karane ki koshisha ki hai| 
rogi ko chikitsaka para vishvasa hona chahie|
unhem vyaktigata rupa se sikhaya jana chahie aura bada mem ve kisi samuha mem sammilita ho sakate haim|
jaba vaha ina vyayamom ko samuha mem karata hai, to use dusarom se protsahana milega aura vaha adhika vishvasapurna anubhava karega tatha behatara prayasa karega|
arama karana dama tatha vatasphiti ke rogi ke lie sabase mahattvapurna hai|
ye rogi hamesha tanavapurna, ghabaraye hue tatha thake hue hote haim kyonki hara samaya ye shvasarodha se bachane ke lie lagatara prayasa karate rahate haim|
yadi apa ina rogiyom ko dhyana se dekhenge to apa tanava ke kai lakshana paenge|
unake hatha mem kampakampi ho sakati hai, jaba unaki ankhem banda ho to phada़ka sakati haim, aura unake vakshasthala ke kshetra mem alpa gatividhi ke paitarna ho sakate haim kyonki ve hara samaya apane shvasana ke bare mem sacheta rahate haim|
ve upari vakshasthala, kandhem, gale tatha merudanda mem adhika gatividhi dikha sakate haim, visheshakara jaba unhem koi dekha raha ho|
yaha tanava tatha ye vyartha ki gatividhiyam durbhagyavasha bahuta adhika urja khatma karate haim|
yadi ve arama se haim, kevala taba hi ve adhika nipunata se aura kuchha sahi tarike se shvasana karana sikha sakate haim|
rogi ko ina vyartha gatividhiyom ke bare mem bataya jana chahie aura udara shvasana ki gudha़ta ko bhi sikhana chahie|
use yaha bataya jana chahie ki uchita shvasana kama kashtakara hoga tatha shvasahinata ke akramana ko bhi kama kara dega|
ye nai pranaliyam alpavadhi pranaliyam nahim haim, aura sthayi rupa se prayoga kie jane chahie|
akramana ke samaya use ina nai pranaliyom ki ora adhika dhyana dena chahie aura use isa nae jnana se labha hoga|
ina rogiyom ko ghara mem anandadayaka aura anukula vatavarana pradana karana chahie|
unhem eka nishchinta pravriti bhi apanani chahie|
jaba rogi ko tanava mukta hona sikhaya jata hai, to eka nihshabdata ka vatavarana hona chahie| 
use usake sira aura ghutanom ko takiye se sahara dekara tatha banhom ko arama se vishrama karate hue arama se lita dena chahie|
use tanavapurna aura tanavamukta mansapeshiyom ke bicha antara ke bare mem batana chahie|
yaha taba taka jari rakhana chahie jaba taka ki vaha kaise unhem puri taraha se tanavamukta karana hai yaha nahim jana jata|
ise banha, kandhe, gale, chehare, vakshasthala tatha udara ki mansapeshiyom mem karana chahie|
mansapeshiyom ke ina tanavom ko mapa ja sakata hai, aura hamem unaki saphalata ke bare mem janakari ho jaegi|
jaba rogi isa tanavamukta hone ki prakriya ko sikha leta hai, to vaha vyayama ke lie taiyara hai| 
vaha kandhe tatha upari vakshasthala ki chhoti mansapeshiyom ko tanane aura tanavamukta karane se prarambha karata hai|
shvasana ke satha-satha isi uddeshya se yojanabaddha kie gae vividha layabaddha aura dolana vyayama karae jate haim|
shvasana vyayamom ko subaha ya shama mem jaba peta khali hota hai, taba karem|
khada़i avastha mansapeshi ki gatividhiyom ke lie svatantrata ka labha pradana karata hai|
sidhe khada़e hom, banhe apake parshva mem hom, dhire se aura gahare shvasa chhoda़em, gale ko jukaem, apane kandhom ko age lae tatha peta ko sikoda़em|
apa paenge ki apa aura adhika hava chhoda़ sakate haim|
aba dhire se shvasa lena arambha karem, vakshasthala, peta ka vistara karem tatha gale ko sidha karem|
thoda़a aura adhika shvasa lene ke lie, apane vakshasthala ko uncha uthaem, peta ko thoda़a khincha le tatha gale ko badha़aem|
kuchha kshanom ke lie shvasa ko rokem aura dhire se shvasa chhoda़na prarambha karem tatha banhom aura eda़iyom ki gatividhi ko ulata dem|
kuchha kshanom ke lie shvasa ko rokem aura ise punah prarambha karem|
jaba pamva ke chapa samatala hom, to rogi ko chalate samaya pida़a ka anubhava hota hai|
ghutane bhi tirachhe sanketa karate haim, isaka artha hai ki pura pamva nitamba sandhi para ghuma gaya hai| 
adhikatara vikritiyam pahale chara mahinom mem utpanna hote haim, jaba bachcha palatane ke lie bahuta chhota hota hai|
kisi bhi sthiti mem bachche ko rakha jaya, janma ke thika bada se hi, vaha apani aramadeha avastha janata hai|
tani hui mansapeshiyam kamaja़ora ho jati haim tatha sikuda़i hui mansapeshiyam shaktishali ho jati haim| 
jaba pida़a tivra aura sandhi uttejita hoti hai to eka tivra avastha ko pahuncha jati hai|
ina rogiyom ka atmavishvasa ke satha upachara hona chahie, rogi taka sandesha svatah hi pahuncha jaega tatha unhem janakari ho jaegi ki ve sahi hathom mem haim|
eka patale suti gadde sahita eka lakada़i ka bencha paryapta hona chahie|
padapata se bachane ke lie paira ko gaddidara gatte para rakha ja sakata hai|
rata mem kevala eka patala takiya dena chahie|
muda़i hui kohani ke satha, bahom ko sharira se juda़a hua rakhana chahie| 
bahom ki sthiti mem nirantara parivartana hote rahana chahie| 
sujana ghatati jati hai, aura marija ko darda se bahuta hi jalda arama mila jata hai| 
niche ki ora sira karake pharsha para leta jaem, hatha ko niche ki ora va hatheliyom ko pharsha para rakhem| 
bahom ko sidha rakhem, sharira ko uthaem aura sabase unche bindu taka nitamba ko uthaem| 
taba sira ko bahom ki disha mem dhire-dhire niche ki ora karem aura usake bada naka ko pharsha ki sataha taka pahunchaem, thudi aura sina donom ko eka hi stara para laem| 

yaha eka kathina vyayama hai, aura ise sarala banane ke lie, hathom ko pairom ki apeksha adhika unche stara para rakha ja sakata hai| 
yaha vyayama apaki bahom, sine aura pitha ke hisse ko majabuta banane ke lie achchha hai| 
hathom ko kamara para rakhakara, pairom ko eka satha rakhakara va sharira ko sidha khada़a rakhakara anrabha karem| 
ghutanom aura nitambom ki ora juke, sharira ko taba taka niche jukaem, jaba taka ki pamva ki pindili va jangham ke bicha lagabhaga 110 ansha ka kona na bana jae| 
aba 15-20 sentimitara ki unchai taka kudem aura paira sidhe karake vapasa jamina para ae| 
yaha sanyojana apaki nirasha ko bhanga karega aura adhika prabhavashali hoga| 
apane peta ki chhantai karana achchha hai| 
apane udara ki peshiyom ko sakhta karem, aura 6 sekennda taka rokem aura phira sansa lem| 
sharira-rachana sambandhi ka़isma apake mahatvapurna angom ko majabuta banata hai aura unaki kriyashilata ko badha़ata hai| 
ye vyayama adhika shramasadhya nahim hote haim| 
ye hara unra aura hara vyavsaya ke vyaktiyom dvara kie ja sakate haim| 
samamitiya vyayama peshiyom ki takata ko kama mem lata hai, kintu unase kisi prakara ki gatividhi nahim hoti hai| 
ye hathom ki pakada़, sidha़i chadha़ne, pratirodhaka chakra va nava khene ke vyayama ke daurana, raktachapa mem vriddhi aura hridaya dara mem bhi thoda़i vriddhi utpanna karate haim | 
ye hridaya ke marijom ke liye adhika labhadayaka haim| 
ye hridavahika sambandhita kathinai utpanna karate haim, jo ki kriyatmaka aura jaldi-jaldi punaravrita hone vali hoti haim| 
eka samanya vyakti jaba vishrama ke bada shramasadhya vyayama karata hai, to aghata kahalane vali pratyeka dhada़kana, jisaki prabalata 4-5 guna badha़ jati hai,usa pratyeka dhada़kana ke satha rakta bahara ki ora nikalata hai| 
shramasadhya vyayama karane ke bada, eka samanya vyakti mem ऑksijana ki khapata 10-12 guna badha़ jati hai| 
vyayama ke daurana bahya sharirika dabava badha़ne se, hridaya ki peshiyom mem rakta apurti mem kami a jati hai|
donom hi sthitiyom mem chikitsakiya prabandhana lagabhaga eka jaisa hi hota hai| 
hridayapeshi rodhagalaka mem, 8-10 haphtom taka sampurna vishrama diya jata hai, jisase rodhagalaka thika ho jae, aura phira kriyaem dhire dhire arambha hoti haim| 
niyamita vyayama se chakriya etheroskalerosisa (chakriya bhitti mem vasa jamane ke karana sankuchana ) ki bhavi pragati rukegi| 
aisa paya gaya hai ki, kama kriyashila basa chalakom ki apeksha, basa ke kandektarom mem, hridaya ki bimariyom ki sambhavana kama hoti hai| 
hridaya ke marijom ke lie, sthiti yojana ka pahala kadama, eka vyayama parikshana karana hai| 
yaha sujaye jane vale vyayama ki tivrata aura samayavadhi ko nirdharita karega| 
adhikatama mapi gayi rogasuchaka hridaya dara ko, 80-85% ki sarvochcha paristhitika hridaya dara, para banae rakhana marija ke lie samanya tarika hai| 
kama se kama pancha se chhah taraha ke yantrikiya upakarana jaise- sidha़i chadha़na, saikila, nava khena, aura chara minata ki pratirodhashakti parikshana ke lie upalabdha honi chahie, jisake anta mem hridaya dara ko, lakshya hridaya dara ke stara ki taraha liya jana chahie| 
taba pratyeka samuha, anya sthanom ya upakaranom taka ghumate haim| 
adhika vyayama nukasana ka karana bana sakata hai| 
sine ya phuphphusiya bimariyom ke lie, eka sahayaka ke rupa mem visheshatah dama, shvasana-shotha, shvasavarodha adi ke lie, sansa lene aura samanya vyayama ke mahatva ko pahachana ja chuka hai| 
marijom ko arama milane ke karana ve isake abhari haim| 
apane dirghakalina anubhava ke satha chikitsaka, ina vyayamom se hone vale phayadom ke lie, apana vishvasa mata prakata karate haim| 
vyayama arambha karane ke pahale, hamem phephada़om aura vakshiya bhitti (sine ki bhitti) ke lochaka lakshana ke vishaya mem jana lena chahie| 
apane lochata ke guna ke karana, phephada़e apane akara mem parivartana hone ka virodha karate haim| 
samanya dasha mem, rikta karane ke lie phephada़om ke jukava ka virodha vakshiya bhitti ko bharane ke lie hone vale jukava dvara hota hai| 
sakhta phephada़e thona sa sthana rikta karate haim aura kada़i vakshiya bhitti bada़i matra mem phephada़om ko niyantrita kara leti hai| 
phephada़om ki matra nishkriya rupa se, usake lochaka guna ke dvara pramanita hoti hai, jo phephada़om ka kriyatmaka shesha matra hoti hai, tatha samanyatah yaha 3000mi.li taka hota hai| 
yaha dhyana dene yogya hai ki, yaha aisi matra hai, jo phephada़e nishkriya rupa se vapasa karate haim| 
kriyatmaka shesha matra se bhinna, phephada़om ki matra mem ae parivartana ke lie, sakriya peshiya prayasa avashyaka hai| 
madhyapata ka sankuchana, vakshiya dabava ko kama karata hai aura isalie phephada़om ke sthana ko badha़ata hai, satha hi satha udara ke sampida़na ke dvara yaha udariya dabava ko badha़ata hai| 
madhyapata ki kriya shvasana hai arthata sansa khinchana| 
yaha ulti va malatyaga ke samaya para bhi sakriya hota hai| 
bahya antaraparshuka peshiyam bhi anntahshvasana mem sahayaka hai| 
antarika antaraparshuka peshiyam shvasa chhoda़ne mem madada karati haim| 
purnatah jvariya shvasana mem, madhyapata khada़i sthiti mem, lagabhaga 1.5 se.mi. taka aura uttana sthiti mem thoda़a adhika, gatishila ho jata hai| 
pasaliyom ka dhancha, khada़i mudra mem 1 semmi. va uttana mudra mem 6 sem.mi. taka badha़ jata hai| 
parshvika visthapana ki apeksha agranitamba visthapana adhika mahatvapurna hai| 
adhikatama matra mem shvasana (gahari sansa) ke daurana, pasali ka ghera 8 se.mi. taka, pashcha bhaga ka vyasa 3 se.mi. taka, aura parshvika bhaga ka vyasa 1.5. se.mi. taka badha़ jata hai| 
adhikatama madhyapatiya vistara 10 se.mi. taka hota hai| 
phephada़om ki matra mem lagabhaga do tihai parivartanom ke karana, madhyapata ka rekhiya visthapana hota hai| 
phephada़om ki matra mem parivartanom ka virodha do takatem karati haim| 
pahali takata hai, usaki lochata, jo phephada़e aura vaksha ki anavashyaka nirdharita karati hai| 
dusari takata hai, peshiya raktasankulana, aura atistravana ke satha shvasana peshiya sankuchana ke karana, samanya phuphphusiya bimari mem vikasita hua vayumarga ka pratirodha| 
shvasana sambandhi bimari hone para, durastha vayumarga ka badha़a hua pratirodha, samipastha vayumarga mem dababa ko ninna (kama) karata hai, taki ye aura bhi asani se sampida़ita ho jae| 
isa prakara se badha़a hua vayumarga pratirodha, yaham taka ki samipastha bhi, atisakriya nihshvasana prayasom ke dvara sahajata se utpanna ho jata hai| 
kama dabava se yukta nihshvasana, aura bhi adhika vayu para dababa dala sakata hai| 
peshiyom ke dvara lagae jane vale atirikta dabava ke karana, shvasana sambandhi bimariyom mem shvasana peshi ka ऑksijana upabhoga 10 guna badha़ jata hai| 
koi bhi vyayama jo uchcha chhoti matra utpanna karata ho, vaha sankata mem dala sakata hai|
aisa shayada ऑksijana ki adhika matra avashoshita karane ke karana hota hai jo kama se kama saiddhantika rupa mem, unake badhe hue prayasom mem nihshvasana peshiyom ke dvara upabhoga ki jaye| 
utakom ka vinasha, phephada़om ke utakom ki lochata ke guna ko ghatata hai| 
sine ki bhitti ki phulane ki pravritti, phephada़om mem ati vayusanchalana karati hai| 
vayumargom mem hone vale yantriki ya sukshmajivi khatare se nipatara peshiyom ke sravana aura asthira nasa ke bicha khansi ki pratikriya dvara angom mem hi kiya jata hai|
phephada़om ki gatividhiyom ke eka sadharana parikshana ka sujava diya jata hai| 
phephada़om ki gatividhi ka parikshana, chikitsaka ko yaha jane ke yogya banata hai, ki vayu ke pravaha mem koi badha hai kya, kintu marija ko phephada़om ka kainsara hai ya nahim yaha nahim batata| 
svayam ke dvara arthata bina kisi dabava ke, vayu ki nishkasita ki jane vali matra hi anivarya kshamata hai| 
eka samanya vyakti mem, pahale eka sekenda mem lagabhaga vayu ka 3/4 bhaga phephada़om mem nishkasita hota hai, aura 3 sekenda ke andara hi yaha purnarupena bahara ho jata hai| 
bimara phephada़om mem vayu ke marga mem badha ho sakati hai| 
atah pahale sekenda mem nishkasita ki jane vali vayu ki matra, bahuta kama hogi| 
yadi eka sekenda mem nishkasita ki jane vali vayu jvariya matra ki 70% jitani hogi, taba sukshma parikshana ke antargata phephada़e samanya honge| 
yadi yaha isase bhi kama hai, to aisa vayumarga mem badha ke karana hoga| 
kya koi sudhara hua hai, ise janane ke lie, isa parikshana ko kuchha haphtom ke bada phira se doharana chahie| 
adhikatama nihshvasana pravaha, shikhara pravaha mapi ke dvara mapa ja sakata hai| 
yaha suvahya upakarana, sidhe-sidhe uchcha pravaha dara ko mapata hai, jo ki vayumarga mem badha ane para ghata jata hai| 
dama mem aksara phephada़e purnatah samanya hote haim, atah kuchha marijom mem shvasavarodha vikasita ho jata hai| 
shvasana nali shotha mem shvasanali va vayumarga mem shleshamala granthi ki ati angavriddhi ke satha chashaka koshika ka bhari anupata ke karana atyadhika kapha utpanna hota hai aura marga mem badha hoti hai|
shvasana sabandhi bimari mem,sankramana ke satha va sankramana ke bina bhi badha ho sakati hai, jo adhika gambhira parinamom ya phoda़e ke nirmana ko badha़a sakati hai| 
dama aura shvasananali shotha ke marija ko, sadharana vyayama ke tarike batane chahie| 
vyayama ke dvara stanom ko, sudridha, vrihada aura sudaula banaya ja sakata hai| 
eka khubasurata mahila, apani diladaula, achchhi mudra, ankhom mem chamaka, chamakati hui tvacha aura apane sharirika hava-bhava ke madhyama se apana prabhava sthapita karati hai| 
adhunika jivana mem, purusha sochata hai, ki vaha kathina parishrama kara raha hai| 
ham vaha karata hai, kintu kabhi-kabhi adhika sharirika kriyaom ko shamila kie bagaira hi| 
sabhi prakara ki adhunika suvidhaem, jivana ki shakti ka prayoga kama karati haim| 
vastava mem, nayi mashinom aura suvidhaom ne, manava ki adatom mem achanaka parivartana la die haim| 
manava lakada़i kata karata tha, pani lata tha aura puratana paristhitiyom ke antargata hi jita tha| 
shiririka shrama hara kadama para shamila hota tha| 
sharira jo kathina parishrama vale karya karata raha ho, vaha achanaka aramapriya kama karane mem sahajata mahasusa nahim karega| 
aramapriya adatem, sharira para aprakritika dabava dalati hai| 
jisase tantrikiya gada़bada़iyam, anidra, siradarda aura tantrakiya rogom ki ashanka badha़ jati hai| 
vyayama utana hi avashyaka hai, jitana ki bhojana karana va sona | 
vyapari jo tahalate haim, ya golpha khelate haim, ve apani samasyaom ko,asani se tanava mukta kara dete haim| 
yadi apake pasa apaki sabase kimati sampatti-apake sharira- ki dekhabhala karane ka samaya nahim hai, to apake pasa bistara para bimara hokara pada़e rahane ka aura apane nirasha se bhare jivana ke vishaya mem dhire-dhire yojana banane ka bahuta samaya hoga| 
yadi apa apane sharira ki dekhabhala achchhe dhanga se karate haim, to apa aura bhi uchchakankshi, dridhanishchayi va utsahi rahenge| 
eka kara mem tela badalane ki, petrola aura pani ki niyamita purti tatha niyamita tyuninga ki jarurata hoti hai| 
apaka sharira kara se bhi jyada najuka mashina hai evam yaha aura bhi behatara dekhabhala ka adhikari hai| 
sira ko uthaem, gahari sansa andara lem, aura jaise hi apa niche jaemm, dhire-dhire sansa chhoda़em| 
banye paira ke bahya bhaga se pharsha ko chhuem, thoda़a age aem aura phira se pairom ke bicha ki pharsha ko chhune ke lie pichhe jaem aura pairom ke bicha mem eka bara phira se chhuem| 
age aem aura daem paira ke bahya bhaga ko chhuem| 
yadi apa pharsha ko nahim chhu pa rahe hai, to kevala usa disha ki ora jaem| 
jaba apa prarambhika avastha mem a rahem ho, taba apani sansa ko thamem| 
yaha vyayama apake pitha ke hisse va udariya bhaga ki mansapeshiyom ko sashakta banata hai| 
sidhe khada़e ho, pairom ko tisa semmi. ki duri para rakhem, banye hatha ko sira ke upara lem jaem, kohani para muda़e| 
pichhale hisse ko sidha rakhem, danyi ora muda़em aura jaham taka sambhava ho danye paira ke satha, danye hatha ko badha़aemm aura usi samaya banye hatha se danyi ora ko dabaem| 
kuchha sentimitara badhem aura phira se danyi ora dabaem| 
jaise hi apa age jukem, sansa chhoda़em| 
prarambhika avastha mem ate samaya sansa ko rokem| 
jitana uncha sambhava ho utana uncha banye ghutane ko uthaem, thika ghutane ke niche aura agra jangha se pamva ko hathom se pakadem | 
prarambhika avastha mem ate samaya sansa ko roke| 
danye ghutane ke satha isi vyayama ko doharaem| 
yaha udariya va jangha ki peshiyom ko majabuta banane mem madada karega| 
bahom ko bandhe, sidhe khada़e ho, pairom ko tisa sentimitara dura rakhem| 
donom hathom ko, ghada़i ki taraha gherekara rupa mem ghumaem| 
lagatara dhire-dhire sansa lem aura gahanata se bahara chhoda़em| 
yaha vyayama apaki bhujaom ko sundara akara dene mem aura apake stanom ko sudaula banane mem sahayata karega| 
apa isa vyayama ko, apani eda़iyom ko bina uthae, aura bhi prabhavashali dhanga se kara sakate haim| 
sakhta sataha para sidhe letem| 
ghutanom ko bina moda़e, donom pairom ko apa jaham taka utha sakate haim uthaie, aura dhire-dhire samakona ki sthiti taka pahunche, pairom ko dhire-dhire niche va upara karem| 
arama se letane ki avastha mem sansa lem, pairom ko uthate samaya sansa chhoda़e, aura pairom ko niche karate samaya sansa ko thamem rahem| 
jaham taka sambhava ho apani pitha ko sidha rakhem aura baithane ki sthiti mem ae| 
isa kriya ke daurana, pairom ke satha apane hathom ko badha़aie, aura antatah anguthom ko chhuem| 
letate samaya sansa le aura uthakara anguthe ko chhute samaya sansa chhoda़em | 
apane anguthe ko chhune ke lie muda़te samaya, apane apa para adhika dabava na dalem| 
yadi avashyaka ho to arambha mem apane pairom ko kursi mem phasaem, pairom ko pharsha para rakhem| 
karavata para leta jaem, paira sidha rakhem, pharsha ke satha nichali bhuja para sira ko rakhem, upari bhuja ka prayoga santulana karane mem karem| 
jaham taka sambhava ho upari paira ko adhika se adhika uthaem| 
dhire-dhire apaka pamva, pharsha ke satha lamba sthiti mem a jayega| 
pairom ko uthate samaya sansa chhoda़em, pairom ko niche ki ora late samaya sansa ko thamem rakhem| 
apake nitambom aura janghaom ko kama karane ke lie, yaha eka achchha vyayama hai| 
sidhe leta jae, pairom ko eka satha evam sidha rakhem, kandhe ke stara para,hathom ko donom ora phailaem | 
baem paira ko lamba avastha mem rakhem, use sharira ke arapara (tirachha) le jaem aura danye hatha se baem paira ke anguthe ko chhune ki koshisha karem, paira ko phira se lamba avastha mem uthaem aura arambhika avastha mem vapasa le aem| 
kandhom ko pharsha se chhute hue, sharira va pairom ko barabara sidha rakhem| 
niche letate samaya sansa andara ki ora lem aura pairom ko uthate va tirachha karane ke daurana, sansa bahara ki ora chhoda़em| 
arambhika avastha mem ane taka, sansa ko roke rakhem| 
jaise hi apa apane sine va pairom ko upara uthaem, dhire-dhire sansa chhoda़e, unhem niche late samaya, sansa ko roke rakhem| 
shakkara ki adhikata, agnashaya para adhika insulina sravita karane ke lie atirikta dabava dalati hai aura jaba vaha aisa nahim kara pata hai, to hamem madhumeha ki shikayata ho jati hai| 
shakkara,rakta mem kolestrola ke stara ko badha़ati hai, jo ki uchcha raktachapa va anya bimariyom ke hone ka mukhya karana hai| 
dhunrapana va sharaba ki apeksha shakkara ka parityaga karana asana hai| 
yu.esa.esa.ara ke priventinga kardiyoloji sansthana ke adhunika adhyayana ne, sandhishotha aura gathiya jaisi gambhira bimariyom ka sambandha parishkrita chini ke satha joda़a hai| 
shakkara, rakta mem mutranla ke stara ko badha़ati hai| 
gariba vyaktiyom ke ahara ko grahana karem aura amira vyaktiyom ki bimariyom se bache| 
daॉ. hinggasana ka dava hai ki, uttari amerika, yuropa aura japana mem kainsara se pida़ita vyaktiyom ke eka bada़e samuha ka sambandha samanya ahara se hai| 
unaka kahana hai ki aisi ashanka thi ki aphrika aura bharata ke jyadatara hissom mem jaham vasa ki matra ahara se juda़i hoti hai vaham peta, vrihadantra, malashaya aura vaksha ka kainsara apekshakrita rupa se kama paya jata hai| 
neshanala kainsara insatatiyuta, yu.esa.e (ena si ai) ke dvara kie gae eka vyapaka adhyyana mem yaha anumana lagaya gaya ki visheshatah amerika mem sabhi taraha ke kainsara ka 35% karana ahara se sambandhita hai| 
atyadhika ahariya vasa aura kama matra mem ahariya tantuom ka sevana, vrihandratra kainsara ki vriddhi ke mukhya karana haim| 
jyadatara ameriki apani kula kailori ka 40% bhaga vasa se prapta karate haim| 
kainsara va hridaya ki bimariyom se bachava ki drishti se vasa se prapta kailori ka stara 30% ya usase kama rakhane ki salaha di jati hai| 
ahara mem tantuom ki vriddhi, mala ki matra aura paragamana ki shakti ko badha़ati hai| 
paragamana samaya, vaha samayavadhi hai, jo ahara sharira se bahara nikalane mem leta hai| 
pashchimi ahara mem yaha 48 se 72 ghante le sakata hai| 
britisha mem nausena ke aphisara va unaki patniyom ke mala ki matra, vajana mem 38 se 223 grama (do chhote chammacha se lekara eka kapa ki matra taka) payi gai| 
britisha shakahari jo yuganda va bharata ke shakahariyom ko pasanda karate haim, unamem mala ki matra 178 se 980 grama taka hoti hai, ausatana nausainika aphsarom se chara guna jyada| 
bhari- bharakama mala, mala mem maujuda kainsarakari tatva ko patala kara deta hai aura shighragami paragamana, kainsarajana ka antom ki bhitti se bahara ane ki kriya ke samaya ko kama kara deta hai| 
atah ahara mem tantuom ki upasthiti, kainsara se suraksha pradana karane mem sahayata pahunchati hai| 
charakola aura maikroveva ke adhika tapamana mem pake mansa mem eka kainsarajanika sanyukta padartha ke nirmana mem madada milati hai|
vartamana varshom mem yaha avalokana kiya gaya hai ki, vitamina si ka adhika prayoga, kuchha hada taka jathariya karkataroga mem kami ke karana ho sakata hai| 
adhika khadayukta bhumi kuchha sabjiyom mem naitreta ki adhika matra ka karana hoti hai, jo ki kuchha hada taka kainsarajanaka hai| 
behatara hoga ki hama khetom ko jaivika padarthom se urvaraka banayem aura isa samasya se chhutakara pane ke lie kama se kama kitanashakom ka prayoga karem| 
mungaphali aura anajom ko nama avastha mem rakhe jane ke karana isamem apha़lataॉksina paya jata hai| 
ahara se sambandhita bahuta si bhrantiyam haim| 
bada़e udyoga bahuta se khadya utpadom ko badha़ava dete haim aura dava karate haim ki ye utpada samagra rupa mem hai aura apako ye shakti va utsaha dete haim evam apako aura bhi akarshaka banate haim aura bimari ki prakriya ko viparita kara dete haim| 
ve janata ko thagane ke lie pratidina nae nae tarikom ki khoja karate haim| 
khadya udyoga apane grahakom ki sehata ki apeksha, grahakom ki svadenndriyom ko santushta karane aura labha arjita karane ke prati adhika chintita rahate haim| 
phasta phuda ki shrrinkhalaem, bharata ke mukhya bada़e shaharom mem khula rahi haim| 
ve pija, hemabargara, aisa krima aura dudha se nirmita peya ko upalbadha karate haim| 
ye khadya padartha atyadhika vasa aura chini ke satha-satha, adhika kailori evam tantuom ki kami vale hote haim| 
bandagobhi aura phulagobhi mem inadola namaka eka rasayana hota hai, jo ki antom mem kainsara ke viruddha karya karata hai| 
yaha bahuta si bimariyom se hamem bachata hai-kabja se lekara hridaya ki bimariyom evam kainsara taka | 
apacha sailulosa mem anya prakara ka rasayana paya jata hai - jaise phalom aura sabjiyom ke rukshansha | 
phala aura unake chhilake, alu, hari pattiyam, palaka, bandagobhi aura phulagobhi, hamare ahara ke lie mahatvapurna haim| 
anajom se prapta hone vale tantu , tantuom ke gauna strota haim| 
phalom, sabjiyom aura dalom se prapta hone vale tantu, komala aura utkrishtha hote haim| 
gehum ke tantu, kolestrola ke stara para koi prabhava nahim dalate haim| 
phalom aura sabjiyom mem bada़i matra mem pae jane vale chipachipe prakara ke tantu, kolestrola ki matra mem bhari kami late haim| 
kritrima madhuraka, sharkariya aura saklametsa ,mutrashaya ke kainsara se sambandhita haim | 
atyadhika talana ya pyaja va masalom ko adhika bhunakara, bhojana banane ke tarikom se, ahara mem haidrokarbana ya kainsarajana ko chhoda़ sakate hai, halanki isaka koi janapadika rogavijnana ke lakshana upalabdha nahi haim| 
vrihadranntra va malashaya ke kainsara, vikasita deshom mem adhika samanya hai| 
isaka prabhava uttari amerika, nyujilainda, yuropa mem adhika aura dakshini amerika,aphrika aura eshiya mem kama paya jata hai| 
sampurna vishva mem ameriki mem vrihadrantriya kainsara ki dara sabase adhika hai| 
gramina janasankhya ki apeksha shahari kshetra ke vasiyom mem adhika kha़tara rahata hai| 
ahara eka mahatvapurna karaka hai| 
dakshini aphrika ke banatu kshetra mem, jaham, ahara mem adhika reshe aura jantu vasa ki matra kama payi jati hai; vaham undukapuchchha, granthyabha (tunkadakashotha), viputishotha, phoda़a utpanna hona, vrihadranntriya aura kolana kainsara ke prabhava kama hote haim| 
pashchimi logom ke ahara mem vasa aura jantu protina ke sevana tatha vrihadranetriya kainsara ke prabhava evam kolana kainsara ki badha़ti ghatanaom ke bicha mem sidha sambandha hai| 
japana mem, grahana ki jane vali kailori (jyadatara asantripta) ki kula matra ka kevala 12% hi vasa se prapta hota hai| 
amerika mem grahana ki jane vali sampurna kailori ka 40-44% vasa se prapta hota hai| 
sikom para bhune hue mansa, dhumem para paki va achara ke rupa mem surakshita machhali aura sharaba ka prayoga aura maiganishiyama va vitamina e ki kami, amashayi kainsara ke karana se sambandhita hai| 
dudha anla ke sravana ko protsahita karata hai| 
ahara aura sone ke samaya ke bicha, dudha ka sevana karana svasthyavardhaka nahim hai| 
gehum ke chhilake mem paintosansa rasayana paya jata hai, jo ki labhadayaka hota hai| 
atah hamesha sabuta gehum ki roti khaem yaha labhadayaka hota hai aura maida, roti, pestri aura maide ki mithaiyom ka parityaga karem| 
uchita prakara ke karbohaidreta, ahara ke ekamatra sabase mahatvapurna tatva haim, ye atisaste, atiposhaka aura sabase kama vasa vale ahara bhi ho sakate haim| 
roti aura chavala karbohaidreta ke sabase samanya strota haim| 
roti ka prayoga purva-baibiliya kala se hota a raha hai| 
ate ko khamira uthane ke lie rakha diya jata hai| 
usake bada khamira uthe ate se roti banane ke lie, ise tandura mem pakate haim| 
yadi apa roti se vastavika labha prapta karana chahate haim to, apako jaisa bhagavana ne upalabda karaya hai, usi rupa mem, sabhi tinom tatvom ko, eka samana anupata mem grahana karana chahie| 
lagabhaga 80 varsha purva manushyom ne paya ki yadi ate se chokara aura kitanu hata diye jate haim to ve sapheda ata (maida) utpanna kara sakate haim jise varshom taka rakha ja sakata hai|
isaka bhandarana kiya ja sakata hai aura adhika mulyom para becha ja sakata hai|
taba se hama breda banane ke lie isa ate (maide) ka prayoga karate rahe haim|
dvitiya vishva yuddha taka samvardhita breda ka nama suna bhi nahim gaya tha|
samvardhana mem gehum ke ate mem kuchha saste sanshlishta vitamina milana bhi sammilita hai|
chavala ko paॉlisha karane ka kama lagabhaga bada ke kala mem para isi karana se prarambha hua|
yadi apa chavala ko usake chokara se nikala lem, to yaha svachchha evam adhika sapheda dikhata hai| 
ise kitanuom se kharaba hue bina varshom taka rakha ja sakata hai|
chokara vitamina bi-kampleksa ke sabase sanriddha srotom mem se eka hai|
bhojana mem isaki kami se beri beri hota hai|
rogi ekagrata ki kami, durbala snriti, raktalpata, aura yaham taka manovikriti tatha sathyape se pida़ita hota hai|
eka nau-sena chikitsaka, daॉ. takaki ne, beri beri ka adhyayana kiya jo nausena mem eka samanya ghatana thi|
eka lambi samudriyatra ke pashchat yaha eka samanya avastha thi aura adhe navikadala isase prabhavita hua karate the|
daॉ. takaki ne, vistrita anusandhana ke pashchat, yaha nishkarsha nikala ki yaha eka ahariya roga tha tatha palisha kiye hue chavala aura protina ki kami ke karana hua tha|
9 mahine ki eka samudra yatra para unhonne sampurna gehum ke breda ko milaya aura dudha ke ansha ko bhi badha़aya|
unhem 300 sadasyom ke navikadala mem se beri-beri ke kevala 9 mamale mile, jabaki pahale unaki sankhya 169 thi|
isa prakara beri beri ki rokathama evam upachara kiya ja sakata hai|
resheyukta bhojana ko adhika chabane ki avashyakata hoti hai tatha yaha kama resheyukta bhojana ki apeksha adhika bhari hota hai aura isilie adhika garishtha tatha kama mota karane vala hota hai|
skarvi shahari garibom kha़asakara unradaraja logom ke bicha jinake ahara mem nibu vansha ke phalom, pattidara sabjiyom aura tamatarom ka abhava hota hai, samanya hota hai|
bhojana ke prati bahuta adhika lagava aura vichitra paushanika svabhava ke karana isaki kami hoti hai| 
bhapa se daba dekara pakana adhunika pakakarma ka suvidhajanaka aura eka tivra pranali hai|
durbhagyavasha yaha utana svasthyakara nahim haim, yaha uchcha tapamana ke karana bhojana se sabhi vitamina si ko nikala deta hai|
skarvi balyavastha ke prathama varsha mem jaba gaya ke dudha ke atirikta nibu vansha ke phalom ke rasa aura sabjiyam nahim di jatim taba vikasita hota hai|
skarvi mem ama maleriya, susti, kamaja़ori, sansa phulane evam haddiyom tatha sandhiyom mem pida़a ki samanya shikayata hoti hai|
sujana aura rakta sravita karate masuda़e samanya hote haim|
skatalainda ke eka nausena shalyachikitsaka, jensa linda ne, 1747 mem, skarvi para nibu aura santare ke achchhe prabhava dikhae the|
bada mem, britani sena ne apane dala ko santare, nibu aura sabjiyam upalabdha karana prarambha kiya| 
vitamina si se skarvi ka upachara ho sakata hai|
aja hamem kuchha adhika janakari bhi hai ki vitamina si kainsararodhi hai|
apake ahara mem adhika phala aura sabjiyam kainsarakari prabhava ko kamaja़ora karate haim aura apaki antom ko adhika nipunata se chalane mem sahayata karate haim|
samanya sardi ke upachara ya rokathama ke lie vitamina si ke prayoga ne atyadhika abhiruchi jagai hai|
upalabdha pramana yaha sanketa karata hai ki isaka prayoga shvasa-sambandhi roga ki ghatanaom ko kama nahim karata hai para akshamata ke dinom ki kula sankhya ko avashya kama kara deta hai|
surya snana svasthyavardhaka hota hai aura bachchom mem behatara asthi vriddhi ko protsahita karata hai| 
riketsa ka mukhya karana surya prakasha, taja़i hava aura vyayama ki kami hai|
surya snana kevala asthiyom ki behatara vriddhi mem hi sahayata nahim karata hai parantu elarji ki rokathama aura vishrama ko badha़ava dene mem bhi sahayata karata hai|
nikotinika anla ki kami se pelagra hota hai|
udayapura manasika chikitsalaya mem bharti pelagra ke 128 rogiyom ke bicha karyanvita eka sarvekshana mem yaha paya gaya tha ki ve mukhya rupa se makai khane vale the|
yaha eka pani mem ghulanashila padartha, nikotinika anla, ki kami ke karana hua tha, jo makai aura jvara ke atirikta adhikatara anajom mem paya jata hai|
yaha uchita hai ki salabhara makai aura jvara ki roti na khaem|
yaha behatara hai ki eka hi dala ka upayoga na karem|
hama yaha nishkarsha nikala sakate haim ki sada bhojana behatara hai|
jaise-jaise hama adhika sampanna hote haim, ahara ka stara ninna hota jata hai|
sampanna desha uchcha rakta chapa, hridaya roga, kainsara tatha madhumeha ke adhika mamalom se pida़ita haim|
hamare purvaja adhika svastha the kyonki ve aparishkrita bhojana khaya karate the|
vajana badha़ane ya ghatane ke ahara, sharira mem bhojana se kailori ki kitani matra li gai aura kailori ki kitani matra kriya-kalapom mem khapata hui, isa para adharita hote haim|
yadi loga jitani khapata karate hai usaki apeksha adhika kailori lete haim, to unaka vajana badha़ta hai| 
vishesha rogom se pida़ita vyaktiyom ke lie vishesha ahara nirdharita kiya jata hai|
kai chikitsaka vishesha hridaya ya vrikka rogom ke rogiyom ke lie kama namaka vale ahara nirdharita karate haim|
kuchha vyakti kuchha khadya utpadom jaise dudha, tamatara, straॉberi, gehum, alu, ande, machhali, kashthaphala, chakaleta ya suara ke mansa ke erlaji sambandhi ya tvacha ki pratikriyaom se pida़ita hote haim|
vishesha samuha ke logom ki jisamem taruna bachche aura vriddha loga bhi sammilita haim, vishesha ahariya avashyakataem haim|
eka santulita ahara mem kisi vyakti ko svasthya rakhane ke lie avashyaka sabhi khadya tatva sammilita hote haim|
kisi ko bhi utakom ke nirmana ke lie protina aura urja tatha ushma pradana karane ke lie vasa aura korbohaidreta ki avashyakata hoti hai|
utakom ki vriddhi aura unhem banae rakhane tatha sharirika kriya-kalapom ko niyantrita karane ke lie khanija aura vitamina ki avashyakata hoti hai|
ahara jisamem kisi bhi avashyaka khadya tatva ki kami hoti hai, se nyunatajanya roga hote haim|
vitamina e ki kami se rataundhi hota hai aura vitamina si ki kami se skarvi hota hai|
dirghakalina prayoga ke bada ina aushadhiyom ke parshva prabhavom ka samaghata sharira para pada़ega|
rogom ki rokathama aura unhem niyantrana mem rakhane tatha aushadhiyom ko ninnatama mem katauti karane ke lie, ahara hi ekamatra upaya hai|
bhojana jise pachane mem kathinai ho, use hara dina nahim khana chahie|
jaba khaya jaya to yaha sampurna bhojana ke ansha se adhika nahim hona chahie|
ve age kahate haim ki yadi koi vyakti eka upayukta ahara leta hai aura sva niyantrana ko vyavahara mem lata hai, to vaha 100 varsha ki sampurna avadhi ya 36,000 ratem bina kisi roga ke jita hai|
ayurveda rogom ki rokathama aura jivana ko badha़ane mem adhika vishvasa karata hai banisbata upachara mem| 
patanjali jinhonne isa se kuchha sau varsha purva yoga ko prarambha kiya, unhem bhojana ke vishaya mem bhi chinta thi|
yogiyom ne kaha hai ki bhojana svasthyakara hona chahie aura kevala indriyom ko santushta karane ke lie nahim khana chahie|
ve yoga ke vidyarthiyom ke lie satvika bhojana nirdharita karate the|
ve vishvasa karate the ki charitra evam svabhava bhojana se prabhavita hote haim|
ve dudha mem pake, aura isamem makkhana mile paundeda chavala ko khane ka paramarsha dete the|
yogi age kahate haim ki kevala pani hi pyasa buja sakata hai|
svasthya ke anurakshana mem tina chijem mahattvapurna haim – ahara, vyayama aura vishrama|
bhojana dekhane mem achchha hona chahie aura isamem badha़iya mahaka honi chahie|
jyom hi ankhe bhojana ko dekhati haim aura gandha naka taka pahunchati hai, pachana arambha ho jata hai, taba lara sravita hoti hai|
hamem bhojana ko sada banana chahie para satha hi satha ise lubhavana aura kshudhavardhaka banane ke lie hamem hamari kalpana ka prayoga karana chahie|
yaha behatara hai ki hama hamare bachchom mem svasthyavardhaka adatem dala dem|
jaise hi apako pata chalata hai ki eka kha़asa bhojana apake svasthya ko prabhavita kara raha hai, to ise badalane ke lie taiyara rahem|
apako aisa karane ke lie eka shaktishali mana aura ichchhashakti ki avashyakata hogi|
ahara eka aisa kshetra hai jisa para ichchhita matra mem shodha nahim kiya gaya hai|
kainsara tatha edsa jaise asadhya rogom ke agamana se, unhem rokane ki hamari chinta aura adhika vastavika hai|
hala hi mem vishva ke vibhinna logom ke bicha bahuta adhika adhyayana kiya gaya hai|
jaham roga adhika prachalita haim, vaham ke ahara se sambandhita ashcharyajanaka tathya sarvajanika chinta ka vishaya bana gae haim|
isa shatabdi ke arambha mem, amariki ahara mem 50% kailori karbohadreta se, 30% kailori protina se tatha 20% kailori vasa se sammilita the|
sampannata ke karana, rashtriya dugdhashalaem bahuta adhika makkhana, chija़ aura dudha nikala rahi haim|
ausata amariki ke bhojana mem vasa ki matra eka chaunka dene vali sankhya, kula kailori ke 40% taka pahuncha gai hai|
bhojana ka matalaba ho gaya hai milka sheka ke kuchha glasa aura gomansa ke tikke ka bada़a tukada़a| 
amariki ahara mem kolestraॉla ki uchcha matra se hridaya rogom tatha daurom mem kha़taranaka vriddhi hui hai| 
jaba sharira ko urja, vriddhi aura marammata ke lie jitani avashyakata hoti hai usase adhika bhojana khaya jata hai, to vaha sharira ke bhagom mem vasa ke rupa mem jama ho jata hai|
hridaya ke lie uchcha ghanatva vale laipoprotina achchhe hote haim|
bahuta amariki shakaharavada ki ora palata rahe haim aura kai salada parlara khula gae haim|
ve svasthya ke prati adhika jagaruka ho rahe haim aura bahuta adhika shaka aura sabjiyam kha rahe haim| 
santripta vasa mukhya rupa se pashu utpada hai aura ghi, makkhana, ande tatha mansa mem paya jata hai|
isa sankarepana ko dhamaniyom ki starom ki kathorata kahate haim (artiosklerosisa) jo hridaya rogom tatha uchcha raktachapa ke lie uttaradayi hai| 
eka anya prakara ki vasa bhi hoti hai jise asantripta vasa kaha jata hai(jo kai vanaspatika telom mem paya jata hai para sabhi mem nahim)| 
anya shabdom mem, yadi apa suara ki charbi ya ghi ki bajaya, suryamukhi ya soyabina tela mem pakaem, to apa dhamaniyom ki starom ki kathorata (artiosklerosisa) se kama prabhavita hogem|
yadi apa bahuta adhika asantripta vasa lene ki ora pravrita hote haim jaise ki japani karate haim, to rakta adhika patala ho jaega aura rakta bahane ka samaya bhi badha़ jaega| 
isaka ullekha karana prasangika hoga ki japaniyom mem hridaya roga ki akasmika ghatanaem kama hoti haim kintu daura pada़ne ke (mastishka ki rakta vahiniyom se sambandhita) rogom ki akasmika ghatanaem adhika hoti haim| 
atyadhika khatara isa bata ka hai ki kai loga asantripta vasa ka upabhoga karana svasthyakara samajate haim|
eka hala hi ke hridaya rogom ko kama karane ke sambhavita upayom para narve mem kie gae adhyayana se yaha nishkarsha nikala hai ki khatare ko kama karane ka sabase achchha tarika hai santripta ya asantripta sabhi vasaom ko kama karana|
amariki vayaska purushom mem kolestraॉla ka ausata stara prati 100 sisi 230 migra. hai aura gramina japana mem yaha prati 100 sisi 160 migra. hai|
hridaya roga yu.esa.e. mem japana ki apeksha adhika hai|
pakka shakahari ahara mem kama kolestraॉla hota hai|
mansahariyom ke lie, mishrita ahara upayukta hai|
apane ahara mem resha badha़aem, rakta mem kolestraॉla aura traiglaiseraidsa kama karane mem isaka achchha prabhava hota hai|
vivekasammata pathyahara ke dvara kolestraॉla ko 15% taka kama kiya ja sakata hai |
poshahara mem vasaom ko niyantrita karake 10% aura kama kiya ja sakata hai |
yaha akalana kiya gaya hai ki yadi uparyukta niyantranom ka anupalana kiya jata hai, to hridaya rogom ke khatare mem do guni kami hona sambhava hai |
jana-madhyama mem adhika prabhava vala prachara vyakti ko yaha vishvasa karane para majabura karata hai ki shishu ahara achchha hai aura yaham taka ki mam ke dudha se bhi behatara hai|
chikitsakom aura vaijnanikom ke shora-gula ke bada isa prakara ke vijnapanom para kuchha pratibandha lagae gae haim aura logom ko yaha batane ka prayatna kiya gaya hai ki mam ka dudha shishu ahara se behatara hota hai |
shishu ahara ke pratyeka dibbe para yaha likhana avashyaka hai ki mam ka dudha shishu ahara se uttama hai |
bharatiya evam antarrashtriya vikhyata chikitsiya patrikaom mem yaha bina kisi sandeha ke siddha kiya ja chuka hai ki mam ka dudha sarvottama hota hai |
stanapana kie hue bachchom ko bahuta kama pratyurjataem aura bimariyam hoti haim |
mam ke rakta se pratirakshi bachchom mem mam ke dudha ke dvara sthanantarita hote haim |
ayurveda mam ke dudha ke mahatva para ja़ora deta hai |
yadi eka vyakti 110 grama chini ka pratidina upabhoga karata hai, to usako hridaya daure ka khatara kevala 60 grama upabhoga karane vale vyakti se 5 guna adhika hota hai |
use rakta mem traiglisaraida aura kolestraॉla ko kama karane ke lie mithai aura madya ko ghatane ki salaha di jati hai |
soviyata aura amariki vaijnanika yaha paramarsha dene ke lie eka satha age ae haim ki jitana apa pratidina chini ke antargrahana ko kama karenge utana hi hridaya aura sharira svastha hoga |
chini jo hamare jivana ko adhika mitha banati hai vastava mem ise katu banati hai aura hamem bimara hone aura udhara ke samaya para jivita rahane ke lie badhya karati hai |
prasiddha soviyata hridaya roga visheshajna daॉ, ena. ji. khaltayeva, isa prasiddha mata se asahamata haim ki chini hridaya ke lie hanikaraka hai, kevala yadi yaha sharirika vaja़na ko badha़ati hai| 
chini karbohaidreta hai, kailori se bhari vitamina, khanija aura reshe se rahita |
jo bhi chini hama khate haim use urja utpanna karane ke lie kharcha kara deni chahie, aura yadi yaha nahim ho sakata hai taba yaha svasthya samasyaem utpanna karegi |
unhonne 10 desha chune jo adhika chini ka upabhoga karate haim aura 10 desha jo kama chini ka upabhoga karate haim |
unhonne paya ki desha jo chini ki sabase kama matra ka upabhoga karate haim vaham hridaya samasyaom ki sabase kama sankhya hai | 
upavasa ke dvara ankhem adhika chamakili aura adhika yuva ho jati haim aura drishti sudhara jati hai | 
upavasa ke bada vrikka aura yakrita behatara kama karate haim |
yadi apa yakrita aura vrikka ki eka prachanda bimari se pida़ita haim, to savadhana rahana behatara hai| kyonki ye anga upavasa ke daurana sharira ke dushita padartha ke utsarjana mem sahayata karate haim, isalie unaka karyabhara badha़ jata hai |
tvacha taja़i aura yuva ho jati hai |
jurriyam, daga-dhabbe, rangahinata aura muhase upavasa ke daurana gayaba ho jate haim |
kisi upavasa ki arambhika avastha para, spanda dara badha़ sakati hai |
dhire-dhire yaha kama ho jati hai aura lagabhaga 40 spandana prati minata para tika jati hai |
upavasa ke anta mem yaha lagabhaga 60 spandana taka phira se badha़ jati hai | 
spanda dara kama se kama 20 spandana prati minata taka bhi kama ho sakati hai |
yadi spanda dara eka lambe samaya taka bahuta kama ya bahuta adhika rahati hai, to yaha upavasa toda़ne ka adesha hai |
ve jo kisi hridaya roga se pida़ita haim vo spashta dekha sakate haim ki unaki spanda dara mem bhinnata bahuta adhika hai aura yaha aniyamita ho sakati hai |
yadi yaha lambe samaya taka aniyamita bani rahati hai to rogi ko upavasa karana banda kara dena chahie |
bahuta se upavasa se darate haim |
unaki dharana hoti hai ki yadi ve upavasa karate haim to ve kamaja़ora ho jaenge aura samadhi ke nikata ja rahe honge |
ve upavasa karane ko bhukhom marane se joda़te haim |
isamem bahuta adhika ichchhashakti chahie |
hama griha dhvansa mem phanse una logom ke bare mem bhi padha़te haim, jo isa kathina paristhiti mem bahuta dinom taka bina khana ya pani ke jivita rahate haim |
jo marate haim ve bhukhe rahane se nahim balki aghata ya bhaya sahita kuchha anya vibhinna karanom se marate haim |
marusthala mem khoe ya jalayana vidhvansa mem phanse vyakti samanyatah pyasa se marate haim na ki bhukhe rahane se |
dhruviya kshetra mem khoe loga samanyatah atyadhika thanda se marate haim aura khane ki kami se nahim jaisa ki yaha trutipurvaka mana jata hai |
udaharana ke lie, khanom mem, majadura samanyatah vishaili gaisom ya gandi vayu ki chapeta mem a jate haim jo antatah unaki nrityu lati hai | 
eshiya aura aphrika ke kuchha deshom mem akala ki sthitiyom mem, logom ko paryapta khana nahim milata hai |
yadi eka purna poshita aura svastha janavara ko bhukha se mara jata hai to usaki arthi taka pahunchane mem usako jo samayavadhi lagati hai vaha usake sharira ki sthiti aura sangrahita vasa para nirbhara karegi |
yaha dekha gaya hai ki marane ke pahale vaha sharira ka lagabhaga adha vaja़na ghatata hai |
kuchha udaharanom mem vaha apane sharira ka adhika se adhika 60% vaja़na ghatata hai |
jaba ki, yadi vaha poshita kiya jata hai, to vaha purnatah bahala ho sakata hai |
ausata vaja़na ka eka svastha vyakti bhukhe rahane ke karana nrityu hone ke pahale apane sharira ka adha vaja़na ghataega |
eka manava upavasa ke daurana pratidina lagabhaga eka paunda vaja़na ghataega | 
aba, mana lijie hama usi vyakti ko 30 dinom ke lie eka chikitsiya upavasa rakhavate haim, to vaha apane sharira ka lagabhaga 1/5vam vaja़na ghataega aura vaha bilakula surakshita hai |
yaha dekha gaya hai ki upavasa ke daurana eka vyakti prathama saptaha mem pratidina lagabhaga eka paunda vaja़na ghatata hai, tina se chara paunda usake bada vale saptaha mem, aura panchave saptaha ke age kevala 1 – 2 paunda saptahika | 
atah hama nishkarsha nikala sakate haim ki atirikta vaja़na vale loga lamba upavasa adhika surakshita rupa se kara sakate haim bajaya unake jo patale haim |
patale logom ko khuda ko upavasom mem dalane ki avashyakata nahim hai, ve eka chhoti upavasa avadhi le sakate haim aura avashyakata ho to ise tina mahine bada dohara sakate haim |
kuchha dinom taka ka eka upavasa bhojana nalika ko sapha karega aura ise dushita padarthom ke sancharana se mukta karega |
yaha apane apa hi vaja़na mem eka prarambhika paryapta kami utpanna karega |
prarambha mem yaha eka dina mem 2–5 paunda ho sakata hai | 
isa kami mem shamila hogi pani ki kami jo samanya namaka ke pratidina antargrahana ko rokane se utpanna hoti hai |
yaha bhranti paida karata hai ki upavasa ke prathama kuchha dinom mem vaja़na shighrata se ghatata hai |
lekina sharira ka vaha bhaga jo mota nahim hai phira se bahala ho jaega jaba samanya khana phira se shuru karate haim |
vastavika kami upavasa ke daurana sharirika vasa ka indhana ke rupa mem upayoga ke karana hoti hai |
eka paunda sharirika vasa 3000 kailori ke barabara hoti hai |
upavasa ke daurana vastavika vaja़na hrasa ina ankada़om se apekshakrita thoda़a adhika hota hai kyonki kuchha avasiya utaka bhi nashta ho jate haim |
eka ja़oradara, purna upavasa avadhi ke daurana, mansa-peshiyam, rakta, yakrita aura anya jaivika angom ka vaja़na ghata jata hai, lekina, ashcharya janaka rupa se, mastishka ka nahim |
eka madhyama matra mem vyayama karata hua aura bhari sharirika shrama mem nahim bandha hua ausata vyakti eka dina mem lagabhaga eka paunda ghatata hai | 
upavasa anubhava ke shuruata mem aura 21vem dina ke anta mem bhi phaॉsphorasa ka hrasa naganya hota hai |
poteshiyama ka hrasa upavasa ke 21vem dina ke anta ki apeksha shuruata mem lagabhaga tina guna tha |
sodiyama jo hamari adatom aura khane mem sadharana namaka (sodiyama kloraida) ki achchhi matra milane ki pratha ke karana hamesha sharira ki avashyakata se adhikata mem hota hai, upavasa ke daurana bahuta adhika sravita hota hai |
hama dekha sakate haim ki upavasa ke shuruata mem sharira se nishkasita sodiyama 21vem dina kama hue ki apeksha lagabhaga 30 guna adhika hai |
upavasa ki avashyakata hai eka shaktishali mastishka aura eka uchita vatavarana |
upavasa karate samaya apako utsahita karane ke lie yadi apake asa-pasa mitra haim to yaha asana ho jata hai | 
upavasa ke daurana, badha़ti hui kamaja़ori ke karana apa manasika rupa se kamaja़ora ho jate haim |
upavasa jari rakhane ke lie apako apani ankhem eka sadhe hue lakshya para rakhani chahie |
apako upavasa ke anta mem svasthya ki disha mem apako milane vale bahuta sare phayadom ke bare mem apane apa ko yada dilana chahie |
upavasa ke daurana apa turanta hi shakti nahim khote haim |
pahale 2-3 dinom taka apa asani se apane dainika karya kara sakate haim |
yadi apaka karya tanava bhara nahim hai, to apa kuchha aura dinom taka vaisa hi karana jari rakha sakate haim |
yadi apa sardi, bukhara ya dasta se pida़ita rahate hue upavasa karate haim, to eka tatkalika pratikriya ke lie apako pura arama karana chahie |
atipati bimari ki avashyakata hai tatkala upavasa karana aura yadi apa patale haim phira bhi upavasa karem kyonki isako kevala 3-4 dinom ke upavasa ki avashyakata hogi |
yadi apako dirghakalika bimariyam haim, to eka achchhi taiyari ke bada hi upavasa karem |
vyakti jo sharirika rupa se maja़buta haim eka, do ya yaham taka ki tina haphtom taka ka eka lamba upavasa rakha sakate haim |
yadi apane pahale upavasa nahim kiya hai, to eka lambe upavasa ka khatara mata uthaie | 
prarambha karane valom ke lie maim daॉ. dive ka subaha ke nashta rahita yojana ki salaha dena chahunga |
yaha vidhi yu. esa. e. mem lokapriya thi |
daॉ. dive ne apane rogiyom ko subaha ka nashta lena banda karane kaha |
madhyahna bhoja ke samaya taka apane pachana tantra ko arama dem |
lekina savadhani ka eka vachana, brancha mem atyadhika na khaem |
subaha ka nashta chhoda़ne ke bada yadi apa 25 ya 50 pratishata adhika khate haim to pura uddeshya samapta ho jaega |
daॉ. dive ka nirikshana tha ki, isa tathya ke bavaja़uda ki bhinna-bhinna chikitsaka apane rogiyom ko bhinna-bhinna davaem dete haim; roga mukti ka pratishata vahi rahata hai |
unhonne nishkarsha nikala ki bimari ko thika karane mem dava ne mushkila se hi koi bhumika nibhai |
unhonne eka gariba yuva lada़ki ka parikshana kiya |
apane darda ke karana vaha pani taka bhi nahim pacha sakati thi |
gariba hone ke karana vaha davaem nahim vahana kara sakati thi |
upavasa ke kuchha dinom bada, apane apa hi vaha pani dharana karane mem sakshama ho gayi thi |
dhire-dhire usane shakti prapta karani shuru kara di |
usaki tvacha sapha ho gayi aura jibha svachchha aura lala |
35vem dina usane khane ke lie manga aura vaha bhi dharita ho gaya tha |
usaka bukhara chala gaya tha, usaki spanda dara samanya ho gayi aura kuchha dinom mem vaha pahale se adhika svastha ho gai thi |
daॉ. dive ashcharyachakita the kyonki ve janate the ki yadi isa lada़ki ko jabaradasti khilaya jata aura dava di jati, to vaha mara gai hoti |
daॉ. dive apane rogiyom ko kama se kama aushadhiyam dene mem vishvasa karate the, jaise ki eka sadharana darda nashaka yadi darda bahuta adhika tha ya virechaka yadi rogi mem koi gati nahim thi |
khadya ke rupa mem vaha thoda़a ubale chavala ka pani ya daliya ka sujava dete the|
unhonne dudha ka nuskha nahim likha|
yadi apa nashta rahita yojana ke satha achchha samanjasya karate haim, to yaha apake tantrom ke lie chamatkara kara sakata hai | taba apa ratri bhoja aura madhyahna bhoja ki avashyakataom ko sudharane ki koshisha kara sakate haim | 
kama matra lem aura mukhya khane mem salada aura hari sabja़iyam lem |
yaha apako 3 dinom ke upavasa para jane ke lie taiyara karega aura upavasa ki yojana ki adata ho jaegi |
ina chhote upavasom ke daurana sharira dushita padartha ki thoda़i matra se chhutakara paega; apako prabala pratikriyaem nahim hongi jaisai ki ye lambe upavasom mem ho sakati haim |
jaba apa upavasa aura isake phayadom se adhika achchhi taraha se parichita ho jate haim, taba apa apane sharira aura upavasa ke daurana yaha kaise pratikriya karega ko samajenge |
taba apa eka bahuta lambe upavasa ke lie taiyara ho sakate haim | 
mainne paya hai ki logom ko kisi vishesha prakara ke khane ka vyasana hota hai | 
yadi ve vahi udipta karane vala khana nahim khate haim jo ve hamesha lete a rahe haim aura adi ho gae haim, to ve santushta nahim mahasusa karenge | |
jaba apa eka bhojana chhoda़te haim, taba khane ki lalasa bahuta badha़ jati hai | 
eka vyasani ki taraha jisaki aushadhi roka li gayi hai apa vinivartana lakshana prapta karenge |
yadyapi isaka mula samaya ke bicha mem kho gaya hai, tathapi upavasa para jane ki parampara ne mukhya dharmom ki pratha mem eka mahatvapurna bhumika nibhai hai |
prachina dharmom mem, upavasa ishvarom ko anukula banae rakhane, ya khadya apurti ko sudharane ki ichchha se sambandhita tha | 
upavasa ke madhyama se, dharmika samuha, apane kukrityom aura papom ke lie kshamadana mangate haim |
kuchha dharmom mem shoka ke daurana eka upavasa kiya jata hai |
anyom mem, loga manate haim ki upavasa unake mastishka ko bhautika vastuom se upara uthaega aura adhyatmika ananda aura khushi ki avastha utpanna karega |
ullekha karane ke lie kuchha hinduvada, islama, isaivada ya jainavada mem upavasa ke mahatvapurna dina hote haim |
yahudi kanuna prayashchita ke dina, yaॉma kippara para eka varshika upavasa ka adesha deta hai 
upavasa ke dinom ke daurana isai loga thoda़a khana khate haim |
bauddha auऱ hindu niyamita rupa se upavasa karate haim, visheshatah dharmika avasarom ke daurana |
mahatma gandhi ne upavasa ka upayoga, prayashchita aura virodha ke eka rajanaitika hathiyara donom ke rupa mem kiya |
vaijnanikom ne sharira para upavasa ke prabhavom ka adhyayana kiya hai|
ve janate haim ki khane ka antargrahana isaka chayapachaya badha़akara sharira ko prabhavita karata hai |
upavasa ke bada, chayapachaya samanya dara se adhika se adhika 22 pratishata kama ho sakata hai |
lekina anusandhanom ne yaha bhi darshaya hai ki upavasa ki lambi avadhi ke bada, svatah chayapachaya ki dara ko kama karake sharira apane apa hi samanjasya karane ki ora pravritta hota hai |
upavasa ke bada, vyakti ko dhire-dhire khana phira se shuru karana chahie |
dharmika samuha manate haim ki upavasa sva-niyantrana ko protsahita karata hai aura ichchhashakti ko prabala karata hai |
usane ise usi mudra mem baithe hue kiya |
asurom dvara haddiyam hathiyara ke rupa mem upayoga ki gayi thim |
kahane ki avashyakata nahim hai ki, unhonne yuddha jita|
yaha bahuta si pauranika kathaom mem se kevala eka hai |
bahuta se lambe upavasom aura dhyana ka varnana bhagavata gita aura ramayana mem diya gaya hai |
upavasa ne dirghayu prapta karane mem sahayata ki hai |
yaha urja sangrahita kara sakata hai jo svasthya badha़anem, dainika dharmika kriyaom ko utsaha ke satha jari rakhane aura adhika mahana lakshyom ko prapta karane mem sahayata karata hai |
bahuta pahale nahim, eka airisha neta baॉbi saindsa 66 dinom ki bhukha hada़tala ke bada belaphasta jela mem, naॉrdana ailainda mem mare |
baॉbi saindsa ne, jo taba 27 ke the, apana eka tihai jivana jela mem bitaya |
isa nidara yuva vyakti dvara yaha upavasa jela adhikariyom ki chaukasa ankhom ki dekha-rekha mem kiya gaya tha |
eka chikitsaka pratidina usaka nirikshana karata tha |
usaki nrityu ke pahale samachara patra-patrika mem usaki sthiti ka vistara mem varnana karate hue usake svasthya para pratidina eka vijnapti prakashita ki jati thi |
‘nrityu taka upavasa’ phaishanebala ho gaya |
yaha una netaom ke hathom mem eka hathiyara hai jo apani mangom ke lie dabava dalate haim |
dharma mem, upavasa eka purani pratha hai |
dharmika neta svasthya ke lie upavasa ke labhom ko janate the aura ise isa prakara samavishta kiya ki ise unake sansarika astitva ka hissa hona chahie | 
eka prakara se, upavasa purva-aitihasika avadhi mem prachalita tha |
hamare purvaja jangala mem rahate the |
unhem khane ke lie shakom aura phalom ya janavarom para nirbhara rahana hota tha |
unhem anta mem kai dinom taka bhukhe rahana hota tha |
sambhavatah yahi karana tha ki ve durusta aura svasthya ke charama mem the |
jangala ki paristhitiyom mem rahane aura shikara ke lie durusta hone ke lie yaha avashyaka tha |
kevala tabhi ve jivita raha sake the, kyonki ve svastha the | 
hamare pasa eka prakritika pravritti hai, bhukha aura jaba avashyakata ho sharira dvara khane ki manga |
masala chhaunka aura talane ke satha, hama khane ko apane svadanukula banate haim aura adhika khane ki ora pravritta hote haim |
hama bahuta jaldi-jaldi khate haim aura isilie hama nahim janate ki vastavika bhukha kya hai |
dhire-dhire hama sharira ki prakritika pravritti ko kho dete haim, jaise ki bhukha, bhojana ke lie sharira ki manga, jaba isako isaki avashyakata hoti hai |
upavasa ko pani ko chhoda़ kara, sabhi aharom se dura rahane ke rupa mem varnita kiya gaya hai |
jaise bhukha manushya ki eka pravritti hai, vaise hi yaha janavarom mem bhi hai |
koi janavara khana tabhi banda karata hai jaba yaha bimara ho aura bhukha nahim ho |
matsya andajanana ki khoja mem pravaha ke viparita disha mem saikada़om milom taka tairate samaya silvara sailmaॉna haphtom taka upavasa karati hai |
rani chinti kuchha bhi nahim khati hai jaba taka ki andom se bachche nahim nikala jate haim| 
bahuta se kita aura pashu shita nishkriyata ke daurana khane se paraheja karate haim | 
aushadhi ke janaka, hippokraita, apane bahuta se rogiyom ko upavasa ki salaha dete the |
unhonne kaha, jaba apa kisi bimara ko khilate haim, taba apa bimari ko khilate haim, dusari tarapha yadi apa khane ko dura rakhate haim, bimari kshina ho jati hai |
itihasa aura vijnana ke bahuta se mahana purushom ne upavasa kiya |
sukarata aura plaito ne manasika aura sharirika durusti ko prapta karane ke lie 10 dinom taka upavasa kiya|
isa masiha ne parvata para 40 dinom taka upavasa kiya aura isilie shaitanom dvara mohita nahim kie ja sake |
mosesa ne parvata para kai dinom taka upavasa aura prarthana ki |
gandhiji ne 21 dinom taka chalane vale kai upavasa kie |
paithagorasa ne alekja़endriya vishvavidyalaya mem pariksha lene ke pahale 40 dinom taka upavasa kiya |
mohammada aura buddha ne kai dinom taka upavasa kiya |
jarmani mem bushingara sainetoriyama ke baida piramaॉnta mem 48 varsha ki avadhi taka 70,000 rasa upavasa sanchalita kie gae |
prakritika svasthyakara sairagahom mem daॉ. albarta bartana nem haja़arom upavasom ko sanchalita kiya |
mahana paraselsasa ne eka bara kaha tha upavasa hamare andara sabase mahana chikitsaka hai|
araba chikitsaka evisina ne 3-5 haphtom ke upavasa ka nuskha bataya |
plutarka ne kaha aushadhi ka upayoga karane ki bajaya behatara hai eka dina ka upavasa karo |
jarmani mem, daॉ. adolpha meyara ne kaha upavasa kisi bimari ko thika karane ka sabase prabhavi sadhana hai|
shikago vishvavidyalaya ke shariravijnana vibhaga ke prayogakartta, daॉ. karlasana aura daॉ. kunde ne eka 30 varshiya vyakti ko 14 dinom ke upavasa para rakha |
upavasa ke anta mem unhonne paya ki sharirika sthiti mem usake utaka eka 17 varsha ke yuva ke jaise the |
daॉ. kunde ne kaha ki, upavasa ke daurana, vaja़na 45% taka kama ho gaya |
18 mai 1933 ko jaba gandhiji apane 10vem dina ke upavasa para the, unake chikitsakom dvara unaka parikshana kiya gaya tha |
unamem se eka ne gandhiji ke svasthya para tippani karate hue kaha sharirika drishti se yaha bharatiya neta chalisa ke kisi vyakti jitana svastha tha |
vaja़na ka sampurna hrasa 157 paunda mula sharirika vaja़na se lagabhaga 38 paunda tha |
bharata mem prakritika rogamukti ke eka patha-pradarshaka, daॉ. vithala dasa modi ne apane karya ke 48 varshom ki avadhi mem, gorakhapura ke apane aspatala arogya mandira mem 35,000 upavasom ka nirikshana kiya | 
unhonne 40 dinom ka eka upavasa rakha aura isa upavasa ki hindi mem prakashita ‘upavasa se labha’ (upavasa ke labha) eka dainiki likhi |
yaha sochana thika nahim hai ki upavasa se koi vyakti kamajora aura durbala ho jata hai |
upavasa ke daurana samvedanaem tikshna ho jati haim |
yaham taka ki khane ki thoda़i si mahaka apake nathunom para hamala kara sakati hai |
eka upavasa ke daurana, koi vyakti behatara dekha sakata hai, behatara suna sakata hai aura behatara socha sakata hai |
jibha para svada kalikaem upavasa ke bada atirikta samvedi ho jati haim |
yaham taka ki sadharana khane ka svada bhi achchha lagata hai |
upavasa ke daurana, apa kama so sakate haim |
lekina yadi apa anidra se pida़ita haim to apako pratidina sambhavatah jyada se jyada 20 ghantom ki behatara ninda aegi |
upavasa hridaya, dhamaniyom aura rakta ko sahayata karata hai |
rakta mem kolestraॉla ka stara niche chala jata hai |
upavasa rakta mem ninna ghanatva vale laipoprotinom ko kama karane mem sahayata karata hai jo hridaya ke lie kharaba haim aura uchcha ghanatva vale laipoprotinom ko badha़ata hai jo hridaya ke lie achchhi haim |
rakta nalikaom mem rakta bharava kama hota hai aura isi prakara rakta chapa kama hota hai |
phephada़om ki sankulata upavasa ke dvara kama hoti hai |
kisi upavasa ke bada apa adhika vayu aura bahuta asani se sansa andara le sakate haim | 
sharira ke andara aise eka mahana kayapalata ke bavaja़uda khana lene ki lalasa se sangharsha karane ki koshisha karem |
jaisai-jaise sharira ke sanchaya sharira ka poshana shuru karate haim ye lakshana dhire-dhire gayaba ho jaenge |
chinta mata kijie; eka dina ke lie phalom para jivita rahane ki koshisha karem |
eka bara mem eka phala, dina mem tina bara lem |
sardi, pha़lu, atisara ya halke darda jaisi chhoti bimari ke daurana, eka chhote upavasa para jaie | 
ahara lena puri taraha se banda kara dem| 
aba shayada apa apana tina divasiya upavasa karane ke liye taiyara haim| 
khuba sara pani piliye| 
apa apani dinacharya jari rakha sakate haim| 
yadi apa jvaragrasta ya kamajora mahasusa karem, to arama kijiye| 
yadi apa dirghakalika bimari se grasta hai taba apa adhika lamba upavasa jari rakha sakate haim| 
eka chetavani , upavasa hara bimari ka ilaja nahim hai| 
yadi apako tapedika, koढ़, kainsara ya hridaya roga ho taba upavasa na karem| 
upavasa ke daurana sadaiva eka prakara ka sankata bana rahata hai| 
achanaka se apako yaha khyala satane lagata hai ki apaki halata bigaड़ gai hai| 
jaba apa isa avastha mem pahunche ghabaraiyega nahim| 
sankata ka ana eka achchha suchaka hai| 
yaha sharira ka eka tarika hai , yaha batane ki vaha hanikaraka padarthom ko sharira se bahara phenkane ki koshisha kara raha hai| 
aisa taba hota hai jaba hanikaraka padartha ko nishkasana ke liye raktapravaha mem phera diya jata hai| 
sankata ke prabhavom ke parinama mutra avarodhana, dasta, siradarda, ulti aura anidra adi ho sakate haim| 
ye chinta ya ninda puri na hone ki vajaha se ho sakate haim| 
isa samaya rogi ko atyadhika arama, taज़i hava aura suryaprakasha ki avashyakata hoti hai | 
upavasa ke daurana suryaprakasha lubhavana lagata hai| 
ise thoड़e samaya ke liye aura subaha-subaha ya phira shama ko lena chahiye jaba surya ki kirane saunya hoti haim| 
jaise-jaise upavasa ki avadhi baढ़ti jati hai vaise-vaise dhupa snana ki avadhi ko dhire-dhire kama karana chahiye | 
apako garama pani pine ki koshisha karani chahiye, usamem apa taज़e nimbu ka rasa mila sakate haim | 
yadi apa 3 dina ka upavasa svayam kara sakate haim, taba apa age chalakara 7 dinom ka upavasa bhi kara sakate haim| 
kisi margadarshaka ke bina dirdhakalina upavasa na karem|
apa upavasa kara sakate haim, lekina margadarshaka apaka sahi margadarshana kara apako apane lakshya taka arama se le jayega| 
vaha apako protsahana dega, upavasa ke daurana anevale sankatom se avagata karaega aura unaka samana karane ke liye taiyara karega| 
yadi apako lambe upavasa ki avashyakata hai taba kisi achchhe svasthya kendra mem jakara dakhila ho jaiye| 
vaham para bahuta se rogi eka satha upavasa karate huye mila jayenge | 
ve eka dusare ko protsahana dene ka kama karate haim| 
eka samuha mem upavasa karana adhika asana hota hai| 
gandhiji ne garama pani ke satha nimbu ka rasa, namaka aura soda bekarboneta lene ki salaha di| 
isase apako sankata ke lakshanom se ubharane va unase rahata pane mem madada milegi | 
kuchha loga garama pani mem taja़a nimbu rasa ke satha shahada lene ki salaha dete haim, do chhote chamacha dina mem tina bara| 
isase apako shakti milegi aura sankata se ubharane mem bhi madada milegi | 
upavasa ke daurana, pancha dinom ke bada bhukha gayaba ho jati hai| 
naड़i spandana kama ho jata hai, rakta chapa mem bhi giravata a jati hai| 
peta mem sravana kama ho jata hai| 
peta nitya ki taraha hi hara roज़ ahara ki asha karata hai aura vaha isi asha mem jathariya rasa ko sravita karata rahata hai| 
tina dina ke bada, peta ashvasta ho jata hai ahara nahim milega aura phira dhire-dhire srava bhi kama ho jata hai| 
isiliye tina dina ke upavasa ke bada peta ke alasara ka darda gayaba ho jata hai| 
svasthya kendra mem apa sharirika va manasika rupa se arama kara sakate haim| 
mainne kuchha sau rogiyom ke satha do-tina haphte ke upavasa satrom ka ayojana kiya hai| 
mainne paya ki 40 dina taka ka upavasa ekadama surakshita hai| 
mera sujava hai ki do-tina haphte ke alpavadhi upavasa karem aura jarurata paड़ne para kuchha mahinom bada ise doharaem| 
yadi apa upavasa nahim kara sakate, to apa alpahara lem sakate haim| 
taज़e phala( kele ko chhoड़kara) khayem aura kuchha dinom ke liye phalom ka rasa piye ya supa va ubali hui sabjiyam khayem| 
eka ya do haphtom ke liye paryapta arama ke satha yaha ahara niyantrana chamatkara kara dega| 
yaha peta ke alasara ke rogiyom mem madadagara siddha hoga aura isase ghava bhi puri taraha se thika ho sakate haim| 
tina dinom ke bada ankhe chamakadara ho jati haim, shravana shakti teज़ ho jati hai aura rakta mem lala koshikaom ki sankhya baढ़ jati haim| 
isaliye isase raktalpata (animiya) ke rogiyom ko bhi madada milati hai| 
jaise-jaise upavasa pura hone ko ata hai apake muha ke jayake mem sudhara ata hai, aruchi gayaba ho jati hai, jibha sapha va gulabi ho jati hai, sansa durgandharahita ho jati hai, peta phira se sravana shuru kara deta hai aura bhukha lauta ati hai| 
mana shanta ho jata hai aura sharira va mana para eka prakara ki shanti chha jati hai jaise ki apa ishvara se milane ke liye taiyara hom| 
apake upavasa chhodane ka yahi sahi samaya hai| 
upavasa chhodate samaya savadhani baratane ki avashyakata hoti hai| 
pahale dina thoड़a-thoड़a khaiye aura phira dhire-ghire apana ahara baढ़aiye | 
alpahara mem bhi sankata ate hai parantu ye bhayanaka kadapi nahim hote aura isaliye inase nipatana asana hota hai| 
yadi apa 2-3 haphte ka alpahara upavasa kara chuke haim, bada mem apa payenge ki apa lamba upavasa bhi kara sakate haim, sankata kama honge aura apa bina kisi pareshani ke lambe upavasa kara sakate haim| 
upavasa ke daurana pachana tantra pachaka rasom ko banana kama kara deta hai| 
aba apa jaisa ki eka bachche ke satha karenge , bilkula vaise hi apako apane pachana tantra ko phusalakara pachana prakriya ke liye prerita karana hoga| 
yaha dhire-dhire kiya jata hai| sadharanatah pachana prakriya ko samanya hone mem utana hi samaya lagata hai jitana ki upavasa mem laga tha| 
shuruata mem , upavasa karane vale vyakti ko asani se pachane vale ahara lene chahiye jisa ke liye bahuta hi kama pachaka rasom ki avashyakata paड़ti ho| 
dusare dina hara do ghante mem eka santara lijiye| 
tisare dina, eka adu ya eka nashapati ya phira hatha bhara angura dina mem tina bara khaiye| 
chauthe dina, nashte mem kisi bhi phala ki do phanke aura dopahara va ratri ke bhojana mem tina phanke lijiye | 
pachavem dina, nashte va dopaha mem do kisma ke phala khaiye aura rata mem ubali hui sabjiyam khaiye| 
chhathe dina, dopahara va rata ke bhojana mem sabjiyom ka supa lijiye| 
satavem dina, nashte mem phala khaiye, dopahara mem supa aura rata mem ubali hui sabjiyam khaiye| 
athavem dina, nashte mem phala khaiye, dopahara va rata mem ubali hui sabjiyom ke satha eka roti khaiye| 
nauvem dina, nashte mem phalom ke satha-satha eka kapa dudha bhi shamila kijiye, dopahara va rata mem ubali hui sabjiyom ke satha do rotiyam khaiye|
dasavem dina, dopahara va rata ke bhojana mem thoड़i-si dala bhi shamila kijiye | 
dhire-dhire, apane ahara mem thoड़e vasa padartha shamila kijiye, sabjiyam pakate samaya ghi ya tela ka upayoga karem ya phira apani rotiyom para thoड़a ghi lagayem| 
upavasa ke bada sharira ki shoshana shakti baढ़ jati hai| 
jaba ahara achchhi taraha se pacha jata hai aura sharira dvara shoshita kiya jata hai, taba rakta mem parivartana ana shuru ho jata hai| 
upavasa ke bada lala rakta kanikaom ki sankhya baढ़ jati hai| 
yaha ullekhita kiya gaya hai ki eka rogi mem, upavasa ke bada lala rakta kanikaom ki sankhya 1 miliyana se baढ़kara 5 miliyana ho gai| 
loha va anya dhatu jo sharira mem sanchita rahate haim sharira dvara liye va upayoga mem laye jate haim| 
upavasa ke bada sharira ki shoshana shakti apani uchchatama stara para hoti hai| 
kharagoshom ke samuha ko sataraha dinom taka upavasa para rakha gaya| 
upavasa ke bada unhonne apane sharira ka 56% vajana dhire-dhire sadharana stara taka laya| 
upavasa ke daurana tuti hui haddiyam va ghava adhika jaldi bharate haim| 
sujana va jalana jaldi kama hoti hai aura sharira javana ho jata hai| 
upavasa sharira ke angom ko pura arama deta hai| 
amataura para anga jo kama karate hai, usaka bhara kama ho jata hai| 
chunki koi ahara nahim liya jata isiliye koi pachana va shoshana prakriya bhi nahim hoti hai| 
atah jo puri shakti isamem lagati hai vo bacha jati hai aura vaha sharira se dushita padarthom ko nikalane mem upayoga mem lai jati hai| 
angom ko arama milata hai, ve thika hote hai aura apani atyavashyaka shaktiyam punah prapta karate haim| 
mula nala hanikaraka baiktiriya se mukta ho jata hai| 
kama atyavashyaka utakom se adhika poshana prapta kiya jata hai| 
srava, sujana, charbi, antahspandana adi jaldi se shoshita kara liye jate haim| 
atah sharira dhire-dhire svayam ko atirikta anachahe padartha se mukta karata hai| 
adhikansha loga enima se darate hai, lekina sabhi prakara ke virechakom mem yaha sabase sarala aura sabase kama hanikaraka hoti hai| 
eka ya do litara pani, sharirika tapamana para nali dvara gude se andara dala jata hai| 
pani malashaya (kolana) ke nichale bhaga mem pahunchata hai| 
yaha pani 2/3 minata ke liye ruka jata hai aura phira jame huye malina padartha ke satha yantravata dhanga se bahara nikala jata hai| 
enima ke upayoga ke bare mem vivada hai| 
kuchha loga upavasa ke daurana hara dina enima lene ki salaha dete haim| 
kuchha loga enima ke bahuta kama upayoga para jora dete haim aura kuchha bilkula hi upayoga na karane para| 
jo loga, enima ko hara roja़ lene ki salaha dete haim, unako lagata hai ki jo dushita padartha malashaya va usake nichale bhaga mem pahuncha jata hai, use vaham jamane nahim dena chahiye , kyonki parisancharita rakta dvara isake punahgrahana karane ka khatara rahata hai| 
subaha-subaha ya shama ke samaya dhupa mem sharira ki sikai, matra vitamina di ke banane ke liye kaphi hoti hai na ki tvacha ko jalane ke liye| 
surya ki kirane jaba khiड़ki ke shishe se hokara gujarati hai taba isaki adhikansha parabaingani kirane sokha li jati hai| 
sardiyom mem, khiड़ki ke shishe ke pichhe se dhupa mem sharira ki sikai ka ananda kabhi bhi liya ja sakata hai, isa prakara atyadhika parabaingani kiranom ke hanikaraka parinamom se bacha ja sakata hai | 
surya ka sukhada aura aramadayi prabhava usaki garmi ke karana hai|
dhupa-snana karate samaya hamem khushahali mahasusa hoti hai| 
garmi sharira ka darda kama karati hai aura peshiyom ko arama deti hai| 
isase apa teja़ bahane vali hava ki thandi lahara se bhi bacha sakate hai| 
dhupa-snana ka asara poshaka para nirbhara hoga| 
sapheda vastra suryakiranom ko paravartita karata hai jisaki vajaha thandaka paida hoti hai| 
vastra kale ho to vaha jyadatara garmi sokha lete hai jisase garamahata paida hoti hai| 
tvacha ko dhupa se julasane ke lie gore logom mem dhupa-snana karane ki adata kaphi prachalita hai| 
niyamita dhupa-snana ke pashchata tvacha ka julasa jana saundarya vriddhi ka karana mana jata hai| 
tvacha mem melanina vaha tatva hai jisase kala ranga banata hai| 
gore logom ki tvacha ke mailapigiyana namaka andaruni stara mem melanina maujuda rahata hai| 
yaha jahira hai ki grishma ki dopahara ki dhupa lagane se tvacha lala hoti hai| 
kuchha dinom bada tvacha ka erithema kramashah ghata jata hai aura sana-taina kahalane vale melanina tatva ke jamane se tvacha bhuri ho jati hai| 
parabaingani kiranom ke samparka mem ane ke lagabhaga 20 ghantom bada melanina ka kuchha ansha sataha ki tarapha badha़ne lagata hai| 
isake bada prarambhika koshika ki parata mem naye tatva ka nirmana hota hai, isa taraha niyamita dhupa-snana karane se tvacha kali hoti jati hai | 
dhupa mem chhuttiyam manane ke bada yaha loga jaba kama para vapasa ate hai to kuchha haptom taka unaki tvacha samvali rahati hai| 
usake bada melanina rogana kaphi phike ho jate hai| 
parantu yadi kuchha dinom taka unako parabaingani kiranom mem anavrita kiya jaye to ve kale ho jate haim| 
jyada dera khula rahane se phunasiyam ubhara sakati hai aura tvacha ki upari parata chhila sakati hai jise sana-barna kaha jata hai| 
nigro logom mem sana-barna ki tivrata kama hoti hai aura tvacha mem atyadhika rogana hone ke karana erithema thoda़ ho sakata hai| 
duniya ke hara hisse mem surya-prakasha se milane vali sasti urja ka istamala karane ki koshisha ho rahi hai| 
yadi tvacha ko baढ़ti matra ki garmi mem khula rakha jae to kramashah garamahata, ushnata aura jalana mahasusa hoti hai| 
jalana paida karane ke lie jaruri vikirani urja ki apeksha dasa guna urja tvacha ko nukasana pahunchane ke lie jaruri hoti hai | 
isi vajaha se kevala dhupa mem khuli rahane se tvacha jalati nahim hai| 
lensa se surya kiranom ko tvacha para kendrita karane se darda tatha jalana ho sakati hai| 
jaba koi surya-snana kara raha ho to use aksara dhupa se garamahata mahasusa hoti hai basharte usaki tvacha pahale se hi sana-barna se piड़ita na ho| 
sharira hara samaya apane upapachaya prakriya se thoda़i si garami bikherata hai| 
ye hrasa ke marga haim - chalana (kandakshana), samvahana (kanekshana) (pasina aura phephadom se nami ka vashpikarana) aura vikirana (redieshana)| 
isa taraha hama jaba barpha ke samparka mem ate hai to hama garami khote haim| 
garama chattanom se hama vaha pate haim| 
saura urja ke bare mem bhi yahi sacha hai| 
jaba dhupa bahuta teja ho to hamem dhupa se garami milati hai| 
isaka kuchha bhaga sidha hama taka ata hai to kuchha hissa ज़mina se paravartita ho kara ata hai| 
agara koi insana registana mem prati ghanta 3 ki.mi. ki raphtara se chale to vaha eka ghante mem lagabhaga 882 grama pani kho dega| 
isi karana garama mausama mem kama karane valom ko bara-bara pyasa lagati hai| 
nami se bhi kaphi asara paड़ta hai| 
agara dhupa-snana patthara ya hariyali ke pasa kiya jae to hariyali ke pani ke vashpikarana ke karana vaham ka tapamana patthara ki apeksha kama hoga| 
vashpikarana pasina nikalane ke satha isa dhanga se santulana banata hai ki sharira ka tapamana atyadhika baढ़ta nahim hai| 
dhupa se sharira ko bachane ka ekamatra upaya melanina rogana ka utpadana hi hai ya nahim isa bare mem thodi galataphahami hai| 
dhyana dene layaka aura eka mahatvapurna ghataka hai tvacha ki satahi parata ka mota hona jise epidarmala motai kaha jata hai| 
parabaingani vikirana prakriya ka tvacha para asara teja hota hai| 
isase bahya tvacha ki do paratem kaॉrniyama aura mailapighiyana ka bahujanana hota hai| 
kaॉrniyama ke motepana se sana-barna ki matra mem kaphi hada taka kami hoti hai, kyonki mailipigiyana ki parata taka pahunchanevala vikirana ghata jata hai | 
isa taraha kaॉrniyama ki chhavi ke rupa mem karyakarita baढ़ti hai | 
tvacha sana-barna ke prati kama samvedanashila banati hai|
krishnavarniya logom mem melanina ke utpadana ke alava kaॉrniyama ka mota hona tvacha ko julasane se bachane ka mukhya karana hai | 
adhika samaya taka aura matra mem dhupa lagane se tvacha ka kainsara hota hai | 
yaha gorom mem jyada aura nigrom mem kama prabala hai | 
ankhe hamari sabase amulya sampatti hai | 
agara surya ki kiranem adhika matra mem ankhom ke pratibimba banane ke parde para yane drishtipatala para pahunche to vaha kharaba ho sakata hai| 
hamare bujurga hamem salaha dete haim ki surya-grahana ko sidhe nahim dekhana chahiye| 
isa se grahana andhatva ho sakata hai | 
aise lakshana barphile kshetra mem teज़ dhupa pada़ne para bhi paida ho sakate haim| 
jaba adhika matra mem suryaprakasha ka prasarita paravartana ankhom taka pahunchata hai taba barpha-andhatva ho sakata hai | 
sadharanatah surya kiranom ke ghataka taranga daidhya ankhom ke drishtipatala ke bahari paratom mem vilina ho jate hai aura koi nukasana nahim pahunchate haim| 
dhupa teja ho to achchhe kisma ke dhupa ke chashme pahanana uchita hai | 
inaki banavata aisi ho ki ve drishtipatala taka pahunchane vali garami ko ghataem | 
yaha sacha nahim ki parabaingani kirane motiyabindu paida karati hai | 
yaha ninna tapa ke srota, jaise ki glasa bloarsa pharnesema, ke vajaha se ho sakata hai, jise hama glasa bloarsa motiyabindu kahate hai | 
paryavarana sambandhita ghatakom ke alava dhupa ke kai achchhe asara hamare sharira para hote haim | 
hava ke tapamana tatha samudri pravaha para dhupa asara karati hai | 
barisha para isaka asara to hoti hi hai, aura sabase mahatvapurna hai paudhom ke khadya utpadana pranali prakasha-sanshleshana para bhi hota hai| 
prithavi para jivana ka srijana surya ke prakasha aura ushma ke avirata pravaha se hi sambhava hua | 
agara surya ka prakasha aura garmi parivartanashila hote to jivana khatarem mem pada़ta; jivana atyadhika garami ya thanda mem jina paड़ta | 
khadya ke lie bhi jivana surya para nirbhara hota hai | 
surya ki garami vayumandala dvara pakada़i jati hai aura jo prakasha ko dharati taka pahunchata hai | 
prakasha prithvi ko tapata hai lekina paida hui garami vatavarana se antariksha mem asani se ja nahim sakati | 
ise grinahausa prabhava kaha jata hai | 
jaba grinahausa ke andara ke paudhom ko garama karane ke lie usaka kancha ka bana chhata dhupa ko andara ane to deta hai para vahi chhata use bahuta dhire-dhire vapasa bahara jane deta hai | 
paudhem apana khadya prakasha-sanshleshana prakriya se batate hai (ye dhupa ki urja ko hava ke karbana dayaksaida aura ज़mina ke pani se mila kara khadya nirmana karate hai ). 
isa prakriya ke daurana paudhem ऑksijana chhodate haim | 
insana aura janavara paudhom ne chhoda hua yahi ऑksijana sansa mem bhara lete haim aura nihshvasa ke samaya karbana-dayaksaida bahara chhodate haim jise paudhe phira istemala karate haim | 
dhupa mem ya dhupa-dipakom mem sharira ko tapane ko surya-snana karane ko kahate hai | 
surya prakasha kai prakara ke prakasha kiranom se bana hai | 
avarakta (inphrareda) kirane sharira mem gaharai taka jati haim | 
ye raktavaha-tantra aura mansapeshiyom ke lie labhadayi hai | 
hamari tvacha parabaingani kiranom ko sokha leti hai jisase sana-barna tatha taininga ho sakata hai | 
inhim kiranom ke asara se hamari tvacha ke rasayanika tatva vitamina-di ya dhupa ka vitamina banate hai | 
surya-dipa parabaingani kirane paida karate haim | 
aja hama surya ko prakasha, ushma aura anya kai taraha ki urja ka srota manate haim | 
manushya, janavara aura vanaspatiyom ka jivana puri taraha surya ki urja para nirbhara karata hai | 
grishma mem rajasthana ke registana mem dhupa aga barasa sakati hai | 
hava itani garama hoti hai mano sidhi angithi se a rahi ho |
yaha tvacha ko julasa sakati hai aura isase lu laga sakati hai | 
mukhyatah hamari sadharana rodhakshamata tatha bhavanatmaka, sharirika sthirata ke liye yaha bahuta mahatvapurna hai | 
manushya sharira para surya ki kiranom ka sidha asara, jo hamari chinta ka vishaya hai, ushnakatibandhiya logom ke liye pareshani bana sakata hai | 
surya prakasha dohara asara karata hai, prakasha-rasayanika pratikriya aura ushnata | 
hama aba janate hai ki vitamina di ka agravarti kaisipha़erala ke sanshleshana ke lie aura saura parabaingani vikirana jaruri hai | 
tvacha dvara kaisipha़erala raktapravaha mem chhoda jata hai | 
yaha harmona tvacha para surya prakasha ke paड़ne se banata hai | 
kaisipha़erala ke bina baढ़ti hui haddiyom mem paryapta kailshiyama jama nahim hoga | 
isake abhava se ghataka vikritiyam aura sukha roga (sukhandi) ho sakata hai | 
haddiyom ke upayukta vriddhi ke lie aura sukhandi ko talane ke lie paryapta surya prakasha ki avashyakata hoti hai | 
prakasha-rasayanika prabhava kuchha simita varnakrama (spektrama) taka hi parimita rahata hai | 
drishti mote taura para surya prakasha ke 0.4 mi. aura 0.65 mi. taranga daidhya ke bicha rahati hai jo surya prakasha ka ehama hissa hai | 
vayumandala se gujarane ke bada surya prakasha prithavi ki sataha taka pahunchata hai | 
vayumandala ki oja़ona parata parabaingani tatha avarakta kiranom ko sokha leta hai | 
kai tarikom se saura vikirana ko roka ja sakata hai | 
mumbai tatha landana aura anya dushita shaharom mem paya janevala audyogika kala dhuam unamem se eka hai | 
uttara bharata aura kai uttara yuropiya deshom ki lambi, kathora thanda bhi bachchom ko dhara ke andara jabarana lambi kaida bhugatane para majabura karati hai | 
yaha bhali prakara batalata hai ki, kyom greta britena va skaindeviyana deshom, madhyapurva deshom evam bharata mem asthi-nriduta evam sukha-roga ,sanyukta rashtra ki tulana mem ama roga hai| 
asthi-nriduta evam sukha-roga se bachane ke lie, bachchom ko prachura matra mem dudha dijiye, aura unhem suraja ki roshani lagane dijie| 
skaindeviya evam greta britena mem kulhe ki haddi tutane ke bada kiye gaye praudha़avastha ke marija़om ki asthi utaka-parikshana mem 25 se 30% logom mem asthi-nriduta dekhi gayi hai | 
adhitara, kama matra mem vitamina di ke sevana evam sharira para paड़ne vale surya ke prakasha mem hone vali kami ke karana yaha roga hota hai|
bachchom mem sukha-roga vitamina di ki kami ke karana hota hai| 
vayovriddha logom mem bhi asthi-nriduta vitamina di ki kami ke karana hota hai| 
suraja ki roshani dvara tvacha mem lagatara vitamina-di ke nirmana aura usake dvara sukha-roga ka upachara evam bachava nissandeha hi siddha ho chuka hai| 
sardi ko dura rakhane , pratirodhaka shakti badha़ane evam svastha arogya ke lie, dhupa khane (mem baithane) ka samarthana kiya gaya hai| 
jaba hama dhupa snana lete haim, yaha sharira ko darda se chhutakara dilane mem sahayaka hota hai, evam aramadeha hota hai| 
sadharana sardi-ja़ukama para, para-baingani kiranom ka prabhava jana-mana hai| 
logom ka eka samuha jisa para para-baingani vikirana dali gai , unamem 40% kama sardi-ja़ukama ke mamale darja kiye gaye| 
prachina kala mem kshaya roga ke nidana ke liye dhupa mem sharira ki sikai ki salaha di jati thi | 
labhaprada prabhava dhupa snana ke karana tha aura samanya rupa se paraheja mem bhi sudhara hota tha| 
tapedika ki pratirodhi davaiyom ke utpadana ke bada, sthana parivartana ya kuchha maha ke liye aspatala mem rahana avashyaka nahim samaja jata hai| 
jaba parabaingani kiranem vayumandala ki oja़ona parata se hokara gujarati haim, to inaka adhikansha bhaga samavishta ho jata hai| 
dopahara ke samaya jaba suraja ki sthiti nabbe digri para hoti hai, taba yaha vayumandala va oja़ona donom ki hi eka bahuta patali parata se hokara gujarati hai| 
isa samaya parabaingani kiranom ki matra bahuta adhika hoti hai, aura jo tvacha ke julasane ka karana bana sakati hai| 
subaha 8 baje se pahale ya shama 4 baje ke bada jaba suraja koniya sthiti para hota hai, taba yaha vayumandala evam oja़ona ki adhika moti parata se hokara gujarata hai | 
atah isa samaya parabaingani kiranom ki matra bahuta kama hoti hai aura sharira ki avashyakata ke lie yaha yatheshta hai| 
tvacha mem vitamina di ki utpatti ke lie parabaingani kiranom ki avashyakata hoti hai| 
jaba apa sacheta rahate haim, isa mastishka rupi duradarshana abhilekha para apaka niyantrana rahata hai | 
yaha nishchita tarka ke anusara karya karata hai| 
jaba apa sote haim, tarka sahita apaka mastishka bhi so jata hai| 
yadi koi tarka nahim hai, to vaham koi anukrama bhi nahim hai| 
yaha mastishka duradarshana abhilekhi kisi disha mem bhatakane chala jata hai| 
apake samane eka sinema ki rila ki taraha, tasvirem, avaja़em aura sabakuchha chalata hai| 
jo kuchha bhi parade para ho raha hai, apa bhi usake sahabhagi hote haim, apa ghatana para pratikriya karate haim, apane mastishka mem uttejana mahasusa karate haim, apa tanava mem a jate haim, aura apa aisa mahasusa karate haim, jaise kisi ne apake sharira ko chota pahunchai hai| 
apa sare anukramom ka samaghata mahasusa karate haim, lekina apa anukrama ki disha ko badala nahim sakate: yaha ghatita hota rahata hai aura apa usamem bhaga lete haim| 
aisa pratita hota hai, ki apa apani apurna ichchhaom, pida़aom, aura yatanaom ke bare mem svapna dekhate haim | 
yaham koi sima bandhana nahim hai, ki apa kya kara sakate haim,ya sharira ke satha kya ho sakata hai| 
apa eka chidiya ki taraha uda sakate haim, havaijahaja chala sakate haim, pahada se phisalakara gira sakate haim ya eka bagha ke satha ladai kara sakate haim|
apa kisi atyanta buri durghatana se grasta ho sakate haim, apa mare bhi jasakate haim, lekina apa yaham ina sabhi pida़aom mem manasika evam sharirika rupa mem saha rahem haim| 
apa kisi udyana mem ya samudra ke tata para ho sakate haim ya kisi lada़ki se mila, usase bata kara, use ghumane le ja sakate haim aura ananda ke pala vyatita kara sakate haim| 
yadi apako ratri srava hota hai| taba bhi apa thake hue, uttejita, aghatita uthenge| 
apa rata ki tulana mem savere kahim adhika thake hue uthate haim, apa svayam ko kamajora mahasusa karate haim, jaise kisi ne apaki sari shakti nichoड़ li ho, aura apako chiड़chiड़apana mahasusa hoga| 
kai bara jagane ke bada bhi lambe samaya taka sapane apake dimaga mem ghumate rahate haim, aura shayada apako shoka aura dukha se bhara dem| 
eka achchhi gahari ninda, apako sapane dekhane ki ajna nahim deti, ya kama se kama dekhane ki anumati deti hai | 
apa dilojana se koshisha kara ise pa sakate haim| 
iseke lie samaya, dhiraja, aura sachche prayasa ki avashyakata hai| 
apane sharira ko eka kapada़e ke thaile mem dala dem; sone se pahale apa apane mastishka ko bilkula shanta kara lem| 
apane mastishka se bata-chita karem, aisa ki jaise apa kisi vyakti se bata kara rahe hom| 
uttejita ya kruddha na hoiye, ise saunya tarike se spashtataya barambara karem| 
apako ashcharya hoga ki apaka prayasa kuchha hi dinom mem ranga layega, aura apa kama se kama sapane dekhenge| 
agara koi jutha sau bara dauharaya jata hai, to vaha sacha ho jata hai| 
isa niyama ka upayoga yaha sabita karane ke lie kiya jata hai ki akashavani, duradarshana va khabarom vali patrikaom mem sirpha punaravritti karane matra se hi hama khabarom para vishvasa rakhane lagate haim| 
mastishka asani se manane vala hota hai, yadi apa prayasa karem, to saphalata jarura milegi | 
sone se pahale pancha se dasa minata chintana karane ki salaha di jati hai| 
chintana chetana mastishka ko achetana mastishka para niyantrana rakhane ki shiksha deta hai | 
maim vaha rasta janata hum jisane bahuta se piड़itom ko rahata dilai hai| 
yaha chintana vipasana yoga kahalata hai| 
chintana karane ka abhyasa sadaiva eka prashikshaka ki sahayata se hi karana chahiye|
yaha eka vyavaharika prashikshana hai, jaise ki eka varishtha shalyachikitsaka nutana shalyachikitsaka ko prashikshana deta ho| 
isa vipasana yoga ko jise ana-pana kahate haim, muje isake kuchha mula prathamika kramom ka ullekha karane dijie | 
apa inaka abhyasa apane ghara para kara sakate haim | 
apa ekagrata ke satha saphalata jarura pa sakate haim| 
isaki vidhi sadharana hai, lekina apa ise eka dina mem nahim sikha payenge, agara apa isake pichhe lage rahenge, to shayada sikhane mem eka haphta, dasa dina ya phira eka mahina bhi laga sakata hai| 
apani shvasana kriya para dhyana dem aura shvasa ko mahasusa karem, ki vayu apake nathunom mem ja rahi hai, aura unhim se bahara nikala rahi hai| 
jaba vayu aisa karati hai, to vaha apake upari otha ke charom ora evam nathune ke antarika bhaga ko sparsha karati hai|
apa hara pala shvasa lete haim lekina apa isase anabhijna rahate haim| 
agara apa apani shvasa para dhyana kendrita karem to apako mahasusa hoga ki vayu apake upari otha evam nathune ke charom ora se hokara guja़ra rahi hai| 
agara apa ko taba bhi mahasusa nahim ho to teja़i se shvasa lem| 
jaba apa shvasa mahasusa kara sakem, dhimi gati se lem, apane samanya shvasana para vapisa a jaem, aura upari otha evam nathunom para vayu ka sparsha mahasusa karate rahiye | 
dhire-dhire apaki samvedanashilata badha़ti jayegi, vayu ka andara jana, evam nathunom se bahara nikalana aura charom ora sparsha karana, yaha saba, kuchha hi dinom mem apako mahasusa hone lagega| 
ye abhyasa 10-15 minata taka karem jaba taka apa so na jaem| 
kuchha dinom bada apa dekhegem ki apa ke sapane dhire-dhire kama va aura kama ho rahe haim, aura apa gahari ninda pane mem samartha ho gaye haim|
yaha aisa hai jaise eka bachcha, jo apako kama karane na de raha ho, usase sidha़i para taba taka upara chadha़te aura utarate rahane ko kahana jaba taka apa use rukane ko na kahem| 
apa apane shvasana ka pratyaksha-bodha karate huye jaba sone jate haim, taba yaha bodha apake chetana mastishka se achetana mastishka dvara le liya jata hai|
puri rata mastishka shvasana ke pratyaksha-bodha ko mahasusa karane mem vyasta ho jaega, aura isa taraha sapane dekhana bhula jayega| 
hama sabhi ke pasa nidra ka eka chakra hota hai| 
agara hama eka vishishta samaya para sone jaem to hamem achchhi ninda ati hai| 
hama mem se kuchha eka ka chakra rata 9.00baje, 10.00baje hota hai, ya phira kuchha logom mem itane adhika vilamba se ki savere 4.00baje bhi ho sakata hai| 
vaha loga jo vyapaka rupa se pure vishva mem yatra karate haim, isa bata ka anubhava prabalata se karate haim| 
vayuyana se niyamita yatra karane vale loga jeta vimana se hone vali thakana ke saunya se lekara gambhira lakshanom se pida़ita rahate haim|
samaya mandala ko para karana samanya manasika aura sharirika laya ke asta-vyasta hone ka karana mana jata hai| 
yadi apa eka rata dera se sote haim, to apa usa adhuri ninda ko dusare dina rata ko jaldi sokara pura nahim kara sakate haim| 
nyuyaॉrka ke mastainphorda vishvavidyalaya evam maॉntiphaina aspatala ke shodhakartaom ne yaha paya ki sharira eka baitri ki taraha nahim hai jise pratidina rata ko arama karake phira se charja karane ki avashyakata hoti ho | 
shayana aura jagrata avastha donom sharira ke tapamana se sambandhita hote haim| 
yaha taya karata hai ki kaba eka vyakti uchita nidra le sakata hai| 
udaharana ke taura para yadi vaha jaldi sokara tapamana chakra toda़ta hai, to usaka vishrama aniyamita aura asantoshajanaka hoga| 
jo loga lambi avadhi taka sote haim, mana lijiye solaha ghante, vaha aisa isa liye karate haim kyonki ve, sharira ko jagane ke prabhavakari sanketa ke abhava mem ninda ke do chakrom se gujarate haim |
anidra roga se pida़ita logom ke liye jalachikitsa atyadhika sahayaka siddha hoti hai| 
achchhi nidra pane ke liye aneka upachara upayogi hote haim| 
eka shithila sharira achchhi nidra pane mem sahayaka siddha hoga|
vibhinna jala chikitsaom ki sahayata se sharira ke tapakrama ki layom ko badala ja sakata hai| 
yadi apa aise kamare mem sote haim jaham para atyanta bhiड़ ho ya asantoshajanaka dhanga se prakasha evam hava ka sanchara hota ho to apaki nidra sphurtidayaka nahim hogi| 
agara apa bhida़-bhada़ vale sthana,ya shoragula aura vyasta saड़ka ke kinare ya relave laina ke pasa rahate haim to apaki nidra ashanta rahegi| 
apaka sharira kuchha hada taka shoragula ka abhyasta ho jata hai| 
kuchha dinom bada shoragula apaki nidra ko kaphi hada taka prabhavita nahim karata, lekina yadi apaki ninda kachchi hai,to isaka asara apaki ninda para pada़ega| 
agara apa so nahim pa rahe haim to kisi shanta sthana mem rahane ki koshisha karem, ya phira sthiti ko sudharane ke liye vatanukulana-yantra lagava lem| 
agara apa so nahim pa rahe haim, chinta mata kariye, utha jaiye| 
udaharana ke taura para, eka aramadeha kursi para chhoti machhalishala ke samane baitha jaiye, aura chhoti machhaliyom ko unake chamakile rangom ke satha dhyana se dekhiye| 
koshisha karem ki kuchha bhi aura na sochem , machhaliyom ke ane jane, tairane, uड़ne, palatane, milane, eka dusare ke pasa se gujarane va pichha karane, ina saba krida़aom para dhyana dem| 
unhem aise dekhate rahem,jaise koi bachcha dekhata hai| 
eka bachcha ina saba chija़om ka ananda aura asani se leta hai| 
vaha chija़om ki kalpana bahuta teja़i se karata hai| 
apa bachche ko eka kahani sunaem aura apa use eka kalpana ki dusari duniya mem le ja sakate haim aura vaha isaka sampurna rupa se ananda leta hai 
eka bachcha banane ki koshisha kariye| 
sochem ki apa eka machhali ki taraha haim aura pani mem taira rahe haim, gatishila haim, palata rahe haim, disha parivartana kara rahe haim, thande aura svachchha pani mem halke aghata kara rahe haim aura yaha saba itana sukhada lagega, ki kuchha hi kshana bada apako apani ankhe bojila lagengi, aura yahi uchita samaya hoga ki apa turanta sone chale jayem| 
eka dusara tarika hoga ki apa koi sidi ya tepa - bhajana, gaja़la ya shastriya sangita ka jo bhi apako sunana sabase adhika pasanda ho,lagakara sune | 
apako pata bhi nahim chalega ki kaba apa nidra devi ki pyari goda mem so gaye haim|
ise amantrita na karem, sone ki utkata ichchha na karem, basa leta jaem aura arama karem |
yadi apa nahim so pa rahe haim to chinta na karem kyonki yadi apa puri rata bhi so nahim pate haim, to basa atha ghantom ke lie leta jaem aura arama karem |
yaha chara ghante ki gahari ninda ke barabara hai |
isa prakara apa kabhi bhi ninda ki kami ke karana pida़ita nahim honge |
yaha apake kama mem badha nahim dalega, isalie ashvasta rahie, yadi apa sone ki utkata ichchha karate haim to apa ise dura bhagate haim |
bina ninda ke do dinom bada, ve ekagrata kho dete haim |
yaham taka ki apane samanya, dainika karyom mem bhi bahuta si galatiyam hoti haim |
koi bhi jo tina dinom taka bina ninda ke rahata hai use sochane-vicharane, dekhane aura sapha sunane mem bahuta adhika kathinai ho sakati hai |
vaha vibhranti ke daura se gujara sakata hai |
na sone vale vastavikata se sambandha kho dete haim aura divasvapna dekhane shuru kara dete haim |
jaba eka vyakti so jata hai, taba sabhi kriyaem ghata jati haim aura mansa-peshiyam arama karati haim |
hridaya dhada़kana aura shvasana dara kama ho jati hai |
jaise hi koi vyakti sota hai, usaka mastishka dhimi lekina adhika bada़i aura usase adhika bada़i tarangem bhejata hai (ilektroensiphelographa ke dvara jo ki ise napane ke lie upayoga kiya jata hai) |
adhikatara vayaska pratyeka ratri 7 se 8 1/2 ghantom taka sote haim |
kuchha vayaska kevala chhah ghantom ki ninda ke satha kama chala sakate haim | 
60 para, logom ko sata ghantom se kama ki ninda ki avashyakata ho sakati hai |
hamem dhimi-taranga vali ninda aura svapna vali ninda donom ki avashyakata hoti hai |
kisi bhi eka prakara ki atirikta ninda dusare prakara ki ninda ki kami ko pura nahim kara sakati hai |
dhimi-taranga vali ninda mansapeshiyom, granthiyom aura anya sharirika tantrom para mastishka auऱ tantrika-tantra ka niyantrana punahsanchita karane mem sahayata karati hai |
svapna vali ninda sikhane, tarka-vitarka, aura bhavanatmaka samayojana jaisi manasika kriyaom ko uchcha banae rakhane ke lie mahatvapurna ho sakati hai |
vaijnanika yaha khojane ki koshisha kara rahe haim ki manava jati ko sone ki avashyakata kyom hai aura kaise yaha sharira ke pra bala ta ko punahsanchita karati hai |
udaharana ke lie, kita nahim sote haim, ve kevala arama karate haim |
ninda vishrama ki vaha avadhi hai jisake daurana sone vala apane prativesha ki chetana kho deta hai |
achchhi gunavatta vali ninda achchhe svasthya ke lie avashyaka hai |
yaha rakta chapa ko niche laegi, pratyurjata ghataegi, udariya samasyaom ko shanta karegi aura tantrikaom ko arama pahunchaegi |
apa eka svachchha mastishka ke satha, urja se bharapura aura eka hrishta-pushta hridaya ke satha uthenge |
apa bina thake hue dina ke sharirika shrama aura tanava lene ke lie sakshama honge |
jaise hi ve takie para apana sira rakhate haim, turanta hi ve eka gahari ninda mem hote haim | |
ratri ke alaga-alaga ghantom samaya mem ninda ki alaga-alaga gaharai hoti hai |
udaharana ke lie, chora hamesha pratahkala ke prarambhika samaya mem lagabhaga 2.00 baje adhika sakriya hote haim jaba adhikatara loga gahari ninda mem hote haim aura shora-gula, aksara unhem jagati hui nahim lagati hai |
aise loga bhi haim jo ratom ko darate haim kyonki pratyeka ratri unake lie thoda़i si bhi ninda prapta karane ke lie eka sangharsha hoti hai |
ve thake hue, chida़chida़e uthate haim aura unhem apane dainika karyom ke lie vastava mem khuda ko dhakelana hota hai | 
isa prakriya mem, dina ke samaya ve jaldi thaka jate haim |
yaha shirsha ke prabandhakom ke satha, pure samaya bahuta sari samasyaom se lada़te hue eka vyapari ke satha, eka susampanna grihani ke satha, sharirika shrama se atyadhika thake hue eka purusha ke satha ya yaham taka ki apane tisa ke dashaka mem teja़i se apana akarshana kho rahi aura apane apa ko lada़kom ke prati kama akarshita pa rahi, aura shadi ke ankusha mem phansane ke lie utsuka eka lada़ki ke satha bhi ho sakata hai| 
kama ninda ankhom ke charom ora kale ghere utpanna karati hai aura isa prakara ise kama akarshaka banati hui tvacha se chamaka uda़a deti hai | 
lada़ke jinhonne apani shiksha puri kara li hai aura apana pesha shuru karane ke lie pratiksha kara rahe haim aura aisa karane mem sakshama nahim haim, shayada paryapta rupa se nahim so paem|
jaba apake pasa eka kusamayojita parivara hota hai taba ninda badhita ho jati hai |
jaba patni pati ke satha sahayoga nahim karati hai |
yadi apane koi rina le liya hai aura ise vapasa bharane mem sakshama nahim haim |
koi bhi chinta apako ninda se dura rakha sakati hai |
hama saba adhikatara pratah kala ke prarambhika samaya mem, jaba ninda halki hoti hai, taba sapane dekhate haim |
ve sukhadayaka sapane ho sakate haim |
bahutom ko ninda ane mem kathinai hoti hai aura jaba ve sone ka prayasa karate haim, to unake pasa bhayanaka sapane hote haim jo dusvapna mem badala jate haim | 
ye sapane dukhadayaka hote haim |
kuchha chite, billiyam, sampa, dekhate haim ya isake atirikta ve apane sharira ko eka pahada़i se girate hue pate haim |
ve uchita rupa se taiyara hone ya apane jute pahanane ya yaham taka ki chala na sakane ke eka bahuta bada़e khatare ka samana karate hue, dikhate haim|
apake paira bhari ho gae haim apa chalane mem sakshama nahim haim; apa dauda़ne ki koshisha kara rahe haim lekina paira age rakhane mem sakshama nahim haim |
apa vahi sapana ratri dara ratri dekhate haim |
apa thake hue, chida़chida़e, dila mem bina kisi utsaha ke jagate haim, apa kamaja़ora mahasusa karate haim jaise ki kisi ne apa ki sari urja nichoda़ di hai, mastishka aspashta hai aura sira bhari hai | 
apa thake hue haim aura kama karane ka koi utsaha nahim hai |
vyaktigata prayatnom se sone ki avastha kabhi nahim prapta ki ja sakati hai |
yaha sabhi manasika kriyaom ke samapta hone ke bada apane anusara ati hai |
lekina apa manasika rupa se taiyari karake ise kara sakate haih achchhi gunavatta vali ninda pane ke lie kisi prayatna mem eka achchhi prishthabhumi |
yadi apa atyadhika thake hue ya tanavapurna haim to turanta hi sone ke lie na jaem |
apa bistara para jane se pahale thoda़a aura samaya lijie |
apane vicharom ko darda ho rahi mansa-peshiyom para sankendrita hone dem aura unhem eka eka karake tanavamukta hone dem |
apani tantrikaom ko dhila chhoda़em jaba taka ki sharira apake apane bistara para kapada़e ke eka tuda़e-muda़e tukada़e ki taraha nahim mahasusa karata hai jaise ki apake pasa na hi mansa-peshiyam aura na hi haddiyam haim |
apa aisa karane ke bada, usi taraha apane mastishka ke satha kijie |
kisi vishishta vichara para dhyana kendrita mata kijie, apane vicharom ko shanta kijie, kisi samasya ko sulajane ki koshisha mata kijie |
usa dina ke sabhi vicharom, prabhavom, samvedanaom ko hatane ki koshisha kijie |
gati ko teja़ mata kijie, ise shanti se kijie jaise ki apa apane mastishka se bata kara rahem haim aura use samaja rahe haim |
jaise-jaise nirantara prayasa ke dina bitate haim, apa mahasusa karenge ki apa hara samaya aura adhika achchha kara sakate haim |
yadi apa nidra takanika para niyantrana kara sakate haim, to apa tanavamukta mahasusa karenge aura mahasusa karenge ki sharira kapada़e ke eka tukada़e ki taraha hai aura yaha ki apa sira se lekara paira taka purna rupa se tanavamukta haim |
basa apane apako eka prashanta avastha mem rakhem | 
apa kabhi anubhava nahim karenge, ki kaba apa eka shantipurna taja़gi dene vali ninda mem jate haim |
yadi apa isa abhyasa mem nirantara rahate haim to apa saphala honge aura eka dina apa ise niyantrita karane mem sakshama honge | 
ninda aba aura eka samasya nahim rahegi |
aise daॉktara haim jo bistara para jane se pahale pancha se dasa minata taka dhyana lagane ki salaha dete haim |
kisi bhi kimata para, ninda ki dava lene se bachem yadi apa thika se so nahim pate haim taba bhi | 
eka ninda ki dava nigalane ke bada jo ninda apa prapta karate haim vaha eka achchhi gunavatta vali ninda nahim hoti hai| 
agali subaha apako eka pashcha-prabhava ho sakata hai, nidralu mahasusa kara sakate haim aura sabase bada़i hani hai ki apa isa dava ke adi ho sakate haim aura bina dava lie apa sone mem sakshama bhi nahim ho sakate haim |
bina kisi sambandha ya ruchi ke anaginata sapane hote haim |
mastishka eka telivija़na kaimare ki taraha hai |
yaha sabhi vicharom, anubhavom, ghatanaom, sukhada kshanom, bhayanaka kshanom, bachapana, vidyalaya ke dinom, mahavidyalaya ke dinom, dukhom aura uttejanaom, asaphalataom, upalabdhiyom, apa para kie gae utpida़nom ko rikaॉrda karata hai aura ye sabhi chija़em apake pasa vapasa ati haim | 
yaha eka asambaddha mashina ki taraha rikaॉrda karata hai aura jo isake raste mem ati haim una sabhi ghatanaom ko rikaॉrda karata hai |
jaba apa mahasusa karate haim ki apa asani se apani sansa adhika dera taka thamem raha sakate haim, to avadhi ko 10-15 sekendom taka badha़aem |
yadi apa kanapati ya sira ke charom ora tanava ya phephada़om mem tanava mahasusa karate haim, to isaka artha hai ki apa sima ko para kara rahe haim |
yaha abhyasa pura karane ke bada kuchha gahari sansem lem aura shavasana karem |
yaha tantrikaom aura mastishka ke bicha talamela vikasita karata hai | 
yadi thika prakara se kiya gaya hai, to apa apane apako urja se bhara hua mahasusa karenge |
yaha apake kama karane ki kshamata ko badha़aega|
yaha unaki sahayata karega jo ninna rakta chapa ya alasya se pida़ita haim |
yaha uchcha rakta chapa ya hridaya roga ke rogiyom ke lie upayukta nahim hai |
bahya shvasana ke bada apa apane shvasana ko roke rakhane ke lie taiyara haim | 
samanya rupa se, asani se aura dhire-dhire jaham taka sambhava ho phephada़om ko rikta karate hue, bina kisi tanava ke shvasa chhoda़e |
jaham taka sambhava ho sansa ko rokem phira samanya rupa se antah shvasana karem, 10 minata taka jari rakhem |
kanapati para dabava aura udara mem hamphane ki anubhuti yaha darshati hai ki apane apani sima para kara di hai |
kuchha gahari sansem lem aura shavasana karem |
yaha tanavamukti ke lie achchha hai aura raktachapa kama karane mem sahayata karata hai |
yaha mastishka ki shanti utpanna karata hai |
ve tina prakara ke hote haim – jalandhara bandha, uddiyana banda aura mula bandha 
ve apake shvasana ki laya para niyantrana pane ke bada antarika pratidharana ke samaya upayoga kie jate haim |
ye do bandha usa samaya shuru kie ja sakate haim jaba apa apani sansa ko 10-15 sekendom taka dharana karane mem sakshama hom |
eka agravarti vidyarthi dvara bahya pratidharana uddiyana ke satha joda़a ja sakata hai | 
bahya pratidharana ke bada, vyakti udara ko bhitara khinchata hai (uddiyana)| 
jaise hi abhyasa ke dvara sthirata prapta hoti hai hama samanya shvasana avadhiyom ki sankhya ko ghata sakate haim |
yada rakhie bandhom ko karate samaya, gardana ko mata khinchie |
kisi bhi bindu para bala prayoga mata kijie |
bandhom ke satha eka karyavadhi mem 6-8 chakrom se adhika bahya ya antarika pratidharana mata kijie |
apani kshamatanusara pratidharana ki avadhi aura chakrom ki sankhya badha़aem| 
yaha dina mem kevala eka bara kiya jana chahie aura eka achchhe yogi se prashikshana prapta karana hamesha hi behatara hota hai |
apake hatha ki mudraem mahatvapurna haim aura shvasana ko parivartita karane aura apake mastishka ko eka shantipurna prabhava dene mem eka bhumika nibhati haim | 
apa unhem niche ki ora ya upara ki ora phalaka kie hue hatheliyom ke satha kara sakate haim |
ise baithe hue kijie aura apani ridha़ sidhi rakhie | 
apa ise padmasana, vajrasana mem, palathi mare baithe hue ya phira kisi kursi para baithe hue bhi kara sakate haim| 
apani tarjani aura anguthom ke shirshom ko chhute hue eka gola banaie |
koi dabava nahim dalie; anyatha gola koniya ho jaega |

aba pancha sekendom taka apani shvasa dhire-dhire aura arama se andara lijie, do sekendom ke lie apani shvasa dharana kijie aura 7 sekendom taka ise bahara nikalie aura eka sekenda ke lie apani shvasa thame rakhie |
apa vayu ko apane phephada़om ke upari khandom mem pravesha karati hui mahasusa karenge |
apane golom ko akshunya rakhie aura tina anya ungaliyom ko adhi muththi ke rupa mem moda़ie aura phira se apane ghutanom ya jangha para niche ki ora phalaka kie hue rakhie aura usi prakara se shvasana shuru kijie, apa apani shvasa ko phephada़om ke madhya khandom mem pravesha karati hui paenge | 
15-sekenda ke isa chakra ko 5 se 10 bara doharaem |
sarvaprathama, apane anguthe ko apani hatheli ke para rakhem jisase ki isaka shirsha chhoti ungali ke adhara ko chhue |
aba eka muththi banane ke lie charom ungaliyom ko anguthe ke upara banda kare |
apani muththiyom ko apani janghom ya ghutanom para niche ki ora phalaka kie hue rakhie aura shvasana chakra phira se doharaem |
apa vayu ko apake phephada़om ke ninna khandom mem pravesha karati hui mahasusa karenge |
tina anya mudraem haim jo apako tanavamukta hone mem sahayata karengim aura unhem karate samaya apa shanti aura prashanti ko apake mastishka para tairati hui mahasusa karengem |
sabhi anya ungaliyom ko phalae rakhie aura phira se apane ghutanom ya janghom para rakhie lekina isa bara hatheli upara ki ora phalaka kie hue |
samanya rupa se shvasana karem aura aba kramika rupa se ise manda karem |
aba apa apani tarjaniyom ke shirshom ko apane anguthom ke madhya joda़ ko chhune ke lie niche laie |
aba apaki tarjaniyom ke shirsha anguthe ke adhara taka aura adhika niche khisakate haim |
hantha khule aura upara ki ora phalaka kie hue haim |
apana bayam hantha upara ki ora phalaka kie hue niche apani goda para rakhie |
apana dayam hantha upara ki ora phalaka kie hue apane bame hantha para rakhie |
yaha gahana tanavamukti ke lie eka mudra hai |
yaha mudra buddha aura mahabira dvara dhyana ke lie upayoga ki gayi thi | 
ina mudraom mem apa sekendom ko na ginem |
apa vayu ka antahpratidharana ya bahya pratidharana na karem |
basa dekhie, kaise apaka shvasana bina kisi sacheta parichalana ke jari rahata hai |
jaba apa pranayama karana jari rakhate haim taba apa apane hathom ki mudraom ko parivartita karate raha sakate haim |
pranayama mem bandhom ko karate samaya pranayama ke satha mahabandha, hathom ki vishishta mudra upayogi hai |
apana dayam agutha apani dayim hatheli para rakhem aura sabhi ungaliyom se banda karem |
usi prakara apana bayam hantha karem |
hanthom ko apani goda para ungaliyom ki ganthem amane-samane chhute hue aura muththiyam upara ki ora phalaka kie hue rakhie |
yaha mudra upayogi hai jaba pranayama vayu ke andara ya bahara pratidharana ke satha kiya jata hai |
vayu ka pratidharana karate samaya ise tina suraksha kapatom sahita satha-satha karana avashyaka hai | 
sira age ki ora jukana chahie, thodha़i ko sine ke nikata laya jana chahie, guda ko andara khincha jana chahie, udara ko andara khincha jana chahie aura jaba taka pratidharana jari rahata hai jari rakha jana chahie |
yaha maha bandha bhi kaha jata hai |
yadi apa pate haim ki apa pranayama nahim kara sakate haim, to ho sakata hai ki apake pasa anukula samaya ya sthana na ho ya ki yaha apaki prakriti se mela na khata ho | 
koi bata nahim, apa kuchha anya adhika joradara chija़ ki koshisha kara sakate haim, aura jisake lie apako eka shanta aura ekanta sthana ki avashyakata nahim hogi |
yaha apaki prakriti ke satha mela kha sakata hai yadi yaha adhika joradara hai | 
aura vaha hai tairaki, adhika uchita rupa se bresta-stroka tairaki |
tairaki karate samaya, apa yada karenge, ki jaba apane pahali bara shuru kiya to apane kitana shvasahina mahasusa kiya tha |
apako sansa eka nishchita avadhi ke lie dharana karani thi aura phira teji se sansa ka antahgrahana |
jaise-jaise apane abhyasa kiya aura adhika anubhavi ho gae sanhalane ki kshamata unnata ho gai | 
tairaki vyayama ka eka sabase achchha rupa hai, kyonki apa taranashila haim aura apane sabhi padom aura sharira ko pratyeka sambhava disha mem chalane mem sakshama haim |
tairaki apako kevala vyayama ki taraha hi nahim sahayata karegi balki isamem pranayama ke sabhi labha sammilita honge |
yadi apa ativaja़ni haim to apana vaja़na kama karenge aura yadi apa adhovaja़ni haim to apana vaja़na badha़aenge | 
jaba yaha kama karata hai, to yaha udariya angom ko niche ki ora dabate hue aura udara ko bahara ki ora phulate hue adhikadhika patala hota jata hai |
uchcha shvasana mem, yaha phephada़om ka upari bhaga hai jo vayu se bhara jata hai |
madhya shvasana mem, kevala madhya aura upari bhagom ka bahuta thoda़a bhaga upayoga hota hai |
udariya shvavasana mem sampurna ninna aura madhya bhaga vayu se bhara jate haim, isalie yaha tinom mem sabase achchha hai | 
tathapi shvasana ka sabase achchha tarika taba hoga jaba phephada़om ke upari, madhya aura ninna bhaga vayu se bhare hote haim |
yaha pranayama shvasana kahalata hai |
yaha pratyeka koshika aura mansa-peshi sahita, pure shvasana tantra ko sakriya rakhata hai |
bahuta se loga apane munha se sansa lete haim, visheshatah bolate samaya |
adatana, bahuta se loga sote samaya apani naka se sansa lena bhula jate haim |
yaha adata niyantrita ki jani chahie |
shvasana ke apane tarikom para adhika dhyana dijie |
dhyana dem aura dekhem ki apaka bachcha kaise sansa leta hai |
yadi vaha eka uchita adata uparjita karata hai to vaha ise apane jivana bhara rakhega | 
bahuta se bachche apane munha se sansa lete haim |
unaki vriddhi kama ho jaegi, aura thairaॉyada uchita prakara se kama nahim karegi |
unako kharaba gale aura taॉnsila ki samasyaem ho jaengi|
edanaॉyada munha se sansa lene ka karana pratita hota hai, lekina yaha isake viparita hai yadi hama apane bachchom ko sahi shvasana sikhate haim to unaka edanaॉyada niyantrita kiya ja sakata hai | 
naka sabhi rakshatmaka yuktiyom se susajjita hai |
vayu jise hama sansa lete haim, vaha dhula ke kanom aura chhote kitom ko nikalane ke lie naka ke balom dvara chhani jati hai |
age phephada़om mem pahunchane se pahale jaba vayu ghumavadara patha se gujarati hai, yadi bahuta thandi hai, to yaha garma ki jati hai, aura yadi yaha bahuta garama hai to thandi ki jati hai, aura isa prakara yaha adhika anukula banai jati hai | 
hamara shvasana, jitana hama kara sakate haim, ina jaharili gaisom se bachane ke lie parivartita hota hai |
edanaॉyada hanikaraka jivanu aura vishanuom se lada़ne ke lie hamari sahayata karata hai jo hamari sansa li isa vayu ke satha a jate haim |
edanaॉyada upayogi haim, isalie jaba apako taॉnsilaitisa hota hai to unako katavane ke lie jaldabaja़i mata kijie kyonki aksara taॉnsilom ko hatae jane ke bada bhi kharaba gale ki shikayata bani rahati hai |
aba hamem vishvasa ho gaya hai ki taॉnsila upayogi haim aura hamare rodhaka tantra ka eka bhaga haim jo hamem bimariyom se lada़ne mem sahayata kara rahi haim |
shvasana ke sahi tarike ko sikhane mem hamem dhire-dhire age badha़na chahie aura apane apako adhika thakane se bachana chahie |
kisi ki bhi chija़ atyadhikata kharaba hoti hai aura yadi pranayama ke abhyasa ke samaya hama aniyamita ya avyavasthita haim to bhi yaha sacha hai | 
gahare antahshvasana mem hama phephada़om ko phailakara madhyapata (dayaphrama) ko niche ki ora dhakelate haim |
udara ke phailava ne vayu ko hamare phephada़om ke ninna bhaga mem pravesha karane ke lie prerita kiya |
sunishchita hone ke lie apa apane hantha ki hatheli ko udara para rakhie aura sanchalana mahasusa kijie |
purusha paenge ki yoga shvasana ka prathama charana kathina nahim hai |
striyam jinhonne khela-kuda mem bhaga nahim liya hai ya vyayama nahim kiya hai ve ise kathina paengi |
lekina ve sabra aura ichchha shakti se apani kathinai ko para kara lengi |
yaha dhyana dena ruchikara hoga ki jo vyakti udariya shvasana nahim kara sakata hai usako pachana samasyaem haim, kabja़ aura striroga sambandhi gada़bada़iyam hoti haim | 
dusare charana mem hama madhya shvasana ko samavishta karane ki koshisha karate haim |
hama apani ninna pasaliyom ko aura chhati ke madhya bhaga ko phailate haim jisase ki thoda़a thoda़a karake vayu madhya chhati mem pravahita hoti hai |
tisare charana mem antah shvasana ke samaya chhati ka pura phulava karate haim |
apa vishvasa karenge ki chhati ke phulava dvara pure antah shvasana ke bada aura sansa andara le sakate haim |
yaha phephada़om ko sahara dega aura upari phephada़om ko vayu se bharane mem sahayata karega | 
abhyasa mem hama bina rukavata ke eka kriya kalapa se dusari mem dhire se chale jate haim |
eka ora se dekhane para, yogi shvasana udara ke upara ki ora se eka taranga ki taraha ki kriya kalapa dikhati hai |
bahya shvasana ke samaya, usi krama mem naka se eka dhimi layatmaka vidhi mem usi prakara se karem |
pahale hama udara mem khinchate haim, ninna pasaliyom ko eka satha dabate haim aura antatah hama hansali aura kandhom ko niche karate haim |
bahya shvasana ke samaya, hama udara aura pasaliyom ko eka satha isa stara taka dabate haim ki bahuta thoda़i vayu pichhe shesha raha jae |
pranayama ke isa sabase asana rupa ka abhyasa, achchha ho ki khali peta mem subaha aura shama karem |
ise pratidina eka minata karane se shuru karem aura dhire-dhire isa samaya ko pancha minata taka badha़aem |
aba apa gahare yoga shvasana mem jane ke lie taiyara haim |
yaha shvasana sardi aura khasi ki pratirodhakata badha़ane mem sahayata karega, aura yadi apa shvasanali shotha se grasta haim to yaha sahayata karega |
jivanu ka virodha karane ke lie eka achchha svastha phephada़a utaka vikasita karega |
eka hina manogranthi vale ya eka aparadha bhavana vale lada़ke aura lada़kiyam apani chhati ko eka satha dabate hue apane kandhom ko age khinchane ki ora pravritta hote haim |
yaha eka kharaba adata hai aura yaha bedhangi mudra unake phephada़om mem kama ऑksija़na jane deti hai | 
khinche hue kandhom vale aise bachchom mem kama vriddhi aura kharaba svastha hone ki sambhavana hoti hai | 
bahuta se mamalom mem halki shvasana ki adata ko parivartita karake bahuta sara sudhara prapta kiya ja sakata hai |
gahare shvasana se adhika ऑksijana phephaड़om taka pahunchata hai| 
jaba ऑksijana rakta vahiniyom dvara pravahita hoti hai to adhika ऑksijana hamare sharira ke vibhinna angom taka pahunchegi aura unaki karya pranali ko behatara banaegi| 
hridaya svastha hoga aura apaka bachcha adhika phurtila aura srijanashila banega|
vyayama ke daurana bhi shvasana kriya baढ़ti hai aura ऑksijana apurti mem vriddhi hoti hai parantu kasarata karate samaya mansa peshiyom dvara yaha upabhukta ho jati hai| 
yoga shvasana ke daurana hama phira bhi isa atirikta urja ko ekatrita kara pranashakti ko baढ़ate hai| 
hama bina thakavata ke lambi avadhi taka karya kara sakate haim| 
manasika taura para, hama loga adhika savadhana rahenge aura svatah ko shanta aura svasthachita rakhenge | 
pranayama ko shanta sthana para karana chahiye; hava svachchha aura shuddha honi chahie| 
khula sthana shreyakara hoga; ya phira bagicha, udyana ya phira khuli khiड़kiyom vala havadara kamara| 
apako kevala andara ke kapada़e ya dhile vastra pahanane chahie|
apako jamina para kambala ya kapaड़a bichhakara baithana chahiye|
sharira, sira, aura garadana sidhi honi chahie, aura apako ankhe banda kara sthira baithana chahie | 
yadi apa baitha sakate haim to padmasana ya vajrasana mem baithiye | 
yaha baithane ki mudra, apake merudanda ko sidha rakhane ki eka svachalita rachana hai | 
yadi apa isa tarike se nahim baitha sakate haim, to eka divara ke sahare sukhasana mudra mem baithiye ya apane merudanda ko sidha rakhate hue koi eka aramadayaka mudra apanaiye | 
yadi apako jamina para baithane ki adata nahim hai,ya phira apa vriddhavastha, ghutanom ke darda ke karana baitha nahim sakate haim taba apa eka sidhi kursi para baithiye, khuda ko khinchakara kursi ki pitha ke sahare satakara rakhiye aura sidha baithiye | 
mudra aramadayaka honi chahiye taki abhyasa ke samaya mudra parivartana ki jarurata na pada़e | 
ushakala, pranayama ke abhyasa ka sabase uttama samaya hai|
isako sandhyakala mem bhi kiya ja sakata hai| 
subaha pahile nitya kriya karake apane ko svachchha kariye aura yadi ho sake to snana kara lijiye | 
ise khali peta karana chahie ya phira bhojana se chara ghante ke bada karana chahie |
apane mastishka ko shanta aura mana ko prasannachita rakhiye| 
pranayama ka abhyasa yantrika kriya nahim honi chahie | 
mastishka ko sacheta rakhana chahie, shvasana kriya ki sthiti aura pravaha ka kshana-kshana dhyana rakhana chahie | 
dhire-dhire apa apani shvasana kriya para niyantrana pa lenge, yadyapi isamem samaya lagega | 
yaha niyantrana usi prakara hai jaise, eka kushala prashikshaka dvara eka kutte ko palatu banana,ya phira eka shera ya hathi ko prashikshana dena | 
shvasana ki dhvani ko eka samana banaye rakhana chahie | 
pranayama ke daurana apa, eka shakti ki nishchita lahara ka anubhava karenge aura mana shanta hone ka bhi anubhava karenge | 
pranayama ke pashchata sharira adhika dhila ho jata hai| 
pranayama ke ananta prakara haim| 
yaham hama chahenge ki apa eka sadharana takaniki ka abhyasa karem aura usamem dakshata hasila karem | 
yadi apaki shvasana prakriya sahaja rupa se karya kara rahi hai to apa thika ujjayi pranayama arambha kara sakate haim| 
shvasa ke hara eka sansa chhoda़ne aura sansa lene ke pashchata anshika virama dijiye| 
isa virama ko dhire-dhire aura asani se baढ़ane ka prayasa karem| 
jora na lagayem aura na hi apana dama ghutane dem | 
apa apane shvasa ko lambi avadhi taka rokakara rakha sakate haim| 
jaba apa apani sansa roka lete haim, hava ko phephaड़e ke durastha kendriya bhagom mem pahunchate hue aura pure sharira mem phailate hue mahasusa karem|
yadi apa thaka hua mahasusa karem to samanya shvasana ko shuru kijiai ya phira bari-bari se samanya shvasana para ja sakate haim| 
shvasana vaha prakriya hai jisake dvara manushya aura dusare prani ऑksijana le kara usaka upayoga karate haim| 
isamem karbana daiऑksaida ko bahara nikalana bhi shamila hai, eka gaisa jo koshaom dvara ऑksijana ke upayoga karane para utpanna hoti hai| 
bahya shvasana ya shvasana prakriya mem paudhe ya prani, vatavarana se ऑksijana grahana karate haim aura karbana daiऑksaida chhoड़te haim| 
jaba ऑksijana jiva ki koshaom mem li jati hai aura karbana daiऑksaida ko bahara kiya jata hai to ise koshaom ka antarika shvasana kaha jata hai| 
manava mem shvasana ke mukhya anga phephaड़e hote haim| 
ye guha ke andara lachili sanrachana hai aura pratyeka mem lakhom chhote-chhote vayu kaksha ya thaile hote haim| 
sansa lene vali sanrachana mem dusare mukhya anga chhati bhitti aura madhyapata hai| 
chhati bhitti mem pasaliyam aura unake bicha ki peshiyam shamila hoti hai| 
madhyapata gumbada ke akara ki peshiyom ki chadara hoti hai, jo chhati guha ko udara se alaga karati hai| 
hava sharira ke andara aura bahara naka aura mukha ke dvara li aura chhoड़i jati haim| 
phairinksa (grasani) (naka aura mukha ka pichhala bhaga), lairinksa (kantha) aura traikiya (shvasanali ) ye vayu ke marga haim, jo naka va mukha ko phephaड़om se joड़te haim | 
shvasana kriya ke daurana jise sansa khichana bhi kahate hai, vatavarana se vayu phephaड़om mem khinchi jati hai| 
nihshvasana ya sansa chhoda़ne ke daurana, vayu phephaड़om se bahara nikali jati hai| 
yadi apa tina minata bhi sansa nahim lete ho, to apaki nrityu ho jayegi hai| 
dama, shvasanali shotha (nali mem sujana) jaise rogom ke karana sansa lene ka tarika badala sakata hai| 
yaha kisi sharirika kriya ya vyayama ke bada bhi badalati hai, taba yaha teja aura gahari ho jati hai| 
jaba hama thoda़i kasarata karate haim aura kuchha sau gaja dauड़te haim, taba hamare phephaड़e tarotaja aura chusta ho jate haim| 
achchhe svasthya ke liye hamare phephaड़om mem hava ka adana-pradana bharapura ho isaliye bahuta-se shvasana vyayamom ka vikasa kiya gaya hai| 
yaha prayukta vishakta karbana daiऑksaida bahara chhoड़ti hai| 
hamari bhavanaom ke satha shvasana ka atmiya sambandha hota hai| 
jaba hama krodhita, uttejita, bhayabhita, udasa hote haim, shvasana eka dusara hi rupa leta hai| 
halanki vibhinna bhavanaom ke satha isaka alaga-alaga rupa hota hai, parantu jaham taka hamara sambandha hai yaha lagabhaga achetana hota hai, isake liye hamem kuchha bhi prayasa nahim karana paड़ta hai| 
sahi shvasana kriya hamare sharirika aura manasika svasthya ko majabuta banane mem sahayaka ho sakati hai| 
yaha svaichchhika hai aura isa para hamara niyantrana hai| 
hama apane shvasana ko gahara ya uthala kara aura kuchha kshanom ke liye roka kara apani ichchhanusara isaka tarika badala sakate haim| 
ise rishiyom aura yogiyom dvara achchhi taraha samaja gaya tha, jinhonne, isako kaise niyantrita kiya jae isamem nipunata prapta ki thi |
unako bodha ho gaya ki yadi bhavanaom ke satha shvasana badalata hai to shvasana mem badalava la kara hama apani bhavanaom para bhi kabu pa sakate haim| 
yaha khoja vakai mem eka bahuta baड़a varadana tha aura hamare rishi muniyom ne sadiyom taka isa vichara para karya kiya hai| 
pranayama shvasana ke sabhipraya niyantrana para nirbhara karata hai, jo ki parinama svarupa hamari bhavanaom para bhi niyantrana karata hai| 
pranayama aura dhyana yahi vo kaड़i hai jo hamem yoga se joड़ti hai| 
jaba hama kisi nadi ya talaba mem nahane jate hai to pani ke andara jitana sambhava hota hai shvasa rokate hai aura dusarom se isaki tulana karate haim| 
jaba hama jalamagna hote haim, taba hama apani sansa ko rokate haim| 
jaba hama vriddha hone lagate haim aura karya mem tallina rahate haim aura chintaom ke lakshana dikhane lagate hai taba shayada hamare pasa isa sadharana avadhi ke liye kama samaya rahata haim| 
genda ko phekane se pahale vaha apani sansa rokega | 
vaha isaliye aisa karata hai kyonki isase usako jyada ekagrata aura takata milati hai| 
eka achchha dhavaka sau gaja ki teja dauड़ mem apani dauड़ aksara bedama hokara puri karata hai| 
eka bhala phekane vala ya eka chakati phekane vala apane sarvochya prayasa ke pahale apani sansom ko roka kara rakhata hai -jitani adhika vaha apani sansa khinchega utane hi lambe samaya taka vaha sansa ko roka sakega | 
eka shitakalina subaha jaba apa thande pani ki bauchhara ke niche jate haim, taba shuruati bauchhara mem apa apani sansa roka lete haim taki ane vale thande pani ko jelane ke liye jyada shakti ekatra kara sakem | 
jaba apa kisi sundara prakritika drishya ko pahali bara dhyana se dekhate hai, taba isaki sundarata se prabhavita hokara kuchha kshanom ke liye ashcharya se apaki sansa ruka jati hai|
jaba apa thaka jate haim, taba apa janhai lete haim aura apaki sansa lene ki kriya dhimi ho jati hai| 
kisi nipuna kalakara dvara banai gai chitrakari ko dekhane para sharira ke rongatem khaड़e ho jate munha hakka bakka raha jata hai aura kuchha kshanom ke liye sanse ruka jati haim| 
pranayama mem sansom ke isa kshanika roka ko kumbhaka kahate haim| 
yaha hamare sharira ki takata ka sadhana hai aura hamari bhavanaom ko niyantrita rakhane mem bhi sahayaka hoti hai| 
sanse sharira, jnanendriyom aura mastishka ke bicha ka eka setu hai| 
hamara uddeshya isa setu ko takatavara banana hai|
svasthya jivana ke liye hamem isako nirmita karana hai, aura isako majabuta banana hai aura sarvopari sharira ki indriyom aura mastishka ke bicha mem talamela vikasita karana | 
hamare prachina rishi muniyom ke liye yaha bahuta baड़i chunauti rahi| 
unhonne avasarom ke anukula sansom para thika niyantrana karane ka bharapura prayasa kiya jisase bhavanaom para purna niyantrana kiya ja sake |
aja pahale se bhi adhika isaka gahara asara ho raha hai kyonki hama audyaugikrita aura bhautikavadi sansara mem tanava se chhutakara pane ki koshisha kara rahe haim| 
adhunika chikitsa vijnana ko isaka bodha ho chuka hai ki dava ki goliyam aja ke vyakula aura chintakula sansara ka hala nahim hai| 
6,000 varshom ke prachina yoga vijnana mem shayada isaka sthayi hala paya ja sakata hai|
manava sharira para yoga, pranayama aura dhyana ke prabhava para unaka shodha sahi siddha ho raha hai| 
ve tanava prerita ya manodaihika rogom ki gambhira paheli para vijaya prapta karane ki apani laड़ai mem asha ki eka kirana dekhate haim| 
jisa tarike se hama shvasa lete haim aksara aparyapta hoti hai, hama alasyapurvaka aura uthale ढ़nga se sansa lete haim isiliye rakta kabhi-kabhara hi paryapta rupa se ऑksijana se bhara hua hota hai| 
aksara yaha bodha nahim hota hai ki kharaba svasthya ke bahuta sare lakshanom ki jaड़ vastava mem dhima rakta pravaha aura isake aparyapta rupa se ऑksijana se yukta hona hai| 
isiliye taba antarika anga, granthiyam aura naड़iyam paryapta rupa se poshita nahim hoti haim| 
utsarjana pranali sucharu rupa se karya nahim karati hai| 
sharira ka mala bahara nahim nikalata hai| 
hama loga hamare uthale ढ़nga ki shvasana kriya se parichita nahim hai kyonki isase hamem turanta koi asuvidhajanaka shvasahinata nahim hoti aura isaki pahale se hi adata ho chuki haim| 
aksara bahuta se loga apane karya ke samaya kursiyom para baithe-baithe tedha़i-medha़i sharirika mudra apanate hai, jaise tankakom ka apane tankanaka mashina para jukana ya phira eka klarka ka apani phailom para jukana, isase hama apane madhyapata ko sankuchita karate haim aura phephaड़om ki kshamata kama karate haim| 
hamari pasaliyam aura riढ़ ki haddiyam itani sakhta aura kaड़i hoti hai ki ye pasali pinjare mem simita jagaha ke karana phephaड़om ko achchhi taraha phailane se rokati haim| 
shvasana prakriya tanga kapaड़om, tanga kamara peti, tanga phulapainto, hamare sharira ke akara ko niyantrana mem rakhane ke liye upayoga mem ane vale andaruni kapaड़om ke dvara simita ho jati hai|
isa taraha hama apani ऑksijana avashyakata ka lagabhaga dasavam bhaga hi sansa dvara andara lete haim evam anumanatah apani phephada़om ki kshamata ka eka tihai bhaga hi prayoga mem late haim| 
aja, eka ama sabhya vyakti sansa nahim le raha hai, balki matra, ghutana ko dura kara raha hai| 
andhavishvasiyom mem eka manyata hai ki hama mem se hara koi apane ginati ki nishchita sanse lekara hi paida hua hai| 
jaba ina sansom ki ginati puri ho jati hai, to hara eka ko yaha duniya chhoda़ni pada़ti hai| 
12-13 bara sansa lene se, ginati dhimi ho jaegi aura apaki jindagi lambi ho jaegi| 
yaha tarka sangata bhi lagata hai, agara apa dhire aura gaharai se sansa lete haim, to apa ka svasthya behatara hoga| 
behatara svasthya ke satha, apa apani sabhi simaom ko nishchita rupa se para kara sakate haim| 
isa prakara se sansa lena kevala pasaliyom ko, kandhom ke upari bhaga ko evam hansali ko hi upara uthata hai| 
sansa kevala phephada़om ke upari bhaga mem hi pravesha karati hai| 
chunki yaha eka sabase chhota hissa hai, vayu thoda़i matra mem hi phephada़om mem pahunchati hai| 
2-3 ghante isi taraha kama karate rahane se , uparisansa lene ke sivaya, dusara koi chara nahim ho sakata| 
apane kandhom ko thoda़a upara uthaie aura gahari sansa lijie| 
apane kandhom ko niche hone dijie aura apane sira ko pichhe kijie, aura apa dekhengem ki apa aba aura jyada sansa le sakate haim| 
age ki ora thoda़a jukate hue, eka meja para baitha jaiye , bilkula ऎse jaise ki hama kama karate samaya baithate haim| 
kyonki kandhe aprakritika rupa se eka uchcha sthana para haim, to sansa kama gahari hi hogi| 
isamem thodi-si udariya sansa shamila hoti hai| 
vayu, upari phephada़om ke bajaya madhya phephada़om mem bhara jati hai| 
adhikatara adami isi tarike se sansa lete haim, jaba ve sidhe khada़e hote haim ya eka bhare hue kamare, ya sinema haॉla mem baithe hue hote haim| 
hamem chhoti-chhoti sansom ka, jaldi-jaldi prayasa karana hai| 
jaba apa agali bara sinema haॉla mem hom, to apane apa ko dhyana se dekhana| 
isaka sabase achchha udaharana, ghoda़e ya kutte jaise janavarom mem dekha ja sakata hai, jo shrama karate samaya sine se sansa lete haim| 
khiladiyom dvara bhi janabujakara isaka abhyasa kiya jata hai| 
inamem madhyapata ko nicha karake phephada़om ke nichale hisse mem vayu phira se bhara jati hai| 
adhikatara adami jaba ve niche lete hue hote haim ya arama kara rahe hote haim to isi taraha sansa lete haim| 
yaha tinom prakara ke sansa lene ki prakriyaom mem sarvadhika sakshama hai, lekina phira bhi aparyapta hai| 
udariya sansa lene ka shuddhatama rupa hrishta pushta svastha manushya mem, sainikom mem, khilaड़iyom mem ,kisanom mem evam parvatiya charavahom mem paya jata hai| 
madhyapata ki eka mahatvapurna bhumika hai, kyonki chhati aura udara ko alaga karane vala, mansapeshiyom ka yaha eka majabuta vibhajaka hai| 
arama ki sthiti mem yaha upara ki ora, vakshiya guhika ki tarapha, vrittakhanda banata hai| 
usane paya ki jo loga purnatah samayojita the evam jindagi ke prati jinaka shanta evam khushanuma drishtikona tha, ve jyada dina taka bane rahe apekshakrita unake jo ina gunom ko apane vasha mem nahim kara payem| 
apa tanava se to dura raha nahim sakate| 
bhojana, vyayama ya prema ki taraha, apako ise lena hi hai| 
lekina apa mem apane tanava ke anukulatama stara ko janane ki kshamata bhi honi chahie taki apa use sahana kara sake| 
ham yaha janane se apako yaha janane ke lie asani hogi ki kaba apa tanava sambandhi bimariyom ke khatare mem haim| 
apani vyaktigata bhavukata ko kama karane ke lie apako, samanjasya banae rakhane mem, dridha़ hona hi hoga| 
adhika thakane se bachiye| 
eka achchhi bharapura ninda lijie evam nahim kahane ki yogyata ko vikasita kijie aura apani simaom ko svikara kijie| 
purushom ki apeksha striyam adhika samaya taka jiti haim| 
anusandhanakartaom ne yaha paya hai ki tanavayukta evam chunautipurna sthitiyom ke pratyuttara mem ,ve purushom ki apeksha kama jivarasayanika kriyaom ko jahira karati haim evam tanava ki sthiti se jaldi hi mukta bhi ho jati haim|
eka mahila ka udaharana hai jisane apane 100vem janmadivasa taka naukari ki evam 113 varsha ki ayu mem apani nrityu taka ,kaphi hada taka apane ko bina chashmem ke padha़ne evam ti vi dekhane yogya, banaye rakha|
una mahilaom ke, jinhonne mahilaom ke adhika ama shaikshika raste ko apanaya, unaki apeksha mem injiniyaringa karane vali mahila vidyarthiyom mem purushom ke samana hi harmona se utpanna hone vale tanava ki pratikriya ko dekha gaya hai | 
nayi mili chhuta se hamem dekhana hai ki yaha kitani hanikaraka siddha hogi aura koronari hradaya bimariyom evam alsara ke prati, logom ke adhika pida़ita hone ki sambhavana ko kitana baढ़aegi |
jaba apaki mahatvakanksha bahuta unchi hoti hai aura apa janate haim ki apa apani jindagi mem, usako pura nahim kara sakate haim, to apa tanavagrasta ho jate haim| 
yaha apake lie eka ubau daura hoga aura apane lakshya pane ke apake prayasom ki dhimi gati, apako kuntha se bhara degi| 
apane uddeshyom ko vibhajita kijie, aura aise lakshya banaiye jinhem ki apa kama samaya mem pura kara sakem|
pratyeka lakshya jo apa hasila karate haim, apaki takata badha़ata hai aura apako aura bhi adhika atmavishvasi banata hai| 
eka vyakti, jisako svayam mem atmavishvasa hai, eka tanava mukta vyakti hai| 
shora pradushana aura janasankhya visphota, snayuom para bahuta adhika prabhava dalate hai| 
apane nagara se dusari jagaha kama para jane vale lakhom karoda़o logom ke lie , apane karya kshetra ane jane mem, bhida़ se bhari relom evam basom mem dhakkama dhakka karana, kunthita karane vale traiphika jama mem gola gola ghumana aura gada़iyom ke dhuam ko pina, pratidina ki ghatanaem hai| 
jine ki evam karya karane ki sthiti mem sudhara ke lie, mummbai mem prarambhika anusandhana adhyayana ke anusara yaha paya gaya ki 71 pratishata kamakaji loga, apane karya kshetra se 10 kimi ke andara hi rahate haim| 
lagabhaga 22 pratishata dasa se 25 kimi taka ka saphara taya karate, sata pratishata ko 25 kimi se bhi adhika saphara karana pada़ta|kuchha 75 pratishata logom ko kama para pahunchane ke lie adha ghante se kama ka aura 20 pratishata ko eka ghante se adhika ka samaya lagata| 
aja ki mumbai mem, pravasi majadurom ke agamana evam janasankhya ki atyadhika vriddhi ke karana , kama se kama 30 pratishata kamakaji logom ko ,rela ya basa dvara donom ora ki yatra mem, do ghante barbada karane pada़te haim| 
dina ke anta taka , ane jane vale loga thakana, chida़chida़epana, ekagrata ki kami, ninda na ana, sira darda, pachana vikara, pitha darda, shaucha kriya mem gada़bada़ aura samanya asuvidha ki bahuta adhika shikayata karate haim|
agara apa bada़e evam bhida़ vale kasbe ki pradushita vayu mem sansa lete haim, to apa tanavagrasta mahasusa karate haim| 
agara jindagi ki raphtara teja़ hai, aura agara hama usase bhi teja bhaga rahe haim, to tanava usase bhi teja bhagega| 
tanava sharira me bharata hi rahata hai| 
bhavishya mem kabhi vaha upara taka bhara sakata hai| jabaradasta tanava ki sthiti mem, chinta ke karana, hamara mastishka atirikta samaya taka kama kara sakata hai aura taba yaha sochane ki shakti ko rokane ko nishana banayega|
jaba hama sone jate haim, hama thakana mahasusa karate haim, hama unghate haim lekina mastishka mem kuchha na kuchha ghada़i ki taraha chalata hi rahata hai| 
hama so hi nahim pate haim| 
hama subaha ko thake hue hi uthate haim, mastishka thika taraha se karya karata hua pratita nahim hota| 
badale mem yaha, aura adhika tanava utpanna karata hai| 
agara sharira tanavagrasta hai, to mansapeshiyom mem bhi khinchava a jata hai|
gardana ki mansapeshiyam, kandhe ki mansapeshiyam evam upari pitha sabhi mem, khinchava ya tanava a jata hai| 
yadi ina mansapeshiyom para hama ungaliyam rakhem aura jora se dabate hue unh khojane ki koshisha karem, to hama mahasusa kara sakate haim ki ye tanim hui haim evam samvedanashila ho gayim haim| 
badale mem, yaha mansapeshiyom ko sikoda़ deta hai|
unamem kama rakta pravaha hota hai aura gardana evam mastishka mem kama rakta pahunchata hai| 
muskarane ke prati koi abhiruchi nahim rahati, chah hama kitani hi koshisha kyom na kara lem| 
jaise hi tanava ka prabhava sharira ke dusare hissom mem pahunchata hai, taba anya pareshaniyam bhi paida ho jati haim| 
pachana kriya badatara ho jati hai, amashaya rasom ka srava adhika hota hua pratita hota hai aura peta darda kara sakata hai| 
hradaya bahuta adhika dhada़kata hua pratita hota hai, nabja bahuta teja chalati hai, aura hama chida़chida़e ho jate haim| 
charom ora ke mahaula mem koi abhiruchi nahim rahati aura manahshakti mem bhi kami ati hai|
sharira mem isa nakaratmaka prakriya ke viparita badalava lane ke yogya hone ke lie, hamem tanavamukta hona sikhana hi chahie| 
hamem mansapeshiyom ko tanana evam dhila karana sikhana chahie| 
maunta sinaya skula ऑpha medisina yu esa e, ke daॉ.stivana je. slaitara kahate haim ki visheshatah vidhurom mem, jivana sathi ka abhava, badha़ti hui bimari evam nrityu se sambandhita hai| 
anusandhanakartaom ke unake dala ne paya ki shveta rakta koshikaom, jo bimari mem sharira ke bachava ke lie uttaradayi haim, unaki karyavidhi mem mahatvapurna rupa se kami thi| 
vidhurom mem, udasi evam bechaini jaisi ama bhavanaom ke karana, rakta ki rasayanika sanrachana mem haॉrmona ke badalava se, kamajora pratirodhi shakti badatara ho gayi thi| 
pratirodhaka vijnana ke eka britisha vaijnanika, daॉ.raॉbarta edara ne eka prayoga ke dvara, mastishka aura pratirodhaka tantra mem eka spashta evam sidha samparka dikhaya | 
unhonem siddha kiya ki sammohana, elarji se utpanna hone vali pratikriyaom ko daba sakata hai|
ina nishkarshom se, bimariyom ke prati daॉktarom ke drishtikona mem parivartana ana chahie evam marijom ke prati unake ravaiye mem bhi| 
yadi eka daॉktara, eka rachanatmaka sujava dete hue, marija ki pareshani ko samajane ki koshisha karata hai to yaha dava likhane se kai guna behatara hoga| 
kuchha prayogom ke anusara, eka nirashajanaka dusara daॉktari nirnaya, daॉktara-marija ke bicha ki kriyaom ko buri taraha prabhavita karata hai evam marija aura usake daॉktara ke bicha hone vali labhadayaka vartalapa mem rukavata dalata hai| 
phurasata sabake lie nahim hai, yaha unake lie bani hai jo manasika ya sharirika rupa se vyasta rahate haim| 
varsha mem kama se kama eka bara chhutti lijie| 
agara paisa nahim bhi haim, to chinta mata kijie, udhara lijie, chhuttiyom ka ananda lijie aura bada mem bhugatana kara dijie| 
pashchima ke deshom me, sanriddha ya dhanadhyata se svatantra logom ke bicha, yaha sarvadhika prachalita hai| 
kyonki unako apane ghara ke lie, apani betiyom ke vivahom ke lie aura apani vriddhavastha ke lie, eka bahuta bada़i rakama ikattha karane ki jarurata nahim hoti hai| 
jaba bachche bada़e ho jate haim to jindagi mem jaldi hi apana ghara chhoda़ dete haim| 
jaise hi unh naukari milati hai, ve eka ghara ya kara kharida sakate haim, aura kishtom mem usaka bhugatana kara sakate haim| 
phurasata ka samaya bitane ka hamara tarika, pashchima ki apeksha kaphi purana hai, hamara dharmika evam samajika parivesha isa taraha ka hai ki hama, bahuta si chhuttiyam lene ke lie taiyara rahate haim| 
phurasata ke kshanom mem uthaya gaya ananda ke bare mem, kaviyom ne kaphi kuchha likha hai| 
unhonne, varsha ke 12 mahinom ke daurana badalati hui jindagi ke bare mem, likha hai, aura sabhi rituom ke daurana prakriti mem hone vale parivartanom ka, gaharai se adhyayana kiya hai| 
gayakom ne kai ragom ko vikasita kiya hai, jo vibhinna rituom mem gaye jate haim| 
unhonne aise ragom ki bhi utpatti ki hai, jo dina mem alaga-alaga samaya para gaye jate haim| 
bharatiya rajakumarom evam uchcha kula ke vyaktiyom ne, apane mahalom ke baga-bagichom mem, nritya dekhate hue evam sangita sunate hue, ghantom bitae haim|
ve pratah kala, shahanai ke madhura sangita ko sunakara uthate the| 
kirtana aura dhvaniyukta arabi shabdom mem mulla ki ajana, sharira va atma mem nai sphurti ka sanchara karane ke lie hote the|
gramina loga, hara roज़ subaha pani bharate hue, kuem ke charom ora, gate hue ghumate the| 
apane bachchom ko sulate samaya mataem, sadiyom se loriyam gati a rahim haim| 
nahane ke liye nadi para jate samaya bhi, gramina auratem gati haim| 
vaham, baicha enda laॉri endarasana ke sukuna dene vale sangita ka, prakriti ki apani vishesha dhvaniyom ke satha sammishrana hota hai, aura yaha samuhagana, ajanmem bachchom dvara garbhashaya mem bhi suna jata hai| 
eka anusandhana sangathana dvara yaha sangita, bechaina bachchom ke lie vikasita kiya gaya hai| 
yaha dekha gaya hai ki jaba bachchom ke tepa bajae jate haim to praudha़ bhi adhikadhika shanta aura tanavamukta ho jate haim |
chikitsiya dhvani ka kshetra vishala hai aura uttejaka sambhavanaem prastuta karata hai |
vikaphorda, eseksa ke ranavela aspatala ke mi. oldasa kahate haim ki jo dhvani hama apani ardha chetana mem sunate haim usaka sambhavatah eka vishala chikitsiya prabhava hota hai | 
apa bistara para jane se pahale adhe ghante taka kuchha sukhada sangita sunem, apako achchhi ninda aegi |
prakriti mem vapasa jaie aura dekhie yaha apako kitana tanavamukta mahasusa karaegi |
jaba apa bahuta mahinom ke eka niyamita dinacharya ke jivana se vyagra ho jaem to apane sharira ki sunem | 
jaba sharira mastishka ka anupalana karata hua nahim lagata, taba apa thake hue aura chida़chida़e mahasusa karate haim |
yaha eka avakasha ka samaya hai |
dainika dinacharya ke vichara se bahara nikalem |
apane apa ko eka susta parivesha se bahara nikalem |
jaba apa eka haphte ki chhutti se vapasa ate haim apa phira se taruna ho jaenge |
sharira aura mastishka adhika chaitanya ho jaega |
antaratma adhika sajiva hogi aura kama karane ki gati bahuta samvriddha |
yugom se, jivana tatvajnana ki prakriti yaha rahi hai ki hamara astitva kevala eka bhautika yana para, paisa kamane, jivita rahane ke lie nahim diya gaya hai |
jivana jine ke lie aura sharira aura atma ki khushi dhundhane ke lie hai |
jaba maim tanavayukta hota hum, to maim eka lambi subaha tahalane jata hum |
tahalana achchha hai aura taba taka tahalem jaba taka sharira darda na ho |
mastishka darda ho rahi mansa-peshiyom ke satha vyasta ho jata hai aura yaha bahuta si chintaem bhula jata hai |
bahuta se loga kahate haim ki vyayama ke bada ve jo labha pate haim vaha unaka unnata sharirika stara hai |
ve kama vyagrata aura udasi ki shikayata karate haim |
ghara para aura vyavasaya sambandhi tanava ka samana karane ke lie adhika atma-vishvasa aura kshamata hoti hai |
eka jiva vaijnanika stara para yaha spashta kiya ja chuka hai ki kaitekolamaina ke sancharana mem eka kami aura bita endorphina mem vriddhi hai jo ki vyayama ke daurana aura usake bada hoti hai |
yaham taka ki yadi hama rakta rasayana mem parivartana ke vishaya mem nahim sochate to akriyashila logom ko jagaruka aura dirghakalika dukha utpanna karane vale vatavarana se dura rakhana aura unako apane pasanda ka vyayama karane mem sahayata karana eka achchha vichara hoga |
yaha taba hua jaba dono samuhom ko eka svasthyaparaka poshahara aura eka samanya vaja़na para rakha gaya |
isa prayoga ne siddha kiya ki vishishta pramana hai ki hradaya roga tanava ke dvara utprerita hote haim |
yadi yaha hamare pragaitihasika purvajom, bandarom ke lie satya hai, to hamare lie bhi satya hona chahie |
ethensa mem, 1981 mem eka bhukampa ke bada, janaleva hradaya daurom ki ghatanaem atdhika teja़i se badha़im |
hradaya daure ka yaha sambandha itana shaktishali hota hai ki ise naja़randaja़ nahim kiya ja sakata hai |
indiyana kaunsila phaॉra medikala risarcha aura neshanala instityuta ऑpha mentala heltha enda nyurosainsa dvara sanchalita eka sarvekshana ne spashta kiya ki nirikshita janasankhya mem se 22 pratishata manasika gada़bada़iyom se pida़ita the |
logom ka atitivra bhavanatmaka sankata sangharsha aura aparadha ke karana tha aura ghora vinasha ki prakriti ke karana nahim |
bada mem isane pichhe bache raha gae logom mem se hone ke lie unamem aparadha ki bhavana utpanna ki aura isalie ve mara gae |
bahutom ne nrityushoka, bhautika kshati aura ayogyata se samanjasya karane mem kathinai mahasusa ki |

yuniyana karbaida karakhane se lagi hui eka kaॉloni ki eka vayovriddha mahila isa ghatana ke bada bola ya suna nahim saki |

kevala manochikitsiya sahayata ke bada usaki shravanashakti aura avaja़ donom vapasa samanya ho gai |
kabhi-kabhi tanava ke antargata, logom ka sahasa uncha ho jata hai, unaki sansakti aura ekata unako unake bhaya se upara uthane mem aura vyagrata ki charama avadhi se ati saphalata purvaka ubarane mem sahayata karati hai | 
dvitiya vishva yuddha ke daurana, irvina jainisa ke anusara britena ke logom ne ati sansaktata ka bhava dikhaya |
bhaya pratikriya havai hamalom ke anta ke 15 minatom bada hi kama ho gayi thi |
logom ne bhompuom para kama dhyana diya |
hiroshima ke nagarikom ne bhi ashcharyajanaka shakti dikhai, jabaki isa dukhanta ke dvara utpanna vinasha, nrityu ghava, bimariyam abhutapurva thim |
pratikula prabhavom ka parimana tivrata mem ashcharyajanaka rupa se kama tha |
akramana ke tina mahinem ke bhitara 140,000 loga shahara mem lauta ae |
israyala ke samajashastri araॉna enthonovsaki dava karate haim ki nrityu ke avirata bhaya aura bhayanaka nrishansata ke antargata apane lambe varshom ki sharirika pida़a ke bavajuda sankendrana shivirom ke aghatom se jo jivita bacha gae unamem se 25 pratishata ne bina manochikitsiya gada़bada़iyom ke shikara hue vaisa hi kiya |
ekata, eka dusare ko naitika sahara, atyadhika samajika shraddha ne unako nayaka bana diya |
ve sabhi apane sarvajanika bhaya aura shatru se lada़ne ke lie eka satha juta sake |
abhava, samanya dukhom, ubharana aura vighatana ka bhaya ka samana karata hua eka kamajora rajanitijna jana jata hai ki shatru ke viruddha yuddha karate hue sabhi chijom para niyantrana karata hai |
kanapura mem 1959 mem eka chikitsa mahavidyalaya mem, eka dina pratahkala subaha, eka phansi ke sabuta ke lie, hama logom ko kendriya karagara le jaya gaya |
eka bhumi vivada mem tina bhaiyom ne eka pure parivara ka saphaya kara diya tha |
eka ko ajivana karavasa diya gaya aura anya do ko usa subaha phansi deni thi | 
yaha bhora tha aura hama phansi die jane vale kshetra se kuchha duri para khada़e the |
hamane dekha do aparadhi bhai pulisa ke adamiyom ke satha a rahe the |
unamem se eka sidhe chala raha tha aura nirbhika laga raha tha |
dusara chalane mem akshama tha |
usake paira kampa rahe the aura vaha bhaya se bhara hua tha |
usako pulisa ke adamiyom ka sahara diya ja raha tha |
ve phansi ke chabutare para lae gae | pahala vyakti usa sthana para khada़a ho gaya jaham use kaha gaya tha |
dusara aparadhi kshama kie jane ki bhikha manga raha tha|
hindu manate haim ki yadi nrityu ke turanta pahale apa ishvara ka nama pukarate haim, to apa svarga mem jaenge |
kala avarana unake sirom para rakha gaya aura phanda unaki gardana ke charom ora lagaya gaya tha |
kuchha kshanom bada, jallada ne hattha khincha aura donom bhai niche gaddhe ke upara apani gardanom ke dvara lataka rahe the |
unake sharirom mem jara bhi kampana nahim tha |
avashya hi, usa jallada ne hattha khinchane ke pahale unako chetaya nahim |
donom bhaiyom ne eka hi aparadha kiya tha aura eka hi saja़a mili |
lekina eka maja़buta aura anta ka samana karane ko taiyara bana raha, dusara nrityu ke samane kamaja़ora ho gaya tha |
atyadhika tanava ke samane vyakti ka svabhava mahatvapurna hota hai |
anya bahuta ja़ra se tanava ke antargata kampa jate haim |
naja़i sankendrana shivirom se jivita bache hue logom ke satha yaha nirdharana karane ke lie bahuta sare sakshatkara kie gae ki ve isa tanava se bachane aura samana karane mem kaise saphala hue the |
unamem se kuchha ne eka nirikshaka ko bataya ki ve eka uddeshya ke lie bache haih eka sambandhi ki sahayata karane ke lie, gavahi ke satha rahane lie aura unake satha jo hua duniya ko dikhane ke lie ya badala lene ki koshisha ke lie |
kuchha sahavasiyom ne apana dhyana khane ki pankti mem sidhe age chale jane jaise chhote-chhote inamom mem kendrita kiya |
manodasha bhi jivita rahane ki prakriya mem eka mahana bhumika nibhati hai aura kuchha hi haim jo atyadhika kathina paristhitiyom mem manodasha banae rakhate haim |
golibari daste ke samaksha eka vyakti se puchha gaya tha ki use eka akhiri sigareta chahie hogi kya, usane kahate hue inkara kara diya nahi dhanyavada, maim sigareta chhoda़ne ki koshisha kara raha hum| 
hasya aura ansu dono hi tanava se rahata dene vale lagate haim |
bulamaina aura kaimili ne vibhinna prakara ke svachalita vahana ghatanaom, ya phutabaॉla ke maidana mem aghata ke dvara utpanna pakshaghata se pida़ita 29 logom ka sakshatkara kiya |
kuchha dusarom se achchhi taraha se samana kara rahe the |
vyakti jo achchha kara rahe the ve the jinhonne satyata ko svikara kiya aura isake satha sakaratmaka rupa se vyavahara kiya |
dusare jaise-taise samana kara rahe the, vishishtatah ve jinhonne apane apa se aghata ka parimana janane se mana kara diya aura chamatkarika rupa se thika hone ki apeksha ki |
kuchha ne socha yaha unaka bhagya tha, asahaya mahasusa kiya aura bhavishya para niyantrana vikasita karane mem akshama the |
rogi jinhonne apani chota ke lie dayitva samaja apane svasthya labha ke lie bhi dayitva samaja aura achchhi taraha se samana kara rahe the | 
balatkara pida़ita se kuchha 26 pratishata ne yaham taka ki chara se chhah varshom bada bhi yaha nahim mahasusa kiya ki ve sharirika akramana se svastha ho gae the |
vidhavaom se sambandhita adhayyana ne yaha khulasa kiya ki unamem se 25 pratishata ne apane patiyom ki nrityu ke pashchata pahale do varshom taka paryapta rupa se vyagra mahasusa kiya |
anya adhyayana mem 40 pratishata nrityu shoka santapta ne yaham taka ki apane priya janom ko kho dene ke do se chara varshom bada bhi vyagrata mahasusa ki |
viyatanama yuddha daksha sainikom ko udasi aura vyagrata ki halke se lekara ati tivra taka eka uchcha dara thi |
nrityu aura vinasha mem, kisi vyakti ko atmatyaga, krodha, saudakari, udasi aura svikriti jaisi pratikriyaom ki shreni se gujarate hue nrityu ka samana karana avashyaka hai |

atmatyaga vichara aura bhavana ko niyantrita karata hai aura vyakti ko dhire-dhire dukhada paristhitiyom se samanjasya banane deta hai |
polio, dahana, ya meru rajju ki chota se eka kamaja़ora aura asahaya pida़ita prarambhika avastha mem tanava ka samana karane ke lie aksara atmatyaga ka prayoga karata hai |
bada mem ve punah pratishthita hona shuru karate haim aura tanava se sidhe samana karane ki ora sthanantarita ho jate hai |
kabhi-kabhi yaha vinashakari ho sakata hai, jaba mahilaem apane stanom mem guthali ke mahatva ko asvikara karati haim ya purusha hradaya daura hote hue darda ke mahatva ko asvikara karate haim |
jaba hama tanava mukta hona chahate haim, to hama eka prashanta, shanta aura ekanta sthana, eka uninda vatavarana jaham koi bhida़ na ho, mem jane ko prathamikata dete haim |
shora-gula, uchcha jivana, sada़kom para bhida़ aura pradushana tanava samvriddha karata hai |
yahi hai vaha jo svisa mem sasa namaka eka alaga-thalaga gamva ke satha hua, jo eka uninda, shanta kshetra tha |

1970 taka, 250,000 se adhika shitakalina paryatakom aura 500,000 grishmakalina kabhi ekanta rahe isa kshetra mem andara a rahe the |
aba yaham eka 10-mahine lamba paryataka mausama hota hai |
sthaniya logom ne haphte mem satom dina, pratyeka dina kai ghantom kama kiya |
jaise hi artha-vyavastha vikasita hui, isa sudura gamva mem sthana lete hue bada़e badalava shuru ho gae |
utkarsha ke pichhale 20 varshom mem, parivara ke bandhana kamaja़ora ho gae |
badha़te hue bachchom mem mata-pita ki antarbhavita kama aura kama ho gai |
paryatakom ki kuchha hanikaraka adate angikrita kara li gayim |
madya aura aushadhiom ka upayoga badha़ gaya hai |
vriddha aura yuva pidha़i ke bicha antarala badha़ raha hai |
yaha druta samajika badalava, chirakalika thakana, chida़chida़apana, anidra, vrana aura uchcha raktachapa ke mamalom ki ora agrasara karate hue, vayaskom ke bicha asara dikhaya hai |
madatyaya yuvaom aura mahilaom ke bicha badha़ raha hai |
talaka aura manasika bimariyam dainika niyamavali haim |
pashchima ke susampanna desha tanava se sabase adhika pida़ita haim |
pashchimi jo yoga sikha rahe the, nirapavada rupa se kahate haim ki yoga ne unhem mana ki shanti di hai |
vishva ke eka sarvadhika shaktishali desha ka sarvadhika shaktishali manava jisaki ungaliyam nabhikiya batana ko niyantrita karati haim apane upa-sahayaka se adhika saphala hota hai |
vaha eka adhika saphala manava hai |
saphalata dirghayu ko samvriddha karati hai, apako jine ke lie kuchha deti hai |
madhya vyavasthapaka adhika tanavayukta hote haim, ve vo vyakti haim jo kathina mehanata karate haim lekina phira bhi karyakarini samiti taka nahim pahuchate |
eka suprasiddha yu. ke. manovaijnanika, viliyama sarjienta, ne mana ki vibhinna tanavayukta sthitiyom ka samana karane ki kshamata pratyeka vyakti ke lie unake svabhava ke anusara bhinna thi |
pratyeka vyakti ko arama aura ninda ki avashyakata hoti hai lekina avashyaka matra pratyeka vyakti ke lie bhinna hoti hai |
adhikatara vayaskom ko ratri mem 7 se 8 1/2 ghante ki avashyakata hoti hai|
pratyeka vyakti ko aksara ninda ane mem samasya hoti hai |
anidra sharirika ya samvedanatmaka gada़bada़iyam darshati hai |
arama aura tanavamukti utane hi mahatvapurna haim jitani ki ninda |
thakanapurna kama ke bada, eka vyakti ko arama ke samaya ki avashyakata ho sakati hai aura anya avadhi mem use kevala thoda़i tanavamukti ya eka sthanantarana ki avashyakata ho sakati hai |
anandadayi aura tanavamukta karane vali kriyaem sharira ko tanava kama karane aura svastha bane rahane mem sahayata karati haim | 
yadi arama aura tanavamukti thakana aura tanava ko nahim ghatate haim, to vyakti ko koi sharirika ya samvedanatmaka samasya ho sakati hai |

joradara vyayama mansa-peshiyom ko sashakta banata hai aura parisancharana tatha shvasana tantra ke karya ko utkrishta karata hai|
tandurusti prapta karane ke lie, eka vyakti ko koi vyayama karyakrama dhire-dhire shuru karana chahie aura kramika rupa se ise usa stara taka nirmita karana chahie jo eka svastha hridaya aura sashakta mansa-peshiyam banae rakhata hai |
dainika vyayama sabase utkrishta labha pradana karate haim |
svasthya sharirika, manasika aura samajika sukha ki eka avastha hai | 
isamem kevala bimari ki anupasthiti se bhi adhika shamila hai |
eka vyakti jo achchhi sharirika sthiti mem hota hai usake pasa sakriya jivana ka ananda lene ke lie aura dainika jivana ke tanavom ka mukabala karane ke lie shakti aura urja hoti hai | 
upayukta poshana, vyayama, arama aura ninda, sapha-saphai aura chikitsiya aura dantya dekha-rekha svasthya vijnana ke avashyaka tatva haim |
tanava ka upachara karana manasika aura sharirika donom bimari se bachane ke lie avashyaka hai |
tanava se koi nahim bacha sakata hai lekina eka vyakti tanava ke karana bimara hone ke khatare ko kama karane mem sahayata ke lie kuchha chija़em kara sakata hai |
niyamita vyayama aura paryapta ninda tanava ke prati sharira ki pratirodhaka kshamata ko shaktishali banata hai |
pratyeka vyakti ko vishrama karake, tahala kara, dhyana lagakara, kisi shauka mem lipta hokara ya kisi anya vidhi jo saphala pai gai ho, ke dvara tanavamukta hona sikhana chahie |
tanava jivana ka masala hai |
bina tanava ke, jivana nirasa aura ubau hoga |
jaba apa tanavapurna hote haim taba apaki gati badha़ jati hai, apaka pratikriyatmaka samaya kama ho jata hai, apa eka samayabaddha parishuddhata sahita apana karya samapta karane mem sakshama hote haim |
apa ka vyavasaya eka chunautipurna kama hoga |
yaha vahi samaya hai jaba apa lakshyom ko nirdharita karate haim |
una taka samaya para pahunchane ke lie, kathina sangharsha kijie |
prapta kie gae pratyeka lakshya ke satha apa shaktishali hote jate haim tatha apa aura unche lakshya taka pahunchane mem sakshama hote haim |
eka konchane vali patni eka achchhi patni hai kyonki vaha apako tanavapurna rakhati hai aura apako mahatvapurna karyom ko samaya para samapta karane ke lie vivasha karati hai | 
tanava vidyarthiyom ke lie apane apako apane padanguliyom para khada़a rakhane ke lie achchha hai |
ve apana grihakarya aura adhyayana uchita dhanga se karane ke lie kathina sangharsha karate haim aura achchhe anka prapta karate haim |
jaba eka eka konchane vali patni chhoti-chhoti chija़om ke lie, pure samaya konchana shuru karati hai aura apako apane hi ghara mem shanti se rahane nahim deti hai, ya eka vidyarthi unche lakshya taka pahunchane ya vishesha yogyata anka prapta karane lie apane abhibhavakom dvara ukasaya jata hai, yahi vaha samaya hai jaba apako tanava hota hai | 
aisa usa vidyarthi ke mamale mem aura adhika hai jo bahuta tivra buddhi vala nahim hai aura vastava mem adhyayana mem adhika prapta karane mem sakshama nahim hai |
yadi apa uchcha lakshya nirdharita karate haim, jo apa ki kshamata ke andara nahim haim ya kuchha aura jisako pura karane mem pura jivana laga sakata hai aura apa abhi bhi ashvasta nahim haim ki apa unhem pura karane mem sakshama honge, to apaka tanava badha़ta hai |
tanava badha़ta rahata hai |
yaha apake bainka khate ki taraha hai, yadi apa bhagyashali haim, to apa mudra jama karana jari rakhate haim |
dhire-dhire, nirantara aura kramika rupa se yaha eka bada़e khate mem badha़ jaega, taba apa eka kara, eka skutara, eka phrija ya tivi kharidane ya eka ghara banane layaka ho jaenge |
yadi apa apani samasyaom ka hala dhundha़ne mem samartha nahim haim, to tanava kramika rupa se aura nirantara usi prakara badha़na jari rahata hai |
yaha apake khate mem dhire-dhire aura kramika rupa se jama hota jata hai |
apa apane badha़te hue bainka khate ke prati jagaruka haim, lekina sharira mem badha़te hue apake tanava ke bare sacheta nahim bhi ho sakate haim |
vastava mem apake tanava khate mem kitana hai aura apake sharirika bainka mem kitana jama ho chuka hai yaha janane ke lie koi parishuddha mapaka paimana nahim hai |
thoda़a tanava masala hai, lekina atyadhika tanava apake astitva ke lie visha ho jata hai |
eka chhoti avadhi ke lie tanava ki kuchha matra sharira ko nukasana nahim pahunchati hai, tanava ke daurana sharira mem jo parivartana ate haim, vapasa samanya ho jate haim jaba yaha samapta ho jata hai |
anya shabdom mem mahatvakankshi loga, jinake pasa apani khuda ki simaom ka koi sapeksha mahatva nahim hota, manate haim ki ve ye saba kara sakate haim |
yadi ve apane abhishta lakshya ko prapta karane mem asaphala hote haim, to ve dusare sarvotkrishta lakshya ko nirdharita karane ke lie mana karate haim |
hama ise uttejaka tanava kahate haim |
yaha sharira mem vidhvansa utpanna kara sakata hai |
tanava adhunika jivana-shaili ka eka hissa hai |
yaha audyogikarana aura bhautikavada ka eka utpada hai |
yaha pashchimi deshom mem eka bada़e paimane mem pahale hi a chuka hai |
jaise-jaise jivana ki gati badha़ti hai, yaha usi ke barabara gati se tanava nirmita karati hai |
jaise-jaise astitva ke lie sangharsha badha़ta hai, usi taraha tanava bhi|
jaba apa naukari pana, ya koi vyavasaya sthapita karana mushkila pate haim, jaba eka ghara, ya apane bachche ke lie eka vidyalaya mem pravesha pane mem kathinai hoti hai, to tanavapurna hone ka koi karana hai |
apa apane apa ko jivana ki adharabhuta avashyakataem ekatra karane mem thaka dete haim |
yaha durdasha pure samaya chalati rahati hai yaham taka ki apake parivarika sadasyom ke satha eka bhavanatmaka samayojana ke samaya bhi |
yaha apake sambandhiyom, mitrom aura apake pada़osiyom ke satha pratispardhaom mem bhi ata hai |
jonsa parivara se achchhi sthiti mem rakhane ki koshisha mem, jo apa nahim hai vaha apa dikhava karane ki ora pravritta hote haim aura sarahana ke lie tarasate haim tatha apane apako jo apa vastava mem haim usase adhika siddha karate haim |
ye sabhi asvasthyakara tanava paida karane mem sakshama haim |
chalie hama 30,000 varshom pahale ke apane purvajom ke samaya mem chalate haim |
unhonem bhojana aura ashraya ke lie sangharsha kiya aura jivita rahe |
yadi eka shikari vriksharahita vana mem pravesha karate hue, apane shikara ko khoja rahe eka khunkhara aura bhukhe bagha ke thika samane a jata hai, samana hota hai, unaki ankhem milati haim |
sharira tanavapurna ho jata hai, vaha vyakti apane hridaya ka tvarita spandana mahasusa karata hai, rakta chapa badha़ jata hai, munha sukha jata hai, usake mathe para pasina a jata hai | 
mushkila se sansa lete hue tatha teji se sochate hue, usa vyakti ko koi nirnaya lena hi hota hai| 
khada़e raho aura usa janavara ka samana karo aura nrityu taka lada़o, ya dusari ora muda़o aura jangala ke andara bhago |
kramika vikasa ne hamem kisi tanavapurna sthiti ki pratikriya karane ke lie sharirika rupa se samartha bana diya hai |
jaise hi sharira tanava sanketa prapta karata hai, sambandhita tantrikaem piyusha granthi ko udipta karane ke lie apana kama shuru kara deti haim |
yaha mukhya kunji granthi hai jo badale mem edrinalina namaka haॉrmona sravita karane ke lie dusari granthiyom ko udipta karati hai |
yaha vaha padartha hai jo sabhi sharirika pranaliyom ko uttejita karata hai |
eka gatimana hridaya aura tivra shvasana ke satha, rakta chapa aura spandana daba ko badha़ate hue adhika ऑksija़na rakta mem bheji jati hai |
asthira peta se vaha vyakti lada़ne ya dauda़ne ke lie taiyara hai |
yadi tanava ghata jata hai, to ye pratikriyaem bhi samapta ho jati haim |
sharira tanava se mukta ho jata hai, aura eka samatulyata ki avastha mem vapasa a jata hai isa prakara eka vyayami apani pratiyogita ke bada eka samanya sthiti mem vapasa a jata hai | 
usake sharira ki pratikriya kama ho jati hai aura sharira ko koi nukasana nahim hota hai |
vahi chija eka vidyarthi ke satha hoti hai jise kisi pariksha se gujarana hai |
yaha isake bada hota hai ki sharira pashcha-tanava samanya sharirika sthiti mem vapasa nahim ata hai |
yaha vaha samaya hai jo khatare se bhara hua hota hai |
aba jaba hama lada़ne ya bhaga jane ke lie badhya nahim haim apani meja़ para baithe hue hamem spandana ho sakata hai | 
eka nirantara bhaya svayam ko eka aisi stithi mem parivartita kara leta hai jo bhige hatha, tivra nada़i-spanda, uchcha rakta chapa, peta ka ati stravana, aura eka tantrikatantra jo gada़bada़ ho jata hai, ko janma deta hai|
tanava tale nishpadana sudharane ke bajaya, hama usa bindu para pahunchate haim jaham aura adhika dabava vastava mem prati utpadaka hota hai aura sharirika vidhvansa mem parinita ho sakata hai |
taba hama ashanta, tanavapurna, chida़chida़a ya udasa mahasusa kara sakate haim |
hama bhukha ki kami, anidra, atyadhika thakana aura yauna shakti ki hani se pida़ita ho sakate haim |
1970 ki avadhi mem tanava aura bimari ke bicha sambandha, visheshatah tanava aura kainsara, hridaya roga, madhumeha, peptika vrana, tapedika, chhajana aura asthama se sambandhita, mahatvapurna anusandhana hua tha |
tanava se utpanna siradarda lagabhaga 1.5 karoda़ amarikiyom ko prabhavita karata hai, jabaki uchcha raktachapa 2 karoda़ ko prabhavita karata hai |
eka mahila atisara se pida़ita thi, lekina na to dava aura na hi poshahara mem parivartana use sahayata kara saka |
antatah, vaha manovishleshaka dvara rogamukta hui thi|
itani gudha़, itani apratyaksha, itani bhinna eka samasya tanava ke karana ho sakati hai |
adhunika chikitsiya duniya manati hai ki mastishka ya antarnihita tanava sabhi bimariyom mem eka prabala aura sabhi bimariyom ke 60 pratishata mem mukhya bhumika nibha sakata hai |
sabhi chunautiyom ka samana karate hue, sabhi durbhagyom ke bare mem sunate hue, isa duniya mem jine ke lie aura prachalita lalacha aura irshya se ubarane ke lie apako tanavamukta rahane aura eka svastha mastishka rakhane ki avashyakata hai |
lalacha aura irshya sabhi manasika aura sharirika bimariyom ke mula karana haim | 
yaha sunishchita kijie ki ve apake mastishka mem pravesha na karem |
chalisa varsha pahale loga haija, chhoti chechaka aura mahamari se mara rahe the |
aura adhika loga dila ke daure, rakta chapa, kainsara aura madhumeha se mara rahe haim |
yaha vo mulya hai jo hama tathakathita pragati, audyogikarana aura adhika susampanna hone ke lie ada kara rahe haim |
yu. esa. e. mem etheroskleraॉsisa risarcha sentara ne yaha janane ke lie eka prayoga karane ki koshisha ki ki kaise tanava dila ke daurom ko pravritta kara sakata hai |
unhonne janavarom ke do samuha lie aura unhem kama vasa aura kama kolestraॉla vala poshahara khilaya aura unake sharirika vaja़na aura rakta chapa ke pathyanka ke lie unaki barabari ki | 
eka samuha ke lie unhonne unako pinjare mem rakhakara unake samanya taura tarikom ko badala kara jina bahuta tanavayukta bana diya |
prayoga ke anta mem, 21 mahinom bada, yaha paya gaya ki tanavayukta samuha apani dhamaniyom mem vasadara jamavom ke gambhira lakshanom se gujara raha tha | 
yaha bahuta pahale ki bata nahim hai jaba manasika vyatha se bhara eka rogi mere pasa aya |
vaha varshom se thika prakara se so nahim pa raha tha |
vaha chida़chida़a tha, usake peta mem atyadhika gada़bada़i, aura gale aura chhati mem jalana thi | 
nyayalaya mem kesa lada़e ja rahe the aura naja़rom mem koi anta nahim tha |
sauharda jane aura ghrina badha़ne ke satha, ina sabaka prabhava manasika se sharirika stara taka utarana shuru ho gaya | 
maine use salaha di ghara jao, vapasa le lo ya haro, lekina sabhi lambita nyayalaya sambandhi vishaya samapta karo phira mere pasa ao |
mere acharaja ke lie usane vaisa hi kiya jaisi salaha di thi |
kuchha hi haphtom mem vaha sharirika aura manasika dono hi rupa se ananda se hrashta-pushta ho raha tha |
vaha eka naya adami ho gaya tha|
usamem jivana ke lie utsaha tha |
uttejana ko bahara nikalane dene ya tanava ko mukta karane ki kshamata samaya-samaya para sahayata karati dikhati hai | 
hama apani bhavanaem vyakta kara sakate haim aura kai vibhinna tarikom se apane apa ko halka kara sakate haim, apani bhavanaem unase bantie jinhem apa janate aura vishvasa karate haim | 
samaya-samaya para apane para hi hansane ki kshamata hamem tanavamukta karati hai |
chikhana aura chillana bhi sahayata karega yadyapi yaha samajika rupa se svikarya nahim bhi ho sakata hai |
nabhikiya sarvanasha ka dara amarikiyom aura yuropiyom ko tanavayukta banae hue hai |
isa duniya ko sau bara samapta karane ke lie paryapta nabhikiya astra bhandarom ka ekatrikarana logom ki vyagrata aura unaki chinta ko badha़a raha hai |
yaha anumana lagaya gaya hai, ki yuropa mem eka nabhikiya yuddha, kuchha 15 karoda़ logom ka tatkala saphaya kara dega |
jivita bache logom mem se adhe logom ko sthayi rupa se ghava ho jaenge |
eka bhautikashastri, mi. josepha haॉtablaॉta ne kaimbrija mem duniyabhara se 30 deshom ke chikitsakom aura chikitsa vaijnanikom ki sangoshthi mem bataya, ki eka nabhikiya sarvanasha vatavarana ko itana sandushita kara dega ki yaham taka ki dina ke samaya bhi purna rupa se andhera ho jaega | 
jivita bache bhi bhukha se mara jaenge kyonki kheti bhi asambhava siddha ho sakegi |
apaki patni, apake bachchom, apake sambandhiyom, apake mitrom ka pyara aura samvedana mahatvapurna hai, kyonki ve apako purnatah se bhara sakate aura apako tanava se mukta sakate haim |
eka adhyayana mem yaha paya gaya hai ki ve mahilaem jinhonne apane premi ya pati ke satha ghanishtha sambandha bante sambhavatah 90 pratishata kama dukhi thim banisbata una mahilaom ke jinake pasa sahare ke lie aise sambandha nahim the |
apani naukariyom se bahara nikale gae karmachari sambhavatah kama dukhi ya bimara hone vale haim yadi unaka sahayaka vaivahika sambandha tha |
pyara aura mitrata para adharita eka vastavika sambandha avyakta bhavanaom ko sugama banane ke lie eka mahana abhivyakti ho sakata hai |

lagabhaga nishchita taura para, yaha isa karana se hai ki achchhi gunavatta vale dudha ke atirikta, pyara jo eka mam apane bachche ko eka snehi alingana mem usako apane sine ke pasa rakhakara deti hai, ashcharyajanaka hai jo bhinna karata hai |
jaba mam ki goda mem liya jata hai, to vaha vishesha sthiti jo ki bachche ki ridha़ leti hai ise sadharana tarike se badha़ne mem sahayata karati hai |
sva-sujava tanava kama karane mem sahayata kara sakata hai |

yadi apa kisi vyakti ko balapurvaka paramarsha dete haim aura usaka atmavishvasa banate haim to yaha usako usaki samasyaom se bahara nikalane mem sahayata karega |
eka dridha़ mastishka kamaja़ora mastishka ko abhibhuta kara sakata hai aura ise tanavamukta karane mem sahayata karega aura sanrachanatmaka sahayata de sakata hai |
manovaijnanika chikitsa aura sammohanavada isi siddhanta para adharita haim | 
phakirom dvara di gai tabija़, sadhuom dvara die gae kale dhage, tantrikom aura jyotishiyom ke dhatvika chhalle, mane jate haim ki sahayata denge | 
paramarsha ke dvara pahanane vala surakshita aura adhika atmavishvasi mahasusa karata hai ki vaha durbhagyom ko abhibhuta kara dega |
ekabara, eka vyakti ne gandhi se puchha tanava se kaise chhutakara paya jae | 
gandhi ne use rama nama (bhagavana ka nama) japane ki salaha di |
bhagavana mem vishvasa logom ko unake dukha se bahara ane mem sahayata kara sakata hai | 
yadi apa bhagavana ka nama astha aura vishvasa se japenge to yaha apako sahayata karega |
yadi apa dridha़ta se mahasusa karengem ki vaha apaki raksha karega, to apa unaki suraksha ka hantha mahasusa karenge |
vishvasa eka mukhya bhumika nibhata hai |
hama eka pratiphala ke lie kama karate haim |
hama dusarom ki sahayata apani avashyakataom mem sahayata pane ki asha ke satha karate haim |
jaba yaha nahim hota hai to hama hatasha hote haim |
gita ka sandesha una logom ki bahuta sahayata karega jo arthahina pratiyogitaom aura hatasha mem lipta haim |
samasyaom aura vyagrata ke lie chikitsiya pulinda ‘nishkama-karma’ (bina pratipala ki ichchha kie kama karana) para adharita hai |
isane logom ko eka karyasiddhi ki ora abhimukha duniya mem prabhavapurvaka kama karane mem sahayata ki jaham pratiphala hamesha nishchita nahim ho sakega |
bharata ke yogiyom dvara nipuna sankendrana auऱ manana ki pranali una shikayatom ka achchha uttara ho sakati hai jinamem chikitsa vijnana sahayata ke lie sakshama nahim raha hai |

yaha sharira ki tanavamukti ki avastha ko eka parde mem prastuta auऱ anuvadita karati hai |
jaise hama tanavamukta hote haim dhvani tarangem halki hoti jati haim yaha nirdishta karate hue ki hama sahi disha mem age badha़ rahe haim |
yaha hamem tanavamukti ke tarikom se parichita hone mem sahayata karata hai |
bada mem jaba hamane takanika para niyantrana kara liya hai, hama mashina ko alaga kara sakate haim aura tanavamukti abhyasa jari rakha sakate haim |
aupaniveshika bharata mem britisha chikitsakom ne charom ora vivarana bheja ki yoga guru dila ki dhada़kana rokana aura jalate hue angarom para bina kisi pratyaksha ghava ke chalana jaise ashcharyajanaka karataba kara sakenge |
isaka karana tha ki yogiyom ne apane sharirom para ativishishta niyantrana prapta kara liya hai visheshatah svayatta tantrika tantra para jo bina chetana niyantrana ke kama karati hai |
dila ki dhada़kana eka anumana ke dvara 5 dhada़kana prati minata kama hoti hai |
isake pramana bhi ekatra kie ja chuke haim ki manana dvara rakta chapa niyantrita kiya ja sakata hai |
yadyapi alpa-spashta hai, isaka dila para eka labhadayaka, pratyaksha kriyatmaka prabhava hai |
manana unake lie sabase adhika labhadayaka hai jo hradaya dhamani ki bimari ke lie adhomukha haim |
hama mastishka aura sharira ko tanavamukta karane ke lie jaba hama chahem takanika sikha sakate haim | 
tanava ke kshana, hamem apane apa ko eka shanta kamare mem lagabhaga adhe ghante ke lie banda kara lena chahie, mastishka aura sharira ko ghatanaom para niyantrana karane dena chahie |
apa isase bahuta tanavamukta hokara bahara aenge |
manana hamare topakhane mem eka prabhavi astra hai |
jaba kabhi hama bahuta tanava mem hote haim hamem isaka prayoga karana chahie |
manana kai tarikom se sahayata kara sakata hai, yaham taka ki aparadha ki ghatana kama karane mem bhi |
yaha uda़isa ke puri jile ke eka aparadha-abhibhuta gamva mem prayoga kiya gaya tha |
mi. prasanna patsani, ema ela e, dvara isa dave ko satyapita karane ke lie, bhuvaneshvara se patrakarom ka eka samuha uda़isa ke isa sabase bada़e gamva achanaka pahuncha, jo tarangita pahada़iyom se ghira, chilka jila ke upara hai, |
eka dadha़i vala, kangresa (ai) neta mem parivartita kesariya poshaka vala naksala, patsani, presa parti ko divara se ghire gamva ke mandira ke 2 ekada़ ke hate mem le gaya, jaham kakshaem hoti thim |
unako eka kaiseta mem tepa kie hue vedikastotra ki sangati ke lie asanom aura usake bada pranayama aura dhyana karane ko kaha gaya | 
mi. patsani ne svikara kiya ki unake nashta hue kama ke samaya ke lie unhem ru. 50 prati maha kshatipurana ke taura para bhugatana kiya gaya kyonki 90 pratishata gamva vale chilka jila mem machhaliyam pakada़ kara nirvaha karate haim |
lekina yahi vaha karana nahim hai jo ve utsahi haim |
unhonne khule mem abhyasa ka sharirika aura manasika labha paya hai |
shavasana (shava ki mudra) tanavamukti ke lie eka shaktishali asana hai jisaka divangata hradayaroga visheshajna daॉ ke. ke. date dvara vyapaka rupa se samarthana kiya gaya tha |
unhonem dridha़ta se vishvasa kiya ki tanavamukti ne hradaya rogom ko dura karane aura uchcha rakta chapa ko kama karane mem sahayata karane mem eka mahatvapurna bhumika nibhai | 
sabase sadharana chija़ jo karani hai vaha hai kida़e-makoda़om aura durgandha rahita sapha hava ke satha eka shanta sthana chunana |
eka lakada़i ke takhta ya ja़mina para eka kambala ya chadara phailaem |
84 jute utarem, belta dhili karem, chashmem utarem ya kevala halke kapada़e pahanem |
jaba apa kisi kathora sataha para bina takiya ke letate haim, to apaki ridha़ sidhi hoti hai, lagabhaga puri ja़mina ko chhuti hui |
yaha sharira ki sabase shithila avastha hai |
yadi apako mahasusa hota hai ki apaki ridha़ ja़mina ko nahim chhu rahi hai, to donom ghutanom ko moda़em, dekhe ki yaha ja़mina ko chhuti hai, aura apane paira alaga phailaem |
apake bagala mem pada़i hui, apani dono banhem kankha se 15-20 digri ka kona banate hue bahara ki ora phailaem, paira halke se alaga aura unhem bahara ki ora samanarupa se girane dem, ankhem komalata se banda |
jaba pura sharira shithila hai, apa mahasusa karenge yaha saba gaya, bhari aura ja़mina mem duba raha hai |
apa mahasusa karenge jaise ki apa anshika taura para ashakta ho gae haim aura yaham taka ki jaba apa koshisha karenge, apa apane paira ya banhem uthane mem sakshama nahim honge |
isa sharirika avastha mem jaba taka raha sakate haim rahem |
apani sansom para adhika dhyana dem; ise nirvighna aura gaharai se chalane dem |
isa sthiti mem 10–15 minata taka rahem |
apa kuchha sukhada vicharom ki kalpana kara sakate haim ya apa apane mastishka ko rikta ya khali mahasusa kara sakate haim, halki nidra ki avastha ka ananda lijie | 
yaha eka gahana arama aura shanti ki avastha hai jo hamare lie achchha svasthya lati hai |
jaba apa uthane ke lie taiyara hom, apane apa se kahem ki apa purnatah tanavamukta haim |
apa uthane ke lie taiyara haim |
apana dayam ghutana moda़em phira bayam ghutana, apani dayim banha moda़em phira bayim banha, apane sharira ko phailaem, jaisi yaha thi, banhom ko sira ke upara phailaem, pairom ko phailaem aura apana pura sharira |
kuchha kshanom ke lie baitha jaem aura apa apane agale nityakarya ke lie taiyara haim | 
yadi apako sone mem pareshani hai, to ninnalikhita vidhi aja़maem |
sabase mahatvapurna yaha hai ki apaka dimaga shanta hona chahie chahe apa ऑphisa mem ho, havai jahaja़ mem, basa mem ya rela gada़i mem , to apa arama se raha sakate haim |
chahe baithe hom ya adhalete hom apane apa ko aramadayaka sthiti mem mahasusa karem |
apani thoda़i uthaem aura phira ise niche karem |
kuchha gahari sansem lijie , phira dekhiye ki apaka mana shanta aura sthira hai|
kalpana kijie ki apa eka chattana para ya samudra tata para baithe haim ya bahuta sare phulom vale eka eka bagiche mem , kisi bhi jagaha ya sthiti ki kalpana kijie jaham apako 
apane mana ki ankhom se phulom ke alaga alaga ranga dekhiye , unaki tajagi ko mahasusa kijie, unase bahakara ane vali sugandha, thandi hava ko apane sharira para mahasusa kijie |
aise eka sthana ki kalpana kijie jaham apa atita mem ja chuke haim aura jo ki apake sharirika aura manasika vishrama ke lie eka adarsha sthana ka pratinidhitva karata hai |
bachche bhi tanavagrasta ho sakate haim aura yaha unake svasthya ko prabhavita karata hai |
isilie kai bachche peptika alsara aura dila ki bimariyom se pida़ita haim|
tokyo se prakashita eka riporta ke anusara skula mem achchha karane ke dabavom ke karana prathamika aura madhyamika skula ke bachchom ke bicha peta ke alsara mem teji se vriddhi 
yada rahe, bhavishya mem hone vale dila ke daure ka bija bachapana se hi bo diya jata hai |
yadi apa apane bachche ko tanava mukta nahim pate haim , tanava ke karana achchhi taraha se so nahim pa raha hai, apa use vishrama ke lie eka bahuta hi sarala vyayama ki sipharisha kara sakate haim.
udaharana ke lie use bistara mem arama se letane dijie |
yadi avashyaka ho aura avaja़ kama karani ho laita banda karake daravaja banda kara dijie |
use paira ke talavom mem chheda hone ki kalpana karane ke lie kahiye | 
use ina chhidrom ke madhyama se sansa andara va bahara lene ki kalpana karane dijie |
use kalpana karane do ki taji thandi hava usaki atma se pravesha kara usake pairom ,janghom, kamara se hoti hui chhati taka pahuncha rahi hai |
thika usi taraha jaba vaha shvasa bahara chhoda़ raha ho to use kalpana karane dijie ki hava kamara , jangha , paira aura pairom mem hue chhidrom se guja़ra kara niche ja rahi hai | 
usako 5-10 minata yaha abhyasa karane dijie |
vaha vyayama karane ke satha puri taraha tanava mukta mahasusa karega aura jyada achchhi taraha se so sakega |
vishrama vashpa snana , sona batha , garma pani snana aura vibhinna thande pani ke anuprayogom jinaka ja़ikra hama pichhale adhyaya jala upachara mem vistara se kara chuke haim, se bhi hasila kiya ja sakata hai|
malisha bhi bahuta aramaprada ho sakati hai|
koi bhi upachara jo mansapeshiyom aura sharira ko arama pahunchata hai vahi dimaga ko bhi prakritika rupa se arama pahunchata hai 
agara hama apani mansapeshiyom ko tanavamukta karate haim to yaha silasila hamare dimaga ke tanava , hamari mansapeshiyom ke tanava ko bhi dura kara sakata hai |
jaba hamari mansapeshiyam tanavamukta hoti haim to hamara dimaga bhi tanavamukta hota hai |
malisha jaba prabala hathom dvara jora lagakara ki jati hai ,jyada labhadayaka hogi , kya yaha sacha nahim hai |
tanavamukta hone ke lie, malisha narama hathom se ki jani chahie aura yaha aramadeha aura sukhada honi chahie |
agara koi madada upalabdha nahim ho to yaha apane hathom se ki ja sakati hai , yaha apane jivana sathi ya eka dosta ya koi rishtedara dvara bhi karavai ja sakati hai|
yaha dabava nahim hai balki sharira ki janakari hai jo ki apako adhika tanavamukta rahane mem madada karata hai|
yaha paira se sira taka yathakrama kiya jana chahie |
yaha eka nishchita tala ke satha aura hamesha bahara se hridaya ki ora kiya jana chahie |
eka daॉktara ne hala hi mem isi siddhanta para prayoga karate hue tanava, siradarda aura peptika alsara ke rogiyom ki madada karane ke lie unase apane hathom ko garma rakhane para dhyana dene ko kaha hai|
isa tarike ko sikhane mem samaya lagata hai|
aisa paya jata hai ki rakta hatheli ki ora bahata hai, aura isake parinamasvarupa sira aura peta jarurata se jyada khuna se chhutakara pa jate haim|
ve ama taura para ina uttejaka vastuom dhula paraga aura dhuem ke prati samvedanashila haim, lekina unaki pareshani ka mula tantrika tanava hai |
dama rogi ka eka dhvanirodhi kamare mem ilaja kiya jata hai aura eka bayophidabaika mashina ke tara se joda़ diya jata hai |
agara vaha sukuna se hai, vaha kevala halki klika hi sunata hai |
use dhula, phula ya jo kuchha bhi asthama ke daurom ke sambhavatah karana ho sakate haim unako drishtigochara banane ke lie kaha gaya |
ama taura para vaha ina vishayom ke bare mem sochakara tanava ko bahuta badha़a leta hai|
rogi ko maluma hai ki vaha daure ki charama sima para hai|
jaba use pata chalata hai ki tanava usaka dushmana hai to use apane abhasom ko sunate hue usa para niyantrana karana sikha sakata hai ya usako kama karane ki koshisha kara sakata hai| 
bayophidabaika sva tanava mukti ki takanika mem nipunata prapta karane mem madada karata hai|
bada mem rogi mashina ke satha puri taraha banta sakata hai|
taba vaha arama se rahana sikha jayega aura apane prayatnom se purna aura pahale se adhika khushahala jindagi ji sakata hai|
mahilaem jo purushom logom ke satha kama karati haim ya rahi haim unaka masika dharma ka chakra sankalita rahata hai |
eka vastukara dosta isa siddhanta para jora deta hai ki ghara ka dija़aina aura yaham taka ki ranga bhi dimaga para prabhava dalata hai| 
ghara isa prakara dija़aina kiye ja sakate haim ki usamem rahane vale manasika shanti mahasusa kara sakem|
aisi parikalpana hai ki bhutahe ghara aura kuchha nahim balki kha़raba dija़aina kiye hue ghara hote haim jo ki mastishka para bhayanaka aura kharaba prabhava dalate haim|
eka adhyayana ke anusara, gaganachumbi vali imaratom ke phlaitom mem bada़e hue bachche adhika manasika samasyaom se grasita hote haim|
ja़mina para khelana aura dauda़na , ja़mina ke adhika naja़dika rahana yaha svastha manovaijnanika rupa se prabhava dalata hai|
agara apa ki samartha mem ho to bangale mem rahane mem rahiye, para eka jopada़i mem rahana bhi utana aramadayaka hoga | 
apane bachchom ko barabara matri-bhumi ke samparka mem rahane dijie |
sanyukta parivara mem rahane vale loga adhika bhavanatmaka rupa se sthira lagate haim|vyaktigata
jo bachche apane dada - dadi ki goda mem badha़te haim unhem paraspara sneha aura samaya milata hai|
isa lie pratyeka vyakti ko bhavanatmaka sankata ke kshanom mem aura adhika madada milati hai|
britisha pichhale yuddha ke karana mata-pita ke pyara aura parivarika ja़indagi ke mamale mem buri taraha prabhavita hua hai|
isa chunava ke parinamasvarupa svatantra samaja para eka bahuta bada़a pratighata hua jisa karana shadi ko purana samaja jane laga |
svachchhanda prema aura dragsa lena chalana mem the|
britena, aja, khuli chhuta ya ekala abhibhavaka parivarom aura jaldi talaka ki duniya ke khilapha hai |
bachchom ko bada़a karane mem mata-pita donom ke hone para vishesha jora dete hue paramparika parivara ke paksha mem bahuta jyada vota pada़e |
yaha sarvekshana mem 15 se adhika unra vale ayu varga ke 1500 vayaskom ke bicha ayojita kiya gaya |
bharata mem, hamari sthiti behatara hai| 
sanyukta parivara abhi bhi hamari parampara mem haim jisa karana svatantrata kuchha kama rahati hai|
pata nahim hama kaba taka surakshita honge jaba ki videshi taura-tarike, riti - rivaja aura jivana yapana ki shaili hamari tarapha teja़i se bahakara a rahi hai |
dhyana vidhiyam alaga alaga hoti haim lekina mulatah sabhi eka hi haim kyonki inaka antima uddeshya antarika jagarukata ko vikasita karana aura usaka phayada uthane mem madada karana hai |
eka taraha se , dhyana ke madhyama se apa apane sharira aura bahara ki duniya ke bare mem aura adhika jagaruka hote haim |
apa behatara sochane mem sakshama hote haim chahe apa kisi bhi prakara ka kama karate haim |
agara apaka sharira svastha ho , to apa aura adhika sakshama honge |
dhyana shanta, aramadayaka vatavarana mem kiya jana chahie |
apane dhyana ko mantrom ya sansa para kendrita karem |
uttamottama dhyana mantra , shabda ya dhvani ki madada se ya apane prashikshaka ke adesha dvara kiya jata hai |
yaha eka asana takanika hai jise jaldi sikha ja sakata hai, isase mastishka shanta aura tanava mukta rahata hai |
vipasana dhyana kisi bhi mantra ki madada nahim leta hai |
yaha sansa ki madada leta hai| 
yaha sabase purana rupa mana jata hai aura bhagavana buddha dvara nirvana pane ke lie istemala kiya gaya tha | 
sharirika aura manasika tanava se mukti dene ke alava yaha rogom ke ilaja mem bahuta upayogi hai |
yaha upavasa ya daitinga ki taraha sharira ko sapha karata hai |.
unhonne apani pustaka yora haildi harta mem hridaya rogiyom ke lie dhyana karane ki sipharisha ki |
vaha age kahate haim ki jaisa ninda mem hota hai vaise hi dhyana ke daurana kardiyaka autaputa 25 pratishata ghata jata hai |
adhikansha loga jo santulita ahara va niyamita vyayama karate haim ve bhi svadishta bhojana va mishthana dekhakara lalacha jate haim |
hara kisi ko kabhi kabhi manoranjana va bhojana ki jarurata hoti hai| 
aisa ahara hamem chunana chahie ki jisaka palana sampurna jivana kiya ja sake |
yadi paushtika bhojana, vyayama aura chatapate khane ke bicha santulana rakha jae to apaka jivana sahi mayane mem sukhadayi hoga |
ye dave bekhabara logom ke lie sabase bada़a khatara haim| 
ve daitinga aura vyayama ko bekara ki chijem manate haim |
sharira ke pahanave para sharira ke vasa para koi prabhava nahim pada़ta hai|
ve ise bhanga nahim karenge |
halanki sanyukta rajya amerika ke khadya evam aushadhi prashasana ka kahana hai ki bahuta adhika pasina asthayi rupa se vajana ghatata hai |
ephadie age kahate haim ki teji se aura atyadhika sapheda pani jana isa pahanave ke karana hota hai jo ki khataranaka hai |
svasthya ki kimata para teja़i se paisa kamana khataranaka vyapara hai |
ilaja mem bhukha damanakari(aimphetamina) ka shayada hi koi mulya hai|
jitane samaya taka inaka upayoga hota hai utane hi samaya taka ye prabhavakari rahate hai |
inake dushprabhava bhi hote haim|
motapa eka ajivana samasya hai to ye davaem shayada hi asaradara hoti haim |
thayaraida harmona ka upayoga sharira mem ऑksijana ki khapata mem vriddhi ke lie kiya jata hai.
vaja़na ke ghatane ka karana bina charbi ke mansa ka ghatana hai na ki vasa utaka |
kaॉsmetika sarjari jaise liposakshana , jisamem vasa ko vastutah khincha liya jata hai ya uchchheda kiya jata hai atah shalya chikitsa ke daurana isamem khatara ho sakata hai |
isaka prabhava sthayi nahim hota hai |
samaya ke satha shesha koshikaem vibhajita ho jati haim aura phira se vasa utaka bana jate haim |
hamem ajivana upachara lena chahie, vyayama karane ki adata dalani chahie , vasayukta aura chiniyukta padarthom se jaham taka ho sake dura rahana chahie |
5, 7 ya 10 dinom ke lie sabhi phalom ke alpahara se shuru kijie| 
apani kshamatanusara ise kijie aura dekhie ki apa kaisa mahasusa karate haim| 
eka samaya mem kariba 250 grama ka eka phala lem, aura dina bhara 3-4 bara lem| 
5-6 dina ke lie apa ,purna upavasa se shuruata kara sakate haim aura phira apa phalom ka alpahara shuru kara sakate haim| 
jaba apa svasthya prakshetra mem haim, to upavasa karana hi adhika upayukta hoga| 
jaba apa kabjiyata mahasusa karate haim, to eka sade halke garma pani ka enima lijie| 
eka ya do dina ke lie, dopahara evam ratri ke bhojana mem ubali hari sabjiyam lem, taba kuchha salada ki matra aura badha़a dijie, aura dopahara evam ratri ke bhojana mem gehum ke ate se bani eka roti, usake bada apani bhukhanusara, do ya tina aura badha़a dijie| 
nashte mem phalom ke satha adha litara dudha lijie, dopahara evam ratri ke bhojana mem kuchha dahi ya mattha, aura dopahara mem kuchha phala lijie| 
phala khane mem adhika dhila deni chahie, jaba kabhi khane ki ichchha ho, to apani pasanda ka eka phala lijie, lekina kele se bachana chahie| 
yadi dudha aura dahi khana apake sharirika gathana se mela nahim khata, aura agara apa thoda़a badalava bhi chahate haim, to nashte mem ankurita munga dala lijie| 
dudha evam dalem, protina ke eka achchhe srota haim| 
agara apako pasanda hai to thoda़i dala lijie| 
atyadhika namaka se paraheja rakhie| 
namaka, sharira mem pani ko roka kara rakhata hai| 
sabjiyom mem atirikta namaka ka istemala na kare aura dahi evam salada bina namaka ke hi lem| 
achara evam papada़ mata lem, kyonki unamem atyadhika namaka hota hai| 
tela, makkhana aura ghi ka kama se kama istemala kijie| 
sabjiyom ko pakane mem, adha chhota chammacha tela hi kaphi hona chahie| 
tale hue bhojya padartho ko na khaem| 
sukhi senki hui braida khaiye ya usa para bahuta thoda़a makkhana lagaiye| 
gehum se bani braida ko variyata dijie| 
sabhi maida utpadanom, telom evam shakkara se paraheja kijie, kyonki ye sabhi adhika urja dene vale srota haim| 
adha kilo makkhana mem 3615 kailorija aura adha kilo chini mem 1820 kailorija hoti haim| 
sabhi mithaiyom, chaॉkaletom, kekom, pestriyom evam aisakrimom se paraheja hi kijie| 
15 dina mem eka bara, apa apani pasanda ki eka mithai kha sakate haim| 
kama kailori vale evam adhika reshe yukta bhojana khaiye| 
adha kilo khire mem kevala 70 kailorija hoti haim| 
adha kilo tamatara mem 105 kailorija, aura adha kilo seba mem 320 kailorija hoti haim| 
apa, thoda़i matra mem mansahari bhojana kha sakate haim| 
ubale va senke hue bhojana, ya nyunatama tela mem pake bhojana ko variyata dijie| 
vasayukta goshta ya vasayukta machhali se paraheja kijie| 
mansahari bhojana ke satha ubali sabjiyam evam salada achchhi matra mem lijie| 
mansahari bhojana reshe vihina hota hai isalie savadhana rahie| 
ubali sabjiyam, salada evam phala jaise reshe yukta ahara, adhika khaiye| 
madira mem bahuta sari kailorija hoti haim, isilie usase paraheja hi kijie| 
vatita peya padarthom se dura hi rahie| 
chaya ya kaॉphi ki adhikata se paraheja hi karana chahie| 
dhunrapana na kijie ya tambaku mata chabaie , yaha apaki vyayama karane ki aura ghumane ki shakti ko kama kara dega| 
sthula logom ko, hamesha bhukha lagati rahati hai| 
dopahara aura ratri ke bhojana se purva unhem, anliya khadya padartha jaise santare, nimbu ya angura khane chahie| 
yaha, bhojana ke prati unaki lalaka ko, kama karega| 
dopahara ya ratri ka bhojana, bhari mata kijie| 
thoda़i matra mem bara-bara khaiye, yaha bhojana ke prati apaki lalaka para niyantrana rakhega evam charbi ke jamane mem bhi rokathama karega| 
phalom ke rasa, malai rahita dudha, mattha, jau ka pani pijie, ina mem bahuta kama kailorija hoti haim| 
chavala ya alu se paraheja karane ki avashyakata nahim haim| 
agara apa chahem to, inako pratidina kha sakate haim lekina matra ubale hue hi| 
sabjiyom ko ovana mem bhunana, behatara hota hai| 
chhilakom sahita sabjiyam adhika svadishta hoti haim| 
bhunane ke bada chhilaka, utara ja sakata hai| 
unako masala dijie evam svadishta banane ke lie thoda़a namaka milaiye| 
bhunane ke lie apa aluom, bainganom, tamatarom, paravalom, bhindiyom ya phira tindom ko variyata de sakate haim| 
khira, pattagobhi, tamatara, gajara, muli, pyaja, shalagama, palaka, shimala mircha ka prayoga karate hue, salada kai tarikom se banayi ja sakati hai| 
eka bharapura salada yukta bhojana ko, apa variyata de sakate haim, jaise ki ajakala restraom mem, saikada़om tarikom se yaha parosa jata hai| 
ghumana sarvottama vyayama hai| 
pratidina pratah jaba taka apa thaka nahim jate, ghumie| 
eka ghante mem apa lagabhaga 6 kimi ki duri taya kara sakate haim| 
agara apa javana evam hrishta-pushta haim, to dhimi gati mem dauda़ie| 
tairaki uttama vyayama hai| 
baidamintana, skvaॉsha ya tenisa jaise joshile khela kheliye| 
khelana, ghumane ya vyayama karane se jyada sarala hai| 
jaba apa vajana ghatate hai aura, teji se ghatate hai, to tvacha dhili pada़ jati hai| 
yaha apaki tvacha mem eka bara phira kasava lane mem sahayaka hoga| 
yaha phurtila bhi hoga| 
garma pani ki apeksha thande pani se snana karana, vajana ghatane mem tivratara sahayaka hota hai| 
saptaha mem do bara, sampurna sharira para lepa laga sakate haim| 
mitti ka lepa ajamaya ja sakata hai|
mitti ko sharira para ragada़ie aura dhupa mem baitha jaiye| 
jaba vaha purnatah sukha jaem, taba thande pani se snana kijie| 
saptaha mem eka ya do bara vashpakaksha mem baitha kara snana ya vashpa snana kijie| 
vaha bharapura pasina lane mem sahayaka hota hai| 
apana vajana teji se ghatane ki koshisha mata kijie| 
apako kamajori,chakkara ana mahasusa ho sakata hai ya phira chhati mem bhi darda badha़ sakata hai| 
prathama do saptahom mem, apaka vajana kaphi teji se ghatata pratita ho sakata hai, sambhavatah saptaha mem 2-3 kilo| 
bada mem vaha ghata kara, saptaha mem 1-2 kilo ke lagabhaga kama ho jayega| 
agara apa tivrata se vajana ghatana chahate hai, to svasthya prakshetra eka surakshita sthana hoga| 
chunki apa ka sharirika bhara adhika hai, isilie apako adhika ऑksijana chahie| 
pranayama kijie, yaha apake tanava ko kama karega, apake phephada़om mem vayu bharega evam apake atyavashyaka angom ki kriya pranali ko behatara banane mem sahayaka hoga| 
apani avashyakataom ke anusara, tanava se mukti dilane vali sarvottama takanika ko ajamaiye| 
dhyana lagaiye, yaha tanava ko kama karata hai aura bhojana ke prati lalaka ko bhi kama karata hai| 
ye yakrita mem bahutayata matra mem sravita hote haim| 
ye sthula logom mem uchcha raktachapa ke lie uttaradayi haim| 
jo loga mote hote haim, samanyatah adhika bhojana khate haim aura usake satha adhika namaka bhi| 
namaka sharira ke andara pani jama karake rakhata hai, jisase raktachapa badha़ta hai| 
ऎse loga samanyatah 12 grama taka namaka kha lete haim| 
behatara hoga hama pratidina 3-5 grama namaka khaye| 
adhikatara sthula logom ko madhumeha bhi ho jata hai| 
yu esa mem sabhi vayaska sthula logom mem se 80% logom ko madhumeha hai| 
agara apa kaphi samaya se sthula ya mote haim, to apako madhumeha hone ki sambhavana adhika hai| 
sharkara ke lie unaki grahya kshamata aksara kama hi hoti hai| 
sharkara ki eka thoda़i matra bhi, unake raktachapa ki vriddhi mem sahayaka hoti hai| 
ve insulina ke prati prabhava shunya ho jate haim| 
saubhagyavasha, jaba apa vajana ghatate haim, prakriya prativarti (dutarapha) hoti hai | 
sthula logom ka peta mota hota hai evam unaki merudanda para lagatara eka khinchava bana rahata hai| 
yaha unaki pitha ke nichale bhaga mem darda ka evam merudanda mem joda़om ke prarambhika darda ka bhi karana banata hai| 
atyadhika sharirika bhara, ghutanom para davaba dalata hai kyonki unhem apake vajana ka boja bhi sahana karana pada़ta hai| 
mote logom mem, ghutanom ke joda़om ka darda ama bimari hai| 
una logom ke rakta mem yurika esida ka stara samanyatah uchcha hi rahata hai| 
garbhavastha ke daurana, uchcha raktachapa evam madhumeha, jatilataem utpanna karate haim| 
agara shishu ka akara bada़a hai to prasava pida़a dera taka sahana karani pada़egi| 
ऎse bahuta se mamalom mem, sija़eriyana ऑpareshana karana hi padata hai| 
vriddhom mem garbhashaya ka tyumara evam kainsara ama bata hai |
mote logom ko apane peta ke karana jyada pareshani hoti hai |
phulana, udara-vayu , apacha evam dayaफ़raigmaitika harniya samanya roga haim |
unaka yakrita kama nipunata se karya karata hai |
tvacha ki sikuड़na mem nami ke karana ina logom mem tvacha ka sankramana jyada hota hai jo samanyatah kamara, bagalom evam stana ke nichale bhaga mem hota hai |
ina logom mem shalya sambandhi khatarom ki sambhavana adhika hoti hai |
asamvedanata (enesthisiya) jyada khataranaka hota hai, shalya-chikitsa mem jyada samaya lagata hai evam ja़khma bharane mem bhi jyada samaya lagata hai |
upachara adhikanshatah bhojana ki matra kama karane, vyayama baढ़ane ya donom para nirbhara karata hai |
jivana bhara ke mutape ke lie vyayama evam santulita ahara donom sammilita rupa se hone chahie |
vastava mem yaha mahatvapurna nahim hai, ki apa kya khate haim |
eka kailori, kailori hi hoti hai chahem vaha kisi bhi srota se ayi ho |
hamem kevala isa bata ka dhyana rakhana chahie ki ahara santulita ho jisase ki antatah hama bimara na pada़em|
adhika karbohaidreta vala ahara motape ko dhire-dhire ghatata hai kyonki adhika karbohaidreta vale ahara ke satha adhika namaka khane ki pravriti hoti hai |
adhika namaka, sharira mem atirikta jala banae rakhane mem madada karata hai |
adhika protina vala bhojana jala ko shighrata se bahara nikalane mem madada karata hai |
parantu bhavishya mem isase ketosisa evam anlaraktata ho jati hai |
kama antarala para bhojana karane se mota na hone mem madada milati hai |
agara apa lambe antarala para bhojana karate haim to apa ja़yada matra mem bhojana karate haim, jisase bhojana aura adhika asantulita ho jata hai |
una logom mem motapa samanya bata hai jo lambe antarala para bhojana karate haim
teji se vaज़na ghatane ke lie upavasa kiya jata hai |
eka haफ़te ya dasa dinom se kama samaya ke upavasa ko protsahana diya jana chahie |
mote logom ke dvara lambe samaya ke upavasa karane se raktakshinata , poteshiyama ki kami , ketosisa , yakrita ka thika se kama na karana evam haipotenshana ho sakata hai |
vivekapurna upavasa apake svasthya mem sudhara layega parantu yaha upayukta nirdeshana mem kiya jana chahie |
vaज़na ghatane ke lie tina pramukha tarike hote haim : upavasa , santulita ahara evam vyayama |
kisi eka para ashrita hone ke bajaya inamem se kinhim do ko milakara apanana achchha rahata hai |
agara upavasa aura vyayama eka satha kiye jae to yaha vaज़na mem teji se giravata la sakate haim |
ise eka achchhe svasthya visheshajna ki dekha rekha mem kiya jana chahie |
shuruata mem vajana mem ghatava teji se hota hai |
aisa antom ke tatvom ke kshaya evam namaka ki niyantrita matra ke karana se hone vali pani ki kami ki vajaha se hota hai |
shuru mem vaज़na ka teji se ghatana motape ke marijom mem bhrama paida kara deta hai |
jaba vaha apane bhojana ki samanya khuraka punah lena shuru karate haim to yaha ghata hua vaja़na teji se baढ़ jaega |
vastavika hrasa taba hota hai jaba charbi ghata jati hai |
aisa lagabhaga taba dekha gaya hai, jaba vasa kailoriyam samanya bhojana dvara pradana ki gayi kailoriyom ke bajaya sharira ke dvara hi apurti ki jati haim |
rasayanika prakriya ki drishti se, shuddha vasa mem prati paunda 4, 000 kailori indhana ki kshamata hoti hai |
sharira ki vasa ki indhana kshamata 3,000 kailori prati paunda hoti hai 
kyonki isamem 25 % nami hoti hai| 
atah eka vyakti jise akara evam gatividhi ke anusara pratidina 3,000 kailori chahie vaha eka dina mem eka paunda kama kara lega| 
eka vyakti jisako 2,000 kailori ki jarurata hai, vaha eka dina mem 2/3 paunda kama kara lega| 
vastavika vaज़na mem kami upara diye gaye ankada़om se thoड़i jyada hoti hai kyonki bhale hi dhire-dhire sahi parantu kuchha bina vasa vale tantuom ka bhi kshaya hota hai| 
jaba apa puri taraha se upavasa karate haim taba mansapeshiyom, rakta, yakrita evam anya mahatvapurna angom ka vaja़na bhi kama hota hai| 
halke vyayama evam upavasa karane vala, samanya adami evam stri eka dina mem eka paॉnda vaja़na kama kara sakate haim| 
paraheja vale kariba 15 se upara santulita ahara haim, jo amariki vasiyom ke bicha lokapriya haim| 
eka upayukta ahara mem, sharira ke avashyaka vitamina evam khanija padarthom ko dhyana mem rakha jana chahie| 
ise protina se 15 % , vasa se 30 % evam karbohaidreta se 55 % kailori nikalane mem samartha hona chahie|
amerika mem achchhi bikri vali kuchha pustake jaise :- epha-plaॉna, etakinsa, pritikinsa evam skarsadela mem motape ke lie kuchha ahara batae gaye haim| 
yaha nishchaya kara pana mushkila hai ki kauna-sa ahara sabase adhika achchha hai jo sharira ke lie avashyaka sabhi pushtikaraka tatvom ki apurti kara sake|
yaha vyakti vishesha para bhi nirbhara karega| 
epha-plaॉna ke antargata kama vasa, jatila karbohaidreta mem uchcha (sabuta anaja va phaliyam) evam vitamina ki bharapura apurti shamila hai| 
eka angreja chikitsaka daॉ. denisa burkita vasa ke sevana ko simita evam protina ki matra ko kama karane, aura adhika reshe vale phalom, sabjiyom evam sabuta anaja ke sevana ko baढ़ava dete haim| 
bevarali hila mula rupa se pahale 9 dinom ke lie eka kama protina vala ahara hai pura bhojana phalom ke rupa mem liya jata hai| 
bada mem, dhire-dhire karbohaidreta, vasa evam protina bhi shamila kiya jata hai| 
pritikinsa ahara jyada reshedara hote hai jisase malotsarga mem asani ho| 
isamem itani kama chikanai hoti hai ki ho sakata hai ki isaka svada achchha na lage evam nigalane mem kathinai ho| 
vaज़na kama karane vale sabhi aharom mem adhika reshe vala bhojana sabase adhika mahatvapurna hai| 
apa ise jyada chabate haim aura khane mem jyada samaya lete haim| 
bhojana bhari hota hai aura apako jyada tripta karata hai, evam yaha chini va tela mem bane parishkrita bhojana ke lie thoड़a hi sthana chhoड़ta hai| 
apa aneka prakara ke nashte evam chikanai yukta chijom ke lie kama lalachate haim aura unake lie bhukha bhi kama rahati hai| 
eka prayoga se yaha sabita ho gaya hai ki jaba apa jaldi-jaldi khate haim to apako bhukha bhi jaldi lagati hai, jabaki bhojana ki vahi matra dhimi gati se khane para apako bhukha dera se lagati hai|
ja़yada reshe vale bhojana mem ज़yada pani hota hai, isalie eka litara ja़yada pani preshaka kala ya sharira ke dvara lie gae bhojana ke samaya ko baढ़ane mem madada karega tatha isa prakara kabjiyata mem arama pahunchaega| 
ja़yada reshe vala bhojana khane se apa mala mem ज़yada kailoriyom ka tyaga karate haim| 
jancha se pata chalata hai ki jaba loga jyada reshe vale bhojana para hote haim taba mala mem kailori tatva ki matra lagabhaga dasa pratishata hoti hai| 
atah apane bhojana ko nigalane se pahale jitane apake danta haim utani bara chabane ki salaha ko manana uchita hai| 
isalie achchhe svasthya evam ikahare badana ke lie apa apane bhojana ko itani achchhi taraha se chabaem ki apaka thosa bhojana pine yogya ho joye tatha tarala bhojana ko thosa ki taraha khaya ja sake|
eka bahuta prasiddha ahara visheshajna laichainsa tarala protina ahara ke viruddha savadhani baratane ko kahate haim| 
yaha hridaya sambandhi aniyamitataom ka karana bana sakati hai| 
unaka dava hai ki amarikiyom ka svasthya sudhara jaega agara ve kevala kama vasa evam adhika jatila karbohaidreta ka sevana karem| 
vasa kana brihadantra (kolona) kainsara se sambandhita hote haim| 
una logom ke bhojana mem bahuta achchha sudhara hoga jo loga 50 % shakahari bhojana evam 50 % mansahari bhojana lete haim| 
laichainsa kahate haim ki agara apa vyayama karane ke lie tatpara haim to apa kisi bhi prakara ka bhojana kha kara bhi vaज़na ghata sakate haim| 
eka stri ne emफ़etamaina (eka aisi dava jo bhukha ko kama karati hai) nirdharita aushadhi lene ke lie eka daॉktara ko dhundha| 
daॉktara ne apana nirdesha paida nikala aura usa para maltivitamina ki goliyam likha kara usa stri ko pakaड़a diya|
daॉktara ne usa stri se kaha ki agara vaha roja 2 mila teji se chala kare to use apane vaja़na se koi pareshani nahim hogi| 
yaha nuskha kama kiya aura usa stri ka vaja़na kaphi kama ho gaya| 
use eka dina ka vyayama bhi nahim chhida़na chahie| 
vyayama mansapeshiyom ko majabuta banata hai, bhukha kama karata hai evam jivana ke prati manovaijnanika drishtikona mem sudhara lata hai| 
svasthya ke prati satarka logom ke lie subaha ka nashta bahuta avashyaka hai| 
apake lie achchha hai ki apa achchhe, mukhya rupa se anaja ka nashta lem, bajaya isake rata ya dopahara ke bhari bhojana ko bhakosem | 
eka bahuta purani manyata bilkula sahi hai, ki bhojana evam sharirika vyayama ke bicha ka asantulana hi motape ke baढ़ne ka mukhya karana hota hai| 
apa jitana kama khayenge aura jitana jyada vyayama karenge utane hi kama mote honge| 
motapa, anekom chikitsa sambandhi samasyaom sahita kai bhavanatmaka evam samajika pareshaniyom ko prerita karata hai| 
anya logom ke dvara mote logom ke satha sahanubhutihina vyavahara kiya jata hai vishesha rupa se vidyalaya mem sahapathiyom dvara|
agara apa apani jarurata se ज़yada kailori ka sevana karate haim to vaha sharira mem charbi ke rupa mem parivartita ho jaega| 
agara apa jarurata se 3,000 kailori ja़yada khate haim to apaka vaज़na eka paunda baढ़ jaega| 
jina logom ka vaja़na, samanya se ज़yada, thika ya kama hota hai ve sabhi shayada eka hi taraha ka bhojana khate haim| 
unake vaja़na mem antara isa bata para nirbhara karata hai ki ve kisa matra mem bhojana karate haim aura usaki tulana mem kitani urja kharcha karate haim| 
sharirika nishkriyata sabhi ayu ke vargom mem motapa la sakati hai vishesha rupa se bachchom evam navayuvakom mem jo sabase adhika kriyashila varga hota hai| 
adhikanshatah mote yuva loga samanya vaja़na vale logom se jyada nahim khate haim| 
balki unamem se adhikatara loga kama khate haim| 
parantu ve itane nishkriya hote haim ki kama bhukha hone ke bavajuda bhi ve apani avashyaka se jyada khate haim aura atyadhika charbi jama kara lete haim| 
kisi bhi vaज़na kama karane vale ahara mem kailori ki matra kama honi chahie| 
agara eka vyakti ko apane jivana ki adatom ke satha apane vaज़na ko banae rakhane ke lie 3,000 kailori ki avashyakata hoti hai to use eka haफ़te mem 2 paunda kama karane ke lie eka dina mem 2,000 kailori khani chahie| 
samanyatah teji se vaज़na kama karana khataranaka hota hai|
vaja़na ghatane vale ahara mem khane ki vastuem susantulita honi chahie| 
arthata - unhem vo sabhi poshaka tatva achchhi matra mem pradana karane chahie jo achchhe svasthya ke lie avashyaka haim| 
isa bata ka koi sabuta nahim hai ki utkrishta ahara udaharana ke taura para kama karbohaidreta ya kama protina vale bhojana,ya vo jo kevala eka hi prakara ke ahara para adharita hom unaka labha susantulita ahara khane se adhika hota ho| 
vaja़na ko kama karane vala bhojana, svadishta tatha kharidane va pakane mem asana hona chahie| 
eka ahara visheshajna ne salaha di hai ki jo vyakti vaज़na kama karane vala ahara le raha hai use ज़yada vyayama karana chahie| 
parantu eka mota vyakti ko jo vaise to svastha hai, use achanaka jyada lambe evam bhari vyayama ki yojana shuru nahim karani chahie|
hridaya para jora dena khataranaka sabita hoga| 
vyayama ke karyakrama ko dhire-dhire baढ़ana chahie| eka achchha tarika hai, pratidina tahalana shuru karana aura tahalane ka samaya baढ़ana|
jaise-jaise vyakti adhika patala aura chusta hota jata hai aura adhika asamanya mehanata vale vyayama sammilita kie ja sakate haim |
jaॉrja varnarda shaॉ ne kaha, khadya prema se adhika vastavika koi prema nahim hai |
sthulata susampanna samaja mem adhika samanya hai |
charlsa dikensa 'pikavika peparsa' mem eka patra, eka mote vyakti, ‘jo’, ka vyakhyana karate haim, jo adhikatara anuchita avasarom para sarabora ho jata hai |
mote loga apane alasya ka ananda lete haim |
ve parishrama ke bada hamphata hua mahasusa karate haim tatha bahuta jaldi thaka jate haim |
ho sakata mote aura patale samana matra mem bhojana khate hom |
patale loga adhika sakriya hote haim aura isalie ve apani kailori ka upayoga kara lete haim|
mote nahim karate haim, unaki atirikta kailori sharira mem sabhi galata sthanom para jama ho jati haim |
sthula loga samanyatah shanta prakriti ke hote haim aura svastha dikhate haim, lekina ve bahuta si bimariyom ke shikara ho jate haim |
jaba koi vyakti atyadhika vaja़na ka hota hai, to usake jivana vritta ka tisa pratishata ghata jata hai|
20% atyadhika vaja़na ko ausata vaja़na ki tulana mem sthula manana behatara hai |
kya koi vyakti mota hai yaha janane ka sabase sadharana tarika hai eka kailipara ki sahayata se usaki tvacha ki parata ki motai napana |
samanyatah, yaha banhom ke pichhe kiya ja sakata hai, yadi yaha purushom mem 23 mimi aura striyom mem 30 mimi se adhika mota hai to vyakti sthula hai |
loga jaise-jaise vriddha hote jate haim ve sthula hone ki ora pravritta hote haim |
ve 50 ki unra ke asa-pasa charama para pahunchate haim
hama kuchha hi sthula logom ko vriddhavastha taka jivita pate haim |
ve samanyatah bahuta si sthulata-sambandhi bimariyom ke shikara ho jate haim |
do prakara ke loga hote haim jo sthula hote haim |
eka prakara ke loga apani puri vriddhi ke daurana sthula hote haim |
sthula striyom ka pratyeka garbhavastha ke satha vaja़na badha़ jata hai |
ve buri taraha se moti hoti haim aura 175% atyadhika vaja़na ki ho sakati haim |
unaka motapa unaki banhom aura pairom ke asa-pasa adhika hota hai |
anya prakara ve haim jo madhya unra mem mote hote haim|
jaise-jaise hama badha़te haim, jivana sthanabaddha, kama aura kama sakriya hota jata hai|
kailori ki avashyakata ninna hoti jati hai |
yaha anumana lagaya gaya hai ki prati dashaka, purushom mem 43 kailori aura striyom mem 27 kailori antargrahana kama ho jata hai |
unra ke satha khane ki adatem nahim badalati haim|
jaba kailori ki avashyakata kama ho jati hai, hama bhojana ki vahi matra lena jari rakhate haim|
atirikta kailori sharira mem vasa jama karana shuru kara deti haim|
yaha samanyatah kendriya sthulata utpanna karati hai, vasa udara aura chhati ke asa-pasa jama hoti hai| 
vasa koshikaem sankhya aura akara mem badha़ti haim aura yaha prakriya puri balyavastha mem jari rahati hai|
asthula logom mem vasa ki koshikaom ka prachura matra mem utpadana balyavastha ke prathama do varshom aura phira vayahsandhi ke thika purva mem hota hai |
yaha dekha gaya hai ki asthula loga, yadi unhem balapurvaka bhojana karaya jata hai, to unaka vaja़na 20-30% badha़ jata hai |
unaki vasa koshikaem akara mem badha़ jati haim lekina sankhya mem nahim aura jaise hi balapurvaka bhojana karana rukata hai, ve apane vastavika vaja़na para a jate haim |
sthula logom mem yadi vasa koshikaem eka ऑpareshana ke dvara nikala di jaem, jaisa ki yaha kabhi-kabhi kuchha utsahi shalyachikitsakom dvara kiya jata hai, to antatah bachi hui vasa koshikaem akara mem badha़ti haim aura prasarita hoti haim aura vaha vyakti eka bara phira atyadhika vaja़na ka ho jata hai |
jaba apa udasa hote haim, to adhika khane ki ora pravritta hote haim |
jaba apa akelapana mahasusa karate haim aura ghara mem asa-pasa pada़e dhera sare khane ke satha karane ko kuchha nahim hota hai, to apa khuda ko hi khilane ki ora pravritta hote haim | 
ausata vidyalaya jane vali lada़kiyam samanyatah patali hoti haim|
jaba ve vidyalaya chhoda़ti haim to ho sakata hai ve vivahita ho jaem|
kuchha eka varshom mem, ve moti ho jati haim |
vidyalaya ke bada unaki sakriyata ghata jati hai |
anya koi bahalava aura manoranjana na hone ke karana ve khane para adhika dhyana kendrita karane ki ora pravritta hoti haim| 
lavanyapurna dikhane ki unaki ichchha bhi kshina ho jati hai |
anuvanshika se adhika samajika-arthika prabhava sthulata ke lie pratyaksha rupa se ja़immedara haim|
eka mam jo bachche ko batati hai ki ghi (makkhana tela) isake lie achchha hai, kisi anuvanshika karana se adhika nukasana kara rahi hai |
manovaijnanika karaka sthula vyaktiyom ko vaja़na kama karane ke lie rokane mem eka mahatvapurna bhumika nibhate haim | 
kuchha doshi sanlakshana aura sharira kalpana badha ke karana ve rata mem khate haim |
eka 22 varsha ki lada़ki ka dina ke samaya eka uttama poshahara hota hai lekina rata mem arddha chetanavastha mem vaha rasoighara taka jaegi aura eka vrihada matra mem khana khaegi|
mote bachche aksara bahuta adhika khate hue dekhe gae haim |
eka bachcha jo 3-4 aisakrima khata hai aura isake upara se do milkasheka pita hai durlabha nahim hai |
eka dulhana vivahotsava para apane 85 paunda ke satha khusha thi |
lekina 10 varshom bada vaha 135 paunda ke sharirika vaja़na ke satha bhi utani hi santushta thi |
eka sthula vyakti ko thika raste para rakhane ke lie bahuta adhika driढ़ nishchaya ki avashyakata hoti hai |
unchai bhara sariniyam kabhi kabhi kuchha bhi upayogi nahim hoti haim |
eka adarsha sharirika bhara vichara-dhara vikasita karana behatara hai |
yaha adhika yathartha hai aura rogi ko patale hone ke lie kathina kama karane ke lie utsahita karata hai |
motai ko adhikatara galati se achchha svasthya samaja jata hai, lekina yaha satya nahim hai | 
eka mota sharira bimariyom ko behatara sharana deta hai |
mote loga samanyatah madhumeha, uchcha rakta chapa, hridaya rogom, pittashaya pathari, gathiya, peta mem sujana aura yakrita samasyaom se gujarate haim |
ye bimariyam, patale logom ki tulana mem sthula logom mem uchcha nrityu sankhya utpanna karati haim |
lekina saubhagyavasha yaha saba ulata ja sakata hai |

sanyukta rashtra (yuesa) mem pratyeka chara logom mem se eka vaja़na ghatane vale poshahara para hai |
vaham atyadhika svasthya jagarukata hai |
haja़arom loga dhire-dhire dauda़te hue aura svasthyakara bhojana lete hue dekhe ja sakate haim |
mote logom mem kolestraॉla stara uchchatara hota hai |
yaha isalie hai kyonki, vasa ke jamava rakta ki dhara mem apana sthana bana lete haim | kolestraॉla jo ki eka vasa hai aura rakta ki dhara mem, yaha rakta ko aura adhika gadha़a banata hai |
yaha rakta vahikaom ki avakashika mem unako patala karate hue jama ho jata hai aura rakta ke lie bahana aura adhika mushkila bana deta hai |
hridaya ki apurti karati hui chakriya hridiya dhamaniyom ki avakashika bhi sankari ho jati hai |
hridaya ki rakta ki apurti ghata jati hai aura yaha hridaya rogom ko janma de sakata hai | 
kolestraॉla pittashaya pathari se bhi sambandhita hai |
kolestraॉla ke satha pitta ki ati santriptata pitta pathariyom ke banane mem sahayata karati hai |
yaha una mahilaom mem adhika samanya hai jo sthula, chalisa ke asa-pasa aura sapha ranga ki haim |
garbhavastha ke samaya aura eka madhumehi bachche ke mamale mem dina mem do bara ki adhika uchcha khurakom ki avashyakata ho sakati hai |
bimara logom mem jaba sharkara ka stara jo avashyaka hai usase ninnatara starom para pahunchata hai to haipoglaisimika pratikriya ho sakati hai |
ina rogiyom ko sharkara kyuba ya biskuta rakhane chahie aura jaba aisi sthiti paida hoti hai taba inhem khana chahie |
injekshana, tvachiya lala dadorom, drava pratidharana aura drishtigata khalala ke sthanom para adhotvachiya vasa ka avashoshana ho sakata hai |
ve salphonila yuriya aushadhiyam hoti haim aura unamem taॉlbyutamaida, klaॉrapropamaida, esetohekja़amaida, taॉleja़amaida aura glaibenklemaida shamila haim |
ve kevala tabhi di jati haim jaba madhumeha poshahara aura vyayama se niyantrita nahim hota hai |
yaha aushadhi agnyashaya para kriya karati hai aura ise adhika insulina sravita karane ke lie uttejita karati hai |
eka amarikana adhyayana batata hai ki taॉlbyutamaida isa bimari ko rokane mem asaphala ho chuki hai aura yaham taka ki ise aura badtara bana diya hai |
yaha sharkara ko avashyakata se atyadhika ninna starom taka ghata sakati hai aura yaha mastishka ko prabhavita kara sakati hai |
mukha se prati-madhumehi aushadhi lene vale atha sau rogiyom para 3-8 varshom taka shodha kie gae the|
yaha adhyayana darshata hai ki isa samayavadhi mem 89 rogi mara gae |
yadi rogi ko isa aushadhi para lambe samaya taka rakha jae to isake parshva prabhava gambhira ho sakate haim |
yaha madya ke prati asahanashilata badha़a deti hai aura mukhiya sampravahana utpanna karati hai
phenaphaॉrmina aura metaphaॉrmina aushadhiyom jaisi bigunaidsa sanyukta rashtra (yunaiteda stetsa) mem khadya aura aushadhi prashasana dvara pratibandhita ho gayi haim |
parshva prabhava haim bhukha ki kami, michali, ulti, atisara aura dugdhika anliyata |
10 se 20 pratishata rogi mukha se lie jane vale karakom se pratikriya nahim dikhate aura upachara prathamika asaphalata kahalata hai |
nayi bimariyam purane sthayi madhumeha se behatara pratikriya dikhati haim |
yadi yaha poshahara se niyantrita nahim ho sakata hai to mukha se li jane vali aushadhiyam di ja sakati haim |
ina rogiyom ko pahale mukha se li jane vali aushadhiyam di jati haim aura yadi yaha asaphala hoti hai to chikitsaka insulina ka sahara lete haim |
jaba madhumeha niyantrita ho jata hai taba rogi ko phira se mukha se li jane vali aushadhi para rakha jata hai |
vyayama aura poshahara ke satha apaki yuva varshom ki pratyasha kama nahim hogi |
yadi apa laparavaha haim, yadi apa apane apa ko niyantrita nahim kara sakate haim, apane apa ko vyayama karane ke lie nahim prerita kara sakate haim ya apako sahi poshahara banae rakhane ka paryapta dridha़ irada nahim hai, taba apa eka bahuta bada़a khatara le rahe haim |
usa samaya, jaba apako pata chalata hai ki apa madhumehi haim, sahasa ekatra kijie aura jivana shaili ko badalane ke lie apane apako badhya kijie | 
yaha koi asana kama nahim hai kyonki apa varshom taka eka vishishta shaili mem jie haim aura isake adi rahe haim, purani adatom ko chhoda़na kathina hota hai |
apa isa prakara jie aura madhumeha prapta kiya |
yadi apa eka madhumehi haim to apako ojapurvaka vyayama karana chahie |
yadi apa vriddha haim tatha yadi apa vyayama nahim kara sakate haim, taba apako kama se kama subaha aura shama, pratyeka samaya eka-eka ghanta avashya tahalana chahie|
dopahara tatha rata ke bhojana mem ubali sabjiyom (adhimanatah 250 grama), saladom, dahi tatha sukhi chapati ka sevana karem
vajana girana shuru hone se purva hi sharkara ka stara ghatana prarambha ho jata hai|
ho sakata hai kuchha dinom taka apa kailori kama karane ke karana kamajori mahasusa karenge kintu 2-3 saptaha mem apa adhika duri taka tahalane ke lie jyada taka़ta aura urja prapta kara lenge|
agara apa halke se madhyama darje ke madhumeha-rogi haim, to do se tina maha mem rakta sharkara samanya ho jaegi |
agara apa uchcha darje ke madhumeha-rogi haim, to mukha se li jane vali prati madhumeha aushadhiyam avashyaka hongi|
parantu, agara apa isa dinacharya mem bane rahe to 6 mahine se lekara eka sala ki samayavadhi mem apa isa aushadhi ko chhoda़ sakate haim|
eka bara samanya rakta sharkara stara prapta kara liya, to sabase kathina karya shuru hota hai|
yadi apa achchhe bhojana adi ki lalasa se bahaka jate haim to apaka sharkara stara punah badha़ jaega aura yaha kathina parishrama vyartha ho jaega|
kai bara jaba yaha rogi rakta jancha ke lie jata hai to use kaha jata hai ki usaki sharkara thoda़i badha़ gai hai|
jaba rogi isa dharana ke satha vapasa ata hai ki usamem kuchha kharabi nahim hai aura use chinta nahim karani chahie tatha yaha ki vaha thoda़i-bahuta pratibandhom ke satha lagabhaga isi bhojana ko jari rakha sakata hai| 
rakta sharkara ko samanya stara para lane ke lie bhojana purnatah pratibandhita hona chahie| 
rakta sharkara ke samanya stara ko para ki anumati nahim deni chahie|
aise rogi ko 6 mahine ke antarala para ya kama se kama sala mem eka bara, rakta sharkara ki niyamita jancha karani chahie 
kai rogiyom ko jaba pata lagata hai ki ve madhumeha ke rogi haim to ve joradara dayatinga karate haim|
durbhagyavasha, eka bara jaba sharkara stara gira jata hai to unhem lagata hai ki ve svastha ho gae haim aura sabakuchha khana shuru kara dete haim|
dayatinga dvara sharkara stara niche lane ke bada, sabhi prakara ke bhojana ko bhakosana prarambha mata kara dijie kyonki sharkara stara turanta chadha़ jaega|
aise kai rogi haim jo apane ahara para pratibandha lagane ka kashta nahim karate haim aura aushadhiyam lete rahate haim|
agara ve kisi samaroha mem jate haim to sharkara vriddhi ko eka goli khakara rokane ka prayatna karate haim|
sharkara stara ko ghatakara samanya karane ke lie apane ahara para pratibandha lagaem, yadi sharkara stara abhi bhi bahuta adhika hai to eka aushadhi se ise anupurita karem|
yadi apa aushadhiyom para hi nirbhara rahate haim, to ye jindagi ko chhoti kara dengi tatha kshati pahunchaengi|
hridayaghata ya kainsara ki bimari hone se yaha behatara hai jo raste sudharane ka shayada hi koi mauka़a dete haim, yadi apa aisa karane ke lie utsuka hom taba bhi| 
atom ko achchhi taraha chalana chahie aura yadi avashyakata ho to sade pani ke enima ka prayoga kiya ja sakata hai| 
rakta sharkara ko kama karane mem isa ahara ki pratikriya adhika tivra hai|
vajana mem yaha kami rogiyom ke lie naitika protsahana ka eka srota hai aura ve asani se prerita ho jate haim| 
jaise-jaise rogi ka vajana ghatate jata hai, yakrita se glukoja़ ka utpadana dhima pada़ jata hai| 
insulina grahiyom ki sankhya mem vriddhi ke satha insulina pratirodhana kshamata kama ho jati hai|
insulina ki kriya adhika prabhavi ho jati hai tatha bita koshikaom ka srava badha़ jata hai|
jaba rogi ko kama kailori ka ahara diya jata hai, to kailoriyom ko, samana rupa se pure dina ke pramukha bhojanom mem, banta diya jata hai 
eka bara ke bhojana mem sharira ko kailoriyom se bhara nahim dena chahie|
yaha atyavashyaka hai kyonki madhumeha mem sharira ki chayapachayi kriya durbala ho jati hai| 
yadi rakta sharkara uchcha hai, to insulina ya anya prati-madhumeha aushadhiyom ki avashyakata ho sakati hai aura taba bhojana ke samaya ko aushadhi ke anusara samayojita karana chahie|
aushadhi ki matra dhire-dhire kama ki jati hai kyonki rakta sharkara stara niche chala jata hai aura adhikatara mamalom mem ise purnatah hataya ja sakata hai|
isa uddeshya ki purti ke lie rakta sharkara stara ki niyamita jancha tatha daॉktara se niyamita paramarsha lene ki avashyakata hogi|
vayaskom ke madhumeha mem, insulina ka srava phira bhi upasthita hota hai aura chunki kama kailori ahara chalata rahata hai, ise aura adhika uttejita kiya ja sakata hai tatha adhika aushadhi ke manga ki avashyakata nahim hogi|
yadi sharkara stara bahuta adhika nahim haim, to karbohaidretom ko dina bhara ke sabhi bhojanom mem vitarita karana chahie tatha khajurom, kelom sahita sabhi phalom ki udara matraom ko sammilita kiya ja sakata hai|
masura mem kama tatha chavala, gehum aura aluom mem prachura matra mem upalabdha, jatila shvetasarom ka adhika sevana karem| 
yadi apa madhumeha ke rogi haim, taba bhi apako alu, chavala aura gehum khane ki anumati di jati hai|
kanda sabjiyom ko kabhi bhi pratibandhita karane ki ja़rurata nahim hai|
phalom ka prachura matra mem sevana kiya ja sakata hai, ye rakta sharkara stara ko badha़ane mem shayada hi koi bhumika nibhate haim tatha inake chayapachaya mem insulina ki avashyakata nahim hoti hai| phalasharkara (phalom mem upasthita sharkara) udarantriya kshetra se sharkara ki apeksha adhika dhimi gati se avashoshita hoti hai aura yakrita dvara li jati tatha chayapachita hoti hai|
isa ga़airainsulina ashrita prakriya se yakrita phalasharkara antatah glaikojana tatha traiglisaraidom mem parivartita ho jati hai|
apa phalom ke prati udara ho sakate haim angura, chiku aura ama jaise mithe phalom ke prati bhi| 
yadi apa seba ya gajara khate haim to apa isake rasa ke satha-satha bahuta sare reshe bhi lete haim|
kuchha gajarem ya seba apako dina bhara ke lie reshom ki paryapta apurti de denge|
ahara mem reshom ki matra prachura honi chahie; ye bhojana ke aise sanghataka haim jinhem sharira avashoshita nahim karata hai|
ye hare saladom, sabjiyom, phalom, chavala ke chokara tatha gehum ke chhilake, aparishkrita karbohaidretom aura jatila shvetasarom mem bhi prachurata mem hote haim|
yadi reshom ki prachurata ho to karbohaidretom ki sharkara badha़ane ki gunavatta bahuta kama ho jati hai|
resha pachana-tantra ka mitra hai, yaha pachana-tantra mem eka jela banata hai jo sharkara ke avashoshana ko dhima kara deta hai| 
yaha vasaom ke avashoshana ko kama kara deta hai jisase antem teji se aura achchhi taraha se chalati haim|
reshe ki vakalata karane vale prathama vyakti batala grika sainetoriyama ke sansthapaka, mishigana, yu.esa.e. ke daॉ. jaॉna harve kelaॉga the|
ve isa tathya se danga the ki kuchha bimariyam jo purva mem ama the ve tisari duniya ke deshom mem ama nahim the, jaise ki madhumeha aura hridaya roga|
unhonne yaha nishkarsha nikala ki parishkrita shakkara aura sapheda dabalaroti ki badha़ti khapata isake lie jimmedara thi| 
parishkrita karbohaidreta ke kai parinama haim, jinamem se eka hai reshom ka hatana|
isaka pachana tatha parisancharana tantra para avanchhaniya prabhava pada़ta hai|
jathariya riktikarana vilambita hota hai tatha karbohaidretom ke sampurna avashoshana aura pachana manda pada़ jate haim|
bharatiya bhojana mem karbohaidretom ki uchcha matra tatha reshom ki uchcha matra svabhavika rupa se upalabdha hoti haim| 
isilie kai bimariyom ki roka-thama ke lie pashchima valom ki apeksha bharatiya bhojana atyanta shreshtha hai|
ahara svasthya avashyakataom ke anusara yaham-vaham kai sanyojana banane honge|
ausata bharatiya bhojana mem 60% karbohaidreta, 20% protina aura 20% vasa hota hai|
yaha darshata hai ki madhumeha-rogiyom ke lie karbohaidreta ki uchcha matra-reshe ki uchcha matra yukta bhojana upayukta hai| 
bharatiya bhojana prayah shakahari tatha anna para adharita hota hai|
pashchimi bhojana mem guara gama, chokara tatha reshom ke anya srotom ko milakara reshom ki matra badha़ai jati hai, kintu bharatiya ahariya udaharanom mem, anna tatha dalem reshe ki uchcha matra ke svabhavika srota haim|
hamem sirpha़ bhojana ko parishkrita karana banda karane ki avashyakata hai, sapheda ata (maida) tatha shakkara jo ki resha rahita parishkrita khadya haim, ka upayoga banda karem|
sabhi mansahari khadyom mem reshe nahim hote haim|
atah shakahari bhojana madhumeha ke rogiyom ke lie behatara hai|
saladom, hari sabjiyom, chhilake sahita gehum tatha impaundeda chavala ki bahuta bada़i matra ki ja़oradara sipha़arisha ki jati hai|
pashchimi loga sabase adhika pida़ita haim kyonki unake bhojana mem resha kama hota hai|
bhojana mem mansahari khadya (resha rahita),vasa (reshe rahita), dudha utpada (resha rahita) tatha shakkara (resha rahita) haim| 
bachchom tatha garbhavati mataom ko reshe ki uchcha matra yukta bhojana lene ki avashyakata nahim hai|
bachchom tatha garbhavati mataom ko adhika poshana ki avashyakata hoti hai tatha reshedara khadya kevala uchcha parimana vala khadya-padartha hai|
eka rogi ko usake bhojana ke bare mem jise vaha jivana bhara ke lie palana karega, margadarshana karana eka jatila karya hai|
isake lie unaki jivana shaili, arthika sthiti, bhojana prathamikataom tatha samajika prabhavom ko dhyana mem rakhane ki avashyakata hogi|
isake lie eka chikitsaka ki ora se apane rogi ko isa santoshaprada lakshya, jo hai ki kaba yaha rogi apane svada aura santushti ke lie kha sakata hai , ka margadarshana karane ke lie vishesha prashikshana dene ki bhi avashyakata hogi|
taba chikitsaka khusha hota hai ki vaha eka aise bhojana ko shamila kara sakata hai jisamem reshe ki bada़i matra hai| 
tanava madhumeha ke lie eka karaka ghataka ho sakata hai|
tanava madhumeha rogiyom ke rakta mem sharkara stara ko badha़a deta hai|
tanava ke karana, mastishka mem adhashchetaka uttejita ho jata hai, jisake parinamasvarupa piyusha granthi uttejita ho jati hai|
adhunika jivana tanava aura parishrama se bhara pada़a hai tatha tanava mukti takanikiyam madhumeha rogiyom ki madada karengi| 
agnyashaya mem insulina ka utpadana hamesha hota hai|
para roga-pratikarakom ke nirmana ke karana yaha ya to aparyapta hai ya yaha prabhavakari nahim hai|
adhikatara mamalom mem ye kevala bhojana tatha vyayamom dvara niyantrita kie ja sakate haim|
anya paristhitiyam bhi haim jaham insulina ka utpadana purnatah banda ho sakata hai jaise ki karkaroga tatha agnyashaya ke sankramana ke mamalom mem|
yaham para rogi ko bhojana niyantrana ke satha-satha insulina ki bhi avashyakata hogi|
kuchha eka mamalom mem, madhumeha bachapana mem hi vikasita ho sakata hai, jaise taruna madhumeha|
isalie bhojana mem paryapta kailoriyam honi chahie|
ye insulina ke bina niyantrita nahim ho sakate|
insulina injekshana tatha khane ke samayom ko isa taraha samayojita kie jate haim ki rakta mem aushadhiyom ke ishtatama stara se bhojana ka antargrahana mela khae|
jinhem melitasa madhumeha hai unaki apeksha ye rogi sankhya mem bahuta kama haim|
madhumeha sampurna janasankhya ke 2-4% ko prabhavita karata hai, jabaki sampurna madhumeha pida़ita rogiyom ka 2% hi taruna madhumeha rogi hote haim| 
taruna madhumeha, yadi aniyantrita ho, to kitonamayata mem vikasita ho sakata hai|
insulina ke charamabhava mem kitona pindom ko rakta mem chhoda़ die jate haim aura vaham mutra mem bahara nikala die jate haim| 
aisa rogi asvastha mahasusa karata hai, michali aura ulti tatha udariya pida़a mem badha़ sakati haim|
bada mem vaha nidralu ho jata hai tatha sammurchchha mem chala jata hai|
yadi anupacharita chhoda़ diya jaya to halata janaleva ho sakati hai
aisi asha ki ina aushadhiyom se rogamukti hogi, kama ho gai|
ina aushadhiyom ko kevala taba hi deni chahie jaba bhojana aura vyayama roga ke niyantrana mem asaphala ho chuke hom| 
gambhira madhumeha mem, kama se kama prarambha mem insulina diya jata hai aura taba khane vali aushadhiyom mem badala jata hai| 
insulina ko tvacha ke thika niche, injekshana se diya jata hai
yaha laghukalina ya dirghakalina kriyashila hoti hai aura eka injekshana mem milai ja sakati hai taki yaha pure dina ke lie paryapta ho|
udakameha mem, piyusha granthi ya adhashchetaka, mastishka ka eka bhaga, samanya rupa se karya nahim karata hai|
bharata mem nrityu ka mukhya karana madhumeha (dayabetisa melitasa ) hai | 
madhumeha se pidita vyakti ka sharira chini ka sadharana upayoga nahim kara sakata , atah chini rakta mem jamane lagati hai | 
gurde atirikta chini ki thoda़i-si matra peshaba mem nikshipta karate haim | 
madhumeha ki gambhira sthiti mem vasa aura protina bhi sadharana rupa se istemala nahim ho pate haim| 
jyadatara chikitsakom ka kabhi yaha manana tha ki agnashaya mem nirmita aura sharira ko madada shakkara jama karane va usaka upayoga karane mem madada karane vala insulina namaka haॉrmona kama hone para hi madhumeha hota hai | 
kai madhumehagrasta marija, khasa kara vaha jinhem 40 sala ki umara mem yaha vyadhi hui ho, samanya ya samanya se adhika insulina bana lete haim | 
para unake sharira para insulina anukula asara nahim dikhata hai | 
atyadhika mutratyaga , jora ki pyasa bhukha aura vajana tatha takata ka ghatana madhumeha ke lakshana hai | 
taipa 1 kisma ka madhumeha kuchha logom para itane akasmika rupa se akramana karata hai ki insulina ki kami ki vajaha se dayabitika ketosidosisa namaka apata sthiti paida hoti hai | 
isamem atyadhika mutratyaga, pyasa, bhukha ki kami, ji machalana ulati hona aura sansa lene mem kashta hona yaha shikayatem hoti hai | 
marija ka turanta ilaja na hone para vaha dayabetika koma mem ja sakata hai , jisake parinama ghataka sabita ho sakatee haim | 
jyadatara taipa 1 madhumeha ke marija socha-samajakara nirdharita khuraka ka palana karate hai, jisamem parimita matra mem karbohaidreta, vasa aura protina hote haim | 
kama kailari yukta khuraka se taipa 2 madhumeha para kabu kiya ja sakata hai | 
dayabetisa se hamari tantrikaom para asara pada़ sakata hai , jisake phalasvarupa sharira ke vibhinna ango mem samvedanashilata ki kami ya phira asvabhavika anubhutiyam mahasusa ho sakati hai | 
yaha hridayaghata, hridaya gati ke rukana ya kotha (gaॅngrina) ka karana bana sakata hai | 
amira pashchatya mem tisare vishva ke deshom ke mukabale madhumeha ka prakopa adhika hai | 
shahari logom mem madhumeha ki bimari gramina logom ke tulana mem jyada hai | 
shahari garibom ke mukabale shahari raisom mem isake shikara adhika haim | 
melitasa shabda jisaka artha madhu hai, usase dayabetisa (melitasisa) nama bana hai | 
madhu jaisi mithi peshaba ke karana isaka madhumeha (dayabetisa melitasa ) namakarana kiya gaya hai | 
peshaba aura khuna mem glukoja ki matra adhika hoti hai |
yaha taba hota hai jaba agnyashaya paryapta matra mem insulina banane mem akshama hota hai kyonki khuna mem paida hue roga-pratikaraka tatva insulina ke samanya karyapranali mem dakhala dene lagate hai | 
donom hi sthitiyom mem khuna mem glukoja ki matra adhika badha़ jati hai | 
jaba hama jyada shakkara khate haim , to khuna ke shakkara ko niyantrita karane ke liye jyada insulina banata hai aura ise samanya matra se adhika badha़ne nahim deta | 
adhika shakkara khane se agara agnyashaya para jyada dera taka bhara pada़e to vaha kama karana banda karane lagata hai | 
kuchha svechchha sevakom ko 14 dina taka adhika shakkara vali khuraka di gai , unamem se eka tihai ke khuna mem insulina ki matra badha़i hui thi | 
isase yaha sabita hua ki aisa eka guta hai jinamem jyada shakkara khane se madhumeha hone ki jyada pravritti hoti hai | 
kuchha logom mem glukoja ki matra adhika hoti hai jinhem pahachanana asana hai jaba kama unra mem madhumeha ki shuruata ho aura glukoja ki matra utani adhika na ho | 
khuna aura mutra ki glukoja ke lie pratah kala ahara ke purva jancha hoti hai | 
bada mem munha se 75 graॅma glukoja़ diya jata hai aura do ghantom taka hara adhe ghante para rakta ka namuna liya jata hai | 
hasare kaॉleja ke vaidyakashastra, daॉ. bi. ke. gupta ne apane tippani ki shuruata mem madhumeha parajora dete hue kahate hai apa agara madhumeha se vakipha ho to apako apani dava bhi maluma hogi| 
madhumeha sharira ke anya kisi bhi vyadhi ko badha़a sakata hai, use pechida bana sakata hai tatha ilaja mushkila kara sakata hai | 
ama taura para madhumeha ki pahachana hai bahumutra (barambara mutratyaga ), rata mem apako kai bara peshaba ke lie uthana pada़ sakata hai , paॉlidisiya (adhika pyasa) | 
samanya ya adhika bhukha ke bavajuda apaka vajana ghata sakata hai | 
apa badana mem darda mahasusa karenge, thakavata ho sakati hai tvacha para ubhare phode phunsiyam jaldi thika nahim hoti | 
apa ko kai kisma ki sankramaka bimariyam jaise ki kshayaroga, gurde ka samnkramana ya pailonephrayatisa ho sakata hai | 
apako drishtipatala sambandhi vilagana ho sakata hai ya andhapana bhi a sakata hai , paira mem hue sankramana se mansa ka saड़ava (gaॅngrina) ho sakata hai jise shuruata mem hi roka na jane para angachchhedana karana pada़ sakata hai | 
madhumeha se grasta garbhavati striyom ko nrita-prasava hone ki jyada sambhavana hoti hai | 
madhumeha sandhishotha ko jatila bana sakata hai | 
sharira ki koi bhi takalipha le lijie aura madhumeha use pechida kara dega | 
agara koi bimara hai aura jalda thika nahim ho raha ho to jarura madhumeha ki sambhavana ki jancha kara lem, khasa kara 40 ke unra mem hara sala madhumeha ki jancha karaem | 
marija ke sahayoga se madhumeha ka ilaja saphalatapurvaka ho sakata hai | 
agara apake peshaba tatha rakta mem glukoja़ ki matra adhika ho to sabase pahale apako ahara para kabu pana chahie | 
agara apako madhumeha nishchita rupa se ho to chini puri taraha se banda karem | 
isase chini ki matra kuchha hada taka ghategi | 
aba isaka prati vyakti upabhoga yuke mem 90 kilograma aura yuesae mem 80 kilograma taka pahuncha gaya hai | 
landana vishvavidyalaya ke poshana ahara vijnana ke mashahura adhyapaka daॉ. jaॉna yudakina kahate haim ki vikasavadi mapaka para nape jane para kaphi badha़i hui yaha shakkara ki khapata itane teja raphtara se ayi hai ki manushya ko isake adi hone ka samaya nahim mila | 
ve anumana lagate haim ki shakkara ki badha़i hui khapata ke karana hridaya vikara aura madhumeha ke mamalom mem bhari vriddhi hui haim | 
vishva yuddha 1 aura 2 ke daurana niyata matra mem khadyanna dene ke karana shakkara ki khapata ghata gai thi aura madhumeha se hone vali nrityu sankhya mem spashta rupa se kami ayi thi | 
poshana ahara visheshajna daॉ. yudakina ne 22 deshom mem shakkara ki badha़i khapata aura madhumeha ke bicha sambandha darshaya hai| 
jaba se shakkara aura sapheda maide ki roti ka prachalana hua hai taba se isaka prakopa badha़a hai | 
madhumeha aura hridaya ki bimariyom ka abhutapurva badha़ta prakopa ka karana hai yaha mahina*** karbohaidreta | 
hamem takata jutakara isa paristhiti ko apani khanapana ki adatem badala kara viparita disha mem ghumana hoga | 
madhumeha ki eka achchhi bata yaha hai ki isake hote hue bhi niyamita jancha ke jariye shakkara ki matra para ankusha rakhane se marija lambi samanya sakriya jindagi ji sakata hai | 
gurde, hridaya athava sharira ka koi ahama hissa madhumeha ki chapeta mem a sakata hai aura eka arthapurna, sakaratmaka aura kamayaba jivana samapta ho sakata hai | 
eka dina mem li gai kailoriyom ka pramana ghatakara aisa kiya ja sakata hai | 
isase khuna mem shakkara ki matra ghatane mem sahayata milegi | 
vyayama eka atirikta phayada hoga kyom ki yaha kailoriyom ki khapata ko badha़ata hai aura vajana ghatana tvarita karata hai | 
baithakara kama karane vale vyakti dvara prati dina ausatana 30 kilo kailori prati kilo grama sharira ke vajana jitani kailora kharcha ki jati hai | 
ahara badale bina agara sharirika parishrama badha़aya jaye to kailori ki khapata badha़egi aura vajana ghatega | 
charbi utaka eka nishkriya pinda hota hai jo sharirika karyom mem bhaga nahim leta | 
sharira mem charbi jitani jami hogi utani hi kailori ki jarurata prati kilograma (sharira ke vajana ke ) kama hogi |
apa agara samanya matra se kama kailori lete haim to apa vajana ghatayenge | 
lambe samaya ke daurana dhire-dhire vajana ghatana jyada uchita hai , na ki kama samaya mem achanaka | 
agara apa kailori ko (ahara mem) santulita matra taka niyantrita rakha sakatem to apaki takata bani rahegi, apa apana niyamita kama kara payenge aura vyayama bhi | 
khaya hua ahara santulita ho aura use achchhi manasika samarthana ki samarthana bhi hona chahiye| 
apa jitani kama kailori lenge utana hi jyada apaka vajana ghatega | 
ahara kailori ke mamale mem to niyantrita ho , para yaha vitaminom tatha sukshma-poshaka tatvom se bhara hona chahiye | 
eka sthulakaya vyakti ke ahara mem kailori bhale hi kama hom para poshana ki drishti se yaha uttama hona jaruri hai | 
yaha kisi svasthyagara mem sahaja sambhava hoga jaham apako anukula vatavarana milega | 
ghara para bada़e paimane para abhiprerana aura daॉktara ki niyamita salaha ki jarurata hogi | 
hama jaise hi kailori-niyantraka ahara shuru karate haim vaise hi sharira ka vajana ghatane se pahale hi plaja़ma (khuna ka tarala hissa) mem glukoja़ ki matra ghatane lagati hai | 
hasila kiya hua glukoja़ ka stara kailori-niyantrita ahara se tikaya ja sakata hai | 
ama taura para sipharisha kiya hua bhojana mem kaphi matra mem hari sabjiyam, salada, phala, ankurita anaja, dahi, dala aura chapati honi chahiye | 
chija़, makkhana, ghi, mithai, samose, pakoड़e, chini, aisakrima, shita peya aura sharaba jaisi uchcha matra ki kailorivali chija़om ko talana chahiye | 
pratidina alaga alaga phala aura ankurita anaja ka nashta aura salada, hari sabjiyom, dahi aura rotiyom ka bhojana karana uttama hoga | 
dopahara mem kuchha phala ya phalom ka rasa thika rahega | 
subaha ghante bhara teja़ kadama chalana chahiye | 
jaise jaise apa pragati karenge, rakta chapa kama hoga | 
rudhira pittasandrava (kolesterala) aura glasaraiड़ ghata kara samanya stara ki ora ayenge | 
marija़ aura chikitsaka ka sambandha mahatvapurna hai | 
kai marija़ khisaka jate haim aura agali jancha ke dina najara nahim ate | 
kuchha marija़ to jana-buja kara ilaja ko nakara denge para jyadatara yaha roga ke bare mem sahi samaja ke abhava se hota hai | 
eka susanchalita klinika mem marija़ ke upasthiti ka likhita tathya hona chahiye tatha avashyaka hone para marija़ ko jancha ke bare mem smarana karana chahiye | 
mutravardhaka davaiyam ahama hai kyonki yaha sharira se atirikta namaka nishkashita karate haim | 
mutravardhaka ki samasya yaha hai ki yaha avashyaka hone vale potaishiyama lavana bhi nikala dalata hai aura phala ke rasa ya potaishiyama purakom se inaki apurti honi chahiye | 
ina parshva prabhavom ke atirikta klonidaina kabja, kampana , pasina chhutana aura anidra ka bhi karana bana sakata hai | 
isase dasta ya atisara, dhima hridayagatimana, thakana aura naka banda hona sambhava hai | 
uchcha rakta chapa ka mukhya karana hai paridhiya rakta dhamaniyom ka sankuchana | 
edrenerjika nyuraॉna blaॉkinga aushadhiyam raktachapa ghatane mem atyanta asaradara hoti haim | 
parshva prabhava hai tvacha ka lala hona, saradarda, dila dhadakana, jatharantriya gada़bada़iyam, sara chakarana tatha mala-mutra ka ruka jana | 
purushom mem napunsakata, potaishiyama ki kami, mutranla ki matra badha़ jana, ji machalana, chakkara ana, lala chakatte, sunnata, junajuni jaise parshva prabhava (saida iphektas ) ho sakate haim | 
pradhana vyavastha para asara karanevali aushadhiyam unindapana aura munha sukhana, khuna ki kami, dasta, tvacha ki jalana bhi paida karate haim , vaise hi 15 dina ke bada davai se bukhara bhi hota hai | 
bita blaॉkarsa (praiktolola aura prophranolola) hridaya ke akunchana ki raphtara aura takata ko ghatate haim tatha hridaya ka utpada kama karate hai | 
ye aushadhiyam unhem prasarita karane aura rakta chapa ghatane mem madada karati hai | 
ye hridaya ka gatimana badha़ate haim aura hridaya ko sankuchana ke liye badhya karate haim | 
lambe arse taka istemala karane se traiglisaraida badha़ sakate haim aura gadha़e kolestrola ghata sakata hai jo hridaya ke lie achchha hota hai | 
shvasahinata aura takhanom ki sujana parshvaprabhava ho sakate hai | 
sambhoga ke daurana purushom ke virya ke srava mem badha utpanna ho sakati hai | 
sadiyom se bharata mem istemala ho rahe raulaphiya sarpentina namaka paudhe se rijarpaina banaya jata hai | 
1. isake gauna prabhava haim hridaya ki dhimi gati, peta ke phoda़e ka badha़na, dasta, unindapana, udasi adi | 
2. ina davaiyom ki karya-paddhati bhinna hoti hai va isake kai gauna prabhava hote haim| 
3. ina davaiyom ko aksara mishrana ke rupa mem diya jata hai taki inaki khuraka tatha anushangika parinama kama kiya ja sake | 
4. yaha marija ki unra, pesha, mausama adi para bhi nirbhara karata hai | 
5. hama ilaja shuru karane se pahale yaha samaja lem ki yaha ilaja ajivana chalega | 
6. inako rokane para rakta ka dabava ilaja ke purva matra para lauta aega ya usase thoda़a jyaada badha़ bhi sakata hai | 
7. sadharanatah dila ka daura rokane ki ummida se rakta chapa ka ilaja kiya jata hai | 
8. kai adhyayanom se yaha maluma hua hai ki isase antima roga ki pragati mem shayada hi koi pharka pada़ta hai | 
9. anya davaiyam achanaka banda karane se hritshula (anjaina pektorisa) badha़ sakata hai ya mayokardiyala imphraॅkshana ho sakata hai | 
10. yaha yada rakhana mahatvapurna hai ki agara hama lambe samaya se koi dava le rahe ho to use dhire-dhire kama karate hue banda karana hara hala mem behatara hai chahe isake lie haphte ya mahine laga jaem | 
11. kaphi sanshodhana ke bavajuda uchcha raktachapa ko kama karane mem aushadhopachara ke phayade ke bare mem abhi bhi shaka hai | 
12. uchcha raktachapa ya ati tanava ko prachalita davadaru se nahim balki niyata ahara aura tanava ghatane ke khasa tarikom se kabu mem laya ja sakata hai | 
13. agara hama raktachapa niyantrana mem rakhem para dhunrapana, vasayukta padarthom ka ahara , vyayama ka abhava aura tanava jaise anya ghataka ghatakom para kabu na rakhem to parinama hanikaraka honge | 
14. jivana paddhati mem badalava lane ka prayasa kiye bagaira sirpha davaiyom para nirbhara karane se aushadhopachara phayademanda na hokara nukasana karega | 
15. hridaya vikara ya daure ko talane mem davaiyom ki asaphalata ko pashchima-deshom ne achchhi taraha samaja liya hai | 
16. arthika sanriddhi aura aishoarama ki sulabhata ke karana dila ke daure se nrityu, pashchimi deshom mem ama bata hai | 
17. hara sala bharata mem 35 miliyana se jyada loga dila ke daure se marate haim | 
18. pashchimi desha svasthya ke bare mem adhika sacheta ho gaye haim | 
19. dhunrapana karane valom ki sankhya prati varsha ghata rahi hai | 
20. unhonne dauda़ne aura bahuta sari reshedara chijem jaise salada, phala tatha ubali sabjiyam jyada khane tatha mansa khana kama karane ki adata dala li hai | 
aushadhopachara se jyada achchha svasthya ke lie jivana paddhati para gaura karane ka samaya a chuka hai | 
22. chini, tela ( ghi aura makkhana bhi ) , namaka, maida, chaya aura kaॉphi kama kijiye | 
23. isase apako jyada mutra tyaga karane aura sharira se mala padartha nishkasita karane mem madada milegi | 
24. agara apako kabja़i mahasusa ho to gunagune pani ka enima lem | 
25. vyayama se raktachapa baढ़ta hai, atah behatara hai ki halke vyayama se shuruata karem | 
26. sharira ko vyayama ki adata hone para raktachapa adhika sthira rahega | 
27. vyayama se rakta sanchalana baढ़ne mem madada milegi | 
28. dhamaniyam banda hokara dila ka daura pada़ne ke asara ghatenge | 
29. mastishka ki koi nada़i phatakara lakhava mara sakata hai | 
30. apaka kama ya pesha agara tanava paida kara raha hom to usase achchhe se nipatana sikhiye | 
31. hara roja shavasana karane ki koshisha karem , tanava ghatane vali takanikom ko ajamaem | 
32. pratidina kama-se-kama 8-9 ghantem gahari ninda soyem | 
33. dopahara mem bhojana ke bada 1-2 ghante vishrama karem | 
34.jaba bhi apa svayam ko vyakula mahasusa kare taba sone se pahale snana kare taki ninda ane mem apako kathinai na ho | 
35. raktachapa ke lie pranayama labhadayi hai kyonki isase phephada़e khula jate haim aura tanava ghatata hai | 
36. raktachapa adhika ho to vashpa snana na karem | 
vaise bhi adhika samaya ke lie yaha na le, haphte mem do bara yaha 3-6 minata se thoda़e adhika samaya ke liye lena behatara hai | 
malisha yadi halki aura hridaya se duragami disha mem ki jae to labhadayi hoti hai | 
thanda ho to malisha dhupa mem karaieem | 
anushithilaka (dayastaॉlika) dabava agara 104 para kara jae to dila ka daura pada़ne ke asara dayastaॉlika echaji ke 85 emaema se niche hone ke halata ke mukabale dasa guna jyada hote haim | 
41. jihva aura namaka ka pyara sadiyom se chalata aya hai | 
jabana ko namakina svada ki itani adata ho chuki hai ki una donom ko alaga karane ka matalaba hoga jindagi ko namakahina bana dena , isa muhavare ka matalaba hota hai ki jine mem koi maज़a na rahana | 
43. jo adata ke karana adhika namaka khate haim, unaka ausata raktachapa namaka kama khane valom se jyada rahata hai | 
44. namaka ka eka anavashyaka ghataka sodiyama ki adhika matra uchcha raktachapa ka mukhya karana hai | 
45. atyadhika sodiyama sharira ko atyavashyaka potaishiyama lavana nishkasita karane para majabura karata hai | 
46. hama jaba adhika namaka khate haim taba gurde para dabava baढ़ta hai | 
47. namaka sharira mem pani banaye rakhane mem madada karata hai | 
48. aba yaha sapha ज़ahira ho gaya hai ki adhika namaka svasthya ke lie hanikaraka hai | 
49. namaka khana kama karane se raktachapa ghatega | 
50. yaha dhamaniyom ke divarom para asara karata hai aura unhem uddipana ke prati adhika samvedanashila banata hai | 
51. ama taura para ज़bana namakina padarthom ki adi hoti hai | 
52. unake bajaya hara dhaniya, pudina , karipatta tatha pyaja, lahasuna, muli ya phira halke masale jaise jira, dhaniya, haldi, rai adi ka prayoga karem | 
53. taiyara khadya padartha tatha achara , jisamem kaphi namaka hota hai, talana chahiye | 
prakriti se gahara samparka rakhane vale, dhimi jindagi jine vale gramina logom ko raktachapa ki samasya nahim hoti | 
55. prashanta dvipavasiyom para kiye gaye eka adhyayana mem yaha samane aya hai ki adhunika taura-tarike vale samaja ke nikata samparka mem aye huem logom ka raktachapa ve jisa samudaya se aye the vaham ke sadasyom se jyada hai | 
56. dhunrapana na sirpha raktachapa tatha hridaya gati badha़ata hai, balki yaha hridaya ko milanevale rakta ke darje aura pramana para bhi asara dalata hai | 
sigareta mem maujuda nikotina hridaya gati aura raktachapa badha़ata hai | 
58. dhuem ke satha shvasa ke andara jo karbana monoksaida liya jata hai vaha ऑksijana preshana ke mamale mem rakta para asara karata hai | 
59. yaha hridaya gati tatha rakta chapa ghatata hai | 
60. hridaya ko pravahita hone vali sahayaka rakta ki matra badha़ jati hai , isase hridaya ki peshivinyasa ko pahunchane vali pure rakta ka parimana bhi baढ़ jata hai | 
61. apake hridaya ko madada karane ke lie sigareta pina chhoda़kara vyayama karana shuru karem | 
62. kushala chikitsaka raktachapa jarura dekhata hai | 
63. koi marija़ agara uchcha raktachapa ka shikara hai to use niyamita rupa se raktachapa ki jancha karate rahane ki salaha deni chahiye | 
64. durbhagyavasha kai sare mamalom mem hridaya vikara hone ke bada, dila ka daura pada़ne ke bada ya nrityu hone ke bada bimari ka pata chalata hai | 
65. isamem samajadari, maitripurna prerana aura eka nanra chikitsaka ki avashyakata hai | 
66. yaha marija़ ko batana avashyaka hai ki use kya hua hai , rakta chapa se sharira ko hani pahunchane mem kitana vakta lagata hai | 
67. ज़yadatara mariज़ halke se lekara thoda़e uchcha rakta chapa se pida़ita hote haim| 
68. aba taka ke kisi bhi eka klinika ke prayasom mem lambe upayukta aushadhopachara se koi khasa phayade najara nahim ae haim | 
69. bajara mem ayi hui nayi davaiyom ko kuchha salom taka parakhane ke purva surakshita evam asarakara manana uchita nahim hoga | 
ye davaiyam raktachapa ko sirpha ghatati haim, para sharira mem honevali lagatara badha़ti hanikaraka prakriya ko rokane mem asaphala hai | 
ahara, vyayama tatha arama ki mahatvapurna bhumika aura dhunrapana na karane ke phayade ko samajana jaruri hai | 
upachara ka uddesha hai dila ke daure ke khatare ko kama karana | 
73. upachara raktachapa ki matra aura usake satha juda़e hue khatarom ke pahaluom para nirbhara hoga | 
74. agara raktachapa jyada ho to pahali jarurata hai use kama karana | 
75. raktachapa madhyama se thoda़a jyada hone ki halata mem aksara dhunrapana banda karana tatha santulita ahara shuru karana gajaba dha sakate haim | 
76. apa agara sthula ho to sharira ka vajana ghatane se raktachapa niche lane mem madada milegi | 
77. taiyara rahem ahara para niyantrana ke lambe saphara ke liye , kama charbi, kama namaka, kama chini kai prakara ke karbohayadretsa aura jyada reshomvali chijem khane ke liye| 
78. salada, hari sabjiyom tatha phalom ke rupa mem apane ahara mem reshom ko jyada khaem | 
79. vyayama se hridaya gati aura raktachapa ghatane mem madada hoti hai | 
80. khilada़iyom ki nada़i ama adami se dhimi chalati hai | 
81. raktachapa ghatane ke lie dhyana karem | 
82. parikshanom mem maluma hua hai ki dhyana ke samaya hridaya ka kama 25 pratishata kama hota hai, jabaki ninda mem hridaya ke kama mem 20 pratishata kami hoti hai | 
83. hridaya ki gati prati minata 5 dhadakanem kama hoti hai | 
84. anveshakom ko raktachapa kama hone ka sabuta mila hai | 
85. dhyana do taraha se madada karata hai - pahale yaha sharira aura mana ko shanta karata hai jo tanava mem jaruri hota hai | 
86. dusara phayada - bhale hi bahuta sapha na ho , yaha hai ki jinhem hridaya ki bimari hone ki sambhavana hai unhem dhyana se hitakari sharirika parinama se madada milati hai | 
87. raktachapa badha़ne ko kai sala lagate haim, aura ise kabu mem lane tatha prakriya ko palatane aura raktachapa se sharira ko hue nukasana ko sudharane mem kuchha mahine lagate haim | 
88. apani niyati ko apane hathom mem pakada़ne ke lie taiyara rahem | 
yaha hridaya ke sankochana ki shakti tatha gatimana, raktapravaha mem rakta ka parinama aura dhamaniyom ki prasarana kshamata para nirbhara hota hai | 
raktachapa mapane ke lie sphigmomainomitara ka prayoga hota hai | 
rakta ka pravaha shuru honevale dabava ko hritkunchana daba kahate haim , yaha hridaya ke sankuchana ke samaya ke rakta chapa ko dikhalata hai | 
isa chapa ko pare ki naliyom para bane gaॉja ya skela ko padha़kara nirdharita kiya jata hai | 
93. isa bindu ke chapa ko hritsphari (dayastolika) daba kahate hai, yaha hridaya ke shithila hone ke samaya ka chapa hota hai | 
94. yaha unra ke satha badha़ta hai kyonki dhamaniyam kama lachili ho jati hai aura rakta ke pravaha ko dhima kara deti hai | 
95. rakta manava sharira ki jivana dhara hota hai, yaha lala ranga ka tarala padartha kai kama karata hai aura isake bina sharira ka koi bhi anga jinda nahim raha sakata | 
rakta sharira ki koshikaom ko kama karane aura badha़ne ke lie avashyaka poshana aura pranavayu ki purti karata hai | 
97. yaha bekara dravyom ke khasa purjom ke pasa pahunchata hai jo ya to unhem sharira se bahara nikalate haim ya nirupadravi padarthom mem badalate hai | 
pratyeka vyakti ke akara aura vatavarana para usake rakta ka parinama nirbhara karata hai | 
hridaya sare sharira mem rakta prasarita karata hai | 
rakta hridaya se dhamaniyom ke jariye bahara ki tarapha bahata hai aura nasom ke jariyem hridaya mem vapisa ata hai | 
bada़i dhamaniyam jo hradaya se nikalati haim chhoti se chhoti vahikaom mem rakta le jati haim |
ina vahikaom mem se sabase chhoti sankari patali bhittiya keshikaem hoti haim |
rakta keshikaom se bada़i shiraom mem bahata hai jaba taka ki yaha sabase badi shira taka nahim pahunchata jo hradaya mem pravesha karati hai | 
chara mukhya bhaga haim plaja़ma (plavika), lala rakta koshikaem, shveta rakta koshikaem aura pletaletsa |
eka hastarekha vijnani hatha ko dekhata hai aura bhavishya ka purvanumana karata hai |
eka jyotishi janma-kundali padha़ta hai aura apake jivana ke bare mem saba kuchha batata hai |
eka chikitsaka bhi rakta chapa napa kara apake bhavishya svastha ka purvanumana lagane mem samartha hota hai |
vaha pushta karane mem sakshama hota hai ki apa kisi aghata ya kisi hradaya daure ke pratyashi haim ya nahim |
yadi apa jivana shaili parivartita karana nahim chahate haim to vaha jivana ki sannikata pratyasha bhavishyavani karane mem sakshama ho sakata hai |
purvanumana ko adhika uchita banane ke lie kuchha atirikta batom ko janana chahie |
uchcha rakta chapa ke satha-satha apako adhika khatare ke karaka haim, to sambhavana bahuta adhika hai ki apa jaldi hi hradaya daura ya aghata ka shikara honge | 
khatare karaka haim ayu, sigareta pina, rakta mem uchcha vasa stara (kolestraॉla aura glisaraidsa), madhumeha, sthulata aura eksa-re tatha ilektro-kardiyograma mem parivartana | 
isa karana se chikitsaka eka sampurna parikshana deta hai, kardiyograma leta hai aura rakta ko vibhinna parikshanom ke lie bhejata hai |
vo aghata aura hradaya daure se bachane ke lie khatare ke karakom aura rakta chapa ko ghatane ki salaha dete haim |
adhikatara loga apane bahuta bada़e nukasana para rakta chapa ko naja़raandaja़ kara dete haim, kyonki adhikatara mamalom mem yaha lakshana rahita hota hai | 
P ve isa laparavahi ke lie aghata ya nrityu ada kara sakate haim |
kuchha logom ko sira darda, parishrama karane para sansa phulana aura dhundhali drishti ho sakati hai |
yaham taka ki yadi apa ise naja़raandaja़ nahim karate haim to apa ina lakshanom se chhutakara pane ke lie goliyam lene ki ora pravritta hote haim, rakta chapa eka jivana bhara ki bimari hai aura eka bara mukhya karana ka pata chalane para usi prakara upachara kiya jana chahie |
sharira ke sabhi bhagom mem rakta apurti banae rakhane ke lie, hradaya ko eka nishchita dabava para rakta bhejana hota hai |
yaham taka ki eka svastha vyakti mem bhi, sharira ki vibhinna avashyakataom ko pura karane ke lie rakta chapa pratyeka kshana badalata rahata hai | 
jaba apa uthate haim, rakta chapa ko samanya stara taka uthane mem thoda़a samaya lagata hai |
dhire-dhire, jaise-jaise samaya bitata jata hai sharira uchcha rakta chapa ka tanava dikhana shuru kara deta hai, ho sakata hai 5varshom, 10 varshom ya yaham taka ki aura bada mem | 
yaha bina kisi chetavani ke aghata ya hradaya daure ke rupa mem akasmata hi hota hai |
jaba apa uchcha rakta chapa se pida़ita hote haim to hradaya ko aura kathina karya karana pada़ta hai | 
dirghakalika tanava ke antargata, dhamaniyam rakta apurti ghatate hue kada़ka aura moti ho jati haim |
akasmata hi, hradaya ko rakta apurti purnataya rokate hue yaha dhamani ko banda kara sakati hai|
jaba kama kiya hua aura thaka hua hradaya, poshana rahita hota hai, to avarodhom se rakta ko dhakelane ke lie use kathina kama karane ke lie badhya kiya jata hai | 
kuchha kiya jana chahie nahim to hradaya daura bahuta apekshita hai |
uchcha rakta chapa ke tanava ke antargata eka chhoti dhamani mastishka mem phata sakati hai aura eka aghata utpanna kara sakati hai |
uchcha rakta chapa ke tanava ke antargata, samaya ke satha vrikka phela ho sakate haim |
pairom mem etheroskleraॉsisa ke karana chalane mem kathinai ho sakati hai, lada़khada़a kara chalana kahate haim |
rakta chapa ne akasmata hi sarvanasha utpanna nahim kiya |
yaha uchcha rakta chapa aura rakta mem sancharita ho rahi vasaom ke dvara kai varshom mem kiya gaya tha|
isa kahani ka sabase bhayanaka anta hoga akasmata nrityu ya apanga bana dene vala eka aghata |
yadi hama dridha़ sankalpi haim, to hama yaham taka ki puri bimari ke samaya ko svastha, taruna aura utpadaka jivana ke aura adhika varshom ke rupa mem badala sakate haim |
rakta chapa bhavanaom aura parishrama ke satha badha़ta hai |
vaijnanika ke pasa pahali bara jane para, yaha samanya se uncha ho sakata hai kyonki apa chintita hote haim |
rakta chapa lie jane se pahale 10 – 15 ka arama lena hamesha hi achchha hota hai |
eka kapa kaॉphi ya pi gayi eka sigareta bhi daba anka badha़a sakati hai |
rakta chapa ajivana badha़ta rahata hai |
yaha navajata se shishuvavastha aura kishoravastha taka teja़i se badha़ta hai |
25 varshom se 40 varshom ke bicha isaka badha़na dhima hota hai |
mahilaom ka rakta chapa purushom se thoda़a ninna hota hai |
yaha neshanala insatityuta phreminghama, maiskyusetsa (yu. esa. e) dvara kie gae adhyayana ka parinama hai |
yadi koi uchcha rakta chapa se pida़ita hai to bhavishya ka purvanumana lagane ke lie khatare ke karaka avashyaka haim | 
eka kampyutara programa kampyutara mem khatare ke karakom ki janakari dala kara banaya jata hai aura atirikta mahatva janane ke lie do rogiyom ka itihasa dala gaya tha | 
eka anya sarvekshana mem, eka uchcha rakta chapa (pare ka 160 mimi sistolika aura pare ka 95 mimi dayastolika) vale vyakti aura eka samanya rakta chapa vale vyakti ke bicha tulana ki gayi thi |
eka uchcha rakta chapa vale vyakti ko hradaya daure ki 3 guna adhika apeksha aura aghata ki 7 guna adhika apeksha hoti hai | 
rakta chapa ko aushadhiyom dvara ghatane ki apeksha khatare ke karakom ko kama karana adhika mahatvapurna hai aura anya khatare ke karakom ko na dekhana hradaya daure ke khatare ko kama nahim karata hai | 
eka hala ke adhyayana ne yaha darshaya ki yadi apaka rakta chapa ninna hai aura anya khatare ke karakom se bhare hue haim to apako kevala rakta chapa ki apeksha bahuta adhika khatara hai |
yaha udghatita kiya ja chuka hai ki yadi apa kolestramaina jaisi aushadhi rakta vasaom ko ghatane ke lie lete haim to yaha khatare ke karaka ko nahim ghatata hai kyonki yaha vasa ko purna rupa se nahim ghatata hai aura hradaya ke lie achchhi vasaom (uchcha ghanatva kolestraॉla) aura kharaba vasaom (ninna ghanatva kolestraॉla) mem vibheda nahim karata hai |
kuchha nishchita aushadhiyom ke satha prayogom ne nakaratmaka parinama ya yaham taka ki badha़i hui nrityu sankhya bhi darshae haim | 
yadi apake abhibhavakom ko uchcha rakta chapa hai to apa uchcha rakta chapa ke bahuta hi anukula pratyashi haim |
lekina apako apake abhibhavakom ki hi taraha usi stara ka rakta chapa nahim bhi ho sakata hai |
koi bhi abhibhavakom ka chayana nahim kara sakata hai lekina vaha jivana shaili badala sakata hai, khatare ke karakom ko kama kara sakata hai aura eka adhika sima taka rakta chapa niyantrita kara sakata hai |
20% atirikta sharirika bhara vale logom ko uchcha rakta chapa hone ki 3 guna adhika apeksha hoti hai |
yadi apa atirikta vaja़ni haim to ghatate hue poshahara para rahana prathama charana hai aura uchcha rakta chapa hona |
shakahari poshahara reshe, manda, protina, vitamina aura khanijom ki matra mem sansahari khadya se bhinna hote haim |
kuchha anusandhana kartaom ne yaha salaha di hai ki agamanavadiyom ka atyadhika dharmika vishvasa bhi unako hradaya daure se bachane mem sahayata karata hai |
pichhale dashaka ke sabase mahatvapurna adhyayanom mem se eka vishvasa yaha hai ki sharira me kolestraॉla jitana ki mulatah samaja gaya tha usase bhi eka bahuta jatila padartha hai |
kolestraॉla rakta mem khula nahim balki laipoprotina namaka protina ke alaga-alaga sanyojana mem sancharita hota hai |
ninna ghanatva laipoprotina (ela di ela) aura uchcha ghanatva laipoprotina (echa di ela) ho sakati haim |
ela di ela ka uchcha stara hradaya rogom ke adhika khatare se sambandhita hai jabaki echa di ela ka uchcha stara kama khatare se sambandhita hai |
echa di ela ka nirgamana kolestraॉla ki matra ko napana jatila karata hai |
echa di ela sabhi unra ke samuhom mem mahilaom mem purushom se mahatvapurna rupa se adhika hota hai |
yaha spashta karata hai ki mahilaem hradaya rogom ke lie kama samvedanashila kyom haim |
echa di ela ki thoda़i matra mem upasthiti anuvanshika hoti hai aura vaja़na ghatane, vyayama, dhunrapana ke sankshipta virama aura tanavamukti ke satha badha़ti hai |
rakta mem kolestraॉla starom ko ghatane ke lie prayoga ki gayi aushadhiyam (kolestairamaina aura klophaibreta) achchhe (echa di ela) kolestraॉla se kharaba (ela di ela) mem vibheda kie bina hi pura kolestraॉla ko ghata deti haim |
sharirika sadhana prayoga karana achchha hai jaise ki vyayama aura vaja़na ghatana, vasa ko kama karana, dhunra pana banda karana aura uchita poshahara lena jo kharaba kolestraॉla (ela di ela) ko ghatata hai aura achchhe kolestraॉla (echa di ela) ko badha़ata hai apekshakrita pure kolestraॉla stara ko ghatane ke lie aushadhiyom para nirbhara rahane se jo hradaya daure ke khatare ko nahim ghatata hai |
yadyapi unhonne hradaya rogom se sambandhita bahuta se vishayom ko sammilita kiya hai, tathapi unhonne dhunrapana ke vishaya para lautana jari rakha |
unhonne eka dina pahale dhuem se bhare eka kamare mem vyavasayi ke satha eka mulakata ko yada kiya |
maine apana tarka sabhi ko bataya kyonki unhem mere pasa ana pada़ta tha daॉ. ephalara tada़ke |
sigareta pina hanikaraka hai |
hani rakta, rakta vahikaom aura hradaya ko hoti hai |
dhunrapana karane vale ki shava parikshaem yaha darshati hai ki parivartana kevala hradaya ko rakta apurti karane vali vrihada chakriya dhamani mem hi nahim hota hai balki sabase patali dhamaniyom mem bhi hota hai jo hradaya ki bhitti mem pravesha karati haim |
ina sabhi dhamaniyom ke antarika bhaga mem eka chhota aura phaila hua pragadha़ka hota hai jo avarodha utpanna karata hai aura ina dhamaniyom mem bahane vale rakta ki matra ghata deta hai |
apa pratidina jitani adhika sigareta piengem ye parivartana utane adhika sudridha़ honge |
ye parivartana sigara aura chilama pine valom mem thoda़e kama hote haim |
dhunrapana karane vale aghata, hradaya daure, enjina pektorisa aura lada़khada़a kara chalane ke prati samvedanashila hote haim jaham pairom ki rakta vahikaem sankari ho jati haim aura thoda़i dura chalane para pairom mem darda hota hai aura jaba aura age chalane se pahale vishrama karane ki avashyakata hoti hai | 
dhunrapana karane vale purushom ki apeksha dhunrapana karane vali mahilaom ko yaha samasya adhika hoti hai | 
dhunrapana na karane valom ki apeksha khatara 20 guna adhika hota hai |
adhika dhunrapana karane valom mem, jo kai varshom se sigareta pi rahe haim, akasmata nrityu ama hai |
yadi ve akasmata aura tivrata purvaka sigareta pina kama kara sakate haim, to khatara avashya hi ghata jata hai | 
jaba hama sigareta ka dhuam andara khinchate haim, to karbana mono ऑksaida rakta mem pahunchati hai aura himoglobina ke satha sanyukta hoti hai aura isaki ऑksijana vahana karane ki shakti pancha pratishata kama kara deti hai | 
dhunrapana do prakara se hama para akramana karata hai |
yaha rakta vahikaom ko sankara karata hai aura rakta ki ऑksijana vahana karane ki kshamata ghatata hai |
aba apa usa sarvanasha ki kalpana kara sakate haim jaba rakta kharaba gunavatta ka hai aura jaba yaha hradaya mem kama matra mem pahunchata hai |
yadi apa eka madhumehi haim, to hradaya roga ka khatara adhika hota hai |
yaha mahilaom mem purushom ki apeksha adhika chalanasara hai |
amadhumehi ki tulana mem atirikta khatara lagabhaga doguna hota hai |
ye rogi hradaya pratiropana ke lie bhi nahim ja sakate haim |
ina rogiyom ko pashchashalyakriyatmaka rupa se di gayi asankranyadamaniya aushadhiyam madhumeha ko natakiya rupa se apakarsha bana deti haim |
yadi apa vyayamom ke ichchhuka nahim haim aura apaka kama sthanabaddha hai, te adhika khatare ki apeksha hai |
yaha anya khatare ke karakom ki antahshakti ko badha़aega |
eka kathina parishrami kisana ya eka godi shramika ko hradaya daure ya akasmata nrityu ka khatara avashya hi kama hota hai | 
eka chakriya dhamani hradaya rogi mem, eka niyamita samanya vyayama ke karyakrama dvara vyayama ki sahana shakti badha़ai ja sakati hai |
unhonne salaha di, yadi apane 40 para kara liya hai, to kama se kama haphte mem eka bara kriya-kalapa karem | 
madhumeha ya rakta mem vasaom (kolestraॉla aura traiglisaraidsa) ka uchcha stara santanom mem hastantarita hota hai | 
samanyatah, hama kharaba khane ki adatem, akriyashilata, sthulata aura dhunrapana uttaradhikara mem prapta karate haim jo khatare ko badha़ate haim |
agrima mamalom mem hradaya dhamani upakaraniya shalyakriya ki jati hai jaham hradaya ki rakta vahika ke bimara bhaga ko sharira ke anya bhaga se li gayi rakta vahika se pratisthapita kiya jata hai |
yaha kabhi-kabhi hi kiya jata hai kyonki isa ऑpareshana ke bada rogi bahuta lamba jivita nahim rahata hai|
yadi apaki rakta vahikaom mem thaktha hai, to ise vilina karane ke lie aushadhi ka prabhavashali dhanga se upayoga kiya ja sakata hai|
upayoga ki jane vali aushadhiyom mem se eka hai heparina|
ye aushadhiyam nasika se raktastravana, aghata, jvara tatha pratyurjata jaise parshva prabhavom se mukta nahim haim|
kama atyavashyaka mamalom mem kaumarina ka upayoga kiya ja sakata hai|
yadi hridaya samanya rupa se kama nahim kara raha hai, to chikitsaka sankuchana badha़ane tatha hridaspandana dara ko kama karane karane ke lie dijitalisa ka nuskha batate haim|
isaka hridaya para eka taॉnika ki taraha kriya hoti hai|
isaki khurakom ki salaha dena kathina hai|
sabhi aushadhiyom ki taraha, dijitalisa ke bhi parshva prabhava hote haim: siradarda, bhukha marana, michali, ulti, dasta, peta darda|
hridshula ka glisarila trainaitreta jaisi aushadhiyom se ilaja kiya ja sakata hai|
yaha rakta vahikaom ko phailati hai tatha rakta ko adhika svatantratapurvaka bahane ki anumati deti hai|
yaha rakta chapa ko asthayi rupa se sudharati hai tatha thoda़e samaya mem hi pida़a ko kama ho jati hai|
isake siradarda tatha sampravaha jaise parshva prabhava hote haim, uchcha khuraka yadi koi apane pamva para khada़a raha to, chakkara ke avega utpanna kara sakate haim|
ye aushadhiyam haim propranaॉla,timolaॉla,metaprolaॉla
propranaॉla ka upayoga uchcha rakta chapa ke lie kiya jata hai, jo hridaya ko hridaya sankuchanom ki tivrata ko dhima karane tatha ghatane mem sahayata karati hai|
ye una logom ko nahim die jate haim jinhem pahale hi hridpata ho chuka ho|
yaha garbhavati, damarogi tatha madhumeha ke rogiyom ko nahim diya jana chahie|
shvasahinata, takhanom para sujana adi parshva prabhava haim|
yadi hridaya rogagrasta ho to ye aushadhidravya avashyaka ho sakate haim|
ahara rakta mem charbi tatha kshati jo pahale hi ho chuki hai ko ghatane mem sahayata kara sakata hai|
rakta chapa ko niyantrita kijie, vyayama karana prarambha kijie tatha dhunrapana karana banda kijie|
yaha hridaya ko jivanakshamata punah prapta karane mem sahayata karegi|
asana rasta mata dhundhie, apa nirbhranta tatha bahakave mem rahenge|
aushadhi apako rahata de sakati hai|
yaha raktachapa niyantrita kara sakati hai, kolestraॉla stara ghata sakati hai|
yaha hridspanda ke avartana ko niyantrita kara sakati hai kintu yaha hridayaghata ke sankata ko nahim ghataegi|
asanna dila ke daure se svayam ko bachane ke lie, apako apani jivana shaili mem parivartana lana hoga, ahariya svabhavom ko parivartita karana hoga, kama se adhika chhutti leni hogi, arama karana sikhana hoga, vyayamom, khelom tatha lambi saira lene mem ruchi utpanna karani hogi|
sharaba, atyadhika chaya aura kaॉpha़i pina banda kara dijie|
yadi apa kevala aushadhiyom para nirbhara kara rahe haim, to apa chaku ki dhara para paga rakha rahe haim|
yadi apa dila ke daure ka anubhava karate haim, to shanta rahem tatha bistara para leta jaem|
chikitsaka ke ane, achchhi taraha se parikshana karane tatha apake kardiyograma lene taka gatividhi na karem|
yaha hamesha behatara hai ki apa kisi achchhe aspatala ke hridiya vibhaga mem bharti ho jaem|
yadi ghara para aspataliya suvidhaom ki vyavastha ho sakati hai, to ghara para upachara karana thika hai|
prayah, aspatala mem jane mem jo shrama hota hai vaha hridaya ko aura adhika kshati pahuncha sakata hai|
isa avastha mem pida़a kama karane tatha hridaya ke avartana ko niyantrita karane ke lie audhashi atyavashyaka hai|
aghata ko kama karane ke lie antarshira dripom ki avashyakata hongi|
antarshira aushadhi upachara ke lie hridaya ki lagatara jancha avashya karani chahie|
prathama chhiyanabe ghante atyanta mahattvapurna hote haim|
lagabhaga 90 pratishata rogi isa kalavadhi mem ventrikulara ektopi kahalane vali atalata se pida़ita rahate haim|
isa samaya para uchita chikitsiya dekhabhala mahattvapurna hota hai|
shanta rahem, kshubdha ya uttejita na hom|
khusha mija़aja bane rahem|
hridpeshiya rodhagalana ke adhikatara jatilatarahita mamalom mem atidakshata vibhaga se rogikaksha ke vibhaga ya niji kaksha mem sthanantarita kiya jata hai jaham hridaya ki jancha kuchha adhika dinom ke lie spandana dara tatha avartana ke lie phira bhi ki jaegi| 
ina rogiyom ko eka dina mem thoda़e antarala ke lie kuchha bara kursi mem baithane ki anumati di jati hai|
4 saptaha mem maithunika gatividhiyom ke punararambha tatha halke-phulke vyayama karane ki anumati di jati hai|
unhem dhire-dhire karyakshama banaya jata hai tatha 6-8 saptaha mem kama para vapasa jane ki anumati di jati hai|
vaha rakta ki kami tatha tat pashchata ऑksijana ki kami se pida़ita rahega|
mastishka mem dhamani ke avarodha se daura pada़ sakata hai|
jaba hridaya ki svayam ki rakta apurti prabhavita hoti hai to yaha enjaina pektorisa kahalane vali hridaya pida़a utpanna kara sakati hai|
jaba shrama dvara hridaya para dabava dala jata hai to ise adhika rakta ki avashyakata hoti hai, kintu sankirnita dhamaniyom ke karana ise yaha nahim mila sakata hai aura ise hridaya pida़a ke rupa mem abhivyakta karata hai|
dila ka daura ya hridpeshiya rodhagalana taba hoga jaba rakta ke thakke ya medarbuda ke hrid-dhamani, vaha dhamani jo hridaya mem rakta apurti karati hai, mem jama hone se rakta apurti achanaka purnatah banda ho jae|
ethirosklerosisa bachapana se prarambha hokara varshom taka dhire-dhire vikasita hota hai|
jaba rakta vahikaom ki antarika parata vikrita hona prarambha karati hai ya ghayala hone lagati haim to ye charbi vahikaom mem ghusapaitha karane lagati haim|
isa charbi mem kolestraॉla, traiglisaraida tatha phaॉspholipida hote haim|
yahi karana hai ki sabhi prakara ke hridaya rogom mem rakta mem ina sabhi charbiyom ke starom ka mapana karana mahattvapurna ho jata hai|
sucharu rupa se kama karane vali mansapeshiya koshikaem dhamaniya divarom ke mukhya anga haim|
protina ke satha lippoprotina kahalane vali yaha parisanchari charbi vahika ke vikrita antarika parata mem pravesha karati hai aura jama ho jati hai|
sucharu rupa se kama karane vali mansapeshiya koshikaem tatha sanyoji utaka isa charbi aura protina jamava ke charom ora phailane lagate haim|
yaha charbi tatha protina chayapachaya se hokara gujarate haim aura achchhe ke lie vahikaom ki divarom para jama ho jate haim|
ye rakta vahikaom ke andara mem dhera ki unnata golika banate haim|
isa unnata golika ko plaka nama diya gaya hai|
vaham para vikasita kshata mem kailsikarana hota hai|
isa plaka ke bhitara chhoti pragatishila rakta vahikaom se rakta srava bhi hota hai|
dhira-dhire plaka ka akara badha़ta jata hai aura yaha adhika thosa ho jata hai|
ina vahikaom mem bahane vale rakta ko eka yantra jo rakta chapa leta hai se mapa ja sakata hai|
dabava ke satha bahane vala rakta bhi charbi jamava ya plaka ke unnata dherom ki sataha ko katata hai|
plaka ki sataha khuradara ho jata hai tatha pattakom ke phansane ke lie eka svabhavika sthana banata hai tatha sangraha shighrata se ho sakata hai|
yaha plaka ko akara badha़ane mem sahayata karata hai|
ye plaka kitani tabahi kara sakate haim yaha unake sthana para nirbhara karata hai|
yadi ve janghom mem haim to ve jyada kha़taranaka nahim haim|
kintu yadi ve hridaya ya mastishka mem haim to ve mara ya pangu kara sakate haim|
plaka banane tatha gati jisase ve akara mem vriddhi kara sakate haim mem kai karaka bhumika nibhate haim|
madhumeha, motapa, dhunrapana, ahara, tanava mahatvapurna haim|
yadi hama khatare ke karakom ko jana lem, to unaki kami hridaya rogom se bache rahane mem sahayata karegi|
raktachapa ke satha, hridaya roga ho jane ya daura pada़ne ka khatara aura adhika hota hai|
yadi uchcha raktachapa 100 anushithilaka se adhika hai, to chikitsiya hastakshepa nyayochita hai|
hridaya roga ke bija balyavastha ke prarambha mem hi bo die jate haim|
yadi apa balyavastha mem hi khatare ka karaka kama karem to hridaya roga ko ja़bardasta rupa se ghataya ja sakata hai|
yadi bachcha sthulakaya hai, to use vyayama karane tatha samanya karbohaidreta khane ke lie protsahita kijie|
eka adhyayana pramanita karata hai ki sharirika vajana mem 15 pratishata ghatava rakta kolestraॉla mem 4 pratishata ki kami, kie gae charama karya mem 7 pratishata sudhara tatha kie gae kula karya mem 16 pratishata sudhara kara degi|
parivara ke sadasya jo asamayika roga hone ke eka itihasa ke lie jane jate haim ve daura ya uchcha raktachapa ke lie upayukta vyakti haim|
unake bachche bhi koronari hridaya roga ke namune dikha sakate haim|
yunivarsiti ऑpha sinasinaiti ऑpha medisina, tusaॉna, yu.esa.e. ke daॉ. charlsa je. glueka ne ethirosklerosisa ki utpatti mem nishchayatmaka rupa se pramanita kiya hai ki hridayaghatom aura daurom ke lie sthiti balyavastha mem hi prarambha ho jati hai|
jisa unra mem ethirosklerosisa sabase adhika prativarti hota hai vaha nishchita rupa se madhya se lekara dera kishoravastha mem hota hai| 
bachche jo adhirata, samayagraha tatha pratiyogitatmakata (taipa e kahalane vale) pradarshita karate haim unamem dila ka daura aura adhika hone ki sambhavana hai banisbata unamem jo adhika aramapasanda (taipa bi kahalane vale) hote haim|
ye lakshana balavihara ke samaya se hi dekhe gae haim|
eka manashchikitsa ke pradhyapaka ka dava hai, ki hamanem pancha sala ke bachchom ko dekha hai jo svayam se adhira haim ya hamesha shighrata mem rahate haim|
bachchom para prayah apane samaya ka labhakari dhanga se upayoga karane ke lie davaba dala jata hai|
bachche suniyojita khelom, sangita prashikshanom, nritya prashikshanom, griha-karya se bandhe hue haim; ve prayah hamamem se adhakatarom se adhika vyasta rahate haim|
bachche, mata-pita dvara jo unake lie sarvashreshtha chahate haim, uttarottara uchchatara lakshya nishchita karane ke lie protsahita kie jate haim|
kuchha parivara shirsha karya se kama kuchha bhi ko chupachapa asvikara karake hi, utkrishta manadandom ka nirmana karate haim| 
isaka yaha abhipraya nahim hai ki eka bachcha jo samuha ke lie parishrama se padha़ta hai ya utsahapurvaka abhyasa karata hai, usako hatotsahita karana chahie|
kathina parishrama tatha prabala karmashakti mem antara karana mahattvapurna hai|
sankshipta mem, samasya taba utpanna hoti hai jaba taipa-e vyavahara ko charama taka le jaya jata hai, prakata rupa se bachche ki sahayata karane ke lie eka achchha udaharana sthapita kijie|
vayaskom ko apane svayam ke taipa-e vyavahara se sacheta rahana chahie jise unake bachche anusarana kara sakate haim| 
yadi pita vishesha rupa se sammelana se sammelana mem jaldi-jaldi jate haim, taba usake bachche ke lie yaha sikhana kathina hoga ki kaise shanta rahe|
vivashata ke bodha se bachane ko lie, bachchom ko unase sambaddha vastuom ke bare mem nirnaya karane mem sammilita karana chahie|
yaha eka bachche ke lie adhika santvanadayaka hoga ki use pariksha mem mile anka ke bare mem puchhatachha karane ki bajaya usase puchha jaya ki usaki samasya asana ho gai hai|
unaki tulana dusarom se karane ki bajaya bachchom ko apana sarvottama karane ke lie protsahita karem|
koronari hridaya roga 50 ke bhitara ki mahilaom mem purushom ki apeksha kama samanya haim|
pashchima mem dhunrapana ke prachalana ke satha, anupata tina se eka taka niche a gaya hai | 
mahilaem purushom ke kama karana chahati thim, sabhi ja़immedariyam aura purushom ke ananda prapta karana chahati thim, isilie unakom isaka mulya ada karana hoga|
50 ki ayu ke bada jo rajonivritti se bhi mela khati hai, bimariyom ki upasthiti purushom ki bimariyom ke lagabhaga samana hi hoti hai | 
isako isa prakara bhi bataya ja sakata hai, ki 50 ki ayu ke bada khatara mahilaom mem badha़ jata hai aura purushom mem ghata jata hai |
yaha isa tathya ke karana hai ki uchcha khatare vale purusha 50 ki ayu ke pahale hi nrityu ke dvara mare jate haim ya unhem pahale hi unaka prathama hridaya daura pada़ chuka hota hai | 
yadi yaha samanya rakta chapa vale vyakti ki tulana mem 160 prakunchaka aura 100 anushithilaka ke barabara uncha hai, to dhyana rakhem |
yadi apa paintalisa para kara chuke haim aura rakta chapa hai, to apa hridaya daure ya aghata ke eka asana lakshya haim |
rakta chapa ko aushadhiyom ki sahayata se niyantrita rakhana asana hai lekina rakta vahikaom mem atharmainasa* (ushma sancharana na hona) parivartanom ko niyantrita karana nahim |
yadi apa rakta chapa ko ninna vasa poshahara, vyayama aura tanavamukti se niyantrita rakha sakate haim, to apa khatare ko avashya hi ghata sakate haim |
phira bhi yada rakhie ki achchha prabhava kevala 3 varshom bada hi shuru hota hai |
pachasavem dashaka ke madhya mem nyu yaॉrka tainsa mem eka lekha aya tha jisamem kaha gaya tha: jantu vasa hridaya rogom se juda़i hui hai, mansa, dudha aura andom mem paya jane vala kolestraॉla ethirosklerosisa se juda़a hai | 
1960 taka kolestraॉla siddhanta – yaha vichara ki khatare ka mukhya karaka unnata sancharita vasa kolestraॉla traiglisaraidsa hai, chikitsiya visheshajnom ke bicha vyapaka rupa se svikrita ho gaya |
yaha dhyana dena mahatvapurna hai ki krishi samaja ki tulana mem shahari aura audyogika jana samudaya mem hridaya ke daurom ka eka vyapaka unnata stara hai |
eka abhramanashila jana samudaya ke dvara santripta vasa, parishkrita sharkara aura purna urja ka adhika grahana nukasanadayaka hai |
janavari 1984 mem, yu.esa. mem eka hridaya, phephada़e aura rakta sansthana dvara 10 varshom ke anusandhana ke bada, hridaya para eka vivarana ne spashtatah pradarshita kiya ki hridaya roga rakta mem uchcha kolestraॉla stara se sambaddha haim |
unhonne paramarsha diya ki kolestraॉla kama karane vala poshahara hridaya rogom ko rokane ki mukhya kunji hai aura pratyeka vyakti dvara angikrita kiya jana chahie |
eka prasiddha hridaya anusandhanakarta , daॉ. jeremiya stamara ne kahah sanriddha poshahara ko pashchimi audyogikrita deshom mem teja़i se phailati hui akala ethirosklerosisa sambandhi bimariyom ki vartamana mahamari ke karana ke rupa mem dikhana yukti sangata hai |
kuchha hridaya anusandhanakarta yaha vichara rakhate haim ki santripta vasa - (jantu vasa jaise ki mansa, dudha aura andom mem) ke upayoga ko kama karana avashyaka hai | 
asantripta vasa (vanaspati tela jaise ki suryamukhi, soyabina ya makke ka tela) ka prabhava bahuta khataranaka nahim hai |
isaka matalaba hai ki yadi khana vanaspati tela (ghi ya makkhana ke sthana para) mem pakaya jata hai to apa ethirosklerosisa ke prati kama samvedanashila hote haim |
khatara yaha hai ki adhikatara loga manate haim ki asantripta vasa ka upayoga svasthyaprada hai |
hridaya rogom ko kama karane para kiya gaya narve ka eka adhyayana isa nishkarsha para pahuncha ki hridaya rogom ko kama karane ka sabase achchha tarika hai kisi bhi prakara ki sabhi vasaom ko ghata dena |
tale-bhune khadya se bachem aura yadi apa alu katale khane ki ja़idda karate haim to sidhe katem aura inhem mota katem |
yadi ve maroda़dara kate haim ya patale kate haim to ve adhika tela sokhenge kyonki unaka satahi kshetraphala badha़ jata hai |
apane gundhe hue ate ya ghola mem tela nahim dalem kyonki jaba talate haim; to ye mulayama honge lekina adhika tela sokhenge |
unake vasa ka upayoga, aura parinamasvarupa, dila ke daurom ki purna sankhya gira rahi hai |
saukada़om-hajarom logom ne santripta vasa aura kolestraॉla mem kama, eka shakahari poshahara angikrita kara liya hai |
yadi hridaya mashina rukati hai, ऑksijana kata jati hai aura vyakti mara jaega jaba taka ki hridaya ko udipta karane ke lie eka vishishta kritrima upakarana ka upayoga nahim kiya jata hai |
hridaya ka pratyeka bhaga eka bhinna prakara ke pampa chalana ka kama karata hai |
hridaya ke dahini ora pravesha karane vale rakta mem eka avashesha utpada, karbana dai ऑksaida hota hai |
hridaya ke dahini ora pravesha karane vala pura rakta hridaya ke banyi ora pahunchane se pahale phephada़om mem jata hai |
phephada़om mem karbana dai ऑksaida nikala diya jata hai aura rakta mem ऑksijana mila di jati hai |
rakta mem, jo hridaya ke banyi ora se bahata hai, taja़i ऑksijana hoti hai | 
ऑksijana sharira ki koshikaom mem urja utpanna karane ke lie upayukta hoti hai |
bina ऑksijana ke sharira ki koshikaem kama karana banda kara deti haim |
eka vayaska hridaya samanyatah eka minata mem lagabhaga 70 bara dhada़kata hai, lekina jitani adhika ya jitani kama ऑksijana sharira ko chahie utani pradana karane ke lie dara svatah hi parivartita ho jati hai |
sharira ko tairaki ya aisa sketinga jaise thakanabhare vyayama ke lie urja ka eka pulinda chahie |
isa karana se, jaba koi vyakti tairata ya sketinga karata hai taba hridaya bahuta teja़i se dhada़kata hai |
rakta ka pravaha badha़akara yaha sharira ko adhika ऑksijana pahuncha raha hai |
yadi vyakti vyayama karana roka deta hai, to hridaya dhire-dhire manda ho jata hai |
jaba koi vyakti gussa, dara hua ya uttejita hota hai taba hridaya teja़i se dhada़kata hai |
yaha vyakti ko lada़ne ya dura bhaga jane ki taiyari karane ke lie mansa-peshiyom ko adhika ऑksijana pahuncha raha hai |
ethirosklerosisa eka sthiti hai jaba vasiya padartha aura kailshiyama ke jamava dhamaniyom ko sankara kara dete haim aura unake samanya rupa se chikane astara ko khuradura bana dete haim |
phati hui koshikaom ke padartha kshata chihna utpanna karate hue pasa ke utakom mem jalana paida karate haim aura dhamani ki bhitti kada़ka, khuraduri aura sankari ho jati hai |
hridaya dhamaniyam jo hridaya mansa-peshiyom ko svayam hi poshita karati haim anya dhamaniyom ki tulana mem adhika bara prabhavita hoti haim |
kada़kapana hridaya ki rakta apurti ko ghata deta hai aura eka darda (enjina pektorisa) utpanna kara sakata hai |
bhitti ki khuraduri sataha sankari naliyom se rakta ke ruka-ruka kara pravaha ke satha-satha eka thakka, thraॉmbasa banane ke lie prerita kara sakata hai | 
eka thraॉmbasa ya embolasa dhamani ko puri taraha se avaruddha kara sakata hai |
mastishka mem eka avaruddha dhamani aghata utpanna kara sakati hai |
kisi hridaya dhamani mem avaruddhata hridaya ka daura utpanna kara sakata hai |
atyadhika tanava rakta ke pravaha mem badha़i hui pratirodhakata ke karana utpanna hota hai | 
amavatika jvara yuvaom ki eka bimari hai, lekina hamesha nahim aura jivanu dvara utpanna hota hai jo rakta vishaktata aura skarleta jvara utpanna karata hai |
janmajata hridaya roga janma ke samaya upasthita eka bimari hai |
eka nile bachche mem janmajata hridaya ki gada़bada़i hoti hai |
sankramana ki ora pravritta rogiyom ko shalyakriya ke pahale aura bada mem prati jaivika ki paryapta khuraka dekara jivanvika antarhridshotha ko roka ja sakata hai | 
phibrileshana eka sthiti hai jisamem hridaya ke mansa-peshiya tantu bina tala-mela ke eka aniyamita laya mem kama karate haim |
abhi bhi jenetika injiyaringa ko upayoga mem lane aura ise samanya adami ke lie upalabdha karane mem eka lamba rasta taya karana baki hai |
hridaya ka daura hamesha achanaka pada़ta hai, yaha taba ata hai jaba hama isaki sabase kama apeksha karate haim |
yaha achanaka a sakata hai jaba hama kama mem kriyatmaka rupa se vyasta hote haim, ya upalabdhi aura sampannata ke charama mem hote haim aura vastava mem jaba apane chalisa ke dashaka mem achchhe svasthya ka ananda le rahe hote haim taba bhi|
taba achanaka hi sabhi chija़ andhakaramaya ho jati haim, dukha aura kabhi na samapta hone vala sannata hota hai |
bahuta se pichhe chhoda़ jate haim chhote-chhote badha़te hue bachche jo padha़ rahe haim, apane age ane vale bahuta se varshom ke satha eka patni, ya eka chura-chura ho gaya vyavasaya ho sakata hai |
kuchha janmajata hridaya rogom ko chhoda़kara hridaya rogom ki ja़bardasta bahusankhya anuvanshika ya kisi krura prakriti dvara hama para thope hue nahim hote haim |
yaha sanriddhi dvara poshita aura tanava dvara janmi adhunika samaya ki sabase samanya aura sadama pahunchane vali bimari ho chuki hai |
agrani hridaya roga visheshajna daॉ. bi. ke. goyala ne mumbai mem eka seminara mem khulasa kiya ki hridaya rogom ki ghatanaem badha़ rahim thim aura vaha bharata mem, usa dina hridaya rogiyom ki sankhya 35 lakha thi | 
dusari ora aisi bimariyom ki ghatanaem pichhale 10 varshom mem yunaiteda stetsa mem ghata rahi haim kyonki ise kai morchom se sambhala ja raha tha |
pashchimi duniya mem, jo hridaya sambandhi bimariyom ke nakaratmaka prabhavom ko bhugata chuka hai, vaham khae jane vale bhojana aura angikrita ki jane vali jivanashaili sambandhita eka badha़ti hui jagarukata hai | 
eka alpaposhita vyakti asvasthyakara sthitiyom mem raha sakata hai aura tapedika ka shikara ho sakata hai |
eka dhani, suposhita, tanavayukta svachchha parivesha mem raha raha vyakti eka asvastha hridaya ka shikara ho sakata hai |
hridaya ki bimariyam prati varsha badha़ rahi haim kyonki jivanashaili aura adhunika jivana ki gati aura teja़ ho rahi hai |
vaha hai yadi hama una tathyom ko samaja sakem jo hridaya ki bimari mem sahayoga de rahe haim aura apani jivana shaili ko parivartita kara sakem |
hridaya dhamani ki bimari koi nayi bimari nahim hai |
eka shaktishali roma ke sineta sadasya sineta ke samane eka prabhavashali bhashana dete samaya acheta ho gae aura sabha kaksha chhoda़ sakane se pahale hi mara gae |
isake lie unake hridaya ki viphalata, eka hridaya dhamani ki kharabi ko uttaradayi thaharaya gaya | 
jaise-jaise hama badha़te jate haim aura varsha bitate jate haim dhamaniyom ki antarika astara aura bhittiyom mem kuchha parivartana hote haim |
rakta vahikaem jo hridaya se rakta le jati haim rakta vahikaom ki antarika astara mem apakshaya se pida़ita hoti haim aura jaise-jaise samaya kramika rupa se bitata jata hai, spashta rupa se yaha marga ko sankara karata hai, halanki arambha mem sharira para isaka koi bura prabhava nahim hota hai|
arambha mem eka phala rasa poshahara achchha hai |
phala rasa phruktoja़ aura poteshiyama ka eka shrota hai aura hridaya ko svastha hone mem sahayata karega |
pani mem taja़e nimbu ke satha shahada ki thoda़i si matra upayogi hogi |
12 ghante bhigi hui aura phira pani mem pisi aura chhani hui kishamisha ki sipharisha ki jati hai |
samaya-samaya para sada pani avashya dena chahie |
rasa ya pani thoda़i matra mem aura barambara dem |
rogi ko malatyaga karate samaya pura ja़ora na lagane ke lie kaha jata hai |
pahale 3-4 dinom taka rogi ko shaiyya malapatra diya jata hai, bada mem vaha surakshita rupa se eka shaiyya-parshva kamoda ka upayoga kara sakata hai |
phala, supa, ubali sabja़iyam aura sampurna gehum ki roti ki sipharisha ki jati hai |
sabja़iyam bina namaka ke ubali jani chahie |
madhyanha bhoja aura ratri bhoja ke satha thoda़a dahi liya ja sakata hai |
yadi apa isa pathyapathya niyama ko banae rakhate haim, to svasthya labha teja़ hoga aura apa bhavishya ke karya bina kisi atirikta hridaya daure ke, samanya sharirika kriya ke satha kara sakate haim, aura vyayama, poshahara, tanavamukti ki sahayata se eka dirghayu ka ananda le sakate haim lekina dhunrapana, madyapana, uchcha kailori bhojana aura tanava jaise sabhi khataranaka karakom se bachate hue |
vyayama ki salaha di jati hai, yaha na to bahuta kama hona chahie na hi bahuta adhika |
adhikatama hridaya kshamata hridaya roga visheshajna dvara napi jati hai aura apako visheshajna ki dekha-rekha ke antargata vyayama avashya karana chahie |
15 minata ke chhote dhime bhramana se shuru karem aura dhire-dhire unhem 45 minata taka badha़aem |
jaise-jaise hridaya shakti prapta karata hai apa apane bhramana ko teja़ kara sakate haim |
eka varsha ke samaya mem shayada apa apani unra ke kisi bhi samanya vyakti ki taraha dhire-dhire dauda़ ya thakana bhara vyayama kara rahe honge aura vo tanava lene mem sakshama ho jaenge jise apa pahale lene mem akshama the | 
vyayama hridaya ki ora rakta ka parisancharana badha़aega |
yaha apako tanavamukta karega aura kolestraॉla stara niche laega |
ninna kailori, ninna vasa aura uchcha reshedara bhojana lem |
subaha ke nashte ke lie phala, gaya ka dudha ya ninna vasa dudha lem |
sharkara se bachem, sharkara utani hi kharaba hai jitana tela auऱ ghi, bina reshe ka eka uchcha kailori bhojana hone ke nate, yaha kolestraॉla badha़ata hai |
apa sabhi prakara ke phala le sakate haim |
nimbu-vansha ke phala aura seba achchhe hote haim |
bhinna-bhinna dinom mem bhinna-bhinna phala lem |

koi bhi phala khana achchha hota hai|
phalom ke rasa ki bajaya phala ko prathamikata dijie |
ye reshom evam vitaminom ke bharapura srota hote haim|
agara apa lena chahate haim to ubali sabjiyam,salada, gehum ke ate ki bina chupada़i roti evanthoda़a dahi ya dala lijie|
hari pattedara sabjiyam jaise, paravala, sapheda kumada़a, tinda़a, turai, patta gobhi, palaka, tamatara, phransa sema, phula gobhi, bhindi, alu adi ko prathamikata dijie|
salada mem, salada patta, patta gobhi, khira, gajara, tamatara, chukandara, pyaja adi ko shamila kijie|
agara salada vayu paida karati hai to atyadhika matra mem na lem|
dopahara ya ratri bhojana ke pashchata mishthanna khane ki bajaya, angura ya seba jaise mithe phala khaiye|
dopahara mem eka gilasa rasa ya nariyala pani ya sabjiyom ka rasa lene ki salaha di jati hai|
shama ko sabjiyom ka supa, ubali sabjiyam, gehum ki do rotiyam aura phala lijie|
tala bhojana, mansa, mithai, chaॉkaleta, aisakrima, thanda़e peya padartha, sharaba evammasalom jaise pachane mem mushkila khadya padarthom se bachie|
bhojana supachya hona chahie|
bicha mem, jaba kabhi kuchha khane ki ichchha ho to, eka phala khaie ya eka gilasa rasa pijie|
eka hi bara bahuta sara pani pine ki bajaya, thoda़i matra mem evam bara bara pani pijie|
pratidina, kama se kama 1ya 2 litara pani pijie|
pratah kala sabase pahale, eka gilasa gunagune pani mem nimbu aura eka chammacha shuddha shahada dalakara pina eka achchhi adata hai|
peta sapha karane mem yaha, apaki madada karega|
bhojana ke satha pani pina chhodie|
dopahara ke bhojana ke 1/2 ghante purva ya eka ghanta bada pani pina behatara hoga|
kyonki yaha pachaka rasom ko patala kara deta hai evam asali pachana kriya mem ada़chana dalata hai|
achchhi ninda dilane mem yaha apaki madada karega|
kama kailori vale bhojana, kaॉlestrola ki matra kama karate haim evam hridaya ke tanava ko kama karate haim|
dhunrapana mata kijie ya tambaku mata chabaie|
nikotina hridaya dhada़kana ko badha़ata hai aura hridaya ke lie hanikaraka hota hai|
dina ke prarambhika ghantom mem surya tapana lijie , subaha ke 8 baje se purva ka samaya behatara hoga|
dina mem lagabhaga eka ghante surya tapana lena sukhada hota hai|
jala upachara bhi mahatvapurna taura se sahayaka ho sakata hai|
atyadhika garma jala ka upachara na lem jaise ki vashpa-snana ya bhapa snana mem lete haim|
prarambhika taura para, dina mem do bara sukhe kapada़e se sharira ko ponchhana achchha rahega|
bada mem, pairom para eka gila kapada़a lapeta lijie|
yaha tanava ko kama karane mem evam ninda lane mem sahayaka hota hai|
dopahara mem ya rata ko sone se purva isako 45 minata taka kijie |
jaba apa chala phira rahe haim to usake badale, peta para bhi gila kapada़a lapeta ja sakata hai|
rata ko sone se purva naha lijie|
yaha tanava ko kama karega evam ninda lane mem sahayaka hoga|
badha़te hue tapamana se pairom ki sikai karana, chhati ke darda ko kama karane mem sahayaka hoga|
tapamana ko badalate hue hathom ki sikai kijie|
yaha hridaya evam shvasana kriya ko pushta banata hai|
chintaom se bachie, jimmedariyom ko apasa mem banta lijie|
bahasom mem mata pada़ie ya uttejita mata hoie|
apane charom ora khushanuma vatavarana rakhie|
hara bara 15 minata ke hisaba se, dina mem do bara, shavasana kijie |
tanava mukta hone ki anya takanikom ko bhi sikhie evam bara bara unhem prayoga mem laie|
dila ke marijom ke lie dhyana lagana labhadayaka hota hai|
pranayama bhi labhadayaka hota hai|
apaki jivana shaili se dila ka daura pada़ sakata hai, ise badalie aura hamesha ke lie dusara dila ka daura pada़ne se bachie|
pittashaya nashapati ke akara ki, lagabhaga eka golpha ki genda jitani eka thaili hoti hai jo yakrita ki dahini ora, niche ki tarapha paim jati hai|
isaka mukhya uddeshya, pitta ko ekatrita kara gadha़a karana hai, jo ki hamare khane vale bhojana mem yukta vasa ko pachane vala eka karaka hai|
pitta, yakrita mem banane vala evam pittashaya mem asthayi rupa se ekatrita hone vala,eka pila sunahara sapha drava hota hai|
jaba hama bhojana vasayukta kara lete haim, taba pittashaya pitta ko sravita karata hai jo nalikaom (jo sistika aura samanya pitta nalikaem kahalatim haim) dvara hota hua chhoti antom taka jata hai, jaham bhojana ka pachana hota hai|
bahuta se sambhava karaka haim jo manushya mem, pittashaya ki samasyaom ke utpanna hone ko, prabhavita karate haim|
yaha mana gaya hai ki ya to yaha, isase sambandhita hai ki hamara sharira bhojana mem yukta kaॉlestraॉla ko kaise upayoga mem lata hai, ya phira yaha pittashaya ke sankramana se sambandhita hai|
pittashaya ki samasyaem, samanyatah pittapathariyom ke hone se utpanna hoti haim|
pittashaya ke andara jaba pitta aura anya dravya rasayanom ka santulana gada़bada़a jata hai, taba pittapathariyam banati haim|
parinamasvarupa, thosa rupa mem badalata hua eka rasayanika mishrana ya talachhata (patthara ya pathari) banata hai|
samaya ke satha, ina talachhatom mem se kuchha, akara mem baढ़ne lagate haim aura pittapathariyam bana jati haim|
agara pittapathari bada़i hai aura sistika nalika mem a jati hai, to yaha pittashaya se pitta ke pravaha ko avaruddha kara sakati hai|
apane avaruddha nirgama dvara ke karana, pittashaya phulana arambha kara deta hai|
jaba pittashaya pitta se puri taraha bhara jata hai, to pitta ka bahava vapasa pichhe bhi ho sakata hai|
parinamasvarupa, pittashaya sankramita ho sakata hai|
lakshanom mem, peta ke dahini ora ya madhya mem tivra darda shamila hai, jo samanyatah bhojana ke pashchata, vishesha rupa se tela mem tali vastuem, sabjiyam jaise tailiya bhojana ke pashchata utpanna ho sakata hai|
darda, pitha ya kandhe ke agrabhaga taka ja sakata hai|
ubakaiyam, ultiyam, jvara ya piliya bhi isake satha ho sakate haim|
agara apako pittashaya sambandhi samasyaem haim, to apaka daॉktara nishchinta hone ke lie paradhvanika jancha kara sakata hai|
apaka svasthya itihasa janane ke pashchata apaka daॉktara, pittashaya sambandhi samasyaom ki ora ishara karane vale lakshanom ,svasthya samasyaom, aura khatare ke karakom ke bare mem pata lagane ke lie , apaki jancha kara sakata hai|
samasya ki nishchintata jnata karane ke lie rakta jancha, paradhvanika jancha evam siti skaina jaisi nidana karane vali jancha-pada़talom ka istemala kiya ja sakata hai|
agara saba kuchha thika-thaka chalata raha hai, to apa shalya-chikitsa ke 2-3 dina bada ghara ja sakate haim|
shalya-chikitsa ko sthagita karana bhi, avashyaka ho sakata hai|
jaba apa shalya-chikitsa ke lie ayem, to apako, bahuta adhika dhana ya kimati vastuem satha na lane ki salaha di jati hai|
aspatala chhoda़ne ke bada agara ghara para apako madada ki avashyakata hai, to kripaya apani paricharika ko suchita kijie |
aspatala ke jana-sevakom mem se eka, apa se aura apake parivara se bata karega|
varda mem apake pahunchane ke pashchata paricharika , apako thaharane mem , apaki madada karegi, aura apake vajana evam tapa, nabja, shvasana dara evam rakta chapa ki jancha karegi|
apaka daॉktara apake svasthya itihasa ke bare mem puchhega aura apaki jancha pada़tala karega|
vaha apake shalya-chikitsa ki vastavikata evam usaki jatilataom ke bare mem ,apako samajayenge|
shalya-chikitsa ke lie daॉktara, apaki svikriti bhi lega|
apaka nishchetaka ayega evam shalya-chikitsa ke lie apaki svasthata ka mulyankana karega|
jancha ke lie apaka rakta bhi liya ja sakata hai, itana ashvasta hone ke lie ki apa shalya chikitsa ke lie ekadama svastha evam thika hai(agara klinika mem pahale se jancha nahim ki gayi hai)|
shalya-chikitsa se purva koi bhi kimati vastuem, suraksha ke lie, apani paricharika ko saumpa dijie|
shalyagara jane se purva apase ,vishesha rupa se banaya hua gauna badalane ke lie kaha jayega|
eka paricharika apako shalyagara taka le jayegi|
shalya-chikitsa ke bada varda mem, bariki se apaki dekharekha ki jayegi|
shalya-chikitsa ke bada nishchetana ke pasharva prabhava ke karana, apa ulti kara sakate haim ya shalya-chikitsa ki jagaha kuchha darda mahasusa kara sakate haim|
agara apa ऎsa mahasusa karate haim, to apani paricharika ko suchita kijie |
apako adhika arama mahasusa karane ke lie, vaha apako eka injekshana de sakati hai|
yaha darda ko kama karane mem madada karega|
agara apa svastha mahasusa kara rahe haim, to apako baithane ke lie ya apane palanga ke charom ora dhire dhire chalane ke lie protsahita kiya jayega|
shalya chikitsa ke pashchata, apako peya padartha ya bhojana diya jayega|
apaki paricharika apako suchita karegi ki apako kya khana ya pina hai|
jitana apa pacha sakate haim, utana hi khana ya pina mahatvapurna hota hai|
subaha ko daॉktara ayega aura apaki jancha karega|
agara vaha apako svastha pata hai, to apa ghara ja sakate haim|
shighra svasthyalabha ke lie chalana-phirana mahatvapurna hai|
apako, apane bistara se bahara baithane ke lie evam thoda़i duri taka chalane ke lie protsahita kiya jata hai|
apako, bhari vastuem na uthane ki ya shramasadhya kama na karane ki salaha di jati hai, kyonki ye saba shalya-chikitsa ki jagaha ke charom ora ki mansapeshiyom para atyadhika davaba dala sakate haim|
phira bhi, sanyama ke satha vyayama karane ke lie apako, protsahita kiya jata hai|
agara apa svastha mahasusa kara rahe haim, to apa kama vasa vala bhojana khane ke yogya ho sakate haim|
apaki paricharika apako, apane ghava ki dekhabhala karana aura ghava mem lali aura sujana jaise sankramana chihnom ko dekhana sikhayegi|
agara apako adhika janakari ki avashyakata hai, to shaikshika pustika upalabdha hai|
nitya ke samanya vyayama punah karane se pahale agale 2 saptahom ke lie, ghara para sanyama se vyayama karana, kripaya jari rakhie|
3 mahinom ke lie, kama vasa vale bhojana ki salaha di jati hai|
apane ghava ki dekharekha ke bare mem, yadi apako kuchha prashna puchhana hai to, kripaya apani paricharika se puchhie|
apane ghava ko, kripaya sapha evam sukha rakhie|
agara ghava ke charom ora lali evam sujana hai to, jitani jaldi ho sake, kripaya apane jipi ya bahuchikitsalaya daॉktara se milie|
yadi apake tankom ko kholane ki avashyakata hai to, ऎsa karane ke lie apaki paricharika, apako usaki tarikha suchita karegi|
yadi apa yaha, bahuchikitsalaya ya jipi mem karane ke ichchhuka haim to, kripaya apana unmochana pramana patra satha laem|
jaba apa ghara jane ko taiyara haim to, jo davaiyam apake lie avashyaka haim, vaha apaka daॉktara nirdharita karega|
davaiyam kaise leni haim, isake bare mem, apaka aushadha vikreta ya paricharika, apako nirdesha denge|
yadi koi prashna ya sandeha hai to, kripaya apane daॉktara ya paricharika se puchhie|
aspatala chhoda़ne se purva, apaki paricharika apako unmochana pramana patra, daॉktari pramanapatra evam daॉktara se milane ki nishchita tithi degi|
davaiyam 18 mahinom( " 300/mahina) ke lie li jani chahie, phira bhi, pittapathariyam punah ho jati haim|
yaha upachara kevala shuddha kaॉlestraॉla pathariyom ke lie hi upayukta hai|
isilie, upachara ka yaha tarika salaha dene ke layaka nahim hai|
peta ki eka bada़i shalya-chikitsa, pittashaya ko nikalane ka eka samanya tarika hai|
khatarom evam vishamataom mem, atyadhika raktasrava ya sankramana shamila hai|
samanya pittashaya shalya chikitsa mem, peta para eka bada़a chira dene ki avashyakata hoti hai|
vartamana kala mem, isa takanika mem, pittashaya nikalana, kevala chara bahuta chhote chhidrom dvara pura kiya ja sakata hai, jisamem pratyeka chhidra, vyasa mem eka sentimitara se bhi kama hota hai|
isamem, eka chhote chhidra dvara eka laiparoskopa( eka durabina jisamem eka kaimara laga hota hai) bhitara dala jata hai, jisase pittashaya ka avardhita rupa (bada़a) dikhai deta hai|
taba dusare chhote chhidrom dvara auja़ara andara dale jate haim|
sistika nalika aura dhamani( pittashaya ko rakta apurti karane vali) ki pahachana hone ke bada, ise chimatiyom se bandhakara kata diya jata hai, isa taraha pittashaya, yakrita kshetra se alaga ho jata hai, aura nabhi ke eka chhidra dvara bahara nikala jata hai|
prakriya puri hote hi chhote chhidra, eka ya do tankom se sila diye jate haim|
kuchha mahinom ke bhitara, chhidra mushkila se hi dikhai dete haim|
laiparoskopika shalya chikitsa mem, vishesha aujara(yantra) chhote chhidrom ke dvara andara dale jate haim|
ye hanirahita chimatiyam sharira ke andara hi chhoda़ di jati haim|
20 semi lambe ama chire ki bajaya, chara chhote chhidrom se shalya-chikitsa karane mem, adhikatara marijom ko shalyakriyottara darda ,ullekhaniya dhanga se kama hi hota hai|
samanya tarike vale shalya-chikitsa ke marijom ki apeksha, ve jaldi hi achchhe bhi ho jate aura samanya gatividhiyam ki ora bhi jaldi hi lauta ate haim|
samanya tarike ki tulana mem, shalya-chikitsa chhote nishana hi chhoda़ta hai|
yadyapi laiparoskopi ke bahuta se phayade haim, phira bhi yaha tarika una kuchha marijom ke lie upayukta nahim hai jinako pahale se hi peta ke nichale hisse mem shalya-chikitsa ho chuki ho ya anya purvavarti chikitsiya bimariyam ho|
daॉktara dvara eka sampurna chikitsiya mulyankana hi yaha nishchita kara sakata hai ki laiparoskopi se pittashaya ka nikalana , apake lie upayukta prakriya hai ya nahim|
apaki shalya-chikitsa ke pashchata, thoda़e samaya ke lie apako, aspatala mem rukana pada़ sakata hai|
apane daॉktara ki salaha manie aura dhire dhire apani takata phira se banaiye|
apake phephada़em sapha karane ke lie, apa se gahari sansem lene evam khansane ke lie kaha ja sakata hai|
shalya-chikitsa ke pashchata jitani jaldi ho sake, chalana-phirana, apaka raktasanchara ko banaye rakhane mem madada karata hai|
jaise hi apaka pachana tantra thika se karya karane lagata hai, apa samanya rupa se khana shuru kara sakate haim|
bukhara(38 digri sentigreda) ya thanda lagana, aura piliya hone jaise kisi bhi parshva prabhava ke jnata hone para, turanta apane 
apako tela ya vasa ki matra kama karani hogi, bada mem apaka sharira dhire dhire tailiya bhojana ke anukula ho jayega|
pittashaya ki shalyachikitsa, sarvadhika saphala prakara ki shalyachikitsaom mem se eka hai|
yaham taka ki, bina pittashaya ke ,adhikatara marija svastha evam samanya jivana jite rahate haim|
isa seva ka uddeshya, apane marijom ko ho sakane vale koi bhi pareshani, lakshana, ya svasthya sambandhi chinta ke bare mem prashna puchhane ke ,atirikta vikalpa upalabdha karana hai|
yaha seva, hamare chikitsa visheshajnom evam parachikitsiya karmachariyom ke paramarsha se, anubhavi paricharikaom ki hamari tima ke dvara upalabdha karaya jata hai|
yaha seva, chikitsiya salaha ya paramarsha ke badale mem nahim hai aura isaka prayoga, nidana ya upachara ke uddeshya se nahim kiya jana chahie|
nishchetana(behosha karana), darda ko anubhava kie bagaira hi marija ki shalya-chikitsa karane ke lie, davaiyom evam vishesha takanikom ka prayoga hai| 
sthaniya nishchetana (lokala anisthisiya ya ela.e.) mem, tvacha ke upari bhaga ya jilli para, eka nishchetaka padartha lagaya ja sakata hai ya tvacha ke bhitara dala ja sakata hai|
apekshakrita surakshita evam sarala vidhi hone ke karana, ela.e. mem, eka nishchetaka ke dvara nirikshana karane ki avashyakata nahim hoti|
kshetriya nishchetana (rijanala anisthisiya ya ara e) mem, darda sanketom ke nasa prasarana ko sharira ke khasa hissom taka pahunchane se rokane ke lie, eka ya adhika injekshana lagate haim|
parinama svarupa, sharira ke chune hue hisse mem hi, sunnapana ata hai|
yaham taka ki, marija ke jage rahane para bhi, shalyachikitsa ki ja sakati hai|
shalyachikitsa ka sthana hamesha, marija se dhaka kara rakha jata hai|
marija ko aushadhi dvara shanta avastha mem bhi rakha ja sakata hai, taki prakriya ke daurana ve, soye rahem|
samanya ninda ke pratikula, pareshana karane para ya pukare jane para bhi marija, nahim jagega|
isa taraha, ji. e. ke antargata, marija apane asa-pasa ki chijom se aura mahatvapurna rupa se, shalya-chikitsa evam darda se, anajana rahega|
sampurna shalya-chikitsa ke daurana, nishchetaka marija ke satha rahega aura shalyachikitsottara darda nivaraka ,upalabdha karayega|
nishchetana se juda़i kai vishamataom evam nrityuom mem, ashcharyajanaka kami hui hai|
marija ki sehata ki dekha rekha karane ke lie nishchetaka ki upasthiti, eka atyanta mahatvapurna tattva hai|
dekha-rekha ke naye upakarana evam nayi aushadhiyom ke avishkara ne, nishchetana mem, behatara suraksha ka yogadana diya hai|
eka atirikta khuraka ke karana, aura samvedanashila yantra, jo ki nishchetana ke lie die gae padartha ki matra ke nirikshana ke yogya haim,isake istemala ke karana evam ऎsi nayi nishchetana ki davaiyom ke prayoga ke karana jinaka prabhava jaldi hi khatma ho jata hai, nishchetana se na jaga pane ki sambhavana, bahuta hi kama hai|
yaha hona asambhava bhi hai, kyonki nishchetana aushadhiyam kevala tabhi banda ki jati haim, jaba shalya-chikitsa purna ho jati hai|
samanyatah, adhikatara marijom ko shalya-chikitsa ke daurana, kuchha darda nivaraka davaiyam di jati haim|
yadyapi vibhinna marijom ke darda ki sima bhinna-bhinna hoti hai, lekina phira bhi, jagane para kuchha darda mahasusa kiya ja sakata hai|
marija ko adhika arama dene ke lie, atirikta darda nivaraka davaem di ja sakati haim|
yadyapi vastava mem ऎsa bahuta hi kama hota hai, phira bhi gambhira jatilataem ho sakati haim|
smarana shakti ka koi dirghakalina hrasa nahim hona chahie|
phira bhi, smarana shakti ka alpakalina hrasa ho sakata hai, jisamem marija ko shalya-chikitsa ke asa-pasa ki ghatanaem yada nahim rahatim|
ऎsa isilie hota hai kyonki chinta ko kama karane ke lie evam manasika uttejana ko shanta karane ke lie di gayim aushadhiyom mem se kuchha mem ,smarana shakti ka asthayi alpakalina hrasa karane ki kshamata hoti hai|
agara apako, aushadhiyom ko jari rakhane ki avashyakata hai, to nishchinta hone ke lie, apane shalya-chikitsaka ya nishchetaka se kripaya puchha lem|
jaba ji.e .ya ara.e .ke antargata eka prakriya chala rahi hoti hai, to upavasa rakhana avashyaka hota hai|ऎsa isalie karana hota hai kyonki jaba marija ji.e .ke antargata hote haim, to ve apani rakshatmaka pratikriyaom ko kho dete haim|
agara peta mem bhojana ki kuchha matra hai, to vaha ulta munha ki ora a sakata hai aura marija ke gale mem phansa sakata hai|
bhojana ke kana, phephada़om mem bhi a sakate haim, aura chhati ke gambhira sankramana ka karana bana sakate haim jo janaleva bhi ho sakate hai|
ara.e. diye jane vale marijom ke lie,upavasa rakhana avashyaka bhi hai kyonki ji.e. mem parivartita karane mem yaha, nishchetaka ke lie sahayaka hota hai, varana nishchetaka ara.e. ke vanchhita prabhava lane mem viphala ho jayega|
jaba hama, naukari chhuta jana, traiphika jama mem phansa jana, ya jaba parikshaem najadika hom jaisi tanavapurna paristhitiyom ka samana karate haim, to hama chintita ho jate haim|
jaba dara adhika tatkalika evam saralata se paribhashita ho, jaise ki khada़i chattana ke upara se dekhana ya eka bada़e se gurrate hue kutte dvara akramana hona, to bechaini utpanna hoti hai|
bhaya ki kuchha matra labhadayaka hoti hai, kyonki yaha, hamare karya ko behatara banati hai aura hamari samasyaom ko hala karane mem hamari madada karati hai|
chinta se vikara taba utpanna hote haim jaba chinta itani bhayankara ho jati hai evam badha़ jati hai ki ve rojamarra ke karyom se samana karane ki yogyata mem badha dalati haim|
sabhi manasika vikarom mem, chinta se utpanna hone vale vikara sarvadhika samanya haim|
bahuta se loga ina vikarom ko bekara samajate haim aura sochate haim ki vyaktiyom ko svayam ki ichchha shakti dvara hi ina lakshanom para kabu pane yoga hona chahie|
lakshanom ko dura karane ki matra chahata se phayada nahim hota, balki upachara, madada kara sakata hai |
vyapaka chinta vikara(jiedi), eka purani badha़i hui pareshani evam tanava hai, yaham taka ki, jaba koi isako uttejita bhi nahim karata|
jiedi vale loga musibata ka purvanumana laga lete haim, aura svasthya, paise, parivara ya kama ke bare mem atyadhika chinta karate haim|
unako tanava mukta hone mem mushkila hoti hai aura ve apani chintaom se pichha chhuda़ane ke layaka nahim hote, jaba ki ve mahasusa karate haim ki unaka chinta karana bekara hai|
unaki chintaom ke satha-satha kampakampi ana, mansapeshiyom mem achanaka jakada़na hona, mansapeshiyom mem tanava hona, sira darda, chida़chida़apana, pasina ana ya achanaka garmi mahasusa hona jaise sharirika lakshana, bhi ate haim|
ve aviveki ho sakate haim ya hampha sakate haim evam ubakai mahasusa kara sakate haim ya bara-bara mutralaya jana pada़ sakata hai|
jiedi se sambandhita hani, samanyatah bahuta thoda़i hi hoti hai evam isa vikara
vale loga, samajika parivesha mem ya naukari mem atyadhika pratibandhana mahasusa nahim karate|
lekina agara prachanda hai to jiedi, vyakti ko kshina bana sakati hai,evam rojamarra ki sarvadhika samanya kriyaom ko nibhana bhi mushkila kara deti hai|
bhayankara vikarom vale loga atyadhika bhaya ko mahasusa kara sakate haim jo bina chetavani die achanaka, aura bara-bara ata hai|
ve anumana nahim laga sakate ki roga ka daura kaba pada़ega aura yaha chinta karate hue ki agala daura kaba aura kaham pada़ega, daurom ke madhya tivra dara utpanna ho sakata hai|
daurom ke madhya, dimaga mem lagatara eka hi chinta bani rahati hai ki agala daura kisi bhi kshana pada़ sakata hai
jaba eka bhayankara daura pada़ta hai, to hridaya jora se dhada़kata hai aura marija, pasina ana,behoshi ana, kamajori ya chakkara ana mahasusa kara sakata hai|
unake hathom mem janajanahata ho sakati hai ya sunnapana mahasusa ho sakata hai evam ve bhavavega mem uttejita ho sakate haim ya bukhara a sakata hai|
chhati mem thoda़i asuvidha, dama ghutane ki anubhuti, avastavikata ki anubhuti, nikata bhavishya mem ghatita hone vale sarvanasha ka dara ya niyantrana khona adi jaise lakshana bhi utpanna ho sakate haim|
kashta bhokta ko sacha mem vishvasa ho sakata hai ki usako dila ka daura pada़ raha hai ya aghata ho raha hai,vaha apana niyantrana kho raha hai ya usaki nrityu samipa hai|
jabaki adhikanshatah daure kuchha eka minata ke lie hi rahate haim, lekina kabhi-kabhi ve dasa minata taka bhi rahate haim|
bahuta kama mamalom mem, ye eka ghante ya usase adhika samaya ke liye bhi raha sakate haim|
bhayankara vikara, aksara udasi ya sharabakhori jaisi anya sthitiyom se juda़a hota hai, aura asvabhavika dara bhi utpanna kara sakata hai|
kuchha logom ki jindagiyam mahatvapurna rupa se simata kara raha jati hai|
jaba eka bhayankara daura pada़ta hai to ve, ऎsi sthiti se bachana chahate hai jo unako lachara mahasusa karava sakati haim|
isa vikara ke karana, jaba unaki jindagiyam itani simata jati hai to isa sthiti ko agoraphobiya kahate haim,jo ki bhayankara vikarom sahita lagabhaga eka tihai logom mem hoti hai|
asvabhavika bhaya matra atyadhika bhaya ki sthiti nahim hai, balki ye atarkika bhaya hai|
agara bhaya ki vastu se bachana sarala hota hai, to asvabhavika bhaya vale loga, upachara karane ki avashyakata mahasusa nahim karate|
eka vishishta asvabhavika bhaya, kisi eka vastu ya sthiti vishesha ke prati, dara matra hai| bahuta se loga, kuchha vishesha vastuom ya sthitiyom ke prati vishishta gambhira, atarkika bhaya ka anubhava karate haim|
samajika paristhitiyom mem apamanita hone ka bhaya ,vishesha rupa se anya logom ke samane khuda para sharminda hone ka atyadhika bhaya hi samajika bhaya kahalata hai |
isamem chhoti chhoti galtiyam vastavikata se adhika badha़i-chadha़i lagatim haim |
sharma se jempana apane apa mem bahuta hi kashtadayaka evam sharmanaka hota hai aura rogi ko lagata hai ki jaise sabhi ankhem usa para kendrita haim ,yaha dara kisi vishishta sthiti mem aura bhi adhika ho sakata hai jaise ki bhashana dene mem ghabarana ,malika ya anya mahatvapurna pradhikari se bata karane mem darana ,ya premi ke satha bahara ghumane jane mem darana |
samajika bhaya samanya jivana ko asta-vyasta kara deta hai ,naukari-peshe mem ya samajika rishtom mem bhi hastakshepa karata hai |
bhida़ se dara lagana kisi bhi sthiti mem hone ka eka prakara ka vaha dara hai , jo aksara akasmika bhaya vali vikriti ke satha hi hota hai aura isa dara se akasmika bhaya vala daura bhi pada़ sakata hai ,aura agara jaba yaha pada़ta hai to isase bachana mushkila bhi ho sakata hai |
bhida़ se darane vale loga ghara se dura yatra karane mem bhi darate haim ya phira simita sthanom para hi jate haim |
samanya chinta aura chinta vikara ke bicha mem eka sima rekha taba utpanna hoti hai jaba bina kisi vastavika khatare ke bhayankara tanava utpanna ho jata hai ,ya jaba chinta ke karana se bachane ke lie charama upaya kie jate haim | 
tanavapurna jivana ki ghatanaem chinta vikara ka karana ho sakatim haim |
chinta ke karana ka nidana hone para amataura para chinta dura ho jati hai ,lekina kabhi-kabhi tanavapurna paristhitiyam khatma hone para bhi lakshana lambe samaya taka bane rahate haim |
chinta vikarom ke hone mem vishamataem taba aura bhi adhika ho jatim haim jaba agara apake mata-pita ya bhai-bahanom mem se kisi eka ki halata aisi hai |
isalie, apaki anuvanshiki banavata aura apake niji anubhavom se pata lagaya ja sakata hai ki apa chinta vikara se rogi honge ya nahim |
chinta vikarom ka asali karana batana aksara bahuta kathina hota hai | 
amataura para rogi apane niji svabhava , jivana mem parivartana aura vishesha paristhitiyam jaise vibhinna prakara ke tanavom ke prati atyadhita samvedanashila ho jata hai |
kabhi-kabhi apani samasya ke bare mem , apane kisi vishvasaniya dosta ya rishtedara jinaki raya para hama bharosa karate haim ,se bata karana tanava kama karane mem bahuta sahayaka hota hai |
ho sakata hai ki khuda unako bhi isi taraha ki musibatem jelani pada़i hom , isalie ve apane anubhavom ko apake satha bantana chahenge |
tanava se mukti dilane ki koi eka taka़nika sikhana ,apako shanta hone mem sahayaka ho sakata hai aura apake tanava aura chinta ko niyantrita bhi kara sakata hai |
tanava se mukti dilane ki takanikom mem pranayama karana , dhyana lagana aura koi eka kalpanika karya karana shamila haim |
kai kitabem aura ऑdiyo tepa upalabdha haim,jinako apa ina takanikom ko apane apa sikhane ke lie prayoga mem la sakate haim |
yaha eka atyadhika bhavuka vartalapa upachara hai, jo rogiyom ko pahachanane mem , samajane mem aura chinta ke karana jo ki tatkala spashta nahim ho pate haim , ko janane mem madada karata hai |
eka khasa sanjnanatmaka vyavahara upachara ,akele ya kisi anya dava ke satha bhi , adhikansha chinta vikarom ko sulajane mem madada kara sakata hai |
isa takanika mem logom ki chinta ko kama karane ke lie ,unaki dara vali sthitiyom ka samana karane hetu unhem sanjnanatmaka yojanaem upalabdha karayi jatim haim |
yaha upachara , saptaha mem eka bara se adhika aura kai haphtom ya mahinom ke lie bhi ho sakata hai |
chinta vikarom ke upachara ke lie vibhinna prakara ki davaiyam upalabdha haim |
ye davaiyam adhikatara upara likhe tarike ke satha hi istemala ki jati haim |
tanavanashaka evam avasadanashaka davaiyam adhikanshatah istemala ki jane vali do davaem haim |
sabase adhika istemala ki jane vali tanava nashaka davai benja़odayaja़epinsa haim |
ye davaiyam chinta ke lakshanom ke lie na kevala dina ke samaya , balki rata mem bhi achchhi ninda ke lie upayogi hotim haim |
halanki yaha eka prabhavi davai hai ,parantu benja़odayaja़epinsa ko lambe samaya taka istemala karane ke bada isa para nirbharata bana sakati hai, jisake parinamasvarupa agara jaba dava banda kara di jati hai ya khuraka mem kami ki jati hai to lakshana dikhai nahim pada़te |
isalie benja़odayaja़epinsa , chinta ke lakshanom ke lie kevala alpakalika ilaja ke lie hi istemala ki jani chahie, na ki lambe samaya taka ke ilaja ke lie |
avasadarodhi davaiyam chinta aura avasada donom ke lakshanom ka ilaja karane ke lie upayoga ki jati haim |
benja़odayaja़epinsa ke viparita, ina para nirbharata nahim banati |
amataura para istemala ki jane vali avasadarodhi davaiyam haim -traisaiklika avasadarodhi davaiyam (ti si e ),slaikativa serotonina ri-apateka inhibitarsa (esa esa ara ai ) , serotonarjika noradrainarjika ri-apateka inhibitarsa ( esa ena ara ai ) evam monoamina ऑksideja़ inhibitarsa |
uddeshyapurna sudhara hone se pahale hi ina davaiyom ko do se chara saptaha ke lie niyamita rupa se liya jana chahie |
chinta vikarom mem kai samasyaem ati haim |
pahali samasya hai kama-kaja mem mana na lagana |
atyadhika ghataka mamalom mem ,vyakti skula jane mem ya naukari karane mem purnatah asamartha ho jata hai |
avasada ya udasi bhi aksara hone vali samasya hai , jisase sthiti badattara ho jati hai aura atmahatya karane ke khatare badha़ jate haim |
chinta vikarom mem kai samasyaem ati haim |
pahali samasya hai kama-kaja mem mana na lagana |
atyadhika ghataka mamalom mem ,vyakti skula jane mem ya naukari karane mem purnatah asamartha ho jata hai |
avasada ya udasi bhi aksara hone vali samasya hai , jisase sthiti badattara ho jati hai aura atmahatya karane ke khatare badha़ jate haim |
madaka dravyom ka sevana bhi aksara eka samasya bana jata hai, ,vishesha rupa se darda nivaraka davai ya sharaba ke satha svayam upachara karana aura madaka dravyom ke sevana karyakrama ke satha eka samanantara upachara ki bhi avashyakata ho sakati hai ,upara batae lakshanom ko pahachanana hi uchita upachara ka pahala kadama hai |
kai rogi to apane lakshanom ko sahana karate rahate haim , kyonki pagala kahe jane ke dara se ve apani samasyaom ko lekara daॉktara ke pasa jane se bhi darate haim |
vastava mem, chinta ya bhaya ke jyadatara rogiyom ko shayada hi kabhi gambhira manasika bimari hoti hai , aura aksara ilaja ke dvara unaki samasyaom ko kama kiya ja sakata haim ya unase chhutakara dilaya ja sakata hai |
dvidhruvi manovikriti jo unmatta avasadagrastata manovikriti (emadipi) ke nama se bhi jani jati hai, eka prakara ki manovikriti hai jisamem rogi ki manodasha badalati rahati hai |
kai logom ki samanya manodasha somavara ki subaha ke avasada se eka halke ullasonmada taka badalati rahati hai |
dvidhruvi manovikriti mem , charama avasada se mahana utsaha ya unmada aura atyadhika sakriyata ki sthiti taka manodasha badalati rahati hai , kabhi-kabhi to vastavikata se bhi samparka tuta jata hai |
halanki, purnataya samanya avastha ki avadhi bhi kaphi lambi hoti hai |
unmadi avasadagrasta rogi eka lambe samaya taka atyadhika manodasha badalane ki sthiti se pida़ita rahate haim aura ina daurom ke daurana unako chikitsakiya rupa se bimara mana jata hai |
dvidhruvi manovikriti ki jancha pada़tala se purva ,eka chikitsaka ya manochikitsaka yaha dekhata hai ki unmada ke daure ke bada avasada bhi hota hai ya nahim |
dvidhruvi manovikriti amataura para kishoravastha mem ya vayaskata ke prarambhika dinom mem ubharane lagati hai aura jivana bhara bani rahati hai |
aksara isako eka bimari nahim mana jata ,aura jina logom ko yaha manovikriti hoti hai ve bekara mem hi salom sala , yaham taka ki kai dashakom taka isase pida़ita rahate haim |
dvidhruvi manovikriti ke karanom ka pata lagane ke lie kapha़i shodha kiye ja rahe hai lekina abhi taka koi nishchita nishkarsha nahim nikale haim |
isa bimari ke prati anuvanshika pravritti ho sakati hai ,jaise ki parivara ke anya sadasya bhi isa bimari se pida़ita ho |
ho sakata hai ki yaha atyadhika tanavapurna ghatanaom ya prasava ke bada hone vale harmonom mem parivartanom ke bada ubhara kara samane ayi ho |
yaha kisi rasayanika asantulana ke karana bhi ho sakati hai jo mastishka ke usa kshetra ko prabhavita karata hai jisase insana ki manodasha ka sambandha hota hai ,aura isilie bhavanaom aura vyavahara mem parivartana ka karana banata hai |
yaha eka aisi sthiti hai jisase rogi mem atyadhika sakriyata ,atyadhika asthira chittata ya chida़chida़apana , avegashila ya laparavaha vyavahara dikhane lagata hai |
jisa vyakti ko haipomaniya hota hai vaha halka mahasusa karata haim aura pida़ita shayada hi kabhi yaha sochata ho ki usake satha kuchha to galata hai aura jaba koi usako yaha batane ki koshisha karata bhi hai to vaha kaphi hinsakata se pesha bhi a sakata hai |
kuchha logom ko apane anubhava se apani isa sthiti ko pahachanana sambhava bhi ho jata hai, lekina aksara manovikriti ki charama sima mem unaki samaja purnatah samapta ho jati hai |
isa daure se rogi kisi ke bhi lie , kisi bhi kalpana ki uda़ana ke daurana ,atyadhika kharcha karane ke daurana , dhokhadhada़i karane ke daurana ,ya atmahatya ke prayasa ke daurana jo sanyoga se kabhi kabhara hi ho sakata hai ,kanuna toda़na bilkula bhi asamanya nahim hai |
kuchha logom ko kabhi-kabhi unmada aura avasada donom ke lakshanom ka mila-jula anubhava hota hai |
jisase atmahatya karane ka khatara badha़ sakata hai |
avasada ya unmada ka samaya bita jane ke bada rogi , amataura para bina kisi bhi sharirika hani ke samanya sthiti mem a jata hai, jaise ki vapasa apane samanya vyaktitva mem , sochane ki kshamata aura urja ke stara para |
yaham taka ki pida़ita vyakti rishtom ya vitta sambandhi mamalom mem hui kshati ke lie pachhatata bhi hai |
jisase atmavishvasa khatma ho sakata hai aura sharmindagi bhi uthani pada़ sakati hai |
daurom ka pada़na pratyeka vyakti mem bhinna-bhinna hota hai ,yaham taka ki eka hi vyakti mem bhi yaha bhinna ho sakata hai |
kuchha loga kai varshom taka bina kisi samasya ke jite rahate haim aura phira bada mem krama se kai aniyamita daure pada़ jate haim |
agara ina daurom ka ilaja na hua ho to pida़ita samaya ke satha apane apa hi thika ho jate haim ,lekina unake jivana para, unake parivarom para atyadhika sadama pahunchane vale prabhava ho sakate haim ,aura kaphi anavashyaka pareshaniyam uthani pada़ sakati haim |
unmada ya gambhira avasada ke rogi ko jaham kahim bhi sambhava ho ,jalda se jalda aspatala mem bharti karana usake shighra svasthya labha hone ko asana bana deta hai |
isake alava, aspatala mem rahakara upachara karavana bhi bimari ke parinamom se rogi ki raksha karata hai ,jaise ki unmada mem utavalepana mem lie gaye nirnayom ya gambhira avasada mem atmahatya ke prayasa mem madada karata hai |
kai mamalom mem (60-80%) manodasha ke parivartana ko niyantrita kiya ja sakata hai, ya manodasha sthirakari jaisi davaiyom se kama kiya ja sakata hai |
sabase jyada istemala ki jane vali davai lithiyama karboneta hai |
isake prabhavi stara ki jancha karane ke lie ,niyamita rakta jancha se isa para savadhanipurvaka nigarani rakhane ki avashyakata hoti hai | 
samajika aura vyaktigata rupa se svikarya starom mem , lithiyama manodasha ko badalata rahata hai aura gambhira se gambhira daurom se dura bhi rakhata hai |
ilaja karane se vyakti eka samanya aura upayogi jivana vyatita kara sakata hai |
durbhagya se ,bahuta hi kama rogiyom ke lie ,isa upachara ke prabhavashali parinama hote haim ya sankhya mem usase bhi kama pida़itom ke lie isake dushparinama hote haim |
lithiyama karboneta ke dushprabhavom mem se kuchheka haim lagatara peshaba ana ,pyasa lagana , hatha kampana aura tvacha para chakatte bana jana adi |
avasada ya unmada ke ilaja mem satha hi satha anya davaiyom ka bhi prayoga kiya jata hai |
kabhi kabhi, eka se adhika dava ki bhi avashyakata pada़ti hai |
ina davaom mem avasadarodhi davaiyam ,manovikarom ke prati prabhavakari davaiyam aura benja़odayaja़epinsa shamila haim |
hara dava ka apana eka dushprabhava hota hai |
ina davaom ka upayoga samanya sthiti ko kevala jalda se jalda vapasa lane ke lie hi kiya jata hai aura shayada hi kabhi kabhara aisa hota hai ki ye davaem lagatara di ja rahi hom |
atyadhika gambhira avasada aura kabhi-kabhara hone vale unmada ke mamalom mem kabhi-kabhi eka aura upachara ilektro kanvalsiva therepi (isiti) prayoga mem laya jata hai |
dekhabhala karate samaya, marija़ ko usaki mansapeshiyom ko shithila karane ki dava evam thoda़i dera ke lie purna behoshi ki dava di jati hai |
sira ke naja़dika rakhe gaye ilektroda se puri taraha se niyantrita eka vidyuta pravaha mastishka mem bheja jata hai |
upachara ki avadhi ke daurana yadadashta ja sakati hai, lekina yadadashta para dirghakalika prabhava ka koi pramana nahim milata hai |
amataura para, isiti se gambhira avasada mem saphalatapurvaka sudhara kie ja sakate haim |
manodasha ke badalate rahane ke prabhava se vaivahika jivana para ,bachchom (ya mata pita) para ,naukari dene valom para evam mitrom para aura yaham taka ki pida़ita vyakti para bhi atyadhika tanava pada़ta hai |
kai mamalom mem tanava ko aura adhika badha़ate hue evam pida़ita ke akelepana ko aura badha़ate hue yaha ekakipana ka ,naukari chhuta jane ka ,ya ghara ke bikharane ka karana banata hai | 
rogi ko kisi na kisi rupa mem paryapta samaya ke lie ya sambhavatah jindagi bhara manodasha sthira rakhane ki dava leni pada़ sakati hai |
lakshanom ko pahachanane ki koshisha jaldi hi karani chahie ,kyonki kabhi-kabhi davaom se upachara jaldi karane se ya kathina paristhitiyom se bachakara, manodasha ko aura adhika prachanda hone se rokana sambhava hota hai | 
sone ki adata mem parivartana aksara bimari ke jaldi punaravartana hone ka eka mahatvapurna suchaka hai |
sharaba aura kaiphina utpadom (chaya, kaॉphi, koka) ka sevana na karake aura santulita ahara apanakara jatilataom ko rokie |
chunki bimari tanava sambandhi hoti hai isalie apa apane tanava ka karana janie aura apane jivana mem isa anavashyaka tanava se bachane ka prayasa kijie aura usake upaya kijie |
sabase pahale, priyajanom aura rishtedarom ko yaha samajana chahie ki rogi ka ajiba vyavahara ,hinsaka ya apamanajanaka bhasha ya lambe samaya ki chuppi , jo bimari ki kuchha avasthaom ki visheshataem ho sakati haim , rogi ke vyaktitva ka pahale se chhipa koi pahalu nahim ,balki manovikriti ka hi eka lakshana hai |
jaba apako bahara jana ho taba apako dhyana rakhana hoga ki apa koi jaharile padarthom ya goliyom ko tale mem chhipa kara jana hoga taki rogi ko khojane para bhi ye chijem na mile |
pulisa ke satha samparka banae rakhane mem darie mata ,jo ki ama taura para madadagara hi sabita hota hai |
rogi ko manochikitsaka ke pasa le jane ke lie protsahita kijie ,ya jaba bhi rogi ki sthiti kaphi ugra ho jae to tatkala madada ke lie puchhie aura puchhie ki usako yaha pata hai ya nahim ki usake satha kuchha galata ho raha hai |
aspatala mem rogi ko dekhane jane se apake vishvasa aura samaja ki asali pariksha ho sakati hai | 
unhem aspatala mem rakhane ka aropa lagane se ,ho sakata hai ki apako bhalai na mile |lekina phira bhi agara apa apane drishtikona mem lagatara atala aura dridha़ bane rahe to isase rogi ko kaphi madada mila sakati hai |
apa paricharya aura chikitsiya karmachariyom se bimari ke bare mem atirikta antardrishti aura samaja prapta kara sakate haim |
dengu eka machchhara-janita vishanu sankramana hai |
dengu bukhara eka tivra, phlu-jaisi bimari hai jo navajatom, chhote bachchom aura vayaskom ko prabhavita karati hai, lekina kabhi-kabhi hi nrityu ka karana banati hai |
dengu ke 2 sabase krura rupa haim dengu himorehajika phivara (diechaepha) aura dengu shaॉka sindroma (diesaesa) |
dengu ke vishanu sankramita mada edisa machchhara ke katane se manavom mem sampreshita ho jate haim |
machchhara samanyatah eka sankramita vyakti ke rakta pite samaya isa vishanu ko uparjita karate haim | 
eka bara sankramita, koi machchhara apani baki ke pure jivana mem ati samvedanashila vyaktiyom ko vishanu sampreshita karane mem sakshama hota hai |
dengu ke vishanu sankramita manavom ke rakta mem 3 – 14 dinom (ausata 4 se 6 dinom) taka sancharita hote haim |
dengu jvara ke sanketa aura lakshana rogi ki ayu ke anusara bhinna hote haim|
navajata aura chhote bachchom ko jvara aura dadora ho sakata hai |
bada़e bachchom aura vayaskom ko ya to halka jvara ya uchcha jvara ka akasmika chadha़ava, tivra sira darda, ankhom ke pichhe darda, mansa-peshi aura joda़om ka darda aura dadore ho sakate haim |
dengu ki samvegi sthiti mem, dengu jvara se diechaepha/diesaesa aura dusari vishanvika bimariyom mem bheda karana mushkila hota hai |
diechaepha/diesaesa ki sankatapurna sthiti sarvadhika bara tapamana ke samanya taka ya niche girane ke 24 ghante pahale ya 24 ghante bada se utpanna hoti hai |
diechaepha ke lakshanom mem jvara, tivra sira darda, ankhom ke pichhe darda, joda़om ke darda aura badana darda, michali aura ulti aura tvachiya dadore shamila haim |
rogi samanyatah sata dinom taka bahuta bimara rahata hai, usake bada kai haphtom taka atyadhika kamaja़ori rahati hai |
pletaleta ki sankhya bimari ke 3 aura 8 dinom ke bicha samanyatah 100,000/mimi3 se kama pai jati hai |
dengu himorehajika phivara jvara, khansi, sira darda, ulti aura udariya darda utpanna karata hai |
khuna ke thakke ki samasyaem ho sakati haim jo ki antarika raktasrava aura nrityu mem parinita ho sakati haim |
yadi dengu himorehajika phivara ka upachara nahim kiya jata hai, to yaha dengu shaॉka sindroma ki ora agrasara ho sakata hai |
rogiyom mem tivra diechaepha ya diesaesa ke satha, jvara aura kuchha dinom ki avadhi ke avishishta sanketom aura lakshanom ke bada rogi ki sthiti mem akasmika hrasa hota hai |
prathamatah rogi thaka hua dikha sakata hai, bechaina ho jata hai aura phira teji se aghata ki sankatapurna sthiti mem chala jata hai |
diechaepha/diesaesa ki sankatapurna sthiti sarvadhika bara tapamana ke samanya taka ya niche girane ke lagabhaga 24 ghante bada se utpanna hoti hai |

isa samayavadhi mem, himorehajika avirbhava hota hai aura parisancharana asaphalata ke sanketa prakata ho sakate haim |
himorehajika ke sanketa samanya haim aura isamem tvacha ke niche raktasrava (petechi aura parapyura), masuda़om se raktasrava, naka aura udarantriya kshetra se raktasrava aura atyadhika masika pravaha sammilita haim |
ve aghata shuru hone ke kuchha dera pahale tivra udariya darda barambara mahasusa karate haim |
dengu himorehajika phivara 3 se 14 dinom taka rahata hai |
isaka ausata udbhavana kala 4 se 6 dina hota hai |
dengu ke vishanu sankramaka nahim hote haim aura eka vyakti se dusare ko sampreshita nahim kie ja sakate haim |
avasada eka bahuta hi samanya anubhava hai |
pratyeka vyakti kabhi-kabhi thaka hua, dayaniya ya dukhi mahasusa karata hai |
samanyatah isaka karana spashta hai – nirasha, hatasha, koi mahatvapurna vyakti ya chija़ khona |
isa prakara ki udasi aura dukha samanya aura jivana ke tanavom ki asthai kriyaem hoti haim |
yaha kisi vyaktigata kamaja़ori ka sanketa ya avastha nahim hai jo ki ichchhashakti ya kamana se dura ki ja sakati hai | 
avasadakari gada़bada़i eka purna-daihika bimari hai |
yaha sharirika aura samvedanatmaka donom rupa se, apake sochane ke tarike aura mahasusa karane ke tarike ko prabhavita kara sakati hai |
avasadakari gada़bada़iyam vibhinna rupom mem ati haim |
adhikatara avasada lakshanom ke eka mishrana se visheshikrita ki jati hai jo hamare kama, ruchi aura parivara aura mitrom ke prati bhavanaom ko prabhavita karate haim |
isamem eka atyadhika aura sthira avasada-grasta manodasha hoti hai jo do haphte taka rahati hai aura dainika karyom mem hastakshepa karane ke lie mahatvapurna kathinai utpanna karati hai |
ye akshama karane vali ghatanaem eka jivanakala mem eka bara, do bara ya kai bara ho sakati haim| 
disthaimiya (manasika avasada) chirakalika, jirna avasadakari lakshanom dvara visheshikrita avasada ka eka bahuta hi halka rupa hai |
yaha avasada-grasta manodasha do varsha ki avadhi ke lie rahati hai aura usake satha-satha urja, bhukha ya ninda mem parivartana ke atirikta svabhimana ki kami aura nirasha ki anubhuti hoti hai |
ye lakshana karya karane mem dukha aura kathinai utpanna karate haim lekina itane tivra nahim hote haim jitane ki mukhya udasi mem |
loga jo disthaimiya (manasika avasada) se pida़ita hote haim ve mukhya avasada ki ghatanaom ke badha़e hue khatare para hote haim |
dohara avasada taba hota hai jaba halka avasada upasthita ho, adhikatara samaya adhika tivra avasadakari lakshanom ki prasangika avadhi ke satha |
avasadakari gada़bada़iyom ke sthanantarana mem anuvanshika ya jaivika karakom ko darshate hue, kuchha prakara ke avasada parivarom mem chalate rahate haim, |
phira bhi, yaha una logom mem bhi ho sakata hai jinake avasada ka koi parivarika itihasa nahim hai |
vanshagata ho ya nahim, mukhya avasadakari gada़bada़i prayah mastishka mem vishesha rasayanikom ke bahuta kama ya bahuta adhika hone se sambandhita hai |
jivana ki ghatanaem avasada ke sahayogi karaka haim |
kisi dukha bhari ghatana ke bada avasada-grasta mahasusa karana bhi bahuta samanya hai lekina samanyata: hama jo bhi hua hai usase sambandhita apani bhavanaom ke satha samajauta karate haim aura unhem anichchhapurvaka svikara kara lete hai |
koi parivartana, gambhira nukasana ya kisi priya vyakti ki nrityu, talaka, kisi naukari ka nukasana, sevanivritti ya kisi nae ghara mem jana jaise tanava, eka avasadakari bimari ko achanaka la sakate haim |
tanavabhari avastha mem samajika sahayoga ki kami bhi eka bhumika nibhati hai | 
yadi hama akele aura samajika rupa se prithaka ho gae haim, anya bahuta si chintaem haim ya sharirika rupa se durbala haim, taba hama avasada ko badha़ane mem adhika samvedanashila ho sakate haim |
avasada ama taura se sharirika bimariyom, visheshataya kainsara aura hridaya roga jaisi jivana ke lie khataranaka bimariyom, gathiya, shvasanali shotha ya aghata jaisi chirakalika aura kashtakari bimariyom, ke satha bhi hoti hai |
kuchha avasadom ke bada imphluenja़a jaise sankramana bhi ho sakate haim |
vyaktitva bhi avasada mem eka bhumika nibha sakata hai |
yadyapi koi bhi vishesha paristhitiyom ke antargata avasada-grasta ho sakata hai, apane vyaktigata sanrachana ya pichhale anubhavom ke karana kuchha loga dusarom se adhika ati samvedanashila hote haim
aisa lagata hai ki striyam purushom se adhika bara avasada-grasta hoti haim |
isaka matalaba yaha bhi ho sakata hai ki striyam adhika tanava mem ho sakati haim, jaise ki kama hona aura usi samaya bachche ki dekhabhala karana |
yaha kisi ashanta karane vali ghatana ki pratikriya bhi ho sakati hai, jaise ki kisi mitra ya sambandhi ki nrityu, kisi mahila-mitra ya purusha-mitra se vichchheda, ya vidyalaya mem asaphalata |
kishora jinamem kama svabhimana hota hai, ve atyadhika sva-alochanatmaka hote haim, aura jo nakaratmaka ghatanaom para niyantrana ka thoda़a sa samvedana mahasusa karate haim, ve jaba tanavabhari ghatanaem anubhava karate haim taba visheshatah avasada-grasta hone ke khatare para hote haim |
vayovriddha mem avasada pata karana kai karakom dvara jatila ho sakata hai |
aksara, avasada ke lakshana budha़e hone ki prakriya ya kisi chikitsiya avastha se sambaddha hote haim bajaya avasada se|
sahayogi karakom mem pati/patni ya kisi ghanishtha mitra ke khone, chirakalika darda aura bimariyom, gatishilata ya yadadashta ka kshaya, badalati paristhitiyom jaise eka ghara se kisi sevanivritta suvidha mem jane ya parivara ke andara parivartanom ka samana karane mem kathinaiyam sammilita haim |
avasada kisi chikitsiya samasya ka koi sanketa bhi ho sakati hai |
yaha ayu badha़ne ke satha alja़imarsa roga ya ऑrgainika brena sindroma jaisi mastishka gada़bada़iyom ke karana bhi jatila ho sakata hai |
manochikitsa aura prati avasadaka aushadhiyam avasadakari gada़bada़iyom ke upachara mem prayoga ki ja sakati haim |
prati avasadaka aura manochikitsa ka eka sanyojana inamem se kisi eka vidhi se adhika achchhe parinama dete hue dikhai pada़te haim 
aushadhi sapeksha rupa se lakshana se turanta rahata degi aura manochikitsa rogi ko jivana ki samasyaom ka adhika prabhavi rupa se samana karane mem sahayata karati hai |
manochikitsa mem shamila hai eka prashikshita paramarshadata ya manochikitsaka se vicharom ke bare mem purnata: bata-chita karana |
kabhi-kabhi, apani vastavika bhavanaem vyakta karana kathina hota hai yaham taka ki ghanishtha mitrom se bhi |
eka chikitsaka, koi bhi jo bina nirnaya die vastava mem sune aura samaje, se bata-chita karana, ashcharyajanaka rahata la sakata hai | 
yadi apaka avasada tivra hai ya eka lambe samaya se chala raha hai to prati avasadaka avashyaka haim |
prayah upayoga kie jane vale prati avasadakom mem traisaiklika entidipreja़entsa (tisi eja़), selektiva serotoniyama ri-apateka (esaesaaraaija़), monoemaina ऑksideja़ inahibitarsa, lithiyama aura kuchha naye prati avasadaka jaise serotonerjika noradrinerjika ri-apateka inahibitarsa shamila haim |
lithiyama aura anya manodasha sthirakari, jaise ki sodiyama valproeta aura karbaameja़paina, prati avasadakom ki prabhavakarita ko badha़ane ke lie prayah upayoga kie jate haim |
yaha samajana mahatvapurna hai ki prati avasadaka, jaise hi apa unhem lete haim turanta hi kama karana nahim shuru karate haim | unaki kriya dhimi hoti hai aura apako pahale haphte ya do haphte taka sahanashila rahane ki avashyakata hai |
jaba apa achchha mahasusa karate haim, to yaha mahatvapurna hai ki apa turanta hi aushadhiyam na banda karem kyonki apa avasada mem phira se avanata ho sakate haim | 
apa apane daॉktara ko salaha dene dem ki kaba rokana hai |
apa eka samanya poshahara kha sakate haim (yadi nahim, to apaka daॉktara apako bataega) aura ve dardanashakom, pratijaivikom aura munha se khai jane vali garbhanirodhaka goliyom ke satha anukula haim |
apako madya se bachana chahie kyonki yaha goliyom ke satha hastakshepa karega aura apako atyadhika susta banaega |
daijepama jaisi prati-vyagrata aushadhiyam prati avasadaka nahim haim |
ve kabhi-kabhi prati avasadakom ke satha likhi jati haim lekina kisi avasadakari gada़bada़i ke lie ve akele nahim li jani chahie |
atyadhika avasada-grasta kuchha hi loga ina upacharom se achchhe nahim hote haim ya aushadhiyom ke prati asahanashila hote haim |
isamem sharira ki mansa-peshiyom ko tanava mukta karane ke lie eka halka nishchetaka aura eka aushadhi shamila hote haim aura taba kuchha sekendom ke lie mastishka mem vidyuta dhara pravahita ki jati hai |
yaha puri prakriya lagabhaga pandraha minata leti hai aura hara rogi isase avagata hota hai ki ve nidra mem chale gae the |
isiti ka eka korsa chhah se 12 upacharom ka hota hai, samanyatah eka haphte mem tina bara diya jata hai |
prati avasadaka upashamaka nahim hote haim halanki ve kama vyathita mahasusa karane mem apaki sahayata kara sakate haim |
ve avasada-grasta logom ko vapasa samanya karane mem bahuta prabhavashali hote haim |
halanki, prati avasadaka kuchha logom mem halke ya samanyatah asthayi parshva prabhava utpanna kara sakate haim | 
phira bhi, yadi parshva prabhava asamanya haim ya apaki dainika kriyaom mem hastakshepa karate haim, to ise apane daॉktara ko batane mem na hichakichaem |
yadi apako hala hi mem kuchha buri khabarem milim haim ya apake jivana mem koi mukhya sankata hai, to apane karibi logom se apa jaisa mahasusa karate haim usase sambandhita bata kijie | 
kuchha vyayama ke lie ghara se bahara nikalie, kevala tahalane ke lie hi |
yadyapi apa kama karane ke kabila nahim mahasusa kara sakate haim, to bhi kuchha halki kriyaem jaise grihakarya karate rahane ki koshisha karana hamesha achchha hota hai |
yaha apake mastishka ko una darda bhari bhavanaom se dura rakhane mem sahayata karega |
achchha, santulita ahara niyamita rupa se khaem, yaham taka ki jaba apa khane jaisa na mahasusa karem taba bhi |
apane dukhom ko madya ke satha dubone ke pralobhana ko rokem |
madya vastava mem manodasha ko dabata hai, isalie yadyapi ye apako tatkalika rahata de sakata hai, yaha bahuta asthira hota hai aura apa antatah aura adhika avasada-grasta ho sakate haim |
yaha apake sharirika svasthya ke lie bhi kharaba hai |
anidra ke bare mem bahuta adhika chinta na karem |
na so pane ke bare mem anavashyaka rupa se pareshana na hom |
yaham taka ki yadi apa vastava mem soye nahim haim phira bhi, apane sharira ko arama dena sahayata karega |
yadi avashyaka ho, daॉktara kuchha prashanta karane vali aushadhiyam bhi likhenge jaba taka ki apaki manodasha sudharati nahim hai |
apane apako yada dilaiye ki avasada kuchha aisa hai jisase anya loga bhi gujara chuke haim, aura yaha ki apa antatah isase bahara nikala aengem |
avasada eka upayogi anubhava ho sakata hai |
paristhitiyam aura sambandha aura adhika spashtatah dekhe ja sakate haim, aura apake pasa aba apane jivana mem mahatvapurna nirnaya aura parivartana karane ki shakti aura viveka hoga, jinase apa pahale bacha rahe the | 
sabase mahatvapurna chija़ jo koi bhi usa avasada-grasta vyakti ke lie kara sakata hai vaha hai uchita roga nidana aura upachara pane mem sahayata karana |
unake lie niyojita bhenta ki vyavastha kijie aura unhem apane satha daॉktara ke pasa le jaie | 
unhem taba taka upachara ke satha rahane ke lie protsahita kijie jaba taka ki lakshana ghatane na shuru ho jaem (kai haphte), ya yadi koi sudhara nahim hota hai to bhinna upachara khojane, aura yadi avasada-grasta vyakti aushadhi le raha hai to nirikshana karane mem sahayata karem |
bhavanatmaka sahara bhi bahuta mahatvapurna hota hai |
unake satha samaya vyatita karem aura unhem bata-chita karane aura kuchha karate rahane ke lie protsahita karem |
lagatara protsahana anamola hota hai kyonki avasada-grasta vyakti mem atma vishvasa ki kami hoti hai aura chintaom aura shankaom ki ora pravritta hote haim |
yadi avasada-grasta vyakti ki sthiti aura kharaba ho rahi hai aura atma hatya ke vichara ane shuru ho gaye haim, to ina kathanom ko gambhirata se lem aura unhem avasada-grasta vyakti ke chikitsaka ko bataem |
manasika bimari kisi ko bhi prabhavita kara sakati hai |
yaha kisi ayu sima, arthika sthiti, jati ya dharma ko nahim janati |
manasika gada़bada़iyam bahuta halki, asthira tanava sambandhita gada़bada़iyom se lekara atyadhika aura dirghakalina gada़bada़iyom taka bhinna-bhinna hoti haim |
singapura mem lagabhaga 16% loga halki manasika gada़bada़iyom se pida़ita haim |
manasika bimari koi akeli bimari nahim hai balki kai vibhinna prakara ki bimariyom se bani hai |
isamem kai prakara ki manasika auऱ bhavanatmaka gada़bada़iyam shamila haim jo karanom, lakshanom aura upachara mem bhinna-bhinna hoti haim |
yaha bimari, jivana ki sadharana jaruratom aura dinacharya ka samana karane ki akshamata mem parinita hote hue chintana, bodha aura vyavahara mem halki se atyadhika taka pareshani utpanna karati hai |
manasika bimari 2 mukhya samuhom mem vibhajita ki ja sakati haih manastapi bimari (mukhya manasika bimari), udaharanatah saija़ophreniya, mainika-dipresiva saikosisa aura bhrantimulaka gada़bada़i |
manastapi bimari vale loga vastavikata se sambandha kho dete haim aura sansara ko dusarom se bhinna rupa se dekhate haim |
ve mithya vishvasa vikasita kara sakate haim matalaba shreshthata, aura utpida़na, aura vibhranti ka jutha vishvasa anubhava karate haim| 
ve jo chija़em vaham nahim haim unhem dekha, suna, sungha, svada le aura anubhava kara sakate haim |
ve apane jivana ki paristhitiyom ke prati jarurata se jyada avasada-grasta ya ullasita ho sakate haim |
amanastapi bimari (mamuli manasika bimari), udaharanatah vyagrata gada़bada़iyam, badhyakari gada़bada़i aura khane ki gada़bada़i | 
amanastapi bimari ke lakshana samanya prakara ke vyavahara aura bhavanaom ke atishayoktipurna rupa haim |
amanastapi bimari vale loga aniyantrita vyagrata, tanava, dara aura avasada mahasusa kara sakate haim |
ye bhavanaem dainika kriyaom ka samana karane mem kathinai mem parinita hote hue mahatvapurna vyaktigata vyatha utpanna kara sakati haim |
manasika bimari ke sanketom aura lakshanom mem sammilita haim aspashta aura avastavika socha, khuda se bata karana, bina kisi achchhe karana ke hansana ya rona, tivrata se aura avirata bata karana, eka vishaya se dusare para kudana, dirghakalika udasi aura chida़chida़apana, atyadhika khushi aura udasi ki bhavana, atyadhika dara, chintaem aura vyagrata, samajika pratyaharana, khane ya sone ki adatom mem akasmika parivartana, krodha, bhranti ya matibhrama ki sashakta bhavanaem, dainika kriyaom ka samana karane ki akshamata, atmahatya ke vichara, bahuta sari avyakta sharirika shikayatem, spashta samasyaom ka inkara ityadi |
manasika bimari, uchcha raktachapa aura madhumeha jaisi bahuta si sharirika bimariyom ki taraha, puri taraha se thika nahim ki ja sakati hai lekina upachara kiya ja sakata hai |
uchita dekhabhala aura upachara samanya kriyaom ke punararambha mem parinita hote hue manasika bimari ke lakshanom ko ghata aura dura kara sakate haim | 
manasika bimari ke upachara mem samanyatah prati manastapaka aura prati avasadaka aura paramarsha ya manochikitsa ka eka mishrana sammilita hai |
manasika bimari ko kisi bhi anya chikitsiya bimari ki taraha dekhana sikhem |
yadi koi vyakti ajiba taraha se vyavahara karata hai to svikara karem ki yaha eka manasika bimari ka lakshana ho sakata hai |
manasika rupa se bimara ke prati sahayaka aura sahanubhutipurna rahem |
manasika bimari ke sanketa vale logom ko manovaijnanika upachara lene ke lie protsahita karem |
apadaem logom ko kai tarike se prabhavita karati haim |
kuchha paristhitiyom mem isaka matalaba hai sambandhiyom, mitrom aura pada़osiyom ko bhukampa, badha़ ya yuddha jaisi apadaom mem khona | 
anya mem, isaka matalaba balatkara, luta, gharelu hinsa, bala durvyavahara, apaharana aura kara ya relagada़i durghatanaom jaise abhighataja ghatanaom ka prakata hona ho sakata hai |
vyakti jo abhighataja ghatanaom ka anubhava kara chuke haim ghatana se sambandhita manovaijnanika tanava se pida़ita hote haim |
adhikatama udaharanom mem, ye asamanya paristhitiyom mem samanya pratikriyaem haim |
ye bhavanatmaka pashchapraghata abhighataja ghatanaom ke turanta bada, ya yaham taka ki haphtom ya mahinom bada bhi prakata ho sakate haim |
isake sanketa aura lakshana tivrata aura avadhi mem bhinna-bhinna ho sakate haim |
kabhi-kabhi abhighataja ghatanaem itani dardanaka hoti haim ki vyakti mahasusa karate haim ki ve apane jivana ka niyantrana phira se prapta karane mem asaphala haim |
isaka yaha matalaba nahim hota ki vaha vyakti divana ya kamaja़ora hai |
yaha kevala darshata hai ki vaha vishishta abhighataja ghatana bina vyavasayika sahayata ke sambhalane ke lie bahuta shaktishali thi | 
sabhi logom ko, jo manasika aghata ka anubhava karate haim, upachara ki avashyakata nahim hoti hai |
kuchha loga parivara, mitrom ya dharmika netaom ki sahayata se achchhe ho jate haim |
anya logom ko kisi atishaktishali abhighataja ghatanaom ka anubhava karane, pratyakshadarshana karane ya pratibhagi hone se utpanna ho sakane vali manovaijnanika kshati se saphalata purvaka achchhe hone ke lie vyavasayika sahayata ki avashyakata hoti hai |
vyavasayika manasika svasthya sahayata ko prapta karane ka vichara karana avashyaka ho sakata hai kyonki ho sakata hai ve posta traॉmaitika stresa disaॉrdara ya pitiesadi se pida़ita hom | 
posta traॉmaitika stresa disaॉrdara (pitiesadi) ka roga nidana aura lakshana eka durbala sthiti hai jo eka bhayavaha ghatana ke bada hoti hai |
lakshana halke ya tivra ho sakate haim, udaharana ke taura para pida़ita vyakti asani se chida़chida़e ya hinsaka avegom se yukta ho sakate haim
tivra mamalom mem unako kama karane ya milanasara hone mem kathinai ho sakati hai |
pitiesadi ke roga nidana kasauti mem vastavika ya bhayabhita karane vali nrityu ya ja़khma, ya usa vyakti ya anya logom ki sharirika akhandata ke prati ashanka shamila kiye hue apati ghatana ke pradarshana ka itihasa sammilita hai | 
isa abhighataja pradarshana ke daurana, vyakti atyadhika dara, asahayapana ya santrasa mahasusa karata hai |
lakshanom ke tina mukhya varga haim jo abhighataja ghatana ke bada hote haim |
aksara pitiesadi se pida़ita logom mem ghatanaem hoti haim jaba ve achanaka, suspashta yadom ke rupa mem abhighataja ghatana ko phira se mahasusa karate haim joki dardanaka bhavanaom ke satha hota hai| 
yaha eka phlaishabaika kahalata hai- eka punarsmarana jo ki itana shaktishali hota hai ki vyakti sochata hai ki vaha vastava mem usi manasika aghata ko phira se anubhava kara raha/rahi hai ya use apani ankhom ke samane prakata hota dekhata hai |
manasika aghata se pida़ita bachchom mem, manasika aghata ka yaha punaranubhava punaravritta krida़a ke rupa mem hota hai|
yaha punah anubhavana daravane sapane ke rupa mem bhi ho sakata hai |
yuva bachchom mem, duhkhada sapane, dushtatmaom ke daravane sapane ya khuda ya dusarom ke lie bhaya mem vikasita ho sakate haim |
kabhi-kabhi, punah anubhava bhavanaom ke achanaka aura darda bhare akramana ki taraha a sakate haim jo bina karana ke hote dikhate haim|
ye bhavanaem samanyatah dukha, dara ya krodha ki hoti haim|
ve jo pitiesadi se pida़ita hote haim prayah parivara, sahakarmiyom aura mitrom ke satha ghanishtha bhavanatmaka sambandhom se bachate haim |
vyakti sunna mahasusa karata hai, kama bhavanaem rakhata hai aura barambara kahata hai ki ve bhavanaom ko mahasusa nahim kara sakate haim visheshatah unaki ora jinake ve sabase karibi hote haim | 
pitiesadi vale loga una paristhitiyom se bhi bachate haim jo unhem abhighataja ghatanaom ki yada dilati haim kyonki ye lakshana aura kharaba ho sakate haim |
udaharana ke lie eka yuddha anubhavi sainika tagada़i pratikriya kara sakata hai jaba vaha vardi pahane logom ko dekhata hai |
pitiesadi se pida़ita bachche apane vichara-bhavom mem apane bhavishya ki ora eka ullekhaniya parivartana dikha sake haim |
udaharana ke lie, bachcha vivaha ya vyavasaya karane ki ummida nahim bhi kara sakata hai | 
pitiesadi vale loga aise bhi abhinaya kara sakate haim jaise ki ve abhighataja ghatanaom dvara nirantara darae ja rahe haim |
ve chida़chida़e ho sakate haim aura unhem dhyana kendrita karane mem kathinai ho sakati hai |
ve teja़ avaja़om se asani se chaunka jate haim |
kabhi-kabhi pitiesadi vale loga atanki akramanom se pida़ita hote haim jaham lakshanom mem jo unhonne manasika aghata ke samaya mahasusa kiya tha usase milata-julata atyadhika dara shamila hai |
ve prasvedita mahasusa kara sakate haim, unhem sansa lene mem kathinai hoti hai aura unaki hridaya gati badha़ti hui mahasusa karate haim |
ve michali aura chakkara ki shikayata kara sakate haim |
bahuta se manasika aghata se pida़ita bachchom mem sharirika lakshana hote haim jaise ki peta darda aura sira darda | 
sabhi uparyukta lakshana eka mahine se adhika raha sakate haim aura mahatvapurna samajika aura vyavasayika gada़bada़iyom mem parinita hote haim | 
pitiesadi se pida़ita vyaktiyom ko prayah sahagami samasyaem hoti haim jaise ki draga aura madya ka dushprayoga |
ve anya lakshanom ka bhi anubhava karate haim jo ki manodasha aura vyagrata ki gada़bada़iyom se sambandhita hote haim jaise ki avasada-grasta, dukhi mahasusa karana aura ninna urja, chida़chida़a, shighra uttejita, naraja़ aura vidveshi hona, jivita rahane ke lie, jinda bane rahane ke lie, anakahi aura anahui chija़om ke lie aparadhi mahasusa karana, dara hua aura vyagra mahasusa karana, sone aura dhyana kendrita karane mem kathinai, anayasa rona, nirasha aura asahayapana ka bhava mahasusa karana jo atmahatya ke vicharom ki ora le ja sakata hai |
isalie abhighataja tanava lakshanom aura sthitiyom ke eka pure samuha ko jaldi la sakata hai |
pitiesadi ka prakatana unamem se jo apati ghatanaom ke samparka mem a chuke haim ke 3% se 58% taka hota hai |
yaha isalie hai kyonki kuchha prakara ki ghatanaem anya se adhika abhighataja hoti haim |
pitiesadi ka khatara yuddha ke kaidiyom, aparadha pida़itom aura sharirika ya yauna dushprayoga aura rajanaitika yantrana jaisi dirghakalika, punaravartita antarvyaktigata hinsa se pida़itom mem adhika hota hai |
upachara ke taura-tarikom ko mote taura para manovaijnanika aura aushadhavaijnanika (aushadhiyom) upacharom mem vargikrita kiya ja sakata hai |
prati avasadakom aura manodasha sthirakarakom ke satha-satha vyagrata kama karane vali aushadhiyam, avasada, phlaishabaika, ati uttejana aura ninda ki samasyaom ke lakshanom ko ghata sakati haim | 
manochikitsa ke satha sammilita bodhatmaka vyavahariya chikitsa, pratyeka vyakti ke lie eka prabhavakari chikitsiya hastakshepa pradana karane ke lie ye aksara sammishrana mem prastavita kie jate haim |
ye upachara niyantrana ki anubhuti ko phira se sanchita kara sakate haim aura vartamana anubhava para bhutapurva ghatanaom ki shakti ko ghata sakate haim |
jitani jaldi logom ka upachara kiya jata hai, unamem manasika aghata pahunchane vale anubhava se ubarane ki utani hi adhika sambhavana hoti hai | 
uchita chikitsa bhi chirakalika manasika aghata-sambandhi gada़bada़iyom vale logom ki sahayata kara sakati hai| 
shoka manane ke lie apane apa ko anumati aura samaya dem |
apa eka abhava se pida़ita haim |
apani shakti aura samana karane ki kshamata para dhyana kendrita karem |
apani prathamikataom ko phira se simankita karem aura apani urja aura sadhanom ko una prathamikataom para sankendrita karem | 
badhaom se mukabala karane mem sahayata karane ke lie chhote-chhote vastavika lakshyom ko nirdharita karem |
svasthyaprada bhojana khaem aura vyayama karem |
yaha tanava se mukta karane mem sahayata kara sakata hai |
apani sanchita shakti ko badha़ane ke lie paryapta arama karem |
anasulaje vishayom ko svikara karem aura upachara ke lie avashyaka parivartana karane ke lie ja़khma aura darda ko eka abhipreraka ki taraha upayoga karem |
bata-chita karana' eka arogyakara aushadhi hai |
priyajanom aura mitrom ka sahara svikara karem |
sharaba evam avaidha davaiyom se bachiye| 
ye madada karane ki bajaya aura adhika pareshaniyam paida karati haim| 
maleriya sansara ki sarvadhika mahatvapurna parajivijanya bimari hai, jisake varshiki 3 miliyana mamale darja hote haim evam 1.5 se lekara 2.7 miliyana nrityuem hoti hai| 
bahuta se logom ko ushna evam uposhniya deshom ki yatra ke daurana maleriya ho jata hai | 
sabasahariya aphrika, dakshina aura dakshinapurvi eshiya, maiksiko, hayati, madhya aura dakshina amarika, papua nyu gina evam solomana dvipom ke adhikanshatah bhagom mem maleriya hota hai| 
yatra karane valom ke lie maleriya hone ka khatara vishesha khatara karakom para nirbhara karata hai, jaise - gramina ilakom ki yatra, ratri samaya ghumana, parade rahita avasom, davaiyom ke prabhava shunya kshetrom ki yatra, yatra ka samaya ( bimari phailane ka uchcha aura ninna mausama) adi| 
2000 mitara se adhika ki unchai para maleriya ka sanchara durlabha hai| 
maleriya eka aisi bimari hai jo eka sankramita enaphilija machchhara ke eka vyakti ko katane aura maleriya ke parajivi ko usake rakta mem chhoda़ne se hoti hai | 
maleriya parajivi ki chara prajatiyam (plaja़modiyama phelsipairama, pi.vaivaiksa, pi.ovaila evam pi.maleriya) manushyom mem sankramana ka sanchara kara sakati haim| 
plaja़modiyama phelsipairama ka sankramana tivrata se badha़te hue janaleva bimari mem badala sakata hai| 
phelsipairama maleriya se pida़ita 1 se 2% marija nrityu ke shikara ho jate haim| 
maleriya ke lakshana phlu jaise haim aura isamem bukhara, thanda, pasina, mansapeshiyom ka darda, siradarda aura kabhi-kabhi ulti evam dasta adi shamila haim| 
phelsipairama maleriya ke marijom mem gurde ki kharabi, ainthana evam sammurchha utpanna ho sakati hai| 
pi.vaivaiksa aura pi. ovela ke sankramana yakrita mem kai mahinom taka suptavastha mem raha sakate haim, aura mahinom uparanta ya yaham taka ki salom ke bada bhi punah upasthita ho sakate haim| 
machchhara ke katane aura maleriya ke lakshanom ke ubharane ke bicha ki avadhi samanyatah 7 se 21 dina hoti hai, lekina agara yatra karane vale ne maleriya ke adhure ya aparyapta bachava sadhana lie hue haim to yaha antarala lamba bhi ho sakata hai| 
yatra karane vale ko yadi, maleriya pravana kshetrom ki yatra ke daurana ya bada mem bukhara ata hai, to isaka karana maleriya na ho, dekhane ke lie turanta chikitsiya madada leni chahie| 
maleriya parajivi ke lie eka rakta namune ki jancha honi chahie| 
yatri maleriya virodhi sadhanom se evam maleriya se bachava ki davaem lekara, apane apa ko bacha sakate haim| 
koi bhi sadhana jo shama evam rata ko katane vali mada enaphilija machchhara ke jokhima ko kama karata hai, sankramana se bachane ke lie eka sarvottama tarika hai| 
agara apaka shayanakaksha vatanukulita ya paradom se dhaka hua nahim hai to palanga para machchharadani lagaie| 
atirikta bachava ke lie, machchharadani para paramethrina namaka kitanashaka chhida़kie| 
baithane vali ya sone vali jagahom para paramethrina ya samana kisma ka kitanashaka chhida़kie ya eka machchhara kaॉyala jalaie jo payarithrama bahara chhoda़ta ho| 
yatra karane valom ko maleriya hone vale kshetrom mem prasthana karane se purva, apane daॉktara se salaha leni chahie| 
maleriya pravana kshetrom ki yatra karane valom ke lie, maleriya se bachava ki davaiyam nirdharita ki jati haim| 
bachava ki avashyaka matra nirdharita karane ke lie, eka samuchita yatravivarana jaruri hai| 
yatra karane valom mem, maleriya se bachava ki svikrita ama davaiyom mem mephlokvina, doksisaiklina aura klorokvina adi shamila haim| 
prabhavapurna bachava ke lie, svikrita khurakom ka sakhta palana aura maleriya virodhi nirdharita davaiyom ki suchi avashyaka hai| 
pratyeka saptaha ke usi dina ya pratyeka dina ke usi samaya goli lijie| 
yatra ke 1 se lekara 2 haphtom se purva, yatra ke daurana aura maleriya pravana kshetrom ko chhoda़ne ke 4 haphtom ke bada bhi svikrita khurakem lem(doksisaiklina ko chhoda़kara, jo yatra se 1 dina pahale li jani chahiye)| 
ghara pahunchane ke bada bhi davaem lena banda na karem,kyonki puri khurakom ko purna karana ati avashyaka hai| 
sabhi davaiyom ke dushparinama hote haim| 
halki mitali, kabhi-kabhi ulti evam chakkara ana adi maleriya virodhi davaiyom ko banda karane mem yatheshta karana nahim haim| 
maleriya ki gambhirata, asthayi parshva prabhavom ko sahana karane ka samarthana karati hai| 
phira bhi, agara koi gambhira parshva prabhava hota hai, to yatra karane valom ko daॉktari madada leni chahie aura maleriya virodhi davaiyom ko lena banda kara dena chahie| 
bujurgom ki vyavaharika evam manasika pareshaniyom ka adhyayana manah jarachikitsa kahalata hai| 
budha़e logom mem, sharirika, bhavanatmaka evam samajika badalava jo badha़ti unra ki prakriya ke satha utpanna hote haim, manovaijnanika pareshaniyam paida kara sakate haim| 
udasi (vishada) eka bhavanatmaka sthiti hai jo atyadhika duhkha ke anubhava, udasi evam asahayata se pahachani jati hai| 
budha़e logom mem udasi evam atmahatya ka khatara javana logom se kahim adhika hota hai, kyonki daॉktari bimariyam, sharirika avastha mem badalava, aura sevanivritti se hui aya ki hani, mam-bapa evam mitrom ki nrityu, jivana sathi ki nrityu evam avasiya vyavastha mem badalava adi jaisi pramukha haniyam jivana ke antima charanom mem hoti haim| 
bahuta se vishada chuka jate haim ya parivara ke dvara samanya se soche jate haim aura isilie anakhoje raha jate haim| 
anavashyaka kashta se bachane ke lie jaldi khoja evam uchita upachara bahuta mahatvapurna hai| 
udasi ke ama lakshanom mem manodasha mem hala ka badalava, vishesha taura para nirashavadi, nirasha aura prasannachitta ki hani adi shamila haim|
kama ekagrata, alasyapana, kriyakalapom mem aruchi, aparadhabodhata, bhukha aura ninda mem badalava adi ho sakate haim| 
bhayankara baicheni ya sharirika shikayate jaise asamanya lakshana bhi ho sakate haim| 
kabhi-kabhi, udasi yadadashta mem badalava ka karana bhi ho sakati hai| 
deliriyama achanaka vyavaharika badalava jaise uttejana, asavadhani, pratikulata evam yadadashta mem badalava jo dina ke badha़ne se upara niche hote rahate haim,isase jani jati hai| 
parivara ke lie, prabhavita vyakti samaya samaya para bhramita laga sakata hai| 
sone-jagane ka chakra samanyatah gada़bada़a jata hai, aura ye utara chadha़ava shama aura rata ko adhika pramukha hote haim| 
deliriyama ko tatkala chikitsiya upachara ki avashyakata hoti hai, kyonki yaha anya gambhira khatarom se juda़i hui hai| 
yaha sankramanom, eka mamuli daure, anidaniya ya asantoshajanaka dha़nga se niyantrita madhumeha, atyadhika tanava(uchcha raktachapa) adi se bhi ho sakati hai| 
kabhi-kabhi kuchha davaiyom ke parshrva prabhava se bhi deliriyama ho sakata hai| 
jisako pahali bara deliriyama hua ho, usa vyakti ko antarnihita karana pahachanane ke lie evam usake upachara se lie aspatala mem bharti karane ki jarurata hoti hai| 
agara deliriyama bina upachara ke chhoda़ diya jata hai, to yaha pagalapana ya nrityu taka badha़ sakata hai| 
pagalapana bodhana (manasika) kshamataom ka kshaya hai jo samajika ya vyavasayika karyom ko prabhavita karata hai| 
deliriyama ke viruddha, pagalapana ke lakshana dhire-dhire utpanna hote haim| 
bodhana kshamataom ke kshaya ka yadadashta, amurta socha, samasya hala karana, aura nirnaya lena, padha़na, likhana, rekha chitrana karana evam bolana adi ko prabhavita karata hai| 
budha़e loga asantoshajanaka alpavadhi yadadashta ka anubhava kara sakate haim, asa-pada़osa ka rasta khojane evam bata karate samaya sahi shabdom ke chunava mem kathinaiyom ka samana kara sakate haim | 
jaise-jaise pagalapana badha़ta hai, budha़om ko khilane mem, unaki saphai karane mem, kapada़e pahanane mem evam shaucha adi jaise roja ke jivana nirvaha ke karyom mem nirikshana evam sahayata ki avashyakata hoti hai| 
bahuta se kashtabhogiyom ko vyavaharika vishamataem nahim hoti| 
phira bhi, jaba ye hoti haim, to avarti matibhrama, ayogya samajika vyavahara evam damva-dola manodasha adi vyavaharika vishamataem ho sakati haim| 
pagalapana se piड़nonmadi sandeha karate haim ki dusare unaki chijom ko chura rahe haim jaise vyavaharika evam vyaktitva badalava ho sakate haim| 
kabhi-kabhi vyavaharika lakshanom mem se kuchha, manodasha mem gada़bada़i ya deliriyama ke karana bhi ho sakate haim| 
agara ye shuru mem hi pahachana lie jaem to prayah saphalatapurvaka inaka upachara ho sakata hai| 
jaise ki pagalapana mem yadadashta ka kshaya samanyatah dhire-dhire hota hai, isilie parivarom ko chhaha mahinom se adhika bita jane para bhi badalava ke bare mem jagarukata nahim hoti| 
elaja़amera ki bimari evam naड़i-sambandhi pagalapana do sarvadhika samanya prakara ke pagalapana haim| 
50% se adhika pagalapana elaja़amera ki bimari ke karana hota hai|
isa sthiti mem, mastishka ki koshikaem samanya se adhika tivra gati se marati jati haim, jisase bodhana karyom ka kshaya tivrata se hota hai|
pagalapana ka dusara sarvadhika samanya prakara naड़i-sambandhi pagalapana hai| 
isaka antarnihita karana ishachimiya(kama rakta pravaha) ya mastishka ke vibhinna bhagom mem ruke hue daure haim| 
adhikanshatah, atyadhika tanava ya dayabitija़ melitasa ki prishthabhumi ho sakati hai jo naड़i-sambandhi pagalapana ko ubharane ke khatarom ke mahatvapurna karaka ho sakate haim| 
pagalapana ke anya karana kama samanya haim| 
pagalapana ka sahi-sahi karana pahachanane ke lie , eka purna daॉktari jancha ki, kabhi-kabhi siti mastishka jancha ki bhi avashyakata hoti hai| 
manoroga gada़bada़iyam, socha-vichara ki avyavastha ko batati hai jo tarka evam vastavikata ke bodha ko prabhavita karati hai| 
matibhrama, vibhrama evam antardrishti hani adi pramukha lakshana haim| 
matibhrama taba hota hai jaba vyakti eka vastu ko dekhata hai jo vaham nahim hai, udaharana ke taura para-eka vyakti avajem suna sakata hai jabaki pasa mem koi srota bhi nahim hota hai ya tvacha para rengata hua koi kalpanika kida़a mahasusa kara sakata hai| 
vibhrama nishchita hote haim, juthe vishvasa asangata hote haim inaka vyakti ki shiksha aura dharmika prishthabhumi ke satha koi lena dena nahim hota hai| 
vyakti apani jindagi se tanga evam dara hua mahasusa kara sakata hai,ya bhramita ho sakata hai ki usaka jivanasathi bevapha hai ya yaha ki ve eka kalpanika bimari se pida़ita haim| 
badha़ti unra mem manovikriti, sunane aura dikhai dene ki kshati evam samajika ekantata se juda़i hui hoti hai| 
uttejana eka sandehaspada lakshana ho sakati hai, aura samarupi udasi mamalom ko jatila bana sakati hai| 
samanyatah manastapi virodhi davaiyam kama matra mem di jati haim| 
chaingi janarala haॉspitala mem, hamare pasa 65 varsha se adhika ayu vale logom ke lie eka vyapaka, bahu-vishayaka raya aura upachara yojana hai jo manoroga sambandhi pareshaniyom ka anubhava kara rahe haim| 
pratyeka vyakti ko unake sharirika, manasika, samajika evam vatavarana ke karakom para purna vistara se raya di jati hai| 
hamare dala mem manasika jarachikitsaka evam jarachikitsaka , manovaijnanika, manoroga ki narse, vyavasayika chikitsaka, malisha adi se ilaja karane vale chikitsaka aura samaja-sevaka adi shamila haim| 
eka bara jaba mulyankana purna ho jata hai, to vyaktigata stara para marija ki avashyakatanusara upachara karyakrama taiyara kiya jata hai| 
jaham kahim bhi sambhava hai, hama isa prakriya mem marijom evam unake parivarom ko sakriya rupa se shamila karatem haim| 
hamara jora, dekharekha karane vale evam sahayata dene vale vatavarana ko pradana karane para, aura unaki purna samarthya taka pahunchane ke lie, unake andara maujuda shakti, hunara ke palana poshana para dhyana kendrita karana hai| 
upachara vikalpom mem vyaktigata manoroga chikitsa, parivarika salaha, davaiyam lene ke hunara ka vikasa, samvedaka uttejana, tanava mukti evam vyayama chikitsa adi shamila haim| 
bujurgom ki bahuta si manasika gada़bada़iyom ka saphalatapurvaka ilaja kiya ja sakata hai| 
uchita hastakshepa, kathinai ko kama karane mem aura jivana ki gunavatta ko behatara banane mem eka antara la sakata hai| 
si ji echa aspatala mem bharti bimarom evam bahya marijom donom ke lie, manoroga jarachikitsiya sevaem pradana karata hai| 
marijom ke lie eka jarachikitsiya kendra bhi upalabdha hai| 
purahstha granthi ki yaha eka altrasaunda jancha hai| 
yaha jancha taba ki jati hai jaba apake daॉktara ko ashanka ho ki apako purahstha granthi kainsara hai| 
purahstha granthi kainsara ka sandeha taba kiya jata hai jaba apake rakta jancha mem prosteta vishesha entijana(piesae) 4 ena ji/ema ela se adhika hota hai ya apaka daॉktara dijitala malashayi jancha karane para purahstha granthi mem sandehajanaka granthika pata hai | 
transa rektala altrasaunda(tiarayuesa) mem samanyatah 15 se 20 minata lagate haim| 
isase apaka daॉktara apaki purahstha granthi ko eka vishesha altrasaunda nalika ka istemala karate hue dekha sakata hai evam kainsara ki upasthiti ki jancha ke lie purahstha granthi ke kai hissom se utakom ke namune le sakata hai| 
transa rektala altrasaunda eka dardarahita prakriya hai evam eka divasiya shalyachikitsa ki taraha ki jati hai| 
isa jancha ke lie koi vishesha taiyari ki bhi avashyakata nahim hoti| 
agara apa kabjiyata se pida़ita haim, to malashaya se mala ko sapha karane ke lie eka rechaka aushadhi ya enima ki avashyakata ho sakati hai| 
yadi apa esapirina ya varapherina jaisi rakta ka thakka na banane vali dava le rahe haim, to rakta bahane ke avasara ko kama karane ke lie, prakriya se purva sata dinom ke lie ina davaiyom ko lena banda kara dijie| 
yadi bayopsi (jancha ke lie purahstha granthi ka eka utaka lene) ki avashyakata hai, to darda dura karane vali aushadhi evam entibayotiksa di jayegi| 
isa jancha ke lie marijom se apane ghutanom ko chhati taka moda़kara, karavata lekara letane ke lie kaha jata hai| 
yaha sthiti malashaya ki altrasaunda salai ko andara dalane mem asana banati hai| 
salai ko pahale chikana kiya jata hai aura taba dhire se apake malashaya ke andara dala jata hai| 
yaha salai dhvani tarangom ko bahara phenkati haim jinhem altrasaunda mashina mem eka kampyutara para bheja jata hai| 
apaki praॉsteta granthi (purahstha granthi) ka eka shveta evam shyama patachitra ti vi maॉnitara para a jayega| 
salai ke eka tarapha eka chhoti nali hoti hai, jo apake daॉktara ko purahstha granthi ki bayopsi karane mem sahayata karati hai| 
jaise hi sui malashaya ki divarom se hokara gujarati hai aura purahstha granthi mem pahunchati hai jaham se utaka ka eka chhota tukada़a le liya jata hai, vaise hi apa eka teja, tivra dhakka mahasusa kara sakate haim | 
darda bahuta kama hota hai aura jaba taka prakriya puri hoti hai, taba taka samanyatah daba hi jata hai| 
khoja karane ke avasarom ko anukulata banane ke lie evam kainsara ki sthiti ko purnataya sthapita karane ke lie,jyada se jyada chhaha ya atha bayopsiyom ki avashyakata ho sakati hai| 
peshaba ya mala ya virya ke strava ke samaya thoda़a rakta ka hona eka bahuta ama bata hai| 
ye lakshana samanyatah samaya ke satha kama ho jate haim| 
pahale eka ya do dinom ke lie atirikta dravya padartha pine ke lie apako salaha di jati hai| 
bayopsi ke bada, mutratyaga karate samaya apako jalana mahasusa ho sakati hai| 
penadola jaisi dardanivaraka ina pareshaniyom se arama dila sakati hai| 
bayopsi ke bada purahstha granthi mem sujana ke karana, mutratyaga karate samaya apako pareshani mahasusa ho sakati hai| 
peshaba ko bahara nikalane ke lie eka peshaba nalika andara dali jati hai| 
mutratyaga karate samaya agara apa lagatara pareshani mahasusa karate haim, to eka antaryamita nalika andara dali jati hai evam apako aspatala mem bharti hona pada़ega| 
yaha mutrashaya ke tala para mutramarga ko ghera leti hai| 
mutramarga vaha rasta hai jo peshaba ko mutrashaya se linga ke antima sire taka le jata hai| 
purahstha granthi shukranuom ke poshana ke lie dravya utpanna karati hai| 
manushya ki unra ke satha purahstha granthi bhi badha़ti rahati hai| 
yaha badha़ta samanyatah hitakari hi hoti hai, lekina asadhya(kainsara) ho sakati hai| 
badha़i hui purahstha granthi mutramarga ka rasta sankara kara deti hai evam dakamutra pravaha ko avaruddha karati hai| 
agara avarodhakata se chhutakara nahim milata, to peshaba sharira mem hi ruka raha jata hai| 
isase mutramarga mem sankramana, mutrashaya ki pathari evam peshaba karane mem achanaka asamarthata adi ho jate haim| 
peshaba karane ki yaha asamarthata eka pida़adayaka sthiti hai, jisamem daॉktara ko eka rabada़ ki nalika(kathetara) mutrashaya ke andara dalane ki avashyakata hoti hai| 
shalya chikitsa dvara mutrashaya ki rukavata se arama pane ke lie, apako aspatala mem bharti hone ki avashyakata hogi| 
yadi apako peshaba karane mem pareshani ho, mutrashaya ka apurna rupa se khali hona, peshaba ka tapakana, aura visheshakara ratri mem bara bara mutratyaga karana adi pareshaniyom ka anubhava hota hai, to apaki purahstha granthi badha़i hui ho sakati hai| 
agara ye lakshana bane rahate haim ya bigada़ jate haim to daॉktara se milie| 
ina lakshanom se arama pane ke lie davaiyam upalabdha haim lekina yaha jaruri nahim ki ye apaki purahstha granthi ki pareshani ko badatara hone se bachaem hi| 
jaise hi mutramarga se mutra ka pravaha kama ho jata hai, mutrashaya ko khali karane ke lie jyada davaba ki avashyakata hoti hai| 
phalatah isase avashyakata se adhika khincha hua mutrashaya, gurdem ki kharabi evam peshaba ka atyadhika jamava adi ho jate haim| 
eka vishesha taura para dijaina kiye gaye yantra se jo mutramarga se hokara gujarata hai, purahstha granthi ka transayurethrala risakshana (tiyuarapi) kiya jata hai| 
tvacha para koi bhi chira nahim diya jata hai| 
badha़i hui purahstha granthi ka bhaga, shalya chikitsa ke daurana nikala diya jata hai| 
yaha santare ke gude ko nikalane jaisa hai| 
santare ke chhilake ki taraha, shalya chikitsa ke bada purahstha granthi ki kaipsula vaise hi rahegi|
isa prakara, shalyachikitsa se puri purahstha granthi ko nahim nikala jata, balki peshaba ke sahi pravaha ke lie jitana rasta kaphi hota hai utana hi utaka nikala jata hai| 
shalyachikitsa mem eka ghante se bhi kama ka samaya lagata hai| 
unnata shishna para koi prabhava nahim pada़ta, lekina skhalita virya ki matra kama ho jayegi| 
yaha isalie hota hai kyonki bada़a rasta bana jane se shukranuom vala virya apake mutrashaya mem chala jata hai| 
shalya chikitsa ke lie ridha़ ka nishchetana prayoga mem laya jata hai| 
ti yu ara pi ke daurana, pravaha marga ka akara badha़ane ke lie badha़i hui purahstha granthi ko nikala diya jata hai| 
prakriya ke daurana apa koi bhi darda mahasusa nahim karenge| 
yaham taka ki jaba nishchetana ka prabhava khatma ho jata hai, taba bhi kashta bahuta hi nyunatama hota hai| 
shalya chikitsa ke bada, mutrashaya mem rakta ka thakka banane se rokane ke lie,eka bada़i nalika mutrashaya ke andara dali jati hai| 
purahstha granthi utakom ke tukada़om ko bahara nikala diya jata hai| 
apaka mutravijnani apako ti yu ara pi prakriya ki avashyakata evam usaki sambhavita vishamataom ke bare mem samajayega| 
agara apako kisi dava se elarji hai ya agara apa esapirina aura rakta ko patala karane vali anya dava le rahe haim, to daॉktara ko suchita karem| 
sabhi chikitsa nirdesha evam jo davaiyam apa vartamana mem le rahe haim ,unhem satha laiye| 
vishesha chikitsaka klinika mem purva-pravesha satra ke daurana diya gaya apaka pravesha phaॉrma apane satha lana yada rakhie| 
agara apako ti yu ara pi ke lie bharti kiya gaya hai to apake lie upachara yojana niche di gai hai| 
vishesha chikitsaka klinika mem apaka daॉktara apake daॉktari vivarana ke bare mem puchhega aura apako, apake shalyachikitsa ki prakriti aura satha hi satha usaki vishamataom ke bare mem bhi samajayega|
jancha karane ke lie apa se, apaka rakta liya jayega aura yaha ashvasta hone ke lie bhi ki shalyachikitsa ke lie apaka svasthya achchha hai aura apa usake yogya haim| 
narsa apake vajana, tapa, nada़i dara, shvasana dara evam raktachapa adi ki jancha karegi| 
apaka daॉktara apake daॉktari vivarana ke bare mem puchhega aura apaki jancha karega| 
vaha apake shalyachikitsa ki prakriti ,usaki vishamataom ke bare mem punah samajayega aura shalyachikitsa ke lie apaki sahamati lega| 
nishchetaka shalyachikitsa ke lie apaki svasthata ka mulyankana karega| 
shalyachikitsa ke lie apa ekadama yogya haim,yaha ashvasta karane ke lie rakta (agara yaha pahale na liya gaya ho) liya ja sakata hai|
shalyachikitsa kaksha ki narsa apaka nama evam byaure ki jancha karegi| 
kuchha logom ko mutratyaga ki avritti, atyavashyakata evam asanyamata ka anubhava ho sakata hai, lekina yaha shalyachikitsa ke bada tisare mahine taka thika ho jate haim| 
3 se 4 haphtom ke lie bhari vajana uthane se ya thakana se bachiye, kyonki atyadhika jora se rakta baha sakata hai| 
agara apake peshaba mem khuna a raha hai, to niche leta jaiye aura arama kijie| 
jitani jaldi ho sakem utani jaldi 1 se 2 gilasa pani pijie| 
agali bara jaba bhi apa peshaba karem, rakta ana banda ho jana chahie| 
agara rakta ana jari rahata hai, to apane mutravijnani ko phona kijie ya apatakalina vibhaga aie| 
apako mutratyaga ki avritti, atyavashyakata evam asanyamata ka anubhava ho sakata hai| 
shalyachikitsa ke bada eka mahine ke lie apako yauna kriya se paraheja karana chahie| 
kabjiyata se bachane ke lie prachura matra mem phala evam adhika reshe yukta bhojana khaem| 
3 dina bada bhi agara apa malotsarga nahim kara pa rahem haim to rechaka aushadhi lem| 
agara apa shalyachikitsa se purva esapirina le rahem the, aura agara apake peshaba mem bahuta adhika rakta nahim a raha hai, to eka saptaha ke bada apa use phira arambha kara sakate haim | 
badha़i hui purahstha granthi ko rokane ke lie koi bhi tarika jnata nahim hai, kyonki yaha unra badha़ne ki prakriya ka eka hissa hai| 
kisi bhi shalyachikitsa ya nishchetaka mem khatara hota hi hai| 
phira bhi, ti yu ara pi ke labha khatarom se kahim adhika haim| 
yadi apako pahale kabhi ullekhaniya bimari ya chikitsiya pareshani hui ho, to apako jyada khatara hoga aura apaka daॉktara apaki purahstha granthi pareshani ke vaikalpika upachara ke bare mem charcha karega| 
sharira mem poshana drava ke samaveshana dvara, sanlakshana utpanna hota hai| 
ti yu ara sanlakshana ke chihnom mem - manasika ulajana, aruchi, ulti, uchcha raktachapa, dhimi hridaya gati evam drishti mem gada़bada़ adi shamila haim| 
ti yu ara sanlakshana davaiyom se thika kiya jata hai jo sharira se atirikta pani ko bahara nikalati haim| 
bahya sankochinii (sphinktara) (apake mutramarga ko banda rakhane ki mansapeshiyam) mem kshati hone ke karana asanyamata ka hona bahuta asamanya hai| 
yaha pahale se hi vidyamana adhika sakriya mutrashaya ke karana, samanyatah jyada ho sakata hai| 
agara mutrashaya ki sthiti yahi hai to apaka daॉktara davaiyom ka nuskha likhega | 
purahstha granthi ki punah badha़ta ke karana punaravritti ho sakati hai| 
badha़i hui purahstha granthi eka hitakari sthiti hai evam usase kainsara nahim hota| 
nikale gaye purahstha granthi utaka ki, kainsara ke lie hamesha jancha hoti hai| 
















kevala kuchha hi mamalom mem ,yaha kisi bhi sankramita vyakti se sidhe samparka mem ane se phaila sakata hai | 
aisa isalie hota hai kyonki isaka vayarasa sankramita vyakti ki lara mem maujuda ho sakata hai, aura isalie matra chumbana jaise samparka se hi sankramana phailane mem madada mila sakati hai |
poliyo vayarasa jaba eka bara apake sharira mem pravesha kara jata hai ,to yaha apake gale aura antom mem apani sankhya mem vriddhi karana arambha kara deta hai |
aura taba yaha apake rakta-sanchara ke madhyama se apake kendriya tantrika tantra taka pahuncha jata hai |
bahuta hi kama mamalom mem ,jaise hi yaha vayarasa apake tantrika tantra mem pahunchata hai ,yaha apaki tantrika koshikaom ko nukasana pahunchane lagata hai |
poliyo vayarasa se sankramita loga ,usake lakshana dikhai dene ke 7-10dina pahale aura bada mem sabase adhika sankramaka hote haim |
yaham taka ki unake mala se chhaha haphtom taka bhi vayarasa phaila sakata hai |
do sau se adhika vibhinna vayarasa haim jinase jukama ho sakata hai |
sabase ama vayarasa ina do samuhom mem se eka ka ho sakata haim - rahainovairasa aura koronavairasa |
eka ke bada eka kai jukama hone ka karana alaga alaga prakara ke kai vayarasa hona hai jinase jukama hota hai aura pratyeka bara jukama eka alaga vayarasa se hi hota hai |
jukama ke vayarasa naka aura gale ki antarika divara para hamala karate haim ,jisase ina hissom mem sujana a jati hai |
jaise hi ye suja jate hai unase adhika balagama banana shuru ho jata hai ,jisake parinamasvarupa naka bahane lagati hai aura chhinkem shuru ho jati haim |
jukama kai tarikom se phaila sakata hai |
yadi apako jukama hai to jaba bhi apa chhinkate ,khansate ya bata karate haim ,to jukama ke vayarasa se yukta pani ki chhoti chhoti bundem hava mem phaila jati haim |
jaba koi aura vyakti sansa ke dvara ina vayarasom ko andara leta hai to vaha bhi jukama se grasta ho jata hai |
pratyaksha aura apratyaksha samparkom ke madhyama se bhi jukama phaila sakata hai |
yadi apako jukama hai aura apa apani naka ya ankhom ko chhute haim aura phira kisi dusare vyakti ko chhute haim ,to apa vayarasa ko usa vyakti taka pahuncha dete haim |
yaham taka ki ,agara apa eka daravaja़e ke haindala ya teliphona jaisi vastuom ko bhi chhute haim to vayarasa una vastuom para chala jata hai |
yadi thoda़i dera ke bada hi koi unako chhuta hai aura phira apane munha ,naka ya ankhom ko chhu leta hai ,to vaha bhi jukama se pida़ita ho jata hai |
halanki 200 se adhika vayarasa samanya jukama kara sakate haim ,lekina una saba mem sabase ama jimmedara vayarasa rahainovairasa hai ,aura yaha behada sankramaka hota hai |
jukama ka vayarasa apake munha ya naka ke madhyama se apake sharira mem pravesha karata hai ,lekina kadachita yaha bhi sambhava hai ki apaki apani khuda ki bimari mem apaka bhi hatha ho | 
jaba koi bimara khansata ,chhinkata ya batem karata hai to yaha vayarasa pani ki chhoti-chhoti bundom ke madhyama se hava mem phaila jata hai |
lekina jisako jukama hai usake hathom ke samparka se bhi yaha phailata hai ya bartanom ,tauliyom ,khilaunom ya teliphona jaisi vastuom ke saje upayoga se bhi yaha phailata hai |
aise samparka ya aise vatavarana ke bada yadi apa apani ankha ,naka ya munha ko chhute haim ,to sambhava hai ki apako bhi jukama ho jae |
samanya jukama jyadatara hathom ke samparka se hi phailata hai |
udaharana ke lie ,jukama se pida़ita vyakti jaba apani naka chhinakata hai ya use chhuta hai aura usake bada yadi vaha kisi dusare vyakti ko chhu leta hai to vaha dusara vyakti bhi vayarasa se grasta ho jata hai |
isake atirikta jukama ke vayarasa pena ,kitabem ,aura kaॉphi ke kapa jaisi vastuom para kai ghantom taka jivita rahate haim aura phira ina vastuom ko chhune se hi jukama ho jata hai |
jabaki yaha eka bahuta hi ama dharana hai ki khansane aura chhinkane se jukama phailata hai lekina phira bhi jukama phailane ki ye prakriyaem vastava mem bahuta hi mamuli haim |
phephada़om ke tapedika se grasta vyakti jaba khansata ya chhinkata hai to isake tyubarakila besili roganuom ko (jo apake phephada़om ko prabhavita karate haim ) sansa dvara andara lene para yaha roga ho jata hai |
halanki, tapedika hona itana asana nahim hai aura isake lie sankramaka vyakti ke satha eka lambe samaya taka bahuta hi nikata samparka mem rahane se hi yaha phailata hai |
tibi ke roganu ke charom ora eka mota avarana hota hai ,jisase vaha shushka vatavarana mem aura ama taura para pratikula sthitiyom mem bhi jivita raha sakata haim (halanki altravaॉyaleta prakasha jaise ki dhupa dvara isako samapta kiya ja sakata hai ) |
jina deshom mem kitanu samapta kie bina hi dudha piya jata hai ,vaham para gojatiya tibi se grasta gayom ke sankramita dudha se tibi phaila jata hai |
taiphaida ke jivanu sankramita logom ke mala mutra mem rahate haim |
khana khane se pahale yadi eka sankramita vyakti apane hatha achchhi taraha nahim dhota hai ,to yaha bimari kisi ko bhi sakati hai ,jo usake satha khana khata hai |
eka bara jaba apake sharira mem jivanu pahuncha jata hai ,to apaki rakta dhara mem vaha vriddhi karane lagata hai ,aura bukhara ke rupa mem bimari ka pahala lakshana dikhai pada़ta hai |
usake bada jivanu apake pachana tantra ,asthi majja ,jigara aura pitta nalikaom mem avashoshita hone lagate haim |
sankramana ke dusare charana mem ,jivanu chhoti anta mem pravesha kara jate hai evam bhayankara dasta hone ke lakshana dikhane lagate haim |
taiphaida ke jivanu prashitana ,thanda aura shushkata mem bhi jivita raha sakate haim isilie khadya svachchhata aura vyaktigata svachchhata ke uchcha manakom ko sunishchita karana sarvopari manana chahie |
pani ki apurti aksara isase sankramita rahati hai aura yaham taka ki barpha se bhi yaha roga phaila sakata hai |
naliyom mem bahanevale sankramita malayukta pani se sipadara machhaliyam dushita ho sakati haim aura britena mem to kaphi pahale ,dibbabanda dushita mansa utpadom ke karana taiphaida eka mahamari ke rupa mem phaila tha |
koi bhi vyakti jisako taiphaida bukhara hai ,bimari ke pahale saptaha ke daurana sabase adhika sankramaka hota hai |
isalie yadi sambhava hai to isa avadhi ke daurana rogi ko alaga rakhana chahie |
chunki yaha bimari sankramaka hoti hai ,isalie isa bimari se grasta vyakti se evam isa bimari ke vahaka se bhi taiphaida ho sakata hai |
bimari ka vahaka vaha vyakti hota hai jisake sharira mem bimari se achchha ho jane ke bada bhi taiphaida ke jivanu hote haim |
vahakom mem bimari ke lakshana dikhai to nahim dete,lekina unake mala ke madhyama se bimari phaila sakati hai |
lagabhaga 5% logom mem hi ,ye jivanu eka varsha ya usase adhika samaya taka bhi rahate haim |
isalie yaha bahuta mahatvapurna hai ki taiphaida ka vahaka bhojana ke vyavasaya mem bilkula nahim hona chahie |
taiphaida ke vahaka ke lie upachara upalabdha haim |
4-6 haptom ke entibayotika upachara se yaha bimari bilkula samapta ki ja sakati hai |
echaaivi ki ekadama sahi utpatti abhi bhi ajnata hai ,lekina adhikatara vaijnanikom ka manana hai ki vayarasa ka eka rupa ,jo esaaivi chimpja़ ( chimpainji ke simiyana inyunodaiphishiensi vayarasa ) ke rupa mem jana jata hai ,aphrika ke kuchha hissom mem rahane vale chimpanji mem maujuda tha aura abhi bhi hai |
eka manyata yaha hai ki yaha vayarasa unhim manushyom mem phaila jo chimpanji ka shikara kiya karate the ,kyonki chimpanja़i ke sankramita rakta ke samparka mem ane se sambhavatah unako yaha ho gaya ho |
taba yaha mana gaya ki kai varshom taka echaaivi vayarasa ka manava mem paya jane vala rupa aphrika ke duradaraja ke eka bhaga taka hi simita tha |
jo bhi ho ,jaba nae parivahana samparkom se aphrika ka vaha bhaga khula , to yaha vayarasa dhire dhire duniya bhara mem phailane se pahale ,aphrika ke anya bhagom mem phaila gaya |
echaaivi vayarasa ,hamari pratiraksha pranali dvara prayoga mem lai jane vali koshikaom ,vishesha rupa se ve koshikaem jo sidi4 ke nama se jani jati haim ,ke anuvanshika koda ko asana bhasha mem badala deta hai aura phira apana hi pratirupa banane mem usa sarala anuvanshika samagri ka prayoga karata hai |
sharira aura adhika sidi4 koshikaem bana sakata haim ,lekina antatah echaaivi vayarasa sidi4 koshikaom ki sankhya isa hada taka kama kara deta hai ki pratiraksha pranali kama karana banda kara deti hai |
virya ya rakta jaise sharirika tarala padarthom ke adana-pradana se echaaivi vayarasa phaila sakata hai |
isalie yaha munha aura guda dvara sambhoga sahita ,laingika sambhoga ke madhyama se phaila sakata hai |
jo loga istemala ki gayim suiyom se avaidha nashili davaem lete haim ,ve bhi echaaivi vayarasa ke khatare ko mola le sakate haim |
yaha sthiti eka mam se usake ajanme bachche mem bhi phaila sakati hai |
halanki ,ise rokane ke lie aba davaem bhi istemala ki ja rahim haim |
echaaivi raktadhanom ke madhyama se phaila sakata hai |
yaham taka ki ,britena mem to 1985 se yaha niti bana di gayi hai ki dana kie gaye sabhi raktom ki echaaivi ke lie jancha ki jaengi |
britena mem jaba se yaha niti shuru ki gai hai ,kisi bhi vyakti ko raktadhana se echaaivi nahim mila hai |
ho sakata hai ki vikasashila deshom mem jancha ki nitiyam itani sakhta na ho jitani ki britena jaise deshom mem haim ,isalie agara apa duniya ke una kuchha bhagom mem raktadhana karate hai to ho sakata hai ki vaham echaaivi ke panapane ka sambhavita khatara ho |
charmaputha (eka prakara ka egjima) baiktiriya ke karana hota hai ,vishesha rupa se ,stephailokokasa ऑriyasa evam stephailokokasa payojinsa namaka do prakara ke baiktiriya se | 
jaba eka bara koi vyakti sankramita ho jata hai to usake karibi sharirika samparka ya usaka tauliya istemala karane ke madhyama se vaha anya logom ko yaha sankramana bahuta asani se phaila sakata hai |
baiktiriya ke samparka mem ane ke prarambhika chara se dasa dinom ke bada taka ,charmaputha ke lakshana nahim dikhate , isalie logom dvara isa sthiti ko phailana bahuta hi asana hota hai kyonki unhem pata hi nahim chalata ki ve sankramita haim ya nahim |
yaha mana jata hai ki yaha sthiti bachchom mem bahuta hi ama hai kyonki unaki pratiraksha pranali abhi taka puri taraha vikasita nahim hui hoti hai |
rebija़ ,eka sankramita janavara amataura para eka kutte ke katane se ya usaki phati tvacha ya ankha ki putali ko chatane se manushyom mem phaila sakata hai |
palatu kutte aura billiyom ko yaha sankramana ho sakata hai agara ve lomada़iyom ,bheda़iyom ,siyarom ,kale saफ़eda ameriki ज़anavarom (skankom ) ,nevalom ,rakunom aura chamagadada़om jaise jangali janavarom ke samparka mem hai |
katane ki jagaha para vayarasa vibhinna avadhi ke lie rahata hai ,vaham se sharira mem pravesha karata hai aura katane se bane ghava ki mansapeshiyom ki koshikaom mem apani vriddhi karata hai |
vayarasa taba tantrika ke sirom mem pravesha kara jata hai aura vaham se ridha़ ki haddi aura mastishka mem pahuncha jata hai |
eka bara kendriya tantrika tantra mem pahunchane ke bada vayarasa ,lara ki gilti ,phephada़om ,gurdom aura anya angom mem phaila jata hai | 
vikasavadhi (ushmayana avadhi ) ( katane aura lakshana ki shuruata hone ke bicha ka samaya )sira se dansha ki duri para nirbhara karati hai ,lekina amataura para yaha do se tina mahine ki hi hoti hai |
rebija़ mem sira ,chehare ya gardana para katane ke gambhira ghava nau dina se jyada nahim rahate |
sira ke niche ke ghavom ke thika hone mem bahuta samaya lagata hai jaise ki kai mahine |
vibhinna prakara ke vayarasom ki vikasavadhi bhi alaga alaga hoti hai , lekina katane aura lakshana ki shuruata ke bicha ausata samaya chara se atha saptaha ka hi hota hai |
lagabhaga 50 % sankramita dansha se bimari phailati hai |
sankramana se bachchom ko jyada khatara ho sakata hai ,kyonki unaka kada chhota hota hai aura jisaki vajaha se unake sharira ka upari hissa jamina ke kaphi kariba hota hai |
ghara se bahara rahane para unako ajnata janavarom ko chhune ya thapathapane se savadhana kiya jana chahie |
korinibaiktiriyama dipthirie namaka roganu se dipthiriya hota hai ,jo gale ko aura kabhi kabhi tvacha ko bhi sankramita karata hai |
dipthiriya bahuta sankramaka hota hai aura chhinka aura khansi ki chhoti chhoti bundom ke madhyama se eka vyakti se dusare vyakti mem phaila sakata hai |
eka sankramita vyakti dvara istemala ki jane vali vastuem jaise kagaja़ ke naipakinom ,kapom ko chhune se ya bahuta hi naja़diki samparka se ya eka dusare ka chumbana lene se dipthiriya phaila sakata hai |
taba munha aura gale ke nama bhagom ke asa-pasa baiktiriya sankhya mem badha़ne lagate haim aura vriddhi karane lagate haim jisase vaham una hissom mem sujana a jati hai |
eka bimara vyakti do se chara saptaha taka sankramaka ho sakata hai ,yaham taka ki agara usamem koi lakshana bhi nahim haim ,lekina vaha eka lambe samaya taka baiktiriya ka vahaka ho sakata hai |
sansa lene vali vayu nali ki divarom mem sankramana hone se kali khansi hoti hai |
sankramana se vayu naliyom mem sujana aura jalana hoti hai aura bahuta adhika balagama banane lagata hai |
kali khansi phailane vala boradetla paratasisa namaka eka jivanu hai |
yaha hava mem bundom ki nami mem rahata hai |
jaba eka rogi bachcha khansata ya chhinkata hai to sankramita pani ki saikada़om bundem hava mem phaila jati haim |
yahi karana hai ki kali khansi itani sankramaka hoti hai |
amataura para bada़e bachche kali khansi dvara gambhirata se prabhavita nahim hote hai, lekina unase sankramana chhote bachchom mem phaila sakata hai |
chikunaguniya eka vayarasajanita roga hai jo machchharom ke madhyama se phailata hai |
isaka jimmedara vayarasa alphavayarasa hai ,jo machchhara ki edija aijipti namaka eka nasla mem rahata hai |
isalie maleriya ,phaileriya aura dengu ke bada ,chikunaguniya machchharom ke vinasha ka eka aura karana bana gaya hai |
kahim bhi ruka hua pani chikunaguniya ke phailane ka mukhya madhyama hai |
edija aijipti machchhara kevala ruke hue pani mem hi apane ande deta hai |
isase pata chalata haim ki bharata mem aja-kala chikunaguniya ka vyapaka prakopa kyom hai ,kyonki bharata julai-agasta ki bhari badha़ se abhi ubara hai |
ayurveda mem chikunaguniya ka ilaja vata dosha vikara ke rupa mem hota hai |
vata hamare sharira ka vayu tatva hai ,jo ki hamare rakta mem ghuli una gaisom ka pratinidhitva karata hai jinhem hama sansa bharate samaya andara lete haim |
kintu jaba vata dosha vikrita ho jata hai, taba roga ki sthitiyam arambha ho sakati haim|
chikunagunya aisi hi bimariyom mem se eka hai, jo vata dosha ke upashamana ke karana hota hai|
yaha vishanu, jaba sankramita vyakti khansata ya chhinkata hai to utsarjita chhoti bundom aura chhote kanom ke dvara vayu ke madhyama se eka vyakti se dusare vyakti mem asani se chala jata hai|
inpha़luenja़a ke vishanu naka ya gale se sharira mem pravesha karate haim|
yaha taba usa vyakti mem lakshana vikasita karane ke lie eka se chara dina ke bicha samaya leta hai|
inpha़luenja़a se pida़ita koi vyakti unake lakshana vikasita karane ke eka dina pahale se lekara sata dina bada taka sankramaka ho sakate haim|
roga logom ke bicha bahuta jaldi phailata hai visheshakara janasankula paristhitiyom mem|
anya paristhitiyom ki apeksha thanda aura sukha mausama isa vishanu ko sharira ke bahara adhika samaya taka jivita rahane ke lie sakshama banata hai, jisake parinama svarupa, mausami mahamari sardiyom ke tapamana kshetra mem prakata hote haim|
inpha़luenja़a ke e aura bi vishanu vishesha pha़lu utpanna karate haim|
e prakara ina prakopom ke lie prayah uttaradayi hota hai jo prativarsha hota hai|
ye vishanu lagatara badalate rahate haim, paida karate rahate haim jo upaprakara ya nasla kahalate haim jo maulika vishanu se bhinna hote haim kintu inaki kuchha vishishtataem banae rakhate haim|
inpha़luenja़a vishanu ki naslem jo phlu paida karati haim, sala-dara-sala bhinna ho sakati haim|
sankramana ke parinama svarupa jaba netra-shleshmala (apaki ankha ki saphedi tatha apaki palakom ki bhitari sataha ko dhakati hui koshikaom ki patali parata) jalane lagati hai to rogajanaka netrashotha hota hai|
kai prakara ke karaka hote haim jo apaki ankha mem phailane ke lie eka sankramana paida kara sakate haim|
koi vishesha sanketa ya lakshana nahim hote haim jo apake samanya chikitsaka ko eka jivanvika sankramana tatha eka vishanuka sankramana ke bicha bheda batane ki anumati dem|
donom prakara ke netrashotha samanyatah svatah hi thika ho jaenge|
yadi netrashotha eka laingika rupa se sancharita sankramana (esatiai) se utpanna hota hai, jaise ki klamaidiya, to yaha prayah apake netrashotha ke haphtom ke badale kai mahinom taka tikane mem parinita ho jaega|
yadi apaki bimari satata hai, to apako sankramana (‘roga nidana’ anuchchheda dekhem) ka karana nirdharita karane ke lie eka ankha phahe se pochhana hoga|
yadi apaka rogajanaka netrashotha eka laingika rupa se sancharita sankramana (esatiai) se utpanna hota hai, to apako isa sankramana ke lie jancha aura upachara prakriya se gujarana hi hoga|
yadi mam ko klamaidiya ho to navajata shishu mem netrashotha dikhai pada़ sakata hai kyonki yaha sankramana prayah janma ke samaya bachche mem chala jata hai|
rogajanaka netrashotha phailane ki apamem adhika sambhavana hai yadi apa kisi ke satha karibi samparka mem raha rahem hai jo ki pahale se hi sankramita hai|
isalie yaha atyadhika mahatvapurna hai ki apa sunishchita karem ki kisi ke satha jisako yaha bimari hai ke samparka mem ane ke bada apa apane hatha achchhi taraha se dhoem|
pama sarkoptisa skaibiyai ke rupa mem jane jane vale manava parajivi se hota hai|
jaba eka mada kutaki apake tvacha ke bhitara chheda karati hai taba jantubadha arambha hoti hai|
yaha kutaki tisa minata ke andara apake tvacha mem chheda kara sakati hai|
nara kutakiyam eka anishechita mada ke satha sambhoga karane ke lie dekhate hue vibhinna chhidra sthalom ke bicha chalate rahate haim|
eka bara sambhoga ho jane ke bada, nara kutaki mara jata hai aura mada kutaki ande dena prarambha karati hai, jinamem se lagabhaga 3-4 dinom mem bachche nikalenge|
ye ande vayaska narom ya madaom mem vikasita hone ke lie 10-15 dina lete haim|
bina prabhavi upachara ke yaha jivana chakra anishchita kala taka jari raha sakata hai|
skaibija़ kutakiyam sabuna aura garama pani ke prati prabhavashunya hoti haim aura ve tvacha se ragada़ kara sapha़ karake bahara nahim ki ja sakati haim|
tivra khujali ka thika-thika karana jo ki skaibija़ se sambaddha hai, ajnata hai, kintu adhikatara visheshajnom ka manana hai ki yaha khujalahata pratirakshi tantra ki kutakiyom aura unake lara, ande tatha malom ke prati pratikriya se utpanna hoti hai|
skaibija़ kutakiyam uda़ ya kuda nahim sakati haim, atah ve kevala eka manava sharira se dusare mem chala sakati haim yadi do logom ka sidhe aura dirghakalina sharirika sambandha ho to|
alpakalina aura simita sharirika sambandha, jaise ki hatha milane ya gale milane se prasarana asambhava hai|
skaibija़ kutakiyam manava sharira ke bahara 24-36 ghante ke bicha taka jivita raha sakati haim, atah yaha sambhava hai ki sandushita kapada़o ya bichhaunom ke satha samparka mem ane para pida़a dene lage|
halanki, yaha bimari prapta karane ka bahuta kama samanya tarika hai|
skaibija़ ki jantubadhaem bahuta jalda phaila sakati haim kyonki logom ko prayah prarambhika sankramana hone ke 2-6 saptaha bada taka pata nahim rahata hai ki unhem bimari hai|
sansimita vatavaranom, jaise ki vidyalayom ya upachara-grihom mem jaham loga eka dusare ke karibi samipya mem rahate haim, vaham para skaibija़(khuja़li) pakada़ne ka khatara badha़a hua hota hai|
loga prayah skaibija़ ko eka bimari ke rupa mem samajate haim jo ki ghatiya vyaktigata saphai aura rahana-sahana paristhitiyom ke parinama svarupa hota hai, kintu aisa koi pramana nahim hai ki yahi sthiti hai|
skaibija़ samaja ke hara kshetra ke logom ko prabhavita karati hai|
koi bhi jivanu ya vishanu tanika shotha utpanna karane ka samarthya rakhata hai|
jivanvika tanika shotha kai vibhinna prakara ke kitanuom se hota hai jo ki naka aura gale ke pichhe svabhavika rupa se rahate haim tatha karibi dirghakalina samparka, khansane, chhinkane aura chumbana dvara phaila sakate haim|
yuke mem jivanvika tanika shotha ka sabase samanya karana maininjokaॉkasi ya nyumokaॉkasi jivanu ke satha sankramana hai, kintu hiba, tibi, i. koli aura bi varga streptokaॉkasi jivanu bhi tanika shotha utpanna kara sakate haim|
vishanuka tanika shotha vishanuom jaise ki koksasaiki, harpija़ simapleksa, mampsa, vairisela jostara, chhoti mata aura bhainsiya dada ke vishanu, poliyo vishanu, ikovairasom (entirovairasa sahita) ke dvara ho sakata hai|
kitanu khansane, chhinkane, ghatiya saphai ya gande pani se pradushita pani ke madhyama se phaila sakate haim|
tanika shotha lime roga, leptospairosisa, taiphasa, tapedika aura dusare sankramanom sahita anya rogom ke bhi eka lakshana ho sakate haim|
maleriya ka eka arambhika roganidana banane ke lie, apaka samanya chikitsaka apase apake lakshanom ke bare mem puchhega aura apake yatra itihasa ko dekhega|
yaha mahatvapurna hai ki apa apane samanya chikitsaka se pichhale 12 mahine mem apane una sthanom sahita jaham apa kevala thoda़e samaya ke lie ruke the, kaham-kaham ki yatra ki hai, ke bare mem kahem|
yadi apa jaham maleriya upasthita hai aise kisi desha ke nivasi haim, ya vaham gae haim, ya vaham se ae haim to apa maleriya ke khatare mem ho sakate haim|
isa arambhika roganidana ke bada, bimari ki pushti karane ke uddeshya se apaka eka rakta namuna liya jaega|
rakta ki thoda़i matra li jati hai aura sukshmadarshi yantra ke niche vishleshana karane se purva eka vishesha ghola ke satha milaya jata hai|
yaha parikshana maleriya ki upasthiti ki pushti kara sakega, aura apake samanya chikitsaka ko yaha bhi suchita karega ki kisa prakara ka parajivi apaki bimari ko paida kara raha hai|
maleriya sashakta rupa se eka ghataka bimari ho sakati hai, kintu turanta roganidana aura upachara se, kai loga purna svastha ho jate haim|
yadi apaka maleriya se nidana ho chuka hai, to apaka upachara jitani jaldi ho sake shuru kara diya jaega|
maleriya ka upachara samanyatah maleriya virodhi davaiyom ka upayoga karake kiya jata hai|
ina davaiyom ke vibhinna prakara upalabdha haim|
yadi apaka vishva mem aura kahim ilaja ho raha hai to apako milane vali davai ka prakara alaga ho sakata hai| 
maleriya parajivi ke prati badha़e hue stara ki pratirodhashakti hone ke karana, kuchha deshom mem vishesha davaiyom ko lagabhaga puri taraha se anupayogi bana diya gaya hai|
yadi apako maleriya ka mamuli roga hai, to apake lie davai ka nuskha likha sakate haim ki apa use ghara le ja sakem|
yadi apaka maleriya adhika prachanda hai, to apako aspatala mem bharti karana pada़ sakata hai|
yadi apa apani davai munha se nahim le sakate haim, to apako maleriya virodhi davai antarshiriya vidhi se (apaki bhuja ki eka shira se) deni pada़ sakati hai|
yadi apane kisi ushnakatibandhi desha mem jane se purva roka-thama ke rupa mem eka maleriya virodhi davai li hai, aura bada mem phira bhi bimari hona jari rahata hai, to apake upachara ke prabhavi hone ke lie apake lie bhinna prakara ki davai ka nuskha likhana pada़ega|
yadi pahali maleriya virodhi davai jisaka apake lie nuskha likha gaya tha, apake lakshanom ko sudharane mem asaphala ho jati hai, to apako vibhinna prakara ki anya davaiyam apake upachara ke hisse ke rupa mem aja़mana pada़ sakata hai|
maleriya ka upachara apako kai haphtom ke lie bahuta kamaja़ora mahasusa karate hue aura thaka hua chhoda़ sakata hai|
yadi apa garbhavati haim, to kuchha davaiyam jinaka samanyatah maleriya ke upachara ke lie upayoga kiya jata hai, anupayukta hote haim kyonki ve apake tatha apake bachche ke lie parshva prabhava utpanna kara sakate haim|
klorokvina aura kvinaina ve davaiyam haim jo garbhavastha ke daurana upayoga karane ke lie surakshita haim| 
ye apako tatha apake bachche ko bina nukasana pahunchae, apaki garbhavastha ke kisi bhi stara para upayoga ki ja sakati haim|
phira bhi, yadi apako maleriya ke roka-thama ke eka tarike ke rupa mem ina davaiyom ka nuskha likha gaya tha, to ve isa roga ke lie upachara ke rupa mem upayukta nahim hongi|
anya davaiyam apa apani garbhavastha ke kisa stara para haim usa para nirbhara karate hue upayukta ho sakati haim|
yadi apa koi maleriya ki taraha ke lakshana vikasita karate haim, ya sandeha karate haim ki apa maleriya ke khatare mem ho sakate haim, to jitani jaldi ho sake apane samanya chikitsaka se samparka karem|
chhoti mata ke lie koi vishesha upachara nahim hai yadyapi hala ke anusandhana ne roga ke ati arambhika avastha mem vishanu virodhi aushadhiyom ka upayoga karake thoda़a labha dikhalaya hai|
pairasitamaॉla, kailpola, krosina isase sambandhita jvara aura mansapeshiya pida़a tatha vedana jo jvara ke satha-satha hote haim, mem sahayata karate haim|
apake chikitsaka tivra mamalom mem esaiklovira namaka eka vishanu virodhi aushadhi ka nuskha likha sakate haim|
vishanu sankramanom mem espirina se paraheja kiya jata hai kyonki yaha reyi ke lakshana namaka eka roga lata hai, jisake khataranaka natije ho sakate haim|
apake chikitsaka khujali kama karane mem sahayata karane ke lie eka aushadhiya malahama, kailamaina malahama ya sodiyama baikarboneta ke eka ghola ka bhi nuskha likha sakate haim|
samanya bekinga soda ka upayoga karem aura ise pani ki thoda़i matra mem gholem|
kuchha honyopaithika daॉktara bachchom ke pratirakshi tantra ko badha़ane mem sahayata karane ke lie honyopaithika dava salphara ki salaha dete haim|
apa prayah apane lakshanom ke vishishtata ke adhara para pha़lu ka nidana svatah kara sakate haim|
anya naidanika parikshanom ka avashyaka hona yaha asamanya hai jaba taka ki jatilataem na hom|
apaka samanya chikitsaka samanyatah nahim kaha sakata ki vastava mem kauna sa vishanu apake pha़lu ki taraha ke lakshanom ko utpanna kara raha hai|
pha़lu vishanu ke mukhya parisanchari prakarom aura naslom ko pahachanana prayogashala nidana ki mukhya bhumika hai|
yaha karane ke lie, eka vishanuka samvardha avashyaka hota hai jo ki eka phahe ka prayoga karate hue apake naka ya gale se liya jaega|
yadi kisi anya roga ka sandeha hota hai (udaharana ke lie, maleriya, yadi apane abhi-abhi kisi kshetra ki yatra ki hai jaham maleriya eka samasya hai), taba dusare parikshanom ya nirdishta karane ki avashyakata ho sakati hai|
yadi apa samanyatah hrishta-pushta aura svastha rahate haim, to apa pha़lu ke lakshanom ka aushadhashalaom mem samanyatah upalabdha aushadhiyom ka upayoga karake ghara para upachara kara sakate haim|
pha़lu ke lie pratijaivikiyom ka nuskha nahim likha jata hai kyonki inaka vishanuom para koi asara nahim hota hai|
phira bhi,kabhi-kabhi jatilataom ko niyantrita karane ke lie yaha avashyaka ho sakata hai, visheshakara gambhira sine ke sankramanom ya nimoniya mem|
jaba taka tivra lakshana bane rahate haim taba taka ke lie apako ghara para aura bistara para rahana chahie jo ki prayah lakshanom ke charama ke bada do ya tina dinom ke lie hota hai|
apako avashyakatanusara ina lakshanom ke upachara ke lie bahuta adhika tarala padartha pina chahie|
sardi-jukamom, pradahita gale aura khansiyom ke lakshana se rahata pane ke lie vishvakosha pravishtiyam dekhem|
yadi apa jana samudaya mem parisancharita ho rahim jatilataom aura e ya bi prakara ke pha़lu ke khatare mem haim to apaka samanya chikitsaka niche suchibaddha aushadhiyom ka nuskha likha sakata hai|
pha़lu ke upachara mem chikitsakom ka margadarshana karane ke lie suvikasita roga-vishayaka mukhya batem haim|
da neshanala instityuta phaॉra klinikala ekselensa (enaaisii) ne jatilataom ke khatare mem raha rahe logom ke lie vishesha upacharom ke prayoga para margadarshika ka nirmana kiya hai|
aushadhiyam jo pha़lu vishanu se lada़ti haim prativishanuka aushadhiyam kahalati haim|
ve vishanu ko dura nahim karati haim kintu lakshanom ki prachandata aura unake thaharane ki samayavadhi ko kama kara sakati haim|
e aura bi pha़lu vishanuom ke upachara ke lie ja़ainamaivira aura oseltamaivira ke upayoga kie jate haim aura jo apake sharira mem vishanu ko vriddhi karane se roka kara kama karate haim|
ja़ainamaivira eka sukha churna hota hai jo ki diskahailara namaka eka upakarana ke madhyama se ise sansa mem lete hue liya jata hai|
apake pahale pha़lu lakshanom ke 48 ghante ke bhitara hi apako ise lena prarambha karane ki jarurata hai| 
adhikatara loga ja़ainamaivira ko asani se le sakate haim, yadyapi yuke mem, yaha samanyatah 65 se adhika unra ke longo ya unake dvara jinakom asthama, tivra avarodhi phuphphusiya roga (siopidi), hridaya roga, vrikka roga, eka kamaja़ora pratirakshi tantra ya madhumeha jaisi chikitsiya samasyaem hoti haim, upayoga kiya jata hai|
phira bhi, isaki 12 varsha se kama unra ke bachchom ke lie salaha nahim di jati hai|
hala hi ke adhyayanom ne darshaya hai ki ve jo 65 se adhika unra ke the tatha ve jinako anya chikitsiya samasyaem thim unake sahita ve jinhonne ise liya apekshita se lagabhaga eka dina pahale hi behatara ho gae|
phira bhi, isane pha़lu ki jatilataom (jaise ki nimoniya) ko rokane mem sahayata nahim ki aura yadi apako asthama ya phephada़e ki samasyaem hom to yaha shvasana samasyaem paida kara sakati hai|
oseltamaivira goli ya drava ke rupa mukha se (munha se) liya jata hai|
ja़ainamaivira ki taraha hi, pha़lu ke pahale lakshanom ke hone ke 48 ghante ke bhitara apako oseltamaivira lena shuru karane ki ja़rurata hai|
isaki khuraka hai lagatara pancha dinom ke lie eka goli dina mem do bara|
phira bhi, yadi apako gurda roga hai to isase eka chhoti khuraka ka nuskha likha jaega|
adhikatara loga oseltamaivira ko asani se le sakate haim, kintu yaha samanyatah 65 sala se adhika unra ke logom ya upara varnita chikitsiya bimariyom valom ke dvara lie jate haim|
phira bhi, isaki eka sala ki unra ke andara ke bachchom ke lie salaha nahim di jati hai|
oseltamaivira se michali, ulti, peta darda ya dasta utpanna ho sakate haim|
ye parshva prabhava gambhira nahim hone chahie kintu yadi ve haim, to apako apane samanya chikitsaka se milana chahie|
isa kshana taka ina davaiyom mem se kisi se bhi pha़lu ki roka-thama ki salaha nahim di jati hai isake atirikta ki jaba eka mahamari ho|
pha़lu ke mausama ke daurana, enaechaesa pha़lu ke starom ka dhyanapurvaka nirikshana karata hai aura yadi roga ke phailava ko rokane ke lie anya kadama uthane ki avashyakata hai to chikitsakom ko suchita karenge|
emantadaina eka anya munha se li jane vali davai hai, jo kevala e prakara ke pha़lu vishanu ke viruddha kama karati hai|
yaha goli ya sirapa ke rupa mem ati hai aura yadi apa abhi-abhi vishanu ke samparka mem ae haim to yaha pha़lu hone se bachane mem sahayata kara sakati hai|
phira bhi, yuke mem isa kshana taka, pha़lu ki roka-thama ya upachara ke lie (apavadatmaka paristhitiyom ke atirikta yadi dusari davaiyam upayukta nahim hoti haim) emantadaina ki salaha nahim di jati hai kyonki yaha dikhane ke lie abhi taka paryapta anusandhana nahim kiya gaya hai ki yaha unaki sahayata karati hai jo ki isa vishanu ke sabase adhika khatare mem haim jaise ki vayovriddha tatha anya chikitsiya roga vale|
emantadaina ke sambhavya parshva lakshana haim michali, chakkara aura anidra roga|
itihasa aura sharirika parikshana chhoti mata ke roga nidana mem prayah sahayata karate haim|
khujali ke satha yaha abhilakshanika tvacha ka dadora aura ninna darje ka jvara roga nidana karane mem sahayata karate haim|
bada़i mata, ka pahale chhoti mata ke satha bhrama hota tha, jisaka aba unmulana kara diya gaya hai|
kabhi-kabhi anya vishanuka sankramana ya sapuyacharmavisphota chhoti mata ke sadrishya ho sakate haim|
yadi sharira ke adhe bhaga para sthaniya dadora dekha jata hai to harpeja़ ja़ostara ya bhainsiya dada ka sandeha kiya jata hai|
tvacha vikshati ke pratyaksha khurachana ko eka sunkshadarshi yantra ke niche parikshana karake, vishanuka samvardhom ya thika hote samaya ke rakta namunom mem badha़te roga-pratikarakom ke stara ke dvara roga nidana ka satyapana sambhava hai|
eka svastha meja़bana mem chhoti mata aura bhainsiya dada ke chikitsiya prabandhana ko jatilataom ke khatare ki kami ki ora nirdeshita kiya jata hai, kyonki yaha roga tattvatah simabaddha hota hai|
gauna jivanuka sankramanom se bachane ke lie snana, stambhaka sukhavanom aura dhyanapurvaka kate gae nakhunom sahita saphai bahuta mahatvapurna hoti hai|
jvara kama karane ke lie pairasitamaॉla ka upayoga karana chahie|
esaiklovira chhoti mata aura bhainsiya dada donom ke lie vikalpa ki aushadhi hai|
yaha rogopachara dadorom ke akramana ke 24 ghante ke andara arambha karana chahie|
2 se 16 varsha ki ayu ke bachchom ke lie , munha se li jane vali khuraka 20migra/kigra, dina mem 4 bara 5 dinom taka (adhikatama 800 migra eka dina mem 4 bara)|
kishorom evam vayaskom ko 800 migra dina mem 5 bara diya ja sakata hai|
chhoti mata ke nimoniya ke lie, antarshiriya esaiklovira ya vidarabaina mahatva ka hai, halanki atyutama sahayaka paricharya dekhabhala anivarya hai|
bachchom mem, chhoti mata ka bahuta achchha purvanumana hota hai|
eka sakaratmaka rakta alepa ya sudridha़ roga-vishayaka sandeha ke adhara para eka bara maleriya ka roga nidana ho jaya taba bina vilamba kie upachara prarambha karana chahie|
klorokvina maleriya ke upachara mem mukhya adhara bana hua hai|
mephlokvina, tetrasaiklina, primakvina, pairimethamaina, proguenila tatha kvinaina sahita anya aushadhiyom ka bhi aksara upayoga kiya jata hai|
klorokvina-pratirodhi maleriya ke mamalom mem hailophaintrina, artesunaita tatha kvinghosu jaisi nai aushadhiyom ke bhi prayoga kie jate haim|
phalsipairama maleriya ke atirikta, rogi jo anya rupom ke lie paryapta upachara pate haim, unhem adhikatara ghatanarahita svathya labha milata hai|
phira bhi, paramparagata aushadhiyom ke prati pratirodhashakti badha़ti ja rahi hai aura yaha chinta ka eka pramukha karana hai|
pramastishkiya maleriya eka chikitsiya apata kala hai aura yaham taka ki sabase badha़iya upachara se bhi nrityu-sankhya dara bahuta adhika hai|
kvinolonesa, kloraimphenikola, kotrimoksaija़ola aura bita-laiktansa sahita taipha़aida jvara ke upachara mem kai pratijaiviki upayogi haim|
munha se di gai kloraimphenikola vikalpa ki aushadhi thi|
kloraimphenikola aura bada mem dusari aushadhiyom ke prati pratirodhashakti ke vikasita hone ke satha vartamana mem kvinolonesa vikalpa ke upachara haim|
durbhagyavasha, kvinolonesa ka bachchom ya garbhavati mahilaom mem niyamita rupa se upayoga nahim kiya sakata hai|
tisare utpadana siphailosporinsa jaise ki sephtriekja़ona aksara upayoga kie jate haim|
bina pratijaivika rogopachara ke, rogiyom ke 90 pratishata mem jvara tatha adhikatara lakshanom ka sankramana ke chauthe saptaha taka samadhana ho jata hai|
phira bhi, kamaja़ori, vajana hani aura durbalata mahinom taka bani raha sakati haim|
uchita upachara ke bada, jvara ka tina se pancha dinom mem prayah samadhana ho jata hai|
vyaktiyom ke eka se chara pratishata purane vahaka bana jate haim aura mala tatha mutra mem salamonela eka sala taka utsarjita karate haim|
isa vahaka sthiti ka unmulana karane ke lie lambe samaya taka pratijaiviki ke upayoga ki jarurata hoti hai|
galasua ka koi ilaja nahim hai, atah upachara puri taraha se lakshanom ke satha vyavahara karata hai|
apaka sharira vishanuom ke prati roga-pratikaraka utpanna karake svatah hi changa ho jata hai, jo ki apake shesha jivana ke lie pratiraksha pradana karati hai|
sankramana ki punaravritti bahuta hi asamanya hai, halanki koi bhi pratirakshana 100% prabhavi nahim hota hai|
phira bhi, yuke pratirakshana karyakrama mem sabhi tike santoshajanaka sanrakshana pradana karate haim|
yadi apako galasua hai, to apako bahuta thande tarala padartha, visheshakara pani, pina chahie, kintu phala ke rasa se paraheja karana chahie kyonki yaha lara utpadana ko badha़ata hai, jo ki kashtakara ho sakata hai|
eka halka dardanashaka, jaise ki ibuprophena, ya pairasitamaॉla, apake darda ko kama karane mem sahayata karega aura eka halke jvara ko kama karega|
phira bhi, 16 varsha ki unra ke, ya usase kama unra ke bachchom ko espirina nahim diya jana chahie|
apaki suji hui granthiyo para eka garama pha़lalena rakhana, yaha bhi aramadeha ho sakata hai|
kishora lada़kom aura mardom ko jinaki shukra granthiyom mem bahuta adhika sujana aura jalana (shukra granthi shotha) hai, adhika shaktishali dardanashaka, ya kaॉrtikosteraॉidsa namaka steraॉida aushadhiyom ka nuskha likha ja sakata hai|
poliyo ka samanyatah lakshanom dvara nidana ho sakata hai, halanki yadi kisi ka isa bimari ka eka halka mamala ho to yaha adhika kathina ho sakata hai|
poliyo ke eka roganidana ki pushti karane ke lie, lara ya mala ka eka namuna liya ja sakata hai aura eka prayogashala mem poliyo vishanu ke lie parikshana kiya ja sakata hai|
kuchha mamalom mem, pramastishka meru-drava ka eka namuna liya jata hai aura isa vishanu ke lie parikshana kiya jata hai|
pramastishka meru-drava vaha drava padartha hai jo ki mastishka aura meru rajju ke charom ora hota hai aura unaki raksha karata hai|
uttara-poliyo lakshana ke eka roganidana ki pushti karane ke lie koi nishchayatmaka parikshana nahim haim|
apaka samanya chikitsaka samanyatah apake chikitsiya itihasa ko dekhakara, aura eka sharirika parikshana karake koi roganidana karega|
ve jo uttara-poliyo lakshana vale haim, unaki isa vishanu se arambhika akramana ke bada ki eka lambi avadhi bina lakshanom ki hogi(prayah 10-40 varsha ke bicha)|
apaka samanya chikitsaka bhi apase puchhega ki kaba apane apane lakshanom ko pahali bara dekha, kyonki uttara-poliyo ke ye lakshana dhire-dhire vikasita hone ki ora jukava rakhate haim|
atah, yadi apake lakshana bahuta hi achanaka prakata hote haim, yaha eka sanketa ho sakata hai ki ye eka bhinna roga ke dvara utpanna ho rahe haim|
kyonki uttara-poliyo ke ye lakshana kai anya rogom ke bahuta samana hote haim, jaise ki sandhishotha, apaka samanya chikitsaka apake roga ko kisi anya sambhavya karanom se alaga karane ke lie parikshana karega|
isaka yaha artha ho sakata hai ki apake samanya chikitsaka ko kya lagata hai jo apaki bimari ko paida kara sakata hai, isa para nirbhara karate hue apako parikshanom, ya prakriyaom ki eka shrinkhala se gujarana pada़ega|
udaharana ke lie, apako eka rakta parikshana karavane ki avashyakata hogi|
uttara-poliyo lakshana vale kisi ka samanyatah samanya rakta parikshana parinama hoga|
phira bhi, yadi apaka rakta parikshana parinama asamanya hai, to yaha sashakta sanketa hai ki apake lakshana eka bhinna roga ke dvara utpanna ho rahe haim|
poliyo vale logom ko samanyatah kuchha dinom ke lie bistara para arama karane ki sakhta salaha di jati hai yaham taka ki yadi yaha bimari ka eka halka mamala ho taba bhi|
vedanaharom (dardanashaka ke rupa mem bhi jana jane vala), jaise ki pairasitamaॉla, ka siradarda, pradahita kantha, aura gale tatha pitha ki akada़na ke darda ko kama karane mem sahayata karane ke lie nuskha likha jata hai|
yadi kisi poliyo vale ko kabja़ hai, to unako adhika asani se mala (vishtha) tyagane mem sahayata karane ke lie virechaka aushadhiyom ka nuskha likha ja sakata hai|
unhem eka aisa poshahara khane ki bhi salaha di jaegi jo reshe mem uchcha ho kyonki yaha mala ko narama karane mem sahayata karata hai aura unhem tyagane ke lie adhika asana banata hai|
kisi poliyo ke eka gambhira mamale vale ki, ya ghataka poliyo ki hamesha aspatala mem chikitsa honi chahie|
yadi poliyo vale eka vyakti ko sansa lene mem kathinai ho rahi hai kyonki usaki shvasana mansapeshiyam vishanu dvara kshatigrasta ho chuki haim, to use eka vayuyantra para rakhana pada़ sakata hai|
eka vayuyantra eka kritrima shvasana yantra hota hai jo phephada़om ke bhitara aura bahara vayu ko chalata hai|
yadi shvasana mansapeshiyam svayam ko durusta karane mem sakshama haim, to rogi ko isa vayuyantra se alaga kiya ja sakana chahie|
yadi poliyo ke parinama svarupa eka vyakti ki mansapeshiyam kshatigrasta ho chuki haim, ya pakshaghata hua hai, to usaki gatishilata aura svatantrata ko sudharane mem sahayata karane ke uddeshya se use prayah bhautika chikitsa aura upajivikajanya chikitsa ki salaha di jaegi|
bhautika chikitsa apake sharira ke angom ka upayoga karane ke lie jo ki isa vishanu dvara prabhavita ho chuke haim, apaki kshamata ko sudharane mem sahayata karane ke lie upacharom ki eka shreni ka upayoga karata hai, jaise ki malisha, vyayama aura jalachikitsa|
eka upajivikajanya chikitsaka pratidina ke kama aura gatividhiyom ko vasha mem karane ke lie adhika asana banane mem sahayata karane ke lie apako vyavaharika sahayata pradana karata hai|
upajivikajanya chikitsa ka uddeshya hai apako jitani svatantratapurva sambhava ho utani jine ke lie prerita karem|
chunki poliyo ke lie, koi nishchita davai, ya upachara nahim hai jo ki uttara-poliyo lakshana ko thika kara sakata hai|
isake bajaya, apake dainika jivana para isa roga ke prabhava ko kama se kama karane mem sahayata karane ke lie isa sthiti ko niyantrita kiya ja sakata hai, aura apake lakshanom ko kama kiya ja sakata hai|
dhanustambha ke satha, debaraidamenta namaka eka shalyaka prakriya ka upayoga karake jitana adhika sambhava ho utana kshatigrasta aura sandushita mansapeshi ko nikalane ke lie yaha niyata karyapranali hai (yadi ghava bada़a hai)|
yaha visha titenospaja़mina jivavisha ki matra ko simita karega jo ki utpanna ho sakata hai|
jaham dhanustambha sthapita ho chuka hai, vaham visha (titenasa prativishom ke rupa mem jana jane vala) ke viruddha kama karane ke lie roga-pratikarakom ke upayoga kiye jate haim|
eka bara jaba dhanustambha ki ashanka hoti hai to pratijaiviki aushadhiyom (jaise ki metronaidaja़ॉla) ki adhika khurakom ko bhi diya jana chahie|
dieja़ipaima aushadhi (eka shire mem sidhe di jati hai) mansapeshiya ainthanom ko niyantrita kara sakati hai|
charama mamalom mem, usa vyakti ko kyureri se shithila karana aura vyakti ke shvasana ko banaye rakhane ke lie eka yantra ka prayoga karana, yaha avashyaka ho sakata hai|
dhanustambha se bachane ke lie, yaha avashyaka hai ki shvasana aura poshana savadhanipurvaka banae rakhe jaem| 
kama se kama protina ke 100 grama ke satha, 3500-4000 kailoriyom ka antargrahana, eka nali ke madhyama se sidhe peta mem ya eka dripa ke madhyama se eka shire mem aksara drava rupa mem liya jata hai|
apa eka samanya sardi-ja़ukama ke lakshanom ka samanyatah upachara kara sakate haim|
sardi-ja़ukama ka koi ilaja nahim hai aura pratijaivikiyom ka nuskha nahim likha jata hai kyonki pratijaiviki vishanuom ke viruddha kama nahim karate haim|
yadi apaka sardi-ja़ukama jatilataom ki ora agrasara hota hai jaise ki shiranalashotha, karna sankramana, ya galatundikashotha, ya yadi apako tivra shvasanali-shotha hai to eka apavada ho sakata hai|
ina mamalom mem, apaka samanya chikitsaka apako pratijaivikiyom ka nuskha likha sakata hai|
vashpa antahshvasana mem garama pani ke eka katore ke upara apake sira ke satha baithana shamila hai|
apako eka tauliya apane sira ke upara rakhana chahie, apani ankhem banda karem aura gahari sansa lene ki koshisha karem|
vashpa shleshma ko dhila karake aura apaki naka ko chhinaka kara sapha़ karane ke lie ise asana banate hue apaki sankulata ko kama karane mem sahayata kara sakata hai|
yadi kuchha ghatakom – jaise ki mithenaॉla, yukeliptasa, kapura, thaimaॉla ya paina tela - ye pani mem milae jate haim to yaha mahasusa kara sakata hai ki apaki naka ke raste sapha़ ho rahe haim|
apani ankhom mem vashpa jane se bachane ke lie apako apana dhyana rakhana chahie kyonki yaha jala sakata hai|
dravadahana ke khatare ke karana bachchom ke lie vashpa antahshvasana ki salaha nahim di jati hai|
isake bajaya, eka bachche ko eka garama, vashpamaya snanagriha mem baithane se phayada ho sakata hai|
isa prakara ki davaiyam mukha se (munha se lie jane vale visankulaka) li ja sakati haim ya apaki naka mem (nasiya visankulaka) eka spre ke rupa mem upayoga ki ja sakati haim|
ye apaki naka ke rastom mem sujana ko kama karake kama karati haim aura shvasana ko asana karane mem sahayata karati haim|
visankulaka kitane karagara haim yaha darshane ke simita pramana haim, aura isilie isa prakara ki davaiyam kuchha logom ke lie kevala lakshana se rahata pradana kara sakati haim|
jaba visankulaka kama karate haim, to ve prayah kevala lakshanom ko eka vahuta chhoti avadhi ke lie kama karate haim|
phira bhi, visankulaka upachara ke eka surakshita prakara haim, aura kabhi-kabhi hi koi gambhira parshva prabhava utpanna karate haim| 
yadi apa nasiya visankulaka ka bahuta bara ya eka lambi avadhi ke lie upayoga karate haim, to apa apani sankulata ko aura bhi bura kara sakate haim|
apako yaha bhi sunishchita karana chahie ki apa do varsha se kama unra ke bachche ko kisi bhi prakara ka munha se liya jane vala visankulaka na dem, kyonki yaha pratikula parshva prabhava utpanna kara sakata hai|
munha se lie jane vale visankulaka adhika sajaga hone ki eka bhavana ki ora agrasara karate hue rakta daba aura hridaya dhada़kana dara mem vriddhi kara sakate haim|
isalie yadi apa sote samaya sidhe kauntara para milane vale visankulaka lete haim, to apako rata mem shayana samasyaem ho sakati haim|
munha se lie jane vale visankulaka jaise ki syudoephidraina, phinailapropenolemaina aura phinailephrina ke kai sidhe kauntara para milane vali sardi-ja़ukama ki davaom mem upayoga kie jate haim|
munha se lie jane vale visankulaka kuchha pratiavasadakom aura bita-blaॉkarsa ke satha eka-dusare ko prabhavita kara sakate haim|
yadi apa ina davaiyom mem se koi eka le rahe haim to munha se lie jane vale visankulaka ko lene purva apako apane samanya chikitsaka se jancha karavana chahie|
yadi apako uchcha raktachapa, hridaya samasyaem ya adhimandha hai to bhi munha se lie jane vale visankulaka ka upayoga karane se purva apako apane samanya chikitsaka se jancha karavana chahie|
nasiya visankulaka vishesha rupa se naka para kama karate haim aura vayaskom tatha bada़e bachchom ke upayoga ke lie prayah surakshita hote haim|
nasiya visankulaka jaise ki ऑksimetaja़olaina, phinailephrina aura ephidraina sidhe naka ke andara lagae jate haim|
ve nasika drapa ya spre ke rupa mem upalabdha haim|
nasiya visankulakom ka 5-7 dinom se adhika ke lie upayoga nahim kiya jana chahie kyonki unhem isase adhika samaya ke lie upayoga mem lane se vastava mem apaki sankulata ko aura adhika kha़raba kara sakata hai|
yadi apa monoemaina ऑksidesa inahibitarsa, ya emaeoai namaka eka prakara ka pratiavasadaka le rahe haim, to apako nasiya visankulakom ka prayoga nahim karana chahie|
shishuom ko bhojana karana kathina laga sakata hai yadi unake avaruddha naka haim to|
lavanayukta (namaka pani) nasika drapa sahayaka ho sakate haim yadi ye bhojana karane ke thika pahale upayoga kie jaem| 
yadi yaha kama nahim karata hai, to tina maha se adhika ke shishuom ke lie ephidraina nasika drapa sidhe kauntara para aushadha vikreta se kharida ja sakata hai|
inaka 3-4 dinom se adhika ke lie prayoga nahim kiya jana chahie aura isake thoda़e hi pramana hai ki ve svatah se lavanayukta nasika drapa se behatara haim|
pairasitamaॉla,ibuprophena ya espririna jvara kama sakate haim aura dardanashakom ki taraha kama kara sakate haim (bachchom ke lie bachchom ke tarala pairasitamaॉla ka upayoga karem)|
sahi khuraka ke lie apako hamesha utpadaka ke nirdeshom ka palana karana chahie|
yadi apako peta ke phoda़e, apacha, dama, ya gurde ki bimari ka eka itihasa hai, to apako ibuprophena nahim lena chahie|
16 varsha se kama unra ke bachchom ya mahilaem jo stanapana kara rahi haim ke dvara espirina nahim liya jana chahie| 
yadi dardanashakom ko garbhavastha ke daurana avashyaka samaja jata hai, taba garbhavastha ke daurana ke jvara aura halke-madhyama pida़a se alpavadhi arama ke lie vikalpa ka dardanashaka pairasitamaॉla hai|
yadi garbhavastha ke pahale 30 haphtom mem eka alpavadhi dardanashaka ki avashyakata hoti hai, taba ibuprophena liya ja sakata hai|
jaise garbhavastha ke daurana upayoga ki jane vali koi bhi davai ke mamale mem, ibuprophena sabase kama avashyaka samaya ke lie sabase kama prabhavi khuraka mem liya jana chahie|
ibuprophena ka upayoga tisare trimasa mem karane ki salaha nahim di jati hai, jaba taka ki upachara chikitsiya nirikshana mem nahim hota hai|
ibuprophena aura pairasitamaॉla ko eka dina ke korsa mem eka ke bada eka liya ja sakata hai agara apa pratyeka ke lie unake adhikatama khuraka ko para nahim kara jate|
phira bhi, bachchom ko ibuprophena aura pairasitamaॉla donom hi nahim diya jana chahie|
apako eka ya dusare mem se koi eka avashya upayoga karana chahie|
donom ka prayoga karana pratikula parshva prabhava utpanna kara sakate haim|
pairasitamaॉla, ibuprophena aura espirina sidhe kauntara para milane vali sardi-ja़ukama ki kuchha davaiyom mem bhi anya ghatakom ke satha shamila hote haim|
yadi apa koi anya dardanashaka le rahe haim, to apako eka sardi-ja़ukama ki davai lene se purva eka aushadha vikreta ya apane samanya chikitsaka se jancha karavani chahie|
phlarbiprophena yukta gale ke lie chusane ki goliyam pradahayukta gale ke lakshanom ko arama karati hui pramanita ki gai haim jo ki sardi-ja़ukama se sambandhita ho sakata hai|
jvara, pradahayukta gale aura siradarda ke lie, kai loga esitaminophena (tailenaॉla, anya) ya dusare halke darda nashakom ki ora muda़te haim|
dhyana mem rakhem ki esitaminophena yakrita ki kshati utpanna kara sakata hai, vishesha rupa se yadi bara-bara ya sipharisha ki matra se adhika mem liya jaya to|
jaba bachchom ko esitaminophena de rahe haim to vishesha rupa se savadhana rahem kyonki khuraka dene ke margadarshana aspashta ho sakate haim|
udaharana ke lie, shishu-drapa sanrupita aushadha bada़e bachchom mem samanyatah upayoga mem lai jane vali sirapa se bahuta adhika sandra hota hai|
yaha reyi ke lakshanom se sambaddha raha hai – eka asamanya kintu sashakta rupa se ghataka roga|
eka asamanya sine ka eksa-re kshayaroga ka suchaka ho sakata hai kintu roganidana ki pushti kevala rogi se lie gae namunom jaise ki balagama (kapha़) mem tibi kitanu ki pahachana karake ki ja sakati hai|
kshayaroga ka pratiyakshmaja aushadhiyom se sammishrana mem, kama se kama chhah mahinom ke lie upachara kiya jata hai|
phira bhi, apa samanyatah aushadha arambha karane ke 2-4 haphtom ke bhitara behatara mahasusa karana shuru kara denge|
upachara mem samanyatah pratijaivikiyom ka eka sammishrana hota hai|
manaka upachara mem aisonieja़ida aura riphaimpisina ka kama se kama chhah mahinom taka diya jana shamila hai|
isake atirikta, upachara ke pahale do mahinom taka pairaja़inamaida aura ethaimbutaॉla bhi die jate haim|
tibi ke upachara mem riphaibutina, saikloseraina, kaipriomaisina aura streptomaisina ke bhi upayoga kie ja sakate haim yadi aushadha pratirodha upasthita dikhai diye haim| nirdeshanusara aushadhiyom ke korsa ko pura karana, yaha atyavashyaka hai|
yadi aisa nahim kiya gaya, to jivanu purna rupa se nahim jaega aura aushadha-pratirodha vikritiyam vikasita ho sakati haim|
isa jivanu ki aushadha-pratirodha vikritiyam eka badha़ti hui samasya bana chuki hai|
pichhale dashaka ne tibi jivanu ki aushadha-pratirodha vikritiyom ka phailava dekha hai|
kuchha vikritiyam aba ina do pramukha aushadhiyom, aisonieja़ida aura riphaimpisina ke prati prabhavashunya haim|
chhoti mata ka nidana karane ke lie samanyatah parikshanom ki avashkata nahim hoti hai, kyonki eka prarambhika halke jvara ke bada dadora, chhale aura papada़iyam atyanta abhilakshanika haim|
chhoti mata ka anya dadorom se bheda karana prayah asana hai|
kabhi-kabhi yaha anya rogom se jo tvacha ko prabhavita karate haim, mila sakata hai, jaise ki kita dansha ya khujali| 
yadi koi sandigdhata hai ki kya apake ya apake vachche ke lakshanom ko utpanna kara raha hai, to isa vishanu ko pahachanane ke lie eka samanya rakta jancha kiya ja sakata hai|
yadi apa garbhavati haim, stanapana kara rahi haim ya apake pasa kamajora pratirakshi tantra hai to apa jatilataem vikasita karane ke adhika khatare mem haim|
chara haphtom se kama unra ke shishuom ko bhi khatara rahata hai|
yadi apa ina samuhom mem se kisi eka mem ate haim aura kisi chhoti mata vale ke samparka mem raha rahe haim, ya apako sandeha hai ki, shayada apa ya apake bachche ne isa roga ko vikasita kara liya hai, taba apako apane samanya chikitsaka se turanta milane jana chahie|
isa roga ki apaki aprabhavyata sthapita karane ke lie apako kuchha parikshanom ki avashyakata ho sakati hai|
yadi apako purva mem chhoti mata hui thi, taba yaha atyadhika asambhavya hai ki apa chhoti mata se dusari bara sankramita honge|
yadi apako pata hai ki apako chhoti mata kabhi nahim hui hai, ya apa anishchita haim ki apako hui thi, taba apako janchane ke lie dekhane ke lie ki apa roga-pratikaraka utpanna kara rahe haim jo sharira ki chhoti mata ke vishanu se raksha karate haim, rakta parikshana ki avashyakata ho sakati hai|
yadi apaka rakta parikshana parinama yaha darshata hai ki apamem ye roga-pratikaraka haim, taba apa beshaka isa vishanu se surakshita kie jaenge|
yadi apamem ye roga-pratikaraka nahim haim, taba apaki achchhi taraha se nirikshana karane ki avashyakata hogi, yadi lakshana vikasita ho jaem to|
chhoti mata ka koi upachara nahim hai, aura yaha vishanu bina kisi upachara ke bhi samanyatah apane apa se sapha़ ho jata hai|
phira bhi, lakshanom se arama pane mem sahayata karane ke lie apa kuchha kara sakate haim|
yadi apa ya apaka bachcha darda mem hai, ya jvara hai, taba apa unhem eka halka dardanashaka de sakate haim, jaise ki pairasitamaॉla ya ibuprophena|
12 varsha se kama unra ke bachchom ko tarala, sharkara-mukta pairasitamaॉla diya jana chahie|
pairasitamaॉla ya ibuprophena mem se kisi eka ka alpavadhi upayoga samanyatah koi parshva prabhava utpanna nahim karega|
pairasitamaॉla ka garbhavastha aura stanapana karane ke daurana kisi bhi stara para upayoga kiya ja sakata hai, kintu ibuprophena ka garbhavastha ke 30 haphtom bada upayoga nahim kiya jana chahie|
yadi apako dama ya amashaya ki samasyaom ka eka itihasa hai, jaise ki peta ke phoda़e, taba bhi ibuprophena anupayukta hai|
yadi apa anishchita haim ki ibuprophena apake ya apake bachche ke lie upayukta hoga, to apake samanya chikitsaka, narsa ya aushadha vikreta se jancha karavaem|
yadi apaka bachcha tina mahine ki unra se chhota hai, to apa apane bachche ko kisi bhi prakara ka dardanashaka dene se purva apane samanya chikitsaka se hamesha bata kijie|
yadi apako sandeha hai ki apake bachche ko chhoti mata hai, ya ve isa roga se nidana kie ja chuke haim, to yaha bahuta mahatvapurna hai ki apa unhem espirina na dem|
solaha varsha se kama unra ke bachchom ko espirina kabhi bhi nahim diya jana chahie, kintu yaha vishesha rupa se mahatvapurna hai ki yadi unhem chhoti mata ki taraha eka vishanuja sankramana ho to apaka bachcha espirina na le|
chikitsiya adhyayanom ne darshaya hai ki bachche jinako chhoti mata hai unaka espirina se upachara karane aura reyi ke lakshana ke rupa mem jane jane vale isa sashakta rupa se ghataka roga ke bicha eka sambandha hai|
yaha roga lagabhaga hamesha bachchom aura kishorom ko prabhavita karata hai, tatha sharira ke anya angom ko prabhavita karane ke satha-satha prachanda mastishka aura yakrita kshata, utpanna kara sakata hai|
yadi reyi ke lakshana ka shighrata se nidana nahim kiya gaya, to yaha vikalangata aura kuchha mamalom mem nrityu ki ora agrasara ho sakata hai|
yadi apa isake bare mem anishchita haim ki kauna si davaiyam apane bachche ko dem, to apane samanya chikitsaka ya aushadha vikreta se bata kijie|
nirjalikarana se bachane ke lie bahuta pani pine ki koshisha karem|
sharkara-mukta barpha-laॉlipaupa tapamana kama karane aura bachchom ko tarala padartha dene ke satha-satha, pradahayukta munha ko yadi yaha sankramita ho gaya hai to arama pahunchane mem sahayata karate haim|
ve chara varsha se adhika unra ke bachchom mem upayoga kie ja sakate haim|
bhojana se paraheja karem jo ki apake munha ko kashtaprada bana sakate haim (jaise ki namakina khadya) aura supa lene ki koshisha karem kyonki ve nigalane mem asana haim (sirpha़ sunishchita karem ki yaha supa bahuta garama na ho)|
gahari kharoncha lagane ko rokane ke lie bachchom ke nakhunom ko svachchha aura chhote rakhem|
rata mem jaba ve soem to unhem dadorom ko kharonchane se rokane ke lie apa apane bachche ke hathom ke upara moje dalane ka bhi vichara kara sakate haim|
yaha papada़iyom ki khujali se bhavishya mem kisi prakara kshatigrasta hone se bhi rokega|
yadi apake bachche ko jvara hai, ya unaki tvacha pradahayukta tatha badha़i hui hai, taba unhem bahuta garma hone ya kampane se bachane ke lie uchita dhanga se kapada़e pahanaem|
yadi apako ya apake bachche ko chhoti mata hai, to dhile-dhale, chikane, suti kapada़e pahanana, yaha sabase achchha hai| 
yaha tvacha ko pradahayukta aura jalana paida karane vale hone se bachane mem sahayata karega|
yadi apake bachche ki tvacha bahuta khujalahatadara ya pradahayukta hai, to apa kailamaina loshana ka upayoga karane ki koshisha kara sakate haim|
yaha loshana upayoga karane ke lie bahuta surakshita hai, aura isaka tvacha para eka aramadeha, shitala prabhava hoga|
yadi apako jaba apa garbhavati haim, taba chhoti mata ho jati hai, to apako esiklovira ke rupa mem jane jani vali prativishanuka davaiyam leni pada़ sakati hai|
ise adarshatah apake dadore nikalane ke 24 ghante ke andara arambha karane ki avashyakata hoti hai|
yaha chhoti mata ka upachara nahim karata hai, kintu lakshanom ko kama tivra avashya karata hai|
yadi apa esiklovira le rahe haim, to apako sunishchita karana chahie ki apa bahuta adhika tarala padartha piem|
apako samanyatah yaha davai dina mem pancha bara (chara ghante ke antarala para) sata dinom taka leni pada़egi|
navajata shishuom, aura eka kamajora pratiraksha tantra vale loga jinhem chhoti mata ho gai hai, ke lie bhi isa davai ka nuskha likha ja sakata hai kyonki ve bhi jatilataem vikasita karane ke adhika khatare mem hote haim|
amataura para eka rogi ke mala ,mutra ya rakta ka parikshana karane se taiphaida bukhara ka pata chala jata hai |
yadi taiphaida jivanu ka pata chala jata hai ,to usaka sahi nidana kiya ja sakata hai |
taiphaida ke lakshana maleriya ke lakshana jaise hi ho sakate haim ,isalie jalda se jalda bimari ka pata laga jana chahie |
mukhya rupa se taiphaida bukhara videsha jane para hota hai ,isalie apake chikitsaka ke lie apaki hala hi ki yatra vivarana ke bare mem janana phayademanda hoga | 
chunki taiphaida bukhara pranaghataka ho sakata hai ,isalie isaki shighra pahachana aura upachara bahuta hi mahatvapurna hai |
vikasita deshom mem ,sankramana ka ilaja karane ke lie aksara empisilina aura kloraimaphainikaॉla jaisi entibayotika davaem istemala ki jati haim | 
apake videsha jane para agara apa taiphaida jivanu se sankramita ho jate haim to apako aspatala mem bharti hone ki jarurata ho sakati hai |
jatharantriya (gaistrointestainala ) (peta aura anta ki ) samasyaom ke karana yadi apa kuchha bhi khane mem asamartha haim ,to apako eka antahshira khuraka (entravinasa dripa) di ja sakati hai |
taiphaida bukhara ke gambhira mamalom mem,apake mastishka ki sujana ko kama karane ke lie kaॉrtikaॉstiraॉyada ki jarurata ho sakati hai |
yadi apa aspatala mem bharti haim ,to apako alaga rakhakara apaki dekharekha ki jaegi aura yaha dekhane ke lie ki apake parivara ke sadasya bimari ke vahaka hai ya nahim ,unake bhi mala ki jancha ki ja sakati hai |
ilaja hone ke daurana ,achchhi taraha se samanya svachchhata banae rakhana bahuta mahatvapurna hai kyonki baiktiriya apake sharira mem maujuda raha sakate haim |
taiphaida bukhara ke rogi ,jinaka ilaja entibayotika davaom se kiya jata hai ,amataura para 2-3 dinom ke bhitara thika ho jate haim aura nrityu to bahuta kama hoti hai |
kuchha bhi ho ,agara eka bara apa ko sankramana ho chuka hai ,to bhi apake sharira mem baiktiriya maujuda ho sakate haim |
isalie ,agara videsha mem taiphaida ke lie apaka ilaja kiya ja chuka ho ,to bhi vapasa ane para apane mala ki jancha kara leni chahie |
yadi apa bhojana vyavasaya se sambandhita haim ,to apako kama taba taka banda kara dena chahie jaba taka ki apake mala ke kama se kama do namune jivanurahita na ho jaem | 
amataura para penisilina ko variyata di jati hai ,lekina erithromaisina ,tetrasaiklina ,ya kloremphainikaॉla bhi istemala ki ja sakati haim |
ilaja jaldi shuru kiya jana chahie ,tabhi vaha asaradara hoga |
yadi bimari beilaja़ chhoda़ di jae ,to pranaghataka bhi ho sakati hai |
sanyukta rajya amarika mem istemala karane ke lie manushyom ke lie enthreksa tike ka laisensa ,abhi hala hi mem diya gaya hai |
kyonki jaivika yuddha mem istemala karane ke lie enthreksa ko eka sambhavita ejenta mana gaya hai ,raksha vibhaga ne hala hi mem ghoshana ki hai ki sabhi amariki sainya karmiyom ka vyavasthita tikakarana shuru kiya jaega |
desha ke logom mem ,jo loga apane peshe mem janavarom ki ayatita khalom ,asthichurnom ,unom ,balom (vishesha rupa se bakari ke bala), aura royom ke samparka mem rahate haim ,aura una logom ko bhi jo naidanika ya anusandhanatmaka gatividhiyom mem sanlagna haim aura jaham ve enthreksa ke bijanuom ke samparka mem rahate haim ,enthreksa ka tika lene ki salaha di jati hai |
tika kevala 18 se 65 sala ki unra taka ke svastha purushom aura mahilaom ko lagana chahie |
garbhavati mahilaom ko yaha tika nahim lagaya jana chahie kyonki abhi taka yaha pata nahim chala hai ki enthreksa ke tike se bhruna ko nukasana ho sakata hai ya nahim |
tvacha sambandhi enthreksa se raksha karane mem tika 93 % asaradara hota hai |
pashuom ke lie bana enthreksa tika manushyom mem nahim lagaya jana chahie |
kevala lakshanom ko dekhakara echaaivi ke roga ki pahachana karana bahuta mushkila ho sakata hai |
yaham taka ki ,echaaivi ka upachara kara rahe visheshajna bhi kabhi kabhi kisi aura bimari ke lakshanom 
isalie ,agara apa echaaivi se sankramita hone ke atyadhika khatare vale samuha mem haim , to yaha bahuta hi mahatvapurna hai ki apa apana echaaivi parikshana karaem |
jitani jaldi echaaivi ki pahachana hogi ,utana hi saphala upachara hone ki sambhavana aura adhika ho jaegi | 
kaphi samaya pahale ,kai loga echaaivi ka parikshana karane ke khilapha hote the kyonki unhem lagata tha ki agara unamem isa roga ki pahachana ho gayi to usa samaya ऎsa kuchha bhi nahim tha joki unhem madada karane ke lie kiya ja sakata tha |
baharahala ,yaha satya nahim hai aura aba to ऎsi davaem upalabdha haim jo echaaivi ki vriddhi ko dhimi aura sthiti ko prabhavi dhanga se sambhalane ke lie istemala ki ja sakati haim |
vayarasa ka pata lagane ke lie apaki rakta jancha se echaaivi ki pahachana ki ja sakati hai |
kabhi bhi ho ,jancha karane se echaaivi ka pata arambhika sankramana se kevala tina mahinom ke bada hi chala pata hai |
isalie ,ama taura para yaha salaha di jati hai ki apani pahali jancha ke tina mahine bada apa eka aura jancha karaem |
apako yaha nahim mana lena chahie ki pahale karai gayi koi bhi rakta jancha echaaivi ki upasthiti ka pata lagane ke lie ki gayi thi |
echaaivi ke lie rakta jancha kevala tabhi ki jati hai jaba apa jancha karane ke lie vishesha rupa se sahamata ho |
apaki echaaivi jancha ka parinama puri taraha se gopaniya rakha jata hai |
lekina,agara apake echaaivi ki pahachana ho jati hai aura apaka svasthya bima ya jivana bima hua hai ,to apaki bima kampani ko bhi suchita kiya jata hai |
yadi apa mem echaaivi hai ,to yaha bahuta jaruri hai ki apake vartamana yauna sathi ,ya anya kisi bhi yauna sathi jo sankramana ke lie apake satha juda़a hua hai , ki jancha honi chahie aura usaka ilaja bhi hona chahie |
apane vartamana ya purva sathi ke satha echaaivi ke bare mem charcha karate vakta kuchha logom ko gussa a sakata hai ,pareshana ho sakate haim ,ya sharmindagi bhi mahasusa kara sakatai haim |
apane daॉktara ya klinika stapha ko apani pareshaniyom ke bare mem batane se darana nahim chahie |
ve apako isase sambandhita vyakti aura usase samparka karane ka sabase achchha tarika bataenge |
apaki anumati se ,apaka klinika apake purva sathi ya sathiyom ko eka 'samparka parchi ' dene ki vyavastha karega |
parchi mem likha hoga ki ve eka yauna sancharita sankramana se juda़e hue haim ,aura unhem apani jancha karane ki jarurata hai |
usa parchi para apaka nama nahim hoga ,aura apaka vivarana bhi puri taraha se gopaniya rakha jaega |
apane sathi ko apane echaaivi ke bare mem batane ke lie apako koi bhi majabura nahim karega ,lekina apako ऎsa batane ki mahatvapurna salaha di jati hai |
agara bina jancha aura bina ilaja kie echaaivi ko chhoda़ diya jata hai to isake vinashakari parinama ho sakate haim aura antatah mauta bhi ho sakati hai | 
yaha dekhane ke lie ki vayarasa ki davaem prabhavi rupa se kama kara rahi haim ya nahim ,davaom ki khuraka(vayarala loda) ko niyamita rupa se mapa jata hai |
eka uchcha vayarala loda ka matalaba ho sakata hai ki apako koi aura nai davaom ki jarurata ho |
alaga alaga samaya mem ,sidi4 sankhya aura vayarala loda donom mem hi utara chadha़ava ho sakata hai ,isalie eka bara jyada ya eka bara kama hone ka matalaba yaha nahim hai ki apaki halata bigada़ rahi hai | 
apake sidi4 sankhya aura vayarala loda ka samanya pravaha ,samaya samaya para,sthiti ke sudhara para naja़ra rakhane mem ati mahatvapurna hota hai |
echaaivi ke lie koi ilaja ya tika nahim hai ,aura shodhakartaom ka manana hai ki agara kai dashaka pahale nahim ,to kama se kama kai varsho pahale se hi yaha paya jata tha |
halanki ,upayoga mem ane vali vishesha davaom ne ,jinako atyadhika sakriya entiretrovairala therepi (harta) ke rupa mem jana jata hai ,sthiti ki pragati ko dhima karane aura unra ko badha़ane mem ullekhaniya saphalata hasila ki hai |
shodhakartaom ka vishvasa hai ki chikitsa mem lagatara sudhara hone ka matalaba hai ki echaaivi vale eka vyakti ki unra bhi utani hi hogi jitani ki anya kisi vyakti ki jisako ki echaaivi nahim hai | isa chikitsa mem kai davaom ka upayoga eka satha hota hai |
ऎsa isalie hota hai kyonki echaaivi bahuta hi asani se kisi eka dava ke anukula ho jata hai aura phira usa dava ke prati prabhavahina ho jata hai |
ye davaem alaga alaga tarikom se kama karati haim ,para ,vayarasa ki vriddhi ko kama karane aura apaki pratiraksha pranali ko bachane ke lie ye sabhi davaem isa taraha se banai gaim haim ki vayarasa ke prajanana chakra ko bicha mem hi roka dem |
echaaivi ke jyadatara logom ko enaaratiai grupa ki do kisma ki davaem aura usake alava eka dava anya grupa ki lene ki salaha di jati hai |
upachara ka lakshya hota hai ki davaom ke dushprabhavom ko kama karane mem vibhinna davaom ka sabase achchha mela kya hona chahie |
echaaivi ka ilaja karane ke lie istemala ki jane vali davaem ,kuchha logom mem apana dushprabhava dikhati haim |
lekina ,kuchha haphtom ke bada jaise hi apaka sharira davaom ka adi ho jata hai dushprabhava kama hone lagate haim |
protiऎja़ inahibitarsa rakta mem kolestraॉla aura sharkara ke stara mem vriddhi ke lie jane jate hai ,jisase apaki dhamaniyam sakhta (dhamanikathinya) ho sakati haim |
dhamanikathinya apake hridaya roga ya daure jaisi hridaya sambandhi bimari hone ki sambhavana ko badha़a deta hai |
paushtika bhojana se ,niyamita vyayama se ,aura dhunrapana na karane se ,dhamanikathinya hone ka khatara kama ho jata hai |
age ke khatare ko kama karane ke lie bhi apako vishesha davaiyam di ja sakati haim |
ina davaiyom mem stetina namaka davaem haim jinase apake rakta mem kolestraॉla ka stara kama ho jata hai |
dhamanikathinya ke bare mem adhika janakari ke lie , 'sambandhita lekha' vala anubhaga dekhem |
kuchha echaaivi virodhi davaom se hatha paira ki nasom ko nukasana pahuncha sakata hai ,yaha sthiti paridhiya nyuropaithi ke nama se jani jati hai |
isase apake hatha paira sunna ho sakate haim ya unamem janajanahata ho sakati hai |
aise koi bhi lakshana hone se ekadama batana chahie ,taki apake tantrika ko aura adhika hone vale nukasana se bachane ke lie apaki upachara yojana mem thoda़a parivartana karana pada़e |
agara apako laga raha hai ki echaaivi upachara se hue dushparinama apake jivana ki gunavatta para pratikula prabhava dala rahe haim ,to apane echaaivi klinika ke stapha ko phaurana suchita karem |
isase apaki davaom ko badalana sambhava ho sakega ,ya anya aisi davaem upalabdha karaim ja sakati haim jo koi bhi dushparinama dura kara sakati hom |
echaaivi ke ilaja ke lie istemala ki gaim davaem sirpha tabhi asaradara hotim haim jaba ve vastava mem nirdharita rupa se li jaem |
yaham taka ki khuraka kuchha kama lene se bhi apake upachara ka thika se kama na karane ka khatara badha़ sakata hai |
udaharana ke taura para ,jaise ki kuchha davaem apako aisi di jaengi jinako kevala khali peta hi liya jana chahie ,jabaki dusari davaom ko bhojana ke satha liya jana chahie |
apako apani jivana shaili mem apani upachara yojana ko phita karane ke lie ,roja़ana ki dinacharya ko vikasita karane ki avashyakata hogi |
jisase apake echaaivi klinika ka stapha apako ,apani salaha aura samarthana dene mem apaki madada kara payega |
agara apa dusari taraha ki davaem bhi le rahe haim to apako khyala rakhana hoga ki echaaivi ka ilaja karane vali kai davaem apratyashita rupa se bhi pratikriya kara sakati haim | 
ina dusari taraha ki davaom mem senta jaॉna vaॉrta jaise harbala upachara , kokena aura daॉktara ke nuskhe ke bina li jane vali davaom jaisi kuchha samanya davaem shamila haim |
dusari koi bhi dava lene se pahale apa hamesha apane klinika stapha ya apane daॉktara se puchha lem |
kabhi kabhi sankramana ke arambhika 72 ghantom ke bada echaaivi ki vriddhi ko rokana sambhava hota hai |
ise posta eksapoja़ra prophilaiksisa (piipi) kahate haim |
piipi mem chara haphtom ke lie echaaivi virodhi davaem di jati haim |
jaba kisi vyakti ko pata chalata hai ki usako nishchita rupa se echaaivi vayarasa se sankramana hua hai to aksara piipi ka prayoga kiya jata hai |
udaharana ke lie ,agara unhonne kisi aise vyakti se sambhoga kiya hai jisake bare mem unhem pahale se hi pata hai ki usa vyakti ko echaaivi vayarasa tha aura sambhoga ke daurana kandoma tuta jata hai ,ya galati se unhonne echaaivi sankramita injekshana sui istemala ki hai |
halanki ,piipi ke purna rupa se kama karane ki garanti nahim hai ,aura jo davaem upayoga ki ja rahi haim unase bhi mitali ,ulti ,dasta aura sira darda jaise kuchha apriya dushprabhava hote haim |
yadi apa khuda echaaivi vayarasa se sankramita hone se chintita haim ,to apa turanta apane sthaniya durghatana evam apatakalina vibhaga ,ya jiyuema (janana -mutra ya yauna svasthya ) klinika se samparka karem |
charmaputha roga samanya rupa se entibayotika krima ( samayika entibayotika davaem ) se saphalatapurvaka thika kiya ja sakata hai |
krima lagane se pahale tvacha ki prabhavita jagahom ko sabuna mile garma pani se dhona chahie |
yaha bahuta hi jaruri hai ki bada mem apa apane hatha turanta dho lem , ya agara upalabdha hai to krima lagate vakta rabada़ ke dastane pahana lem |
ilaja ke prati upachara ki pratikriya samanyatah sata dinom ke bhitara dikha jani chahie |
agara upachara ki pratikriya nahim dikha rahi hai ya charmaputha bahuta jyada phaila hua ya kashtadayaka hai ,to entibayotika goliyam ( khane ke lie ) di jati haim |
khane ki entibayotika davaom ka korsa samanyatah sata dina ke lie diya jata hai | 
yaha jaruri hai ki lakshana samapta ho jane para bhi apa ya apaka bachcha entibayotika davaom ka korsa pura kara sakate haim | 
agara sambhava ho ,to sandigdha janavara ko pakada़na chahie aura 5-10dinom ke lie usa para nigarani rakhi jani chahie |
isake bada yadi rebija ke koi lakshana dikhai nahim de rahe haim ,to yaha mana ja sakata hai ki vaha janavara pagala nahim hai |
janavara ke dvara kata gaya vyakti agara davai le raha hai to vaha rebija़ ka upachara roka sakata hai |
yadi janavara mara diya jata hai ya vaha khuda mara jata hai ,to rebija़ vayarasa ki upasthiti ke lie isake dimaga ki jancha ki ja sakati hai |
manushyom mem isaki pahachana lara ki jancha ke dvara ki jati hai |
yadi sankramita vyakti mem gambhira lakshana vikasita ho jate haim aura usaki nrityu ho jati hai ,to rebija़ ki pushti karane ke lie postamartama ya dimaga ka phlorosenta entibaॉdi testa (phaeti) kiya ja sakata hai |
yadi koi janavara jo ki sambhavatah rebija़ se sankramita hai ,apako katata hai to apaka ilaja turanta kiya jana chahie |
kisi bhi janavara ke kate jane para chikitsaka ki salaha li jani chahie , aura titanesa ka injekshana bhi diya jana chahie |
amataura para katane ke turanta bada uchita ilaja shuru karane se lakshanom ko badha़ne se roka ja sakata hai |
yadi chehare para kuchha kata le to prarambhika upachara vishesha rupa se mahatvapurna hai |
yadi apako kisi ke katane ke pahale hyumana rebija़ inyunolobulina pratirakshana tika nahim laga hai jise ki nishkriya tika bhi kaha jata hai (echa ara au ji ), sharira mem vayarasa ke phailane se pahale use nishkriya karane lie kate gae sthana ke asa-pasa isaka injekshana lagaya jata hai |
yaha padartha vayarasa ko rokata hai, aura jisase pratiraksha pranali isako nashta kara sake |
isake alava, lambe samaya taka chalane vali eka rebija vaiksina (sakriya pratirakshana) 30 dinom mem 5-6 khuraka mem di jani chahie |
tika hatha mem diya jata hai aura yaha apekshakrita dardarahita hai ,sira darda, bukhara, michali, mansapeshiyom mem darda aura chakkara ana ke atirikta isake kama se kama dushprabhava haim |
yadi apako kisi ke kate jane se pahale hi vaiksina ki khuraka di ja chuki hai(purva arakshita tika ) , vaiksina ki do khuraka di jayegi
isa ilaja ki pratikriya sharira mem jalda hi dikhani chahie |
yadi sankramita vyakti ka ilaja nahim kiya gaya, aura lakshana ubhara aem (dikhane laga jaem), to aisa kaha jata ki rebija़ ka hona siddha ho chata hai |
isa sthiti mem, unako kuchha rahata dene ke alava aura kuchha bhi nahim kiya ja sakata hai |
unaki eka andhere kamare mem dekharekha ki jani chahie aura unake lakshanom ko upakarana aura dava se niyantrana mem rakhana chahie |
hridaya aura sansa ki gati ko banae rakhane ke lie , shanta karane vali aushadhi dekara aura achchhi dekhabhala ki suvidhaom ka istemala karake rebija़ ke bahuta thoda़e marija़om ka saphalatapurvaka ilaja़ kiya gaya hai , lekina jo loga bacha jate haim unhem dirghakalina vikalangata ka samana karana pada़ta hai |
poliyo ki gambhira avastha ke samanya ilaja mem marija़ ki lakava mari hui shvasana mansapeshiyom ke sthana para kritrima phephada़e (lohe ke phephada़e)ka istemala kiya jata hai- yaha eka bada़i mashina hai jo ki hridaya ki mansapeshiyom mem tanava aura khinchava utpanna karake kama karati hai | 
kritrima phephada़e ke karana kharaba anga aksara gatihina ho jate haim |
kai deshom mem, jaham poliyo abhi bhi eka chinta ka vishaya hai, kuchha ventiletara aura (lohe ke) kritrima phephada़om ka abhi bhi upayoga kiya jata hai|
pahale pairalitika poliyo aura snayavika vikasa jisamem purna rupa se na hua ho aise poliyo ka gharelu upachara paryapta nahim tha |
halanki jisamem snayavika vikasa purna rupa se na hua ho poliyo ke aise halke mamalom ka sadharanatah bahuta sara tarala padartha dekara aura vishrama karakara upachara kiya jata tha |
roga ki pahachana lakshanom para nirbhara karati hai jinako tuma anubhava kara rahe ho aura sharirika lakshana , udaharana ke lie , gale ki jilli ka ranga badalana aura gale ka sujana |
yadi apaka chikitsaka sochata hai ki apako dipthiriya hai to phahe ke dvara naka ,gale aura tvacha ke ghavom ,kate hue sthana ya alsara se sankramana aura baiktiriya ke namune liye jayenge jinaki prayogashala mem maikroskopa se jancha ki ja sakati hai |
agara kisi mem si. dipthiriya baiktiriya paya jata hai to daktara yaha bata sakata hai ki bimari kisa hada taka badha़ sakati hai aura yadi sankramita vyakti kisi ke bhi samparka mem ata hai to kya use bhi koi khatara ho sakata hai |
lakshana dikhane ke 48 ghantom ke bhitara entitaॉksina upachara diya jata hai aura diptheriya taॉksina pratirodhaka entibaॉdija़ ki sharira mem matra mapane ke lie rakta ke namune ki bhi jarurata hoti hai |
sharira ki pratiraksha pranali dvara bimariyom aura sankramanom se lada़ne ke lie utpanna kiye jane vale koshanuom ko entibaॉdija़ kaha jata hai |
entibaॉdija़ ko punah mapane ke lie rakta ka dusara namuna kuchha haphtom bada liya jata hai |
bimari aura na phaile isake lie apako dusare marijom se alaga kamare mem aspatala mem bharti kara diya jayega |
si.diptheriya ke jivanuom ko marane ke lie apako 7 se 14 dinom ke lie entibaॉyotika ka korsa diya jayega jisamem pahale injekshana diya jayega aura bada mem goli di jayengi |
sharira mem taॉksina ka prabhava kama karane ke lie apako entitaॉksina ka injekshana bhi diya jayega |
jaba taka sare parikshana apake sharira ko sankramana-mukta nahim dikhate aura jaba taka puri taraha se thika nahim ho jate taba taka apako alaga hi rakha jayega |
bilkula svastha hone ke bada aspatala se nikalane se pahale apako bustara ya diptheriya vaiksina ka pura korsa diya jaega | 
koi bhi jisane sankramita vyakti se samparka kiya ho jaise ki ghara ke sadasya , agantuka aura aisa koi jisane use chuma ho ya usake satha yauna sambandha kiya ho, use turanta apane daॉktara ke pasa jakara diptheriya ke lakshanom ki jancha karavani chahie |
isamem pratyeka vyakti se sata dinom taka roja़ naka aura gale ke balagama ke namune lena shamila hai |
agara jaruri hua to unhem entibayotika ka upachara aura vaiksina ki bustara doja़ di jati hai |
kama para sahayogiyom se ya skula ke dostom se diptheriya ke sankramana ke hone ka khatara kama hota hai |
diptheriya eka aisa roga hai jisaka ilaja karane ke bajaya use roka jana chahie hai , lekina agara shuru mem hi roga ki pahachana ho jae tabhi upachara ya niyantrana ke ye tarike sankramana ko niyantrita karane mem prabhavashali hote haim |
upachara hamesha bahuta jaruri hai aura isamem deri nahim ki jani chahie |
kali khansi ke rogiyom ko baiktiriya ko marane ke lie sadharanatah entibayotika di jati hai |
entibayotika ke bada bhi khansi ki tivrata 6-8 haphtom taka bani rahati hai | 
5 dinom taka entibayotika lene ke bada apa sankramita nahim rahenge |
yadi apake bachche ko halki kali khansi hai, use bahuta arama diya jana chahie , bahuta adhika tarala padartha pina chahie aura jaba taka vaha sankramita ho taba taka use anya bachchom se dura rahana chahie (jaba se unhem lakshana prarambha hom taba se do chara haphtom ke lie )|
atirikta balagama ya ulti ko hata dem taki apaka bachcha use nigala na le aura jisase unaka gala avaruddha ho jae |
aba taka vishva svasthya sangathana se koi chikanaguniya ke lie dava nirdharita nahim hui hai|
daॉktara darda ko kama karane lie esaparina ,bruphina, kaॉmbiphlema , pairasitamaॉla ityadi ka istemala karate haim para elopaithika mem sarvada manya abhi taka koi bhi davai nahim hai|
kuchha logom ko kuchha davaiyom se phayada hota hai, jabaki dusarom ko nahim hota hai |
kuchha chikunaguniya ke marija़ kuchha dinom bada apane -apa thika ho jate haim |
jaham taka chikanaguniya ka sambandha hai chikitsa vijnana puri taraha vishvasa nahim karata hai |
aisa dekha gaya hai ki jyadatara loga chikanaguniya ka ayurvedika upachara karate haim |
ayurveda bhi chikanaguniya ka puri taraha se ilaja karane mem sakshama nahim ho sakata hai, lekina aisa ho sakata hai ki yaha sharira ko roga ke satha mukabala karane ke lie avashyaka pratirodha pradana kare |
ayurveda mem kuchha jada़i buti aisi haim jo ki chikanaguniya ke lakshanom ko kama karane ki kshamata rakhati haim|
matra lakshanom ke adhara para inphluenja ke karana hui sansa ki bimari aura roganuom ke karana hui sansa ki bimari mem pharka kara pana mushkila hai |
halanki prayogashala mem inphluenja ke prakopa ki pushti hoti hai para jyadatara loga inphluenja se sankramita sansa ki nali ke ilaja ke lie chikitsakiya salaha lena chahate haim |
varshika phlu mahamari ki pushti ke lie prayogashala ki jancha avashyaka hogi |
apane bachchom ko hamesha prithvi mata ke samparka mem rahane dijie |
jo loga sanyukta parivara mem rahate haim ve loga bhavanatmaka rupa se adhika sthira lagate haim |
jo bachche dada - dadi ki goda mem badha़te haim unhem paraspara sneha aura samaya milata hai |
isa prakara hara vyakti bhavanatmaka sankata ke kshanom mem aura adhika samarthana prapta karata hai |
pichhale yuddha ki vajaha se, britisha ko mata pita ke pyara aura parivarika jivana ke sandarbha mem bahuta bhugatana pada़a |
isa sarvekshana se yaha nishkarsha samane aya hai ki aja ke samaja mem shadi jaisi pavitra rasma ka koi mahatva nahim hai |
svatantra pyara aura dragsa lena phaishana mem the|
britena, aja-kala svachchhanta aura ekala abhibhavaka parivarom aura jalda talakom ke khilapha़ hai | 
bachchom ko bada़a karane ke lie donom abhibhavakom ke hone ke paksha mem eka bada़e paimane para paramparika parivara ko vishesha ahamiyata di gai hai |
15 varsha ki ayu varga ke 1500 vayaskom ke bicha yaha sarvekshana ayojita kiya gaya tha | 
bharata mem, hama behatara haim|
sanyakta parivara abhi bhi hamari parampara hai aura svachchhanta abhi kuchha kadama dura hai |
pata nahim hama kaba taka teja़i se chalane vali isa duniya mem videshi adatom ,tarikom , riti-rivaja़om aura jine ke tarikom se bache raha sakate haim |
dhyana vidhiyam alaga alaga hai para vastava mem ve saba eka hi haim kyonki unaka antima uddeshya to vikasa mem madada karana aura antarika jagarukata ko badha़ane mem phayada uthana hai |
eka taraha se dhyana ke madhyama se, apa apane sharira aura bahara ki duniya ke prati aura adhika jagaruka ho jate haim|
apa jisa bhi taraha ka kama karate ho apa usamem behatara sochane mem samartha ho pate ho |
apa aura adhika kushala ho jate haim, apaka sharira aura adhika svastha ho jata hai|
dhyana eka shanta, aramadayaka vatavarana mem kiya jana chahie |
apane dimaga ko mantra va sansa para kendrita kijie |
apake prashikshaka dvara bole gae mantrom , shabdom aura avaja ya sanketa se eka uttama dhyana mem madada milati hai |
takanika asana hai , jaldi sikhi ja sakati hai aura damaga ko shanti deti hai aura apako tanavamukta avastha mem le jati hai|
vipasana dhyana kisi bhi mantra ki madada nahim leta hai |
yaha kevala sansa ki madada leta hai |
yaha sabase purana rupa mana jata hai, nirvana prapta karane ke lie bhagavana buddha dvara isaka istemala kiya gaya tha |
yaha sharirika aura manasika tanavamukti dene ke alava, rogom ke ilaja mem bhi adhika mulyavana hai|
parajiviyom ke unmulana ki bajaya rogi ko tatkala upachara upalabdha karana jyada jaruri hai |
maleriya rodhi upachara ke tarike vibhinna deshom mem mahamari vijnana ke adhara para roga ke sankramana ,davai ki rodhaka shakti evam rajanitika aura arthika sandarbha ke anusara badalenge |
svastha-chikitsaka apaki bimari ke lakshanom ko dekhakara inphluenja़a ko pahachana lete haim khasakara agara asa-pasa ke samudayom mem bimari ke eka samana lakshanom vale kai mamale samane ate haim to svasthya adhikari inphluenja़a ko pahachana lete haim, aura sthaniya svasthya vibhaga phlu ke prakopa ka hona pakka kara dete haim |
jina logom mem phlu ke pratikatmaka lakshana bhi hom , aise logom ki phlu ki samanya jancha hona jaruri nahim hai |
vishishta phlu vayarasa ko apa shayada hi rakta parikshana ya naka ya gale ke balagama ke namunom ke parikshana dvara pahachana sakate haim | 
amataura para phlu ke lakshanom ke lie griha upachara kaphi hota hai |
halanki lakshanom ki avadhi aura gambhirata ko kama karane ke lie entivayarala davaem upalabdha haim |
ye davaem vishesha rupa se budha़e vayaskom aura dusarom , jinhem phlu se pareshaniyom ke vikasa ke lie khatara hai, ke lie upayogi hoti haim |
pahala lakshana ke dikhane ke 2 dinom ke bhitara ye davaiyam shuru kara di jani chahie |
adhikatara kanjektivaitisa se sankramita mamalom mem apaka daॉktara lakshanom se aura pariksha dvara sthiti ka pata lagane mem sakshama ho jaega |
sadharanatah kanjektivaitisa ke sankramana mem ankhem lala aura suja jati haim aura chipachipe kichada़ se bhara jati haim ye pramukha lakshana haim |
jyadatara kanjektivaitisa ke sankramana vale kesa bina kisi upachara ke eka ya do haphtom mem thika ho jate haim |
yadi apa abhi bhi do saptaha ke bada kanjektivaitisa ke lakshanom se pida़ita haim, to apako apane daॉktara ko phira se dikhane ki jarurata hai jisase vaha roga ki pahachana aura upachara para punah vichara kara sake | 
apaka daॉktara sunishchita karega ki apa anya taraha ki taka़liphom jaise elarji, ya takalipha़ dene vali kanjektivaitisa adi se pida़ita nahim ho jisase apake upachara para prabhava pada़e jo ki apako diya ja raha hai|
apaka daॉktara yaha bhi jancha karega ki apa ankha ki kisi anya taka़lipha se pida़ita to nahim haim jaise blaiphairaitisisa jisake karana palakem lala aura suja jati haim |
agara apa ina lakshanom ko mahasusa karate haim to apake lie chikitsakiya madada lena bahuta jaruri hai kyonki ho sakata hai ki yaha eka sanketa ho ki isase adhika gambhira taka़liphem hom jo inaka karana hom |
yahi karana hai ki apake chikitsaka ke lie jaruri hai ki vaha anya lakshanom ko pahachana kara unaka nidana kare taki ye bada mem gambhira samasyaom ka karana na bane |
gambhira glukoma- glukoma ka yaha eka durlabha rupa hai jo ki ankha mem dabava ke karana banata hai |
darda aura drishti ka khona adi lakshana bahuta teji se vikasita ho sakate haim |
anupacharita gambhira glukoma ke parinamasvarupa drishti hamesha ke lie ja sakati hai |
kairatitisa-jaba apaki kaॉraniya (eka paradarshaka jilli jo ki apaki ankhom ke samane hoti hai jisase hokara prakasha apaki ankhom mem ata hai )lala ho jati hai aura kabhi -kabhi suja bhi jati hai |
gambhira mamalom mem , isaki vajaha se kaॉraniya para nishana bana jata hai , aura jisase hamesha ke lie drishti chali jati hai |
airitisa-isake karana apaki ankha ki putali (kaॉraniya ke pichhe jo rangina bhaga hota hai) suja jati hai |
yadi apako airitisa hai, aura isaka upachara nahim kiya jae , to isake karana putali lainsa ke samane ke bhaga se chipaka jati hai jo ki tarala padartha ko pyupila se nikalane se rokata hai |
isase apaki ankhom ko hamesha ke lie nukasana ho sakata hai|
yadi apaka daॉktara apake sankramana ke prati nishchita nahim hai ,ya usako sankramana ka karana janane ki avashyakata hai ,to vaha apaki sankramita ankhom ki saphai karata hai|
apaki pareshani ka karana pata lagane ke lie prayogashala mem isaka parikshana kiya jata hai| 
tabhi apaka daॉktara apaka sabase upayukta ilaja kara sakata hai|
apaka daॉktara jaba apaki ankhom ki saphai ke parinamom ka intajara kara raha hai ,taba isa bicha apake lakshanom ko niyantrana mem rakhane ke lie apako entibayotika davaom ka eka sankshipta korsa de sakata hai|
eka bara jaba apaka daॉktara ankhom ki saphai ke parinama jana leta haim ,to vaha apaki isa sthiti ke karanom ke lie sabase upayukta ilaja nirdharita kara sakata hai|
yadi apaka bachcha 28 dinom ka hai ,ya usase bhi chhota hai ,aura usamem sankramaka kanjektivaitisa (netrashleshmalashotha )ke lakshana dikha rahe haim ,to apane daॉktara se sidha samparka karana bahuta jaruri hai|
kai bachchom ki 'chipachipi ankhem ' ho jati haim|
amataura para aisa taba hota hai jaba ashruvahini (lekrimala dakta ) thika se ansu bahara nahim nikala pati|
jaba ansu thika se bahara nahim nikala pate ,to mavada banani shuru ho jati hai ,jo sankramaka kanjektivaitisa ki taraha dikhati hai|
lekina ,yaha halata gambhira nahim hai aura yaham taka ki isake tatkala upachara ki bhi avashyakata nahim hai|
yadi apake bachche ki ankhom mem lalima bhi hai ,to yaha ankha ke sankramita hone ka sanketa hai|
agara aisa hai to apaka daॉktara apake bachche ki jancha bariki se karega|
kanjektivaitisa se sankramita kisi bhi navajata shishu ko sidhe netra roga visheshajna ( ऑphthailamolaॉjista) ke pasa le jana chahie ,taki ankhom mem hone vali kisi bhi kshati ko rokane ke lie unaki halata sambhali ja sake aura unaka ilaja kiya ja sake|
bahuta hi kama mamalom mem ,navajata shishuom mem sankramaka kanjektivaitisa (navajata kanjektivaitisa ) se bahuta hi gambhira jatilataem ho sakati hai ('jatilataem' anubhaga dekhem )|
halanki ,jyadatara bachche shighra upachara se bahuta jaldi thika ho jate haim|
sankramaka kanjektivaitisa ke adhikansha mamalom mem kisi bhi prakara ke chikitsa upachara ki avashyakata nahim hoti|
chunki adhikatara sankramana bina upachara ke eka se do saptaha ke bhitara hi thika ho jate haim ,isalie apaka daॉktara shuru mem koi bhi vishesha dava ya upachara nahim likhega |
sankramaka kanjektivaitisa ke adhikansha mamalom mem ,entibayotika davaom se sehata mem thoda़a sudhara ho jata hai|
ho sakata hai ki halata apane apa hi sudhara jae isalie ,yaha dekhane ke lie apako sata dinom ke lie dava na lene ki bhi salaha di ja sakati hai|
lagabhaga 10% loga jinake sankramaka kanjektivaitisa ka ilaja entibayotika davaom se ho raha hai , davaom ke dushprabhava jelate hi haim|
bagaira ilaja़ vale sankramaka kanjektivaitisa se kisi bhi prakara ki jatilata ka khatara bahuta hi kama hota hai ,aura isilie entibayotika davaom se ilaja karana shayada hi kabhi kabhi avashyaka hota ho| 
yadi apaka sankramaka kanjektivaitisa vishesha rupa se gambhira hai ,aura use do saptaha se bhi adhika ho gaye hai ,to apako entibayotika ilaja़ ki avashyakata ho sakati hai|
kuchha skula,ya krida़a samuha isa bata para jora dete hai ki eka bachche ka ilaja़ entibayotika davaom se kiya jana chahie isase pahale ki unako yaha dobara ho (halanki aisa bahuta hi kama hota hai - 'nivarana' anubhaga dekhem)
aura isilie ,yadi aisa hota hai,to apaka daॉktara entibayotika davaem likhane ke lie raja़i ho sakata hai|
yadi apako ankhom ki dava(ai draॉpa)dalane ke lie kaha gaya hai,to usako dalane se jaldi hi apaki drishti dhundhali ho sakati hai|
ai draॉpa dalane ke turanta bada apako koi vahana nahim chalana chahie ya koi mashina nahim chalani chahie ,aura,aisa karane se pahale, apa hamesha yaha sunishchita kara lem ki apaki drishti spashta hai ya nahim|
yadi do saptaha ke bada bhi lakshana maujuda haim ,to yaha bahuta hi mahatvapurna hai ki apa apane daॉktara ke pasa jaem ,taki apaki halata ka punarmulyankana kiya ja sake ,aura apake ilaja ki samiksha ki ja sake|
skebija़ (khujali) ke ghuna ke bilom ko vishishta rupa se eka syahi parikshana ke dvara darshaya ja sakata hai|
khujali vali tvacha ke asapasa syahi ko mala jata hai, aura phira use eka sprita se bhigi rui se ponchha diya jata hai|
agara skebija़ ke ghuna ke bila maujuda haim,to kuchha syahi vaham raha jayegi aura eka kali laina ke rupa mem bila ka pata laga jaega|
roga ki pahachana ki pushti ke lie ,tvacha ke prabhavita kshetra se eka namuna halke se kharonchakara liya jata hai,jisaka parikshana maikroskopa se kiya jata hai| 
tvacha ka namuna ghuna ko ,unake ande aura unake mala ko dekhane ke lie istemala kiya jata hai|
agara parivara mem eka se adhika sadasyom ko eka jaise hi lakshana haim to roga ki pahachana ko aksara gupta bhi rakha ja sakata hai|
halanki skebija़ ke lie daॉktara ke bina likhe bhi ovara-da-kauntara (otisi) upachara upalabdha haim ,phira bhi apako apane daॉktara ke pasa jane ki salaha di jati hai agara apako pahali bara skebija़ hua hai|
aisa isalie kaha jata hai kyonki tvacha ki anya dusari adhika gambhira bimariyam bhi skebija़ ki taraha ke hi lakshana paida karati haim|
isilie apake daॉktara ke lie apake lakshanom ka karana janana bahuta hi jaruri hai| 
yadi apake parivara ke kisi anya sadasya mem pahale se hi skebija़ ki pahachana ho gayi hai ,to apako apane daॉktara ke pasa jane ki jarurata nahim hai aura apa apana upachara karava sakate haim|
skebija़ ke lie sabase vyapaka rupa se istemala ki jane vali do krimem paramethrina krima aura melathiyana krima haim|
amataura para sabase pahale paramethrina di jati hai ,aura agara paramethrina krima aprabhavi hoti hai to melathiyana krima istemala ke lie di jati hai|
donom krimom mem kitanashaka hote haim jo skebija़ ke ghuna ko mara dete haim|
yadi apa paramethrina krima laga rahe haim, to apa ise apani thoda़i aura kanom se lekara niche ki ora lagate hue apane pure sharira para malem|
yadi apa do varsha se kama unra ke bachche ko krima laga rahe haim,to unaki khopada़i ,chehare aura kanom para bhi krima lagani chahie|
krima lagate vakta,hatha paira ki ungaliyom para,aura nakhunom ki tvacha ke niche vishesha dhyana dena chahie|
krima ko lagabhaga 8-12 ghante ke lie laga rahane dijie|
krima lagate vakta yadi apa apane hatha,ya sharira ke kisi anya bhaga ko dho lete haim,to krima ko puna: lagane ki avashyakata hogi|
sata dinom ke bada upachara phira doharaya jana chahie|
melathiyana krima lagane ke nirdesha bhi bilkula paramethrina krima lagane jaise hi hai ,sivaya isake ki ise 8-12 ghante ke bajaya 24 ghante ke lie laga rahane dijie|
sata dinom ke bada phira se upachara doharane ki avashyakata hoti hai|
usa dina apa pahale krima lagaem aura sare kapada़e ,chadarem aura tauliye 50C(122F) se jyada ke tapamana para dhone chahie|
yadi apa kuchha vastuom ko dho nahim pa rahe haim ,to unhem kama se kama 72 ghante ke lie eka plastika ke thaile mem rakha dijie ,jisase ki itane samaya mem ghuna svayam hi mara jaegem|
pharnichara,palatu janavara ya kalina adi ko dhuam dene ki avashyakata nahim hai|
yadi apa mem skebija़ ki pahachana ho gayi hai,to apa anya logom ke satha karibi evam lambe samaya taka ke sharirika samparka se taba taka bachem jaba taka ki apa pahale krima na laga lem|
apa ghara ke anya sadasyom ke satha bhi nikata samparka se bachem jaba taka ki unaka ilaja pura na ho jaem|
eka bara krima lagane ke bada bachche aura vayaska apane apane skula ya kama ko ja sakate haim| 
apake daॉktara ko kama steraॉyada vali krimem likhani chahie ,jo khujali ke lakshanom mem rahata dene mem madada karati haim| 
daॉktara ke bina likhe bhi ovara-da-kauntara (otisi)upaya,kaimomaila loshana tvacha mem khujali se rahata dilane ke lie madada kara sakate haim|
khane ke lie di gayi entihistemaina dava khujali se rahata dilane aura sone mem madada kara sakati hai|
inako apani sthaniya pharmesi se bina daॉktara se puchhe bhi kharida ja sakata hai|
halanki,inase unindapana ho sakata hai,aura agara apa aisa mahasusa kara rahe haim to vahana ya bhari mashina ko chalane se bachem|
yadi apa nishchita nahim haim, to apane daॉktara ya pharmasista se jancha karaem|
khujali ke lakshana upachara ke bada bhi kai haphtom taka dikha sakate haim kyonki apaki pratiraksha pranali abhi bhi skebija़ ke nrita ghunom aura unake apashishta utpadom ke prati samvedanashila hai|
agara khujali ke lakshana chhaha saptaha se adhika jari rahate haim to apane daॉktara ke pasa jaem|
vishesha lakshanom ke adhara para daॉktara amataura para roga ki eka dridha़ pahachana kara leta hai|
prayogashala mem parikshana ke lie , gale mem pichhe se lara ka eka namuna le liya jata hai|
yaha sankramana karane vale jivanu ki pushti karane mem madada karata hai|
kuchha mamalom mem,eka rakta parikshana ki bhi avashyakata ho sakati hai|
isaka matalaba yaha hai ki apake chikitsaka ko aise kisi bhi mamale ko sthaniya svasthya adhikariyom ko batana chahie jisase ve roga ke prasara para nigarani kara sakem|
skarleta jvara ke sabase sadharana mamalem bhi yaham taka ki bina upachara ke bhi lagabhaga eka haphte mem dura ho jate haim |
halanki,teji se svasthya labha ke lie aura kaisi bhi jatilataom se bachane ke lie apako ilaja karane ki salaha di jati hai|
upachara karane se,jyadatara loga 4 se 5 dinom mem hi thika ho jate haim|
entibayotika davaom ka 10 dina ka eka korsa hi skarleta jvara ke lie samanya upachara hai|
isamem amataura para khane ke lie penisilina li jati hai|
yadi apake bachche ko penisilina se elarji hai,to usaki bajaya irithromaisina namaka entibayotika li ja sakati hai|
agara entibayotika davaem thika taraha se li jaem to lakshana amataura para kuchha hi dinom mem samapta ho jate haim|
yaha nishchita karane ke lie ki sankramana puri taraha se sapha ho gaya hai ya nahim,upachara ka pura korsa samapta kiya jana chahie|
amataura para entibayotika korsa shuru karane ke 12 se 24 ghante ke bhitara hi bukhara gayaba ho jata hai|
skarleta jvara vale vyakti ko khasakara jaba thoda़i bhi bhukha lagati hai to atyadhika matra mem thande tarala padartha pine chahie| 
kamare ka tapamana bhi thanda rakhana chahie|
darda mem rahata ke lie evam teja bukhara ko kama karane ke lie perasitamola bhi li ja sakati hai|
skarleta jvara ko yadi bina ilaja ke hi chhoda़ diya jae, to usake kai haphtom bada taka tvacha utarati rahati hai aura jatilataom ka khatara bhi badha़ sakata hai|
roga ke lakshanom aura naidanika lakshanom ,rakta mem kitanuom ke samvardhana aura kamara mem chheda karake mastishkameru drava ke vishleshana ke adhara para aisa mana gaya hai|
kamara mem chheda karane ke bare mem adhika janakari ke lie 'sambandhita lekha' anubhaga dekhem|
yadi maininjaitisa ka sandeha hai, to roganu samvardhana se pushti hone taka ke intajara ke bina hi entibayotika davaem tatkala di jani chahie|
roganuom ke rakta pravaha mem pravesha karane (saiptisimiya)se agara lala chakate hue haim to eka gilasa parikshana kiya ja sakata hai jisamem kharoncha ya lala chakatom ke nishana para pani pine ke eka sapha gilasa ko dabakara jancha jata hai ki ve lala chakate phike ho rahe haim ya nahim|
yadi chakatom ke nishana phike nahim ho rahem haim,to ina lala chakatom ke hone ka karana saiptisimiya ho sakata hai|
kuchheka mamalom mem lala chakatte pahale to phike ho jate haim lekina bada mem badala jate hai aura phira phike nahim pada़te|
baiktiriyala maininjaitisa vale vyakti ke lie entibayotika davaom aura uchita aspatala prabandhana se tatkala upachara bahuta avashyaka hai|
jitani jaldi roga ki pahachana hogi aura usaka ilaja hoga,utani hi jaldi svasthya labha hone ki sambhavana adhika se adhika hogi|
sankramita vyakti ke satha jo koi bhi pratyaksha rupa se, karibi, lambe samaya taka samparka mem haim (samanya rupa se parivara ke sadasya aura jinake lie khatara jyada hai)yadi avashyaka ho to unako uchita surakshatmaka entibayotika davaem di jani chahie|
vayarala maininjaitisa entibayotika davaom se thika nahim hota aura usaka upachara achchhi narsinga dekhabhala para nirbhara hota hai|
svasthya labha samanya rupa se puri taraha ho jata hai, lekina haphtom ya mahinom taka sira darda ,thakana, avasada bane raha sakate haim|
machchharom dvara manushya mem phailane vala maleriya eka ushnakatibandhiya roga hai, aura yaha 100 se adhika deshom mem paya jata hai|
vishesha rupa se havai adde ke karyakartaom aura una vyaktiyom se jo anya deshom se ayatita vastuom ke samparka mem rahate haim, ke dvara inglainda mem ekadha bara maleriya ka chhita puta prakopa hua hai|
britena mem prati varsha kula milakara maleriya ke lagabhaga 2,000 mamalem paye jate haim|
maleriya mukhya rupa se aphrika, dakshina aura madhya amerika, eshiya aura madhya purva ke deshom ko prabhavita karata hai|
yaha bimari vishesha rupa se upa sahara aphrika mem bada़e paimane para phaili hui hai, jaham 90 % se adhika mautem maleriya se sambandhita hoti haim|
maleriya se sambandhita sabhi mautom mem se lagabhaga do tihai mautem vishva ki janasankhya ke 20% sabase gariba logom ki hoti haim|
tritiyaka maleriya mamuli hota hai aura apekshakrita ilaja karane ke lie asana hota hai|
melignainta maleriya bahuta gambhira ho sakata hai, aura kabhi kabhi ghataka bhi hota hai|
yadi asadhya maleriya ki ashanka hoti hai, to shighra chikitsiya vishesha dekharekha ki avashyata hoti hai|
chhoti mata eka atyadhika sankramaka roga hai jo vishanuom ke eka samuha mem se eka harpisa namaka vishanuom dvara utpanna hota hai|
kai vibhinna prakara ke harpisa vishanu hote haim, kintu eka lakshana jo una sabhi mem hai ki ve prarambhika sankramana ke pashchat sharira mem prasupta rahate haim, aura bahuta bada mem punah sakriya ho sakate haim|
bhainsiya dada ke rupa mem jane jane vale chhoti mata ke vishanu ke punah sakriya hone se yaha roga utpanna hota hai aura yaha taba ho sakata hai jaba pratirakshi tantra manda pada़ jata hai|
pha़lu (inpha़luenja़a ke rupa mem bhi jana jane vala) phephada़om aura upari vayumargom ka eka roga hai jo eka pha़lu vishanu ke sankramana se utpanna hota hai|
yaha vishanu phephada़om aura upari vayumargom mem phaila jata hai|
e, bi aura si ke rupa mem bhi jane jane vale, tina pha़lu vishanu hote haim| isake pramukha lakshana haim uchcha tapamana jo shighra ata hai aura samanya darda tatha pida़a|
apa bhukha ki kami, michali aura khurakhuri sukhi khansi ka bhi anubhava kara sakate haim|
apake lakshana samanyatah 2-3 dinom mem choti para pahuncha jate haim aura apako 5-8 dinom ke bhitara bahuta behatara mahasusa karana shuru kara dena chahie, halanki khansi aura samanya thakana 2-3 haphtom taka raha sakata hai|
yaha pha़lu vishanu samanyatah eka sankramita vyakti dvara vatavarana mem khansi ya chhinki hui lara ki chhoti bundom mem phaila jata hai|
vishanu se sandushita hathom se sidhe samparka se bhi yaha vishanu sankramana phaila sakata hai|
sankramita hone se purna lakshana hone taka pahunchane mem yaha 1-4 dina (ausatana 2 dina) leta hai|
pha़lu vale vyakti samanyatah lakshana prarambha hone ke eka dina purva sankramaka hote haim aura pha़lu lakshanom ke prarambha hone ke bada lagabhaga 5 dinom ke lie sankramaka bane rahate haim|
kamatara pratirakshi tantra vale bachche aura vyakti adhika lambe samaya ke lie sankramaka bane raha sakate haim|
atah apako isa sankramaka avadhi ke daurana dusarom ke satha sabhi anavashyaka samparka se bache rahane ki koshisha karana chahie|
yuke mem pha़lu samanyatah jada़e ke mahinom ke daurana, aktubara se apraila taka hota hai|
jatilataem jaise ki sine ka sankramana una logom ko prabhavita kara sakata hai jo vayovriddha haim, ya unhem jo vishesha chikitsiya roga vale haim|
yaha gambhira roga mem parinita ho sakata hai aura jivana ghataka ho sakata hai|
jada़e ke daurana pha़lu ki taraha roga vale logom ki sankhya jo apane samanya chikitsaka se paramarsha lete haim, sala-dara-sala atyadhika parivartita hoti hai (samanyatah prati 100,000 jana sankhya ke lie 50 aura 200 ke bicha)|
phira bhi, pha़lu vale svastha vyaktiyom ko apane samanya chikitsaka se paramarsha lene ki avashyakata nahim hoti haim|
yadi hara haphte prati 100,000 jana sankhya ka 400 se adhika loga pha़lu ya pha़lu ki taraha ke roga ke satha apane samanya chikitsaka se paramarsha lete haim to eka mahamari ghoshita ki ja sakati hai|
pha़lu ki taraha ka roga anya rogom ka eka prakara batata hai, jo samana lakshanom mem parinita hote haim|
yakrita-shotha ke sabhi tivra mamalom ka lagabhaga adha eka vishanu sankramana ke karana hote haim|
yakrita ko hipaitaitisa vishanu ke kai prakara sankramita kara sakate haim, kintu sabase samanya haim hipaitaitisa e aura bi vishanu|
hipaitaitisa e ghatiya vyaktigata saphai ya svachchhata ke madhyama se bhojana aura pani ke vishtha (mala) ke satha dushana se hota hai|
hipaitaitisa bi rakta aura sharirika dravom ke adana-pradana se phailata hai|
yaha asurakshita yauna sambandha, aroganurahita sui, suinoka aghata (eka prayukta sui se tvacha ka akasmika chhidana), ya sandushita rakta utpadom se ho sakata hai|
hipaitaitisa si bhi rakta aura rakta utpadom ke adana-pradana se phailata hai|
yaha sui adana-pradana karane aura suinoka durghatanaom se phailata hai|
yaha sitambara 1992 se pahale, jaba hipaitaitisa si ke lie jancha karana lagu kiya gaya tha, taba rakta adhana se bhi phailata tha|
si se ji taka nama rakhe gaye chara anya svikrita hipaitaitisa vishanu haim| hipaitaitisa e aura i kevala tivra sankramana utpanna karate haim, jabaki hipaitaitisa bi aura si jirna (chalate a rahe) roga utpanna karate haim|
hipaitaitisa di kevala hipaitaitisa bi se sankramita logom mem hi upasthita hota hai| granthiya jvara vishanu bhi hipaitaitisa ka eka karana ho sakata hai|
hipaitaitisa e aura hipaitaitisa bi (alaga-alaga ya mishrita) ke lie, aura hipaitaitisa e tatha taipha़aida ke eka sammishrana ke rupa mem bhi tike upalabdha haim|
sva-pratirakshita hipaitaitisa jirna hipaitaitisa (dirghavadhi sujana aura jalana) ka eka bahuta asamanya karana hai|
karana abhi bhi ajnata haim ki kyom pratirakshi tantra joki samanyatah kisi vishanu, kitanu ya jivanu para akramana karata hai, angom ko ina bahari pindo mem se eka ke rupa mem bhula karate hue sharira para akramana karana arambha kara deta hai|
sva-pratirakshita hipaitaitisa mem apaki shveta rakta koshikaem (limphosaitsa kahalate haim) jirna sujana aura jalana tatha kshati utpanna karate hue apake yakrita para akramana karati haim|
yaha taba adhika gambhira samasyaom jaise ki yakrita viphalata ki ora agrasara ho sakati hai|
prati varsha 100,000 mamalom mem kevala 1 nidana kiye jane ke satha sva-pratirakshita hipaitaitisa yuke mem bahuta asamanya hai|
10 mamalom mem lagabhaga 7 samanyatah 15-40 ki unra ke bicha ki mahilaom mem haim|
phira bhi, adhika unra vale samuhom mem mahilaom ki apeksha purusha adhika prabhavita hone ki ora jukava rakhate haim| 
yadi apa ina lakshanom mem se kisi ko bhi prakata karana arambha karate haim to apa apane samanya chikitsaka se shighra milem taki eka prarambhika nidana ke lie parikshana kiye ja sakem|
sva-pratirakshita hipaitaitisa ke upachara mem ve davaiyam shamila haim jo sujana aura jalana ko kama karane ke lie pratirakshi tantra ko daba dene mem sahayata karati haim|
steraॉida aushadhi (prednisolona) kai haphtom mem dhire-dhire apake sujana ko kama kara sakata hai, aura taba apake lakshanom ko niyantrita karane ke lie upayoga kiya ja sakata hai|
hipaitaitisa e hipaitaitisa e vishanu dvara utpanna eka sankramana hai, jo yakrita ke sujana aura jalana ki ora agrasara hota hai|
yaha vishanuka hipaitaitisa ka sabase samanya prakara hai|
yuke mem hipaitaitisa e hota hai, kintu yaha kuchha anya deshom jaham svachchhata aura malajala nishkasana ghatiya ho sakati hai, visheshakara aphrika, uttari evam dakshini eshiya, madhya amarika tatha dakshini aura purvi yuropa mem adhika samanya hai|
yaha mukhya rupa se bachchom aura javana vayaskom ko prabhavita karata hai|
yaha vishanu prayah apake munha mem kuchha dalane se hota hai jo ki kisi hipaitaitisa e vale ke mala (vishtha) se dushita ho chuka hai| hipaitaitisa e samanyatah eka tivra (alpavadhi) sankramana hai aura halanki lakshana apriya ho sakate haim, yaha kabhi kabhara hi gambhira hota hai|
hipaitaitisa e kisi bhi unra mem ho sakata hai|
udbhavana kala (isa vishanu ke samparka mem ane se isa sankramana ke vikasita hone taka ka samaya) lagabhaga 2-6 haphtom ke bicha hota hai|
hipaitaitisa eka samanya shabda hai jisaka artha hai yakrita ki sujana aura jalana|
yakrita ka sujana aura jalana sankramana, madira ki arakshitata se, vishishta aushadhiyom, rasayanom, ya vishom, ya pratirakshi tantra ke vikara se parinita ho sakata hai| 
hipaitaitisa e hipaitaitisa e vishanu (echaevi) ke sankramana se utpanna yakrita ke sujana aura jalana ki ora ishara karata hai|
echaevi una kai vishanuom mem se eka hai jo hipaitaitisa utpanna kara sakata hai aura yunaiteda stetsa mem 3 sabase samanya hipaitaitisa vishanuom mem se eka hai|
anya 2 haim hipaitaitisa bi aura hipaitaitisa si. hipaitaitisa bi aura hipaitaitisa si se bhinna, hipaitaitisa e jirna (chalata a raha, dirghavadhi) roga nahim utpanna karata hai|
halanki yakrita uttejita ho jata hai aura suja jata hai, yaha adhikatara logom mem bina kisi dirghavadhi kshati ke puri taraha se thika ho jata hai|
eka bara hipaitaitisa e hone ke bada, apa ajivana pratiraksha vikasita kara lete haim aura isa roga ko punah prapta nahim kara sakate|
kyonki jisa taraha se yaha phailata hai, hipaitaitisa e vishanu mahamariyom aura prakopom mem hone ki ora jukava rakhate haim|
yunaiteda stetsa mem 3 logom mem jyada se jyada 1 ke pasa echaevi ke prati roga-pratikaraka hote hai, matalaba unhem vishanu ho chuke haim kintu adhikatara bimara nahim pada़te haim| yunaiteda stetsa mem hipaitaitisa e ke mamalom ki sankhya vibhinna sampradayom mem alaga-alaga hai aura 1990 ki shuruata se hipaitaitisa e ke tike ke samaveshana se mahatvapurna dhanga se prabhavita nahim hue haim|
hipaitaitisa bi eka vishanu hai jo eka sankramita vyakti ke rakta aura sharirika dravom ke madhyama se phailata hai|
hipaitaitisa bi echaaivi se 100 guna adhika sankramaka hai, yaham taka ki bahuta se logom ko yaha ahasasa nahim hota hai ki ve isa vishanu se sankramita ho chuke haim|
yaha isalie kyonki lakshana sambhavatah turanta, ya thoda़a bhi vikasita na hom|
udbhavana kala (isa vishanu ke samparka mem ane se isa sankramana ke vikasita hone taka ka samaya) eka se chhah mahine ke bicha hota hai|
vishanu ka pata lagane ke lie eka rakta parikshana kiya jata hai|
yaha vishanu sharirika dravom jaise ki rakta, lara, virya aura yoni drava mem upasthita hota hai|
yaha eka vyakti se dusare vyakti ko diya ja sakata hai, jaise ki asurakshita yauna sambandha ke dvara(bina eka kandoma ke upayoga kie) ya aushadhi bhitara pahunchane ke lie sui adana-pradana karane se|
sankramita mataem bhi isa vishanu ko apane bachchom mem prasuti ke daurana prasarita kara sakati haim (prayah usa mahila ki janakari ke bina ki vaha sankramita hai)|
yaha hipaitaitisa bi vishanu eka jirna (dirghavadhi) roga utpanna karana jari rakha sakata hai, jo eka tivra sankramana ka anusarana karata hai|
yaha shishuom aura nanhem bachchom (kishorom) mem bahuta samanya hai, kintu yaha vayaskom mem bhi ho sakata hai|
yadi apa mem jirna hipaitaitisa bi vikasita hota hai, to apa achchhe bane raha sakate haim, kintu apa isa vishanu ko de sakate haim kyonki apa eka vahaka haim|
lakshana a aura ja sakate haim, ya apa gambhira yakrita kshati vikasita kara sakate haim|
hipaitaitisa bi ki upasthiti upa-sahariya aphrika, dakshina purva eshiya, aura prashanta dvipom, jaise ki havai dvipom, salomana dvipom, aura phiji mem sabase adhika hai|
hipaitaitisa bi ka sabase kama apatana astreliya, nyuja़ilainda, uttari aura pashchimi yuropa, tatha uttari amarika mem paya gaya hai|
sansara bhara mem isa vishanu ke lagabhaga 350 miliyana (3,500 lakha) vahaka haim|
hipaitaitisa bi se bachane ke lie eka tika upalabdha hai| khasara eka atyadhika sankramaka vishanuka roga hai|
yaha roga jvara, khansi, aura vishishta lala-bhure dagom sahita lakshanom ki eka shrinkhala utpanna karati hai|
yaha sankramana vayu ke madhyama se lara ki chhoti bundom se phailata hai|
apako eka sankramita vyakti se sidhe samparka dvara ya vayu ke madhyama se jaba ve khansate ya chhinkate haim taba, khasara ho sakata hai|
ye chhoti bundem sataha para kuchha ghantom ke lie jivita, aura sankramaka bani raha sakati haim|
khasara 1-4 varsha ki unra ke bicha ke bachchom mem adhika samanya hai, kintu kisi ko bhi jisaka isa roga se pratirakshana nahim hua hai, ho sakata hai|
khasara ke lakshana sankramana shuru hone ke 9 se 11 dinom ke bada prakata hote haim, aura 14 dinom taka rahate haim|
yaha sthiti prathama lakshanom ke prakata hone ke bada, aura dadore vikasita hone se purva sabase adhika sankramaka hoti hai|
khasara ke lie samanyatah upachara ki avashyakata nahim hoti hai kyonki sharira ka pratirakshi tantra (vishanuom se suraksha) prayah do haphtom mem sankramana ko khadeda़ sakata hai|
vishishta rupa se, eka bara jaba kisi ne khasara ke sankramana ko khadeda़ diya hai, to ve isake prati pratiraksha vikasita kara lete haim|
khasare ki jatilataom mem nimoniya, kana aura ankha ke sankramana, aura krupa (phephada़om aura kantha ka eka sankramana) shamila haim|
adhika gambhira jatilataem, jaise ki mastishka ka sujana aura jalana (mastishkakopa), adhika asamanya haim kintu ghataka ho sakate haim|
khasare se prati varsha sansara bhara mem eka miliyana (dasa lakha) nrityuem hoti haim|
khasare ko rokane ka sabase prabhavi tarika hai khasara-galasua- rubela (emaemaara) ka tika, jo balyavastha ke do anya sashakta rupa se ghataka rogom – galasua aura rubela se bhi suraksha pradana karata hai|
emaemaara tike ki saphalata ka artha hai ki, yuke mem, khasare ke mamale asamanya haim|
phira bhi, hala ke varshom mem, khasare ke mamale ki sankhya mem vriddhi hui hai|
udaharana ke lie, 2001 ke daurana 70 mamalom ki tulana mem 2006 mem 739 mamale the|
yaha socha hai ki khasare ke mamalom ki sankhya mem yaha vriddhi mata-pita ka apane bachchom ko isa emaemaara tike se tika na lagavane ka parinama hai|
yaha shayada emaemaara ka svalinata (eka roga jo ki mastishka ke vikasa para prabhava dalata hai, aura bolachala tatha samajika antahkriya mem samasyaom ki ora agrasara kara sakata hai) se sambandha ke anumana ke karana hai|
1991 mem, prachara ne emaemaara injekshana aura svalinata ke bicha eka sambandha ka dava karate hue eka riporta ko vishishta rupa se darshaya tha|
phira bhi, isa dave ki jancha karane ke lie prarambha kie gae bahuta se adhyayanom ne emaemaara tike aura svalinata ke bicha koi sambandha nahim paya|
rubela vishanu se utpanna tivra sankramana, khasara, do haja़ara se adhika varshom se abhijnata eka roga, atyadhika sankramaka hai|
yaha vishva ke hara desha mem dekha gaya hai|
tikom ke upayoga se purva, khasare ki mahamari hara do se pancha sala mem hota tha|
khansi, sardi-ja़ukama, jvara aura eka tvacha dadora jo ki prarambhika lakshanom ke isa roga ki visheshata batane ke kai dinom purva prarambha ho jata hai|
khasare se svasthya labha samanya hai, kintu shvasana aura kendriya tantrika tantra ki gambhira jatilataem ho sakati haim|
chhoti mata eka sarvavyapi aura atyadhika sankramaka vishanuja roga hai|
vyapaka tvacha ke dadorom se visheshita, yaha prarambhika roga, vairisela jostara vishanu ke dvara utpanna hota hai|
sankramana ki punaravritti anyatha bhainsiya dada ya harpisa jostara ke rupa mem jane jane vale eka simita(sthaniya) tvacha ke dadore mem parinita hoti hai|
purva kala mem bada़i mata se prayah milati chhoti mata, vishva bhara mem mausama ke anusara aura mahamari mem hoti hai|
yaha vishanu shvasana marga se phailata hai aura bachchom mem eka samanya sankramana hai|
maleriya vikasashila deshom mem manava jati ke pramukha abhishapom mem se eka hai|
akalana kiya gaya hai ki yaha prati varsha eka se tina miliyana (dasa se tisa lakha) nrityu ka karana banate hue 500 miliyana (5000 lakha) se adhika logom ko prabhavita karata hai|
halanki, ka़aphi hada taka, uttari amarika aura yuropa ke adhika bhagom se isaka unmulana ho gaya hai, phira bhi yaha ushnakatibandha ke adhikatara kshetrom mem vyapta hai|
ushnakatibandha ke yatriyom ko vishesha rupa se maleriya ke khatare se svayam ki suraksha karana avashyaka hai|
taipha़aida jvara salmonela taiphi aura pairataiphi dvara utpanna, udariya lakshanom ke satha jvara ke rupa prastuta karata hua, eka tivra, sarvangi sankramana hai|
19vim shatabdi se purva, taipha़sa aura taipha़aida jvara ko eka jaisa samaja jata tha|
taipha़aida ke lie antriya jvara eka vaikalpika nama hai|
salmonela taiphi aura pairataiphi kevala manavom mem basate haim|
sankramita vyakti ke manava mala se dushita bhojana aura pani ke antargrahana dvara yaha jiva arjita kiya jata hai| 
yaha eka varsha se adhika unra ke bachchom mem dekha gaya hai|
taipha़aida ke prakopa vikasashila deshom mem uchcha nrityu-sankhya mem parinita hote haim|
hala hi mem, pratijaivika pratirodhi jivom ka vikasa adhika chinta utpanna kara raha hai|
poliyo, poliomailitisa ke bhi rupa mem jana jane vala, eka atyadhika sankramaka vishanu dvara utpanna eka roga hai|
adhikatara logom ke lie, poliyo eka halka roga hai aura kevala pha़lu ki taraha ke lakshana utpanna karata hai|
phira bhi, dusarom ke lie, poliyo eka sashakta rupa se ghataka roga hai, jo prachanda mansapeshi pakshaghata utpanna kara sakata hai (angaghati poliyo)|
yaha isalie, kyonki poliyo ke eka prachanda mamale mem yaha tantrika koshikaom para jo mansapeshiyom ke karya mem sahayata karati haim, akramana karata hai|
atita mem, poliyo eka bahuta samanya roga tha|
yaha vishva bhara mem, vishesha rupa se bachchom (kishorom) mem, nrityu aura pakshaghata ka eka nitya karana tha|
phira bhi, aja, vyapaka tike ke parinamasvarupa, poliyom ke mamalom ki sankhya mem prabhavashali tarike se kami ai hai|
1993 se, yuke mem poliyo ka eka bhi mamala nahim hai, aura aba kevala chara desha haim jaham yaha roga eka gambhira samasya bani hui hai – naijiriya, bharata, apha़ganistana, aura pakistana|
isalie isa roga se nipatane ka sabase achchha tarika hai isaki rokathama karana|
atah yaha bahuta mahattvapurna hai ki apa yaha sunishchita karem ki apaka bachcha apane sabhi avashyaka pratirakshanom ko prapta karata hai kyonki ve badha़ rahe haim|
poliyo ke bada ka lakshana eka roga hai jo una logom ke lagabhaga 20-66% ko prabhavita karata hai jinhem poliyom hua tha|
apa poliyo ke bada ke lakshana ko apake prarambhika roga ke 10-14 varshom ke bicha vikasita kara sakate haim|
yaha lakshana eka nai mansapeshi pida़a aura kamaja़ori utpanna kara sakata hai, aura apako parishranta aura susta mahasusa karate hue bhi chhoda़ sakata hai|
yaha apake lie asa-pasa jana, ya nitya niyata kama aura gatividhiyom ko karana kathina bana sakata hai|
yaha jnata nahim hai ki kya poliyo ke bada ke lakshana ko utpanna karata hai|
kuchha siddhanta yaha sanketa karate haim ki poliyo vishanu kabhi-kabhi apake prarambhika sankramana ke pashchat apake sharira mem supta raha sakata hai kyonki apake pratirakshi tantra ne ise puri taraha se nashta nahim kiya hai|
yaha supta vishanu taba ‘punah-sakriya’ ho sakata hai, halanki yaha jnata nahim hai ki kya isa punah-sakriyana ko prarambha karata hai|
anya siddhanta yaha sanketa karate haim ki yaha lakshana kshatigrasta tantrika koshikaom ka parinama hai jo dhire-dhire adhikala mem vikrita hota hai|
mukhya rupa se 1940 vem aura 1950 vem dashaka ke daurana poliyo mamalom ki uchcha sankhya ke karana, hala ke varshom mem, yuke mem poliyo ke bada ka lakshana eka adhika samanya roga ho gaya hai|
phira bhi, aba jabaki poliyom yuke mem prakritika rupa se aura sakriya nahim hai, poliyo ke bada ka lakshana bhavishya mem aura adhika asamanya hona chahie|
dhanustambha (titenasa) jivanu (kitanu) se utpanna hota hai jo ki apake sharira mem eka ghava, jaise ki bagavani karate samaya lage eka chire, ya eka jantu kata ke madhyama se pahunchata hai|
apa dhanustambha (titenasa) vikasita karane ke adhika khatare mem haim yadi ghava gahara hai ya yadi yaha mitti athava khada se maila ho jata hai, kintu yaham taka ki chhote ghava jaise ki kante se eka chubhana dhanustambha (titenasa) utpanna karane ke lie apake sharira mem paryapta jivanu ko jane ki anumati de sakata hai|
eka bara apake sharira ke andara jane ke bada, yaha jivanu jivavisha (visha) sravita karata hai jo taba dhanustambha (titenasa) utpanna karata hai, halanki yaha roga vikasita hone mem 21 dina taka le sakata hai|
dhanustambha (titenasa) apaki nasom ko prabhavita karata hai, aura ghataka ho sakata hai|
dhanustambha (titenasa) yuke mem atyanta asamanya hai kyonki loga isase tikakrita hote haim|
dhanustambha (titenasa) pratirakshana ke eka pure korsa mem tike ke pancha khuraka hote haim|
yaha tika apake sharira ko dhanustambha (titenasa) jivavisha ke viruddha roga-pratikaraka utpanna karane ke lie sakshama banata hai, jo phira apako roga se yadi apa isa jivanu se sankramita hone vale the to, suraksha karata hai|
sardi-ja़ukama eka sankramana hai jo ki upari shvasana marga – naka, gala, shiranala (khopada़i mem ankhom aura naka ke pichhe ka chhota sthana), shvasa-pranala (mukhya vayu marga jo phephada़om taka hoti hai), kanthanali (svara-yantra) aura shvasa naliyom (phephada़o mem vayumarga) ko prabhavita karata hai|
sardi-ja़ukama vishanuom dvara hote haim aura kai tarikom se phaila sakate haim|
yadi apako sardi-ja़ukama hai aura apa chhinkate, khansate ya bolate haim, to sardi-ja़ukama ke vishanu yukta dushita drava ki chhoti-chhoti bunde vayu mem prakshepita ho jate haim|
yadi ye kisi aura ke dvara shvasa mem lie jate haim taba ve sankramita ho sakate haim|
sardi-ja़ukama pratyaksha aura apratyaksha samparka se bhi phaila sakate haim|
yadi apako sardi-ja़ukama hai aura apa apane naka ya ankhom ko chhute haim aura phira kisi aura ko chhute haim, to apa unamem vishanu pravesha kara sakate haim|
vikalpa rupa se, yadi apa eka vastu ko jaise ki daravaja़e ka hattha ya teliphona ko chhute haim, to yaha vishanu usa vastu mem sthanantarita ho sakata hai|
yadi koi usa vastu ko thoda़i dera ke bada chhuta hai aura phira apane munha, naka ya ankhom ko chhuta hai, to ve sankramita ho sakate haim|
sardi-ja़ukama ke lakshana prayah apake sankramita hone ke 2-3 dinom bada prarambha hote haim, aura 2-14 dinom taka rahate haim| 
adhikatara loga sardi-ja़ukama se eka hapha़te ke andara thika ho jate haim|
apa sabase adhika sankramaka hote haim jaba apaki chhinkati, bahati naka aura khansi ho jo ki vikasashila sardi-ja़ukama ke prathama sanketa hote haim|
ve mukhyatah sardiyom ke mahinom mem hote haim, halanki sardi ka mausama svatah sardi-ja़ukama ka eka karana nahim hai| 
vayaskom ki apeksha bachche sardi-ja़ukama hone ki ora adhika pravritta hote haim|
yaha isalie kyonki vayaskom ne kuchha vishanuom jo sardi-ja़ukama utpanna karate haim ke prati pratiraksha badha़a li hai| 
ausatana, vayaskom ko eka sala mem do aura chara ke bicha sardi-ja़ukama hote haim, jabaki bachchom ko adhika se adhika atha ya dasa ho sakate haim|
eka samanya sardi-ja़ukama apake upari shvasana marga – apaki naka aura gale ka eka sankramana hai|
yaha prayah hanirahita hota hai, halanki yaha isa taraha nahim mahasusa ho sakata hai|
yadi yaha eka bahati naka, pradahita kantha aura khansi nahim hai, to yaha jaliya ankhe, chhinka aura sankulata hai – ya upara ke sabhi ho sakate haim|
vastava mem, kyonki 200 se adhika vishanuom mem se koi eka samanya sardi-ja़ukama utpanna kara sakata hai, lakshana bahuta hi bhinna hone ki ora jukava rakhate haim|
durbhagya se, yadi apa adhikatara vayaskom ki taraha haim, to apako sala mem do se chara bara samanya sardi-ja़ukama hone ki sambhavana hai|
bachche, vishesha rupa se shishu vidyalaya ke chhatrom ko, adhika se adhika salana chhah se 10 bara eka samanya sardi-ja़ukama ho sakata hai|
achchhi kha़bara yaha hai ki apako aura apake bachche ko lagabhaga eka ya do hapha़tom mem behatara mahasusa karana chahie|
yadi eka samanya sardi-ja़ukama ke lakshanom mem utane samaya mem sudhara nahim ata hai, to apa yaha sunishchita karane ke lie apane chikitsaka se milem ki apake phephada़om, shiranalom ya kanom mem eka jivanuja sankramana ki jatilata nahim hai|
upari shvasana marga ke vishanuja sankramana ke rupa mem bhi jana jane vala, samanya sardi-ja़ukama, eka sankramaka roga hai jo vishanuom ki kai vibhinna prakarom se ho sakata hai|
vishanuom ki atyadhika sankhya ke karana jinase sardi-ja़ukama ho sakate haim aura kyonki nae sardi-ja़ukama ke vishanu vikasita hote rahate haim, sharira kabhi bhi una sabhi ke prati pratiraksha nahim badha़a pata hai|
isa karana se, sardi-ja़ukama eka niyamita aura bara-bara ane vali samasya hai|
vastava mem, ausatana, shishu vidyalaya ke bachchom ko eka sala mem nau sardi-ja़ukama, jo balavada़i mem haim, unhem eka sala mem 12 sardi-ja़ukama, aura kishorom tatha vayaskom ko, prati varsha sata sardi-ja़ukama hote haim|
tapedika (tibi) eka sankramana hai jo tyubarakala besilasa ya maikobaiktiriyama tyubarakulosisa namaka eka kitanu se utpanna hota hai|
lagabhaga 50 varsha purva jaba prabhavi prati-tyubarakulosisa aushadhiyom ka prachara kiya gaya tha, taba tibi nrityu ka pramukha karana tha|
kai deshom mem tibi abhi bhi eka pramukha samasya hai|
hala ke varshom mem vikasita deshom mem inaki sankhya mem vriddhi hui hai, sambhavatah jaham yaha samanya hai una kshetrom se havai yatra aura logom ke avagamana mem vriddhi ke karana aisa hai|
yaha phephada़om aura sharira ke dusare bhagom jaise ki lasika parva (tyubarakulosisa edinaitisa ya kanthamala (skrophula)), tvacha aura haddiyom ko prabhavita kara sakata hai|
tyubarakala besili sakriya rogom ko utpanna karane se pahale varshom taka prasupta raha sakata hai|
adhikatara mamalom mem phephada़e ka sankramana pratirakshi tantra dvara achchhi taraha se niyantrita ho jata hai, aura koi lakshana nahim dikhata hai|
yadi pratirakshi tantra kama prabhavi hote haim to sakriya phephada़a roga ho jata hai|
eka vyakti ko bina usaki janakari ke tyubarakulosisa se sankramana ho sakata hai|
eka tyubarakulina tvacha parikshana, hipha (mainataॉksa) parikshana se isaka pata lagaya ja sakata hai|
jaba sakaratmaka hai, to yaha sanketa karata hai ki usa vyakti ke pasa svabhavika pratiraksha ka eka stara hai|
ve vyakti jisake nakaratmaka parikshana haim unake pasa yaha pratiraksha nahim hoti hai aura ve tibi se sankramana ke prati adhika samvedanashila hote haim|
tyubarakulina-nakaratmaka vyakti ko bisiji tike se labha ho sakata hai|
yaha tibi kitanu ke eka sanshodhita sanskarana se bane eka tike ka prayoga karata hai|
yaha tikakritom mem se lagabhaga 70% mem tibi vikasita hone ke khatare ko lagabhaga 15 varshom ke lie kama karata hai |
sitambara 2005 se, bisiji tike ki eka nai lakshyapurna yojana prachalita ki gai|
yuke mem sabhi skuli bachchom ke bajaya kevala una logom ko tika lagaya jaega jinhem tibi hone ka adhika khatara hai|
chhoti mata vairisela-jostara (vairisela chikanapaॉksa ka chikitsiya nama hai) namaka eka vishanu se hota hai|
chhoti mata eka halka, kintu atyadhika sankramaka roga hai jo adhikatara bachchom ko kabhi na kabhi hota hai|
marcha aura mai ke bicha isa roga ka hona sabase adhika samanya hai| apake vishanu se samparka mem ane ke bada yaha lakshanom ko prakata karane ke lie 10-21 dina leta hai|
chhoti mata 2-8 varsha ki unra ke bachchom mem sabase adhika samanya hai, halanki apako kisi bhi unra mem chhoti mata ho sakata hai|
apa dadore prakata hone se lagabhaga do dina purva se lagabhaga pancha dina bada taka sankramaka hote haim|
atah, apako, ya apake bachche ko, ghara para rahana chahie jaba taka ki sabhi chhalom ki papada़i puri taraha se samapta nahim ho jati, aura yaha prayah pahale chhale prakata hone ke 5-7 dinom ke bada hota hai|
antima chhale ke phutane aura papada़i samapta hone ke bada, apa aba aura sankramaka nahim hote haim|
chhoti mata eka sankramita vyakti ke chhinkane aura khansane se, lara aura nasikiya shleshma ki chhoti bundom mem phailati hai|
yaha vishanu ina bundom mem pahale se hi hota hai, jisake karana yaha itani jaldi phailata hai|
apako eka bara chhoti mata ho chuki hai, to apako dusari bara chhoti mata kabhi kabhara hi hogi|
yaha isalie kyonki apaka sharira isa chhoti mata ke vishanu ke prati pratiraksha vikasita kara leta hai, jo apako punah sankramita hone se rokata hai 
chhoti mata ke sankramana ke pashchat, yaha vishanu sharira ke tantrika utakom mem rahata hai (prasupta rahata hai)|
isase koi nukasana nahim hota hai kyonki ise pratiraksha pranali dvara niyantrana mem rakha jata hai jo ki sharira ka eka bhaga hai jo sankramana se lada़ai karata hai |
yaha vayarasa jivana mem bada mem kisi bhi samaya alaga prakara se jaise dada ke rupa mem sakriya ho sakata hai , samanyatah jaba apa vayaska ho | 
amataura para dada ka pahala lakshana hai prabhavita nasom mem darda hona hai |
sata dinom ke bada amataura para dane dikhai dete haim, usake bada phaphole jo ki apake sharira ke kevala eka paksha ko prabhavita karate haim |
jise chechaka nahim hui ho aura yadi apako dada hai to apa usake lie sankramaka ho sakate haim |
halanki, yaha jaruri nahim hai ki jise chechaka hai , use dada ka sankramana ho |
90 pratishata se adhika garbhavati mahilaem chechaka ke vayarasa ke sankramana se mukta haim (kyonki jivana mem pahale unhem yaha ho hi chuka hoga ) atah adhikatara mahilaom ko isa roga ke hone ki chinta nahim karani chahie |
yaham taka ki agara garbhavastha ke daurana apako dada ho jae to isase apake bachche para kisi bhi taraha ka prabhava nahim pada़na chahie |
chechaka lagabhaga 1000 garbhadharana mahilaom mem se tina mem hota hai, aura apa aura apake bachche donom ke lie gambhira jatilataem paida kara sakata hai|
taiphaida bukhara bhi antrajvara ke rupa mem jana jata hai, eka gambhira sankramana hai joki jivanu salmonela taiphi ki vajaha se hota hai|
roga eka manushya se dusare manushya ko lagata hai aura yaha taiphaida ke baiktiriya yukta khana khane se aura pani pine se phailata hai |
hara sala, inglainda aura velsa mem taiphaida ke lagabhaga 150 se 200 mamalom ki riporta hoti hai|
halanki,eka anumana ke anusara duniya bhara mem, 13 se 17 lakha mamalom mem se 600,000 logom ki mauta hui haim|
bharata, aphrika aura dakshina amerika, jaise vikasashila deshom mem taiphaida bukhara eka ama samasya hai kyonki ina deshom mem khadya svachchhata aura saphai ke manaka vikasita deshom ki taraha utane uchcha nahim haim |
ina deshom mem, taiphaida kabhi-kabhi mahamari ka karana banata hai |
isalie, yadi apa eka vikasashila desha ka daura kara rahe haim, to taiphaida tikakarana ke lie salaha di jati hai |
salmonela taiphi salmonela samuha ka hai joki taiphaida bukhara ka karana to hai hi , satha hi halki bimari jaise phuda paॉyaja़ninga ka bhi karana hota hai |
pairataiphaida salmonela ke karana hota hai , entaritaidisa pairataiphi e , bi ya si taiphaida ke hi taraha ke sankramana haim para ye kama gambhira hote haim |
besilasa enthraisisa baiktiriya se hone vala eka gambhira sankramaka roga enthreksa hai |
enthreksa pashuom mem sabase adhika hota hai, lekina yaha insanom ko bhi sankramita kara sakata hai |
enthreksa krishi kshetrom mem sabase ama hai jaham yaha janavarom mem hota hai |
halanki enthreksa vishva stara para paya ja sakata hai, yaha aksara deshom ke lie kama manakikrita aura prabhavi sarvajanika svasthya aura pashu svasthya karyakramom ke lie eka khatara hai |
vartamana mem adhika jokhima ke rupa mem suchibaddha kshetrom mem dakshina aura madhya amerika, dakshini aura purvi yuropa, eshiya, aphrika, kairebiyana, aura madhya purva haim |
yaha amataura para sankramita pashuom ya vyavasayikom ke khule utpadom ke karana manushyom ko sankramita karata hai |
shramika jo anya deshom se nrita janavarom aura pashu utpadom (audyogika enthreksa) ke samparka mem ae haim unamem enthreksa hona adhika ama hai, ve bi enthraisesa se sankramita ho sakate haim |
enthreksa yunaiteda stetsa mem janavarom mem bahuta kama hota hai jabaki vahim isaki tulana mem yaham manushyom mem bahuta kama hota hai |
dengu bukhara eka vayarasa dvara hone vali aisi bimari hai joki sankramita machchhara ke katane se hoti hai, viskaॉnsina mem isaka hona asamanya hai|
roga kairibiyana, meksiko, dakshina aura madhya amerika, prashanta, eshiya aura ushnakatibandhiya aphrika ke kuchha hissom ke adhikansha deshom mem kuchha dvipom sahita sabase adhika ushnakatibandhiya aura uposhnakatibandhiya kshetrom mem ama hai |
dakshini teksasa mem kabhi - kabhi hone vale mamalom ka arambha sanyukta rajya amerika mem hi hua tha | 
kabhi kabhi, kisi desha ke nivasiyom ya vaham ae agantukom se jaham dengu bukhara hota hai unase usa desha mem bhi dengu bukhara phaila jata hai |
manava inyuno vayarasa (echaaivi) eka yauna sancharita (esatiai) vayarasa hai joki sharira ki pratiraksha pranali para akramana karata hai, jo ki roga aura sankramana ke khilapha eka prakritika raksha pranali pradana karata hai |
echaaivi vishesha koshikaom, jinhem sidi 4 koshika kaha jata hai, jo apake rakta mem paya jata hai ,ko sankramita karata hai, aura sankramana se lada़te haim |
sankramita hone ke bada, sidi 4 koshikaem echaaivi dvara nashta ho jati haim |
unaki sankhya mem antatah giravata aura pratiraksha pranali kama karana banda kara deti hai ,halanki sharira sidi 4 koshikaom ka adhika utpadana karane ka prayasa karega | 
isake karana vyakti mem gambhira echaaivi ke sankramana ka khatara badha़ jata hai, ya kainsara jaise roga, hai chhoda़ deta hai|
echaaivi vayarasa eka vishesha prakara ka vayarasa jo ki retro vayarasa ke rupa mem jana jata hai |
retrovayarasa hamari koshikaom mem phailakara unake dienae ko toda़ deta hai aura phira se ise joda़kara isaki pratilipiyam banata hai |
retrovayarasa ilaja ko chunauti de rahe haim kyonki ye vayarasa ke teji se nae upabhedom ke rupa mem badala (badalana) sakate haim |
echaaivi ke bada ke charanom ka varnana karane ke lie ekvayarda inyuna dephisiensi sindroma (edsa) eka shabda hai , jaba pratiraksha pranali kama karana banda kara deti aura vyakti ke lie janaleva paristhiti utpanna ho jati hai jaise nimoniya (langasa ka sankramana) |
'edsa' shabda pahale daॉktarom dvara istemala kiya gaya tha jaba echaaivi vayarasa ki satika prakriti ko puri taraha samaja nahim gaya tha |
halanki, yaha shabda vyapaka rupa se prayoga nahim kiya jata hai kyonki kisi ko bhi alaga-alaga paristhitiyom mem echa ai vi sankramita hone ke lie bahuta hi samanya shabda hai|
visheshajna aba echaaivi sankramana ke lie 'agrima' ya 'antima sthiti' shabda ka upayoga karana jyada pasanda karate haim |
echaaivi eka vaishvika mahamari hai, jisaka matalaba hai ki yaha eka aisi bimari hai jo duniya bhara mem phaila gayi hai |
vartamana mem vishva svasthya sangathana (dablyuechao) ka anumana hai ki duniya bhara mem 33 lakha loga echaaivi se sankramita haim |
1981 se jaba se mahamari shuru hui hai taba se yaha bhi anumana hai ki 25 miliyana logom ki echaaivi se nrityu ho gai hai |
vayarasa vishesha rupa se upa sahara aphrika mem bada़e paimane para phaila hai--aphrika ke desha jo ki sahara registana ke dakshina mem haim -jaise dakshina aphrika, ja़imbave aura muja़ambika | 
britena mem 73,000 loga echaaivi ke satha ji rahe haim, aura visheshajnom ka manana hai ki 30% logom ko pata nahim hai ki unhem echaaivi hai | 
jina logom mem echaaivi ka pata chala hai unaki sankhya 21 vim sadi ki shuruata ke bada se badha़ gayi hai|
aisa mana jata hai ki logom ki yaha galata socha hai ki echaaivi aba sarvajanika svasthya ke lie khatara nahim hai |
echaaivi vayarasa rakta, virya aura yoni tarala padartha ke rupa mem sharirika tarala padartha ke vinimaya ke madhyama se phaila sakata hai |
echaaivi phailane ka sabase ama karana sambhoga ya maukhika aura guda seksa hai |
sui ke dubara istemala aura eka garbhavati mahila se usake ajanme bachche ko yaha vayarasa phaila sakata hai |
halanki, upachara ke kshetra mem pragati ke karana, aba yaha sambhava hai ki mam dvara bachche ko vayarasa parita hone se roka ja sake |
halanki, isaka matalaba yaha nahim hai ki halata kevala samalaingika samudaya ke lie chinta ka vishaya hai |
britena mem, yaha anumana hai ki dasa mamalom mem se vishamalaingika seksa ke daurana eka mamala echaaivi arjita kara raha hai |
tvacha ke lie impaitigo eka behada sankramaka jivanu sankramana hai |
70% se adhika gaira phapholedara impaitigo mamalom ka lekhankana hona bahuta samanya hai |
ama taura para do aura chara varsha ki unra ke bachchom mem ye lakshana bahuta samanya haim |
khasakara agara ve sena ke bairakom jaise simita parivesha mem rahate haim to ye paristhitiyam vayaskom ko bhi prabhavita kara sakati haim |
samanyataya impaitigo bina kisi ilaja ke do se tina haphtom mem thika ho jayega |
samanya rupa se bimari ke khatare ka kama se kama prabhava ho isake lie entibayotika krima sankramana ke ilaja ke lie upayoga mem lai jati haim |
impaitigo samanya rupa se gambhira nahim hai, anya logom mem khasakara navajata shishuom mem yaha na phaile isake lie savadhani baratana bahuta jaruri hai | 
isamem tvacha ke prabhavita kshetrom ko chhune ke bada savadhanipurvaka hathom ko dhona aura kisi anya ki tauliya aura bistara ki chadara ko na prayoga karana shamila haim | 
adhikansha loga entibayotika upachara ke 48 ghante ya eka bara unake ghavom ke sukha chukane ke bada ve sankramaka nahim raha jaenge |
unamem sankramana limpha nodsa (lasikaparvashotha) ya tvacha ki gahari parata (selyulaitsa ) taka mem phaila rahata hai |
aisa bahuta kama hota hai ki sankramana gurdom taka bhi phaila jae jabaki aisa 1% se bhi kama mamalom mem hota hai | 
rebija़ kendriya tantrika tantra ka eka sankramana hai jo ki manava sahita sabhi garma khuna vale jivom ko prabhavita kara sakata hai |
rebija bhi jalantaka ke rupa mem jana jata hai |
mastishka ki sujana (sujana) rebija vayarasa ke karana hoti hai aura eka sankramita pashu ke katane, ama taura para eka kutte ke katane se lara ke madhyama se khuna mem phailata hai |
eka bara kisi vyakti mem roga puri taraha pramanita hone para , unamem roga ke lakshana dikhane shuru ho jate haim to use hamesha nrityu ki ora le jate haim | 
vishva svasthya sangathana (dablyuechao) ka anumana hai ki rebija़ sankramana vale kisi janavara ke kata lene ke parinamasvarupa 40,000 aura 70,000 ke bicha hara sala loga rebija se mara jate haim |
shahari rebija़ kuttom aura billiyom ko prabhavita karata hai, silavena rebija़ (jangali) chamagadada़ aura lomada़iyom aura anya jangali janavarom ko prabhavita karata hai |
rebija 150 se adhika deshom mem paya jata hai lekina britena sahita 30 deshom ko mukhya rupa se , aba adhikarika taura para sakhti se pashuom ko dusari jagaha le jane para niyantrana ke madhyama se rebija़ mukta rupa mem vargikrita kiya gaya hai |
chechaka, chhoti chechaka ke rupa mem bhi jana jata hai, eka gambhira aura behada sankramaka roga hai |
vayarala sankramana eka vyakti se dusare ko ,sidhe samparka ke madhyama se ,chechaka ke chhalom ke phuta jane se aura havai bundom ke madhyama se phailata hai |
jina logom ko kabhi chechaka nahim hua hai unhem kisi aise ke satha rahane mem jisako yaha bimari hai sankramana ho sakata hai |
jo pahale sankramita nahim hue haim una logom mem vayarasa ke lie hamesha hi samvedanashilata hai |
isa vayarala bimari ke lakshana haim sukhi khujali(pruritus) hona tvacha para lala chakattom ke satha tarala padartha se bhare phaphole ho jate haim jinake phutane para tvacha para papada़i pada़ jati hai|
halka bukhara aura kamajori ke bada chechaka ke danom ki shuruata hoti hai |
sankramana ki avadhi se tina dinom taka rahati hai| isase pahale lala chakatte dikhate haim usake bada sabhi phaphole papada़i bana jate haim |
kisi ke chechaka ke sankramita hone mem jaba taka roga bahara ata hai aura lakshana dikhai dete haim ,isake bicha badha़ne ki avadhi 10 se 21 dina taka hoti hai|
adhikansha bachche vayarasa dvara 10 varsha ki unra se sankramita ho jate haim|
sankramana ke bada amataura para sankramana dobara na phaile isake lie pratirodhaka shakti upasthita rahati hai, halanki jisako pahale chechaka ho chuki hai use bada mem haraphija़ ho sakati hai |
bharata mem, yaha amataura para varsha ke pahale chhaha mahinom mem hota hai |
poliyo (poliyomailitisa bhi kaha jata hai) eka sankramaka, aitihasika vinashakari roga hai jisaka 20 vim sadi ke antima chhamahi mem pashchimi golarddha se lagabhaga saphaya kara diya gaya tha |
halanki prachina kala se manushya poliyo se grasta hai, isaka sabase vyapaka prakopa 1900 ki pahali chhamahi mem hua , tatpashchata tikakarana jonasa saॉlka ke dvara shuru kiya gaya , jo ki 1955 mem vyapaka rupa se upalabdha ho gaya |
1952 mem poliyo mahamari charama sima para tha, akele sanyukta rajya amerika mem lagabhaga 3,000 se adhika mautom ke satha 60,000 mamalom ki suchana mili |
halanki, bada़e paimane para tikakarana se , ugra-prakara ka poliyo, ya poliyo jo ki prakritika sankramana se hota hai use 1979 taka sanyukta rajya amerika se hata diya gaya aura pashchimi golarddha se 1991 taka hata diya gaya |
dipthiriya kaॉrinabaiktiriya dipthiriya namaka baiktiriya se hone vala eka gambhira aura behada sankramaka roga hai |
yaha roga upari shvasa sambandhi marga(naka, gale, avaja baॉksa aura upari sansa ki nali)aura kabhi kabhi, tvacha ko bhi prabhavita karata hai |
yaha kisi ko bhi prabhavita karati hai, kisi bhi unra mem kara sakati hai aura ghataka ho sakati hai |
1940 se bachapana se hi niyamita rupa se tikakarana ke karana britena mem dipthiriya aba atyanta durlabha hai, lekina dipthiriya baiktiriya abhi bhi duniya mem maujuda haim aura britena mem lagabhaga sabhi nae mamale videsha yatra ke daurana hote haim |
vishesha rupa se garibi aura paryapta svasthya dekhabhala ke abhava mem yaha bimari duniya ke kuchha deshom mem rahane vale logom mem abhi bhi sakriya hai |
vishva svasthya sangathana (dablyuechao) ki riporta hai ki hala hi mem purvi yuropa ke kai deshom mem dipthiriya ki bada़i mahamari phaili hai |
mahamari eka vishesha jagaha ya kshetra mem achanaka vyapaka rupa se phailane vala eka sankramaka roga hai |
pichhale kuchha varshom mem dakshina amerika, dakshina purva eshiya, bharata aura rusa ke kuchha hissom mem bhi mahamari hui hai |
britena mem do, tina aura chara mahine ke bachchom ko niyamita rupa se dipthiriya ke khilapha (tika) tripala tikakarana ko hissom diya jata hai |
bustara khuraka skula shuru karane se pahale di jati hai aura phira 16 varsha ki unra aura 18 sala ke bicha di jati hai |
pashchimi duniya mem pratirakshana karyakrama ki saphalata ki vajaha se, mata-pitaom ki pidha़iyam bina isa roga ke bare mem kuchha jane bada़e ho gae haim |
shika parikshana mem dipthiriya taॉksina protina ki eka chhoti matra banha mem kalai ki tvacha mem antahkshipta ki jati hai jisase apa dekha sakate haim ki kya apa abhi bhi dipthiriya ke lie pratirakshita haim |
kali khansi jo kukura khansi ke rupa mem bhi jani jati hai, eka bahuta hi sankramaka roga hai jo shishuom mem nrityu ka karana bana sakati hai, vishesha rupa se chhaha mahine ki unra se kama unra ke bachche.
bada़e bachchom aura vayaskom ke lie, kali khansi amataura para gambhira roga nahim hai.
bachchom ko kali khansi se bachane ke lie dusare, tisare aura chauthe mahine mem tike lagae jate haim, aura dubara unake skula shuru karane se pahale bhi.
vaiksina shuru hone se pahale inglainda aura velsa mem prati varsha aksara 100,000 roga ke mamale dekhe jate the, lekina 1950 ke dashaka mem य़ha vaiksina shuru kiye jane ke bada, isaki dara mem eka sala mem 2000 mamalom mem prabhavashali tarike se giravata dekhi gayi. shabda chikanaguniya vyutpattishastra sambandhi tanjaniya ki Makonde bhasha hai, jaham isaka matalaba hai mem nihita hai ki jukata hai jo '.
halanki, logom ko abhi bhi kali khansi ati hai,aura bachche abhi bhi isa roga se marate haim, atah yaha bahuta mahatvapurna hai ki bachchom ko pratirakshita rakha jae.
yadi loga apane bachchom ko pratirakshita nahim karate haim,to yaha roga eka mahamari ka rupa le sakata hai.
chikanaguniya vartamana mem duniya ka sabase bhayavaha roga bana gaya hai.
dengu aura sarsa ki mahamari ke bada, aba chikanaguniya ne duniya ke logom ka dhyana akarshita kiya hai.
duniya ke kuchha bhagom mem, chikanaguniya bhi chikana gini ke rupa mem jana jata hai, halanki yaha eka mithya hai, chikanaguniya ka chikana ke satha koi sambandha nahim hai.
chikana gini shabda sirpha chikanaguniya ka eka angreja़ikarana hai.
shabda chikanaguniya vyutpattishastra ki drishti se tanjaniya ki Makonde bhasha hai, jaham isaka matalaba hai jo jukata hai.
isa roga ka nama dardanaka lakshanom ke karana pada़a jisamem rogi ke joda़om ki haddiyam upara ki ora muda़ jati haim
chikanaguniya ki utpatti madhya aphrika mem 1952 se huyi, jaham se yaha vartamana mem eshiya mem bada़e paimane para phaila gayi. 
yaha roga hi eka mahamari ke rupa mem prakata hota hai, aura aniyantrita hone para, yaha eka vishvamari mem badala sakata hai.
chikanaguniya ki adhunikatama mahamari vartamana mem dakshina bharata mem phaila rahi hai.
dakshina bharata mem manasuna mem bhari badha़ ayi jo isa roga ke vyapaka prakopa ke lie jimmedara hai.
dakshini bharata ke rajya andhra pradesha, kerala aura tamilanadu isase sabase aghika prabhavita haim.
bharata mem donom elopaithika aura ayurvedika kshetrom mem chikanaguniya sambanghi anusandhana para bahuta jora diya ja raha hai 
adhikansha logom ko ayurveda mem chikanaguniya ke lie samadhana prapta hua hai.
sanskrita mem sandhi jvara ke rupa mem jana jane vale roga ke chikanaguniya jaise (samana) lakshana haim.
sandhi jvara ka matalaba hai 'joda़om mem darda' aura yaha chikanaguniya ke prathamika pahaluom mem se eka hai.
isa karana se, chikanaguniya ko sandhi jvara ke barabara mana ja sakata hai.
chikunaguniya eka vayarala roga hai jisake prachalita lakshana aura etiyolaॉji dengu bukhara jaise hi haim.
chikanaguniya ke lie jimmedara vayarasa alphavirus, jo togoviridae parivara ke antargata ata hai.
yaha vayarasa machchhara katane ke madhyama se sankramita hota hai.
kevala eka vishishta prakara ka machchhara vektara(sadisha) hai. jo chikanaguniya roga ke lie vahaka ka karya karata hai
yaha edija aegyptimachchhara hai, jo dina ke samaya hi sakriya hota hai.
jo insanom ke bicha dengu bukhara ke sancharana ke lie jimmedara edija aegyptimachchhara yaba vahi machchhara hai.
chikanaguniya ke halata ama taura para ghataka nahim hote isase pancha se sata dinom ke bhitara uchita upachara ke satha roga ka nivarana kiya ja sakata hai.
roga ki buniyadi visheshata bukhara ke satha joda़om mem darda gathiya ki taraha ka hona (jise sanskrita nama diya gaya hai.)
inphlyuenja ka karana eka vayarasa hai jo mukhya rupa se upari shvasa nali, naka, gale aura brankai para hamalom ke karana hota hai,aura kabhi kabhara phephada़om para bhi.
sankramana amataura para lagabhaga eka saptaha ke lie rahata hai.
isaki visheshata uchcha bukhara, mansapeshiyom mem pida़a, sira darda aura jukama, gambhira rupa se thakana, sukhi khansi, gale mem kha़rasha, aura rhinitis ke achanaka shuruata se hoti hai
adhikansha loga bina kisi chikitsiya upachara ki avashyakata ke eka se do saptaha ke bhitara thika ho jate haim.
chhote bachchom, bujurgom aura chikitsiya halatom jaise phephada़om ke roga, madhumeha, kainsara, gurde, ya dila ki samasyaom se pida़ita logom ke lie inphluenja eka gambhira khatare ka rupa le leta hai.
ina logom mem sankramana antarnihita rogom jaise, nimoniya aura nrityu ke gambhira jatilataom ko age badha sakati hai.
inphluenja teji se mausami mahamari ke rupa mem duniya bhara mem phailata hai aura aspatala aura anya svasthya sevaom ka kharcha aura utpadata mem kami ke karana logom para kaphi arthika boja pada़ta hai.
varshika phlu mahamari se janasankhya ke 5-15% loga upari shvasa nali ke sankramana se prabhavita haim.
aspatala mem bharti hone vali mautom aura mukhya rupa se uchcha jokhima vale samuhom (bujurga, lambe samaya se bimara) mem hote haim.
halanki akalana karana mushkila hai, ina varshika mahamari se tina aura pancha lakha mamale gambhira bimari ke evam 250 000aura 500 000 duniya bhara mem hara sala hone vali mautom ke haim.
adhikanshatah vartamana mem audyogika deshom mem inphluenja se sambanghita logom ki nrityu 65 varsha se adhika unra ke logom mem hoti haim.
vikasashila deshom mem inphluenja ke prabhava ke bare mem bahuta kama jnata hai.
halanki, ushnakatibandha pradesha mem inphluenja ka prakopa jaham vayarala sancharana samanya rupa se pure varsha rahata hai,roga ka prakopa tivra aura durghatanajanya nrityu ki ora hota hai.
udaharana ke lie, 2002 mem medagaskara mem eka inphluenja prakopa ke daurana, tina mahine ke bhitara 27000 se aghika mamalom ki suchana mili aura turanta upachara ke bavajuda 800 logom ki nrityu ho gayi.
isa prakopa ki eka jancha, vishva svasthya sangathana (dablyuechao) dvara samanvita ki gayi,aura paya ki vaham paryapta svasthya natijom ke lie simita pahuncha ke satha kuposhita abadi aura aparyapta svasthya sevaem jimmedara haim(inphluenja, medagaskara, julai - agasta 2002 ka prakopa, dekhana saptahika mem gambhira svasthya parinama the mahamari vijnana ke rikarda).
katibandhom mem inphluenja ke sahi evam samayika aura gambhira mahamari ke varshika boja ke ankada़om ka bahirveshana karana sambhava nahim hai. 
enthreksa mukhyatah shakahari stanadhariyom ki eka bimari hai, halanki aisa jnata hua hai ki anya stanadhariyom aura kuchha pakshi bhi isa bimari se grasita ho jate haim 
manushya ama taura para sankramita pashuom ya sandushita pashu utpadom ke peshevara pradarshana se pratyaksha ya paroksha rupa se roga adhigrahana karata hai.
isalie ina ghatanaom mem kami ke lie pashuom mem niyantrana mahatvapurna hai.
eka vyakti se dusare vyakti mem sancharana ke lie koi likhita pramana patra maujuda nahim hai.
roga ke prabhava janavara aura manava svasthya para vinashakari ho sakate haim
dablyuechao ne manushyom aura pashuom mem enthreksa ki nigarani aura niyantrana ke lie disha - nirdesha prastuta kiye haim.
enthreksa ka preranarthaka ejenta jivanu hai,besilasa enthrasisa, jisake bijanu jo salom ya dashakom taka vatavarana mem jivita raha sakate haim, agale mejabana mem udgrahana ka intajara karate haim.
yaha roga upa Sahelian aphrika aura eshiya ke adhikansha deshom mem pashuom aura manushyom mem abhi bhi maujuda hai,evam kai dakshini yuropiya deshom mem, amerika mem, aura ऑstreliya ke kuchha kshetrom mem.
pashuom mem bimari phailane ke chhitaputa udaharana anya deshom mem bhi milate haim.
manava mem enthreksa tina prakara ke hote haim:, tnachiya enthreksa,taba uparjita hota hai jaba eka bijanu kisi kata ya gharshana ke madhyama se tvacha mem pravesha karata haim,jatharantra enthreksa mukhya rupa se dushita bhojana, evanjukama usa janavara ka mansa khane se jo bimari se mara ho,
tvachiya enthreksa 95% manava mamalom ka karana hai,ya vishva stara para aura adhika ke lie uttaradayi hai .
enthreksa ke sabhi tinom prakara ghataka haim agara turanta ilaja nahim kiya jae.
haija eka tivra atisara(dasta ki bimari) hai jo jivanu vibriyo kolara ke antargrahana ke karana hoti hai.
sancharana pratyaksha sandushana mala - mukha se ya dushita pani aura bhojana ke antargrahana ke madhyama se hota hai.
roga ka achanaka hamala gambhira tivra jaliya atisara se hota hai jo gambhira pani ki kami aura gurde ke kama na karane ke karana nrityu ki ora agrasara ho sakata hai 
atyanta kama ushmayana avadhi - do ghante se pancha dinom taka- sashakta rupa se phatanevale prakopa ko badha़ata hai, jisase rogiyom ki sankhya mem bahuta teji se vriddhi ho sakati haim.
haija se sankramita logom mem lagabhaga 75% ke bare mem koi bhi lakshana vikasita nahim hote haim.
halanki, roganu unake mala mem 7 se 14 dinom taka bane rahate haim,aura phira vatavarana mem jada़ne lagate haim, sashakta rupa se anya vyaktiyom ko sankramita karane ke lie.
haija eka atyanta sankramaka roga hai jo bachchom aura vayaskom donom ko prabhavita karata hai.
anya atisariya rogom ke viparita, yaha kuchha ghantom ke antargata hi svastha vayaskom ko mara sakate haim.
kama pratiraksha vale vyaktiyom jaise ki kuposhita bachchom ya echaaivi ke satha ji rahe logom mem nrityu ka adhika khatara rahata hai yadi vaha haija se sankramita hote haim.
19 vim shatabdi ke daurana, haija apane mula jalashaya ya srota,se bharata mem ganga ke delta se dakshina eshiya mem ghatane se pahale shesha vishva mem kai bara phaila.
chhaha mahamariyam darja ki gayi,jisane yuropa, aphrika aura amerika bhara mem lakhom logom ko mara dala.
satavim mahamari, jo abhi bhi jari hai,dakshina eshiya mem 1961 mem shuru huyi,1971 mem aphrika aura 1991 mem amerika pahunchi.
yaha bimari aba kai deshom mem sthanika mani jati hai aura roganu jo haija ke karaka haim vartamana mem paryavarana se samapta nahim kiye ja sakate haim.
maleriya eka aisi bimari hai jo sabhi unra ke logom mem sankramita ho sakati hai.
yaha plajmodiyama prajatiyom ke parajivi ke karana eka vyakti se dusare vyakti ko sankramita machchharom ke katane se phailata hai.
bukhara, siradarda, thanda lagana, aura ulti - ama prathama lakshana 10 se 15 dina mem eka vyakti ke sankramita hone ke bada dikhai pada़te haim.
prabhavakari davaom ke satha agara turanta ilaja nahim kiya jae to maleriya gambhira bimari paida kara sakata hai jo ki aksara ghataka hoti hai.
maleriya sancharana ki tivrata sthaniya karakom,jaise varsha paitarna, machchhara prajanana saitom aura machchhara prajatiyom ki anantarata aura niyamitata para nirbhara karati hai.
kuchha kshetrom mem pure sala maleriya ke mamalom ki sankhya sthira rahati hai - yaha maleriya sthanika hai jabaki anya kshetrom mem maleriya mausamiya hai, jo amataura para barasata ke mausama ke satha samanata rakhata hai.
bada़i aura ghataka mahamari una kshetrom mem bhi phaila sakati jaham logom ka maleriya parajivi ke satha bahuta kama samparka hota hai, isalie unamem sukshma ya koi bhi rogakshamata nahim hoti.
ye mahamariyam mausama ke dvara sakriya ho sakati haim,evam jatila apata sthiti ya prakritika apadaom se aura adhika bhada़ka sakati haim.
lagabhaga duniya ki abadi ke 40% loga, jyadatara duniya ke sabase gariba deshom mem rahane vale,maleriya ke khatare mem haim.
hara sala, 500 lakha se adhika loga maleriya se gambhira rupa se bimara ho jate haim.
adhikansha mamale aura mautem upa sahara aphrika mem hoti haim.
halanki, eshiya, laitina amerika, madhya purva aura yuropa ke kuchha hisse bhi prabhavita ho rahe haim.
maleriya se mukta kshetrom mem jaba yatri jate haim jaham maleriya sankramana atyadhika hai asurakshita ho jate haim,unamem kama ya rogakshamata nahim ke barabara hoti hai,apane desha lautate vakta aksara vilamba ya maleriya ke galata nidana ki chapeta mem ajate haim.
rebija eka pashujanya vishanujanita roga hai jo gharelu aura jangali janavarom ko sankramita karata hai.
yaha anya pashuom aura manushyom ko sankramita janavarom (arthat katane, kharoncha, tuti hui tvacha aura shleshma jilli para chatane) se lara ke satha nikata samparka ke madhyama se phailata hai.
rebija donom pashuom aura manushyom ke lie ghataka hai,jaba eka bara isa bimari ke lakshana vikasita ho jaem.
rebija ke prathama lakshana ama taura para anishta aura shvasana, jatharantra aura/ya kendriya tantrika pranali ke sahabhagita ke sanketa dete haim.
tivra charana mem atisakriyata ke sanketa (ugra rebija) ya pakshaghata (gungi rebija) prabala hote haim.
donom ugra aura gungi rebija mem, antatah argha lakava ke bada sabhi mamalom mem,amataura para sansa banda ho jane ke karana koma aura mauta ho jati hai. 
saghana upachara ke bagaira,bimari ke pahale sata dinom ke daurana rogi ki mauta ho jati hai.
vikasita deshom mem, rebija mukhya rupa se jangali janavarom mem poshaka ke rupa upasthita rahati hai, jinase roga gharelu pashuom aura manushyom mem bhi phaila jata hai.
hala hi mem, baita rebija duniya ke kuchha bhagom mem eka mahatvapurna mahamari ke srota ke rupa mem (yani amerika aura ऑstreliya) mem ubhari hai.
uttara amerika mem sabase aghika darja manava rebija, silvara balom vali baita rebija vayarasa ke sankramana ke karana se hone vali mautem,aura ऑstreliya mem purva aparichita rebija vayarasa ke sankramana se kama se kama do manava mautem hui.
dakshina amerika mem, vanyajiva rebija, vishesha rupa se baita rebija ki sankhya badha़ rahi hai. 
2003 mem pahali bara dakshina amerika mem kuttom ki tulana mem aura adhika logom ki nrityu vanyajiva ke katane ke karana rebija se huyi. 
halanki, kutte aphrika aura eshiya mem mukhya poshaka ke rupa mem haim aura duniya bhara mem pai jane vali manava rebija se hone vali mautom ke liye sabase aghika jimmedara bhi haim.
niyamita rupa se manushyom ka rebija se sankramita hona, sankramita kuttom aura billiyom, jangali mansabhakshi prajatiyom jaise, lomada़iyom, raccoons, skunks, gidada़, aura bheda़iye, aura kitabhakshi aura pishacha chamagadada़ ke katane se hota hai.
pashu, ghoda़e, hirana, aura anya shakahari rebija se sankramita ho sakate haim halanki ve sambhavatah anya janavarom ko va aura logom ko vayarasa sancharita kara sakate haim, lekina aisa shayada hi kabhi hota hai.
ama sardi ki taraha yaha bhi vayu dvara phailata hai.
kevala ve loga jo apane phapheda़om mem tibi hone se bimara haim, sankramaka hote haim|
jaba sankramaka vyakti khansate, chhinkate, bolate ya thukate haim, to ve besili ke rupa mem jane jane vale, tibi kitanuom ko hava mem phenkate haim|
eka vyakti ko sankramita hone ke lie inaki kevala eka chhoti sankhya ko sansa lene ki avashyakata hoti hai| 
anaupacharita chhoda़ diya jaya, to sakriya tibi roga vala pratyeka vyakti ausatana 10-15 logom ko hara sala sankramita karega|
kintu tibi besili se sankramita vyakti avashyaka rupa se isa roga se bimara nahim hoga|
pratirakshi tantra isa tibi besili ko ‘divara se banda kara deta hai’ jo, eka mote moma jaise avarana se surakshita, varshom taka prasupta raha sakata hai|
jaba kisi ka pratirakshi tantra kamajora hota hai, to rogi hone ka khatara aura adhika hota hai|
sansara mem koi na koi naya tibi besili se prati sekanda sankramita hota hai|
vartamana mem, sampurna vishva ki janasankhya ka eka tihai tibi besili se sankramita hai|
vishva svasthya sangathana (dabluechao) akalana karata hai ki 2005 mem tibi ke nae mamalom ki sabase adhika sankhya dakshina-purva eshiya kshetra mem paya gaya, jo sarvabhaumika rupa se prasanga mamalom ka 34% mana gaya hai|
phira bhi, upa-sahariya aphrika mem akalita ghatana dara dakshina-purva eshiya kshetra ki apeksha lagabhaga duguna hai, prati 100,000 janasankhya mem lagabhaga 350 mamale|
2005 mem tibi se 6 miliyana (60 lakha) nrityuem parinama hui|
aphrika kshetra mem sabase adhika nrityu ki sankhya aura prati vyakti sabase adhika nrityu-sankhya donom haim|
aphrika mem tibi ki mahamari 1990 ke dashaka mem teji se badha़i, kintu yaha vriddhi prati varsha kama ho rahi hai, aura ghatana darem aba sthira ho gai haim ya girana prarambha kara chuke aise dikhai pada़te haim|
2005 mem, sabhi chhah dabluechao kshetrom mem akalita prati vyakti tibi ghatana sthira thi ya gira rahi thi|
phira bhi, prati vyakti ghatana darom mem dhimi giravata janasankhya vriddhi ke dvara barabara kara diya gaya hai|
parinamasvarupa, prati varsha badha़ti hui nae mamalom ki sankhya aphrika, purvi bhumadhyasagara aura dakshina-purva eshiya ke dabluechao kshetrom mem sarvabhaumika rupa se abhi bhi badha़ rahi hai|
laingika rupa se prasarita sankramana (esatiaisa) aise sankramana haim jo prarambha mem eka vyakti se dusare vyakti mem laingika samparka se phailate haim|
30 se adhika vibhinna laingika rupa se sankranya jivanu, vishanu aura parajivi haim|
kai, vishesha echaaivi aura upadansha mem, mam se bachchom mem garbhavastha aura prasuti ke samaya, aura rakta utpadom tatha utaka sthanantarana se bhi prasarita ho sakate haim|
inpha़luenja़a (pha़lu) upari shvasa sambandhi eka vishanuja roga hai jo jvara, sharirika pida़a, siradarda, thakana, bhukha ki kami, eka sukhi khansi, aura eka sujana ya sukha gala jaise lakshana utpanna karate hue, achanaka hota hai|
pha़lu samanya sardi jukama ke samana nahim hai; pha़lu ke lakshana prayah adhika gambhira hote haim, aura apa aksara sardi jukama ke satha ki apeksha adhika kama ya skula chhoda़enge|
adhikatara loga bina samasyaom ke thika ho jate haim, kintu kabhi-kabhi yaha roga eka jivanuja sankramana, jaise ki eka karna sankramana, nasika sankramana, ya shvasanali-shotha ki ora agrasara ho jata hai|
achchha griha upachara ina sankramanom se bachane mem sahayata kara sakata hai|
kuchha logom mem aura adhika gambhira jatilataem, adhika samanya rupa se nimoniya, ho sakati haim|
jatilataom ke lie badha़e hue khatare para ke loga, jinhem aspatalavasa ki avashyakata hoti hai ve haim chhote bachche, 65 varsha ke vayaska aura adhika vriddha, tatha ve jinako gambhira chikitsiya samasyaem hoti haim|
apaki ankhom ka sapheda bhaga aura apaki palakom ki bhitari satahem netra-shleshmala ke nama se jani jane vali eka paradarshi jilli (koshikaom ki patali parata) se dhaki hoti hai|
yadi netra-shleshmala pradahita hoti hai, to apako netrashleshmala shotha namaka eka roga hai|
netrashleshmala shotha ke tina prakara hote haim – kashtakaraka, pratyurja (elarjika), aura rogajanaka|
pratyeka prakara ka netrashleshmala shotha vibhinna karakom se hote haim|
kashtakaraka netrashleshmala shotha taba hota hai, jaba eka takalipha़ dene vali vastu, jaise ki klorina, ya eka barauni, apaki ankhom mem chala jata hai|
yaha apaki ankhom ko pradahayukta kara sakata hai aura yadi apane unhem mala, to yaha isa pradaha ko aura bhi kharaba kara sakata hai| 
takalipha़ dene vali vastu se bachane aura apaki ankhom ko na malane se sahayata milegi| 
phira bhi, yadi apaki ankhe bahuta adhika lala aura kashtakara haim, to apako turanta hi chikitsiya sahayata talashani chahie| 
pratyurja (elarjika) netrashleshmala shotha taba hota hai jaba apaki ankha eka 'pratyurjaka' ke samparka mem ati hai|
pratyurjaka eka vishesha padartha hai jo sharira ke prabhavita bhagom mem jalana aura pradaha utpanna karate hue apake sharira ke pratirakshi tantra ko asamanya rupa se pratikriya karane ke lie taiyara karata hai|
rogajanaka netrashleshmala shotha eka vishanu se, jivanu se ya, asamanya mamalom mem, eka laingika rupa se prasarita sankramana, jaise ki klemaidiya, ya sujaka se ho sakata hai|
sabase samanya lakshanom mem ankhom ka lala hona aura pani dena shamila hai|
apa apani barauniyom para eka chipachipi parata bhi dekha sakate haim, vishesha rupa se jaba apa subaha mem pahali bara jagate haim, jo apaki ankhom ko yaha mahasusa kara sakati hai jaise ve eka satha phansa gai haim|
rogajanaka netrashleshmala shotha eka bahuta samanya roga hai, aura samanya chikitsa shalyachikitsaom mem abhilikhita sabhi ankha-sambandhita samasyaom ke 35% ke lie jimmedara hai| 
yaha bachchom aura vayovriddhom mem adhika samanya hai|
rogajanaka netrashleshmala shotha ko kabhi-kabhara hi kisi chikitsiya upachara ki avashyakata hoti hai kyonki yaha sankramana prayah 1-2 haphtom ke andara samanyatah svatah se thika ho jata hai| 
adhikatara logom ke lie, rogajanaka netrashleshmala shotha koi jatilataem utpanna nahim karati hai|
ve jo rogajanaka netrashleshmala shotha se jatilataem hone ke sabase adhika khatare para hote haim, ve haim navajata shishu, jo 28 dinom, ya usase kama unra ke hote haim|
eka bahuta chhoti unra mem ankha mem sankramana sthayi kshati pahuncha sakata hai| 
yadi apako rogajanaka netrashleshmala shotha hai jo ki laingika rupa se prasarita sankramana (esatiaisa), jaise ki klemaidiya, se hua hai, to apaka roga haphtom ki apeksha, kai mahinom taka raha sakata hai| 
khujali eka sankramaka tvacha roga hai jisase atyadhika khujali hoti hai|
yaha sarkoptisa skaibiyai namaka chhoti kutakiyom se hoti hai jo tvacha mem chheda karati haim| 
khujali naja़diki sharirika samparka se, kama samanya rupa se, kapada़o aura bichhaune ke satha apradhana samparka se phaila sakati hai| 
khujali vishesha rupa se una deshom mem vistrita phaili hai jinaki uchcha janasankhya ghanatva aura chikitsiya seva ki simita pahuncha hai| 
inglainda aura velsa mem mamalom ki sahi sankhya ka akalana karana kathina hai kyonki bahuta loga isa roga ka upachara apane samanya chikitsaka se bina mile khuda karate haim| 
phira bhi, eka adhyayana ne yaha sanketa kiya hai ki pratyeka 1,000 mem 1 ko hara mahine khujali ka sankramana hoga|
khujali bachchom aura mahilaom mem adhika samanya hai aura uttari inglainda mem adhika vistrita phaile hue se dikhai pada़te haim| 
khujali ke sabase adhika prakopa sardiyom mem hote haim, sambhavatah kyonki sala ke isa samaya mem loga adhika samaya andara bitate haim aura eka dusare ke karibi nikatata mem hote haim|
kai karanom se jo puri taraha se nahim samaje haim, khujali ke prakopa hara 20 varshom mem kai mamalom ke satha charama para pahunchate hue, eka chakra ka anusarana karate hue dikhate haim|
khujali samanyatah eka gambhira roga nahim hai,kintu tivra khujali apriya ho sakati hai aura isake jivana shaili para pratikula prabhava ho sakate haim|
khujali ka samanyatah vishesha krimom ka prayoga karake saphalatapurvaka upachara kiya ja sakata hai|
punah-sankramana ko rokane ke lie, yaha mahattvapurna hai ki parivara ke sabhi sadasyom ke satha-satha koi bhi laingika sathi jo apake pasa the, ka upachara ho|
kabhi-kabhi, khujali eka gauna tvacha sankramana ki ora agrasara ho sakati hai kyonki tvacha atyadhika khujali se ugra evam pradahita ho sakati hai|
yadi apako pahale kabhi khujali nahim hui hai, to skaibija़ kutaki se apake prarambha mem sankramita hone ke 2-6 hapha़te bada lakshana shuru honge| 
yaha isalie kyonki apake pratirakshi tantra ko- jisaka skaibija़ se sambandhita khujali ke lakshanom ko utpanna karane ka irada hai – sankramana se pratikriya karane ke lie samaya ki avashyakata hoti hai|
yadi apako atita mem khujali hui thi, to apake lakshana sankramana ke 48 ghante ke andara prarambha ho jaenge kyonki apaka pratirakshi tantra eka skaibija़ sankramana ke prati pratikriya karana ‘sikha’ chuka hoga|
skaibija़ apaki tvacha ko tivrata se khujali hone jaisa mahasusa karata hai| 
ye lakshana rata mem, aura garama pani se phuhara-snana, ya snana ke bada aura kharaba hone ki ora jukava rakhata hai|
ye skaibija़ kutakiyam apaki tvacha para chhote lala dane aura lakirem bhi chhoda़engi, jo unake dvara apaki tvacha mem chheda karane se utpanna hue chihna haim| 
skaraleta jvara himolitika streptokaॉksi ke rupa mem jane jane vale eka prakara ke jivanu dvara utpanna eka sankramana hai, jo e streptokaॉkkasa jivanu samuha ka hai|
skaraleta jvara samanyatah streptokaॉkkasa jivanu, jaise ki strepa throta se eka gale ya tvacha sankramana ke bada hota hai| 
skaraleta jvara vaha dadora hai jo streptokaॉkkasa jivanu ki eka jatilata hai| 
skaraleta jvara skaraletina ke rupa mem bhi jana jata hai, halanki yaha shabda kabhi-kabhi roga ke halke rupa ka ullekha karane ke lie prayukta hota hai|
yaha yuke mem strepa sankramana ke lie pratijaivikiyom ke niyamita prayoga ke karana purnataya asamanya hai|
samanyatah yaha adhika samanya rupa se 4 aura 8 varsha ki unra ke bhitara ke bachchom ko prabhavita karata hai, kintu yaha kisi bhi unra ke vyakti ko ho sakata hai, halanki yaha 2 varsha ke andara ke bachchom mem atyanta asamanya hai|
halanki skaraleta jvara eka atyanta gambhira roga hua karata tha, ajakala adhikatara mamale atyanta halke hote haim| 
skaraleta jvara ko khansi aura chhinkom se vayuvahita bundom mem sthita jivanu ke madhyama se diya ja sakata hai|
kuchha mamalom mem yaha ghavom aura phaphalom sahita, anya sthanom se sankramana ka pichha kara sakata hai| 
yaha sankramita hone ke bada lakshanom ko vikasita karane ke lie lagabhaga 2 se 4 dina leta hai| 
yadi apa sochate haim ki apake bachche ko skaraleta jvara hai, to apake samanya chikitsaka se salaha lena eka achchha vichara hai|
yaha sanketa karane ke lie koi pramana nahim hai ki jaba apa garbhavati haim taba skaraleta jvara hone se apa apane bachche ko khatare mem dalate haim|
matishkavarana (maininajaitisa) vaha jilliyam hai jo matishka aura merurajju (kendriya tantrika tantra) ko dhakati haim| 
ve kendriya tantrika tantra aura shesha sharira ke bicha sankramana ke prati atirikta avarodha ke rupa mem kama karate hue, eka avarodha ki taraha kama karate haim| 
yaha jivanu ya vishanu se ho sakata hai|
yuke mem prati varsha lagabhaga 3500-4000 ullikhita mamale haim|
jivanuja matishkavarana shotha (maininajaitisa) roga ka eka kama samanya rupa hai|
vishanuja matishkavarana shotha (maininajaitisa) adhika samanya hai kintu samanyatah kama gambhira hota hai|
jivanuja matishkavarana shotha (maininajaitisa) kai vibhinna prakara ke kitanuom se hota hai, jo dasa mem se eka mem prakritika rupa se naka aura gale ke pichhe rahate haim aura najadiki dirghakalina samparka, khansane, chhinkane aura chumane se phaila sakata hai|
kuchha logom mem hi ye kitanu sharira ki surakshaom ko tabaha kara dete haim aura matishkavarana shotha (maininajaitisa) utpanna karate haim|
yaha jivanu sharira ke bahara adhika samaya taka nahim raha sakata hai atah yaha jala apurtiyom, tarana-talom, bhavanom ya karakhanom se nahim pae ja sakate haim aura kevala vishesha prakara ke jivanu (meninjokokala si) janasankula kshetrom mem adhika teja़i se phailate haim| 
jivanuja matishkavarana shotha (maininajaitisa) ke lie rogotpadana kala 2 aura 10 dinom ke bicha hai| 
vishanuja matishkavarana shotha (maininajaitisa) eka kama gambhira roga hai kintu phira bhi atyanta durbalakaraka ho sakata hai, aura bahuta kama bara, siradarda, jvara aura unindepana se gahari sammurchha mem vikasita ho sakata hai|
vishanuja matishkavarana shotha (maininajaitisa) ke lie rogotpadana kala 3 haphtom taka ho sakata hai| 
yaha eka drishya hai jo sabhi jagaha pratidina gharom mem chalata hai| 
bachche skula se ate haim aura khane ke lie kuchha dekhate hue sidhe rasoighara ki ora aguvai karate haim|
apa kaise sunishchita kara sakate haim ki apake bachche ko poshana milata rahe kintu phira bhi svasthyavardhaka bhojana ke lie jagaha rahe|
halanki bachchom ko kama bara alpaharom ki avashyakata hoti hai, jaise-jaise ve bada़e hote jate haim, yaha ashcharyajanaka nahim hai ki adhikatara skula ke bada bhukhe hote haim| 
kai bachche dopahara ka bhojana jaldi karate haim – 11:30 ya isase pahale- aura phira unake agale khane ke avasara se pahale taka aparahna kakshaem aura shayada skula ke bada ki gatividhi bhi| 
yaha ashcharyajanaka nahim hai ki dina ke anta mem alpahara vikraya mashina (vendinga mashina) bahuta adhika akarshaka dikhata hai| 
apake bachche ki unra aura skula ke bada ki dinacharya para nirbhara karate hue, apa shayada hamesha yaha niyantrita karane mem samaksha na hom ki vaha dera aparahna mem kya khata hai| 
kintu thika abhi hara mata manie|
apane bachche ka margadarshana karane ke lie skula ke bada achchhe alpahara ka kadama uthaem jo santoshajanaka hoga aura phira paushtika ratri-bhojana ke lie jagaha bhi chhoda़ega|
apa svayam ko apane bachche ki jagaha rakhem aura usake eka samanya karya-divasa ke khana-pana ki suchi para dhyana rakhem| 
kuchha chhote bachche madhya-subaha alpahara le sakate haim, kintu adhikatara bada़e skuli unra ke bachche nahim lenge| 
ina savalom ke jababa dijie : dopahara ke bhojana ka samaya kaba hai ?
dopahara ke bhojana mem apaka bachcha kya aura kitana khata hai ?
kya apane skula karyakrama ke pashchata apaka bachcha alpa bhojana leta hai ?
yaha apako samajane mem madada karega ki jaba vaha ghara ata hai taba apaka bachcha kitana bhukha hoga|
manyatah ratri ka bhojana parosati haim isake bare mem bhi apa sochana chahengi|
eka bachcha jo ghara mem 3 baje bhukha ata hai aura bhari matra mem alpa bhojana khata hai vaha agara ratri ka bhojana 5:30 baje hai to sambhavatah bhukha nahim hoga|
isi taraha, eka bachche se jisake abhibhavaka 7:30 baje dera taka chalane vala karya karate haim dopahara ke bhojana ke pashchata kuchha nahim khae ki asha karana, yaha nyayasangata nahim ho sakata hai|
apane bachche ke karyakrama ke bare mem sochem aura usake anusara yojana banaem|
age, isa bare mem bata karem ki apaka bachcha alpa bhojana ke samaya kauna-se alpa bhojana lena pasanda karega|
eka satha svasthyavardhaka vikalpom ki eka suchi sonche aura vividha prakara ke taje phalom aura sabjiyom ko usamem shamila karana na bhulem|
jabaki eka keka ka tukada़a ya kuchha alu ke chipsa nishiddha bhojana nahim hone chahie, para aise alpaposhita alpa-bhojana skula ke bada ke pratyeka dina ki bhojana suchi mem bhi nahim hone chahie|
agara apa yaha kara sakate haim, to apane bachche ko kirane ki dukana mem satha le jaem aura poshaka tathyom ki chippi padha़ne evam utpadom ki tulana karane mem kuchha samaya bitaem|
protina, resha, kailshiyama, aura anya mahatvapurna poshaka tatvom ki matra para dhyana dem, tatha inake ansha parimana para bata karane ka avasara na gavaem|
eka satha, una alpa-bhojanom ko chunem jinamem shakkara, charabi aura namaka kama haim|
isa prakriya mem sammilita hona ise adhika sambhava banata hai ki apaka bachcha svasthyavardhaka bhojana chayana karana sikhega|
svasthyavardhaka alpa bhojanom ko unhem unake bhojana dibbom mem ya baikapaikom mem paika karake ya unhem ghara para drishya tatha khane ke lie taiyara rakhakara asani se upalabdha karaem|
agara apa skula ke bada ghara para haim, to eka chhota bachcha eka lakada़i para chitiyam (mungaphali ke makkhana (pinata batara) aura kishamisha se dhaki sileri chitiyam), andde ki navem(kada़e ubale andde ke vaijom ke upara eka chija़ pala), ya phala kababa jaise eka maulika alpa bhojana banane mem apaki madada karane mem ananda le sakata hai|
bada़e bachche phala smudi, hyumasa dipa ke satha mini-pita, ya purna-anaja biskuta ke upara chija aura nashapati ke tukada़om ka annada le sakate haim|
bada़e bachche prayah apane svayam ke alpa bhojanom ko banana pasanda karate haim, atah unhem samagriyam aura kuchha sadharana anudesha pradana karem|
yadi ratri ka bhojana bahuta hi pasa ho to jaba taka apa parivara ka bhojana banana khatma karem, eka prathama korsa jaise ki thoda़a salada ya atirikta sabji dene ki sochie|
una ratom ke lie jaba ratri ka bhojana ghantom dura ho to apa eka adhika santoshajanaka alpa bhojana jaise ki adha saindavicha ya purna-gehum ke ate ke taॉrtila se bane eka kvesadila, kama charabi vali chija़, tatha maikroveba mem garama kiye salsa ka chunava kara sakate haim|
eka achchhe alpa bhojana ko banane mem jitana samaya lagata hai usase adhika samaya use khane mem lagana chahie|
yadi apaka bachcha eka skula ke pashchat ke karyakrama ya eka dekha-rekha karane vali ke ghara jata hai to pata kijie ki vaham alpa bhojana diya jata hai ya nahim|
yadi apako apa jo sunate haim vaha pasanda nahim ata hai, to vikalpa sujayem ya eka atirikta alpa bhojana bandhe jise apaka bachcha skula ke bada kha sakata hai|
purna anaja se bana kama charabi vala grenola bara eka achchhi pasanda hai|
anya vikalpom mem khadya mishrana, kashthaphala, kama-mitha sukha siriyala, purna-anaja ke chhote-mapha़ina, aura sukhe meve sammalita haim|
agara apaka bachcha skula ke bada eka khela khelata hai aura use jaldi takata badha़ane ki avashyakata hoti hai to isa prakara ke khadya padartha vishesha rupa se madadagara hote haim|
phira se, rasoighara ke patala para samane va madhya mem ya rephrijaretara mem kuchha svasthyavardhaka chhoda़na sabase achchha upaya hai|
eka bhukha bachcha, eka bhukhe baliga jaise hi, sambhavatah sabase kama pratirodha vala rasta apanata hai|
niyamita janchem apake bachche ki vriddhi aura vikasa ka nirikshana karane mem daॉktara ki madada karati haim|
varshom se, daॉktara unchai aura vaja़na ke mapom ko usi unra ke anya bachchom ki tulana mem eka bachche ki sharirika vriddhi ka mulyankana karane ke lie apane prathamika upakaranom ke rupa mem istemala karate haim|
biemaai eka ganana hai jo eka bachche ki unchai aura vaja़na ka upayoga yaha ankane ke lie karata hai ki unamem kitani shariri charabi hai|
daॉktara biemaai ka upayoga bachche ka vaja़na eka nishchita unchai aura unra ke lie kitana uchita hai, yaha nirdharita karane ke lie karate haim|
bachche hamesha rasoighara ke asapasa jame rahate haim, khasa kara teraha se unnisa varsha ki ayu mem|
adhika khane ki vikriti vale logom ko, pahale bhojana samposhana aura sukha pradana karata hai, lekina bada mem yaha avishvisaniya aparadha va duhkha ka kendra hota hai|
ve samanyatah bhojana ki bahuta adhika matra jaldi- jaldi khate haim aura ve aisa karate hue, purna rupa se niyantrana ke bahara mahasusa karate haim, tatha ve samaya-samaya para hi ja़yada nahim khate haim balki kaphi hada taka niyamita rupa se karate haim|
ye bartava khane ka eka tarika़a bana jata hai aura khane ke paraheja (daitinga) ke satha bari-bari a sakata hai|
adhika khane ki vikriti una logom mem adhika samanya hai jo sthulakaya hote haim, parantu yaha svastha vajana vale logom ko bhi prabhavita karata hai|
phira bhi, isa para thoda़i janakari hai ki isa vikriti se kitane bachche aura teraha se unnisa varsha ki ayu vale prabhavita haim, kyonki isa paristhiti ko hala mem hi pahachana gaya hai, aura kai loga isake lie madada mangane mem bhi bahuta adhika lajjita hote haim|
ilaja karate baliga aksara kahate haim ki unaki pareshaniyam balyavastha ya kishoravastha mem shuru hui thim|
phira bhi, yaha vishvasa kiya jata hai ki amariki vayaskom ke 2% - lagabhaga 1 miliyana se 2 miliyana logom ko - yaha vikara hai|
aura halanki khane ke anya vikarom vale bahusankhya logom mem mahilaem haim, yaha akalana kiya gaya hai ki adhika khane ki vikriti vale vyaktiyom ke eka tihai se adhika purusha haim|
bachche aura teraha se unnisa varsha ki ayu vale jo kabhi-kabhi bahuta adhika khate haim jaruri nahim hai ki unhem adhika khane ki vikriti ho|
bachche bahuta adhika bhukhe ho sakate haim, visheshatah vriddhi uchhala ke daurana, jaba unhem apane badha़te sharira ko urja dene ke lie adhika poshaka tatvom ki avashyakata hoti hai|
atah yaha nishchita karana mushkila ho sakata hai ki eka bachche ko adhika khane ki vikriti hai|
lekina kai lakshana kisi eka svasthyavardhaka bhukha vale se kisi ko jo adhika khata hai, alaga karate haim| mata-pitaom ko jaba ve rasoi bhandara ya rephrijaretara se bahuta adhika matra mem bhojana gayaba pate haim to pahale yaha eka samasya laga sakati hai, halanki yaha kalpana karana kathina hai ki eka bachche ne itana adhika khaya hoga|
loga jo adhika khate haim unakom aise anubhavom ka bhi anubhava ho sakata hai jo khane ki kai gada़bada़iyom jaise ki avasada, chinta, aparadha bodha, ya sharma ke lie samana hote haim|
ve skula, karya, ya dostom ke satha milana-julana tala sakate haim kyonki ve apani adhika khane ki samasya se sharminda hote haim|
adhika khane ki vikriti ka karana ajnata hai, halanki bharatiya svasthya sansthana (enaaiecha) ne vivarana diya hai ki una sabhi logom mem se adhe se jyada ko jinhem yaha hai unake avasada ka bhi eka itihasa hai|
phira bhi, yaha aspashta bana hua hai, ki adhika bhojana karana avasada lata hai ya ki loga avasada se isa vikara ki ora pravritta haim|
kai loga jo adhika khate haim ve kahate haim ki ye ghatanaem gussa, duhkha, ya vyagrata dvara sakriya ho sakate haim|
vaijnanika yaha bhi parikshana kara rahe haim ki kaise dimaga karya karata hai aura sharira chayapachaya karata hai (vaha tarika jisase sharira urja ka upayoga karata hai) ye adhika khane ko prabhavita kara sakate haim|
adhika khana khane ki gadabada़i ko anya khane ki gadabada़iyom jaise ki bulimiya narvosa (manasika karanom se atikshudha) aura enoreksiya narvosa (manasika karanom se bhukha na lagana) se alaga karana, yaha mahatvapurna hai|
bulimiya narvosa (kabhi-kabhi adhika khana-julaba lakshana kahalata hai) vale loga adhika khana khate haim aura phira ulti karate haim ya vajana badha़ane se bachane ke lie virechaka aushadhiyom ka prayoga karate|
ve loga upavasa bhi kara sakate haim (kuchha samaya ke lie khana banda kara dete haim) ya adhika khana khane ke bada badhya hokara vyayama karate haim|
jo loga adhika khane ki gada़bada़i se grasta hote haim, usi taraha ve jo bulimiya narvosa vale haim bhojana ko bahuta adhika matra mem bara-bara khate haim aura isake vishaya mem aparadha va sharma mahasusa karate haim|
phira bhi, bulimiya (atikshudha) se bhinna, adhika khane ki gada़bada़i vale vyakti julaba nahim lete haim, aura isilie lagatara vajana badha़ta jata hai|
enoreksiya (akshudha) narvosa, eka anya khane ki gada़bada़i mem bhi khane ke bare mem aparadha mahasusa karana sammalita hai|
jabaki adhika khane ki gada़bada़i vale loga lagatara adhika bhojana karate haim, enoreksiya (akshudha) vale loga sambhavatah apane sharira ko jivana-santarjaka hani pahunchate hue, khuda ko bhukha rakhate haim|
ve bhi kama vajana prapta karane ke lie badhya hokara vyayama kara sakate haim, enoreksiya (akshudha) ethaletika ke rupa mem jnata eka paristhiti|
daॉktara ke satha parivarika gatiki, khane ke tarike, aura bhavanatmaka mudde para bahasa karane ke bada hi eka bachche ki adhika khane ki gada़bada़i ka roga nidana kiya jaega|
daॉktara purna sharirika parikshana bhi karega aura motapa tatha paushanika paryaptata ki kuchha samasyaom ko nirdharita karane ke lie prayogashala janchom ka adesha de sakate haim|
kisi bhi khane ki gada़bada़i ki taraha, yaha bhi mahatvapurna hai ki vaha bachcha chikitsa prapta karem|
vibhinna prakara ki chikitsa adhika khane ki gada़bada़i ke ilaja mem madada kara sakati haim|
udaharanasvarupa, parivarika chikitsa aura sanjnanatmaka vyavaharika chikitsa, logom ko nirikshana karane aura unaki bhojana karane ki adatom mem parivartana karane ke takanika tatha unako tanava se pratikriya karane ke tarike sikhati haim|
parivarika chikitsa vyakti ko madada karane ki prakriya mem sammpurna parivara ko sammilita karati hai|
sanjnanatmaka vyavaharika chikitsa logom ko khuda ko harane vali sochom ko badalane mem madada karane ki paddhati ko unake vyavahara mem parivartana karane ke satha milati hai|
salaha, dusarom ke satha unake sambandhom para vichara karane mem bhi madada karati hai aura unaki una kshetrom mem karya karane mem madada karati hai jo unamem vyagrata utpanna karate haim|
kuchha mamalom mem, daॉktara chikitsa ke satha sanyojana mem upayoga karane ke lie avasadarodhi aushadhi ka nuskha likha sakate haim|
phira bhi, kisi bhi khane ki gada़ba़di ke lie shighra upachara nahim hai|
ilaja kai mahine ya adhika samaya le sakata hai jaba vyakti yaha sikhata hai ki bhojana ke prati eka adhika svasthyavardhaka paddhati kaise rakhem|
halanki vaja़na niyatranna karyakrama adhika khane ki gada़bada़i se grasta kuchha logom ke lie madadagara haim, bachchom aura teraha se unnisa varsha ki ayu valom ko daॉktara ke paramarsha aura nirikshana ke bina bhojana niyatranna ya vajana-niyatranna karyakrama shuru nahim karana chahie|
yadi apako apake bachche mem adhika khane ki samasya hone ka sandeha hai, to paramarsha ke lie apane daॉktara ya svasthya dekha-rekha kartta aura nirdeshom ke lie shikshita dimagi svasthya peshevarom se samparka karem jinhem bachchom mem khane ki gada़bada़iyom ka ilaja karane ka anubhava ho|
apane bachche ko ashvasana dijie ki madada ke lie ya kevala sunane ke lie apa vaham haim|
khane ki gada़bada़i hone ko svikarane mem kathinai ho sakati hai, aura apaka bachcha apane mem eka samasya ke hone ko manane ke lie taiyara nahim ho sakata hai|
apa bhi svasthyavardhaka khane ki adatom ko apane khuda ke bhojana aura vyayama ke satha sakaratmaka sambandha ka pratirupana karake aura bhojana ko eka puraskara ke rupa mem upayoga na karake protsahita kara sakate haim|
apake parivara aura daॉktara ki madada se, apaka bachcha phira se svastha khane ki ora vapasi daura shuru kara sakata hai|
apane shishu ko stana-pana karane ya shishu ahara ka pana karane ka chunava karana una prathama nirnayom mem se eka hai jo ashanvita mata-pita lenge|
halanki koi sahi ya galata pasanda nahim hai, phira bhi, balachikitsa amariki akadami (da amerikana akedemi ऑpha pidiyatriksa, eepi), anya sansthaom jaise ki amariki chikitsa samiti (da amerikana medikala aisosieshana, eemae), amariki ahariya samiti (amerikana daitetika aisosieshanaedie), aura vishva svasthya sangathana (varlda heltha ऑrganaijeshana, dabluechao), ke satha stana-pana ko shishuom ke lie sarvashreshtha ke rupa mem sipha़arisha karane mem shamila hai|
visheshatah, eepi kahati hai ki shishuom ko shuruati 6 mahinom taka kevala stanapana karana chahie aura yadi mam aura shishu donom ichchhuka haim to yaha stanapana 12 mahinom (aura age) taka jari rahana chahie|
lekina stanapana (ya poshana) sabhi striyom ke lie mumakina aura behatara nahim ho sakata hai|
apake shishu ko stanapana ya shishu ahara pana karane ka nirnaya apake stanapana karane ke satha-satha apaki jivana shaili sahita apake sukha ke stara para adharita hona chahie|
stana ka dudha navajatom ke lie poshana ka adarsha rupa hai, lekina una mataom ke lie jo stanapana karane mem asamartha haim ya jo aisa nahim karane ka nirnaya leti haim, unake lie shishu-ahara eka achchha vikalpa hai|
yadi apa apane shishu ko vyavasayika rupa se taiyara shishu-ahara khilate haim, to yaha nishchita kara lem ki apake shishu ki poshana jaruratem puri hongi|
aura apa phira bhi apake shishu ke satha purnatah achchhe se juda़e rahenge|
akhirakara, chahe stana ka dudha ya shishu-ahara, bhojana karana mam aura shishu ke bicha sanyojana ka eka mahatvapurna samaya hai|
apake shishu ko stanapana ya shishu-ahara pana karane ka nirnaya eka atyanta vyaktigata hai|
kintu ye kuchha tathya haim jisa para jaba apa yaha nirnaya lete haim ki kauna sa apake aura nae shishu ke lie sarvashreshtha hai, apa vichara karana chaha sakate haim|
stanapana karana mam aura shishu donom ke lie eka anokha anubhava ho sakata hai|
yaha adarsha poshana aura eka vishishta bandhane vala anubhava pradana karata hai jise kai stanapana karati mataem sanjoti haim|
anya tathya shishu ke pratirakshana tantra mem yogadana karake, sankramana ke prati avarodhom ko badha़akara aura jivanu tatha vishanuom jaise jivom ki vriddhi ko kama karake stanapana kiye eka shishu ko sakramanna se bachane mem madada karate haim|
stana ke dudha mem prakritika rupa se sabhi vitamina aura khanija bhi hote haim jinaki eka navajata ko avashyakata hoti hai|
khadya aura aushadhi prashasana (da phuda enda draga edaministreshana, ephadie) yaha sunishchita karane ke lie shishu ahara banane vali kampaniyom ko niyantrita karati hai ki ve sabhi jnata avashyaka poshaka-tatvom ko apane shishu-ahara mem upalabdha karaem|
kyonki stana dudha ke kuchha adhika jatila padarthom ka nirmana karana bahuta kathina hai aura kuchha ki abhi bhi pahachana nahim hui hai|
stana dudha ki eka senta bhi kimata nahim hoti hai aura kyonki pratiraksha aura roga-pratikaraka unamem unaki mam ke stana ke dudha dvara pahuncha jate haim, stanapana kiye shishu prayah una shishuom se kama bimara hote haim jo shishu–ahara grahana karate haim|
udaharanasvarupa, visheshajnom ne nishchita kiya hai ki jo shishu 4 ya adhika mahinom taka kevala stanapana kiye haim unamem kana ke sankramana 40% kamatara ghatita hote haim|
isaka artha hai ki ve daॉktara ke karyalaya mem kama se kama daura karate haim, jo kamatara ada karane tatha nuskhom aura kauntara para milane vali aushadhiyom ke lie kama paise dene ke barabara mana jata hai|
usi taraha, jo striyam stanapana karati haim unaki apane bimara shishu ki dekha-rekha karane ke lie karya se samaya nikala pane ki kama sambhavana hai|
balachikitsa pustika ke apraila 1999 ke prakashana mem prakashita eka kimata adhyayana mem, shodhakarttaom ne nirdharita kiya hai ki shishu jinhem kabhi bhi stanapana nahim karaya gaya tha ve prati varsha "331 se "475 taka atirikta chikitsa kharcha ke uttaradayi hote haim|
eka stanapana karati mata ko stana dudha ke utpadana ke lie hara roja 500 atirikta kauloriyom ki jarurata hogi, jisaka artha hai ki use vyapaka vividha prakara ke susantulita aharom ka sevana karana chahie|
yaha stanapana karane vale bachchom ko unaki mataom ke stana ke dudha ke madhyama se vibhinna svadom se parichaya karata hai, jinamem unaki mataom ne kya khaya hai isa para nirbhara karate hue alaga-alaga gandha hoti haim|
milane aura roganurahita karane ke lie bina botala ke aura dukana se shishu-ahara ke lie antima minata ki bhagadauda़ ke bina, stana ka dudha hamesha taja aura upalabdha hota hai|
aura kyonki stana ka dudha hamesha sahi tapamana para hota hai, atah adhi rata mem botala garma karane ki koi avashyakata nahim hai|
stanapana karane vali mataom ke lie sakriya rahana - aura apane bachchom ke satha bahara tatha jaham-taham jana - bhi asana hai aura ve janati haim ki jaba kabhi bhi unake bachchom ko bhukha lagegi, unake pasa bhojana upalabdha hoga|
hala ke adhyayanom ne yaha sanketa kiya hai ki stanapana karane se bachapana aura vayaska ke mutape ko rokane mem sahayata mila sakati hai|
rashtriya mahila svasthya suchana kendra (neshanala vumensa heltha inpharmeshana sentara) (yu.esa. ke svasthya aura manava sevaom ke vibhaga ka eka bhaga) ke anusara bachche jinhem stanapana karaya jata hai, kama anavashyaka vajana badha़ne ki ora jukava rakhate haim, jo unhem bada mem kama ativajani hone mem sahayata kara sakate haim|
hala ke adhyayanom ne yaha sanketa kiya hai ki bachche jinhem 6 mahine taka kevala stanapana karaya gaya tha, unaka aikyu 5 se 10 anka adhika hai banisbata una bachchom ke jo shishu-ahara ka pana karate the| kai stanapana karane vali mataem apane bachchom ke satha bahuta adhika ghanishtha rupa se juda़ne ke isa anubhava ka sachamucha ananda uthati haim|
aura isa tvacha-se-tvacha ke samparka se mam aura shishu ke bicha bhavanatmaka sambandha badha़ sakata hai|
eka shishu ko purna rupa se poshita kara pane ki yaha kshamata eka nai mam ko apane shishu ki dekhabhala karane ki apani kshamata para atmavishvasa mahasusa karane mem bhi sahayata kara sakati hai|
stanapana kailori ko bhi jalata hai aura garbhashaya ko sikuda़ne mem sahayata karata hai, atah stanapana karane vali mataem apani purva-garbhavastha ki banavata aura vajana mem adhika shighrata se vapasa ane mem sakshama ho sakati haim|
isake atirikta, adhyayana darshate haim ki stanapana purvarajonivritti-sambandhi stana kainsara ke khatare ko kama karane mem sahayata karata hai aura garbhashaya tatha andashaya kainsara ke khatare ko bhi kama karane mem sahayata kara sakata hai|
halanki yaha bachchom ke lie sabase achchha poshanika srota hai, phira bhi, stanapana kuchha chintaom ke satha ata hai jise kai nai mataem adana-pradana karati haim|
jabaki kuchha ke lie yaha prarambha se hi asana hota hai, yaha chunautipurna ho sakata hai|
kabhi-kabhi, mam aura shishu donom ko bahuta adhika dhairya aura stanapana ki dinacharya se abhyasta hone ke lie driढ़ta ki avashyakata hoti hai|
kintu yaha saba prayasa prayah mam aura usake bachche donom ke lie lambe samaya mem mulyavana hota hai|
adhikatara mata-pita apane bachchom ko eka garama aura aramadayaka kaॉpha़i ka kapa dene ka sapana bhi nahim dekhenge, kintu ve niyamita rupa se kaipha़inayukta nridu-peya parosa sakate haim|
halanki yaha sambhava hai ki apaka bachcha kabhi na kabhi kaipha़ina antargrahana karega, para kaipha़ina ke upabhoga ko vishesha rupa se chhote bachchom mem kama se kama taka rakhana, yaha eka achchha vichara hai|
halanki yunaiteda stetsa ne abhi taka kaipha़ina antargrahana aura bachchom ke lie koi margadarshana nahim banaya hai , kanadai margadarshana yaha sipharisha karata hai ki shishu vidyalaya ke bachchom ko eka dina mem 45 miligrama se adhika kaipha़ina na mile|
jo soda ke eka 12 aunsa (355mililitara) kaina ya 1.5 aunsa (43 grama) dudha ke chaॉkaleta bara mem pai jani vali kaipha़ina ki ausatana matra ke barabara hota hai|
eka uttejaka padartha jo bachchom aura vayaskom ko samana rupa se prabhavita karati hai, kaipha़ina eka nashila padartha hai jo kai paudhom ki pattiyom aura bijom mem prakritika rupa se banati hai|
kaipha़ina kritrima rupa se bhi banai jati hai aura kuchha khadyom mem milai jati hai|
kaipha़ina ko eka nashile padartha ke rupa mem paribhashita kiya gaya hai kyonki yaha kendriya tantrika tantra ko uttejita karati hai|
nichale stara para, kaipha़ina logom ko adhika satarka aura jaise unamem adhika urja hai, aisa mahasusa karati hai|
pratidina eka 12 aunsa (355mililitara) mitha kiya gaya nridu peya pina, eka bachche ke mutape ke khatare ko 60% badha़a deta hai| kevala kaipha़inayukta peyom mem hi khali kailori (kailori jo koi poshaka tatva nahim pradana karate haim) nahim hote haim, kintu jo bachche unase peta bharate haim, unhem paushanika kamiyam utpanna karane ke khatare mem rakhate hue, vitamina aura khanija nahim pate haim jinaki unhem svasthyavardhaka srotom se avashyakata hoti hai|
vishesha bachchom ko, jo bahuta adhika soda़a pite haim (jo samanyatah tisari aura athavi kakshaom ke bicha prarambha hota hai) kailshiyama milana chhuta sakata hai jise maja़buta haddi tatha danta banane ke lie dudha se pane ki avashyakata hoti hai|
kaipha़ina eka mutravardhaka hai jo ki sharira se pani nikalane (peshaba karane se) ka karana hota hai, jo nirjalikarana mem sahayoga kara sakata hai|
peyom mem upasthita ki matra vastava mem nirjalikarana utpanna karane ke lie paryapta hai ki nahim, yaha phira bhi spashta nahim hai|
yaha isa para nirbhara kara sakata hai ki peya pine vala vyakti kaipha़ina ka abhyasta hai ki nahim aura kitana kaipha़ina usane usa dina liya hai|
surakshita rahane ke lie, garama mausama mem, jaba bachchom ko prasvedana se khoe pani ko pratisthapita karane ki avashyakata hoti hai, taba atyadhika kaipha़ina ke upabhoga se bachana, yaha buddhimata ho sakati hai|
kaipha़ina ko achanaka banda karane se pratyahara lakshana (siradarda, mansapeshiya darda, asthayi avasada, aura chida़chida़apana), utpanna ho sakate haim, vishesha rupa se una logom ke lie jo bahuta adhika upabhoga karane ke adi haim|
kaipha़ina hridaya ki samasyaom ya snayu tantra ki gada़bada़iyom ko badha़a sakata hai, aura kuchha bachchom ko yaha jnata nahim bhi ho sakata hai ki unako khatara hai|
kaipha़ina samvedanashilata kaipha़ina ki usa matra ki ora sanketa karati hai jo kisi mem prabhava utpanna karega|
yaha matra eka vyakti se dusare vyakti mem badalati rahati hai|
ausatana, parshva prabhava utpanna karane ke lie jitana chhota vyakti, utani kama kaipha़ina ki avashyakata hoti hai|
phira bhi, kaipha़ina samvedanashilata rojana kaipha़ina upayoga ki matra se sabase adhika prabhavita hoti hai|
ve loga jo kaipha़inayukta peya niyamita rupa se pite haim, jaldi hi kaipha़ina ke prati ghati hui samvedanashilata utpanna karate haim|
isaka artha hai ki unhem samana prabhava prapta karane ke lie jo kaipha़inayukta peya pratidina nahim pita hai jaise kisi se kaipha़ina ki uchchatara khurakom ki avashyakata hoti hai|
atah, jitana adhika kaipha़ina apaka bachcha leta hai, usake samana prabhava mahasusa karane ke lie use utani hi adhika kaipha़ina ki avashyakata hogi|
akara ke adhara para kaipha़ina ke prabhava se adhika asani se prabhavita hone ke atirikta, chhote bachche kaipha़ina ke prati adhika samvedanashila hote haim kyonki ve bada़e bachchom ya vasyakom jitana isake samparka mem nahim rahate haim|
kaipha़ina isake upabhoga karane ke kuchha ghantom ke andara sharira ke eka chhora se dusare chhora taka chalati hai aura phira peshaba se nikala jati hai|
yaha sharira mem sanchita nahim hoti hai, kintu apaka bachcha isake prabhava ko 6 ghantom taka mahasusa kara sakata hai yadi vaha isake prati samvedanashila hai to|
dudha aura anya kailshiyama-sanriddha bhojana hamesha bachchom ke aharom mem avashya hone chahie aise bhojana haim|
akha़irakara, kailshiyama maja़buta, svastha haddiyom ke lie eka pramukha mulabhuta anga hai|
kintu 9 se 18 ki unra ki lada़kiyom ke 85% aura lada़kom ke 60% se adhika kailshiyama ki pratidina sipharisha ki gai 1,300 miligrama pane mem asaphala hote haim|
yaha ashcharyajanaka nahim hai jaba apa yaha vichara karate haim ki ajakala kai bachche dudha ki apeksha soda adhika pite haim, jo kailshiyama ke sabase badha़iya srotom mem se eka hai|
aura kishora-kishoriyam jo dhunrapana karate ya soda, kaipha़inayukta peya, athava madira pite haim, unhem isase bhi kama kailshiyama mila sakata hai kyonki ve padartha sharira ke kailshiyama ke avashoshana aura upayoga ke tarike mem badha dalate haim|
kintu shaishava se kishoravastha ki hara unra mem kailshiyama eka aisa poshaka tatva nahim hai jise bachche asani se chhoda़ dene ka kha़tara utha sakate haim|
bachapana aura kishoravastha ke daurana, sharira maja़buta haddi banane ke lie isa kailshiyama khanija ka upayoga karata hai- eka prakriya jo kishoravastha ke anta taka sabhi kintu purna ho jata hai|
haddi ka kailshiyama navayuva vayaskata mem ghatana prarambha ho jata hai aura jaise-jaise hamari unra badha़ti hai haddi ki kramika kami hoti jati hai, kha़asa taura para aurato mem|
kishora-kishoriyam, kha़asakara lada़kiyam, jinake ahara asthi nirmana karane ke lie apane adhikatama samarthya taka poshaka tatva nahim pradana karate haim, ve asthiroga asthi-sushirata(ऑstioporosisa) vikasita karane ke aura adhika khatare mem hoti haim, jo kamaja़ora haddiyom se asthi-bhanga ke khatare ko badha़ati hai|
kailshiyama mansapeshi ke sankuchana, tantrika se sandesha prasarita karane, aura harmonom ko nikalane mem eka mahatvapurna bhumika nibhata hai|
yadi rakta mem kailshiyama ke stara kama haim(kailshiyama ke kama antargrahana ke karana), to samanya koshika karya ko sunishchita karane ke lie kailshiyama asthiyom se liya jata hai|
jaba bachche bachapana aura kishoravastha mem paryapta kailshiyama aura sharirika gatividhi prapta karate haim, to ve mumakina sabase adhika maja़buta haddiyom ke satha apana vayaska jivana prarambha kara sakate haim|
dudha aura anya dugdha utpada kailshiyama ke una sabase achchhe aura sabase suvidhajanaka srotom mem se haim jise apa pa sakate haim|
dugdha pratyurjata(elarji) ki sambhavana ke karana 1 sala se kama ke shishuom ko sampurna dudha ya dugdha utpada kabhi bhi nahim lena chahie|
apane bachche ke prathama varsha mem poshana ke pramukha srota ke rupa mem stana ke dudha ya shishu ahara ke satha juda़e rahem|
1 aura 2 varsha ki unra ke bachchom ko unake samanya vriddhi aura mastishka vikasa ke lie avashyaka ahariya vasa pradana karane mem sahayata karane ke lie sampurna dudha lena chahie|
2 varsha ki unra ke bada, adhikatara bachche kama-vasa ya vasarahita dudha mem badala sakate haim, halanki apako isake lie pahale apane bachche ke chikitsaka se vichara-vimarsha kara lena chahie|
achchhi kha़bara yaha hai ki sabhi dudha mem - vasarahita se sampurna taka - prati sarvinga (eka bara mem khane ka bhojana) kailshiyama ki lagabhaga samana matra hoti hai|
2005 ke ahariya margadarshana 2 se 8 varsha ke bachchom ke lie pratidina kama-vasa ya vasarahita dudha ka 2 kapa (473 mililitara) ya dugdha utpada aura 9 varsha tatha isase adhika ke bachchom ke lie 3 kapa (710 mililitara) ki sipharisha karata hai|
dugdhasharkara asahishnuta vale bachchom mem antriya enjaima (laiktasa) jo dugdha utpadom ke sharkara (dugdhasharkara) ko pachane mem sahayata karata hai, paryapta nahim hota hai|
ina bachchom ko dudha pine ya dugdha utpadom ko khane ke bada maroda़ ya dasta ho sakata hai|
saubhagyavasha, kama dugdhasharkara aura dugdhasharkararahita dugdha utpada ke satha-satha laiktasa drapa haim jise dugdha utpadom mem milaya ja sakata hai aura goliyam haim jo li ja sakati haim taki jinhem dugdhasharkara asahishnuta hai ve dugdha utpadom ka ananda le sakate hai aura unamem phayada dene ke lie jo kailshiyama hote haim unase labha mila sakata hai|
sakhta, adhika dinom ke chija़om (jaise ki chedara) mem bhi dugdhasharkara kama hoti hai, aura dahi jisamem sakriya samvardha (kalchara) hote haim, pachane mem adhika asana hote haim aura dugdhasharkara ki samasyaom ko utpanna karane ki bahuta kama sambhavana hoti hai|
dudha ki protina kuchha logom mem pratyurja pratikriyaem utpanna kara sakati haim|
sampurna dugdha protina ke lagabhaga 80% ke lie jimmedara kesina, gaya ke dudha mem pradhana protina hai|
kesina vaha hai jo dahi banata hai jo taba banata hai jaba dudha ko khatta hone ke lie chhoda़ diya jata hai|
bache hue 20% gaya ke dudha ke protina dahi ke pani, panila bhaga jo dahi ko hata lene ke bada bachata hai, mem hote haim|
eka vyakti protina ya to kesina ya dudha ke pani vale bhaga aura kabhi-kabhi donom ke prati bhi pratyurja ho sakata hai|
yadi apa sochate haim ki apaka shishu ya bachcha dudha ke prati pratyurja ho sakata hai, to apane bachche ke chikitsaka se bata karem|
dudha ke prati pratyurjata vale shishuahara die jane vale shishuom ko soya-adharita ya haipoelarjenika shishuahara mem badalane ki avashyakata ho sakati hai|
adhika bada़e bachchom ke lie, dudha aura dugdha utpadom ke kuchha achchhe vikalpom mem kailshiyama-sanriddha chavala ya soya dudha(yadi soya sahana hota hai) ya vigana( purna shakahari) utpadom, jaise ki vigana ( purna shakahari) chija़ aura dusare soya-adharita (phira se, yadi soya sahana hota hai) ya chavala-adharita himibhuta dejarta, sharabata, pudinga, aura barpha paॉpa shamila haim|
bachchom aura kishora-kishoriyom ke mata-pitaom jo ovo- shakahari haim (ve ande khate haim, kintu dugdha utpadom ko nahim) ya vigana( purna shakahari)( ve kevala vanaspati srotom se prapta bhojana karate haim) ko isa bare mem chinta ho sakati hai ki eka dugdha rahita ahara unake bachchom ko paryapta kailshiyama ki apurti kara sakata hai ya nahim|
halanki kevala eka vanaspati ahara mem sipharisha ki gai kailshiyama ki matra milana, yaha eka adhika chunauti ho sakati hai, kailshiyama ke achchhe srotom mem garahi hari pattidara sabjiyam, braॉkoli, chana, aura santare ka rasa, soya aura chavala peya, evam dhanyom (siriyalom) sahita kailshiyama pushta kie hue utpada shamila haim|
vishesha rupa se kishoriyam, alpa khuraka lene ka nirnaya le sakati haim aura dugdhahara khane se paraheja kara sakati haim jise ve sochati haim ki ye unhem moti banati haim|
kintu apake kishora-kishoriyom ka yaha samajana mahattvapurna hai ki 8-aunsa (240mililitara) ke eka gilasa vasarahita dudha mem kevala 80 kailori aura shunya vasa hota hai aura kishoriyom ke lie pratidina sipharisha ki gai kailshiyama antargrahana ke eka chauthai ki apurti karata hai|
vastava mem, loga jo kailshiyama sanriddha ahara khate haim unaka vajana sachamucha kama ho sakata hai aura unamem sharirika vasa bhi kama hota hai|
eka adhyayana mem, kishoriyam jo hara dina 300 milagrama atirikta kailshiyama, jo eka gilasa dudha ke barabara hota hai, li thim, unaka vajana 2paunda (907 grama) taka kama tha banisbata una lada़kiyom ke jinhem atirikta kailshiyama nahim mili thi|
apa sampurna dugdha utpadom - aura soda tatha sharkarayukta phala ke rasa jinake bahuta kama paushanika mahattva haim, ke svasthyavardhaka vikalpom ke rupa mem kama vasa aura vasarahita dugdha utpadom ko bhi pesha kara sakate haim|
yadi apaka kishora/kishori rasa pita/piti hai, to unhem kailshiyama pushtakrita 100% phala ke rasom ko de(magara, bahuta adhika rasa nahim, kyonki isamem bahuta adhika sharkara aura kailori sammilita ho sakati hai|)
apake kishora/kishori se asthi-sushirata tatha eka svasthyavardhaka ahara mem dugdha utpadom aura anya kailshiyama-sanriddha khadyom ke mahattva ke bare mem bhi bata karem|
halanki bachchom ke lie yaha sarvottama hai ki kailshiyama jisaki unhem avashyakata hoti hai, eka kailshiyama-sanriddha ahara se mile, kabhi-kabhi yaha bilkula bhi sambhava nahim ho sakata hai|
yadi apa chinta karati haim ki apake bachche ya kishora/kishori ko paryapta kailshiyama nahim mila raha hai, to kailshiyama anupurakom ke lie apane bachche ke chikitsaka se vichara-vimarsha karem|
kailshiyama avashoshana ke lie vitamina di avashyaka hai, atah yaha mahattvapurna hai ki apaka bachcha ina poshaka tattvom ki paryapta matra le|
jaba tvacha suryaprakasha mem khuli rakhi jati hai taba sharira dvara nirmita, vitamina di pushtakrita dugdha aura anya utpadom, machhali, aura andapitom mem bhi paya jata hai|
aura, apane bachche ko niyamita sharirika kriya-kalapom aura vyayama mem shamila hone ke lie prerita karana na bhulem, jo asthi svasthya ke lie bahuta mahattvapurna hota hai|
vajana-sambhalane vale vyayama jaise ki rassi kudana, dhire-dhire dauda़na tatha chalana bhi maja़buta haddiyam banane aura banae rakhane mem sahayata kara sakati haim|
vastava mem, hala hi ka vaijnanika pramana yaha sanketa karata hai ki, bachche ya kishora/kishoriyom ke lie, vyayama kailshiyama antargrahana ki apeksha behatara asthi svasthya se aura bhi adhika maja़buti se juda़a hua ho sakata hai|
inamem se adhikatara, eka anukaraniya vyakti ke rupa mem kama karate haim aura dugdha utpadom tatha anya khadyom ka jo uchcha paushtika tatva vale hote haim, upabhoga karate haim. sambhavatah apa apane apa se bhi kailshiyama ka upayoga kara sakate haim|
karbohaidreta sharira ke sabase mahattvapurna aura asani se upalabdha urja ke srota haim|
halanki inhem 2000 ke dashaka mem eka kharaba ilja़ama bhi mila hai aura prayah amerika mem mutape ke prakopa ke lie bhi isa para dosha lagaya jata raha hai, phira bhi karbohaidreta bachchom tatha vayaskom donom ke lie svasthyavardhaka ahara eka avashyaka bhaga hai|
jaise-jaise sharkara ka stara badha़ta jata hai, agnyashaya insulina namaka eka harmona sravita karata hai, jo sharkara ko rakta se koshikaom mem jaham sharkara ka urja ke srota ke rupa mem upayoga kiya ja sakata hai, hatane ke lie avashyaka hai|
kuchha khadyom mem yaha karbohaidreta (adhikatara ve jinamem sadharana sharkara aura atyadhika shuddha anaja, jaise ki maida aura sapheda chavala) asani se rasayanika parivartana karate haim aura apake bachche ke rakta sharkara stara to shighrata se badha़ane ka karana banate haim|
dusari tarapha़, jatila karbohaidreta (sampurna anaja mem pae jane vale), sharkara stara ko adhika dhire-dhire badha़ne ki anumati dete hue adhika dhire-dhire rasayanika parivartana karate haim|
eka aisa ahara khana jisamem adhika khadya hote haim jo rakta sharkara mem teja़i se vriddhi karate haim, eka vyakti ke madhumeha aura hridayarogom ki taraha ki svasthya samasyaem vikasita karane ke khatare ko badha़a sakata hai, halanki ye adhyayana adhikatara vayaskom mem kie gae haim|
karbohaidreta kama karane ke hala ki sanaka ke bavajuda, nirnayaka bindu yahi hai ki karbohaidreta yukta sabhi khadya apake bachche ke lie hanikaraka nahim haim, chahe ve jatila hom, jaisa ki sampurna anaja mem, ya chahe sadharana karbohaidreta, jaise ki ve jo phalom mem pae jate haim|
adhikatara khadyom mem ye hote haim isaka vichara karane para yadi karbohaidreta ki itani manahi hoti, to hamem eka bahuta bada़i samasya hoti|
kintu, nissandeha, kuchha karbohaidreta-saghana khadya, anyom ki apeksha adhika svasthyavardhaka hote haim|
kyonki kai chikitsiya visheshajna ye sochate haim ki parishkrita karbohaidretom (khadyom mem pae jane vale parishkrita sharkara aura kaindi tatha soda jaise peyom, aura parishkrita anaja jaise kai pasta aura bredom mem pae jane vale sapheda chavala aura maida) ke ja़rurata se ja़yada upabhoga ne yunaiteda stetsa mem mutape ki akasmika vriddhi mem yogadana kiya hai|
kintu kaise koi eka prakara ka bhojana itani bada़i samasya paida kara sakata hai?
beshaka , koi bhi vyakti kasarata na karane se aura avashyakata se adhika khana khane ke karana motape ka shikara ho sakata hai |
lekina tathakathita bure karbohaidreta chini aura shuddha khadya padartha ,atirikta kailori ke lie mahatvapurna yogadana karate haim |
logom ka rukha jarurata se jyada shuddha khadya padartha khane ka hota hai |
aura ,aksara khadya-padartha jaise kola aura kaindi se kuchha bhi poshana nahim milata hai atah inako khane ki koi avashyakata nahim hai |
lekina hala ke kuchha varshom mem shuddha karbohaidretsa ki kada़i alochana hui hai para isaka yaha matalaba nahim hai ki sari shakkara kharaba hoti haim |
samanya karbohaidreta bahuta sare poshaka khanom mem paye jate haim-jaise phala , sabjiyam aura deyari utpadom jo avashyaka poshaka tatva pradana karate hai aura vikasa aura sampurna svastha mem madada karate haim |
udaharana ke lie, taja phalom mem samanya karbohaidretsa hote haim ,para unamem vitamina aura phaibara bhi hote haim |
2005 ahara nirdesha-pustika sabako- bachchom aura kishorom sahita sabita anaja ki khapata badha़ane aura shakkara kama karane ki salaha deti hai |
bachchom ke ahara ka kama se kama adha bhaga sabuta anaja ke sevana se ana chahie | 
sabuta anaja mem pae gae aparishkrita karbohaidretsa achchhe hote haim , parishkrita anaja utpadom ko pholika esida (pholeta bhi kaha jata hai), loha , aura anya poshaka tatvom ke satha paushtika banaya ja sakata hai, aura jisake phalasvarupa isamem poshaka tatva sabuta anaja ki tulana mem adhika hote haim |
anaja ki vastavika rashi apake bachche ki unra, linga, aura sharirika gatividhi ke stara ke adhara para alaga alaga hogi |
skula jane vale bachchom ko ausatana 4 - 6 aunsa ke samakaksha anaja roja khana chahie aura isa khane ka kama se kama adha bhaga sabuta anaja ka hona chahie |
sabhi aunsa eka bara parose gae eka khane ke barabara hote haim |
atah roti ka eka tukada़a, khane ke lie taiyara 1 kapa siriyala ya adha kapa pake hue chavala , pakaya hua pasta, ya garma siriyala 1 aunsa ke barabara mana ja sakata hai |
khadya-padartha jinamem adhika chini milai gai ho (soda, kukija़, keka, kaindi, jame hue desarta, aura kuchha phalom vale peya) inamem kailori adhika aura jaruri poshaka tatva kama hote haim |
aksara motape ka karana adhika chini vala ahara hota hai|
mithe khadya padartha bahuta khane ke karana danta mem sada़na hoti hai |
chini ki khapata para niyantrana rakhana mahatvapurna sanyama hai |
kuchha khadya padarthom mem chini dalane se svada badha़ta hai ,khasakara siriala aura dahi jaise poshaka khadya padarthom mem thoda़i shakkara dali jae to isase apake bachche ke lie koi hani nahim hai aura danta ke daॉktara ke pasa jane ki bhi avashyakata nahim hogi |
apane bachchom ko adhika chini aura kama poshana tatva vala bhojana dene ke bajaya svasthya-vardhaka vikalpom jaise -aise phala jo ki prakritika rupa se mithe hom va karbohaidretayukta nashta jinamem phaibara aura vitamina hom jinaki bachchom ko avashyakata hoti hai , dena chahie | 
chini para niyantrana ke lie upara se chini dalana soda aura mithe peya padarthom ko chhoda़na hoga |
pratyeka 12 aunsa (355 mili litara) karboneteda, mitha shitala peya 10 chammacha chini aura 150 kailori (49 mililitara) ke barabara hota hai |
kaha jata hai ki mitha peya amerika ke bachchom ke dainika ahara mem chini ka sabase bada़a srota haim |
prati dina 12 aunsa (355 mili litara) mitha shitala peya pine se bachchom mem motape ka khatara badha़ jata hai |
bachchom ko soda ya rasa peya (jinamem aksara shitala peya ke taraha hi jyada chini hoti hai) ke sthana para kama vasa vala dudha, pani, ya 100% phalom ka rasa dena chahie | 
halanki 100% phalom ke rasa mem chini nahim dali jati hai , prakritika phalom ke rasa mem upara se chini dalane ke karana kailori badha़ jati haim |
ameriki bala roga akadami (apa) salaha deti hai ki sata sala ki unra ke bachchom ko 4-6 aunsa (118-177 mililitara) taka phalom ka rasa lena chahie aura usase adhika nahim , isase bada़e bachchom evam kishorom ko 8-12 aunsa ( 237-355mililitara )lena chahie |
bachchom ko santulita , poshaka ahara mile yaha sunishchita karana utana mushkila nahim hai jitana dekhane mem lagata hai |
samanya rupa se apane ghara mem achchhe karbohaidretsa ke evam svastha-vardhaka vikalpom (sabuta anaja , phala , sabjiyam aura kama vasa vala dudha aura deyari utpadom )ko bharapura matra mem rakhem , aura jinamem chini dali gai ho aise khadyom para niyantrana lagaya jana chahie (khasataura para jinamem poshaka tatva kama hom ) aura hara roja bachchom ko sakriya rahane ke lie protsahita karana chahie |
jaba apane bachchom ke bare mem bata hoti hai to kai mata pita kolestraॉla ke bare mem nahim sochate haim |
lekina adhika kolestraॉla hridaya roga aura daure ka pramukha karana jana jata hai, aura adhunika chikitsa anusandhana se pata chalata hai koronari dhamani ki bimari ki jada़em bachapana se hi hoti haim |
abhi taka bachapana ke kolestraॉla ke stara ka pata nahim lagaya ja saka hai para kuchha visheshajna sochate haim ki bachchom mem adhika kolestraॉla ka hona isa pramukha sarvajanika svasthya samasya ko kama ahamiyata di gai hai|
pichhale kuchha varshom mem, adhyayana bachchom mem natakiya dha़nga se motapa badha़na aura adhika vajana ke karana hone vali samasyaom ko bhi darshata hai |
aura kuchha bala roga visheshajnom ki riporta ke anusara bachchom mem kuchha sala pahale ki tulana mem kolestraॉla ke stara mem ullekhaniya vriddhi dekhi hai | 
bachchom ke svasthya aura kolestraॉla ke bicha koi sambandha karana logom ke lie mushkila hota hai kyonki uchcha kolestraॉla se juda़i samasyaem ama taura para salom taka nahim dikhati haim | 
isalie yaha mahatvapurna hai ki apako apane bachche ke kolestraॉla ke stara ke bare mem pata ho, khasakara agara mata ya pita ka kolestraॉla adhika ho |
kolestraॉla livara dvara nirmita eka komala padartha hai |
apake sharira dvara banaye gaye lipida ya vasa mem se eka hai aura jinaka koshika ki divarom ke nirmana karane ke lie aura kuchha harmona aura utakom ke rupa mem istemala kiya jata hai |
aisa kaha jata hai ki eka dina mem kolestraॉla ka sevana kama se kama 300 miligrama kiya jana chahie |
poshana ke suchaka patrom ko padha़kara apane parivara ko khane mem kitana kolestraॉla khana chahie ,nirdharita kijie |
isake alava, amerikana harta esosieshana ka kahana hai ki kula vasa ka sevana apani kula kailori ka 30% ya kama hona chahie aura santripta vasa 10% ya kama hona chahie |
agara apa ne aisakrima ka dusara kapa ya eka aura chija़bargara nahim khaya to apake sharira ko sucharu rupa se chalane ke lie paryapta kolestraॉla hota hai|
aisa isalie hai kyonki apaka livara svastha sharira ke karya karane ke lie paryapta kolestraॉla banata hai |
vastava mem, livara eka dina mem 1,000 miligrama kolestraॉla banata hai , baki jo khadya padartha apa khate haim usase ata hai |
sistika phaibrosisa (CF) eka vanshagata bimari hai jo ki shvasana aura pachana pranali ko prabhavita karati hai |
siepha़ upakala koshikaom ko asta-vyasta karake bachchom ko bimara kara deti hai , ye vo koshikaem haim jo ki tvacha mem pasine ki granthiyom ko banati haim aura ye phephada़om , yakrita, agnyashaya, aura pachana aura prajanana pranali ke andara bhi pai jati haim |
ina koshikaom mem dosha , sharira mem namaka aura pani ke santulana mem samasyaom ka karana banata hai , isake karana gadha़a balagama jo ki phephada़om ko avaruddha karata hai jo ki sankramana aura anya sansa ki samasyaom ki vajaha banata haim |
siepha़ ke satha bachchom mem, yaha balagama pramukha antom mem vasa aura poshaka tatvom ke samanya avashoshana ko rokane mem badhaka hota hai,jisake karana kharaba pachana, dhima vikasa, vajana badha़ne mem kathinai , antra gati dhimi hone ke karana mala visarjana mem kathinai , aura sankramana se lada़ne ki kshamata ko kama karata hai |
nidana ke samaya mem adhikatara rogiyom ki halata ko agnashayi kami kaha jata hai, isaka matalaba hai agnyashaya mem banae gae enja़aimom jo ki vasa , starcha, protina adi ko pachata hai , antom mem thika se parita nahim karata hai|
yaha avashyaka matra mem kai mahatvapurna poshaka tatvom ko avashoshita karane mem samasyaom ka karana banata hai |
atirikta vasa aura poshana tatva kailori aura nirdharita enjaimom ke satha achchha samagra poshana pradana karake apa apane bachche ko si epha ke satha svastha aura majabuta banane mem madada kara sakate haim |
shodha se pata chalata hai ki achchha santulita, poshaka ahara khane se koronari hridaya roga ke jokhima, stroka, kuchha taraha ke kainsara, aura haddiyom ki kamajori ko kama kara deta hai|
aura achchha santulita, poshaka ahara ki paikejinga vale khadya padarthom se kirane ki dukanom ke shelpha bhare hue haim |
apane parivara ke lie svastha bhojana lene ke samaya usa para likhe vadom se pare bhojana mem poshana mulyom, samagri, aura kailori ki ginati ko dhyana mem rakhate hue yaha dekhana bhi jaruri hai ki yaha apake parivara ke lie kitana svasthyavardhaka hai |
apako svasthyavardhaka khadya vikalpom ko chayana karane evam unase apake parivara ki poshaka jaruratem kaise puri hoti hai ?khadya-padarthom ke lebala para yaha suchana di hui rahati hai |
khadya evam aushadhi prashasana (ephadie) aura ameriki krishi vibhaga (yuesadie) ke sare paikda khadya-padarthom ke lebalom para poshaka suchana padha़ne yogya aksharom mem likhi rahati hai |
janakari ama taura para pichhe ki tarapha ya shirshaka poshana tathyom ke niche paikejinga ke tarapha dikhai deti hai| yaha bhi kirane ki dukanom mem taja khadya padarthom , phala, sabjiyom aura machhali ke pasa pradarshita rahati hai |
khadya-padarthom ke utpadaka svasthyavardhaka vasa-mukta ya kolestraॉla-mukta khadyannom ke utpadana ka dava karate haim , kai logom ko ashcharya hai ki kya ve ina davom ko sacha manakara bharosa kara sakate haim |
tathya yaha hai ki ephadie ko yaha dava karane ke lie khana nirmataom ko vaijnanika sabuta pradana karane ki avashyakata hoti hai |
agara aisa hai to bhi, davom ko savadhanipurvaka padha़na aura unhem achchhe se samajana ki unaka kya artha hai ? yaha eka achchha vichara hai |
kama vasa ka matalaba hai ki vaha utpada usi branda ke niyamita utpada ki tulana mem 25% kama vasa vala hai |
halke ka matalaba hai ki eka aisa utpada jo ki niyamita utpada ki tulana mem 50% kama vasa vale utpada hai |
kama vasa ka matalaba hai eka utpada ke prati sevana mem vasa ki matra 3 grama se kama honi chahie |
pratyeka khadya-padartha ke lebala ke upara apa pratyeka sevana ki matra ki suchi dekhenge |
ama taura para khadya nirmata yaha nirdharita karate haim aura yaha amataura para logom ke khane ki matra para adharita hota hai |
bhojana ke poshana ki sari janakari lebala para pratyeka sevana ki matra ke adhara para suchibaddha rahati hai |
agara 2 biskuta sevana ki matra likhi hai aura apa 4 biskuta khate haim jo ki sevana ka dugani matra hogi - atah apa ko poshana janakari ki dugani matra ki jarurata hai|
pratyeka kantenara mem sevanom ki sankhya se apa pure paika mem sare sevanom ki matra ko jana sakate haim |
to agara eka sevana 1 kapa hai, aura pure paiketa mem 5 kapa haim, to pratyeka paiketa mem 5 sevana hongi |
kailori urja ki eka ikai hai jisase pata chalata hai ki koi bhojana apake sharira ko kitani urja pradana karata hai ?
kailori ki sankhya jo ki khadya-padarthom ke lebala para suchibaddha rahati hai jisase pata chalata hai ki eka sevana mem kitani kailori haim ?
nambara do ,vasa se kailori-eka sevana mem vasa se kula kitani kailori ati haim aisa yaha batata hai |
lebala para vasa ko suchibaddha kiya hua rahata hai jisase loga apane ahara mem vasa ki matra ki jancha kara sakem |
ama taura para ahara visheshajna ka kahana hai ki pure dina ke khane se 30% se kama kailori vasa se milati haim |
isaka matalaba hai ki agara apa eka dina mem kula 2,000 kailori vala khana khate haim, isamem vasa se 600 se adhika nahim ana chahie |
ye sankhya dikhati hai ki 1 grama vasa mem kitani kailori , karbohaidreta aura protina hai |
yaha janakari hara poshaka tathyom ke lebala para sandarbha ke lie mudrita rahati hai |
dahine hatha ke kaॉlama mem dainika poshaka tathyom ke mulyom ko pratishata mem suchibaddha kiya hua rahata hai aura ve batate haim ki eka vyakti ko bhojana ke eka sevana se kitane vishishta poshaka tatva milenge |
adarsha rupa mem, pratyeka dina una poshaka tatvom ka 100% lena hi lakshya hai |
bhojana ke eka sevana mem 18% protina hota hai, usa hisaba se agara apa prati dina 2000 kailori ka khana lete haim to vaha khana apaki jarurata ke lie pratidina 18% protina deta hai |
suchibaddha kiye gaye kuchha vishishta poshaka tatvom ki nirdharita pratishata se roja ke khane mem yaha poshaka tatva adhika ya kama matra mem upalabdha haim , yaha taya karane mem bahuta madada milati hai |
agara bhojana mem poshaka tatvom ki matra 5 % ya usase kama ho , to usa khane mem poshaka tatvom ki matra kama mani jati hai |
bhojana ko poshaka tatvom ka eka achchha srota mana jata hai yadi usamem poshaka tatvom ki pratishata 10 se 19 ke bicha ho |
yadi bhojana mem dainika mulya ke 20% pratishata se adhika poshaka tatva haim, to yaha poshaka tatvom mem uchcha mana jata hai |
khadyom ke lebala para prati dina ke ausata ahara ki janakari 2000 kailori para adharita rahati hai, lekina bachchom ko vastava mem kitani kailori aura poshaka tatvom ki jarurata hogi , usaka stara unaki unra , vaja़na , linga aura sharirika kriyaom ke adhara para bhinna- bhinna hoga|
bhojana samanyataya paushtika hai ya nahim , isake lie ama taura para khadyom ke lebala ko eka gaida ke rupa mem istemala karem para itani bhi chinta nahim karem ki poshaka tatvom ki ganana prati aunsa taka karem, basa itana taka ho ki apana bachcha svastha aura achchhi taraha se badha़ raha hai |
yaha sankhya ingita karati hai ki bhojana ke eka sevana mem kitana vasa hai aura yaha amataura para grama mem mapa jata hai |
halanki bahuta adhika vasa khane se motapa aura svasthya sambandhi samasyaem ho sakati haim, para hamare sharira ko hara dina kuchha vasa ki jarurata to hoti hi hai |
vasa urja ka eka mahatvapurna srota hai - usamem prati grama karbohaidreta ya protina dugani matra mem urja hoti hai |
vasa tvacha, haddiyom aura antarika angom ke lie rodhana evam lachilapana pradana karate haim |
vasa kuchha khasa vitaminom (e,di, i aura ke ) ko vahana karati haim evam sanchaya mem madada karata hai |
bhojana mem bahuta adhika vasa hridaya roga sahita anya svasthya samasyaom ka karana bana sakati hai, para vayaskom aura 2 sala ki unra se bada़e bachchom ki dainika kailori mem vasa ki matra kevala 30% honi chahie |
vasa ke tahata asantripta vasa bhi suchibaddha hai |
kamare ke tapamana para ye vasa tarala rupa mem ho jati hai |
vanaspati telom, meve,mungaphali aura machchhali adi khanom mem asantripta vasa bahuta adhika matra mem hoti hai | 
asantripta vasa aksara achchhi vasa kahi jati hai kyonki vaha kolestraॉla ke stara ko nahim badha़ati hai jaisa ki santripta vasa karati hai |
kolestraॉla, amataura para miligrama mem mapa jata hai, jo ki vasa janakari ke antargata suchibaddha hai |
kolestraॉla sharira mem vitamina di aura kuchha harmonom ka utpadana karata hai aura sharira mem kai anya mahatvapurna padarthom ka nirmana bhi karata hai |
yadi rakta mem kolestraॉla ki matra bahuta adhika hai to yaha samasya bana sakata hai , jisake karana ethrosaklorosesa,dhamaniyom mem rukavata aura unaka sakhta hona hota hai jisaki vajaha se dila ka daura ya stroka hone ke jokhima badha़ sakate haim |
eka vyakti ki jarurata ke lie adhikansha kolestraॉla jigara dvara nirmita kiya jata hai |
yaham taka ki,mansa aura murgi utpada,ande aura sampurna dugdha deyari utpada jaise ahara srota bhi kolestraॉla ke stara mem apana yogadana dete haim|
namaka ka eka ghataka,sodiyama,poshana tathya lebala para miligrama mem suchibaddha hota hai|
sodiyama ki thoda़i hi matra sharira ke tarala padarthom ka uchita santulana banae rakhane ke lie avashyaka hoti hai, lekina atyadhika matra uchcha raktachapa badha़a sakati hai|
lagabhaga sabhi khadya padarthom mem sodiyama ki thoda़i bahuta matra svabhavika rupa se hoti hi hai, lekina kai sansadhita khadya padarthom mem matra kaphi adhika hoti hai|
khadya lebalom para sampurna karbohaidreta ke tahata suchibaddha ahariya phaibara mem svayam koi kailori nahim hoti hai aura vaha paushtika ahara ka eka jaruri hissa hota hai|
uchcha phaibarayukta ahara antra niyamitata ko badha़ata hai, peta ke kainsara ke khatare ko kama karane mem madada karata hai aura kolestraॉla ke stara ko bhi kama karane mem madada kara sakata hai|
sampurna karbohaidreta ke tahata suchibaddha shakkara bhi jyadatara khadya padarthom mem payi jati hai|
phalom mem prathamika sharkara hoti hai lekina phaibara, pani aura vitamina bhi hote haim, jisase ve eka svastha vikalpa bana jate haim|
dusari tarapha,nashte ke khadya padarthom, kaindi aura soda mem shakkara aksara atyadhika matra mem upara se bhi dali jati hai|
halanki karbohaidreta mem prati grama kevala 4 kailori hi hotim haim,peya padarthom aura nashte ke khadya padarthom mem uchcha shakkara ki matra ka artha hota hai ki apa jaldi hi kailori grahana kara lete haim,aura isa rikta kailori mem amataura para bahuta hi kama anya poshaka tatva hote haim|
yaha suchi apako batati hai ki apake eka bara ke khane mem kitani protina hai aura yaha matra amataura para grama mem napi jati hai|
mansapeshiyom, tvacha aura pratiraksha pranali sahita sharira ka adhikansha bhaga protina se bana hota hai|
yadi sharira ko paryapta vasa aura karbohaidreta nahim milata hai,to urja ke lie vaha protina ka upayoga kara sakata hai|
ande ,mansa,murgi utpada,machhali,dudha, panira, dahi, sukhe meve, soyabina, aura sukhe sema adi khadya padarthom mem atyadhika protina
bachchom ki hara dina ki 10 se 20% kailori kaise bhi protina se hi ani chahie |
vitamina e aura si vishesha rupa se do mahatvapurna vitamina haim, isilie inako poshana tathya lebala para suchibaddha hone ki avashyakata hai|
pratyeka sarvinga mem hara vitamina ki matra pratishata dainika mulya mem mapi jati hai isalie vitamina si ka 80% dainika mulya ka khana apako dina bhara ki vitamina si ki jarurata ka 80% pradana karata hai|
vitamina aura khanijom ki khadya lebala suchi para vitamina e ka nama ama taura para sabase pahale likha hota hai|
vitamina e achchhi drishti ke lie mahatvapurna hota hai aura tvacha ko svastha banae rakhane mem madada karata hai|
yaha gajara aura skvaisha jaisi narangi ranga ki sabjiyom mem aura gahari hari pattedara sabjiyom mem paya jata hai|
sanyoji utakom ka nirmana karane aura unhem banae rakhane ke lie , ghavom ko bharane ke lie aura sankramana se lada़ne ke lie sharira vitamina si ka upayoga karata hai|
vitamina si khatte phalom, anya phalom aura kuchha sabjiyom mem paya jata hai|
lebalom para ina do mahatvapurna khanijom ka pratishata likhana avashyaka hai aura yaha dainika pratishata mulyom mem mapa jata hai|
sharira mem kailshiyama ke bahuta sare upayoga hai, lekina svastha haddiyom aura dantom ke nirmana mem apani bhumika ke lie yaha sabase achchha jana jata hai|
dudha aura anya deyari utpada kailshiyama ke utkrishta srota hote haim|
1 aura 3 varsha ke bicha ki unra ke bachchom ko prati dina 500 miligrama kailshiyama ki jarurata hoti hai, jabaki 4 - 8 sala ke bachchom ke lie 800 miligrama ki jarurata hoti hai|
9 se 18 unra ke bachchom ke lie kailshiyama ki avashyakata prati dina 1300 miligrama taka hoti hai jo 4 se 4 1/2 kapa dudha ke (lagabhaga 1 litara) barabara hoti hai|
yaha dekhana bahuta asana hai ki sanyukta rajya amerika mem adhikansha kishorom ko hara dina paryapta kailshiyama kyom nahim milata, jabaki kailshiyama driढ़ikrita santare ke rasa, dahi, panira, aura hari pattedara sabjiyom jaise anya khadya padarthom mem bhi paya jata hai|
ayarana nayi, svastha lala rakta koshikaom ke utpadana mem sharira ki madada karata hai|
lala rakta koshikaem ऑksijana ko sancharita karati haim, isalie paryapta ayarana lena mahatvapurna hai|
kishora lada़kiyom aura mahilaom ko mahavari ke daurana rakta mem hui hani ki kshatipurti ke lie atirikta ayarana ki jarurata hoti hai|
lala mansa ayarana ka sabase achchha srota hai, lekina yaha ayarana se bharapura anajom, kishamisha, aura gahari hari pattedara sabjiyom mem bhi paya jata hai|
khadya lebala para vaha samagri bhi shamila karani chahie jo usa utpada mem maujuda hoti hai|
khadya padartha mem unaka kitana ansha shamila hota hai isi ke anusara samagri suchibaddha hoti hai|
agara apake parivara mem kisi ko khadya elarji hai to samagri suchi padha़na vishesha rupa se mahatvapurna hota hai|
elarji,asthama aura inyunolaॉji ki amerikana akadami ka anumana hai ki sanyukta rajya amerika mem 2 miliyana ya 8 % bachche khadya elarji se prabhavita haim aura 90% khadya elarji dudha,ande,mungaphali,gehum,soya,machhali,sipadara machhali aura sukhe mevom jaise atha khadya padarthom se hoti haim|
2006 ke bada se,khadya nirmataom ke lie khadya lebalom para (samagri ki suchi ke nikata ya usake bada) spashta rupa se yaha likhana avashyaka kiya gaya ki ina utpadom mem elarji paida karane vale tatva maujuda haim ya nahim|
kuchha mamalom mem, lebala para suchibaddha samagri ko janchane se yaha pahachanana asana ho jata hai ki khane ke lie kya
halanki, kuchha samagri ko jisase ki elarji ho sakati hai,eka aparichita nama se bhi suchibaddha kiya ja sakata hai (udaharana ke lie, arekisa tela, mungaphali tela ke lie eka aura shabda hai, jise mungaphali se elarji valom ko paraheja karana chahie )
eka ahara visheshajna sujava de sakata hai ki kina kina khadya padarthom se paraheja karana chahie evam chhipi hui samagri se savadhana rahana chahie|
saja parivara mem bhojana ke paushtika hone ki adhika sambhavana hoti hai aura jo niyamita rupa se apane parivarom ke satha hi khana khate haim una bachchom dvara nashte mem asvasthyakara bhojana karane ki sambhavana bahuta hi kama aura phala, sabjiyom aura sabuta anaja khane ki sambhavana adhika hoti hai|
adhyayanom se pata chalata hai ki jo niyamita rupa se parivara ke satha hi bhojana karate haim una kishorom mem dhunrapana,madirapana ya marijuana aura anya nashe ki davaom ke upayoga ki sambhavana bahuta hi kama hoti haim,aura ve vayaskom ki taraha hi svastha ahara adika lete haim| 
svasthya aura poshana ke alava,parivara ke satha bhojana karana parivara se joda़ne ke lie eka bahumulya avasara pradana karata hai|
jaba bachche bada़e ho jate haim to yaha aura bhi adhika mahatvapurna ho jata hai|
parivara ke satha bhojana karane ki adhika yojanaom ke lie, kailendara mem dekhakara eka aise samaya ka chayana karem jaba ghara mem sabhi loga maujuda hom| 
parivara ke satha adhika bara bhojana karane mem ho rahi badhaom jaise vyasta karyakrama, ghara mem samagri ka na hona, khana banane ke lie samaya na hona ka hisaba rakhie|
parivara se madada aura vicharom ke lie puchhem ki ina badhaom ko kaise hataya ja sakata hai|
udaharana ke lie, parivara ke satha bhojana ke lie kirane ka samana kharidane ke lie yojana banaem|
ya agara khana pakane ka samaya eka samasya hai,to saptahanta para kuchha taiyari karane ki koshisha kijie ya puri taraha eka disha bhi pahale se banakara aura use phrija़ra mem dalakara rakhane ki koshisha kijie|
eka bara jaba apake pasa sari samagri maujuda hai,to taiyari mem bachchom ko bhi shamila kijie|
chhote bachchom se madada lena thoda़a sa eka atirikta kama ho sakata hai, lekina aksara yaha phayademanda hota hai|
meja para pletem rakhana, salada banana,peya padartha dalana, naipakina taha karake rakhana, ya eka khana chakhane vale ke rupa mem hona jaise sarala karya skula na jane vale aura jane vale bachchom ke lie upayukta kama hote haim|
aura adhika sahayata karane mem bada़e bachche sakshama ho sakate haim jaise samagri lana,sabjiyam dhona, milana aura chalana aura khana parosana|
yadi apake pasa kishoravastha ke bachche haim, to unase eka rata ke lie khana pakane ka vichara banaem aura apa eka sahayaka ke rupa mem unake satha rahem|
yaham taka ki agara apa rata ke khane ke bada hone vale kamom (bartana sapha karana,dopahara ka khana banana adi) ko bhi pura karane ke bare mem socha rahe haim,to rata ke khane ke daurana usa para dhyana kendrita karane ki koshisha na karem|
khate vakta apane samaya ko khushanuma banae aura sabhi ko dina bhara ke tanava se chhutakara pane ka mauka dijie aura eka parivara ke rupa mem eka satha hone ka ananda lijie|
ho sakata hai apake bachchom ko bahuta bhukha laga rahi ho, lekina khana khane ke bajaya apane bachchom ko pratiksha karane dem jaba taka ki sabhi loga khane para na baitha jaem|
khane se pahale jaba taka ki khana meja़ para na a jae ,shanti ka mahaula banaye rakhem| 
aisa karane se khana pakane vale ka mana badha़ane aura dhanyavada dene ka , hara kisi ko eka achchhe bhojana ki shubhakamana dene ka, ya eka dusare ko dudha ka eka gilasa aura tosta age badha़akara prastuta karane ka eka mauka mila jata hai|
aisa karake apa eka achchha mahaula banate haim aura achchhe shishtachara aura dhairya ko rupa dete haim|
parivara ke satha bhojana karana sabhya vyavahara sikhane ke lie eka achchha samaya hota hai jise bachche restaram aura dusarom ke gharom para upayoga karate haim, isalie apa apani jagaha baithe rahana ,chija़om ko apane apa khane ki bajaya dusarom ko dena,naipakinom ko goda mem rakhana, aura munha bhara hone para bata na karana jaise niyama sikha sakate haim|
jaba ve niyama toda़em to apa shishtata se unhem yada dila sakate haim, lekina rata ke khane ke daurana tanava aura anushasana kama se kama rakhane ki koshisha karem|
apaka dhyana isa bata para kendrita rahana chahie ki apake bachche ko lagata rahe ki usaki paravarisha achchhi taraha ho rahi hai, vaha parivara se juda़a hua hai aura usaka eka hissa hai|
karbohaidreta ki hi taraha vasa para bhi ajakala bura hone ka aropa ,galata tarike se lagaya ja raha hai| halanki kuchha vasa dusarom se nishchita rupa se behatara haim, kuchha vishesha prakara ki vasa vastava mem apa aura apake bachche ke lie achchhi haim aura svastha ahara ka eka mahatvapurna hissa haim|
apake bhojana mem vasa,ya lipida,ve poshaka tatva hote haim jo apaka sharira tantrika utakom (mastishka ki taraha) aura harmonom ka nirmana karane ke lie upayoga karata hai| 
apaka sharira bhi vasa ka upayoga indhana ke rupa mem karata hai|
yadi vaha vasa jo apane khane mem grahana ki hai , urja ke rupa mem prayoga nahim hoti hai ya sharira ke nirmana mem istemala nahim hoti to sharira dvara vaha vasa koshikaom mem jama ho jati hai|
yaha apake sharira dvara age ke lie sochane ka tarika hai, bhavishya ke upayoga ke lie vasa ki bachata karake, apaka sharira usa samaya ke lie sochata hai jaba khana durlabha ho sakata hai|
vasa se bhojana ka svada aura banavata banati hai, lekina isamem kailori bhi atyadhika hoti hai, aura vasayukta khadya padarthom ki atyadhika matra se (kisi bhi bhojana ki atyadhika matra ki taraha hi) kai svasthya khatare paida ho sakate haim|
bachchom aura kishorom ke lie ,mithai aura nashte (alu ke chipsa, chaॉkaleta, keka, donata, pestri, aura kukija़ sahita) vasa ka eka mahatvapurna srota hote haim|
bachchom ko gadha़e dudha ke utpadom aura bekana, haॉta daॉga, aura charbi vale lala mansa jaise uchcha vasayukta mansom se bhi vasa prapta hoti hai|
nissandeha,phasta phuda aura bahara ke bhojana mem ghara mem bane khane se adhika vasa hoti hai ,aura restaram ke tale hue vyanjanom mem vasa samagri sabase adhika hoti hai|
vasa aksara malaidara, paniradara,ya makkhani saॉsa ya dresinga ke rupa mem khadya padarthom mem chhipi rahati hai|
phira bhi, halanki, vasa svastha ahara ka eka mahatvapurna hissa hoti hai lekina tabhi jaba apaka bachcha sahi prakara ki vasa (visheshatah kuchha machhaliyom mem) batai gai sahi matra mem leta hai|
halanki ajakala ke bachche sharirika rupa se pichhali pidha़iyom ki tulana mem bahuta hi kama svastha pae jate haim aura sanyukta rajya amerika mem 1976 ke bada se mote bachchom ka pratishata dogune se adhika ho gaya hai, nishchita rupa se akele vasayukta khadya padartha motape ki mahamari ke lie doshi nahim haim|
vyayama na karana ,hamesha baithe rahana (tivi dekhana, kampyutara para samaya bitana, aura vidiyo gema khelana), aura apani avashyakata se adhika khana , motape ki mahamari ko dosha dene ke lie shera jitana jyada khana bhi doshi haim| 
nihsandeha, achchhe svasthya ko badha़ava dene mem aura logom ke motape ko rokane mem aura eka svastha vajana ko banae rakhane mem madada ke lie kama vasa vale bhojana sujae gae haim|
halanki, bachchom aura vayaskom donom ke lie,eka svastha ahara mem vasa ki matra kitani honi chahie isa bare mem bhi bhrama hai|
protina ya karbohaidreta ki tulana mem vasa mem dugani kailori hoti hai (vasa ka 1 grama 9 kailori pradana karata hai, jabaki karbohaidreta aura protina donnom ke 1 grama 4 pratyeka grama kailori pradana karate haim)
isalie vasa urja ka eka bada़a srota hai, lekina bhojana mem vaha dugani kailori pradana karata hai|
halanki kuchha mata pita ki adata hoti hai ki ve kula milakara vasa ki katauti karane ki koshisha mem rahate haim ya jarurata se jyada use na lene ki pabandi lagate haim , phira bhi yaha vasa apake bachche ke ahara ka eka mahatvapurna hissa hai|
bure dabava ke bavajuda bhi vasa dushmana nahim hai|
vishesha rupa se, yuva bachchom ke lie, vasa aura kolestraॉla mastishka ke vikasa mem mahatvapurna bhumika nibhate haim|
aura 2 sala ki unra se kama bachchom ke lie to vasa lene para pratibandha nahim hona chahie|
2 varsha ki unra se, bachchom ko vividha ahara khane chahie jisamem 30- 35% kailori vasa se milani chahie|
4 se 18 sala ki unra ke lie, 25 -35 % kailori grahana karana uchita hai|
isa unra mem nanhem bachchom mem unake bhojana aura unaki khane ki adatom se parivartana hote haim jabaki shishuom ka ahara vaha hota hai jo apa apani ichchha se dete haim| 
apaka kama apane bachche ki jihva ko naye svada evam rupa se vikasita karana hai|
bhojana variyataem jivana mem jaldi hi sthapita ho jatim haim isilie svastha bhojana ke lie svada vikasita karane mem apane bachche ki madada kijie|
nanhem bachchom ke peta chhote hi hote haim, isalie unako vaha bhojana dijie jo una poshaka tatvom se bharapura ho jo unake svastha aura majabuta vikasa ke lie jaruri haim , aura mithai aura rikta kailori para pabandi rakhie|
apaka nanha pahale ungaliyom se aura phira lagabhaga 15 se 18 mahinom ki unra mem svayam hi bartanom se khana sikha jaega|
ina adatom ka abhyasa karane ke lie apane bachche ko kai avasara dijie, jaba pareshani ho to usaki madada ke lie taiyara rahem|
jaise hi ye adatem vikasita hone lagem , apa pichhe ho jaem aura bachche ko apane apa karane dem|
nanhem bachche bhi apani svatantrata para jora dena shuru kara dete haim, aura khane ki meja़ vaha jagaha hai jaham apako una para apana niyantrana rakhana chahie| 
apane bachche ko bhukha aura paripurnata ke antarika sanketom ke prati pratikriya dikhane dem ,lekina simaem jarura lagu karem|
bhojana mem paushtika khadya padarthom ki vividhata ko apa svayam taya karem aura apaka bachcha phaisala karega ki usako usa bhojana mem se kya khana hai , kitana khana hai, ya saba kuchha khana hai|
dudha nanhem bachchom ke ahara ka eka mahatvapurna hissa hota hai kyonki yaha kailshiyama aura vitamina di pradana karata hai jo haddiyom ko majabuta banane mem madada karata hai|
2 varsha se kama ayu ke bachchom ko samanya vikasa aura mastishka ke vikasa ke lie avashyaka ahariya vasa ke lie gadha़a dudha pina chahie|
jaba apaka bachcha 2 varsha ka ho jata hai, to apa use sambhavatah kama vasa ya bina vasa ka dudha de sakate haim, lekina aisa karane se pahale apane bachche ke daॉktara se salaha lena eka achchha vichara hoga|
12 aura 18 mahine ki bicha ki unra kapa se dudha pilane ke lie eka achchha samaya hoti hai|
sare samaya botalom se dudha pilane ko eka bara mem hi chhuda़ane ki bajaya,khane ke samaya se shuru karake apa dhire-dhira botala chhuda़a sakati haim| 
jaise hi bachcha bhojana lena shuru kara deta hai tabhi se apa use kapa se vasayukta gadha़a dudha de sakati haim|
yadi apa stanapana kara rahi haim,to kevala kapa se hi dudha pilaie aura botala ki adata bilkula bhi mata dalie | 
pahali bara mem kuchha bachchom ko gaya ka dudha pasanda nahim ata kyonki yaha stana dudha se alaga hota hai ya isake banane ka phaॉrmula bhi
yadi aisa hai,to vasayukta gadha़e dudha ko pharmula dudha mem ya stana dudha mem milana thika hota hai aura dhire - dhire mishrana ke anupata ko thika se aise milati rahem ki antatah yaha 100 % gaya ka dudha bana jae |
chunki aba apa kuchha anya khadya padartha jo apane abhi taka nahim die the (dudha,khatte phala,pure ande) dene shuru kara sakati haim,isalie koi bhi naya bhojana dete vakta elarji pratikriyaom ko avashya dekhem |
agara apake parivara mem kisi karibi sadasya ko khadya elarji hai to apane bachche ke daॉktara ko jarura bataie, aura ho sakata hai ki apa usa padartha ko aura amataura para una padarthom ko bhi jinase khadya elarji hoti hai, jaise mungaphali aura samudri khana ,kuchha dinom ke bada dena chahem |
ho sakata hai kuchha khadya padarthom ko shuru karane ke lie apako taba taka intajara karana pada़e jaba taka ki apaka bachcha 2 ya 3 sala ka na ho jae,ya jaba daॉktara elarji ke parikshana ka sujava de |
una khadya padarthom ko dene se bachem jinase gale mem atakane ki sthiti paida hone ka khatara ho ,jaise paॉpakaॉrna, kada़i kaindi, haॉta daॉga, kachchi sabjiyam aura kada़e phala,sabuta angura,kishamisha aura sukhe mevem adi| 
khate vakta apane bachche ka hamesha dhyana rakhem|
apane bachche ko dina bhara mem tina bara sampurna bhojana aura do ya tina bara nashta dem,lekina dhyana rakhem ki nanhem bachchom ke lie bhojana chhoda़ dena samanya si bata hai|
bachche dvara bhojana chhoda़ne dena kai mata pita ke lie eka kathina manobhava hota hai, lekina bachchom ko bhukha aura peta bhare hone ke apane svayam ke antarika sanketom ke prati pratikriya dikhane dena chahie |
yaha bata mahatvapurna hai ki agara bachcha bhukha nahim hai to use jabaradasti khana nahim khilana chahie |
dusari ora, bachche ko dina bhara khana mangate rahane bhi nahim dena chahie |
apane bachche ke prathama 3 mahinom ke daurana,sabhi avashyaka poshana pradana karane ke lie stana dudha ya pharmula dudha dena jari rakhem|
lekina jaise hi apaka shishu sharirika aura manasika rupa se vikasita hota hai,pine ki prakriya bhi vikasita hoti jati hai|
hara bara pilane para apaka bachcha samanya se adhika dudha pina shuru kara deta hai, isalie bara bara pilane ki ja़rurata nahim hogi aura aba vaha rata mem jyada dera taka bhi sona shuru kara deta hai|
lekina agale varsha ke daurana aura vishesha rupa se,jivana ke prathama 3 mahinom mem kai bara aisa hota hai jaba vikasa ki teji se apake bachche ki bhukha bhi badha़ti hai|
mangane para dudha pilati rahem aura avashyakata ke anusara dudha pilane ki sankhya bhi badha़aem |
jaise jaise haphte gujarate jaegem apaka shishu adhika satarka bhi hota jaega , dhimi avaja mem bolana shuru kara dega aura eka samajika muskana bhi dena shuru kara dega |
dudha pilane ki prakriya ke daurana shayada isilie apa aura apake bachche ke bicha parasparika sambandha aura adhika badha़ jata hai|
aura yahi vaha samaya hai jaba adhikanshatah shishu thosa ahara lena shuru kara dete haim|
amerikana ekedami ऑpha pidiyatrika (aapa)(amariki bala roga akadami) ne abhi hala mem sujava diya hai ki jaba bachcha 6 mahine ka ho jata hai to use dhire - dhire thosa ahara dena shuru kara dena chahie|
jaise bhi ho, apake bachche ki tatparata aura poshana sambandhi jaruratom ko dekhate hue apaka daॉktara aura bhi jaldi yani 4 mahine ka hone para bhi thosa ahara dene ki bata kaha sakata hai|
koi bhi thosa ahara shuru karane se pahale apane daॉktara se puchha lem|
ho sakata hai apake bachche ko thosa ahara lena sikhane ke lie kuchha samaya lage|
ina mahinom ke daurana apa stana dudha ya pharmula dudha bhi samanya rupa se pilati rahem ,agara apaka bachcha pahali bara mem kuchha khadya padarthom ko khane se mana kara de ,ya khane mem bahuta adhika dilachaspi na dikhae ,to bilkula bhi chintita na hom |
jaba apaka bachcha thosa ahara ke lie taiyara hai aura apake daॉktara ne bhi apani sahamati de di hai to dina mem eka samaya aisa chuna lijie jaba apaka bachcha thaka hua ya chida़chida़a nahim hota hai|
apa apane bachche ko thoda़i bhukha lagane dem, lekina itani bhi nahim ki vaha bhukha se mara raha ho, to aise samaya mem apa apane bachche ko stanapana kara sakati haim ,ya hamesha ki taraha botala se thoda़a dudha de sakatim haim|
apane shishu ko apani goda mem ya shishuom ki sidhe baithane ki sita mem bithaem|
jo shishu amataura para 6 mahine ke asapasa achchhi taraha se baithana shuru kara dete haim,unako suraksha patti bandhakara eka unchi kursi para bithaya ja sakata hai|
amataura para,bachche ka prathama ahara stana dudha ya pharmula dudha mem mishrita shishuom ka anaja thoda़e se ayarana vala chavala hota hai|
prathama ahara kisi tarala padartha mem puri taraha mila thoda़e se anaja se jyada kuchha bhi nahim ho sakata hai|
apane bachche ke honthom ke pasa chammacha rakhem, aura bachche ko bhojana ki gandha aura svada lene dem|
agara yaha pahali chammacha asvikara ho jati hai to hairana na hoem |
isa unra mem bachche ko diya gaya adhikansha bhojana usaki thoda़i, biba, ya unchi kursi ki tre para hi gira jata hai| 
apane bachche ki botala mem dudha ke satha koi bhi anaja taba taka na milaem jaba taka apake bachche ka daॉktara apako aisa karane ke nirdesha na de, kyonki aisa karane se bachche ka vajana badha़ sakata hai aura bachcha thosa ahara khana nahim sikha paega| 
eka bara jaba apaka shishu chammacha se anaja khana banda kara de, to usa samaya apa eka phala ya sabji dena shuru kara sakati haim|
eka samaya mem eka hi taraha ka bhojana dem aura kuchha bhi naya dene se pahale kai dinom taka intajara karem| 
aisa karane se apako pata chalega ki kauna se khadya padartha se apake bachche ko elarji ho sakati hai| 
ama taura para kuchha khadya padarthom se elarji kaphi bada taka bhi ho sakati hai|
bachche ke pahale janmadina ke bada taka bhi ande, gaya ka dudha, khatte phala aura jusa, aura shahada mata dem|
visheshakara agara bahuta chhoti unra mem diye jaem to ando (vishesha rupa se sapheda bhaga) se bhi elarji ho sakati hai|
khatti chija़em atyadhika anliya hoti haim jisase bachche ko dardanaka dayapara chakatte (elarji) ho sakate haim|
shahada mem kuchha vishesha prakara ke bijanu ho sakate haim , jo vayaskom ke lie to hanikaraka nahim hote lekina bachchom mem una bijanuom se khadya vishaktata (botulija़ma) ho sakati hai|
gaya ke niyamita dudha mem shishuom ke lie jaruri poshana nahim hota hai|
machhali aura samudri bhojana, mungaphali aura mungaphali ke makkhana, aura peda़ ki giriyom se bhi shishuom mem elarji ho jati hai,aura ye saba taba taka nahim dena chahie jaba taka bachcha 2 ya 3 sala ka nahim ho jata ,aura yaha isa bata para bhi nirbhara karata hai ki bachche ko khadya elarji ke prati khatara adhika hai ya nahim|
bachche ko khadya elarji ka khatara tabhi adhika hota hai jaba agara parivara ke eka ya adhika karibi sadasyom ko elarji ho ,ya khadya elarji, ekjima, asthama jaisi elarjiyom se sambandhita sthitiyam ho|
adhika se adhika vitamina aura khanija lavana banae rakhane ke khana pakane ke tarikom se apane bachche ke bhojana mem poshaka tatvom ko banae rakhane ki koshisha kijie|
phalom aura sabjiyom ko ubalane ki bajaya bhapa se ya beka karake pakaem kyonki ubalane se unake poshaka tatva nashta ho jate haim|
jo khadya padartha apa abhi upayoga nahim kara rahe haim unhem dibbe mem banda karake rakhane ki bajaya phrija़ mem rakha dem|
jaba apa bachche ko alaga-alaga khadya padarthom ko khane ke lie saphalatapurvaka de chuke haim ,to aba do khadya padarthom ka pisa hua mishrana de sakate haim|
jaba apaka bachcha 9 mahine ka ho jata hai, to vaha danedara,tukada़edara bhojana khane yogya ho jata hai kyonki bhojana ke prati usamem parivartana hone shuru ho jate haim aura aba vaha meja para parosa gaya bhojana adhika lene lagata hai|
yadi apa bachche ko vyavasayika taura para taiyara kiya gaya dibbabanda ahara de rahe haim, to bachche ko khilane ke lie kuchha chammachabhara bhojana eka katori mem parosem|
apane bachche ko sidhe jara se bhojana na khilaem , kyonki ho sakata hai ki bachche ke munha ke baiktiriya shesha bhojana ko bhi dushita kara dem|
bachche ke khule jara ke bhojana ko eka ya do dina ke bhitara hi nipatana bhi eka buddhimani ka kama hoga|
6 mahine ki unra ke bada phalom ka rasa diya ja sakata hai, jo ki bachche ko kapa mem dena shuru karane ki eka thika unra hai|
bada़e haindala ka aura dhakkanadara pinevala eka kapa kharidem , aura apane bachche ko use thika se pakada़na aura usase pina sikhaem|
ho sakata hai,apane bachche ke lie eka uchita kapa khojane ke lie apako kuchha alaga alaga taraha ke kapa dekhane pada़em|
gandagi ko sapha karane se bachane ke lie sabase pahale pani ka upayoga karem|
phalom ka kevala 100 % rasa hi dem, na ki rasa ka koi peya padartha ya paudara vala koi anya rasa |
rasa botala mem na dem aura eka dina mem kula milakara 4 aunsa (120 miligrama) se kama hi dena yada rakhem |
bahuta jyada rasa dene se stana dudha ya pharmula dudha ke poshana ke bina hi bachche mem atirikta kailori badha़ jati hai| 
bahuta adhika rasa pine se vajana adhika badha़ jata hai aura dasta bhi ho sakate haim|
amataura para shishuom ko gajara aura shakarakandi jaise phala aura mithi sabjiyam achchhi lagati haim ,lekina anya sabjiyom ki bhi upeksha na karem|
agale kuchha mahinom ke daurana apaka lakshya bhojana mem vibhinna prakara ke khadya padarthom ko shamila karana hai|
yadi apake bachche ko koi vishesha bhojana achchha nahim laga raha hai, to use bada vale bhojana ke satha punah arambha karem| 
isase pahale ki apaka bachcha kuchha khadya padarthom ke lie apani ruchi dikhae ,ho sakata hai ki isake lie apako kaphi kuchha koshisha karani pada़e|
jyadatara bachche 8 mahine ke hote hote , shishuom vale ayarana se bharapura una anajom aura pisi sabjiyom aura phalom ke adi ho jate haim jo stana dudha ya pharmula dudha ke satha unake ahara mem dene shuru kara die jate haim| 
agale kuchha mahinom mem, apaka bachcha meja para parose gae khadya padarthom ko khana shuru kara denga|
jaise jaise apa apane bachche ki jibha ka svada badha़a rahim haim , to kisi bhi khadya padartha se elarji pratikriyaom ke lie dekhane ke lie nae nae khadya padarthom ka jancha parikshana ( kuchha dinom se lekara eka saptaha taka ke lie )jari rakhem|
apane bachche ko pura anda, khatte phala, machhali aura samudri bhojana, sukhe meve (mungaphali aura mungaphali ke makkhana sahita), ya shahada mata dem|
isa avastha parivartana ke daurana, ho sakata hai apa mansa khilana shuru karana chahem aura naye taraha ka daradara bhojana dena chahem jise use adhika chabane ki avashyakata pada़egi|
apa shishu ahara bhi kharida sakati haim jisase apake shishu ko nae svada aura nayi banavata vala bhojana milega,ya jo kuchha bhi parivara ke baki logom ke lie apa bana rahim haim vahi bhojana kante se masalakara ,katakara ,ya pisakara bachche ko de sakatim haim|
apa ise thoda़i jyada dera taka taba taka pakaem , jaba taka ki yaha bahuta narama na ho jae, aura use chhote chhote tukada़om mem kata dem taki apaka bachcha use chaba chabakara khae jisase gale mem atakane ka khatara kama ho sakega|
jaba bachche lagabhaga 9 mahine ke ho jate haim, to amataura para ve tarjani aura anguthe ke bicha bhojana pakada़ne ke talamela mem nipuna ho jate haim aura ve apani ungaliyom se khuda khane ki koshisha karane lagate haim|
jaba apa nae nae khadya padartha dene shuru kara rahe haim to apane bachche ke svabhava ko hamesha dhyana mem rakhem|
yadi nayi banavata vale bhojana se usake pratirodha ka samana karana pada़ raha hai, to unhem thoda़a thoda़a bhojana parosem aura usa bhojana ke satha use mila dem jo apake bachche ko pasanda hai|
jo bachche jaldi hi bahuta uttejita ho jate haim ,unako chammacha se havai jahaja ki taraha khelakara munha mem khana khana bahuta achchha lagata hai|
jabaki adhika samvedanashila bachche ko khana khilane mem, nyunatama manabahalava ki avashyakata ho sakati hai|
stana dudha se aura shishuom ke pharmula dudha se badha़te shishuom ko mahatvapurna poshaka tatva milate rahate hai, lekina eka sala ke hote hote ve kama pina shuru kara dete haim|
adhika poshaka tatva aba unako anya vividha prakara ke khadya padarthom se mila rahe haim jo unhonne khane sikha lie haim aura ve unako pasanda bhi karane lage haim| 
ho sakata hai apa chintita ho jaem ki apa apane bachche ko bahuta adhika khila rahe haim ya paryapta nahim khila rahe haim|
apane bachche ke sanketom ki ora dhyana dijie ki vaha bhukha to nahim hai ya usaka peta bhara hai|
agara bachche ka peta bhara hai to vaha bemana se dudha piega, isalie turanta apana dudha pilana banda kara dem,ya botala hai to use hata dem|
thosa khadya padartha dene para,ho sakata hai apaka bachcha apana munha phera le,ya apana munha hi nahim khole, ya bhojana ko bahara ugala de| 
jaba apa asali mem bachche ko khana khila rahim haim to use apani ungaliyom se ya chammacha se khane dem|
shishu unra ki yaha achchhi taiyari hai jaba apaka bachcha svayam khana sikha jata hai| 
aura yadi apane pahale se hi niyamita rupa se khane ke lie adhika samaya ka nirdharana nahim kiya hai|
amariki bala roga akadami (aapa) aura anya peshevara samuha jo navajata shishuom ki dekhabhala ke lie chintita hai ,stanapana ko sabase achchha batate hue use karane ka sujava dete haim|
aapa ka manana hai ki vishesha rupa se prathama 6 mahinom taka shishuom ko kevala stanapana hi karana chahie|
thosa ahara dete hue bhi bachche ke prathama varsha ke daurana stanapana karate rahana chahie evam agara age bhi ichchhuka hom to karati rahem|
ho sakata hai ki stanapana karana sabhi mahilaom ke lie sambhava ya behatara na ho|
samanyatah dudha ke lie bachcha mam ki sahuliyata para nirbhara hota hai ki mam use stanapana karaegi ya phira botala se dudha pilaegi|ho sakata hai ki kuchha maom aura bachchom ke lie stanapana thika hi na ho|
agara apa ke mana mem stanapana ko lekara kuchha sandeha hai ki apa apane bachche ko stanapana kara sakati haim ya nahim ,to apane daॉktara se puchhem|
yada rakhem,chahe apa stanapana karaem ya pharmula dudha dem, apake bachche ki bhavanatmaka evam poshana sambandhi sabhi avashyakataem puri honi chahie |
vyavasayika rupa se taiyara kie gae shishu ahara stanapana ke paushtika vikalpa hote haim|
botala se dudha pilana mam ke lie adhika svatantrata evam anukulata pradana karata hai evam isase asani se pata chala jata hai ki bachcha kitana dudha pi pa raha hai| 
chunki pharmula dudha dhire dhire pachata hai isalie jo bachche pharmula dudha pite haim ,unhem stanapana karane vale bachchom ki apeksha kama bara pine ki jarurata hoti hai|
pharmula dudha ke hone se sarvajanika sthalom para dudha pilana asana ho jata hai aura pita evam parivara ke anya sadasyom dvara bhi bachche ko dudha pilane mem madada mila jati hai jisase parivarika sambandha bhi majabuta hote haim|
kuchha bachche - yaham taka ki sabase adhika poshana satarka – kahenge ki ve eka achchhe resha-sanriddha bhojana ke lie tarasate haim|
halanki reshe ki socha sabhi unra ke bachchom mem pratibandhom aura ahom ko prakata kara sakate haim, phira bhi, vastava mem kai phalom se sampurna-anaja siriyala taka ke kshudhavardhaka khadyom ki prachurata reshe ke achchhe srota haim|
aura sambhavatah apaka bachcha unhem bina jane hue bhi kha raha hai|
kevala varishtha nagarikom ki bhida़ ke lie hi nahim, khadya jo reshe ke achchhe srota haim, labhaprada haim kyonki ve garishtha hote haim aura, isilie, adhika khane ko nirutsahita karate haim - halanki resha svayam bhi koi kailori nahim badha़ata hai|
isake atiriktta, jaba yaha paryapta dravom ko pine, uchcha-reshedara bhojana khane ke satha sammilita kiya jata hai, to bhojana ko pachana tantra mem sarakane mem sahayata karata hai aura anta kainsara tatha kabja़ se bachata hai|
bena ke dopahara ke bhojana ko khana prarambha karane ke kuchha dera bada hi, usake shishupalana poshanakarta ne dhyana diya ki vaha apane munha mem khujali ko kharochane ki koshisha karate hue dikha|
usake ulati karane aura ja़ora-ja़ora se sansa lena prarambha karane ke pashchat, usa palana poshanakarta ne bena ke lie chikitsiya upachara manga, jo bada mem eka khadya pratyurjata (elarji) ke satha nidana kiya gaya, isa mamale mem mungaphali se|
dudha, ande, soya, gehum, peda़om ki giriyom, aura ghonghe (sipadara machhali) ke satha-satha, mungaphali sabase samanya bhojanom mem se haim jo pratyurjataem (elarji) utpanna karate haim|
eka khadya pratyurjata (elarji) se, sharira aise pratikriya karata hai ki vishishta khadya utpada hanikaraka hai|
jisake parinamasvarupa, sharira ka pratirakshi tantra (jo sankramana evam roga se lada़ta hai) isa khadya pratyurja, khadya mem upasthita vaha padartha jo pratyurjata (elarji) ko prarambha karata hai, se lada़ne ke lie roga-pratikaraka utpanna karata hai|
agali bara vyakti jaba ise chhune ya khane ya isake kanom ko sansa lene se usa khadya ke samparka mem ata hai, to sharira svayam ki suraksha karane ke lie eka histamaina namaka sahita rasayanom ko sravita karata hai|
ye rasayana pratyurja lakshanom ko prarambha kara dete haim jo shvasana tantra, jatharantra patha, tvacha, ya hridavahini tantra ko prabhavita kara sakata hai|
ina lakshanom mem bahati naka, khujalahatadara tvacha ke dadore, jibha, honthom, ya gale mem siharana, sujana, peta darda, ya gharagharahata bhi shamila ho sakate haim|
chikitsaka kisi anya rogom ke lie bhi parikshana karega jo ye lakshana utpanna kara sakate haim|
udaharana ke lie, yadi apake bachche ko dudha pine ke bada dasta hai aisa lagata hai, to chikitsaka yaha dekhane ke lie parikshana kara sakata hai ki banisbata khadya pratyurjata (elarji) ke dugdhasharkara ki asahishnuta karana ho sakata hai|
selieka roga - eka roga jisamem vyakti ate ke lasa (glutena), eka protina jo gehum aura kuchha anya anajom mem paya jata hai, ko sahana nahim kara sakata hai – aura yaha khadya pratyurjataom ke lakshanom ka anukarana bhi kara sakata hai|
yadi chikitsaka ko eka khadya pratyurjata (elarji) ki ashanka hoti hai, to sambhavatah apako eka pratyurjata (elarji) visheshajna ke pasa bheja jayega, jo aura adhika prashna puchhega, sharirika parikshana karega, aura roganidana karane mem sahayata karane ke lie sambhavatah parikshana karega|
parikshana mem khadya pratyurjatotpadakom ke drava sarom ko apake bachche ki agrabhuja ya pitha mem, tvacha ko chheda karake rakhana, aura yaha dekhane ke lie intaja़ara karana ki 15 minata ke andara lala uthe hue daga (jo sphota kahalate haim) banate haim ki nahim, shamila hai|
eka vishesha khadya ka sakaratmaka parikshana kevala yahi darshata hai ki apaka bachcha usa khadya ke prati pratyurja ho sakata hai - pratyurjata visheshajna satyapana ke lie atirika parikshana bhi kara sakata hai|
khadya pratyurjata se apake bachche ka roganidana karane ke pashchat, pratyurjata visheshajna upachara yojana banane mem apaki sahayata karega|
koi bhi aushadhi khadya pratyurjataom ka upachara nahim kara sakata hai, atah upachara ka samanyatah artha hai, isa pratyurjatotpadaka aura ve sabhi khadya jinamem yaha hota hai, se bachana|
apako khadya suchaka patrom se svayam ko parichita karane ki avashyakata hogi taki apa isa pratyurjatotpadaka se bacha sakem|
2006 se, khadya suchaka patra lagane ke eka naye kanuna ne ise thoda़a asana bana diya hai|
paiketa khadyom ke nirmataom ko isamem ya isake nikata ke upadanom ki suchiyom, isa utpada mem dudha, ande, machhali, sipamachhali(ghongha), peda़om ki giriyam, mungaphali, gehum, ya soya hai ki nahim, spashta rupa se vyakta karane ki avashyakata hoti hai|
halanki khadya pratyurjataom ke lie koi upachara nahim hai, phira bhi aushadhiyam gauna aura gambhira donom lakshanom ka upachara kara sakati haim|
pratyurja pratikriya se sambandhita shitapitti, bahati naka, ya peta darda jaise lakshanom ke upachara ke lie pratihistamainom ke upayoga kie ja sakate haim|
yadi apake bachche ko khadya pratyurjata ke parinamasvarupa sansa ki gharagharahata ya dama ki bhada़kanem (daure bhi kahalate haim) haim to chikitsaka sambhavatah yaha sipharisha karega ki eka shvasanali vispharaka jaise ki elbutiraॉla (jise eka hatha mem pakada़ne vale pampa upakarana se sansa liya ja sakata hai) ko shvasana ki kathinaiyom ko kama karane ke lie turanta liya ja sakata hai|
yadi apa eka dushtoshaniya (bahuta mina mekha nikala kara chunane vala) khane vale ko pala-posakara bada़a kara rahe haim, to apako chinta ho sakati hai ki apake bachche ko uchita vriddhi aura vikasa ke lie avashyaka poshaka tatva nahim mila raha hai|
kintu sambhavatah apako chinta karane ki jarurata nahim hai jaba taka apaka chikitsaka apako yaha nahim kahata hai ki apaka bachcha usa unra sima ke lie samanya dara se vriddhi nahim kara raha hai
samaya ke satha, adhikatara naka chadha़akara khane valom ko bhi unaki avashyakataom ki purti karane ke lie paryapta kailori aura poshaka tatva mila jate haim|
bhukha sharira ka sanketa hai ki ise bhojana ki avashyakata hai|
eka bara jaba hama hamare sharira ki avashyakataom ko pura karane ke lie paryapta bhojana kara lete haim, to bhukha chali jati hai jaba taka ki hamara peta phira se khali nahim ho jata hai|
kuposhana bhukha ki taraha vahi chija nahim hai, halanki ve aksara satha-satha chalate haim| 
loga jo kuposhita hote haim, unamem uchita svasthya aura vikasa ke lie avashyaka poshaka tatvom ki kami hoti hai|
koi samaya ki eka lambi ya thoda़i avadhi ke lie kuposhita ho sakata hai, aura yaha sthiti halki ya gambhira ho sakati hai|
loga jo kuposhita haim unake bimara hone ki adhika sambhavana hoti hai aura, gambhira mamalom mem, mara bhi sakate haim|
durbhagyavasha, duniya mem aise lakhom loga haim jinhem adhikansha samaya khane ke lie paryapta nahim milata hai aura kuposhana ke khatare mem rahate haim|
dirghakalina bhukha aura kuposhana mahattvapurna svasthya samasyaem utpanna kara sakata hai|
loga jo hara samaya bhukhe rahate haim unaki apane akara ke eka ausata vyakti ki apeksha ullekhaniya rupa se kama vajana hote hue, samanya se kama vaja़na vale hone ki sambhavana hoti hai|
unaki vriddhi unhem ausata se bahuta adhika chhota banate hue, ruka bhi sakati hai|
duniyabhara mem, 5 varsha se kama unra ke bachchom ka adhika se adhika 27% samanya se kama vaja़na vale haim|
loga jinhem paryapta bhojana nahim milata hai aksara bhukha ka anubhava karate haim, aura bhukha lambi avadhi mem kuposhana ki ora agrasara hoti hai|
kintu koi kai karanom se kuposhita ho sakata hai jinaka bhukha se kuchha lena dena nahim hota hai|
yaham taka ki ve loga bhi jinake pasa khane ke lie prachura hai, kuposhita ho sakate haim yadi ve aisa bhojana nahim khate haim jo uchita poshaka tatva, vitamina, aura khanija pradana karata hai|
yadi kisi vyakti ko eka vishesha poshaka tatva ki paryapta matra nahim milati hai, to vaha kuposhana ka eka rupa hai(halanki isaka avashyaka rupa se yaha artha nahim hai ki vyakti gambhira rupa se bimara ho jaega)|
vishva mem kuposhana ka sabase samanya rupa lauha ki kami hai, jo sansara ke 80% janasankhya ko prabhavita karati hai - adhika se adhika 4 se 5 biliyana (4 se 5 araba) loga|
lauha lala mansa, andapitom, aura pushtikrita ate, breda, aura siriyala jaise khadyom mem paya jata hai|
sansara mem sarvatra, jo loga nirdhana haim ya jo nirdhanata-akranta kshetrom mem rahate haim, ve bhukha aura kuposhana ke sabase adhika khatare mem haim|
nirdhana deshom mem, prakritika apadaem jaise ki sukha aura bhukampa bhi samanya khadya utpadana aura vitarana ko asta-vyasta karake bhukha aura kuposhana mem yogadana kara sakate haim|
yunaiteda stetsa mem, khadya utpadaka vishishta paushanika kamiyom ko rokane ke lie kuchha samanya khadyom ko vitaminom aura khanijom se pushta karate haim|
udaharana ke lie, namaka mem ayodina ki anuvriddhi kuchha avatu (thairaida) granthi ki samasyaom (jaise ki galaganda) ko rokane mem sahayata karati hai, bhojana mem milaya gaya pholika anla vishesha janma doshom se bachane mem sahayata kara sakata hai, aura atirikta lauha lauha-nyunata ki raktalpata ko rokane mem sahayata kara sakata hai|
kuposhana hara unra ke logom ko prabhavita karata hai, halanki shishuom, bachchom aura kishora/kishoriyam sabase adhika pida़ita hote haim kyonki kai poshaka tatva vriddhi aura vikasa ke lie upayogi haim|
adhika vriddha vyakti mem kuposhana ho sakata hai kyonki unravriddhi, bimari, aura anya karaka kama bhojanechchha ki ora agrasara kara sakate haim, atah ve paryapta kha nahim sakate haim|
madira, poshaka tatvom ke avashoshana mem badha dala sakata hai, atah madyavyasaniyom ko ve jina vitaminom aura khanijom ka upabhoga karate haim, unase labha nahim bhi ho sakata hai|
jo loga draga aura madira ka durupayoga karate haim yadi ve thika se nahim khate haim to ve kuposhita ya samanya se kama vajana vale ho sakate haim|
vishesha ahara le rahe bachche aura kishora/kishoriyom – jaise ki shakahariyom – ko uchita poshaka tatva pane ke lie santulita bhojana aura kai prakara ke khadyom ko khane ki avashyakata hai|
shakahariyom aura shuddha shakahariyom (viganom, jo mansa ke atirikta dudha ya dugdha utpadom ko bhi nahim lete haim) ko yaha sunishchita karana chahie ki unhem paryapta protina aura vitamina jaise bi12 mile|
yadi eka garbhavati mahila kuposhita hai, to janma ke samaya usake bachche ka kama vajana ho sakata hai aura jivita rahane ki bhi sambhavana kama ho sakati hai|
vikasashila deshom mem kuposhana se vitamina e ki kami nivarya andhepana ka mukhya karana hai, aura gambhira vitamina e ki kami vale bachchom mem bimara hone ya sankramanom jaise ki dasta ya khasare se marane ki aura adhika sambhavana hai|
kuposhana ka eka anya rupa, ayodina ki kami, manasika mandana aura dhime vikasa ka karana ho sakati hai|
lauha ki kami bachchom ko kama sakriya aura dhyana kendrita karane mem kama sakshama bana sakati hai|
jo kishora/kishoriyam kuposhita hoti haim unhem skula mem sikhane mem aksara musibata hoti hai|
saubhagyavasha, kuposhana ke kai hanikaraka prabhavom ko ulata ja sakata hai, vishesha rupa se yadi bachcha halka ya kuchha samaya se kuposhita hai|
kuposhana ka upachara isake karana para nirbhara karata hai|
eka chikitsaka ya ahara-visheshajna apaka bachcha jo bhojana khata hai usake prakarom evam matraom mem vishishta parivartanom ki salaha de sakata hai|
yadi isa kuposhana ko utpanna karane vali eka antarnihita samasya hai, to chikitsaka apake bachche ko avashyaka poshaka tatva mile yaha sunishchita karane ke lie tarike dhundhane mem apaki sahayata karega|
mata-pita ko aksara yaha chinta hoti hai ki jo bachche pinata batara saindavicha (mungaphali ke makkhana ka saindavicha) mem jite hue se lagate haim ya sabjiyom ko dekhane matra se chhipate haim, svasthya bane rahane ke lie shayada paryapta nahim khate haim| 
yunaiteda stetsa aura anya vikasita deshom mem kuchha bachche gambhira kuposhana jo tisari duniya ke deshom mem dikhai pada़ta hai, usi taraha ka anubhava karate haim|
yaham taka ki naka chadha़akara khane valom ko bhi samanyatah paryapta kailori aura poshaka tatva milate haim|
mata-pita ke lie bachchom ka thika se poshana ho raha hai yaha sunishchita karane ka sabase achchha tarika hai ki svasthyavardhaka khadyom ke vibhinna prakarom ko parosem aura asvasthyakara alpaharom ko simita karem|
yadi apako chinta hai ki apake bachche ki urja stara kama ho raha hai ya ki vaha usi unra ke anya bachchom jitani jaldi nahim badha़ raha hai, to apake chikitsaka se apaki chinta bantem|
lauha eka aisa poshaka tatva hai jo eka bachche ke niyamita ahara ka eka avashyaka bhaga hai aura lala rakta koshikaom (arabisi) ke aksijana-vahaka ghataka, rudhira-varnika (himoglobina) banane ke lie avashyaka hai|
lala rakta koshikaem sampurna sharira mem isaki sabhi koshikaom taka aksijana pahunchane ke lie parisancharita hoti hai| 
paryapta lauha ke bina, sharira paryapta arabisi nahim bana sakata hai, aura sharira ke utakom aura angom ko aksijana nahim milega jisaki unhem achchhi taraha se kama karane ke lie avashyakata hoti hai|
atah bachchom aura kishora/kishoriyom ke lie yaha mahattvapurna hai ki unhem unake dainika ahara mem paryapta lauha mile| 
lauha ki kami (jaba sharira ke lauha bhandara kama hone lagate haim) kuchha bachchom, vishesha rupa se nanhem bachchom aura kishora/kishoriyom (khasakara una lada़kiyom ko jo bahuta adhika masika dharma ka anubhava karati haim) ke lie eka samasya ho sakati hai|
vastava mem, kai kishoriyam lauha ki kami ke khatare mem haim - yadi unhem samanya masika dharma hota hai taba bhi - yadi unake ahara mem masika raktastrava ke daurana hui lauhayukta lala rakta koshikaom ki kshati ki purti karane ke lie paryapta lauha nahim hota hai to|
aura, kishora/kishori vyayami atyadhika vyayama karate samaya prasvedana aura anya margom se lauha khote haim|
12 mahine ki unra ke bada, nanhem bachche lauha ki kami ke khatare mem hote haim kyonki ve lauha-pushtita shishu-ahara aba aura nahim pite haim athava shesha ko pura karane ke lie shayada lauha-pushtita shishu siriyala ya paryapta anya lauhayukta khadya nahim khate honge|
lauha ki kami bachche ki vriddhi ko prabhavita kara sakata hai aura sikhane ki tatha vyavahara-sambandhi samasyom ki ora agrasara ho sakati hai|
aura yaha lauha ki kami vali raktalpata mem vikasa kara sakati hai, eka roga jisamem sharira mem lala rakta koshikaom ki sankhya mem kami hoti hai|
lauha ki kami vali raktalpata vale kai logom mem koi sanketa aura lakshana nahim hote haim kyonki sharira ki lauha apurti dhire-dhire kama hoti hai|
isamem koi ashcharya nahim hai ki mata-pita ko svasthyavardhaka khane ka kya artha hai yaha samajane ke lie kuchha sahayata ki avashyakata hai|
ahara margadarshaka piramida se phuda phaida (prachalita ahara jo avashyaka rupa se paushtika na ho) taka, yaha bahuta pechida ho sakata hai|
achchhi kha़bara yaha hai ki apako eka svastha bachcha palane-posane ke lie poshana mem kisi stara ki avashyakata nahim hai| 
eni ke lie pahali kaksha paravarisha ka eka kathina varsha raha|
usaka 6 varsha ka putra, jastina, apane pinata batara saindavichom (mungaphali ke makkhana ke saindavichom), pinata batara kraikarom (mungaphali ke makkhana ke kraikarom), aura apane sansadhita khadyom mem chhipe una sabhi mungaphaliyom se sashastra shatakom vidyarthiyom ke satha kaipheteriya mem aparahna bhojana karana prarambha kara diya|
jastina ke lie, jise mungaphali se atyadhika gambhira pratyurjata (elarji) hai, isaka artha hai mungaphalirahita meja para baithana|
yuesa phuda enda draga edaministreshana (ephadie)(khadya tatha aushadha prashasana) akalana karata hai ki 3varsha ki unra se kama ke bachchom ke 6% ko unhem ghara para ya, aura adhika kha़taranaka rupa se, ghara se dura pratyurja pratikriya ke khatare mem rakhate hue kisi na kisi prakara ki pratyurjata (elarji) hai|
mungaphaliyam sabase samanya pratyurjata (elarji) utpanna karane vale khadyom mem se haim|
kintu, kyonki mungaphali ki pratyurjata (elarji) ki anya khadyom ki pratyurjata (elarji) se adhika badha़ jane ki kama sambhavana hoti hai, yaha bada़e vachchom aura vayaskom ke bicha adhika samanya ho jata hai|
yaha sambhava hai ki adhika amariki kisi anya khadyom ki apeksha mungaphali ke prati pratyurja haim|
mungaphaliyam vastava mem vastavika kashthaphala nahim haim, kintu eka phali hai (matarom aura masurom ki taraha usi varga mem haim)| 
jaba mungaphali ki pratyurjata (elarji) vala koi mungaphaliyom ke samparka mem ata hai, to pratirakshi tantra dhokhe se yaha vishvasa karata hai ki mungaphali ke protina (ya pratyurjatotpadaka) sharira ke lie hanikaraka haim|
pratirakshi tantra inyunoglobyulina i (aijii) namaka roga-pratikarakom ko utpanna karata hai jo taba sharira mem raktapravaha mem rasayanom ko sravita karane ke lie pratyurjata (elarji) koshikaem (masta koshikaem kahalati haim) utpanna karate haim, jinamem se eka histamaina hai|
yaha histamaina taba vyakti ki ankhom, naka, gale, phephada़om, tvacha, ya jatharantra patha para kama karata hai, aura pratyurja pratikriya ke lakshanom ko utpanna karata hai|
mungaphali ki pratikriyaem, yaham taka ki bahuta kama matra ke samparka se bhi atyadhika gambhira ho sakati haim|
yaha isalie ho sakata hai kyonki pratirakshi tantra mungaphali ki protina ko anya khadya protinom ki apeksha adhika asani se pahachana leta hai|
mungaphali ke pratyurjatotpadaka sanrachana mem peda़ ke kashthaphalom ke pratyurjatotpadakom ke samana hote haim|
yaha spashta kara sakata hai ki kyom lagabhaga adhe logom ko jo mungaphali ke prati pratyurja haim, ve peda़ ke kashthaphalom, jaise ki badamom, braja़ila natom, akharotom, pingala phalom, maikadimiyom, pishtom, bhidurakashtha phalom, aura kajuom ke prati bhi pratyurja hote haim|
jo loga eka vriksha kashthaphala ke prati pratyurja hote haim prayah kama se kama eka ya do anya vriksha kashthaphalom ke prati bhi pratyurja hote haim|
mungaphali ki taraha, vriksha kashthaphala ki pratikriya bhi atyadhika prachanda ho sakati hai, yaham taka ki thoda़e se samparka se bhi|
shodha ne darshaya hai ki mungaphali ghataka khadya pratyurjata (elarji) pratikriyaom ki #1 ki doshi hai|
yadi apa apane ghara mem kashthaphalom ki anumati dete haim, to prati-samparkavikara ke lie dekhate rahem jo baratanom aura khana pakane ke baratanom se ho sakata hai|
udaharana ke lie, yaha sunishchita karem ki jo chaku apa pinata batara saindavicha (mungaphali ke makkhana ka saindavicha) banane ke lie upayoga karate haim, kashthaphala pratyurjata (elarji) vale eka bachche ka bhojana banane ke lie usaka upayoga na kiya jaya, aura vaha kashthaphala breda anya breda ki taraha usi tostara mem na senka jaya|
pake bhojanom, jise apane svayam nahim banaya hai ya upadanom ki ajnata suchi vale kuchha bhi ko apa na parosem|
skula ke kaipheteriya karmachari varga ke vetara aura mahila vetara se lekara hara eka se isa pratyurjata (elarji) ke bare mem kahie jo apake bachche ke khane ke bhojana ki dekha-rekha karata hai|
yadi kisi restaram ke prabandhaka ya malika ko mungaphali - ya kashthaphala rahita bhojana ke lie apaka anurodha kashtaprada lagata hai, to vaham mata khaie|
apane bachche ke skula ke aparahna bhojana, ke atirikta alpahara aura parti, khelane ki tithi, parti adi ke lie ghara se bahara rata bitate samaya, skula ke karyakramom, aura anya sairom mem le jane ke lie bhojana banane ki sochem|
apake bachche ke jane se purva shishu sadana ke paryavekshaka ya skula ke pradhanacharya se bata karem|
khadya pratyurjata (elarji) apatakala karya yojana banane ke lie satha-satha kama karem|
epainephrina ko hara samaya sulabha rakhem – apake kara ke dastana khane mem nahim, kintu apake satha, kyonki eka tivragrahita (eniphailaiksisa) ki ghatana ke samaya dusara mahattva hota hai|
thoda़i si taiyari, aura rokathama ke satha, apa sunishchita kara sakate haim ki apake bachche ki pratyurjata (elarji) eka sananda, svastha, roja़marra ki ja़indagi ke raste mem na ae|
yunaiteda stetsa mem samanya se adhika vaja़na vale bachchom ke pratishata mem kha़taranaka dara se vriddhi ho rahi hai|
kula milakara, bachche vyayama karane mem kama samaya aura adhika samaya tivi, kampyutara, ya vidiyo-gema kansola mem bita rahe haim|
aura aja ke vyasta parivara ke pasa bina avakasha ke, paushtika, ghara mem pakaya bhojana banane ke lie khali pala bahuta thoda़e haim|
nai sahasrabdi mem, javana aura vriddha, kai logom ki manasikata phasta phuda se lekara ilektraॉniksa taka, jaldi aura asana hoti si lagati hai|
1970 ke dashaka se, yunaiteda stetsa mem samanya se adhika vaja़na vale bachchom aura kishora/kishoriyom ka pratishata dugune se adhika ho gaya hai|
aja, 2 se 5 varsha ki unra ke 10% aura 6 tatha 19 ke bicha ki unra ke bachchom ke 15% se adhika vaja़na vale haim|
yadi apa bachchom ke pratishata jo samanya se adhika vaja़na vale haim ko bachchom ke pratishata jo samanya se adhika vaja़na vale hone ke khatare mem haim ko milate haim, to lagabhaga tina bachchom mem se eka prabhavita hai|
apake bachche ko samanya se adhika vaja़na vala hone se rokane ka artha hai jisa taraha se apa aura apaka parivara khate aura vyayama karate haim tatha jisa taraha se apa milakara samaya bitate haim, vaha tarika apanana|
apake bachche ko svastha jivanashaili arambha karane mem sahayata karane ki shuruata apa, mata-pita se, aura udaharana dvara margadarshana karate hue hoti hai|
samanya se adhika vaja़na vale bachche madhumeha, uchcha rakta daba, aura uchcha kolestraॉla - jaise gambhira svasthya roga, jise kabhi kevala vayaskom ke roga samaje jate the, ke khatare mem haim|
samanya se adhika vaja़na vale bachche kama svabhimana ki ora bhi pravritta ho sakate haim jo ki chidha़ae jane, dhaunsa diye jane, ya sathiyom dvara upekshita kiye jane se utpanna hota hai|
samanya se adhika vaja़na vale bachche krida़a sathi ke rupa mem chune jane ke lie prayah antima hote haim, yaham taka ki shishu vidyalaya jitana pahale bhi|
bachche jo apane vaja़na se nakha़usha haim unamem ausata vaja़na vale bachchom ki apeksha asvasthyakara alpa khuraka lene ki adatem aura khana-pana ki gada़bada़iyam, jaise ki kshudhahinata (enoreksiya narvosa) aura atikshudha (bulimiya), hone ki adhika sambhavana hai, tatha ve avasada ke satha-satha dhana sampatti ka durupayoga karane ki ora adhika pravritta ho sakate haim|
kai karaka haim jo akele hi milakara, samanya se adhika vaja़na vale hone mem yogadana karate haim|
anuvanshika karaka, jivanashaili ki adatem, ya donom shamila ho sakate haim|
kuchha udaharanom mem, antahstravi samasyaem, anuvanshika lakshana, aura aushadhiyam atyadhika vaja़na vriddhi se sambandhita ho sakati haim|
jo hama khate haim usaka adhika - vasa se lade hue phasta phuda se lekara maikroveva aura pahale se paiketa mem bhare hue bhojanom taka jaldi aura asana hota hai|
dainika karyakrama itane khachakhacha bhare hue hote haim ki adhika svasthyavardhaka bhojanom ko banane ya kuchha vyayama mem samaya dene ke lie bahuta kama samaya hota hai|
khuraka ka akara, ghara mem aura bahara, prachanda rupa se badha़ gaya hai|
isake atiriktta, jivana aba hamesha se adhika, nishkriya hai – bachche sakriya rupa se bahara khelane ki apeksha ilektraॉnika upakaranom, kampyutara se lekara hatha mem pakada़e jane vale vidiyom gema sistamom ke satha khalane mem adhika samaya bitate haim|
8 se kama unra ke bachche ausatana 2.5 ghante tivi dekhane ya vidiyom gema khelane mem bitate haim, aura 8 ke aura usase bada़e bachche 4.5 ghante tivi ke samane ya eka jvayastika ko maroda़te hue date rahate haim|


jo 2 ghante ya usase kama dekhate haim, una bachchom ki tulana mem bachche jo eka dina mem 4 ghante se adhika dekhate haim, unaki samanya se adhika vaja़na vale hone ki adhika sambhavana hai|
ashcharyajanaka rupa se nahim, shayanakaksha mem tivi bhi samanya se adhika vaja़na vale hone ki sambhavana ko badha़ane se sambandhita hai|
anya shabdom mem, kai bachche jaba eka bara skula se ghara a jate haim, to ve lagabhaga rata ka khana khane, grihakarya karane, aura bistara para jane ke lie taiyara hone se pahale ka sabhi khali samaya eka ya dusare parde (skrina) ke samane bitate haim|
aura halanki skulom mem sharirika shiksha (pii) bachchom ko uthane aura chalane-phirane mem sahayata kara sakata hai, phira bhi, saba milakara adhika se adhika skula pii karyakramom ko kama kara rahe haim ya tandarusti-nirmana sharirika kriyakalapom ko vastava mem karane mem lagae jane vale samaya ki matra ko kama kara rahe haim|
eka adhyayana darshata hai ki tisari kaksha ke bachchom ko jima kakshaem prati saptaha sirpha़ 25 minata ka joradara kriyakalapa pradana karati haim|
anuvanshiki bhi eka bhumika nibhata hai – jina apake sharira ke prakara aura kaise apaka sharira vasa ka bhandarana evam khapata karata hai, ko thika usi prakara nirdharita karane mem sahayata karate haim jaise ve anya vilakshanataom ko nirdharita karane mem sahayata karate haim|
kyonki jina aura svabhava donom eka pidha़i se dusari pidha़i taka age badha़ae ja sakate haim, atah eka parivara ke kai sadasya vaja़na se sangharsha kara sakate haim|
bhavanaom se samajauta karane ke lie phuda phiksesa para adhika vishvasa bhi vaja़na ki vriddhi mem yogadana kara sakata hai|
kuchha loga, jaba ve dukhi, atyadhika thake hue ya ube hue mahasusa karate haim to adhika khane ki ora pravritta hote haim|
samana parivara mem logom ka samana khana-pana ka tarika़a hone ki ora jukava hota hai, sharirika kriyakalapa ka samana stara banae rakhate haim, aura samanya se adhika vaja़na vale hone ki ora samana pravriti apanate haim|
adhyayanom ne darshaya hai ki yadi eka bachche ke eka ya adhika mata-pita samanya se adhika vaja़na vale ya sthulakaya haim, to usake mutape ka khatara bahuta hi badha़ jata hai|
isake atirika isake kai svasthya labha haim, stanapana atyadhika vaja़na vriddhi ko rokane mem bhi sahayata kara sakata hai|
halanki thika thika prakriya jnata nahim hai, para stanapana karae gae shishu apane khuda ke antargrahana ko niyantrita karane mem adhika sakshama hote haim aura apane khuda ke antarika kshudha sanketom ka palana karate haim|
bachchom ke sakriya rahane ki svabhavika pravritti ko protsahita karem aura bachchom ko vibhinna prakara ke svasthyavardhaka khadya pesha karem|
eka bachcha kisi nae khadya ko svikara karega isase pahale yaha 10 ya adhika koshishem le sakata hai, atah prayasa na chhoda़e|
bachchom ko pratidina sharirika rupa se sakriya rahane ke lie protsahita karem, chahe vaha eka suniyojita krida़a dala ho ya madhyavakasha ke samaya ka phutabaॉla ka eka phurtila khela|
apake bachche ko saira karane aura ahate mem khelane ki taraha ke roja़marra ke kriyakalapom ke madhyama se ghara mem bhi sakriya rakhem|
unhem achchhe khadya vikalpa chunane mem adhika shamila hone dem|
kishora/kishoriyam phasta-phuda pasanda karate haim, kintu unhem bhune hue murga saindavichom, saladom jaise adhika svasthyavardhaka vikalpom aura chhote akarom ki ora parichalita karane ki koshisha karem|
yadi ve suniyojita khelom mem nahim hai, to charadivari ke bhitara ke karyakrama, tandarusti kakshaem jaise ki yoga ya pailetsa, ya sketabordinga, inalaina sketinga, athava mauntena baikinga (pahada़ para saikila chalana) ki taraha vaikalpika khela ki sipharisha karem|
tivi, kampyutara, aura vidiyo gema ke samaya mem katauti karem aura tivi dekhate samaya khane ko hatotsahita karem|
kai prakara ke svasthyavardhaka khadyom ko parosem aura jitani bara sambhava ho satha-satha khana khaem|
eka dina ke unake ahara mem phalom tatha sabjiyom ke 5 sarvinga shamila karane ki koshisha karem, svasthyavardhaka alpaharom ki yojana banaem, aura bachchom ko hara dina subaha ka nashta karane ke lie protsahita karem|
kai prakara ke kriyakalapom ko karane ka prayatna karane ke lie apane bachchom ko protsahita karem|
koi eka khela ya kriyakalapa ko karane ke lie dabava na dalem – aura ve kya pasanda karate haim use pane mem unaki sahayata karem aura phira unhem unake prayasom mem sahayata dem|
kai chhote bachche akhadya vastuom ko kisi na kisi samaya apane munha mem dalate haim|
ve svabhavika rupa se apane vatavarana ke bare mem jijnasu hote haim, aura udaharanartha, ve reta ke bakse mem se kuchha dhula kha sakate haim|
pika vale (chaka, mitti khane vale) bachche, phira bhi, apane asa-pasa ke isa abodha khoja se age jate haim|
bachchom ke adhika se adhika 25% se 30% mem (aura unake 20% manasika svasthya chikitsalayom mem dekhe jate haim) pika kahalane vali eka khane ki gada़bada़i hoti hai, jo akhadya vastuom ko khane ki satata aura anivarya utkata ichchha se visheshita hai|
pika shabda maigapai, eka pakshi jo apane vishala aura andhadhundha bhukha ke lie jana jata hai, ke lie laitina shabda se ata hai|
pika svalinata aura manasika mandana sahita vikasatmaka vikalangata vale logom mem aura 2 tatha 3 varsha ke bicha ki unra ke bachchom mem sabase samanya hai|
halanki 18 se 24 mahine se kama ke bachche akhadya vastuom ko khane ki koshisha kara sakate haim, yaha usa unra mem avashyaka rupa se asamanya nahim samaja jata hai|
pika eka acharana bhi hai jo una bachchom mem dikhai de sakata hai jinhem unake vikasa ko prabhavita karate hue eka mastishka kshati pahunchi ho|
yaha kuchha garbhavati mahilaom, ke satha-satha miragi vale logom ke lie eka samasya bhi ho sakati hai|
halanki kuchha vastuom ka upabhoga hanirahita ho sakata hai, phira bhi pika ko khana-pana ka eka gambhira vikara samaja jata hai jo kabhi-kabhi gambhira svasthya samasyaom jaise ki shisha vishaktata aura lauha ki kami vali enimiya mem parinita ho sakata hai|
udaharana ke lie, kuchha garbhavati mahilaem, lauha ki kami vali enimiya, jo pika vali garbhavati mahilaom ke bicha eka samanya roga hai, ke lie upacharita hone ke bada akhadya vastuom ko khana banda kara dengi|
phira bhi, jaba eka paushanika kami, ko sudhara jata hai to hara koi pratikriya nahim dikhata hai, jo karana hone ke bajaya pika ka eka parinama ho sakata hai, aura pika vale kuchha logom mem eka pramana prastuta karane vali paushanika kami nahim hoti hai|
jiophaijia ke rupa mem jana jane vala, prithvi ke padarthom jaise ki mitti ya dhula ko khana, pika ka eka rupa hai jo lauha ki kami utpanna kara sakata hai|
pika ko spashta karane ka eka mata hai ki kuchha sanskritiyom mem, mitti ya dhula ko khana michali (aura isalie, pratah vamana) ko dura karane, dasta ko niyantrita karane, larastravana ko badha़ane, jivavishom ko hatane, aura gandha ya svada ke avabodhana ko badalane mem sahayata kara sakata hai|
kuchha loga mitti ya dhula ke svada aura prakriti ko pasanda karane ka dava karate haim, aura ise eka dainika svabhava ke bhaga ke rupa mem khate haim (bahuta kuchha anyom ke lie dhunrapana karane ki eka dainika dinacharya ki taraha)|
aura kuchha manovaijnanika mata pika ko tanava ki eka vyavaharavadi anukriya ya eka sanketa ke rupa mem batate haim ki usa vyakti ko eka maukhika grastata hai (use apane munha mem vastuom ko rakhane se sukha milata hai)|
eka anya spashtikarana hai ki pika vishesha dharmika krityom, loka aushadhi, aura jadui vishvasom ki eka sanskritika visheshata hai|
udaharana ke lie, vibhinna sanskritiyom mem kuchha loga vishvasa karate haim ki dhula khana unake sharirom mem jadui atmaom ko samavishta karane mem unaki sahayata karega|
halanki, inamem se koi bhi mata, pika ke hara rupa ko spashta nahim karata hai|
eka chikitsaka ko kya isa roga ko utpanna kara raha hai, ise samajane ki koshisha karane ke lie hara mamale ka vyaktigata rupa se upachara karana chahie|
apaka chikitsaka pika-sambandhita acharana ko niyantrita karane aura rokane mem apaki sahayata karane mem eka mahattvapurna bhumika nibhaega|
vaha apake bachche ko svikarya aura asvikarya khadya padarthom ke bare mem sikhane ke lie apako shiksha degi|
chikitsaka una akhadya vastuom ko khane se rokane ke tarike dhundhane mem bhi apake satha kama karega jisake lie apaka bachcha tarasata hai (jaise ki bachchom ke lie suraksha talom (chaildsa-sepha़ti laॉka) aura unche shelpha़ ka upayoga karana, aura gharelu rasayanom aura aushadhiyom ko bachche ki pahuncha se bahara rakhana)|
kuchha bachchom ko vyavaharavadi hastakshepa ki avashyakata ho sakati hai aura parivara ko eka manovaijnanika ya anya manasika svasthya anubhavi vyakti ke satha kama karane ki avashyakata ho sakati hai|
eka bachche ki unra aura vikasatmaka stara para nirbhara karate hue, chikitsaka bachchom ke satha unhem adhika upayukta rupa se khane ke tarike sikhane ke lie kama karenge|
yadi pika vyavaharavadi upacharom ke prati pratikriya na dikhate hue mahatvapurna vyavaharavadi samasyaom se sambandhita hai to aushadhi ka nuskha bhi likha ja sakata hai|
yadi sanketa hogem to apaka chikitsaka raktalpata ya anya paushanika kamiyom ke lie jancha kara sakata hai|
eka bachcha jisane eka sashakta rupa se hanikaraka padartha, jaise ki sisa, ko kha liya hai, to use sisa aura anya vishaile padarthom ke lie jancha jaega aura parajiviyom ke lie mala parikshana kiya ja sakata hai|
kuchha mamalom mem, kya khaya gaya tha ya anta samasyaom, jaise ki eka avarodha, ka parikshana karane ke lie eksa-re ya anya tomographi sahayata kara sakata hai|
saubhagyavasha, pika samanyatah eka asthayi roga hai jo jaise hi bachche bada़e ho jate haim ya garbhavastha ke bada sudharane lagata hai|
kintu vikasatmaka aura manasika svasthya samasyaom vale vyakti ke lie, pika eka adhika lambi chinta ho sakati hai|
upachara ke bada, yadi apake bachche ka pika vyavahara hastakshepa karane ka prayasa karane ke bavaja़uda bhi kai hapha़tom bada taka bana rahata hai, to atirikta upachara ke lie apake chikitsaka se phira se samparka karem|
yada rakhie ki pika ka upachara karane mem rogi mahatvapurna hai kyonki kuchha bachchom ke lie akhadya vastuom ko khane ki chahata ko rokane mem yaha samaya le sakata hai|
dopahara ka khana skula mem kharidane mem apake bachche ko shayada pahali bara isa bare mem uttaradayitva milata ki kauna sa khadya vaha khaega|
achchhi kha़bara yaha hai ki ina salom mem svada aura poshana donom mem skula ka dopahara ka khana behatara hua hai|
durbhagyavasha, kai skula ke dopahara ka khana protina, vitaminom, kailshiyama, aura lauha ke stara ko to pura karate haim, kintu vasa ki sipharisha matra ko phira bhi para kara jate haim|
kuchha skulom ne bhune murga ka saindavicha aura salada jaise behatara dishom ko parosane ka eka prayasa kiya hai|
nakaratmaka pahalu spashta hai: vishishta skula kaipheteriya mem, apaka bachcha eka la karte ya vendinga mashina mem prayah upalabdha kama paushtika bhojana ka labha lete hue, eka asvasthyakara khadyom ka mishrana phira bhi chayana kara sakata hai|
udaharanartha, eka bachcha eka haॉta daga jaise bachchom ko achchhe lagane vala vaisa hi bhojana, kai dinom taka lagatara kharidane ka nirnaya le sakata hai|
apane bachche ko kaipheteriya ke bhojana ke satha-satha phala, sabjiyom, bina charbi ka mansa, aura sabuta anaja ,sapheda breda ke bajaya gehum ki breda ko chunane ke lie protsahita kariye |
jaba taka sambhava ho tala hue khadya padarthom se bachem aura dudha ya pani ko eka peya ke rupa mem chayana karem |
yadi apa apane bachche ki madada kara rahe haim to usako dopahara ke bhojana ke buddhivardhaka khadya aura alpahara dena shuru kijie jise vaha khana pasanda kare |
purane khadyannom jaise pinata batara aura jeli ke sthana para, pitha breda ya grilda chikana ya sabjiyom se bhare saindavicha ka istemala karake dekhem |
agara apake bachche ki ichchha ho to supa aura salada ka prayoga karem aura rata ke bache hue khane ko dopahara ke khane ke dibbe mem rakhana na bhulem |
bachchom ke lie pahale se paika kiya gaya dopahara ka bhojana bahuta hi prachalita aura suvidhajanaka hota hai , lekina vaha manhaga aura kama poshaka tatvom vala hota hai |
isake bajaya, khuda hi apane lie svastha samagri ka upayoga kara dopahara ke bhojana ko paika karane ke taiyara karem |
apane bachche ke skula mem jancha karake sunishchita karem ki bachchom ko unake lancha mem kya paika kara sakate haim isa para koi pratibandha to nahim hai |
aura isa prakriya mem bachchom ko shamila karana mata bhulem jisase ki svastha lancha unaka lakshya ho jisake lie ve prayasa bhi kara sakate haim |
eka paika kiye hue dopahara ke bhojana ko khane ke samaya taka surakshita rakhane ke lie atirikta jimmedari vahana karani pada़ti hai |
isaka matalaba hai ki garma khane ko garma evam thande khane ko thanda hi rakhem |
eka adhyayana mem paya gaya hai ki eka tihai se kama mata pita jina khadyom ko thandaka ki jarurata hoti hai aise khadyom jaise dahi, deli mansa saindavicha, aura anya khadya padarthom ko hi paika karate haim |
agara bachcha shakahari ahara hi leta hai aura phira bhi use sare jaruri poshaka tatva jo ki svastha aura balavana banane ke lie avashyaka hote haim, milate haim to kai mata-pita ashcharya hi karenge ,jisase shakaharavada adhika prachalita aura lokapriya ho jata hai |
mansa se bachana eka bura vichara laga sakata hai , lekina jyadatara ahara aura chikitsa visheshajnom ka manana hai ki suniyojita shakahari ahara vastava mem khane ke lie svasthavardhaka hota hai |
beshaka, jaba bachchom aura kishorom ko shakahari ahara khilana ho to vishesha dekhabhala ki jarurata hoti hai khasakara agara usamem deyari aura anda utpada shamila na hom | 
kisi bhi ahara ke rupa mem apako sunishchita karana hoga ki na kevala apake bachche ko paryapta matra mem jaruri poshaka tatva milem balki yaha bhi samajana hai ki bachchom ke badha़ne ke satha unaki poshaka tatvom ki jarurata bhi badalati hai |
kai aise karana haim jinaki vajaha se bachche ya parivara shakahari ahara ko apana sakate haim |
chhote shakahari amataura para eka parivara ka hissa haim jo ki svasthya ya anya karanom se shakahari bhojana khate haim |
bada़e bachche janavara, paryavarana, ya svayam ke svasthya ki chinta ki vajaha se shakahari banane ka phaisala kara sakate haim |
jyadatara mamalom mem, agara apaka bachcha shakahari hona chunata hai to apako chintita nahim hona chahie |
sunishchita karem ki apaka bachcha svastha aura paushtika khana pasanda karata hai ,apane bachche ki pasanda ko samajane ka sabase achchha tarika yaha hai ki usase usake bare mem charcha karem ki usaka isase kya tatparya hai aura vaha isaka kaise prayoga karana chahata hai ?
pratyeka 12 aunsa (355 mili litara) karboneteda, mitha shitala peya ka sevana 10 chammacha chini aura 150 kailori (49 mililitara) ke barabara hota hai|
amerika ke bachchom ke dainika ahara mem mithe peya chini ka sabase bada़a srota haim |
prati dina 12 aunsa (355 mili litara) shitala peya lene se bachchom mem motape ka khatara badha़ jata hai |
soda ya rasa peya ke bajaya (jinamem atirikta chini hoti hai), bachchom ko kama vasa vala dudha, pani ya 100% phalom ke rasa dena chahie |
halanki 100% phalom ke rasa mem atirikta chini nahim hoti hai , lekina phalom ke rasa mem payi jane vali prakritika sharkara bhi kailori ko badha़a sakati hai |
ameriki bala roga akadami (apa) salaha deti hai ki 7 varsha se kama unra ke bachchom ke lie rasa ke sevana ki sima4-6 aunsa (118-177 mililitara) aura bada़e bachchom aura kishorom ke lie adhika se adhika 8-12 aunsa (237-355 mililitara)rasa lene ki sima simita ki hai |
sunishchita karem ki apake bachchom ko eka santulita, poshaka ahara mile , yaha utana mushkila nahim jitana yaha dekhane mem laga sakata hai |
sadharanatah achchhe karbohaidreta(sabuta anaja, phala, sabjiyam, aura kama vasa vale dudha aura deyari utpadom) banaem, apane ghara mem svasthavardhaka khadyom, atirikta chini vale khadyom ka simita matra mem (vishesha rupa se kama ya bina poshana vale padarthom) bhandara rakhem aura apane bachchom pratidina sakriya rahane ke lie protsahita karem |
jaba yaha apane bachchom ke lie hota hai, kai mata pita kolestraॉla ke bare mem sochate taka nahim haim |
lekina kolestraॉla ka uchcha stara hridaya roga aura stroka ke lie eka pramukha yogadana karaka jana jata hai, aura navinatama chikitsa anusandhana se pata chalata hai ki koronari dhamani ki bimari ki jada़em bachapana se hi hoti haim |
abhi taka bachapana mem kolestraॉla ke stara ka pata nahim laga tha , lekina kuchha visheshajna sochate haim ki bachchom mem uchcha kolestraॉla ka hona eka kama riporta ki gai eka pramukha sarvajanika svasthya samasya hai |
pichhale kuchha varshom mem adhyayana se, bachchom ke motape mem natakiya vriddhi aura samasyaem aura adhika vajana hone ke dushparinama pata chale haim | 
aura kuchha bala roga visheshajnom ki riporta yaha dikhati hai ki kuchha sala pahale ki tulana mem badha़e hue kolestraॉla ke stara vale bachchom ki sankhya mem ullekhaniya vriddhi hai hai |
sadharanatah badha़e hue kolestraॉla se sambandhita samasyaem kai salom taka nahim dikhati haim , jisase kai logom ke lie bachchom ke svastha aura kolestraॉla ke bicha koi sambandha kara pana mushkila hai |
eka sala ki unra ke bada, shuddha shakahari ahara se badha़te bachchom ko paryapta avashyaka vitamina aura khanija, vitamina di, vitamina bi 12, loha, kailshiyama, aura jasta nahim milata hai |
isalie yaha mahatvapurna hai ki majabuti dene vale anaja aura poshaka tatvom se bharapura khane ka sevana karem , satha hi satha apane bachche ke phaibara lene para bhi dhyana dem |
jina yuva bachchom ka ahara paryapta poshaka tatva pradana nahim kara pata hai unako ahara mem atirikta vitamina lene ki salaha di gai hai | 
bachche khane ke mamale mem bahuta mina-mekha nikalane vale hote haim isake parinamasvarupa kuchha bachchom ko shakahari ahara se badha़ne ke lie paryapta kailori nahim mila pati ho | 
shakahari bachchom ke thika-thaka poshana ke lie sabjiyom ki paryapta matra ki avashyakata hoti hai , unake chhote se peta ke lie kailori shayada bhari ho sakati haim |
bachchom ki mina-mekha nikalane ki avastha ke daurana shakahari mata - pita ko yaha maluma hona jaruri hai ki apaka 
apa apane shakahari bachche ke ahara mem paryapta vasa aura kailori prapta kara sakate haim, para apako savadhanipurvaka khane ke samuha ko nikala kara yojana taiyara karani hogi |
kishoravastha se purva ke balaka evam kishora aksara apani hi pasanda ka hi khana khana chahate haim jo unhem pasanda ho |
eka kathora vaktavya ke lie mansa khana banda karane ka nirnaya lena hai |
yaha kishorom mem bahuta samanya hai jo ki ve janavarom ke haka ke samarthana mem , svasthya karanom ke lie ya dosta aisa kara rahe haim ,shakahari hone ka nirnaya le lete haim |
agara yaha karana sahi hoga, vastava mem eka mansa mukta ahara kishorom ke lie eka achchha vikalpa ho sakata hai, vishesha rupa se vichara karate hue ki aksara shakahari adhika khadya padartha khate haim jabaki adhikatara kishorom ko paryapta rupa se phala aura sabjiyam nahim mila pati haim | 
deyari utpada aura ande sahita shakahari ahara hai ki (lacto-ovo) badha़te hue kishorom ke lie sabase achchha vikalpa hai |
eka shuddha shakahari ahara kishorom ki jarurata ke khasa poshaka tatvom jaise loha , protina ja़inka , kailshiyama aura vitamina di aura bi 12 ke rupa mem pura karane mem viphala ho sakata hai |
yuva shakahariyom aura unake mata -pita ke lie achchhi khabara yaha hai ki kai skula salada bara aura anya svastha shakahari vikalpom sahita shakahari bhojana ki suvidha dene mem adhika jagaruka hote ja rahe haim|
agami dopahara ke bhojana ke menu ki suchi skulom mem prakashita karate haim , use dekhakara apa sunishchita karem ki apake bachche ke lie eka shakahari vikalpa usamem mila jayega |
agara nahim, to basa apane bachche ke lancha ko paika karem |
prachina sthapanna ke anusara mungaphali ka makkhana aura jeli saindavicha eka mahana shakahari dopahara ka bhojana hai |
yadi shakahari balaka ya kishora apane skula ka dopahara ka bhojana khuda ke lie bana lete haim ya dopahara ka bhojana kharidana pasanda karate haim ,to dhyana rahe ki apaki tulana mem apake bachche ke vichara mem eka svastha shakahari bhojana (yani, phrencha phraija़ aura eka soda) bahuta alaga ho sakata hai |
apane bachche se sahi khane ke bare mem bata karem khasataura para jaba vaha shakahari ahara ka palana kara raha ho |
atah savadhana rahem yadi apaka bachcha khuda hi atma pratibandhaka ahara ka bahuta palana karata hai|
eka kishora agara khane mem gada़bada़i karata hai to usaki kailori mem teja़i se kami a jati hai, ya vasa aura karbohaidreta mem katauti a jati hai to ise shakahara kahate haim kyonki yaha samajika rupa se svikarya aura svastha mana jata hai|
agara koi kishoravastha se purva ka balaka ya kishora eka svastha tarike se shakahari bana raha hai to unake lie yaha samajana mahatvapurna hai ki unake shakahari ahara mem kauna se poshaka tatvom ki kami hai , atah ve una poshaka tatvom ki kami se bacha sakem isake lie ve unake sthana para dusare poshaka khadyom ka istemala karem | 
ho sakata hai ki apaka pura parivara apane bachche ke ahara nirnaya ke samarthana ke lie aura apane balaka ya kishorom ke khadya padarthom ke prati jagarukata ko badha़ava dene ke lie saptaha mem kama se kama eka rata shakahari bhojana khana chahe | 
eka shakahari ahara sabhi bachchom ke lie eka svastha vikalpa ho sakata hai, jaham taka mumakina ho ise thika prakara se socha-samajakara aura paryapta poshaka tatvom ke satha khaya jae |
eka chhoti si khoja se, apako itane shakahari vikalpa milenge jitane apane soche bhi nahim honge |
eka shakahari ahara ki yojana ke siddhanta thika usi prakara se haim jaise ki koi bhi svastha ahara yojana khadya padarthom ki aneka kisma pradana karate haim aura jisamem khadyom ke sabhi samuha shamila haim |
eka santulita ahara apake bachche ko poshana tatvom ka eka aisa uchita sammishrana pradana karata jisaki use jarurata hoti hai |
lekina apa ko apane bachche ke ahara mem sambhavya poshaka tatvom ki kami ke bare mem pata hona chahie aura pata lagane ki koshisha karem ki kaise unhem pura karem |
yadi apa ashvasta nahim haim ki apake bachche ko jaruri poshaka tatva jinaki use jarurata hai , mila rahe haim ya nahim , agara shakahari ahara ke bare mem yadi apako kuchha savala haim to kuchha utkrishta upalabdha sansadhanom jaise parivara ke daॉktara, bachchom ke chikitsaka, ya eka panjikrita ahara visheshajna ki madada se apa apane bachche ke ahara ke plana karane mem sankocha na karem |
jaham taka apane bachche ko khilane ki bata hai to aisa lagata hai ki chakkara dilane vali niyamom sankhya palana karane ke lie hai| 
apake bachche ko majabuta aura svastha banane ke lie poshaka tatvom ki jarurata hoti hai, lekina apako bhi apane bachche ko sima ke andara bhojana khilana chahie jisase apake bachche ka vajana na badha़e aura svasthya samasyaom ka vikasa bhi na ho |
motapa sanyukta rajya amerika mem eka ama samasya banata ja raha hai |
lagabhaga do tihai ameriki adhika vajana ya motape se grasta haim, aura unamem se lagabhaga adhe bahuta kama sharirika kriya-kalapa karate hai |
ameriki poshana adhikari madada karane ki koshisha kara rahe haim |
ameriki krishi vibhaga (USDA) dvara ahara dishanirdesha banaya gaya hai ki apane bachche ko svastha, santulita ahara kaise dem isaki vyavaharika salaha pradana ki gai hai |
disha nirdesha sujava dete haim ki atita ki tulana mem bachche adhika phala, sabjiyam, aura sabuta anaja khate haim aura ve hara dina 30 se 60 minata taka madhyama se joradara vyayama karate haim |
bachchom ke lie unra, linga, aura vyayama ki adatom ke adhara para salaha di gai haim |
yuesadie ki vebasaita para laॉginga karake apa pata laga sakate haim ki apake bachche ke ahara ke lie kya dishanirdesha upayukta haim |
bachchom aura mata pita ko ahara disha nirdeshom ko samajane ke khadya gaida piramida banaye gaye haim |
piramida ke andara chhaha dhariyam pancha khadya samuhom ka pratinidhitva karati haim-apake bachche ko hara dina vasa aura tela ka prayoga achchhe se karana chahie | 
piramida para ki sidha़iyam vyayama ke mahatva ka pratinidhitva karati haim aura jisa para pratyeka dina apaka hara samanya kadama apane bachche ke svasthya sudhara ke pratinidhitva ke lie hoga |
apake bachche ko pratyeka dina jisa urja ki avashyakata hai use aise anaja samuha vala ahara jaise roti, chavala aura pasta adi pratyeka dina pradana kariye |
ina khadya padarthom mem bahuta adhika karbohaidreta hote haim , joki sharira ke lie upayukta indhana haim, aura apane bachche ko khelane ke lie urja , skula mem dhyana lagane ke lie, aura kai anya kriyakalapom ke lie apane bachche ko aisa ahara dijie |
anaja bhi anya mahatvapurna poshaka tatva pradana karate haim jaise vitamina bi kaॉmpleksa (pholeta) hai, jo ki apake bachche ke sharira mem mansapeshiyom ke nirmana ke lie avashyaka protina ke upayoga mem madada karata hai | 
hara dina yadi apaka bachcha jo bhi anaja istemala karata hai usaka kama se kama adha bhaga sabuta anaja , daliya, brauna chavala, aura rai ki roti ke rupa mem hona chahie |
sabuta anaja phaibara ahara hai joki hridaya roga aura madhumeha jaisi bimariyom se raksha karane mem madada kara sakata haim aura apake bachche ke vajana ko niyantrita karane mem madada karata hai |
ve parishkrita anaja se alaga haim ve isa prakara haim jaise sapheda breda aura sapheda chavala, jo sansadhita kiya gaya hai, aura aneka poshaka tatva unamem se nikala liye jate haim |
sharira ke kriya-kalapa ko banae rakhane ke lie vasa aura tela avashyaka poshaka tatva haim lekina inaka sanyama se istemala kiya jana chahie |
vasa sharira mem vitamina e, di, i, ke aura bita kairotina ko avashoshita karane mem madada karata hai | 
halanki achchhe svasthya ko banae rakhane ke lie vasa ki jarurata ho sakati hai, para unamem kailori shamila hoti haim isalie unhem sima mem rakhana eka achchha vichara ho sakata hai |
tela vanaspati vasa haim joki ki kamare ke tapamana para tarala ho jate haim, ye amataura para khana pakane mem upayoga mem laye jate haim |
kai tela alaga alaga paudhom aura machhali se a sakate haim |
kuchha anya ama telom ke satha-satha jaituna ka tela, makka ka tela, soyabina tela aura surajamukhi tela bhi shamila haim |
kuchha khadya padarthom mem jaise jaituna, machhali, aura avakado adi mem prakritika rupa se tela adhika hota hai |
adhikansha telom mem monoasantripta vasa ya paॉlianasechureteda vasa adhika hoti hai |
isa vasa ke karana echadiela (achchha) kolestraॉla, jo ki dila ki samasyaom ko rokane mem madada karata hai apake bachche mem isaka stara badha़a deta hai, aura (bura) eladiela kolestraॉla ke vaha stara ko kama karata hai, jo ki dila ki samasyaom ko badha़ata hai |
khalisa vasa, makkhana, anya snigdha padartha aura marjarina , aura adhika santripta vasa ya transa vasa, jo (bura) rakta mem eladiela kolestraॉla ke stara ko badha़a sakate haim aura apake bachche ke lie hridaya roga ke khatare ko bhi badha़a sakate haim |
vasa 2 varsha se kama ayu ke bachchom mem pratibandhita nahim kiya jana chahie |
chhote bachche ke vikasashila mastishka aura anya angom ke samuchita vikasa ke lie vasa ki eka nishchita matra avashyaka hoti hai |
apane bachche ko tvarita urja khuraka pradana karane ke lie khuna mem jaldi avashoshita ho jane vali shakkara dijie |
apa apane bachche ko kaindi, mithai, aura anya khadya padarthom ke dvara sharkara eka simita matra mem khilaem yaha eka achchha vichara hai |
aisa isalie hai kyonki sharira atirikta chini bhandara ko rakhata hai use turanta itani vasa ki jarurata nahim hoti hai |
isaki vajaha se vajana aura svasthya sambandhi anya samasyaem badha़ sakati haim |
mahilaom ke lie prasava se purva dekhabhala aura garbhavastha ke daurana chikitsa sambandhi dekhabhala karane ki salaha di jati hai |
prasava purva ki achchhi dekhabhala ke uddeshya se kisi bhi sambhavita samasyaom ko jaldi pata lagakara, yadi sambhava ho to unhem paryapta poshana, vyayama, vitamina ka sevana adi ki salaha ke madhyama se rokane ke lie, aura upayukta visheshajnom dvara mahilaom ko aspatalom adi ka marga-darshana karana bhi avashyaka hai |
niyamita prasavapurva dekhabhala ki upalabdhata ne, vikasita deshom mem ,matri nrityu darom, garbhapatom aura janma doshom, janma ke samaya kama vajana hone aura shishuom ki anya nivarya samasyaom ko kama karane mem eka bhumika nibhai hai|
halanki prasavapurva dekhabhala ki upalabdhata mem vyaktigata svasthya aura samajika labha kaphi hote haim, lekina phira bhi kai vikasita evam vikasashila deshom mem ,samajika arthika samasyaem isake sarvavyapi svikarana mem rukavatem dala rahim haim|
garbhavati mam ke lie, prasavapurva dekhabhala ki eka nitya adata honi chahie- pholika esida ki kama se kama 400 miligrama matra ke satha vitamina lena, jisase nyurala tyuba dosha ko rokane mem madada milati hai|
mataom aura shishuom ke svasthya mem sudhara karane ki isaki kshamata ki vajaha se ,sabhi garbhavati mahilaom ko prasavapurva dekhabhala karane ka sujava diya jata hai|
prasavapurva dekhabhala ki aparyapta adatem janma ke samaya shishu ke kama vajana hone, samaya se pahale janma hone, navajata nrityu dara, shishu nrityu dara aura matri nrityu dara jaise badha़te hue khatarom se juda़im hoti haim| 
apake garbhavati hone ke samaya ki gai dekhabhala hi prasavapurva dekhabhala kahalati hai| 
yaha dekhabhala eka chikitsaka, dai, ya anya kisi svasthya ki dekhabhala karane vale peshevara dvara upalabdha karai ja sakati hai|
prasavapurva dekhabhala ka uddeshya, garbhavastha ki pragati ki nigarani karana aura isase pahale ki kuchha sambhavita samasyaem mam ya bachche ke lie gambhira ho jaem,unaki pahachana karana hai|
jo mahilaem garbhavastha ke daurana niyamita rupa se kisi daॉktara ke pasa jatim haim, to unake shishu adhika svastha hote haim, unake dvara samaya se pahale janm dene ki sambhavana bhi kama hoti hai,aura garbhavastha se sambandhita anya gambhira samasyaom ke hone ki sambhavana bhi kama hoti hai|
jaise hi apako lagata hai ki apa garbhavati haim, to apane daॉktara ko yaha pata lagane ke lie phona kijie ki apa prasavapurva dekhabhala ki apani prathama jancha ke lie kaba aem | 
apani garbhavastha ke daurana apa, prasavapurva dekhabhala ki apani sabhi janchom ke lie daॉktara ke pasa jaem ,yaham taka ki agara apa bilkula thika bhi haim|
daॉktara se samaya lena kabhi kabhi mushkila ho sakata hai ya apako lage ki isamem samaya barbada ho raha hai|
lekina phira bhi apane bachche ke lie, prasavapurva dekhabhala ko prathamikata dijie|
garbhavati hone se pahale hi daॉktara se pasa jana adarshatah sabase achchha hota hai|
pahale jane se apa garbhavati hone se pahale ki chintaom aura muddom ke barem mem daॉktara se charcha kara sakati haim|
jo mahilaem garbhavastha ke daurana niyamita rupa se kisi daॉktara ke pasa jatim haim, to unake shishu adhika svastha hote haim, unake dvara samaya se pahale janm dene ki sambhavana bhi kama hoti hai,aura garbhavastha se sambandhita anya gambhira samasyaom ke hone ki sambhavana bhi kama hoti hai|
jisa stri ki chikitsiya halata kaphi purani hai ya usaki garbhavastha adhika khatare vali hai,to use daॉktara ke pasa jaldi jaldi jane ki avashyakata ho sakati hai|
apani garbhavastha ke daurana apa, prasavapurva dekhabhala ki apani sabhi janchom ke lie daॉktara ke pasa jaem ,yaham taka ki agara apa bilkula thika bhi haim|
jo khana apako achchha lagata hai apani garbhavastha ke daurana use puri taraha chhoda़ne ki jarurata nahim hai| 
apako sirpha taja bhojana karana hai aura itana dhyana rakhem ki apaka adhikanshatah bhojana paushtika ho| 
apane bachche ki vriddhi aura vikasa ke lie pratidina apako kevala 300 atirikta kailori ki jarurata hoti hai|
apane bachche ki vriddhi mem sahayaka hone ke lie jyadatara garbhavati mahilaom ko pratidina 300 atirikta kailori ki jarurata hoti hai| 
isalie taja bhojana karem aura vaha bhi kevala paushtika bhojana| 
pratidina pratyeka panchom ahara shreniyom mem se kuchha na kuchha khadya padartha khane ki koshisha karem |
ye shreniyam ve mahatvapurna poshaka tatva pradana karati haim jo apa aura apake bachche ke lie jaruri haim| 
pholika esida vitamina bi ka eka prakara hai jo mastishka aura ridha़ ki haddi ke janm doshom (jinhem nyurala tyuba dosha kaha jata hai) ko rokane mem madada karata hai|
prasava unra ki sabhi mahilaom ko garbhavastha se pahale aura garbhavastha ke prarambhika dinom ke daurana pratidina,paushtika ahara ke eka hisse ke rupa mem pholika esida ki 400 maikrograma matra sahita bahuvitamina khuraka leni chahie | 
apake paushtika ahara mem ve khadya padartha shamila hone chahie jo pholika esida aura pholeta se bharapura ho (pholeta eka prakara ka pholika esida hai jo bhojana mem prakritika rupa se paya jata hai) |
dinabhara mem tina bara jyada khane ki bajaya, thoda़a thoda़a karake chara se chhaha bara khaem , jisase bachche ke vikasa se hone vali ghabarahata aura bechaini se rahata milane mem madada milati hai|
prasuti-vijnani vaha chikitsaka hoti hai jo garbhavastha , prasava ke samaya aura prasava se svasthyalabha ke daurana mahilaom ki dekhabhala mem mahira hoti hai|
10 mem se lagabhaga 8 garbhavati striyam prasuti-vijnani ke pasa jatim haim|
parivara ka daॉktara vaha daॉktara hota hai jisako parivara ke pratyeka sadasya ke svasthya dekhabhala ke sabhi pahaluom para prashikshana hasila hota hai|
apake parivara ka daॉktara hi apaki garbhavastha se pahale, apaki garbhavastha ke daurana aura garbhavastha ke bada mem apake svasthya ki dekhabhala karane vala daॉktara bhi ho sakata hai, aura yaham taka ki apake bachche ka bhi daॉktara ho sakata hai|
eka pramanita narsa ya dai vaha panjikrita narsa hoti hai , jisako garbhavati mahilaom aura janm lene vale bachchom ki dekhabhala ke lie uchcha shreni ka vishesha prashikshana aura anubhava prapta hota hai|
pramanita narsa ya daiyom ko prasava se pahale, prasava ke daurana aura prasava ke bada dekhabhala karane ke lie laisensa prapta hote haim|
eka matri-bhruna chikitsa visheshajna vaha dai hoti hai jisako adhika khatare vale garbhadharana vali mahilaom ki dekhabhala mem vishesha prashikshana prapta hota hai|
yadi apa ke andara koi bhi jokhima karaka haim jo apaki garbhavastha ko jatila bana sakate haim, to prasavapurva dekhabhala karane vali apaki daॉktara apako prasuti-bhruna dava visheshajna ke pasa jane ki salaha degi|
prasavapurva parikshana vo parikshana hote haim jo apa aura apake bhruna ko taba karane hote haim jaba apa garbhavati hotim haim|
ina parikshanom se svasthya ki dekhabhala karane vali daॉktara ko yaha pata laga jata hai ki janma se pahale bachcha kaisa hai aura apani garbhavastha ke daurana mam kaisi hai|
rakta parikshana, mutra parikshana aura raktachapa ki jancha adi kuchha aise niyamita prasavapurva parikshana haim jinako sabhi garbhavati mahilaom ko karane hote haim|
ulvavedhana(emaniosenatesisa) jaise anya parikshana ko ,niyamita rupa se karane ki salaha nahim di jati, jabataka ki apako ya apake bachche ko agara kuchha samasyaom se ya kisi anya sthiti se badha़ta hua khatara na ho|
apaki daॉktara jo bhi prasavapurva parikshana karaem ,usake bare mem apa jyada se jyada janakari ekatra karem| 
agara apako jara bhi sandeha hai ki ye prasavapurva parikshana hone chahie ya nahim ,to apane pati,parivara ke anya sadasyom,kisi adhyatmika salahakara aura/ya anuvanshika paramarshadata se salaha lem |
yada rakhem, jyadatara mahilaom ko yaha ashvasta samachara rahata hai ki unake bachche thika haim|
apaki sampurna garbhavastha ke daurana, prasavapurva dekhabhala ke hisse ke rupa mem, niyamita chikitsiya parikshana hote rahenge|
ina parikshanom se apake svasthya ki dekhabhala karane vali daॉktara ko yaha pata lagata rahata hai ki apa aura apake bachche ka svasthya kaisa hai|
vaise to ye sabhi niyamita parikshana haim, lekina ye apake aura apake bachche ke svasthya ki raksha karane mem eka mahatvapurna bhumika nibhate haim|
daॉktara ke pasa jane para hara bara,apaki daॉktara apake mutra ki jancha kara sakati hai|
garbhavastha ke daurana, enimiya ke lie kama se kama eka bara aura rakta parikshana ho sakata hai|
apaki daॉktara sistika phaibrosisa (si epha) ke lie bhi eka skrininga jancha kara sakati hai, yaha eka vanshagata bimari hai jo shvasana kriya aura pachana kriya ko gambhira rupa se prabhavita kara sakati hai |
sistika phaibrosisa ka koi ilaja nahim hai|
jisa bachche ko,mata pita donom se eka eka asamanya jina prapta hota hai,usako yaha roga ho jata hai|
da amerikana kaॉleja ऑpha ऑbsatitrishiyansa enda gainekolaॉjistsa (esiऑji) ke anusara, si epha skrininga jancha una sabhi pati patniyom ko karani chahie ,jinhonne garbha dharana kara liya hai ya usaki yojana bana rahem haim|
apa karaem ya na karaem,yaha apaka eka vyaktigata nirnaya hai|
anuvanshika paramarsha se apako apana nirnaya lene mem madada mila sakati hai|
pratyeka bara,apaki daॉktara protina ke lie apake mutra ki jancha karegi aura apake raktachapa ko mapegi|
mutra mem protina aura uchcha raktachapa garbhavastha se sambandhita aise lakshana haim jinase uchcha rakta dabava ho jata hai aura isa sthiti ko prakagarbhakshepaka (prieklaimpasiya )kaha jata hai|
prakagarbhakshepaka (prieklaimpasiya ) se 5-8 pratishata garbhavati mahilaem prabhavita hotim haim|
agara isako bagaira ilaja ke chhoda़ diya jae,to isase aparyapta bhruna vikasa sahita gambhira samasyaem utpanna ho sakati haim|
bahuta hi kama mamalom mem, isase jivana ke lie khatara vali sthiti paida ho sakati hai jise eklaimpasiya kaha jata hai|
eklaimpasiya ke rogi ko daure pada़tem hai aura kabhi kabhi vaha koma mem bhi chala jata hai|
prieklaimpasiya ka avalokana aura nigarani bahuta hi bariki se hone chahie, kyonki yaha bhi eka vaja़ha hai jisase ki apaki sabhi prasavapurva jancha mahatvapurna bana jatim haim|
isake alava, hara bara jancha ke lie jane para apaki daॉktara daॉpalara namaka hatha ke eka yantra se apake bachche ke dila ki dhada़kana bhi sunegi|
lagabhaga 20 saptaha ke bada, apake bachche ke vikasa ko dekhane ke lie vaha apake peta ko bhi napegi |
dila ki samanya dhada़kana aura vriddhi dara mahatvapurna sanketa haim jinase pata chalata hai ki apake bachche ka vikasa thika thaka chala raha hai|
bahuta si daॉktara sabhi garbhavati mahilaom ki altrasaunda jancha karati haim|
skrina para bachche ki tasvira dikhane ke lie, altrasaunda dhvani tarangom ka upayoga karata hai|
daॉktara transadyusara namaka eka yantra se garbhavati mahila ke peta para malisha karati hai ya eka yantra usaki yoni mem dalati hai|
jaise jaise daॉktara transadyusara chalati hai, mahila ko dabava mahasusa hota hai lekina darda nahim hota |
prasavapurva dekhabhala eka chikitsiya dekharekha hai jo garbhavati mam aura usake bachche ke lie ki jati hai|
isamem mam ke lie khuda apani dekhabhala bhi shamila hai,jaise-apane daॉktara ki salaha ka palana karana, achchha paushtika ahara lena, khuba arama karana ,samajadari se vyayama karana, aura una chija़om se bachana jo usako ya usake bachche ko nukasana pahuncha sakati haim | 
jo adatem eka vayaska ko nukasana nahim pahuncha sakati hai,ho sakata hai ve apake bachche ke vikasa ko nukasana pahuncha sakati hom|
yada rakhem ki apake bachche ka svasthya usake paida hone se bahuta pahale hi shuru ho jata hai|
uchita prasavapurva dekhabhala se, apa apane bachche ki svasthya samasyaom ke khatare ko kama kara sakati haim|
dhunrapana karane vali garbhavati stri ko usaki garbhavastha mem pareshani hone ka kaphi khatara rahata hai|
dhunrapana na karane vali mam ke bachchom ki tulana mem ,dhunrapana karane vali mam ke bachchom mem janm ke samaya ausata vaja़na kama hota hai, jisase samaya se pahale janm dara mem badha़ottari hoti hai, aura sadana inaphainta detha sindroma (achanaka shishu nrityu lakshana) se mauta ka khatara bhi adhika ho jata hai|
adhyayanom se pata chalata hai ki jo mahilaem apani garbhavastha ke arambhika dinom mem dhunrapana chhoda़ deti haim,unake bachchom mem kuchha bhi nukasana hone ka khatara kama ho jata hai|
yaham taka ki garbhavastha ke akhiri mahine mem bhi dhunrapana chhoda़ne se ,prasava ke daurana use ऑksijana ki matra jyada milane mem madada ho sakati hai|
agara jaba apa garbhavati haim ,to sharaba pine se surakshita sthiti ki koi garanti nahim hai|
koi bhi sharaba pine se vaha apake aura apake bachche donom ke rakta mem pravesha kara jati hai|
garbhavastha ke daurana sharaba pine se phetala elkaॉhala sindroma (epha e esa) ho sakata hai|
epha e esa se pida़ita bachchom ko vyavaharika, sharirika aura manasika samasyaem ho sakati haim|
apani daॉktara ki salaha ke bina, pahali bara koi bhi dava na lem|
yaham taka ki daॉktara ki salaha ke bina li jane vali davaom jaise ki entihistemainsa ya darda nivaraka davaom se ,jinamem espirina ya aibuprophena hota hai,bachche ke vikasa ko nukasana ho sakata hai|
kevala daॉktara ke dvara hi batai gayi prasavapurva vitamina ki davaem lem,lekina apane apa koi bhi atirikta vitamina ki goli na lem|
halanki apako ayarana, kailshiyama, pholeta jaise kuchha tatvom ki adhika jarurata hoti hai, lekina anya poshaka tatvom ki atyadhika matra apake bachche ko nukasana bhi pahuncha sakati hai|
eka svastha bachche ki shuruata paushtika bhojana se hi honi chahie|
jaba apa garbhavati haim, to jo kuchha bhi apa khati ya piti haim ,usase apake bachche ka bhi poshana hota hai |
isilie yahi karana hai ki paushtika bhojana khana bahuta mahatvapurna hai |
garbhavati mahila ko apani jaruratom ko pura karane ke lie ,dinabhara mem kevala 300 kailori ki atirikta jarurata hoti hai ,jisase vaha apane bachche ko avashyaka poshaka tatva de sake|
isalie,atyadhika kailori, vasa,chini aura sodiyama se paraheja karate hue,apaka lakshya behada paushtika ahara lena hi hona chahie|
prasavapurva dekhabhala svastha garbhavastha ka eka mahatvapurna hissa hoti hai|
chahe apa parivara ke chikitsaka,prasuti -vijnani, dai ya narsa mem se kisi ko bhi chunem ,prasavapurva dekhabhala apaki garbhavastha ke daurana apake aura apake bachche ke svasthya ki nigarani ke lie bahuta hi mahatvapurna hoti hai |
prasavapurva dekhabhala ki kuchha arambhika janchom mem se kuchha yaham batai ja rahim haim|
jaise hi apako lagata hai ki apa garbhavati haim,to pahali bara daॉktara ke pasa jana hi apaki prathama prasavapurva jancha hogi| 
apaki daॉktara, apake masika dharma ,garbha nirodhaka, pichhala garbhadharana, aura koi bhi elarji ya anya chikitsiya sthitiyom ke bare mem janakari lene ke lie apase kai savala puchhegi|
apa jo bhi davaem le rahim hom ya daॉktara ki salaha ke bina bhi jo davaem le rahim hom ,unhem apani daॉktara ko avashya bataem|
parivara ki kisi bhi janmajata asamanyataem ya anuvanshika bimariyom ke itihasa ko apane daॉktara se jarura bataem|
koi bhi janakari jo apa apane daॉktara ko batatim haim ,apa aura apake bachche ke svasthya ki dekhabhala karane mem madada karati hai| 
garbhavastha mem jaldi hi apake prasava ki niyata tarikha batakara ,apaki daॉktara jitana sambhava ho sakata hai ,utane hi sahi dhanga se apake bachche ke vikasa para najara rakhati hai|
apaki niyata tarikha ka ankalana karane ke lie, apaka svasthya sanrakshana data apake akhiri masika ki shuruata se agale 40 saptahom ki ganana karega|
apaka svasthya sanrakshana data apake vajana, lambai aura raktadaba ki jancha karega|
vaha apake hridaya ko sunega aura apake sampurna svasthya ka ankalana karega|
apaka svasthya sanrakshana data kisi bhi sankramanom ya asamanyataom ke lie apaki yoni aura apake garbhashaya ke mukha (garbhashaya griva) ka parikshana karega|
graiva kainsara ki jancha karane ke lie apako eka paipa parikshana ki avashyakata ho sakati hai|
garbhashaya griva aura apake garbhashaya ke akara mem parivartana apaki garbhavastha ke stara ko nishchita karane mem sahayata kara sakata hai| 
apaka svasthya sanrakshana data apake rakta ke prakara ka nirdharana karane ke lie, araecha ghataka - lala rakta koshikaom ki sataha para eka vishesha protina sahita, rakta ki jancha karega|
rakta jancha yaha bhi bata sakate haim ki apa upadansha (siphilisa), khasara, galasua, rubela ya yakrita-shotha bi (hepaitaitisa bi ) ke samparka mem rahe haim| apako echaaivi, vaha vishanu jo edsa utpanna karata hai, ki jancha ke lie prastava diya ja sakata hai| 
chhoti mata aura toksoplasmosisa ki nirapadata ki bhi jancha ki ja sakati haim|
apake mutra ka vishleshana mutrashaya ya gurde ke sankramana ke bare me bata sakata hai|
apake mutra me shakkara ya protina ki bahuta adhika upasthiti madhumeha ya gurde ki bimari ki ora sanketa kara sakata hai|
svastha jivanashaili ka chayana apake bachche ko sabase achchhi shuruata dene mem madada kara sakata hai|
apaka svasthya sanrakshana data apase poshana, prasavapurva ke khanijom, vyayama aura jivana shaili ke anya vishayom ke bare mem bata karenge|
yadi apa dhunrapana karate haim, to apaka svasthya sanrakshana data apako chhoda़ dene mem madada karane ke lie sujava dega|
prasavapurva ki jancha apake shishu ke svasthya ke bare mem apako bahumulya janakari de sakati haim|
apaka svasthya sanrakshana data bhrunasambandhi asamanyataom ka pata lagane ke lie paradhvanika chitrana (altrasaunda), rakta jancha ya anya anvikshana parikshanom ki salaha de sakata hai|
pahali timahi mem prayah hara chara se chhah saptahom mem niyata- prasavapurva ke nirikshana, bada mem - sambhavatah pahale se aura kama ho jaenge|
apaka svasthya sanrakshana data apake vajana aura raktadaba ki jancha karega, aura apa apane sanketom aura lakshanom para vichara-vimarsha karenge|
sambhavatah apaki garbhavastha ke bada taka apako eka aura shroniya pariksha ki avashyakata nahim hogi|
pahale timahi ki samapti ke naja़dika, apa apane shishu ki dila ki dhada़kana ko eka chhote yantra jo dhvani tarango ko apake shishu ke dila se dura uchhalata hai, se sunane mem sakshama ho sakate haim|
yada rakhe, apaka svasthya sanrakshana data apaki garbhavastha ke shuru se anta taka apaki madada karane ke lie vaham hai|
prasavapurva ke milane ke nishchita samaya, una vicharom sahita jo kashtaprada ya lajjajanaka ho sakate haim, kisi bhi prashna ya chintaom ke bare mem vichara-vimarsha karane ke lie adarsha samaya hote haim|
yaha bhi jnata kijie ki milane ke nishchita samaya ke bicha apake svasthya sanrakshana data taka kaise pahunche|
yaha janana ki jaba apako isaki avashyakata ho, sahayata upalabdha hai, yaha mana ko amulya shanti pradana kara sakata hai|
jyom hi apako shanka hoti hai ki apa garbhavati haim, to apake garbhavastha svasthya sanrakshana data, jaise ki prasuti visheshajna / striroga visheshajna, ke satha bhenta ka samaya nishchita karem|
yadi apa eka griha garbhavastha jancha se apane sandeha ko pakka kara bhi chuke haim, to bhi niyojita bhenta ki nirantarata banae rakhana buddhimata hai|
yaha sunishchita karega ki apa aura apake shishu ko eka achchhi shuruata mile|
apake aura apake shishu ke svasthya ke prati ashvasta hone ke lie apaki garbhavastha ke shuru se anta taka apake svasthya sanrakshana data ke satha niyamita bhenta mahatvapurna haim|
chikitsiya dekha-rekha ke atirikta, prasavapurva ki dekha-rekha mem garbhavastha va bachche ke janma para shiksha, sahita sahara tatha margadarshana sammilita hai|
apake svasthya sanrakshana data ke satha niyamita bhenta apake shishu ke vikasa ki pragati ko banae rakhane ki anumati deta hai|
bhenta apako prashna puchhane ka bhi mauka dete haim|
isake satha hi, adhikatara svasthya sanrakshana data pratyeka bhenta mem, apake jivana-sathi ke satha-satha dilachaspi lene vale parivarika sadasyom ka bhi svagata karate haim|
pahali bhenta apaki garbhavastha ko nishchita karane aura apake samanya svasthya ko nirdharita karane ke lie banaya gaya hai|
isake atirikta, bhenta apake svasthya sanrakshana data ko kisi bhi jokhima karakom ke sanketa dega jo ki apaki garbhavastha ko prabhavita kara sakata hai|
yaha bhavishya ke bhentom se vishesha rupa se adhika lamba hoga|
apa se pichhali garbhavasthaom aura shalya-chikitsaom, chikitsa paristhitiyom aura kisi bhi sankramaka bimariyom se samparka ke bare mem puchha jaega|
isake satha hi, apake svasthya sanrakshana data ko kisi bhi aushadhiyom (nuskhe dvara nirdharita ya bina nuskhe ke sidhe kauntara se li gai) ke bare mem, jo apane li haim ya vartamana mem le rahe haim, suchita karem|
koi bhi prashna jo apake pasa ho sakate haim use apane pradata se puchhane mem sankocha na karem|
sabase adhika sambhava hai, ki ve vahi prashna haim jinhem apake pradata sabase adhika bara sunate haim prasavapurva ki dekha-rekha, vaha dekha-rekha hai jo mahila ko garbhavastha se pahale aura usake bachche ke janma se purva milati hai|
prarambhika aura niyamita prasavapurva ki dekha-rekha milana mam aura vikasashila shishu donom ke lie mahatvapurna hai|
prasavapurva ki dekha-rekha mem garbhavastha ke bhinna pahaluom ko kaise sambhalem, isake bare mem shiksha aura margadarshana ke satha svasthya ki dekha-rekha bhi sammilita hai|
eka svasthya sanrakshana data aneka vishayom para vichara-vimarsha kara sakata hai, jaise ki poshana aura sharirika kriyakalapa, janma ki prakriya ke daurana kya asha karem, aura eka shishu ki dekha-rekha karane ke lie adharabhuta kaushala|
isake atirikta, svasthya sanrakshana data aba eka mahila ko yaham taka ki usake garbhadharana karane ki koshisha se pahale hi, garbhadharana ke purva ki dekha-rekha ke lie eka svasthya sanrakshana data se milane ki salaha de rahe haim|
aneka svasthya sanrakshana data salaha dete haim ki eka mahila jo garbhadharana karane ke bare mem socha rahi hai, vaha, garbhadharana se purva usaka achchha svasthya hai, yaha sunishchita karane ke lie eka svasthya sanrakshana data se mile|
yu.esa. sarvajanika svasthya seva (yu.esa. pablika heltha) salaha deta hai ki bachcha janane ke uchita unra ki mahila, hara dina pholika esida ka kama se kama 400 maikrograma, bhojana / ya purakom ke madhyama se le|
aneka svasthya sanrakshana data garbhadharana karane ke tina mahine pahale aura garbhavastha ke kama se kama pahale tina mahinom taka ahara ko pholika esida se pura karane ki salaha dete haim|
mahilaem jo garbhadharana karane ke bare mem sochati haim unhem yaha nishchita karana chahie ki unaka vishesha rogom ke lie jo vikasashila bhruna ko hani pahuncha sakate haim (jaise ki chhoti-mata ya rubela), uchita rupa se tikakarana ho chuka hai|
garbhadharana karane se pahale ina tikom ko lena, mahatvapurna hai kyonki ve vikasashila bhruna ko hani pahuncha sakate haim|
svastha vajana aura ahara hona tatha niyamita sharirika kriyakalapa karana garbhavastha ke daurana mam aura bhruna donom ki sahayata kara sakate haim|
dhunrapana, madirapana, ya nashile padartha ke istemala se bachana bhi mahatvapurna hai|
isake satha-satha, vishesha aushadhiyom aura vatavarana se samparka bhruna ke lie hanikaraka ho sakate haim aura garbhavastha ke daurana inase bachana, yaha mahatvapurna hai|
prarambhika aura niyamita prasavapurva ki dekha-rekha milana eka svastha garbhavastha ko badha़ava dene ka sabase achchha tarika hai|
prasavapurva ki dekha-rekha sirpha़ svasthya ki dekha-rekha se adhika hai; isamem prayah garbhavastha ke vibhinna pahaluom ko kaise sambhale, isake bare mem shiksha aura margadarshana sammilita hota hai, jaise ki poshana aura sharirika kriyakalapa, svanya janma se kya asha karem, aura apake shishu ki dekha-rekha ke lie adharabhuta kaushala|
prasavapurva ki bhenta apako aura apake parivara ko apake svasthya sanrakshana data se kisi bhi prashnom ya vishayom, jise apane apaki garbhavastha, janma, ya pitritva se joda़ liya hai, ke bare mem bata karane ka eka mauka bhi deti hai|
aneka svasthya sanrakshana data salaha dete haim ki eka mahila jo kevala garbhadharana karane ke bare mem sochati hai, vaha garbhadharana se purva ke svasthya sandarbha mem svasthya sanrakshana data se mile|
aise kadama haim jise vaha kuchha pareshaniyom ke jokhima ko kama karane ke lie utha sakati hai|
mahilaem jo garbhadharana karane ke bare me socha rahi haim unhem sunishchita karana chahie ki unaka vishesha bimariyom jaise ki rubela (jarmana khasara bhi kahalata hai), ke lie uchita rupa se tikakarana hua ho aura pratirakshita ho chuke hom |
yadi eka mahila ko jaba vaha garbhavati hai taba rubela ho jata hai, to bhruna baharapana, hridaya ki samasyaem, ankhom ke motiyabinda, aura manasika mandana, sirpha nama lene ke kuchha sahita, vibhinna prakara ki pareshaniyom ke badha़e hue khatare para hota hai|
ye samasyaem, jo eka satha janmajata rubela kahalati haim (kaॉn-jinn-ita-ula ru-bela-yuha) eka bachche ke rupa mem rubela hone ke prabhava se bahuta adhika prachanda hoti haim| 
svasthya sanrakshana data isa sankramana se pratiraksha ke lie prayah eka mahila ke rakta ki jancha karate haim, ya ve eka mahila ke svasthya vivarana se balyavastha ke pratirakshanom ke pramana ko jnata karane ki koshisha karate haim|
yadi eka mahila rubela se pratirakshita nahim hai, to use tika lagana chahie|
amariki mahavidyalaya ke dhatu-vijnani aura prasuti-visheshajna (amerikana kaॉleja ऑpha ऑbstetrishiyansa enda gainikolaॉjistsa (esioji)) salaha dete haim ki mahilaem garbhadharana karane ki koshisha karane se pahale tika lene ke kama se kama eka mahine bada taka intajara karem|
esioji yaha salaha bhi dete haim ki isa avadhi mem mahilaem garbhavastha se bachane ke lie kadama uthaem|
yaha samayavadhi, tika/bimari jo bhruna ko prabhavita kara sakati hai, ke kisi bhi vilamba sanketom ke viruddha raksha kara sakati hai|
eka mahila jo pahale hi se garbhavati hai use rubela ka tika nahim lagavana chahie| 
isake satha-satha, yadi eka mahila ko jaba vaha garbhavati hai, taba chhoti mata ho jata hai, to bhruna kai samasyaom ke badha़e hue khatare para hota hai, jo eka satha kaॉnjinaitala verisela (janmajata chhoti mata) (vaira-i-sela-uha) kahalate haim|
verisela nama verisela ja़ostara vishanu kahalane vale usa vishanu ke nama se ata hai jo chhoti mata utpanna karata hai|
kaॉnjinaitala verisela (janmajata chhoti mata) eka bachche ke rupa mem hone vali chhoti mata ke prabhavom se adhika gambhira hota hai|
yaha tvacha ke dagadara hone , etrophi (eta-ro-phi) kahalane vale bahom aura pairom ke kamajora ya apakshaya hone, aura ankho ki asamanyataom mem parinita ho sakata hai|
jaise ve rubela ke lie karate haim, thika vaise hi svasthya sanrakshana data sambhavatah verisela se pratiraksha ke lie eka mahila ke rakta ki jancha karenge, ya ve unake svasthya vivarana se balyavastha pratirakshana ke pramana ko jnata karane ki koshisha karenge|
yadi eka mahila verisela se pratirakshita nahim hai, to use tika lagana chahie|
roga niyantrana aura roka-thama ke kendrom ka bhaga, pratirakshana karyapranaliyom mem salahakara samiti (esiaipi) salaha deta hai ki bachchajanane ki unra vali sabhi agarbhavati mahilaom ko verisela ke virudhda tike lagaye jaem|
esiaipi aura amerikana ekedemi ऑpha pidiyatriksa (balaroga-vijnana ka amariki akadami) bhi salaha dete haim ki mahilaem garbhadharana karane ki koshisha karane se pahale tika lene ke kama se kama eka mahine bada taka intajara karem|
yaha samaya, tika/bimari jo bhruna ko prabhavita kara sakati hai, kisi bhi vilamba sanketom ke viruddha raksha kara sakata hai| 
kevala vitamina aura pratirakshana hi aisi vastuem nahim haim jo svastha garbhavastha ko badha़ava dene mem madada kara sakati haim|
vastuem jaise ahara, sharirika kriyakalapa, aushadhiyam, dhunrapana, madirapana ya nashile padartha ka istemala, aura paryavarika karaka sabhi garbhavastha ko prabhavita kara sakate haim|
eka svastha ahara, vajana stara, aura niyamita sharirika kriyakalapa stara garbhavastha ke daurana mata aura bhruna donom ki samasyaom ko kama karane mem madada kara sakata hai|
isa karana se, aneka svasthya sanrakshana data sujava dete haim ki jo mahilaem garbhadharana karane ke bare mem socha rahi haim ve garbhadharana karane ke pahale apane khuda ke svasthya stara ko badha़ane ya banaye rakhane ke lie kadama uthaem|
eka svasthyavardhaka ahara yaha sunishchita karane mem madada karata hai ki bhruna ke pasa badha़ne aura samanya rupa se vikasita hone ke lie sabhi poshana haim jisaki use avashyakata hoti hai|
svastha vajana banaye rakhana, donom garbhavastha ke daurana aura pahale, mata ke sharira se tanava kama karane aura garbhavastha ki vishesha gaड़baड़iyom ke jokhima ko kama karane mem madada kara sakata hai|
yadi svasthya sanrakshana data dvara anumodita ho to, garbhavastha ke daurana aura pahale sakriya rahana, mahilaom ko apane svastha vajana banaye rakhane mem sahayata kara sakata hai aura parisanchari, hridavahini, aura kankala tantro ke karya ko behatara kara sakata hai|
thika usi prakara, mata ke sharira ko jo kha़taranaka ho sakate haim aisi chijom se dura rakhana bhi mahatvapurna hai|
udaharanasvarupa, vibhinna bimariyom aura paristhitiyom ka ilaja karane ke lie istemala ki gaim aushadhiyam bhruna ki vriddhi aura vikasa ko prabhavita kara sakati haim|
kuchha jada़i-buti yukta anupurakom aura vitaminom ki uchcha matraem bhi eka mahila ke garbhadharana karane ko aura adhika kathina bana sakati haim, aura garbhavastha ke daurana bhruna ke svasthya para gahara prabhava dala sakati haim|
yaham taka ki kuchha padarthom jaise ki penta aura kitanashakom ke asa-pasa rahana bhi bhruna ke svasthya ko jokhima mem dala sakata hai|
mahilaem jo garbhadharana karane ke bare me socha rahi haim, unhem apane svasthya sanrakshana data ke satha ina sabhi karakom para vichara-vimarsha karana chahie|
garbhavastha ke daurana ki samasyaom se bachane ke lie aushadhi aura anupuraka ke upayoga mem kuchha parivartana, ya vatavarana mem parivartana ki salaha di ja sakati hai|
shodha darshata hai ki garbhavastha ke daurana dhunrapana, madirapana, ya nashile padarthom ka upayoga, yaham taka ki abhi aura phira se ya kama matraom mem sevana bhi, bhruna ke lie svasthya samasyaem utpanna kara sakate haim, unamem se kuchha gambhira ho sakate haim|
inamem se aneka samasyaom ke ajivana prabhava ho sakate haim|
madira aura nashile padarthom ka sevana bhi kuchha mahilaom ke lie garbhadharana karane ko aura adhika kathina bana sakata hai|
garbhavastha ke daurana ki samasyaom ke jokhima ko kama karane ke lie, svasthya sanrakshana data salaha dete haim ki mahilaem garbhadharana karane ki koshisha karane ki shuruata se pahale hi jitani jaldi ho sake dhunrapana banda karem, madirapana banda karem, aura nashile padarthom ke sevana ko purna rupa se roka dem|
unhem isa tambakurahita, madirarahita aura nashile padartharahita jivanashaili ko apane garbhavasthaom ke shuru se anta aura prasava ke bada taka banaye rakhana chahie, kyonki inamem se aneka padartha stanapana ke dvara shishu ke sharira mem ja sakate haim|
yadi apa garbhadharana karane ke bare me socha rahi haim, to apane svasthya sanrakshana data se kaise eka svastha garbhavastha ko sabase achchha badha़ava dem, ke bare mem bata karem, eka taraha se yaha apaki chikitsa vivarana aura jivanashaili ko dhyana mem leta hai|
bimari niyantrana aura nivarana kendra (sentarsa phaॉra disija़ kantrola enda privenshana (sidisi)) ke anusara lagabhaga 4miliyana (40 lakha) amariki mahilaem hara varsha janma deti haim|
unamem se lagabhaga eka tihai ko kuchha prakara ki garbhavastha-sambandhi jatilata hogi| 
ve jinhem prasavapurva samuchita dekharekha nahim milati hai ve aise jokhima uthate haim ki aisi jatilataem gupta raha jaengi ya paryapta shighrata se hala nahim ho paengi|
jo, parinamasvarupa, mata aura usake shishu donom ke lie sashakta rupa se gambhira natijom ki ora agrasara ho sakate haim|
ye ankada़e apako bhayabhita karane ke uddeshya se nahim haim, parantu nishchaya hi jitani jaldi ho sake - adarshatah, yaham taka ki apake garbhadharana karane se pahale hi prasavapurva ki dekharekha ko prarambha karane ke mahatva ko batalana hai|
nissandeha, yaha hamesha sambhava va vyavaharika nahim hai|
parantu apaki garbhavastha mem jitani jaldi apa shuruata karate haim, apake svayam ke svasthya aura vahi apake shishu ke lie bhi sunishchita karane ke behatara avasara hote haim|
adarshatah, purvaprasava ki dekha-rekha apake garbhavati hone se purva prarambha honi chahie|
yadi apa garbhavastha ki yojana bana rahe haim, to sampurna parikshana ke lie apake svasthya sanrakshana data se milie|
vaha yaha sunishchita karane ke lie samanya jancha kara sakata hai ki apaka svasthya achchha hai aura apako koi bimari ya anya roga nahim haim jo apaki garbhavastha ko prabhavita kara sakate haim|
yadi apa koi asamanya lakshana anubhava karate rahe haim, to yaha unhem batane ka eka achchha avasara hai|
yadi apaka pahale se hi eka jirna roga, jaise ki madhumeha, dama, uchcha raktachapa (uchcha rakta daba), hridaya samasya, pratyurjataem (elarjisa), lampasa (eka uttejaka vikara jo kai sharirika tantrom ko prabhavita kara sakata hai), avasada, ya kuchha anya roga, ke lie upachara ho raha hai, to apako apane chikitsaka se isake bare mem bata karani chahie ki yaha garbhavastha ko kaise prabhavita kara sakata hai|
kuchha mamalom mem, bhruna ke lie khatare ko kama karane ke lie apako aushadhiyam badalane ya chhoda़ dene ki avashyakata ho sakati hai – vishesha rupa se prathama traimasika avadhi ke daurana (12 haphte)|
ya, apako apane roga ko niyantrita karane ke bare mem aura bhi adhika satarka rahane ki avashyakata ho sakati hai|
udaharana ke lie, madhumeha vali mahilaom ko donom samaya, apane garbhavati hone ki koshisha karane ki shuruata karane se pahale aura apane garbhavastha ke daurana, apane rakta sharkara stara ko niyantrana mem rakhane ke bare mem vishesha rupa se savadhana rahana chahie|
asamanya stara janma doshom aura anya jatilataom ke khatare ko badha़ate haim|
yaha apane svasthya sanrakshana data se apani adatom, jaise ki madira pana ya dhunrapana jo ki apake shishu ke lie eka khatara khada़a kara sakata hai, ke bare mem bata karane ka bhi eka achchha samaya hai|
eka prasavapurva ke vitamina jisamem pholika anla, kailshiyama, aura lauha hote haim, ko prarambha karane ke bare mem puchhem|
mahilaem jo garbhavati hone ka upaya kara rahi haim unake lie yaha vishesha rupa se mahattvapurna hai ki ve pahale se hi pholika anla vale vitaminom ko lem , kyonki tantrikiya nali ke dosha (merudanda aura tantrika tantra ke samanya vikasa ki samasyaem) garbhavastha ke prathama 28 dinom mem hote haim, prayah isase pahale jaba eka mahila ko yaha pata lagata hai ki vaha garbhavati hai|
yadi apaka ya apake sathi ka eka mahattvapurna anuvanshika vikara ka eka parivarika itihasa hai aura apako yaha sandeha hai ki apa mem se koi eka usaka vahaka ho sakata hai, taba anuvanshika parikshana uchita ho sakata hai|
apake svasthya sanrakshana data se isa para bata karem, jo yadi avashyakata ho to apako eka anuvanshika paramarshadata ke pasa bheja sakata hai|
yadi apako yaha saba karane se pahale hi pata chalata hai ki apa garbhavati haim, to chinta na karem|
yaha vaha seva pane ke lie bahuta dera nahim hai jo apake aura apake shishu ke svasthya ko sunishchita karane mem sahayata karega|
yadi apa svastha haim aura apaki garbhavastha tatha prasava se jatilataom ki pratyasha karane ka koi karana nahim hai to inamem se koi bhi sanrakshana data eka achchha chayana hai|
phira bhi, narsa-daiyom ko prasava ke lie agara sija़eriyana sekshana (garbhashaya mem chira lagakara bhruna ko bahara nikalana) karana avashyaka ho to eka chikitsaka rakhane ki avashyakata hai|
yadi apaki garbhavastha adhika khatare mem nahim hai phira bhi svasthya sanrakshana data badalane ka yaha eka achchha samaya hai yadi apa apane vartamana chikitsaka se khusha nahim haim to|
apako apani garbhavastha ke 6 se 8 hapha़tom ke daurana, ya jaba apaki masika avadhi 2 se 4 hapha़te dera ho to apako pahali jancha ke lie phona karana aura samaya niyata karana chahie|
abhi aisa karana apake svasthya sanrakshana data ko apake garbhavastha ki avadhi ankane aura apaki prasava tithi ka purvanumana karane mem sahayata karata hai|
apaki pahali bhenta ke daurana, apa shroni tatha malashayi jancha sahita, sampurna sharirika jancha hone ki ummida kara sakate haim|
aphriki ya bhumadhyasagariya vanshaja ki mahilaom ka sikala sela (hansiya ke akara ki koshika) vishishtata ya roga ke lie parikshana ho sakata hai kyonki ve sikala sela raktalpata hone - eka jirna rakta roga - ya vishishtata ko vahana karane ke uchchatara sankata mem hote haim jo unake bachchom mem ja sakate haim|
isa pahali bhenta ke daurana, apa parikshana ke lie mutra ka namuna dene aura shroni ke kainsara ke lie pepa testa (ya smiyara) hone ki bhi ummida kara sakate haim|
pepa smiyara karane ke lie, apaka svasthya sanrakshana data garbhashaya griva (garbhashaya ka mukha jo yoni ke thika shirsha para sthita hota hai) ke bhitara dhire se kharonchane ke lie eka bahuta lamba maskara dandi ya rui ke phaha jaise dikhane vali vastu ka prayoga karata hai|
yaha samanyatah nahim dukhata hai; kuchha mahilaem kahati haim ki ve thoda़i tisa mahasusa karati haim, kintu yaha eka sekanda taka hi rahata hai|
hara jancha mem, apaka vajana aura rakta daba samanyatah likhe jate haim|
22vem saptaha ki shuruata mem, yaha nirdharita karane ke lie ki bhruna samanya rupa se badha़ aura vikasita ho raha hai ki nahim, apake garbhashaya ki lambai-chauda़ai tatha akara ko bhi mapa ja sakata hai|
apake bhentom ke eka ya adhika ke daurana, apase sharkara tatha protina ka parikshana karane ke lie eka chhota mutra namuna dene ke lie puchha jaega|
protina prakgarbhakshepaka (prieklaimpasiya) ki ora sanketa kara sakata hai (eka roga jo garbhavastha ke uttarakala mem vikasita hota hai aura rakta daba ki achanaka vriddhi aura drava dharana tatha mutra mem protina ke satha, atyadhika vajana vriddhi se visheshita hota hai)|
madhumeha ke lie parikshana una mahilaom ke lie samanyatah 12 haphtom mem hota hai jo garbhavastha ke madhumeha (madhumeha jo garbhavastha ke daurana hota hai) hone ke uchchatara sankata mem hote haim|
anya sabhi garbhavati mahilaom ka 24 se 28 hapha़tom mem madhumeha ke lie parikshana kiya jata hai|
isa parikshana mem eka sharkarayukta drava pina aura eka ghante bada rakta glukosa parikshana (jisamem rakta nikalana shamila hai) karavana nihita hai|
yadi rakta mem sharkara ka stara uchcha hai, to garbhavastha ke madhumeha ka nidana karane ke lie aura adhika parikshana kie ja sakate haim|
kai bhavi mata-pita ninnalikhita prasavapurva ke parikshanom mem se eka ya adhika ko karavane ka bhi nirnaya lete haim jo sambhavana ka purvanumana karane ya yaham taka ki bhruna mem vishesha vikasatmaka ya gunasutra-sambandhi asamanyataom ki upasthiti ka bhi pata lagane mem sahayata kara sakate haim|
16 aura 18 haphtom ke bicha, mahila ke rakta mem bhruna dvara utpadita eka protina, alpha-phetoprotina ka stara mapa ja sakata hai|
yadi stara uchcha hai, to vaha eka se adhika bhruna ya ayukta merudanda (spaina baiphida) ya anya tantrikiya nali dosha vale eka bhruna ko vahana kara sakati hai|
eka uchcha stara yaha bhi sanketa kara sakata hai ki garbhadhana ki tithi ki galata ganana ki gai thi|
yadi stara ninna hai, to bhruna mem gunasutra-sambandhi asamantaem, jaise ki dauna sindroma ho sakati haim|
yaha dimaga़ mem rakhana atyavashyaka hai ki ninna ya uchcha parinama svatah se eka samasya ki ora sanketa nahim karate haim, ve aura parikshana ki avashyakata ki ora sanketa karate haim, jo kai mamalom mem samanya parinama batate haim|
16 aura 18 haphtom ke bicha bhi, apaka svasthya sanrakshana data eephapi ke satha-satha, apake rakta mem vishishta harmonom ke starom ko mapa sakata hai|
isa parikshana ke lie, eephapi ke stara ke satha-satha, echasiji (hyumana korionika gonadotropina) aura estriola, jo jarayunala dvara utpanna kie jate haim, ke starom ko mapane ke lie mam se rakta ka eka namuna nikala jata hai|
rakta mem ina tina padarthom (isilie yaha parikshana prayah triguna parikshana ya triguna chihnaka kahalata hai) ke stara chikitsakom ki eka bhruna ke vishishta janma vikarom ya gunasutra-sambandhi asamantaom ke khatare ko pahachanane mem sahayata karate haim|
jaba inahibina-e namaka, eka atirikta padartha ke stara ko bhi mapa jata hai, to yaha parikshana chatushparikshana (ya chihnaka) kahalata hai|
isa parikshana mem, garbhashaya se ulva (enniyotika) drava ka eka namuna nikalane ke lie eka sui ka prayoga kiya jata hai; yaha samanyatah 15 aura 20 haphtom ke bicha kiya jata hai|
drava ka parikshana vishishta ghataka asamanyataom jaise ki dauna sindroma ya ayukta merudanda (spaina baiphida) ki pahachana kara sakata hai|
vishishta rupa se, yadi yaha vishvasa karane ka koi karana hai ki aise rogom ke khatare, shayada matri-ayu (35 ya adhika), asamanya eephapi ya vividha chihnaka parikshana parinamom, ya parivarika itihasa ke karana samanya se adhika hai, taba hi enniyosentesisa (sui se udara bhitti mem se hokara ulva-thaili mem chheda karana) ki salaha di jati hai|
halanki isa parikshana mem purva prasava aura preranatmaka garbhapata hone ka thoda़a khatara hota hai, para isaki bahuta bada़i sankhya bina kisi samasya ke karyanvita ki jati haim|
yaha prakriya prathama traimasika avadhi ke daurana eka enniyosentesisa ke samana uddeshya ke lie upayoga mem lai jati hai|
isamem utaka jo ulva-thaili (bhruna ke charom ora ki thaili) ko garbhashaya ki divara se chipakata hai, usaka eka namuna lena sammilita hai|
enniyosentesisa ki taraha, siviesa bhi vishishta rupa se taba hi kiya jata hai jaba vishesha khatara karaka hote haim; isaka prathamika labha yaha hai ki parinama jaldi se mila jate haim|
siviesa mem bhi garbhapata aura anya jatilataom ka thoda़a badha़a hua khatara hota hai|
garbhavastha samanya rupa se vikasa kara raha hai yaha sunishchita karane ke lie aura prasava ki apekshita tithi nishchaya karane ke lie apako sambhavatah kama se kama eka paradhvani parikshana karavana hoga|
samanyatah, shishu ki sharirika sanrachana ko dekhane ke lie 18 se 20 hapha़tom mem eka paradhvani parikshana kiya jata hai, para yaha aura jaldi ya bada mem tatha kabhi-kabhi eka se adhika bara bhi kiya ja sakata hai|
eka paradhvanika chitrana se apako ya apake shishu ko koi khatara nahim hota hai|
kuchha svasthya sanrakshana dataom ke pasa ऑphisa mem paradhvanika chitranom ko pradana karane ke lie yaha upakarana aura avashyaka prashikshita karmachari hote haim, jabaki anya apako sthaniya aspatala ya vikirana chikitsa kendra mem bheja sakate haim|
jaham kahim bhi paradhvanika chitrana kiya jata hai, eka takanikajna apake peta ko eka jela se dhaka dega aura taba eka chhada़i ki taraha ka yantra isake upara chalaega|
apake sharira se uchcha-avriti ki dhvani tarangem pratidhvanita hoti haim aura bhitara ke bhruna ka eka chitra eka sanganaka ke parde para banati haim|
apake shishu ka chhayachitra lene ke eka tarike ke rupa mem shaॉpinga malom mem paradhvanika chitrana upalabdha haim|
phira bhi, upakarana ka prayoga karane vale sabhi vyakti paradhvanika chitrana vale takanikajnom ki taraha avashyaka rupa se prashikshita na hom|
inamem se eka ko karavane ke lie vachanabaddha hone se pahale, apake svasthya sanrakshana data se vichara-vimarsha karana, buddhimani hogi|
kuchha mahilaem purvaprachalita chikitsiya rogom jaise ki madhumeha, aura kaise ve garbhavastha ko prabhavita kara sakati haim ke bare mem chinta karati haim|
ina chintaom ke bare mem apake svasthya sanrakshana data se vichara-vimarsha karana atyavashyaka hai, jo aushadhi ya upachara paddhatiyom mem parivartana ki salaha de sakata hai jo ki apaki chintaom ko kama kara sakata hai|
garbhavastha ka madhumeha - garbhavati mahilaom ke 8% taka mem, samanyatah prathama traimasika avadhi ke bada yaha roga hota hai|
garbhavastha ke daurana, jarayunala, jo bhruna ko poshana aura aksijana pradana karata hai, harmona bhi utpanna karata hai jo ki insulina ke kama karane ke tarike ko badala dete haim|
insulina eka padartha hai jo agnyashaya dvara banaya jata hai|
yaha bhojana ke sharkara ko sanchita karane mem sharira ki sahayata karata hai taki bada mem yaha urja mem parivartita kiya ja sakata hai|
jaba kisi ko garbhavastha ka madhumeha hota hai, to insulina ke satha ki samasya uchcha rakta sharkara stara ki ora bhi pravritta hoti hai|
prakgarbhakshepaka (prieklaimpasiya) – eka asamanya roga jo chhah mahine ke bada hota hai, yaha uchcha rakta daba, idima ( hatha, paira, ya chehare ke sujana mem parinita sharira ke utakom mem drava ka sanchayana), aura mutra mem protina utpanna karata hai|
araecha ghataka adhikatara logom ki lala rakta koshikaom mem paya jane vala eka padartha hai (eka samanya rakta parikshana apake araecha ghataka ko nirdharita kara sakata hai)|
yadi apa mem yaha nahim hai, taba apako araecha rinatmaka mana jata hai|
yadi apake shishu mem yaha ghataka hai aura araecha dhanatmaka hai, to jaba shishu ki rakta koshikaem apake rakta pravaha mem pravesha karati haim taba samasyaom mem parinita ho sakati haim|
yaha isalie kyonki apaka sharira roga-pratikarakom ko utpanna karake pratikriya kara sakata hai jo bhruna ke rakta pravaha mem pahuncha sakate haim aura lala rakta koshikaom ko nashta kara sakate haim|
ye roga gambhira kintu niyantraniya haim, isalie apane apa ko unake bare mem shikshita karana aura unake bare mem apake svasthya sanrakshana data se vichara-vimarsha karana atyavashyaka hai|
garbhavati mahilaem prayah vajana vriddhi ke bare mem bhi chinta karati haim|
samanyatah yaha salaha di jati hai ki samanya vajana ki eka mahila garbhavastha ke daurana lagabhaga 25 se 30 paunda badha़ae|
una vyaktiyom ke lie jo apani garbhavastha mutape se prarambha karate haim, unaki kula vajana vriddhi 15 se 25 paunda ke asa-pasa honi chahie|
aura jo dubale-patale haim unhem 28 se 40 paunda badha़ana chahie|
garbhavastha alpahara prarambha karane ka eka achchha samaya nahim hai, yaha svasthyaprada bhojana khana arambha karane ka eka utkrishta samaya ho sakata hai|
yaha niyamita, kama-prabhavi vyayama karane ka bhi eka achchha samaya hai|
garbhavastha mem bada mem vajana vriddhi ko niyantrita karana adhika kathina hai, atah pahale kuchha mahinom ke daurana bahuta adhika vajana badha़ane se bachane ki koshisha karem|
phira bhi, paryapta vajana nahim badha़ana, aparyapta bhruna vriddhi aura asamayika prasava jaisi samasyaem bhi paida kara sakati haim|
chikitsaka samanyatah salaha dete haim ki mahilaem vikasashila bhruna ko poshana pradana karane ke lie apane pratidina ke antargrahana mem lagabhaga 300 kailori ki vriddhi karem|
halanki protina se inamem se adhikatara kailoriyom ki apurti honi chahie, phira bhi apake ahara ko taje phalom, anajom, aura sabjiyom sahita, susantulita hona ja़ruri hai|
apaka svasthya sanrakshana data apako paryapta pholika anla, lauha, tatha kailshiyama mile yaha sunishchita karane ke lie sambhavatah prasavapurva ke eka vitamina ka nuskha likhega|
bina nuskhe ke sidhe kauntara para milane vali davaiyam bhruna para unake sambhavita prabhavom ke karana samanyatah pratibandhita mani jati haim|
adhikatara chikitsaka salaha denge ki apa bina nuskhe ke sidhe kauntara para milane vali koi bhi davaiyam bilkula na lem, para ve unaki eka suchi prastuta kara sakate haim jinhem ve lene ke lie surakshita samajate haim|
prakritika aushadhiyom, anupurakom, aura vitaminom sahita davaiyom ke bare mem apake svasthya sanrakshana data se kisi bhi prashna para vichara-vimarsha karana sunishchita karem|
jaba apa garbhavati haim, to ahara-janya rogom jaise ki listeriyosisa aura tokja़oplasmosisa, jo aprasuta shishu ke lie jivana- santarjaka ho sakata hai aura janma vikara ya garbhapata kara sakata hai, se bachana, bhi mahattvapurna hai|
apako sharka, tega machhali, kinga maikerela, ya tailamachhali khane se bhi paraheja karana chahie|
halanki machhali aura kavacha prani apaki garbhavastha ke ahara ka eka atyadhika svasthyaprada bhaga ho sakate haim (unamem labhadayaka omega-3 vasa anla aura protina adhika tatha santripta vasa kama hote haim), phira bhi isa prakara ki machhaliyom mem pare ke uchcha stara ho sakate haim, jo eka bhruna ke vikasashila mastishka ko kshati pahuncha sakate haim|
yadi apa ina parshva prabhavom mem eka ya adhika ka anubhava karate haim, to dhyana rakhem ki apa akele nahim haim|
apake chikitsaka se kisi bhi kashta ko kama karane ke upayom ke bare mem bata karem|
jaba apaka sharira sharirika parivartanom se gujara raha hai jo apake lie purnatah naya ho sakata hai, to apake svasthya sanrakshana data se bata karana, hamesha asana nahim hota hai|
ho sakata hai apako ashcharya ho raha hai ki apa sambhoga kara sakate haim ya nahim athava bavasira ya kabja़ ke bare mem kya karem, ya ho sakata hai ki apa prasava ke bare mem bahuta adhika chinta anubhava kara rahe hom|
inhem ya anya prashnom ko puchhane mem apa lajja anubhava kara sakati haim, para aisa karana atyavashyaka hai - apake svasthya sanrakshana data sambhavatah ina saba ko pahale hi suna chuke honge|
apake niyojita bhentom ke bicha prashnom ki nirantara suchi rakhem, aura usa suchi ko hara bhenta mem apane satha le jaem|
maleriya ka khatara, unake lie jo prabhavita kshetrom ki yatra kara rahe haim, ullekhaniya hai, aura isilie yaha bahuta mahattvapurna hai ki ise rokane ke lie savadhani baratem|
hara sala, lagabhaga 2,000 yatri maleriya ke satha yuke vapasa jate haim, jinamem se adhikansha ise aphrika se prapta kie hue hote haim|
ausatana, 12 loga hara sala isa roga se marate haim|
maleriya ke kai mamalom ko yaha janchakara ki jisa kshetra ki apa yatra kara rahe haim vaha jokhima bhara hai ya nahim, sahi tarika़e se maleriya virodhi aushadhiyam lekara, aura yadi bimara ho jate haim to shighra chikitsa salaha prapta karake roka ja sakata hai|
gantavya sthala' vibhaga ki yatra ke lie upayukta vebasaita dekhakara apa yaha jancha kara sakate haim ki apa jisa desha ki yatra kara rahe haim vaham ke lie apako nirodhatmaka maleriya upachara lene ki avashyakata hai ya nahim| ('chuninda kada़iyam' vibhaga dekhem)|yadi apane maleriya virodhi aushadhi pahale bhi li hai, to apa yaha na mane ki vaha bhavishya ke daurom ke lie bhi upayukta hogi|
vishishta kshetra mem jaham ki apa yatra kara rahe haim, machchharom dvara phailae gae maleriya ki nasla para nirbhara karate hue aushadhi jisaki apako avashyakata hai use badalane ki ja़rurata ho sakati hai|
maleriya ke sankramana (maleriya-virodhi) ko rokane hetu aushadhi lena una sabhi yatriyom ke lie jaruri hai jo aise kshetrom ki yatra kara rahe haim jo ki maleriya ke khatare mem haim|
yadi apako miragi (daure) ho, to apako klorokvaina aura mephlokvaina se bachana chahie kyonki ve ainthana (daure) paida kara sakate haim|
daॉksisailaina lene se pahale apako apane samanya chikitsaka se paramarsha lena chahie kyonki kuchha aushadhiyam jo apa miragi ko niyantrita karane ke lie shayada le rahe honge daॉksisailaina ko kama prabhavi bana sakati hai|
mephlokvaina le rahe kuchha loga ne chinta aura avasada ka anubhava kiya hai|
isalie, yadi sambhava ho, to apako mephlokvaina ko lene se bachana chahie yadi apaka gambhira manovikriti-sambandhi bimari ka itihasa ho to|
yadi apako yakrita ki koi samasya ho to apako apane samanya chikitsaka se maleriya-virodhi aushadhiyom ke chayana para vichara-vimarsha karana chahie|
yadi apako echaaivi ya edsa hai, to apako apane samanya chikitsaka se apani yatra yojana aura aushadhi chayana ke bare mem kahana chahie|
yadi apako gurde ki samasyaem haim, to apako progyuenila nahim lena chahie|
yadi anya maleriya nivarana aushadhiyom ki avashyakata hai to apako apane samanya chikitsaka se milana chahie|
yadi apa bita-blaॉkarsa ya anya aushadhiyam lete haim jo apake hridaya gati ko niyantrita karane mem madada karati haim, to apako mephlokvaina se paraheja karana chahie|
daॉksisailaina lene ke prathama tina saptahom taka, apako garbhanirodha ki eka atirikta paddhati ke upayoga ki avashyakata hogi, jaise ki eka kandoma, kyonki daॉksisaiklaina maukhika garbhanirodhaka goli ke prabhava ko kama kara sakata hai|
apane samanya sata dina ko chhoda़kara apako apani goli lena bhi jari rakhana chahie|
apako apane samanya chikitsaka ya parivara niyojana salahakara ke satha yaha vichara-vimarsha karana chahie|
yadi apa progyuenila le rahi haim, to apani yatra se pahale apako apane samanya chikitsaka ke satha yaha vichara-vimarsha karana chahie kyonki kabhi-kabhi progyuenila varapherina ke prabhava ko badala sakata hai|
yaha bahuta mahatvapurna hai ki apa sacheta haim ki kisi bhi phlu jaisi- bimari, ya raktalpata, jo ki una jagahom ki yatra se vapasi ke tina mahine ke andara hota hai jaham maleriya upasthita hai, to yaha maleriya ho sakata hai, bhale hi apane apani aushadhi li ho|
yadi apa bimara hote ho jaba apa vapasa ate ho, to apako apane samanya chikitsaka se milana chahie aura unhem batana chahie ki avakasha mem apa kaham the, aura yaha ki sambhavatah apa maleriya ke samparka mem ae honge|
sharad (aktubara/shuruati navambara) mem tika injekshana se lagana chahie|
e aura bi vishanu ke adhunika prakarom ke viruddha, phlu-virodhi tike, upalabdha haim aura lagabhaga 75% prabhavashali haim|
yadi apa tikakrita haim aura phlu hota hai, to isa bimari ki vikriti bahuta kama gambhira hone ki ora pravritta hoti hai|
enaechaesa ka phlu ke injekshana prapta karane vale logom ki sankhya ke lie lakshya hai tatha svasthya vibhaga varshika adhara para udgrahana akadom ka akalana karata hai|
jinaki unra 65 ya jyada hai unake lie, lakshya 70% hai aura, hala ke varshom mem, yaha adhika ho gaya hai|
yadi apa aise deshom ki yatra kara rahe haim jaham yaha vishanu samanya hai, jaise ki bharatiya upamahadvipa, aphrika, madhya aura dakshina amerika, durastha purvi bhaga aura purvi yuropa to hipetaitisa e ke viruddha pratirakshana ki salaha di jati hai|
apake samanya chikitsaka ka shalya-kaksha apako hipetaitisa e, miyadi bukhara, tanika shotha si, titenasa aura poliyo ke muphta tike dene mem sakshama hoga|
yatri svasthya chikitsalaya bhi yaha tike de sakate haim, lekina vaham thoda़a shulka ho sakata hai|
rakta ki jancha yaha dikhaegi ki apa pichhale sankramana ke karana, pahale se hi hipetaitisa e ke vishanu se pratirakshita haim ya nahim|
anyatha, apako apani yatra ke 4-6 saptaha pahale eka injekshana lena chahie aura 6-12 mahine bada eka vardhaka khuraka|
hipetaitisa e ke viruddha sanrakshana hipetaitisa e ke tike ki pahali khuraka ke do saptaha bada prarambha hota hai|
eka prarambhika tika aura vardhaka khuraka 10 varshom se adhika ke lie sanrakshana deta hai aisa mana jata hai|
yadi apa eka sankramita vyakti ke satha najadiki samparka mem haim ya yadi apaki yatra se pahale apake pasa pratirakshana ke lie samaya nahim hai, to apa inyunoglobyulina namaka roga-pratikarakom ke eka injekshana se alpavadhi sanrakshana (3-6 mahinom taka tikau) pa sakate haim|
inyunoglobyulina adhikatama sanrakshana ke lie hipetaitisa e vishanu se samparka ke bada do saptahom ke bhitara diya jana chahie|
hipetaitisa e ke viruddha eka mahatvapurna rokathama achchhi vyaktigata saphai ka abhyasa karana hai|
kevala apane hathom ki achchhi aura bara-bara dhulai kai sankramanom, vishanuom aura jivanuom ke viruddha sanrakshana mem apaki madada kara sakati hai|
yaha jaruri hai ki shauchalaya ke upayoga ke bada tatha bhojana taiyara karane ya khane se pahale apane hathom ko dhoem|
atirikta savadhaniyom mem tauliyom, khane ke bartanom, ya tuthabrashom ka adana-pradana na karana sammilita hai|
purna sanrakshana ke lie apako hipetaitisa bi ke tike ki tina injekshanom ki 4-6 mahinom ki avadhi mem jarurata hogi|
tisari khuraka ke eka mahine bada phira yaha janchane ke lie ki pratirakshanom ka asara hua hai, eka rakta parikshana ki jati hai |
apako phira kama se kama pancha varshom ke lie pratirakshita hona chahie|
prarambhika injekshana ke 5 varshom bada eka samvardhaka injekshana amataura para diya jata hai|
yadi apa una deshom - udaharanasvarupa, dakshina-purva eshiya, upa-sahariya aphrika, ya prashanta mahasagariya dvipom, jaise ki havai ke dvipom, solomana dvipom, aura phiji ki yatra karane ki yojana bana rahe haim jaham yaha roga kha़asa taura para samanya hai to apaki yatra se pahale, apako hipetaitisa bi ka tika prapta karane ke lie apane samanya chikitsaka se milane ki vyavastha karani chahie |
hipetaitisa bi ka tika isa samaya nishulka upalabdha nahim hai atah apaka samanya chikitsaka apase isake lie shulka lega ya apako eka vishesha yatra chikitsalaya mem bheja dega|
sankramita mataom se janmem shishuom ko sankramita hone se rokane mem madada karane ke lie unake janma ke bada inyunoglobyulina namaka roga-pratikarakom ka eka injekshana diya jata hai|
usi samaya hipetaitisa bi tika ka eka khuraka diya jata hai, jisaka 12 mahine bada eka samvardhaka khuraka sahita pratyeka ke bicha mem eka mahine ke antara ke satha do khurakom dvara anusarana kiya jata hai |
vishanu ke samparka mem ae kisi ko bhi turanta inyunoglobyulina ke eka injekshana ke satha-satha tika bhi dena chahie|
yadi apa sochate haim ki apake bachche ko khasare ka sankramana ho sakata hai to apako use dadora pragata hone ke bada kama se kama pancha dinom taka anya bachchom se alaga rakhana chahie|
pratirakshita bachche, aura kisi ko bhi jise pahale hi khasara ho chuka hai, unhem khasara hone ki atyadhika asambhavana hai|
prathama emaemaara tika sabhi bachchom ko 13 mahine ki unra ke asapasa diya jana chahie, satha hi eka samvardhaka khuraka unake skula ki shuruata ke pahale diya jana chahie| (jaba ve 3-5 varsha ki unra ke bicha hom)
prathama khuraka ke bada bachchom ke 5-10% ke bicha purna pratirakshita nahim hote haim, atah samvardhaka injekshana sanrakshana badha़ane mem madada karata hai, aura 1% se kama ke khatare mem bane rahane mem parinita hota hai|
yadi eka bachcha jo 13 mahine ki unra se kama mem khasare ke vishanu ke samparka mem ata hai to unhem bimari mem vikasita hone se rokane ke lie uthaya gaya kadama isa para nirbhara karega ki ve chhah mahine ki unra se kama haim, ya adhika|
halanki inlainda mem khasare ke aneka mamale nahim haim, para sansarabhara mem lagabhaga 30 miliyana (3 karoda़) mamalem haim, aura prati varsha lagabhaga eka miliyana (dasa lakha) nrityuem hoti haim|
pratirakshana ke bina, khasara phaila jaega aura eka mahamari utpanna karega|
vishva svasthya sangathana (dabluechao) ne 2010 taka khasare ka sarvabhaumika rupa se unmulana karane ke lie eka lakshya taiyara kiya hai|
isamem sansara ki janasankhya ke 95% ko pratirakshita karane ki avashyakata hogi|
eka purvanumanita samparka se bahuta pahale eka jivita tika dekara khasare ko roka ja sakata hai|
aba yaha salaha di gai hai ki sabhi svastha bachchom ko pandraha mahine ki unra mem jivita khasare ke tike die jaem|
aksara khasara-gala ganda roga-rubela (emaemaara) ke rupa mem balyavastha mem di jane vali dusari khuraka, aba samanya hai|
prathama tika chhah mahine aura nau mahine ki unra ke madhya una paristhitiyom mem diya ja sakata hai jaham khasare ka apatana eka varsha ki unra se pahale hi adhika hai|
5-15 pratishata bachchom mem tika lagane ke lagabhaga eka saptaha bada asthayi bukhara aura dadora hota hai|
kosha-madhyasthatakrita pratiraksha mem doshom vale vyaktiyom aura garbhavati mahilaom ke lie jivita khasare ke tike ko anupayukta bataya gaya hai|
roga-pratikarakom sahita nishkriya pratirakshana ki unake lie sipharisha ki jati hai jo gambhira khasara vikasita karane ke adhika khatare mem haim aura unake lie bhi jo sankramana se prabhavita ho chuke haim|
udaharanasvarupa, asadhya bimari vale bachche aura ve jo kosha-madhyasthatakrita pratiraksha mem doshom vale haim|
prabhavi hone ke lie, nishkriya pratirakshana ko samparka mem ane ke chhah dinom ke bhitara avashya diya jana chahie|
kyonki chhoti mata mem adhika apradhana akramana dara hai, atah chhoti mata se asani se prabhavita hone vale logom ko sankramana se grasta marijom ke samparka mem ane se bachana chahie|
kyonki chhoti mata sadharanatah eka susadhya bimari hai atah bachchom ke lie roganirodha chikitsa ka bahuta kama auchitya hai|
phira bhi, chhoti mata ki rokathama ke lie tika upalabdha hai aura isaki sadharanatah 10-12 varsha ki unra ke bicha ke una bachchom ke lie sipharisha ki jati hai jinhem pahale chhoti mata nahim hua ho|
isaki una inyunokampromaija़da rogiyom ke lie bhi sipharisha ki jati hai jo pahale kabhi chechaka ke samparka mem nahim aye haim|
verisela jostara inyuno globyulina (vija़edaaiji) aura verisela jostara plajma (vija़edaaipi) adhika jokhima vale vyaktiyom mem lakshanatmaka chhoti mata ko rokane ya sudharane mem upayogi haim|
rokathama vaha kshetra hai jisa para adhikatama dhyana hona chahie|
sthanika kshetrom mem machchharom ke dansha ki barambarata ko kama karane ke kadama mahatvapurna haim|
uchita vastrom, kita nirodhakom, machchharadaniyom adi ke upayoga ki salaha di jati hai|
machchharadaniyom aura nirodhakom ke vyapaka upayoga se maleriya ke apatana ko kama hota dikhaya gaya hai|
sthanika kshetrom mem machchharom ki janasankhya ko kama karane ke prayasa kie gae haim|
phira bhi, kimoprophailaiksisa ki una logom ke lie aksara salaha di jati hai jo maleriya-sthanika kshetrom mem yatra karate haim|
klorokvina jnata klorokvina-pratirodhaka kshetrom ko chhoda़kara jaham para mephlokvina ya daॉksisaiklaina upayoga kie jate haim kimoprophailaiksisa ke lie sabase adhika bara pasanda ki aushadhi hoti hai|
rakshatmaka kshamata 70 se 90 pratishata hai|
yaha tika ya to injekshana ke rupa mem ya eka munha se li jane vali kaipsula ke rupa mem upalabdha hai|
isa korsa mem hara dusare dina tina kaipsula ya eka mahina chhoda़kara do injekshana shamila hota hai|
yaha sanrakshana kama se kama tina varshom taka ka़ayama rahata hai|
tike surakshita haim aura inamem alpatama jatilataem hoti haim|
khasara, gala ganda roga aura rubela ka tika (emaemaara) rashtriya pratirakshana karyakrama ke bhaga ke rupa mem do khurakom mem diya jata hai|
prathama khuraka lagabhaga 13 mahine ki unra mem di jani chahie aura dusari bachche ke skula ki shuruata ke thika pahale (3-5 varsha ki unra ke bicha)|
jaham avashyakata hai (udaharanasvarupa, yadi apake bachche ka tikakarana karyakrama chhuta gaya hai), pratirakshana ki do khurakem kisi bhi unra mem di ja sakati haim, agara pratyeka khuraka ko kama se kama eka mahine ke eka antarala dvara alaga kiya gaya ho to|
phira bhi, yaha salaha di gai hai ki bachche apani dusari khuraka skula prarambha karane se pahale prapta karem|
bachche jinhem tika nahim lagaya gaya hai aura jinhem gala ganda roga ho jata hai unhem suji hui gilatiyom ke akramana se kai dinom taka skula aura jitane jyada loga sambhava hom se dura rakhana chahie|
yaha isalie kyonki gala ganda roga bahuta sankramaka hai aura yaha khansane aura chhinkane se phailata hai|
yaha sunishchita karane ke lie ki ve poliyo vishanu se pratirakshita bane rahem apake bachche ko poliyo tike ki pancha khurake leni hi hongi|
prathama khurakem apake shishu ko taba di jati haim jaba ve do, tina, aura chara mahine ki unra ke hote haim|
ye sabhi khurakem ditiepi/ aipivi/echaaibi tike ki hongi|
lagabhaga tina varsha aura chara mahine ki unra mem, apake bachche ko unaki chauthi khuraka milegi|
yaha tika ditiepi/aipivi ki eka khuraka hogi, jo poliyo, mansa santanika, dhanustambha aura kali khansi ke viruddha sanrakshita karata hai|
isamem echaaibi ke lie tika sammilita nahim hai, kyonki apaka bachcha do, tina, aura chara mahine ki unra mem jo tika prapta karata hai vaha unhem pratiraksha pradana karane ke lie paryapta hota hai|
13-18 varsha ki unra ke bicha, apake bachche ko tike ki usaki akhiri panchavim khuraka milegi|
yaha tika tidi/aipivi hai, aura poliyo, mansa santanika aura dhanustambha ke viruddha sanrakshita karata hai|
akhiri khuraka ke bada, apaka bachcha ina bimariyom se jivanabhara ke lie pratirakshita bana rahega|
pratirakshita hona dhanustambha ko rokane ka ekamatra upaya hai|
dhanustambha pratirakshana ke eka pure korsa mem tike ki pancha khurakem shamila hoti haim|
yaha tika apake sharira ko dhanustambha ke jivavisha ke viruddha roga-pratikarakom ki rachana karane mem samartha banata hai jo phira yadi apa isa jivanu se sankramita ho jaem to apaki roga se suraksha karata hai|
sabhi bachchom ko samanyatah dhanustambha pratirakshana pradana kiya jata hai|
eka vayaska ke rupa mem, agara apa sunishchita nahim haim ki apako puri taraha se pratirakshita kiya gaya hai ya nahim (apane pure pancha khuraka nahim li hai), apane jipi ya narsa ko dikhaiye |
ve apako 'pratirakshana ' injekshana ke bare mem salaha dene mem sakshama honge |
kuchha logom ko injekshana sthala ke asa-pasa thoda़i sujana aura lalima ho jati hai, lekina yaha ama taura para kuchha dinom ke bada chali jati hai |
anya dushprabhavom mem asvastha mahasusa hona, halka sira darda, anya darda, aura mamuli bukhara shamila haim |
halanki, ye lakshana kevala eka ya do dina taka hi rahane chahie |
isaki gambhira pratikriya ho aisa bahuta kama hota hai |
yadi apa vakai mem chintita haim to apako apane jipi ya narsa ke satha apane lakshanom para charcha karani chahie |
yadi apako gambhira ghava ya eka aisa ghava hai joki vishesha rupa se ganda hai, to apako titanesa ka khatara hai aura apako turanta apane sthaniya aspatala ke durghatana aura apatakalina vibhaga jana chahie |
eka svasthya adhikari apa se puchhega ki kya apa ne titanesa vaiksina ke sabhi pancha khuraka li haim ?
yadi apane sabhi pancha khuraka nahim li haim to ve ama taura para apako vaiksina ka eka injekshana de denge |
chahe apane tike ki sabhi pancha khuraka li haim ya nahim , svasthya adhikari apake ghava ka akalana karake taya kara sakega ki apako tetanasa siramarodhi injekshana ki jarurata hai ya nahim |
tetanasa siramarodhi mem entibaॉdi haim joki apako tetanasa baiktiriya ke khilapha atirikta suraksha de sakata hai , lekina yaha eka tika nahim hai |
jaham ghava vishesha rupa se ganda hai yaha siramarodhi adhikatara atirikta suraksha ke rupa mem vaham prayoga kiye jate haim , udaharana ke lie ghava mem sudriढ़ khada, aura bada़i matra mem baiktiriya ke satha, pravesha karana hai |
yadi apane apane apa ko kata ya jala liya ho, ya kisi ne kata liya ho lekina yadi apaka ghava utana gambhira ya vishesha rupa se ganda nahim hai, to apako jancha karani chahie ki kya apa titanesa vaiksina ki puri pancha khuraka le chuke haim ?
yadi apane sabhi pancha khuraka nahim li haim to apako titanesa hone ka khatara ho sakata hai |
apako apane jipi, ya narsa ke pasa vaiksina ke injekshana ke lie jana chahie |
yadi apako vaiksina ke injekshana ki ja़rurata hai to yaha mahatvapurna hai ki jitana jaldi sambhava ho sake ghava hone ke do dinom ke bhitara apa apane jipi ya narsa ke pasa jaem | 
yadi apa titanesa vaiksina ke sabhi pancha khuraka le chuke haim aura apaka ghava gambhira ya vishesha rupa se ganda nahim hai,to apako titanesa ka khatara nahim haim to apako titanesa ke tike ke atirikta injekshana ki jarurata nahim hai |
itane sare alaga alaga vayarasa ama sardi ka karana ho sakate haim, inake khilapha koi tikakarana abhi taka vikasita nahim kiya gaya hai |
halanki, agara apako sardi hai to apa kuchha aisa kara sakate haim jo inhem phailane se rokane mem madada kare |
apane hatha achchhi taraha se aura kai bara dhoem , aura apane bachchom ko hatha dhone ke mahatva ko sikhaem |
jaba sabuna aura pani upalabdha nahim hom to sharaba yukta hathom ko nuka़sana na pahunchane vale dravya jisamem 60 pratishata taka sharaba ho aisi eka botala lekara chalem |
ye jaila saba kitanuom ko mara dete haim , aura bada़e bachchom ke svayam ke upayoga karane ke lie surakshita bhi haim |
khasakara jaba apake parivara mem kisi ko sardi hai to rasoi aura batharuma ko bilkula sapha-suthara rakhem |
yadi apake bachche ko chechaka hai, to apa apane bachche ke skula ya narsari ko sthiti ki suchana deni chahie |
svasthya sanrakshana ejensi salaha deti hai ki bachche skula ya narsari ke lie vapasa tabhi ja sakate haim jaba chhale puri taraha se papada़i vale ho jaem, aura yaha amataura para pahala chhala dikhane ke 5-7 dina bada hota hai |
akhiri chhale ke phata jane ke bada aura papada़iyukta, bachchom ko aba sankramaka nahim mana jata hai |
halanki, ve abhi bhi isa samaya taka sankramaka haim |
yadi apako , ya apake bachche ko hala hi mem chechaka hui hai, to apa apane dostom ya rishtedarom ko aspatala mem dekhane jane mem sakshama nahim ho sakate haim -pahale apa ko jancha karane ke lie varda ko teliphona karana chahie |
yaha bhi sabase achchha hai ki apa ya apaka bachcha garbhavati mahilaom, navajata shishuom, aura kisi ko bhi, jinaki pratiraksha pranali kamajora hai (aise loga jinaki rasayana chikitsa chala rahi ho ya echaaivi ya edsa vale logom) ke satha samparka se bachem |
yadi apake bachche ko chechaka hai, to ho sakata hai ki antima nishana ke dikhane ke chhaha dina taka use havai jahaja se uda़ne ki anumati nahim di jae |
jaba apaka bachcha sankramaka sthiti ko para kara chuka ho aura chhalom para papada़i pada़ chuki ho , taba vaha uda़ana bharane ke lie surakshita hota hai para yaha sabase achchha hoga ki pahale apani eyaralaina ki niti ki jancha kara lem |
jaise hi bimari ka nidana kiya jata hai aura apa chikitsa salaha le lete haim taba jaldi se jaldi apako eyaralaina ko suchita kara dena chahie |
yaha bhi mahatvapurna hai ki sunishchita karem ki agara apako deri se jana ho ya apako chhutti radda karani pada़e ya agara apako apane rukane ke samaya ka vistara karana pada़e jaba taka apaka bachcha ghara ke lie uda़ana bharane ke layaka na ho jaye taba apaki bima kampani ko pata hona chahie ki apa isake lie kavara haim ya nahim |
chechaka ka vayarasa kabhi kabhi nirjiva vastuom, ya vastu jo ki pahale se chechaka ke vayarasa se sankramita haim jaise ki bachchom ke khilaune, bistara, kapada़e ya ke madhyama se phaila sakata hai |
vayarasa ke prasara ko rokane ke lie, apa kisi bhi vastu ya satahom ko miltana jaise staralaija़ dravya se sapha kara sakate haim aura sunishchita karem ki koi bhi sankramita kapada़e ya bistara sapha rahem aura niyamita rupa se dhoe jaem |
1898 mem taiphaida ke khilapha pahala tikakarana sau sala pahale vikasita kiya gaya tha |
taba se, tike mem kaphi sudhara hua hai aura parshva prabhava bahuta kama paida hone ki sambhavana haim |
yadi apa taiphaida baiktiriya se sankramita haim to taiphaida baiktiriya vaiksina apake sharira ko taiphaida baiktiriya ke khilapha entibaॉdi banane ke lie taiyara karati hai, taki apa bimari se lada़ sakem |
yadi apa kisi desha ke lie yatra kara rahe hai jaise eshiya, aphrika aura dakshina amerika jaham bhojana aura pani ki svachchhata ke manakom ka stara kharaba hai , apaka taiphaida tikakarana hona chahie |
yadi apa yatra ki taiyari kara rahe ho , sthaniya parivahana para yatra karate ho ya aura buniyadi avasa mem rahate ho , to yaha vishesha rupa se mahatvapurna hai ki apaka tikakarana hua ho |
vaiksina ke kai prakara haim joki salmonela tiphi se raksha karate haim |
taipheriksa ya taiphima Vi injekshana sabase adhika istemala hote haim |
yadi apa jyada khatare vale desha ko ja rahe haim to apane sharira mem pratiraksha shakti vikasita karane ke lie apako kama se kama do saptaha purva injekshana lene ki avashyakata hai | 
eka maukhika pratirodhaka dava hai jise vivotipha kaha jata hai |
halanki, kevala eka injekshana prabhava mem lana asana hai, kevala eka khuraka ki avashyakata hai aura tina sala taka prabhava rahata hai |
isa samaya ke jo abhi bhi khatare mem unake lie bustara khuraka ki salaha di jati hai |
taiphaida tika ena echa esa para upalabdha hai |
halanki, apa apane jipi ya narsa se nih shulka tikakarana prapta karane mem sakshama ho sakate ho, atah yaha sabase achchha hoga ki apa yatra se pahale ise lagava lem |
agara unhonne tikakarana aura pratirakshana atirikta seva se bahara nikalana chuna hai to jipi vaiksina dene se mana kara sakate haim |
isalie apako apane jipi ke chikitsa kendra jancha karane jane ki avashyakata hai jisase ki apaka jipi apako pratirodhaka tika pradana karane mem sakshama ho sake |
yauna sancharita (esatiai) echaaivi sankramana ko rokane ka sabase achchha tarika hai ki hamesha seksa ke daurana ya maukhika aura guda seksa ke daurana kandoma ka upayoga karem | 
tela adharita chikanaiyukta, vesilina ya bachche ke telayukta kandoma ka prayoga karane se bachem kyonki yaha kandoma ko kamajora kara sakate haim aura isake karana yaha phata bhi sakate haim |
jala adharita KY jeli valom ko snehaka adhika pasanda karate haim, aura ye saba suparamarketa, ya pharmasista se kharide ja sakata haim|
danta bandha - jo leteksa ka eka ayatakara tukada़a hai joki munha mem phita baithata hai -maukhika seksa ke daurana sankramana ke prasara ko rokane ke lie yaha bhi istemala kiya ja sakata hai |
apako hamesha pratyeka upayoga ke bada, aura sharira ke vibhinna bhagom para una ka upayoga karane ke bada seksa ke khilaune ko sapha karana chahie |
eka sathi ke istemala ke bada dusare ke istemala ke bicha mem unhem sapha kara liya jana chahie |
donom sathi ko yadi echaaivi hai to yaha mahatvapurna hai ki donom yauna sathiyom ke bicha surakshita yauna sambandha hom |
yaha isalie sambhava ho saka kyonki jaba apa apane apa mem vayarasa ki eka nai nasla dekhate haim aura jise apaki echaaivi virodhi dava bhi isako niyantrita karane mem sakshama nahim hogi |
kisi ka janabujakara echaaivi se sankramita hona eka aparadhika jurma hai|
yadi apa injekshana dvara draga lete haim to eka-dusare ki suiyom ko istemala na karem kyonki isake karana apa echaaivi se arakshita ho sakate haim, aura to aura kai sthaniya adhikariyom aura pharmesiyom dvara haipetaitisa si. jaise anya rakta janita vayarasa se bachane ke lie sui vinimaya karyakrama, jaham istemala ki gai suiyom ko svachchha suiyom se badala sakate haim |
yadi apa heroina istemala karate haim, to apako methadona karyakrama mem namankana karana chahie |
methadona tarala ke rupa mem liya ja sakata hai,jisase apako echaaivi hone ka jokhima kama ho jata hai |
yadi apako echaaivi hai to apako sankramana ke jokhima se bachane ke lie atirikta savadhani baratani chahie |
apako apani pratiraksha pranali ko badha़ane ke lie bhi kadama uthane chahie |
jaisa ki impaitigo bahuta sankramaka hai baiktiriya ko phailane se rokane ke lie yaha mahatvapurna hai svachchhata sambandhi savadhaniyam lem aura anya logom ko yaha hone se bachaem |
vartamana mem sanyukta rajya amerika mem yaha salaha di gai hai ki 2 mahine aura 6 sala ki unra ke bicha ke bachchom ko poliyo nishkriya tikakarana (IPV) ke chara khuraka di jani chahie |
albarta sabina dvara vikasita ki gai maukhika poliyo vaiksina (opivi)ko 1964 se svikarya bana diya gaya hai |
puri abadi ko opivi se pratirakshita karane ke lie svikara kara liya gaya hai kyonki yaha prashasana ke lie asana tha, aura yaha samparka tikakarana, jisaka artha hai ki eka bina tikakarana vala vyakti yadi kisi 'samparka pratirakshana' kiye hue bachche ke samparka mem ata to vaha bhi pratirakshita ho jata hai | 
opivi ke satha samasya yaha thi ki, bahuta kama mamalom mem, pratirakshita bachchom mem pairalitika poliyo ho sakata hai ya jinake samparka mem ye loga ate haim |
1979 se (jaba jangali poliyo sanyukta rajya amerika mem samapta ho gaya tha), isa desha mem prati varsha poliyo ke lagabhaga 10 mamale opivi dvara pata chale haim |
agara yaha isake satha samparka mem ata hai to aipivi eka aisa tika hai joki sharira ki pratiraksha pranali ko ( entibaॉdi ke utpadana ke madhyama se) vayarasa se lada़ne ke lie badha़ava deta hai | 
poliyo unmulana ke sabhi prayasom ke satha-satha ve sabhi mamale joki tike se sambandhita haim , roga niyantrana aura rokathama kendra (sidisi) ne bhi kevala sanyukta rajya amerika mem die gae tika aipivi ko banane ka phaisala kiya hai |
vartamana mem jaba bachche ko skula mem pravesha kara rahe haim , sidisi aura amerikana akadami ऑpha pidratiksa (apa) ne 18 mahine ki unra se pahale IPV ki tina khuraka samaya-samaya para dene ki raya di hai, aura eka IPV ki bustara khuraka 4 se 6 unra ke bicha dene ki raya di hai |
yadi apa sanyukta rajya amerika ke bahara yatra karane ki yojana bana rahe haim, vishesha rupa se aphrika aura eshiya (jaham poliyo abhi bhi maujuda hai), sunishchita karem ki apa aura apaka bachcha poliyo tikakarana ka eka pura seta le chuka ho |
do, tina aura chara mahine ki unra ke sabhi bachchom ko diphthiriya taॉksaida , titanesa taॉksaida aura kali khansi taॉksaida ka tikakarana (kali khansi) (ditipi tika) diya jana chahie |
3 aura 5 varsha ki unra para aura phira 16 se 18 sala ke bicha bustara khuraka di jati haim |
jaham kevala kuchha hi abadi pratirakshita hai una deshom mem dipthiriya mahamari abhi bhi hoti hai aura ina kshetrom ke lie jo loga yatra kara rahe haim unako yaha roga laga sakata hai |
120 se adhika deshom ke lie yatriyom ko khuda ko bachane ke lie dipthiriya ke khilapha tikakarana karae jane ki raya di jati hai |
yaha bahuta mahatvapurna hai ki bachche kali khansi ke khilapha pratirakshita hom |
yaha sankramana ka eka nishkriya prarupa hai aura isa sankramana se lada़ne ke lie sharira entibaॉdi paida karata hai atah apa isase pratirakshita haim |
pratirakshana eka samaya ke bada khatama ho jata hai, atah bachchom ko skula shuru hone se pahale bustara jeba ka tika jarura dena chahie |
bahuta chhote bachchom ko jaiba tika dene ka karana yaha hai kyonki taba chhote bachche kali khansi jaise rogom se bada़i gambhirata se prabhavita hote haim |
jaiba tike bahuta surakshita haim aura yaha sambhavana bhi nahim hai ki apake bachche ko koi bhi dushprabhava hoga |
halanki, agara apake bachche ko isake dushprabhava hote haim , to unamem chida़chida़apana ana , bukhara ana aura jaham sui lagai gai hai vaham mamuli sujana ana bhi shamila ho jate haim |
bimari ka mukhya karana edisa aigyapti machchhara mana jata hai, vastava mem yaha eka sthaniya kita hai |
apane pure jivanakala mem, yaha eka chauthai kilomitara bhi yatra nahim karata hai |
atah aisa nahim hai ki machchhara hi mahamari ka karana hai |
logom ke eka prabhavita kshetra se dusare aprabhavita kshetra ko yatra karane ke madhyama se chikanaguniya eka mahamari bana jata hai 
isa karana se, yaha bahuta avashyaka hai ki yatriyom ko viniyamita kiya jae , vishesha rupa se una yatriyom ko jo chikanaguniya ke kai mamalom vale sthana se dusari jagaha ja rahe hom |
tikakarana inphluenja ko rokane aura mahamari ke prabhava ko kama karane ke lie pramukha upaya hai |
inphluenja tikom ke vibhinna prakara upalabdha haim aura 60 se adhika varshom se istemala kiye ja rahe haim |
ve surakshita haim aura inphluenja ke donom halke aura gambhira parinamom ko rokane mem karagara haim |
aisi raya di jati hai ki bujurga logom ko, aura kisi bhi unra ke vyaktiyom ko jo inphluenja se sambandhita antarnihita svasthya ki sthitiyom ki jatilataom ke karana adhika khatare mem haim, unhem tika lagaya jana chahie |
bujurgom mem tikakarana se 60% rugnata ko kama karana aura inphluenja se sambandhita nrityu dara ko 70-80% kama karane ka socha gaya hai |
svastha vayaskom mem inphluenja ki rugnata ko kama karane ke mamale mem vaiksina bahuta prabhavi (70-90%) hai, aura isa ayu varga mem tikakarana ke karana paryapta svasthya sambandhi aura arthika labha dikhayi dete haim |
inphluenja tika ki prabhavashilata sarvaprathama unra aura vaiksina praptakarta ki pratirodhaka shakti aura pratirodhaka tika aura vatavarana mem vidyamana vishanuom ki samanata ki matra para mukhya rupa se nirbhara karata hai |
inphluenja tikakarana svasthya dekhabhala ki lagata aura inphluenja bimari se juda़e nukasana ki utpadana kshamata donom ko kama kara sakata hai | 
vartamana mem phlu ke sabhi nishkriya tikom mem andom ke protina ki alpamatra hoti hai aura ande ke protina se elarji vale vyaktiyom dvara ise istemala nahim kiya jana chahie |
manava aura pashuom donom mem enthreksa ki rokathama ke lie sthaniya pashuom ki achchhi dekhabhala jaruri hai jisamem enthreksa sankramita pashuom ka sahi tikakarana va sankramita pashuom ke shavom ka surakshita nipatana avashyaka hai |
nashta karane ka sabase karagara tarika hai-bhasmikarana yani isa taraha se jala kara bhasma karana ki antarnihita mitti ka roganunashana sunishchita tarike se pura ho jae|
vyavaharika rupa mem, kai sthanika bimariyom vale deshom mem, sthaniya paristhitiyam ina sarala niyantrana upayom ko lagu karane mem rukavatem paida kara deti haim|
audyogika deshom mem, achchhi krishi evam audyogika svachchhata mem hi rokathama nihita hai|
pashuom aura manushyom ke lie tike upalabdha haim|
halanki manushyom mem unaka upayoga atyadhika khatare vale samuhom ke lie hi simita hona chahie jaise ki una logom ke lie jo vyavasayika rupa se logom ke samparka mem rahate haim aura kisi sainya parivesha mem rahate haim |
rogi ko alaga rakhane ki avashyakata nahim hai aura kisi bhi prakara ki sangarodha avashyakata bhi nahim hai|
kapada़e aura anya dushita samagri ko bhasmikarana dvara puri taraha nashta kiya jana chahie|
jina logom mem lakshana vikasita ho rahe haim,unaki 80 % bimari kama ya ausata gambhira hoti hai|
shesha 10% -20% logom mem ,nirjalikarana ke lakshanom ke satha pani vale gambhira dasta ho jate haim|
yadi bimari ko bina ilaja ke chhoda़ diya jae, to jyada se jyada do vyaktiyom mem se eka vyakti mara sakata hai|
uchita upachara se, nrityu dara 1% se niche hi rahani chahie|
vartamana dashakom mem, haija ki rokathama ke upaya bahuta jyada nahim badale haim, aura jyadatara upaya to sashakta rupa se prabhavita abadi ko sapha pani aura uchita svachchhata upalabdha karana hi hai|
svasthya shiksha aura bhojana ki achchhi svachchhata bhi samana rupa se mahatvapurna hoti haim|
vishesha rupa se, vyavasthita tarike se hatha dhona sikhaya jana chahie|
eka bara jaba bimari ke prakopa ka pata chala jata hai,to nrityu dara ko kama karane ke lie samanya madhyavarti upaya haim - shighra ilaja karavana aura bimari ke prasara ko niyantrita karana|
80 % taka rogiyom ka ilaja paryapta rupa se orala rihaidreshana saltsa (dablyuechao / yunisepha oaraesa manaka paiketa) ki vyavastha karake kiya ja sakata hai|
atyadhika gambhira nirjalita rogiyom ka ilaja ringara laikteta jaise antahshira tarala padarthom se kiya ja sakata hai| 
atyadhika gambhira mamalom mem, dastom ke antarala ko kama karane,avashyaka punarjalikarana tarala padartha ki matra ko kama karane aura vibriyo baiktiriya ke utsarga ki avadhi ko kama karane ke lie uchita entibayotika davaem di ja sakati haim|
logom mem entibayotika davaom ,ya jana rasayanaroganirodha ke niyamita ilaja se haija ke prasara para koi prabhava nahim pada़ta,lekina roganurodhi pratirodhaka kshamata badha़ jane se pratikula prabhava ho sakata hai|
samaya para ilaja prapta karane ke lie ,prabhavita abadi vale ilakom mem jaba bhi sambhava hom, haija upachara kendra sthapita karane chahie|
surakshita pani aura svachchhata ka pravadhana eka durjeya chunauti hai, lekina haija phailane ke prabhava ko kama karane mem eka mahatvapurna karaka bani hui hai|
manakikrita bimari prabandhana sahita svikrita niyantrana vidhiyom ne, bimari se hone vali nrityu dara ko kama karane mem,apane apa ko prabhavi siddha kiya hai|
rukavatom ko sahi marga dikhane ke lie vistrita nigarani ankada़e sarvadhika mahatvapurna hote haim aura pratyeka vishishta sthiti mem unhem upayogi banana chahie|
isake alava, haije ki rokathama aura niyantrana, akele svasthya sansthaom dvara nipataya jana koi bada़a mudda nahim hai|
pani, saphai, shiksha aura sanchara anya kshetra haim ,jaham amataura para dusari sansthaem shamila hoti haim|
sambhavita haija prakopa se nipatane ke lie, eka vyapaka bahu-vishayaka drishtikona apanaya jana chahie|
manava rebija ki rokathama, pashu chikitsa adhikariyom aura sarvajanika svasthya adhikariyom donom ko shamila karake eka samudaya prayasa hona chahie|
bada़e paimane para kuttom ke tikakarana para mukhya rupa se dhyana kendrita karake, rebija unmulana karyakramom ko bada़e paimane para rokathama karyakramom ko rokane ki bhavi bachata dvara nyayasangata thaharaya gaya hai|
halanki, jaba taka kutte sehone vale rebija ko samapta na kiya jae ya kama se kama achchhi taraha se niyantrita na kiya jae, taba taka rebija hone para manushyom ke lie surakshita aura adhika kiphayati ilaja़ haim-tantrika utaka tike ke upayoga ke bajaya koi anya vanchhaniya vikalpa |
audyogika deshom mem rebija se khatare vale janasamudaya mem rebija ko rokane ke lie, rebija hone se purva kiya gaya tikakarana vyapaka rupa se aura saphalatapurvaka istemala kiya ja chuka hai aura bachchom vale una kshetrom mem bhi unaka prachara kiya jana chahie jaham kutte se rebija hona atyadhika sthanika hai |
agara rebija puri taraha samapta nahim hota hai, to vikasashila deshom mem, manushya aura pashu donom mem roga ki rokathama se sambandhita kharchom ke badha़ne ki sambhavana adhika ho jaegi| 
sankramaka netrashleshmalashotha (kanjektivaitisa) ke prasara ko rokane ka sabase achchha tarika hai ki apa apani sankramita ankhom ko chhune ya davai dalane ke bada apane hatha achchhi taraha dho lem|
agara apako sankramana nahim hua hai, lekina apake kisi naja़diki vyakti ko hua hai,to usake samparka mem ane para hara bara apa apane hatha jarura dhoem|
sankramana ko phailane se rokane ke lie tauliyom,takiyom aura chadarom ko apasa mem istemala karane se bachem| 
jaba taka ki apake bachche ke skula mem sankramaka netrashleshmalashotha ka prakopa na ho,apa apane bachche ko ghara para na baithaem|
svasthya sanrakshana ejensi ke anusara- agara bachchom ko jukama ya sankramaka netrashleshmalashotha jaisi koi halki-phulki ya anya sankramaka bimari ho gai hai,to unako skula ya khela maidana na bhejana koi buddhimani nahim hai|
yaha, karibi sharirika samparka ke madhyama se ya eka sankramita vyakti ke balagama ko chhune se phailata hai|
yadi apake bachche ko shonatvag jvara (skarleta jvara) ho gaya hai,to use skula aura anya logom se taba taka dura rakhem jaba taka ki usane entibayotika davaom ka kama se kama 5 dina ka korsa pura na kara liya ho| 
skarleta jvara se pida़ita kisi bhi vyakti ke balagama se dushita sabhi kapada़e ke rumalom aura kagaja़i rumalom ko turanta dhoya ya phenka jana chahie |
agara apane unhem chhua hai to apa apane hatha achchhi taraha dho lem|
agara kisi vyakti ko streptokokasa baiktiriya se tvacha sankramana ho gaya hai,to usa vyakti ko chhune se ya khane ke dushita bartana,kapa aura gilasa,kapada़e, bistara ki chadarem ya taulie adi apasa mem istemala karane se bhi baiktiriya dusare logom mem chale jate haim|
skarleta jvara ama taura para vayaskom ko nahim hota hai, lekina agara apa garbhavati haim to skarleta jvara hone se bachane ke lie aura atyadhika bukhara aura gale mem kharasha jaise lakshanom se bachane ke lie,apa una bachchom se dura rahane ki koshisha karem jinake bare mem apako maluma hai ki unhem skarleta jvara hai|
kyonki maininjaitisa ke lakshanom ko pahachanana mushkila hota hai, isalie yaha jaruri hai ki apako khatare ke sanketom ke bare mem pata hona chahie|
yadi koi bimara ho jata hai to usaki halata jaldi hi bigada़ne lagati hai,isalie agara apako maininjaitisa ya saiptisimiya hone ka shaka hota hai to chikitsiya salaha le leni chahie|
vayarala maininjaitisa ka hona yuva vayaskom mem sabase ama bata hai aura vishesha rupa se skulom aura kaॉlejom mem, eka thoda़e se prakopa ke rupa mem yaha phaila jata hai|
koi bhi lakshana dikhane para chikitsaka se turanta jancha karana bahuta jaruri hai|
kuchha khasa prakara ke baiktiriyala maininjaitisa ke lie tike upalabdha haim|
ye haim- samuha e aura si ke mainingokokai baiktiriya aura hemophailasa inpha़luenja़i (hiba),jinase maininjaitisa ho sakata hai|
bachchom ke svasthya ke lie , meningokoksala si aura hiba , aba niyamita rashtriya tikakarana karyakrama ke hissa bana gae haim|
bada़e bachchom aura vayaskom ko meningokoksala si ki sirpha eka hi khuraka ki avashyakata hoti hai aura yaha salaha bhi di jati hai ki 25 varsha se niche ki ayu ke sabhi yuva logom ko isa tike se pratirakshita kiya jana chahie|
jo loga usa vyakti ke karibi samparka mem rahate hai jise e ya si prakara ka baiktiriyala maininjaitisa ho gaya hai, to unhem bhi usa vishesha prakara ke maininjaitisa ka tika lagava lena chahie, agara ve grupa bi ke satha samparka mem haim,to tika lagavane ki koi avashyakata nahim hai|
prarambhika katane ke bada maleriya ke lakshana, 10 dina se lekara 4 saptaha ke bicha dikhane shuru hote haim|
halanki, kuchha mamalom mem,apa jisa parajivi se sankramita hote haim usake prakara para nirbhara karate hue ,apamem lakshana ubharane mem eka sala bhi laga sakata hai|
jo maleriya ghataka nahim hota, usake lakshana phlu ke lakshanom ke samana hi hote haim,jaise ki-bukhara ana (40 digri selsiyasa tapamana), thanda lagana, ama taura para asvastha mahasusa karana, mansapeshiyom mem darda, sira darda, dasta, mitali aura ulti adi|
agara koi davai na li jae to ye lakshana kai haphtom taka raha sakate haim, lekina ama taura para ,uchita ilaja se, kuchha hi dinom mem thika ho jate haim|
ghataka maleriya, plajmodiyama phalsiperama namaka eka parajivi ke karana hota hai|
ama taura para ,ghataka maleriya sadharana maleriya ke jaise lakshanom se hi shuru hota hai, lekina bada mem aksara, shvasa samasya, jigara ka kama na karana aura sadama jaisi jatilataom ke rupa mem vikasita hone lagata hai|
ghataka maleriya dimaga aura kendriya tantrika tantra ko bhi prabhavita kara sakata hai|
lekina jaba yaha ho jata hai,to bimari ki yaha vikriti aksara ghataka bhi ho jati hai|
phlu , sansa lene ke upari marga ka eka sankramaka roga hota hai (jo upari shvasa patha ke rupa mem jana jata hai)|
isake lakshana haim-jaldi jaldi bukhara ana (38-40 C),pasina ana aura bukhara jaisa mahasusa hona,mansapeshiyom mem samanya darda, samanya thakana mahasusa hona, khushka evam chhati mem jami balagama vali khansi, chhinkem ana, bahati ya banda naka evam sone mem takalipha़ hona adi|
lakshanom mem michali,ulti, aura vishesha rupa se bachchom mem dasta hona bhi ho sakate haim|
phlu ke kai lakshana to samanya jukama ke lakshanom ki hi taraha hote haim, aura bahuta se loga gambhira jukama ko,galati se phlu hi samaja lete haim|
donom hi vayarasa se hote haim , lekina jukama ki tulana mem, phlu mem phephada़em kahim adhika prabhavita hote haim|
aura amataura para phlu mem atyadhika sira darda bhi nahim hota,jaise ki jukama mem hota hai| 
kuchha mamalom mem, logom mem hepetaitisa e ke koi bhi lakshana nahim dikhate hai, jo ki vishesha rupa se chhote bachchom mem dikhate haim|
saba logom mem hepetaitisa e ki avadhi aura ugrata samana nahim hoti|
eka saptaha ke lie apako, piliya ke satha-satha phlu jaise lakshana ho sakate haim,jo dhire - dhire thika ho jate haim|
eka ya do mahine taka apako thakana bhi mahasusa ho sakati hai|
kuchha hi mahinom mem apa puri taraha se thika ho sakate haim ,lekina kuchha logom ko punah hone ki sambhavana bhi ho sakati hai, aura lakshana chhaha mahine taka bhi bane raha sakate haim|
eka bara jaba apa hepetaitisa e se bilkula thika ho jate haim ,to apa isase puri taraha pratirakshita ho jate haim aura phira apa kabhi bhi isa vayarasa se pida़ita nahim ho sakate|
kai logom ko jaba hepetaitisa bi ho jata hai to koi bhi lakshana nahim dikhate,halanki taba bhi, dusare logom ko unase roga ho sakata hai|
kabhi kabhara, phulaminenta hepetaitisa bi namaka gambhira ghataka haipetaitisa ho jata hai| 
lakshanom mem achanaka bimara paड़na, tvacha aura ankhom mem gambhira rupa se pilapana (piliya) ,aura peta mem sujana adi shamila haim|
agara turanta ilaja nahim ho paya to yaha ghataka bhi ho sakata hai |
hepetaitisa bi vale bahuta se logom mem,sankramana thika ho jata hai aura ve bimari se mukta ho jate haim|
phira bhi,yaha roga dirghakalika bhi ho sakata hai ,agara apa isase chhaha mahine se adhika samaya taka sankramita rahate haim|
kuchha loga roga ke vahaka hote haim aura unamem roga ka koi lakshana bhi nahim hota hai, aura jaba unake sharira se vayarasa ka asara khatma ho raha hota hai ,to ve svastha rahate haim|
unako yaha bhi pata nahim chalata ki ve roga se prabhavita bhi the|
dusarom mem, jaise ki vishesha rupa se shishuom aura bachchom mem, roga ke lakshana dikhate haim,jo ate-jate rahate haim|
ye lakshana haim -thakana,mitali,peta darda,vajana ghatana aura piliya adi,yaha atyadhika gambhira sthiti ke samana hota hai|
ina logom ke jigara para taba aparivartaniya chakatte pada़ jate haim,jise sirosisa kaha jata hai, aura antatah jigara kainsara ho sakata hai|
prathama lakshanom ke 3 se 4 dina ke bada ,lala-bhure ranga ke dhabbedara chakatte ho jate haim,jo atha dinom taka bane rahate haim|
amataura para,chakatte kana ke pichhe ubharana shuru hote haim, phira sira aura gardana ke charom ora phaila jate haim, aura 2-3 dinom ke bada pairom aura sharira ke baki hissom mem bhi phaila jate haim| 
chakatte chhote chhote banana shuru hote haim, lekina jaldi hi bada़e ho jate haim aura aksara apasa mem eka satha mila jate haim|
isi taraha ke chakattom se khasara hone ki galataphahami ho sakati hai, lekina khasara mem chakattom ke alava dusare anya kai lakshana bhi hote haim|
ushmayana avadhi eka se do saptaha ki hoti hai aura vayaskom mem aksara yaha jyada hoti haim|
asvasthata, bukhara, bhukha na lagana, netrashleshmalashotha, khansi, thanda lagana jaise lakshanom se bimari shuru hoti hai aura kai dinom taka bani rahati hai|
isake bada munha mem nile-bhure ranga ke dane (kopalika spaॉta) ho jate haim aura phira ye dane chehare para phailana shuru ho jate haim jo phira sare sharira para phaila jate haim|
dane pancha dina taka rahate haim aura phira usake bada khala utarani shuru ho jati hai|
kai dinom taka danom ki upasthiti ke bada, bukhara utarana shuru ho jata hai|
khasara ki sabase ama jatilataem shvasa nalika evam tantrika tantra mem hoti haim|
baiktiriya ka atyadhika sankramana madhya kana mem hota hai ya atyadhika gambhira mamalom mem sankramana se nimoniya bhi ho sakata hai|
khasara sankramana ke bada, insephelaitisa gambhira ya dirghakalika bhi ho sakata hai|
gambhira rupa se khasara una vyaktiyom mem ho sakati hai jo pratirakshita nahim hote haim, jaise ki vo loga jinaka kisi asadhya roga ka ilaja chala raha ho ya jina logom ko edsa hai|
vikasashila deshom mem, kuposhana ke shikara bachchom ko bhi gambhira khasara ho sakati hai|
yaham taka ki garbhavati mahilaom mem, jarmana khasara (rubela) ke viparita, khasara (rubiola) se koi bhi janmajata visangatiyam paida nahim hotim|
chechaka se bahuta hi kama bukhara aura bechaini hoti hai,aura sari tvacha para lala chakatte ho jate haim|
jisase khujali,bhukha na lagana aura susti jaise lakshana dikhane lagate haim|
shuru mem dane chhote hote haim aura paradarshi tarala padartha se bhare hote haim, lekina dhire - dhire aparadarshi hone lagate haim aura una para papada़i jamane lagati hai|
shuruata mem aksara dane osa ki bunda ki taraha lagate haim |
dane dhada़ se shuru hote haim aura 2 se 4 dinom mem sharira ke anya bhagom mem teji se phaila jate haim |
1 se 2 saptaha mem papada़i pada़ kara puri taraha se gira jati hai |
kshirna pratiraksha shakti vale bachchom (kami ai pratirodha) ke bahuta se ghavom se aksara khuna nikalata hai |
chechaka bhi insephelaitisa (mastishka jvara) aura anumastishkiya lakshanom ka karana ho sakata hai |
khasakara ke vayaskom aura alaga-alaga vyaktiyom ki pratirodhaka shakti ke lie chechaka , nimoniya eka gambhira aura jivana ghataka ho sakata hai |
agara prasava se thika pahale garbhavati mahilaem chechaka se prabhavita ho jati haim to chechaka mata aura bhruna ki mauta ka karana bana sakati hai |
yadi garbhavastha ke shuru mem hi sankramana hota hai, to bachche vikrita paida ho sakate haim |
maleriya ke prarambhika lakshana gaira vishishta haim |
rogi amataura para bimara mahasusa karate haim aura sira darda, thakana, mansapeshiyom mem darda aura halki peta ki pareshani rahati hai |
paramparika maleriya mem achanaka teja bukhara ke karana niyamita antarala para , thanda lagana aura adhika pasine ke satha daure jaise ate haim |
sharirika parikshana se kuchha jancha parinama samane ate haim jaise tilli spashta dikhane lagati hai ,kuchha spashta badha़a hua yakrita aura halka enimaya ho jata hai |
phalsiperama maleriya mem chetana mem badalava a jata hai aura sharira mem akaड़na jaise bure prabhava dikhai dete haim |
mastishka maleriya mem plajmodiyama phalsiperama ke karana tantrika tantra ki bhagidari dekhi jati hai , jo ki maleriya ka sabase ghataka rupa hai |
gurde ki viphalata phalsiperama maleriya mem bhi kabhi kabhi dekhi jati hai |
baiktiriya ki sankhya ke adhara para bhinna-bhinna bimariyom ki ushmayana avadhi pancha se bisa dina hoti hai |
kevala 20 se 40 pratishata rogiyom ke vyakhyana ke samaya peta mem darda hoga |
thanda lagana, pasina ana, bhukha na lagana , sira darda, hani, kamajori, gale mem kha़rasha aura mansapeshiyom mem darda ke rupa mem gaira vishishta lakshana aksara maujuda rahate haim |
5 se 10 pratishata rogiyom mem manovikriti aura ghabaड़ahata utpanna ho jati hai |
rogi ke sharirika parikshana se rogi ki bimari ki tivrata ka pata chalata hai |
teja़ bukhara ke karana (sapeksha mandaspandana) nada़i gati mem hone vale asantulana ko nota karem |
tisa pratishata rogiyom ke dhada़ para halke gulabi chakkatte ho jate haim |
jancha ke samaya , peta ki komalata ko badha़e hue jigara aura tilli ke satha dekha ja sakata hai |
bimari ke akhiri charana mem taiphaida bukhara ki anupacharita kai jatilataem sankramana ke tisare ya chauthe saptaha mem samane ati haim |
isamem antom ka rakta-srava ya chhidra hona ,viryakosha mem sujana, jigara ya tilli mem phoda़a hona shamila haim |
andashaya (lada़kiyom mem) ya vrishana (yauvana ke bada lada़kom mem) ke asapasa sujana ho sakati hai |
lagabhaga eka tihai logom mem kanthamala ke roga vale vayarasa ke koi lakshana vikasita nahim hote haim aura, anya logom mem, lakshana kaphi halke hote haim |
kanthamala ka roga mem, eka ya donom lara granthiyam suja jati haim aura dardanaka ho jati haim |
isaki yaha eka visheshata hai ki isamem vishesha rupa se kana ke thika niche aura kana ke samane, chehara 'hainsatara' ki taraha suja jata hai |
granthiyom ki sujana do se tina dina taka badha़ti hai aura dhire-dhire kama ho jati hai vaise hi jvara bhi kama hone lagata hai |
lagabhaga atha dinom taka sujana ka rahana yaha asamanya nahim hai |
kanthamala roga ke lie ushmayana avadhi (sankramana ke lakshana dikhane shuru hone se pahale apake sistama mem vayarala sankmana rahata hai ) 14-21 dinom ke bicha hoti hai |
halanki, eka vyakti jise kanthamala roga ka sankramana hai vaha granthiyom ke sujane ke chhaha dina pahale se sankramita rahata hai aura granthiyom ki sujana ke pancha dinom bada taka vaha sankramita rahata hai |
apa kevala tabhi kanthamala roga ko hastantarita kara sakate haim agara apa vartamana mem sankramita haim |
99% mamalom mem poliyo mem samanya sthiti mem bahuta kama lakshana samane ate haim |
vastava mem, kuchha loga koi bhi lakshana ko nahim dekha paenge |
amataura para halke poliyo ke mamale phlu jaise lakshanom ke karana hote haim jinamem gale mem kharasha, bukhara, matali, ulti, dasta aura kabja shamila ho sakate haim |
kisi ke vayarasa se sankramita hone ke bada ye lakshana 3-21 dinom ke bicha dikha sakate haim |
pakshaghata vale poliyo ka prakara bahuta kama paya jata hai jisamem kevala 1% loga hi isake shikara hote haim |
isa prakara ke poliyo ka pahala lakshana bukhara hai |
amataura para yaha sansa lene ki mansapeshiyam haim (phephada़e aura madhyapata) jo mukhya rupa se prabhavita hoti haim |
yadi sansa ki mansapeshiyom ko laka़va mara jata hai , sharira paryapta ऑksijana prapta karane mem asamartha hota hai, aura samanya rupa se eka ventiletara (kritrima sansa lene mashina) ki avashyakata hoti hai |
isa bare mem adhika janakari ke lie 'ilaja' anubhaga dekhem |
isa prakara ka poliyo janaleva ho sakata hai |
kitani tantrika koshikaem prabhavita hui haim , pakshaghata ki sima isa para nirbhara karegi |
yadi kevala koshikaem kshatigrasta hui haim, pakshaghata se jaldi svathya labha hone ki sambhavana hai |
halanki, agara koshikaem puri taraha se nashta ho chuki haim, to pakshaghata ke hamesha rahane ki sambhavana rahati hai |
prarambhika bimari ke 10-40 sala ke bada poliyo ke bada ke sahalakshana dikhai dene lagate haim |
adhikansha logom ko poliyo ke bada ke sahalakshana dhire-dhire vikasita hote hue lagate haim | 
kabhi kabhi, titanesa ka pahala aura ekamatra sanketa sankramita ghava ke pasa mansapeshiyom ki ainthana hai |
halanki, agara eka bara visha apake khuna mem ho jata hai, to amataura para lakshanom ki shuruata chehare se hoti hai |
sabase ama sanketa mansapeshiyom ka bara-bara ainthana hai jisaki vajaha se apana munha kholana bahuta kathina ho jata hai (isalie amataura para ise 'laॉkajaॉ' kaha jata hai )|
ainthana apake gale ki mansapeshiyom mem ho sakati hai, isaki vajaha se apako kuchha bhi nigalana mushkila ho jata hai |
yaha ainthana chehare ki mansapeshiyom mem ho sakati hai jisake karana apa muskurate hue lagate haim |
ainthana anya mansapeshiyom mem bhi phaila sakati hai - sira ka gardana ki tarapha juka jana , chhati ki tarapha juka jane se sansa lene mem mushkila hona , peta ki tarapha ya hatha aura pairom ki tarapha bhi phaila sakata hai |
yadi ainthana pitha ki mansapeshiyom mem phaila jae to ridha़ ki haddi bilkula pichhe ki ora dhanushakara ho sakati hai |
yaha sankramana bachchom mem bahuta ama hai |
anya lakshanom ke satha-satha sparsha ke lie charama samvedanashilata , teja़ bukhara, gale mem kharasha, teji se dila ki dhada़kana, sansa lene mem kathinai, siradarda, antom mem khuna ka bahana aura dasta shamila haim |
mauta ke pratyaksha karana rakta vishaktata, mansapeshiyom mem ainthana ke karana ghutana (shvasavarodha), dila ka rukana (dila rukana ), gurde ki viphalata ya thakavata bhi ho sakata hai |
yadi anupacharita chhoda़ diya jaye to lagabhaga 60% mamalom mem mauta hoti hai |
sardi ke lakshana ke pahale 2-3 dinom ke daurana kharaba hote haim aura phira agale kuchha dinom mem kama ho jate haim |
kuchha prakara ki sardi do saptaha taka rahati hai |
kuchha loga phlu ko (inphluenja)buri thanda ke rupa mem ullekha karate haim |
halanki, apa bahuta sardi mem bahuta bimara mahasusa karate haim, phlu eka bhinna sthiti aura vayarasa hai jinake karana phlu hota hai ki karana sardi se alaga haim.
sardi ke lakshanom ko vikasita hone mem kuchha dina lagate haim , jabaki phlu ke lakshanom ki achanaka shuruata ho jati hai , aksara kuchha ghantom mem vikasita ho jate haim |
amataura para eka ama sardi ke lakshana thanda vale vayarasa ke samparka mem ane ke eka se tina dina ke bada prakata hote haim |
eka ama sardi ke lakshanom mem banda naka aura pani bahana, gale mem kharasha, svara baithana, khansi, aura shayada thoda़a bukhara aura sira darda shamila haim 
kai loga sardi ke karana thakana aura darda mahasusa karate haim |
ye lakshana amataura para tina se 10 dinom taka rahate haim |
bukhara ke satha phephada़om sambandhi tibi ke lakshanom mem thakavata , bhukha aura vajana ki hani, rata mem pasina ana aura lagatara khansi ana shamila haim |
kapha vali khansi rakta ke kuchha nishana ke satha ho sakati hai |
yakshma plurisi (phephada़om ke asapasa ki jilliyom ko prabhavita karata hai )ke karana phephada़om ke asapasa ki khali jagaha mem tarala padartha bhara jata hai(ama taura para jilliyom ke bicha ke khali sthanom mem ) aura jisa vajaha se phephada़e anshika rupa se kharaba ho jate haim |
kabhi kabhara , phephada़om ki tibi dhamani ko dhire-dhire nashta kara deti hai , isa karana phephada़om mem bhayanaka rupa se rakta-srava hone lagata hai |
tibi rakta-pravaha (jvara ya bajare jaisa tapedika kaha jata hai ) ke madhyama se sharira bhara mem vyapaka rupa se phaila sakata hai |
chechaka ke sabase adhika abhijnata lakshana lala lala chakatte hona , jinase sharira bhara jata hai |
halanki, chakatte hone ke pahale, apa ya apake bachche ko kuchha halke phlu jaise lakshanom ka anubhava ho sakata hai |
ye phlu jaise lakshana (vishesha rupa se bukhara) bachchom ki tulana mem vayaskom mem badatara ho sakate haim |
ina lakshanom mem matali, bukhara, darda, dardanaka mansapeshiyam, sira darda, ama taura para asvastha mahasusa karana, aura bhukha ki kami shamila ho sakate haim |
prarambhika lakshanom ke phaurana bada chakatte vikasita hona shuru ho jate haim |
samanya taura para ye dane apratyashita rupa se prakata hote haim, aura amataura para kana ke pichhe , chehare para, chhati aura peta para aura sira para hatha aura paira para bahom ke niche paye ja sakate haim |
dane chhote, khujali, lala dhabbe ke rupa mem shuru hote haim |
lagabhaga 12-14 ghantom ke bada, ye dane tarala padartha se bhare chhalom mem vikasita ho jate haim jinamem bahuta adhika khujali hoti hai |
ye chhale apake hathom ki hatheliyom aura apake pairom ke talavom para ho sakate haim |
eka se chara dinom ke bada ye chhale sukhane laga jayenge aura ina para papada़i pada़ni shuru ho jayegi |
eka se do saptaha ke bada, papada़i tvacha se svabhavika rupa se jada़ jaegi |
yaha bahuta mahatvapurna hai, agara apa garbhavati haim, ya apaki pratiraksha pranali kamajora hai aura yadi apa kisi aise kisi ke samparka mem rahe haim jise chechaka hai,chahe apa ko chechaka ke lakshana hom ya na hom isaki paravaha kie bina apane jipi se sidhe samparka karem | 
agara apaka shishu chara saptaha se kama ka hai aura vaha chechaka vayarasa se sankramita hai to apako apane jipi se samparka karana chahie |
agara anupacharita chhoda़ diya jae to chechaka ina logom mem gambhira samasyaom ka karana ho sakati hai |
isalie yaha jaruri hai ki apa chikitsaka ki salaha lem jisase ki apako koi bhi avashyaka savadhaniyam ya upachara pradana kiye ja sakem |
apako bhi apane jipi se sidhe samparka karana chahie agara apa ya apake bachche mem koi bhi asamanya lakshana vikasita hote haim |
udaharana ke lie, agara tvacha ke chhale sankramita ho jate haim, ya apa, ya apake bachche ke sine mem darda, ya sansa lene mem kathinai ka anubhava hona shuru ho jata hai |
taiphaida ki gambhirata bhinna-bhinna rupa se ho sakati hai aura kitana baiktiriya ne sankramita kiya hai isa para bhi nirbhara karata hai |
lakshana halke ho sakate haim aura eka saptaha taka raha sakate haim, ya ve adhika gambhira ho sakate haim aura eka gambhira bimari ka karana ho sakate haim jo ki do mahine taka chala sakati hai |
roga se sankramita hona aura lakshana (ushmayana avadhi) dikhane ke bicha ki avadhi ama taura para 1-3 saptaha taka hoti hai |
taiphaida bukhara ke lakshana vishishta rupa se do alaga charanom mem hote haim |
taiphaida bukhara ka pahala lakshana sira darda aura bukhara hai |
apaka tapamana dhire-dhire 39 si 40 si (103 epha 104 epha) taka badha़ega |
isamem bhukha na lagana , joda़om ka darda , pasina ana, gale mem kha़rasha aura samanyataya asvastha hona mahasusa hota hai |
apa kabja aura peta darda ka anubhava karate haim aura apako chapate aura gulabi ranga ke chakatte ho sakate haim |
kabhi-kabhi, prarambhika lakshanom ke satha-satha khansi, naka mem se khuna ana aura laghushanka mem darda hona bhi shamila honge |
taipha़aida jvara vale bachchom ko prayah ulati aura dasta hote haim|
pahala charana samanyatah lagabhaga eka saptaha taka rahata hai|
roga ke dusare charana ke daurana, apaka jvara bana rahega aura, yadi upachara nahim kiya gaya to aura bhi bura ho jaega|
apaka nada़i spanda kamaja़ora ho sakata hai aura apake dila ki dhaड़kana dhimi pada़ sakati hai|
apa atyadhika thaka hua mahasusa karenge aura vibhramita tatha bhrantimulaka ho sakate haim|
apaka prarambhika kabja़ dasta mem badala jaega aura apake mala mem rakta ho sakata hai|
apaka yakrita aura pliha bada़e ho sakate haim, jisaki vajaha se ve pasaliyom ke niche, donom tarapha़ mahasusa kie ja sakate haim| 
apaka peta komala aura kashtakara mahasusa hoga|
samanyatah, taipha़aida jvara ke upacharita mamalom mem, yaha chauthe ya panchave saptaha taka nahim hota hai jaba apaka tapamana samanya para lauta ata hai aura apaki samanya sthiti mem dhire-dhire sudhara ana prarambha ho jata hai|
phira bhi, yadi anupacharita chhoda़ diya gaya, to taipha़aida jvara ghataka ho sakata hai|
yadi apa upachara nahim karate haim, to apako kai mahinom taka jvara ata rahega aura udaravaranashotha (peritonaitisa) jaisi jatilataem utpanna ho sakati haim jo anta ki divarom ke chhidrita hone ka parinama hota hai|
logom ke 10-30% ke bicha, jinako taipha़aida hai aura upachara nahim kiya jata hai, mara jate haim|
jo sabase adhika khatare para haim ve haim vayovriddha , bahuta chhote bachche aura ve loga jo kuposhita haim|
adhikatara enthreksa ke sankramana taba hote haim jaba jivanu tvacha para ke eka kata ya kharoncha mem pravesha karata hai, jaise ki jaba sankramita pashuom ke sandushita una, chamada़e ya roma utpadom (vishesha rupa se bakare ke bala) ko vyavahara mem late haim| 
tvacha ka sankramana uthe hue khujalahatadara sujana ke rupa mem prarambha hota hai jo eka kita dansha ke sadrisha hota hai kintu 1-2 dinom ke andara isa ghava mem phaphola paड़ jaega aura phira 1-3 semi ke vyasa mem, madhya mem eka kale parigalita (marate hue) kshetra ki vishishtata ke satha, eka dardarahita phoड़e mem vikasita ho jata hai|
sate hue kshetra ki lasika granthiyam suja sakati haim|
tvacha sambandhi enthreksa ke anupacharita mamalom ke lagabhaga 20% ghataka hote haim|
prati rogotpadaka jivanuom se sambandhita rogopachara ke satha nrityu asamanya haim|
prarambhika lakshana eka samanya sardi-jukama ke sadrishya ho sakata hai|
kai dinom ke bada, ye lakshana prachanda shvasana samasyaom aura aghata ki ora agrasara ho sakate haim|
shvasa enthreksa samanyatah tivra lakshanom ke akramana ke 1-2 dinom mem nrityu mem parinita ho jata hai|
anta sambandhi roga ka enthreksa ka rupa sandushita mansa ke upabhoga ke phalasvarupa ho sakata hai aura antra patha ke eka tivra jalana se visheshita hota hai|
michali ke prarambhika lakshana, bhukha ki kami, ulati, jvara ke bada peta ka darda, khuna ki ulati, aura tivra dasta haim|
anta sambandhi enthreksa 25% se 60% mamalom mem nrityu mem parinita hote haim|
echaaivi ki prarambhika avastha prathamika echaaivi sankramana ke rupa mem jana jata hai|
jo echaaivi se sankramita haim, una logom mem se 60% mem, samanyatah unake echaaivi se sarvaprathama sankramita hone ke 2-6 hapha़tom ke bada sambandhita lakshana vikasita hogem|
ye prarambhika lakshana prayah bahuta halke hote haim, atah unhem eka anya roga jaise ki saradi, ya granthiya jvara, samajane ki bhula karana asana hai|
upara darja prarambhika lakshanom ke bada, echaaivi prayah kai varshom taka koi aura lakshana utpanna nahim karega|
anupacharita chhoda़ne para, echaaivi apake sharira mem sidi4 koshikaom ki sankhya ko eka khataranaka tarike se ninna stara taka kama kara dega aura apaka pratirakshi tantra kama karana banda kara dega|
vishanu pratirakshi tantra ko isa taraha se kshati pahunchane mem samanyatah dasa sala leta hai|
eka kshatigrasta pratirakshi tantra nissandeha apamem eka gambhira sankramana vikasita karane mem parinita ho sakata hai| 
tivra asadhya sapuyacharmavisphota (nana-buliyasa impetigo) ke lakshana; samanyatah naka aura munha ke asa-pasa ke kshetra mem lala tvacha sankramanom ke prakatana se prarambha hota hai|
ye tvacha sankramana gadha़i, pili-bhuri sunahari papada़iyam chhoda़te hue turanta phuta jate haim|
ye papada़iyam phira eka lala nishana jo vishishta rupa se bina daga़ pada़e bhara jati haim, ko chhoda़te hue sukha jaengi|
ye tvacha sankramana kashtaprada nahim hote haim, kintu ve khujalahatadara ho sakate haim|
yaha mahattvapurna hai ki tvacha sankramanom ko na chhuem, ya kharonche kyonki yaha isa sankramana ko apake sharira ke anya bhagom mem, aura dusare logom mem phaila sakata hai|
sankramana ke anya lakshana jaise ki jvara aura suji hui granthiyam asamanya haim kintu ve adhika gambhira mamalom mem ho sakate haim|
susadhya sapuyacharmavisphota (buliyasa impetigo) ke lakshana, samanyatah dhada़, banhom aura tangom para drava se bhare phapholom ke prakatana se prarambha hota hai|
ye phaphole turanta phaila sakate haim aura phira kai dinom bada pili papada़i chhoda़te hue phuta jaenge, jo phira bina daga़ pada़e bhara jaenge|
ye phaphole kashtaprada nahim hote haim, kintu unake asa-pasa ki tvacha ka kshetra khujalahatadara ho sakata hai|
phira se, yaha mahattvapurna hai ki tvacha ke prabhavita kshetrom ko na chhuem ya kharonche|
jvara aura suji hui granthiyom ke lakshana susadhya sapuyacharmavisphota (buliyasa impetigo) mem adhika samanya haim|
jalantaka (raibija) kata ke sthana para darda ya siharana, jvara, bhukha ki kami aura siradarda se prarambha hota hai|
agale kuchha dinom ke daurana vyagrata, tanava, ananukulana, gardana ki akada़na, aura kabhi-kabhi daure (daure) ya ainthana (prabala ya achanaka gatividhiyam) ki eka badha़ti anubhuti hoti hai|
putaliyam phaili hui (vivardhita) dikhai pada़ sakati haim aura dhvani, prakasha aura tapamana ke prati badha़i hui samvedanashilata ho sakati hai|
eka haphte ke andara, kai sankramita vyakti nigale jane ka eka bhaya darshate haim|
atyadhika tivra pyasa ke bavajuda, pine ka koi bhi prayasa gala ghutane, shvasarodhana aura ghabada़ahata ki eka badha़ti anubhuti ke satha gale ki mansapeshiyom aura madhyapata (dayaphrama, phephada़om ke niche ki mansapeshi) mem prabala ainthana utpanna karata hai|
yaha jalatanka (haidrophobiya, pani ka bhaya) kahalata hai|
jaise-jaise yaha aura bigaड़ta jata hai, yaham taka ki pani ka drishya ya avaja ina prabhavom ko le ati hai aura pitane, thukane, katane aura unmada ke satha pagala vyavahara ke antarala hote haim|
jaba vyakti tivra vyagrata aura manasika dukha bhogata hai to ye akramana spashta-manasikata ki avadhi ke satha bari-bari se badalate haim|
ankha ki gatividhiyom aura chehare ke hava-bhavom ko niyantrita karane vali tantrikaem shithila ho jati haim aura hridaya tatha phephada़e kama karana banda karane lagate haim|
yadi ve anupacharita rahate haim to samanyatah prachanda lakshanom ke vikasita hone ke lagabhaga eka saptaha bada, kariba-kariba 100% mamalom mem sampurna lakava, sammurchha aura nrityu parinama hota hai|
vyakti jinhem akalaprasuta poliyo ya aghataja poliyo hota hai samanyatah purna roga nivritti karate haim|
phira bhi, ghataja poliyo, jaise ki isaka nama suchita karata hai, mansapeshiyom ka lakava utpanna karata hai- aura yaham taka ki nrityu mem parinita ho sakata hai|
ghataja poliyo mem, vishanu antra patha ko chhoda़ deta hai aura tantrikaom para akramana karate hue, rakta pravaha mem pravesha kara jata hai (akalaprasuta poliyo ya lakshanarahita poliyo mem, vishanu samanyatah antra patha se age nahim jata hai)|
yaha vishanu shvasana kathinai tatha hathom aura pairom ka lakava utpanna karate hue, hatha-pairom ke mansapeshiyom aura shvasana ke lie avashyaka mansapeshiyom ko niyantrita karane vali tantrikaom ko prabhavita karata hai|
rohini (dipha़thiria) ke lakshana achanaka a sakate haim|
sankramana ke samparka mem ane ke 1-9 dinom ke bicha (rogotpadaka jivom ke sharira mem pravesha karane tatha roga lakshanom ke utpanna hone ke bicha ki avadhi) roga ke lakshana prarambha ho sakate haim|
rohini (dipha़thiria) ke prathama lakshanom ko vikasita karake eka bachcha eka dina ke andara gambhira rupa se bimara ho sakata hai|
gale ki lasika ganthem itani suji hui ho sakati haim ki gardana adhika vistrita ho jati hai, prayah sanda़ ki gardana ke rupa mem ullikhita hoti hai|
apaka sharira bhi aghata mem ja sakata hai, jo ninna rakta daba, tivra dhada़kana, pilapana, thandi tvacha aura pasina utpanna karata hai|
kabhi-kabhi sankramana sirpha़ naka mem hota hai aura eka durgandhayukta, raktaranjita srava utpanna kara sakata hai|
jivanu samanyatah gale para akramana karate haim, kintu asamanya mamalom mem, yaha tvacha, vishesha rupa se khule ghavom ya chhalom mem raha sakata hai|
jivanu eka shaktishali visha (eksotaॉksina) utpanna karate haim, jo gale ke andara koshikaom ko marate hue tatha eka prakara ki bhuri-sapha़eda jilli utpanna karate hue, asa-pasa ki utakom mem nikala jata hai|
gale ki jilli thakka bane sirama (phaibrina), shveta koshikaom, jivanu, nrita satahi utaka koshikaom se banati hai aura samanyatah galatundikaom (taॉnsilom) ko dhakati hai|
yaha sataha se itani maja़buti se chipaki rahati hai ki ise chimati (phaॉrasepsa) se hatane ka koi bhi prayasa raktastrava utpanna karata hai|
jilli se yaha khatara hai ki yaha upari vayu margom ko avaruddha kara sakati hai, jo shvasa ko badhita kara sakati hai ya apake shvasa ko puri taraha se roka sakati hai|
yadi aisa hota hai to eka apatakala shvasa-pranala chhidrikarana (traikiyostaॉmi) ki avashyakata hoti hai|
tvacha sambandhi rohini (dipha़thiria) atyanta asamanya hai aura tvacha ko prabhavita karata hai, samanyatah tanga, pamvom aura hathom ke nichale bhagom para|
tvacha ki sataha para phaphole (phaphole) aura piba bhari phunsiyam (puyasphotika) prakata hoti haim aura akara mem kai sentimitara taka ka eka phoड़a (alsara) bana jata hai|
yaha phoड़a (alsara) eka gadha़e rupa-ranga ka hoga aura isake asa-pasa ki tvacha gulabi ya baigani dikhai degi tatha 1-2 hapha़tom ke lie kashtakara ho sakata hai|
rohini (dipha़thiria) ke jivanu ghavom se risate hue sankramita dravom (udaharana ke lie piba aura rakta) dvara phaila sakate haim|
ye phoda़e samanyatah 2-3 mahinom mem thika ho jate haim kintu samanyatah eka nishana chhoda़ denge|
yaha prakara ushnakatibandhi deshom mem adhika samanya hai kintu yaha ghatiya vyaktigata sapha़ai vale, beghara logom aura jale hue rogiyom mem dekha gaya hai|
yaha kitani jaldi vikasita hota hai aura isa roga ki prachandata, si. dipha़thiria jivanu visha (visha) utpanna karata hai ya nahim tatha pratiraksha (suraksha) ki matra jo usa vyakti ke pasa pahale ke dipha़thiria pratirakshana se pahale se hi hai, para nirbhara karata hai|
sankramana ke bada lakshana prakata hone mem 5-21 dina laga sakate haim, kintu adhikatara mamalom mem ve 7-10 dinom ke bada prakata ho jate haim|
naka aura munha mem bahuta sara shleshama ya najala-jukama ho sakata hai|
ina lakshanom ke prarambha hone ke bada, bachcha lagabhaga do saptaha taka atyanta sankramaka hota hai aura roga ko phailane se rokane ke lie, use anya bachchom, vishesha rupa se shishuom se alaga rakha jana chahie|
eka bara jaba unhem pratijaiviki di ja chuki haim, to ve kevala pancha dinom ke lie sankramaka honge|
jaise-jaise sardi-jukama ke lakshana sudharana prarambha karate haim, khansi ka daura ho shuru jata hai, aura rata mem vishesha rupa se kharaba ho sakate haim|
khansi ke ye daure bachche ke chehare ko lala kara sakate haim aura unaki ankhom se pani gira sakate haim|
yadi apaka bachcha atyanta asvastha lagata hai ya chehara nila pada़ jata hai to chikitsiya dekha-bhala ki talasha karem|
khansi bachche ka sansa lena kathina kara sakati hai|
khansi ke eka daure ke anta mem jyom hi ve hava ke lie teji se sansa lete haim ve khom khom ki avaja kara sakate haim- isa roga ko isaka nama isa avaja se milati hai|
khansi ke eka daure ke anta mem kabhi-kabhi bachcha khom khom karega phira bimara (ulati) ho jaega|
chhote shishu prayah bimara hote haim kintu khom khom nahim bhi kara sakate haim|
yadi apaka bachcha bara-bara bimara hota hai, to ve bahuta sara vajana kho denge aura kuchha dinom ke lie aspatala mem jana pada़ sakata hai|
chhote shishu, vishesha rupa se chhah mahine ke andara vale, sabase gambhira rupa se prabhavita hote haim|
bada़e shishuom aura bachchom mem aise prachanda lakshana nahim hote haim|
vayaskom mem, kukurakhansi ke lakshana samanyatah kevala sukhi, takalipha़deha khansi hoti hai jo kuchha mahinom taka rahati hai|
samanyatah khansi ke daure, yaham taka ki pratijaiviki upachara ke satha bhi 6-8 hapha़tom taka rahate haim|
pha़lu se jvara, khansi, kampane vali juda़i, sharirika pida़a, siradarda, aura thakana hoti hai|
ye lakshana samanyatah 3-4 dinom taka rahate haim, jisake bada apake pasa eka sukhi khansi, bahati naka, aura pradahayukta ya khuradura kantha ho sakata hai|
rogotpadaka jivom ke sharira mem pravesha karane tatha roga lakshanom ke utpanna hone ke bicha ki avadhi (inakyubeshana kala)- pha़lu vishanuom ke samparka se apake lakshana vikasita karane taka ka samaya- 1 se 4 dinom ka hota hai|
halanki loga prayah kisi bhi prakara ki halki bimari jisake pha़lu ki taraha ke lakshana hote haim, (jaise ki samanya sardi-jukama ya eka amashaya vishanu) ka varnana karane ke lie pha़lu shabda ka prayoga karate haim, phira bhi pha़lu vishishta lakshanom vala eka alaga vishanuja roga hai, aura yaha sala ke eka vishesha samaya- dera sharada ritu aura shita ritu mem hone ki ora pravritta hota hai|
rogajanaka netrashotha ke lakshana samanyatah eka ankha mem prarambha honge|
yadi apa rogajanaka netrashotha se pida़ita haim, to apako eka upari shvasana patha ka sankramana bhi ho sakata hai|
eka upari shvasana patha ka sankramana vaha hai jo apake gale aura vayumargom ko prabhavita karata hai|
skarleta jvara mem samanyatah 1 se 4 dinom ka inakyubeshana kala hota hai|
yaha roga prayah pradahayukta kantha ya eka tvacha sankramana se prarambha hota hai|
eka vyakti jise kabhi chechaka nahim hua ho, usa vyakti jisamem yaha vayarasa ho, ke samparka mem ane ke bada do se tina saptaha (samanyatah 13 se 17 dina) mem roga vikasita kara sakata hai|
chechaka vale vyakti ko dadora ubharane ke eka ya do dina pahale bukhara ho sakata hai ya asvastha mahasusa kara sakata hai|
samanyatah dadora sira para lala dhabbom se shuru hota hai aura agale paॉncha dinom mem nae dhabbe ubharana shuru ho jate haim|
dhabbe jaldi hi chhale bana jate haim jo pani se bhare hote haim, kuchha dinom mem pani vale dane sukha jaenge aura papada़i banaenge|
dadora kapha़i khujalidara ho sakate haim jo bachchom ko chida़chida़a aura baichena bana sakate haim|
adhikatara bachchom ko samanyatah halki bimari hoti hai yadyapi vayaska bahuta bimara ho sakate haim|
chechaka hava ke dvara phailata hai jaba chechaka grasta vyakti khaॉnsata, chhinkata ya sansa leta hai|
chechaka ke chhale ke pani ko chhune se bhi chechaka phaila sakata hai|
chechaka ke vyakti eka se do dina chahale se aura dadora ubharane ke 5 dina bada taka sankramana phelate haim|
adhikatara paristhitiyom mem prithakkikarana avashyaka nahim hai kyonki chechaka ka sthanantarana dadora vikasita hone ke pahale hona sambhava hota hai|
halki bimari vale bachchom ko skula ane ke lie svikriti de deni chahie jaba vo, danom ki sthiti ke bavaja़uda sabhi kriyaom mem bhaga lene ke lie|
chechakagrasta bachchom ko, dadore ki prathama avastha mem anya samuhom ke samparka mem ane se bachane ke lie, prithaka kara dena chahie kyonki vaha dadore ke chhathe dina taka ya jaba taka sabhi dadore para papada़i nahim jama jati taba taka abhi bhi sambhavatah sankramita hote haim|
chechaka vale vyakti ko jana yatayata ke sadhana mem sapha़ra nahim karana chahie|
nahane ke pani mem ½ kapa bekinga soda dalakara snana karo aura chechaka ke chhalom para kailemaina loshana lagaem|
aksara hatha dhoem khujalane se phailane vale sankramana se bachane mem sahayata ke lie agunli ke nakhuna chhote aura sapha़ rakhem|
18 varsha se kama ayu ke bachchom ko jinhem chechaka hai ko esprina ya e esa e hota hai nahim dem| 
esprina ‘reye sindraॉma’ ke khatare ko badha़a sakati hai jo jinako chechaka hai unake yakrita aura mastishka ko nukasana pahanchati hai|
ghara me yadi kisi ko chechaka ho to use ghara mem prithaka sthana para rakhane ki koshisha mata karie|
prithaka karane se ghara mem rahane vale dusare logom taka chechaka phailane se rokana lagabhaga asambhava hai|
yadi apa ya apaka bachcha chechaka vikasita karata hai to bimari ki suchana dene ke lie naja़diki jana svasthya kendra ko phona karie|
jana svasthya adhiniyama me chechaka eka suchaniya roga hai|
pratiraksha tantra ki samasyaom vale vyakti, garbhavati mahilaom jinhem kabhi chechaka na hui ho, vayaska jinhem kabhi chechaka na hui ho ke lie chechaka bahuta gambhira ho sakati hai|
yadi apa khatare ki kagara para haim ya kisi vyakti jise chechaka hai ke samaparka mem haim to age ki salaha ke lie apane pasa ke jana svasthya kendra para samudayika svasthya narsa ya apane parivarika chikitsaka ko phona karie|
vishishta ayu vargom aura sankatagrasasta shreniyom ke lie aba chechaka ke lie pratirakshana upalabdha hai|
una samuhom mem se eka ki taraha, shreni 5 ke bachchom ko 2001 ke arambha mem chechaka ke lie tika lagaya gaya|
de keyarsa (dainika dekhabhala kendra) ya skulom ke jaise sthanom mem vo vyakti jo bachchom ke nikata samparka mem ate haim, sardi jukama ke sankramana ke lie adhika khatare mem hote haim|
eka vyakti ko sardi jukama hota hai jaba vaha isako phailane vale vayarasa ke samparka mem ata hai|
sardi jukama ka vayarasa hastantarita ho sakata hai jaba sankramita vyakti khansa kara ya chhinka kara hava mem vayarasa chhoda़ta hai taka jise svastha vyakti sansa dvara andara leta hai|
sardi jukama ka vayarasa taba bhi hastantarita hota hai jaba eka svastha vyakti eka sataha jaise ki daravaja़e ki kundi ko chhuta hai jise sankramita vyakti ne chhua ho aura phira vayarasa ko sharira mem pravesha karane ke lie ankhom aura naka ko chhua ho|
bahati hui ya banda naka (sabase adhika samanya lakshana), eka kharaba ya khichakhichahata vala gala, kuchha rogiyom mem bukhara aura sukhi khansi, sardi jukama ke sanketa evam lakshana haim|
achchhi svasthya sambandhi adatem jinaka pratidina palana kiya jata hai sardi jukama se bachava ka eka mahatvapurna anga hai|
jaham taka sambhava ho anya vyaktiyom, jinhem sardi jukama hai, ke samparka se dura raho|
samanya sardi jukama ke lie koi vishesha upachara nahim hai lekina arama aura garma tarala padarthom (peya) ko pina lakshanom mem arama dene mem sahayata kara sakate haim|
panchava roga kapha़i samanya balyavastha ka roga hai jo ki samanyatah sardiyom aura basanta mem hota hai|
panchava roga samanyatah eka halki bimari hai jise sharira para eka dane aura chamakadara lala galom dvara pahachana jata hai|
panchave roga ko ‘‘slaipda chika ’’ sanrakshana ki taraha jana jata hai|
panchava roga skula ayuvarga ke bachchom mem adhika ama hai, lekina chhote bachchom aura vayaskom ko bhi bimari ho sakati hai|
adhinkasha vayaskom ko panchava roga ho chuka hota hai jaba vaha bachche the aura unhem dubara yaha nahim hoga agara vaha sankramita vyakti ke samparka mem haim|
purani raktaalpata ya pratirodhaka tantra ki samasyaom vali garbhavati mahilaom aura bachchom ko chikitsaka se samparka karana chahie agara ve panchave roga ke samparka mem haim|
samanyatah kuchha dinom taka ke lie roga eka halke bukhara, siradarda aura darda se shuru hota hai, chehare para eka lala dhabba nikalata hai, jo eka ‘slaipda chika’ki taraha dikhata hai, panchavem roga ka karana ho sakata hai|
eka se chara dina bada eka lakira ki taraha ka dhabba hatha, paira aura sharira para ubharata hai, agale eka se tina saptahom taka dhabba ata hai aura jata hai aura eka bara jaba dhabba ubhara jata hai samanyatah bachcha achchha mahasusa karata hai|
panchave roga se grasta vayaska samanyatah adhika bimara hote haim aura adhikatara unhem tivra joda़om ka darda hoga| vayaskom ko kai dinom taka bistara para rakhane ke lie joda़om ka darda tivra ho sakata hai|
panchava roga usi prakara phailata hai jaise ki samanya sardi jukama phailata hai|
panchave roga ka vayarasa sankramita vyakti ke shleshmala aura thuka mem hota hai aura jaba vyakti sansa leta hai, thukata ya chhinkata hai hava dvara phailata hai|
eka vyakti jise panchava roga hai, ke shleshamala aura thuka ke samparka mem ane para bhi vayarasa phailata hai|
eka vyakti jise kabhi panchava roga nahim hua hai, sankramita vyakti ke samparka mem ane para 4 se 14 dinom mem roga vikasita kara sakata hai|
panchave roga ka upachara hai, bukhara aura darda ka esitaminophena (udaharana ke lie tailenaॉla, tempra, painadaॉla) ke satha upachara karo|
paॉnchavem roga se bachava karana kathina hai kyonki eka sankramita vyakti lakshana prakata nahim kara sakata hai aura roga mukhyatah dadora dikhane ke kuchha dinom mahale phailata hai|
usa karana ke lie bara jaba dadora ubharata hai bachchom ko skula ya ‘dainika dekhabhala kendra’ se ghara para rahane ki avashyakata nahim hai| 
panchavem roga ko phailane se rokane ke lie bachche ki naka ponchhane ke bada ya upayoga mem lae gae tishuja़ ko chhune ke bada apane hatha sabuna aura garma pani se dhoem|
panchavem roga ko rokane ke lie bachchom ko upayoga mem lae gae tishuja़ ko kuda़edana mem dalana aura khansane, chhinkane ora nama ponchhane ke bada achchhi taraha hatha dhona sikhaem|
rubela ya jarmana khasara eka halka roga hai jo vayarasa dvara phailata hai|
rubela ‘3-de mija़ilsa’ ki taraha bhi jana jata hai kyonki dadora 3 dinom taka rahata hai|
aba rubela bahuta jyada ama nahim hai kayonki adhikansha bachchom ya yuva vayaskom ko pratirakshana diya gaya hai aura adhikatara bada़e vayaskom ko jaba vaha bachche the roga ho chuka tha|
garbhavati mahilaom ke atirikta rubela adhikansha logom ke lie eka nukasana dayaka roga nahim hai|
rubela garbhapata, nrita shishu ya janma sambandhi dosha utpanna kara sakata hai|
chara mem se eka mahilaom ke jinhem garbhavastha ke prathama ti mahinom ke daurana rubela ho jata hai janmajata doshom ke satha shishu ka janma hoga|
rubela shishu ke ankhom, kanom hdaya, nada़i tantra aura mastishka ko kshati pahuncha sakata hai|
lakshana samanyatah halke hote haim aura rubela vale kuchha logom mem koi bhi lakshana nahim ho sakate haim|
halka bukhara, siradarda, darda hona, gardana ke pichhale bhaga para aura kanom ke pichhe chhoti suji hui granthiyam|
eka gulabi dadora jo chehare para shuru hota hai aura sare sharira para niche ki ora aura hathom aura pairom para ubharata hai|
kishorom aura vayaskom vishesharupa se mahilaom me joda़om mem darda|
vayarasa jo rubela ka karana hai sankramita vyakti ke munha naka aura gale mem paya jata hai|
rubela hava dvara phailata hai jaba eka sankramita vyakti khansata, chhinkata ya sansa leta hai|
vayarasa sankramita vyakti ke shleshma ke sidhe samparka mem ane para jaise ki chumana, dvara bhi phailata hai|
rubela grasta vyakti dadora ubharane ke lagabhaga pancha dinom pahale ya kama se kama chara dinom bada taka, vayarasa phaila sakata hai|
rubela ke lie koi vishesha upachara nahim hai lekina bukhara aura darda ke upachara ke lie esitaminomphena (uda. tempara, tailenaॉna, penajaॅla) ka prayoga karem|
rubela grasta vyakti ko dadora ubharane ke chara dinom bada taka skula, dainika dekhabhala kendra ya kama ki bajaya ghara para rukana chahie|
rubela ko phailane se bachane ke lie pratirakshana sabase achchha upaya hai|
sabhi bachchom ko rubela ka tika eka varsha ki ayu para aura skula shuru karane se pahale eka bustara ki khuraka leni chahie
koi bhi jise rubela nahim hua ho ya usane rubela ka tika nahim liya ho, kisi bhi jana svasthya kendra para tika lagava sakata hai|
eka mahila ko jo shishu ko janma dene ke lie kapha़i bada़i ayu ki ho garbhavati hone se pahale rakta ki jancha karavani chahie ki vaha rubela ke viruddha surakshita hai|e
agara vaha surakshita nahim hai to use rubela ka tika lena chahie|
eka garbhavati mahila ko apane chikitsaka se ‘rubela kakta jancha’ ke lie kahana chahie|
eka garbhavati mahila surakshita nahim hai, use shishu ke janma ke bada rubela ka eka tika lagavana chahie|
hatha, paira aura munha ka roga eka halki bimari hai jo hathom, pairom aura munha ko prabhavita karati hai|
hatha-paira aura munha ka roga samanyatah 10varsha ki ayu se kama vale bachchom mem utpanna hota hai lekina una logom ko jinhem kabhi bimari na hui ho, ko asani se phaila sakata hai|
hatha-paira aura munha ka roga eka vayarasa dvara phailata hai aura adhikatara garmiyom aura sharada mem hota hai|
hatha-paira aura munha ka roga, khura aura munha ke roga jo janavarom ko hota hai se sambandhita nahim hai|
hatha-paira aura munha ka roga adhikansha logom mem koi lakshana nahim dikhata|
halka bukhara, sira darda, kharaba gala, bhukha na lagana aura shakti ki kami samanya lakshana haim|
ghava jo munha mem, jibha para aura kabhi-kabhi masuda़om aura hothom para vikasita hote haim|
ghava vikasita hone ke lagabhaga do dina bada hathom ki hatheliyom para aura pairom ke talavom para eka lala dadora chhale ke satha dikhai deta hai|
vayarasa jo hatha-paira aura munha ke roga ko utpanna karata hai sankramita vyakti ke naka va gale ke shleshma satha hi satha mala mem hota hai|
hatha-paira aura munha ke roga ka vayarasa hava dvara phailata hai jaba eka sankramita vyakti khansata, chhinkata ya sansa leta hai|
hatha paira aura munha ka roga taba bhi phaila sakata hai jaba rogagrasta vyakti naka ya chehara ponchhata hai aura jaba sankramita bachche ka daipara (pothada़a) badalate haim ya usaki shaucha kriya mem sahayata karate haim|
hatha-paira aura munha ka roga lakshanom ke prakata hone ke thoda़e samaya pahale aura phira bimari shuru hone ke eka se do dina taka bhi phaila sakata hai|
eka vyakti jise roga hai ke samparka mem ane para tina se sata dina bada taka eka vyakti hatha-paira aura munha ka roga vikasitakara sakata hai|
hatha-paira aura munha ka rogaeka adhika gambhira roga ki taraha dikhai de sakata hai, yadi apa sochate haim ki eka bachche ko hatha-paira aura munha ka roga hai, eka chikitsaka dvara usaki jancha ki jani chahie|
hatha-paira aura munha ka roga ke bukhara aura darda ka esitamainophena (uda. tailenaॉla, tempra, penadaॉla) dvara upachara kijie|
agara bachche ka gala kharaba hai to use nagama, bina masale vale khadya padartha dem aura bachche ko aise tarala padartha dem jo munha mem jalana utpanna na karem|
hatha-paira aura munha ke roga ke phailava ko kama karane ke lie bachche ki naka ponchhane ke bada ya prayoga mem lae gae tishu ko chhune ke bada garma pani aura sabuna se hatha dhoem|
bachchom ko upayoga mem lae gae tishuja़ ko kacharapatra mem dalana aura khansane, chhinkane aura naka ponchhane ke bada hatha dhona sikhaem|
haintavayarasa eka roga hai jo sabase pahale nrigachuhom dvara phailaya jata hai|
haintavayarasa logom taka, chuhe ya usaki minga, mutra ya lara dvara phailata hai|
alabarta mem haintavairasa sankramana kaphi kama hota hai lekina 65 se 77 pratishata rogiyom mem yaha ghataka hota hai|
aja taka kishora evam vayaska hi vaha ayuvarga hai jo ki haintavairasa dvara sankramita hua hai|
haintavairasa ke lakshana phlu ki taraha hota hai aura isamem 1040 F (40 0C) taka bukhara, kampakampi, badanadarda aura sansa lene mem takalipha़ shamila haim|
haintavairasa ke lakshana nriga chuhe ya usaki mingom mutra ya lara ke samparka mem ane ke do saptahom ke andara dikhai dene lagate haim|
kuchha rogiyom mem haintavairasa ke lakshana ramparka mem ane ke 45 dinom bada taka lakshita nahim hote haim|
hipetaitisa e yakrita ka eka sankramana hai jo eka vairasa dvara hota hai|
hipetaitisa e ‘sankramaka hipetaitisa’ bhi kahalata hai kyonki yaha dusare prakara ke hipetaitisa ki apeksha asani se eka vyakti se dusare ko phailata hai|
bimari samanyatah do se tina sptahom taka rahati hai lekina kuchha vyaktiyom mem yaha achchhi hone ke lie kai mahine le leti hai|
asadharana rogiyom mem hipetaitisa e nrityu ka karana bhi bana sakati hai|
hipetaitisa e kisi ko bhi ho sakata hai aura kisi vyakti ko hipetaitisa e eka bara ho chuka ho to use samanyatah dubara nahim ho sakata|
hipetaitisa e koi bhi lakshana nahim utpanna kara sakata ya kuchha logom mem, mukhyatah chhote bachchom mem, kevala bahuta halki bimari hoti hai|
jaba lakshana dikhai dete haim , adhikatara vaha achanaka bukhara thakana, kama bhukha, ji michalana aura peta darda ke satha shuru hote haim|
kuchha dinom se lekara kuchha saptaha bada mutra gahara ho sakata hai, mala halke ranga ka ho sakata hai aura ऑnkhem va tvacha pili ho sakati hai (piliya)
jisa vyakti ko yaha roga hai ke samparka mem ane ke do saptaha se do mahine bada vyakti ko hipetaitisa e ho sakata hai|
adhikanshatah hipetaitisa, lakshanom ko prakata karane mem lagabhaga eka mahina leta hai|
vayarasa jo hipetaitisa e phailata hai, sankramita vyakti ke mala mem paya jata hai|
hipetaitisa sabase adhika mala se hathom taka phelata hai jaba eka vyakti sanyogavasha mala ko chhuta hai jaise ki malatyaga karate samaya, potada़a (daipara) badalate samaya ya bachche ko shaucha mem sahayata karate samaya|
taba hipetaitisa ka vairasa hathom se dusari chija़om taka phailata hai jaise ki bhojana|
taba dusare vyakti ke hipetaitisa e ho sakata hai jaba vaha vayarasa vala sankamita bhojana khata hai ya pani pita hai|
vastuom jinamem lara hoti hai jaise ki tuthabrasha, khane ke bartana, pine ke kapa ko apasa mem upayoga lane se bhi hipetaitisa roga phaila sakata hai|
hipetaitisa lara ke samparka mem ane para asani se nahim phailata|
hipetaitisa e vale loga dusarom taka vayarasa phaila sakata hai jaise hi vaha bimara mahasusa karane lagata hai usase eka ya do saptaha pahale aura bimari shuru hone ke eka aura saptaha bada taka
hipetaatisa eka ka koi vishishta upachara nahim hai|
hipetaitisa e vale vyakti ko samanyatah bahuta adhika arama ki avashyakata hoti hai|
yadi vyakti bimara mahasusa karata hai aura usaki bhukha kama hai to bara-bara kama matra mem bina masaledara khana sahayaka ho sakata hai|
shauchagriha ka prayoga karane ya potada़a badalane ya khana banane ya khane ke bada hathom ko hamesha achchhi taraha dhoem|
una vastuom ko jo munha se chhuti haim jaise ki khana khane ke bartana, pani pine ke bartana ya tuthabrasha ka kabhi sammilita prayoga na karem|
yadi apa eka vayakti ke samparka mem haim jise ‘hipetaitisa e’ hai to turanta apane naja़diki jana svasthya kendra para salaha ke lie phona karem|
kuchha vyaktiyom ko hipetaitisae ke vikasita hone se bachava ke lie inyuna globyulina ki (khuraka) ki avashyakata ho sakati hai|
hipetaitisa e ka tika una logom yatra jivana shaili ya upachara mem hastakshepa ke karana sankramana ke adhika khatare mem hai, ke lie upalabdha hai|
hipetaitisa bi vayarasa dvara hone vala yakrita sankramana hai|
hipetaitisa bi ‘‘ sirama hipetaitisa’’ bhi kahalata hai kyonki roga sankramita vyakti ke rakta (sirama) ke samparka mem ane se bhi phailata hai|
yaha bimari samanyatah do se tina saptahom taka rahati hai lekina kuchha logom mem yaha thika hone mem kai mahine le leti hai|
kuchha rogiyom me hipetaitisa bi nrityu ka karana bhi bana sakati hai|
adhikansha loga hipetaitisa bi se lagabhaga tina mahinom mem thika ho jate haim aura dubara roga hone se surakshita hote haim|
lagabhaga 10 pratishata loga thika ho jate haim aura dubara svastha mahasusa karate haim lekina abhi bhi dusare vyaktiyom mem hipetaitisa bi phaila sakate haim|
hipetaitisa bi grasta vyakti mem koi bhi lakshana nahim ho sakate haim ya bahuta halke lakshana ho sakate haim visheshataya agara vyakti eka chhota bachcha hai|
jaba lakshana utpanna hote haim vaha adhikatara dhire-dhire dikhai dete haim aura thakana, kama bhukha lagana, joda़om ka darda, ulti, ji michalana aura peta darda ko shamila kara sakate haim|
kuchha dinom se lekara eka saptaha bada mutra gahara ho sakata hai, mala ranga mem halka ho sakata hai aura ankhem aura tvacha pili ho sakati haim (piliya)|
hipetaitisa bi ka vayarasa adhikatara rakta, virya, yoni strava aura bahuta kama bara thuka ke samparka mem ane se phelata hai|
sankramita vyakti ke satha suiyom ke sammilita prayoga (jisamem kana chhedane, taitu gudavane, vidyuta apaghatana aura ekyupankchara mem prayoga ki jane vali suiyam shamila haim|
sankramita vyakti ke satha yauna sammbandha banane se bhi hipetaitisa bi ho sakata hai|
hipetaitisa bi, aisi vastuem jinamem kama matra mem rakta hota hai jaise ki reja़ra, tuthabrasha ya kana ke kundala ke sammilita prayoga dvara bhi phailata hai|
hipetaitisa bi, eka navajata shishu mem usa ki mam se garbhavastha ya janma ke samaya bhi pahuncha sakata hai|
eka bara hipetaitisa bi ka vayarasa sharira mem pravesha kara lene ke bada, isake lakshanom ke lie, yadi koi prakata hote haim to, 50 se 180 dina laga sakate haim|
hipetaitisa bi ke lie koi vishishta upachara nahim hai|
hipetaitisa bi vale vyakti ko atyadhika arama ki avashyakata hoti hai|
pratirakshana hipetaitisa bi ko phelane se roka sakata hai|
panchavi shreni me sabhi bachchom ko hipeaitisa bi ka tika lagana chahie|
anya vyakti jo hipetaitisa bi ka tika chahate haim adhika janakari ke lie naja़diki jana svasthya kendra para phona kara sakate haim|
suiyom ya reja़ra ke bledsa ko sakhta dibbom jaise ki eka blicha baॉtala ya kaॉphi ke dibbe mem dalane ke dvara savadhani purvaka phenkie|
kaॉndaॉma ka prayoga karem ya hipetaitisa bi se bachane ke lie yauna sambandhom se bachem|
apaka niyamita sathi hipetaitisa bi ke tike dvara roga se bacha sakata hai|
tuthabrasha, kana ke kundalom ya reja़ra ka sammilita prayoga na karem| ina vastuom para bahuta kama matra ke rakta mem bhi hipetaitisa bi ka vayarasa ho sakata hai|
kisi bhi prakara ki suiyom ka sammilita prayoga na karem (jaise ki dava ki suiyaॉm, kana chhedane vali, taitu banane ki suiyaॉm, vidyuta apaghatana ki suiyaॉm aura ekyupankchara ki suiyaॉm)|
agara apa eka vyakti ke samparka mem haim jise hipetaitisa bi hai, naja़diki jana svasthya kendra para paramarsha ke lie phona karem|
kuchha vyaktiyom ko hipetaitisa bi ke vikasita hone se bachava ke lie inyuna globulina ke injekshana aura hipetaitisa bi ke tike ki avashyakata ho sakati hai|
hipetaitisa si yakrita ka eka sankramana hai jo vayarasa dvara hota hai|
hipetaitisa si, hipetaitisa e aura hipetaitisa bi se bhinna hai|
hipetaitisa si ka vayarasa kshatigrasta yakrita, sutrana roga aura kabhi kabhi kainsara bhi paida kara sakata hai|
hipetaitisa si ke vayarasa vale adhikansha vyaktiyom ko yakrita roga ka anubhava nahim bhi ho sakata hai|
yadyapi hipetaitisa si kai varshom se charo tarapha hai entibaॉdi (roga pratikaraka) ke lie jaॉncha hamane abhi hala mem hi vikasita ki hai|
lagabhaga 80% loga jinaki jaॉncha hipetaitisa si vayarasa ke lie sakaratmaka hoti hai jaॉncha mem hamesha sakaratmaka aengem|
bahuta chhoti sankhya ke loga sankramita hona ke bada unake rakta se hipetaitisa si ke vayarasa ko dura kara pate haim|
hipetaitisa si vale adhikansha logom mem koi lakshana nahim hote haim ऑra kapha़i svastha mahasusa kara sakate haim|
hipetaitisa si grasta anya vyakti thaka sakate haim aura unhem ji michalana, ulti, bhukha na lagana, bukhara, siradarda aura puta darda ho sakata hai|
hipetaitisa si mem kuchha vyaktiyom ko tvacha mem khujali, ankhom aura tvacha ka pila hona (piliya), gahara mutra ya halke ranga ka mala ho sakate haim|
eka vyakti ke sankramita hone ke bada hipetaitisa si ka vayarasa lakshana utpanna karane ke lie 6 saptahom se 9 mahine le sakata hai|
kai vyaktiyom, jo hipetaitisa si se sankramita haim, mem lakshana 20 varshom ya aura adhika samaya taka lakshana dikhai nahim dete haim|
hipetaitisa bi vayarasa mukhyatah rakta, virya, yonistrava aura bahuta kama bara thuka ke samparka dvara phailata hai|
1990 se pahale dana kie gae rakta ki hipetaitisa si vayarasa ke lie jaॉncha nahim hoti thi|
eka vyakti jise 1990 se pahale raktadana kiya gaya ho hipetaitisa si vayarasa se sankramita ho sakata hai|
bahuta kama rogiyom mem hipetaitisa si eka navajata shishu mem usaki maॉm se garbhavastha ya janma ke daurana istantarita hota hai|
hipetaitisa si grasta vyakti ko samanyatah bahuta adhika arama ki apashyakata hoti hai|
suiyom ya reja़ra bledsa ko kathora dibbe jaise ki blicha baॉtala ya kaॉphi tina mem dalane ke dvara savadhanipurvaka phenkie| hipetaitisa si se bachane ke lie dhakkana ko banda kijie, banda karake tepa lagaie aura kuda़e mem rakhie|
kandoma ka prayoga karem ya yauna sambandhom se bachem agara apa ya apaki sathi ko rajahstrava hora raha hai|
apaka niyamita sathi hipetaitisa si vayarasa ke lie parikshita hona chahie|
tuthabrashom, kana ki baliyom ya reja़ra ka sammilita prayoga na karem| ina vastuom para yahaॉm taka ki bahuta kama matra mem bhi hipetaitisa si ka vayarasa ho sakata hai|
kisi bhi prakara ki suiyom ka sammilita prayoga na karem (davai ki suiyam kana chhedane vali, taitu ki suiyam, vidyuta apaghatana ki suiyam aura ekyupankchara ki suiyam|
edsa ka artha hai ekvayarda inyunodephishiensi sindraॉma|
edsa isa roga ki vikasita avastha hai jo ki hyumana inyunodephishiensi vayarasa (echa ai vi) ke karana hota hai,
vikasita echa ai vi sankramana ya edsa vale vyaktiyom ke pratirakshana aura tantrika tantra kamajora hote haim|
eka vyakti ke eka bara echa ai vi dvara sankramita ho jane para sharira vayarasa se lada़ne ki koshisha karane ke lie, rogapratikaraka utpanna karata hai|
yadyapi ye roga pratikaraka vayarasa ko nashta nahim kara sakate haim, eka rakta parikshana bata sakata hai ki vyakti mem yaha rogapratikaraka haim, isaka artha hai ki vaha purusha ya mahila echa ai vi se sankramita hai|
sankramana ke bada echa ai vi, ina roga prati karakom ke prakata hone mem 6 mahine taka le sakata hai|
jabaki kai vyakti jo echa ai vi se sankramita haim bimari ke koi sanketa nahim darshate haim|
phlu ke samana lakshana jo sankramana ke lagabhaga ti tina se chara saptahom mem prakata hote haim|
echa ai vi ke kai prakara ke lakshana jo ki kai mahinom taka chalate haim mem shamila haim – vaja़na ghatana, thakana, bukhara, rata mem pasina ana, sukhi khansi, dasta, granthiyom mem sujana, yadadashta khona, sambhranti aura avasada| nimoniya, yista (thrasha), kainsara ya manasika rogom ki taraha|
echa ai vi eka vyakti jo echa ai vi se sankramita hai, ke rakta, virya ya yoni strava ke samparka dvara phailata hai|
kisi vyakti, jo ki sankramita hai ke satha asurakshita sambhoga (yoni, guda ya mukha dvara) karana|
koi vyakti jo sankramita hai ke davai injekta karane vali, taitu banane vali, tvacha mem chheda karane vali ya ekyupankchara ki suiyom ka sammilita prayoga|
garbhavastha ke samaya ya janma ke samaya ya stanapana karane se sankramita mam se usake bachche ko|
eka vyakti, jise navambara 1985 se pahale rakta hastantarita kiya gaya ho, bhi echa ai vi se sankramita ho sakata hai|
yadyapi kanada mem aba sabhi rakta aura rakta utpada echa ai vi rogapratikarakom ke lie parikshita haim|
echa ai vi sparsha, chhikanem, pani ke phavvarom, shauchalaya sita ya machchhara ya anya kida़om se nahim phailata hai|ee
echa ai vi sankramana ya edsa ka koi upachara nahim hai aura echa ai vi sankramana ko rokane ke lie koi tika nahim hai|
edsa grasta adhikansha vyaktiyom mem sankramana, kainsara aura anya lakshanom ka upachara lambe samaya taka jivita rahane mem sahayata kara sakata hai lekina yaha echa ai vi sankramana ka upachara nahim karata hai|
echa ai vi ko phailane se roka ja sakata hai- yauna sambandha na banakara, surakshita yauna kriyaem karake jisamem kandoma ka prayoga shamila hai ; suiyom aura dava dalane (injekta karane) ke anya upakaranom ka sammilita prayoga na karake|
impetigo eka tvacha sankramana hai jo baiktiriya dvara hota hai|
impetigo phailata hai jaba eka dadora, jaise ki ekjima ya kolda sorasa, sankramita ho jata hai|
samanyatah impetigo naka, munha aura kanom ke charom tarapha ki tvacha ko prabhavita karata hai lekina sharira para kahim bhi ubhara sakata hai|
impetigo ke chinhom aura sanketom mem shamila haim- chhote lala dhabbe ka eka dadora, chhala jo dhabbe ke upara banata hai aura phira phuta jata hai, bhure pile ranga ka khuranta banata hai|
impetigo sankramita kshetra se sidhe samparka dvara phailata hai ya una vastuom ko chhune se jase sankramita kshetra ke sidhe samparka mem rahi hom|
impetigo ka upachara rogapratikarakom dvara kiya jata hai jo baiktiriyaom ko mara dete haim|
khuranta ko hatane ke lie aura tvacha ko thika hone mem sahayata karane ke lie sankramita kshetra ko pratidina sabuna ke pani se bhi dhoya ja sakata hai|
mono nyukliosisa sankramana akasara epstrina bara vishanu dvara hone vali sancharana bimari hai|
sankramita mono nyukliosisa bada़e bachchom tatha javana vayaskom samanyataya jyada agrasara hota hai| 
mononyukliosisa sankramana ke chinha tatha lakshana bukhara gale mem darda, thakavata, chhote-chhote dane, tilli badha़na tatha livara ho sakate haim|
lagabhaga adhe loga jinamem mononyukliosisa sankramana vikasita hote haim mem kuchha athava kuchha bhi nahim lakshana hote haim|
adhikatara javana bachchem halke bimara hote haim jabaki vayaska mononyukliosisa sankramana se jyada bimara ho jate haim|
kisi ke thuka ke, jinake sankramana haim samparka mem ane se mononyukliosisa sankramana phailata hai|
kisi sankramita vyakti ke samparka mem ane ke 4 se 6 saptaha bada vyakti mem mononyukliosisa sankramana vikasita hota hai|
mononyukliosisa sankramana ka vyakti bachchom ki dekhabhala ke skula athava kama mem shamila ho sakate haim jaba taka ve samanya karyakalapa mem bhaga lene mem khuda thika na mahasusa karem|
halanki mononyukliosisa sankramana ko koi vishesha ilaja nahim hai, bimari vale vyakti ko daॉktara ki dekhabhala mem rahana chahie|
mononyukliosisa sankramana ke chinhom ka dhyana rakhie, ye samparka ke bada 4 se 6 saptaha taka rahate haim| 
sahabhagi vastuom khane ki vastuem, khane ke bartana ko nahim bante kisi vyakti jisako mononyukliosisa sankramana hai ke thuka se ninnatama samparka ke lie|
yadi apake ghara mem kisi ko mono nyukliosisa sankramana hai to vyakti ke khane aura pine ke upayoga hone vali vastuem dishavashara mem dhoe ja sakate haim|
inphluenja athava phlu vishanu dvara utpanna hota hai aura adhikatara kai loga, isa phlu se eka saptaha mem hi thika ho jate haim| 
jabaki, bachche athava vayaska lambi bimari se tatha 65 se adhika vayaskom mem ugra bimari vikasita ho jati hai jaise nimoniya|
phlu ke chinha tatha lakshana haim teja़ bukhara, aksara thanda, sira darda mansapeshiyom mem darda, bhukha khatma hona, khansi (sadharanataya), gala sukhana tatha atyadhika thakana|
samane ane ke bada phlu ke lakshana eka se pancha dinom taka raha sakate haim|
phlu hava ke dvara phailata hai jaba sankramita vyakti sansa leta hai, khansata, chhinkata hai|
phlu ka vishanu phlu vale vyakti ke ....... athava la ke samparka se bhi phailata hai|
phlu ke vayaska tina se pancha dinom taka sankramita tatha javana bachchom mem lakshanom ke samane ane ke bada sata dinom taka rahate haim|
adhikatara rogiyom mem phlu ke lakshanom ka bistara mem arama, jyada matra mem tarala padartha pine tatha bukhara tatha darda ke lie eketaminophena (udaharana, tayalenola) lene se thika kiya ja sakata hai|
jinamem phlu haim una 18 sala se chhote bachchom ko esprina athava koi bhi esprina athava esa yukta utpada nahim dijie|
jo loga pratidina adhika longom ke samparka mem ate haim, jaise shikshaka, pratyeka sala pratirakshita hokara phlu se bimara hone ke jokhima ko kama kara sakate haim|
hara sala pratirakshana jaruri haim kyonki phlu vishanu pratyeka sala badalata hai|
phlu ke phailava ko kama karane ke lie bachche ki naka ponchhane athava tishu ke upayoga ke pashchata sabuna tatha garma pani se hatha dhovem|
phlu ke phailava ko rokane ke lie bachchom ko upayoga kie gae tishu ko kachara dana mem dalana tatha khansi chhikane athava apani naka nikalane ke bada hatha dhona sikhaem|
khasara eka vishanu dvara hone vali gambhira bimari hai jo lala khasara athava sabelom se bhi jani jati hai|
khasara khataranaka bimari ho sakati hai kyonki yaha shvasa nali mem sujana, kana sankramana tatha nimoniya ki ora agrasara ho sakati hai|
jatila rogiyom mem, khasara mastishka kshati tatha kai bara nrityu ka bhi karana bana sakata hai|
khasara aba sadharanataya hone vali bimari nahim hai kyonki adhikatara bachchom tatha javana vayaskom ko isake viparita pratirakshita kiya ja chuka hai aura kai bada़e vayaskom ko bachapana mem yaha bimari ho chuki hai|
koi bhi jo pratirakshita nahim hai athava jinhem ye bimari purva mem nahim hui ko khasara ho sakata hai|
shishu tatha vayaska jinako khasara hua hai sadharana tatha bahuta khatare mem hote haim|
garbhini mahila athava loga jo kuposhita haim athava jinamem pratirakshana tantra ki samasya hoti hai ko bimara hone ki jyada sambhavana hoti hai|
chunki khasara eka khataranaka bimari hai sabhi janakari ke tatha sandhikdha rogiyom ko rijanala pablika helpa ko suchita karana chahie|
khasare ke chinha tatha lakshana khansi se shuru hote haim, naka bahana, teja़ bukhara, ankhom mem sujana, pani ana, prakasha ke prati samvedanashila, munha me sapheda chhote chhale bhi dikhai de sakate haim|
khasare ka vyakti samanyataya 10 se 14 dinom ke lie bimara hota hai, lala dhabbem jo antima 3 se 4 dinom mem pahale chehare para ate haim aura sharira, hatha tatha pamva mem phaila jate haim|
khasare ka vishanu naka tatha rogi ke gale mem paya jata haim aura hava se phaॅlata hai jaba vyakti khansata, chhinkata aura sansa leta hai|
khasare vala vyakti vishanu ko anya longom mem dhabbom ke ane se pahale tina se pancha dinom mem phaila sakata hai|
eka vyakti ko kisi jisake bimari hai ke samparka mem ane ke 6 tatha 21 dinom bada khasara vikasita ho sakata hai|
khasare ke lie koi vishesha ilaja़ nahim hai lekina bimari ka dhyana se nirikshana karom aura daॉktara ko bulao athava apane pasa ke pablika heltha sentara se salaha lem|
khasare ke vyakti ko dhabbom ke chauthe dina ke bada taka dhara mem rahana chahie|
khasare ke phailava se bachane ka uttama tarika pratirakshana hai|
sabhi bachchom ko unaki unra ke eka sala mem khasare ka tika aura dusari khuraka anake pancha sala ki unra mem skula purva bustara ke satha lagani chahie|
1957 se purva janmem vayaskom mem sambhavayataya khasara ho gaya hoga aura punah nahim hoga|
yadi apa khasare grasita vyakti ke samparka mem ate haim, turanta apane pasa ke pablika heltha sentara ko salaha ke lie bulaem
kuchha logom ko khasare ke vikasa se bachane ke lie pratirakshana pratirakshana ke bundo ki jarurata hoti hai|
meningitisa jalana athava jilli ka sankramana hai jisamem mastishka tatha ridha़ ki haddi ko sanrekhita karata hai|
meningitisa jivanu (udaharana meningo ko kala) meningitisa athava vishanu (vairala meningitisa) ke karana hota hai|
kuchha sankhya ke logom mem sharira mem jivanu ke akramana se hota hai aura parinamasvarupa meningitisa athava anya gambhira sankramana ho jate haim|
meningitisa kisi bhi unra mem sankramita ho sakata hai, lekina samanyataya pancha sala se chhoti unra ke bachchom tatha 15 se 25 sala ke bada़e javana vayaska ko hota hai|
meningitisa ke prarambhika chinha bukhara, sira darda, matali, ulti aura samanya bimara sa mahasusa karana hai|
meningitisa ke ye lakshana samanyataya phlu ke lakshanom se jyada gambhira hote haim aura teji se adhika vikasita hote haim|
javana bachchom mem meningitisa ka janakari mem mukhya lakshana vyavahara mem badalava, ninda, chidha़na athava bahuta adhika rona bhi shamila hai|
jivanu jo meningitisa ka karana hai naka mem tatha sankramita vyakti ke gale mem paya jata hai|
meningitisa vyakti se vyakti taka vastuom ke bantane se phailata hai jaise sigareta, tarala tatha khane ke bartana jisamem se dushita ho sakate haim athava antaranga samparka se jaise chumbana|
jivanu meningitisa hava se bhi phailata hai jaba eka vyakti niyamita sankramita vyakti jo khansata tatha chhinkata hai ke samparka mem ata hai|
vishanu jo meningitisa ka karana haim sankramita vyakti ke mala tatha thuka mem paya jata hai|
vairala meningitisa vishanu jyadatara mala se hatha taka phailata hai jaba eka vyakti durghatana vasha mala ko chhuta hai jaise jaba antada़i khisakana, daipara badalate athava bachche ko tayaleta upayoga mem madada karana hai|
phira vairala meningitisa vishanu hatha se anya chija़om jaise bhojana taka phailata hai|
anya vyakti vairala meningitisa ko prapta karata hai jaba vo khana, khate haim athava pani pite haim jo vishanu se dushita haim|
vibhajita vastuem jinamem thuka shamila hota hai jaise tuthabrasha, khane ke bartana athava pine ka kapa vairala meningitisa phailata hai|
lekina vairala meningitisa bimari thuka ke samparka se asani se nahim phailata|
vaira meningitisa hava se nahim phailata|
bektiriyala meningitisa ka ilaja aspatala mem jivanunashaka se hota hai|
meningitisa se hama bachava kara sakate haim vastuem jaise sigareta drava tatha khanem ke bartana ko na banta kara|
khansate tatha chhinkate munha tatha naka ko dhaka kara rakhem aura meningitisa ke bachava ke lie bachchom ko bhi yahim karana sikhaem|
yadi apa eka vyakti jisako meningitisa hai ke najadika se samparka mem ate haim, apane naja़dika ke pablika heltha sentara para raya ke lie phona karem|
meningitisa ke vikasita hone se bachava mem madada karane ke lie apako entibayotika di jaegi|
mamapsa eka sankramana aura lara granthi ki sujana hai|
mamapsa aba samanya bimari nahim, vishanu ke dvara hone vala sankramana hai|
adhikatara bachche tatha javana vayaskom ko mamapsa ke viruddha pratirakshita kiya hua hota hai tatha adhikatara bada़e vayaskom ko bachapana mem yaha bimari ho chuki hoti hai|
koi bhi jise pratirakshita nahim kiya athava jisako pahale ye bimari nahim hui ko mamapsa ho sakata hai|
kuchha loga mamapsa mem jyada़a bimara nahim hota, jabaki anya bahuta bimara hote haim|
mamapsa ka mukhya lakshana eka athava donom kano ke niche samane ki aura sujana tatha darda hona hai|
mamapsa ke anya lakshanom mem bukhara, sakhta naka tatha munha tatha gale mem darda bhi shamila hai|
eka vyakti jisako mamapsa hai ko nigalane athava bolane mem mushkila hoti hai tatha khatta bhojana nigalane mem darda hota hai|
mamapsa ke karana asthai baharapana bhi ho sakata hai kyonki sujana kanom ke najadika hoti hai|
kishorom tatha vayaskom mem ovari tatha testsa mem sujana kara deta hai lekina prajanana mem akshamata asamanya hai|
vishanu jisake karana mamapsa hota hai vyakti jisako bimari hai ke thuka tatha shleshma mem paya jata hai|
vishanu hava se phailata hai jaba mamapsa sankramita vyakti khansata, chhinkata athava sansa leta hai|
mamapsa vishanu sankramita vyakti dvara bahati naka ko ponchhane ke lie upayoga kiye gae tishu mem paya jata hai tatha kisi ko phaila sakata hai jisane usa prayoga kiye gae tishu ko chhua|
mamapsa vishanu sankramita vyakti vishanu ko anya logom mem sujana prarambha hone ke bada sata dinom se no dinom mem phaila deta hai|
eka vyakti mem mamapsa kisi ke jisako bimari hai ke samparka mem ane ke bada 12 se 15 dinom (samanyataya 16 se 18 dina) mem vikasita ho jata hai|
mamapsa ka koi vishesha ilaja nahim hai, esitaminaphena (uda. tayalenala, tempra, panadaॉla) darda tatha bukhara ke ilaja ke lie upayoga karem|
eka bachche ko mamapsa ke satha skula se ghara mem athava bachche ki dekhabhala mem sujana prarambha hone bada no dina taka rahana chahie|
mamapsa ke phailava ke bachava ke lie uttama tarika pratirakshana hai|
sabhi bachchom ko mamapsa ka tika eka sala ki unra mem tatha skula prarambha hone se pahale bustara ki khuraka dedi chahie|
kisi ko jise mamapsa athava mamapsa tika nahim laga ko pablika heltha sentara se tika mila sakata hai|
phlu ankha ki putali ke avarana ka sankramana hai|
kisi ko bhi ai phlu ho sakata hai, lekina yaha bachchom mem samanyataya hoti hai|
yaham mukhyatah ti prakara ka phlu hota hai baiktiriyala phlu, vairala phlu, elerjika phlu|
baiktiriyala phlu aksara bachchom mem pinka ai kahalata hai|
baiktiriyala phlu vishanu ke karana hota hai aura anya ko asani se phailata hai|
vairala phlu vishanu dvara hota hai aura anya ko bhi phailata hai|
elerjika phlu elarji ke karana hota hai tatha sala ke kisi vishesha samaya mem hata hai visheshatah yadi vyakti ko bukhara ho athava anya vatavarana elarji ho|
elerjika phlu ankhom mem elarji (koi chija़ jisase vyakti ko elarji ho) hone ke karana bhi ho sakata hai|
bektiriyala phlu athava pinka ai ankha ka sapheda hissa lala dikhai deta hai tatha chakshu mem sujana hoti hai|
vairala mem ankha ka sapheda gulabi dikhai deta hai tatha ankhe dukhati hai tatha adhikatara pani ata hai|
bektiriyala tatha vairala phlu hathom se phailata hai|
phlu vala vyakti jaba sankramita ankha ko chhuta hai aura usake bada kisi anya ko chhuta hai to sankramana phailata hai|
eka vyakti ko bektiriyala phlu (pinka ai) kisi anya ke samparka mem ane ke bada jisako sankramana hai do ya tina dina mem ho jata hai|
pinavaॉrma chhote sapheda kida़e hote haim jo sankramita vyakti ki antom mem rahate haim|
vayaska pinavaॉrma lagabhaga 10 mimi lambe chhote hote haim aura sapheda dhage jaise lagate haim|
pinavaॉrma sankramana eka gambhira samasya nahim hai, lekina yaha sankramita vyakti ke lie aramadayaka nahim hota kyonki guda ke asa-pasa khujali hoti haim|
kisi ko bhi pinavaॉrma ho sakata hai, javana bachchom adhikatara sankramita hote haim kyonki ve samanyataya apane hatha dhone mem laparavaha hote haim|
kai logom me sakramana ke koi chinha nahim hote aura unhem ye pata bhi nahim hota ki vo sankramita haim|
anya loga, mukhyatah bachche, rata mem achchhe se so nahim pate athava khujali ke karana thika se baithane mem pareshani hoti hai|
ugra sthiti mem, sankramita vyakti ka vajana kama ho sakata hai aura guda ke asapasa khujali se lala tatha sujana a jati hai|
pinavaॉrma kevala sankramita loga, jo samanyataya hatha se guda ko chhune se phailata hai|
pinavarma ande sharira ke bahara do se tina saptaha jivita raha sakate haim|
pinavaॉrma ka jivana kala do se atha saptaha mem khatma hota hai|
vayaska kida़a guda se bahara rengata hai aura mada kida़a apane ande guda ke asa-pasa de deti hai|
jaba sankramita vyakti apani guda ko chhuta hai ande anya vyakti ko phaila jata hai athava usi vyakti ko punah sankramita kara deta hai|
pinavaॉrma asani se eka vyakti se dusare ko phailata hai, visheshatah eka hi ghara ke logom ke bicha|
daॉktara sankramana ke lie sadharana jancha kara sakata hai aura usake bada dava niyata karata hai|
pinavaॉrma sankramita vyakti ko pratyeka dina nahana usake nichale bhaga sabuna tatha pani se dhona chahie|
pinavaॉrma ke phailava se bachane ke lie apayoga ke bada tatha khane ki taiyari aura khane se pahale hatha sabuna tatha pani se achchhe se dhoem|
jaise senkramana ho jae sankramita vyakti ke kapada़e bistara tatha taulie ko garma pani se dhaem|
jisa dina se ghara mem sabhi davai lena prarambha karate hai yaha sunishchita kijie ki sabhi ko bistara, taulie, sone ke kapada़e tatha antahvastra sapha hom, yadi sankramana hai|
pinavaॉrma vale vyakti ko skula, bachche ki dekhabhala athava kama ke lie ghara mem baithane ki avashyakata nahim haim|
ringavaॉrma phaphundi ke karana hone vala tvacha sankramana hai|
suji hui tvacha eka chhalle ki taraha dikhai deti hai jabaki isa sankramana ka kida़o se koi kama nahim hai|
ringavarma ke lie tina adhikatara samanya jagaha sira, sharira tatha pamva haim (ise khilada़i ka pamva bhi kaha jata hai|)
sira ka ringavarma, phaphundi eka chhoti phunsi se prarambha hokara golakara phailati hai|
sankramita kshetra ke bala gira jate haim, eka asthai ganjakhujalinuma tatha sakhta nishana chhoda़ jate haim|
sharira ke ringavaॉrma mem sankramita kshetra mem nishana vikasita hote haim jo akara mem ubhare hui kinarom ke chhalle jaise hote haim|
pairom ke ringavarma mem,paira ki tvacha, samanyataya panje ke asapasa khujali chalati hai aura panje ke bicha ki tvacha mem darare tatha papada़idara ho jati hai|
phaphundi longo, janavara tatha dhula se uthai ja sakati hai|
phaphundi eguliyom ka satha nahim chhoda़ti aura jaba sankramita vyakti kahim ora chhuta hai to phaila sakati hai|
phaphundi kuchha vastuom ke bhi satha hoti hai kagam ......... sira ka phita, topi, kapada़e, tauliya bistara tatha phailati hai jaba koi anya ina vastuom ka upayoga karata hai|
ringavaॉrma jamina, nahane ka taba tatha shaॉvara stala se bhi phailata hai jaba sankramita vyakti isaka upayoga karata hai|
ringa varma phaphundi ke tvacha mem pravesha se nishana ke ane tatha kai dina leti hai|
phaphundi ringavarma ke lie sira ki khala utarane mem 10 se 15 dina tatha sharira ke ringavarma ko pradarshita hone mem 4 se 10 dina leta hai|
sira athava sharira ke ringavarma ko daॉktara dvara pata lagana chahie jo phaphundi pratirodhaka roga chikitsa aushadhi pradana karega|
pamva ka ringavaॉrma phaphundi pratirodhaka pratirodhaka krima, loshana athava pavadara dvara thika kiya ja sakata hai| 
sabhi prakara ke ringavarma ke lie sankramita kshetra ko pratidina halke se sabuna ya pani se dhoem ora achchhi taraha se sukhaem|
ringavarma ke phailava se bacha ja sakata hai janavarom ko chhune tatha kama karane athava dhula mem khelane ke bada hatha dhone se|
kanga, brasha, tauliya athava bahari vastra kisi se bhi nahim bante yadi apa ringavaॉrma se bachava karana chahate haim|
jaise hi sankramana hota hai sankramita kshetra ko chhune ke bada sabuna tatha pani se hatha dhoem|
ringavarma sankramana hote hi sarvajanika suvidhaom mem jaise vyayamashala, khela athava vyayama kendra, skula tatha satha mem nahana athava shaॉvara suvidhaom mem upayukta chappala pahane|
khujali sukshma kita ke karana hoti hai jo maitsa kahalata hai jo itane chhote hote haim ki anhem sirpha maikroskopa dvara dekha ja sakata hai|
khujali sukshma kita ke karana hoti hai jo maitsa kahalata hai jo itane chhote hote hai ki anhem maikroskopa dvara dekha ja sakata hai|
mada maitsa tvacha ki upari sataha para rahate haim jaham yaha pratyeka dina do se tina ande deti hai|
kisi ko bhi khulaji ho sakati hai kishora athava vayaska, jyadatara bachchom mem hati hai kyonki bachche eka dusare se aksara naja़dika se samparka mem ate haim aura aksara vyaktigata padartha bantata hai|
khulaji ka saphai se koi lena dena nahim hai, loga jo bahuta sapha rahate haim| jo loga kama sapha hote haim ki tulana mem jaldi hi khujali prapta kara sakate haim|
khulaji vale vyakti ko khujali se nishana ho jate haim|
dane aksara kalai ke asa-pasa pae jate haim, kohani tatha ghutane mem, anguliyom ke bicha, dhariyom ke nishana, lekina yaha sharira ke kisi bhi garma kshetra mem paya ja sakata hai|
do sala se kama unra ke bachchom mem nishana chehare tatha sira, hatha ki hatheli athava paira ki sataha mem bhi ho sakate haim|
kulhe para lala chhote-chhote dane, chhote chhale, sapheda gola rekhaem (bila) kuchalane ke nishana tatha sakhta papada़i dikhai deti hai|
khaja khujali ke kitanu logom ki tvacha mem hote haim, jinase khaja-khujali haim|
kitanu kapada़om taulie athava khujali yukta vyakti ke prayoga kiye gae bistara mem bhi pae ja sakate haim|
kitanu anya logom mem phailata hai jaba vo khujali yukta vyakti ki tvacha ko chhuta hai athava usa vyakti ke kapada़e, tauliya athava bistara ka upayoga karata hai|
khaja-khujali janavarom ko nahim phailati tatha eka vyakti janavara athava dhula dabbe se khaja-khujali prapta nahim karata|
loga jinake kabhi khaja-khujali nahim hui hai mem kitanuom ko tvacha mem pravesha karane ke bada do se chhah saptaha mem daga tatha khujali vikasita ho jaainge|
jina logom ke khaja-khujali purva mem hui hai kitanuom ke tvacha mem pravesha karane ke bada eka se chara dinom mem samanyataya teja़i se lakshana vikasita honge|
khaja-khujali ke ilaja ka prathama charana eka vishesha krima athava loshana (skebisaitha kahalata hai) kitanuom ko marane ke lie prayoga kiya jata hai|
khaja-khujali ke ilaja ka dvitiya charana sankramita vyakti ke prayoga kiye kapada़e bistara tatha tauliya-garma pani mem dhoem jaem athava sila plastika thaili mem banda aura do se pancha dinom ke lie phrijara mem rakhe|
khaja khujali ke ilaja ke ilaja ki vistrita janakari ke lie apane najadika ke pablika heltha sentara ko phona karem|
jaba vyakti ko khaja-khujali ho jae, ghara mem sabhi ka eka hi samaya mem ilaja hona chahie, usako bhi jisake pahale daga athava khujali nahim hui hai|
koi bhi jisaki tvacha se tvacha ka samparka usa vyakti se hota hai jise khaja-khulaji hai ka ilaja hona chahie|
bachche jinako khaja-khujali hai skula se ghara me rahana chahie jaba taka ilaja khatma nahim hota|
vayaska jinako khaja-khujali ho ko bachche ki dekhabhala, aspatala, ghara ki dekhabhala vistrita dekhabhala suvidhaem athava vyaktigata dekhabhala vyavastha ke karya taba taka nahim karane chahie jaba taka ilaja khatma nahim hota|
seksa se prasarita bimari athava STD, vibhinna sankramanom ka samuha hai jo seksa anga athava jananendriya kshetra ko prabhavita karate haim|
STD, vishanu athava jivanu ke dvara hota hai jise jine ke lie garma, andhakara tatha nami ki avashyakata hoti hai|
koi bhi jo aise vyakti se seksuala samparka karata hai jise STD hai sankramita ho jata hai|
STD ke chinha tatha lakshana vibhinna prakara ke sankramana se badalate rahate hai|
jananedriyom tatha mutra dvara se pasa nikalata hai|
STD ke chinha tatha lakshana jaba vyakti peshaba jata hai to jalana vala darda, khujali athava jananendriyom ke asa-pasa jalana, jananendriyom ke asa pasa darda, aurata ke nichale peta mem darda hona hai|
STD vyakti se vyakti taka antarana seksuala samparka se phailata hai visheshatah vejainala samagama maukhika seksa|
STD sakasmika samajika samparka se nahim phailata jaise chubanna, hatha milana athava khane ke bartana bantana athava sarvajanika suvidhaom ke upayoga se|
dengu ko prathama ghoshita mahamari 1779-1780 eshiya, aphrika tatha amerika mem hua|
adhikatara STD ka ilaja kiya ja sakata hai athava ilaja mem kama gambhira hota hai|
ilaja STD ke prakara para adharita hai badalata rahata hai|
STD hona bhavishya ke sankramana ke viruddha suraksha nahim deta|
eka vyakti jisaka STD ke lie saphalata purvaka ilaja hua hai ko aba bhi dubara bimari ho sakati hai|
kuchha marijom mem, sankramita vyakti ke seksuala sathi ko samparka kiya aura STD ke phailava se bachane ke lie ilaja kiya gaya|
STD ke phailava se bachane ka uttama tarika hai seksuala samparka se bachem|
anya tarika eka hi sathi jisake bare mem apa janate hai ki sankramita nahim ke satha seksa karem aura kisi anya ke satha seksa na karem|
pratyeka bara seksa karate vakta sahi prakara se kandoma ka prayoga kara STD prapta karane ki jokhima ko kama kara sakate haim, lekina kandoma 100 pratishata suraksha pradana nahim karata|
steptokokala (strepa) sankramana samuha A streptokasa jivanu ke karana hota hai|
adhikatara strepa sankramana strepa gala athava lala bukhara hai|
kuchha logom ko sirpha strepa gala athava kuchha ko sirpha lala bukhara hota hai|
kisi ko bhi strepa sankramana ho sakata hai, parantu yaha bimari adhikatara sakula ki unra mem bachchom mem hoti hai|
kuchha logom ko satrepa sankramana mem koi lakshana nahim hote|
anya logom mem, strepa gala samanyataya bukhara, siradarda, petadarda tatha gale darda se shuru hota hai|
gala tatha tanansila lala tatha suje hue hote hai aura samanyataya sapheda ghava hote haim|
lala bukhara samanyataya vyakti ke thika na mahasusa karane ke bada 24 ghante ke andara shuru ho jata hai|
sankramita vyakti lala, suje hue hotha, jibha lala tatha stroberi jaise lala dhabbe ho jate haim|
dagom ke ane ke bada pancha dinom mem anguli tatha panjo ke kinarom, hatheli tatha paira ki pagathali para tvacha atarane lagati hai|
gambhira hradaya athava kidani samasyaem vikasita ho sakati hai yadi strepa sankramana ka entibayotika se ilaja nahim kiya jata|
strepa jivanu sankramita vyakti ke thuka mem tatha naka tatha gale mem paya jata hai|
jivanu jaba sankramita vyakti bolata, khansata athava chhinkata hai taba hava ke dvara phailata hai|
strepa jivanu bhi sankramita vyakti ke thuka athava shleshma jaise chumbana athava pine ka kapa athava khane ke bartana bantane se phailata hai|
jaba strepa sankramana ke vyakti ka 24 ghante ke lie entibayotika se ilaja kiya jata hai to samanyataya vo jivanu ko nahim phaila sakate|
yadi ilaja nahim kiya jae, strepa kiya jae, strepa sankramita vyakti bimari ko 10 se 21 dinom ke lie kuchha paristhitiyom mem isase jyada dinom ke lie phailata hai|
eka vyakti mem strepa sankramana 12 ghante se 4 dinom mem usa vyakti jo sankramita hai ke samparka ke bada vikasita ho jata hai|
strepa sankramana entibayotika se thika kiya jata hai, jo 10 dina leta hai|
sankramita vyakti achchha mahasusa karana prarambha karata davaiyom ke prarambha hone bada|
eka vyakti jisamem strepa sankramana ke chinha vikasita ho gae hai ko apane daॉktara se jaldi se jaldi milana chahie|
yadi apako purva mem gathiya bukhara hua hai to turanta daॉktara se mile yadi apaka samparka kisi se hua jise strepa sankramana hai|
strepa sankramita vyakti ko skula bachche ki dekhabhala athava kama para nahim jana chahie jaba taka enti bayotika pure dina (24 ghante) na lenle|
dengu (DF) tatha dengu hemarejika bukhara (DHF) chara samana vastu jaivika rupa se bhinna, phlevivayarasa jati ke vayarasa sirotaipa (DENI1, DEN2, DEN3, DEN4) mem se eka ke dvara hota hai|
inamem se kisi eka sirotaipa se sankramana kevala usi sirotaipa se jivana pratiraksha deta hai atah dengu pravabhita kshetrom mem jivana ke daurana eka se adhika dengu se sankramana ho sakata hai|
dengu (DF) tatha dengu hemarejika bukhara mukhyatah ayanavrita tatha upaayanavrita pradeshom ki bimari hai| tatha chara vibhinna dengu sirotaipa eka chakra mem vyavasthita rahate haim jisamem manava tatha edisa machchhara shamila hai|
halanki edisa eka gharelu dina mem katane vala machchhara hai jo ki manava ko bhojana ke rupa mem pasanda karata hai, taka bahuta hi samanya edisa prajati hai|
sankramana roga vishayaka bimari jisaka spektrama samanya vayarala rindroma se lekara gambhira tatha ghataka hemarejika roga taka phaila hai, utpanna karata hai|
DHF ke mahatvapurna jokhima karakom mem sankramita vayarasa ka kshati pahuchana, tatha satha hi unra tatha marija ki purva dengu sankramana ka itihasa shamila hai| 
STD akasmika samajika sambandha jaise pyara karana tatha hatha milana ya juthe baratana ya jana suvidhaom ke prayoga karane ke dvara nahim phaila sakata hai|
tina mahadvipom para lagabhaga satha-satha prakopa ka hona yaha darshata hai ki ye vayarasa tatha inake rogavahaka machchhara ushtakatibandha mem 200 sala se adhika samaya se pura phailava hi|
inamem se adhikansha samaya, DF eka halki, aghataka ushtakatibandhiya agantukom ki bimari samaja jata tha|
samanyatah do mukhya mahamariyom ke bicha labem antarala (10-40 varsha) the mukhyatah kyonki eka ati samvedanashila janasankhya mem eka nae sirotaipa ka pradurbhava tabhi dikhai deta hai vairasa tatha unake rogavahaka machchhara tabhi jivita rahate haim jaba vo janasankhya kendrom ke bicha tairane vale jalapotom ke dvara unaka dhire vahana kiya jata hai|
dengu ki mahamari dvitiya vishva yuddha ke bada shuru hui thi tatha usake bada sari prithvi para phaila gai| 
bahuta sare sirotaipa ke karana mahamariyaॉm lagatara hoti hai, dengu vayarasa tatha inake rogavahaka machchharom ka bhautika phailava badha़ chuka hai tatha DHF pesiphaka kshetra tatha amerika me avirbhava ho chuka hai|
dakshinapurvi eshiya mem, DHF mahamari sabase mahale 1950 mem prakata hui thi, parantu 1975 taka usa kshetra ke bahuta se deshom ke bachchom mem yaha aspatala mem bharti hone va nrityu ka pramukha karana bana gaya tha|
1980 mem DHF ka dvitiya vistara eshiya mem arambha hua jaba shrilanka, bharata tatha maladvipa dvipom mem pahali pamukha DHF mahamari hui, pakistana ne dengu bukhara ki mahamari 1994 mem sabase pahale suchita kiya|
shrilanka tatha bharata mem mahamari vibhinna vayarasa sirotaipa se juda़i hui thi parantu DEN3 adhika prabhavashali tha tatha yaha pahale una deshom mem sankramita logom ke DEN3 vayarasom se bhinna tha|
35 salom ki namonjudagi ke bada dengu bukhara mahamari donom taivana tatha china ke logom ke ganarajya mem 1980 mem punah paida hui thi|
china ke logom ke ganarajya mem charom sirotaipom ke dvara eka ke bada eka mahamari hui thi tatha DHF ki pahali pramukha mahamari DEN2 ke karana hui thi tatha heniyana dvipa para 1985 mem suchita ki gai thi|
20 varsha se adhika ke saphala niyantrana karyakrama dvara andhikasha sankramana ki rokathama ke bada bhi 1990 se 1994 mem dengu/DHF ka singapura mem bhi punarutthana hua|
eshiya ke anya deshom mem jahaॉm DHF eka kshetriya mari hai, pichhale 15 varshom mem yaha mahamari ka adhika vistara hua hai| 
25 se adhika varshom ki namaujudagi ke bada paisiphika mem dengu vayarasa 1970 ke arambha mem phira se pradurbhava hua|
abhi ke kuchha varshom mem charom sirotaipa ke dvara mahamari gatividhiyaॉm kai dvipom para DHF mahamari ke satha badha़ chuki hai|
aphrika mem kama nigarani ke bavajuda 1980 ke bada chara sirotaipom ke dvara dengu phivara mahamari akasmika rupa se badha़ chuki hai|
adhinkasha gatividhi purvi aphrika mem hui tatha pramukha mahamari sabase pahale seshalsa (1977) kenya (1982, DEN2) mojambika (1985, DEN3), diboti (1991-92, DEN2), somaliya (1982, 1993 DEN2), tatha saudi araba (1993, DEN2), suchita ki gai thi| 
aphrika ya madhyapurvi mem DHF mahamari suchita nahim ki gai parantu mojambika, diboti tatha saudi araba se kuchha chhuta puta DHF ke samana rogavishayaka ghatanaaim suchita ki gai|
dengu/ DHF ka pramukha jana svasthya samasya ke rupa mem amerikiya kshetra mem akasmika pradurbhava hua hi|
shahari pile bukhara ko rokane ke prayasa mem jo ki e. ejipti ke dvara prasarita hota hai pena ameriki svasthya sangathana ne eka abhiyana shuru kiya jisane e.ejipti ka adhikansha madhya tatha dakshina ameriki deshom se 1950 tatha 1960 mem unmulana kiya|
e.ejipti unmulana karyakrama jo ki 1970 mem yunaiteda stetsa mem adhikarika rupa se samapta ho gaya tha, dhire-dhire anya jagahom mem kama ho gaya tatha machchhara una deshom me jahaॉm unmulana ho gaya tha phira se paida hone shuru ho gae|
1970 mem amerika mem kevala DEN2 vairasa maujuda tha halanki kolambiya tatha pyuratoriko mem pramukha phailava DEN3 ka ho chuka tha|
1977 mem DEN1 ka pradurbhava hua tatha sampurna kshetra mem 16 varsha se adhika samaya taka pramukha mahamari ka karana raha| DEN4 1981 mem aya tatha samana hi mahamari ke phailava ka karana bana|
1981 mem DEN2 ki nai prajati dakshina purvi eshiya mem ayi tatha amerika (kyuba) mem DHF mahamari ka prathama mukhya karana bani|
2003 taka ameriki kshetra ke 24 deshom ne DHF ghatanaom ki janakari ki pushti ki tatha aba kai deshom ki DHF sthaniya mahamari hai|
DEN3 vayarasa 16 salom ki namaujudagi ke bada amerika mem punah dikhai diya|
lagabhaga sabhi, DEN3 ko panama mem tatha 1995 ki shuru-ata mem kostarika mem pushti ki gai|
2005 mem sabase pramukha manava ko prabhavita karane vali machchharom se paida hone vali vairala bimari dengu hai|
dengu ka kshetriya vistara maleriya se tulanatmaka hai tatha lagabhaga 2.5 araba logom ki jindagi ina kshetrom mem mahamari, sankramana ke lie khatare mem hai|
hara sala dasa karoda़ DF ki ghatanaem hoti hai tatha lagabhaga sau se hajara ghatanaem DHF ki 
bahuta se deshom mem DHF ke dvara nrityu sankhya lagabhaga 5% hai parantu sahi upachara ke dvara ise 1% taka kiya ja sakata hai|
mahadvipi yunaiteda steta mem dengu ke prakopa ka kama khatara hai| 
edisa ejipti unase bhinna hai jo maleriya phailate haim tatha samanyatah dina ke samaya katate haim|
e. ejipti tatha edisa elbopiktasa do sakshama rogavahaka machchhara upasthita haim tatha kuchha paristhitiyom mem pratyeka dengu vairasa ko phala sakata hai|
isake atirikta kai sare vairasa hara sala ayanavriti kshetrom se lautane vale sailaniyom jahaॉm dengu vairasa sthaniya mahamari hai ke dvara ate haim|
1977 se 2004 taka ayatita dengu ki kula 3806 anumanita ghatanaem yunaiteda steta mem suchita ki gai|
halanki kuchha ikadhdhe kie gae namune prayogashala roga nidana ke lie paryapta nahim the, 864 (23%) ghatanaom mem dengu ki pushti ki gai|
yunaiteda steta mem nishkiya nigarani tatha bimari ko pahachane ke lie chikitsaka para bharosa, marija ki purva yatraom ki janakari, sahi jaॉncha ke namune tatha kesa ki suchana adi ke karana kai sare kesa asuchita hi raha jate haim|
ऑnkada़e yaha darshate haim ki dakshini tatha dakshini purvi yunaiteda steta ke rajyom mem jahaॉm e.ejipti paya gaya hai mem dengu phailava tatha chhutaputa ghatanaom ka khatara hai|
halanki saphara dvara dengu tatha niyam̍trita prakopa mahadvipi yunaiteda steta mem bhi hota hai, adhikansha yu.esa. nagarikom mem dengu ki ghatanaem sthaniya mahamari ke yu.esa. pradesha ke logom ke bicha sthanavarana se hoti hai|
CDC ne pyureto riko mem pyureto riko ke svasthya vibhaga ke satha milakara prayogashala adharita dhairyayukta nirikshana kiya|
DF/DHF ke akasmika vyapaka avirbhava ke karanom mem prathama hai vyapaka janasankhya parivartana hona jisamem se atyadhika mahatvapurna aniyam̍trita shaharikarana evam satha satha janasankhya vriddhi ka hona hai|
janasankhyika parivartanom ke parinamasvarupa ninnastariya griha vyavastha tatha doshapurna jala, nikasi evam kachara prabandhana vyavastha hui jina sabhi se e.ejipti ki janasankhya ghanatva mem vriddhi hui tatha e. ejipti janita bimariyom ke prasara mem sugamata hui| 
adhikansha deshom mem jasvasthaya ka adhara bhuta dha़acha bigada़a hai|
simita vittiya evam jana sansadhanom tatha pratispardhi prathamikataem ‘‘sankatiya vicharadhara (kraisisa menteliti)’’ ke rupa mem parinita hui jisamem mahamari ke sankramana ke rokathama ke karyakrama banane ki bajaya mahamari ke pratyuttara mem apatakalina niyantrana karyakrama chalane para bala diya gaya| 
havai jahaja ke dvara badha़ gai yatraem dengu vairasa se sankramita logom ke janasankhya kendrom ke madhya parivahana ka eka adarsha tarika pradana karati hai jo dengu vairasa tatha anya paithojinsa ke tivra sthanantarana mem parinita hoti hai|
anta mem kai dengu prabhavita deshom mem prabhavashali machchhara niyantrana vastava mem maujuda nahim haim|
purva mem bada़e machchharom ke niyantrana hetu altra lo volyuma (sukshma kama ghanatva) kitanashi sthaniya chhida़kava para vishesha mahatva diya gaya jo ki e. ejipti ke niyantrana ke lie apekshakrita nishphala tarika hai|
dengu ka koi tika upalabdha nahim hai|
halanki hala hi mem kendideta vairasa kshina karane vale tike banae ja chuke haim|
manava volantiyara para prabhavotpadakata jaॉncha aba arambha hona baki hai|
dusari pidha़i ke punah sanyojaka vairasa tike ke lie bhi anusandhana kie gae haim|
5 se 10 varsha taka janaupayoga ke lie prabhavashali dengu tike upalabdha nahim ho sakegem|
badha़ti hui mahamarika gatividhiyom ke tatkalika chalana evam dengu ke bhaugolika vistara ke virodhatmaka paridrishya ashajanaka nahim haim|
kai kshetrom mem dengu vairasa ke lakshana tatha sirotaipa ke avirbhava ki sambhavana hai jahaॉm e.ejipti ka janasankhya ghanatva uchcha stara para hai|
machchharom ko niyantrita karane ki takanika ke abhava mem hala hi ke kuchha sala mem jana svasthya adhikariyom ne bimari se bachava tatha logom ke sammilita prayasa se larva paida karane ke sthanom ko kama karake machchhara niyantrana para bala diya hai|
lambe samaya ke bada yaha tarika sambhavatah prabhavashali hoga| nikata bhavishya mem bimari ka sthanantarana ka prabhava nahim hoga|
isalie hamem eka unnata, pratikriyatmaka prayogashala adharita nigarani rakhane vale yantra ka nirmana karana chahie jo ki asanna dengu mahamari ke lie jaldi chetavani pradana kara sake|
kama se kama sarvekshana ke parinama logom ko koi kadama uthane tatha chikitsaka ko roga pahachanane tatha DF/DHF kesom ko sahi taraha se ilaja karane ke lie sajaga kara sakate haim|
sthaniya mahamari se paryaya hai ki bimari lagatara tatha kisi vishesha kshetra ya janasankhya para usaki niyamitata anumanitata ke satha hoti hai|
mahamari sankramita bimari ka vyapaka phailava hai jahaॉm kai sare loga eka hi samaya para sankramita ho jate haim|
dengu hemarejika bukhara (DHF)/DSS eka adhika khataranaka bimari hai jisake karana nrityu bhi ho sakati hai|
dengu bukhara ke chinha evam lakshana anayasa teja bukhara ana, teja agrabhaga mem siradarda ho|
dengu bukhara ke anya chinha evam lakshanom mem ऑnkhom ke pichhe darda (ritiro arabitala pena) jo ki ऑnkhom ki gatishilata ko kama karane, joda़o evam maॉnsapeshiyom ke darda, svada tatha pachana shakti ke jnana ki kami sine tatha upari angom para khasare jaise dane tatha ji michalana evam ulti hai|
halke hemarejika (rakta strava) ka avirbhava jaise naka ya masuda़e se khuna ka bahana ho sakata hai|
limphedenopethi lyukopeniya ke satha tatha sambandhita limphosaitosisa samanya hai|
dengu henerejika bukhara tatha dengu shaॉka sindroma ke chinha tatha lakshana dengu bukhara ke samana hai|
bukhara ke sudhara vale charana ke daurana marija ki halata spashta rupa se aura kharaba ho jati hai atayadhika kamajori, atyadhika baicheni, chehare para pilapana tatha yadakada teja tatha lagatara peta darda/ livara mem badha़a ho sakata hai|
hemarejika ghatanaem nirantara hoti rahati hai tatha inamem sakaratmaka raktabandhata jaॉncha peteche, jaldi se chota pahunchana, nada़i chhidrom se rakta bahana, episateksisa, munha tatha masuda़om se rakta bahana tatha tvacha para dada shamila hai|
GI kshetra se rakta ka bahana eka ashubha sanketa hai jisake bada samanyatah lambem samaya taka dahashava rahati hai|
hepetomegali (yakrita) samanya hai parantu yaha piliya ke satha nahim hoti hai|
marija sadamem mem ja sakata hai jisake pramana ho sakate haih pili thandi chipachipi tvacha, susti ya baicheni, marija ko pyasa ka anubhava hona tatha munha ka sukhana, teja kamajora nada़i tatha shvasa lene mem pareshani|
dengu ke nishchita kesa ka samana sthana tatha samaya ka hona|
dengu bukhara ka prabandhana lakshanatmaka tatha sahayogatmaka hai| isamem perasitamola do parantu esprina ya bruphena nahim|
dengu pure bharata mem adhinkasha mahanagariya shaharom tatha kasbom mem prachalita hai|
1996 ke daurana dilli mem dengu/DHF ka atyadhika avirbhava hua jaham ki lagabhaga 10252 kesa tatha 423 nrityu suchita ki gai|
dengu se sankramita vyakti bimari ke arambha hone ke 6 se 12 ghante pahale se machchharom se sankramita ho jata hai tatha yaha 3 se 5 dina taka rahata hai|
edisa ejipti dengu tatha dengu hemarejika bukhara ka mukhya karaka hai|
edisa elbopiktasa bhi dengu bimari ka sthanantarana karata hai|
edisa ejipti eka chhota sapheda pattiyom ke satha kala tatha lagabhaga 5 mimi akara ka machchhara hai|
edisa ejipti apane sharira mem vairasa ke vikasita hone mem (bahya udbhavana samaya) lagabhaga 7 se 8 dila leta hai tatha bimari ko sthanantarita karata hai|
edisa ejipti machchhara kisi bhi prakara ke manava nirmita patrom ya sangraha karane vale patrom jisamem pani ki chahe thoda़i hi matra ho mem janma lete haim|
edisa ejipti ke ande eka sala se bhi adhika bina pani ke jinda raha sakate haim|
edisa ejipti ke paida hone ke pasandida sthana dejarta kulara, drama, jara, pota, balti, phuladana, paudha lagane ki katori, tenka, kunda, botalem, tina, tayara, chhata ke ghatara, phrija ke patra, simenta ke blaka, kabra, baॉnsa ke dande, nariyala ka khola, peda़om ke chheda, tatha kai aise aura sthana jahaॉm barisha ka pani ikaththa ya sangrahita hai|
kyonki edisa ejipti patrom mem janma lete haim atah aise patrom ko dhudhanna tatha machchhara paida hone ke mula stronto ko hatana eka mahatvapurna karya hai|
samanya adami ko bimari tatha rogavahaka ke bare mem vibhinna midiya sadhanom jaise tivi rediyo, sinema, slaida adi ke dvara janakari batana|
logom ko edisa paida hone vale sthana ko dhudhanne tatha hatane ke bare mem jagrita karana tatha sammilita karana|
kulara tatha anya chhote patrom se kama se kama saptaha mem eka bara pani hatana|
dina mem sote samaya machchhara dani ya machchhara bhagane vale sadhanom ka prayoga karana|
samaya tatha sthana ke satha sandehatmaka kesom ki suchana mem achanaka badhottari tatha mahamari/bimari ke mausama vishesha mem vyavasthita hona| 
vahaka machchharom ke sandarbha mem gharelu larva suchakanka/vahaka suchakanka/ britom sanchakanka ke dvara darshayi gai vahaka ghanatva|
kuchha DF marijom mem bhi halke hemarejika pradarshana hote haim|
pahale dengu sankramana ke samaya DHF ka khatara 0.2% hai parantu yaha dengu vairasa ke dusare sirotaipa ke satha sankramana ke samaya 10 guna badha़ jata hai|
DF tatha DHF jvara janya charana ke daurana ka pradarshana tatha prabandhana samana hi hai|
di epha vishesha machchhara jo aidija़ ijipti kahalata hai ke katane se hastantarita hota hai|
aidija़ unase bhinna hota hai jo maleriya hastantarita karate haim aura samanyatah dina ke samaya katate haim|
edija़ ijipti ekatrita pani, gaddhe adi mem asani se paida hote haim|
machchhara sankramita hota hai jaba vaha dengu bukhara vale vyakti ko katata hai aura phira svastha vyakti ko vayarasa hastantarita karata hai jaba unhem katata hai|
dengu bukhara bachchom aura vyaskom donom ko prabhavita kara sakata hai|
achanaka bukhara ana, siradarda, eka asthai sharirika dora, kharaba gala aura khaॉnsi dengu bukhara ke prathama lakshana haim|
dengu bukhara mem atyadhika sharira darda, bhukha khatma hona, ji michalana aura ulti bhi ho sakati haim|
chara se chhah dinom ke bada bukhara utara jata hai aura eka samanya dadora ubhara sakata hai|
kabhi kabhi dengu bukhara atyadhika gambhira hota hai aura 
dengu bukhara ke lakshana vishishta nahim hote haim aura anya kai bimariyom se milate haim, shuruata mem use pahachanana kathina hota hai|
vayarasa ke viruddha koi vishesha upachara nahim hai lekina sahayaka savadhaniyaॉm bahuta mahatvapurna haim|
halke sankramana ka ghara para sadharana tarikom bistara para arama, bukhara ke lie tarala aura pairasitamola lena dvara upachara kiya ja sakata hai|
dengu vayarasa para pratijivanu davaiyaॉm asara nahim karati hai lekina di ja sakati haim agara eka dvitiya jivanu sankramana ka sandeha ho|
adhinkasha rogi thika ho jate haim lekina alpasankhya mem jatilataem ya yahaॉm taka ki nrityu bhi ho sakati hai|
chara prakara ke dengu vayarasa hote haim aura eka ke dvara sankramana kevala usi prakara ke viruddha pratirodhaka kshamata deta hai|
dengu bukhara ko vikasita karana abhi bhi sammbhava hai agara dusare prakara ke vayarasa se sankramita hai|
kyonki dengu bukhara ka koi vishesha upachara nahim hai, bachava atyadhika mahatvapurna hai|
machchharom ki paidavara ko kama karane ke lie jana svasthya ke upachara mahatvapurna haim|
vayaska machchharom ki uda़ana sima simita hoti hai aura samanyatah usa kshetra mem katate haim jahaॉm paida hote haim|
machchharom ko svachchha pani ki apashyakata hoti hai aura ghara ke andara aura bahara ekatrita pani mem paida hote haim jaise ekatrita pani, varsha jala, phenke hue patra ya tayarsa aura yahaॉm taka ki phuladana ke andara ka pani|
vyaktigata stara para machchharom ke katane se bachava ke lie uchaya karem jaise ki lambi mataluna aura kamija़|
machchharadani adhika upayogi nahim ho sakati kyonki ye machchhara dina ke samaya katate haim|
machchharom ko dura rakhane vali krima prabhavashali hoti haim agara sahi dhanga sam prayoga ki jaem|
eka navinatama adhyayana ne nishchita kiya hai ki davaiyaॉm daithaila -3 mithailabenjamaida (di i i ti) ki taraha bhi jani jati haim, jinamem daithaila taॉlumaida hoti hai, adhikatama suraksha pradana karati haim|
sharira ke sabhi khule bhagom para krima lagani chahie aura hara chara se paॉncha ghantom mem dubara lagani chahie|
machchhara pratirodhi tikiyaem evam vashpitra ghara ke andara prabhavashali suraksha pradana karate haim|
yadyapi machchhara kundaliyaॉm, prabhavashali ho sakati haim, dhume se lambe samaya taka ke samparka ke prabhava ajnata haim|
dengu eka vyakti se dusare ko pratyaksha rupa se nahim phaila sakata hai|
pratyeka varsha dengu ke mausama mem rakta aura rakta utpadom ki atyadhika kami hoti hai|
rajadhani mem dengu marijom ki lagatara badha़ti hui sankhya ke satha, shodhakarta vishesha dengu vayarasa ke gudha़ artha para bhi kama kara rahe hai jisamem shahara mem sarakari aspatalom para atyadhika rogiyom ka bhara badha़aya hai|
abhi taka ke shodhom ne darshaya hai ki isa varsha paya jane vala dengu vayarasa ghataka prakara ka nahim hai|
dengu, vayarasa 1,2,3 aura 4 vayarasa ke karana hota hai jisamem se 2 aura 4 hanikara (vishakta) prakara haim|
hamem bheje gae namunom ka vishleshana karane ke bada hamane paya ki adhikansha sankhya mem darja kie gae rogi dengu vayarasa 3 ke haim|
vayarasa 3, rajya mem sabase pahale 2003 mem dekha gaya vayarasa tha aura jabaki bada ke varshom mem samanya janasankhya mem vayarasa ki upペsthiti mishrita rupa se dekhi gai| isa varsha dengu vayarasa 3 ne
vishleshana kiya gaya hai ki dengu vayarasa ki pravriti do se tina varshom mem chakra ko doharane ki hoti hai aura hama 2003 ki pranaravriti dekha sakate haim|
desha ke mukhya chikitsiya sansthana, da ऑla indiya instityuta ऑpha medikala samisa (AIInS), ne bhi dengu vayarasa 2 paya hai|
samanya janasankhya mem dengu vayarasa 2 ki upasthiti bahuta simita hai|
1996 ke prakopa mem dengu vayarasa 2 se raktastrava ke karana bhari nrityu dara hui thi|
dengu vayarasa 3 ke karana bahuta bhari nrityu sankhya nahim hoti hai|
dengu aidija़ machchhara jo charom mem se kisi bhi eka prakara ke vayarasa se sankramita hai ke katane se hastantarita hota hai| 
aidija़ vishcha ke ushnakatibandhiya aura upa ushnakatibandhiya kshetrom mem paya jata hai|
dengu bukhara eka jvara janya bimari hai jo shishuom, chhote bachchom aura vyaskom ko prabhavita karati hai|
lakshana halke bukhara se lekara tivra siradarda, ऑnkhom ke pichhe darda, mansapeshiyom aura joda़om mem darda aura dadore ke satha, akshama bana dene vale teja़ bukhara taka hote haim|
dengu ke lie koi vishesha vayarasarodhaka davai nahim hai|
jalayojana ko banae rakhane ke lie dengu bukhara mahatvapurna hai aura esitaila sailisaiklika esida (uda. espirina), naॉna stiraॉidala jalana kama karane vali davaiyaॉm (uda.ibyuprophena) ke prayoga ki salaha nahim di jati hai|
dengu raktastrava bukhara, (petadarda, ulti, raktastrava) sambhavatah eka pranaghataka samasya hai jo mukhyatah bachchom ko prabhavita karati hai|
dengu, manushyom mem sabase adhika hone vala machchharom se utpanna vayarala roga hai jo ki adhunika varshom mem antarrashtriya janasvasthya ka eka mukhya mamala bana gaya hai|
vishvastara para 2.5 araba loga una kshetrom mem rahate haim jahaॉm dengu vayarasa hastantarita ho sakata hai|
machchhara rogavahakom aura vayarasom, donom ka bhaugolika phailava dengu bukhara mahamari ke vaishvika punarautthana ka karana bana hai|
ushnakatibandhi pradeshom ke kai shahari kendrom mem atyadhika sthanikata ke vikasa ke satha pichhale 25 varshom mem dengu raktastravayukta jvara (dengu/di echa eka) ka pradurbhava|
mukhya rogavahaka ‘endija़ aijitapi’ machchhara dvara hastantarita hone vale chara alaga, lekina ghanishtha sambandhita, vayarasa haim jo dengu phailate haim|
eka ke dvara sankramana se svasthya labha, usa sirotaipa ke virudva jivanaparyanta pratirodhaka kshamata pradana karata hai lekina anya tina se bada mem hone vale sankramana ke viruddha kevala anshika aura asthai bachava pradana karata hai|
sansadhanom ke purna vatavarana mem echa.ai.vi. ke upachara ke satha sambandha rakhane mem ऑnkada़om aura anubhavom ke abhava ke karana sasti enti ritrovayarala (e ara vi) uvaiyaॉm upalabdha karane mem, bharatiya jatigata aushadhiya udyoga ki kshamata ka aba svasthya-sanrakshana-prabandhakom dvara purnatah durupayoga hua hai|
di echa epha sarvaprathama 1950 mem philipinsa aura thailainda mem dengu mahamari ke daurana pahachana gaya|
1970 taka nau deshom ko di.echa.epha़ mahamari ka anubhava ho chuka tha aura aba yaha sankhya chara guna se jyada badha़ chuki hai aura lagatara badha़ti ja rahi hai|
aja ubharate hue ‘di.echa.epha.’ ke rogi amerika ke aura eshiya mem yahaॉm charom dengu mahamari phaila rahe haim|
di.echa.epha. aspatala mem bharti hone ka aura kai deshom mem bachchom mem nrityu ka mukhya karana bana gaya hai|
ajakala dengu aura di.echa.epha. bachava evam niyantrana ke lie rogavahaka niyantrana upalabdha vidhi hai lekina janasvasthya upayoga ke lie dengu ke tike para anusandhana chala raha hai|
dengu/di echa epha ke bachava aura niyantrana ke lie vishvastariya yojana dablu echa o dvara aura kshetriya yojana, amerika, dakshina purva aishiya aura pashchimi prashanta dvara vikasita ki gai hai|
ऑstreliya aura antarrashtriya stara donom mem sankramaka roga eka mahatvapurna jana svasthya prathamikata bani hui hai|
ऑstreliya jina samasyaom ka samana kara raha hai vaha kai taraha ki haim –bhojana se hone vali bimariyaॉm, sukshmajivanurodhaka ke prati pratirodhaka shakti vale jivanu, yauna sambandhom dvara hastantarita hone vali bimariyaॉm, rogavahakom dvara phailane vali bimariyaॉm aura tike dvara bachava ki jane vali bimariyaॉm|
nae aura ubharate hue roga jaise ki baita laisi vayarasa aura bovaina spaॉnjipharma ensephailopethi (bi esa i) satha hi satha jaivika karakom ke antarrashtiya phailava ka khatara, jana svasthya ke lie sambhavita khatare paida karate haim|
svasthya sanrakshana ka karyalaya (o echa pi) sankramaka roga ke prasara aura usase sambandhita samajika aura arthika prabhava ko kama karane ke lie kai prakara ke karyom mem laga hua hai| 
svasthya sanrakshana ka karyalaya sankramaka roga prakopa ke javaba mem rashtriya karyavahi ke sanyojaka aura sararvottama prakiya margadarshikom ke vikasa aura rashtriya janasvasthya niti ke neta ke rupa mem mukhya bhumika nibhava hai|
netavarka (tantra) dvara eka rashtriya pariprekshya sampadita kiya gaya hai jaise ki ‘kanyunikebala dijija़eja़ netavarka ऑstreliya’ (si di ena e) ora likhita pramana jaise ki neshanala kanyunikebala dija़ijeja़ sarvilensa etraiteja़ija|
svasthya sanrakshana ka karyalaya, rashtriya rupa se sarvettama svasthya prakiya ko sunishchita karane ke sandarbha mem takaniki evam niti sambandhi salaha pradana karata hai|
vistrita rashtriya aura antarrashtriya tantriya suvidhaom ki pahuncha sankramaka rogom aura sambandhita kshetrom mem visheshajnom ke nikaya taka hai jo ki roga ke rashtriya mahatta vale phailava ke prati uchita aura sahi samaya para pratikiya tatha jana svasthya nirnayom, nitiyom aura karyakramom mem uchcha gunavatta nivesha ko sunishchita karata hai|
kverentaina ekta 1908 ke manava svasthya paksha ke prashasana mem sammilita hone ke satha satha astreliya ki kverenataina ensapekshana sarvisa, krishi, machhali palana aura vanakhanda vibhaga-astreliya aura phuda staindarda ऑstreliya ke satha ghanishtha samparka hone ke satha, nyujilainda roga niyantrana ke lie svasthya sanrakshana karyalaya (ora echa pi) ke bahuayami prastava ko sudridha़ karata hai|
ajakala si di ena e eka pariyojana ‘neshanala kanyunikebala dija़ijija़ sarvilensa streteji’ (ena si di esa esa) karyanvita kara raha hai jise mukhya svasthya adhikariyom ka sahayoga purna anudana mila hai tatha ऑstreliyai svasthya mantriyom ki paramarsha samiti dvara samarthana mila hai|
‘ena si di esa’ eka yojanapurna dhaॉncha uplabdha karata hai jo nirikshana sambandhi ikrayaom ke samanvaya aura yojana ko sudridha़ banane tatha nai ya ubharate hue sankramaka rogom ka javaba dene ke lie yojanaem aura prathamikataem vikasita karane ke lie sarakara aura jana svasthaya adhikariyom ki sahayata karega|
isa yojana ka uddeshya ऑstreliya ki janasankhya para sankramaka rogom ke samajika aura arthika prabhava ko kama karana hai|
yaha yojana adhika vyapaka janapadika-rogavijnana ke ऑnkada़om jina para khatara sambandhi prabandhana aura jana svasthya niti adharita hai, ko upalabdha karane ke lie roga nirikshana ko sudharane para dhyana deti hai|
yaha yojana adhika niyamita aura roga ke phailane para prabhavashali prati̍ikaya ko badha़ava dene ke lie tantra ko sudridha़ karane ka uddeshya rakhati hai|
bhojana se utpanna bimariyom ke lie eka rashtriya nirikshana vyavastha ka vikasa jo ki susangata rashtriya suchanaem pradana karega, khadyasamabandhi rogom ke adhika vyapaka ऑnkada़om ke sangraha ko badha़ava dega aura rashtriya prachalita vishleshana ke lie eka avasara pradana karega|
sankramaka rogom ke bachava aura rokathama ke lie nirikshana mulasiddhanta hai|
kai sankramaka rogom ke lie ‘o echa pi’ (svasthya sanrakshana karyalaya) ka tantriya prabhaga aura nirikshana niti, rashtramandala ke ऑnkada़a sankalana aura samanvaya ka prathama kendra hai|
o echa pi ऑnkada़om ke sankalana aura/ya sankramaka rogom para anusandhana karane ke lie anya aijensiyom se bhi samanvaya aura anubandha karati hai|
yaha kendra haim – ‘neshanala sentara ina echa ai vi epidemiyolaॉji enda klinikala risarcha’ (ena si echa i si ara) aura ‘neshanala sentara phaॉra inyunaijeshana risarcha enda sarvilensa ऑpha vaiksina piventebala dija़ija़eja intelijensa’ (si di ai) ke dvara pracharita kie jate haim|
si di ai kabhi kabhi kuchha rashtriya nirikshana yojanao, jisamem neshanala maikrobaikatiyala saivilainsa sistama, da ऑstreliyana maikobaiktiriyala raipharainsa leboretari netavarka, da neshanala niseriya netavarka, ऑja़phuda neta, rota vayarasa saॅvilainsa aura da saintinela chikina skima shamila haim, para riportasa prakashita karata hai|
si di ai mem niyamita rupa se suchita rashtriya nirikshana yojana para janakari, si di ai mem suchita ki gai nirikshana paddhati dastavejom me vistara purvaka haim|
vebasaita ka sankramaka roga nirikshana vala bhaga sankramaka roga nirikshana ki janakari deta hai|
o echa pi ne kai ena echa ema ara si karyakarini dalom ko vaijnanika aura prashasanika sahayoga diya hai aura si di ena e, pi echa ela ena aura anya vibhagom ki karyakarini dalom ko isa prakara ka sahayoga de raha hai|
niti ke vikasa ya punaravalokana ke lie prathamikata vale kshetrom ki pahachana si di ena e dvara ki jati hai aura vibhinna prakara se vyavahara kiya gaya hai jaise ki vishishta karyaya pranali vale samuha, vishishta nitiyaॉm vikasita karane ke lie aura/ya sankramaka rogom ki samasyaom ki jaॉncha karane ke lie visheshajna ejensiyom se sauda kiya gaya|
si di ena e aura vishesha salahakara prakrayaom dvara rajya aura rajyakshetra rashtriya niti vikasa mem sahayoga de sakate haim|
rashtriya aidsa niyantrana ke anumana ke anusara bharata mem 4.58 karoda़ lo echa ai vi/ edsa ke satha raha rahe haim|
si di ena e vishesha salahakara prakiyaom dvara rajya aura rajyaksherashtriya niti vikasa mem sahayoga de sakate 
si di ena e aura vishesha salahakara prakiyaom ke anusara bharata mem 4.58 karoda़ loga echa ai vi/ edsa ke satha raha rahe haim|
desha mem echa ai vi sankramana ke tivra prasara ke prati kadama uthane ke lie eka prabhavashali tika kai upayom mem se eka mana gaya hai|
bachavakari echa ai vi tika echa ai vi sankramana se bachane ke lie banaya gaya hai jabaki chikitsiya echa ai vi tika vayarasa se pahale se sankramita vyakti mem pratirodhaka pratikiya ko badha़ane ke lie banaya gaya hai|
vikasashila deshom mem chikitsiya tike se adhika bachavakari echa ai vi tike ko badha़ava dene ka naitika dharmasankata kuchha tikakarom dvara svikara kiya gaya hai|
bharata mem echa ai vi sankramana ke satha raha rahe adhikansha loga unaki atisamvedanashilata aura unake echa ai vi prasthiti se anabhijna haim|
eka bachavakari echa ai vi tika ila logom ke jivana ki gunavatta mem natakiya parivartana nahim bhi la sakata hai|
purva prayasom aura anya vaijnanika ऑnkada़om ke parinama batate haim ki bachavakari echa ai vi tike ka vyaparika utpadana aura vitarana kai dashakom dura hai|
vayaskom mem echa ai vi hastantarana ka lagabhaga 90% asurakshita sambhoga ke dvara hota hai|
sambhoga ke dvara echa ai vi hastantarana mem kami kandoma ke lagatara aura sahi prayoga dvara sambhava hai|
echa ai vi hastantarana ki ati samvedanashilata aura pahachanane aura niyanntrita karane ki yogyata, linga aura samajika arthika stara ke anusara asamana rupa se vitarita hai|
echa ai vi se bachava ke upayom mem sanrachanatmaka karakom ko bhi sambodhita karana chahie jo ki mahilaom ko bhi sambodhita karana chahie jo ki mahilaom aura simavarti janasankhya ki atisamvedanashilata aura echa ai vi sankramana ke satha raha rahe logom ki samajika samarthana ki avashyakata mem sahayaka hote haim|
kisi ko bhi bachavakari echa ai vi tike mem punji lagane ke manavajatiya ashaya ka vishleshana aura chikitsiya echa ai vi tike mem punji na lagane ke avasarom ka mapana savadhani purvaka karana chahie|
chikitsiya echa ai vi tike, mahamari ke lie vistrita pratikriya aura svasthaya dekhabhaga pradana karane ki yojana ke tatkalina labhom ko sudridha़ banane ke bada़e avasara pradana karate haim|
eka chikitsiya tika roga ke boja ko kama karane mem aura echa ai vi sakramana ke satha raha rahe logom ke jivana ki gunavatta ko badha़ane mem sahayoga de sakata hai|
chikitsakiya echaaivi tike adhika se adhika avasara dete hai jo mahamari ki vyapaka pratikriya ko majabuta aura tatkalika labha svasthya dekhabhala vitarana pranali ke lie hai
eka chikitsakiya tika roga ke boja mem kami lane ke lie aura echaaivi sankramana se grasta logom ke jivana ki gunavatta badha़ane mem yogadana de sakate hai 
nirodhaka tike ke lie samarthana upachara ke lie eka echaaivi tike ki jarurata ka samajauta nahim hona chahiye
eka ki sankramana ke bada arogya prapti sirotaipa ke khilapha ajivana unmukti pradana karata hai, lekina bada ke anya tina sankramana ke khilapha kevala anshika aura kshanika sanrakshana pradana karata hai
vartamana manakikrita earavi chikitsa jaldi hi upachara ke vikalpa ke bahara chali jayagi, kyoki ve bada़i abadi para kama prabhavi ho gai hai
eka akramaka chikitsakiya echaaivi tike ke anusandhana karyakrama ke vaikalpika samadhana ki peshakasha kara sakate haim.
asurakshita abadi para echaaivi tike ke parikshana mem savadhani se vichara kie jane hai
vikasashila deshom mem echaaivi nirodhaka tike ke anusandhana ke bare mem savala puchhana jaruri hai
nirodhaka tike ke parikshana mem bharti hone ke lie pratibhagiyom ka bahumata kamajora abadi se haim
kamajora abadi kai gaira nirdharita jokhima legi- echaaivi se sankramita hone ka jokhima jaivika;ajnata pahalu aura lambi avadhi ke tike ke prabhava.
hamem age parikshana svayansevakom jo nishchita rupa se hara kimata para gaira sankramita rahana chahate hai aura shodhakartaom jinhe kuchha logom ko sankramita karane ki 'ja़rurata hai' yaha sabita karane ke liye ki unake lagae tike vastava mem sanrakshita haim, ke bicha hita ke sangharsha para age charcha ki jarurata hai
eka nirodhaka tike ki naitikata para charcha logom ke echaaivi bimari ke bare mem janakari aura tika prabhavakarita ke adhara para kiya jana chahie; parikshana mem bhaga lene ki ichchha ka prabhava aura takaniki avadharanaom ki unaki samaja jaise plesabosa aura dabala netrahina paddhati para adharita ho
muddom para jaise sansadhana mem gariba setingsa ke pralobhana ka artha; sahamati prakriyaom ke bataye nyunatama manakom aura gopaniyata avashyakataom para vaham deta hona chahie
tippanikarom ne kaha hai ki bharatiya niyamaka ejensiyam kamajora haim aura medikala kaunsila doshi daॉktarom ke khilapha karravai ke liye mana kara deti hai
bharatiya medikala anusandhana parishada(aisiemaara), bharatiya echaaivi tike sangha mem pramukha bhagidara, abhi taka una logom ko saja़a nahim de paya hai jo usake naitika disha nirdesho ka ullanghana karate hai
vikasita deshom mem, deta aura suraksha nigarani borda svatantra naidanika anusandhana visheshajnom ki samitiyam hai jo deta ki samiksha karate hai jaba eka naidanika parikshana ka karya pragati para ho aura sunishchita karate hai ki pratibhagiyom anuchita jokhima mem nahim haim.
vikasita deshom ko deta ki samiksha ka adhikara ho aura sujava de sakate haim ki koi parikshana sanshodhita ho ya ho agara vaham suraksha chintaem haim, ya agara parikshana uddeshya prapta kiya gaya hai
koi sabuta nahim hai ki achara samitiyom sadasya ne tike ke naitika nihitartha prastava parikshana ka kada़ai se vishleshana kiya hai
bharata mem prastavita echaaivi tike ke parikshana ka vistrita naidanika parikshana protokaॉla ki anupasthiti mem, jokhima vivarana ke satha aura kaise mulyankana kiya gaya - samudaya paramarsha ka mulya eka sarvajanika sambandha vyayama ho kara raha gaya hai
bharata mem echaaivi ke tike ke parikshana para paramarsha 'eka shveta patra' ke adhara para jokhima aura labha ke parikshana mem bhaga lene ke aura prathamikata ke adhara para eka gahana bahasa aura nirodhaka tika banama chikitsakiya tike ke adhara kiya jana chahie
bala roga ki anuthi visheshata, anya upa visheshatao ke viparita, isaka prathamika dhyana hai aura roga ki rokathama ko samarpana hai
asvasthata aura nrityu ka pramukha karana jaise achanaka shishu nrityu sanlakshana, durghatanaom aura chotom, hinsa, motapa, aura sankramana ka behatara niyantrita nivaraka upaya ke dvara kisi bhi sambhavita sudhara mem shighra nidana ya prabandhana prayasom se adhika hai
isa sammelana mem pratuta sabhi sankramaka roga visheshajna bachchom kabhi ki apeksha se adhika svastha aura khusha haim -- kaiyo ke jora ke satha ki vikasa ki isa pragati ke lie aura prabhavi aura surakshita tike ki upalabdhata pramukha karana hai
2005 mem bala chikitsa niyamita tikakarana karyakrama mem do nae tike joड़e gaye, eka sanyugmi meningokoksala tika (menakatra, sanophi-pashchara) aura eka kali khansi bustara tika (aidasela, sanophi-pashchara aura bostrikasa, galekasosamithaklina)
eka aura sanyojana vaiksina, emaemaara/vi jada(prokyoda, meraka), hala hi mem ameriki khadya aura draga edaministreshana (ephadie) dvara manjura, khasare, kanthamala ka roga, rubela, aura chhoti chechaka dada (vi jada) ke khilapha pratirodhaka kshamata prapta karane mem kai injekshana ke prashasana ki jarurata ki madada mem hala hoga
agale 12 mahine ke bhitara shuru hone vale anya tike rotavayarasa aura papilloma vayarasa ke lakshita bimari ko rokane ke liai hai
isake atirikta, sanyojana tika ditiaipi / poliyo / hiba (pentasela, sanophi-pashchara) pahale se hi dakhila laisensa avedana ki ephadie svikriti prapta hai, ke rupa mem diaraesa dvara ki ghoshana ki. lairi pikaringa, roga niyantrana aura rokathama (sidisi) ke lie kendra aura margareta renelasa, mairilainda vishvavidyalaya ke chikitsa skula, kaॉleja parka mem hai.
hala hi mem shuru ki sanyugmi meningokoksala tika, menakatera, vartamana mem 11-54 varsha ayu varga ke vyaktiyom ke lie manjuri de di hai, lekina 2 taka ki um taka ke bachchom ka adhyayana kiya gaya hai
pratirakshana acharana para salahakara samiti ki sipharishem (aisiipi) aura apa meningokoksala tika ke sarvabhaumika prashasana ke lie 11 se 12 ki unra ki palatana ke lie haim, 14 sala ki unra mem ya uchcha vidyalaya ke pravesha para, aura kaॉleja ane vale nae logom jo dolamitari mem rahege
anya uchcha jokhima samuhom jinhe tika milana chahie sukshma jivavijnana, sthanikamari vale ya mahamari kshetrom ke lie yatri, sena ke javana, aura satha ki kami ya aisaplenia ke puraka shamila hai
dusare kama jokhima vale vyakti, yani, vyakti 16-54 varsha ayu varga ke jo meningokoksala bimari ke apane jokhima ko kama karana chahate haim, vaiksina ke lie bhi patra haim.
da. ophita ne kaha ki kama jokhima para logom ke lie tikakarana ki lagata sankramana ka mauka 125,000 mem 1 se bachane ke lie lagabhaga aura mauta ya gambhira siक़yela ka mauka 1,250,000 mem 1 se "80 ho
vartamana tika samuha se suraksha bi meningokoksala bimari pradana nahim karata hai, tanava yuva shishuom mem sabase ama hai.
achchhi khabara yaha hai ki samuha bi roga ki milate ghatana krama 43% 1990 se 1992 mem, 34% 1997 se 2003 taka ghatakara sabhi meningokoksala sankramana ke lie hai
chikinapoksa tike ko 1995 mem upayoga ke lie ephadie dvara anumodita kiya gaya tha.
chikinapoksa tika 1998 taka nahim tha ki bachchom ki 50% se adhika yaha vaiksina diya ja raha tha.
donom niji (apa) aura sarvajanika kshetra (aisiipi) ki sipharisha ke turanta bada svikriti di gai ke 12 se 18 mahinom ki unra ke sabhi bachchom ko chhutakara de
bada ke prayasom ko 'kaicha apa pratirakshana' mem nirdeshita aura varsha 2000 taka rahe, roga ki ghatanaom mem kaphi katauti ki sarahana ki thi
nae niyamita tikakarana ke prayasom ki prabhavakarita ka samarthana deta hai, lekina vaham abhi bhi chikinaposa tike ke satha anasulaje mudde hai, jaise daॉ. aini gershona, dvara samiksha [4] kolambiya vishvavidyalaya, nyuyaॉrka, nyuyaॉrka ne ki hai.
da. gershona ne kaha ki hala hi mem eka adhyayana mem roga ke khilapha 85% sanrakshana dara sanketa diti hai, sanshodhita bimari ke satha bachchom ke shesha mem manaya pratirakshita ko.
sabase mahatvapurna, data gambhira bimari se bachava mem 97% ki dara dikhai hai.
bada ke tike yuga mem, roga niyantrana ke lie kendra ki riporta se 66 varshika mauta banama 1995 se pahale 145 hara sala, jaba tika pesha kiya gaya tha.
gershona ke anusara halanki kuchha adhyayanom se pata chalata ki samaya ke satha chhoti chechaka mem saphalata [dara] mem koi vriddhi nahi hai, dusarom karate haim
kuchha dina dekhabhala kendrom aura skulom mem chikinapoksa ke prakopom ke phailane ki kuchha riportom rahe haim.
sanyukta rajya amerika mem, inyonoloज़ika anuvarti tika praptakartaom ke upara hai entibaॉdi aura sela ke hatha-madhyasthata chhoti chechaka ke pratiraksha pratikriya-5 se 10 sala ke bada tikakarana ke lie dada vayarasa sanketa dite hai.
vaiksina ke tike ki viphalata ka karana anuchita bhandarana shamila ho sakate haim.
isake atirikta, kuchha deta 15 mahine ki unra se pahale ki pratirakshana sujava ki kama surakshatmaka prabhava ke satha juda़a hua hai.
vartamana mem, sidisi viज़eda tike ki dusari khuraka ki sipharisha ki suraksha badha़ane para vichara kara raha hai.
yaha donom mamalom ki suchana saphalata hai aura kyonki 4-6 sala ki unra mem eka bustara khuraka ke lie sirama entibaॉdi vriddhi darja ki gai hai lagabhaga 6 titersa guna ke lie karanom mem shamila haim.
hala hi mem emaemaara/viज़eda sanyojana tike ko manjuri bustara khuraka ki anumati ke lie eka hi samaya emaemaara tike ki jarurata ki dusari khuraka samanya rupa se kiya jata hai para die jane ki injekshana ki kula sankhya mem vriddhi ke bina, hota hai.
dada sankramana ke punarsakriyana una mem se veksina ke bicha kama ama hai jo inyuno samajauta vyaktiyom ke bicha prakritika sankramana ki darem mem shamila tha.
da. gershona ne samajaya ki, jo pahale leukemika veksina mem manaya gaya tha, vaham bada mem dada ke lie 1 juda़a dilachaspa pahalu hai
karana hai eka kharoncha jo shighra hi vaiksina prashasana ke bada vikasita hui
lekimiya ke rogiyom ke eka samuha ke adhyayana mem, 3.6% mem dada vikasita dane the, halanki dane ke bina kevala 0.4% mem bada mem punarsakriyana roga tha.
eka naya dada tika, jisamem viज़eda vayarasa ki bachapana ke tike se eka bahuta kama matra hai, dikhaya hai ki roga aura naidanika abhivyaktiyam ki gambhirata ko kama karane ke lie hai
60 se adhika ayu ke 38,000 svayansevakom mem,jinamem se adhe ko tika mila, niyantrana ke 642 ki tulana mem 315 mem dada vikasita hua
vaham bhi darda ya bechaini mem 61% ki kami thi jo tivra sankramana ke satha tha aura bada mem herapetika nasom ke darda mem 66% ki kami thi
da. gershona ne sujava diya ki agara bachapana mem tike lagae abadi ke bicha dada ka asvikarya ghatana hai, nai dada tika 40 se 50 varsha ayu sima mem vayaskom ko diya ja sakata hai
apa lala kitaba se chalu samiti ki sipharishom ke kai dama, ekjima ke satha bachchom ko tike ke prashasana ke lie prasangika aura pratiraksha ki kami haim
ekjima ke satha bachche sabhi vartamana mem tikom ki sipharisha ko prapta kara sakate haim jaba taka ve kula matra mem 20 miligrama / dina se jyada ya 2 migra / kigra se jyada kortikosteroidasa prapta kara rahe haim agara ve kama se kama 10 kilo vajana ke hai
konatikosteroida ke kramashah matra mem sabhi vyaktiyom mem dabe huai inyuno haim, jo ama taura para raha rahe haim vayarasa tike ke prashasana ke lie eka viparita sanketa hai.
kai pratiraksha kami vale rogiyom ko tika nahim lena chahie.
apavada chayanatmaka ijiai ya aijiji sabakalasa ki kamiyom ke satha hote haim aura unake satha jinahe edsa hai, jo gambhira rupa se inyuno daba nahim haim
inyuno ke dabava mem kimotherepi para rogiyom ke lie anushansaem samasyagrasta hai kyonki vaham simita deta ke phaisale margadarshana kara rahe haim.
vyaktigata ऑnkolaॉji karyakrama aksara apane rogiyom ke lie protokaॉla ke naidanika anubhava para aura sahitya ki apani vyakhya ke adhara ki sthapata hai.
vyaktigata ऑnkolaॉji karyakrama aksara apane rogiyom ke lie protokaॉla ke naidanika anubhava para aura sahitya ki apani vyakhya ke adhara ki sthapata hai.
ama taura para tikom para kama se kama 3 mahine para roka lagai jani chahie aura kimotherepi ke bada banda kara de aba jaba upachara adhika inyuno saparesiva sarakarom ke satha kiya gaya hai
pratiraksha samaroha ki jancha ke lie tikom ke prayoga ke bare mem nirnaya vishishta istemala kiya ja sakata hai.
sabhi bala roga visheshajnom tike suraksha muddom aura pratirakshana sandeha ke satha sambandhita hai
mata pita, sabase susangata ga़latapha़hami (uttaradataom ka 25%) ka eka vyapaka sarvekshana prashnavali mem pratirakshana ke mana karane ke lie agrani raha tha ki bachchom ki pratiraksha pranali bhi kai tikom ki prapti se kamajora ho jaega
sarvekshana se pata chala ki para ke satha chintaom ko kaphi kama hai, jo sambhavana hai ki pichhale kuchha varshom mem vyapaka apa dvara shiksha ke prayasom ka parinama hai
peshevara sangathanom ke bicha pratirakshana ama sahamati ko kayama rakhata hai.
unaki chikitsa ghara para sabhi bachchom ke lie pratiraksha taka pahuncha pradana karana.
tike ka anupalana sunishchita karane ke lie sabase achchhi rananiti ke lie skula mem pravesha ya jari rakhane ke lie pratirakshana ki avashyakata hai.
sanghiya sarakara ki sipharisha ki tikom ke samanya prashasana ke lie majabuta samarthana pradana karata hai, alaga alaga rajyom mem isa samaya skula mem pravesha ke lie apane tike ke pralekhana mem badalati haim.
sabhi rajyom dipthiriya, tetanasa, poliyo, khasara ki avashyakata hai, aura rubela, lekina 6 ko chhoड़ kara sabhi rajyom mem kali khansi ki avashyakata hai aura 5 ko chhoड़ kara sabhi mem hepetaitisa bi tikakarana ki avashyakata hai.
anya vartamana mem sipharisha ki tike (kanthamala ka roga, hemophilasa inphluenja bi, chikinapoksa, niyomokokasa, menigokokasa, aura kali khansi ke bustara khuraka) kevala kuchha rajyom mem avashyaka haim.
isake viparita, amataura para chaildakaiara kendrom rajya skula sistama se apani tika avashyakataom mem adhika sakhta haim 
da. merakusa ne yaha bhi bataya ki alaga alaga rajyom mem unake pratirakshana ke lie chhuta ke bare mem bhinna kanuna hai
sabhi 50 rajyom chikitsa bahishkarana ka samarthana karate hai; 49 dharmika anumati aura 20 niji ya darshanika chhuta dete haim.
mata pita ka drishtikona jo apane bachchom ke lie tikom ko mana karate hai charcha ka eka anya mahatvapurna mudda tha
chikitsa nishchita rupa se apane rogiyom ko adarsha svasthya seva pradana karana chahate haim aura ina prayasom ki ora kisi bhi sima ko kathinai se svikara karate hai
vaham dekhabhala ke darshana mem kaphi matabheda haim.
mata pita ke nirnaya ko badalane ke lie kai prayasa kiye gaye hai
isase pahale ki eka rogi ko svasthya suvidha se chhutti di jaai, agrima notisa diya jana chahie aura pahuncha ke lie eka vikalpa para dhyana karana chahie aura dastaveja para vichara vimarsha kiya jana chahie
apa ne hamesha bala roga visheshajnom ke karyalaya se pratirakshana karyakrama ko protsahita kiya hai.
dekha gaya hai ki prathamika dekhabhala pradataom dvara prabandhita bachchom ke lie tike (viphasi) karyakrama mem rajya ya shahara prashasita svasthya klinikom se adhika saphala hai
vikasa aura sudhara ke lie anya kshetrom pratirakshana suchiya aura sthaniya pratirakshana gathabandhana.
pratyeka chikitsaka ko unake abhyasa ki sthapana ka akalana, kshamata chuka ke avasara para mukhya rupa se dhyana kendrita karana chahie aura sabhi karyalaya badhaom ko dura karana chahiai.
usa ayu mem jaba tikom ki sipharisha ki gai hai anusmaraka aura bachchom ki yada ke lie eka pranali honi chahie
pratiraksha di jani chahie, bhale hi eka samanya pariksha ke karana anusuchita lekina abhi taka nahim hai.
vaham bhi teji se traika tikakarana ke lie eka pranali kevala anurodhom aura eka hi dina tikakarana kevala daura hona chahie
narsom ke lie chikitsakom dvara pratyaksha paryavekshana ke bina bachchom ko pratiraksha adhikara diya jana chahie
sankramaka roga ke drishtikona se, chaildakaiara kendra ya skula shishuom aura chhote bachchom ke lie khataranaka ho sakata hai jaha ve samaya vyatita karate hai
roga jinaki vriddhi ki ghatana hui hai bachchom mem ghara mem suvidhaom ki kai haim: dasta, tivra otitisa midiya, nimoniya, sinusitisa, aura grasanishotha.
vishishta rogajanakom mem khoja, adhika uchcha ghatanaom ke karana chinta ki hepetaitisa e aura bi, ketomegalovayarasa (siemavi), chhoti chechaka-dada, simpleksa vayarasa, antra, hemophilasa inaphuainaज़ grupa bi, aura satrepatokokasa nimoniya harpija bimari ki vajaha se hai
mera bachcha chaildakaiara/skula kaba ja sakata hai? eka bahuta hi ama savala hai jo mata pita chikitsakom se puchhate hai.
javaba kaphi kuchha rogajanakom, jisake lie sankramaka ki avadhi ko dhyana se adhyayana kiya gaya hai ke vishaya mem spashta kara rahe haim
anya sankramanom ke lie, deta adhure haim
eka satepatokokala sankramana samuha ke satha bachche (grasanishotha ya tvacha sankramana) ko chaildakaiara/skula mem roganurodhi upachara ke shuru hone ke kama se kama 24 ghante taka nahim lautana chahie; isa samaya ke daurana parivara aura anya bachchom ke satha nikata samparka (chumbana) se bacha jana chahie.
bachche mem chikinapoksa mem ghavom ki papada़i ke satha, jo mamuli mamalom mem lagabhaga hamesha shuru hone ke bada kama se kama 6 dina hota hai.
dada ke satha bachche ko bahara rakha jane ki ja़rurata nahim hai jaba taka unake baratana shamila nahim kiye jate
bachche jinako khasara hai 4 dinom ke bada ghara para rahane ki ja़rurata nahi hai; rubela ke satha unako 7 dinom ke lie ghara mem rakha jana chahie
kanthamala ke satha bachche karnamuliya sujana ke shuru hone ke bada 9 dinom mem lauta sakate haim.
hepetaitisa a ke satha bachche bimari ki shuruata ya piliya ke bada 7 dinom ke lie ghara mem rahana chahie
kali khansi ke satha bachche ko vapasa ane se pahale roganurodhi upachara ke 5 dinom ko pura karana chahie
kai atisariya roga vayarala rogajanakom ke satha juda़e hai jaise rotavayarasa, aidinovayarasa aura nora vayarasa ke rupa mem, ya ama baiktiriyala ejenta ke satha jaise salmonela aura kenyalobaktera ke rupa mem, anya bachchom se apavarjana ki avashyakata nahim hai jaba taka mala langota mem nahim ho sakata hai
bachche antrashotha ke satha shigavisha ke karana - ekshcherichia kolai ke utpadana (i. kolai O157: H7 sahita) ya jaba taka dasta aura 2 mala sanskriti parinama nakaratmaka rahe shigela ghara para rakha jana chahie
salmonela tyaphi sankramana ke satha bachche ko vapasi se pahale 3 sanskriti nakaratmaka parinamom ki avashyakata hai
vayarala ke satha upari aura kama shvasana patha sankramana sambandhita vapasi ki sipharishem adhika samasyagrasta haim
bachche jiname sansa ka syankyatiala vayarasa aura inphluenja sankramana kabhi kabhi vayarasa 3 saptaha ke bada sabase lakshana ki roka-thama se lambe samaya ke lie nikala jata hai
tivra netrashleshmalashotha ke satha bachchom,jise sapheda ya pile ranga ki naja़ra nirvahana ke satha gulabi ya lala netrashleshmalashotha ke rupa mem paribhashita kiya hai, ko jaba taka eka klinishiana dvara manjuri na mile ghara mem rahana chahie
jo gaira tivra netrashleshmalashotha ke satha hai,pani najara nirvahana ke satha bhi sapha hai, ko dina dekhabhala ya skula se vanchita karane ki avashyakata nahim hai.
rokathama ka eka aunsa nishchita rupa se ilaja ke adha kilo ke mulya ke barabara hai aura isa kahavata kai samarthana ka vaijnanika prastutikarana apa ki baithaka ki 75vi varshagantha se behatara kabhi nahim hua
nishchita rupa se prakasha dala gaya tha nae tikom mem jinmem bachchom ke svasthya ka sudhara jari rahega lekina yaha bhi dekha kara khushi hui ki kitani achchhi taraha hamare nivaraka upaya aja kama kara rahe haim
gambhira phephada़e ki bimari se eka bara thika hone para tapedika phira se roga pakada़ne ke khilapha koi suraksha pradana nahi karata hai, daॉktarom ki khoja hai
rogi jinko tibi ke do hamalom huai hai,pahale avasara para baga ke unmulana mem basa nakama rahe haim
yaha tathya ki prakritika sankramana apako unmukti nahim deta hai jivanuom ke khilapha adhika prabhavi tike ke vikasa ko mushkila kara dega
dakshina aphrika ke shodhakartaom ne guruvara nyu inglainda jarnala ऑpha medisina mem khabara di hai ki parikshana mem loga jinaka tapedika vapasa aya paya gaya ki nae jivanu mula baiktiriya se eka alaga anuvanshika sanrachana tha
yadi prakritika sankramana surakshatmaka pratiraksha pradana nahim karata hai, tapedika ke khilapha behatara tike ka vikasa vishesha rupa se chunautipurna hoga aura tapedika ke khilapha mejabana suraksha ko behatara samajane ke lie vishesha rupa se mahatvapurna hoga.
vishva svasthya sangathana vartamana mem agale dashaka mem eka bada़e paimane para duniya bhara mem tibi hone vali mautom ki sankhya mem kaphi kami ke uddeshya se karyakrama chala raha hai
vartamana mem, 33 lakha phephada़om ke hamalom ke shikara ke roga se hai
jivanu bahuta se logom mem hota hai, lekina aksara yaha bimari mem parinamita nahim hota
yaham taka ki jinamem roga grahana karata hai, vaham aksara sankramana aura roga ke bicha eka lamba thaharava hai, aura vaijnanikom ko pata nahim aisa kyom hai
loga jinaki pratiraksha pranali achchhi echaaivi, ya samanya gariba poshana aura asvasthyakara jivana shaili ke rupa mem anya sankramanom ke karana achchhi taraha se kama nahim kara rahi haim ke lie tibi vishesha rupa se khataranaka hai
echaaivi sankramana ki dara mem uchhala ko aphrika mem tibi ke mamalom mem vriddhi ke lie jimmedara thaharaya gaya hai
tibi ke ilaja mem davaiyom ka eka lamba pathyakrama shamila hai, aura vikasashila deshom mem yaha sunishchita karana ke marija ne puri taraha se pathyakrama pura kara sunishchita karana mushkila hai
goliyam lambe samaya ke bada jaba vaha ya vaha ko achchha lage le li jani chahie, yaha sunishchita karane ke lie ki sabhi tibi ke baiktiriya ko mara dala hai
riporta mem jila aspatala, saharanapura mem 12.10.2002 aura 6.12.2002 ko subaha ke 10 baje ke bicha mem bharti kula 57 logom ki nrityu ke bukhara ke mamalom the
manaka mamale paribhasha aura uchita dastaveja ke abhava mem, mauta ke sahi karana ka akalana karana mushkila hai
ina mautom ke mamale rikaॉrda ke vishleshana ke adhara para 45.6% mautom ka insephelaitisa ki vajaha se hone ka sandeha tha
1.11.2002 se 5.12.2002 ke bicha kula 13 ensephalaitisa maute mujaphpharanagara ke jila svasthya adhikari dvara suchita kie gae
pichhale varsha ki tulana mem chalu varsha ke daurana bukhara se sambandhita ankada़om ke vishleshana se pata chala ki pravesha aura pichhale 3 varsha ki mautom mem ensephalaitisa mautom mem koi ullekhaniya vriddhi nahi hai
bimari achanaka eka shuruata hai aura teji se beshaka kama avadhi ke bhitara nrityu ke lie agrani hai.
bukhara, ulti, badala sensoreyama, samanyikrita taॉnika aura ya avamotana akshepa, aura koma, ke satha prastuta mamalom.
simita nyurolaॉjikala sanketa nahim dekhe gaye aura dane ka koi itihasa nahim dekha gaya tha
10 varsha taka ki ayu ke bachchom ki sabhi hone vali mautom ki ke bicha 60% maute sabhi hone vali mautom pancha mem sthana le liya hai
parivara, pada़osa aura svasthya karyakartaom ke bicha koi madhyamika mamale suchita nahi kie gae
mauta ke mamale mem mahamari ya ama jokhima karaka ke sambandha nahi paye gaye
aktubara 2002 ke bada se jila aspatala mem bharti karaye insephelaitisa ke satha sangata bimari ko khojane ke prayasa kiya gaye, lekina sangata bimari ka eka hi mamala mila
neisidi timom ne kula 14 gamvom, chara prathamika svasthya kendrom aura shahari saharanapura ke do bastiyom ka daura kiya
chara prathamika svasthya kendrom ke dasa prabhavita gamvom mem kie gae teja bukhara sarvekshana mem pichhale 15 dinom ki sandarbha avadhi mem 1.72% bukhara ki dara ka pata chala
maleriya parajivi ke lie rakta smiya nakaratmaka paye gaye
kita vijnani adhyayana ne jei aura dengu vaiktara aura bahuta kama ghanatva ke maleriya vaiktara ka koi sabuta nahim dikhaya
53 rakta, 16 si.esa.epha. namune aura 16 gale ke kula sandigdha mamalom aura nrityu ke mamalom ki samparkom se ekatra kie gae
saharanapura aura mujaphpharanagara jilom mem paya gaye insephelaitisa ke mamalom mem bikhare hue aura chhitaputa prakriti mem haim
naidanika tasvira aura prarambhika jancha prayogashala vayarala ensephalaitisa ke nidana samarthana karate haim.
isake alava unnata vayarasa sanskriti, pisiara aura jina anukramana adhyayana ki taraha prayogashala jancha ke rashtriya sanchari roga sansthana, dilli aura enaaivi pune mem pragati ke tahata hetukavijnani ki pahachana nishchita kara rahe haim.
roga ki janapadika roga vijnani nigarani ke prabhavi prasara aura ke bimari sabhi pahaluom ki jancha hai jo uchita niyantrana hai
kisi vishesha bimari ki nigarani vyavasthita sangraha aura rugnata aura nrityu dara mem deta ke mulyankana shamila hai, mahamariyom ki jancha ki riporta prayogashala jancha ke pata lagane ke lie prerana ka ejenta ka prayoga karate haim aura jaiva, kitanashakom aura anya niyantrana mem prayukta samagri, unmukti ke mulyankana ke prabhava apriya sthiti ki janasankhya aura anya karravai ke lie sambandhita deta dete hai.
mahamari vijnana sevaom mem prayogashala takanikom ka parichaya avadharana kranti la di hai aura satha hi roga nigarani ki gunjaisha bhi
ajakala prayogashala samarthana nigarani ke samvedanashila pranali ka eka abhinna anga mana jata hai.
prayogashalaom svasthya dekhabhala sevaom ke satha rogiyom ki pahali bada़i samparka ke bindu para sthita haim.
vikasashila deshom ki prayogashalaai prathamika svasthya kendra ya samudayika svasthya kendra mem hi upalabdha hai (prathamika svasthya kendrom ke unnayana).
prayogashalaom vyakti ke satha hi satha samudaya ke lie nivaraka aura upacharatmaka protsahaka sevaom ke lie takaniki sahayata upalabdha karati hai.
prayogashalaom ko simita buniyadi jancha karana chahie uda. maikro maleriya ke lie pratilipi aura kshaya roga, ina prayogashalaom ko loga sangraha, bhandarana aura chikitsiya namunom ki prayogashala ke lie jila temporaijeshana para dhyana dena chahie
paridhiya prayogashalaom mem eka takanishiyana aura eka prayogashala sahayaka / parichara karmachari ko shamila hona chahie.
paridhiya prayogashalaom mem upalabdha spesa kama se kama eka prayogashala evam karyalaya / abhilekha kaksha shamila karana chahie (16 phuta x10 phuta) aura eka stora-anya sevaom ke satha sanyukta kamare (16 phuta x10 phuta).
parivahana aura sanchara ki suvidha bhi ho aura rogiyom ke namune, apurti aura vyaktigata charcha ki kharida ki sipha़arisha ke lie paridhiya aura madhyavarti prayogashalaom ke bicha hona chahie
prayogashalaom ko sarvajanika svasthya ka parikshana karane ke satha hi naidanika prasangikata bhi karani chahie.
sarvajanika svasthya prasangikata ke parikshanom mem se, adhika se adhika janapadika roga vijnani mahatva ke rogom prathamikata di jani chahie
jaham taka sambhava ho, parikshana vishvasaniya, samvedanashila vishishta, tvarita, asana aura lagata prabhavi hona chahie.
rakta sabase mahatvapurna aura jila stara para bara bara ekatra hone vala naidanika namuna hai
rakta banja (adhimanatah dispojebala) sirinja aura suiyam upayoga jani chahie
rakta ki matra kama se kama 4-5 miligrama hona chahie.
shishi jisamem rakta ekatra kiya jata hai, banjara sukhi aura thika se lebala hona chahie
sui aura sirinja shishi bhi istemala ke rupa mem, rakta ikattha karane se pahale puri taraha se shushka hona chahie.
khuna nikala ne ke bada, sui sirinja se khuna ki shishi ko sthanantarita karane se pahale sirinja se hataya jana chahie.
ekatra rakta ki shishi ko mata hilao.
khuna 4-6 ghante ke lie kamare ke tapamana para rakhane ke bada, sirama ki prakriya ko alaga kie jane chahie.
yadi sirama ke vibhajana ke lie suvidhaem upalabdha nahim haim, to yaha 4 digri selsiyasa para prashitita kiya jana chahie (sthira nahim).
eka banja pashchara vinduka ka prayoga karem, bedakhala rakta ke tarala bhaga se thakka hataai; eka sapha banja apakendritra rabara kaga hone tyuba mem tarala bhaga hastantarana karem
tarala philtara pepara para lathapatha namunom ki dhulai eka sarala aura surakshita prakriya hai.
sharira ke tarala padartha ke sabhi prakara philtara pepara stripsa (vatamana nambara 1) para avashoshita kiya ja sakata hai.
shamila philtara pepara ke satha karda ke rupa mem chihnita sarkala hone stripsa (ama taura para 0.6 se 1.8 semi vyasa mem) vyavasayika rupa se upalabdha hai
rogi ka vivarana eka bola pena ke satha karda para likha hai.
rakta ungali chubhana se le ya eka shishu ki chubhana changa se.
saita 70% sharaba ke satha sapha aura banja dhundha ke satha sukhi sapha ho jata hai.
panchara eka banja prayojya nukila ke satha banaya jata hai aura rakta ki pahali bunda sukhi rui se pochha diya jata hai
philtara pepara (chihnita kshetra) dhire se khuna ke dusare chhoda़ para chhua diya jata hai aura sokhane ki anumati hai jaba taka purva nirdharita kshetra puri taraha se bhara nahim jata

khuna ka namuna hava mem 3 ghante ke lie bina kisi bhi sataha, sidhe garmi ya dhupa ke satha sidhe samparka mem aye bina, kshaitija sthiti mem sukha gaya hai
namuna eka plastika ki thaili mem rakhe ya eka liphapha mem aura bhali bhanti banda karake parivahana ya melinga se pahale taya ho
namuna ke sangraha ke bada, ise turanta mamala jancha pharma ke satha bhara jana chahie aura pratyeka namuna ke satha bhara jana chahie
namuna adhimanatah 2-8 digri si thanda rakha jana chahie aura prayogashala mem sambhavata jaldi bheja jana chahie.
deri ke mamale mem namune ko sabase pasa prayogashala mem le jane se pahale 2-8 digri si para sangrahita karana chahie
samanya mem, sankramaka samagri ka bhandarana aura parivahana ke daurana eka kama tapamana para rakha jana chahie, si.esa.epha. ke ekatra namune ko chhoda़kara phayojenika maininjaitisa jo kamare ke tapamana para rakha jana chahie.
mula naidanika namunom mem rogajanakom ya entibaॉdi ki matra ka bhandarana ya parivahana jo gambhira naidanika parinamom ko prabhavita karata hai ke daurana asvikara kara sakate haim.
isalie vishesha dhyana se pahale ya daurana samagri ki prayogashala ke lie paragamana liya jana chahie taki unhem garma ya sukhane se bachaya jae
namunom ki thanda aura doharaya vigalana se bachae.
namune ko tabhi phriज़ karem ashvasana diya ho ke parivahana -20 digri selsiyasa para hoga
2-8 digri si para bhandarana aura parivahana kare, si.esa.epha. phayojenika maininjaitisa ke sandigdha mamalom ko chhodakara
namuna mamale ki jancha se sambandhita kantenara shoshaka namuna asapasa ke samagri ke satha eka plastika ki thaili mem rakha jana chahie ki bhale hi pure namuna bahara phaila, yaha avashoshita ho jae.
prayogashala ki riporta pharma ka eka alaga plastika ki thaili mem banda kiya jana chahie aura daura lipate namuna ya namunom ke baॉksa ko majabuti se juda़a hona chahie.
samagri eka achhuta dapha़ti / kairiyara mem paika kiya jana chahie prayogashala ke lie namuna ka parivahana ke lie .
sabhi namunom sambhavita rogajanaka aura tadanusara antararashtriya stara para bayoheज़da lebala ke satha svikara kie jate haim
namune /aisoletsa samagri anya prayogashalaom mem bheja jaega aura namune ke parivahana ke lie paikinga ke sakhta disha nirdeshom ke satha bheja jana chahie
bandhyakarana sabhi sukshma jine ka pura vinasha saporesa sahita jivom para karata hai
kitanushodhana jivom ki vanaspati rupom ke vinasha jo bimari ya bhojana ki vikriti adi ka karana ho sakate hai
kitanushodhana donom puna: prayojya aura prayojya kancha ke bane padartha aura dushita ya sanskriti samagri ke satha dushita lekha prayogashala mem sarvadhika mahatvapurna hai.
prayogashala mem prapta sabhi namunom ko sambhavita rogajanaka mana jana chahie.
aisi samagri ke sabhi prayojya ko jalana aura puna: prayojya mem ane valo ko aotokleva dvara shuddha karana eka adarsha vidhi
eka nissankramaka samadhana yukta jara mem vastu tyagem.
eka dhatu ke bartana mem ya ya tre ke satha kavara mem samagri sthanantarita karem, pani dale aura 15 minata taka ubalem
sabhi vastu yukta ya sandushita prayoga samagri ka tyagem eka dhatu baॉksa ya eka balti mem karem
kancha ke bane padartha ko sabuna ke ghola yukta eka bola mem rakho aura andara brasha ke satha sapha़ karo
eka eka karake sabhi vastuo ko nala ke pani se dhoe phira sabhi kancha ki vastuo ko nala ke pani se bhare ke kantenara mem dale (koi ditarjenta ke nishana na chhode, anyatha yaha galata parinama de sakata hai)
upayoga mem aye pipettesa ko eka kancha ke jara mem tyage jisamem paryapta matra mem sodiyama hyapokloraida/paॉta. dayakrometa ka ghola ho.dhone se pahale 24-48 ghante ke lie chhode
jiva vijnana prayogashala mem visankramina ke ama tariko ko mote taura para visankramita samagri ke adhara para tina shreniyom mem vibhajita kiya ja sakata hai
upakarana jaise ki inaklaketinga lopasa aura searaiga aairna ka isa takanika se visankramina hota haim.
inokyoletinga pasha ke visankramana ke lie, kuchha sekanda ke lie lau ke nile shanku para khada़i pasha pakade aura dhire se tara ke pure garma lala hone taka upara ki tarapha badhe
pashadharaka ko teji se niche le taki pasha dharaka ke kai incha bhi thoda़a garama ho
garma hava ka visankramana sukhe kancha ke bartano ke lie jaise testa tiyuba, philaska, pipettesa, pertidishesa, kancha ki ekatrita sabhi suiya, savaba aura banda samana ke lie sabase achcha tarika hai jo bada tapamana jela sake aura jaha bhava dvara nahi pahucha ja sake 
garma hava dvara visankramana bijali ke garma ovana dvara asani se ho sakata hai
tharmosateta tapamana kabu karane ke lie laga hai
bada़i ikaiyom mem parisanchari hava ke pankhe lage hone chahie jo ovana ke vibhinna bhagom mem samana tapamana sunishchita karem
otokaleviga dabava mem nama garmi se visankramana karana hai aura yaha jivanu midiya, plastika ke samana adi ka visankramana karane ke lie istemala hota hai
bada mem otokaleviga vivarana mem varnita kiya gaya hai
eka paridhiya prayogashala mem, sukshma jancha teji se aura arthika prakalpita roga-nirnaya pradana kara sakati hai jo niyantrana aura rokathama rananiti mahatvapurna prabhava dala sake
eka kuchha ghantom ke lie sabuna yukta pani ke ghola ke bartana mem slaida rakhe.
slaido ko ya to daudate nala ke pani mem ya kuchha ghantom ke lie svachchha jala ke kai badalava mem rakhe
slaida sukhe, svachchha, eka prakara ke vriksha mukta kapada़e ka upayoga kara sapha kiya jana chahie
hamesha sapha slaida ko kinarom se sambhale taki anguli ke nishana se bachem
slaida ko kama se kama 60 minata ke lie 1-2% haypo kloraita ke samadhana mem rakhe
garama pani ke sabuna mem donom pakshom ko brasha ke satha dhokara, vishesha dekhabhala ke satha kevala kuchha slaida ko eka bara mem dhoe taki sakrecha na pade
hara slaida ko mahina tara ki jali ya rui ke satha sapha karem
taja ditarjenta samadhana mem slaida ka hastantarana karem
nala ke chalate pani ya svachchha jala ko kai bara badala kara dhoye
shuru mem, dhulai aura saphai ke bada, slaida sukhi jagaha mem ya garma hava ke kaibineta mem rakha jana chahie.
slaida ko l0 ke paiketom mem sangrahita kiya jana chahie, jo mote kagaja mem lipate ho aura chipakane vala tepa ya rabara bainda ke satha surakshita ho.
phyojenika maininjaitisa meningesa ka eka tivra jivanu sankramana hai, samanyatah naisseria meningetidisa, streptokokksa nimoniya aura hemophilasa inphluenja ki mahamari ke rupa mem hota hai.
sela ginati shveta rakta koshikaom ki samanya prakriya dvara neyubaora kaksha ke upayoga aura drava ki siemaema prati leyokosytesa ki ginati se kiya jana chahie
tapedika mahana sarvajanika svasthya ke mahatva ki eka bimari hai jo maikobaiktiriyama tapedika aura maikrobaiktiriya ki kuchha anya prajatiyam ki vajaha se hoti hai
phephada़e mem kshaya roga ka nirnaya maikroskopi dvara rogi ke thuka mem dandanu pradarshana dvara sthapita kiya ja sakata hai
adhimanatah subaha jaldi thuka ka namuna le lijie
ishtatama parinamom ke lie, lagatara 3 dinom ke namunom ka parikshana kiya jana chahie.
thode thuka ke mamale mem, 24 ghante ke sangraha ki jancha ki ja sakati hai.
nebuliज़eda aura garama khara haypartonika thuka utpadana ke lie prerita kiya ja sakata hai rogiyom mem jo thuka utpadana mem asamartha hai
thuka tanga dhakkana ke satha jivanurahita vyapaka kantenara mem ikattha kiya jana chahie
thuka namuna nyunatama deri ke satha prayogashala mem diya jana chahie.
namune ko jo uddhara nahim ho sakata ho ya jisa para turanta karravai na ho sakti ho 4-8 digri si para adhikatama 3-4 dina ke lie rakha ja sakta hai
tika huda mem ya kamare mem, chehara pe masaka pahane hue, petri disha mem thuka ke eka hisse ka hastantarana kare
eka lakada़i ki chhada़i ka upayoga kara, keseyasa, purulenta ya khuni samagri ka chhota sa hissa kate aura eka sapha slaida mem sthanantarita karem
eka hi lakada़i ki chhada़i ka upayoga se ya tara pasha se, samana rupa se eka bada़e kshetra mem phaile isa samagri ko phaila le jo slaida ka kama se kama do tihai hissa ho
plega manava jati ka eka prachina sankata hai.
plega jivanu bimari hai jo yersinia pestisa ke karana hoti hai.
adami mem plega, mukhya rupa se tina prakara, bubonika, pneumonika aura paseptikemika se hota hai
plega ke prakalpita roga nirikshana sukshma pariksha dvara sthapita kiya ja sakata hai.
bubonika plega mem bubo esapireta jama karana chahie
sabhi suraksha upaya ke tahata bubo ko jivanurahita chamada़e ka sirinja pankachara kara aura eksaudeta nikala kara bubo esapireta nikalo
l0 mili ya 20 mililitara sirinja, l8/l9 geja sui aura jivanurahita namakina ke kuchha mili sirinja mem taiyara ke satha sashastra, bubo esapireta ki akanksha ke lie istemala kiya jana chahie.
bubo ko pankachara kara aura chushana lagu kiya jata hai.
yadi akanksha ka utpadana drava nahim karata hai, to namakina bubo mem injekata kara phira nikale
phephada़e mem plega ke thuka ke namune ko jivanurahita chauda़e munha pencha kantenara mem rakhe
eksudeta/thuka ko savadhani barate huai eyarosaulja़ ko pharma ke eka hi bhaga ke tina smiyarom mem banaem
daga smiyarom ya to nila mithena/grama dhundhala/veyasana daga dvara hai
grama daga eka niyamita prayogashala ko bekateriologika ejenta shamila sandigdha namunom ki jancha ke lie istemala ki prakriya hai.
pratyaksha namunom aura sanskritiyom ka sukshma parikshana eka teji se prakalpita nidana pradana kara sakate haim.
grama parinama prakoshtha ke, akara (kokki, besili), sela vyavastha ke prakara (ekala, shrrinkhalita, kalastarda) prakasha maikroskopi ke tahata dekhe daga, kya etolojika ejenta ho sakata hai ki eka tvarita mulyankana pradana kara sakate haim.
grama daga sukshma jivanu vijnana ki adharashila hai.
grama daga l00 sala pahale hansa isai grama dvara bataya gaya tha.
kristala baingani (kirata vayaleta) prathamika daga ki kuchha baiktiriyala koshikaom ke sela divarom mem peptidogalyakena hoga.
ayodina eka chubhata ko dai thika kara ke rupa mem joda़ata hai.
yadi koshika divara mem peptidogalyakena shamila nahim hai to kristala baingani asani se anla ya sharaba (ranga bina) se dhoya ja sakata hai
eka madhyamika dai, sepharenina (daga hatane ke lie), ranga hatane ke bada ka kadama hai
yadi prathamika ranga badhya nahim ho, koshikaai asani se sepharenina sokha legi
grama, sakaratmaka koshikaom baingani ranga ke haim jabaki grama nakaratmaka koshikaom gulabi/lala.
vai pesatisa ke rupa mem prakata, vasa, kama grama nakaratmaka 1i ke bare mem kokkobakilli 0.5i dvara.
grama daga amataura para bibo respayaretrasa, tilli, jigara aura thuka smiyarom ke kalachara/sabakalachara para hota haim.
jaba garama daga samagri chhote kokkoida grama-nakaratmaka besili, samagri ka pata chalata hai aura alaga kalachara aura pahachana ke lie chahie
vai pesatisa ke rupa mem prakata, vasa, kama grama nakaratmaka 1i ke bare mem kokkobakilli 0.5i dvara.
veyasana daga upayoga ke rupa mem upasthi yerasinia aura pesteorella eka sambhavita parikshana ke lie eka aneka rangom ka antara daga hai
buniyadi phusachina aura veyasana daga mem samanya phusachina aura nila mithayalina baiktiriyala koshikaom jo dvidhruvi, banda suraksha pina ke akara koshikaom ke rupa mem prakasha maikroskopi ke tahata pradarshita se bandhe haim 
aneka rangom ka antara jivom ke kai alaga alaga prakara ke satha dekha ja sakata hai, isalie veyasana daga akele vya.pesatisa nidana ke lie nahim hai
jaba daga samagri eka visheshata suraksha pina ho, vayasana sakaratmaka daga hai.
yadi vayasana dvi-dhruviya jnata jivom suraksha pina hai dhundhala ho jana dekhane ke bada nahim kiya ja sakata hai, jancha abhikarmakom aura sambhava hai takaniki samasyaom ke lie jancha karem.
doharaem jaba taka daga visheshata kai parinama niyantrana kalachara ke satha prapta kara rahe haim.
testa veyasana ke eka bahuta daga yerasinia/pasteyaurella nama istemala karate haim. (sakaratmaka niyantrana) kolai ya anya nakaratmaka niyantrana ke rupa mem baiktiriya antom ke satha hai.
jaba utaka smiyarom ki jancha, niyantrana slaida sankramita aura asankramita besili utaka smiyarom plega ke satha taiyara ki bhi jancha ki jani chahie.
visheshata nile dvidhruvi dhundhala ho jana dikha besili; sujava ye.pesatisa jivom ki upasthiti.
maleriya eka parajivi plajmodiyama prajatiyom ki vajaha se bimari hai
bharata mem maleriya samanyatah pi. vaivaiksa aura pi. phalsiperama ke karana hota hai.
prayogashala jancha rogi ke paridhiya rakta philma mem parajivi ke vibhinna charanom ke pradarshana para adharita hai.
2-3 ghante ke daurana tapamana ke shikhara ke bada khuna le lijie.
maleriya ke rakta namune ke baem hatha ki anamika ki noka se ekatra ki jani chahie.
ungali pe komala dabava lagu karem aura rakta ka eka chhota para slaida ke bicha mem chhoda़ kara ikattha karem.
age aura adhika khuna eksapresa aura patali philma ke lie irada chhoda़ se 1 semi ke bare mem slaida para 2 ya 3 bada़i bundem lene ka dabava lagu karem.
shesha khuna rui ke satha ungali se sapha kara lem.
phailanevala ke kone ka prayoga, jaldi khuna ki bundom mem shamila hone ke bada़e aura unhem eka moti philma banane ke lie phaila de.
khuna nahim hona chahie jarurata se jyada hada़kampa macha gaya, lekina eka golakara ya ayatakara rupa mem 3-6 se phaila sakata hai
moti philma ke lie eka phlaita stara makkhiyam, dhula aura atyadhika garmi se sanrakshita sthiti mem sukhe ki anumati dem
eka pena ya markara pensila ke satha sukhi philma patali philma rogi ke nama, ya sankhya aura tarikha ka mota hissa bhara mem likhe
bola pena ka istemala slaida lebala ke lie nahim karem
sukhi slaida ko sapha kagaja mem lapetem aura jitani jaldi ho sake prayogashala ke lie rogi ke rikaॉrda pharma ke satha bheje
maleriya ke mamale se eka gilasa slaida para moti aura patali smiyara banao.
eka gilasa 5 minata ke lie asuta jala se yukta jara mem eka philma ko urdhvadhara sthiti mem rakhakara moti smiyara se dehaemoglobinaeja kare
parajivi ke ninnalikhita charanom eka paridhiya rakta ki patali smiyara mem paya ja sakata hai.
phaileriya limaphatikasa ki eka bimari ka mukhya karana nematoda vuchereria benakraphatati hai aura kabhi brujia malayi dvara hota hai.
prayogashala jancha ke mamalom ki paridhiya rakta mem parajivi ke kida़e ke bachche ka charana ke pradarshana ke adhara para / vahaka hai.
rakta adhimanatah rata l0 baje aura subaha 2 ke bicha hona chahie vishesha rupa se jaham sukshma phaileriya rata avadhi dikhati hai
dipthiriya eka bimari hai chhada़ ke akara grama sakaratmaka jivanu koryanebakateriyama ke karana hoti hai
prayogashala jancha (sambhavita) chhada़ ke akara ke jivanu eka chini shabda chinha ke pradarshana para (koniiphoma vyavastha) aura metakrometika grevela ki gale mem savaba ki sandigdha dipthiriya ke mamale pradarshita ki upasthiti batata hai
dipthiriya ke sandigdha mamale se sakhti se gale ke pichhe ragada kara donom tonasila aura kisi bhi kshetra mem sujana aura jilli gathana ko asankramita rui ko dandi para laga kara gale ka savaba ikattha karem
eka svachchha, sundara slaida lo, eka hira pensila se slaida ka sidhe anta kone para istemala kara lebala karo.
ekatra gale ke savaba ka upyoga kara ghumanevali shaili mem slaida ke kendra mem eka dhabba taiyara karem, slaida ka kama se kama 2/3 bhaga shamila karate hue
adarsha pratirakshavijnani parikshana, jo teji se karane vala parikshana hai aura jila prayogashala mem asani se ho sakata hai, leteksa samuhana (laa) ka parikshana hai.
hepetaitisa bi vayarala sankramana ka roga niradharana bahuta mahatvapurna hai, na kevala vayarala haipetaitisa aura livara sirosisa rogiyom ke mamale mem, balki data rakta ke namunom ki jancha mem bhi, yaha sunishchita karane ke lie ki surakshita rakta adhana hua hai ya hepetaitisa bi sankramana ke asurakshita raktadhana ke madhyama se prasara ki rokathama ke lie
eka alaga samuhana hota hai, jaba sirama echabieji yukta namuna leteksa ke satha mishrita kanom ko shuddha aura uchcha pratikriyashila virodhi ke satha entibaॉdi echabieji lepita hota hai; koi samuhana nahi hoga jaba sirama namune mem echabieji shamila nahim ho
marija / rakta data se lie sirama para parikshana kiya jata hai
nishkriya parikshana ko garma ya sira namune ka niyantrana mata karo
agara jancha mem deri ho, to parikshana ke sirama namunom ko rephrijaretara ya dipa phrijara mem rakhe, yaha dhyana rakhe ki namune ko bara bara thanda aura vigalana na kiya jaai
abhikarmakom ko kamare ke tapamana para rakhe, aura dhire se shishiyam hila kara sunishchita lem ki leteksa abhikarmaka puri taraha se nilambana mem hai
abhikarmakom ko sankramana se bachane ke lie, sunishchita kara le ki shishi ki topi thika hai aura turanta hi shishi pe laga di gai hai
kaipa aura dopara ka balana sandushana aura galata parinama de sakata hai
anuchita mishrana aura epaliketara chhadi ka vinimaya galata parinama deta hai
slaida ko ज़ora se hilane para kha़raba samuhana ho sakata hai
gunavatta niyantrana ke upayom ke rupa mem, sakaratmaka aura nakaratmaka niyantrana satha satha lagaya jana chahie
galata nakaratmaka parinama echabiesaaiji ki bahuta adhika matra (proज़ona prabhava) ke namunom mem paya ja sakata hai.
aise mamalom mem vishishta sindroma (gambhira piliya, jipiti/jioti padonnati) spashta ho jaega.
usa mamale mem namuna 1:40 kama karane ke bada parikshana samanya namaka ke satha doharaem.
vidiaraela slaida phalokkuleshana uchcha samvedanashilata aura vishishtata ke satha ka parikshana hai aura pratikriyashila sira namunom ki teji se aura satika matratmaka taitreta karana ke lie istemala kiya ja sakata hai
vidiaraela pratijana kana, jo khurdabina ke niche chhote phusiphoma suiyom ke rupa mem dekha jata hai, kalempa mem phlokyoleta (chhote, madhyama aura bada़e), ve jaba pratikriyashila (pojitiva) sirama ke satha samparka mem a jae
vidiaraela pratijana kardiolipina, lesitina aura nishchita anupata mem kolestraॉla ka eka mishrana ke hote hai aura vyavasayika taura para upalabdha hai.
pratyeka banda glasa mem emapule (suvidhajanaka vapasi ke lie paryapta atirikta ke satha) 0.5 miligrama shamila haim.
emoplesa pratijana thande, kale sthana mem sangrahita kiya jana chahie. emoplesa mem gada dikhe to nikala diya jana chahie.
10 emapaulesa pratyeka sankula ke satha diya jata hai jisamem 5 mililitara pratyeka vidiarala pratijana ho
bapharda khara parikshana ke lie pratijana payasana taiyara karane ke lie avashyaka hai.
matratmaka parikshana sabhi sakaratmaka pratikriya (pradarshana) sirama namunom aura namune jo kamajora (dablyu) ya kisi na kisi gunatmaka parikshanom mem pratikriya dikhane para hai.
hara sirama daylushana namune jo tyara kiye gaai hai, alaga namune ke satha gunatmaka parikshana kie gae hai
parinama padha़ne se pahale maikroskopa ke antargata vargikrita haim.
raipida plajma rigena (arapiara) parikshana sephilitika rogiyom mem kardiyolipina ke khilapha entibaॉdi ke gathana ka pata lagata hai
siphiletika rogiyom ke khuna mem gathita regena pratijana ke phlokkuleshana ke karana hai, jo koyala kanom ke satha saha egalutenetasa hote hai aura chhote kale klalpasa dete hai jo khurdabina ke bina asani se dikhai de rahe haim
arapiara pratijana aura niyantrana sira sthira aura sakriya rahega, samapti lebala para mudrita ki tarikha taka, basharte ve 2-8 digri selsiyasa ke bicha eka phrija mem jama ho jati hai
rakta ke namune upavasa rogiyom se ekatra ki jani chahie, kyonki bahuta lepemika namune juthi sakaratmaka pratikriya de sakate haim.
namune ke sabase jyada kamajora pada़ne para, nishchita sakaratmaka pratikriya deta hai, namuna ke titare ke rupa mem mana jata hai.
agara titare 1:32 se adhika ho to, usa jagaha se pare dabala dayalushana ke satha jari rakha, jaba taka titare prapta ho
vidala parikshana salmonela thyephi aura salmonela perathyephi ke khilapha entibaॉdi ka pata lagane ke lie eka samuhana parikshana hai jo ama antom ke bukhara ka karana hai
jaba sirama namuna esa thyephi aura esa perathyephi e, bi ke khilapha entibaॉdi sambandhita pratijanom ke satha mishrita hote ho to samuhana hoga
yadyapi vibhinna prakara ke rupa mem nirdharita manaka sambhava nahi hai, ko dekhane ke sarvajanika svasthya ki jancha ki bata se yaha ama taura para yaha kahana ki koi mala sankramana hua hai paryapta hai
koliphoma pani mem maujuda baiktiriya hanikaraka nahim hai, lekina ve sanketa dete hai ki pani mala se dushita hai, isalie adhika khataranaka jivom ke satha sankramana ho sakata hai.
manava mula ke koliphoma besili mala pradushana ka sabase vishvasaniya sanketaka haim.
jamina gilasa satoparsa 230 mililitara ki kshamata ke presterileज़ida botalom mem pani le lijie jo rima para lataka rahi ho
autokalevinga dvara botalem ko asankramita kare. vaikalpika rupa se, ऑto kaleva eka tanga skru ke satha plastika ki botala dhakkana ke satha istemala kiya ja sakata hai.
jaba pani nala se liya jaai, nala ka munha kholo aura botala bharane pahale pani ko pancha minata ke lie dauda़ne do
usake munha se botala dalem data, pani ki sataha ke niche eka paira aura pani se bharane ke satha banda kara diya
botala ko sataha para le ao aura data ko badala do
sataha ke pani ka sangraha se bachem kyonki isame karbanika padartha hai.
pani ke sangraha ke daurana, namune ko sankramana se bachaai.
namunom ke sangraha ke bada pani jitani jaldi ho sake testa karem
yadi 3 se adhika ghante ki deri ki sambhavana ho, prayogashala bhejane ke liea pani ka namuna barpha mem paika karem
jaba klorinetida pani ke namune ho, to asankramana se pahale namune ki botala mem sodiyama thiyoslapheta ki matra joda़em.
mishrana karane ke lie pani ke namune ko 25 bara ulatem
mishrana karane ke lie pani ke namune ko 25 bara ulatem
l8-24 ghante ke lie 37 digri si para sabhi tyuba sene.
saba esida aura gaisa dikha tyuba prakalpita sakaratmaka mana jata hai.
mekardi sankhyikiya talika se koliphoma namune ke 100 mililitara mem maujuda jivom ki sambhavita sankhya ki ganana ka prayoga kiya ja sakata hai.
tyuba prakalpita ka koliphoma ginati, sabakalchara mem nae sire se eka shakti hai mekkonkeya shoraba ya anda sena adhikansha sambhavita sankhya (emapiena) namune ki sakhya/100 mili, mem anla aura gaisa ke eka 50 mililitara ki jancha ka eka seta, pancha 10 mililitara ke lie dikha raha hai, aura pancha se 1 mililitara matra.
donom esida aura gaisa dikhane vali tyubem mala koliphoma ke lie sakaratmaka rupa mem liya jana chahie.
mekardi sarani ka prayoga prakalpita parikshana ke rupa mem mala ka koliphorma ki sankhya ki ganana.
eka bhi mala koliphorma paya gaya jala manava upabhoga ke lie ayogya hai.
hala ke varshom mem eka, sarala vishvasaniya aura asana (aprashikshita karmiyom dvara bhi), 'ham- nahim', pani ki gunavatta jivanu ke parikshana karane ke lie taiyara hai.
echa2esa-patti parikshana hai, jo kshetra mulyankana manakikarana aura gunavatta ke antargata vartamana mem hai paridhiya svasthya karyakartaom aura samudaya pratibhagiyom ke hatha mem jala ki gunavatta ki nigarani ke lie philda parikshana ke rupa mem parita hone ki ummida hai.
pine ke pani mem koliphorma jivanuom ki upasthiti haidrojana salphaida (echa2esa) ke satha juda़e - jiva bana raha hai, aura pani ke mala pradushana echa2esa utpadana ka pradarshana dvara sthapita kiya ja sakata hai.
yaha vibhinna karyakartaom dvara dava kiya gaya hai, ki echa2esa -patti vidhi paramparika emapiena parikshana ke satha 80% samajaute se upara varnita hai.
pura sistama asankramita hai aura pani ke parikshana ke samaya khole jane ki jarurata hai
370C para sete haim ya parivesha (30-370C) ke tapamana para khada़e anumati, koi mashina kshetra ki shartom ke tahata avashyaka hai, ke rupa mem ki botalom ki jeba aura sharira ke tapamana mem ayojita kiya ja sakata hai ka upayoga kiya ja sakata hai.
agara botala ki samagri kale ho to mala pradushana ka sanketa hai.
haija sarala aura prachura jaliya atisara ke achanaka shuru hone ki visheshata hai.
balagama aura mithai gada़bada़ gandha ke guchchhe ke satha pani dasta haija ke lakshana haim
haija vibriyo haija ke nama se jiva ke karana hota hai jo kara rahe haim grama nakaratmaka amataura para eka dhruviya phegeluma dvara koma aura motaila ke akara se ghumavadara hai.
prayogashala nidana vibriyo haija ke pradarshana para mala namune pe adharita hai.
bistara se ekatra nahim hona chahie taki paina ke lie bahara baiktiriya ya nissankramaka bistara paina sapha istemala se hastakshepa se bachane ke sakata hai.
rogi ko chauda़a munha kantenara uda mem mala shunya ke nirdesha die ja sakata hai aisakrima kapa asankramita pencha topi botala mem mala aura 3-5 grama hastantarana.
jaba bhi mala ikattha sambhava nahim ho, eka guda savaba ke namuna ekatra karem
yadi upalabdha ho, asakammita samanya namaka mem savaba ko gila kare
guda dabanevala yantra ke madhyama se malashaya mem 4 semi gaharai se 2 jada़u parichaya aura 90 digri se bari bari se aura jada़u vapasa le lem.
stora satoparasada kantenara ya parivahana ke madhyama uda mem jada़u se sapha़ karana kere-bleyara, taki sukhane se bache
jalda se jalda prayogashala mem bheje aura deri ki sthiti mem keri-bleyara madhyama ka upayoga karem aura pasa ki prayogashala mem bheja
keri-bleyara parivahana madhyama eka ardha thosa parivahana ama taura para chhote tuma botalom mem saplai madhyama hai.
keri-bleyara parivahana madhyama hava rahita tanga kantenara mem sangrahita kiya jana chahie taki sukhe na.
eka/do guda savaba vahi rogi se madhyama mem liya ho taki pure savaba madhyama mem duba ho
chhada़iyom ke atirikta bhaga se toda़o aura pencha topi ko badalo.
namune ke prayogashala mem ane ke bada, prasanskarana se pahale ise rajistara mem darja karana chahiai aura prayogashala koda sankhya diya jana chahie
jivanuom ki vriddhi ko lete huai lataka elakalina pepatona bunda ya sidhe mala nilambana banao
eka sukshma prayogashala mem byo surakasha bahuta avashyaka hai aura mula rupa se tina ghataka para nirbhara karati hai: prayogashala dijaina, sanchalana aura upakarana, chayana aura avashyaka byo surakasha upakarana aura surakshita prayogashala prakriyaom ke prayoga ke buniyadi stara
prayogashala mem khane, pine, dhunrapana se aura khadya samagriyom ke bhandarana se bachem.
karya kshetra ko kama se kama dina mem eka bara shuddha kare aura sambhavita sankramaka samagri ke phelane ke bada adhika bara
sankramaka samagri prahastana ke bada apane hathom ko dhoai
prayogashala kota/gauna prayogashala mem pahano aura inako bahara nahim le jana chahie.
una sabhi prakriyaom ke lie jo khuna ya sankramaka samagri ke satha durghatana, sidhe samparka ke lie shamila kara sakate haim dastane ka upayoga karem.
nipatana se pahale sabhi tarala ya thosa kachare shuddha kare.
eyarosola gathana ko kama karane vali sabhi takaniki prakriyaom ka isatemala kare
prayogashala suraksha prakriyaom mem karmachariyom ko paryapta prashikshana pradana karem
jaham taka sambhava ho sakriya rupa se bimariyom ke khilapha karyakartaom ko chhutakara dilae, ejenta, jo ki unake dvara niyantrita kiya jata hai.
kevala chikitsa phita karmachariyom ko hi naidanika prayogashalaom mem kama para rakhe
durghatana aura bimari ki janakari turanta sambandhita adhikariyom ko de
prayogashala prakriyaom ke surakshita chalana ke lie paryapta jagaha aura roshani pradana karem.
asani se sapha hone vali divare banae, chhata aura pharsha jo tarala padartha aura rasayanom aura disainaphektenata ke lie pratirodhi ko abhedya jana chahie
eka bharosemanda aura achchhi gunavatta ki jala apurti sunishchita karem.
upayukta susajjita 'prathamika sahayata' kamara asani se sulabha banao.
khushahala jivana ke liye, suvasta evam bimari mukta rahana jaruri hai
apane dekha hoga ki loga bimari se grasta hai aura samaja hoga kaise bimari eka vyakti ko kamaज़ora kara deti hai
roga sharira mem sankramana ka parinama ho sakata hai jaise taiphaida ya chikinapoksa, ya ve sharira ke phijiyolaॉji se sambandhita ho sakata hai jaise uchcha raktachapa aura madhumeha
kuchha avashyaka padartho ki kami ke roga haim jaise rata ka andhapana jo vitamina e ki kami ki vajaha se hota hai
roga ki sthiti svasthya ki sthiti ke viparita hai
apane dhyana diya hoga ki jaba eka vyakti ko eka bimari hoti hai vaham kuchha sanketa aura lakshana hote hai
pratyeka roga se lakshana jude hote hai aura vishesha lakshana jo ki bimari ke lie haim
jaba apake mitra ko chikinapoksa hua, chhote dane usake sharira, chehare aura gardana para vikasita ho gaye
maleriya, netrashleshmalashotha (eka netra roga), taiphaida, chikinapoksa kuchha ama roga haim
apako pata hai ki agara parivara mem kisi eka thanda laga jaya, yaha dusarom ko nahi chhodti hai
halanki apake dada uchcha raktachapa se pida़ita hai, kisi aura ko yaha nahi hota
ama sardi vyakti se vyakti mem phailati hai jabaki uchcha raktachapa nahim phailata
sanchari roga sankramaka rogom ke ke rupa mem jane jate hai
sanchari roga eka vyakti se dusare vyakti mem phailate hai, matalaba, eka svastha vyakti eka marija se yaha pakada़ sakata hai
eka roga sanchari kahalata hai agara yaha eka vyakti se dusare vyakti mem phailata hai
eka roga nisanchari kahalata hai agara yaha eka vyakti se dusare vyakti mem nahi phailata hai
eka sanchari roga ya to hava ya chhune se phailata hai.
sanchari rogom ke prasara ke do tarike haim lekina vaham aura bhi haim.
eka roga bhojana aura pani ki bhi phailata hai.
sanchari roga hava, pani, bhojana aura samparka ke madhyama se phailata hai.
inphlyuenja, poliyo, taiphaida, khasara, kanthamala ka roga, chikinapoksa, tapedika, edsa kuchha sanchari roga haim.
ina dinom hama tapedika (tibi) ya haija jaise sankramaka rogom ke bare mem kama sunate hai
dusari ora, bahuta bara hama rogom ke bare mem sunate hai jaise madhumeha , uchcha raktachapa, motapa, hridaya roga, adi
jivana shaili jine ka tarika hai jisaka hama palana karete - kaise hama kama karate hai, kya aura kaba hama khate hai, kaise aura kaba hama sote hai, adi
adhika se adhika yuvaom ko dhunrapana karate aura pite dekha jata hai halanki ye svasthya ke lie hanikaraka hai
hamara paushtika, santulita bhojana phasta phuda aura janka phuda ko jagaha de raha hai, sabjiya panira aura makkhana ke paksha mem asvikara ki ja rahi hai, shitala peya dudha ki jagaha le rahe haim
hama chalane ki bajaya eka gada़i ya basa ka upayoga karana pasanda karate haim
adhika se adhika mashinom pratyeka dina hamare kama mem hamari madada karane ke lie vikasita ki ja rahi hai.
logom ko svastha rahane ki disha mem unaki adatom ko badalane ki jarurata hai. 
hama jivana shaili ki bimariyom ko rokane ke lie bahuta kuchha kara sakate haim. 
yoga ya dhyana ka abhyasa jivana mem tanava se bachane ke lie kare.
hamara eka bahuta hi svastha jivana ho sakata hai agara hama apane jivana shaili ke prati sacheta rahe
yaha sambhava hai ki hama roga ko niyantrana mem rakha sakate hai, agara hama apani jivana shaili mem samajadara parivartana kare
sanchari roga bahuta chhote jivo ke dvara hota hai jinahe kitanuom aura parajivi bulaya jata haim.
ye roganu hara jagaha maujuda haim, hava, pani, mitti, adi mem
jaba roganu eka svastha sharira mem pravishta karate hai, sankramita bhojana aura pani, hava, samparka ya kida़e,ke madhyama se, ve guna hote hai aura sharira ki samanya kamakaja ko pareshana karate hai, isa prakara, roga ke lakshanom ka nirmana karate hai
tunhem pata hai ki upara ullekha kiya hai ki roganu vikasa ke lie kuchha samaya lete hai aura sharira mem roga utpanna karate hai
kitanuom ke mem sharira pravesha aura bimari ke lakshana ke antarala ko ushmayana avadhi kaha jata hai
jo taiphaida se pida़ita hai, 2-3 saptaha pahale, tayapha़ayada roganu se sankramita bhojana khaya hoga
vibhinna rogom ke lie ushmayana avadhi alaga hoti hai
tunhem pata hai ki jaba eka vyakti kitanuom ke samparka mem ata hai, use roga ho jata hai.
apako jana kara hairani hogi ki roganu ka asana samaya nahim hota
kuchha logom kitanuom ke samparka mem ane para bhi bimara nahim hote hai.
apake sharira ki raksha pranali apaki roganu se raksha karati hai.
dusare shabdom mem apa vayarala bukhara se rogakshamata hai.
apake sharira ki raksha pranali sapheda rakta koshikaom ka bana hota hai jo sainikom ke rupa mem kitanuom se lada़te hai
sapheda rakta koshikaai kitanuom se lada़ne ke lie eka padartha paida karate haim jise pratipinda bulaya jata hai
pratipindom aura kitanuom ke bicha ki lada़ai mem, agara pratipindom ka virodha karane ke lie aura phira isa bimari ko roka jata hai aura usa vyakti ne kaha hai kitanuom ko nashta kara pa rahe haim
pratipindom vishesha rasayanom jo khuna mem pae jate haim aura kitanuom ke khilapha karravai karate hai
roga ke lakshana dikhai dete haim, agara kitanuom ke entibaॉdi para prabandhana karane ke lie 
dusare shabdom mem, vyakti ko isa roga se chhutakara nahim hai
rogakshamata sharira ki kshamata eka vishesha bimari ka virodha karane ke lie hai
prakritika rogakshamata janma ke samaya maujuda hai aura vishesha rogom ki ora hi hai.
svastha, svachchha paryavarana aura achchhi adatem bhi tumako bimariyom se lada़ne ke lie kshamata dete hai.
jaba apa kisi bimari se grasta hotala hai, taba roganu sharira mem maujuda hote haim
to pratipindom jo rakta mem raha vikasita hote hai bimari se ubarane ke bada aura bimari ke bhavishya ke hamale ke khilapha lada़ne mem madada karate haim
rogakshamata vayarala bukhara ya jivana mem jaba taka khasara ya chhoti chechaka ke mamale mem jaise eka chhoti avadhi ke lie kiya ja sakata hai hasila ho gai hai.
jaba kamajora roganu apake sharira mem tikakarana ke madhyama se pravesha karate hai, entibaॉdi rakta mem eka vishesha bimari ke hamale ko rokane ke lie vikasita hoti hai
rogakshamata adhigrahana ka utpadana karane ke lie, hamare pasa rashtriya karyakrama hai jise sarvabhaumika tikakarana karyakrama bulaya jata hai
vyapaka tikakarana karyakrama bachchom ke svastha rahane ke lie jaruri hai.
kshaya roga, galaghontu, kali khansi, tetanasa, khasara, poliyo kuchha isa taraha ke rogom haim.
apane akasara apani mam ko bhojana aura pani ko kavara rakhane ke lie suna hoga
khula khana aura pani kitanuom aura roga se sankramita ho jane ki sambhavana hai.
asvasthyakara paristhitiyom se taiyari dvara khadya aura pani dushita ho sakata hai
pine ka pani eka surakshita srota se liya jana chahie.
nala se jo jala hame milata pine ke lie surakshita hai.
nala ka pani kuchha tarikom se sapha kiya jata hai jo kitanuom ko marata hai isase pahale ki hamare gharom ke lie bheja hai.
kuom, talabom, nadiyom aura hatha ke pampom se milane vala jala amataura para asurakshita isame roga karane vale roganu sakate hai
haipetaitisa, haija, taiphaida, adi jaise roga bhojana aura pani se phailate hai 
kya apa bata sakate haim kya hota hai jaba eka roga se pida़ita vyakti inphluenja, diphthiriya jaise, khasara, kanthamala ka roga, adi khansane ya jora se bolata ya chhikata hai?
pida़ita vyakti hava mem roga ke kitanuom ko chhodata hai jo ki apake sharira mem pravesha karete hai jaba tuma andara sansa lete ho
bhida़ bhare sthanom ki hava, bura samvatita gharom, sinema gharom adi ki adhika sambhavana hai ki yaham bimari phelate vale roganu hai
sanchari roga samparka ke madhyama se phaila sakata hai
sidha samparka matalaba apa vastava mem eka vyakti ko chhu rahe hai jo bimari hai
jabaki tuma eka rogi ke satha apratyaksha samparka karate ho jaba apa kuchha bhi jaise kanghi, tauliya, kapa, adi upayoga karate ho
mano apaka bhai inphluenja se pida़ita hai
jaba apaka bhai khansata hai vaha apane hatha se usaka munha kavara karata hai aura phira apane dosta ke satha hatha milata hai.
roga roganu sidhe samparka ke madhyama se sidha samparka ke madhyama se apake bhai se usake dosta mem parita ho gaye
kai bimariyom kida़e ke madhyama se phailati hai
makkhiyom apane sharira para kachare se roganu uthati hai aura jisa khane para ve baithati hai vaha sankramita ho jata hai
makkhiyom haija jaisi bimariyom ka karana hai aura machchhara karana hai maleriya ka
nirjalikarana khataranaka ho sakata hai, to adhika tarala padartha ka sevana de, orala rihaidreshana saॉlta (oaraesa) de.
rogi ko bistara mem rakhem jaba taka use bukhara hai aura jaba taka usaki bhukha samanya hoti hai
davaom ke satha bukhara ko niyantrana kare aura bhapa ke satha khanse aura sansa le
bharapura karbohaidreta ahara ki haipetaitasa rogi ke lie sipharisha hai
krama se honevali sarda aura pasina taiphaida ka eka lakshana hai.
bhida़ bhare sthanom ki hava mem roga ke roganu ho sakate hai
roga apratyaksha samparka ke madhyama se phaila sakata hai jaba tuma marija ka upayoga karane vali vastu upayoga karate ho
apa mem se adhikansha edsa jo sindroma ekvayarda inyuno kami kahalata hai ke bare mem suna hoga
edsa eka gambhira bimari hai jo pichhale kuchha dashakom ke daurana dikhi hai
edsa eka vayarasa ke karana hota hai jo sharira ki prakritika suraksha pranali para hamala karata hai
roganu sharira para hamala karega aura vyakti roga pakada़ega yadi sharira mem koi raksha pranali nahi hai
edsa vayarasa sankramita vyakti ke satha yauna samparka se phailata hai, sankramita sharira dravyom ka vinimaya jaise raktadhana
edsa vayarasa se phailata hai sankramita sui ke prayoga se ya janma para sankramita mam se bachche ko garbhavastha ke daurana ya janma ke samaya 
tunhem yada rakhana chahie ki edsa ka prasara hatha milane, galom para chumbana, usi taॉyaleta sita ka upayoga karake aura sankramita vyakti ke pasa khada़e hone se nahi hota
tunhem yada rakhana chahie ki edsa hava ke madhyama se, phaila nahim sakata jaise kapa ke madhyama se chhinkane se, khansane se, chashme se, thaliyo se, jala ya bhojana se.
abhi taka edsa ke lie koi ilaja nahim hai, aura na hi logom ki raksha ke lie koi tika hai.
thika se parikshata rakta ka agraha kare jaba rakta adhana ki jarurata ho
agara edsa se sankramita ho, to garbhavastha se bachem
edsa se bachane ke lie hamesha injekshana mem staralaija sui ka upayoga karem.
rashtriya sanchari roga sansthana (rashtriya sanchari roga sansthana) ko 30 julai, 1963 ko sthapita kiya gaya tha, maleriya instityuta ऑpha indiya ko vistara karane ke lie aura gatividhiyom ke punarnirmana ke lie bharata sarakara ke nirnaya ke bada jo 1909 mem apani sthapana ke bada se vibhinna namom ke tahata astitva mem bana raha
mahamari vijnana aura sanchari rogom ke niyantrana ke lie vibhinna vishayom mem shikshana aura anusandhana ke lie rashtriya kendra ko vikasita karane ke lie punargathita sansthana sthapita kiya gaya sthapita kiya gaya tha
sanchari roga ke niyantrana mem ekikrita visheshajnata aura bahu anushasanika upalabdha karane ke lie kendra ke adhikansha ke rupa mem karya karane ke lie rashtriya sanchari roga sansthana (rasanrosam) ki parikalpana ki gai thi
rashtriya sanchari roga sansthana (rasanrosam) ko vishvasaniya tivra arthika mahamari upakarana vikasita karane ka karya saumpa gaya jo kshetra mem sanchari rogom ke niyantrana ke lie prabhavi rupa se lagu kiya ja sake 
rashtriya sanchari roga sansthana (rasanrosam) ka uddeshya mote taura para tina gatividhiyom ko kavara karana hai arthat., prashikshana, seva aura sanchari rogom aura unaki rokathama aura desha mem niyantrana ke kshetra mem karya anusandhana 
rashtriya sanchari roga sansthana ke vyapaka lakshya aura karya sarakara ko desha mem sanchari rogom ke niyantrana aura rokathama ke sambandhita muddom para salaha dena hai 
rashtriya sanchari roga sansthana, sanchari rogom para puchhatachha aura jancha arambha karata hai aisi samasyaom sahita jo svasthya aura parivara kalyana mantralaya, bharata sarakara dvara aura samaya samaya para rajya sarakarom dvara sansthana ko saumpa ja sakata hai
rashtriya sanchari roga sansthana kshetra jancha mem unake dvara kiya ja sakata hai aura jaham avashyaka takaniki sahayata pradana karane ke satha unhem rajya svasthya pradhikariyom ko sahayata pradana ki hai
rashtriya sanchari roga sansthana vyapakata aura sanchari rogom aura unaki rokathama ke prasara para parichalana anusandhana chalata hai
rashtriya sanchari roga sansthana mahamari jancha, manachitrana-sthanikamari ke kshetrom mem, bharata mem sanchari rogom ki vyapakata ka adhunika tarja para ankada़e aura ama taura para spashta karane ke lie siddhantom antarnihita sanchari roga ka adhyayana karate hai
rashtriya sanchari roga sansthana sanchari rogom aura upachara para chikitsiya jancha chalati hai. shighra nidana ki takanika ka adhyayana karane ke lie aura adyatana karane ke lie aura sansthana adhunika praudyogikiyom ka pata lagane ke lie aura anavika vidhi mahamari ki aura nigarani pravana rogom.
anusandhana karyakarta ko sambandhita vishayom mem rashtriya sanchari roga sansthana prashina deta hai
rashtriya sanchari roga sansthana janapadika roga vijnana aura sanchari rogom ke niyantrana mem karmiyom prashina deta hai
rashtriya sanchari roga sansthana vaijnanika parinama, takaniki disha nirdesha, prashikshana mainuala aura buletina prakashita karata hai.
rashtriya sanchari roga sansthana poliyo ke lie janapadika roga vijnana aura prashikshana, rebija janapadika roga vijnana aura repharala prayogashala jaise samayika hita ke vishayom para karyashalaom ka ayojana karata hai
rashtriya sanchari roga sansthana vargikarana, jiva vijnana, paristhitiki, pareshana aura sadisha roga ke niyantrana ke upayom ki gatishilata para kshetra adhyayana chalata hai
rashtriya sanchari roga sansthana samudaya mem nae upakarana, takanika, ke tarike mem jaldi pata lagane ke uddeshya se / nidana aura sankramanom ki mahamari mulyankana mem sudhara vikasita karata hai
sansthana vibhinna rashtriya svasthya karyakramom aura roga nigarani gatividhiyom ke lie anusandhana visheshajna salaha aura parichalana gatividhiyom ke madhyama se sahayata pradana karata hai.
sansthana desha mem sanchari rogom aura unake niyantrana se sambandhita vibhinna muddom para sarakara ko salaha deta hai
sanchari rogom aura unake niyantrana ke jnana ko badha़ava dene ke lie chikitsa sangathanom ko apurti karane ke lie vaijnanika aura jaivika samagri deta hai
rashtriya sanchari roga sansthana ke pustakalaya 1909 mem kasauli (himachala pradesha) mem kendriya maleriya byuro ki sthapana ke satha astitva mem aya.
pustakalaya suvidhaom mem vriddhi ki gatividhiyom ke satha aura bharatiya maleriya sansthana ke rupa mem nama ka vistara karane ke lie jari rakha.
1963 mem, vistara ko protsahana mila jaba bharatiya maleriya sansthana ki gatividhiyom ko phailaya gaya aura punargathita kiya gaya aura isako rashtriya sanchari roga sansthana ke rupa mem nama diya gaya
desha mem rashtriya sanchari roga sansthana ka pustakalaya hai jisamem desha mem prachalita maleriya ke kshetrom mem aura anya rogavahaka janita rogom ka kaphi purana sahitya hai
pustakalaya mem sabhi vektara vahana roga, sanchari rogom aura sambandhita pahaluom arthat. jivanu vijnana, perasitoloji, sukshma jiva vijnana, sankramaka roga, mayakoloji, jaiva rasayana, immunologi adi para navinatama sahitya milega 
kula 126 (patrika 85 videshi aura 45 rashtriya) sadasyata prapta hai 
isake alava 100 patrikaom mudra para ya manartha adhara para prapta ki ja rahi haim
pustakalaya mem, kula 29825 pustakom aura patrikaai, 251 nakshe, vibhinna vishayom para 89 photo, aura vibhinna sangathanom ki 1276 varshika riporta hai
kuchha mahatvapurna anukramana aura sara patrikaom pustakalaya mem upalabdha ushnakatibandhiya bimariyom buletina, svachchhata, ekasapekta medika ka sara (immunologi aura sukshma jiva vijnana), suchakanka medikasa (masika) aura helaminatoloji ka sara haim
pustakalaya peparo ki ki eka bada़i sankhya, aura indiyana jarnala ऑpha maleriya mem prakashita lekha aura jarnala sanchari roga ka paryapta staॉka rakhata hai jo desha aura videshom mem anusandhana karyakartaom ke lie apurti ke lie aura vibhinna pathyakramom ke pratibhagiyom ke lie rashtriya sanchari roga sansthana mem rakhata hai
pustakalaya mem bhi isa sansthana ka anusandhana karyakartaom ke lie shodha patrom para dalem prapta hai
kendriya pashu sabha rashtriya sanchari roga sansthana ke vibhinna prabhagom ke lie prayogashala pradana karata hai aura anya sarakari, ardha sarakari aura niji sansthaom ko bhi
pashu mukhya rupa se etiyalaji ke prakopom ki aura kaloniyom prajanana ke lie machchharom ki vibhinna prajatiyom mem se pahachana ke lie nidana prayojana ke lie upayoga mem laye jate hai
kendriya pashu sabha dvara ina pashuom ki rashtriya sanchari roga sansthana ke vibhinna prabhagom ke lie vibhinna prayogatmaka prayojanom ke lie, sapheda chuhom, rebija roga ka nidana, mausa tika parikshana, aura rebija ke lie pratijana ki taiyari ke upayoga ke lie apurti hoti hai
rakharakhava pratijana taiyara karane ke lie toksoplasama gonadi (toksoplaja़mosija़ ki serodiagnosisa mem) upayoga kiya
kitavijnana rakharakhava chuhe ke dehika (ज़enopsayalla cheposisa) kitanashakom ke avashishta prabhava ka adhyayana karane ke lie
machchharom ko khilane ka kitavijnana (stepahenase maleriya ka machchhada़ aura sahayakom igyapti, balu makkhi aura chuhe munda़na) 
edsa ka vibhaga rashtriya sanchari roga sansthana mem varsha 1995 mem sthapita kiya gaya tha
1995 se pahale edsa ka vibhaga edsa ke sandarbha prayogashala ke rupa mem 1985 se astitva mem tha
vartamana mem vibhaga ko vividha gatividhiyom ko shamila karane ke lie vistarita kiya hai
echaaivi / edsa ke lie rashtriya sandarbha kendra (echaaivi sirama vijnana yunita shamila hai)
echaaivi / edsa ke lie rashtriya sandarbha kendra 1985 se karya kara raha hai
sandarbha seva mem shamila hai rashtriya echaaivi parikshana disha nirdeshom ke anusara nigarani kendrom dvara utpanna sirama vijnani pariksha parinamom ki pushti
echaaivi sankramana ke lie vibhinna jokhima vale samuhom ki niyamita nigarani
videshi chhatrom aura bharata mem ae videshi paryatakom ki niyamita aura nirantara skrininga.
kuchha vishesha jokhima samuhom ke rogiyom samaya samaya para nigarani udaharana ke lie kshaya roga, kala-ajara, kushtha roga, aura dragsa ke nasheda़i, adi hai
dablyuechao sahayoga kendra ऑstreliya mem karya dvara ayojita antarrashtriya gunavatta ashvasana karyakrama mem bhagidari banae rakhane ke lie antararashtriya stara para manyata prapta rashtriya sandarbha kendra ke rupa mem sthiti 
adhyayana ke pratiraksha propha़aila echaaivi sankramita / edsa rogiyom mem pragati para najara rakhane ke lie roga aura pratikriya ka virodhi rekatovayarala therepi
shakuna mahatva ke pratiraksha pairamitarom ka adhyayana aura bimari ki sthiti ki dridha़ta isa prakara chikitsakom ko upachara ki pratikriya khojane ke lie madada karati hai
isa jancha mem sidi4/ sidi 8 ki ginati shamila haim, tiecha1 / tiecha2 ka adhyayana, vishishta sayatokina propha़aila, pi24 pratijana parakha, niyopatrrina ke sirama akalana, aura bita-2 mayakarolobulina aura vayarala loda parakha hai 
naidanika sevaem edsa sankramana juda़e bharata mem prachalita karane ke lie pradana ki jati haim
naidanika sevaom hai ki edsa ke mamale ko prabhavita kara sakata hai avasaravadi parajivi aura anya rogaja़naka़om ke lie pradana ki jati haim. 
paramarsha ke satha etiologikala naidanika sevaem ama ke lie yauna sancharita sankramanom pradana ki jati haim
pisiara ne sandigdha echaaivi sankramita rogiyom ki anvika nidana hai
vyaktiyom ki vibhinna shreniyom mem vibhinna jokhima vyavahara jo rashtriya sanchari roga sansthana ke lie riporta echaaivi ke parikshana ke lie purva parikshana, posta parikshana aura paramarsha jari rakhane, ke rupa mem vyaktigata mamala yogyata hai
gaira sarakari sangathana chala raha hai jaham svaichchhika jancha evam paramarsha kendra, jo rashtriya edsa niyantrana sangathana, bharata sarakara dvara vittaposhita hai
edsa vibhaga mem visheshajna echaaivi paॉjitiva vyakti ke lie paramarsha pradana karate haim
chikitsa aura sahayaka karmiyom ke lie prashikshana bharata bhara se prayogashala nidana echaaivi sankramana ki aura bayosephati hai
edsa divijana ke sankaya sadasyom dvara niyamita yatraom bharata ke vibhinna rajyom ke lie prashikshana karyashalaom ka sanchalana karane ke lie prayogashala ke kshetra mem nidana echaaivi sankramana aura bayosephati ki
rashtriya gunavatta niyantrana karyakrama disha nirdeshom ka nirupana netavarkinga ke lie rajya edsa karyakrama ke adhikariyom ke satha aura satha hi sansthaom ke sukshma jiva vijnaniyom ke satha baithaka ki
echaaivi nigarani karyakrama ka mulyankana karane ke lie vibhinna rajyom ka daura hua
edsa divijana ki nodala sansthana ke rupa mem rashtriya edsa niyantrana sangathana bharata sarakara dvara isa gatividhi ko pura karane ke lie pahachana ki gai hai
echaaivi jancha mainuala prayogashala nidana bayosephati aura gunavatta niyantrana, rashtriya sanchari roga sansthana dvara bahara laya gaya hai aura echaaivi parikshana prayogashalaom prayogashala prabhariyo satha hi bharata bhara ke chikitsakom ko vitarita kiya gaya hai 
divijana echaaivi parikshana kita ka bhandarana ke lie tina kamare thanda rakhata hai.
kula 238 jorapake (jonala rakta parikshana kendra) aura 9 (nai) sira nigarani kendrom uttara bharata ke vibhinna rajyom mem bikhare hue hai 
manava mula ke rakta utpadom manava mem echaaivi ke lie aushadha niyantraka janarala bharata ke lie parikshana upayoga kara rahe haim 
elisa aura vestarna blata kita jo vitarita kara rahe haim prayogashalaom / nigarani kendrom / rakta bainka pradarshana visheshataom ke lie mulyankana kiya jata hai 
jaiva rasayana aura jaiva praudyogiki divijana ke sakriya rupa se shamila hai roga nidana mem mahamari ke daurana aura phailane, apareshanala anusandhana, janashakti vikasa, salahakara ki bhumika aura anya vividha gatividhiyom ki rokathama ki disha mem aura niyantrana ke mahamari ka eka jarana-pravana roga bada़a sarvajanika svasthya ka mahatva hai
divijana ke rupa mem kendra dablyuechao sahayoga janapadika roga vijnana ke lie aura prashikshana ka ayojana tina mahine philda janapadika roga vijnana prashikshana karyakrama madhyama stara ke svasthya prabandhakom ke lie hara sala deshom ke adhina dablyuechao dakshina purva eshiya kshetra.
kendra bahara aura desha ke bhitara blyuechao sathiyom ko janapadika roga vijnana ke kshetra mem deta hai
kendra hara sala 4 saptaha ka prashikshana karyakrama peramedikasa se dakshina purva eshiya kshetra ke desho ko sanchari rogom ke niyantrana aura rokathama mem ayojita karata hai
10 dinom ke prashikshana karyakrama ka samaya samaya para rajya stariya tivra pratikriya timom ke lie avashyakataom para nigarani pranali ko majabuta karane ke lie nirbhara se ayojita kiya jata hai
laghu avadhi prashikshana karyakrama rajya/ jila stariya svasthya adhikari ke lie ayojita kara rahe haim jo japani insephelaitisa, dengu, haija, vayarala haipetaitisa jaisi mahamari ki rokathama aura niyantrana mem pravana rogom, meningokoksala meninjaitisa adi
divijana ne nigarani ke prashikshana maॉdyula vikasita ki hai
prabhaga bharata mem do rashtriya svasthya karyakramom ke lie kendra bindu hai arthat. sanchari rogom ke lie rashtriya nigarani karyakrama aura yaja unmulana karyakrama
rashtriya nigarani karyakrama sanchari rogom ke lie 1997 mem shuru ki gai jo roga nigarani pranali ko majabuta karane ke lie taki jaldi chetavani sanketa asanna se pahachane ja sake aura uchita samaya para karravai shuru ki ja sake
rashtriya nigarani karyakrama ka mukhya uddeshya jila aura rajya stara para kshamata nirmana hai
isa karyakrama mem rajya ke svasthya adhikariyom ko nigarani pranali ki chikitsa aura arddha karmiyom ke prashikshana ke madhyama se,-prayogashalaom ko sudridha़ karane ke unnayana ke dvara maujuda svasthya buniyadi suvidhaom ke upayoga dvara karyanvita ki ja rahi hai aura teji se sanchara aura deta sansadhana pranali ka vikasa kiya ja raha hai
karyakrama chayanita jilom mem prayogika pariyojana ke rupa mem sanchalita hai
karyakrama mem 100 jilom ko kavara karane ke lie charanabaddha dhanga se 9vi pancha sala ki yojana avadhi ke dvara parikalpana ki gai hai
yaja eka gambhira virupita aura gaira akshama-vasanta treponemala sankramana hai aura samasya bharata mem duradaraja ke durgama parvatiya jangala aura adivasi ilako mem hai
kendriya kshetra karyakrama 1996-97 ke bada se isa uddeshya ke satha sankramana ke transamishana ko rokane ke uddeshya se karyarata hai
rananiti ke rupa mem rashtriya sanchari roga sansthana ki sakriya pahachana aura mamalom ki upachara aura pasa samparka ke lie
isa karyakrama mem nau sthanikamari vale rajyom mem sthaniya svasthya pradhikariyom dvara karyanvita ki ja rahi hai
divijana rajyom ke svasthya adhikariyom ke rupa mem roga phailane ki jancha upakrama mem takaniki sahayata pradana karata hai
2000 ke daurana ki jacha mem mahatvapurna prakopa hai aiaiti dilli ke parisara mem haija; jei andhra pradesha, asama aura bihara mem; mumbai mem leptospayarosisa, betula madhya pradesha mem maleriya, saharanapura, uttara pradesha mem insephelaitisa hai
anusandhana pariyojanaom ki sankhya vartamana mem vibhinna sanchari rogom ke janapadika roga vijnani pahaluom para kiye ja rahe haim
purva phelarioloji prabhaga ko vistarita kiya gaya aura helaminatholojikala prabhaga ke rupa mem 1980 mem nama pada
helaminatholojikala prabhaga phorosisa, gini krimi aura antra parajivi rogom para anusandhana, prashikshana aura sevaai dete hai
prabhaga prayogatmaka aura samudaya adharita anusandhana adhyayana ko spashta karane ke lie mahamari vijnana ke bare mem janakari, kita vijnani, sirama vijnani aura chemotheraputika pahaluom limpha़ isa bimari ke prabhavi niyantrana ke lie phaileriya
aisiemaara ke sahayoga se phaileriya para shodha gatividhiyom aura dablyuechao, samanvita yojana banai hai aura samaya samaya para eka upayukta niyantrana rananiti taiyara karane ke lie mulyankana kara hai
prabhaga sentrala sarvekshana dala jo jnata gaira mem phaileriya ke prasarana para naja़ra rakhata sangha shasita kshetrom aura anya sangathanom ke rajyom se anurodha para sthanikamari vale kshetrom mem desha ki sthanikamari vale kshetrom mem aura bhi sarvekshana hai
prabhaga niyojana, nirdeshaka, samanvayana aura gini krimi unmulana karyakrama ke karyanvayana ka mulyankana ke lie kendra bindu hai jo desha bhara mem sata sthanikamari vale rajyom mem arthat., rajasthana, madhya pradesha, karnataka, maharashtra, gujarata, andhra pradesha aura tamilanadu satha hi gaira sthanikamari vale rajyom mem hai
svasthya adhikariyom aura sarvajanika svasthya injiniyara ke lie jidabyupi para prashikshana pathyakrama bhi ayojita kara rahe haim
vibhaga dvara antra parajivi sankramana para asana niyantrana rananiti vikasa ke lie anusandhana gatividhiyom satha hi prashikshana korsa kara rahe haim
karmiyom ko prashikshita badha़ane ke lie rashtriya phaileriya niyantrana karyakrama mem prashikshana pathyakrama philariala niyantrana ke lie niyamita rupa se tina shakhaom mem vyavastha kara rahe haim 
sayakalopasa ke sanrakshita namunom, gini krimi, ova aura antra parajivi aura manava phaileriya slaida ke puti vibhinna shodha aura shikshana sansthaom / sangathanom ko saplai kara rahe haim
vaham tina kshetriya phaileriya prashikshana evam anusandhana kendra para karya kara rahe hai kerala ke kalikata mem, rajamundari andhra pradesha mem aura varanasi uttara pradesha mem
nidana aura phaileriya para repharala sevaem sarvajanika aura chikitsa sansthaom ko pradana ki jati haim.
gini krimi roga ke karana hota hai isa parajivi drakunkulusa medinesisa aura pine ke pani ke madhyama se asurakshita srotom se sancharita hai jaise kadama thika hai, talabom yukta sankramita pani, adi 
‘‘chechaka unmulana’’ ki saphalata se utsahita bharata sarakara ke ministri ऑpha heltha enda epha. dablu. ne 1983-84 mem ‘‘neshanala gini varma iredikeshana programa’’ (ji dablu i pi) prarambha kiya|
desha mem ji dablu i pi ke mulyankana, margadarshana, tala-mela aura yojanaom ke lie naॉdala ejensi ke rupa mem da neshanala instityuta ऑpha kanyunikevala dija़ija (ena ai si di) dehali karyarata hai|
praimari heltha keyara sistama ke jarie kshetriya steta heltha dayarektarsa ke dvara karyakrama lagu kiya gaya hai|
da ministari ऑpha rurala davalapamenta, bharata sarakara, steta pablika heltha injiniyara dipartamentsa (rurala vatara saplai) aura phaileriya se prabhavita kshetrom mem, asurakshita pine ka pani ke sadhanom ka rupantarana aura sapha pine ka pani ki apurti ka rakha-rakhava aura prabandha karane ke karyakramom mem madada karate haim| 
1996 ke daurana jila jodhapura (rajasthana) ke tina gamvom mem nau phaileriya ki ghatanaem varnita ki gai hai, unamem mem se bhi paryapta rupa se nisantrita thi|
ji dablu bimari ka bharata me unmulana hala hi ka dusara mukhya rashtriya upalabdhi hogi|
sabhi karyakrama ke satha-satha kuchha svatantra daksha logom ke dvara unnati ka mulyankana kiya jata tha|
e neshanala kamishana phaॉra phekashana ऑpha ̍gini varma iredikeshana ki sthapana ministari ऑpha heltha enda phemili velpheyara ke dvara ki gai thi|
da kamishana antha bahuta anubhavi svatantra daksha pablika heltha ke kshetra mem sammilita karata hai joki desha bhara se laye gaye haim aura dairektara aura jvainta dairektara aura neshanala instityuta ऑpha kanyunikevala dija़ija़ ka pramukha vyakti (helaminathelaॉji)
1999 julai mem desha ne eka aisa kida़a jo dhage ke samana patala hoti hai (phaileriya) para jivi se hone vali bimari se mukta avadhi ke tina sala pure kiye|
8.1.2000 ko vijnana bhavana mem bharata se iredikeshana ऑpha gini varma, ko lakshita karake eka sabha ka ayojana kiya gaya tha|
da intaraneshanala sartiphikeshana tima (aira si ti) ne jeneva mem pharavari 2000 mem eka sabha mem ai si si di i ko bharata mem bahuta patale parajivi phaileriya se bimari ki sthiti para eka riporta di|
ai si ti ki riporta ke adhara para bharata ko gini varma bimari se aja़ada desha ghoshita kiya gaya hai|
bharata mem gini varma bimari shirshaka se eka kitaba ena ai si di dehali ke helaminathelaॉji divijana ke dvara chhapi gai haim aura aba bikri hetu bajara mem ane vali hai|
prachina kala se manava jati ko prabhavita karane vali lasika vahiniyom mem hone vali jalana eka bhanyakara bimari hai|
lasika vahiniyom mem hone vali jalana ka karana ina parajiviyom mem se eka hai – vakereriya banakrophti brugiya maleyi, ya brugiya timori|
machchhara ke katane se parajivi eka adami se dusare adami mem sancharita ho jata hai|
kyuluksa kvinakyuphasasiyetasa machchhara se sancharita dablu banakrophati ke karana lasika vahiniyom mem hone vali jalana va sujana ki bimari bharata mem prabala rupa mem hai|
kyuluksa kvinkyuphisiyetasa machchhara gandi va maili dashaom mem palata hai|
brugiyana phaileriyasisa bharata mem kuchha hi niyanntrana taka simita hai aura yaha machchharom ke manasoniyodesa samuha dvara sancharita hota hai|
dablu branakrophti ki kevala dainika rupa se saba pirayodika vividhata desha mem muthabheda़ kara chuki hai joki nikobara ke chhah chhote dvipom ke vastavika nivasiyom taka hi simita hai|
yaha bimari apane sthayi rupa mem bahuta si sharirika virupata aura ayogyata, limpha se sambandhita nalikaom ki badhaom ke karan hota hai jisaka parinama manushyom mem chamada़i moti hona, haidroile aura garde se sambandhita bimariyam hoti hai|
limphengitisa ke tivra akramana ke daurana utpadakata me hani se alaga, sthayi phaileriya ke marijo ki atpadakata mem 60 pratishata hani ka anumana lagaya gaya hai jisaka parinama arthika rupa se hani mem hoti hai|
di e ela vai ke mukhya karanom mem se eka ke rupa mem bimari aba pahachani jati hai|
sansara mem phaile hue 120 miliyana loga phaileriyala parajivi kida़o se sankramita haim|
eka viliyana logom se adhika phaileriyala parajivi kida़o ke khatare para hai|
limphetika phaileriyasisa (limph se sambandhita jalana) ki samasya dakshini purvi eshiya, eshiyana, dablu echa o ke dakshini prashanta kshetra, purvi aphrika istarna meditereniyana rijana aura rijana ऑpha amerika mem pramukha hai|
indoneshiya, bharata, china mem lasika vahiniyom meka hone vali jalana va sujana eka mukhya janasadharana svasthya samasya hai|
sampurna anumanita sankramita logom ka lagabhaga do tihai hissa china, bharata aura indoneshiya hai|
sampurna prithvi para limphetika phaileriyasisa (limphlasika vahini ki sujana va jalana) se sankramita lagabhaga 120 milina loga anumanita kiye gaye haim, bharata ne isa prithvi mandala ki samasya mem 39 pratishata yogadana karata hai|
31 disambara 1996 ke antima anumana ke anusara 428.28 miliyana loga bharata mem 18 kshetriya rajyom aura yuniyana teritarija mem rahate haim|
315.11 miliyana loga gramina kshetrom mem rahate hue limphetika phaileriyasisa (limpha se sambandhita jalana) ke khatare para haim|
bharata mem 49 miliyana sankramita vyaktiyom mem se 20 miliyana loga phaileriyasisa ke sthayi rupom se kashta pate haim|
malayana phaileriyasisa, andhrapradesha, uda़isa, madhyapradesha, kerala, pashchimi bangala, tamilanadu aura asama rajyom ke gramina kshetrom taka simita kiya gaya hai|
sansara mem bainakrophtiyana phaileriyasisa ko niyantrita karane ke lie uda़isa mem 1949 se 1954 taka pahali bara eka margadarshaka yojana shuru ki gayi thi|
bharata sarakara ne neshanala instityuta ऑpha kanyunikevala dija़ija़ (ena ai si di) ke tahata 1955 mem bharata mem neshanala phaileriya kantrola programa prarambha kiya|
karyakrama ke kriyakalapom ko saja-sajja ke dvara punah nirikshana kiya gaya hai|
shahari sthaniya kshetrom mem nai niyantrana ikaiyom ka pratishthapana|
ena epha si pi ko 100 pratishata kendriya pratibhutika yojana banayi jani chahie|
gramina janasankhya ko surakshita karane ke lie praimari heltha sentarsa ke satha ena epha si pi ko milakara purna kiya jana chahie|
da vileja heltha gaida (vi echa ji) enda maltiparapaja़ varksa (ema pi dablu) di i si ke satha phaileriyasisa ki naidanika ghatanaom ka upachara ho sakata hai|
pratishata ema epha dara aura kama se kama 20,000 logom ke satha gamva mem phaileriya ikai sthapita ki ja sakati hai|
yadi niyama se hone vala sarvekshana pura ho gaya ho, sarvekshana ikaiyom ko purane nirakshita jilom ke punah sarvekshana ke lie lagaya jana chahie| 
kendra ke dvara jaise urvana maleriya skima mem sancharana ko rokane ke lie ena epha si pi kasbom ke lie eka vishesha prajati ke paudhom ka arka pradana kiya ja sakata hai|
brugiya malayi sankramana ke anmulana para upaya jo ki sambhava haim 1996 mem prarambha kiye gaye hogem|
maleriya sancharana ko nipatane ke lie kasbe jinamem unche maleriya ki ghatanaem varnita karate hue phaileriya ke lie eka prajati ke paudhom ka arka ki do pratishata apurti bhi ki ja sakati hai|
phaileriyasisa ke prashikshana aura shodha ke lie kendriya sangathana helaminatholaॉji dovijana ऑpha neshanala instityuta ऑpha kanyunikevala dijija (ena ai si di) dilli mem hai|
ena ai si di ki tina shakhaem padanamita ki hai jaise rijanala phaileriya dreninga enda risarcha sentarsa (ara epha ti enda ara si) joki kailikata (kerala) rajamundri (andhrapradesha) aura varanasi (uttara pradesha) mem sthita haim|
sanshodhita rananiti ko vyavasthita karane vali eka rashtriya karyashala limphetika phaileriyasisa ke niyantrana ke lie janavari 1996 mem dilli mem sangathita ki gai thi|
sarvajanika svasthya mapa ke rupa mem 1 dina vali chikitsa pranali, 12 dinom vali chikitsa pranali ke samana hi prabhavashali hai|
rashtriya phaileriya divasa 13 jilom mem manane ke tahata bharata sarakara 1996 mem di i si sankramita bimariyom ke upachara ke lie varshika utsava ki margadarshaka yojana shuru ki thi|
pancha varshom ki avadhi ke lie ema di di ki yojana banai gai haim jaba se pajivi ke badha़ottari karane vale jivana ka alpa samaya lagabhaga pancha varsha haim|
1997 se eka margadarshaka yojana 13 jilom mem shuru ki ja chuki hai|
dablu echa o ne pachasavim varda heltha aisembalike dvara mai 1997 mem eka dridha़ sankalpa ke badalava ke satha phaileriyasila elimineshana programa varlda vaida, yaha kahate hue limphetika phaileriyasisa ka bahishkara eka sarvajanika svasthya samasya ke rupa mem prarambha kiya tha|
svasthya sevaom ke tahata 1996 se 1998 taka ema di di ke daurana ji i si kavareja enda kaॉmpaleansa ka mulyankana ne sanketa kiya ki kevala adhi lakshita janasankhya ne hi thika se draga ka upabhoga kiya|
rugnavastha ki vyavastha ke lie suvidha ena ai si di ki tina shakhaom varanasi, kelikata, rajamundi mem sthapita ki gai haim|
hastakshepa ka parinama aura sangramika sammbhavayata ke parikshana (vishleshana) ke lie eka vijnapti (purva lekha) vikasita kiya gaya tha|
varlda heltha ऑrgenaijeshana (dablu echa o) kama se kama chara sau miliyana skula jane vale bachche (chhoti unra) ko shamila karate hue anumana lagata hai ki desha ki eka chauthai janasankhya sthayi rupa se mitti se sancharita krimiyom se sankramita ki gai hai|
jalavayu ki dashaom, mitti ke prakara, svachchha sthitiyom bhojana aura vyaktigata svasthya para nirbhara vibhinna kshetrom mem tivrata aura vyapakata vishesha rupa se ghata-badha़ sakati hai|
anta ke parajivi ke sankramana ki bahuta vyapakata garibi, kharaba paryavarana se sambandhita svasthya vijnana aura bhaddi bekara svasthya sevaom se bahuta juda़i hui hai|
anta ke parajivika sankramana bahuta jyada phailane vala aura adhika vyapaka hai lekina tulanatmaka rupa se asani se niyantrana karane yogya hai|
isa samasya ka vistara bharata mem sahi pramana mem nahim hai|
neshanala instityuta ऑpha kanyunikevala dija़ija़, dehali mem divijana ऑpha helamintholaॉji ne san 1968 se desha ke vibhinna bhagom mem paramparagata sarvekshana karane ki jimmedari utha raha hai|
sarvekshana kshetrom ke bahuta khatare vale ilakom mem niji rajya/jila svasthya adhikariyom ke satha talamela banate hue sarvekshanom ko yojanabaddha kiya gaya tha|
desha ke vibhinna rajyom mem 110 sarvekshanom ka samuha (yoga) ko pura karane ki jimmedari athayi gayi thi|
1990 se 16 sarvekshanom ka eka samuha 14 rajyom mem pura kiya ja chuka hai|
anta sambandhi parajivi ki vyapakata 22.8 pratishata se jila puri, uda़isa mem, pashchimi ilake mem 78 pratishata taka parivartana karane ke lie payi gai|
3837 namunom ka eka samuha 7 paryavarana sambandhi kshetrom/anchalom mem 96 skulom se ekatrita kiya gaya hai (4 shahari, 6 gramina aura janajatiya aura ajanajatiya samudayom pratyeka se 1)
apani prashikshita manavika shakti aura susthapita antarika dhanche ke satha da divijana ऑpha treninga enda maleriyolaॉji mem rashtriya va antarrashtriya prashikshana karsesa (pathyakrama) sanchalita karata hai|
dadivija़na ऑpha treninga enda maleriyolaॉji pure dushabhara se rajya/jila stara ke maleriya adhikariyom ko prashikshana ke dvara neshanala enti maleriya programa (ena e ema pi) ka samarthana karata hai|
da divijana, desha ke vibhinna bhagom mem bimariyom ke achanaka udvega vishesha rupa se maleriya ki jancha aura rogavahaka ki jancha ke karya ke lie puri taraha se samagri-sampanna hai aura niyantrana mapa bhi sujata hai|
divijana ka maleriya chikitsalaya, yakayaka bimari ke visphota ki jancha ke daurana sangrahita slaida se, aspatalom se bheje gae marijom se parajiviyom aura maleriya ki upasthiti ke lie blada-slaidsa ka parikshana aura jancha-parakha karata hai|
da divijana, narsiga studentsa, ema.di. (pi esa ema) vidyarthiyom aura si echa esa adhikariyom ke lie chhote prashikshana karseja़ (pathyakrama) sanchalita karata hai|
da junesisa divijana, plega, kala-aja़ara aura erabovayarala sankramana ke lie neshanala repharensa leboretrija़ rakhata hai| 
da lunosisa divijana ko varlda heltha ऑrgenaijeshana ke dvara manyata di gai hai jaise dablu echa o kolavaretiva sentara phaॉra rebija़ aipidemiyolaॉji phaॉra sautha ista eshiya rijana|
hala hi mem ina adholikhita bimariyam jo janavarom se manavom mem sancharita ho jati hai una para karya pura kiya ja raha hai- plega, rebija, kala-ajara, eravosaparosisa, rikaitasiyosisa, haidautidosisa, nyurosaistiserakosisa aura ainathareksa|
rebija leboretrarija kama kara rahi hai jaise 1985 se dablu echa o kolevoretiva sentara, si rijana aura rebija mahamari vijnana ke lie kama karata hai|
rebija leboretrari, janavarom mem rebija ke nidana ke lie janata ko aura pashuchikitsalayom ko nidana ki sevaem pradana karati hai|
da leborutri, sandehapurna haidrophobiya ke keseja़ mem rebija ke lie enti maॉrtama nidana pradana karati hai|
rebija leboretri, sandehapurna rebija़ se prabhavita janavarom ke namuno se vayarasa ki visheshataom ka varnana aura alagava ko pura karati hai|
da divijana nai nidana vidya se sambandhi takanikiyom ke vikasa aura manakikarana mem laga hua hai|
serekanavarjana (rogapratikaraka jaise vaiksina se sambandhita) parikshana, adami aura janavarom danom mem bimari ke viruddha pahale aura bada mem rokathama kiye jane vale upachara sambandhita pure kiye ja rahe haim| 
da leboretri eka gunavatta purna yojana, rebija ke lie dainika nidanika prakrai ke lie chalati hai jisamem desha ki vibhinna sansthaom ki prayogashalaem shamila hoti haim|
da loboretri, rokathama ke lie upachara, asankranyakata, rebija virodhi sasti davaiyam va raste ka vikalpa dhundhane mem niratnara rupa se lagi hui hai|
da laboretri, eka rebija se prabhavita janavara ke dvara kate jane vale vividha prakara se pida़ito ke kesa mem bimari ke achanaka phutane ki jancha-pada़tala ko pura karati hai|
eravovayarasa leboretri desha mem dengu aura je i ke bimari ke achanaka visphota ki jancha-pada़tala ke lie prayogashala samarthana pradana karata hai|
eravovayarasa leboretri desha mem dengu aura je i ke bimari ke achanaka visphota ki jancha-pada़tala ke lie prayogashala samarthana pradana karata hai|
prayogashala, dengu aura je i vayarasa ke prithakkarana ke lie si 6/36 saila lainsa aura vero saila lainsa ki suvidhaem rakhata haim|
eravovayarasa leboretri, serolajikala (blda sirama se sambandhita) nidana ke prayoga ke lie vikasita prayogashala takanikom ke manakikarana ka prayasa karata hai|
lishamaniya leboretri dilli mem sabhi aushadhalaya aura aspatalom ko nidana se sambandhita sevaem pradana karati hai|
lishamaniya leboretri donom perasitayolaॉjikala (parajivi ka utpida़na) aura serolaॉjikala (blda sirama se sambandhita) nidana ke lie suvidhaem rakhati hai|
lishamaniya leboretri, lishamaniya strensa ke rakha-rakhava, kroe prijaraveshana (jivita koshikaom ki suraksha) aura prithakkarana mem lagi hui hai|
lishamaniya leboretri svayam mem yatheshta hai, serolajikala prakriyaem jaise i epha e aura si ai i pi ti i ela ai esa e, ai ephae aura si ai i pi ke lie rasayana aura rakta mem upasthita hanikaraka tatvom ko nashta karane vale padarthom ka manakikarana aura purva taiyari mem hai|
lishamaniya leboretri, prarthana para entijana (eka hanikaraka padartha jo sharira mem entibaॉdija paida karane ka karana hai) ki bharata mem dusari prayogashalaom ko satha hi satha dusare deshom jaise nepala ko apurti karati hai|
lishamaniya leboretri desha mem vibhinna prakara ki bimariyom ka achanaka visphota ki jancha ke lie prayogashala sahayata pradana karata hai|
lishamaniya leboratri kala ajara ke marijom ke upachara ke lie paramarshatmaka sevaem pradana karati hai|
lishamaniya leboretri kala-ajara ke nidana ya tivra aushadhalayom ke lie nidana vijnana ki sevaem pradana karati hai|
taikjoplajama leboretri dilli mem sabhi aspatalom aura aushadhalayom ke lie nidana vijnana ki sevaem pradana karata hai|
taiksoplajma leboretri svitjalainda ke sapheda dhabbedara balom va khala vale chuhom ke taiksoplajma gondai ke ara echa strena bana rahi hai|
manga hone para sansthaom ko ara echa strena ki apurti ki ja rahi hai|
kendriya plega prayogashala QP-2 stara ki prayogashala hai|
kendriya plega prayogashala plega ke sabhi pahaluom para karya karane ke lie sakshama hai|
kendriya plega prayogashala rajyom ki plega niyantrana gatividhiyom ka samanvaya karati hai|
kendriya plega prayogashala, rajya plega niyantrana ikaiyom ke sahayoga se desha mem plega para nigarani rakhati hai|
kendriya plega prayogashala, plega para nigarani rakhane ke lie jivanu vijnana, sirama vijnana tatha kita vijnana ka adhyayana karati hai|
kendriya plega prayogashala desha mem kahim bhi plega ke sandehaspada phailava ki jancha pada़tala ka kama karati hai|
kendriya plega prayogashala karyakshetra namunom se prithaka sandehaspada vai paiptisa jaise jivanu ki pahachana karati hai| 
kendriya plega prayogashala vai paiptisa ke prithaka kana ko banae rakhati hai|
kendriya plega prayogashala sirama vijnana ki nayi takanikom jaise elisa, aijiema ke vikasa ka kama karati hai| 
kendriya plega prayogashala, prayogaashalaom ke bicha guna –ashvasna ko banae rakhane ke lie karyakrama chalati hai|
kendriya plega prayogashala, plega para rajyiya talamela kamechi ki niyatakalika sabhaom ka ayojana karati hai tatha una ki vvastha karati hai|
kendriya plega prayogashalasa pratikaraka jaivika samagri ki prapti hetu tatha suchanaom ke adana pradana mem dablyuechao sahayoga kendra ki sahayata karati hai| 
leptospira prayogashala dilli ke sabhi aspatalom ko nidanasuchaka sevaem pradana karati hai|
leptospira prayogashala mahamari ke phailava ki jancha pada़tala ke lie prayogashala sahayoga pradana karati hai| 
leptospira prayogashala leptospirosisa ki sirama nigarani para bahukendriya adhyayana prarambha kara rahi hai| 
haidetida aura riketasiya prayogashala riketasiya aura haidetida sankramana ki karyavahi karati hai tatha dilli ke sabhi aspatalom ko nidanasuchaka sevaem pradana karati hai|
haidetida aura riketasiya prayogashala haidetida bimari aura riketasiya sankramana ke sirama vijnana sambandhi nidanasuchaka ka karya karati hai|
busela prayogashala niyantrita janasankhya para sirama sambandhita nigarani ke adhyayana ki jimmedari leti hai|
sisitisarakosisa prayogashala miragi ke rogiyom mem sistisarakosisa ke prachalana para adhyayana karati hai|
sistisarakosisa prayogashala niyurosistisarakosisa ke nidanasuchaka getu sirama vijnana sambandhita parikshana jaise elisa ke nirdharana ka kama kara kara rahi hai|
enthreksa desha ke vibhinna bhagom chhuta ki bimari ke phailava ki jancha pada़tala ke lie prayogashala sahayata pradana karati hai| 
baingalora sthita plega dekharekha ki gatividhiyom mem talamela baithane aura nirikshana karane ka kama kendriya plega prayogashala, junosisa divijana, enaaisidi, mukhyalaya, dilli karati hai|
plega dekharekha ikai plega sankramana ke pramana hetu dantom se kutarane vale jangali jantuom ke mausami prachalana para adhyayana ka karya karati hai|
plega dekharekha ikai plega sankramana ke pramana hetu dantom se kutarane vale jangali jantuom ke jivanu vijnana aura sirama vijnana sambandhita parikshana ka karya karati hai|
plega dekharekha ikai plega ka karana janane ke adhyayana ka bahishkara karane hetu chuhom ke hvasa ki jancha pada़tala ka kama karati hai|
plega dekhaऱekha ikai kita vijnana adhyayana jisamem kita se kita marane ki dava taka ki roga pravanata ka adhyayana shamila hai, ka karya karati hai| 
plega dekharekha ikai, plega dekharekha tatha rajya plega niyantrana ikaiyom ki gatividhiyom mem talamela baithati hai|
yaha vibhaga vibhinna junotika bimariyom para samaya samaya para prashikshana karyakrama ayojita karata hai|
yaha vibhaga jaii, rebija, kalajvara, plega tatha leptospirosisa jaise vibhinna rogom para nirdeshikaem tatha pustikaem bana chuka hai| 
insatityuta ऑpha kanyunikebala dijija़ dvara pradatta jantu suvidha 1000 varga mitara ke kshetra mem phaaili hui haim, tatha isamem prajanana suvidha (800 varga mitara) aura prayogika suvidha (200 varga mitara) bhi shamila haim|
yaha suvidha sipiesaie arthata jantuom para prayoga ke nirikshana niyantrana ke lie banai gai kameti minisitari ऑpha soshala jastisa enda emapavaramenta, nai dilli ke satha namankita ki gai hai jisaka namankana kramanka 16/1999/sipisiesaie|
chuhom ko tina bandhi hui kaॉloniyom mem jo ki phaundeshana staॉka, pedigrida eksapenshana staॉka tatha podakshana staॉka hai, mem ikattha kiya jata hai|
janana vijnana mem anishchita sangraha mem kharagosha, murgi, suara, bheda़, hansa aura kabutara bhi pale jate haim|
janavarom ko sambhavita sankramana se bachane ke lie prayogika suvidha ko janana kshetra se alaga rakha jata hai|
prayogika suvidha ko mukhya shodha blaॉka se joda़a jata hai aura sansthana ke vaijnanika karmachariyom ke lie sulaba hota hai| 
seva kshetra mem gadde aura sapha dhulai bhaga, bhojana, bistara, pinjare tatha bekara samana nikasi ikai ka bhada kshetra shamila hai|
vyavasayika chhoti goliyom ki khuraka ko bangala grena ki jagaha upayoga mem liya jata hai| gajara aura pattedara sabjiyam, murgi, suara tatha kharagoshom ke lie upayoga mem liya jata hai|
manaviya upayoga hetu emasididvara prapta kitanu mukta pani ka prayogashala ke sabhi javanarom ko unhe bimariyom se bachane ke lie diya jata hai|
svastha prayogika janavarom ke svasthya ka khyala pashu chikitsaka ki madada se rakha jata hai|
bimara janavarom ko alaga rakha jata hai tatha unaka ilaja karavaya jata hai|
nrita janavarom ko jala diya jata hai|
bharata mem haija eka sthaniya bimari hai isa bimari ke phailava ki kai suchanaem mili haim| 
haija mem jo tivra gati se phailane ki kshamataem haim jo tivrata se sarvajanika svasthya samasyaom ki tarapha agrasara hoti haim isa roga ki tarapha vishesha dhyana diya jana avashyaka hai tatha haija ke suchita mamalom para turanta karyavahi ki jani chahie| 
haija ka sandehaspada mamale ke bada mem sthaniya svasthya adhikari ko sandeshavahaka, teliphona ya phaiksa se turanta suchita karana chahie|
agara sahi se savadhanipurvaka vichara kiya jae to haija eka simita nivasa sthana taka hi rahata hai|
kshetra mem pahale haije ke mamale ke bare mem turanta sthaniya svasthya adhikari ko suchita karana chahie|
prayogika santushti jalda se jalda prapta karake parinamom ko turanta sthaniya svasthya adhikari ko suchita karana chahie| 
vaibriyo kolera 01 ya 0139 ko dasta ke rogi ke mala ke namune se alaga rakhana|
haija ke satha adhika patale dasta ke mamale ke vishesha, ilaja ke lie sabhi rogiyom mem haija ka sandeha kiya jana chahie|
haija ke mamuli mamalom ke lakshana naidanika rupa se avishishta tivra dasta ki bimari se aprameya hota hai
nabbe pratishata se jyada mamalom mem haija mamuli hota hai|
halahi mem lagabhaga sabhi haija ke phailava iaiti ya bayotaipa ke karana se hue haim|
1980 se lekara aba taka jitane bhi klasikala bayotaipa ke karana mamale rahe haim unake bare mem suchita nahim kiya gaya hai|
klasikala bayotaipa ki tulana mem iaitiora bayotaipa jyada anupata mem upagami sankramana phailata hai tatha vatavarana mem lambe samaya taka bana rahata hai|
1992 ke anta mem naye sairogrupa – vi klora 0139 ke karana lambe stara para bharata aura bangladesha mem mahamariyam utpanna huim|
naidanika lakshanom ke nidana ke bada bhi rogiyom mem samanyatah sankramana bana rahata hai|
kabhi kabhi roganu vahaka avastha kai mahinom taka bani rahati hai|
kitanunashaka dava jo sugamata se karya karati hai, sankramata ki avadhi ko kama karati hai|
vi kolera vatavarana mem lambe samaya taka raha sakati hai tatha kuchha jaliya paudhom tatha janavarom ke satha bane rahate haim tatha pani ko sankramana ka bhandara banane mem sakshama hai|
samanyatah sankramana pradushita jala tatha bhojana se phailata hai|
bimari paida karane ke lie vi kolera ka khuraka vyakti ki atisamvedanashilata para nirbhara karati hai|
sthaniya bimari vale kshetrom mem stanapana navajata tatha bada़e bachchom ki suraksha karati hai|
kai mamalom mem aurala rihaidreshana thairepi dvara shuruati ilaja se, haija ka ghataka prabhava eka pratishata se bhi kama kiya ja sakata hai|
agara dasta aura ulti ke lakshana bane rahate haim to dravya aura lavana ke hvasa ke phalasvarupa pani ki kami, esidosisa tatha poteshiyama ki kami ho sakati hai|
pani ki purti hetu chikitsa ko lagatara hone vale lavana aura dravya ki kami ko pura karane ke lie chalu rakhana chahie|
rogi ki chikitsakiya sthiti ko pani ki purti ke pahale va bada taba taka dhyana mem rakhana chahie jaba taka dasta ruka na jaem|
bada़e tatha kuposhita bachchom ka vishesha dhyana rakha jana chahie|
mataom ko eka litara pani ko napana sikhaya jana chahie|
o ara esa ka ghola banane ke lie pani ko ubalane ki koi avashyakata nahim hai|
jo sapha pani ghara mem pine ke lie kama mem laya jata hai, oaraesa banane mem upayoga mem laya ja sakata hai| 
oaraesa banane se pahale hatha dhone chahiem|
eka litara pani mem oaraesa ka pura paiketa upayoga mem lena chahie|
samanyatah mata isa paiketa ka kuchha bhaga bada ke upayoga ke lie bachakara rakha legim joki nahim hona chahie|
isa bata para jora diya jata hai ki pura paiketa eka litara pani mem milaya jana chahie|
yaha ghola 24 ghantom ke andara andara upayoga mem liya jana chahie agara na liya jae to phenka dena chahie|
lagabhaga 80 se 90 pratishata rogi bina kisi antahshirachikitsa ya kitanunashaka dava ke sirpha oaraesa ke ghola dvara thika kie ja sakate haim|
pahale 3 se 6 ghante mem 100 mili/kilo vajana ke barabara oaraesa ghola pani ki kami ko thika karane ke lie dem|
agara rogi pyasa mahasusa karata hai aura pani pina chahe to use oaraesa pine dem|
jina rogiyom mem gambhira rupa se pani ki kami ho unhe jalda se jalda oaraesa ka gholapine ke lie dena chahie jaise hi ve pine mem sakshama hojaem yaha prakriya prarambhika antahshira dravya chikitsa ke pure hone se pahale hi shuru kara deni chahie|
antahshira chikitsa una logom ke ilae avashyaka hoti hai jo gambhira haija se pida़ita haim|
aivi dravya ka upayoga prarambhika rogi ki pani ki purti hetu tatha una logom ko jo sadamem mem hom dene mem tatha jo gambhira pani ki kami se pida़ita hom, ko dena chahie| 
antahshira pani ki purti hetu ringara ka lekteta ghola ko prathamikata di jati hai|
agara ringara ka laikteta ghola upalabdha naho te samanya lavaniya ghola yadyapi yaha kama prabhavi hota hai, antahshira dvara pani ki purti hetu upayoga mem liya ja sakata hai|
sadharana glukoja ghola aprabhavi hai tatha upayoga mem nahim liya jana chahie|
antahshira dravya chikitsa ke daurana rogiyom ki nirantara atarala para dhyana rakha jana chahie taki rogi ki chikitsakiya sthiti mem sudhara ke chinha ki jancha ki ja sake|
isa bata ka bhi dhyana rakha jana chahie ki rogi ko jarurata se jyada ghola na diya jae|
agara jyada matra mem aivi dravya diya jae to phuphphusiya odema utpanna hota hai khasataura para aisa taba hota hai jaba upapachayi emidonsisa ko thika na kiya jae| 
uparokta taba hota hai jaba ama lavana ko dakara aivi panipuraka tatha oaraesa ghola nahim diya jata hai|
jaba aivu rihaidreshana ke nirdeshana ka anusarana kiya jate haim to phuphphusiya odema nahim hona chahie| 
oaraesa ka ghola dene se phuphphusiya odema nahim hota hai|
agara aivi dravya kama matra mem diya jaye, jada़ta jatake ko turanta thika na kiya jae ya jada़ta phira se hoti ho joki vishesha rupa se 80 sala se upara ke logom mem hota hai to gurda phela ho sakata hai| 
agara gambhira pani ki kami ko thika na kiya jae aura samanya pani ki purti ko nirdeshika ke anusara banaye rakhem to vrikka viphala nahim hota hai| 
gambhira mamalom mem kitanunashaka dava dasta ke prabhava vistara tatha avadhi ko kama kara sakate haim tatha kolera vaibriyoja ke utsarjana ki avadhi ko kama kara sakate haim|
jaise hi ulti ana banda ho jaem turanta pani ki purti ke prarambhika tina chara ghantom ke bhitara kitanunashaka dava munha ke madhyama se die ja sakate haim|
injekshana dvara die jane vale kitanunashaka dava se koi labha nahim hai, kyonki ve bahuta mahange hote haim|
mamuli mamalom mem kitanunashaka dava ka upayoga nahinka iya jana chahie|
kitanunashaka dava ka chayana kitanunashaka dava ke pratirodhaka ki sthaniya namunom ke adhara para kiya jana chahie|
nikatatama tatha juda़e hue kshetrom mem kitanunashaka dava samvedanashilata ke namunom ki janakari hona avashyaka hai|
haija ke nidana ke lie dastanashaka, vamananashaka, enthananashaka, kaॉrdiyotaॉnika tatha kortikostiroyada dava nahim deni chahie|
rudhira ka aka dhamani se dusari dhamani mem pravesha tatha vaॉlyuma vistara avashyaka nahim haim|
kai sankramaka bimariyam bharata mem mahamariyam haim|
roga niyantrana karyakrama ko yojana banane, lagu karane tatha nirikshana karane hetu eka prabhavi nigarani vyavastha ki avashyakata hai|
kai roga mausami evam chakriya chalana mem hote haim jinaka nigarani vyavastha se pata lagaya ja sakata hai|
roga phailava ka karana bana sakate haim jo tivra gati se phailata hai tatha aneka nrityu ka karana banata hai| 
desha mem plega, maleriya tatha dengu raktasrava bukhara ne nishchita rupa se roga nigarani vyavastha ko dridha banane ki avashyakata para jora dala hai taki shuruati chetavani ke chinhom ko pahachana jae evam sahi samaya para sahi anusaraniya kadama uthae jaem| 
kai uchcha prabhava vali kametiyom ne roga nigarani vyavastha ko dridha banane tatha shuruati phailava ki pahachana karane ke lie kadama uthane ka samarthana kiya hai|
svasthya tatha parivara kalyana mantralaya, bharata sarakara ne chuninda jilom mem sankramaka rogom ke lie rashtriya nigarani karyakrama shuru kiya hai|
prabhavashaali roga nigarani vyavastha hetu, ankada़a sangraha ka aparivartita tatha nirantara rahana avashyaka hai|
suchana mem ashvasta aparivartita hetu nigarani karyakrama ke tahata ‘mamala paribhasha’ ka dhyana rakha jana chahie|
mote taura para mamalom ka vargikarana sandehaspada sambhavita tatha nishaachita rupa mem kiya gaya hai|
nigarani vyavastha ke uddeshya se vo sabhi mamale jo sambhavita ke antargata ate haim sakaratmaka mamale ke rupa mem lie jane chahie|
agara suvidha upalabdha ho to roga nirnaya nirdharana hetu sabhi sandehaspada mamalom ki jancha karani chahie|
vaibriyo kolara 01 tatha 0139 ka dasta ke rogi ke stulasa saimpala se prithakkrarana|
haija ke samaya adhika patale dasta ke mamale ke vishesha ilaja ke lie sabhi rogiyom mem haija ka sandeha kiya jana chahie|
tina ya tina se jyada patale mala ka pichhale 24 ghantom mem vamana ke satha ya vamana ke bina, pani ki kami ke satha ya pani ki kami ke bina|
tivra bimari ninnalikhita chikitsakiya varnana ke anusara piliya, darka yurina, bhukha na lagana, ghabarahata, atyadhika thakana, peta ke upari bhaga mem dahini tarapha darda hona| 
tivra sankramana ke lie vishesha rupa se bane echabii aijiema nirodhi testa hara jagaha upalabdha haim|
echabiesaeji joki prayah upalabdha hota hai tivra naye sankramana tatha kronika hepetaitisa bi ki badatara sthiti mem pharka nahim kara pata|
chhah maha se jyada echabija़ eji ki sirama sakaratmakata purane sankramana ki suchaka hai|
maleriya ke sabhi rogi bukhara se pida़ita hote haim, chinha tatha lakshana bhinna bhinna hote haim| 
siradarda, kamaradarda, sardi, pasina ana, gathiya michali tatha ulti ke lakshana haim|
spalainomaigali tatha khuna ki kami maleriya ke sambandhita lakshana haim|
dengu vayarasa ka sirama jiva dravya shaaveta kta kanika, shava parikshana namune se prithakkarana|
japani mastishka jvara vayarasa sankramana phaibaraila bimari ki bhinna gambhirataom ke satha satha nada़i sambandhi lakshana jo siradarda se mastishka jvara taka bhi simita hota hai mem phalita hota hai|
mastishka jvara ko chikitsakiya rupa se dusare kendriya snayutantra sankramana se alaga nahim kiya ja sakata|
paudhom se nikalane vala chipachipa padartha phira gardanatoda़ bukhara, shita jvara tatha echa inphluenja ka pata lagata hai|
dengu tatha dengu raktastrava bukhara eka tivra vayarala bimari hai| 
degu bukhara eka svayam simita bimari hai tatha dengu sankramana ke mamalonki adhikata ka pratinidhitva karati hai|
kuchha ajiba mahamari paristhiyom mem joki vayarasa ke visheshata samvahana tatha aprabhavita stara para nirbhara karati hai, dengu sankramana ke satha raktastrava tatha shaॉka mem jatilataom ko prakata karati hai| 
bharata mem dengu sakramana eka shatabdi se adhika taka astitva mem rahata hai|
dengu sankramana duniya ke kai bhagom mem jisamem bharata bhi shamila hai prabhavapurna rugnata tatha nrityu ka karana banata hai| 
yaha vayarasa pahali bara kalakatta, pashchimi bangala mem 1945 ke daurana prithaka kiya gaya|
dengu vayarasa ke sabhi 4 prakara prithaka kie gae haim tatha bharata ke vibhinna bhagom mem suchana mili hai| 
raktastravita prakatikarana se sambandhita sabase pahala bada़a phailava 1963 mem kalakatta mem hua tha|
diechaepha mamalom ki ghatanaom mem eka ashcharyajanaka chadhanava dekhane ko mila tha tatha bada़i matra mem phailava ki suchanaem desha ke hara kone se mili thim| 
1996 mem dilli se eka bada़e phailava ki suchana mili jisamem 10252 mamale the tatha 423 mautom ke bare mem rajya nideshalaya ko suchita kiya gaya|
dengu sankramana ki visheshata tivragati se phailana hai jo tivra sarvajanika svasthya samasyaom ko janma deta hai, isaki nigarani rokathama tatha niyantrana hetu dhyana diya jana chahie|
diepha/diechaepha ke mamalom ki masika suchana sambandhita rajya ke adhikariyom ke madhyama se rashtriya maleriya rodhi karyakrama nideshalaya, 22 shyamanatha marga, dilli 110054 ko di jani chahie|
agara diepa/diechaepha ke karana akasmika mamala tatha nrityu mem badhottari ho to turanta jila svasthya karyalaya ya nikatatama nirikshaka suchita karana chahie|
jila svasthya karyalaya, tivra sanchara sadhanom dvara sambandhita svasthya adhikari ko suchita karem (variyata dem teliphona phaiksa ya i mela) 
phailava ki vistrita janakari jisamem jancha pada़tala, niyantrana tatha mapana bhi shamila haim, ko jitani jaldi ho sake suchana ka anusarana karana chahie|
isa sambandha mem uthae gae kadamom ke bare mem sankramaka rogom ki rashtriya sanstha janakari rakhati hai|
diepha/diechaepha, eka samuha bi arabovayarasa dvara janita hota hai jisamem 1,2,3,4 stiriyotaipsa shamila haim|
yaha sankramana eka sakramita mada machchhara edesa edajipati ke katane se phailata hai|
edesa ejipati machchhara samanyatah dina mem katata hai|
machchhara agara kisi dengu sankramita vyakti ko katata hai to machchhara sankramita ho jata hai|
eka bara machchhara sankramita ho jae to taunra aisa hi rahata hai|
mada machchhara samanya tapamana tatha nami mem 3 haphtom se adhika ji sakate haim|
mada machchhara viresika asvastha vyakti ko katane para sankramita ho jate|
mada machchhara jisane ki eka sankramita ashrayadata ko katata hai, vayarasa hastantarana ke paryapta stara taka phailane mem 8 se 11 dina leta hai| 
dengu prasarana ke lie vatavarana tapamana sima 160 selsiyasa se 400 selsiyasa hai, 160 selsiyasa se niche edesa ejipati katana banda kara deta hai|
bharata mem dengu prasarana ka mukhya rogavahaka edesa ejipati hai|
dengu phailava ke lie edesa elbopiktasa kokarana mana gaya hai|
machchhara ki pitha tatha tangom para vishesha rupa se sapheda dhariyam hoti haim|
edesa elbopiktasa machchhara taigara machchhara ke nama se bhi jana jata hai|
yaha machchhara gharom mem, gupta jagahom para tatha andheri jagahom para rahata hai|
mada machchhara apane ande gharom, skulom tatha karyolayom ke andara tatha asa pasa sapha pani ke bartanom mem chhoda़ti hai|
lava machchhara ande ko toda़kara bahara ata hai tatha pani mem lagabhaga eka saptaha taka raha sakata hai|
phira larva eka gola pyupala avastha mem badalakara do tina dina taka rahata hai jisake bada vayaska machchhara katane ko taiyara ho jata hai| 
yaha machchhara eka gharelu janmadata hai|
machchhara prajanana kisi bhi pani lage ya pani ikattha karane vale dibbom jaise kulara, upari tainka, phalatu baltiyom, tayarom, bartanom tatha barisha ka pani ikattha karane vale dibbom jinhe samaya para khali aura sapha nahim kiya jata mem ho sakata hai|
kyonki pani machchharom ke jivana ke pahale 8 dinom mem bahuta jaruri hai, eka haphte mem khali hote bartanom se machchharom ki prajanana prakriya kaphi daha taka kama ho jati hai jisase dengu bukhara ka khatara bana jata hai|
edsa machchhara 400 mitara taka ki simita kshetra mem hi uda़ sakata hai parantu manushyom dvara upayo kie jane vale vibhinna yatayata ke sadhanom dvara lambi duri taka phailane ki kshamata rakhata hai|
daingu ka phelava jyadatara manasuna ke bada ke samaya mem jaba machchharom ka prajanana adhika hota hai, mem ghatita hota hai|
una kshetrom mem jaham pani ki kami ke chalate lambe samaya taka pani sanchita karake rakha jata hai, mem bhi phailava ho sakata hai|
samanyatah shahari kshetra ko jyada khatara rahata hai jaham adhika janasankhya ghanatva, kharaba svachchhata, kularom ki adhikata, tainka, bekara pada़i baltiyam, tayara, bartana adi adhika matra mem rahate haim joki machchhara prajanana ko badha़ava dete haim|
degu bukhara/diechaepha gramina kshetrom mem bhi phaila sakata haijaham vatavarana machchhara prajanana ke anukula hota hai| 
machchhara prajanana pani ekatra karane ke bartanom, maveshiyom ke khane pine mem, phalatu pada़e tinom, tayarom, botalom mem jinhe samaya para khali tatha sapha nahim kiya jata hai mem ho sakata hai|
dengu hai marejika phivara dengu bukhara ki gambhira avastha hai|
diechaepha akasmata hi teja bukhara, siradarda, bhukha na lagana, vamana tatha peta ke nichale bhaga mem darda ke satha shuru hota hai|
pahale pancha dinom ke daurana, bimari klasikala dengu se mela khati hai, parantu maikyulopaipyulara raisha kama samana hota hai|
haimarejika dayalisisa samanyatah ghara para phaile hue peteshiya ke chehare, puchha tatha kancha dvara pradarshita kiya jata hai|
sakaratmaka dhamani pida़na sanyantra parikshana tatha venipanchara drishya para ragada़ hamesha rahati hai|
naka, masuda़om, peta tatha antom ki karya pranali se khuna bahane ke lakshana bhi dekhe ja sakate haim|
livara samanyatah bada़a komala tatha najuka hota hai|
lagabhaga 50% rogiyom mem vyapaka lasikagranthi kartana paya jata hai
2-7 dina bada jaba bukhara kama hota hai to gambhira avastha a jati hai|
isake satha hi kuchha samaya pashchata sharirika tapamana mem tivra giravata ati hai jisamem vibhinna starom para karyapranali mem gada़bada़ hoti hai|
kama gambhira mamalom mem mukhya chinhom mem hone vale parivartana kama tatha asthayi hote haim|
rogi anayasa hi thika ho jata hai ya thairepi ke kuchha samaya bada thika ho jata hai| 
sambhavita tatha nishchita dengu bukhara ka mamala tathaa teja, kama, sankuchita nadi dabava ke paraspara viphalata ka pramana tatha chipachipe, thandi tvacha tatha badale manasika sthiti ki unra mem uchcha raktachapa dvara prakata hota hai|
phephada़om tatha udara sambandhi guhikaom se jivadravya ka risava joki haipovolmika shaॉka ka agrasara hota hai diesaepha se hone vali nrityu ka mukhya karana hai| 
bimbanu ginati mem giravata tatha lohayukta khanija stara mem chadha़ava ke bicha samayavadhi sambandha diechaepha mem vilakshana pratita hota hai|
mastishkajvara ke chinhom ke satha khopada़i keandara raktastrava, upapachayi, ilektrolaita gada़bada़ tatha hipautika viphalata bhi ho sakati hai|
phaaibaraila bimari ke shuruata mem mandatmaka nidanasuchaka mem vayarala tatha baiktiriya sankramana ka bada़a vistara shamila hota hai|
sangami haimokentreshana ke satha promabosaitopeniya ka spashta upasthiti diechaepha/diesaesa ko dusare rogom jaise indotaॉksina shaॉka jo baiktiriya se hota hai, se alaga rakhati hai|
jina rogiyom mem gambhira raktapravaha, joki pluyurala bahava tatha haiprotenimiya ka pramana dete haim jivadravya risava/bahava ka sanketa dete haim|
diepha/diechaepha ka nidanasuchaka sirama vijnana parikshana dvara nishchita kiya ja sakata hai|
isa parikshana mem aijiema pratijanatatva nashaka jo lakshanom ke shuruati pahale saptaha ke anta taka dikhai dene lagate haim aura tivra ghatanakrama ke 1-3 mahine ke bada pata lagae ja sakate haim ki jancha shamila haim| 
dengu vayarasa ka sirama, jivadravya, shveta rakta kanika, shava parikshana namune se prithakkarana|
parasparika aijiji va aijiema pratijanatatvanashaka taitarasa ka juda़vam sirama namune mem eka ya jyada dengu vayarasa ke chauguna ya jyada pradarshana|
aiie dvara inyunohisto kemistri ya inyunophlorensansa ya sirama namune dvara dengu vayarasa shava parikshana tishyu mem entigama ka pradarshana|
shalya kriya shodha tatha phailava jancha pada़tala ke lie entigama ka simita sankhyaom mem neshanala insatityuta ऑpha vayaroloji (enaaravi, pune) utpadana kiya jata hai| 
eka sirama mem sankramana unra bhara ke lie himoglobina pratiraksha kshamata pradana karata hai parantu dusare sirama prakarom ke viruddha suraksha pradana karata nahim karata hai isake bajaya agami sankramana ko aura badatara bana sakata hai| 
kshetra mem dengu ki sthiti ka dekhane ke lie nigarani avashyaka hai, agami phailava ke samaya se pahale pata lagane tatha samayanukulana nivarana tatha niyantrana mapadandom ko shuru ke lie niyamita rupa se shuru kiya jana chahie| 
nigarani mem mahamari sambandhita, kita vijnana sambandhita tatha prayogashala mapadanda shamila hone chahie|
sthaniya kshetra mem rogavahaka ke phailava ki sima ka pata lagatane ke lie manasuna se purva tatha manasuna ke bada larva nigarani mahatvapurna hai|
jina kshetrom mem edeja़ ejipati anupasthita rahata hai ya bahuta kama hai tatha dengu ka phailava hota hai vaham sthaniya rogavahaka nigarani banane ke lie rogavahaka ke pahachanane ke payasa karana avashyaka hai|
edija़ machchhara tatha larva ki kitavijnana adhyayana byavara siti mem 5 janom mem ki gai|
shahara ke tatha pada़ausi rajyom, jilom va kshetrom ke mamalom ki sharirika sthiti ka pata lagaya jana chahie|
haipitaitisa vayarasa ke karana, haipitaitisa yakrita ka eka tivra sankramana hai|
chhah prakara ke haipitaitisa haim e, bi, si, di, i aura ji| 
chikitsakiya nidana para adharita, haipitaitisa vayarasa ki 100,000 ghatanaem varshika darja hoti haim| 
kisi bimari ki vastavika varshika ghatanaom ke nambara joki darja the purnatah galata anumanita the|
yaha pramukha hai ki sandigdha vyakti para nigarani rakhane ki yojana mem sudhara hota hai isi prakara, kisi bimari ke utpanna hone se jaldi parichita hone ke lie (haipitaitisa i vayarasa) ya shaktishali uchcha kshetra, samuha vyavahara aura abhyasa ke lie avashyaka hai thika usi prakara sankramita sirinja aura sui ke prayoga se (haipitaitisa bi aura si)
rajya svasthya adhikariyom ki salaha ke dvara haipitaitisa vayarasa ki ghatanaem masikiya hailtha intelijensa ke kendriya byuro mem riporta honi chahie|
yadi haipitaitisa vayarasa ke karana achanaka se nrityu ya ghatanaom ka badha़na ya guchchhita hona, suchana turanta agale uchcha sthana taka suchita (visheshatah toliphona ya phaiksa ke dvara) honi chahie|
yaham kama se kama chhah vayarasa haim jo ki haipitaitisa vayarasa ka karana ho sakate haim|
yaha chhah vayarasa e,bi,si,di,i aura ji haipitaitisa vayarasa haim|
echaevi se echadivi ke lie prakritika parajivi kevala manushya hi hoti hai|
echaivi aura echajivi ka bhandara ajnata haim|
echaivi ke lie manava satha hi satha janavara sambhavatah bhandara haim, echaivi chimpainji mem, suara, temerina, aura sayanomolanasa, maikakyusa mem pahunchaya jata hai|
haipitaitisa e aura haipitaitisa i mem jarna mayurana roganuvahaka anupasthita haim|
haipitaitisa bi, si, di aura ji dridha़tah sankramana ka netrittava kara sakate haim| (jirna ya purana roganuvahaka)| 
jirna ya purana roganuvahaka bimari ko dusarom ko pahuncha sakata hai|
pani aura bhojana ke dushita hone se echaevi aura echaivi phesiyo orala ruta ke dvara bheje jate haim|
echabivi ke sampreshana ki pramukha kriyavidhi pairinatala avastha mem mam se shishu mem pahunchana, paitrika (sankramita sui aura sirinja, khuna ke eka sthana se dusare sthana taka pahunchane ke dvara) aura yauna sambandhi marga haim|
visheshatah shishu aura bachchom mem ghanishtha sharirika sambandha echabivi sankramana ke sampreshana ke karana bina pramana ke svikara bhi kie ja chuke haim|
echasini prayah pa2trika marga ke dvara sampreshita hota hai: sampreshana ka khatara yaha hai ki gharelu sambandha aura yauna sambandhi kriya kalapa bahuta kama pratita hote haim|
echadivi jisako echabivi ki upasthiti ki jarurata isake bahuvidha ke lie hoti hai echabivi ke rupa mem bheja ja sakata hai|
echajivi ke lie kevala simita suchanaem upalabdha haim yaha vayarasa paitrika marga ke dvara nishchita rupa se sampreshita hota hai|
ghatanaom ki bahutayata mem, visheshatah chhote bachchom mem sankramana kama hai bina kisi spashta chinha ke ya kevala kuchha bhaga piliya hai| 
adhunika tivra ghatanaom mem, akramana prayah bukhara ke satha tivra bhukha ki kami sadharana svasthaya bimari ka ehasasa, peta sambandhi kashta sabaka anusarana karate hue kuchha hi dinom mem piliya ki ghatana| 
danyi upari chaturthiya udara sambandhi komalata tatha hepara sambandhi mahatvapurna upalabdhiyam haim|
yakrita ke kriyakalapom ka parinama eelati mem eka uthava 8 bara sammilita karata hai ya samanya stara se jyada aura sorama viliruvina 2 ng% se jyada| 
haipitaitisa bi, si aura di dridhatah sankramana (jirna ya purana roganuvahaka) ka netrittva kara sakate haim| 
lagatara hone vale sankramana ka parinama purane sakriya haipitaitisa, yakrita sirohosisa aura yakrita karasinoma ko sammilita karata hai|
hamari janasankhya ka lagabhaga 3-5% bhaga ko haipitaitisa bi ke purane roganuvahaka ka hona mana jata hai|
janata aura chikitsakiya karyakarta vibhaga varnita ghatanaom, jinakom ve ankhom aura tvacha ke pile ranga ke dvara pahachana sakate haim (piliya), mem sahayata kara sakate haim|
prapta hue sakriya kesa kisi bimari ke utpanna hone ki karyavahi ke daurana visheshatah pramukha haim|
haipitaitisa vayarasa ke prakarom ke dvara nishchita rogom ka nidana kevala prayogashala ke testa ke dvara hi kiya ja sakata hai|
prayah haipitaitisa e bachchom mem kama paya jata hai lekina vayaskom mem mem kaphi tikshna ho sakata hai|
bharata mem lagabhaga sabhi bachche dasa varsha ki unra se pahale punah upachikitsakiya pradarshana ka anusarana karate hue isa vayarasa mem pratiraksha vikasita karate haim|
vishesha rupa se haipitaitisa e eka tivra, achanaka, sankramaka jvara ke samana akramana, pramukha rogom mansapeshi mem darda, siradarda, bukhara aura ghabarahata ke satha karata hai|
haipitaitisa e prayah itana asahaniya ya lamba samaya taka rahane vala nahim hai jitana ki haipitaitisa bi|
haipitaitisa e jarna haipitaitisa ka netritva nahim karata ya roganuvahakom ki dasha echaevi sankramana ke lagatara hone vale sankramana ke satha purana haipitaitisa 12 mahinom se jyada dekha nahim ja chuka|
haipitaitisa e se hone vali asvasthata, eka hi unra ki agarbhavati mahilaom ke mukabale garbhavati mahilaom mem jyada nahim hoti aura na hi haipitaitisa e ka upachara garbhavastha ke daurana prayah bhinna aura haipitaitisa e ke karana bhruna ka tedha़a medha ho jane ka likhita varnana nahim kiya gaya hai| 
haipitaitisa bi ka akramana prayah bhukha marane vala roga, aspashta, udara sambandhi bimari, ubakai aura ulti ke lie ghataka hai, kabhi – kabhi joda़om ki bimari aura krodhita hona prayah piliya mem vriddhi karata hai|
haipitaitisa bi, haipitaitisa e ke mukabale jyada lamba upachara karavata hai|
echabivi ke karana sthayi livara ki bimari (kronika haipitaitisa, livara sirosisa aura livara karasinoma) kevala una logom mem payi jati hai jo echabiesa eji roganuvahakom ke lie sthayi hote haim|
rakshaka asankranyata sankramana ka anusarana karati hai yadi eviesaeji (enchi echaviesa) aise padarthom ko utpanna karate haim jo hanikaraka pratijana tatvom ko nashta karate haim aura echabiesaeji asakriya ho jate haim|
chikitsakiya adhara para haipitaitisa si, haipitaitisa bi se milata julata hai|
tivra avastha ke daurana haipitaitisa si ka asara kama hota ja raha hai aura sthaniyata mem vriddhi haipitaitisa bi ke mukabale bara bara jyada ho rahi hai|
chikitsakiya echadivi sakramana, chahe tivra ho ya sthayi, tikshna bimari ki aura hota ja raha hai|
yadyapi kevala kuchha echabivi ki sankramana ka karana piliya hai, lagabhaga sabhi echadivi sankramana (koinphekshana aura supara inphekshana) ke karana, eka chikitsakiya tikshna haipitaitisa ka ansha piliya ke satha hota hai| 
phulaminenta bimari ka khatara, chikitsakiya echadivi aura echabivi ke satha satha hone vale sankramana ke lie 10 pratishata aura echadivi ke sankramana mem 20 pratishata taka pahuncha sakata hai| 
haipitaitisa i ka chikitsakiya upachara, haipitaitisa e ke chikitsakiya upachara ke hi samana hai|
echaivi ka 50% se adhika sankramana enikatorika raha sakata hai|
piliya ka prabhava badha़ti unra ke satha badha़ta hua dikhata hai| 
haipitaitisa i ki jyadatara ghatanaem javana vayaskom mem payi jati hai|
kisi roga ke utpanna hone ke daurana dusari gharelu ghatanaem asadharana haim|
kevala garbhavati mahilaom ko chhoda़kara akala nrityu ki ghatanaom ki dara usa echae ke bhi samana hai jaham siephaara bisa pratishata taka garbhavastha ke tisare traimasika ke sankramana ke bicha pahuncha sakata|
haipitaitisa ji ke vayarasa ki khoja kuchha samaya pahale hi ki ja chuki hai|
echajivi ko tivra aura jirna haipitaitisa se joda़a ja chuka hai|
echajivi ke satha vyaktigata sankramana ko bahutayata, yakrita ki bimari ka koi ki chikitsakiya pramana nahim rakhati haim|
akala nrityu ki ghatanaom ki dara, haipitaitisa aura i sankramana ke lie samanyatah kama hoti hai|
garbhavati mahilaom mem haipitaitisa vayarasa gambhira bimari ka karana ho sakata hai|
lagabhaga 20% taka siephaara garbhavati mahilaom jinamem echaivi ke karana haipitaitisa hota hai, dekha ja chuka hai|
chalisa varsha ki unra ke upara ki akala nrityu ki ghatanaom ki dara, haipitaitisa bi sankramana ke lie, aspatala mem bharti hone ki ghatanaem lagabhaga 1% jyada hai|
akala nrityu ghatanaom ki dara badha़ti hai yadi haipitaitisa bi aura di vayarasa vayarasa ke eka satha ghatita hone vale sankramana hote haim| 
garbhavati mahilaom mem haipitaitisa e aura i sankramana ke lie samanyatah kama hoti hai|
garbhavati mahilaom mem haipitaitisa vayarala gambhira bimari ka karana ho sakata hai|
lagabhaga 20% taka siephaara garbhavati mahilaom jinamem echaivi ke karana haipitaitisa hota hai, dekha ja chuka hai|
chalisa varsha ki unra se upara ki akala nrityu ki ghatanaom ki dara, haipitaitisa bi sankramana ke lie aspatala mem bharti hone ki ghatanaem lagabhaga 1% jyada haim|
akala nrityu ghatanaom ki dara badha़ti hai yadi haipitaitisa bi aura di vayarasa ke eka satha ghatita hone vale sankramana hote haim|
tivra haipitaitisa vayarala eka vishesha prakara ke paryapta chikitsakiya lakshana, joki prayah roga ke nidana mem koi karyavahi nahim vyakta karata hai|
jyadatara roga nidana ki kita ayata ki jati hai, mahangi aura ielaaiesae ridara ke lie avashyaka hai| 
kuchha upakaranom ki lagata (visheshatah haipitaitisa si, haipitaitisa di, aura haipitaitisa i ke lie) lagabhaga 30,000 hai aura pratyeka parikshana ki matraka lagata 200 se 600 rupaye taka pahuncha sakati hai|
haipitaitisa bi ke lie roganuvahakom ki dara khuna ke parikshana ke dvara, eka samanya takanika aura saste abhikarma ka prayoga karate hue nirdharita ki ja sakati hai| 
haipitaitisa ke vayarasa se kisi roga ka achanaka se phutana prayah haipitaitisa i ke karana hota hai joki dushita pani aura khane ke padarthom ke dvara phailata hai|
haipitaitisa e hone vala roga aksara hone vala nahim hai kyonki echaevi ke karana bahuta kama ya bachchom mem hone vala upa chikitsakiya sankramana aura hamare desha mem jyadatara loga bachapana mem ina vayarasa ke karana sankata mem a jate haim|
bhavishya mem hama isa prakara ke rogom ka utpanna hovana dekha sakate haim jisa prakara bahuta se loga isa sankata se bachapana mem hi ‘svachchhata’ aura svasthya mem sudhara ke karana bacha jate haim|
echaivi ke karana kisi roga ke utpanna hone ka khatara garmiyom mem aura manasuna ke bada ke samaya mem jyada hota hai| 
haipitaitisa e aura i ka sancharana vishtha hai vyaktigata svasthya aura paryavaraniya svachchhata ki napa, joki dusari antarika sankramanom ke nimantrana kisi bhi avastha ke lie adhara hai|
haipitaitisa i ke lie koi rogamukti ka tika upalabdha nahim hai| 
haipitaitisa i ke lie upalabdha koi rogamukti ke tike mahamari vijnana mem sambandhita salaha manana aura bahuta unchi dara ke karana lagata se prabhavita nahim hote|
haipitaitisa bi, si aura di vayarasa marija ya roganuvahaka ke sharira ke tarala ke dvara phailate haim| 
haipitaitisa bi, si, di aura ji ko rokane ka samanya adhyayana, suiyom aura upakaranom ke visankramana ko sammilita karata hai, joki haipitaitisa bi aura haipitaitisa si ke lakshanom ke lie tvacha ka chubhana aura khuna ki padarthom ka alaga hona aura surakshita maithuna ko badha़ava dena hai| 
rokathama ke lie lagaya jane vala tika kevala haipitaitisa bi ke viruddha upalabdha hai| 
svasthaya shiksha aura janata ka jagaruka hona aura sahayoga haipitaitisa vayarala ko rokane aura niyantrita karane ke lie mahatvapurna hai|
batae gae padartha aura svasthya shiksha sandesha, joki vyaktigata ya sthaniya prayoga ke lie apanae ja sakate haim|
bharata mem lagabhaga sabhi haipitaitisa vayarala ke rogom ka phutana haipitaitisa i ke vishtha ke pheisi oreli sancharana ke karana hai| 
haipitaitisa e ka kabhi kabhi hone vale roga ka likhita varnana kiya ja chuka hai|
yadyapi haipitaitisa vayarasa se hone vale rogom ka haipitaitisa bi ke karana badha़na aspatala, vyavastha mem pata lagaya ja chuka hai, ise samudaya vyavastha mem koi roga ke utpanna hone ka karana pramanita nahim kiya ja chuka| 
enaaisidi ne 1997 mem haipitaitisa bi se 3 rogom ka utpanna hone ki karyavahi ki ho joki aparyapta chune hue kshetrom mem ashikshita chikitsakom ke dvara aparyapta visankramita sui aura sirinja ke prayoga ke karana paye gae|
haipitaitisa i ke karana kisi roga ka utpanna hona dushita pani ki apurti ke karana hai|
roga ke utpanna hone para roka aura niyantrana hone ki mapa nishchita rupa se surakshita pani ki apurti se sidhe sambandhita honi chahie|
kisi bada़e roga ka utpanna hona dushita pani ki paipom mem apurti ke karana paya gaya|
kuchha kshetrom mem ghatanaom ka samuha sadharana pani ki apurti mem sambandhita una paintom ka pata lagane mem jaham dushita hona paya jata hai, sahayata karega|
una paintom para pani ki paipa lainom ki marammata phira se haipitaitisa i ke phailane ko rokegi|
kisi roga ke utpanna hone ke daurana, surakshita pani ki apurti ke lie adarsha rupa se vaikalpika vyavastha banani chahie| 
satha hi satha usi samaya mem, dushita hone ka sadhana prathamika taira para hata dena chahie|
yadi sthaniya pani ka sadhana aba bhi prayoga kiya jata hai, samudaya ko pani ubalane ke lie salaha deni chahie|
haipitaitisa i vayarasa sadharana khuraka mem klorina yukta pani mem nashta nahim hota, vayarasa pani ko ubala ka khatma kiya jata hai|
sabhi manushyom ka vishtha (mala-mutra) tivrata se aura surakshita dhanga se hata dena chahie|
chahe roga haipitaitisa e ke karana ya haipitaitisa i ke karana utpanna ho, usake rokathama aura niyantrana ke upaya samana hai|
yadi haipitaitisa bi se koi roga utpanna hota hai, parichaya aura khatare ko hatane ke vahaka roga ko niyantrita karenge|
haipitaitisa bi (aura yadi sambhava hai to haipitaitisa si ke lie nishchita surakshita injekshana aura sabhi rakta dana ki jaivika mahatvapurnata sabhi paitrika sancharita sankramanom (jaise – haipitaitisa bi, si, di aura ji) ko pramukharupa se rokegi| 
neshanala edsa kantrola programa maithuna ko badha़ane ke lie echaaivi sankramana rokane ke lie mapadanda़om ka prayoga karata hai| 
ye mapa bhi haipitaitisa bi ki rokathama ki ora aura dusare paitrika haipitaitisa vayarala ke sancharana mem yogadana karengi|
haipetaitasa bi ke do prakara ke tike upalabdha haim – punah sanyoga di ena e tika tatha jivadravya dvara prapta|
punah sanyoga di ena e tika tatha jivadravya dvara prapta donom tike surakshita tatha prabhavita haim|
punah sanyoga di ena e tika tatha jivadravya dvara prapta tike nirmata dvara nirdeshana ke anusara li jani chahie|
punah sanyoga di ena e tika tatha jivadravya prapta tike eka satha eka se jyada khuraka vali shishiyom mem bhi upalabdha haim|
jaba phailava shuru hota hai aura jaba isaka khatara badha़ jata hai, to dusari sarakari vibhaga, gaira-sarakari karyalaya tatha samaja ki sahayata lena avashyaka ho jata hai|
tivra atisara, taiphaida bukhara, tatha piliya ke mamale ki nigarani banae rakhana|
vayarala haipetaitasa bi mem roga lakshana prakata hone ki avadhi samanyatah 1-2 mahine hoti hai| (15 di se kama nahim)
kuchha bimariyom jinaki roga lakshana avadhi chhoti hoti hai jo chehare va munha ke raste phailati hai|
ve bimariyam tivra atisara (roga lakshana prakata hone ki avadhi kuchha dina) aura taiphaida bukhara (roga lakshana avadhi 1-3 haphte)
samanarupa se, jala pradushana se tivra atisara tatha taiphaida bukhara mem badha़otari vayarala hepetaitasa i se hoti hai|
pani ki gunavatta ke niyamita dekharekha ke lie klorina ke rasayanika kriya uparanta bache avashesha ke namune ka nirikshana tatha kolipharma jivanu ka yadakada nirikshana ki yojana|
atisara tatha taiphaida bukhara mem badha़otari tatha pani ki kharaba gunavatta ki ghatanaem ane vale vayarala hepetaitasa ki mahamari ki ora sanketa karati hai|
astitva mem rahati atisara tatha taiphaida bukhara ke mamalom mem badha़otari ko vyavasthita kiya jae tatha ane vale vayarala haipetaitasa ke phailava ko rokane ke lie sahi mapana liya jae|
vishesha taura se manasuna ke pahale, gande pani ke nale ka bahava tatha pani purti ki vyavastha mem risava para samayika sarve kie jane chahie|
jina kshetrom mem pani ruka-ruka kara ata hai vaham laina para teja vaॉlteja vale pampa paipalaina mem nakaratmaka dabava dala sakate hai jisase gande nale ka pani sapha pani ki paipalaina mem khinchane lagata hai|
chehare munha ke dvara hastantarita hone vale rogom ka khatara kama karane ke lie samaja mem ai i si gatividhiyam shuru ki jani chahie|
tivra vayarala hepetaitasa eka aisa purnataya vishishta chikitsakiya lakshana hai jo samanyatah nidanasuchaka mem koi pareshani paida nahim karata|
bhautika vibhajana una kshetrom tatha samuhom ki pahachana karana jo isa phailava se prabhavita ho, ye vibhinna kshetra una binduom ke bare mem batate haim jaham pandushana ho raha hai|
isa bata ko sunishchita kara liya jae ki hara tikakarana mem alaga kitanurahita sui ka upayoga kiya jata hai|
isa bata para jora diya jae ki sui ko punah upayoga mem lene se pahale 20 minata taka ubala gaya hai tatha silabanda sui ka upayoga kiya jata hai|
japani mastishka javara isaki mahamari vishishtata tatha adhika nrityu-dara ke mamalom ki vaja़ha se sarvajanika svasthya mahatva ka roga hai|
pahala bada़a JE phailava pashchima bangala mem bankura tatha burdavana jile mem 1973 mem hua tha taba se yaha desha ke kai rajyom/yu tija़ mem phaila chuka hai|
japani mastishka jvara eka machchhara dvara utpanna junotika roga hai|
japani mastishka jvara ka vayarala mukhya taura para janavarom tatha pakshiyom ko sankramita karata hai|
manushya, japani mastishka jvara JE vayarasa ka mahatvahina shikara hai|
isaka khatara una kshetrom mem jyada hai jaham paristhitika vijnana paristhitiyam manushyom ke hastantarana ko asana banati hai|
yadyapi je i mukhya rupa se gramina krishi vale kshetrom mem jaham rogavahaka machchhara suarom tatha dusare jantuom ke bhandara mem bada़i sankhya mem utpanna hote haim|
jei ki mahamari ki suchana shahari ilakom ke asapasa bhi mili haim jaham samana sthitiyam pai jati hai|
agara jei ke phaॅlava ka sandeha ho to turanta jila svasthya adhikari ko suchita karana chahie|
jisase pratyeka gamva mem pratyaksha mamale kama honge tatha sthaniya karmika vibhaga ko bhi apane kshetrom mem mastishka jvara ke bare mem satarka rahana chahie|
samudaya ke pramukha vyaktiyom ke madhyama se sakriya nigarani ko jyada khatare vale laghusamudayom mem protsahana diya jana chahie, udaharana ke lie vo jagaha jaham vyavasayika rupa se suarom ka samuha rakha jata hai|
japani mastishka jvara bi arabovayarasa ke samuha se hota hai|
yaha vayarasa bina janana sambandha ke dusare phulevivayarasa jisamem dengu, vesta naila tatha yallo phivara vayarasa shamila haim se sambandhita hota hai|
je i sankramana eka sankramita kyulisaina machchhara ke katane se phailata hai|
manushyom mem virammiya halka hota hai tatha thoda़i avadhi ke lie rahata hai|
manushya se manushya mem jei ke hastantarana ka koi pramana nahim mila hai|
jei vayarasa chakra ka hastantarana janavarom tatha pakshiyom mem hota hai|
jei vayarasa ka svabhavika chakra jangali tatha gharelu ridha़ ki haddi vale jivom tatha machchharom mem paya jata hai|
jei vayarasa ka svabhavika itihasa mem pani mem rahane vale pakshi jaise bagula, maveshi palatu murgiyam tatha battakhom ki pramukha bhumika haim|
ridha़ ki haddi vale jivom mem suara jei vayarasa ke pramukha shikara mane jate haim|
yadyapi sankramita suara khuleama roga ke lakshanom ko nahim dikhate parantu uname bhayankara viremiya panapane lagata hai jo bada़i matra mem suarom mem pae jate haim|
vartamana mem upalabdha pramanom ke adhara para maveshi mem jei hastantarana ke koi sanketa nahim milate haim|
ghoda़o mem sakriya roga vikasita hota hai parantu viremiya unche titara mem tatha lambe samaya ke lie durlabha milate haim|
ghoda़e, machchhara sankraman ke asambhajya sadhana haim|
manushyom mem sankramana, janavarom mem mukhyatah suarom ke satha najadika rahane ke satha joda़kara dekha jata hai|
kyulaiksa vishnui samuha se juda़e machbharata mem pramukha rogavahaka nasla haim|
machchharom ki gyaraha aura raslom ko jei ke rogavahaka hone ke lie doshi thaharaye gaye haim|
nyuleksa machchhara samanyatah saghana vanaspati ke satha jala jivom mem paida hote haim|
sinchita chavala ke kheta, khali gadde tatha pokhara kyuleksa machchhara ke paida hone ke ama sthala haim|
kyuleksa machchhara juphilika hote haim, tatha janavarom tatha jangali pakshiyom ko katate haim|
kyuleksa machchhara gharom ke bahara vanaspati tatha dusare chhaya vale sthanom para rahate haim| parantu garmiyom mem gharom ke andara bhi paye jate ja sakate haim| 
machchhara andara tatha bahara poshita hote haim|
mahamariyam samanyatah manasuna ke satha tatha bada ki avadhi mem ghatita hoti haim jaba rogavahaka ghanatva uchchatama hota hai|
sthaniya kshetrom mem, aniyamita mamale salabhara ghatita hote rahate haim|
mada kyuleksa machchhara viremika vayarasa ko ashraya dene vale se sankramita ho jate hai|
mada kyuleksa machchhara 9 se 12 dinom ke bahya roga lakshana prakata ki avadhi ke bada vayarasa ko dusare ashraya dataom mem hastantarita kara sakata hai|
kyuleksa machchhara lambi duri taka uda़ sakate haim| (1-3 kimi. ya usase bhi jyada)
kyuleksa machchhara unrabhara sankramita rahate haim tatha eka machchhara ki ausata jivana avadhi lagabhaga 21 dina hoti haim|
kyuleksa machchhara ke katane para manushyom mem roga lakshana prakata hone ki avadhi sambhavatah 5 se 15 dinom mem bhinna-bhinna hoti hai|
jei ki chikitsakiya visheshataem mastishka jvara ki hoti haim|
rogi bukhara ke satha tivra shuruata tatha vyavahara mem badalava evam samvedanashilata jo ki 24 ghante se jyada dera taka bana rahata hai ke itihasa ko dikhate haim|
samvedanashilata mem gada़bada़, alasa, nindra, chida़chida़apana, udasinata tatha achetana ke rupa mem dikhai deti haim|
rogi bolane mem kathinai ke satha dusare nada़i sambandhi kamiyom jaise ankha mem lakava haimiplejiya, kampanna tatha mansapeshiyom ka kama karana banda kara dena jaisi samasyaem vikasita ho sakati haim|
pi. phailsipairama maleriya ki pareshaniyom mem se ema mastishka shodha haim|
kyonki maleriya bhi samanyatah sambandhita hai to maleriya ka roganirdharana bhi kiya jana chahie|
agara jancha pada़tala maleriya janita sthiti ki ora sanketa kara, bimari chikitsakiya drishti se sangata kara rahi hai tatha maleriya virodhi ilaja prabhavashali hai to maleriya ka sambhavita bhatakava mem le raha hai|
rakta slaida ke taiyari tatha parikshana ki suvidhaem pratyeka pi echa si ko upalabdha karavai jani chahie|
aiphaida bukhara eka dusara pramukha sthayi bimari hai jo japani mastishka jvara se milate julate lakshanom ko badha़ava deti hai|
avashyakata pada़ne para esa taiphi ke lie savadhani purvaka itihasa, sharirika tatha rakta jivanu samuha ke nirikshana ko taiphaida bukhara ko alaga karana chahie|
mahamari tatha kita vijnana sambandhi jancha pada़tala jei ke roga anumanita roga nirdharana mem labhadaya hai|
jei ka roga nirdharana ki pushti rogavahaka tatha use badha़ane vale ashrayadata jaise khataranaka tatva ki upasthiti karate haim|
japani mastishka jvara ka roganirdharana prayogashala parikshana dvara sunishchita kiya gaya hai|
gambhira mamalom mem nrityudara uchcha hoti hai| isamem 20 se 40 pratishata ki suchana mili hai|
jo rogi tivra ghatanakrama se ubara jate haim unamem nada़i sambandhi parinama dikhai de sakate haim|
je i roga nirdharana ki sirama vijnana parikshana dvara pushti ki gai hai|
isa parikshana mem ai ji ema pratijana tatvanashaka ka pata lagaya jata hai jo lakshanom ke shuruvati pahale saptaha mem dikhai deti hai tatha tivra ghatanakrama ke eka se tina mahine bada pata lagane layaka ho jate haim|
10 ya jyada dina ke antarala para juda़va sirama mem lie gae ai ji ji pratijanatatvanashaka taitara mem chara guna badha़ottari nishchita hai|
parikshana ayojana hetu entigama vyavasayika vyavasayika rupa se abhi upalabdha nahim hai|
roga nirdharana hetu si esa epha se vayarasa ke alaga karane ka kama niyamita rupa se nahim kiya jata hai|
rakta se vayarasa ki prathakata lagabhaga asambhava kyonki viremiya halka aura thoda़e samaya ke lie hota hai| 
sirama vijnana adhyayana hetu rakta sanchaya karate samaya vishavyapaka purvavadhana ko dhyana mem rakhana chahie|
japani mastishka ka koi upayukta ilaja nahim hai|
sahayatapurna ilaja tatha achchhi dekhabhala pramukhatah jei ki nrityu dara ke ghataka stara mamalom ko kama kiya ja sakata hai|
isalie yaha mahatvapurna hai ki jei ke mamalem jalda se jalda aspatala mem bheja diya jata hai agara mastishka jvara ka sandeha ho|
sirama vijnana prayogashala ke parinama ane se purva hi ilaja shuru kara dena chahie|
isa bata para jora diya jata hai ki jei ke karana hui nrityudara ko kama karane ke mamalom ke lie chikitsakiya sahi vyavastha hi eka rasta hai|
isa bata ka kai pramana nahim hai ki gaimaiglobulinasa tatha koratikostaraॉyada ke prabhava ke prabhava labhadayaka haim|
nigarani vyavastha ko sthapita karana chahie taki mastishka jvara ke kisi bhi mamale ke bare mem sthaniya svasthya adhikari ko turanta suchita kie jae|
rogavahaka tatha badha़ane vala ashrayadata ka pata lagane ke lie rogi ke ghara ke ilake ko janchane ke lie avashyaka karyashretra jancha pada़tala shuru karani chahie|
rogavahaka ghanatva ko kama karane ke lie tatha vyaktigata suraksha hetu machchhara katane ke rokane ke lie nivaraka mapana nirdeshita kie gae haim|
kyonki sankramana ko badha़ava dene vale ashrayadata jo ki suara haim ve khuleama isa bimari ke chinha darshate haim isalie sankramita janavarom ko pahachanana sambhava nahim hai to isaka prithakkarana tatha vinasha nahim ho pata|
isalie kitanunashakom ke chhida़kava dvara machchharom ke ghanatva ko kama karane ke lie turanta mapaka ki manga ki jati hai|
isalie janata ko machchhara katane ke khilapha savadhani baratane ke lie puri banha ke kapada़e pahanane, machchharadani ka upayoga tatha machchhara bhagane vali krima ka upayoga upayoga karane jaisi avashyaka suchana deni chahie|
lambe samayavadhi mapanom mem neshanala maleriya iredikeshana programa ke tahata rogavahaka niyantrana hetu die nirdesha shamila haim|
agara asapasa ke kshetrom mem svasthya suvidhaem upalabdha nahim ho to rogi ko jila aspatala mem bheja dena chahie|
jyada khatare vale laghusamudayom ki pahachana karani chahie taki niyantrana mapana svasthya shiksha gatividhiyom va karyakshetrom nirikshana ke adhara para ina kshetrom para jyada dhyana diya ja sake|
aise kshetrom mem gramina tatha shahara ke asapasa ke kshetra jinamem suara palana kiya jata hai aura vishesha rupa se agara ye dhana khetom ke naja़dika hom jo kyuleksa machchhara ke paida hone ke pramukha sthana hai, shamila haim| 
rogiyom ka prithakkarana tatha malotsarjana va sharira se ise alaga karane ka asanka mila karane ki avashyakata nahim hoti kyonki jui vayarasa eka isanna se su dusare insana mem hastantarita nahim hota|
eka kilda jei tika saitrala risarcha insatityuta kasauli dvara dudhamuhe chuhe ke mastishka mem nakayama jei strena mem bharakara banaya jata hai|
manavi tikakarana JE ke khilapha, akela hi lambe stara para JE ko rokane ke lie upalabdha haim|
bharata mem bahuta se sankramaka roga sthaniya hai|
sandigdha vyakti ki dekha rekha karane vala eka prabhavi tarika avashyaka hai ya bimari ko niyantrita karane vale karyakrama ki yojana, lagu karana ya chetavani dena|
bahuta si bimariyam mausami ya chakriya pravriti rakhati haim jo ki dekha-rekha karane ke tarikom ke dvara samaji ja sakati haim|
bimariyam bahuta teji se phailane va sambhavya samarthya ke satha phutane ka karana bhi ho sakati haim aura bahuta si nrityu ka karana bhi|
nai bimariyom ka phutana aura dubara se ubharane vala sankramana bhi ho sakata hai|
yakayaka phutane vali bimari ka purvanumana hone para savadhani ke sadhana lena, tivrata se hone vale janata ke akasmika svasthya sankata ko roka sakata hai aura jivana bacha sakata hai|
jaba achanaka kisi bimari ke phutane ka pahale se na to pata chale aura na hi rokathama ki ja sake, taba chetavani ke chinhom ki pahachana, samaya se jancha aura vishesha niyantraka sadhanom ka upayoga bimari ki tivrata ko aura nrityu ko roka sakata hai|
niyantrana karane vale sadhana bahuta prabhavashali hote haim jaba chune hue vyavadhana (hastakshepa) pahale se prayoga kiye jaem|
achanaka bimari ke phutane ki jancha karane ka mukhya addeshya,bimari ko niyantrita karana, isaki tivrata ko dusare kshetrom mem phailane se kama karana aura ankana ki kaise rokathama ki rananiti bhavishya mem aise bimariyom ke khatare ko kama karane va dura karane ke lie maja़buta ki ja sakati hai|
chikitsa va svasthya se sambandhita karmika vibhaga ko kita vijnana se sambandhita nirdharita sima, vibhinna prakara ke lakshanom ko dekhakara roga nirnaya prayogashala mem pushti karana, vishesha niyantrana karane vale vyavadhana aura shuru mem hi chetavani ke sanketom ki pahachana ko shamila karate hue yakayaka bimari ke phutane ke jancha-pada़tala ke siddhantom mem prashikshita hona chahie|
kuchha prayogashala parikshana pi.echa.si. stara para kiye jane ki asha ki jati hai, dusare kevala jile (kshetra) stara para hi upalabdha ho sakate haim|
kuchha prayogashala parikshana, namunom ke lie pradesha kshetriya va rashtriya stara para bheje jane ki avashyakata ho sakati hai|
parikshana ke satha, teliphona nambara, phaiksa na. purapata likha prayogashala ki eka lista usake rakha-rakhava ke lie lagane ka sujava diya jata hai|
naॉdala aphisara ko prayogashala ki naukari mem pratyeka stara para antara (darara) ko pahachanana chahie jo ki diye hue sadhanom se bhare ja sakate haim|
chikitsiya namunom, ke thika sangraha, unake bhandarana parivahana para vishesha mahatva diya jana chahie|
eka prabhavashali nirikshana vyavastha pratyeka jile mem sthapita honi chahie|
ऑkada़om ki niyamitata se jancha honi chahie jaisa ki prithaka mapa ‘‘saravelansa ऑpha aipedemika prona dija़ija़sa ’’ mem sujava diya gaya hai jo ki neshanala instityuta ऑpha kanyunikevila dija़ija़ja mem upalabdha hai|
mahatvapurna hai ki sandigdha vyakti ki nigarani ke ankada़o ki prabhavi va teja jancha ke lie jile ke stara paryapta suvidhaem sthapita honi chahie|
naॉdala ऑphisara aura dusare mukhya karmika vibhaga ko kampyutarsa ke payoga mem prashikshana lena chahie|
bahuta si bimariyom se sambandhita suchi ki jancha va rakha-rakhava mem i pi ai va inapho jaise saॉphtaveyara vishesha rupa se upayogi haim|
isa prakara ke parikshana, bada़e khatare ke samuha kshetra aura tathya se sambandhita mulyavana mahamari vijnana se sambandhita suchanaem pradana karate haim|
kisi eka bimarike achanaka phutane ki ghatana mem, rajya ke naॉdala ऑphisara ko turanta isaki ghoshana (suchana dena) karana chahie|
asamanya bimari ke achanaka phutane ki ghatana mem ya lakshanom ko dekhakara roga ka nirnaya na kara pane mem distarikta ऑphisara ko kuchha vishesha takaniki va dusare samarthana (sujavom) ki avashyakata ho sakati hai|
jaba se neshanala instityuta ऑpha kanyunikevala dijijesa (ena ai si di) eka rashtriya stara ka naॉdala ऑphisa hai, yaha asha ki jati hai ki yakayaka bimari ke phatane para turanta ena ai sidi ko sanketika rupa se yadi kiyi takaniki sujava ki avashyakata hai, suchana di jayegi|
rashtriya bimari ki dekharekha ke karyakrama ke tahata yaha asha ki jati hai ki ena ai si di ko i mela va phaiksa ke dvara jile va pradesha stara para joda़a jayega|
teliphona suvidhaem pi echa si stara para asha ki jati haim|
bimariyom ke achanaka phutane ki rokathama va niyantrana ke lie samudaya ka thosa va sakriya sahayoga avashyaka hai|
amudayika stara para ai i si ki sakriyata ka samarthana hona chahie taki bimariyom ke niyantrana va rokathama ke vishaya mem kuchha mukhya sandesha jane jate haim|
svasthya shiksha samagri jo ki pahale se hi taiyara ki gai hai aura kshetriya parikshana yadi kisi kshetra mem achanaka bimari phuta pada़e taba yaha bahuta upayogi honge kyonki isa prakara ke samagri chhoti ghoshana para bhi avashyaka ho sakati hai|
chikitsa va svasthya karmika vibhaga ko samudaya netaom va dusare mukhya karmika vibhagom unake kshetrom mem sambandha sthapita karane chahie joki jaldi se chetavani ke sanketa prapta karane mem aura bimari ke daurana vaishya samudayom ke samarthana mem upayogi honge|
jila stara para naॉdala ऑphisara ko sambandhita karyakrama ऑphisara ke satha paramarsha apurti gauna svasthya suvidhaom mem hai aura jila stara para paryapta matra mem upabhokta vastuom mem bhari mulya antara ko rokane ke lie bhandara ikaththe kiye jate haim|
ghatanaom ke ashavana mausami badha़ta se pahale suchiyam janchi jani chahie|
kuchha jivana bachane vali davaiyom jaise o ara esa paiketa gamva starapara rakhe ja sakate haim vishesha rupa se unchem khatarom ki jebom mem aura kshetrom mem jaham ki manasuna ke daurana pahanchana asambhava ho sakata hai|
yaha dhyana kiya jana chahie ki achanaka bimari ke daurana adhika pratishata mem bahuta si gambhira ghatanaem ho sakati haim|
gambhira bimari ke khatarom ke sanketa ke bare mem samudaya mem simita jnana ya chhote bachchom mem bahuta adhika kuposhana ki dara jaise kuchha nishchita tathyom ke karana akala nrityu ki dara badha़ sakati hai|
purva savadhani baratane vale sadhanom ko nishchaya va liye jane ki avashyakata hai jaba samaya se chikitsiya sahayata ki khoja ke lie|
jaba kabhi nyumoniya ke sanketa va lakshana prakata ho, taba ghatana ke sahi upachara ke lie mata ko svasthya suvidhaom para paramarsha lena chahie|
yaha kevala tabhi sambhava hogi yadi mataem nyumoniya ke sanketa va lakshanom ke bare mem jane udaharanartha badha़ta shvasa sambandhi dara aura sina|
jaba eka bimari ki achanaka phutati hai ya taba ina bimariyom ka khatara bhi uncha hai, taba dusare sarakari vibhagom ka asarakari ejensiyom ka aura samudayom ka yogadana prayah avashyaka hota hai|
isa prakara ki sahayata adhika agami hogi yadi bimari ke achanaka akramana se pahale hi takanikem parasparika vartalapa ke lie vikasita ki jame|
bimariyom ki ashatita mausami vriddhi se pahale, samudayom ke paratinidhiyom, ena ji osa, aura sambandhita vibhagom se eka sabha ekatrita karana upayogi ho sakata hai|
kuchha takanikem adesha dene ke lie bulai gai sabhaom ke lie sthapita honi chahie|
bimariyom ko jada़ se ukhada़ne vale va dura karane vale uddeshya nishchita kiye gaye haim|
isa prakara ke bimariyom ki koi akeli ghatana eka achanaka phutane ke rupa mem mani jani chahie jaise poliyo, charmaroga, phaileriya|
spashta rupa se adhika matra mem svasthya kharaba hone ki ghatanaom ke samudaya mem phailane para eka bimari ka yakayaka akramana va mahamari ko paribhashita kiya jata hai|
jabaki eka achanaka bimari ka akramana prayah chhote kshetra taka hi simita hota hai aura mahamari eka bada़e bhaugolika kshetra ko ghera leti hai aura eka se adhika kendra bindu rakhati hai|
bahuta si ghatanaom ko achanaka bimari ka phutana kahe jane ki avashyakata hai jo ki anekom tathyom ke anusara badalata hai|
kyonki kuchha bimariyam eka akele ghatana mem eka bimari ka visphota ka karana banati hai| (tivra poliyo, vaijnanika parikshana ka kida़a, ajnata roga ka karana janane ki bimari ki asvabhavika, tivra gambhira kada़i) 
pradeshom aura jilom ko bahuta si ghatanaom para eka mapadanda sthapita karana chahie jo ki sthaniya sthitiyom para eka mahamari ka adhara banae|
eka pradesha mem achanaka bimari ka phuta padha़ne ka nirnana karane ke lie roga ke lakshana prakata hone ki avadhi ke bhitara hi tivra akramana ke eka jaisi pancha ghatanaem ya eka gamva mem eka nrityu ko manaka ke rupa mem prayoga kiya ja raha hai|
ghatanaom ki puri sankhyaom mem badha़ta kisi bhi prakara se bimari ke vistara mem badha़ta ki ora sanketa nahim karati|
suchana dene vali saitsa ki sankhya mem bhinnata, suchanaom ki paripurnata, vaha sthana jaham se chhatra skula mem jate haim usaka bhaugolika akara aura janasankhya ka akara aise karaka haim jo ki suchanaom ki jancha-pada़tala karate samaya dhyana mem rakhane chahie|
chetavani ke sanketa ki eka chhoti suchi rakhana upayogi hota hai jisase ki jancha-pada़tala ka silasila shuru karana chahie|
eka sandigdha ki nigarani karane ki yojana ka mukhya uddeshya bimari ke yakayaka phutane ko rokana ya shuru ke stara para unaka pata lagana aura isalie svasthya karmika vibhaga ko chetavani ke sanketom ke lie attaradayi hona chahie|
bimari ke visphota ki jancha-pada़tala ka pahala kadama bahuta si suchita ghatanaom ke sambhava hone para lakshanom ko dekhakara roga ka nirnaya nishchita karana hai|
bimari ka galata nidana hone ke karana bahuta sa samaya va prayasa bekara ho sakate haim|
eka medikala ऑphisara ke dvara, lakshanom ko dekhakara roga ka nirnaya nishchita karane ke lie varnita ghatanaom ki jancha-pada़tala ki jani chahie|
ghatanaom ki adhikata se ghatana ki manaka vyakhya ke bicha mem girane ki asha ki jati hai|
sandeha ki sthitiyom mem chahe svasthya kharaba hona ghatana ki paribhasha se milata hai, eka dusara vichara khoja ja sakata hai|
bimari ka achanaka phutane ki prathama suchana naidanika lakshano para adharita hogi aura lakshanom ko dekhakara roga ka pata lagana sambhavita hoga|
naidanika lakshanom va bimari ka yakayaka visphota jo ki adhika sadharanatayah varnita hai ke lie sujava banaye gaye haim|
pahale sadharana karanom ko bahara nikalane ke lie, prayogashala, mahamari vijnana se sammbandhita, kita vijnana se sambandhita kuchha jancha-pada़tala ki jani chahie|
naidanika lakshanom va mahamari vijnana se sambandhita nirdharita simaem ke vishaya mem purna jnana bimarike yakayaka visphota ki jancha-pada़tala ke lie mahatvapurna hai|
naidanika-lakshana lekhakara roga pahachanane ki ghatanaom ki prayogashala nishchitata ya roga ke karana janane ka adhyayana karane vala abhikarta ki pahachana kabhi-kabhi avashyaka ho sakati hai|
aisi sthiti ke tahata, kuchha chuni hui ghatanaom ki prayogashala nishchitata ya roga ke karana janane ka adhyayana karane vala abhikarta ki pahachana kabhi-kabhi avashyaka ho sakati hai|
sari ghatanaom se namune ekatrita karana avashyaka nahim hai jaise ki niyantrana mapa aura bimari ke udvega ki jancha-pada़tala ko prabhava (parinama) ki avashyakata nahim haim|
anavashyaka namunom ka sangraha ko hatotsahita kiya jata hai jaise ye prayogashala para bhari vaja़na lete haim aura kuchha parikshana bahuta kharchile hote haim|
yadi bimari ka udveva asadharana hai, ghatana akala nrityu dara bahuta unchi hai, roga ke karana bhi nishchita nahim ho sakate ya yadi naidanika lakshana usa kshetra mem pahile varnita na kiye gaye ho, steta ऑphisara aura ena ai si di se jile ke adhikariyom ko jancha pada़tala mem sahayata lane ki asha ki jayegi|
prabhavashali jancha-pada़tala aura bimari ke udvega ko niyantrita karane ke lie, bimariyom ke naidanika aura mahamari vijnana se sambandhita lakshanom ko janana bahuta avashyaka hai|
asana ullekha ke lie mahamari vijnana se sambandhita nirdharita sima, suchi ki taraha likhe hue krama mem honi chahie|
kuchha mahatvapurna aura sadharana naidanika lakshanom ke antaratmaka, lakshanom ko dekhakara roga kana nirnaya karana, isa lekha patra mem diya gaya hai|
medikala aphisarsa ko suchi ko bada़a karane aura dusari dashaem jo unake kshetrom mem vyapaka haim shamila karane ke lie utsahita kiya jata hai|
bimari ke udvega ko bahuta achchhe avasara ki taraha dekha jana chahie ki ve kyom hoti hai isa jancha ke lie, jyada khatarom vale kshetra va samuha ke parichaya va niyantrana mapa ke mulyankana ke lie|
sabhi bimariyom ke udvega satha hi satha dhamakane vali bimariyom ke yakayaka visphota ki jancha ke lie prayasa kiye jane chahie|
kisi eka bimari ke udvega ke nishchita ho jane ke bada, sancharana ki riti aura bimari ke udvega ke karanom para vishishta hastakshepa (vyavadhana) nirbhara honge|
samaya se upachara milana aura ghatana va samparka ke samipa hona nrityu ko kama karane ke lie mahatvapurna hai|
bimari ke udvega ke astitva mem ane ke bada aura lakshanom ko dekhakara roga ko pramanita karane ke bada, ghatanaom ki bilkula thika vyakhya karana va ginana bahuta mahatvapurna haim|
bimari ke yakayaka udvega ke samaya ke daurana bimari ki sari ghatanaem, vichara-vimarsha ke tahata, jina kshetrom mem hoti haim unako parichaya va suchi baddha hona chahie|
sandigdha vyaktiyom para sakriya nigarani karana, ghatanaom ke lie sakriya khoja hai|
mukhya samudayika pratinidhi ena ji oja, gona svasthya karmika vibhaga aura dusare sarakari vibhagom se karmika vibhagom ke dvara, sandigdha vyakti para sakriya nigarani eka adhika suchana ghatanaom ke bare mem pradana karati hai joki sarakari svasthya suvidhaom mem nahim dekhi ja sakati|
mukhya samudaya ke pratinidhiyom se samparka ke dvara mulyavana suchanaem prapta ki ja sakati hai vishesha rupa se yadi bimari ka yakayaka udvega kendriya hai|
sandigdha para sakriya nigarani ko taba taka jari rakhana chahie jaba taka bimari ka udvega pura na ho jae| (antima ghatana ke bada jaba roga ke lakshana prakata hone ki dohari avadhi vyatita ho jae prayah yaha taba hota hai|)
bimari aura bimari ke udvega ki jancha pada़tala karane ke lie jo sadhana upalabdha haim, una para nirbhara karate hue, ghara-ghara jakara bhenta karane ke lie vishesha rupa se ghatanaom ke samparka ke gharom para jana adhika anukula hai|
sandehaspada ghatana ki vyakhya, ghatanaom ki pahachana ke lie prayukta ki ja sakati hai|
kuchha paristhitiyom mem, ghara-ghara jakara bhenta karane ke lie samudaya ki sahayatako suchibaddha kiya ja sakata hai|
rashtriya karyakrama ki gunavatta ko ankane ke lie (yadi tarka sangata hai) bhi suchanaem avashyaka hai|
eka uttama adarsha tarike mem, sandigdha vyakti ki nigarani se suchanaem likhita varnita ho, yaha mahatvapurna hai|
jo loga bimara haim aura kesa dephinishana se milate haim kyonki bimari ke lie visphota ko, jo ki ‘‘lainalistinga’’ kaha jata hai ke upara pravesha karana chahie|
bimari ke visphota ki jancha pada़tala karate hue bimari ke visphota ka samaya, sthana aura vyakti ke sambandha mem vistrita varnana pradana karana avashyaka hai|
bimari ke prarambha hone para isako ghantom ke dvara suchitrita karana chahie (udaharanartha, phuda paesaninga) dinom, saptaha ya mahinom jaise bhi uchita ho|
mahamari ka ghumava ghatana aura ghatanaom ki suchi ko parichita karane mem sahayaka ho sakata hai aura sancharana ke dhangom ko aura namunom ke lie sujava de sakata hai|
bimari ke visphota ke daurana, ghatanaom ka samaya se parikshana, mahamari ke jukava ke pramana patra ke lie au bimari ke vistara ko rokane ki mapa ki prabhavashilata ko chetavani ke lie sahayata karega|
kshetriya bimariyom ke kesa mem, rekhachitrana para pahale ankada़om ko prastuta karana bhi upayogi hai|
yaha rekhachitrana bimari ke yakayaka udvega ke vistara jo ki purva ke suchanabaddha ghatanao se tulana kiye jate haim, bimari ke phailane ki tivrata aura niyantrana ke prayasom ke mulyankana ko pradarshita karane mem sahayaka hote haim|
kshetra ka eka naksha ya eka bhatti akriti jaham pratyeka suchanabaddha ghatana ghatanaom se tulana kiye jate haim, bimari ke phailane ki tivrata aura niyantrana ke prayaso ke mulyankana ko pradarshita karane mem sahayaka hote haim|
kuchha dashaom mem kramabaddha turanta taiyara kiye nakshe, haphte aura mahine mem (ya bimari ki pidha़i ke dvara) bimari ke phailane ke namunom ko janchane ke lie antadrirshti pradana kara sakata hai|
ghatanaom ka samuha ke lie jukava aura nivasiya sthana nirdharana ke juda़ava mem nakshe para prabhavita skulom va dusari sansthaom ko ankita karane ke lie upayogi ho sakata hai|
isa prakara ke nakshom ka banana chhuta ke sadhanom ko pahachanane mem sahayata kara sakata hai|
ye ghatanaem ayu, seksa, vyavasaya, samajika arthika nirdharita sima pravasana, chechaka ke tike ka itihasa aura dusari tarkasangata charitrika visheshataom ke sambandha mem varnita honi chahie|
suchi banate hue, janasankhya ki charitrika visheshataem udaharana ke lie ayu aura seksa se sambandhita, sahi prakara ke samuha mem honi chahie|
paryavarana se sambandhita dashaom, isaka janasankhya aura karyotpadaka karaka ke satha parasparika prabhava ka gativijnana ka shodha, mahamari ke utpatti grantha para upakalpana ko banane mem sahayata karegi jo ki kauna se niyantraka manaka apanane chahie isake lie adhara apanaega| 
mausama vijnana se sambandhita kendrom, jilom mem prakritika vatavarana ke satha, barasata, ardrata, tapamana ke ankada़e upalabdha haim| 
prakritika apadaom jaise badha़, tuphana, sukha, bhukampa adi ki suchana jaise ki upalabdha hai vaha mahamari ke varnana ke lie prayoga ki ja sakati hai| 
manava nirmita sthiti jaise ki sinchai va udyogom para vikasatmaka yojanaem paryavarana se sambandhita dashaem utpanna kara sakati haim joki bimari ke sancharana ke lie samvahaka hai| 
isi prakara pine ke pani ki apurti aura paryavarana ki saphai se sambandhita suchana jancha pada़tala ke lie kathina hai|
rogamukti arakshana stara ka dridhanishchaya jatila hai yadi udvega ka karana vaiksina se roke jane vali bimari hai|
samudaya se roga mukti ke stara ka mulyankana rogamukti abhivyakti ke lekha jokha joki piesapi para upalabdha hai, ke dvara kiya ja sakata hai|
pratishata rogamukti vistara ka, khurakom ke pure yoga ko vibhajita karake (khasara aura bisiji) ya vaiksina ki tisari khurakom (opivi aura dipiti) ka prabandha eka vishesha ayu samuha ke lie kiya gaya hai, jo prabhavita gramina ya shahari kshetro mem puri janasankhya ke ayu samuha dvara anumana lagaya jata hai| 
rokathama ke lie tika lagavaye hue eka jaisi unra ke samuha ka mulyankana, bada़e bachchom ke samavesha ke satha puri taraha se kiya jana chahie joki unche arakshana ka jutha artha de sakata hai|
rogamukti ki gatividhiyam, rogamukti satra ki niyatakalikata ko shamila karate hue, tike ki prapta matra aura kolda chena sistama ka purnanirikshana hona chahie|
rogamukti arakshana para suchana, purane tikakarana arakshana mulyankana sarvekshanom ke dvara upalabdha karayi ja sakati hai| 
rogamukti stara eka mukhya vivaranatmaka satha hi satha vishleshanatmaka nirdharita sima hai|
marija ke dvara li gai tike ki khuraka ki matra ko nishchita karane ke lie pratyeka ghatana ke rogamukti stara ko savadhani purvaka jancha pada़tala honi chahie| 
rogamukti stara ko pramanita karane ke lie rogamukti kardasa ya rogamukti rajistara ko janchana chahie|
yadi aise lekhe jokhe prapta nahim kiye ja sakate haim taba kevala maukhika itihasa prayoga kiya jana chahie|
adarsharupa se kolda chena ke guna ko ankane ke lie rogamukti ke sthana ka parikshana kiya jana chahie| 
bimari ke udvega ki jancha pada़tala ki likhita vivarana mem, prayogashala jancha pada़tala ke parinamom ko shamila kiya jana chahie|
kuchha parikshanom ke parinama sthaniya rupa se aura teji se prapta kiye ja sakate haim|
yadi pani se honevali bimariyom ka sandeha kiya ja sakata hai taba isa prakara ke parikshana pani ki gunavatta ko chetavani deti hai|
niyantraka gatividhiyam, marijom ka upachara, nidana ki lambita prayogashala nishchayatmaka ko dera nahim ki jani chahie|
karya shuru kiya jana chahie joki naidanika mahamari vijnana se sambandhita aura kita vijnana se sambandhita nishkarshom para adharita hom|
uchita sangraha, parivahana para nama patti ankita karana aura naidanika namunom ke bhandara para vishesha jora diya jana chahie|
prarambhika riporta suchana ko jama karate samaya prayogashala parikshanom ke parinamom ke lie intajara karana avashyaka nahim hai, joki jile ke bahara ki prayogashalaom ko bheje ja chuke haim aura joki kuchha samaya le sakate haim|
parinamom ko antima vivarana mem joda़a ja sakata hai|
jancha pada़tala ki avadhi ke daurana, sarvabhaumika sankramana niyantraka savadhaniyam lagu kiye jane ki asha ki jati hai| 
raktastravi bukhara ke yakayaka visphota ki jancha padatala karate samaya ya sancharana ka antrettara marga sandeha kiya jata hai, surakshatmaka aujara jaise dastane, nakaba, adi svasthya vibhaga dvara marijom ki jancha ya upachara karate samaya pahanane ki asha ki jati hai| 
rakta aura dusare naidanika namunom ke sangraha ke lie jo bhi prakriya avashyaka ho, savadhani purvaka purna ki jani chahie|
kitavijnana se sambandhita jancha pada़tala kshetriya jancha pada़tala ka eka mahatvapurna pahalu hai jaise ki bahuta adhika bimariyom ka yakayaka udvega rogavahaka ke karana utpanna hota hai| 
kita vijnana se sambandhita jancha pada़tala bimari ke achanaka udvega ka sambhavita karana ko nishchaya karane ya bahara nikalane ke lie upayogi hai, vishesharupa se yadi prayogashala parikshana ke parinama upalabdha nahim haim| 
adhika rogavahaka ghanapana, asanntoshajanaka kshetriya karyanviti ki chhaya aura rogavahaka niyantrana mapakom ko chetavani hai| 
rogavahaka ghanapana adhika hona eka chetavani ka sanketa hai jaise rogavahaka se bimari ka yakayaka visphota ka khatara inhi dashaom mem badha़ta hai| 
dusari tarapha, khatara kama hai yadi jatila stara ke niche ghanepana ko thika kara liya jata hai| 
yadi dengu bukhara ya dengu raktastrava jvara ki shanka ho ya nishchita ho gaya ho, taba bahuta adhika prathamikata ke satha rogavahaka ki nigani rakhana aura rogavahaka niyantrana gatividhiyom ko shuru kiya jana chahie| 
dusare bahuta adhika khatare vale sthanom mem rogavahaka ki nigarani turanta ki jani chahie yadi kshetra se koi bhi aisi ghatana suchita na ki gai ho|
ayu samuha ke sambandha mem bimari ke adhika khatara hone para janasankhya samuha, bhaugolika sthiti sakriyata rogamukti stara, aura dusari visheshataom adi ko paribhashita karane ke lie varnanatmaka kita vijnana sahayata karega| 
thika sankhya ke bajaya, akramana dara ko nishchita karana adhika sangata hai kyonki vibhinna samuhom ke janasankhya akara mem parivartanom ko dara dhyana mem rakhati hai|
isa prakara ki darom ki, janasankhya samuha mem ghatanaom ki sankhya ke vibhajaka dvara aura usi samuha ke janasankhya ke akara dvara samanyatah ganana ki jati hai|
anuvarti bhramana kika avadhi va avritti bahuta se tathyom para nirbhara karegi aura sthaniya stara para nirnaya lie jayengem| 
vaha tathya jo nirnaya ko prabhavita karenge, ve honge bimari ki gambhirata, dusare kshetrom mem isaka teji se phailana, adhika se adhika roga ke lakshana prakata hone ki avadhi, pratidainika svasthya sevao taka prabhavita kshetra ki sulabhata|
bimari ke yakayaka visphota ki jancha pada़tala mem, mahamari ka pralekhana eka mahatvapurna kadama hai|
yadi pralekhana pura hai aura ankada़e puri taraha se vishleshita haim, taba upayogi sabaka sikhe ja sakate haim|
bhavishya mem bimari ke udvega ke khatare ko kama karane ke lie lambi rananiti banane ke lie aura yadi vaha bimari hoti hai to inake prabhavashali sanchalana ke lie vaha suchana bahuta upayogi hogi|
samachara patra aura takaniki jarnala mem riporta ka prakashana, suchana ki vistrita pahuncha ko nishchita karega|
antarvibhagiya samiti ke sadasyom aura teji se pratikriya dene vali chima ke sadasyom ke satha bimari ke yakayaka udvega ki mukhya prapti para bhi vichara vimarsha kiya jana chahie|
svasthaya shiksha aura janata ki jagarukata ya sahayoga, eka bimari ke udvega ko niyantrita karane ke lie mahatvapurna hai| 
yadi samudaya janata hai ki kaise bimariyom ke visphota ka phailava hota hai aura kya manadanda ve apane parivarom mem le sakate haim, jisase khatara yatheshtha rupa se kama kiya ja sake|
yadi suchana upalabdha hai to halachala ya ghabarane ki sambhavyata kama hoti hai aura samudaya ka samarthana bhi sulabha hoga|
jabaki mukhya sandesha avashyaka rupase sabhi kshetrom ke lie eka jaisa rahega, bhasha aura shaili sthaniya avashyakataom ke lie anukula banane ki avashyakata hai| 
yaha sujava diya jata hai ki svasthya shiksha samagri ki purva taiyari ke lie karya aura mukhya sandesha mahamari pranata bimariyom ke lie liye jate haim|
vishesha bimariyom para likhita pramana mem mukhya sandeshom ko vikasita karane ke lie kuchha sujava diye jate haim|
janata ko yaha suchita karana vishesha rupa se mahatvapurna hainki mahamari pranata bimariyom ki adhikansha ghatanaem vyavasthita ki ja sakati hai yadi prarambhika avastha se hi upachara shuru kiya jae|
yaha bhi mahatvapurna hai ki prabandhaka aura janata anuchita mapakom se jagaruka ho taki adhikansha samaya aura sadhana manadandom ke lie dusari disha mem na moda़em jae joki aprabhavi hai| 
bhandara kie hue pani va bhojana ka surakshita prayoga ke lie samudaya ki bhagidari bimari ke yakayaka visphota ko rokane ke lie avashyaka hai| 
aushadhi dvara sankramita bimari ki rokathama ka parinama kevala bimari ke phailava ko rokane mem aprabhavi hi nahim hai balki sadhanom aura manava shakti prabhavashali manadandom se apavartana karata hai|
eka mulyavana upachara se vanchita gambhira rupa se bimara marijom, pratijivanu mukabalaka ubharana ke lie vividha deshom mem isase yogadana diya hai|
gharelu samparko ki chuni hui (kemotherepi) sankramita bimari ka upachara ki mulyavatta bhi sandehapurna hai| 
haija virodhi tike joki ajakala upalabdha haim, ve bahuta adhika kshamata dara nahim rakhate haim|
unamem joki do khuraka se rogamukta kiye gaye haim, suraksha kevala 3-6 mahinom mem anta kara deti hai|
tikakarana abhiyana adhika upayogi niyantrana gatividhiyom se sadhanom aura manava shakti ko moda़ deta hai|
kisi bhi desha ko haija tikakarana pramanaka rakhane ke lie yatra avashyaka nahim haim|
desha ke vibhinna kshetrom mem ya deshom ke bicha mem yatraa aura vyapara para pabandi haija hai phailava ko nahim rokata| 
sankramita vyaktiyom ki bahutayata koi bhi lakshana nahim rakhati|
chekaposta ki sthapana (dhancha) ko thaosa niveshom ki avashyakata hai aura jo dusare adhika upayogi niyantraka manadanda़om se dhyana moda़ta hai|
bimari ke yakayaka visphota ki jancha pada़tala ka pramukha uddeshya, uchita upachara aura shighra nidana dvara nrityu dara ko kama karana hai| 
akala nrityu dara sarthaka rupa se, prabhavashali ऑrala rihaidreshana therepi (oarati) dvara kama kiya ja sakata hai|
bimari ke yakayaka visphota ko rokane ke lie ashavadi prabhava ke lie gatividhiyom ki hara suchi para rogavahaka niyantraka mapadanda़ lagu kiye jane chahie|
rogavahaka ke jivana chakra ki sabhi avasthaom ki tarapha, niyantraka mapadanda़om ko nirdeshita kiya jana chahie|
audyogika gatividhiyom aura badalati hui jivana shaaili ke karana shahari aura shahara ke charom tarapha ke kshetrom mem rogavahaka se hone vali bimariyom ka khatara badha़ chuka hai jisase aisa paryavarana bana chuka hai joki machchharom aura dusare rogavahakom ke prajanana mem adhika sahayaka hai|
da aidija machchhara (dengu bukhara ka vahaka) eka gharelu prajanaka hai joki sapha pani ke patrom ko tarajiha deta hai|
sadharana savadhanipurna mapadandom ke dvara machchharom ke prajanana ko niyantrita karane ke lie samudaya ka sahayoga atyavashyaka hai|
eka samayika dhanga se prajanana sthalom ko nishkasita kiya jata hai yaha nishchita karane ke lie niyamita chetavani dena avashyaka hai|
kuchha rogavahakom se hone vali bimariyom ka yakayaka udvega ka khatara janavarom va pakshiyom ki upasthiti mem badha़ta hai|
suarom ko japana ke mastishka jvara (jaipanija़ enake phlaintisa) ke vayarasa ka vistara karane vala paraposhi mana jata hai|
jaham suara bada़e sthapita ho chuke haim, vaham niyamita chetavani aura niyatakalika rogavahaka virodhi mapadanda nirdeshita kie jate haim|
yadi parivartita samvedana kshetra ke sathakoi tivra bukhara ki ghatana hai, taba sthaniya pradhana aura panchayata sadasyom ko jagaruka hona chahie aura sthaniya svasthya adhikariyom ko turanta suchita kiya jana chahie| 
babonika plega jo ki chuhe ke pissu se hota hai sharira se antarika angom se sambandhita linshamaniya sisa dusari mukhya rogavahaka se hone vali bimariyam haim|
aise kshetrom mem jaham lijamaniyasisa vyapaka hai ya vaham lijamaniyasisa ya plega ka tivrata se phailava hai, rogavahaka ko chetavani dene vala aura aiisi gatividhiyom ka anumodana kiya jata hai| 
bhojana se hone vali bimariyom sukshma jivi aura rasayanika tatvom satha hi satha bhojana se paida hone vala sankramana (udaharanartha salamonela sankramana) ke dvara paida hone vale visha ke karana phuda paॉijaninga ko shamila karati hai| 
vastava mem sabhi pani se hone vale sankramana (vayarasa se hone vala, baiktiriya se hone vala, parajivi se hone vala) bhojana ke dushita hone se sancharita kiye ja sakate haim| 
hamare desha mem bhojana se hone vali bimariyom ka yakayaka udvega bahuta sadharana hai|
jabaki pahale ghara mem bana bhojana khula rakhane para kama hi vyaktiyom dvara dushita hota tha, jabaki udyoga ya bada़e paimane para taiyara kiya gaya tatha vitarita dushita bhojana adhika sankhya mem logom ko prabhavita karata hai|
bhojana se hone vale sankramanom ka udvegom ki jancha karate hue sabhi ka sakshatkara lene ke lie prayasa kiye jane chahie, ve sabhi jinhe dikhavaya ja chuka hai bimari aura bhojana kshaya ke itihasa ke lie yadi koi| 
ve jinhone eka vishesha bhojana ke padartha ka upabhoga kiya aura nahim kiya unamem bimari ki dara ki tulana ki jati hai aura pratyeka bhojya padartha ke lie sambandhita khatara anka jata hai|
isa prakara ka ulajava vala bhojana bahuta unchi akramana kara dega ya bahuta uncha sambandhita khatara|
eka bahuta bada़e bimari ke yakayaka udvega mem, janasankhya ke eka namune se sakshatkara kiya ja sakata hai ya jancha pada़tala se niyantrita kiya ja sakata hai|
bhojana ke sanchalaka aura shankapurma bhojana ko jancha pada़tala ke daurana dhyana diye jane ki avashyakata hai|
hava se hone vali pramukha bimariyam joki bimari ke yakayaka udvega ka karana ho sakati hai – khasara, diphthiriya, kukara khansi, chhoti chechaka, mastishka aura merurajju se sambandhita bimari tantrika shotha inphluenja|
prathama tina bimariyam tike se roke jane yogya haim aura unaka yakayaka udvega samudaya mem bekara rogamukti ko darshata hai|
inphluenja aura chhoti chechaka ke lie bharata mem ajakala koi bhi tika upalabdha nahim hai, aura kisi vishesha roga nidana ki anupasthiti mem, ina udvegom ka niyantrana eka mushkila vicharartha prastava hai|
adhika ayu ke samuhom chikanapaॉksa (chhoti chechaka) prayah gambhira hoti hai aura bahuta jyada marijom ke svasthya suvidhaom mem visheshakara sankramita bimari vale aspatalom (inphaikshiyasa dijija haॉspitala) mem bharti karane ki avashyakata hai|
mastishka va merurajju ki bimari tantrika shotha ki dasha mem, yadyapi haidrojana ke do paramanuom se sanyoga karane vala tika upalabdha hai, lekina isako niyamita rupa se prayoga nahim kiya jata hai|
mastishka va merurajju ki bimari tantrika shotha ke yakayaka udvega mem chune hue adhika khatarom vale samuhom ka tikakarana kevala sipharisha ki gai hai ya paramarsha kiya gaya hai|
injekshana, echaaivi aura haipitaitisa bi aura si ko lete hue sankramanom ki vividhata ka sancharana kara sakata hai|
aparyapta rupa se visankramita suiyam, sirinja, teja aujara joki khala ko bhedate haim aura anajancha rakta, antrettara rupa se sancharita sankramanom ke sadharana sadhana haim|
suiyom ka uchita visankramana (kama se kama 20 minata taka ubalana) aura saire raketadana ki jancha echabiesaoji, echaaivi ke lie|
nrityu ki ghatanaom ki badha़ne ki aura samuha ki suchana ke lie jagaruka svasthya karmika vibhaga aura aspatalom ko dena chahie|
samasta svasthya suvidhaom ko, aupidi mem shamila karate hue marijom ka lekha jokha thika karana chahie|
yadi kisi kshetra mem isa prakara ki ghatanaem achanaka se badha़ jati haim taba kshetriya jancha pada़tala aura avashyaka sudharatmaka karya karyanvita kiye jane chahie|
eka prabhavashali nigarani rakhane ka tarika prarambha mem chetavani sanketa de sakata hai aura bimari ke udvega ko roka sakata hai|
svasthya karmika vibhaga paryapta rupa se prashikshita hai aura paramarsha die hue margadarshaka tarikom ka aspatalom mem palana kiya jata hai, yaha nishchaya hona chahie|
maleriya, degum bukhara aura dusare vayarala phivara ke karana bukhara ki tivra ghatanaom ke achanaka badha़ta aura samuhom mem hona ho sakata hai|
yadi vriddhi bimari ke visphota ko thika karati hai, avashyaka naidanika mahamari vijnana se sambandhita prayogashala kita vijnana se sambandhita jancha pada़tala lakshanom ko dekhakara nishchaya karane ke lie sanchalita kiye jane chahie|
chhoti chechaka va khasara bahuta teji se bukhara phailane ka karana ho sakata hai|
jatila naidanika prastuti, prabhavita ayu samuha chechaka ke tike ke itihasa ke dvara (khasara) lakshanom ko dekhakara roga ka nishchaya kiya ja sakata hai|
vayarasa ki vividhata hi tivra bukhara ka karana ho sakata hai|
kevala prayagashala parikshana ke dvara vayarasa ka prakara pahachana ja sakata hai|
kisi prakara naidanika prastuti ka prakara aura dusare mahamari vijnana se sambandhita va kita vijnana se sambandhita nirdharita sima sambhavata lakshanom ko dekhakara roga ka nirnaya karane mem sahayata kara sakate hai|
kuchha bimariyam jaise degum bukhara isa gambhira dasha mem prastuta ho sakata hai jaise ki raktastravika bukhara aura tela lakshana|
raktastravika bukhara ki eka ghatana ki suchana eka kshetra se jo ki dengu bukhara ke satha sangata tivra bukhara ki ghatanaom ki suchita vriddhi rakhata hai satha hi satha aidesa aijipati ka bahuta ghanapana dengu bukhara ke paksha mem eka shaktishali sanketa haim|
maleriya tulanatmaka rupa se hamare desha mem samanya hai|
maleriya, bimari ke yakayaka visphota ka karana mana jata hai|
tivra bukhara ki ghatanaom mem vriddhi motijara bukhara ke karana ho sakati hai joki tulanatmaka rupa se desha ke bahuta se bhagom mem sadharana haim|
motijara bukhara ki ghatana vyakhya ke satha yadi naidanika prastuti anukula hai to yaha dekhane ke lie marijom ki jancha ki jani chahie|
raktastrava bukhara ‘ghatanaem’ tulanatmaka rupa se mushkila se aura sankhya mem kama payi jati hai|
raktastrava se sambandhita bukhara ki jancha dengu bukhara ke visphota ko pahachana sakati hai|
tivra bukhara mem ghatanaom ka eka guchchha (samuha) jo suchita kiya ja chuka hai aura dengu jvara ka naidanika karanom para sandeha kiya gaya hai aura kita vijnana se sambandhita jancha-pada़tala, eka kshetra se raktastrava jvara ki eka suchita ghatana lakshanom ko dekhakara roga ko nirnaya karegi|
raktastrava se sambandhita bukhara chikanaguniya vayarasa ke karana bhi ho sakata hai|
yaha bimari naidanika rupa se dengu bukhara se abhinna haim|
tela lakshanom ki ghatanaem suchita nahim ki jati aura akala nrityu dara kama hoti hai|
raktastrava se sambandhita bukhara ke gambhira rupa unchi akala nrityu dara ke satha kuchha deshom se pita jvara aura evola jvara ke karana suchita kiye ja chuke haim|
bahuta adhika sankhya mem yatriyom ki vriddhi aura parivahana ke tivra sadhanom ke karana, ina sankramanom ke ayata ka vaham bahuta khatara hai|
yadi uparyukta sankramana ka sandeha kiya jata hai to yatra itihasa ya jo loga hala hi mem videshom ki yatra kiye ho unake satha samipa ke samparka banane chahie|
rajya ya rashtriya stara ke adhikariyom ko sanchara ke sabase teja marga ke dvara turanta suchana dena chahie|
jaba jancha karate samaya, marijom ka upachara karate hue, sankramita jaivika samagri ko kabu mem rakhate hue sankramana niyantrana savadhaniyam abhyasa mem lani chahie|
prayah jei sankramana, siradarda ke satha halke bukhara ya bina kisi naidanika lakshanom ke satha manda hota hai|
jei ke satha marija mastishka jvara ke chinha bhi prastuta kara sakata hai|
prayah eka gamva mem je.i. ke eka ya do se adhika ghatanaem nahim hoti|
chaubisa ghanto se adhika ke lie sthayi samvedana tantra ya vyavahara mem badalava aura bukhara ke satha tivra akramana ya itihasa je i ke marija denge|
kendriya tantrika vijnana se sambandhita kami prastuta ki ja sakati hai ya nahim bhi ki ja sakati|
mastishka-sambandhi badhaom ko alasya nidra, chida़chida़apana ya chetanata ki kami ke rupa mem pradarshita kiya jata hai|
marija bolane ki pareshani aura dusari nada़i se sambandhita kamiyam jaise netra sambandhi lakava, kevala eka tarapha lakava, sharira ka anachaha kampana, aura anguliyom ka thika se kama na karana vikasita ra sakata hai|
kendriya tantrika vijnana sembandhita chinha sthira ya dhire-dhire badha़ne vale ho sakate haim|
ghatanaom ki adhikata mem hara vyakti sankramana ke bada asankranyata vikasita karata hai|
isalie prayah pandraha sala ke niche ke bachchom mem yaha ghatanaem sthaniya kshetrom mem adhika dekhi jati haim| jaise vayaska janasankhya pahale se hi prakritika sankramana se surakshita hai|
nrityudara ko kama karane ke lie marija ko turanta aspatala mem bharti karana chahie|
je i khuna ke hanikaraka tatvom ko nashta karane vale padartha ke lie raktodaka vijnana ke dvara ghatanaom ko nishchita kiya jata hai|
mastishka ke tantuom ka samuha ya si.esa.epha. se vayarasa ka prithakkarana niyamita rupa se nahim kiya jata|
mastishka ki nasom mem sujana se hane vala roga ki eka ghatana prayah unchi digri ke bukhara ka achanaka ghavom ke satha, gambhira sira darda, badalane yogya samvedana tantra ke satha ya bina gardana ko moda़ne mem darda ko prastuta karati haim|
gardana sambandhi dridha़ta ki upasthiti aura sakaratmaka karaninga aura brudajinasaki chinha naidanika lakshanom ko dekhakara roga ka pata lagane ka nishchaya karate haim|
mastishka ki nasom mem sujana se hone vala roga, vayarala ya mavada banane vali sajiva rachana, kshaya roga paida karane vale ke karana ho sakata hai|
pradarshana ke lie lada़khada़ate hue chalate hue chheda hona ya sajiva rachana ka prithakkarana si.esa.epha se avashyaka hai|
eka vyaparika rupa se upalabdha chipakane vala padartha ka parikshana mastishka ke nasom mem sujana se hone vala roga ke kuchha karakom ka jnana pradana kara sakata hai udaharanartha-staiptokokasa nyumina haimophilasa inphluenja bi, meninagokosi e aura si|
maleriya ki bahuta si jatilataom mem se eka karana pi.phailasipairama mastishka ki eka patanashila bimari se sabandhita haim|
jaba se maleriya tulanatmaka rupa se sadharan hai, maleriya ke lakshanom ko dekhakara nirnaya karana sabase pahale dhyana me rakhane chahie|
lakshanom ko dekhakara roga ka nirnaya tabhi kiya ja sakata hai yadi kshetriya jancha maleriya sambandhi dashaom ko sanketa karati hai, bimari naidanika rupa se sangata hai aura maleriya virodhi upachara prabhavashali hai|
rakta kanikaem banane ke lie unake parikshana ke lie pratyeka pi echa si para savidhaem upalabdha honi chahie|
matijara ka bukhara ki prayah visheshataem haim dhire-dhire baicheni ghabarahata ka prarambha hona, siradarda, mansapeshiyom mem darda, bhukha ki kami hona, dhire-dhire bukhara ka pancha se sata dinom ke daurana 39 se 42 digri C taka pahunchana|
eka panasika udasinata, aura nirasata sadharana hai aura bukhara se hone vali manasika uttejana vikasita ho sakati hai|
isa dasha mem rogi, bukhara badale hue samvedana tantra ke satha prastuta kara sakata hai|
motijara ba bukhara hamare desha mem bahuta sadharana hai, ise hamesha hi savadhana itihasa ke dvara, sharirika parikshana aura esa. taiphi ke lie rakta samvardhana ke dvara bahara nikala jana chahie|
kuchha bimariyam jaise maleriya, dengu bukhara, khasara, kali khansi ki jatilataom ke parinama svarupa badala hue samvedanatantra ke satha bukhara ho sakata hai|
isa prakara ki jatilataem ghatanaom ke tulanatmaka rupa se kama anupata mem ho sakati hai|
mastishka ki eka bimari ki eka ghatana isalie tulanatmaka rupa se samudaya mem bimari ki bahuta adhika ghatanaom ko pradarshita karati hai|
daॉktarom ko isa prakara ki ghatanaom ki turanta suchana dene ki avashyakata hai taki age bhi kshetriya jancha ko chalu rakha ja sake|
kshaya roga, mastishka ki nasom mem sujana se hone vala roga hamare desha ke kai hissom mem tulanatmaka rupa se sadharana haim|
yadyapi bahuta adhika ghatanaem suchita ki gai haim ti.bi. mastishka ki sujana se hone vala roga se bimari ke visphota ki dasha mem hone ki asha nahim ki jati hai|
kisi bhi prakara se daॉktarom ko yaha bata dhyana mem rakhani chahie ki ti vi mastishka ki nasom mem hane vala roga eka visheshaka roga hai, vishesharupa se yadi marija chhota bachcha hai|
yadi mastishka ki eka bimari ki koi bhi ghatana ajnata roga ke karana ka jnana ke karana eka kshetra mem hoti hai je i ko avagata nahim hai, isaki turanta suchana uchita jancha-pada़tala ke lie rajya svasthya adhikariyom ko deni chahie|
vayarasa prithakakarana ke lie namunom ko vishesha kabu mem rakhane ki avashyakata hai yaha uchita hai ki kuchha vishesha sansthaem jese ena ai si di dilli aura ena ai vi pune, prambhika stara para jancha-pada़tala mem sammilita kiye gaye haim|
pani va khane se paida hone vali bimariyam naidanika lakshanom ki vividhata ke satha prastuta ha sakati hai|
tivra pani jaisa atisara chhote bachchom mem sabase jyada samanya samasya hai|
manasuna mem aura manasuna ke bada ke mahinom mem mausami vriddhi ke satha ghatanaem pure varsha bhara hoti haim|
tivra pani jaisa atisara ke yakayaka phuta sakata hai|
khasara ke yakayaka udvega ka anugamana karate hue isa prakara ke bimari ke achanaka visphota suchita kiye ja chuke haim|
bimari ke phailava ki jancha ke lie aura munha se pani ki kami hone para sharira mem pani sokhane ki tarakiba pradana karane ke lie joki jivana rakshaka hai, shighra sakriya hona mahatvapurna hai|
o ara ti ki anupasthiti me unchi nrityu dara varnita ki ja chuki hai|
tivra pani jaisa atisara ke gambhira ghatana krama ke kuchheka ghantom ke bhitara hi nrityu ho sakati hai|
chhoti unra ke bachchom ka samuha, sugamata se prabhavita ho jata hai, unake sharira mem pani ki kami ho jati hai|
jaba atisara vali ghatana vayaskom mem bhi ho sakati hai isake prayah gambhira rupa se sharira mem pani ki kami ho jana ya nrityu hona parinama nahim hote|
yadi tivra pani jaisa atisara ke karana sharira mem gambhira rupa se pani ki kami hona ya nrityu hone ki ghatanaem pancha ki ayu se bada़e marijom mem hati hai, haija ka sandeha kiya jana chahie| kshetriya stara para niyantrana mapa margadarshana ke rupa mem shuru honi chahie|
pechisha ka achanaka udvega bachchom mem ho sakata hai isa prakara ki bimari ka yakayaka visphota bahuta adhika nrityu ka karana ho sakata hai jaba taka vishesha prakara ke upachara samayika danga prarambha nahim kiye jate haim|
samudaya aura gauna svasthya karmika vibhaga mala mem (khuna jaisa atisara) chinha ke khatare se jagaruka ho, yaha mahatvapurna hai taki daॉktari sahayata turanta khoji jae|
tivra shvasa se sambandhita sankramana ke karana shitajvara hota hai jo bharata mem asvasthata aura nrityu ka karana haim vishesharupa se bachapana mem|
baiktiriya, vayarasa, phangi aura parajivi aura itani chhoti rachana jo maikroskopa se dekhi jae ki vividhata ke dvara ve karana ho sakate haim|
kuchha bimariyam jaise plega, chhuta ki bimari, baiktiriya se hone vali bimari, inphlunja meliyodosisa aura hanta vayarasa sankramana mukhya rupa mem phephada़o se sambandhita ulajava ke satha gambhira bimari ke udvega mem prakata ho sakati hai|
yadi bada़e bachchom mem aura vayaskom mem kuchha ghatanaem eka satha dekhi jati hai to ina sankramanom se bhavishya mem ghatita hone vali yakayaka bimariyom ke lie turanta hi jancha pada़tala shuru ki jani chahie taki bimari ke achanaka visphota ko rokane va ghatanaom ke uchara ke lie uchita karya turanta kiye jaye|
tivra shita jvara ke marijom mem prayah bukhara, thithurana khansi, sine me darda aura dusare samanya lakshana shvasa se sambandhita aparyaptata ki ghatana-badha़ta ansha aura sine ke eksa-re para andara ghusana dekhe jate haim|
eka bara shita jvara ka sandeha ho jane para,age sancharana ko kama karane va nrityu ko rokane ki sahi vyavastha ke lie, lakshanom ko dekhakara roga ka nirnaya hone para roga ke kuchha vishesha karana janana avashyaka hai|
yadyapi piliya bahuta se karanom se ho sakata hai para hamare desha mem piliya ki bahuta adhika ghatanaom ke lie vayarasa se hone vali jigara ki sujana uttaradayi hai|
kama se kama vayarasa se hone vali jigara ki sujana ke lie yaha chhah karaka (echa e vi, echa bivi echa si vi, echa di vi, echa i vi aura echa ji vi) karana ho sakate haim|
bharata mem vayarasa se hone vali jigara ki sujana ka yakayaka visphotom ke lie maukhika rupa se sanchalita hepetaitisa i vayarasa uttaradayi hota hai|
ina bimariyom ka achanaka visphota, pani apurti ke dushita hone se lagatara juda़a hua hai|
piliya ke hava-bhava badha़ti hui unra ke satha badha़ane ke lie prakata hote haim|
hepetaitisa i ghatanaom ki bahutayata javana vayaskom mem hoti hai|
bimariyom ke visphotom ke daurana, gauna gharelu ghatanaem asadharana haim|
garbhavastha ke tisare traimasika ke daurana una sankramitom ke bicha akala nrityu dasa bisa pratishata taka pahuncha sakati hai|
hala hi mem rajasthana hariyana, gujarata rajyom ke gramina samudayom mem hepetaitisa bi ki bimari achanaka phaili joki ayogya sadharana chikitsaka ke dvara asurakshita injekshana ke prayoga se joki mahamari vijnana se sambandhita hone para juda़i hui thi| 
unchi akala nrityu dara ke dvara bimariyom ka yakayaka visphotom ko dekha gaya tha|
tivra vayarasa se hone vali jigara ki sujana paryapta rupa se vishesha naidanika lakshana haim joki prayah lakshanom ko dekhakara roga ka pata lagane mem koi pareshani nahim paida karata|
marija kuchha samanya se lakshana vikasita karata hai jaise ghabarahata, kamajori, bhukha ki kami, ji michalana, ulti, bukhara aura peta mem halka darda|
piliya ki avadhi uparivartaniya hai lekina prayah eka se tina haphtom mem samapta hoti hai|
vyaktigata marijom mem vayarasa se hone vali jigara ki sujana ke vishesha prakara ko naidanika karanom para bhinna nahim kiya ja sakata|
desha ke bahuta se bhagom mem piliya ka eka pramukha karana eka prakara ke baiktiriya ke karana adami ya janavarom ko hane vali bimari ubhara raha hai|
achanaka unchi shreni ka dhava mansa peshiyom me darda, ankha ki bhitari jilli ka phailava ke dvara prayah yaha bimari pahachani jati hai|
yadi marija mem piliya dikhai deta hai to gurdom ki ulajana aura raktastrava, baiktiriya se hone vali bimari nishchitata se sandeha kiya jana chahie|
mastishka ki nasom me sujana se hone vala roga phephada़o se sambandhita va hdaya sambandhi ulajana kuchha ghatanaom mem marija mem ho sakati hai|
jaba taka kitanu nashaka davaom se turanta upachara nahim kiya jata, taba taka unchi akala nrityu dara ke dvara hi chinhita kiya jata hai|
pahale haphte ke bada mutra se aura tivra bimari ke daurana khuna se baiktiriya se hone vali bimari ke prithakkarana ke lie ya khuna mem hanikaraka tatvom ko nashta karane vale padarthom ka pradarshana ke dvara bimari ko nishchita kiya jata hai|
bachchom mem teji se halka lakava (e epha pi) ka sabase mahatvapurna karana poliyo vayarasa ka sankramana haim|
poliyo ki sambhavana e epha pi ki koi bhi ghatana se mana jana chahie, jo kshetra o pi vi arakshana stara ke haim aura bahuta kama poliyom ki ghatanaem haim unamem mana jana chahie|
pairalaitika paॉliyomailitinsa ke lakshanom ka nidana kevala dusare lakshanom, jo ki pahale se hi upasthita ho, ke bada hi khatma hone chahie|
tivra pairalaitika paॉliyo mailitisa ka charitra chitrana bukhara ke dvara jo ki sharira ke angom para kamajori ya lakava ka achanaka se akramana karata hai jo ki shuru ke tina dina taka unnati nahim karata|
lakava janma se hi upasthita nahim hota hai aura na hi gambhira chota mem ya manasika gatirodha mem sahayaka hota hai|
sarvajanika svasthya parinama ka eka vistrita sarvekshana, naॉji mem sana 1997 mem vibhinna prakara ki apadaom ke rupa mem paya gaya hoga|
sharirika parikshana ki jatila jancha shamila karati hai tivra phlesida़ lakava mansapeshi ki komalata koi kendrika kami nahim, gahari tenada़sa nasa ki anupasthiti ka prabhava aura asamana jancha|
prabhavita mansapeshi ka nasha eka dera se hui jancha hai, lakshanom ke satha dinom ke akramana ke bada shesha lakava ya nrityu ya e epha pi mem lagatara ajnata ghatana, sambhavita lakshnom ka nidana karati hai|
e epha pi se khataranaka bimariyom ke alaga hone ki ghatana ya sambandha, tivra paॉlimailitisa ke lakshnom ka nidana nishchita karata hai|
nitamba tantrika ki chota ke karana, pairalaitika paॉliyomailitisa prayah guliyana bare sindroma (ji bi esa) transavarsa mailetisa aura manasika lakava ke satha bhramita karata hai| 
galata sthana para adhika injekshana dene se nitamba tantrika ki chota ke karana manasika lakava mem, purana itihasa aura sharirika parikshana ke dvara antara bataya ja sakata hai|
jibiesa mem bukhara prayah anupasthita aura lakava eka samana va durastha hota hai|
gahari pratibimbita nasa (jo mansapeshi ko haddi se joda़ti hai) ki sarvペtrika anupasthiti aura sarvatrika sammohana hai| 
lakava phuraphuri paida karana aura hatheliyom aura talave ki behoshi prayah ji bi esa mem payi jati hai|
prarambha mem paॉliyo ki kaphi ghatanaom ke ji bi esa ke rupa mem daksha logom ke dvara nidanita kiya ja chuka hai| jinhonne prastavita kiya ki poliyo vayarasa ke lie pancha sala se kama unra ke logom ka ji bi esa ki pratyeka ghatana mem malamutra tyagane ka sthana janchana chahie|
koraputa jile ne sana 1991 julai-agasta mem tivra jvara roga (ji.i) ke achanaka se utpanna hone ke prabhavom ki suchana di| isa samaya ke daurana lagabhaga 2270 ghatanaem aura 413 nrityu pae gae|
jyadatara ghatanaem atisara aura ulti ane ki hi hoti hai|
sana 1991 mem 18 mai se 19 juna ke samaya ke daurana chikitsiya sandigdha motijara bukhara ki 392 ghatanaem pi echa si galora ke bahisam vibhaga, jila hamirapura, himachala pradesha para upacharita ki gai thi|
praveshita rogiyom mem chikitsiya ruparekha ke ankanom ki jancha pradarshita karati hai ki ve prarambha mem (100%) bukhara, (74%) siradarda, (17%) ulti, (15%) atisara, (7%) kabja, (57%), sparshaniya yakrita ke satha upasthita the|
25 khuna ke namunom mem se dasa mem salamonela taiphi alaga kiya gaya tha, jisaka parikshana shimala medikala kaॉleja ki prayoshala mem kiya gaya tha|
praveshita ghatanaom ki eka krama suchi prathama svasthya kendra, gailora mem upalabdha thi|
suchi jisane 101 ghatanaom ke nama, unra linga, pata aura lakshanom ke akramana ki dinanka ki suchana pradana ki|
bharata mem lagabhaga achanaka se vayarasa se hone vale yakrita shodha phesiyo orali hipaitaitisa i ke utsarjana ke karana haim|
achanaka se bimariyom ka phutana, dushita pani ki apurti se lagatara sambandhita hai|
achanaka se bimari ka phuta pada़na hepetaitisa e (yakrita shodha) ke karana ho sakata hai|
kuchha samaya purva, hipaitaitisa bi ipidadaimiolojikali ke kuchha rogom ka achanaka se phutana asurakshita injekshana (jo ki suchita kiya ja chuka hai) se sambandhita hai|
nartha vesta dehali (esa di vilauka pitamapura janasankhya 1435) mem rahane vale nivasiyom ne apraila 1994 mem piliya ki ghatana mem aprayaya vriddhi mahasusa ki|
blokom para madhyamika darje se sambandhita logom ke dvara ya upari sosiyo-ikonomika strata ke dvara rokathama lagai gai|
sabhi nivasi prayoga karane ka adhikara rakhate the aura kevala sapha-suthara shauchalaya prayoga karate them|
blaॉkom mem paipom se pani ki apurti hoti thi aura vaham koi aura pani ka sadhana nahim tha|
pani ki apurti ruka-ruka kara thi aura logom ki jarurata puri nahim hui|
isalie nivasiyom ne ऑna-laina-bustara (vidyuta vardhaka yantra) yantra, pani ko upara tanki taka pahunchane ke lie prayoga kie|
eka ghara-ghara jakara sarvekshana mem sana 1435 mem piliya ki 27 ghatanaom ko, janasankhya ki tina mahine ke nirdeshita samaya mem pradarshita kiya|
piliya ke akramana ka mulyankana 1.9 pratishata paya gaya tha|
bahuta sari bimariyom ka phutana abhilekha mem hai, jahaॉm piliya ki ghatanaom ka ghatata-badha़ta krama ka nambara (hajarom mem se kuchha ghatanaem) kuchha samaya mem hua|
27 ghatanaom mem esa di vilauka eka bimari ka phutana mana gaya tha|
tumane paya ki jyadatara ghatanaem hepetaitisa i ke karana thi (prayogashala jancha)
tuma sandeha karate ho ki isa bimari ka phutane ka bada़a karana paipa mem ata dushita pani hai|
esa di vilauka mem achanaka se bimari ka phutana hepetaitisa i ke karana tha|
eka rogi bukhara, siradarda, sharira mem darda aura kampane ki bimari ki shikayata lekara gamva banamani (janasankhya 3403) se pi echa si ke o pi di mem 7.10.1991 ko aya|
apane chikitsiya adhikari ko bada़i sankhya mem ghatanaem joki gamva mem eka samana bimariyam rakhati haim ke bare mem suchana di jinamem kaphi loga mara gaye|
pi echa si se daॉktara ke eka samuha ne 8.10.1991 ko gamva mem sarvekshana kiya|
unhonne 756 logom ka parikshana kiya, jinamem 692 (92%) bukhara satha hi bukhara se pahale ane vali kampa-kampi aura thanda lagane ki bimari ko sahana kara rahe the|
gamva banayani ke kisi bhi rogi mem mastishka ki nasom mem sujana hone se bhayankara roga ke lakshanom ka hona nahim paya gaya|
eka chikitsiya visheshajna ne gamva banayani ke 63 rogiyom ke bukhara ka parikshana kiya|
81 bhaga maleriya parajivi ke lie sakaratmaka paye gae (plajamoda़ima baiveksa 66, plajmodiyama phelasiperama|
8.10.1991 ko sarvekshana arambha ke daurana 466 khuna ke namune bukhara ki ghatana se ekatrita hue|
pi echa si ne ise eka bimari phailasipairama maleriya ka hona mana gaya|
jila adhikariyom ne rogom ke achanaka se utpanna hone ke bare mem suchana di aura gamva banayani mem rogiyom ke upachara ke lie eka upachara kendra sthapita kiya gaya|
jila svasthya adhikariyom ne roga ke phutane ke bare mem 1.11.91 ko rashtriya sanstha kanyunikevala roga ko suchana di|
samachara isa prakara varnita tha ki jila pharukha bada ke eka gamva mem bada़i sankhya mem bukhara ki ghatanaem aura 21 nrityu payi ja chuki thi|
sana 1991 mem 1-3 navambara ke daurana ena ai si di samuha ne prabhavita kshetrom ka daura liya|
2.11.91 taka 2294 khuna ke namune gamva banayani se ekatrita kie gae the| 325 maleriya parajivi ke lie sakaratmaka paye gae the (pi phalasiperama 131, pi vaivaiksa 194)
lagabhaga chalisa pratishata sakaratmaka namune pi phalesiperama ke karana the|
thika usi samaya antarala ke daurana, 2184 namune pi echa si ke dusare gamva se ekatrita kie gae the kevala gamvana banayani ke 5 kilomitara ke andara|
jinamem se 208 namune maleriya parajivi (pi phalasiperama 36 P vaivaiksa 172) ke lie sakaratmaka paye gaye|
gamva banamani ki sada़ko para jaba jaya ja raha tha yaha dekha gaya ki eka bada़i sankhya mem nivasi apane makanom ke samane bida़i (lokala sigaretsa) bana rahe the| 
puchhatachha para yaha paya gaya ki bida़i bichhana aura krishi, gamva banamani aura usake asa-pasa ke gamvom ke pramukha vyavasaya the|
gamva banamani mem bida़i ludha़kane ne kisi roga ka utpanna hone ki shuruata ke tathya pradana kie|
yaha mahatvapurna hai ki pichhale tina salom mem jila pharukhabada ne phailasipairama maleriya ki ghatana ki koi suchana nahim di|
prayagashala mem kancha ke samana dhone ki aura kitanuom ko marane ki suvidha honi chahie, jo ki chikitsiya aura vatavaraniya namunom ko ekatrita aura unake parivahana aura satha hi satha khuna ke namunom se sirama ko alaga karane ke lie avashyaka hai|
prayogashala ko shishtachari avishkari ya mula padarthom ki adhikata rakhani chahie jo ki nigarani aura avishkara ke lie jaruri ho sakata hai ya apadaom mem sahayata pradara kara sakate haim|
hastachalita kriyavidhiyom ke kama karane ka stara prayogashala mem upasthita hona chahie|
paryapta stapha ke sadasya joki shikshita hai aura ya bahumulya ya rogom ki nigarani ke kama ki jimmedari ke lie prashikshita haim|
likhita varnana ke rakharakhava ke lie upayukta lekhana samagri aura usa saba ka dusare kendrom para bhejana|
prayogashala ko, tivra parinama ke utsarjana ke lie ya jaldi chetanapurna sanketom ke lie adhikata mem vidyutiya sanchara netavarka rakhana chahie|
atisara, tivra shvasaniya sankramana, chechaka, aparyapta poshana aura sthaniya manushyom mem prachalita bimari, maleriya panchom jarurata me nrityu ke sadharana karana haim|
aparyapta poshana ko chhoda़kara sabhi prakashita karane yogya vimariyam sidhe vatavarana svasthya sthiti se sambandhita thi aura prakashita karane yogya bimari ke anusara aparyapta poshana bada़e rupa se khijane vali bimari hai|
apadaom se prabhavita loga vishesha rupa se prakashita bimari ko ghava pahunchane yogya haim| apada ki pratirodhakata ko aparyapta poshana, dabava aura thakana ke karana kama karate haim aura jaba bada mem ane vali apadaom ki rahane ki sthiti gandi hom|
sancharita bimariom ka niyantrana svasthya vatavarana (svachchha pani, paryapta saphai, vahaka para niyantrana, sharana) para nirbhara karata hai| rogom se mukta karana aura svastha karyakarta jo ki shuruati nidana aura upachara mem shikshita the|
prabhavi vatavaraniya svasthya jnataom ko dhanyavada apadaom ka anusarana karate hue mahamari bahuta lambi va sadharana nahim haim|
chhua-chhuta ki bimari mem apavada hai ki bhayankara purane roga, sukhe aura nagarika kalaha ke dvara hote haim thika usi taraha ve aphrika mem 1980 tatha 1990 mem pae gae|
mahamari ki prakashita hone yogya bimari hai ki vaha aphrika aura sansara ke dusare bhagom mem jada़a-ponchha ke lie sharanarthi shivira rakhate haim|
karyakrata bimari ki yojana para nigarani aura akhanda paryavaraniya svasthya seva janata ke svasthya ki raksha karane .......... aura jaba ye bimariyam eka samaya mem apadaom mem payi jati hai, jatila hai|
eka bhanyakara roga ko badha़ava dene ki dasha dusare prabhavom ke karana hi jyadatara hoti haim naki prathamika khatarom se, bharava ki ghatana ko chhoda़kara, jo ki pani se utpanna aura vahakom se utpanna bimariyom ko badha़ava de sakati haim|
dusari ghatanaom se pradushana ya khada़e pani ko chhoda़na ya pine ke pani ki apurti mem badha utpanna karana ho sakata hai|
teja havaem, samudriya tuphana, mitti ki slaidsa aura bhukampa sabhi ruke hue pani mem ho sakata hai, visheshatah jaham para sharirika prayatnom ke jalaprapata ho sakate haim|
udaharana ke lie taraha-taraha se barpha aura pala ka pighalana, bharavom ka banana, mitti ka girana jvalamukhiya visphotom ke lie asadharana nahim hai|
badha़om ke karana, bhukampa, bhuskhalana joki nadiyom ko banda kara deti hai para bhi pratikriya kara sakata hai|
pratyeka ghatana mem, adhikata mem ruka hua pani, kitom ki bimari ke vahakom ko paida ya pani ki apurti ko bekara ya maila ekatra karane ke kshetra ke satha dushita kara sakata hai|
prakritika apadaom aura bhujaom ka talamela na rakhane ka parinama pramukha rupa se pani ka toda़na ya vidyuta ki apurti jo ki pani ke pampom ke lie jaruri hai mem hastakshepa karana ho sakata hai|
maila ekatra karane ki paipa aura maila ekatra karane ke kshetra ka ilaja ka karya khatma kiya ja sakata hai ya use arpita karana sarthaka|
pani ke dvara vahaka ke dvara lai gayi bimari ke atirikta vaham para bahuta sari sparsha se phailane vali bimariyam ho sakati hai jo ki apasi sparsha se phailati hai| 
vo lamba samaya jise manushya asthayi rupa se basane mem vyatita karata hai, eka bimari ke utsarjana ke khatare ka eka mahatvapurna dridha़nishchaya haim|
asthayi sharana mem, sharanarthiyom ke sthayi nivasa mem kevala kuchha samaya ke lie eka jatila avastha jo ki inaphekshana ki bimari se chhuachhuta jaise bhayankara roga utpanna karati hai|
bhukhamari ke daurana bhojana pradana karane ke lie sthapita kaimpa eka vishesha ghatana hai, jaise bada़i sankhya mem loga joki mahale se hi kamajora aura sambhavatah bimara haim,eka lambe samaya ke lie apani ichchha se aise kaimpa me raha rahe haim|
kajaika ripablika ke purviya kshetra 1997 ki garmiyom mem vibhinna prakara se bhare hue the|
200000 se jyada loga vidyuta ke bina aura 30000 gaisa ke bina the|
lagabhaga 3500 kuem aura dusare pani ke sadhana dushita the aura kharaba pani se paudhom ka upachara karana nirarthaka arpita karana tha|
kajaika neshanala instityuta ke maikrobaoloji kendra ke janata ke svasthya ne turanta kriya ki kshetriya svasthya steshana ke satha kama karane mem|
miyadi bukhara, salamonelosisa, sigailosisa, tivra atisara, hipaitaitisa e ka vayarasa, tuleriyagiya, inavaisiva mainishamokokela bimari, tokyoplajamosisa, laiptospaisesisa aura limpa bimari ke utpanna hone ka varnana mulyankita hai|
eka vishesha hipaitaitisa e vaisinesana karyakrama kuchha chune hue kshetrom mem 3-15 salom ke bicha mem prastuta kiya gaya|
bada vali apadaom ka parikshana yaha darshata hai ki laiptosairosisa tina guna badha़ chuka hai lekina tinom mem se koi bhi dusari bimariyom ko lakshita banane mem purnatah pradarshana nahim hua|
janasankhya ko hipaitaitisa e ka tika lagane se koi bhi vayarasa ki suchana nahi mili|
prastavita samipiya napom ne, chetavani dena aura niyantraniya dantom se butarane vale jantuom ko sammalita kiya|
arama se hone vale ऑpareshana kiapa kalina avastha ke daurana nrityu dara 10000 prati dina ke anusara, achanaka se hone vale parivartanom ki jasusike lie dikhani chahie|
sadharanatayah svasthiya karmachariom ko salaha lena chahie jaba bachche nrityu dara (si ema ara) sharanarthi ki janasankhya mem 100000 dina mem roja badha़ti he ya jaba pratyeka pancha ke bicha mem 2/10000 pratidina nrityu dara badha़ti hai| 
julai 1994 mem purvi jaire mem, eka miliana ravanada़na bicha biema ara sharanarthiyom ki shreni 34.1 -54.5/10000 pratidina (uchcha pratyeka lekha jokha mem) badha़ti hai|
sharanarthiyom ki janasankhya 6 se 10% ke bicha mahine ke daurana jaiira ke pahunchane ke bada mari|
uchcha nrityu dara jyadatara purnarupa se atisara bimari ke bhanyakara roga aura aparyapta pani ki apurti ke karana thi|
sharanarthiyom ke arntapravaha ke tisare saptaha ke dvara, arama ke lie shuru kie gae prayatna eka prabhavashali asara chhoda़ti hai|
pratidina ki napa, jaise chechaka asankranyata, vitamina e ki kami puri karana, bimari ka upachara ko phailana ka karyakrama pratyeka kaimpa me sthapita kie gae the aura pani ki pareshani, pratyeka vyakti ko 5-10 litara taka pani pradana karake dura ki gai|
lakava janma ke samaya maujuda nahim hota aura gambhira chota ya manasika gatirodha se sambandhita nahim hota|
aisi samasyaem una jagahom para jyada pai jati hai jaham pridisaastara sthitiyam asvachchha hoti hai, jaise ki ghani abadi vale mahanagara| 
garibi ghatane ke lie jagarukata aura vyavastha ke stara ko uncha uthane ke lie, samanya svasthya ko badha़ane ke lie kuchha kadama uthae gaye, saphai sambandhita sevaem samudaya ke lie kuchha suraksha pradana karengi yadi apadaem akramana karati hai| 
apada hone se pahale hi purvataiyara mapaka apanae jaem, to mahamari rokane va sancharita hone vali bimariyom ko niyantrita karane ki kshamata ko achchhi prakara badha़a sakate haim| 
apatti ki sthitiyom mem, tivra sansa se sambandhita sankramana aura dayariya prayah mukhya ghataka hote haim| 
unhe rokane ke lie, svasthya ko badha़ava sapha pani ki paryapta matra ka pravadhana, svachchhata ki suvidhaem, uchita sharana bilakula avashyaka haim|
khasara ke yakayaka udvega apatti ke samaya mem prayah unchi akala nrityu dara ke satha eka sadharana khatara hai| 
kisi bhi aisi ghatana ke prakata hone se pahale hi, shighra tikakarana abhiyana shuru kiye jane chahie| 
janata-svasthya ki nigarani, uchita kadama uthane hetu yogya banane ke lie, svasthya suchana ka sangraha, vishleshana aura prachara hai|
janata ke svasthya ki nigarani ke lie vishesha svasthya karmachariyom ki niyukti karana mahatvapurna hai|
samipya aura samudaya ke svasthya karmachariyom satha hi satha asthayi sahayata kendrom aura aspatalom ke karmika vibhaga, bimariyom ki kisi ekasuchi ke satha marijom ke lie prastuta karate hue sacheta hona chahie – motijara jvara ya pairataiphaita jvara, haija, tantrika jvara, plega mastishka jvara ya tantrika shotha satha hi satha (bhojana ki vishaktata ko shamila karate hue) vishaktata ki bahuta adhika matra ya maleriya ki ghatanaem| 
prithaka bimari ke sadhana se, parichita samparkom se ina marijom se, purani ghatanaom (itihasom) ko liya jana chahie|
bahuta bada़e paimane parajanasankhya andolana ki bahuta buri sthitiyom ke tahata kuchha sima taka janata – svasthya samasyaom ki nigarani sambhava ho sakati hai|
prastuta riportinga yojana ko kshetra vistrita nigarani rakhane vali yojana ko banane ke lie badha़aya ja sakata hai ki joki prathamikata bimari jaise gambhira pani va saphai se sambandhita mahamari bimariyom ko shamila karate hue, ka dhyana rakhati hai|
janasankhya andolana ki sakriya nigarani yojanabaddha apatti ke samaya hastakshepom ke lie aura samanya bimari ki nigarani ke lie ankada़e pradana kara sakati haim| 
suchana dene ke tarikom aura rastom ko badha़ane ke lie, joki prashasakiya dhanchom dvara prayoga kiya unakshetraom mem jaham loga pahuncha rahe haim prayah sabase jyada prabhavashali pahuncha haim|
sandehapurna bimari ka yakayaka udvegom, eka svasthya nigarani ke tarike se suchana ke dvara sanketika, mulyankana ke lie unche stara ke vijnapti ka prayoga karate hue teji se jancha pada़tala ki jani chahie(medikala sensa phrantiyararsa 1997e., varlda heltha ऑrgenaijeshana 1999bi)
mulyankana ko ina nirnayom bimari ke udvegom ko kaise niyantrita kiya jata hai ko lene ke lie shakti pradana karana chahie|
sankramaka rogom ke yakayaka udvega ko niyantrita karane ke lie do mukhya rananiti haim – surakshatmaka gatividhiyom ke dvara bahuta adhika ghatanaom ko kama karana aupa prabhavi upacharava shighra kesa ki pahachana hone ke dvara bimari ke karana nrityu dara ko kama karana|
ina mapakom ko teji se kiya jana chahie aura jabaki bimari ke jancha pada़tala mem prayogashala nishchayatmakata ke lie intajara karate hue dera nahim karani chahie| 
bimari ke udvega ka eka mukhya mahamari ka rupa dharana karane se pahale hi, bimari ke udvega ko niyantrita karane ka prabhavashali tarika teji se pratikriya dena hai|
apatakalina dashaom mem samuha mem asankranyata prathamikata hai, jaham loga visthapita kie gaye gaim, vaham samanya sevaom ka vighatana hai, jaham bhida़ bhada़ vala ilaka ya svachchha sthitiyam ya jaham kahim bahuta phaila hua kuposhana hai, vaham asavadhani se ya khasara ki eka ghatana suchita ki gai ho ya nahim| 
haija ke eka nishchita kesa ko, sare dayariya ke kesamaja ko teja karana chahie, haija ki taraha upachara kiye jane ke lie|
ina tina gatividhiyom mem seva pradana karane vale dala ke karyakramom ki bhumika mahatvapurna|
haija eka udaharana ke rupa mem prayoga kiya jata hai kyonki laitina amerika, eshiya, aura aphrika ke bahuta se bhagom mem bimari sthaniya hai| 
1990 ke prarambha mem, haije ki mahamari ne laitina amerika aura aphrika mem lakhom logom ko prabhavita kiya tha|
haije ki rokathama aura niyantrana apatti mem ,dusari mahamariyom ke lie samanya tarike svikara kiye jane ke lie udaharana prastuta karata hai|
bahuta se vikasita deshom ki samanya janasankhya mem thosa haije ke vahaka aba sadharana haim (udaharanartha loga bina kisi pradarshana ke bimari ke vibariyo kolariya late haim|) 
yadyapi haije ki bahuta si ghatanaem manda aura sadharana mapakom se upachara ke yogya haim, bimari bahuta teji se badha़ sakati hai aura sharira mem pani ki kami hone ke karana isaka parinama nrityu ho sakata hai|
haija bahuta asani se phaila sakata hai jaham kahim bahuta kama saphai ho aura chipachipa ho jaise ki sharanarthi shivira mem hota hai|
isalie haije ko rokane ke lie shivirom mem pani ki apurti, saphai aura svachchha bhojana ki sahi vyavastha ko yojana banana bahuta mahatvapurna hai| 
niyamita svasthya nigarani ke lie jo uttaradayi hai, haija ki sambhavana se sacheta hona chahie, bimari ke chinhom se parichita hona chahie aura turanta shankapurna ghatana ke bare mem adhikariyom ko suchita karana chahie|
yadi haija phaila chuka hai to jo avashyaka kadama uthae jane ke lie yojanaem bhi banani chahie|
inhe vishleshita karana chahie jo atirikta svasthya aura paryavarana sambandhi mapaka liye jane haim satha hi satha kaise haije ke marijom ka upachara karana hai| 
kisi kshetra mem jaham bimari ki upasthiti ka pata na chale jaba pancha varsha ka marija ho ya gambhira rupa se sharira mem pani ki kami ho jae ya tivra pani jaise atisara se nrityu ho taba haije ki eka ghatana ka sandeha kiya jana chahie|
eka kshetra mem jaham kahim haija ka yakayaka udvega hota hai koi bhi pancha varsha ka marija ya tivra pani jaise atisara ki adhika vikasita hai ulti hoti ya bina ulti ke sh(vishva svasthya sangathana 1993bi) 
yadyapi haije ka upachara kiya ja sakata hai, yaha tikakarana aura masa kemotherepi se niyantrita nahim ki ja sakati| 
haije ke niyantrana karane vale ina manadandom mem svasthya vijnana shiksha ki bhumika jatila hai|
haije se pida़ita sandehapurna marijom ko aise sthana mem upachara kiya jana chahie joki isi uddeshya ke lie sahi kiya gaya hai|
kisi kshetra mem jaham haija ka yakayaka udvega sambhava hai, apurti karata hai aura udvega ki prathama avastha para hi sakriya hone ke ilae udvega ko niyantrita aura upachara karane ke lie sthaniya rupa se sangrahita rakhana chahie|
janata ko kisi eka roga ke utpanna hone ke khatare ke bare mem jagaruka karana chahie|
atisara ke marijom ko shighra hi spashtakarana ke lie bhejana surakshatmaka mapadanda ve le sakate haim|
haija ke naidanika upachara para eka nai suchana ke lie dekhem – vishva svasthya sangathana (1993bi)|
apatakala mem sankramaka bimari ke niyantrana ke lie benasana (1945), perina (1996), medisinala sansa phrantiyarsa (1997e), spihara projekta (2000) ko dekhom|
bahuta jyada bhida़ vale kshetrom mem rahane vale logom mem haija bahuta teji se phaila sakata hai| 
eka turanta upachara karane ki suvidha sthapita karani chahie|
marijom se alaga, jo loga mulakata karate haim suvidha ko kiya jana chahie unake lie jo dekhabhala karate haim|
sangrahita pine ka pani, mukta avashishta klorina ke kama se kama 0.2 (miligrama) ema ji se hara eka litara ko sapha kiya jana chahie|
klorina ki isa matra (gadha़apana) 0.05% (0.5 ema ji eka litara para) dhone ke lie, pani ke satha sodiyama haipokloreta aura kailshiyama haipokloreta ko milana chahie| 0.2% (2 ji eka litara para) divarem aura pharsha dhone ke lie, 1% (10 ji eka litara para) dushita bistara va kapada़e ko asankramita karane ke lie aura shauchalayom ki saphai ke lie|
prabhavita kshetrom mem yadi sambhava ho to kuom ko dhaka kara rakhana chahie|
pratyeka pani se bhari balti ke satha sodiyama haipokloraita aura kailshiyama haipokloraita ka parikshana karane ke lie kisi ko niyukta karana|
adarsha rupa se haija ko dura karane ke sabhi mapadanda pratyeka kume para jaba pani bhara ho hone chahie|
ghatanaom ka pata lagane ke lie svasthya adhikariyom ko roja gharelu upakaranom ka pata lagana chahie| 
yadi eka bimari ka achanaka udvega hua hai to logom ke juda़ava ko roka diya jana chahie|
jo dushita bhojana bajarom mem becha jata hai, unhe kama karane ke lie savadhani ke mapadanda़ ko chalu rakhana chahie|
ishchirikiya koli ki upasthiti ke ilae pani ke namunom ka parikshana|
parikshana kharaba, pradushana aura baiktiriya ki sambhavya upasthiti joki atisara ka karana hai, ki tarapha sanketa karate haim|
haije ki upasthiti ko nishchita karane ke lie yadi sambhava hai to prayogashala parikshana ke lie malamutra namune bhejem| 
mahamari aura bhi badatara hoti ja rahi hai ya janata svasthya manaka eka sakaratmaka prabhava rakha rahe haim, isako ankane ke lie chikitsalayom aura upachara kendrom para achchhe rikaॉrda rakhana (ghatanaom ora nrityu ki sankhya) madada karega| 
shivira ke nakshe para bimari ke udvega ko chinihta karane ke lie marijom ka lekha jokha prayoga karo|
varlda langa phaundeshana, eshiya mem prathama kendra ko malti draga resisatenta tyubarakolosisa se lada़ne ke lie parinama deta hai| 
da tropikala dijija phaaundeshana, da varlda langa phaaundeshana ke satha sajedari mem hai aura manila mem suvidha kekhulane paradusare paisa lagane vale sajedara jashna manate haim|
da varlda langa phaundeshana ne yaha ghoshana ki ki eka anudana praptakarta, da tropikala dijija phaaundeshana prathama antarrashtriya tyubarakolosisa sentara eshiya mem khola, malti draga resisatenta tibi (emadiara – tibi) se lada़ne ke lie|
da steta ऑpha da arta suvidha eshiya aura philipinsa mem adhika avashyaka aushadhiyom ki tivra apurti, prashikshana, shodha, upachara pradana karengi|
naya kendra, tibi tritamenta sentara, eka tibi prayogashala aura eka naya treninga sentara ediara tibi mem sutradhara karega, emadiara tibi bimari ki sabase khataranaka aura teji se badha़ne vale rupom mem se eka hai| 
mai 2008 mem chikitsa ke lie naye prashikshana pathyakrama shuru kiye jayenge| 
shighra pata lagana aura upachara dena tibi se lada़ne ke lie sabase prabhavi hathiyara siddha hue haim lekina shuru ka kama karane ke lie takaniki, arthika aura manava sansadhanom ki bahuta jyada kami hai| 
eshiya ke asa pasa aura philipinsa mem tibi se lada़ne ke lie lage hue eka takaniki rupa se unnata kendra ke nirmana mem sahayata dene ke lie varlda langa phaundeshana garva hai| 
hama vishvasa karate haim ki yaha eka aisa sthana hoga jaham tropikala dijija phaundeshana ke bahuta adhika gunavana yogya loga, kshetra ke marijom ke labha ke lie apana jnana unnata va banta sakate haim|
kshaya roga – eka bimari jise roka bhi ja sakata hai aura usaka upachara bhi kiya ja sakata hai, isaka prabhava pure sansara mem lagabhaga 80 lakha logom para pada़ta hai|
12 mahine ke samaya ke upachara ke upara,eka bina upachara kiya vyakti sakriya tibi ke satha 10 se 15 dusare logom ko prabhavita karega| 
pratyeka varsha dasa lakha se jyada loga bimari se marate haim|
bimariyom ka chhatha netritva karane ka karana tibi hai aura philipinsa ke madhya nrityu ka chhatha netritva karane ka karana hai|
sansara ke 45 rashtrom mem kuchha samaya purva, intaraneshanala hailtha epidemika ऑpha tibi ko joda़te hue eka nai strena – vibhinna prakara ki davaiyom ke dvara haija ko rokane ke vistara para likhita pramana diye ja chuke haim|
ina chetanayukta sankhyiki ko prakasha mem lane ke lie, kendra bimari se lada़ne ke lie eka naya avishkarita kadama utha chuke haim| 
yaha eka svatah chalita aushadhi krama baddha yojana ko lagu karega – philipinsa ke lie nai hai lekina dusare deshom mem lagu ho chuki hai – joki taji davaiyom ki vishvasaniyata apurti ko nishchita karega, va ghataka tibi ki davaiyam ke vitarana, sangrahana vyavasthapana ki lagata ko kama karega, svasthya dekhabhala ko adhika prabhavi aura kiphayati banayega| 
yaha nai yojana rediyo phrikvensi aidentiphikeshana (araephaaidi) teknoloji niyojita karegi, jo svatah chalita rupa se pahachana karane ke lie chhote kampyutara chipsa prayoga karata hai aura davaiyom ki manga va apurti ko teji se va uchita dhanga se khoja karata hai|
yaha hastachalita yojana para jatila sudhara haim jisake tahata bahudha krama mem galatiyam hongi jisaka parinamasvarupa davaiyom ki kami ya akushala atireka|
intaraneshanala tyubarakolosisa setara phaॉra eshiya ki shuruata varlda langa phaundeshana ke sahayoga se sambhava hui; alaya kaॉrporeshana; da englo kinga phaundeshana; misasa entonite megelensa – phyu; pavalo ara entoniyo dijaina aura kansalatensi inka yunilivara; metarostonerika aura aragasa knsatrakshana| 
varlda langa phaaundeshana ne varlda heltha ऑrgenaijeshana ke vishva vyapta tambaku ke likhita varnana ki prashansa mahatvapurna ghatana ke rupa mem ki|
da varlda langa phaundeshana (dablyuelaepha) ne varlda heltha ऑrgenaijeshana ke globala tobeko epidemika 2008 para dablyuechao ki suchana’ sansara mem ruka ruka kara tambaku se hone vali nrityu ko rokane ke lie rashtriya sahayata ki ora eka simachinha ki taraha ki| 
da dablyuechao riporta blumabargaphailenthopisa ke dvara nishiddha, sansara ki prathama hamesha se hi vyapaka riporta hai – eka sau unasi deshom mem kitane loga tabaku prayoga karate haim aura tambaku prayoga ke svasthya prabhava ko kama karane ke lie unaki sarakara kya kara rahi hai|
dablyu echa o ki riporta ne eka chhah rananiti karyakrama para apani ruparekha di emapavara prithvi para hone vali nrityu ko kama karane ke lie joki isa shatabdi eka biliyana pahuncha saki| 
chhuachhuta ka bhayankara bimari ki chetavani aura rajanitiyom ko rokana|
dusari bara smoka ya kisi aura ki prayoga ki hui sigareta pine se logom ko bachana|
tambaku ke khatarom ke bare mem chetavani|
tambaku ke vijnapana aura badha़ava para dabava mem vivaha ki ghoshana aura taiksa aura mulyom ko badha़ana|
luisa jensa di viala kestala dablyu ela epha cheyaramena ne kaha ; dablyuechao ne eka prapta karane yogya karyakrama, tambaku se hone vale bimari ka virodha karane ki siddha karane ke raste para eka riporta prastavitathi|
isaka chhah rananitiyom ka emapavara paikeja eka naksha hai joki tambaku utpida़na ka avashyaka rupa se miliyana se bhi adhika jivana para dava se pahale kama karane ke lie sansara mem pratyeka desha margadarshaka ke pada se sahayata karata hai| 
dablyu ela epha tambaru prayoga aura phephada़om ki bimari se lada़ne vale karyakramom aura sangathanom ko samarthana dene ke lie shodha (praujekta) vikasita karata hai, bahuta si dusari tambaku niyantrita gatividhiyom mem aura suchana banane mem dablyu echa o aura blumabarga inausietiva ke satha eka sakriya sajedara hai| 
nyuय़ॉrka mem dablyu echa o ka likhita varnana aja eka presa kaॉnphrensa mem bataya gaya tha|
dablyu echa o ke likhita varnana se pata lagaya gaya hai ki prativarsha panchha lakha tambaku se sambandhita nrityu lagabhaga eka sala mem atha lakha taka pahuncha jaegi jaba taka ki sarakara majabuti se karya nahim karati|
una nrityu mem se lagabhaga 80% niche se madhyamika aya vale deshom mem payi jaengi jinamem se chhuachhuta aba dusari ora hai, bada़e rupa se kyonki tambaku ki indastri ki globalayuddha niti ka lakshya una logom mem javana aura vayaska loga haim| 
riporta mem paya gaya hai ki pure sansara ki pancha pratishata janata surakshita hai kisi pancha tambaku ko niyantrita karane vali rajanitiyom ka netritva karane ke dvara jaharila, vivaha ki ghoshana ka vijnapana, janata ki shiksha, dhae se rahita sthana aura anta karane ke sahayoga ki adhikata|
isa sansara mem koi desha puri pancha rajanitiyom ko uchcha stara para aura pure dabava ke satha purnatah lagu nahim kara paya|
likhita varnana joda़te haim joki adhe se jyada sansara ki janasankhya deshom mem rahati hai joki deshom mem tambaraku se hone vali mahamari ke skopa ke bare mem bahuta kama ya koi suchana nahim rakhati|
dablyu ela epha phephada़om ki bimari ke globala chhuachhuta ki bimari ke sambandha mem sthapita kiya gaya, joki 10 lakha logom ko prativarsha marata hai| 
dablyu echa o mem sanlagna karate hue dablyu ela epha ke sajedara intaraneshanalayuniyana agensta tyubarakolosi enda langa dijija, da staॉpa tibi pirtanarashipa aura da kaimpaina phaॉra taubeko phri kidsa ko shamila karata hai|
dablyu ela epha pancha prathamika kshetrom mem kama karata hai : tambaku, kshayaroga, echaaivi/edsa chailda langa helpa aura asthama|
kshayaroga :eka upachara karane yogya bimari joki 2 miliyana logom ko prativarsha marati hai|
do araba loga - vishva ki abadi ka eka tihai - jivanu se sankramita haim jo kshayaroga ka karana hai|.
do lakha loga hara sala kshaya roga se mara jate haim|
isa roga ke ilaja ke lie prabhavi davaem 50 sala se jyada samaya se upalabdha haim, parantu hara 15 sekanda mem, vishva mem koi eka kshayaroga se mara jata hai|
isase bhi adhika khataranaka, prati dina pratyeka sekenda eka vyakti ko tibi se naya sankramana hota hai|
ilaja na kie jane para, eka kshayarogi prativarsha ausatana 10 se 15 anya vyaktiyom ko sankramita karata hai|
vishva ke 80 pratishata kshayaroga ke mamale 22 hai-bardanda vikasashila deshom mem kendrita haim, parantu vishva ka koi bhi kona surakshita nahim hai|
bhramanashila abadi, HIV/AIDS ki mahamari, tatha roga ki pratirodhi upabhedom mem bahuta adhika vriddhi ne vishva svasthya sangathana (WHO) ko 1992 mem kshayaroga ko vaishvika apatakala ghoshita karane ko tatpara hona pada़a|
kaha jata hai ki 22 hai-bardanda deshom mem se 9 aphrika mem haim, aura ki aphrika echaaivi ke satha jine valom mem se 60 pratishata ka ghara hai, aphriki svasthya mantriyom ne kshayaroga ko aphrika mem 2005 mem apatakala ghoshita karake suta ka palana kiya|
dabalyu echa o ka lakshya hai 2005 taka 70% naye kshayaroga ke mamalom ko dhundhana aura usamem se 85% ka nidana karana|
lekina, adhika sthaniya aura antarrashtriya prayasom ke bavajuda mem kevala sata hai bardanda deshom ke ina lakshyom ko prapta karane ki sambhavana hai|
vastava mem, sabase hala ke deta haim ki vishva stara para, nae sankramaka rogiyom ka kevala 53% hi paye jate haim|
hatostahita karane ke bajaya, antarrashtriya samudaya ne kshayaroga ko rokane ko karya ko shakti pradana ki hai|
dablyuechao, tibi roko bhagidari , aura anya rashtriya aura antararashtriya ejensiyom ne eka mahatvakankshi dasa varshiya yojana vikasita ki hai joki tibi ke antima unmulana ke lie nimva rakhane jaisa hoga|
isa vaishvika yojana ke tibi ko 2006-2015 taka rokane ke lie karyanvayana agale dashaka taka 14 lakha jana bachaega|
phaundeshana apane sahayogiyom ke satha ina lakshyom ko prapta karane tatha - aura adhika karane ke lie kama jari rakhega|
varlda langa phaundeshana aura usake sahayogiyom ne 22 deshom mem, jaham duniya ki tapedika mamalom ka 80 pratishata paye jate haim para dhyana kendrita kara rahe haim|
sahayoga ke satha hama ina deshom mem globala yojana ko lagu karane tatha globala yojana mem dekhe gaye abhinava samudaya samadhana ka samarthana karane ke lie rashtriya kshayaroga niyantrana karyakrama ke satha kama kara rahe haim|
prativarsha 30 lakha se jyada bachche tivra shvasana sankramana jaise ki nimoniya, kshayaroga, echaaivi sambandhi phephada़om ki bimari, tatha asthama se pancha varya ki ayu taka pahunchane se pahale hi mara jate haim|
ina bachchom se adhikansha gambhira nimoniya se marate haim jo vikasita deshom ki tulana mem vikasashila deshom mem pancha guna adhika samanya hai, eka nrityu dara ke satha jo ki 10 se 50 guna adhika hai|
vikasita duniya ke bhaga mem tikakarana tatha sasti aura karagara entibayotika davaom ki upalabdhata ki vajaha se nimoniya kama hota hai tatha kama ghataka hai|
bahuta se kama aya vale desha tikom ya pratijaivikom ko vahana nahim kara sakate, tatha prayasa isake alava aparyapta dava vitarana se avasrddha hote haim|
isake alava, vaham amataura para bachapana mem hone vale nimoniya ke ilaja ke lie koi spashta protokaॉla tatha upachara nahim hai, isalie yadi bimara bachche eka apatakalina kaksha mem maujuda hai, to unaka sakshya adharita takanika ka prayoga ka karate hue ilaja nahim hoga|
sanyukta rashtra sahasrabdi vikasa lakshyom mem se eka 1990 aura 2015 ke bicha pancha varsha se kama ayu ke bachchom ki nrityu dara ko do-tihai taka kama karana hai|
isa duniya phephada़e phaundeshana antararashtriya prayasom antarrashtriya kendriya kshaya roga aura phephada़e ki badha़ rahi bimari ke phephada़e bala svasthya pariyojana ke khilapha isa taraha ke rupa mem, bachche phephada़om ki sehata mem sudhara karane ke lie navina drishtikona ka samarthana karake isa lakshya taka pahunchane ke lie karane ke lie yogadana deta hai
phephada़e bala svasthya pariyojana bhi bachche nimoniya se prabhavi ho sakata hai manaka mamale prabandhana aura anya prashikshana aura prabandhana takanikom kshayaroga niyantrana ke lie vikasita ki hai ki prayoga ka pradarshana kiya.
malavi mem, jaham prarambhika pariyojana bachchom mem pancha ke antargata 20-25% se sabhi mamalom ki 10.3% karane ke lie jagaha kama ho gai thi, nimoniya mautom liya.
parikshana ki eka kisma aba elarji shartom ka nidana karane mem maujuda haim, yaha jnata elarji ki pratikriya ke lie tvacha ka parikshana ya maujudagi aura allergen ke starom vishesha IgE ke lie rakta ka vishleshana shamila haim.
elarji ke lie upachara elarji vali vastu se parihara, entithistemainsa steraॉeda ya anya mukha se li jane vali davaom ke prayoga, elarji vali vastu se pratikriya ko asamvedanashila karane ke lie inyunothairepi, aura lakshita therepi shamila haim|
avadharana elarji mulatah 1906 mem vinija़ bachchom ka chikitsaka klemensa vaॉna pireta dvara, yaha dekhane ke bada ki usake kuchha rogi dhula, paraga, ya kuchha khadya padarthom jaise samanya rupa se ahanikara vastuom ke lie atyanta samvedanashila the pahachana mem ai|
pireta isa tathya ko elarji grika shabda elosa se kaha jisaka artha hai anya tatha eragaॉna ka artha karya|
aitihasika rupa se, atisamvedanashilata ke sabhi rupom elarji ke rupa mem vargikrita kiya gaya hai, aura sabhi ke lie pratiraksha pranali ke eka anuchita sakriyana ko karana mana gaya tha|
bada mem, yaha spashta ho gaya ki kuchha anya roga tantra, pratiraksha pranali ke eka beka़ayada sakriyana ke lie ama kada़i se juda़e the|
1963 mem, eka nae vargikarana yojana philippusa jaila aura raॉbina kombsa dvara banai gayi atisamvedanashilata ki pratikriyaom ke chara prakarom ka varnana kiya gaya tha, jo prakara I se prakara IV samvedanashilata ki taraha jani jati haim|
isa nae vargikarana ke satha, isa shabda elarji kevala prakara I
elarji ke tantra ko samajane mem eka bada़i saphalata inyunoglobyulima i (IgE) namaka shreni ke pratirakshi ki khoja thi – kimishiga ishija़aka aura sahakarmi 1960 ke dashaka mem IgE ko alaga karane va varnana karane vale pahale vyakti the|
elarji prabhavita anga naka ke samanya lakshana nasika nyukosa ki sujana (rinathisa ki elarji) 
ankhom mem lalima aura kanjaktiva mem khujali (kanjaktiva ki elarji)
eyaraveja chhinkane, khansate, bronkokaॉnsatrikshana, gharagharahata aura dispaniya, kabhi kabhi asthama ke sidhe hamale, gambhira mamalom mem enjiodema namaka sujana ke karana earave kaॉnsatriktsa|
kanom ka bhara bhara lagana, sambhavata: darda, aura kastaichiyana tyuba se nikasi ke abhava ke karana uncha sunana|
jatharantra sambandhi marga peta darda, phula, ulti, dasta
bahuta se elarji vale padartha hava mem uda़ne vale hote haim jaise dhula ya paraga|
ina mamalom mem, lakshana samparka mem kshetrom mem hava ke satha, ankhom, naka aura phephada़om jaise utpanna hoti haim.
udaharana ke lie, elarji yukta rinitisa, jo ki he phivara ke rupa mem bhi jana jata hai ke karana se naka mem bechaini, chhinkana tatha ankhom mem lalima evam khujali hoti hai|
sansa se andara jane vale elarji padartha esthametika lakshana bhi prakata kara sakate haim, jo vayumarga ki sikuda़na (braॉnkonsitrikshana) aura phephada़om mem shleshma ka utpadana badha़na, sansa ki kami (dispinia), khansana aura gharagharahata ke karana hote haim|
eka tarapha ina parivesha elarji vale padarthom se, elarjika pratikriyaem khadya padarthom, kida़e ke katane se, aura espirina ki taraha davaom ki pratikriyaom, aura penisilina jaise entibayotika ka parinama ho sakati hai|
bhojana se elarji ke lakshanom mem shamila haim, peta darda, phulana, ulti, dasta, khujali yukta tvacha, aura tatha hivsa ya enjiyodema ke dairana tvacha ki sujana|
khadya elarji se shayada hi shvasa (esthametika) pratikriyaem, rinitisa ka karana hoti hai|
kita ka katana, entibayotiksa aura kuchha davaem bhi eka pranaligata elarji pratikriyautpanna karati haim jo ki enaphileksisa bhi kaha jata hai; pachana tantra, shvasana pranali, aura sanchara pranali sahita kai pranaliyam prabhavita ho sakati haim|
gambhirata ki dara ke adhara para, yaha tvacha sambandhi pratikriyaom, braॉnkonsitrikshana, idema, haipotenshana, koma aura mauta ka karana bhi ho sakata hai|
isa prakara ki pratikriya achanaka shuru ho sakati hai ya shuruata mem deri ho sakati hai|
elarji ki pratikriya ke isa prakara ki tivrata mem aksara epinephrina ke eka injekshana ki avashyakata hoti hai, kabhi kabhi eka epi – pena svatah injektara ke nama se jane jane vale upakarana ke madhyama se|
tivragrahita ka svabhava isa taraha ka hai ki pratikriya drishya rupa se sabasaidinga ho sakate haim, lekina samaya ki eka lambi avadhi mem punaravritti ho sakati hai|
tvacha ke samparka mem ane padartha jaise ki laiteksa elarji pratikriyaom ke bhi samanya karana haim, samparka tvachashotha ya egjima ke rupa mem jane jate haim|
skina elarji aksara kharoncha, athava sujana evam tvacha ke bhitara jalana, jo eka vhila evam phleara pratikriya jo haivsa tatha enjiodema ki visheshataom ke rupa mem jana jata hai|
elarji ke jokhima karakom ko do samanya shreniyom mem rakha ja sakata hai, namatah mejabana aura paryavaraniya karakom|
mejabana karakom mem shamila haim anuvanshikata, linga, jati aura unra, anuvanshikata aba taka ka sabase mahatvapurna hai|
elarji vikarom ki ghatanaem hala hi mem badha़ gai haim, halanki, inaki akele anuvanshika karakom dvara vyakhya nahim ki ja sakati hai.
isa chara mukhya paryavaraniya karaka, shuruati bachapana ke daurana sankamaka bimariyom mem sammilana, paryavarana pradushana, allergen starom, aura ahara mem badalava hai| 
elarji roga parivarika haim: abhinna juda़vaom eka hi elarji rogom ke hone ki 70% sambhavana hai, gaira abhinna juda़vaom mem eka hi elarji hone ki 40% sambhavana hai|
elarjika mata pita se bachchom ke elarjika hone ki sambhavana adhika hai, aura unaki elarji una se jo gaira elarjika mata pita se hue hom adhika takatavara hone ki sambhavana hai |
halanki kuchha elarji vanshavaliyom satha sangata nahim haim, mata pita, jo mungaphali ke lie elarjika hom ke bachchom ko, jo regavida se elarji ho sakati hai, ya bhai bahanom se alaga jinhe kisi anya chija se elarji hai|
aisa lagata hai ki elarji ke vikasita hone ki sambhavana virasata aura pratiraksha pranali kama karane mem kuchha aniyamitata ke karana hai, lekina kuchha vishishta elarji padartha jo elarji ke vikasa ke karana hote haim, nahim haim|
elarji samvedikarana ka jokhima aura elarji ke vikasa unra ke satha badalata rahata hai, chhote bachchom mem sabase jyada khatare ke satha|
kai adhyayanom ne darshaya hai ki IgE stara bachapana mem sabase adhika haim aura 10 aura 30 varsha ki unra ke bicha teji se girate haim|
he phivara ka sarvadhika prachalana bachchom aura yuva vayaskom mem tatha asthama ki ghatanaem 10 se kama ke bachchom mem uchchatama haim|
kula mila kara, lada़kom mem elarji ke vikasita hone ki sambhavana lada़kiyom ke mukabale jyada hai, halanki, kuchha bimariyam namatah asthama yuva vayaskom mem, mahilaom ke prabhavita hone ki sambhavana adhika hai|
jatiyata, lekina nasliya karakom mushkila ho gaya hai paryavaraniya prabhavom aura parivartana pravasa ki vajaha se alaga karane ke lie kuchha elarji mem eka bhumika nibha sakate haim
dilachaspa bata hai ki, yaha sujaya gaya hai ki alaga alaga anuvanshika loci asthama ke lie uttaradayi haim, vishesha rupa se, kokeshiyana, hispainika, eshiyana, aura aphriki mula ke logom mem jimmedara hai|
antarrashtriya bhinnataem janasankhya mem vyaktiyom ki sankhya ke satha sambaddha kiye gaye haim jo elarji se grasta haim|
elarji roga adhika paramparika evam krishi adharita deshom ka tulana mem audyogika deshom mem adhika samanya haim, tatha shahari abadi mem gramina abadi ke viruddha elarjika rogom ki dara jyada hai, halanki yaha vibhinnataem kama paribhashita ho rahi haim|
elarji ke lie eksapojara vishesha rupa se prarambhika jivana mem, elarji ke lie eka mahatvapurna jokhima karaka hai|
etopika elarji mem vriddhi ke lie vartamana mem sukshmajivom ke prati eksapojara mem vriddhi sarvadhika satyabhasi vivarana hai|
bachche jo ki bada़e parivarom ya bhida़ gharom, ya de keyara mem rahate haim, mem elarjika bimariyom ki ghatanaem kama hoti haim|
bachapana ke daurana eka rishta baiktiriya aura vayarasa ke lie jokhima ke bicha prastavita kiya gaya hai|
elarji ke vikasa ke viruddha sanrakshana jise - svachchhata parikalpana kaha gaya hai|
endotoksina aura baiktiriya ke anya ghatakom ke prati eksapojara etopika rogom ko kama kara sakata hai.
endotoksina eksapojara sapheda rakta koshikaom (lyukosaitsa) jo ki rakta mem sancharita hote haim mem se jvalanashila saitokainsa jaise tinapha-a, iphana intaralyukina 10 tatha intaralyukina 12 ke utsarjana ko kama kara deta hai|
kuchha maikroba – sensinga protina, sharira mem koshikaom ki sataha para paya jane vala tola jaise grahaka ke rupa mem jana jane vala samvedana-riseptarsa ki taraha hai jo isa prakriya mem sanlipta mana jata hai|
gutavarma aura isi prakara ke parajivi vikasashila deshom mem anupacharita pine ke pani mem upasthita haim, aura vikasita deshom ke pani mem pani ki apurti mem niyamita klorineshana aura pani ke shuddhikarana taka upasthita the|
hala ke anusandhanom ne dikhaya hai ki antra kida़e jaise kuchha ama parajivi, (udaharana hukavarma ) ke anta ki divara mem sravita rasayanom (aura isalie khuna ki dhara) pratiraksha tantra ko dabane aura parajivi para hamala karane se sharira ko rokate haim|
yaha svachchhata parikalpana siddhanta ke eka naye drishtikona ko badha़ava deta hai - ki adami aura parajivi ke saha vikasa ne eka aisi pratiraksha pranali ki ora unmukha kiya hai jau parajiviyom ki upasthiti mem hi sahi dhanga se kama karati hai|
unake bina, pratiraksha pranali asantulita evam atisamvedanashila ho jata hai|
vishesha rupa se, anusandhana sujate haim ki shishuom mem elarji anta vanaspati ki deri sthapana ke satha mela khati ho sakati hai|
tathapi, isa siddhanta ka samarthana karane ke lie anusandhana paraspara virodhi haim, china aura ithopiya mem kie gaye kuchha adhyayanom mem intestainala kida़e se sankramita logom mem elarji mem vriddhi ka pradarshana kiya gaya hai|
kuchha elarji ke upachara mem kuchha kida़e ki prabhavashilata ki jancha karane ke lie naidanika parikshana shuru kiya gaya hai|
aisa ho sakata hai ki shabda 'parajivi' anupayukta ho, aura ho sakata hai eka aba taka pahale se na socha gaya simbayosisa isa vishaya para adhika janakari ke lie karya kara raha ho. helaminthika thairepi dekhem|
elarji pratikriyaom ki patha phijiyolaॉji ko do charanom mem vibhajita kiya ja sakata hai|
pahala elarjika padartha ke prati pradarshana ke tatkala bada ki eka tivra pratikriya hai|
isa charana ya to dhima pada़ sakata hai ya eka dera charana pratikriya jo eka pratikriya ke lakshanom ko lamba khincha sakata hai aura utakom ko nukasana ka parinama de sakata hai, ke rupa mem pragati kara sakata hai|
yadi eka hi elarji padartha ke prati bada mem pradarshana hota hai, to elarjana aejii anuom jo masta koshika ya besophilsa para hote haim ke satha juda़ sakata hai|
jaba eka se adhika aejii-grahi sankula usi elarjenika anu ke satha antahkriya karate haim, tatha samvedi koshika ko sakriya kara dete haim taba aejii aura ephasi grahiyom ki apasa mem juda़ne ka ghatana hoti hai|
tivra pratikriya ke rasayanika madhyasthom manda hone ke bada, leta pheja़ pratikriyaem aksara ho sakati haim|
yaha anya lyukokaitsa jaise nyutrophilsa, limphokaita, osinophilsa tatha maikrophejeja़ ke prarambhika saita ki tarapha pravasa ke karana hai|
yaha pratikriya ama taura mula pratikriya ke 2-24 ghante bada dekhi jati hai|
masta koshikaom se sautokainsa bhi dirghakalika prabhavom ki jada़ta mem eka bhumika nibha sakate haim|
asthama mem dekhi gayi leta pheja़ pratikriyaem anya elarjika pratikriyaom se thoda़i bhinna hoti haim, halanki, ve abhi bhi osinophilsa se madhyasthom ke nikalane ke karana hota hai abhi bhi TH2 koshikaom ki sakriyata para nirbhara hai|
elarji ka parikshana karane vali mashina ko lekalainda vayu sena adde mem naidanika immunologi prayogashala mem sanchalita kiya ja raha hai|
elarji roga ka nidana ke pahale pushti ki ja sakati hai, upalabdha lakshanom ke anya sambhavita karanom ko dhyana se vichara kiya jana chahie|
udaharana ke lie, vesomotora rinitisa kai buraiyom mem se eka hai jo ki ki elarjika rinitisa ke lakshanom se milati hai, peshevara vibhedaka nidana ki avashyakata ko rekhankita karate hue|
eka bara jaba asthama, rinitisa, tivragrahita, ya anya elarji roga ka nidana kiya gaya hai, elarji ke karanatmaka ejenta ki khoja ke lie kai tarike haim upalabdha hai|
elarjana vishishta aejii entibaॉdi ki upasthiti ko mapane ke lie elarji tvacha parikshana rakta elarji parikshana se adhika pasanda kiya jata hai kyonki yaha adhika samvedanashila evam vishishta, prayoga karane mem asana, tatha kama kharchila hai|
rogi ki tvacha mem chhote panchara ya chubhana banane ki shrrinkhala ke karana tvacha ka parikshana panchara parikshana aura chubhana parikshana ke rupa mem bhi jana jata hai| 
sandigdha elarjana tatha/ ya unake sara ki chhoti matra (paraga, ghasa, ghuna protina, mungaphali ka satva, adi) tvacha para pena ya dai se chinhita hokara saitom ke prati parichita ho jate haim| (syahi/dai ko dhyanapurvaka chayanita hona chahie kyonki yaha svayam bhi elarjika pratikriya ka karana hai)
eka chhoti si plastika ya dhatu ki divaisa tvacha ko panchara karane ke lie ya chubhana ke lie prayoga kiya jata hai|
kabhi kabhi, elarjana ko rogi ki tvacha mem eka sui aura sirinja ke satha antahkshipta injekta kiya jata hai| 
parikshana ke lie ama kshetrom mem hatha tatha pitha shamila haim|
yadi rogi kisi tatva ke prati elarjika hai, to eka drishya jvalanashila pratikriya amataura para 30 minata ke bhitara ghatita hogi|
adhika samvedanashila rogiyom mem yaha pratikriya tvacha ka thoda़a lala hone se eka paripurna haina taka ho sakati hai| ( vhila evam phleyara kahi jane vali)|
tvacha chubhana pariksha ke parinamom ki vyakhya samanya rupa se elarjista dvara gambhirata ke eka paimane para, + / - arthata baॉrdara laina pratikriya, 4 + arthata eka bada़i pratikriya hone ke satha ki jati hai|
teji se, elarjista vhila evam phleara pratikriya ke vyasa ko rikaॉrda kara rahe haim tatha mapa rahe haim|
achchhi taraha prashikshita elarjista ki vyakhya aksara prasangika sahitya dvara nirdeshita hai|
kuchha rogi vishvasa kara sakate haim ki unhone avalokana se apani elarjika samvedanashilata nirdharita hai ki hai, para eka tvacha parikshana ko marija avalokana ke mukabale elarji ka pata lagane ke lie adhika behatara dekha gaya hai|
yadi eka gambhira jivana ke khatare vali enaphilaiktika pratikriya mem mulyankana ke lie eka marija ko laya gaya hai mem, kuchha elarjista pahale tvacha chubhana parikshana karane se pahale eka prarambhika rakta parikshana ko pasanda karenge|
tvacha parikshana eka vikalpa nahim ho sakata hai agara marija ya vyapaka tvacha roga hai tatha kai dinom ke lie entithistemainsa se paraheja nahim kiya hai|
vibhinna khuna elarji parikshana tarikom bhi vishishta padarthom se elarji ka pata lagane ke lie upalabdha haim|
parikshana ke yaha prakara eka kula aejii stara ko mapate haim - rogi ke sirama ke bhitara nihita aejii ke eka anumana ke anusara|
yaha rediyometrika aura kalarametrika inyunoesesa ke prayoga ke madhyama se nirdharita kiya ja sakata hai|
rediyometrika esesa ke rediyoelarjorbasenta parikshana (raasata) prakriya, jo aejii – baindinga (enti aejii) rediyodharmi aisotopa ke lebala vale entibaॉdi ka prayoga rakta mem aejii pratipinda ke stara taya karane ke lie karata hai|
anya nae tarike rediyosakriya aisotopsa ke sthana para kalarametrika ya phlorometrika takanika ka upayoga karate haim|
kuchha skrininga parikshana tarike, gunatmaka pariksha parinama upalabdha kara sakate haim sandigdha elarji samvedikarana ke satha rogiyom mem eka ham ya nahim javaba ke satha|
aisi hi eka vidhi eka bada़e adhyayana ke anusara 70.8% ki samvedanashilata aura 72.6% ke eka sakaratmaka bhavishyavani mulya ki hai|
kama aejii stara samanyatah sansa se ane vale elarjana ke lie samvedikarana ko bahara karane ke lie paryapta nahim hai|
sankhyiki vidhiyom, jaise aeosi karva, pridiktiva mulya ganana, aura sambhavana anupata ka vibhinna parikshana tarikom ka eka dusare se sambandha ki jancha karane ke lie istemala kiya ja raha hai|
ina vidhiyom ne darshaya hai ki eka uchcha kula aejii vale rogiyom mem elarji samvedikarana ke eka uchcha sambhavana hai, lekina eka savadhani se chune gae elarjana ke lie vishesha elarji parikshanom ke satha age ki jancha aksara avashyaka hai|
yaha lekha ya bhaga mukhyatah sanyukta rajya amerika ke satha sambaddha hai tatha yaha isa vishaya ke eka vishvavyapi drishtikona ka pratinidhitva nahim karata hai|
elarji paristhitiyom ke ilaja ke lie istemala mem ane vale chikitsa upacharom mem bhari sudhara kiya gaya hai|
paramparika ilaja aura elarji ke prabandhana mem paristhitiyom mem elarjana se bachana anyatha anyatha jokhima ko kama karana shamila hai|
udaharana ke lie, billi elarji vale logom ko una se bachane ke lie protsahita kiya gaya| 
halanki parihara lakshanom ko kama karane jivana ke khatare enaphileksika ko kama karane mem madada kara sakati hai, yaha paraga tatha samana hava se hone vali elarji ke lie prapta karana kathina hai|
sakhta parihara abhi bhi prabandhana mem eka bhumika hai, halanki, aura aksara khadya elarji ke prabandhana mem prayoga kiya jata hai| 
kai pratirodhi davaem koshikaom aura digrenyuleshana prakriyaom ke sakriyana ko rokane ke lie elarji madhyasthom, ki karravai blaॉka karane ke lie upayoga ki jati haim| 
inamem entithistemainsa, krostisona, deksomethasona, haidrokoristosona, epinephrina (edrinalina) thiyophilina aura kromolina shamila haim|
tathapi, mahilaom ke bicha 40 pratishata echaaivi sankramanom mem se kevala 0.5% yaunakarmiyom mem hote haim| 
dragsa elarji ke lakshanom ko kama karane mem madada karata hai, aura tivra tivragrahita se ubarane mem anivarya haim, lekina elarji vikarom ke purane upachara mem chhoti bhumika nibhate haim| 
asamvedanashilata ya haiposensitaijeshana eka upachara hai jisamem rogi ko paristhiti mem dhire dhire elarjana ka uttarottara adhika khuraka se tika lagaya jata hai|
yaha ya to gambhirata ko kama kara sakate haim ya puri taraha atisamvedanashilata ko samapta kara sakate haim|
etopija़ mem dekhe gaye atyadhika aejii utpadana ko rokane ke lie, yaha aejiji pratipinda utpadana ke pragatishila skivinga mem rahata deta hai| 
eka artha mem, vyakti elarjana ki badha़ti hui matra ko badha़ata hai|
adhyayanom ne nai elarji ke vikasa ko kama karane mem dirghakalika prabhava aura inyunotherepi ke surakshatmaka prabhava ka pradarshana kiya hai|
meta - vishleshana ne bhi bachchom mem elarjika rinitisa aura asthama mem ilaja ki prabhavakarita ki pushti ki hai| 
rochestara mem meyo klinika ki samiksha ne, elarjika rinatisa evam knjakchivaitisa, asthama ke elarjika prakara, tatha kitom ke katane ki achchhe vaijnanika adhyayana para adharita ki prabhavakarita ki pushti ki| 
isake atirikta, rinitisa aura asthama mem injekshana inyunotherepi ki chikitsiya prabhavakarita satha hi suraksha pradana ki pushti karem rashtriya aura antararashtriya dishanirdeshom ki sipharishom ka palana kara rahe haim| 
inyunotherepi ka eka dusara rupa monoklonala enti antahshira injekshana- aejii entibaॉdi shamila hai| 
yaha bainda tu phri tatha bi - sela se juda़e aejii , apane vinasha ka signala dete hue| 
ve pahale se hi basophilasa aura mastula koshikaom para ephasi riseptara ke lie badhya aejii ki badhyata nahim hai, kyonki yaha elarji bhada़kau pratikriya protsahita karega. 
isa varga ka pahala ejenta omaliज़umaba hai. 
halanki imayunotherepi isa prakara ki bahuta etopi ke kai prakara ke upachara mem karagara hai, yaha khana elarji ke satha logom ke bahumata ke upachara mem upayoga nahim kiya jana chahie. 
eka tisara prakara, sabaliguala imayunotherepi, jo khadya padarthom aura nivasi baiktiriya jaise gaira maukhika pratiraksha sahishnuta ka labha-rogajanaka pratijanom leta eka maukhika rupa-prashasita chikitsa hai. 
yaha chikitsa vartamana mem yuropa mem elarji upachara ke 40 pratishata ke lie khatom. 
sanyukta rajya amerika mem, sabaliguala imayunotherepi aura paramparika ellerjisatesa bicha ka samarthana prapta hai, jo elarji ka ilaja chikitsakom ne samarthana kiya hai. 
eka prayogatmaka upachara, enjaima potenatiatida (ipidi), dashakom ke lie, lekina koshisha ki gai hai amataura para prabhavi rupa mem svikara nahim hai. 
ipidi olarjina aura eka enjaima, bita ke dilushana ka upayoga karata hai-galukuronidesa hai, jo karane ke lie ti viniyamaka limaphoketasa disensetaijeshana, ya niche paksha-viniyamana, bajaya samvedikarana dvara pratikriya dene vali haim. 
ya sabita prabhavashilata ki manjuri de di ipidi bhi otoimayuna rogom ke upachara ke lie, lekina phira se koshisha ki gai hai ephadie nahim hai. 
vaikalpika chikitsa mem elarji ke ilaja ke lie kai apane chikitsakom dvara, vishesha rupa se, harbala chikitsa, honyopaithi, paramparika chini dava aura kinesioloji nechuropethika varnita haim. 
vyavasthita sahitya khojom ke meyo klinika dvara 2006 ke madhyama se ayojita kiya, lekha asthama aura upari shvasa patha ke sankramana sahita kai sthitiyom, padha़ai ke saikada़om shamila hai, aura koi antara nahim palesabo ke satha tulana mem kisi bhi vikalpa ke upachara ka koi prabhava ko dikhaya. 
isa lekhaka yaha hai ki bachapana aura kishoravastha bimariyom ke lie honyopaithi ki sabhi prakara ki kathora naidanika parikshanom ke adhara para, vaham hai ki vaikalpika upachara ke prayoga ka samarthana karata hai koi thosa sabuta hai sampanna hua. 
taipa 1 madhumeha, rumeti sandhishotha aura elarji rogom jaise kai bimariyom sujana se sambandhita-ghasa ka bukha़ara aura asthama ko pashchimi duniya mem pichhale 2-3 dashakom mem vriddhi hui hai. 
isake atirikta, echaaivi prasara shahari kendrom se gramina abadi ke lie ghuma raha hai: gramina 58.6%, shahari 41.4%, 72.2% gramina, ki sabase adhika prabhavita haim samuhom mem shamila 27.8% shahari abadi ka eka alaga se. 
vikasashila deshom mem etopi ke mamale ama taura para kaphi kama raha hai. 
10 lakha elarji asthama se janasankhya ka (ke bare mem 3%) dama pida़ita haim. 
asthama ki vyapakata 1980-1994 se 75% ki vriddhi hui. 
vyaptata 39% aphriki amerikiyom mem uchcha gorom ki tulana mem asthama hai. 5.7 miliyana (lagabhaga 9.4%). 
chhaha aura sata sala olds asthama mem 18.4% se 20.9% karane ke lie pancha varshom mem, dara 31% se 24.7% karane ke lie 13 se 14 varsha olds ke lie kama hi samaya ke daurana vriddhi hui. 
1960 aura 1990 ke bicha 3% se 10% karane ke lie bachchom mem badha़ gaya hai vyaptata. 
varsha jahara kita ke karana prati tivragrahita kama se kama 40 mautem. 
ke bare mem 400 mautem tivragrahita penisilina ke karana. 
tivragrahita ke bare mem 220 mamalom aura prati varsha 3 mautem latekasa elarji ki vajaha se haim. 
eka anumana ke anusara 150 logom ko salana tivragrahita khana elarji ki vajaha se mara jate haim. 
1999 aura 2006 ke bicha 48 logom ko mauta ke pancha mahine 85 sala ki unra se lekara mem hui. 
vayaskom ke 15% lagabhaga kita visha hai halke, sthaniyakrita elarji pratikriyaom. 
pranaligata pratikriyaom vayaskom ke 3% mem aura bachchom ke kama se kama 1% hote haim. 
aushadha elarji anaphyalakatika pratikriyaom 400 mautom ka karana penisilina ke lie. 
khadya elarji bachchom ke 2-4% aura vayaskom ke 1-2%. 
mungaphali ke dane aura / ya peda़ nata (jaise akharota, badama aura kaju) elarji ke bare mem tisa lakha amerikiyom, ya janasankhya ka 1.1% ko prabhavita karata hai. 
shishuom ke 5-7% aura vayaskom ke 1-2%. mungaphali elarji mem 117.3% vriddhi 2001 se 2005, inglainda mem eka anumana ke anusara 25700 logom ko prabhavita kara rahe haim karane ke lie manaya gaya. 
(dama, ekjima aura elarji rahinitesa satha) 2.3 miliyana (lagabhaga 3.7%), vyaptata 48.9% dvara 2001 aura 2005 ke bicha badha़ gaya hai. 
halanki anuvanshika karaka maulika etopi roga ki samvedanashilata shasana, etopi mem badha़ta bhi kama eka samaya sima ke andara abadi mem eka anuvanshika parivartana ke bare mem vistara se bataya jana hua hai. 
aisa lagata hai ki kama baiktiriyala aura vayarala sankramana jivana mem pratyaksha ko paripakva pratiraksha pranali dura tiecha1 prakara pratikriyaom se jaldi, anargala tiecha 2 pratikriyaom ke lie hai ki elarji mem vriddhi ke lie anumati agrani socha hai. 
isa svachchhata parikalpana sankramana kama karane ke lie bachchom ko ujagara karata hai ki uchcha jivana stara aura svachchha tarike se sthiti rakhata hai. 
darem aura sankramana akele ke prakara mem parivartana halanki gai elarji roga mem manaya vriddhi, aura hala hi mem sabuta ki vyakhya karane ke lie gaistrointestainala maikrobiyala paryavarana ke mahatva para dhyana kendrita kiya hai asamartha hai. 
aisa lagata hai ki ina sankramanom gambhira tiecha 1/ tiecha 2 viniyamana pherabadala karake unake prabhava lagu hai. 
nae svachchhata ke hayapothessa mahatvapurna tatva bhi endotokasina se khulasa ho, palatu janavara ke lie pradarshana aura eka kheta para badha़te shamila haim. 
jo amerika ke borda elarji aura immunologi (ebiei) dvara pramanikarana ayojita sanyukta rajya amerika ke chikitsakom mem saphalatapurvaka eka manyata prapta shaikshanika karyakrama pura kara liya hai. 
eka elarjisata-imayunolojisata eka chikitsaka ke prabandhana ke lie vishesha rupa se prashikshita aura asthama ke upachara aura anya elarji rogom hai. 
eka elarjisata banana- imayunolojisata prashikshana ke kama se kama nau varsha ke pura hone ki avashyakata hai. 
medikala skula ko pura karane aura eka chikitsa ki digri ke satha snataka karane ke bada, eka chikitsaka to (eka chikitsaka) ya bala chikitsa banane ke lie (eka bachchom ka chikitsaka) banane ke lie antarika chikitsa mem prashikshana ke tina sala se gujarana hoga. 
eka bara eka chikitsaka ina visheshtao mem prashikshana samapta kara diya hai, ve ki pariksha pasa karani hogi ya to ameriki borda bala chikitsa ke (ebipi) ya ameriki borda antarika davai (ebiima) ke. 
inataranisatasa ya jo upa para dhyana kendrita karane ke lie elarji ki visheshata-immunologi to eka adhyayana ke atirikta do sala kama se kama eka phailoshipa, elarji mem bulaya-immunologi prashikshana karyakrama pura ichchha bala roga. 
jo ebiei ke rupa mem suchibaddha haim elarjisata banana- imayunolojisata -pramanita saphalatapurvaka ameriki borda elarji aura immunologi (ebiei) ke pramanita pariksha bita chuke haim, unake phailoshipa ninnalikhita. 
n ne yunaiteda kingadama, elarji samanya dava ya bala chikitsa ke eka sabaisapeshieliti hai. 
kramashah) eka daॉktara eka visheshajna rajistrara ke rupa mem kai varshom ke rupa mem janarala medikala kaunsila visheshajna rajistara ke lie yogyata se pahale kama karata hai (emaarasipi ya emaarasipisiecha snatakottara pariksha prapta karane ke bada. 
elarji sevaom ko bhi imayunolojisata dvara diya ja sakata hai. 
eka 2003 raॉyala kaॉleja ऑpha phijishiyana ke bare mem kya britena mem aparyapta elarji sevaom hone laga rahe the ke sudhara ke lie eka mamale ki riporta prastuta ki. [65] 2006 mem, sadana ko yahova ki hai ki 2007 mem riporta ki eka upasamiti bulai. 
yaha vaise hi hai ki elarji sevaom yahova ke lie eka elarji mahamari ke rupa mem aura isake samajika lagata kya bheja se nipatane ke lie aparyapta the sampanna, yaha kai anya sipharishom ko banaya hai. 
echaaivi ka pahala mamala chennai, tamilanadu 1986 mem yaunakarmiyom ke bicha mem nidana kiya gaya. jalda hi, jancha kendrom ki sankhya mem desha bhara mem sthapita kie gae the ke bada. 
prarambha mem phokasa skrininga videshiyom para, khasakara videshi chhatra tha. dhire dhire, para dhyana kendrita blada bainkom ki jancha karane ke lie para chala gaya. jaldi 1987 taka prayasa pramukha shahari kshetrom mem echaaivi jancha kendrom ki eka rashtriya netavarka sthapita karane ke lie banae gae the. 
echaaivi pratyeka aura hara bharatiya rajya aura kshetra mem phaila gaya hai. 
andhra pradesha, karnataka, maharashtra (visheshakara mumbai), manipura, mijorama, nagalainda, aura tamilanadu mem, 45 jilom ke satha echaaivi ke eka vishesha rupa se uchcha vyapti / edsa hai. 
gova aura gujarata mem mahamari jaise anya rajyom mem kuchha jilom mem kendrita hai. 
bharata mem (nako) ko echaaivi / edsa karyakrama mukhyatah pravasi majadurom, traka chalata hai, yaunakarmiyom aura nasom mem nasha karane valom jaise uchcha jokhima samuhom para dhyana kendrita kiya hai.
bharata ka rashtriya edsa niyantrana sangathana (nako) nae echaaivi jari / 6 julai, 2007 ko desha ke lie edsa ka anumana hai. 
kyonki ve eka vistarita nigarani aura sudhara paddhati para adharita haim ye sankhya kaphi mahamari ki vyapakata ke pichhale anumanom ki tulana mem aura adhika satika haim kama kara rahe haim. 
donom yuenaedsa aura vishva svasthya sangathana ka samarthana nae echaaivi / edsa ka anumana hai. 
isaki satata prayasa isaki mahamari behatara pata karane ke bhaga ke rupa mem, bharata sarakara, 2006 mem, echaaivi para eka khanda / edsa ke rashtriya parivara svasthya sarvekshana mem-3, eka yadrichchhika sarvekshana samanya janasankhya ke adhara para shamila kiya gaya hai. 
deta isa vyayama aura se nigarani kendrom ke netritva mem aura kama anumana naya. 
halanki kula milakara, echaaivi mahamari, vaham rajyom aura janasankhya samuhom ke bicha mem bhinnata hai ki hala ke varshom mem eka sthayi pravritti ko darshata hai. 
achchhi khabara yaha hai ki tamilanadu aura uchcha echaaivi boja jaham prabhavi hastakshepa jagaha mem kai varshom ke lie kiya gaya hai ke satha anya dakshini rajyom mem echaaivi ki vyapti mam giravata ya sthira banana shuru kara diya hai. 
echaaivi ke uttari rajyom mem uchcha vyapti ke chayanita jeba ke satha nae kshetrom mem ubharane ke lie jari hai. 
isamem uchcha vyapakata ke satha 29 jilom, pashchima bangala, uda़isa, rajasthana aura bihara ke rajyom mem vishesha rupa se kara rahe haim. 
yaha ankada़a 2006 hai jo davaom aura jo purushom (emaesaema) ke satha yauna sambandha rakhane vale purushom injekshana logom jaise kai uchcha jokhima samuhom ke bicha echaaivi sankramana mem vriddhi ko darshata hai. 
uda़isa, panjaba, pashchima bangala, uttara pradesha aura kerala jaise chennai, dilli, mumbai aura chandigadha़ aura rajyom ke metro shaharom saba sui se nasha karane valom ke bicha uchcha vyapti dikhao. 
halanki hai yaunakarmiyom ke bicha dakshini rajyom mem echaaivi prasara ka stara girata sujava deta, isa samuha ke bicha samagra vyaptata stara sarpila upara bani hui hai. 
bharata ke lie naya anumana dakshina aphrika aura naijiriya ke niche kahate haim. 
nai sarvekshana paddhati ko kama sankhya ko samajane ki kunji hai. 
pahale, yuenaedsa ne 5.7 miliyana sankhya mem nigarani kendrom para pure desha mem garbhavati mahilaom aura nasha karane valom aura veshyaom chara mahine eka varsha se adhika jaise uchcha jokhima samuhom, parikshana dvara pahunche. 
visheshajnom ka nishkarsha hai ki kyonki sarakara nigarani kendrom mukhya rupa se garibom dvara jaldi ho rahe haim aura yaha rashtriya anumana skewed upara hone ki vajaha se uchcha jokhima samuhom ke dvara. 
navinatama paddhati) aura jo rakta ke namunom ka parikshana aura chehare sahita 200,000 logom (unra 15-54), ke bare mem shamila eka janasankhya adharita sarvekshana, donom nigarani deta lagabhaga (se 400 kendrom ko shamila-desha bhara-chehara sakshatkara, disambara 2005 ke bicha ke lie aura agasta 2006. 
adhika vyapaka, vichara aura svatantra vaishvika sangathanom, bharata ki nai epi-pahachana paddhati dvara vaidhikrita kama sankhya mem aura kai sthaniyakrita mahamari ko darshata hai. 
bharata eka bada़i durghatana ki sambhavana vale desha hai aura 8 pramukha prakritika apadaom hara varsha ki eka ausata ke shikara hai. 
jabaki gambhira bhukampom aura relave, khadanom aura karakhanom mem pramukha durghatanaom samaya samaya para ghatita badha़, chakravata, sukha aura mahamari lagatara kara rahe haim. 
iarasiesa balki anya deshom mem na kevala bharata mem apani gatividhiyom ko rahata bahara vahana karati hai. 
apada rahata karya mem iarasiesa savadhana kiya hai aura vibhinna anubhava 2001 mem gujarata mem ae bhukampa mem 1934 bihara mem bhukampa ke satha shuruata. 
yaha apada pida़itom ke lie karoda़om yeomena seva pradana ki gai hai ina avadhiyom ke bicha. 
rahata mem 1992 ru. 72,20,607.00 aphrika, jo sukhe aura anya apadaom se prabhavita ho rahe haim ke kai deshom ke lie pradana kiya gaya. 
1992 aura 1998, rahata samana layaka rupaye se adhika ke bicha. 32,53,66,460.00 badha़ ke pida़itom ke lie bharata ke kai bhagom mem pradana kiya gaya. 
1993 aura 1999 ke bicha, samaja rupae se jyada kharcha kie. 13,53,00,000.00 ko rahata aura punarvasa pariyojanaom para maharashtra, madhya pradesha aura bilkula pradesha mem tina bada़e bhukampa ke bada. 
aura suraksha balom ke ghayala adami bimara eka ke prathamika dayitvom ke IRCS hai ki paravaha hai. 
dvitiya vishva yudgha ke daurana kuchha 500 prashikshita mahila kalyana adhikariyom ke aspatalom mem kama para jaham ghayala lada़akom ka ilaja kiya ja raha the. 
uni aura buna hua vastra yuropa mem yuddha ke bharatiya kaidiyom ko 20.000 parsala eka saptaha ki dara se ki pika avadhi mem bheja gaya. 
bharata eka bara phira ke daurana 1965 aura 1971 ke pakistani sangharsha, upahara parsala donom bharatiya aura pakistani karane ke lie yuddha ke kaidiyom ko di gai. 
kalyana sevaom jo vishesha rupa se isa kama ke lie prashikshita kiya jata hai iarasiesa mahila kalyana adhikari dvara bimara aura ghayala sainikom ke lie sainya aspatalom mem chalae ja rahe haim. 
reda krasa homa bangaluru mem akshama karmiyom ke lie 1946 mem sthayi rupa se vikalanga ya akshama sainikom ke lie hai, jo dvitiya vishva yudgha ke daurana lada़i gathana kiya gaya tha. 
isake alava, yaha kaidiyom ko paryavekshana aura eka adhikshaka aura arddha karmachari ke margadarshana mem sabhi gharelu arama chikitsa dekhabhala pradana karata hai. 
isa bata ko bhi chikitsa ke bada ki dekhabhala kosha 1941 mem arthika rupa se jo maim aura dvitiya vishva yuddha mem bhaga liya jaruratamanda purva sainikom ki madada karane ke lie banaya hai, kashmira, laddakha aura nipha apareshana aura bharata aura pakistana ke sangharsha. 
isa kosha ke rupae se adhika ki sthapana ke bada se. 60 lakha 28,635 purva karane ke lie vitarita kie gae haim sainikom 1991 ke anta taka. 
kisi bhi desha ke svasthya ke nikata apane bachchom ke svasthya se juda़a hai. 
isa ki janakari hone ke karana yaha iarasiesa prayasa donom mam aura bachche ke lie sarvottama sambhava dhyana dene ke lie. 
dekhane mem yaha 1931 mem rashtriya mukhyalaya mem prasuti evam bala kalyana byuro ki sthapana ki isa bhavana ko dhyana mem rakhate. 
taba se isa gatividhi badha़i hai aura sabhi dishaom mem vistara kiya. 
vaham se adhika 30 emasidabyu paliklinika aura 375 emasidabyu kendra ne desha ke vibhinna bhagom mem reda kraॉsa shakhaom dvara chalae ja rahe haim. 
ve, prasavapurva prasavottara parivara kalyana sevaom, bachchom ko poliyo pradana aura niyamita svasthya jancha ko pura-apa, prasuti mamalom aura chalane narsari mem upasthita. 
nivasa ka sevaom pranali ke eka netavarka ina sansthaom pitha. 
isake alava svasthya shiksha mem, poshana aura dudha khila karyakrama niyamita rupa se ayojita ki jati haim. 
yaha bhi takaniki prashikshana aura shakhaom aura anya sangathanom ko salaha pradana karata hai. 
yaha tihari ke pichhada़e pahada़i kshetrom mem kalyana ke kama mem agrani raha hai gadha़vala, jaunsara-bavara, alamora, pithauragadha़, udhama sinha nagara aura uttara pradesha mem nainitala jilom. 
ye kshetra yojanaom chara mukhya prasuti aspatalom, 20 upa kendrom aura 12 bala vikasa kendrom shamila haim. 
sabhi reda kraॉsa emasidabyu aspatalom aura kendra parivara kalyana yojana ka kama bahara le jana. 
nai phasalom ka parichaya, aparichita bimariyom ke lie logom ko ujagara nai phasala kita aura ve kheti samudayom ko uthane ke roganuom akarshita ho rahe haim. 
ve chhote parivara ke manadandom ko apanane ke lie aura unhem pradana garbha nirodhakom aura anya parivara niyojana upakaranom yogya joda़om prerita. 
ina kendrom mem se kai, naidanika preraka aura shalya chikitsa ki avashyakataom ko pura karane ke lie susajjita haim. 
janasankhya mem vriddhi chikitsa karmiyom ki manga ke satha vastava mem, narsinga dekhabhala ke lie vishesha rupa se gharom mem aspatalom mem tanava kama karane ke lie badha़ raha hai. 
reda kraॉsa emasidabyu ikaiyom sahayaka narsa dhatriyom, mandapa aura grihiniyom ko ghara narsinga para prashikshana pradana. 
uchcha adhyayana aura narsinga vishayom mem visheshajnata ko protsahita karane ke lie, chhatravritti narsinga vidyarthiyom ko sammanita kara rahe haim. 
isa avasara aearasiesa aura ninna aya samuha ki kamai ke avasarom aura vyavasayika kaushala badha़ane ke lie samajika rupa se mahilaom ki sthiti ko badha़ane aura arthika rupa se sikhane aura netritva ke lie unhem avasara upalabdha kara deta hai. 
samaja vi.ti.si. shuru kara diya hai desha ke jo kakshaem ya adivasi abadi pichhada़e ka prabhutva ho alaga jeba mem. 
laingika samanata aura arthika vikasa ki prakriya mem mahilaom ki bhagidari badha़ane ke lie ke lie sambandha mem aearasiesa ne varsha 1989 aura 1993 mem 2 vyavasayika prashikshana kendrom, eka pratyeka salta leka, kolakata (pashchima bangala) mem hai aura arakkonama (tamilanadu) mem shuru kiya tha. 
jaruratamanda mahilaom ko bhi kendra bindu ke rupa mem mahilaom ko badha़ava dene ke lie vikasita kiya hai / bachche ke vikasa gatividhiyom ko prashikshana pradana karane ke atirikta ina kendrom. 
ye bhi vastra utpadana kendra ke rupa mem apane rahata karya mem samaja ke upayoga ke lie seva kendra. 
sakshama karane se mataom adhika udaratapurvaka kalyana para kharcha karane ki aura unake bachchom ki paravarisha. 
arthika svatantrata do aura samudaya mem mahilaom ki sthiti mem sudhara. 
parivara ke lie hai aura jisamem logom ko jine ke samudayom jivana ki gunavatta mem sudhara. 
eka svasthya ki bahali ke lie sabase chikitsatmaka agrima ke rupa mem rakta adhana shumara hai. 
rakta hamesha na kevala jivarnabala para hi jivana ka eka padartha ke rupa mem dekha gaya hai. 
rakta bainka aearasiesa ki 1962 mem jaba sangraha matra 212 yunita rakta salana jabaki isa para 40000 ikaiyom ke lie 1990 mem nukila tha kamakaja shuru kara diya. 
paramparika manyataom, mithakom aura tebusa khuna ke sangraha mem badhaom rahe the. 
, vyaktigata drishtikona, patrachara ke madhyama se bada़e paimane prerana abhiyana isa kauntara karane ke lie, samuha ke drishtikona shuru kie gae. 
reda kraॉsa rakta bainka apane prashikshita karmachariyom ke madhyama se vibhinna svayansevi sangathana, skula aura kaॉlejom adi ke sahayoga se prerana karyakrama badha़ane ka kama chala raha hai 
halanki 300cc rakta ka dana bilkula eka chikitsakiya yogya vyakti se hanirahita hai khuna se adhika 3,00,000 yunita salana dilli mem rogiyom aura bhi adha nahim karane ke lie avashyaka haim svaichchhika dataom se ata hai. 
sabhi upakaranom prayukta banja haim aura dispojebala isalie, edsa sahita kisi bhi bimari ke samparka ka khatara bhi nahim hai. 
peshevara dataom nishiddha haim ki suprima korta ke phaisale ke anusara 1.1.98 se kyonki rakta peshevara dataom se ekatra khataranaka hai aura jokhima se bhara hua. 
yaha rakta bainka rashtriya mukhyalaya se puri taraha se, jo dilli aura dilli ke asapasa ke rogiyom ko di jati hai vibhinna ghatakom mem pure rakta ko alaga karane ke lie susajjita hai. 
vaham thailesimiya ke bare mem 1200 rogiyom ho reda kraॉsa rakta bainka ke satha darja ki gai. 
thailesimiya rogiyom ko unake astitva ke lie reda sela dhyana hara 2-4 haphtom ki avashyakata hoti hai. 
reda kraॉsa rakta bainka muddom khuna thelasimika rogiyom ko muphta. 
apane nama ke layaka hara sangathana nirantarata aura saphalata sunishchita karane ke lie yuva ke sahare ki jarurata hai. 
vastava mem abhi taka kama ki svaichchhika aura manaviya svabhava ke karana kisi bhi anya sangathana se bada़a reda kraॉsa ki avashyakata hai. 
1925 jearasi gatividhiyom mem sthapana eka tina para karyakrama guna: adharita haim samvardhana svasthya, seva aura dosti ki. 
sangathita shaikshika sansthanom mem chhatrom ke bicha mem paida karane ki koshisha karata vibhinna manaviya sevaom mem eka apekshakrita kama unra mem unhem shamila kara reda kraॉsa andolana ke sata pradhanadhyapakom. 
jearasi / vyaarasi eka lakha ke sadasyom se bhi adhika hai, desha mem phaile. 
kaॉlejom jearasi / vyaarasi yuva reda kraॉsa ke rupa mem kaha jata hai mem. 
aiarasiesa jo anadekhi aura apariharya paristhitiyom aura unhem dhundhane ke anya sabhi ka artha hai majabura nahim jukenge hai parinamom ke madhyama se apane priyajanom se alaga kara diya gaya hai ki antarrashtriya stara para logom ko yada kara ke anurekhana ke satha hi sambandhita hai. 
aiarasiesa bhi hai jo isi taraha ki paristhitiyom mem alaga kara diya gaya hai parivarom ko milata hai. 
eka javana lada़ki bangladeshi chechaka (1973) se sankramita. 
chechaka ke tike ke vikasa ke lie dhanyavada, isa roga sarakari taura para 1979 mem khatma ho gaya tha. 
eka mahamari (ya vaishvika mahamari) hai ki eka vyapaka bhaugolika kshetra se adhika logom ko prabhavita karata hai eka bimari hai. 
justiniana ko, 541 se 750 ke lie, 50 aura 60 pratishata ke bicha yuropa ki janasankhya ka mara plega. 
1352 se 1347 ke kale mauta yuropa mem 5 varshom mem (anumanita 25 aura yuropa, eshiya ki abadi ke 50% ke bicha hone ka, aura aphrika 25 lakha ko mara - vishva ki abadi ka samaya 500 karoda़ tha) mem. 
15 vim aura 16 vim shatabdiyom ke daurana yuropiya khojakarta dvara chechaka, khasara aura madhya aura dakshina amerika ke kshetrom ke lie sannipata ki shuruata ke mula nivasiyom ke bicha mem mahamariyam ke karana hota hai. 
1518 aura 1568 roga mahamariyam ke bicha 20 lakha se 3 karoda़ ke girane se meksiko ki janasankhya ka karana hota tha. 
pahala yuropiya imphluenja mahamari 1556 aura 1560 ke bicha mem, 20% ke eka anumana ke anusara nrityu dara ke satha hua. 
chechaka ne 18 vim sadi mem akele eka anumana ke anusara 60 lakha gorom ko mara dala. 
unra ke una bachchom ki 80% sahita sankramita ke tahata 5 varsha ki 30% pratishata karane ke lie, upara ke roga se hai, aura eka ke bache ke tisare andha mara gaya tha. 
isa inphlyuenja paindemika 1918 ke (ya spenisha phlu) 1.7 araba duniya ki abadi ka (lagabhaga 2%) 25-50 lakha logom ko mara dala. 
aja inphlyuenja 500000 karane ke lie hara sala duniya bhara ke bare mem 250.000 marata hai. 
jyadatara mamalom mem, sukshmajivom sadbhava mem apane mejabana ke satha rahate haim. 
isa taraha ke kai ushnakatibandhiya vayarasa aura kida़e, bandara, ya jisamem ve rahate haim aura anya janavarom ke lie bhi aisa hi mamala hai. 
kyonki roganuom aura unake mejabana saha haim vikasita, mejabana dhire dhire ke sukshmajivom ke lie pratirodhi bana jate haim. 
jaba eka sukshma jiva eka arase se pashu mejabana se eka insana, yaha eka hanirahita parajivi aura rogajanaka bana jana banda mai ke lie kudata hai. 
sabase naya sankramaka bimariyom ke satha, kuchha manaviya karya hai, taki eka maujuda sukshma jiva eka nai jagaha mem nivasa upara le ja sakate haim ki vatavarana mem badalava shamila hai. 
nae gamvom aura gramina kshetrom mem avasa gatividhiyom ka nirmana janavarom ke satha samparka mem logom ko lata hai - aura ve bandaragaha ke roganuom.
jaisa desha, jangalom ke madhyama se sada़kom ke nirmana aura nipatana ya vyavasayika udyamom ke lie kshetrom samashodhana dvara unake varsha vanom ka istemala kara, logom ko kida़om aura anya janavarom ko pahale ajnata sukshma jivanuom harboring muthabheda़. 
kai vikasashila deshom mem shaharom ka teji se vikasa gariba svachchhata ke satha bhida़ bhare ilakom mem logom ki bada़i sankhya dhyana kendrita karane ke lie jata hai. 
sankramaka rogom ki ina shartom palaka sancharana. 
jahajom aura anya mala vahaka aksara ki dura gantavyom ke lie roga phaila sakata hai anabhipreta yatriyom bandaragaha. 
antararashtriya jeta ke satha halanki-havai yatra, logom ko eka bimari ke satha sudura, ya unake parivarom ke lie ghara ke lie ise le sakata hai, to apana pahala lakshana prakata hone se pahale sankramita. 
jarmana daka tikatom chara entika maikroskopa chitrana. 
maikroskopi mem pragati sankramaka rogom ke prarambhika adhyayana karane ke lie avashyaka the. 
abu ali ibna sina (evisenna) ko jaldi 11 vim sadi mem sankramaka rogom ke sankramaka prakriti ki khoja ki. 
abu ali ibna sina (evisenna) sangarodha ke chikitsa topa ki, lagabhaga 1020 mem sankramaka aura sankramaka rogom ke prasara ko simita ke eka sadhana ke rupa mem prastuta kiya. 
abu ali ibna sina (evisenna) bhi beimani se videshi sansarika nikayom dvara sankramita hone se pahale sandushita hai ki sharirika srava haim, para vaha bimari ke mula karana ke rupa mem unhem dekhane nahim diya. 
jaba kale mauta bubonika ala pahuncha plega-enadalusa 14 vim shatabdi, ibna khatima aura ibna ala mem khatiba ki sankramaka roga hai jo manava sharira mem pravesha sankramaka sansthaom ke karana hota hai dharana prastuta ki. 
isa taraha ke vicharom aura yuropa mem punarjagarana ke daurana itali ke bhikshu jiromalo pharekasatoro ke lekhana ke madhyama se vishesha rupa se lokapriya ho gaya. 
entona vaina liuvenahoeka (1632-1723) ke sukshmajivom ka palana karane ke lie pahali bara kiya ja raha hai, jivanu ke asana drishya ke lie anumati maikroskopi ke vijnana unnata. 
lui pashchara shaka hai ki kuchha bimariyom sankramaka ejentom ke karana ho pare, siddha aura rebija ke lie eka tike ka vikasa kiya. 
raॉbarta kaॉkha, eka vaijnanika adhara kaॉkha ke postulates ke rupa mem jana ke satha sankramaka rogom ke adhyayana mem pradana ki. 
edavarda jennara, jonasa saॉlka aura albarta sabina chechaka aura poliyo, jo bada mem unmulana aura ina rogom ke lagabhaga unmulana, kramashah para natija hoga karane ke lie prabhavi tike ka vikasa kiya. 
alekjendara phleminga ne duniya ki pahali penisilina entibayotika ki khoja ki. 
jeraharada domagka entibayotika davaom Sulphonamides, pahali vyapaka spektrama sinthetika vikasita karata hai. 
sankramaka rogom ki chikitsa upachara imphekashanoloji ki chikitsa ke kshetra mem pada़ta hai aura prachara ke adhyayana janapadika roga vijnana ke kshetra se sambandhita kuchha mamalom mem. 
samanyataya, sankramana shuru prathamika dekhabhala chikitsaka ya antarika chikitsa visheshajnom dvara nidana kara rahe haim. 
udaharana ke lie, eka gaira nimoniya samanyatah chikitsaka ya pulamonolojisata (phephada़om chikitsaka) dvara ilaja kiya jaega. 
isa imphekashanolojisata ka kama donom rogiyom aura samanya chikitsakom ke satha, satha hi prayogashala ke vaijnanikom, imphekashanolojisata, bakatiriyolojisata aura anya visheshajnom ke satha kama karana hai. 
yaha roga jarura eka prarambhika jacha ke bada ka nidana nahim kiya gaya hai. 
marija imayunokomapromaiज़da hai (udaharana ke lie, edsa ya rasayana chikitsa ke bada mem). 
ushnakatibandhiya rogom jaise yaha sankramaka ejenta eka asamanya prakriti ka hai. 
yaha roga pahali pankti pratijaivikom ko javaba nahim diya. 
yaha roga anya rogiyom ko, aura khataranaka ho sakata hai ki marija ko alaga hona hoga. 
lekimiya ya khuna lukemiya ya asthi majja ka eka kainsara hai aura yaha eka asamanya prasara rakta koshikaom, amataura para sapheda rakta koshikaom (leukokyatasa) se guna karake (utpadana) se hoti hai. 
lekimiya rogom ki vyapaka samuha ka hissa hematolokajikala niyoplasama kaha jata hai. 
lekimiya eka vyapaka shabda rogom ke eka spektrama shamila hai. 
lekimiya hai naidanika aura petholojikali isaki tivra aura jirna rupom mem vibhajita. 
tivra lekimiya aparipakva rakta koshikaom ka teji se badha़ jata hai. 
yaha bhida़ asthi majja svastha rakta koshikaom ka nirmana karane mem asamartha ho jata hai. 
lekimiya ke tivra rupom bachchom aura yuva vayaskom mem ho sakata hai. 
vastava mem, lekimiya ghataka roga ke kisi anya prakara ki tulana mem amerika mem bachchom ke lie mauta ka eka aura ama karana hai. 
tatkala upachara ke tivra leyukemia teji se badha़ne aura jo to khuna hai aura sharira ke anya angom mem phaila mem se bahara ho jana isa ghataka koshikaom, ke jama ho jane ke karana avashyaka hai. 
halanki isa roga kabhi kabhi kapaliya tantrika palasi paida kara sakata hai kendriya tantrika tantra (sienaesa) shamila asamanya hai. 
chirakari lekimiya apekshakrita paripakva ka nirmana atyadhika dvara, lekina aba bhi asamanya, rakta koshikaom vishishta hai. 
amataura para pragati karane ke lie sala ke lie mahinom lene, koshikaem samanya koshikaom ki tulana mem eka bahuta uchcha dara para, khuna mem bahuta asamanya shveta rakta koshikaom mem jisake parinamasvarupa utpadana kiya jata hai. 
chirakari lekimiya jyadatara purane logom mem hai, lekina hota siddhantatah kisi bhi ayu varga mem ho sakata hai. 
jabaki gambhira lekimiya turanta ilaja kiya jana chahie, kronika rupa kabhi kabhi kuchha samaya se pahale ilaja chikitsa ke adhikatama prabhavashilata sunishchita karane ke lie nigarani kara rahe haim. 
kronika therepi aura dhire dhire apane sharira ke madhyama se chalata hai. 
isake alava, isa roga hai jo ki kainsara se grasta parivartana hai ki samanya rupa se limaphokyatika pharma karane ke lie para chala jata hai majja sela ka eka prakara mem hui sanketa limaphobalasatika ya limaphosaytaka leyukimiya, ke rupa mem vargikrita kara rahe haim. 
nyeloida ya nyelojenausa lukemia hai, jo ki kainsara se grasta badala ki samanya rupa se lala koshikaom pharma karane ke lie para chala jata hai majja sela ka eka prakara mem hui sanketa, sapheda koshikaom ke kuchha prakara, aura pletaletom. 
lukemia roga bhi, vayaskom ko prabhavita karata hai, khasakara una unra 65 aura purane. 
bachchom mem 85% aura vayaskom mem 50% asthi majja aura pranaligata roga niyantrana, prasara jaise ki rokathama CNS chirakari lekimiya limaphosaytaka karane ke lie jyadatara aksara 55 sala ki unra para vayaskom ko prabhavita karata hai. 
Leukemias kabhi kabhi chhoti vayaskom mem hote haim, lekina yaha bachchom ko prabhavita karata hai lagabhaga kabhi nahim. 
bachchom ki tulana mem vayaskom mem tivra nyelojenosa kimiya adhika samanya hai, aura adhika samanya mahilaom ki tulana mem purushom mem. 
40% asthi majja aura pranaligata roga niyantrana, sienaesa ka vishesha upachara hai, agara lage. 
vayaskom mem mukhya rupa se lagatara nyelojenosa lekimiya.
bachchom ki eka bahuta chhoti sankhya mem bhi isa roga ka vikasa. 
ke bare mem balom sela lekimiya prabhavita logom ke 80% vayaska purushom rahe haim. 
nahim 100% karane ke lie dasa sala asadhya para hai, lekina ilaja asani se chhote bachchom ke mamale mem, 96% ki suchana di. 
sathanantrana ke jarie asthi majja ko nukasana, aparipakva sapheda rakta koshikaom ki uchcha sankhya ke satha samanya asthi majja ki koshikaom hai, jo rakta mem thakke prakriya mahatvapurna haim blada pletaletsa ki kami mem parinama hai. 
yaha chota adhika ho sakata hai, khuna ya chidha़ balida (petechie) vikasita lekimiya logom ke satha hai. 
jo rogaja़naka़om lada़ai mem shamila haim shveta rakta koshikaom, daba diya ja sakata hai ya bekara. 
yaha eka sadharana sankramana se lada़ne mem asamartha ho ya rogi ki pratiraksha pranali ke karana sakata anya sharira ki koshikaom para hamala shuru karane ke lie. 
anta mem, lala rakta koshikaom ki kami araktata, jo dyspnea karana mai ki ora jata hai. 
sabhi lakshana anya rogom ke lie jimmedara thaharaya ja sakata hai, nidana, rakta parikshana aura eka asthi majja ki pariksha ke lie avashyaka haim. 
jo 'shveta rakta', uchcha shveta rakta koshikaom mem roga ke nama se prapta hota hai isaka matalaba yaha vachana lekimiya, gina jata hai ki sabase lekimiya rogiyom ko ilaja se pahale ki hai. 
sapheda rakta koshikaom ki uchcha sankhya mem jaba eka khuna ka namuna eka khurdabina ke niche dekha jata hai spashta kara rahe haim. 
aksara, ina atirikta sapheda rakta koshikaom aparipakva haim ya bekara. 
koshikaom ki atyadhika sankhya bhi anya koshikaom ke stara ke satha, rakta ki ginati mem eka hanikaraka asantulana paida hastakshepa kara sakata hai. 
kuchha lekimiya rogiyom drishya eka niyamita rupa se khuna ki ginati ke daurana gina uchcha sapheda rakta koshikaom ki ja़rurata nahim hai. 
yaha kama ama halata lyukemiya kaha jata hai. 
asthi majja abhi bhi jo rakta koshikaom ki samanya utpadana badhita kainsara shveta rakta koshikaom mem shamila hai. 
halanki, lyukemika koshikaom ke bajaya khuna hai, jaham ve eka rakta parikshana mem dikhai jaegi pravesha ki majja mem raha rahe haim. 
eka lyukemika rogi ke lie, sapheda rakta koshikaom ko rakta mem samanya kiya ja sakata hai ya kama gina jata hai. 
lyukemia kisi ke lekimiya chara pramukha prakara ka mem hai, aura vishesha rupa se balom sela lekimiya mem ama hai ho sakata hai. 
sabhi lekimiya ke vibhinna prakara ke lie koi bhi jnata karana hai. 
yaha alaga lyukemia ki sambhavana hai, aura bahuta kama kya una karanom ke bare mem kuchha khasa hai vibhinna karanom hai. 
shodhakartaom chara sambhava karanom: prakritika ya kritrima aionaija vikirana, rasayana ke kuchha prakara ke bare mem shaka majabuta hai, kuchha vayarasa 
lekimiya, anya kainsara ki taraha hai, jo onakojensa sakriya ya tyumara dabanevala jina nishkriya ke dienae mem sharirika nyuteshanom se parinama hai, aura koshika nrityu, bhedabhava ya prabhaga ke viniyamana badhita. 
ina nyuteshanom anayasa ho sakate haim ya eka vikirana ya kasinajana padarthom se samparka ka aura anuvanshika karakom se prabhavita hone ki sambhavana haim. 
kauhota aura mamale ko niyantrita adhyayana benjina jaise, aura bala ranjaka lekimiya ke kuchha rupom ke vikasa ke lie petro rasayana se khulasa ho, linka kara liya hai. 
vayarasa bhi lekimiya ke kuchha rupom se juda़ gaya hai. 
ya udaharana, sabhi ke kuchha mamalom mem manava inyuno vayarasa ya manava ti vayarasa (echatilavi -1 aura -2 ya to dvara vayarala sankramana ke satha, vayaska ti sela lekimiya / limaphoma karana) juda़e rahe haim. 
phenakoni araktata bhi tivra mayelogenausa lekimiya ke vikasa ke lie eka jokhima karaka hai. 
karana ya lekimiya ka karana banata hai jaba taka pae jate haim, vaham isa roga ko rokane ke lie koi tarika nahim hai. 
yaham taka ki jaba karanom aura isalie khasakara nivarana prayojanom ke lie upayogi nahim hai, ve tatkala, jaise niyantraniya nahim kiya ja sakata hai svabhavika rupa se hoti prishthabhumi vikirana jnata ho jate haim. 
tivra limaphosayatika lekimiya (sabhi) ke uchita prabandhana asthi majja aura pranaligata anya saitom para (sampurna sharira) roga ke rupa mem achchhi taraha ke rupa mem kainsara ki rokathama, vishesha rupa se kendriya tantrika tantra (sienaesa) ke niyantrana para jora deta hai. 
prerana kimotherapi chhuta ke bare mem lane ke lie - ki, lyokemika koshikaom aba asthi majja namunom mem paya jata hai. 
vayaska sabhi ke lie, manaka shamila karane ki yojana, vinkraristine predanisone, aura eka enthesayakalina dava; anya dava yojanaom ela esaparajinesa ya sayakalophosaphamida shamila ho sakate haim shamila haim. 
kama jokhima hai, amataura para tina aushadhiyom ke (prednisona, ela esaparajinesa, aura vankrisatina) ilaja ke pahale mahine ke lie shamila manaka chikitsa ke satha bachchom ke lie. 
vayaskom mem ekikarana therepi (1-3 mahine; bachchom mem 4-8 mahine) hai ki aba bhi sharira ke andara chhupa rahe haim kisi bhi lekimiya koshikaom ko samapta karane ke lie. 
kemotherapeutika davaom ka eka sanyojana pratirodha vikasita karane se shesha lekimiya koshikaom rakhane ke lie prayoga kiya jata hai. 
marijom ke satha kama ausata karane ke lie jokhima methotekseta aura 6 jaise entimetabolaita davaom ke satha chikitsa-merkapatopurina (6-sansada) prapta sabhi. 
uchcha jokhima vale marijom uchcha dava khuraka plasa upachara atirikta kemotherapeutika davaom ejenta ke satha prapta karate haim. 
sienaesa prophilaiksisa (nivaraka chikitsa) mastishka aura tantrika tantra ko phailane se kainsara ko rokane ke lie. 
kapaliya (sira) vikirana plasa spainala nala ya intharathekala (aiti) prasava ke spainala kaॉrda aura mastishka ke asapasa ke antariksha () ne nashili davaom ke methotatekseta mem. 
ti ke satha kevala bachchom-sela lekimiya, eka uchcha sapheda rakta koshikaom ki ginati, ya lekimiya ne mastishkameru drava (siesaepha) mem koshikaom ke satha hi aiti chikitsa kapaliya vikirana ko prapta karane ki avashyakata hai. 
bachchom mem eka gahana 6 mahine ke upachara ke karyakrama ki jarurata hai ke bada prerana, rakharakhava rasayana chikitsa ke 2 sala ke bada. 
uchcha jokhima vale bachchom daunorubisina jaise ina davaom ke satha satha eka anthrasayaklina prapta kara sakate haim. 
kyonki yaha vishishta jenetika asamanyataem ki pahachana karane mem madada mila sakati hai eka prayogashala parikshana polimaresa shrrinkhala pratikriya (pisiara) ke nama se sabhi rogiyom ke lie salaha di jati hai. 
isa taraha ki asamanyataem aura, natijatana, upachara yojanaom roga ka nidana para eka bada़a prabhava hai. 
pisiara jancha vishesha rupa se marijom ke lie jinaki bimari bi hai prakara sela mem mahatvapurna hai. 
bi sela sabhi amataura para manaka sabhi chikitsa se thika nahim hai. 
isake bajaya, uchcha pratikriya darom ki gaira karane ke lie upayoga kiya jata hai ki akramaka, sayaklophosaphamaida adharita regimensa-hodakina hai limaphoma ke satha prapta kara rahe haim. 
kyonki ina sabhi marijom ke satha anya vyaktiyom ki tulana mem eka bura roga ka lakshana hai, kai onakolojesata (elobiemati), elojenika asthi majja pratyaropana suja chhuta sankshipta kiya ja sakata hai ninnalikhita paramparagata sabhi rasayana chikitsa ke bada se. 
jo asthi majja pratyaropana prapta loga, pha़iltarda hava, banja khana sahita aspatala mem suraksha alagava, avashyakata hogi aura anta mem sukshmajivom ka bandhyakarana, unaki kula sapheda rakta koshikaom (dabyubisi) ginane taka 500 se upara hai.
avartaka tivra limaphoketika lekimiya (sabhi) rogiyom ko amataura para atirikta rasayana chikitsa akele se labha nahim hai. 
yadi sambhava ho to, ve, elojenika asthi majja pratyaropana ke dvara (elobiemati) bada phira se shamila rasayana chikitsa prapta karana chahie. 
vaikalpika rupa se, avartaka sabhi ke satha marijom elobiemati, pratiraksha pranali ejenta ke nae parikshana mem bhaga lene se, aura kemotherapeyotika ejentom, ya kama khuraka rediyotherepi, labha ho sakata hai yadi sharira ya sienaesa bhara mem kainsara rekarsa. 
chirakari limaphoketika lekimiya (sielaela) sambhavatah vartamana upachara dvara asadhya hai. 
lekina, saubhagya se, sielaela rogiyom ke eka bada़e samuha ke upachara ki jarurata nahim hai. 
adhyayana ki steja eka sielaela vale logom (hai ki, jo vistrita limaphoida utaka ke kama tina kshetrom hai vyaktiyom) shighra upachara se labha nahim hota hai. 
donom mancha hai aura rogi ke lakshana para adhikansha onkolojisata adhara sielaela ilaja. 
udaharana ke lie, jo (raya steja 0) eka rudha़ivadi ghada़i kama jokhima prarambhika stara ki bimari hai aura drishtikona intaja़ara liya ja sakata hai, purane rogiyom mem (60 + sala). 
isake viparita karake, sielaela satha bada़i vyaktiyom-jatilataom ya adhika unnata roga (raya steja kshkshksh ya iv) sambandhita rasayana chikitsa aura upachara eka kortikosateroeda (uda se predanisone, predanisolone) ko labha ho sakata hai. 
koratikosateroidasa pahala haim pratiraksha pranali sielaela dvara badala diya gaya hai jisase logom ke lie rekha ejentom. sielaela otoimayuna karana senadroma mai, jisamem rogi ki pratiraksha pranali ke hamalom ko nashta kara deta hai aura usaka khuda ka rakta koshikaom. 
jaba lala rakta koshikaom ko prabhavita kara rahe haim, sthiti inyomanohemolyatika animiya ke rupa mem, lala rakta koshikaom ki sankhya kama karake jo thakana, chakkara karana mai visheshata, jana jata hai aura sansa ki kami. 
jaba khuna pletaletom prabhavita haim, yaha pratiraksha ki madhyasthata tharmaboketopenia, jisamem pletaletsa ki sankhya kama se khuna baha raha karane ke lie netritva mai kaha jata hai. 
jo lakshana anubhava kara rahe haim yuva marijom ke lie, chikitsaka (elobiemati; otobiemati) jaldi rasayana chikitsa, plasa emalojenarika ya ऑtolaॉgasa asthi majja pratyaropana vichara kara sakate haim. 
yadi marija ko rakta pravaha samasyaom ke sanchalana mem lekimiya koshikaom ki uchcha sankhya ke karana anubhava hai, chikitsaka, bhi epharesisa ke rupa mem, sapheda rakta koshikaom ko alaga karane ke lie jane jate haim, rasayana chikitsa ke pahale lyukepheresisa suja sakata hai. 
ki limpha nodsa ki vriddhi karane ke lie eka kshetra mem ya sambandhita haim chuke pliha ke pliha ke sthaniyakrita, kama matra rediyotherepi ya sarjikala prabandhana saplenektomi dvara (hatane ke dvara ilaja kiya ja sakata hai lakshana). 
yadi lekimiya hai kai alaga alaga saitom mem, kula sharira vikirana (tibii) ki jarurata ho sakati hai ki limpha nodsa hamala kiya. 
madhyavarti ya unnata bimari se loga eka chikitsiya parikshana mem bhaga lene se madada ki ja sakati hai. 
vartamana samaya, naidanika parikshana (jaise, bayarostetina, dolasatetina 10, aura PSC 83 - eka cyclosporine dava aushadha pratirodha ko dura karane ke lie rasayana chikitsa ke satha diya) (jaise, intarapheronasa, monoklonala entibaॉdi) aura satha hi nae kemotherapeyotika ejenta imayonolojika yaugikom ka upayoga kiya ja raha hai . 
tivra melojenausa lekimiya (aiamala) jyadatara vayaskom ke lie ama hai, aura purushom ki tulana mem mahilaom ko prabhavita kara rahe haim.
kai alaga alaga kemotherapyutika yojanaom aiamala ke ilaja ke lie upalabdha haim. 
kula milakara, agara rananiti shamila hai jabaki kendriya tantrika tantra (sienasa) ke lie, vishesha upachara prasada (sampurna sharira) roga asthi majja aura vyavastha ko niyantrita karane ke lie hai. 
samanya taura para, sabase onakolojisata davaom ke sanyojana para rasayana chikitsa ke arambhika, shamila charana ke lie vishvasa karate hai. 
isa taraha ke sanyojana kimotherapi amataura (bimari) aura roga pratirodha ki eka kama jokhima kama jaldi chhuta ka labha pradana karata hai. 
ekikarana ya rakharakhava upachara roga punaravritti eka bara chhuta hasila kiya gaya hai ko rokane ke lie di ja sakati hai. 
ekikarana upachara aksara shamila rasayana chikitsa ya joda़a davaom ke satha gahanata rasayana chikitsa ka eka punaravritti satha hai. 
isake viparita karake, rakharakhava ilaja hai ki una logom ko shamila charana ke daurana prashasana se kama ho dava khuraka shamila hai. 
isake atirikta, vishishta upachara yojana hai, lekimiya ka nidana kiya gaya hai ki isa prakara ke adhara para istemala kiya ja sakata hai. 
jo bhi yojana hai, yaha hai ki aura nirnaya diya ja raha hai ki upachara ke vikalpa ke pichhe banane ki prakriya ko samajane ke lie rogi ke lie mahatvapurna hai. 
aiemala ka prarambhika upachara amataura para shamila rasayana chikitsa ke satha nashili davaom ke sanyojana ka upayoga kara daunorubikena (dienaara) jaise, syatrabineshana (ara-si), indrubikena, theogonine, eptosaida, ya mitoksatrone, ainabolika steraॉyada shuru hota hai. 
asthi majja ya anya srotom se bhi nava nidana roga bhi stambha koshika transaplanteshana (esasiti) ke lie vichara kiya ja sakata hai ke satha marijom. 
elojenika asthi majja pratyaropana mukhyatah rogiyom ke lie unra ki hai jo eka sangata parivara data hai 55 varsha se kama arakshita hai elobiemati. 
nae rogiyom ka nidana aiemaela ke lagabhaga adhe se isa ayu samuha mem, 75% shamila karane aura samekana upachara ke bada eka puri chhuta (siara) ko prapta karane ke satha haim. 
ailojenika asthi majja pratyaropana aiemaela ke satha sabhi rogiyom ke bare mem 15% ke lie upalabdha hai. 
durbhagya se, yaha hai ki sabhi aiemaela rogiyom ka kevala 7% isa prakriya ka upayoga kara thika ho jaega anumana hai. 
jo pha़iltarda hava, banja khana sahita aspatala mem suraksha alagava, ki avashyakata hoti hai (esasiti, elobiemati) stema koshika transaplanteshana prapta loga, aura anta mem sukshmajivom ka bandhyakarana, unaki kula sapheda rakta koshikaom (dabyubisi) ginane taka 500 se upara hai. 
upachara sentrala narvasa sistama lekimiya ki hai, agara vartamana, (jaise, ketarabina ya ara, si, methotatreekseta) mastishka aura merurajju ke asapasa ke kshetrom mem kemotherapeyotika davaom ke injekshana shamila ho sakate haim. 
eka bara marija chhuta mem hai, vaha uchcha khuraka ara-si (echadiesi) ke satha satha, samekana chikitsa / enathrasayalina davaom ke bina) udaharana ke lie samekana ya rakharakhava chikitsa, prapta hoga. 
vriddha aiemaela rogiyom ko vishesha ilaja chintaom hai. 
ve kama spetimia (rakta vishaktata) garenulokyatopenia ke satha juda़e, sahana karane ke lie aura ve aksara melodyatopenia ( 'preleukemia') sindroma (emadiesa) ki uchcha dara hai sakshama ho sakate haim. 
jo unra 75 ya jo mahatvapurna chikitsa shartom ko prabhavi rupa se kama khuraka ara ke satha ilaja kiya ja sakata hai si khatma ho gai haim vyaktiyom. 
uchcha matra ke bada shamila kimotherapi bujurga marijom ke sahana karane ki sambhavana nahim hai. 
abhi hala taka, ilaja ki yojana aura aiemaela ke satha bachchom ki pratikriyaom jyada una vayaskom ke se alaga nahim tha. 
phira bhi nae, adhika gahana pravesha aura samekana upachara uchcha chhuta dara aura lambi suraksha ke parinamasvarupa hai. 
kai loga shamila parikshana achchhe parinama (jaise, daunobine, doksorobisina) syatrabina (ara-si) ke satha satha eka entrasyalina ke sanyojana ka upayoga karane ka utpadana kiya hai. 
unra ke tahata 3 varsha ke entrasyalina shamila karane ke lie istemala savadhani se chuna jana chahie bachchom mem se doksorobisina aura vishaktata se bhi sambandhita mautom ka karana hota hai. 
ekikarana chikitsa jatila hai, lekina yaha uchcha khuraka ara ke kama se kama do pathyakramom-si (echadi) shamila hona chahie. 
bachche, jo eka gariba roga ka lakshana hai tivra lukimiya ke (bahuta adhika sapheda rakta koshikaom), vishesha rupa se monosyatika ema5 lekimiya hai. 
nava chirakari nyalojenosa lekimiya (siemaela) nidana ka ilaja karane ki chunauti ko isa roga ko niyantrita karane ke lie sabase achchha samagra rananiti nirdharita karane ke lie hai. 
leukepheresesa, yaha bhi eka paridhiya rakta stema sela pratyaropana ke rupa mem jane jate haim, stema sela kreyoprajraveshana (phrojana bhandarana) se pahale kisi anya upachara karane ke satha. 
rogi ke rakta ki aura stema koshikaom ko hatata hai, to rogi ke lie rakta ke ritarna eka mashina ke madhyama se parita ho jata hai. 
lyukapheresisa amataura para pura karane ke lie 3 ya 4 ghante lagate haim. 
isa stema sela ya kisi bhi kainsara koshikaom ko marane ke lie davaom ke satha ilaja nahim kiya ja sakata hai sakate haim. 
isa stema sela to jama ho jati hai jaba taka ve rogi mem vapasa pratiropita kara rahe haim. 
echaelae (hyumana lyukosaita entijana) unra 60 ke tahata sabhi rogiyom ke taipinga, ke rupa mem achchhi taraha se bhai bahana, mata pita ki taipinga ke rupa mem, aura bachchom, yadi upalabdha hai. 
isa prakriya hai ki eka sangata data stambha koshika transaplanteshana ke lie upalabdha hai nirdharita karega. 
purva upachara prajananata upaya (jaise, virya ke kayaroprasarveshana purva upachara karane ke lie, eka garbhavastha ke pura hone se pahale ilaja ke lie), jo apane parivarom ko pura nahim kiya hai yuva rogiyom mem. 
samanya mem, siemaela upachara vikalpom do gutom: una hai ki astitva mem vriddhi nahim karate aura una hai ki kara mem vibhajita haim. 
kemotherapeyoteka davaom hayadroeksaurea (hyadea) aura bosulaphena (melerana) jaise samaya ki avadhi ke lie rakta ke manaka ke anusara ganana kara sakate haim, lekina ve astitva mem vriddhi nahim karate. 
ve aksara marijom mem jo esasiti gujarana nahim kara sakata hai ya jo unra ya chikitsa vicharom ki vajaha se intarapheraॉna chikitsa ka javaba nahim hai khuna ki ginati ko niyantrita karane ke lie upayoga kiya jata hai. 
gliveka, kainsara ki davaom ke kainsara koshika ko nishakraya karane ka asamanya enjaima ka eka naya varga ki, yaha marata hai, lekina svastha koshikaom ko lagabhaga achhuta chhoda़ jata hai. 
rasayana chikitsa jaise anya kainsara chikitsa,, satha hi kainsara koshikaom, aura aksara gambhira apriya dushprabhavom ke satha marijom ko chhoda़kara svastha koshikaom para hamala. 
juna 2006 ke mem, khadya evam aushadhi prashasana (ephadie) ki anya chikitsa ka javaba nahim hai siemaela ke ilaja ke lie (sapryasela (tiema)) maukhika thayarosina kinesa avarodha karanevala desatiniba manjuri de di hai. 
ki siemaela astitva para asara karata hai eka ilaja elojeneka asthi majja pratyaropana, uchcha khuraka rasayana chikitsa aura vikirana eka data asthi majja ka nishechana ke bada ke prayoga hai. 
yaha prakriya rogiyom ka eka bada़a pratishata mem hai aura unake lie rogahara hai karomosomala vishamata ko hata. 
ke bare mem 20% rogiyom ko asamanya kromosama aura sudhara bachane ke intarapheraॉna dikhane unmulana lene ke 30% karane ke lie. 
hala ke nishkarsha yaha bhi kaha ki intarapheraॉna ke satha sanyojana mem kama khuraka sayatrabina (ara-si), aura adhika intarapheraॉna akele se adhika labhadayaka ho sakata hai ki salaha dete haim. 
rogiyom ke lie jo intarapheraॉna, ऑtolaॉgasa ya elojineka karane ke lie stambha koshika pratyaropana javaba nahim hai ki kevala vaikalpika hai. 
unnata charanom bimari ke satha marijom sayetotoksika davaom ke satha ilaja kiya ja sakata hai. 
udaharana ke lie, vyaktiyom meloida parivartana dikha ki aiemaela mem chhuta prerita karane ke lie upayoga kiya jata hai dragsa - ki, daunorubikena hai aura sayatarabina, ke satha ya bina 6-thiyoguanine ya etoposaida di ja sakati hai. 
visphota koshika sankhya asthayi taura para kama ho jaegi, lekina ve phira se 3 se 6 saptaha ke bhitara badha़ jaega. vyaktiyom limaphoida parivartana dikha eka thoda़a behatara drishtikona hai. 
ve dragsa tivra limaphokayatika lekimiya (sabhi) ke prabandhana mem upayoga ke satha vyavahara kara rahe haim - yaha hai ki hai, venakrisatina, predanisona aura donorubikena, ke satha ya bina ela-esaparaginesa. 
ki siemaela ke naidanika parikshanom mem adhyayana kiya ja raha hai nai davaom intarapheraॉna ke satha homoheringatonine-alpha (ienaepha-e) shamila hai , pekalitaksela (teksola), QS21 (eka sanyantra nikalane ki koi dushprabhava kama karata hai ki pratiraksha pratikriyaom ko badha deta hai), aura emiphostina (eka rasayanika rasayana chikitsa). 
isake atirikta, chikitsiya parikshana tike jaise,), yaha hai ki rogi ki pratiraksha pranali ke kainsara koshikaom ko marane ke lie pratyaksha kara sakate haim monoklonala entibaॉdi (inyonolojika padarthom padarthom ki sambhavita labhom ka mulyankana kara rahe haim aura harmonom (jaise, vikasa karakom, intaralyokinasa). 
balom sela lekimiya (echasiela) jisamem nyotetida eka lailaja, akarmanya rakta vikara, anshika rupa se bi koshikaem asthi majja mem sanchita paripakva hai. 
balom sela ka nama lekimiya taraha ve, thika hai, chhote balom ke satha shamila haim anumanom akara ka hai jo dekho koshikaom ke akara se prapta hota hai. 
kisi bhi anya lekimiya ke viparita, echasiela kama sapheda rakta koshika gina jata hai. 
jo lakshana-amataura para tatkala upachara prapta nahim kara mukta haim balom sela lekimiya ke satha marijom. 
marijom balom sela lekimiya mem niyamita khuna ke karya aura pariksha hara tina se chhaha mahine se bimari pragati para najara rakhane ke lie aura koi bhi naya lakshanom ki pahachana chaukasa intajara sanlagna haim. 
upachara amataura para jaba rogi sanketa aura lakshana kama rakta koshika mayane, lagatara sankramana, aspashtikrita ghava, araktata, ya jaise sho avashyaka mana jata hai ki kaphi marija ki rojamarra ki jindagi ko badhita karane ke lie mahatvapurna hai thakana. 
jo rogi jo sabase nava, amataura para ya to kaledaribina ya penatosatetina hai, jo kemotherapyotika davaom purina enaloga ya nyukaliyo ke rupa mem jana ke eka varga mem donom prapta echasiela ke mamalom ka nidana bhi shamila hai ilaja ki jarurata hai. 
jyadatara mamalom mem, ilaja ke eka daura mem eka lambe samaya taka chhuta ka utpadana hoga. 
anya upachara ritukshiba jalaseka ya atma intarapheraॉna se injekshana-alpha shamila haim. 
simita mamalom mem, rogi ko tilli ka (hatane) spenekatomi se labha ho sakata hai. 
kyonki unaki saphalata ki dara kaledaribina ya penatosatetina se kama ho to ye upachara amataura para eka naya rogi ke lie pahale ilaja ke rupa mem nahim diya jata hai. 
isa alpavadhi mem, vishesha rupa se jaba nyutophaila gina, kai rogiyom ko bhi apane sapheda rakta koshikaom ki ginati samanya stara se baramada kiya hai jaba taka entibayotika lena kama kara rahe haim. 
isa karana, nidana, upachara mem mahatvapurna anusandhana, aura lekimiya ke roga ka nidana kiya ja raha hai. 
klinikala parikshana ke saikada़om ki yojana banai ja rahi haim ya phira kisi bhi samaya para ayojita ki. 
adhyayana upachara ke prabhavi ka matalaba hai, isa roga ke upachara ke behatara tarikom para chhuta ya ilaja ke bada mem rogiyom ke lie jivana, ya uchita dekhabhala ki gunavatta mem sudhara lane dhyana kendrita kara sakate haim. 
1998 ke rupa mem, yaha hai ki hara sala lagabhaga 30,800 vyaktiyom ko sanyukta rajya amerika mem lekimiya aura 21,700 vyaktiyom ke satha isa roga se mara jaega nidana ho jaega anumana hai. 
kshaya roga (tuberkla dandanu ya tapedika ke lie tibi ke rupa mem) eka ama aura ghataka sankramaka roga maykobakatirya ke karana hota hai, mukhya rupa se maikobaiktiriyama kshayaroga sankshipta. 
kshaya roga sabase adhika hamale phephada़om (phuphphusiya tibi ke rupa mem) para bhi sentrala narvasa sistama ko prabhavita kara sakata hai, lasika pranali ke parisancharana tantra ke jenitorineri pranali, haddiyom, joda़om aura yaham taka ki tvacha. 
maikobaiktiriyama bovisa, maikobaiktiriyama ephrikenama, maikobaiktiriyama keneti jaise anya maykobakatirariya, aura maikobaiktiriyama mikoti bhi hai, lekina kshayaroga paida kara sakata hai ina prajatiyom amataura para svastha vayaskom ko sankramita nahim karate. 
eka ne duniya ki abadi ki eka tihai se adhika ko tibi ke jivanu ke lie, aura samane a gaya hai nae sankramana eka prati sekanda ki dara se hote haim. 
hara koi nahim sankramita ko purna vikasita roga vikasita; sparshonmukha, avyakta tibi sankramana se sabase ama hai. 
halanki, eka dasa gupta sankramanom mem jo, agara anupacharita, chhoda़ apane shikara ki tulana mem adhe se adhika marata sakriya tibi roga se pragati karega. 
2004 mem, nrityu dara aura rugnata ankada़e jyadatara vikasashila deshom mem 14.6 miliyana kronika sakriya tibi mamalom, 8.9 lakha nae mamale, aura 1.6 miliyana mautem, shamila haim. 
isake alava, vikasita deshom mem logom ki badha़ti sankhya tapedika ke karana unaki pratiraksha pranali inyonosupresiva davaom, nashe, ya echaaivi se samajauta kara rahe haim / edsa ke samparka kara rahe haim. 
echaaivi sankramanom mem vriddhi aura tibi niyantrana karyakrama ki upeksha tapedika ka eka punarutthana sakshama hai. 
dava ke ubhara-pratirodhi upabhedom ke satha bhi isa nai mahamari ke lie, 2000 se 2004 ke lie, tibi rogiyom ka 20% manaka upachara aura 2% dusari karane ke lie pratirodhi laina davaom ke lie pratirodhi ja raha yogadana diya hai. 
tibi ghatana vyapaka rupa se, pada़osi deshom mem bhi, svasthya dekhabhala pranali mem antara spashta rupa se ki vajaha se bhinna hota hai.
rajya ke svasthya nideshaka ya rajyapala adhikara eka sarvajanika svasthya apatakala ki ghoshana ki hai. 
eka sarvajanika svasthya apata sthiti ke mamale mem rajyapala ne jana svasthya ki raksha ke lie eka apata adesha jari karane ka vyapaka adhikara hai. 
ji esa ke antargata 166a-6, rajyapala, samudayika kaॉlejom, vishvavidyalayom, chaildakaiara aura praudha़ divasa dekhabhala suvidhaom aura vyavastha ki hai ki logom ka koi sarvajanika ayojanom ko roke bada़e honge sankhya eka bhautika sthana mem ikattha kara rahe haim sabhi skulom ko banda kara sakate haim. 
rajyapala ne sabhi gaira anivarya rajya sevaom aura vyavastha anivarya sevaom ke sanchalana ke kariba raha sakate haim. 
isa taraha ke eka adesha ke abhava mem, ejensi sira sthaniya ke satha aura vyaktigata sthiti ki gambhirata ka nirdharana karane ke lie kya karravai ki jani chahie ka nirdharana karane ke lie / rajya loka svasthya adhikariyom se paramarsha karenge. 
sarvajanika svasthya ke adhikara ko rajya ke svasthya nideshaka aura pratyeka sthaniya svasthya nideshaka ko karana hai. 
pratyeka rajya ejensi vaise rajya ya sthaniya sarvajanika svasthya ejensiyom ke kisi bhi sanchari roga adesha karane ke lie eka sanchari roga ke sancharana ko rokane ka palana karenge. 
prabandhana karmachariyom ko suchita karega aura karmachari karyasthala mem anya logom ke sanchari roga ke koi sabuta ke svasthya ki suchana prabandhana ko dege jo vyapaka asuraksha phela sakata hai 
prabandhana tatkala sthaniya svasthya vibhaga ko suchita karega. 
ejensiyom ne apareshanom ki nirantarata ke bhitara isa yojana ke disha nirdeshom protokaॉla ko asani se paribhashita kiya hai. 
riporta bimariyom ka sabase hala ki suchi ke rupa mem rajya svasthya ayoga dvara sankalita prashasanika sanhita mem 10a enasiaisi41ai.0101 paya jata hai. 
isa suchi mem lagatara nae rogom mem ubharane ke rupa mem adyatana kiya jata hai. 
sarvajanika svasthya alagava, sangarodha ke adhikara, aura anya sanchari roga niyantrana ke upayom ko rajya svasthya nideshaka aura / ya sthaniya svasthya nideshaka (ji esa 130a-145) ke satha ghara ya unake disainisa ke kaunti mem rahata hai. 
adesha mem vyakti ko vyakti se sancharana ko kama karane ke lie, pratyeka ejensi jagaha mem turanta rajyapala aura / ya sarvajanika svasthya adhikariyom se adesha ko lagu karane ke lie eka samajika dura niti honi chahie. 
samajika dura logom ke bicha ghanishtha samparka ke lie avasarom ko kama karane ke dvara eka bimari ke prasara ko simita karane ke lie dija़aina kiya gaya hai. 
eka ejensi vaikalpika karyashetra ya telivarkindaga ke upayoga dvara samajika dura ka abhyasa karane ke lie chuna sakate haim. 
ejensi sira jaldi telivarka vyavastha sthapita karane ke lie, samanya avashyakataom ko bayapasa kara, jaisa ki ejensi mem, apatakala ki ghoshana ke daurana antarika telivarkindaga niti aura prakriyaom bahirrekha adhikrita kara rahe haim. 
karmachariyom ko niyamita vetana prapta hoga kama karane ke lie avashyaka. 
pratyeka ejensi sira pahale se dekhana hoga aura anivarya apareshana namoddishta aura ina apareshanom karmachariyom ke lie karmachariyom ko namita. 
yadi ve ya bimara alaga kara rahe haim agara ve tatkala parivara ke eka sadasya ke lie hai jo alaga hai ya bimara dekhabhala karane ke lie avashyaka haim anivarya karmachariyom ko kama se, mapha kiya ja sakata hai. 
isa ejensi ne maujuda karmachari nama aura samparka janakari sahita sthiti se anivarya karmachariyom ki eka suchi, kayama karega. 
yaha ejensi sira mamale mem namita karmiyom kama karane mem asamartha haim karmiyom ke lie eka vaikalpika yojana ka vikasa hoga. 
vaikalpika karyakartaom jo anivarya rupa mem nirdishta nahim kara rahe haim, lekina jo anivarya karmachariyom ke lie mem bharane ke lie kaushala vartamana karmachari shamila ho sakate haim, theka majadura ya anya asthayi karmachariyom. 
yaha vishesha rupa se eka mahamari mem kai saptaha ya mahinom ke lie pichhale mai mahatvapurna hoga. 
karmachari anivarya karmiyom aisi padanama aura ke lie riporta karane ke lie ya para, apata sthitiyom mem kama karate rahane ki avashyakata ko adhisuchita kiya jaega ke rupa mem namita. 
yadi anivarya karmiyom samaya ki eka vistarita avadhi ke lie karyakshetra para rahane ke lie avashyaka haim, ejensi ya vishvavidyalaya paryapta avasa aura bhojana ki vyavastha karega. 
vyaktiyom anivarya karmachari ke rupa mem namita anushasanatmaka karravai karane ke lie, upara ke adhina ho sakati hai aura rojagara ki samapti, iechchheka viphalata ya kama para rahane ke lie riporta karane ke lie shamila hai. 
pratyeka sthiti eka mamale para mamale adhara para uchita karravai ka nirdharana karane ke lie samiksha ki jaegi 
jaba eka ejensi ya banda hai, jaba prabandhana ki kevala anivarya karmachari kama para riporta karane ke lie avashyaka haim nirdharita karata hai. 
yaha anivarya karmachariyom ko samaya aura eka pradana kiya jaega aura sabhi adhe ghante ke lie bhugatana karate haim ya sabhi ghante ki karmachari para rahane ke lie saita hai, dhana ki upalabdhata ke vishaya ki avashyakata hoti hai kama kiya. 
yadi dhana upalabdha nahim haim, ke karmachari eka bada ki tarikha ya eka mem atirikta adha samaya vetana diya jaega adha pratikaratmaka samaya. 
yaha pravadhana hai jo mukta hote haim aura gaira mukta sapha shrama manaka adhiniyama (phaelaesaai) karmachariyom sabhi para lagu hota hai. 
yaha pravadhana kisi bhi paristhiti mem asthayi karmachariyom ko shamila nahim karata hai. 
prabandhana karmachariyom ke lie hai ki ve apane ko chhoda़, jisase una lakshanom eka sanchari roga ke satha itana hai ki ve anya karmachariyom ko sankramita nahim hai ghara ke satha juda़e rahane ke lie protsahita kare tatha saja़a nahim hogi tanava chahie. 
jaba eka karmachari alaga hai, karmachari ko bhugatana diya jaega samaya samapta hota hai ke dvara nirdishta avadhi taka ya karmachari jo bhi pahale ae ko sanchari roga se bimara ho jata hai chhoda़o. 
yadi ejensi prabandhana ki eka karmachari ko eka sanchari roga se jude lakshana hai manana hai ki prabandhana ke karmachari aura kama karane ke lie chhoda़ kisi bhi upalabdha pratipuraka chhutti, bimara chhoda़, avakasha chhutti ya bonasa ka upayoga karane ke lie riporta karane ke lie nahim ki avashyakata ho sakati hai. 
halanki sabhi prayasom ko gaira anivarya karmachariyom ko eka vaikalpika sthana se kama karane ki anumati dene ke lie kiya jana chahie, yaha hamesha sambhava nahim ho sakata hai. 
jaba aijensi banda hai, ya jaba ejensi prabandhana ki kevala anivarya karmachariyom jo kama karane ke lie bhugatana kiya pradana kiya jaega avashyaka nahim haim, ne gaira anivarya karmachariyom ko kama karane ke lie riporta karane ke lie 30 kailendara dina chhoda़ avashyaka haim nirdharita karata hai. 
isa karmachari ke vetana karmachari prapta hoga usi dara para karmachari kama kara raha tha bani rahegi (samanya rupa se prapta bhugatana koi badalava primiyama sahita). 
yadi samayojana ki ja़rurata hai, ve agale pecheka mem kama karane ke lie vapasa ane ke bada, vetanaishegi samaya sima para nirbhara karata hai ki bhugatana ki avadhi ke lie kiya jaega kiya jaega. 
yadi eka gaira anivarya karmachari jaba ejensi banda hai kama karane ke lie chuna, karmachari atirikta bhugatana prapta nahim karega. 
karmachari kauna karega para aropa upayukta khate mem anusuchita dina ki chhutti ke anta taka chhutti chhutti chhutti ya bimara chhoda़ niyojita kara rahe haim. 
yadi karmachari bimara ho jata hai aura yaha shramika muavaja adhiniyama ke shramika muavaja niti ke anusara kama karane ke lie kiya ja sambandhita nirdharita kiya jata hai lagu hota hai. 
yadi karmachari alaga hai ya bimara ke parinama ke rupa mem banda ho jata hai naukari jokhima hai, bimara chhoda़ niti lagu hoti hai. 
parivara aura chikitsa ke upabandha aura niti chhoda़o parivara bimari bhi lagu honge niti chhoda़o. 
karmachari jo aura eka sanchari roga ke lakshana haim ghara rahane ke lie ya jo sanchari roga ke satha jaba taka ve kahate haim ki ve puri taraha se thika ho rahe haim yakina hai ki kama para vapasa jane ke lie nahim kiya jana chahie chetavani bimara haim avashyaka haim. 
ejensiyam eka svasthya dekhabhala pradata se kama karane ke lie phitanesa ka pramana patra ki avashyakata ho sakati hai. 
agara alaga, yaha karmachariyom ki jimmedari eka likhita satyapana ke satha eka sarvajanika svasthya adhikari se ejensi pradana karana hai. 
isa ejensi ko banane-taka ka ho sakata hai samaya 24 mahine yadi avashyaka karane ke lie. 
yaha bhi bado ki dekhabhala ke lie lagu hota hai. samiksha niti pravadhana ki eka ejensi se adhika 30 dinom ki chhutti aura isa niti ka muavaja pravadhanom ke lie aura samiksha ki jaegi ya to, sanshodhita ya nae sire se samapta banda ho gaya hai yadi. 
isa ejensi ke pramukha ne rajyapala ke karyalaya aura rajya bajata ke nideshaka ke satha paramarsha se karravai ki upayukta pathyakrama ka nirdharana karega. 
ina pravadhanom ke navinikarana para eka nirnaya vicharadhina hai, karmachari jaba taka koi nirnaya kiya jata hai (,, bimara pratikaratmaka, bonasa), agara, upalabdha chhutti chhutti lene ke lie anumati di ja sakati hai. 
agara chhoda़, chhoda़ upalabdha nahim hai unnata kiya jaega aura pahale 24 mahinom ke bhitara bhugatana kara diya hai. 
eka apatakalina layoff vetana ki vajaha se dhana upalabdha nahim hai eka asthayi alagava hai, kama ya anya apatakalina sthiti ke karmachariyom ne karyakshetra se dura rahane ke lie avashyakata ke karana upalabdha nahim hai. 
niyokta ki sthiti badala jaegi aura jalda hi ke rupa mem sambhava ke rupa mem karmachariyom ko yada karane ka irada rakhata hai vishvasa rakhata hai. 
eka apatakalina jabri yadi ejensi puri taraha ya anshika rupa se banda rahata samaya jana svasthya apata sthiti ki vajaha se eka anishchita kala ke lie banda banda ki ghoshana ki ja sakati hai. 
isa ejensi ke pramukha ne rajyapala ke karyalaya aura rajya bajata ke nideshaka ke satha paramarsha ke bada yaha phaisala karana hoga. 
eka apatakalina katauti ke daurana, karmachari, jo rajya svasthya yojana mem bhaga lene ke hakadara haim dura rakha jata hai. rajya ejensiyom ke niyokta anshadana ka bhugatana karega. 
isa ejensi ne bhi katauti ke bada pravadhana hai ki karmachariyom ko kisi bhi yogadana unaki ora se kie ke lie rajya chukane karega ke satha isa maha ke lie karmachari ka yogadana ada kara sakata hai. 
eka karmachari ne apatakala ke samaya chhoda़ne ke lie bhugatana nahim hoga, chhantani, lekina chhutti aura bimara chhoda़ ko banda karmachari ke khate mem jabri chhutti se lautane para jama ho karane ke lie karana ke daurana jama karate rahenge. 
yadi koi kami bala mem karmachari ritarna se pahale ghatita hona chahie, jabaki para utara anya aprayukta chhutti ke satha bhugatana kiya jaega / bonasa hai ki hatha para layoff ke samaya tha chhoda़ dena hai, avakasha sanchita chhoda़o. 
apake bachche ki khansi, se chhutakara) kapada़e ka eka tukada़a mem kuchha ainisida (ajavaina) lapeto pane ke lie hai aura ise banane eka genda mem. 
garmi eka lohe ya 'tava' aura dhire se apake bachche ki chhati para yaha jagaha ke satha kapada़e ki genda. 
itana garma nahim yakina hai ki kapada़a hai karo, ki vaha apane bachche ki tvacha jala jati. 
eka aura tipa tulasi pana ko kuchalane ke lie aura isa pesta karane ke lie shahada ki kuchha bundem joda़ne hai. 
maim apane dasa para mahine ke bete, yaha koshisha ki hai aura yaha sacha mem kama karata hai. 
apa bhi apane bachche ko kuchha garma dudha mem haldi ki eka chutaki ke satha pine ke lie mila de sakata hai. 
maim apane 14 mahine purane ke lie jaba vaha eka thanda tha isa prabhavi upaya ka istemala kiya hai. 
hita paudara ke satha kapura kuchha nariyala tela aura ise eka gunaguna karane ke lie rajya ko thanda karane ke lie anumati dete haim. 
eka bara vaham, chhati, pitha, naka, gale aura mathe ka tela lagu hote haim. 
isa thanda mem kami ke bare mem ama taura para 2 dinom ke lie avedana lagatara do bara eka dina ke bada. 
yadi apa khansi hai / raktadhikya, aura adha nimbu ka rasa garma pani ke eka glasa ke lie shahada ka eka bada़a chamacha joda़em. 
jaba bhi mere 4 sala ki unra mem, maim use adaraka ka rasa aura shahada aura phira kuchha garma dudha de eka khansi hai. 
nami bhi hai, lekina madada agara apa eka nahim hai, eka vyapaka bartana ke pani ke satha apane bachche ke kamare mem bhari jagaha hai. 
prathamika dekhabhala chikitsakom ki kami kai vikasita deshom mem eka badha़ti hui samasya hai. 
sanyukta rajya amerika mem, medikala chhatrom parivara pratha prashikshana mem pravesha ki sankhya 50% se 1997 aura 2005 ke bicha mem gira diya. 
1998 mem, antarika davai nivasiyom ka adha hai, lekina 2006 se, 80% se adhika visheshajna ya asapatala bane prathamika dekhabhala chuna. 
2004 mem, amerika mem visheshajnom ka madhya aya do bara hai ki PCPs ki, aura isa antara ko chauda़a hai. 
sanyukta rajya amerika mem prathamika dhyana prathaom adhikadhika videshi chikitsa snatakom para khali padom ko bharane ke lie nirbhara karate haim. 
vikasashila deshom prathamika dekhabhala chikitsakom mem eka aura bhi adhika mahatvapurna asamanata samana karana pada़ta hai. 
isa pena amerikana svasthya sangathana 2005 mem kaha ki ... amerika kshetra hai, lekina svasthya ke kshetra mem mahatvapurna pragati ki hai mahatvapurna chunautiyom aura asamanataom rahane ki suchana di. 
inamem sabase mahatvapurna avashyakata ki abadi ke sabhi kshetrom ke lie gunavatta svasthya dekhabhala pradana karane ke lie hai. 
pichhale 27 sala se adhika ka anubhava hai ki svasthya pranali hai ki prathamika svasthya dekhabhala ke siddhantom ka palana donom vyaktigata aura sarvajanika svasthya ke sambandha mem adhika se adhika kushalata aura behatara svasthya parinama upaja ko darshata hai. 
karyakarta ki sankhya aura gunavatta sakaratmaka pratirakshana kavareja, prathamika dekhabhala ki autaricha ke satha juda़e rahe haim aura shishu, bachche aura matri bachane. 
daॉktarom ki gunavatta aura unake vitarana ke ghanatva ko hridaya rogom mem sakaratmaka parinama ke satha sahasambandhi dikhaya gaya hai. 
svasthya pranaliyom mem, (prathamika dekhabhala) karyakartaom dvarapala aura nevigetarsa ko, prabhavi ya bekara, anya sabhi sansadhanom ka upayoga davaom, tikom aura apurti jaise ke lie ke rupa mem karya karate haim. 
vartamana mem mahatvapurna kami 2.4 lakha daॉktarom, narsom aura dhatriyom ke eka vaishvika ghate ke barabara ke satha 57 deshom mem haim. 
kai deshom mem, simita abhi taka mahangi peshevara ke kaushala achchha svasthya jaruratom ke sthaniya propha़aila mem milana nahim kara rahe haim. 
sabhi deshom mem bhejane ki kami shahari ekagrata aura gramina ghate dvara visheshata se pida़ita haim. 
amira deshom jo uchcha dhyana mangom ke satha purana aura apakshayi rogom ki disha mem eka badalava ka karana hoga kama prajanana aura bujurga logom ki bada़i abadi ka eka bhavishya samana karana pada़ta hai. 
badha़ti antarala gariba kshetrom se svasthya karmiyom ka bahirvaha para bhi adhika se adhika dabava lagu hoga. 
sata deshom mem 6,000 prathamika dekhabhala chikitsakom ka eka sarvekshana batata hai ki dhyana ki gunavatta ko prabhavita kai kshetrom mem asamanataom ka pata chala. 
matabheda dekhabhala ki lagata ka rujana ka palana nahim kiya, aura sanyukta rajya amerika mem prathamika dekhabhala chikitsaka anya deshom mem apane samakakshom ke pichhe, jo amerika ke do se tina guna adhika prati vyakti kharcha isa tathya ke bavajuda. 
ke lie prabandha-ghante dekhabhala ke bada lagabhaga do bara ke rupa mem nidaralainda mem, jarmani aura nyujilainda kanada aura sanyukta rajya amerika hai, jaham marijom ko apatakalina suvidhaom para nirbhara hoga ke rupa mem ama the. 
anya pramukha asamanataom pranaliyom ke bare mem palana marijom ki dekhabhala ko yada dilane ke lie svachalita shamila haim, rogiyom ko parikshana ke parinama de ya hanikaraka dava sahabhagitaom ki chetavani di. 
isamem antara ke rupa mem achchhi taraha se bicha mem prathamika dekhabhala daॉktarom, vittiya protsahana ke bare mem dekhabhala ki gunavatta mem sudhara karane ke lie gae the. 
Hantavirus jo vayarasa ya unake gobara, mutra athava lara se se sankramita haim hirana chuhom se logom mem phaila raha hai. 
hanta vyarasa jaba chhote kana ki vayarasa bana eyaraborna shamila mem sansa hai. 
hirana chuhe, kisi na kisi bhuri hoti haim aura puri peparakorna ki taraha laga rahe eka sapheda andarabaili aura unake gobara hai. 
hirana chuhom gharom aura bahari imarato mem rajadhani svasthya kshetra mem phansaya gaya hai, lekina ve aura adhika gramina kshetrom mem ama hai.
tibi ki rokathama aura niyantrana do samanantara drishtikona leta hai. 
mem pahali, tibi ke satha loga aura unake samparkom ki pahachana kara rahe haim to ilaja kiya. 
sankramana ki pahachana aksara tibi ke lie uchcha jokhima samuhom parikshana shamila hai. 
dusare drishtikona mem, bachchom ko tibi se unhem bachane ke lie tika lagaya jata hai. 
durbhagya se, koi tika hai ki vayaskom ke lie vishvasaniya sanrakshana pradana karata upalabdha hai. 
ushnakatibandhiya kshetrom mem jaham vishesha mayko bekatiriya ki ghatanaom, tibi sahita mayko bekatiriya karane ke lie jokhima adhika hai tibi ke khilapha kuchha sanrakshana deta hai. 
kai deshom mem unake tibi niyantrana karyakrama ke bhaga ke rupa mem, vishesha rupa se shishuom ke lie bisiji vaiksina ka upayoga karem. 
bisiji vaiksina tibi ke lie pahala tika tha aura pashchara sansthana phransa mem 1905 aura 1921 ke bicha mem vikasita kiya hai. 
bisiji ke satha tathapi, samuhika tikakarana dvitiya vishva yuddha ke bada jaba taka shuru nahim kiya. 
tibi ke gambhira rupa bachchom mem (jaise maininjaitisa) ko rokane ke lie bisiji ki rakshatmaka kshamata 80% se adhika hai, kishorom mem phuphphusiya tibi aura vayaskom ko rokane ke lie apani rakshatmaka prabhavakarita, 0 se 80% se lekara chara raha hai. 
dakshina aphrika mem tibi ke sarvochcha prasara ke satha desha, bisiji sabhi bachchom ke lie tina varsha ki ayu ke tahata di jati hai. 
halanki, bisiji ki prabhavashilata hai kshetrom mem jaham mayko bekatiriya kama prachalita haim kama, bisiji isalie ina deshom mem puri abadi ke lie nahim diya hai. 
amerika mem, udaharana ke lie, bisiji tika hai jo vishishta manadandom ko pura karane ke lie logom ko chhoda़kara sipharisha nahim hai. 
shishuom ya nakaratmaka tvacha ke satha bachchom ko jo lagatara ya anupacharita marijom ka ilaja karane ke lie ya lagatara samparka mem rahate haim vibhinna dava-pratirodhaka tibi se avagata karaya jaega pariksha parinama. 
helthakeyara karmachariyom jisamem emadiara ke eka uchcha pratishata-tibi ke rogi paya gaya hai setingsa, emadiara ke sancharana-tibi mem eka vyakti ke adhara para mana jata hai, aura tibi niyantrana savadhaniyom ki sambhavana hai karyanvita kara diya gaya hai aura saphala nahim rahe the. 
kai nae tike vikasita kie ja rahe haim tibi sankramana ko rokane ke lie. 
pahala punah sanyojaka tapedika vaiksina 2004 mem, neshanala instityuta ऑpha elarji aura sankramaka roga (niaida) dvara prayojita sanyukta rajya amerika mem klinikala parikshana mem pravesha kiya. 
eka 2005 ka adhyayana dikhaya hai ki paramparagata rasayana chikitsa ke satha satha hi phira se bachava sankramana chuhom mem baiktiriya ke gayaba hone ko gati kara sakate haim di eka dienae tibi vaiksina. 
dienae tibi vaiksina chara se pancha sala insanom mem upalabdha hone laga sakata hai. 
eka bahuta ashajanaka tibi vaiksina, emaviai85a, vartamana charana mem dakshina aphrika mem dvitiya parikshanom eka samuha ऑksapha़orda vishvavidyalaya ke netritva hai, aura eka anuvanshika rupa se parishkrita chechaka vayarasa para adharita hai. 
vartamana tike, shodhakartaom aura niti ki simaom ke karana puraskara, kara protsahanom aura agrima bajara pratibaddhataom sahita tika vikasa ki nai arthika maॉdala ko badha़ava dene rahe haim. 
vishva svasthya sangathana ke anusara (dablyuechao), lagabhaga 2 biliyana loga eka ne duniya ki abadi ka tritiya ne kshayaroga rogaja़naka़ ko ujagara kiya gaya hai. 
salana, 8 lakha logom ko kshayaroga se, aura bimara ho 2 karoda़ loga isa bimari se duniya bhara mem mara jate haim. 
2004 mem lagabhaga 14.6 miliyana logom ko 9 lakha nae mamalom se sakriya tibi roga tha. 
varshika ghatana dara 356 100000 prati aphrika mem 41 100000 prati karane ke lie amerika mem se bhinna hota hai. 
kshaya roga prajanana ayu ki mahilaom ki duniya ki sabase bada़i sankramaka hatyara aura echaaivi se grasta logom mem nrityu ka pramukha karana / edsa hai. 
2005 mem, tibi ke sarvochcha anumanita ghatanaom se desha svajilainda, 100,000 logom ko prati 1262 mamalom ke satha tha. 
bharata, para 1.8 karoda़ mamalom ke satha sankramanom ki sankhya sabase adhika hai. 
vikasita deshom mem, tapedika kama ama hai aura yaha mukhya rupa se eka shahari roga. 
britena mem tibi ghatanaom 40 prati 100000 landana mem se 5 pratishata kama 100000 gramina dakshina pashchima mem inglainda ki tulana mem se lekara, rashtriya ausata 13 pratishata hai 100000. 
pashchimi yuropa mem sabase adhika darom purtagala mem haim (42 100000 prati) aura spena (20 pratishata 100000). 
ye dara 113 prati 100,000 china mem aura 64 pratishata 100000 mem brajila ke satha tulana karem. 
sanyukta rajya amerika mem, samagra tapedika mamale ki dara 4.9 100000 vyaktiyom prati 2004 mem tha. 
kshayaroga ke mamale unra ke satha badalati hai. 
aphrika mem tibi ke mukhya kishorom aura yuva vayaskom ko prabhavita karata hai. 
halanki, desha mem jaham tibi uchcha sanyukta rajya amerika jaise kama bhara, para chala gaya hai, tibi mukhya rupa se purane logom ki eka bimari hai. 
isamem jnata karakom ki eka sankhya hai ki logom ko aura adhika tibi sankramana karane ke lie atisamvedanashila bana rahe haim. 
saha echaaivi sankramana ke satha upa mem eka vishesha samasya ka sahara aphrika, echaaivi ke uchcha ghatana ke lie ina deshom ke karana hota hai. 
dhunrapana karane se adhika 20 sigareta bhi do se chara se tibi ka khatara badha़ta hai eka dina bara. 
tyubarakulara kshaya misra ke mammi ke rida mem paya gaya hai. 
kshaya roga ki gai manava mem prachinakala se hi maujuda hai. 
maikobaiktiriyama kshayaroga ka shighra pata lagane ke khalima bijona ke 18000 varshom dinankita vartamana se pahale bani hui hai. 
halanki, ki kshayaroga pashuom mem aura udgama to manushya ke lie, tabadala ya eka ama mulapurusha se diverged, vartamana mem yaha spashta nahim hai. 
kankala, aura tibi tha tyubarakulara kshaya mammi ke rida mem 3000-2400 i.pu. se paya gaya hai pragaitihasika manushya (4000 i.pu.) dikhata rahata hai. 
yakshma tapedika ke lie eka grika shabda hai, lagabhaga 460 i.pu., hippokretsa ke samaya rakta aura bukhara hai, jo lagabhaga hamesha ghataka tha khansate shamila ki sabase vyapaka yakshma roga ke rupa mem pahachana ki. 
anuvanshika adhyayana ki tibi dakshina amerika mem lagabhaga 2000 varshom ke lie upasthita the sujava dete haim. 
dakshina amerika mem, tapedika ke prarambhika sakshya ke Paracas ke satha juda़a hua hai-100 i. ke lagabhaga karane ke lie kavera sanskriti (lagabhaga 750 i.pu.). 
audyogika kranti se pahale, tapedika kabhi kabhi vemapayarapana mana jata raha ho sakata hai. 
jaba eka parivara ke eka sadasya ko tapedika se mara gaya, vaha dhire dhire apane svasthya kho jaega sankramita the anya sadasya haim. 
loga kahate haim ki kshayaroga mula pida़ita ke parivara ke anya sadasyom se jivana khali ke karana kiya gaya hai. 
isake alava, jo tibi ke lakshanom logom vempayara guna hone ka kya vichara karane ke lie bhi isi taraha ka pradarshana kiya tha loga. 
tibi ke satha loga aksara ankhom aisi, sujana lala ke rupa mem lakshana haim (jo bhi chamakadara roshani karane ke lie), pili tvacha eka samvedanashilata banata hai aura rakta khansate hai, pida़ita khuna chusa kara rahi thi khuna ke isa nukasana ko bharane ke lie eka hi rasta hai ki isa vichara ka sujava de. 
eka aura loka vishvasa ko jimmedara thaharaya, rata, pari mem bhaga lene ke lie hai, isalie hai ki pida़ita ko dura arama ki kami ke karana barbada majabura kiya ja raha, yaha vishvasa sabase jaba eka majabuta sambandha mem pariyom aura nrita ke bicha dekha tha ama tha. 
isi taraha, lekina kama samanyatah, yaha pida़itom khinna kiya ja raha karane ke lie-thaharaya tha ghoda़om mem chuda़ailom (hegasa) dvara unake rata baithakom, phira arama ki kami jisaka parinama ki yatra karane ke lie parivartita kiya ja raha. 
tibi unnisavim sadi mem prachalita hui thi. 
bahuta se loga tibi se sapesa pithiseka ya ki 'asha' ke rupa mem sandarbhita ullasonmada ki bhavanaom ka utpadana vishvasa ke tapedika़. 
ke rupa mem isa roga pragati yaha hai ki tibi pida़ita jo kalakara the vishvasa tha rachanatmakata ke barsata tha. 
yaha bhi hai ki tibi pida़ita se pahale ve jo mahilaom ko aura adhika sundara aura purushom aura adhika rachanatmaka banaya mara urja ke eka antima phata adhigrahita mana jata tha. 
isa chikitsa ki topa ibna sina (aivisena) dvara 1020s mem likha ko tapedika tarikhom ke isa adhyayana se pahale. 
ibna sina eka sankramaka bimari ke rupa mem phuphphusiya tapedika ki pahachana karane ke lie pahali chikitsaka tha. 
yaha madhumeha ke satha sahayoga se pahachana karane ke lie, aura kaha ki kshayaroga ka sujava sabase pahale pahali mitti aura pani ke satha samparka ke madhyama se phaila sakata hai. 
adesha mem tapedika ke prasara ko simita karane ke lie ibna sina sangarodha ki paddhati vikasita ki. 
yadyapi yaha kaha gaya hai ki phephada़e ke pharma 'ke satha' daॉ. richarda moratana dvara 1689 mem, tibi ke lakshanom ki vividhata ke karana tubarakalesa juda़e the sthapita kiya gaya tha. 
tibi ke 1820s jaba taka eka bhi roga ke rupa mem pahachana ki hai aura nahim tha 1839 ji. la. sakonalena dvara jaba taka ‘tapedika’ ka nama nahim tha. 
isa varsha 1838-1845 ke daurana, daॉ. jaॉna karogahana, memota gupha ka malika hai, lagatara tapamana aura hava gupha ki pavitrata ke satha isa roga ilaja ki asha mem gupha mem kshayaroga pida़ita ke eka nambara laya: ve eka varsha ke bhitara nrityu ho gai . 
pahale tibi aspatala 1859 mem golabarasadropha mem, jarmani haramana (aja sakolovasako, polainda) dvara barehamara khola. 
isa dandanu karana tibi, maikobaiktiriyama kshayaroga, ki pahachana ki gai thi aura 24 marcha, 1882 mem raॉbarta kaॉkha dvara bataya. 
isa dandanu karana tibi, maikobaiktiriyama kshayaroga, ki pahachana ki gai thi aura 24 marcha, 1882 mem raॉbarta kaॉkha dvara bataya. 
raॉbarta kaॉkha phijiyolaॉji ya dava mem 1905 mem isa khoja ke lie nobela puraskara prapta kiya. 
kaॉkha ki gojatiya (pashu) aura manava tapedika hai, jo sankramana ke eka srota ke rupa mem sankramita dudha ki manyata deri samana the vishvasa nahim tha. 
kaॉkha, bula 1890 mem tapedika ke lie eka ilaja ke rupa mem tayubara kali bekeli ke eka glycerin nikalane ki ghoshana ki hai vaha 'tayubarakolena'. 
tayubarakolena, lekina prabhavi nahim thi bada mem purva ke lie eka parikshana rogasuchaka tapedika ke rupa mem rupantarita kiya gaya. 
tapedika ke khilapha pratiraksha mem pahali vastavika saphalata albarta kalameti aura kemili guerina dvara paya bobaina-tanava kshayaroga se 1906 mem vikasita kiya gaya tha. 
kalameti aura gurena ke kshayaroga ke khilapha pratiraksha kaha jata tha bisiji' (dandanu). 
isa bisiji vaiksina pahali manushyom para 1921 mem phransa mem istemala kiya gaya tha. 
bisiji dvitiya vishva yuddha ke bisiji hai ki amerika mem vyapaka svikriti prapta, greta britena ke bada jaba taka, aura jarmani nahim tha. 
kshaya roga, ya 'upabhoga' ke rupa mem samanyatah jnata tha, shahari garibom ke eka sthanika bimari ke rupa mem 19 vim aura jaldi 20 vim shatabdiyom mem sabase vyapaka sarvajanika chinta ke karana hota hai. 
1815 mem, eka inglainda mem chara mautom mem upabhoga ki gai thi, 1918 se eka phransa mem chhaha mautom mem abhi bhi tibi ke karana the. 
20 vim sadi mem eka anumana ke anusara 100 lakha logom ko mara dala kshayaroga. 
isa 1880s mem sthapana ke bada kaha ki isa roga sankramaka tha, tibi britena mem eka darja karana pada़a roga banaya gaya tha. 
aise abhiyanom ko sarvajanika sthanom mem thukana banda karane ke lie, aura sankramita the protsahita senatoriya mem pravesha ke lie gariba the ki jaisa jelom britena mem. 
madhya aura uchcha varga ke lie senatoriya utkrishta dekhabhala aura lagatara dhyana die jane ki peshakasha ki. 
sabase achchha shartom ke tahata senatoriya mem taja hava aura parishrama ki jo bhi poraportida labha bhi hai, jo pravesha ke 50% pancha sala (1916) ke bhitara mara chuke the. 
jana svasthya abhiyana tibi ke prasara ko rokane ki koshisha ki. 
ke rupa mem eka taraha se tapedika karyakramom ke lie paise jutane ke lie krisamasa muharom ka samvardhana denamarka mem 1904 ke daurana shuru hua. 
krisamasa ki padonnati sanyukta rajya amerika aura kanada ke lie 1907-08 mem rashtriya kshayaroga esosieshana mem madada karane ke lie (bada mem ameriki phephada़e esosieshana) namaka vistrita muharom. 
sanyukta rajya amerika ke tapedika ke prasara ke bare mem, chinta mem sapitonsa mem chhoda़kara sarvajanika thukana nishedha karane ke lie andolana mem eka bhumika nibhai. 
yuropa mem, tibi se nrityu 500 se 100.000 ke 1850 mem se 50 ko 100.000 ke 1950 se bahara gira gai. 
sarvajanika svasthya mem sudhara pratijaivikom ke ane se pahale hi tapedika kama the. 
halanki tapedika ki bimari sarvajanika svasthya ke lie eka mahatvapurna khatara bane rahe. 
jaba chikitsa anusandhana parishada ke britena mem 1913 mem apane shuruati dhyana tapedika shodha ka gathana kiya gaya tha. 
yaha 1946 mem entibayotika satrepatomayasina hai ki prabhavi upachara aura ilaja sambhava hone ke vikasa ke satha jaba taka nahim tha. 
pahale entibayotika ki shuruata karane ke lie, senatoriya alava kevala upachara shalya hastakshepom tha. 
tibi phira ina sankramanom ke lie ilaja ke hisse ke rupa mem sarjari shuru ki hai. 
yaham, sine keveti ke sarjikala hatane, satha hi satha phephada़om mem jivanuom ki sankhya kama ho jaega khuna mem davaom ke lie shesha jivanu ke jokhima ko badha़ana, aura isalie isa rasayana chikitsa ki prabhavashilata ko badha़ane ke lie socha hai. 
ummida hai ki isa bimari ko puri taraha samapta kiya ja sakata hai dava ke udaya-1980 ke dashaka mem pratirodhi upabhedom se pani phira gaya. 
udaharana ke lie, britena mem tapedika ke mamalom, 117.000 lagabhaga 1913 mem nambara, lagabhaga 5000 ke lie 1987 mem gira gai thi, lekina mamale phira, gulaba aura 2000 mem 6,300 taka pahunchane aura 2005 mem 7600 mamalom ka. 
nyuyaॉrka mem sarvajanika svasthya suvidhaom aura echaaivi ke udbhava ke unmulana ke karana, vaham dera se 1980 ke dashaka mem punarutthana tha. 
una davaom ka unake korsa ko pura karane mem asaphala rahane ki sankhya adhika hai. 
kshayaroga ke punarutthana eka vaishvika svasthya apata sthiti ki ghoshana mem vishva svasthya sangathana dvara 1993 mem hui.
kshaya roga stanadhariyom dvara kiya ja sakata hai, billiyom aura kuttom jaise palatu prajati, amataura kshayaroga se mukta ho, lekina jangali janavarom vahaka ho sakate haim. 
kuchha sthanom mem, viniyamom navinata palatu janavara ke svamitva simita tibi ke prasara ko rokane ke lie lakshya, udaharana ke lie, amerika ke kailiphorniya rajya palatu jerabilasa ke svamitva rokati. 
maikobaiktiriyama bovisa pashuom mem kshaya roga ka karana banata hai. 
eka prayasa ke pashu aura nyujilainda ke hirana junda se gojatiya kshayaroga unmulana ke lie chala raha hai. 
aisa lagata hai ki junda sankramana adhika kshetrom mem jaham ऑstreliyai brasha jaise sankramita vektara prajatiyom-posumasa kheta mem gharelu pashuom / jada़i simaom ke satha samparka mem ane punchha sambhavana hai paya gaya hai. 
posuma unmulana ke madhyama se vekatara niyantrana aura niyamita rupa se nigarani ke madhyama se pashuom jundom mem roga ke stara ki nigarani eka do ke rupa mem isa bimari ke nyujilainda ridindaga karane ayami rukha dekha hai 
isa ayaralainda aura britena mem, bejarsa gojatiya tapedika ke prasarana ke lie eka vektara prajatiyom ke rupa mem pahachana ki gai hai. 
parinamasvarupa, sarakarom kuchha timahiyom se dabava mem, mukhyatah deyari kisanom, gojatiya tibi ki ghatanaom ko kama karane ke uddeshya se kuchha kshetrom mem bejarsa ke unmulana ke eka sakriya abhiyana maunta karane ke lie ae haim. 
britena sarakara isa mudde para nahim, kama se kama hi pratibaddha nahim hai kyonki yaha logom ki raya bhaya: bejarsa eka sanrakshita prajati haim. 
pashuom mem tibi ki ghatanaom para kolinga ki prabhavashilata proponentasa aura virodhiyom ko apani padha़ai unaki sthiti ka samarthana karane ke lie dekara ke satha eka vivadaspada mudda hai. 
eka svatantra adhyayana dala bejara ki sambhavana prabhavakarita 18 juna 2007 ko suchana di hai ki yaha karagara hone ki sambhavana nahim thi kolinga ke dvara eka 9 sala ka vaijnanika adhyayana hai aura vastava mem tibi ke prasara ko badha़aya ja sakata hai. 
svatantra adhyayana dala sara jaॉna baॉrna ki adhyakshata mem kiya gaya aura do sankhyikividom, pradhyapaka karistala doneli aura sara devida kaॉksa hai, yunaiteda kingadama mem sabase pratishthita sankhyikivid shamila the. 
doneli aura kaॉksa jo vibhinna nitiyom ke prabhava ke bare mem vistrita matratmaka purvanumana banane ke lie istemala kiya gaya tha bejara abadi ka eka parishkrita stokasatika maॉdala ka utpadana kiya. 
isa baॉrna ki riporta ki sipharishom ko mantriyom ke lie eka ashcharya ke rupa mem aya tha. 
britena sarakara ke mukhya vaijnanika salahakara sara devida kinga punah karane ke lie eka samiti ki riporta ki jancha baॉrna bulai. 
raja ka samiti 30 julai ko eka riporta taiyara ki, baॉrna ki 9 sala ke adhyayana, sapha una ke baॉrna riporta aura sipharisha bejara ka khandana kiya jisaka nishkarsha ke prakashana kevala eka mahine ke bada. 
raja samiti aura kisi bhi sankhyikividom ko shamila nahim kiya sankhyikiya maॉdala ke doneli & kaॉksa ka istemala nahim kiya. 
parinamasvarupa, bejara ke mudde behada ko yunaiteda kingadama mem vivadaspada bana hua hai. 
pratiraksha kami sindroma ya adhigrahana inyuno sindroma (edsa ya edsa) ekvayarda lakshana aura sankramana pratiraksha pranali manava mem manava inyuno vayarasa (echaaivi) ke karana hota karane ke lie vishesha nukasana se utpanna hone ke eka sangraha, aura anya prajatiyom mem isi taraha ke vayarasa (esaivi hai, phaivi, adi). 
sthiti ki dera mancha vyaktiyom avasaravadi sankramana aura tyumara ko atisamvedanashila chhoda़ta hai. 
yadyapi edsa ke lie upachara aura echaaivi vayarasa ki pragati dhima karane ke lie maujuda haim, abhi koi ilaja nahim jana jata hai. 
echaaivi eka shleshma jilli ya khuna ke sidhe samparka ke madhyama se eka sharirika drava rakta, virya, yoni drava, priseminala drava jaise echaaivi, yukta ke satha, aura stana dudha phailata hai. 
yaha sancharana, yoni ya guda maukhika linga, raktadhana, chamada़e ke niche sandushita sui, mam aura bachche ke bicha vinimaya ke rupa mem garbhavastha, prasava ya stanapana, ya eka ke upara sharirika tarala padarthom ke lie anya pradarshana ke daurana a sakata hai. 
jyadatara anusandhanakarta ki echaaivi mem utpanna upa sahara aphrika ne bisavim shatabdi ke daurana vishvasa karate haim. 
2007 mem, eka anumana 33.2 miliyana loga isa bimari se duniya bhara mem rahate the, aura yaha 330000 bachchom sahita eka anumanita 2.1 karoda़ logom ke jivana, ka dava kiya. 
ina mautom ki tina chauthai se adhika upa mem hui sahara aphrika, arthika vikasa retarding aura manava punji ko nashta. 
entiritrovairala upachara, lekina entiritrovairala dava ke niyamita upayoga sabhi deshom mem upalabdha nahim hai, donom ke nrityu dara aura echaaivi sankramana ke rogom ki sankhya kama kara deta hai. 
edsa ke lakshana mukhyatah hai ki samanya rupa se svastha pratiraksha pranali ke satha vyaktiyom mem vikasita nahim hai paristhitiyom ka parinama hai. 
ina sthitiyom se jyadatara sankramana baiktiriya, vayarasa, kavaka aura kaha ki samanya rupa se pratiraksha pranali ke tatvom dvara niyantrita kara rahe haim parajivi ke karana hota hai ki echaaivi harjana. 
avasaravadi sankramana edsa ke satha logom mem ama hai. 
edsa ke satha loga bhi kaposi hai sarakoma, garbhashaya griva ke kainsara aura pratiraksha pranali lemaphomasa ke nama se kainsara jaise vibhinna kainsara vikasita hone ka eka jokhima badha़a hai. 
isake atirikta, edsa ke satha logom ko aksara bukhara jaise sankramana ka pranaligata lakshana hai, sweats rata (visheshakara) mem, sujana granthiyom, thanda lagana, kamajori hai, aura vajana ghatane. 
edsa ke nidana ke bada, entiretrovairala therepi ke satha maujuda ausata uttarajivita samaya (2005 mem) ke rupa mem karane ke lie kiya ja adhika 5 varsha ka anumana hai banaya hai. 
nai upachara aura vikasita karane ke karana echaaivi ke ilaja ke lie pratirodha vikasita karane ke lie jari hai, astitva samaya ke anumanom mem parivartana karana jari rakhane ke lie jari ki sambhavana hai. 
entiretrovairala therepi ke bina, mauta samanya rupa se eka varsha ke bhitara hota hai. 
adhikansha rogiyom ko avasaravadi sankramana ya meliganesisa pratiraksha pranali ke pragatishila viphalata ke satha juda़e se mara jate haim. 
naidanika bimari badha़ne ki dara mem vyapaka rupa se dikhaya gaya hai aura vyaktiyom ke bicha mejabana samvedanashilata aura pratiraksha samaroha mem svasthya dekhabhala aura saha jaise kai karakom-sankramana hai, satha hi karakom ko vayarala tanava se sambandhita se prabhavita hone ka badalata hai. 
edsa rogiyom bhaga mem bhaugolika kshetra mem ina sankramanom ki vyapakata para nirbhara hai ki vikasa ki vishishta avasaravadi sankramana rogi rahata hai jisamem. 
nimokayatisa nimoniya (mulatah nimokayatisa ke rupa mem, aura kareni nimoniya jnata abhi bhi pisipi, jo aba nimokayatisa nimoniya ke lie khada़a ke rupa mem) apekshakrita svastha, imayonokomaponenta logom ko hai, lekina bicha mem ama echaaivi mem durlabha hai sankramita vyaktiyom sankshipta. 
pashchimi deshom mem prabhavi nidana, upachara aura dinacharya prophilaiksisa ke agamana se pahale, nimokayatisa jeroveki mauta ka eka ama tatkala karana tha. 
jaba taka ki sidi4 ginati 200 μla pratishata se kama hai, yadyapi yaha amataura para ghatita nahim hai vikasashila deshom mem, yaha abhi bhi eka amparikshata vyaktiyom mem edsa ka pahala sanketa ki hai, 
kshaya roga (tibi) echaaivi sankramana ke satha juda़e mem advitiya hai. 
kyonki logom ko kshayaroga imasunokomaponenta ko shvasana marga ke madhyama se, eka bara asani se pahachana mem upacharya hai, sankramaka hai prathamika avastha echaaivi roga mem, ho sakata hai aura nashili davaom ke upachara ke satha nivaraniya hai. 
halanki, bahu dava pratirodha sambhavatah eka gambhira samasya hai. 
prarambhika avastha echaaivi sankramana (sidi4 ginati mem> μla prati 300 koshikaom), tibi amataura para eka phephada़e ki bimari ke rupa mem prastuta karata hai. 
unnata echaaivi sankramana mem, tibi aksara vishesha eksatrapalamoneri (pranaligata satha) roga eka ama suvidha prastuta karata hai. 
lakshana amataura para aura samvaidhanika haim eka vishesha saita para sthaniyakrita nahim haim, to aksara trektasa, jigara, kshetriya limpha nodsa asthi majja, asthi, mutra aura gaistrointestainala prabhavita hai, aura kendriya tantrika tantra. 
aisophagetisa ke ghutaki (nareti ke nichale anta ka astara ke eka sujana ya nigalane tyuba ke peta karane ke lie agrani hai). 
echaaivi sankramita vyaktiyom mem yaha samanya rupa se (harpasa simpleksa-1 ya saytomegalovyarasa) sankramana (kaindidiasisa) ya vayarala kavaka ke karana hai. 
echaaivi sankramana mem aspashtikrita purana dasta sahita kai sambhavita karana haim, ki vajaha se hai ama jivanuja (salmonela, shigela, listiriya, kaimpailobaiktara, ya esacherichiya koli). 
kuchha mamalom mem, dasta kai davaom ko echaaivi ke ilaja ke lie, ya yaha basa prathamika echaaivi sankramana ke daurana vishesha rupa se echaaivi sankramana, satha mai prayoga kiya jata ka eka paksha prabhava ho sakata hai. 
yaha bhi pratijaivikom dasta ke jivanuja karanom ka ilaja karane ke lie prayoga kiya jata ka eka paksha prabhava (kalosatridiyama belagama ke lie) ama ho sakata hai. 
echaaivi sankramana ke bada ke charanom mem, dasta intestainala patha poshaka tatvom avashoshita ke tarike mem badalava ka eka pratibimba ho, aura socha hai echaaivi ka eka mahatvapurna ghataka se sambandhita barbada ho sakata hai. 
toksoplaja़mosija़ eka bimari ke ekala sela parajivi toksoplaja़ma gondi bulaya ke karana hota hai, yaha ama taura para mastishka toksoplaja़mo insephelaitisa karana hota hai lekina yaha sankramita kara sakate haim aura ankhem aura phephada़om ki bimari ka karana. 
pragatishila maltiphokla lyukoenasephelopethi (piemaela) jisamem tantrika koshikaom ke axons kavara nyana mem nyelin ke kramika vinasha tantrika udyami ke sancharana eka dilimitetiga roga, hai. 
piemaela eka vishanu ke dvara jo avyakta rupa mem janasankhya ke 70% mem hota hai jesi vayarasa kaha jata hai, jaba kevala roga pratirakshana pranali ko buri taraha kamajora ho gaya hai karana, ke rupa mem edsa rogiyom ke lie bhi aisa hi mamala hai karana hota hai. 
edsa manobhransha parisara (aidisi) eka chayapachaya ainasephelopethi echaaivi sankramana se prerita hai aura echaaivi sankramita mastishka mayakrophegasa aura mayakrogaliya donom mejabana aura vayarala mula ke jo lukana nyurotokasinasa ke pratiraksha sakriyana dvara hai. 
vishishta snayavika ki kamiya echaaivi sankramana ke varshom ke bada, vyavahara, aura motara asamanyataem sanjnanatmaka ghatita dvara prakata kara rahe haim aura kama sidi4 ke satha juda़a hua hai + ti sela ke stara aura uchcha plajma vayarala loda. 
vyaptata pashchimi deshom mem 10-20% hai lekina bharata mem echaaivi sankramana ke kevala 1-2% hai. 
(isa jilli) ke kavaka karepatokokasa nyophormansa dvara mastishka aura merurajju ko kavara karepatokokala maininjaitisa ke meninx ke eka sankramana hai. 
karepatokokala maininjaitisa, bukhara, siradarda, thakana, matali paida kara sakata hai aura ulti. 
marijom ka bhi daura pada़ne aura bhrama ka vikasa ho sakata hai; anupacharita chhoda़ diya, yaha ghataka ho sakata hai. 
marijom ko echaaivi sankramana ke satha kaphi hada kai ghataka kainsara ki ghatanaem badha़ gai haim. 
yaha mukhyatah saha ki vajaha se hai sankramana eka oncogenic dienae vayarasa se, vishesha rupa se epasatina-barra vayarasa (esabivi), keposi hai sarakoma-juda़e harapesavyarasa (keesaechavi), aura manava pepilomavyarasa (echapivi). 
keposi hai sarakoma (esa) echaaivi mem sabase ama tyumara-sankramita rogiyom ki hai. 
1981 mem yuva samalaingika purushom mem esa tyumara ki upasthiti eka edsa mahamari ka pahala sanketa tha. 
eka gamaherapesa vayarasa ke karana hota kaposi hai serakoma bulaya-juda़e dada vayarasa (keesaechavi), yaha aksara tvacha para thoda़a baingani pinda ke rupa mem prakata hota hai, lekina anya angom ko prabhavita kara sakata hai, vishesha rupa se munha, jatharantra sambandhi marga, aura phephada़om. 
Burkitt hai limaphoma, borakitasa-limaphoma jaise jaise uchcha greda bi sela limaphoma, visarita bada़e bi sela limaphoma (dielabisiela), aura prathamika kendriya tantrika tantra limaphoma adhika bara echaaivi se sankramita rogiyom upasthita. 
ye vishesha kainsara aksara eka gariba purvanumana purvabhasa. 
kuchha mamalom mem ina limaphoma edsa ko paribhashita kara rahe haim. 
esatena-barra vayarasa (ibivi) ya keesaechavi ina limaphoma ke kai karana. 
echaaivi sankramita mahilaom mem garbhashaya griva ke kainsara-mana jata hai edsa-paribhashita. 
saravaikala kainsara manava pepilomavyarasa (echapivi) ke karana hota hai. 
edsa ke alava-tyumara ke upara suchibaddha, echaaivi sankramita rogiyom ko paribhashita karane mem kuchha anya tyumara ki vriddhi ki jokhima para hojakina roga aura guda aura guda karanikoma jaise haim. 
halanki, stana kainsara ya brihadantra kainsara jaise kai ama tyumara, ki ghatanaom, echaaivi mem vriddhi nahim karata hai sankramita rogiyom. 
kshetrom mem jaham echaaiaiarati vyapaka edsa, bahuta se edsa ki ghatanaom ke ilaja ke lie istemala kiya jata hai sambandhi malignancies kama hai. 
ghataka kainsara samagra echaaivi ki mauta ka sabase ama karana-sankramita rogiyom bana gae haim. 
edsa rogiyom ko aksara avasaravadi sankramana hai ki gaira ke satha upasthita vishesha lakshana, vikasa vishesha rupa se kama greda jvarom aura vajana ghatane. 
ye maikobaiktiriyama eviyama-intraselayulara aura sayatomegalovayarasa (siemavi) ke satha sankramana shamila haim. 
siemavi, ke rupa mem hai, aura upara varnita siemavi retinaitisa andhapana paida kara sakata hai brihadantrashotha paida kara sakata hai. 
penisiliosisa penisiliama ke karana maranephaphe aba tisari sabase ama avasaravadi sankramana hai (eksatrapalamoneri kshayaroga aura kerepatokosisa) mem echaaivi ke bada dakshinapurva eshiya ke sthanikamari vale kshetra mem sakaratmaka vyaktiyom. 
edsa echaaivi ke sankramana ka sabase gambhira tvarana hai. 
echaaivi ki mukhyatah sidi4 jaise manava pratiraksha pranali ka mahatvapurna anga infects eka retarovayarasa + ti koshikaom hai (ti koshikaom ke eka sabaseta), mekarophegasa aura vriksha ke samana koshikaom. 
jaba echaaivi sidi4 + ti koshikaom ko mara dalata hai, taki vaham kama 200 sidi4 + microliter (μla) khuna ke prati ti koshikaom rahe haim, selulara rogakshamata kho diya hai. 
kuchha deshom mem, sanyukta rajya amerika jaise, yaha edsa ke nidana ke lie le jata hai. 
anya nyayalaya mem, aise mem kanada ke rupa mem, edsa kevala jaba eka vyakti echaaivi se sankramita eka ya kai edsa ke adhika se sambandhita avasaravadi sankramana ya kainsara ke satha ka nidana hai nidana hai. 
entiritrovairala chikitsa ke abhava mem, echaaivi sankramana edsa se pragati ke bicha ka samaya nau se dasa varsha, aura edsa ke vikasa ke bada bicha ka astitva samaya hai kevala 9.2 mahine hai. 
halanki, naidanika bimari badha़ne ki dara mem vyapaka rupa se vyaktiyom ke bicha do saptaha taka 20 varshom se bhinna hota hai. 
purane logom ko, aura isalie yuva logom ki tulana mem teji se pragati ki bimari ka eka bada़a khatara hai kamajora pratiraksha pranali hai. 
bechari tapedika jaise ki svasthya dekhabhala aura sapaye gaye sankramana ke astitva ke lie upayoga mem bhi teji se pragati ki bimari ke lie logom ko predisaposa mai 
sankramita vyakti ke anuvanshika virasata kuchha loga echaaivi ke kuchha upabhedom karane ke lie pratirodhi rahe haim eka aura mahatvapurna bhumika nibhata hai. 
yaha homoज़yagosa sisiara5 logom ke satha hota hai ki eka udaharana-? 32 badalava karane ke lie echaaivi sankramana ke kuchha upabhedom ke satha pratirodhi rahe haim. 
echaaivi anuvanshika chara raha hai aura jo naidanika roga mem pragati ke vibhinna darom karana alaga upabhedom, ke rupa mem maujuda hai. 
atyadhika sakriya entiretrovairala therepi ka prayoga edsa aura bicha ka astitva samaya mem pragati ke donom ke bicha ka samaya prologa. 
vaijnanikom aura karyakartaom ke eka chhote se alpasankhyaka tarikom echaaivi aura edsa, echaaivi hi hai, ya vartamana parikshana aura upachara ki vaidhata ke astitva ke bicha ke sambandha savala. 
galataphahami ki sankhya echaaivi asapasa hui hai / edsa. 
tina sabase ama hai ki edsa akasmika samparka ke madhyama se phaila sakata hai, eka kumvari, aura edsa ke ilaja karenge ki echaaivi kevala samalaingika purushom aura nasha karane valom ko sankramita kara sakate haim ki yauna samparka. 
anya galataphahami hai ki samalaingika purushom ke bicha guda sambhoga ke kisi bhi kritya edsa sankramana karane ke lie, aura ho sakata hai skulom mem samalaingikata aura echaaivi ki khuli charcha samalaingikata aura edsa ki vriddhi dara ko badha़ava milega haim. 
5 juna, 1981 ke bada se kai paribhashaem mahamari ki nigarani ke lie bangui paribhasha hai aura 1994 ka vistara vishva svasthya sangathana edsa mamale paribhasha itani ke rupa mem vikasita kiya gaya hai. 
ke rupa mem ve na to samvedanashila haim lekina, rogiyom ke naidanika manchana ina paddhatiyom ke lie eka ichchhita ka upayoga karate haim, nahim tha aura na hi vishishta.
yaha thika se sthapana hai ki mata ki shiksha ka vikasashila deshom mem bala poshahara para sakaratmaka asara pada़ta hai
anya logo ki shiksha ka asara kama adhdhayana hua hai - mataom 'mitrom, pada़osiyom aura parivara se
svatantra ke prabhava ki jancha karane ke lie mata, pita aura dadi ki shiksha aura bachche ki unchai para aura unra aura vajana ke lie jada skora, aura samudaya ki bhumika ka stara matri saksharata aura abhibhavakom ki shiksha aura anya vyakti-stara ke karaka.
5692 bachchom ke anubhagiya deta andhra pradesha rajya bharata aura viyatanama mem sthalom se namuna ke lie (20 pratyeka desha se) aura phira samudaya (31 viyatanama se aura 102 bharata se) vishleshana kiya gaya
bahustara pratigamana vishleshana saha-nirmatao ke khate aura bhaugolika klastaringa ki tippaniyom
bala poshana sakaratmaka aura svatantra rupa se mata, pita aura dadi ki shiksha se juda hai. 
sankhyikiya mahatvapurna mem kevala bharata mem dadi ki shiksha ke sangathana bharata ke namune aura lada़kom ke lie majabuta tha
samayojita matalaba antara mem unchai-unra ke antara jo lada़ke eka shikshita dadi ke satha rahate hai aura jo rahate hai ke bicha ka jada-skora 0.64 (95% CI 0.29 se 0.99, pi<0.001)
viyatanama ke namune mem, bachche poshana samudaya mem shikshita mataom ke anupata ke satha, mata pita ki shiksha aura anya mapadanda (unchai 0,81, 95% CI 0,29 ke lie 1.31 samayojana karane ke lie, = 0,002 pi) juda़e the. 
poshana svasthya ke lie eka nyunatama paikeja ke lie eka yojana da. chavaज़ rashtriya poshana sansthana meksiko siti mem ke lie ke dvara karyashala ke lie shuru ki gai thi. 
yaha thika se sthapita hai ki mata ki shiksha vikasashila deshom mem bala poshahara para sakaratmaka prabhava pada़ta hai 
skula mem, lada़kiyam kaushala prapta kara sakate haim jo ki bada mem adhunika svasthya sevaom aura samaja svasthya sandesha ka upayoga karane ke lie upayoga kiya jata hai. 
kama tatola, halanki, asara anya vyaktiyom ki shiksha-mataom 'mitrom, pada़osiyom aura parivara jo sidhe bachche poshana prabhava mai se hai (halanki chaildakaiara) ya paroksha (halanki matri shiksha ke prabhava ka sanshodhana)
hala hi mem sudhara ke bavajuda, purvaskuli bachchom ki lagabhaga adhi eshiya jo kama vjani hai, jo duniya mem sabase uchchatama stara hai. 
bharata mem 62 lakha bachchom kama bhari haim aura viyatanama mem 5 lakha haim. 
donom deshom mem hai chinta ka vishaya raha hai ki arthika pragati bachche poshana mem sudhara mem parilakshita nahim kiya gaya hai. 
shiksha ke samagra rananiti mem eka pramukha tatva ke rupa mem vikasashila duniya mem kuposhana ko kama karane ke lie dekha hai.
asara visheshakara mahila shiksha ke lie majabuta hai. 
bharata mem, shiksha ke bhi ninna stara ki vyakhya karane ke lie shiksha kuposhana aura nrityu ke sthanika klastaringa, aura bachche ke jivita rahane ki sambhavanaem aura svasthya-vyahavara sambandhita badha़ane ke lie dikha gaya hai
matri shiksha ka sakaratmaka prabhava bhi, sarvavyapi nahim hai.
viyatanama mem, eka taja adhyayana ke anusara rashtriya pratinidhi sarvekshana ka upayoga kara do bachche poshana para matri shiksha ka koi mahatvapurna prabhava paya gaya. 
yaha shayada mahilaom para badha़a dabava ke ghara ke bahara kama karane ke lie chaildakaiara aura stanapana prathaom ke lie nakaratmaka parinama ho gaya hai jo shiksha ki kisi bhi sambhavita labha kahim bhari haim
tarka diya gaya hai ki aisa lagata hai ki bachchom ki mataom ke lie adhika se adhika eksapoja़ra samaya 'ke rupa mem pita ki tulana mem mataom ke lie adhika se adhika shiksha ka asara hai kyonki pita ghara ke bahara kama karate haim
agara, halanki, rojagara mataom ko ghara mem chaildakaiara se dura le jata hai, to ghara ke sadasyom ki upasthiti jo mam se behatara shikshita hai chaildakaiara ke samagra gunavatta mem sudhara ke lie eka sambhavita prastava rakha rahe hai
yaha tarka anya adhyayanom dvara samarthita hai ki bada़e bhai bahana aura dada dadi dvara chaildakaiara mem bhumika hai
jana shiksha ki avadharana pahali kalsvala dvara 1980 mem shuru ki gai thi aura Cleland aura jejiboya, jo dakshina eshiya se pradarshita karane ke lie sarvekshana deta istemala se vikasita ki hai
vikasashila deshom mem hai ki donom mataom aura unake chhote bachchom sabase naidanika kuposhana se samvedanashila isa samuha ka gathana koi shaka nahim kiya ja sakata hai.
isa sahitya ki samiksha ki hai ki praudha़ shiksha aura bala poshahara ke adhikansha adhyayana mata pita para hi dhyana kendrita kiya hai
yadyapi hala ke anusandhana ki ina dusarom bachche poshahara ko nirdharita karane mem eka mahatvapurna bhumika nibha sakate haim kuchha sujava dete haim, dusare ghara ke ya samudaya ke sadasyom ke bicha shiksha ke prabhava ki jancha ki hai. 
adhyayana ke yuva ja़indagiyam (2002 mem) viyatanama aura andhra pradesha rajya mem dakshina bharata mem ke pahale daura se para ke anubhagiya deta ka upayoga karata hai. 
yuva ja़indagiyam adhyayana ke pahale daura se deta britena sarvajanika puralekha, adhyayana sakhya esa na 5307 se upalabdha haim. 
prahari sthalom, aura saitom ke bhitara samudayom, ardha prastavita chayana kiya gaya.
gharom yadrichchhika namuna karane ke lie eka samana paddhati ka upayoga karake chayana kiya gaya.
isa adhyayana dijaina para vistrita janakari aura namuna tarikom kahim aura prakashita kara diya gaya hai. 
halanki deta rashtra ke pratinidhi nahim haim, hama bhavishya mem isa data bharata se ki ja rahi karane ke lie aura sandarbhita karenge viyatanama. 
manavashastriya suchakanka bachche poshana ka sanketaka ke rupa mem, prakriyaom ko vishva svasthya sangathana ki sipharisha ke bada istemala kiya gaya.
bachche jine bauna ke rupa mem vargikrita kiya gaya tha (kronika kuposhana ka eka sanketaka) agara unra- unchai jeda-skora 2 se niche hai aura vajana ke antargata (donom purani tatha gambhira kuposhana ka suchaka) yadi unra- bhara ka jeda-skora 2 se niche hai 
shiksha mapa gaya, aupacharika shiksha ke pratyeka ghara ke sadasya ko pura kiya tha kya stara ko prativadi puchhakara.
isa deta ke chara mukhya shreniyom mem darja kie gaya - koi nahim, prathamika (unra 6-11 sala ke lie), madhyamika isi (11-14/15 varsha) ya uchcha (15 varsha ya isake bada).
dadi ke lie, sabhi shiksha shreniyom sanyukta the kyonki dadi ka kevala eka chhota sa sankhya mem uchcha madhyamika ya uchchatara shiksha prapta ki
samudayika stara para matri saksharata mataom ke anupata ke rupa mem mapi gai thi, jo samudaya mem, jaisa ki namuna se anumanita sakshara the (paitrika saksharata para data upalabdha nahim tha).
saksharata puchha dvara mulyankana kiya gaya kya tuma padha़ sakate ho aura eka patra ya akhabara ko asani se samaja sakate ho, mushkila se, ya bilkula nahim, kisi bhi bhasha mem satha? 
eka samudaya ki pasanda stariya saksharata chara, ke bajaya eka shiksha chara, isa parikalpana para adharita hai ki saksharata shiksha ke mahatvapurna parinama hai aura kaha ki shiksha ke akara saksharata tantra ke madhyama se mukhya rupa se phaila hua hai
hama chi prayukta-chukata parikshanom varnanatmaka vishleshana ke lie, chhatra ka ti-parikshana aura epha-anupata ke bicha matabheda ke sankhyikiya mahatva ka mulyankana karane ke lie parikshana, isaka ausata hai do ya do se adhika ka ausata hai, kramashah
bahubhinnarupi pratigamana vishleshana ekadhika saha sansthapakom ke lie eka satha samayojita karane ke lie istemala kiya gaya tha.
saha sansthapaka kuposhana ke nirdharakom ke lie yunisepha ke maॉdala ke adhara para chayana kiya gaya, jo donom sthula aura sukshma stara para jaivika aura samajika karanom shamila hai.
sankhyiki sahabhagitaom eka dami samparka shabda darja karake maॉdala mem akalana kiya gaya
satika pi mana patha mem aura sankhyikiya mahatva prastuta kara rahe haim ki 5% ke stara para, sambhala tha, jo ki 10% stara upayoga sankhyikiya sahabhagitaom, ke alava
pravritti ke lie eka parikshana hai jo mem spashta chara aura isa chara ko chhoda़kara eka maॉdala ke rupa mem nirdishta hai satata kula milakara eka maॉdala ke bicha ki sambhavana anupata pariksha avedana ke dvara kiya gaya.
eka tina stara ke maॉdala ke ankada़om ki saupanika sanrachana ke lie khate mem, prayoga kiya jata hai, 32 33 jisamem stara 1 vyakti bachche ka mapana karane ke lie aura bachche ke ghara, stara 2 samudaya aura stara 3 sthala.
nitishastriya nikasi ke yunaiteda kingadama aura hara nniha jana adhyayana desha rahata hai se anusandhana sansthanom mem bhaga lene se prapta kiya gaya 
sakshatkara se pahale suchita sahamati pratibhagiyom se prapta ki thi.
padha़ai ki vyapakata aura kama 19% aura viyatanama mem 24% aura 28% hai aura bharata mem 43% thi
isa bicha desha ke anurupa matabheda kya kahim aura dikhaya gaya hai, viyetanami bachchom ke satha behatara bharatiya bachchom ki tulana mem ausatana narishada kiya ja raha
vayaskom क़ viyatanama ki tulana mem andhra pradesha mem skuli shiksha ke eka uchcha stara ki hai spashta hai (pi < 0.001), jo bhi pichhale sahitya se mela khati hai.
isa adhyayana se bachche poshana ke bicha eka sakaratmaka sangha ka pradarshana kiya gaya hai - donom ashcharyajanaka aura vajana ke antargata - aura vayaska bachche ke parivara aura samudaya ke bhitara rahane ki shiksha
vishesha rupa se dilachaspa, pichhale sakshya ke abhava di, sakaratmaka sahayoga bachche poshana aura bharata aura samudaya mem dadi ki shiksha stara ke viyatanama mem matri saksharata ke bicha mem dikhaya jata hai
isa adhyayana se pata chalata hai ki bachchom ka poshana kai vayaska gharelu aura samudaya ke sadasyom ki shiksha ke satha juda़a hua hai
vishesha rupa se dilachaspa, pichhale sakshya ke abhava di, ko dhundhane ki eka namuna mem bala poshahara (bharata) sakaratmaka dadi ki shiksha ke satha juda़a hua hai
dusare namune mem (viyatanama), bachche poshana sakaratmaka samudaya ke satha matri saksharata ke stara juda़a hua hai 
stana ka poshana mahatva pilanevali sabhi sarvajanika svasthya karyakramom mem bala ki avashyakata hai, lekina itana khila ke anya rupom ke tarkasangata parichaya bhi hai
isa shodha se do pramukha nitigata nihitartha haim
sabase pahale yaha sarvabhaumika buniyadi shiksha ke maujuda prayasom ka samarthana aura vayaska saksharata karyakrama, kyoki shiksha sakaratmaka rupa se bachche poshana ke satha juda़a hua hai
dusari nihitartha adhika shreshtha hai - vartamana nitiyom bachche poshana mem sudhara lane mem pramukha abhinetaom mam ke alava anya pahachanane se labha
svasthya aura poshana ke lie hastakshepa prajanana ayu ki mahilaom lakshyikarana ka ama tarika parivara ya samudaya mem prabhavashali dusarom ko anadekha kara sakate haim.
aba jaba ki bharata mem mahilaom ki badha़ti sankhya aura viyatanama ke ghara ke bahara, rojagara ko le ja rahe hai, dadi aura bhai bahana jaise anya dekhabhalakartaom ki bhumika teji bachche poshana ka pata lagane mem mahatvapurna ho jane ki sambhavana hai
atah isa nishkarsha poshana niti aura karyakramom ke dhyana ko chauda़a karane ke lie eka kaॉla samarthana mam- bachcha joda़i ke prati se apane parivara aura samaja ke vyapaka sandarbha mem
bharata mem dadi ki shiksha ke satha sakaratmaka sahayoga se mahila shiksha ke svasthya labha ki bhari sabuta diya mumakina hai aura hama chaildakaiara mem dadi 'jimmedariyom ke bare mem kya janate diya
yaha, ki vyavahara svasthya aura prajanana kshamata ka pata lagane mem dadi ki bhumika para prakasha dala hai pichhale adhyayanom ka samarthana karata hai, sakaratmaka ya nakaratmaka, ki kya hai aura jo vyavahara-badala hastakshepom mem dadi ka eka bada़a bhaga lene ke lie
viyatanama mem bala poshana aura samudayika stara ke matri saksharata ke bicha ka sakaratmaka sahayoga aura mata pita ki shiksha aura anya vyaktigata stara ke saha sansthapaka aura jokhima karakom se upara hai ki bachchom ko sakshara samudaya mem rahane se phayada ho sakata hai
pichhale 20 varshom ke arthika parivartana ke bavajuda, viyatanama ke samajika dhanche ko aba bhi mukhya rupa se samajavadi yojanabaddha arthavyavastha se prabhavita raha hai
eka batachita ke anya sadasyom ki saksharata isalie vastavika jnana aura chaildakaiara ke nakali ki sajedari ke madhyama se prabhavita kara sakata hai
loga aksara sevaom aura buniyadi suvidhaom ki manga banane aura adhika rajanitika takata chahate haim aura kama se kama shikshita abadi 
yaha shodha ka samarthana karane ke lie sakshya poshana bachche para praudha़ shiksha ka eka sakaratmaka prabhava ke lie kahate haim
shiksha-prabhava donom unchai aura vajana ke lie unra ke jeda-skora ke lie isi taraha ke the, ve madhyama se adhika lambi avadhi ke lie sanchalita sujava de.
isa adhyayana se yaha bhi pata chalata hai - pichhale kuchha anya adhyayanom ke viparita - usa pita ke 'shiksha bachche poshana ke satha, aura kaha ki isa sangathana ke rupa mem mataom ki shiksha ke rupa mem majabuta tha juda़a hua hai.
yaha mumakina hai ki pita, aksara gharom ke pradhana, nirnaya lene aura tarike ki pratyaksha ya paroksha chaildakaiara prabhavita mem vyavahara karate haim 
bharata aura viyatanama ke bicha nishkarshom mem antara sanskritika mahatva ko badha़aya parivara ya samudaya para rakha mem matabhedom ke bare mem vistara se bataya ja sakata hai
bharata aura viyatanama mem kanyunsa mem dadi chaildakaiara para majabuta prabhava lagu karane ke lie jane jate haim, taॉ isa adhyayana se unaki shiksha ka eka asara dhundhata apratyashita nahim hai
desha ke bicha matabheda bhi vyavasthita karakom ka parinama ho sakata hai, akara aura sanrachana ki do data setom ki jaise
isi prakara, viyatanama mem dadi 'shiksha ka eka prabhava ki kami (bharata ke satha viparita) anshika rupa se kama se kama, dadi ke namune mem shamila ki chhoti sankhya se, samajaya ja sakata hai, 
desha ke bicha matabheda darashata hai, donom hi tarike aura sanskritika karakom isa prakara ke matabhedo ko
isake atirikta, apako pata hona chahiye ki, jaba parinama ki vyakhya kare, ankade anubhagiya the, karaniya matalaba hai ki aura kaha ki age anudairdhya shodha nishkarshom ki pushti karane ke lie avashyaka hoga sabita nahim kiya ja sakata hai.
ina khamiyom ke bavajuda, isa chhote se adhyayana shikshavidom, arthashastri aura poshana visheshajna ke bicha hita ke mahatvapurna muddom ki eka sankhya ko joda़ne mem saphala ho gaya hai 
poshana, stana dudha kahim behatara hone ki sambhavana hai, bachche ko prapta karenge kisi bhi anya bhojana, vikasashila duniya mem vishesha rupa se adhika hai.
kuchha bachchom ko vikasita karane aura chhati para se lamba chhaha mahinom akele dudha ke lie samaroha mem sakshama hote haim, lekina samana rupa se, anya bachchom ko chara mahine pahale anupurana jarurata pada़ sakati hai.
vaham poshana niti isa shodha se ubhara ke lie do nihitartha haim
sabase pahale yaha sarvabhaumika buniyadi shiksha aura praudha़ saksharata karyakrama ke maujuda prayasom ka samarthana karata hai, shiksha ke sakaratmaka rupa mem bachche ke poshana ke satha juda़a hua hai.
dusari nihitartha adhika shreshtha hai - vartamana nitiyom bachche poshahara mem sudhara ko pahachanane aura pramukha abhinetaom ne mam ke alava anya sahita se labha
yaha mumakina hai ki svasthya ya poshana ke lie hastakshepa prajanana ayu ki mahilaom lakshyikarana ke ama drishtikona
chaildakaiara mem hai aura isalie prabhava ke upayogi avasara yada prabhavashali dusarom ko anadekha kara sakate haim.
isa adhyayana se pata chala ki mataom bada़e mem bachche poshahara ko nirdharita karane mem eka bhumika hai, satha hi pita, dadi aura samudaya khela dikhata hai
bicha mem chara se chhaha mahine bachchom kuchha anupuraka khilane ki jarurata karane ke lie, vishesha bachche ke vyaktigata sharirika jaruratom ko mam ka dudha utpadana para nirbhara hai aura shuru ho jaega.
mam ka apane svasthya aura sharirika samaroha mem kaphi sudhara kiya ja sakata hai, kama aura ama ki bhavana ke lie eka bada़i kshamata ke lie achchhi taraha agrani-kiya ja raha
isake alava, karyakrama sambandhi anusandhana jisamem prabhavashali aura ghara mem mam sahita samudaya ke sadasyom ko sasurala valom ko shamila karane ke lie vyavaharika tarikom ka pata lagane ke lie, poshana hastakshepom mem shikshakom aura sthaniya netaom, karyakrama ke prabhava ko badha़ane ke lie avashyaka hai.
yaha bara bara dava kiya jata hai ki hama paryapta poshana svasthya mem mahatvapurna sudhara lane ke lie ke bare mem pata hai
avashyaka arthika sudharom ko lagu karane ke lie hai hamare rajanitika anichchha badha hai
yaha spashta hai ki tisari duniya mem kai garbhavati aura lakatetiga mataom ke ahara ka sevana urja aura kai poshaka tatvom donom ke lie vartamana sipharishom se kaphi kama hai
mata ke ahara mem anupurana isa stara se upara sanskaranom badha़ava nahim, vaham hai majabuta paristhitijanya sakshya halanki yaha hai ki garbhavastha ke daurana ke rupa mem
lakatetiga mataom ka khana sevana jaba taka apavadita kama hai, dudha utpadana ke bare mem 750 mililitara para / nri. pika karane ke lie lagata hai
garbhavastha ke lie, eka mukhya vichara hai ahara shiksha kshetriya khadya padarthom ke vibhinna mishrana ke lie garbhavati mam ko salaha di jati hai
shikshana satra postarom aura parchom ke sahayoga se kiya jaega.
satika kshetra ke adhara ki jarurata ke anusara, yaha ahara shiksha chikitsatmaka lauha aura vitamina ki apurti dvara samarthita ho jaega
vishesha dhyana bhi shikshana svachchhata ko usake antima matritva ke lie mahila ko taiyara karane ke lie diya jaega
garbhavastha ke anta mem navajata ko turanta stanapana ke mahatva para jora diya jaega, aura bachche ke janma ki shuruata mem asapasa ke paryapta saphai para
jaba bachche ko do mahine ho jaya taba mam ko puraka ahara taiyara karane ke lie sikhaya jaega.
phira, isa kshetra para nirbhara, vitamina ki sthiti mem bachche ko prabandha dvara ki raksha ki jaegi e, si ya di, ke rupa mem upayukta.
chara aura chhaha mahine puraka ahara ke pravadhana ke bicha ki sipharisha ki jaegi, vishesha rupa se jinaka vajana kama hai
adhyapana bhi pradana ki jaega ki kesai bachche ko parivara ahara mem ekikrita kiya jaya
isa kshetra aura samajika arthika stara para nirbhara karate hue, pura dudha chhuda़ane ka vayu chhaha aura chaudaha mahine ke bicha ke unra mem hota hai
lagabhaga chhaha mahine, chikitsatmaka loha bhi avashyakatanusara pradana ki jaegi.
jaba bachcha eka varsha ke unra taka pahunchata hai, usaka parikshana hoga aura parajivi pravaha ke lie ilaja kiya jaaiga
atirikta ahara shiksha pradana ki jaegi, kshetriya upalabdha bhojana para vishesha dhyana dete huai.
mam aura bachche ke lie yaha poshana paikeja rediyo aura telivijana ke madhyama se rashtriya shaikshika upayom dvara samarthita ho jaega, aura eka kama lagata vale khadya padarthom ki pranali ki bikri dvara vitarita ho jaaiga
sahayaka shiksha bhi skula aura anya sarakari sangathanom mem hi kiya jaega
jaba vikasashila deshom mem simita sansadhanom hai aura vaham unake lie mangom se pratispardha hai, yaha sapha़ hai ki vaham saphalata atyadhika skoringa ke satha prathamikataem nirdharita karane ke lie dabava hoga
kuchha deshom ke samuhom ke satha jo ve lakshya taka pahuncha chuke haim prabhavashilata ka parikshana karake karyakrama ki vyavaharyata ka mulyankana kiya hai
shrilanka mem desha vyapi karyakrama, udaharana ke lie, 75 pratishata khatare mem mataom taka pahuncha gaya, lekina dusare aura tisare digri protina urja kuposhana ke satha bachchom ke satha saphalata dara kevala 50 pratishata thi
yaha nishkarsha nikala ja sakata hai, ki, mataom eka behatara isthiti mem thi
phira bhi, nirnaya kevala labharthiyom ki sankhya para nahim banaya jana chahie; kavareja se eka karyakrama ki prabhavashilata ka ekamatra akalana nahim kiya ja sakata hai, bhale hi yaha ankada़om mem achchhi kitaba dikhata hai.
eka poshana hastakshepa karyakrama poshana sambandhi sthiti mem sudhara ke sandarbha mem, mulyankana kiya jana chahie, lekina durbhagya se saphalata aise manadanda ka upayoga kara pahachana karane ke lie kathina hai
shrilanka mem, matri anupurana ke eka dashaka ke bada, kama janma ki ghatanaom ka anumana-vajana aparivartita hi rahenge.
isi taraha, vartamana sarvekshana purva-skula vale bachche mem kuposhana, 1975/76 mem kie sarvekshana mem parivartana nahim dikhate 
saphalata ki kami eka ama khoja hai; beatona aura ghassemi ne hala hi mem kai antararashtriya karyakramom ki samiksha ki hai, aura yaha pradarshita karana asambhava tha ki unamem se kisi mem eka mahatvapurna sudhara hua hai.
kyom yaha nirdharita karane ke lie itana asana nahim hai, lekina kama se kama kuchha dosha rasadiya asaphalataom para bhi jana chahie
yaha shrilanka mem mana jata hai, udaharana ke lie, ki samudaya ki bhagidari eka sarakara ki saphalata karyakrama prayojita karane ke lie mahatvapurna hai
samanya shiksha ko bhi eka mahatvapurna karaka hona dikhaya gaya hai
soysa svasthya sevaom ke apane upayoga ke lie shrilanka aura mahilaom ki shiksha ke stara se sambandhita hai aura isamem eka sakaratmaka sambandha paya gaya hai
soysa ne dava kiya hai ki shrilanka mem janma dara mem kami ke lie jimmedara eka mukhya karaka hai
soysa ne kaha ki poshahara sandeshom ke madhyama se gramina kshetrom mem nahim milane ka eka karana raha ki shiksha ke paramparagata tarikom mem sudhara lane ke aparyapta prayasa ki vajaha se
khadya sahayata shrilanka mem pahachana ka eka ama risava antara-parivarika aura shayada hi vyavaharika drishtikona pure parivara ikaiyom ki ora hai.
yaha eka bhojana ki lagata sahayata karyakrama mem vriddhi karega, lekina kisi bhi sanskriti mem koi aurata apane svayam ke upayoga ke lie bhojana ko pratibandhita nahim karegi; vaha hamesha apane parivara ke satha saja karengi
isa prakara, agara pure parivara ko sahayata prapta ho to yaha eka behatara saphalata dara ko pa sakate haim, na sirpha una naidanika khatare mem.
svasthya sudhara ke khilapha kama karane sabhi jatila, arthika, samajika aura rajanitika karanom ke satha, yaha ashcharyajanaka nahim hai ki vaham eka uchcha spashta viphalata dara hai
vaham, phira bhi, sakaratmaka upalabdhiyam hai jo shayada hi khate mem jate haim
eka vishesha poshana karyakrama mam aura usake bachche para nirdesha ka parichaya eka vishishta svasthya ke mudde para eka taraha se hai ki majaduri ke rupa mem aura adhika badha़ jati hai aise apratyaksha tarikom se kabhi nahim ho sakata sarakarom ka dhyana kendrita mem saphala ho.
yaha rajanitika aura samajika rupa se rashtriya niti ko akara dene mem eka mahatvapurna karaka hai
sapha़ anusandhana sakshya kai deshom se se ana chahie, pahale anupuraka khila ekamushta ki ninda ki hai jo puri taraha se nishprabhavi hai aura isalie kama prathamikata ka hai 
yaha alaga vikasashila deshom ke bicha eka avedana prakriti ke sahayogatmaka anusandhana ki avashyakata para jora dalati hai
apake bachche ka janma apake jivana ka sabase romanchaka ghatanaom mem se eka hai, eka pala jo hamesha yada rakhage
ina prasuti purva dekhabhala satra ka uddeshya shrama aura vitarana ki prakriya ko asana karane ke lie hai, garbhavastha ke bare mem apake prashnom ke uttara pradana karane ke lie, garbhavastha ke daurana samasyaom ko kama karane ke lie
shrama aura vitarana, apane bachche ki dekhabhala aura akara mem phira se vapasa lane mem madada karane ke lie
yaha 4 satrom ki eka shrrinkhala hai, jo vyaktigata haim aura eka poshana aura stanom ki jancha ke satha pailvika manjila ko majabuta banane aura erobika vyayama ke satha 20 haphtom mem shuru hoti hai.
yaha agale satra se 26-28 saptaha mem eka poshana ki jancha ke satha abhyasa aura erobika gatividhiyom ko khinchane ke satha ata hai.
tisara satra amataura para jaham maim eka shrama aura prasava ke eka visidi dikhata hu aura isa satra ke lie upasthiti pati ke satha isa puri prakriya ke madhyama se rogi avagata karata hu.
chauthe satra prasava ke bada kiya jata hai, jo prasava ke bada abhyasa ke satha aura akara mem vapasa hota hai
vikasashila deshom mem bachchom mem, kuposhana bimari aura mauta mem yogadana ka eka mahatvapurna karaka hai.
bachapana ke daurana kuposhana bhi vikasa kshamata aura rugnata aura jivana ke bada ke varshom mem nrityu dara ke jokhima ko prabhavita kara sakata hai.
bachchom ke kuposhana amataura aparyapta ya anuchita bhojana ka sevana ke sanyojana ke karana hota hai, gaistrointestainala parajivi aura anya bachapana rogom bimari ke daurana aura anuchita paravaha hai.
bachchom ke kuposhana lambe samaya se bharata mem eka gambhira samasya ke rupa mem svikara kiya gaya hai, lekina rashtriya stara aura kuposhana ke karanom ke stara para ankade durlabha hai
1992 – 93 mem, rashtriya parivara svasthya sarvekshana (rapassa) eka rashtriya pratinidhi namune se unchai aura chara varsha se kama unra ke bachchom ke vajana para ekatra manavashastriya deta ikatta kiya
yaha sarvekshana bharata mem bachche ke kuposhana ke nirdharakom aura stara ka adhyayana karane ke lie eka advitiya avasara pradana karata hai
isa enaephaechaesa buletina mem yaha mudda bachche kuposhana ke stara ka anumana hai aura mam ki shiksha aura bachchom ke poshana ki sthiti ke bare mem anya janasankhyikiya aura samajika karakom ke prabhava ki jancha karata hai.
parinama se sanketa milata hai ki chara sala ki unra ke tahata sabhi bachchom ki adhe se jyada kuposhita haim
bachche jinaki mataraai kama ya shikshata nahim haim poshana sambandhi sthiti ke lie adhika shikshita mataom ke bachchom se kama hai 
mahilaom ki shiksha aura saksharata karyakramom ke bachchom ke poshana ki sthiti mem sudhara karane mem eka mahatvapurna bhumika nibha sakata hai.
apraila 1992 aura sitambara 1993 ke bicha, enaephaechaesa ne unra chara ke tahata sabhi bachchom para 88562 parivarom ki eka rashtriya pratinidhi namune mem se 25 rajyom mem janakari ekatra ki (dilli sahita, jo ki rajya ke sarvekshana ke bada bana)
ina 20rajyom mem, sarvekshana mem bachchom ka vajana aura unchai ke bare mem janakari ekatra ki
andhra pradesha, himachala pradesha, madhya pradesha, tamilanadu aura pashchima bangala mem, kyonki unchai-mapane bordom upalabdha nahim the kevala bachchom ka vajana prapta kiya
20 rajyom ke lie jaham donom unchai aura vajana napa gaya, enaephaechaesa bachchom ke poshana ki sthiti ke tina suchakanka pradana karata hai : unchai ke lie unra, vajana ke lie unchai aura vajana ke lie unra.
pratyeka suchakanka mem eka bachche ke poshana ki sthiti ke bare mem kuchha alaga hi janakari pradana karata hai
unchai ke lie unra upayom raikhika vriddhi mandata, mukhya kronika (dirghakalika) kuposhana darshati hai 
vajana ke lie unchai ke sambandha mem unchai upaya sharira dravyamana, mukhyatah tivra (alpakalika) kuposhana darshati hai.
vajana ke lie unra donom chirakalika aura tivra kuposhana ko darshata hai.
ye tina suchakanka manaka vahana ke sandarbha mem eka antararashtriya sandarbha abadi ke lie madhya se vyakta kara rahe haim jo vishva svasthya sangathana (dablyuechao) dvara nirdishta 
bachche jinaki unchai ke lie unra ke sandarbha manjala niche se adhika do manaka vahana pha़alsa ke rupa mem avikasita paribhashita kara rahe haim
bachche jo kisi bhi ina suchakankom ke sandarbha manjala niche se adhika tina manaka vahana gira gambhira, vyartha ya kama avikasita rupa mem paribhashita kara rahe haim
kai alaga alaga visheshataom eka bachche ke poshana ki sthiti ko prabhavita kara sakate haim, kisi eka chara ka asara bhramaka ho sakata hai jaise mam ki shiksha 
isa karana se, mam ki shiksha ke svatantra prabhava rasadiya vapasi aura ekadhika vargikarana vishleshana ka upayoga karane ka anumana ke prabhavom ko niyantrita karane ke lie 12 anya - sambhavita bhrama-chara hai.
isa vishleshana mem, chara lagatara apane matalaba mulyom mem unhem setinga ke dvara ayojita karate haim
bharata mem bahuta se chhote bachchom kuposhita haim
bachchom ke kuposhana kaphi shahari kshetrom ki tulana mem gramina kshetrom mem adhika hai.
dablyuechao manakom ke dvara paribhashita, kai gambhira kuposhana pida़ita hai
29 pratishata gambhira avaruddha hai, 3 pratishata gambhira rupa se barbada hai, aura 22 pratishata gambhira rupa se vajana ke antargata haim
bachchom ki apekshakrita kama anupata barbada ki tivra kuposhana se kama bharata mem kuposhana se adhika purana ama ka sanketa hai
tivra kuposhana aba bhi antararashtriya manakom se adhika hai.
bharata mem yuva bachchom mem barbada ki vyapakata ko antararashtriya sandarbha abadi mem lagabhaga 8 bara stara kya hai, aura gambhira barbada ki vyapakata ke bare mem 25 bara antararashtriya stara ki hai.
bachchom ke poshana ki sthiti ka vyapaka rupa se rajya se bhinna hota hai.
kuposhana ke uchchatama aura ninnatama stara ke satha bihara aura kerala rajya hai
kerala mem bhi, jaham bharata mem sabase kama shishu aura bala nrityu dara hai, eka se adhika-javana bachchom ke chauthe avaruddha hai, aura isi taraha ke anupata hai
kuposhana ke apekshakrita ninna stara ke satha anya rajyom gova, dilli, manipura, mijorama, aura nagalainda haim
bachche ki apekshakrita uchcha stara ke satha kuposhana ke anya rajya madhya pradesha aura uttara pradesha hai 
jinaki bachcho ki mata kuchha shikshata haim lekina madhya vidyalaya ko pura nahim kiya hai, unaki bahuta kama avaruddha ho jane ki sambhavana hai 
bachche jinaki mataom madhya vidyalaya ya uchcha shiksha puri kara li hai ke kama kuposhana grasta hone ki sambhavana hai.
mata ki shiksha ko bhi 12 anya janasankhyikiya aura samajika chara ke sambhavita prabhavom ka bhrama ke lie niyantrita karane ke bada eka bachche ka poshana ki sthiti ke bare mem eka majabuta svatantra prabhava hai
tajjuba ki bata hai, bharata mem lada़kiyom ke khilapha bhedabhava ki vyapaka sabuta diya, anupata avikasita aura kama karane ke lie samana lada़kom aura lada़kiyom ke lie haim.
anupata barbada kuchha lada़kom ke lie adhika hai
mam ki shiksha ke lie niyantrana aura dusari janasankhya aura samajika chara kisi bachche kuposhana ke tina suchakankom mem laingika antara para vastutah koi prabhava nahim hai
rajya ke alaga vishleshana stara ke ankada़e darshate haim ki bihara mem bhi, jaham mahilaom ki sthiti vivadaspada rupa se bharata mem sabase kama hai, kuposhana stara lada़ke aura lada़kiyom ke lie samana haim.
bharata mem kai anya adhyayanom aura vyapaka linga bhedabhava ke satha anya vikasashila deshom mem bhi bachche kuposhana mem eka laingika antara ka pata lagane mem asaphala raha hai.
anya bhavishyavakta chara mem vishleshana mem shamila, bachche ki unra, bachche ke janma ka krama, aura gharelu arthika sthiti sabhi poshana sthiti ke bare mem svatantra prabhava hai.
chhaha mahine se kama shishuom ke kuposhita hone ki sambhavana hai bada़e bachchom se kama hai, shayada kyonki unhem stana pana karaya jata hai
tina ya adhika bada़e bhai bahana ke satha bachche ki tulana mem chhote parivarom ke bachche ki adhika kuposhana se grasta hone ki sambhavana hoti hai, shayada, kyonki parivara ke satha khane ke lie pratiyogita badha़ jati hai
parivara ke akara tivra kuposhana para eka mahatvapurna prabhava nahim hai, halanki, barbada ke sanketa diye hai.
parivara ke akara ki taraha, ghara ki arthika sthiti jirna bachche kuposhana para eka majabuta prabhava dalati hai, vajana ke antargata deta ke adhara para kara dikhaya, barbada sanketa ne diya
dusare chara mem se koi bhi vishleshana maja़buta, sangata, bachche ke kuposhana para tina upayom dvara svatantra prabhava hai.
shishu aura bala nrityu dara ina rajyom mem bhi rashtriya ausata se adhika haim
dablyuechao manakom dvara vikasita ke adhara para, bachchom ki 52 pratishata unra chara ke antargata, avaruddha kara rahe haim, 17 pratishata barbada hai, aura 54 pratishata vajana ke antargata hai
barbada ka nichala stara, vajana ke antargata ki tulana mem, ki purane (dirghavadhi) kuposhana aura adhika tivra (gambhira para alpavadhi) kuposhana se adhika ka sanketa hai. 
bachche mem poshana sudhara lane ke prayasa ke lie rajyom para dhyana kendrita kara sakate haim jaise bihara, madhya pradesha aura uttara pradesha jaham samasya vishesha rupa se gambhira hai.
bachche mem kuposhana ke uchchatama starom se anya rajyom mem bala poshahara mem sudhara karane ke lie kadama mem badha nahim ani chahie 
kerala mem bhi, rajya jaham bala kuposhana ke ninnatama stara para hai, eka chauthai se adhika sabhi bachchom mem se unra chara ke kuposhana se pida़ita haim.
yaham vishleshana kiye gaye sabhi janasankhyikiya aura samajika chara mem, mam ki shiksha bachche ke kuposhana para sabase sashakta svatantra prabhava dalati hai
isa vishleshana ke parinama mahilaom ki shiksha aura bharata mem saksharata karyakrama ko majabuta banane ke paksha mem kai anya bahasa karane ke lie jora dalate hai
halanki kuposhana anapadha़ mahilaom ke bachchom ke bicha sabase adhika hai, jinaki mataom ne madhya vidyalaya ya uchcha shiksha puri kara li hai unake bachche 40% se adhika jirna kuposhana se grasta hai
yaha sujava dete haim ki mahilaom ki shiksha behatara banane ke prayasom ko aura vishishta karyakramom ke satha joda़ne aura stanapana mem sudhara lane ki jarurata hai
vishleshana dikhata hai ki chhote parivarom mem chhote bachche ke kronika kuposhana se pida़ita hone ki sambhavana kama hai tina ya adhika bada़e bhai bahana ke satha bachchom banisapata
yaha khoja parivara ke akara ko simita karane ke lie karyakrama ki yojana ke lie eka majabuta auchitya pradana karati hai 
ilektraॉnika midiya ka mata ke dravyamana para bachche ke kuposhana ke stara para koi sankhyikiya mahatvapurna prabhava nahi pada़ta hai.
isaka matalaba yaha hai ki bachchom ke poshana ki avashyakataom ko bharata mem rediyo aura telivijana karyakramom para aura adhika sandeshom - ya adhika prabhavi - sandeshom ko shamila karane ki jarurata hai.
ina nishkarshom ki satarkata ki eka digri ke satha vyakhya ki jani chahie kyonki isa sarvekshana mem kuchha unra ke galata riportinga ke sabuta hai
unra chara se niche kuchha bachchom ko vastava mem se bade ke rupa mem riporta kiya gaya
isaka matalaba hai ki shoshana aura vajana ke niche ke anumana, jo unchai ke lie unra aura vajana ke lie unra para adharita haim, kuposhana ki hada ko atiranjanaka kara sakati hai 
anya parivarom ke bajaya anapadha mam ya kama samajika sthiti ke satha ke parivarom chhote bachchom ki unra ki galata suchana dene ki sambhavana jyada hai 
jivana ke pahale varsha aura adhika teji se vriddhi aura vikasa jivana ke kisi bhi anya ki tulana mem samaya ka eka samaya hota hai
amataura se bachche ka janma ke pahale 4 mahine ke andara vajana dabala ho jata hai aura pahale janmadina taka janma vajana se tripala ho jata hai
isa ashcharyajanaka vriddhi ke lie, shishu ko kailori aura avashyaka poshaka tatvom ki eka paryapta sevana ki avashyakata hai
akele achchha poshana samanya vikasa ki garanti nahim hoga, lekina uchita khilane ke bina pyara bhara vatavarna adhura hai 
apake bachche ko apaki taraha poshaka tatvom ki jarurata hai: protina, karbohaidreta, vasa, jala, vitamina aura khanija. 
apake bachche se svastha vriddhi aura vikasa ke lie 40 se adhika vibhinna poshaka tatva avashyaka haim
purna avadhi bachche amataura poshaka tatvom ke paryapta bhandara ke satha janma lete haim, vishesha rupa se jala, pahale kuchha dinom ke lie
dusare ya tisare dina taka, apake bachche ko, kailori, pani aura poshaka tatvom ki jarurata hogi
halanki stanapana adhikansha shishuom ke lie sabase achchha hai, yaha sabhi parivarom ke lie sambhava nahim ho sakata hai
chahe apa stana-pha़ida ko chune ya vanijyika shishu pharmula ka prayoga karem apake bachche ki poshana ki jarurata ko surakshita aura paryapta rupa se apurti kiya jaega 
nai mataai jo kama karane ke lie vapasa jane chaheti hai aksara stanapana pharmula phidiga ko aksara sanyojana karane mem sakshama hai
narsinga mam ko amataura para prati dina atirikta 500 kilo kailori ki jarurata hai, protina ke adhika 20 grama aura kailshiyama ki adhika 400 miligrama ke satha
yaha apurti eka gilasa dudha joda़kara ki ja sakati hai, eka thoda़a mansa ke sevarata bada de, aura eka anda ya roti ka eka tukade ke satha
khuba pani pine se avashyaka tarala pradana karane mem madada milegi.
eka navajata shishu ke peta ki kshamata ¼ kapa se kama hai
12 mahine mem bachche ka peta lagabhaga 1 kapa ayojita karane mem sakshama honge
kyonki bachche eka samaya mem bahuta kama kha sakate haim, ve hara do ya tina ghante mem khate hai
pahale kuchha saptahom ke daurana bachche aniyamita samaya para bhukhe ho sakate hai
jaise hi ve vikasata hote hai, ve aura adhika niyamita ho jate hai aura aba khilane ke bicha aura sakshama ho jate hai
khilane para manga aura adhika lokapriya ho gayi hai halanki kuchha mata pita aba bhi niyamita anusuchiyam sthapita karane ko prathamikata jo unake aura unake bachchom ke lie suvidhajanaka ho
pratyeka parivara ko chunana hoga ki sabase achchha poshana unake bachchom ke lie kya hai.
bada़e bachchom aura vayaskom jaise, bachchom ko dudha ke satha hi pani ki jarurata hai
ye dainika drava mutra ke gathana ke lie sharira se kachare ko hatane mem madada karane ke lie avashyaka haim
bachchom ko sharira ke vajana ke paunda prati tarala padartha ko 8 kilograma / 18 paunda karane ke 1 / 3 kapa ki jarurata hai.
bhari vajana para, tarala padartha ki avashyakata kama hai. udaharana ke lie, 5½ kilo / 12 paunda ke bachche ko eka dina mem 4 kapa (1 litara / 1 chauthai gelana) drava ki jarurata
drava se adhikansha stana dudha ya pharmula se ana chahie.
kai bachchom ko, khasakara garmiyom ke mausama mem atirikta pani chahie 
yadi apaka bachcha jaldi khane ke bada rota hai aura apa koi karana nahim khoja sakate hai, eka sapha botala mem thoda़a pani khilane ki koshisha kariye (lekina sunishchita karem ki apane pani ubala aura phira thanda kiya). 
savadhana raho, itana pani mata do ki bachche ko paryapta dudha lane mem pareshani ho
mata ke dudha aura vanijyika pha़armulom mem vitamina aura khanijom ki paryapta matra shamila hai jo samanya shishuom ke lie hai.
yadyapi mam ka dudha aura pharmula mem vitamina si ki bahuta chhoti matra hoti haim, yaha eka bachche ki jaruratom ko pura karane ke lie paryapta hai.
bachche ko atirikta vitamina aura khanija dena shayada samanya paristhitiyom mem anavashyaka hai aura yadi atyadhika matra mem diya jata hai to khataranaka ho sakata hai
stana ke dudha mem amataura para vitaminom aura khanijom ke rupa me eka shishu dvara istemala kiya jane ki vishesha rupa se achchhi taraha se avashoshita hai
udaharana ke lie, vaham stana ke dudha mem kama lauha hai, lekina yaha eka bahuta hi upayogi rupa mem maujuda hai
isake alava, eka purna avadhi shishu eka achchhi poshita mam se lauha bhandara ke satha paida hota hai lagabhaga chhaha mahina taka rahane ke liye kaphi hai
svastha dantom ke vikasa ke lie phlyuraida mahatvapurna ho jata hai, lekina chhota anusandhana kiya ja chuka hai ki jivana ke pahale chhaha mahinom mem phlyuraida kitana mahatvapurna hai
pharmula khilaya jane vale bachchom ko pani se paryapta phlyuraida milega (agara yaha phlyuraida ka ilaja ho) ki pharmule ke satha milaya jata hai.
kshetrom mem, jaham pani phloraida ka ilaja nahim hai, amerikana akadami ke bala chikitsa prati dina 0.25 miligrama ki eka phlyuraida puraka ki anushansa shishuom mem donom ke lie stana aura botala dvara ke liye hai
anupurana janma ke bada do saptaha mem shuru karana chahie.
peta ka darda eka paristhiti hai jo ki kuchha navajata shishuom ko prabhavita karati hai
peta ka darda eka vikara hai jaham shishu rona aura chida़chida़apana dikhata hai jisake lie koi spashta karana nahim hai.
kai bachche khilane ke bada rone gamagina charanom ke madhyama se jate hai
peta ke darda ke kuchha parinama ho sakate hai jo bachche ne khaya hai ya kuchha aura mam ne khaya hai jo ki usake dudha mem prakata hota hai.
yadyapi jyadatara bachche udhama machate hai ya samaya para koliki prakata hotala hai, yaha dekhane layaka ho sakata hai ki agara khane ke kuchha prakara yaha paida hota hai. 
kuchha aramadayaka takanika ka abhyasa karem, jaise apake bachche kamala ya bata karem ya use dhire se ga ke sunaai 
bachchom paristhitiyaॉ ki kisma mem achchhi taraha se vikasita hote hai
yaha mata pita ke lie sabase mahatvapurna hai ki nirnaya karana hai ki parivara ki sthiti sahi ho
jo bhi khilane ka tarika chune, bachche ko pyara ka eka mahaula mem khilaya jana chahie.
khilane bara ke daurana bachcha pasa hona chahie, chhua jana chahie, hilana chahie aura bata karani chahie 
isa ke bina nivida bhautika samparka, aksara bachchom ko vikasita karane ke lie viphala rahata hai
yadyapi, dosta aura rishtedara apane anubhavom ko batate hai, isa ka phaisale ke kaise aura jaba apake bachche ko khilana hai nirbhara karega apake bachche ki aura apaki khuda ki jarurata hai.
yaham taka ki chhote bachchom bhi samaja sakate haim mahasusa kara sakate hai ki abhibhavaka kaba arama ke bajaya pareshana hai 
bebi ka mukhya bhojana stana dudha ya pharmula jari rahega
khane ke satha prarambhika anubhavom ka bada mem khane ki adatom para prabhava pada़ sakata hai
sikhana svikara karane ki svadom aura dijaina ki kisma mahatvapurna hai.
anusandhana se pata chalata hai ki mam ka khaya bhojana ka svada usake dudha mem dikhai dete haim.
bachche ko shayada shighra mam ke dudha mem prakata hone vale svada ka ananda lena hoga jo bhojana mem prakata hota hai
eka adhyayana mem sujava diya gaya hai ki stana pana vale shishu bada ke jivana mem nae svadom ki koshisha karege
phira bhi, mata pita ko savadhana rahana chahie ki unake bachche para svada variyataom ko lagu karane ke lie nahim kare
kai dashakom se mata pita ko thosa ahara jaldi lagu karane ke lie kaha gaya tha
kuchha mamalom mem bachchom ko janma ke bada tina saptaha mem anaja dena jaldi shuru kiya gaya
vartamana sipharisha hai ki bachche ko 4 se 6 mahine ka intajara karana hai
tina mahine ki unra taka, shishu ki jibha ke satha javaka pusha karane ke lie prakritika palata hai 
yaha chusane mem upayogi hai lekina bachche ko thosa ahara hilane se rokata hai munha ke age se pichhe nigalane ke lie
lagabhaga 4 se 6 mahinom ki unra mem, bachchom sira ko niyantrana karate hai aura adhika asani se sidhe baitha sakate hai
ina donom vikasa ke mila ke patthara se prakata hota hai ki bachcha chammacha se khane ke lie taiyara hai
4 se 6 mahine ki unra se pahale bachche ko thosa khadya padarthom mem poshaka tatvom jarurata nahim hai
isa samaya taka bachcha apane janma ke samaya vajana doguna kara lega aura bhukha rahane lagega
kuchha bala chakitsaka aura poshana visheshajna ko vishvasa hai ki thosa khadya padarthom ke shighra parichaya se bachche ko peta bhara khane ke lie sikhata hai
shishuom se agraha karana mata pita ke lie asana hai adhika khane ke lie vastava ki tulana mem chahate haim; adhikansha bachchom ko nahim pata kaise virodha kare
apane najuka bachche ko eka samaya mem kevala eka naya khana de
eka aura joda़ne se pahale bachchi ko istemala karane ke liai kuchha dina de
kuchha bala chakitsaka ki sipharisha hai ki lauha-gadha़vale shishu chavala anaja stana dudha ya pharmula eka pahala thosa ahara ke rupa mem de
thosa ahara apake bachche ko lohe ka eka achchha srota deta hai, satha hi karbohaidreta, protina, aura vasa ke bicha kailori ka eka achchha vitarana hai 
anya anajom ke bajaya chavala se elarji pratikriyaom paida hone ki kama sambhavana hai 
jaba apa pahali bara thosa khadya padarthom ka parichaya karate hai to apake bachche ko ashvasta karane ki avashyakata ho sakati hai
apake bachche ko pakade use batane ke lie ki yaha naya anubhava thika hai
7 mahinom ke asapasa mem sabjiyom aura phalom ke bada anaja ka anusarana karem.
yadi apake bachche mem chabane ke kuchha lakshana dikhane shuru hotala hai, to apa use dena chahenge kuchha maishda sabjiyom, ya mote anaja taki chabane ke kaushala para kama karane ke lie
8 se 9 mahine ke bachche ke eka ya do danta hoge aura shayada vaha dheledara khadya padartha sambhala sakega 
yadi a parivara ke lie taiyara ahara shishu ko dena shuru nahim kiya hai, to aba shuru karem
asana jao. asani se maisha hone vale khadya padartha se shuru karo jaise pakaya hua alu ya gajara, kela, ya dibba banda phala.
agara kisi bhi poshaka tatva ki kama apurti ho, to yaha lauha hai
kai bala chakitsaka isa samasya ko pahachanate hai aura lauha puraka ki salaha dete haim.
lauha-gadha़vale anaja khasakara lohe ke sutrom achchhe haim.
shishuom mem svada ki majabuta bhavana hoti hai aura unhem svada vardhaka vayaskom dvara sahayayukta ki jarurata nahim hai
eka bara jaba apaka bachcha chijom ko sambhala lene ke lie sakshama hota hai, to apa ungali se khane ki peshakasha sakate haim jo samanvaya vikasita karane mem madada karega
isa stara para, eka ungali vala khana kuchha narama ho sakta hai jo kaphi dera taka bachche ke lie ise thali se munha taka dharana kara sake
jaba bachche ke danta ana shuru hote haim taba raska eka achchha ungali khadya hai
thanda phala mishrana (shuddha kainda ya narama phala) danta ke ana ke samaya bachche ke masuda़om ko taja़agi dete hai
chhaha mahine ki unra taka, bachchom ki jarurata ya to mam ka dudha ya vanijyika pharmula poshaka tatvom ke mukhya srota ke rupa mem hota hai halanki bachchom ko anya khadya padartha khana shuru karana chahiye
navajata shishu aura mata pita ko jo santushta haim 12 mahine taka kisi eka mam ke dudha ke satha ya vanijyika pharmula ka upayoga karana jari rakha sakate haim
amerikana akadami ke bala chikitsa ke dvara stana ke dudha ya vanijyika pharmula ka upayoga karane ki 12 mahine taka ki sipharisha ki jati hai 
gaya ka dudha bachche ke lie 12 mahinom taka nahim diya jana chahie.
shayada 10 mahine taka apake bachche khaye aura ananda le anaja ki kisma, sabjiyam aura phala ka.
halanki mansa protina ka achchha srota hai, jyadatara protina bachche ko khuba sare dudha se milata hai.
mansa ka sabase mahatvapurna yogadana shishu ke ahara mem lauha ka hai
lagabhaga , 10 se 12 mahinom mem, ande ahara mem joda़a ja sakata hai
yadyapi lagabhaga 10 mahine mem, ande ka yaॉrka ahara mem joda़a ja sakata hai, pahale sala ke anta taka apake bachche ko ande ka sapheda padratha ya pura anda nahim dena chahie
yaha kyonki ande ka sapheda agara shuru mem jaldi diya to aksara elarji pratikriyaom ka karana ho sakata hai 
panira aura thoda़i matra mem dahi ka bhojana karane ke lie aba joda़a ja sakata hai.
shishuom samanyatah pahale varsha ke anta taka 
bachcha kapa se adhika dudha pina sikhata hai, taba botalom ya stanapana dhire dhire banda kiya ja sakata hai. 
eka bachcha jo dudha ki bhari matra pina jari rakhata hai kaphi thosa khadya padartha apani badha़ti poshana sambandhi jaruratom ko pura karane ke lie.
1 varsha ki unra se bachche ko khadya padarthom ki eka kisma aura eka dina mem kevala 2 kapa dudha khana chahie 
bachche ka khana garma karane ke lie maikroveva ovana ka upayoga karane ka dhyana rakha jana chahie 
maikroveva khana garma-spaॉta bana kara garma kara sakate haim
eka chammacha khane ki matra thandi ho sakati hai, para agale thoda़i matra bachche ka munha jala sakata hai
shishu kamare ke tapamana ko aura thanda bhojana svikara kara sakate hai, to vastava mem garama karana itana avashyaka nahim hai
adhikansha svasthya aura bachche ki dekhabhala karane vale peshevara bachche khana garma karane ke lie maikroveva ovana ka prayoga karane ke khilapha ki sipharisha karate hai
yadi apa eka maikroveva ovana mem garma karate hai, atyadhika satarkata ka upayoga karem, aura khana achchhi taraha se hilaye sunishchita karane ke lie ki khana sahi tapamana para hai 
12 mahine ki ayu ke niche ke shishuom ke lie hani anushansita nahim hai kyonki isamem botulija़ma saporsa ho sakate hai 
bachchom aura vayaskom ka pachana tantra ina hanikaraka saporsa ko nashta kara sakate haim lekina eka bachche ka nahim
saporsa peta mem sakriya raha sakata hai aura unake ghataka visha ka utpadana kara sakata hai
yaham taka ki pakana shahada mem saporsa ko nashta nahim kara sakate 
sare shahada botulija़ma saporsa liye nahim hote lekina kyonki shishu ko shahada ki jarurata nahim hai jokhima uthane ka koi karana nahim hai
bitasa aura palaka svabhavika rupa se naitreta ghatanevala ke uchcha sandrata hai jo bachche ki himoglobina ki kshamata ऑksijana parivahana karana kama kara sakati haim
bitasa aura palaka sanchalana mem prayoga kiya jana chahie ya bilkula nahim jaba taka bachcha usake pahale janmadina taka pahunchata hai. 
kachche ande aura kachche dudha bachchom ke lie upayukta nahim haim.
kachche ande aura kachche dudha mem sankramana ka srota ho sakata hai jo shishuom ke lie khataranaka ho sakata hai 
bachchom ke lie jo anya khadya padarthom nahim hai, unamem desarta, karboneteda shitala peya, kaiphina-yukta shitala peya, kaindi aura shamila haim.
agara ve paushtika khadya aura peya padarthom ki jagaha le, to ve hanikaraka ho sakata hai.
vidyuta shitala peya chini ya nutra suvita (tiema) ke satha mitha miksa kara bachchom ke lie achchha nahim kara rahe haim
nutra suvita (tiema) bachchom aura vayaskom ke lie udara matra mem surakshita mana jata hai, lekina abhi taka puri taraha se sthapita nahim hua hai ki ye bachchom ke lie suraksha hai
isake alava, bachchom ko vriddhi aura vikasa ke lie kailori ki jarurata hai.
yaham eka garbhavati mahila ke lie poshana avashyakataom mem kai parivartana hote haim
isa taraha ke parivartanom ko anshika rupa se isa bhruna ka poshana karane ke lie aura anya sharirika bhinnarupom ki avashoshana aura poshaka tatvom ke chayapachaya ke karana anshika rupa se prabhavita haim.
ye parivartana bachchi ke samanya vikasa mem madada aura duddha nikalana ke bada ki manga , ya narsinga (stanapana)
poshana avashyakataom mem parivartana garbhavastha mem sharira ka anukulana karane ke lie sambandhita garbhavastha ke karana parivartana bahuta jaldi kevala bhruna ki jarurata ka javaba dete hai
isa taraha ke badalava bi-12, pholeta, vitamina bi -6, aura lipida, traiglisaraidsa mem vriddhi, aura rakta mem kolestraॉla ilektrolaitsa, protina, glukoja, vitamina ki kami shamila haim
garbhavati mahilao ko, pholika esida, sodiyama, aura chini ka sevana lohe ke vibhinna matra ki avashyakata hoti hai
matri kuposhana ka parinama mam ke lie svasthya samasyaom aura janma ke samaya shishu ka kama vajana aura anya kamiyom bhi shamila ho sakate haim 
khadya evam poshana borda ke rashtriya vijnana akadami garbhavati aura duddha nikalane vali mahilaom ke lie anushansita dainika ahara bhatta (radiai) badhane ke nidesha hai
obatetrishiana aura gayanekolojisata ameriki kaॉleja ke anusara, garbhavati mahilaom ko unake samanya sarvingsa khadya padarthom ki kisma ka badha़ana chahie
prayah, puraka poshana garbhavati mahilaom ke lie, pradana ki jata haim, lekina eka juna 1990 ki instityuta ऑpha medisina ke (IOM) dvara jari riporta ke anusara, adhyayana ina puraka chhote ya koi mulya nahim ho paya.
adhikansha chikitsakom sahamata hai ki radiesa, lohe ko chhoda़kara, uchita ahara ke madhyama se prapta ki ja sakati hai
rashtriya anusandhana parishada ke anusara, garbhavastha ke bada ke charanom mem vishesha rupa se lohe ki bada़i matra ki jarurata hoti hai aura akele ahara dvara ise pura nahi kiya ja rakata 
bhruna mangom ki vajaha se atirikta loha na kevala hai, balki isalie bhi kyonki mata ke rakta ki matra ke rupa mem adhika 30 pratishata ke rupa mem vriddhi ki ja sakati hai.
yaha khanija svastha lala rakta koshikaom ke nirmana ke lie avashyaka hai, eka mahila ke lie isaka upabhoga karana mushkila hai khadya padarthom se banae rakhane ke lie eka paryapta apurti khuda aura usake peta ke bachche ke lie
paryapta loha bina, bhruna mam se isaki apurti akarshita karega, aksara use lohe bina aura thaka hua chhoda deta hai
garbhavastha patte ke rupa mem aurata ki ja़rurata ko dabalsa kara deti hai (ullasa esida )
halanki, sabhi garbhavati mahilaom ke lie patte ki khuraka ki avashyakata para vaishvika sahamati nahim hai.
kuchha adhyayanom mem, vitamina phrolika esida nyurala tyuba dosha rokane mem mahatvapurna hoti hai,
phrolika esida ki jarurata lala rakta koshikaom ke nirmana ke lie avashyaka hai.
phira bhi, ioma painala ko sipharisha karane ke aparyapta sabuta mile hai ki sabhi mahilaom pholeta yukta saplimenta le, jo livara, gurde, hari pattedara sabjiyom aura sema aura matara sukhe mem paya ja sakata hai 
jabaki anusandhana jari hai, ioma ne garbhavati mahilaom ke lie hi saplimenta ki anushansa jo dhunrapana karane vali, nasha karane vali, sharaba pine vali, ya sakhta shakahariyom haim.
mitapa kama karane vale, sipharisha ka nirnaya karege jo vyaktigata avashyakataom para adharita ho aura vishishta chikitsa bina kisi karana ke maltivitamina ki puraka ka sujava nahim dege
kaphi chikitsako ki raya hai ki garbhavastha namaka kharaba karane vali halata hai, jo, sharira mem samanya se adhika namaka ka upayoga kara sakate haim
isake alava, sodiyama hani ke bhruna ke lie hanikaraka ho sakata hai.
eka dina 2000 se 8000 miligrama sodiyama ki matra ka sevana amataura para garbha mem sipharisha ki jati hai, jo samanya rupa se prati dina 1100 se 3300 miligrama ke lie sipharisha ki hai
chini bhi garbha mem eka samayika chinta ka vishaya hai.
lagabhaga sabhi mahilaom ko aura adhika glukoja nikalana hoga (chini ke eka pharma) unake mutra mem jaba ve garbhavati haim.
madhumeha vali mahilaom ki najadika se nigarani ki jani chahie sunishchita karane ke lie unaki rakta sharkara mulya ya samanya ke pasa hai.
garbhavastha mem eka aurata ka kailori sevana pratibandhita hai, use paryapta, vitamina aura khanijom protina nahim mila sakata hai, usake peta mem pala rahe bachche paryapta poshana karane ke lie
kama kailori sevana ke berakadauna mem parinama kara sakate haim jo moti mam mem sangrahita hota hai, usake khuna aura mutra mem madaka dravyom ke utpadana ke lie agrani jise kitona bulaya jata hai
kitona ka utpadana eka bhukha se bhukhamari ki nishani hai
kitona ka lagatara utpadana manasika rupa se manda bachchom mem parinama kara sakate haim
rashtriya vijnana akadami ke anushansa hai ki garbhavati hone se pahale ki tulana mem uneha pahali timahi mem garbhavati mahilaom ko ausata 150 kailori prati dina ya adhika khana chahiye aura 350 kailori prati dina ya adhika bada ke do timahiyom mem khana chahiye 
amataura para 25 se 30 paunda ke kula vajana badane ki sipharisha hai, vastavika svarupa mem adhika mahatvapurna paunda ki sankhya se adhika ka vichara hai
pahali timahi ke daurana vajana labha isake nyunatama para rahana chahie, aura hone vali mam se nirantara badha़ana chahie, use tisari timahi mem sabase adhika vajana, jaba bhruna aura plesenta sabase jyada badha़ta haim
tisa sala pahale, rashtriya anusandhana parishada ke khadya evam poshana borda ne garbhavastha ke daurana mahilaom ko salaha di hai ki unahe 20 se 25 paunda prapta karana chahiye
adhyayana se pata chalata hai ki kama vajana vali mahilaom ko, ya jo garbhavastha ke daurana 20 paunda se adhika labha, khatare mem kama janma-vajana shishu vitarita karate hai
1990 ke adhyayana para adharita, ioma aba anushansa deta hai ki 25 se 35 paunda ka vajana eka samanya garbhavastha ke daurana isa jokhima ko kama karane ke lie hai
kishorom aura kale mahilaom, jo aksara chhote bachchom, aba dridha़ta se adhika ki rashi prapta karane ke lie salaha di jati hai.
vajana badha़ane mem yaha sipharisha ki vriddhi hone ka peta bhara kha karane ke lie mataom ke lie eka hare ranga ka prakasha nahim de
yadyapi atirikta poshaka tatvom ki avashyakata haim, prati dina kevala 300 kailori ki vriddhi ki sipharisha ki jati hai
garbhavastha ke daurana vajana labha kramika hona chahie.
ostetrikiansa aura gyanekojisata ka amerikana kaॉleja pahale tina mahinom mem 3 se 4 paunda ki anushansa deti hai aura baki ke garbhavastha ke daurana prati maha 3 se 4 paunda hai
lagabhaga kula 6 se 8 paunda vajana ke bachche aura baki ka bhara mem bada़e stana aura badha़a tarala padartha, garbhashaya, dravya, aura plesenta ki matra shamila hai
garbhavastha ke daurana, vasa ki jama se adhika vriddhi se adhika kula rashi ka eka tihai eka aurata garbhavati hone se pahale 
jyadatara mahilaai janma ki prakriya mem ya kai haphtom ke bhitara vajana kama kara deti hai
stanapana se garbhavastha ke daurana jama vasa kama karane mem madada milati hai
eka aurata jo stanapana nahim karati hai aura jo karati hai use 600 se 800 kailori adhika kharcha karani padhati hai 
vaha aurata jo usake bachche ko narsom karati hai use vishesha poshaka tatvom ki jarurata badha़ jati hai
hama janate haim ki apake bachche ke svasthya ko apa atyanta mahatva pradana karate hai aura isalie hama chahate haim ki apake bachche ke sarvottama sambhava chikitsa dekhabhala ho aura apaki madada karane ke lie ninnalikhita suchana aura linksa pradana ki hai
sarakara labha karyakrama vebasaita apako sarakari karyakramom jisake lie apa aura apaka bachcha yogya hai dhundha़ne mem madada kara sakate haim
hara ameriki rajya mem sarakari karyakrama hai jo apako janakari pradana karate hai, aura svatantra ya kama lagata ki chikitsa dekhabhala aura khana upalbadha karata hai 
bala svasthya bima karyakrama bachchom ko muphta svasthya bima pradana karata hai.
bala poshahara karyakrama poshahara ki janakari aura mukta aura kama lagata vala khana pradana karate hai
bala kalyana ejensi pratyeka rajya ke lie garbhavastha ke satha perenatinga aura palaka dhyana mem madada kara sakata hai
svasthya vibhaga garbhavastha aura garbhavastha ke bada sambandhita poshana ki janakari ke satha pradana karati hai
shiksha vibhaga skula poshana ke bare aura garbhavastha karyakrama ki janakari pradana kara sakata haim
khadya stampa karyakrama kama aya vale vyaktiyom ko khana prapta karane ke lie madada karata hai
dabluisi ni: shulka bhojana, poshana paramarsha, aura sipha़arishem koi shulka bina pradana karata hai
chikitsa pratyeka rajya mem muphta sevaom apa aura apake bachche ko dene ki peshakasha ke bare mem javaba dene ke lie apaki madada kara sakata hai
amerikana akadami ke bala chikitsa (aapa) daॉktarom ka eka sangathana hai jo shishu aura bala dekhabhala mem visheshajna hai
pure jivana avadhi mem, poshahara manava jivana, svasthya aura vikasa ka eka buniyadi stambha hai
bhruna ke vikasa ke prarambhika charanom se, janma para, aura shaishava ke madhyama se, bachapana, kishoravastha aura vayaska mem, uchita bhojana aura achchha poshana bachane ke lie avashyaka hote haim, bhautika vikasa, manasika vikasa, pradarshana, utpadakata, svasthya aura achchhi taraha se kiya ja raha hai
sabhi bachche mauta ke adhe se adhika kuposhana ke satha juda़e rahe haim, jo bimari se sharira ki pratirodha kamajora karate hai.
bechari ahara, lagatara bimari, aura aparyapta ya anyamanaska dhyana yuva bachche kuposhana ka karana bana sakata hai.
agara eka aurata garbhavastha ke daurana kuposhita hai, ya agara usake bachche jivana ke pahale do varshom ke daurana kuposhita hai, bachche ke bhautika aura manasika vikasa dhima ho sakata hai 
jaba bachcha bada hota hai to yaha banaya nahim ja sakata hai - yaha bachche ko usake baki ke jivana ke lie prabhavita karega
bachcho ko dekhabhala karane ka adhikara hai, surakshatmaka vatavarana aura paushtika bhojana karane ke lie aura buniyadi svasthya dekhabhala bimari se unhem bachane ke lie aura vriddhi aura vikasa ko badha़ava dene ke lie
dayariya poshana ke antargata ka eka mahatvapurna karana hai
kyonki dayariya ke daurana poshaka tatva ki avashyakata badha़ jati hai, anya sankramaka rogom ke daurana, jabaki poshaka tatva ka sevana aura avashoshana amataura para kama ho jata hai
dayariya ka pratyeka prakarana vajana kama karane aura vikasa hinata paida kara sakata hai.
aura bhi, yadi dayariya bara bara hota hai, vaham episoda ke bicha vikasa pakada़ne ke lie bahuta kama samaya ho sakata hai, parinama samanya vriddhi vakra ka vinasha
bachche jo tivra atisara ke lagatara episoda ka anubhava karate hai, ya jinahe lagatara dayariya hai, poshana ke antargata banane ki sambhavana jo bachche dayariya ke kama ya chhote episoda ka anubhava karate hai se jyada hai 
samanya mem, poshana ki sthiti mem dayariya ka prabhava bachche ke hara sala dayariya rahane ke anupata mem hai
krama mem, poshana ke antargata dayariya ki samasya mem yogadana deta hai
bachchom mem, jo aparyapta khane ke parinama svarupa poshana ke antargata rahe haim, pichhale atisariya episoda, ya donom, tivra atisariya episoda adhika gambhira hai
lagatara dasta bhi adhika hota hai aura aura pechisha aura adhika gambhira hai.
lagatara dayariya ke prakarana se marane ka khatara ya pechisha mem kaphi vriddhi jaba bachcha pahale se hi poshana ke antargata hai
samanya mem, ye prabhavom poshana ke antargata ki digri ke anupatika haim, ye sabase bada़a taba jaba poshana ke antargata gambhira hai.
isa prakara, dayariya aura poshana ke antargata eka dushchakra pharma ka sanyojana karate hai jo, agara tuta nahim, to antatah parinama mauta ho sakati hai
antima ghatana gambhira vishesha rupa ya dayariya ka lamba prakarana ho sakata hai, jaba gambhira kuposhana maujuda ho, anya gambhira sankramana jaise nimoniya ho
dayariya se hone vali nrityu, vastava mem, amataura para poshana ki kami ke satha judi hai
aspatalom mem, jaham nirjalikarana ke achchhe prabandhana ka abhyasa hota hai, bachchom mem dayariya se lagabhaga sabhi nrityu poshana ki kami se hoti hai
dayariya, hakikata mem, poshana ki bimari aura ilektrolaita aura tarala padartha ke rupa asantulana, aura jaba taka isa roga ke donom pahaluom ka ilaja chikitsa paryapta nahim hai
phira bhi, tarala pratisthapana ke viparita, dayariya ka poshana prabandhana achchhe khana pana ki prathaom ki avashyakata hai donom bimari ke daurana aura dayariya ki ghatanaom ke bicha, jaba bachcha bimara nahim hai.
jaba yaha ho gaya hai aura poshana ki kami ya to roka hai ya sahi, dasta ke eka prakarana bhavishya bahuta kama ho raha hai se mauta ka khatara.
khilana shishuom aura chhote bachchom ki dekhabhala ka eka mahatvapurna pahalu hai.
upayukta khila prathaom ko dhyana dene aura manasika, samajika vikasa ke satha sambandhom ko protsahita karana hai
ve behatara poshana aura sharirika vikasa ke lie, ama bachapana ke sankramana ko kama grahakatva aura behatara virodha una logom ke satha samana karane ke lie.
chhote bachchom mem jivana avadhi mem behatara svasthya ke lambe samaya se sthayi svasthya prabhava padate hai, badha़ pradarshana aura utpadakata sahita, aura kuchha gaira kama jokhima sanchari rogom se 
halanki yaha shayada hi kabhi pratyaksha karana ke rupa mem suchibaddha hai, kuposhana bachapana mem hone vali mautom ke adhe se jyada mem yogadana deta hai.
jyadatara bachchom ke lie, khane ke lie upayoga ki kami ke karana kuposhana ka ekamatra karana nahim hai.
kharaba khila prathaom aura sankramana, ya donom ka sanyojana, donom kuposhana ke pramukha karaka haim. sankramana - vishesha rupa se lagatara ya lagatara dayariya, nimoniya, khasara aura maleriya - poshana sambandhi sthiti batati hai 
kharaba khila prathaom, jaise aparyapta stanapana, galata ahara dena, aparyapta matra dena, aura yaha sunishchita nahim karana ki bachche ko paryapta khana mila, kuposhana mem yogadana dete hai 
kuposhita bachche, bari mem, adhika roga se asurakshita hai aura dushchakra ki sthapana ho jati hai
behatara khila prathaom jo kuposhana ko rokane ya ilaja prati varsha 800,000 jindagi bacha sakata hai
kishoravastha svastha vayaska jivana ke lie taiyara karane ki eka khida़ki ka pratinidhitva karati hai. 
kishoravastha ke daurana, poshana sambandhi samasyaai jo prarambhika jivana mem ati hai sambhavita sahi kiya ja sakata hai, vartamana valom ko sambodhita karane ke atirikta
yaha bhi svastha bhojana ke akara ko aura majabuta aura jivana shaili vyavahara ki samaya avadhi, poshana ki shuruata-vayaskata mem sambandhita purane rogom ko rokane ya talane ke lie
jaise kishorom mem sankramana ka prasara kama hai chhote bachchom ki tulana mem jaise nimoniya aura antrashotha 
vaham poshana mudde haim, jisamem kishora-vishishta hai, aura jisake lie vishishta rananitiyom aura drishtikona ki jarurata hai
sarvajanika svasthya karyakramom ke prashasana mem hamesha nirdharana aura mulyankana jora se adhika ka khatara rahata hai
jaisa ki jelliphe aura jelliphe ne hala hi mem kaha, kabhi kabhi spashta ka sabuta aura avashyaka nivaraka upayom ki shuruata ka avarodha vaijnanika jancha para badhyakari agraha vilamba kara sakate haim 
hama mulyankana ke satha apane apa ko chintana karane ka karana hai, phira bhi, sansadhana teji se simita ho rahe hai, aura yaha pata karana avashyaka hai ki hamane kya hasila kiya hai aura hama kya prapta karane mem viphala rahe haim
suvidhapurvaka mulyankana kasauti ki chara shirshakom ke antargata charcha ho: nrityu, rugnata, vikasa, aura anta mem, suchakanka karya kshamata ke adhara para
nrityu dara, khasa taura para shishu nrityu dara, janata ke svasthya ka eka ama suchaka hai, aura eka se chara varsha ke ayu samuha mem nrityu dara aksara tulanatmaka antararashtriya sarvekshana mem vibhinna deshom mem poshana ki kami ki vyapakata ke suchakanka ke rupa mem prayukta kiya gaya hai 
antararashtriya stara para shishu mem aura purva skula mem nrityu dara mem hala ke varshom mem ghatane ka kaphi natakiya sabuta hai
janasankhya gatishilata ke upaya hamesha adhika nrityu ke akalana ka khulasa kara rahe haim
phira bhi, yatharthavadi hone ke lie, adhikansha vikasashila deshom ke lie vishvasaniya ankada़e lene mem kuchha samaya lagega, lekina apekshakrita chhote namunom para bhi kaphi sarala purvavyapi adhyayana mulyavana ho sakati hai.
mataom se prashna puchhe ja sakata hai jaise, apake katane bachche hai, kitane aba bhi jivita haim aura kisa unra mem ve mara gaye? 
demographarsa ne shishu aura bala nrityu ke bare mem suchana ke stara ko aura adhika paramparagata bayanom mem parivartita karane ke lie takanika ka vikasa kiya hai 
nrityu deta ki vyakhya mem yaha spashta rupa se mahatvapurna hai ki mauta ke karana ka pata lage
udaharana ke lie, gambiya mem, meka gregora ne dikhaya hai ki 1973 se purva, pancha varsha ki ayu se pahale ke 50 pratishata ke bachche mara gaye the
isake viparita, gvatemala mem, yahi ankada़a kevala lagabhaga 25 pratishata tha.
poshana-avasatha aura maleriya ke bicha ka rishta jatila hai; donom kula pratiraksha kshamata para pratikula prabhavi dalati hai
jo bhi karana hai, yaha bala dena hai ki poshana sambandhi sthiti ka nrityu sanketaka ke rupa mem eka bahuta munhaphata sadhana ho sakata hai vishishta karanom se sambandhita atirikta janakari ke bina
eka ko mauta ki unra ke bare mem vistrita janakari ki jarurata hai. isa riporta ke sandarbha mem jivana ke pahale varsha ke daurana mautom ke vitarana ki jancha karane ke lie yaha vishesha rupa se prasangika hai ki 
laitina amerika aura kairibiyana mem piaiechao ke sarvekshana mem isa adhyayana ke adhikansha kshetrom mem, eka aura chhaha mahine ke bicha mem ghatita chhaha aura baraha mahinom ke bicha ke rupa mem hai
satha hi shishu nrityu dara ki vyakhya mem ahara sankramana para vichara ke mahatva ka eka aura udaharana ke rupa mem, santa mariya kokuai, gvatemala unra para dayariya bimari ki ghatana se monatiaila ke nishkarsha mahatvapurna haim
choti rugnata chhaha aura baraha mahine ke bicha hota hai, yani ke nrityu ke pika ke bada, sujava hai ki dayariya uchcha shishu nrityu ka pramukha karana nahim hai.
dusari tarapha, mata kostarika mem aura adhika hala ke adhyayana mem, shishu nrityu dara aura dasta se nrityu ke bicha eka bahuta spashta rishta dikhaya gaya hai.
isa ghatana ke eka satika anumana prapta karane ke lie rogom ki aura gambhirata ki poshana rajya se sambandhita ho sakata hai, kathina hai; atisariya roga ki sthiti hai jisane yuva bachche ke sambandha mem sabase adhika dhyana liya hai
ghatana para eka vishvasaniya jancha ke lie, prekshana vastava mem janasankhya ka namuna para kie jane ki jarurata hai kama se kama hara do haphte mem, aura agara karana bhi nirdharita karana ho to, yaha eka atyanta kushala aura durjeya upakrama ho jata hai 
mausami ghatanaom mem bada़e julom dvara vyakhya jatila ho sakati hai, jaise gambiya, naijiriya, aura nepala mem dekha gaya hai
hame na kevala sankramana ke episoda ke avritti balki unaki avadhi ki bhi pata karane ki avashyakata hai, jo tivrata ka eka mahatvapurna suchaka hai.
donom mata kostarika aura tomakinasa naijiriya ne dikha diya hai ki sankramana ki avadhi eka kuposhita ki tulana mem eka achchhi taraha se poshita vyakti mem jyada hai halanki episoda ki sankhya tulanatmaka ho sakati hai.
yaha kevala samapta karane ke lie uchita hai ki, halanki rugnata mem kami se, atisariya roga vishesha rupa se, mam ke bachche hastakshepa ki saphalata ka eka bahuta hi prasangika sanketa ho sakata hai.
isa drishtikona ke adhika upayukta sarvajanika svasthya karyakrama ke niyamita mulyankana ke lie, chara ki sankhya aura sansadhanom avashyakata hai,
shishuom mem poshana sambandhi sthiti ke mulyankana ke bare mem aura chhote bachchom ki anthropometrya ke bare mem kaphi kuchha likha gaya hai
phira bhi, hamesha ki taraha anubhagiya drishtikona hai ki, apekshakrita badalane ke lie asamvedanashila jaba taka ki ise natakiya hai 
adhika satika janakari vikasa ki gati ki mapa ke dvara prapta kiya ja sakata hai.
anudairdhya upayom ki svabhavika rupa se avashyakata hai, lekina phayada yaha hai ki kaphi chhote namunom sarthaka parinama de sakata hai.
jaisa ki pahale se hi nrityu dara aura asvasthata ke sambandha mem bala diya gaya hai, yaha alaga alaga unra mem bachchom ko dekhane aura samuha mem nahi ke lie mahatvapurna hai 
vaham manavashastriya akalana mem isake lie eka atirikta karana hai.
nashili davaom ke parche sahayata karyakrama mukta ya kama lagata vali dava kama aya vale vyaktiyom aura parivarom ko pradana karate hai
phri medikala / dentala klinika mukta klinikom ke bare mem janakari apako pradana karate hai.
rashtriya sangathana durlabha vikarom ke lie marija ka sahayata karyakrama hai jo loga anyatha unaki davaom ka kharcha nahim utha sakate haim ko muphta dava pradana karata hai.
rogiyom ki sahayata ke lie patra hone ke lie aura anya karyakrama vittiya vishishta manadandom ko pura karana hoga.
kuchha rajya mahilaom ke lie davaom ke sahayata pradana karate haim jo chikitsa dvara kavara nahim haim
isake atirikta, jinako ja़rurata hai unako muphta davai ki apurti karane ke lie kai dava kampaniyam apane svasthya seva pradata ya chikitsaka ke satha kama karenge 
hila-bartana aspatalom aura narsinga homa ki suvidhaa hai jise sanghiya sarakara se paisa mila hai aura aba hara sala muphta mem eka nishchita rashi ya kama lagata vali svasthya sevaai pradana karana hoga jo bhugatana nahim kara sakate
tuma kisi bhi samaya hila bartana sahayata ke lie avedana kara sakate haim dekha-bhala ke pahale ya bada mem hila bartana dvara sahayata ke lie puchha kara 
bachche ke lie tike karyakrama parivarom ko daॉktarom ko muphta tike upalabdha karake madada karata hai jo patra bachchom ki seva karate hai aura sidisi dvara rashtriya stara para dekha jata hai
rajyom aura patra amerika pariyojanaem shuru karane ke lie yogya rogiyom ki seva aura unra 18 varsha sahita, chhota karane ke lie koi bahara ka jeba kharcha ke satha niyamita inyunaijeshana pradana karata hai
sabase adhika abadi mem pahale do sala ke bada ghatati barbada ki vyapakata, badhi hai
adhika janakari vikasa ghate ke mahatva ke bare mem ki jarurata hai.
eka bindu, tathapi, jo paryapta nahim hai para jora diya hai ki diye gaye ghata ka jokhima bahuta chhote bachchom ke lie adhika se adhika hai.
parinamasvarupa dahalija ya kata, jisa para atirikta jokhima prashansaniya hai bindu se unra ke satha badalati hai.
bangladesha mem eka bahuta mahatvapurna adhyayana mem, nrityu aura barbadi ki digri ke khatare ke bicha ke sambandha ka pata lagaya gaya hai.
bachche, jo buri taraha barbada aura avikasita the ki bahuta uchcha nrityu dara thi lekina avikasa ki udaravadi digri koi atirikta jokhima uthane nahim diya.
nrityu ka eka bahuta karyatmaka hani kachche aura charama suchaka hai.
kuchha deshom mem, 50 pratishata taka skuli bachche manadanda €ke rupa mem avikasita vargikrita ho sakate hai (unchai-unra 90 pratishata sandarbha se kama), yaha bahuta mahatvapurna ho jata hai ki avikasa ke bare mem aura adhika janakari ho
kuchha shramika bhojana ki kami ko avikasa ke saphala anukulana ke rupa mem dekhate hai, halanki kaphi bachche ke lie puri taraha se apane paryavarana mem parivartana para kabu pane mem saphala nahim hai
vaham sanketaka hone mem koi phayada nahim jaba taka ki yaha nirnaya lene ke lie upayogi ho
agara nirnaya khine ke karyakrama ki avashyakata ke bare mem hai, to hamari ajnanata ke karana aura avikasa ke sharirika mahatva sharmanaka hai.
mahilaom ke prajanana ayu ke poshana svasthya ki raksha karane ke lie, purva garbhavati ki sthiti, garbhavastha ke daurana aura duddha nikalane para vichara karana jaruri hai
sabase sarthaka mapana vajana ke lie unchai aura jangha paridhi haim.
deshom mem, jaham kavashiorakora protina ka pramukha rupa-urja kuposhana hai, plajma elabumina ki mapa mahatvapurna hai, jaise ki rakta himoglobina jaham araktata hai.
garbhavastha ke daurana sabase adhika prasangika manavashastriya mapa bhara ki rashi hai jo apane korsa ke daurana prapta ki hai.
britena ke lie 12.5 kilograma (12) rashi ki sipharisha ki gai hai, aura yaha avanchhaniya kiya jaega agara janasankhya ki ausata vriddhi 10 kilo se niche gira gayi 
anusandhana ki sakhta jarurata hai jisamem manavashastriya badala tulanatmaka sambandhita ho, mam ke svasthya evam kalyana donom ke lie, satha hi garbhavastha ki saphalata ke lie aura bada mem bachche ki vriddhi aura vikasa ke lie 
adhe se adhika bachcho ki mauta kuposhana ke satha judi hai, jo bimari se sharira ki pratirodha ko kamajora karati hai.
kharaba ahara, lagatara bimari, aura aparyapta ya anyamanaska paravaha chhote bachchom ke kuposhana ka karana bana sakate haim
agara eka aurata garbhavastha ke daurana kuposhita hai, ya agara usake bachche jivana ke pahale do varshom ke daurana kuposhita hai, bachche ka sharirika aura manasika vikasa dhima ho sakata hai
yaha banaya nahim ja sakata jaba bachche bada़e hai – yaha bachche ko jivana bhara prabhavita karega
bachche ko dekhabhala karane ka adhikara hai, surakshatmaka vatavarana ka aura paushtika bhojana karane ka aura buniyadi svasthya dekhabhala jo bimari se unhem bachaye aura vriddhi aura vikasa ko badha़ava de
kuposhana upa sahara aphrika ki tulana mem bharata mem aura ama hai 
duniya mem kuposhita bachchom mem eka hara tina mem bharata mem rahata hai.
lagabhaga 50 pratishata bachapana ki mautom ke lie kuposhana jimmedara thaharaya gaya hai. 
kuposhana vikasa aura janane ki kshamata ko simita kara deta hai
bharata mem, lagabhaga tina varsha ki ayu se niche ke 46 pratishata sabhi bachche apani unra se chhote haim, 47 pratishata vajana se kama hai aura kama se kama 16 pratishata barbada hai
kuposhana ki vyapakata rajya bhara mem badalati hai, yaha madhya pradesha mem uchchatama dara rikaॉrdinga (55 pratishata) aura kerala mem sabase kama (27 pratishata) hai. 
bachchom mem kuposhana akele bhojana ke sevana se prabhavita nahim hai.
kuposhana svasthya sevaom ke upayoga, bachche ke lie dekhabhala ki gunavatta aura garbhavati mam satha hi achchhi svachchhata prathaom dvara bhi prabhavita hota hai
lada़kiyom ko unake kama samajika sthiti ki vajaha se lada़kom ki tulana mem kuposhana ka khatara aura adhika hai
bachapana mem kuposhana ka dirghakalika parinama gambhira hai, kyonki yaha, motara, samvedi, sanjnanatmaka samajika aura bhavanatmaka vikasa mem badha dalata hai 
kuposhita bachche ke skula mem kama achchhi taraha se pradarshana ki sambhavana hai aura adhika kuposhita vayaskom mem vriddhi hone ki sambhavana hai, roga aura jaldi mauta ka adhika se adhika jokhima para
sabhi vayaska mahilaom ki eka tihai kama bhari hai
mahilaom aura lada़kiyom ke aparyapta dhyana, khasataura para garbhavastha ke daurana, kama vajana ke shishuom ka janma parinama hai
navajata ke lagabhaga 30 pratishata ka janma ke samaya kama vajana hota hai, unhem age kuposhana aura bimariyom ko kamajora bana deta hai
vitamina aura khanija ki kami bachchom ke astitva aura vikasa ko prabhavita karati hai.
raktalpata tina varsha ki unra ke antargata, 74 bachchom ke pratishata ko, 90 kishora lada़kiyom se adhika aura 50 pratishata mahilaom ko prabhavita karata hai
ayodina ki kami, jo sikhane ki kshamata ko 13 phisadi se kama kara deta hai, vyapaka hai kyonki sabhi gharom ke adhe ayodinayukta namaka ka upayoga karate hai
vitamina e ki kami, jo andhapana aura purva-skuliyom mem rugnata aura nrityu ke karana badha़ jate hai, sarvajanika svasthya samasya bana hua hai.
0-3 ayu varga ke bachche vanchita samuhom se aura paryapta rupa se vidyamana aisidiesa dvara sevata nahim hai 
kuposhana ke mahanatama starom ke satha kshetrom para spashta niti dhyana ki kami hai.
khina shishuom aura chhote bachchom ki dekhabhala ka eka mahatvapurna pahalu hai.
upayukta khila prathaom ko dhyana dene aura manasika, samajika vikasa ke satha sambandhom ko protsahita karate hai
ve behatara poshana aura sharirika vikasa ke lie, ama bachapana ke sankramana ko kama grahakatva aura behatara virodha una logom ke satha samana karane ke lie hai
chhote bachchom mem behatara svasthya parinama lambi jivana avadhi mem sthayi svasthya prabhava dalati hai, bade pradarshana aura utpadakata sahita, kuchha gaira aura kama jokhima sanchari rogom ki 
kuposhana saba bachapana mautom ka adhe se jyada mem yogadana deta hai, halanki yaha shayada hi kabhi pratyaksha karana ke rupa mem suchibaddha ho
jyadatara bachchom ke lie, bhojana ki kami kuposhana ka ekamatra karana nahim hai.
kharaba khilane ki pratha aura sankramana, ya donom ka eka sanyojana, kuposhana ke donom pramukha karaka haim.
sankramana - vishesha rupa se lagatara ya jaldi dayariya, nimoniya, khasara aura maleriya - khanom ke tahata poshana sambandhi sthiti hai.
kharaba khila prathaom, jaise aparyapta stanapana, galata ahara dena, aura aparyapta matra mem de rahi hai aura sunishchita nahim karana ki bachche ko paryapta bhojana milata hai, kuposhana mem yogadana de rahe hai
bari mem, kuposhita bachchom ke, adhika roga aura dushchakra asurakshita sthapita hai
sudharati khanapana prathaai jo kuposhana ko rokati hai ya ilaja ke lie prati varsha 800,000 jivana bacha sakati hai
kishora mauka eka svastha vayaska jivana ke lie taiyara karane ki eka khida़ki ka pratinidhitva karata hai.
kishoravastha ke daurana, poshana sambandhi samasyaom ko prarambhika pahale ke jivana mem sambhavita sudhara ja sakata hai, vartamana valom ko atirikta sambodhita karane ke lie.
yaha bhi eka samaya avadhi hai akara aura svastha khane ko majabuta jivana shaili aura vyavahara, ya poshana ki shuruata-vayaskata mem sambandhita purane rogom ko talati hai
jaisa ki kishorom sankramana ka prasara kama hai jaise nimoniya aura antrashotha chhote bachchom ki tulana mem
jirna roga ki tulana mem unra ke logom ke satha, ama taura para una para, kama svasthya aura poshana dhyana diya gaya hai, prajanana svasthya chintaom ko chhoda़kara 
vaham poshana mudde haim, jo kishora-vishishta haim, aura jo vishishta rananitiyom aura drishtikona ke lie kahate haim.
echaitivi dvara isa karyakrama ki maulikata ka dava nahim kiya jata hai.
hama bahuta se logom aura srotom ke lie hamara abhara svikara karate hai jisaka kama aja़adi para khincha hai
karyakrama pustikaem, patraka, postara aura suchanatmaka gaida jo marathi aura angreji mem hai ke dvara puraka aura samarthita hoga, aura aja़adi se svasthya karyakarta steshanom, aspatala, skulom, aura adhika para upalabdha hoga 
donom mataom aura bachchom ka 40% maharashtra, bharata mem chirakala poshana ki kami hai, aura kama pancha ki nrityu dara prati 1000 jivita janmom 58 mautom mem hoti hai, ya 1 hara 17 bachchom mem.
ina mautom ki eka bahuta bada़i sankhya nirjalikarana se dayariya se utpanna hoti haim, bachapana ki nrityu ka sabase asani se nivaraniya karana. 
garbhavastha ke daurana, prasava aura tatkala janma ke bada ki avadhi mem kushala upasthiti
mataom ko narsa ya dai se stanapana ke lie prashikshana pradana karaya jaega.
pahale adhe ghante mem janma ke bada kolostrama pradana karane ke mahatva ke bare mem protsahita karana, aura navajata ya janma ke bada ki avadhi ke bare mem anya prashna ke bare mem salaha di jati hai 
bimariyom ke khilapha uchita khila prathaom aura rogakshamata ke labhom ke bare mem spashta disha nirdeshom ke samvardhana ke madhyama se surakshita rakhem.
bachche ko pahale adhe ghante mem janma, vishesha rupa se stanapana ke bada eka bachche ke jivana ke pahale chhaha mahinom ke daurana kolostrama pradana karane ki pratha ko badha़ava dena chahiye
chhaha mahine se upayukta puraka khilane ke satha aura do sala ya pare nirantara stanapana ke lie, vitamina e ki anupurana aura anya poshana ke satha avashyakatanusara
puraka khila mam ke dudha ke alava khadya padartha de rahi hai.
kuposhana upa ishtatama stanapana prathaom se, kharaba gunavatta puraka ahara, hanikaraka khilane ki prathaom, puraka bhojana aura khine ke bartana aura sandushana ka parinama ho sakate haim, 
shishu ki prathama varsha ki dusari chhamahi eka vishesha rupa se samvedanashila samaya hai kyonki shishu khana sikha rahe hai aura bara bara aura dhairyapurvaka narama bhojana khilaya jana chahie.
yadi poshana sevana aparyapta hai, to parinama jivana bhara rahega
anubhava dikhata hai ki khana shishuom aura tivra atisara ke satha bachchom se hui nahim hona chahie
unake khila sthiti para nirbhara karate hue, bachchom ko stana dudha ya dudha patala prapta kare
nirjalikarana ke mamalom mem, jaise hi prarambhika rjalikarana chikitsa pura kara liya gaya hai ke rupa mem peshakasha ki jani chahie
jaisa hi bhukha ae uchita sthaniya upalabdha khadya padarthom (anaja) ke rupa mem jalda hi peshakasha ki jani chahie
dayariya ke bada, bhojana ki matra samanya se adhika eka chhoti avadhi ke lie diya jana chahie
koi vishesha shishu pharmula ka upayoga (laiktoja mukta utpada) dayariya ke mamalom ke lie dridha़ta se hatotsahita kiya jana chahie ve kevala shayada hi kabhi avashyaka haim aura mahange hote haim
eka bachche ke jivana ke pahale varsha ke bhitara khasara tikakarana ko badha़ava dene ke dvara dayariya ki ghatana ko kama karana hai.
khasare ke khilapha 100% kavareja pratirakshana karyakrama ka lakshya hai.
mataom ke bhrama ko lakshita karane ke lie aura kaise pahachana ke bare mem samajane ki kami digri ka mulyankana aura atisariya nirjalikarana ka ilaja ke lie hai
mataom ko, tatkala tarala pratisthapana, tarala aura khadya ke badane, ghara ka bana hua tatha anya rainkom paika kaise sahi dhanga se taiyara karane ke lie, anaja adharita orala rihaidreshana salta, kaba aura kyom, isaka istemala karane ke lie, aura satata khila, stanapana sahita ke lie mahatvapurna jarurata padha़aya jaega
yadi mataai jaldi se ghara mem nirjalikarana ko pahachana sake aura ilaja kara sake, to bachchom ke bahumata ko atirikta chikitsa dekhabhala ki jarurata nahim hoti
jabaki maharashtra mem mataom ka kevala 41% sahi lakshana ki nirjalikarana ke lie chikitsa upachara ki jarurata hai, bachche ki pahachana ka sujava de sakata hai, 77% svasthya suvidha mem dayariya se bachche ko le jate hai.
homa-meda ghola ko paribhashita karem aura badha़ava de -- ghara mem banaya ghola (atirikta chavala ke satha, chaya, chavala pani, adi) ki spashta paribhasha ko paribhashita karem 
prathama stara ke svasthya karyakarta, samudayika svasthya karyakarta, jisaki jnana ke prasara mem aura dayariya ke prabandhana kaushala ke lie mahatvapurna bhumika hai, rupa mem donom ghara upachara aura anya rainkom ke prayoga se. 
orisa paiketa paryapta apurti mem aura pure svasthya pranali stara para upalabdha hona chahie
agara yaha sambhava nahim hai, adhikatama prayasa jari rakhana chahie aura yaha anta ki ora, isa bicha mem, pure ke rupa mem banane ke rupa mem sambhavata ka prayoga kiya jana chahie, simaom ko pahachanana chahie
jaba poteshiyama lapata sanghataka ho, rogiyom ki poteshiyama mem (phalom ke rasa) amira tarala padartha pine ke lie protsahita kiya jana chahie.
bade tarala aura ghara ka bana aura paika drava
orisa drava ki taiyari ke lie sabase surakshita pani ka prayoga kiya jana chahie 
mishrita sandeshom ki varshom se nirmita bhrama sahi karo, rjalikarana drava samagri ke mapa ke bare mem
mataom ko plastika ki eka litara ki botala milegi, lebala ke satha ki kaise chinha pahachana ke bare mem aura nirjalikarana ki digri, kaise ghara tarala padartha rjalikarana taiyara karane ke lie aura kaise mishrana taiyara karane ke lie aura ghara ka bana hua tatha anya rainkom paika
isake alava, ve prapta karenge 2 tarapha chammacha namaka aura chini ko sahi dhanga se mapane ke lie
maukhika rjalikarana chikitsa, agara thika se abhyasita ho, dvara shishu aura bala nrityu dara mem kama se kama adhe katauti kara sakate haim aura, aspatalom, svasthya kendrom, aura sabhi deshom mem klinika mem mahanga yatraom ke anaginata lakhom ki avashyakata ka nirakarana 
lekina yaha sambhava hai? maukhika rjalikarana lavana (orisa) ke paiketa vishva mem upalabdha nahim haim aura sarala chini, namaka samadhana ka kha़taranaka prabhava ho sakate haim agara yaha anuchita mishrita hai
anaja-orati adhara donom ka kamiyo ka samadhana para kabu kara sakate haim abhi taka eka aura maukhika rjalikarana upachara ka vikalpa pradana karake
surakshita jala pranali jala gunavatta hastakshepa hai jo sasti sarala aura majabuta praudyogiki vikasashila duniya ke lie upayukta hai
uddeshya jala kitanushodhana aura surakshita bhandarana ke madhyama se istemala ke bindu para surakshita banane ke lie hai.
pvainta ka dushita pani ke upayoga ki upachara haidokaloraida samadhana sodiyama ka istemala kara sthaniya kharide aura pani aura namaka se isa samudaya mem utpanna eka ilekatoletika sela ka istemala karate haim.
vyavahara mem badalava ki takanika, samajika vipanana, sahita samudaya jutana, sakshatkara, preranatmaka sanchara, aura shiksha, dushita pani aura bimariyom ke bicha ke sambandha ke bare mem jagarukata badha़ane ke lie. hai
surakshita pani ke labha, svachchhata aura vyavahara ko prabhavita karane ke lie kharida sahita pani ke bhandarana ke bartana aura samuchita upayoga aura nissankramaka hai 
surakshita jala sangrahana plastika ke kantenara mem eka sankirna munha ke satha, dhakkana, aura pani ki puna sandushana ko rokane ke lie hai
saura vikirana ka prayoga kara kitanurahita gande pani ke satha isa mukta aura asana takanika hai
saura vikirana pani se paradarshi kantenara bharane ki aura pura karane ke lie surya ke prakasha ke bare mem pancha ghante ke lie rakhane ki prakriya hai jo pani mem rogaja़naka़om ko nashta kara deta hai
gande pani ka ilaja karem aura roga ke prasara ko roke ubalate pani ki pratha ko badha़ava dene ke dvara aura klorina, ayodina, ya yaham taka ki ghara ke blicha ka upayoga, ghara pani kitanushodhana ka sanchalana karane ke lie. 
svasthya karyakartaom ko ina tarikom ka upayoga karane ke lie prashikshita kiya jaega aura mataom ke lie takanika pahuncha denge
sabhi skulom mem shauchalaya ki suvidha sthapita karane ke lie maharashtra rajya aura bharata sarakara ke karyakramom mem avashyaka bhagidari vikasita karane ke lie protsahita karate haim aura yogadana dete haim 
bharata sarakara ne desha mem gharom mem 2010 taka shauchalaya badha़ane ke lie eka kendrita abhiyana ko shuru kiya hai
isake atirikta, hamem lada़kom aura lada़kiyom ke lie alaga shauchalaya ke lie tatkala avashyakata lakshya hoga, satha mem maharashtra mem hara skula mem hatha dhone ki suvidha 
dayariya ki ghatanaom ko kama karane ke lie sabuna aura pani se hatha dhone ki adata ko badha़ava de
sabuna ki paryapta upalabdhata sunishchita karane ke lie, sthaniya nirmataom ke satha sajedari isa pratha ko badha़ava dene ke lie upayoga kiya jaega.
dayariya se mauta rokem aura bachche ko dayariya samvedanashilata episoda ke bada sabhi svasthya dekhabhala pradataom ko shikshita karake aura mataom jinka anupurana mem
kendrita aura ekikrita abhiyana ke dhyana madhyama se, aura sthaniya nirmataom ke satha bhagidari ke madhyama se, hama jasta ki khuraka ki upalabdhata mem vriddhi karege 
13 mahine ka kailendara, jivana sandeshom ke lie 13 tathya karane ke lie, jivana bachane ki janakari ko asani se upalabdha karate hai
bachchom ke lie manasika svasthya hastakshepa, batai gai samasya aura ayu ke lie aura bachche ki anya vyaktigata visheshataom ko dhyana mem rakhate hue alaga-alaga hote haim|
yadyapi ina hastakshepom mem kuchha tarike eka se hote haim, upachara ki vidhiyam eka dusare se kapha़i bhinna ho sakati haim|
manochikitsa evam ‘manovishleshana’ paribhashika shabdom ke prayoga me vishesha antara ke satha bachche ke upacharom ka varnana karane vale paribhashika shabda vishva ke eka bhaga se dusare bhaga mem alaga alaga ho sakate haim|
isa karana se isa lekha mem sabhi shabdom ki paribhashaom ko samajane ke lie pathakom ko vishesha savadhani leni chahie|
yaha lekha vyavaharika, manoparivartanatmaka ya anya kshetrom mem bachchom ke lie sabhi prakara ke upacharom ko shamila karata hai|
bachchom ke manovishleshana upachara ko apanane ke lie kai vibhinna takanikiyam haim|
agara bachche bahuta chhoti ayu ke haim, eka anukulita manovishleshaka takanika avashyaka ho sakati hai|
kuchha rogiyom mem abhibhavaka-shishu manochikitsa eka sambhavana hai|
do takanikiyom ki vivechana ki jaegihabhibhavaka – shishu manochikitsa aura mentaliserena bevorderenda kindara therepi (ema bi ke ti)|
abhibhavaka-shishu manochitsa mata-pita aura bachche ke bicha samanya sambandhom se sambandhita samasyaom ko sambodhita karati hai|
eka bi ke ti shishu ki vastavikata aura kalpana mem antara karane ki yogyata se sambandhita samasyaom ko sambodhita karata hai|
yadi abhibhavaka-shishu ke bicha surakshita lagava ka surakshita silasila tuta chuka hai, abhibhavaka-shishu manochikitsa eka takanika hai jo isa bandhana ko bachane ke lie prayoga mem laya ja sakata hai|
isa takanika ke lie abhibhavaka, shishu aura chikitsaka ke bicha trimargiya sambandha ki avashyakata hoti hai|
chikitsiya baithakom ke daurana abhibhavaka apane vichara aura manobhava prakata karata/karati hai jo ki shamila hane vale kai karakom para adharita hote haim|
bachche ke bhavishya ke lie abhibhavaka ki apekshaem aura ashaem|
chikitsaka ki bhumika abhibhavaka aura shishu ke bicha hone vali kriya ke avalokana aura vyakhyata ki taraha hoti hai|
chikitsaka abhibhavaka ke satha bachche ke vyavahara se sambandhita apane kuchha vicharom ko banta sakata hai aura isa taraha karake abhibhavaka ko bachchom ko anubhava karane ka eka vaikalpika tarika bata sakata hai|
ema bi ke ti takanika shishu ke satha surakshita lagava ko punah pane ke lie abhibhavaka ko usake apane bachapana ke anubhavom ke satha muddom ko sulajane mem madada karata hai|
eka bi ke ti bhavishya mem bachche ke manoroga vikriti ke vikasa ke khatare ko kama karane mem madada karata hai|
eka mahatvapurna sampratya hai ‘‘manovrittiyom ki yogyata’’|
manasikarana ki yogyata apane apa ko ya dusare vyaktiyom ke vyavahara ko kalpanika panasika sthitiyom me dekhane ki kshamata hai|
‘‘manasikarana ki yogyata’’ sampratyaya ke satha vibhinna prakara ki manasika prakriyaem bhumikaem nibhati haim|
nidaralainda saikoenalitika instityuta ena pi ai (uchcha saiko enalitika instityuta) 2003 se bachchom ke lie upachara ke eka prakara jise ‘‘mentaliserena bivorderende kindara therepi ’’ kahate haim, ke satha kama kara rahi hai|
‘‘mentaliserena bivorderende kindara therepi ’’ ka ‘‘bachchom ke lie unaki manasikarana ki yogyata ko badha़ane ka upachara’’ ke rupa mem anuvada kiya gaya hai|
yaha upachara phaॉneji (3) (2002) ke siddhanta para anshika rupa se adharita hai jisamem manasikarana (ya manahkarana) mukhya vishaya hai|
manasikara purna hota hai jaba do vikasatmaka sampratyayom ko milaya ja sakata hai|
visheshataya, samasanyojaka karyapranali aura abhinaya karya pranali|
eka balaka jo kevala samasanyojaka pranali ke anusara karya karata hai vastavikata aura kalpana ke bicha antara ka anubhava nahim karata hai|
balaka jo kevala abhinaya karyapranali mem karya karata hai ke lie vastavikata aura kalpana mem antara hota hai yadyapi ve eka dusare se alaga-alaga hote haim|
balaka jaba abhinaya kara raha hota hai vastavikata stara ke bare mem jagaruka nahim hota hai|
jaba donom karyapranaliyam juda़i hoti hai balaka ko janakari hoti hai ki vo abhinaya kara raha hai aura taba hama manasikarana ki bata karate hai|
manahkarana eka cheta ke satha-satha acheta prakriya hai jo logom ko yaha dekhane yogya banata hai ki svayam ki aura dusarom ki kriyaem evam vichara sammilita vicharom, iradom aura abhivrittiyom ke dvara prerita hote haim|
eka balaka ke svaniyantrita tantra ka vikasa aura manasikarana ki yogyata (jo ki jaba satha hote haim intaraparsanala intarapritetiva phakshana, ai ai epha) kahalate haim| apane prarambhika lagava sambandhom para adharita hote haim|
manasika aghata ki sthiti balaka ko manasika aghata se juda़e hue vyaktiyom ya ghatanaom se sambandhita bahuta darda bhari bhavanaom mem rahane ke lie prerita karati hai|
una bhavanaom ko samakalita karane ki ayogyata, karya-pranali jisamem balaka kriya karata hai, ko prabhavita kara sakati hai|
chinta ka bahuta adhika bahava, udaharana ke lie, balaka ko kayra karane ki samakalina karyapranali mem vapasa jane ke lie prerita kara sakata hai|
anya ghataka jo ki balaka ke vikasa mem nirnayaka haim aura abhibhavaka se lagava ki gunavatta para mukhyatah adharita bhi haim, ‘‘dhyana niyamana’’ aura ‘‘prabhava niyamana ’’|
jaba donom mem se kisi ka bhi niyamana paryapta nahim hai, donom ke bahava ko prabhavita karane ke lie eka upachara sahayaka ho sakata hai|
‘‘dhyana niyamana’’ mem arambhika bindu hai ki balaka samasanyojana pranali se abhi bhi karya kara raha hai jisaka artha hai yaham uparyapta prabhava pradarshana haim|
yaham mukhya uddeshya antarika duniya (bhavanaom, vicharom, ichchhaom aura avegom) ki ora balaka ka dhyana nirdeshita aura kendrita karana hai|
prabhava niyamana mem adhikatara chikitsiya sambandha ke antargata phaile hue prabhavom ko khola jata hai|
yaham mahatvapurna hai ki balaka apani svayam ki bhavanaom se gujarata hai aura manasika kalpana mem unhem pahachanata hai aura nirmita karata hai|
jaise yaha ‘manasikarana ki prakriya’ vikasita hoti hai, balaka natakiya karyapranali mem karya karata hai aura pratikatmaka kalpana ka prayoga karane yogya ho jata hai|
hastakshepa, manasika sthitiyom aura manasika prakriyaom ke bare mem chintana ko sudharane para adharita hote haim|
ema bi ke ti prati saptaha do ya pancha baithakom ke satha tivra svarupa ki taraha mana ja sakata hai|
pancha baithakem prati saptaha batachita aura khelom ka sanyojana hota hai|
balaka ki antarika duniya ke satha samparka banane ke lie chikitsaka balaka ke satha khelega aura batachita karega aura isalie sabhi anshom aura tukada़om sahi akara dega jinhem sanyojita hone ki avashyakata hai|
yadyapi ‘lagava siddhanta ‘adhunika manovijnana mem vistrita, gahari anusandhana dharaom me se eka ke satha samajika-samvegika vikasa dharaom mem se eka ke satha samajika-samvegika vikasa ka eka mukhya vaijnanika siddhanta bana chuka hai|
adhunika samaya taka lagava siddhanta, bahuta kama anubhavika samarthana vale siddhantom ki apeksha chikitsa ke lie bahuta kama prayoga mem lai gai hai|
lagava siddhanta anshika rupa se svayam bolabae ke dvara chikitsiya upayoga para kama dhyana dene ke karana aura anshika rupa se vyavasayiyom ke bicha ‘lagava’ shabda ke artha ka vistrita prayoga hone ke karana ho sakata hai|
samasyatmaka vali lagava dekhabhala purna antahkriyaom ko eka adhika uchita vikasatmaka marga mem badalane ke lie marvina evam anya (2002) dvara vikasita kiya gaya – yaha eka abhibhavaka shiksha aura manochikitsiya hastakshepa hai|
yaha kaha gaya he ki yaha spashtataya samakalina lagava aura samanurupa vikasatmaka siddhantom para adharita hai|
purvalekha ka uddeshya ina vicharom ko abhibhavakom ke lie upabhoktamaitripurna, samanyabodha phaishana mem prastuta karana hai ki ve bauddhika aura samvegika rupa se samaja sakem| 
yaha vuttakara rupa mem bachche ki avashyakatao aura lagava ke dhanga ke graphiya prastutikarana dvara kiya jata hai jisamem dekhabhalakarta dvara bachche ki avashyakataom aura surakshita ashraya ka sankshiptikarana hota hai|
abhi taka yaha purvalekha 4 varsha ki ayu taka ke purvavidyalayi bachchom para parikshita aura kendrita hai|
bachche ke usase alaga jakara khojabina karane aura vapasa arama aura shanti para vapasa ane se sambandhita sanketom ke lie dekhabhala karane valom ki samvedanashilata aura uchita pratikriyom ko badha़ane ke lie|
svayam ko pratibimbita ki aura lagava dekhabhalapurna antahkriya ke prati balaka ke vyavahara vichara aura bhavanaom ke prati yogyata ko badha़ane ke lie|
anake apane itihasa ke anubhavom ko pratibimbita karane ke lie jo unake vartamana dekhabhala ke svarupa ko prabhavita karata hai|
yaha bada ka bindu dekhabhalakarta ke bachava ke tarikom ke galata prayoga ko batane ka lakshya rakhata hai|
yaha eka prarambhika mulyankana hai jo ‘anajana paristhiti prakriya (ainsavartha 1978), avalokana, abhibhavaka vikasa sakshatkara ka prayoga karate hue eka vidiyo tepa sakshatkara (abera aura anya 1989) aura vayaska lagava sakshatkara (jiऑrja evam anya 1984) aura bachche ke bare mem dekhabhalakarta prashnavali ka upayoga karata hai|jiऑrja evam anya 1984) aura bachche ke bare mem dekhabhalakarta prashnavali ka upayoga karata hai|jiऑrja evam anya 1984) aura bachche ke bare mem dekhabhalakarta prashnavali ka upayoga karata hai|jiऑrja evam anya 1984) aura bachche ke bare mem dekhabhalakarta prashnavali ka upayoga karata hai|jiऑrja evam anya 1984) aura bachche ke bare mem dekhabhalakarta prashnavali ka upayoga karata hai|jiऑrja evam anya 1984) aura bachche ke bare mem dekhabhalakarta prashnavali ka upayoga karata hai|jiऑrja evam anya 1984) aura bachche ke bare mem dekhabhalakarta prashnavali ka upayoga karata hai|jiऑrja evam anya 1984) aura bachche ke bare mem dekhabhalakarta prashnavali ka upayoga karata hai|jiऑrja evam anya 1984) aura bachche ke bare mem dekhabhalakarta prashnavali ka upayoga karata hai|jiऑrja evam anya 1984) aura bachche ke bare mem dekhabhalakarta prashnavali ka upayoga karata hai|jiऑrja evam anya 1984) aura bachche ke bare mem dekhabhalakarta prashnavali ka upayoga karata hai|jiऑrja evam anya 1984) aura bachche ke bare mem dekhabhalakarta prashnavali ka upayoga karata hai|jiऑrja evam anya 1984) aura bachche ke bare mem dekhabhalakarta prashnavali ka upayoga karata hai|jiऑrja evam anya 1984) aura bachche ke bare mem dekhabhalakarta prashnavali ka upayoga karata hai|jiऑrja evam anya 1984) aura bachche ke bare mem dekhabhalakarta prashnavali ka upayoga karata hai|jiऑrja evam anya 1984) aura bachche ke bare mem dekhabhalakarta prashnavali ka upayoga karata hai|jiऑrja evam anya 1984) aura bachche ke bare mem dekhabhalakarta prashnavali ka upayoga karata hai|
bachche ki ‘lagava prarupa’ ya to ainsavartha ya pi e si (purva vidyalayi lagava vargikarana tantra) ka prayoga karate hue vargikrita kiya jata hai|
phira chikitsa ka pratyeka ‘lagava-dekhabhala svarupa’ yugma ke anusara vyaktigatakarana kiya jata hai|
karyakrama me jo 20 saptaha taka saptahika rupa se hota hai samuha-baithaka, vidiyo punarniveshana shabda-chitra aura mano-shaikshika aura chikitsiya vartalapa shamila hota hai|
dekhabhalakarta sikhate haim, samajate haim aura phira bachchom ke lagava vyavaharom aura unaki apani dekhabhalapurna pratikriyaom ke bare mem avalokana ki ora anumanika kaushalom ka abhyasa karate haim|
yu esa e mem suraksha ke chakra ka ‘heda starta’ /arli heda starta, karyakrama ke antargata karyakshetra parikshana kiya gaya|
vikasakom ke anusara karyakrama ka uddeshya eka siddhanta aura pramana adharita hastakshepa sanlekha vikasita karana hai jise vijnana adharita lagava prakriyaom mem prashikshita vyavasayikom ke madhya sajedari mem prayoga kiya ja sakata hai|
aisa bataya gaya hai ki 75 yugmom ke ankada़om ke vishleshana ke prarambhika parinama, avyavasthita se vyavasthita prarupa ki ora vastavika badalava aura surakshita lagava ke vargikarana mem badha़otari ka sanketa dete haim|
yaha eka madhyastha karyakrama hai jo shishuom para kendri hai jinhonne pratikula arambhika dekhabhala aura dekhabhala mem vighatana ka anubhava kiya hai|
‘madhyastha karyakrama protsahana dene vale dekhabhalakartaom ki isa tathya ko pahachanane mem vishesha sahayata pradana karata hai ki protsahana purna dekhabhala mem rakha gaya chhota balaka
madhyasthata karyakrama mukhya batom ka lakshya rakhta haih shishuom ka palana poshana karana jaba dekhabhalakarta palana poshana karane mem suvidha na mahasusa karate hom, shishu vyavaharom ke prakarom ki pratikriya ko dabane ki pravrittiyom|
madhyasthata karyakrama avashyaka rupa se pratinidhi dekhabhalakartaom ke lie eka prashikshana karyakrama hai|
pariposhana pradana karana jaba yaha svabhavika rupa se na a raha ho|
parinamom ke adhara para ki protsahana bachchom ke lagava anupatahina rupa se avyavasthita hane ki sammbhavana hoti hai aura dimaga ki eka anasulaji ya asvikrita avastha vali protsahana mataom ke bachche avyavasthita lagava vale hone ki sammbhavana hoti hai|
dojaera va anyom ki vyakhya hai ki protsahita bachchom ko apane lagava tantra ko vyavasthita karane mem kathinai hoti hai jaba taka ki unhem pariposhita protsahana dekhabhala na mile|
protsahana milane vale abhibhavakom ki sahayata hai yadi ve apane lagava stara ke bare mem svachalita na bhi hom to|
protsahana dekhabhala mem shishu aksara pariposhana prapta karane mem asaphala hote haim|
protsahana dekhabhala ka jukava shishuom ke vyavahara ke prakarom ki pratikriya ki ora hota hai|
yadi protsahita shishu eka bachavakari ya pratirodhatmaka tarike mem vyavahara karata hai, protsahana dekhabhala isa prakara kama kara sakati hai jaise ki shishu ko usaki avashyakata nahim hai|
uddeshya yaha hai ki protsahana dekhabhala kartaom ke eka palana-poshaka ke rupa mem karya karane ke lie prashikshana dena yadi sanketa dene vale shishu se dura hom taba bhi|
protsahana dekhabhala mem shishu aksara sharirika, vyavaharika aura samvegika stara para avyavasthita hote haim|
protsahana bachchem aksara tanava harmona kaॉrtisola ka asamanya utpadana darshate haim|
yaha nirdharita nahim hua hai ki ye bada ke vikarom ke badha़e hue khatare ke lie sarthaka hai ya nahim lekina vayaskom mem manorogom ke prakarom ke satha bahuta ninna va bahuta uchcha stara juda़e hue haim|
yaham uddeshya hai bachche ki agrata ka anusarana karane mem aura adhika anukula samajika sathi banane mem protsahana abhibhavakom ki sahayata karana|
protsahana dekhabhala mem shishu aksara darane vali paristhitiyom ka anubhava karate haim|
abhibhavakom ke karyom mem se eka hai bachchom ko vastavika ya anubhava kie jane vale khatarom se bachana|
yaha aksara protsahana bachchom ke lie barabada karane vala aura bahuta kharaba hota hai aura dekhabhalakarta svayam eka khatare ke rupa mem ho sakate haim|
mukhya udaharana haim bachche ko alaga le jane ke lie ya hata dene ke lie vyavahara para akasmika khatare|
khatare ki paristhitiyom ko anubhava karane vale balakom ki pratikriyaom ka prasara simita hota hai aura ve aksara sudhara ke tarike se alaga ho jate haim|
isake lie sambhavita sabuta protsahana bachchom mem avyavasthita lagava svarupa ki anupatahina sankhya ho sakati hai|
bachche para prabhava ko samajane mem sahayata karane ke dvara protsahana abhibhavakom mem khatare vale vyavahara ko kama karana uddeshya hai|
hastakshepa ke purva aura bada mem dekhabhala karta aura bachche ke vyavaharom ko jancha jata hai jaise ki bachche mem nada़i antahstravi kriyaom ka niyamana hota hai|
10 baithakom vala hastakshepa vyavasayi samajika karyakartaom dvara dekhabhalakartaom ke gharom mem kiya jata hai|
punarniveshana aura imanadari ke lie baithakom ka vidiyotepa banaya jata hai|
hala hi mem 200 protsahana parivarom ko shamila karate hue neshalana istityuta ऑpha mentala heltha ke sahayoga se yadrichchhita naidanika jancha mem hastakshepa ka mulyankana kiya gaya hai|
adhe shishu tulanatmaka hastakshepa ke rupa mem parivarika karyakrama ke lie vikasatmaka shiksha ko saumpe jate haim|
vikasaka svayam ingita karate hem ki ve dekhabhalakarta ke vadom ka parikshana nahim karate haim yadyapi ve kahate haim ki jaise ki ve balaka ke parinamom ke sambandha mem dekhabhalakartaom ke vadom ko eka alochanatmaka paksha ke rupa mem manate haim, yaha eka alochanatmaka truti ho bhi sakati hai ya nahim bhi ho sakati hai|
janmadata abhibhavakom ke lie eka parivartita rupantara arambha kiya gaya hai jo ki eka chhote samuha para parikshita kiya gaya hai|
yaha 1998 mem si echa jinaha aura je.e. leriyu dvara banaya gaya eka protsahana dekhabhala hastakshepa hai|
5 sala se kama unra ke bachchom jisake satha durvyavahara kiya gaya ho aura jinhem protsahana dekhabhala mem rakha gaya ho ki vikasatmaka aura svasthya avashyakataom ko batane ke lie yaha karyakrama banaya gaya hai|
baryakrama ko lusiyana ki rajya sarakara aura niji koshom ne punji sahayata di hai|
palana-poshana dekhabhala hastakshepa ka uddeshya bachchom aura palana-poshana kartaom ke madhya eka lagava vale sambandha banane ko sahara dena hai, phira bhi lagabhaga 12-18 mahinom ke bada lagabhaga adhe bachche akhirakara apane mam-bapa ke pasa lauta ate haim|
rupankaka, bachchom aura palana poshana kartaom (e bi si) ke madhya sambandhom ke vikasa ko age badha़ane ke lie mairi dojaira programa aura protsahita bachchom ke roga lakshana vijnana aura palana-poshana dekhabhala ke lagava stara ke madhya sambandha darshane vale asake karya ka vivarana dete haim|
karya ina parinamom para adharita hai ki bachche ka vivarana dene vale protsahana abhibhavakom ke gunatmaka guna aura bachche ke satha sambandha protsahana abhibhavakom ke satha gaharai se juda़e haim|
uddeshya tha ki protsahana dekhabhala ka eka hastakshepa ki taraha namuna banane ke lie karyakrama vikasita karana|
riektiva ataichamainta disaऑrdara ki sambhavanaom aura karanom aura ara e di aura anya manoroga vijnanom ke khatare vale karakom para adhunika, yadyapi simita, anusandhanom ko karane ka yaha eka sacheta prayasa hai|
dekhabhala mem ane ke turanta bada ‘protsahana dekhabhala’ mem bachchom ko tivra sahayata milati hai aura phira bahurupatmaka upachara pate haim|
protsahana dekhabhala ka mulyankana aupacharika taura para eka purva niyojita naidanika sakshatkara ka prayoga karake bhi kiya ja sakata hai jo vishesha taura para protsahana abhibhavakom ke lie bachche ke artha ko shamila karata hai|
pratyeka balaka ke lie ayu para adharita vyaktigata hastakshepa socha nikale gae haim, balaka palana-poshana dekhabhala para naidanika prastutikarana aura janakari apasa mem milate haim|
‘mulyankana dala’ hastakshepa pradana karane mem shamila rahata hai|
jo karyakrama chala rahe haim niyamita durabhasha aura jakara samparka banana kayama rakhate haim aura ‘protsahana abhibhavakom’ ke lie samarthana samuha haim|
isa sthiti ke dura hone ke pahale balaka ki apane abhibhavakom ke satha sambandha ki bigada़i hui pravriti|
nae lagava sambandhom ko sthapita karane mem sambandhom mem gambhira gada़badiyam mahatvapurna sahayogi karaka samaji jati haim|
manasika aura vastavika durvyavahara ke itihasa aura anya aparadhi kriyaem ama haim|
bachchom mem vikasatmaka vilamba ama haim aura niyantrana, samajika samvegika aura vikasatmaka samasyaom ki eka mahatvapurna shrrinkhala hoti hai|
bachcha sambandhom ko asangata aura vishvasaniya ki taraha mana sakata hai|
age, kathora aura asangata vyavahara ke bavaja़uda kai bachche apane abhibhavakom se juda़e rahate haim, nae lagava sambandhom ke vikasa ko jatila banate haim|
dhatri abhibhavaka svastha lagava sambandhom ke banane mem badha bhi utpanna karate haim| baॉlabae para adharita dekhabhala pradana karane vala tantra vayaskom mem jaiva vyavaharika tantra ki taraha bhi dekha jata hai jo ki ‘balaka lagava tantra’ ka sampuraka hai|
sabhi dhatri dekhabhalom mem yaha sudridha़ jaiviki vyavastha nahim hoti hai kyonki kai adhika lagava aura nukasana hone se darate haim, kai paisa kamane ke lie ise prabhavapurna tarike se kara rahe haim aura kuchha bachchom ko kshatigrasta ki taraha dekhate haim aura samvegatmaka rupa se dura aura kama sammilita ho sakate haim| 
hastakshepom mem shamila haim dhatri abhibhavakom ko samvegom ke niyamana mem bachche ki sahayata karana sikhane mem sahara dena, bachche ke tanava ke prati prabhavakari pratikriya dena aura bachche ke chinhom ko samajana sikhana|
vishesha rupa se aise bachchom ke sanketom ki taraha galata manobhava aksara unake bhayanaka, asangata aura sambhranta pichhale sambandhom ke parinama ke karana sambhrantapurna hote haim|
dhatri dekhabhala ko isa bata ki pahachana karana sikhaya jata hai ki aise bachchom ki vastavika avashyakata kya hai na ki vo kya sanketa dete haim apani avashyakata ko batane ke lie|
aise bachche uttejaka aura virodhatmaka vyavahara dikhate haim jo ki dekhabhala kartaom mem samanyatah asvikriti ki bhavanaom ko badha़a sakate haim|
asamajika bachche upekshita aura sahi mayane mem atmanirbhara ho sakate haim, aviveki bachche jaise ve haim usase adhika achchhi taraha niyantrita kie ja sakane valom ki taraha mane ja sakate haim|
dhatri dekhabhala se unaki avashyakataom aura pragati ke mulyankana ke lie niyamita rupa se samparka aura kulakata ki jati hai|
jaise ki 2005 mem 250 bachchom ne karyakrama mem bhaga liya|
2001 mem prakashita parinama ankada़e usi bachche ke apane abhibhavakom ke pasa vapasa ane para ‘durvyavahara uparadha-vyasana’ mem 68% ki kami ora usi maॉm ke bada ke bachche ke lie uparadha –vyasana mem 75% kami darshate haim|
lekhaka dava karate haim ki karyakrama dhatri abhibhavakom ki nae lagava banane mem hi sahayata nahim karata hai balki vapasa laute bachche ke dekhabhala mem na hone ke lambe samaya bada parivarika vikasa para mahatvapurna prabhava bhi dalata hai|
bachchom mem lagava ke lie sarvottama hastakshepa paddhiti para pramana adharita nishkarsha nikalane ki koshisha karane ke lie yaha ankada़om ko ekatra karane aura sanshleshita karane ka eka prayasa tha| 
abhibhavaka samvedanashilata aura shishu lagava suraksha para pratyeka hastakshepa prabhavashali hai|
hastakshepa karyakrama ka prakara ora samaya vishishtata rakhata hai|
hastakshepa karyakrama hamesha aura sarvabhaumika taura para prabhavashali hote haim|
abhibhavakom ki samvedanashilata mem parivartana lagava suraksha se yom hi juda़i hoti hai|
chunava mapadanda adhika se adhika sambhavita hastakshepa adhyayanom ko shamila karane ka irada rakhate hue bahuta vistrita the|
samvedanashila parinama 7,636 parivarom ko shamila karate hue 81 adhyayanom para adharita the|
lagava suraksha 29 adhyayanom aura 1,503 bhagidarom ko shamila karati hai|
prayoga mem lae gae mulyankana mapaka the ensavartha sensitiviti retinga, ensavartha evam anya (1974)
nishkarsha yaha tha ki matri samvedanashilata para puri taraha se vyavaharatmaka dhyana dene ke satha hastakshepa, na kevala matri samvedanashilata ko badha़ane mem balki bachchom ki lagava suraksha ko badha़ane mem bhi sabase adhika prabhavashali haim| 
hastakshepa prakriyaom jinake achchhe parinama samane ae haim ko samajane ke lie prayara aura glaisara dvara ti adhyayanom ko chuna gaya|
kohana evam anya (1999) ‘‘yaha hastakshepa samudayika svasthya sevaom ke lie bheji gai mataom aura shishuom ko shamila karata hai|’’
batai gai samasyaom mem khilana-pilana, sona, vyavaharika niyamana, matri avasada aura juda़ava ya lagava mem asaphalata mahasusa karana shamila haim|
yadrichchha rupa mem niyata kie gae niyantrana samuha ne manogatyamaka manochikitsa ka uttaradayitva liya hai|
yaha ‘shishu-abhibhavaka manochikitsa mem shishu eka prarambhaka rupa mem’ ki dharana para adharita hai|
adhe satra ke lie mam shishu ke satha ja़mina para niche baitha jati hai, usaka avalokana karati hai aura kevala shishu ke sutrapata para hi anih kriya karati haim|
vichara yaha hai ki yaha avalokanatmaka chintanashila mudra ko protsahana dene ke dvara matri samvedanashilata aura pratikriyashilata ko badha़ata hai, jaba sharirika rupa se bhi suganya hota hai|
shishuom ko bhi unaki mataom ke satha unake sambandha banane ka anubhava hota hai|
dusare adhe satra ke lie mam apane avalokana aura anubhavom ki charcha karati hai|
nigarani, pratiksha aura vismaya samuha mem shishu, manogatyamaka manochikitsa samuha mem shishuom ki apeksha eka surakshita ya sangathita lagava vargikarana mem jane ke lie sarthaka 
yaha bataya gaya hai ki yadyapi lagava (surakshita lagava ke lie agragayi) ke lie dekhabhalakartaom ki vishesha pratikriyaem nahim mapi gai thim|
‘‘mainipuleshana ऑpha sensitiva resapaॉnsivanesa’’ vana dena buma (1994) yaha hastakshepa chida़chida़e shishuom vyavaharatmaka paimane para sahayata die gae vali ninna samajika-arthika samuha mataom para kendrita hai|
yadrichchhika rupa se niyukta kiya gaya samuha ko 6 aura 9 mahinom ki ayu mem, shishu ke nakaratmaka aura sakaratmaka chinhom para matri pratikriyaom ke adhara para 3 upachara satra milate haim|
hastakshepa ensavarthasa sensitiva responsivanesa kamponenta para adharita tha arthata sanketa ko dekhana, sahi taraha se isaki vyakhya karana, uchita pratikriya ka chunava karana aura pratikriya ko prabhavashali rupa se karyanvita karana|
yaha paya gaya ki ina shishuom ne saikita, svasantosha aura rone mem kami vale niyantrita shishuom ki apeksha sarthaka rupa se uchcha praptanka pae|
sabhi matri ghatakom mem sudhara aya| usake bada 12 vem mahine para eka ‘ aparichita sthiti’ mulyankana kiya gaya jisane niyantrita samuha mem 75% ki tulana mem asurakshita ki taraha vargikrita kevala 38% pae gae|
‘‘maॉdiphaida intaraikshana gaidainsa’’ benaॉita evam anya (2001) – yaha hastakshepa embiensa (patipikala maitaranala biheviyara insatrumenta phaॉra asaisamenta enda klasiphikeshana) para mape gae dekhabhalakartaom ke aparyapta vyavaharom ko kama karane ka lakshya rakhata hai|
yaha aparyapta vyavaharom ko asangathita lagava ke lie yogadana dene vala mana jata hai|
khela adharita hastakshepa (ema ai ji) ki tulana ahara adharita hastakshepa ke vyavahara rupantarana ke satha ki gai|
ema ai ji samuha mem anuchita matri vyavaharom aura vighatita sampreshana mem sarthaka kami pai gai|
manochikitsa mem pravesha karane vale bachchem aksara parivara hinsa ya anya sadama grasta anubhavom ke samparka mem ate haim|
sadamagrasta bachchom ka upachara karana adhikatara chikitsakom ke lie pratinidhika sadama grastata ke karana, eka chunauti ho sakati hai|
asahayata ki anubhuti aura jatila naidanika rogiyom ko asadhya manane ki sva surakshatmaka pravritti se bachane ke lie vicharashila sarvekshana taka pahuncha ki avashyakata hai| 
purva anuchchheda mem batae gae jaiva rasayanika mapa yaham taka ki garbhavastha ke daurana adhika mahatvapurna ho sakate haim|
udarahana ke lie protina ki kami elbyumina sandrata mem giravata ko badha़a sakati hai jo ki sadharanatah rakta ke patalepana ke karana garbhavastha mem pai jati hai|
plaja़ma amino-anlaprarupa bhi jnanaprada ho sakata hai yaha anusandhana ka eka vishaya hai|
vartamana samaya mem ina mapom ko janasvasthya karyakramom ke lie upayukta nahim bataya ja sakata hai|
janma ke samaya sagarbhata ayu ke sapeksha mem shishu ka janmabhara, unchai aura sira ki golai nirdharaka prachala haim|
vikasashila jagata mem ninna arthika paristhitiyom mem janma-bhara yuropa aura uttari amarika ki apeksha kama haim|
yaha kevala ahara sambandhi kami para purnatah aropita nahim kiya ja sakata hai kyonki maleriya jaise sankramana janmabhara ko gambhira rupa se prabhavita kara sakate haim|
2.5 kilo se kama bhara ke satha janme bachchom ka anupata vishesha taura para sarthaka ankada़a hai kyonki navajata shishu aura shishu nrityudara isa bindu niche teja़i se badha़ti hai|
anusandhana karyakramom mem aparabhara bhi eka mahatvapurna mapa hai|
stanyakala ke daurana mata ke poshana stara ka mulyankana aura adhika kathina hai|
stanyakala ke daurana samanyatah vaja़na mem 570 grama/mahina (14) ki dara se kami ati hai lekina jaisa ki bhaga 3-2 mem vyakhya ki gai hai kapha़i upapachayi anukulana hote haim aura atyadhika bhara mem kami nahim dekhi gai hai jaba taka ki khana khane mem atyadhika kami na ho|
eka mahatvapurna mapa shishu ka dugdha arntagrahana hai aura vyaktigata stara para eka se pancha mahinom ki ayu ke bicha 500mili./24 ghante se niche ki matra ko nishchita rupa se aparyapta mana ja sakata hai|
650 mili. eka janasakhya aurata matra ko kama mana jaega| jaisa ki sharirabhara ke satha yaha anumanita hai ki ahara sambandhi kamiyom ke sabase kharaba prabhava ke viruddha uchita upapachayi anukulana dugdha apurti ko bachata hai|
eka stanapana karane vali mahila yaha sambhavita hai ki purna svasthya aura tandurusti ka mulyankana satha mem eka chusta jivana ke lie usaki kshamata ka sabase adhika prabhavita hone ki sammbhavana hai|
uchcha sambhavita mulya ke mapa ka eka antima prakara haih kriyatmaka kshamata ka mapa, sharirika kriya ka stara hone ka eka svabhavika udaraharana|
thaॉnsana (15) , skuli bachchom ko dudha dene ke prabhava ka parikshana karane ke lie 1920 mem baॉeda aura anya ke dvara sanyukta rashtra mem varnaniya khoja ki vyavastha ki gai|
vriddhi ki tivrata nishchita thi aura bachchom ka parikshana aura shikshakom ki rirpotom ne bataya ki dudha pie hue bachchom ki samanya sthiti mem sudhara aya tha aura adhika satarka ho gae aura niyantrita karane ke lie dusarom ki apeksha adhika tivra aura kathina hote haim|
sampurita ya asampurita bachchom ke bicha unchai vriddhi mem antara 6 mimi./varsha tha lekina kriya mem antara ke adhika mahatvapurna hone ki sambhavana hai|
anarthakari nahim lekina aparyapta urja lene vale bachchom mem kriyaom ke stara mem kami yuganda mem rahane vale gramina bachchom mem bhi batai gai hai|
gaॉtamala mem ai ena si e pi adhyayanom usi taraha yaha darshaya tha ki naitrojana santulana ya vriddhi ko prabhavita kie bina purva skuli bachchom ke urja arntagrahana ko 90 se 80 ki.kailori/kilograma se kama kiya ja sakata hai|
garbhavastha aura stanapana karane vali mahilaom ke lie kriya aura bhi adhika sangata kriyatmaka prachala hai kyonki visheshataura para ye vikasashila jagata ke gramina kshetrom mem shramika vyavasayiyom ke eka mahatvapurna ghataka ko darshate haim|
jaisaki 1.17 vibhaga mem vivechana kiya gaya hai shrilanka mem adhyayanom ne darshaya hai ki lauha tatva apurti karya utpadana ko badha़ane yogya banati hai|
karya kshamata aura urja utpadana mem khejabina ko kalpanashilata ora lachilepana ke satha taiyara karane ki avashyakata hai|
eka stri ke lie usi karya ko ninnatama bahari prayasa ke satha ya adhika ke satha pura karana sambhava hai|
yaha sujava dena aviveki nahim hai ki karya karane ki bada vali shaili tandurusti ka uchcha bhava aura samanya prasannata pradana karata hai|
jambiya mem garbhavati aura stanapana karane vali mataom dvara viskuta apurti ke prachalana ke lie bataya gaya prathama sarvamata karana hai ki isane unhem karya ke lie adhika shakti di|
abhi taka jisa prakara koi pramana nahim hai ki ve adhika vastavika karya kara rahi thim, kevala yaha nishkarsha nikala ja sakata hai ki isa kathana ka artha hai ki mataem uchchatama sima ki apeksha apani kshamata ke andara achchha karya karane ke lie adhika yogya thim|
yaha kapha़i spashta hai ki kriyatmaka parikshanom ka prayoga eka vishaya hai jo anusandhana ke adhika bada़e nivesha ko labha pahunchata hai|
unaki antarika manovaijnanika ruchi se kapha़i alaga, yaha poshana ke hastakshepa karyakramom ke labhom ko , rashtriya vikasa yojana ke lie tarkasangata rupa mem anuvadita karane ke lie yogya banaega|
unako rajanitijnom aura sarakari adhikariyom dvara chikitsiya aura jaiva rasayana mapakom ki apeksha jinaki sarthakata kama svabhavika hai, adhika gambhirata se liya ja sakata hai|
vivechita sabhi prakara ke sanketaka upayogi ho sakate haim jaba taka ki unaki simaom ko dhyana mem rakha gaya|
unako poshana aura svasthya para hastakshepa ke prabhava ki jancha karane ke svabhavika pratyaksha uddeshya ke lie lekina samajika vikasa ke apratyaksha sanketaka ki taraha bhi prayoga mem laya ja sakata hai|
bada ke upayoga ka eka udaharana hai eka vrihada stariya sanghatita gramina vikasa karyakrama ko hala hi mem nepala mem britisha sarakara dvara di gai punji|
nepala me krishi, pashupalana, vaniki, sinchai aura shiksha satha hi satha svasthya se nivesha haim|
isa uddeshya ke lie poshana stara suchakom ka prayoga karane ke sampratyaya ko utsahapurvaka svikara kiya gaya hai|
yadyapi isa utsaha ko banae rakhane ke lie aura donom deshom ki sarakarom ko arthapurna janakari pradana karane ke lie|
eka upayukta pranali vijnana, jisamem ankada़e eka avirala rupa se ekatrita kie jaem, ko sthapita karane ke lie yaha avashyaka hoga|
samana rupa se mahatvapurna, jisake parinamom ka vishleshana kiya jaega aura turanta uplabdha karae jaengem jisase jaba avashyaka ho pure karyakrama mem avashyaka parivartana lae ja sakem|
nrityudara ankada़om ki vyakhya karane mem nrityu ke karanom ko samajana svabhavika rupa se mahatvapurna hai|
gaॉtamala mem sadrisha ankada़e kevala 25 pratishata ke lagabhaga the|
svasthya aura buddhimatta manava tandurusti ke do ghanishtha sambandhi paksha haim|
ai kyu praptankom aura sanjnanatmaka yogyata ke anya mapom mem manava samuha antarom ko samajane mem buddhimatta para svasthya ka prabhava sabase mahatvapurna karakom mem se eka hai|
kai karakom ke karana sarthaka sanjnanatmaka apurnata ho sakati hai visheshataura para jaba ve garbhavastha aura balyavastha mem hote haim jaba mastishka vriddhi kara raha ho aura rakta-mastishka badha kama prabhavashali ho|
ye apurnata kai bara sthai, kai bara anshika ho sakati hai ya bada ki vriddhi ke dvara purna rupa se kshatipurti ho sakati hai|
vikasita deshom ne kai sanjnanatmaka kriyaom ko prabhavita karane ke lie jane jate poshaka tatvom aura jiva visha se sambandhita kai svasthya nitiyom ko lagu kiya hai|
inamem kuchha khadya utpadom ke sudradha़ikarana ke lie avashyaka niyama aura pradushakom (uda. sisa, para aura karbanika kloraida) ke surakshita stara sthapita karane ke lie niyama shamila haim|
bachchom mem sanjnanatmaka apurnata ki kami ka lakshya rakhate hue vyapaka niti sujava batae gae haim|
kuposhana vriddhi ke kai vibhinna kalom jaise ki garbhavastha, stanapana ke daurana, shaishavavastha ya balyavastha ke daurana utpanna ho sakata hai|
yaha vibhinna poshaka tatvom jaise ki sukshma poshaka tatva, protina ya urja ki kami ke karana bhi ho sakata hai|
yadyapi kuchha avalokanakarta tarka karate haim ki jivana ke prathama chhah mahine isa mayane mem sabase sankatapurna hai ki kuposhana usa kala mem bada ke jivana mem kuposhana ki apeksha sanjnanatmaka vikasa ko adhika nukasana pahunchata hai|
philipainsa se eka adhyayana tarka karata hai ki kuposhana ka jivana ke dusare varsha mem jivana ke prathama varsha mem kuposhana ki apeksha bada़a nakaratmaka prabhava ho sakata hai|
garbhavastha ke daurana alpaposhana aura anya karaka intraॉtaraina grotha ritardeshana (ai yu ji ara) paida kara sakate haim jo ki nyuna janma bhara ka eka karana hai|
yadyapi, yaha sujava diya gaya hai ki ai yu ji ara mem mastishka chayanita tarike se aparyapta ho sakata hai|
mastishka vriddhi samanyatah sampurna sharira bhara ya lambai ki apeksha kama prabhavita hota hai|
vikasita deshom se kai adhyayanom ne paya hai ki mastishka vriddhi aura alpa aghatom ko bhi prabhavita karate hue uchcha intrayutarina grotha ritaradeshana ke apavada ke satha ai yu ji ara manasika pradarshana aura kishoravastha ya vayaskavastha mem vyavahara para kama ya amapaniya prabhava dalata hai|
udaharana ke lie 1944 ke dacha akala ke daurana kuchha mahinom ke lie tivra alpaposhana ke karana kuchha kshetrom mem ausata janma bhara mem kami ai|
yaha bada mem niyantrita kshetrom ki apeksha ina kshetrom se 18-19 varsha ayu ke dacha purusha dastom ke lie ai kayu parikshana ke pradarshana mem parivartana ke satha sambandhita nahim tha|
yaha vyakti purva prasava avastha mem akala ke samparka mem ae the lekina janma ke bada nahim| akala ke daurana ninna esa i esa valom mem janma kama hue jabaki akala ke bada ninna esa i esa valom mem janmom mem kshatipurti vali vriddhi thi| 
kyonki esa i esa ai kyu se sambandhita hota hai isamem kuposhana ke karana chhupa hua eka prabhava ho sakata hai|
eka purana vishvasa ki bachchom ke ai kyu mem vriddhi stanapana se juda़i hoti hai, 2006 mem britisha medikala janarala mem prakashita eka lekha mem chunauti mili thi|
parinamom ne darshaya ki stanapana nahim maॉm ka ai kyu, bachchom ke ai kyu praptakom mem antara batata hai|
isa adhyayana ke parinamom ne tarka kiya ki pahale adhyayanom ne maॉm ke ai kyu ko dhyana mem nahim rakha|
kyonki mam ka ai kyu isa bata ka bhavishya suchaka tha ki bachche ko stanapana karaya gaya ya nahim, adhyayana ne nishkarsha nikala ki ``stanapana (svayam mem) ka bachchom mem buddhimatta para bahuta kama ya koi prabhava nahim hota hai|
isake bajaya, mam ke ai kyu ka usake bachche ke ai kyu ke satha sarthaka sahasambandha tha, chahe bachche ko stanapana karaya gaya ho ya stanapana nahim karaya gaya hai|
anya adhyayana ne abhibhavaka ke ai kyu ko niyantrita karane ke bada bhi stanapana ka sakaratmaka prabhava paya|
anya adhyayana ne paya ki stanapana ai kyu mem 8.3 anka badha़ata hai|
eka adhyayana darshata hai ki yadi shishu mem epha e di esa tu jina ka ‘si’ rupantara tha, stanapana ai kyu mem 7 anka badha़a sakata hai|
unako jinaka ‘G’ rupantara hota hai ai kyu ka koi labha nahim milata hai|
chili mem 18 parshiya hai skula snatakom para do adhyayanom ne paya ki prathama varsha ke daurana poshaka stara ne ai kyu, skula upalabdhi aura mastishka ayatana ko prabhavita kiya|
sukshmaposhaka tatvom ki kamiyam (uda.ayodina aura loha) buddhimatta ko prabhavita karati hai aura vikasashila jagata mem eka samasya hai|
sukshma poshaka tatva sampurana ki upalabdhata ko badha़ane ke lie niti sujava banae gae haim aura buddhimatta sambandhi vikasatmaka samasyaom ka virodha karane ki sambhavanaom mem sahi sabita hue haim|
udarahana ke lie kopena haigana sarva sammati batati hai ki ayodina aura loha tatva donom ki kami apurna mastishka vikasa se sambandhita hai aura yaha bahuta bada़i sankhya mem logom ko prabhavita kara sakati hai|
yaha anumanita hai ki 2 araba logom (kula vishva janasankhya ka eka tihai) 285 miliyana 6 se 12 varsha ki ayu ke bachchom ko shamila karate hue ayodina ki kami dvara prabhavita hote haim|
vikasashila deshom mem, yaha anumanita hai ki chara varsha ya usase kama ayu ke bachchom ka 40% anake ahara mem aparyapta loha tatva ke karana rakta alpata se grasita haim|
vitamina aura khanijom ki kami para sammilita kathana kahata hai ki ina kamiyom ki tivrata ka artha hai ‘‘ badha़te hue sau karoda़om mastishkom ki apurnata aura rashtriya ai kyu mem kami’’|
kulamilakara, pahale se loha tatva ki kami vale bachchom mem sanjnanatmaka kriya ko loha tatva apurti se sudhara ja sakata hai ya nahim anusandhana kara rahe adhyayanom ne mishrita parinama die haim kyonki sambhava hai sankatagrasta vriddhi kalom mem abhava aparivartaniya kshati ka karana bane|
adhika achchhe prarupa ke satha kai adhyayanom ne vastavika labhom ko darshaya hai|
lauha tatva ki kami ko rokane ke lie eka vikalpa hai vishishta apurti dena udaharana ke lie goliyam
yadyapi yaha manhaga hai, vitarana vyavasthaem aksara aprabhavashali hoti hai aura anupalana ninna hai|
bada़e paimane para sukshma poshaka tatvom ke vitarana ke lie mukhya bhojana (anaja, ata, chini, namaka) ka sudridha़ikarana sabase adhika sthai aura pradana kiya ja sakane vala vikalpa hai phira bhi sarakarom aura khadya udyoga ki ora se sahamati ki avashyakata hai|
vikasita desha kai khadya padarthom ko kai sukshma poshaka tatvom dvara apurita karate haim|
vikasita jagata mem bhi atirikta vitamina khanija apurti ka prabhava ho sakata hai|
eka adhyayana ne ‘‘kamakaji varga’’ pahale hisapenika ki apurti karate hue, 3 mahinom ke lie sanyukta rashtra mem 6-12 varsha ki ayu ke bachchom mem 2 se 3 ai kyu paॉintsa mem ausata vriddhi pai|
yaha adhikatara bachchom ke bahuta bada़e upa samuha mem bahuta jyada badha़otari dvara samajaya ja sakata hai, sambhavatah kyonki ye adhikansha ki taraha paryapta rupa se poshita nahim hote|
adhyayana batata hai ki skula ke bachchom ke abhibhavakom ko shaikshika pradarshana nishkrita hai, eka sambhavita nidanashastra ki taraha unake bachchom ke poshana stara ke mulyankana ke lie poshana ki ora abhimukha hue chikitsakom ke pasa jane ke lie sahi salaha milani chahie
adhika anumanatah anya poshaka tatva bhavishya mem mahatvapurna sabita ho sakate haim|
eka adhyayana mem garbhavati aura dudha pilane vali mataom ko machhali ke tela ke sampuraka ko badha़i hui sanjnanatmaka yogyata se sambandhita paya gaya hai|
vriddhavastha mem vitamina B12 aura pholeta sanjnanatmaka kriyaom ke lie mahatvapurna ho sakata hai|
anya adhyayana ne paya ki garbhavati mahilaom jinhonne prati saptaha vasa anlom ke satha 340 grama pora yukta machhali ka sevana kiya hai ko phayada hai ki para vishaktata ke khatare ka prabhavashali hona|
unamem kama maukhika ai kyu, gatyatmaka samanvaya aura vyavaharatmaka samasyaom vale bachche hone ki kama sambhavana hai|
yadyapi, pare ki uchcha matra vale ahara jaise ki sharka, svaॉrdaphisha, kingamaikerala aura tailaphisha manasika mandata paida kara sakate haim|
vikasashila desha gaॉtemala se eka adhyayana ne paya ki shaishavavastha mem kama janma bhara ki apeksha ninna vriddhi sanjnanatmaka aura upalabdhi parikshanom mem kishora pradarshana nakaratmaka rupa se sambandhita the|
bada mem isase sambandhita lambi avadhi ke adhyayana ne gaॉtamala mem eka uchcha protina-urja vale peya ko ahara ke eka sampuraka ki taraha dene ke prabhava ko 6-24 mahinom ki ayu ke bachchom mem dekha|
skula hajiri ki sambhavanaom ki vriddhi aura prathama kaksha mem uttirna hone, 13 varsha valom dvara shreni ki vriddhi, vidyalayi upalabdhi purna karane ki vriddhi aura 25-40 varsha vale vayaskom ke ai kyu parikshana akom ki vriddhi mem eka sarthaka sakaratmaka aura kapha़i sthai prabhava paya gaya|
vikasashila jagata mem 5 ki ayu se kama vale 31% bachche samanya rupa se (ayu ke lie unchai manaka vichalana rinatmaka 2 se kama) ya tivra avikasita (rinatmaka 3 manaka vichalana se kama) the|
yaham taka ki vyapakata pahale bhi bahuta adhika thi jabaki avaruddhata ki vishvavyapi vyapakata prati varsha pratishata bindu se lagabhaga adhi taka kama ho rahi thi|
jamaika mem 9-24 mahinom ki ayu vale ruki hui vriddhi vale bachchom para kie gae adhyayana ne paya ki jaba 17-18 varsha ki ayu ke the unhonne sanjnanatmaka aura shaikshika parikshanom aura mano samajika kriyaom para vikasita samuha ki apeksha sarthaka rupa se kama se kama anka prapta kie|
pahale se avikasita ina bachchom ko poshana apurti dene ka (prati saptaha 1 krigra dudha adharita nuskha) bada ke praptanko para koi sarthaka prabhava nahim pada़a lekina manosamajika utprerana (mam aura bachche ke si prati saptaha khela satra) ka sakaratmaka prabhava pada़a|
kuchha jiva-visha jaise ki sisa, para, arsenika, taluina aura pi si bi tantrika vikasatmaka vikara ke jane mane karana haim|
ina khatarom ki pahachana ne bachava ke pramana-adharita karyakramom ko protsahana diya jaise ki petrola mem sisa sanyoji padarthom ka nishkasana|
yadyapi yaha bachava abhiyana kaphi sapha़la hue haim, bahuta se vastavika deri ke bada hi shuru hue|
sisa prabandhana ki nitiyam deshom ke madhya antara rakhati haim vishesha rupa se vikasita aura vikasashila jagata mem|
adhikatara vikasita deshom mem sisayukta gaisolina ka prayoga kama ya nishkasita ho gaya hai aura yu esa bachchom mem sise ke stara, sisa nyunikarana se sambandhita nitiyom dvara sarthaka rupa se kama ho gaya hai|
yaham taka ki 24 mahinom ki ayu ke asapasa uchcha sisa stara 10 varsha ki ayu mem bauddhika aura shaikshika pradarshana kamiyom se sambandhita haim|
kuchha, kama se kama pahale kaphi prayoga mem lae jane vale karbanika kloraidsa jaise ki daiऑksinasa di di ti aura pi si bi sanjnanatmaka kamiyom se juda़e hue haim|
eka lainsaita punaravalokana ne manava vayasakom mem roga-vishayaka tantriyavishakta prabhavom ko utpanna karane ki yogyata vale 201 rasayanom ko pahachana hai, jaisa ki sathi samuha sarvekshana vale vaijnanika sahitya mem bataya gaya hai|
kai atirikta rasayanom ko prayogashalayi namunom mem tantriyavishakta hona darshaya gaya hai|
lekha batata hai ki bachche adhika samvedanashila hote haim aura tarka kahata hai ki nae bachavakari tarikom ki jo vikasita ho rahe mastishka ki vishesha atisamvedana shilata ko pahachanate haim, prayoga para pratibandha shuru karane se pahale purva vastavikata se bachane ke lie rasayanom ke parikshana aura niyantrana ke lie avashyakata hai|
isake atirikta parishishta mem suchibaddha kie hue audyogika rasayana tantrika visha ki taraha mane gae haim|
vartamana kainabisa prayoga, khuraka nirbhara vyavahara mem ai kyu praptako mem kami ke satha sarthaka rupa se saha sambandhita tha|
yadyapi, isa prakara ki giravata rogiyom mem nahim dekhi gai jo aupacharika taura para kainabisa upabhokta the aura davai lena banda kara diya tha|
lekhakom ne nishkarsha nikala ki kainabisa ka buddhimatta para lambe samaya taka prabhava nahim hota hai|
jaba tambaku ya madira ke prayoga ke viparita prabhavom ke kapha़i likhita pramanom se tulana karem to bhruna para prabhava sabase kama hota hai|
bhruna madira prabhavana, jo bhruna madira sanlakshana ka karana banata hai, pashchimi jagata mem manasika avaruddhata ke pramukha karanom mem se eka hai|
garbhavastha ke daurana mata ka tambaku dhunrapana badha़i hui kriyaom, ninna dhyanakendrikarana aura ghati hui bauddhika yogyataom se sambandhita hai|
yadyapi eka adhunika adhyayana pata hai ki matri tambaku dhunrapana ka bachche ke ai kyu para koi prerita karane vala sidha prabhava nahim hota hai|
ai kyu aura shiksha dvara mapi gai matri bauddhika yogyata ke lie samayojana ne kama ai kyu aura tambaku dhunrapana ke madhya sahasambandha ko dura kiya|
anya adhyayana ne vatavaraniya tambaku dhunrapana pradarshana ke madhya sambandha dekhane ke bajaya, 
adhyayana ko linga, jati, pradesha, garibi, abhibhavaka shiksha aura vaivahika sthiti, phairitina aura rakta sisa sandrata ke prati niyantrita kiya gaya|
tvacha ko gore karane vale utpadom ka, samanyatah eshiyai mahilaom mem prachalita, ai kyu ke prati hanikaraka prabhava hota hai|
tvacha ko gore karane vale utpadom mem aksara sakriya ghatakom ki taraha vishaile rasayana hote haim jaise ki para aura haidrokyunaॉna
tvacha ko gora karane vale utpadom ke eka bada़e bhaga ki ina vishaile rasayanom ki upasthiti ke karana, visheshataya eshiya ke charom aura dukanom mem prasiddha bikane vale, kaiyom dvara alochana hui hai jaise ki epha di e|
jaba lagae jate haim, para aura haidrokyunaॉna tvacha ke dvara raktapravaha mem avashoshita hote haim|
adhyayanom ne darshaya hai ki rakta mem pare ki matra mem 100 maikrograma ki badha़otari bachchom mem ai kyu ko ausatana 14 ankom se badha़ati hai|
pare ki vishaktata aura haidrokyunaina ki vishaktata ka prabhava jaise ki manasika aura sharirika vikara, para yukta aura haidrokyunaina yukta kantivardhaka utpadom ke prayoga ka parinama hota |
tvacha ko gora karane ke utpadom ka prayoga visheshataya hanga kanga mem prachalita hai|
yadyapi vaham bikane vale adhikatara utpadom mem isake sakriya ghataka ki taraha pare ki matra aksara svikrita matra se bahuta bada़i matra mem 27,000 se 60,000 guna hote haim|
tvacha ko gora karane vale utpada jo adhikatara eshiyai deshom mem bikate haim, para ya haidrokyuinona yukta hote haim, garbhavati mahilaom mem garbhastha shishu ke mastishka vikasa ke lie bahuta hanikaraka bhi hote haim|
garbhavastha aura bachche ke janma ke samaya svasthya dekhabhala, jisako pahunchana aksara niti dvara sanchalita kiya jata hai, sanjnanatmaka vikasa ko bhi prabhavita karata hai|
bachchom mem kama buddhimatta ke bachavakari karanom mem shamila haim sankramaka roga jaise ki tanika shodha parajivi aura mastishkiya maleriya, purva prasava davaiyam aura madira prabhavana, navajata shvasavarodhana, kama janmabhara sira ki chotem aura antahstravi vikara|
eka pratyaksha niti ka balyavastha ki sanjnanatmaka yogyata ke nirdharakom para dhyana dene para bala diya|
eka adhunika siddhanta batata hai ki purva balyavastha tanava vikasita ho rahe mastishka para prabhava dala sakata hai aura nakaratmaka prabhava paida kara sakata hai|
balyavastha mem hinsa ke samparka mem ana sabhi jatiyom ke bachchom mem vidyalayi shreni mem kami aura kama ai kyu se sambandhita rahi hai|
bada़i matra mem aphrika amerika ke prathama shreni ke shahari bachchom aura unake dekhabhala kartaom ka mulyankana sva-vivarana, sakshatkara aura ai kyu parikshana ko shamila karate hue pramanika parikshanom dvara kiya gaya|
adhyayana ne bataya ki chhote bachchom mem hinsa aura sadama sambandhita dukha se samana ai kyu aura pathana upalabdhi se sambandhita the|
hinsa ya sadama se samparka ai kyu mem 7.5 bindu (esa di 0.5) ki kami aura pathana upalabdhi mem 9.8 bindu (esa di 0.66) ki kami ki ora le gaya hai|
hinsa ka ai kyu para nakaratmaka prabhava ho sakata hai ya ai kayu hinsa ke viruddha rakshatmaka ho sakata hai|
akasmika prakriya aura karana-karya sambandha ki disha anabhijna hai|
ada़osa-pada़osa ke khatare ko 2006 se eka anya adhyayana mem aphrika-amerika kishorom ke ninna skula shreniyom se joda़a gaya hai|
roga ka samana hone para maleriya prati varsha aphrika mem 300-500 miliyana vyaktiyom ko adhikatara pancha varsha se kama ayu ke bachchom ko prabhavita karata hai|
hipetaitisa bi ka vayarasa manushyom ko shamila karate hue homonoida ke yakrita ko prabhavita karata hai aura hipetaitisa namaka pradaha ko paida karata hai|
hipetaitisa bi vayarasa eka di ena e vayarasa hai aura kai asambandhita vayarasom jo vayarala hipetaitisa utpanna karate haim mem se eka hai|
roga ko mula rupa se ‘‘sirama hipaitaitisa’’ ki taraha jana jata tha aura eshiya aura aphrika ke bhagom mem mahamari ka karana bana|
hipetaitisa bi china aura eshiya ke anya bhagom mem sthanika hai|
vararasa se sankramita vishva janasankhya ka anupata 3 se 6% para anumanita kiya gaya hai lekina tisare taka para prabhava ho chuka hai|
vayarasa ke karana paida hue gambhira bimari ke lakshanom mem yakrita pradaha, ulti, piliya aura kabhi kabhi nrityu shamila haim|
purana hipeaitisa bi antatah yakrita ka sirosisa aura yakrita kainsara vartamana kimothairepi ke prati bahuta kama pratikriya karane vali eka ghataka bimari utpanna kara sakata hai|
hipetaitisa bi vayarasa ka sankramana gambhira (sva niyantrita) ya purana (lambe samaya taka rahane vala) ho sakata hai|
sva-niyantrita sankramana vale vyakti saptahom se mahinom ke andara svatah hi sankramana dura kara lete haim|
sankramana ko dura karane ki sambhavana vayaskom ki apeksha bachchom mem kama hoti hai|
95% se adhika vyakti jo eka vayaska ya bada़e bachchom ki taraha sankramana grasita haim purna punalabhi ke mukama ko prapta karenge aura vayarasa ke prati sanrakshatmaka pratirashrana vikasita karenge|
yadyapi, kevala navajata shishuom ke kevala 5% jinhonne janma ke samaya apani mam se sankramana liya hai sankramana dura karenge|
eka se chhah ki ayu ke madhya vo loga jo sankramita haim 70% sankramana dura karenge|
hipetaitisa bi vayarasa ke satha tivra sankramana tivra vayarala hipetaitisa ke satha sambandhita hai eka bimari jo samanya bure svasthya bhukha mem kami, ji michalana, ulti badana darda, halke bukhara, gahare mutra ke satha shuru hoti hai aura phira piliya ke vikasa mi ora badha़ti hai|
isa bata para dhyana diya gaya hai ki khujalidara tvacha sabhi hipetaitisa vayarasa ke prakarom ke sambhavita lakshana ka eka sanketa raha hai|
adhikansha sankramita vyaktiyom mem bimari kuchha saptahom taka chalati hai aura phira dhire-dhire sudharati hai|
kuchha rogiyom mem adhika tivra yakrita ki bimari (akasmata badha़ne vali yakrita kharabi) ho sakati hai aura isake parinama ke karana nrityu ho sakati hai|
sankramana purnatah alakshanika ho sakata hai aura bina pahachana ke ja sakata hai|
hipetaitisa bi vayarasa ka purana sankramana alakshanika ho sakati hai ya yakrita ke purane pradaha (purana hipetaitisa) ke satha juda़e ho sakate haim, kai varshom ki avadhi mem sirosisa
isa prakara ka sankramana natakiya rupa se hipetosenyulara karasinoma (yakrita ka kainsara) ki ghatana ko badha़ata hai|
purane samvahakom ko madira pana se bachane ke lie protsahita kiya gaya kyonki yaha sirosisa aura yakrita kainsara ke khatare ko badha़ata hai|
hipetaitisa di sankramana kevala hipetaitisa bi vale sahagami sankramana ke satha hi paida ho sakata hai kyonki hipetaitisa di ka vayarasa hipetaitisa bi vayarasa ki sataha entijana ko kaipasida banane ke lie prayoga mem lata hai|
hipetaitisa di vala sahasankramana yakrita sirosisa aura yakrita kainsara ke khatare ko badha़ata hai|
paॉliarateraitisa nodosa hipetaitisa bi sankramana vale logom mem adhika hota hai|
purane sankramana vale vyakti ke mem hipetaitisa bi vayarala entijansa aura rogapratikaraka dhundha ja sakata hai|
hipetaitisa bi vayarasa sankramana ki pahachana ke lie parikshana, esasesa kahalane vala, mem sirama ya rakta parikshana shamila karata hai jo ya to vayarala pratijanom (vayarasa dvara utpanna protina) ya mejabana dvara utpanna pratirakshaka ki pahachana karata hai|
hipetaitisa bi sataha pratijana (echa bija़ e ji) isa sankramana ki upasthiti ko pahachanane ke lie sabase adhika prayoga mem laya jata hai|
echa bija़ e ji sankramana ke daurana prakata hone vala sabase pahale pahachana gaya vayarala pratijana hai|
yadyapi, sankramana ke prarambh mem yaha pratijana upasthita nahim bhi ho sakata hai aura yaha sankramana mem bada mem bina pahachana ke raha sakata hai kyonki yaha mejabana dvara dura kara diya gaya hai|
sankramita viriyana vayarala jinoma ko lapete hue eka antarika ‘‘sata kana ’’ rakhata hai|
aiko sahedrana sata kana sat protina, vaikalpika rupa se hipetaitisa bi kora entijana ya echa bisi e ji, ki 180 ya 240 pratilipiyom se bana hota hai|
echa bija़ e ji ke prakata hone ke turanta bada dusara pratijana hipetaitisa bi i pratijana (echa bi i ji) ke nama vala prakata hoga|
paramparagata rupa se, mejabana ke sirama me echa bi i e ji ki upasthiti, vayarala pratikriti aura badha़i hui sankramakata ki adhika uchcha dara ke satha ambandhita hai|
eka sankramana ki prakritika prakriya ke daurana echa bi i e ji dura ho sakata hai aura turanta bada mem ‘i’ pratijana (prati-echa bi i) ke lie roga pratikaraka utpanna hote haim|
yaha rupantarana samanyatah vayarala pratikriti mem natakiya kami ke satha sambandhita hai|
yadi mejabana sankramana dura karane yogya hai, akhirakara echa bi esa e ji pahachanane yogya nahim rahega aura hipetaitisa bi sarphesa entijana aura kora entijana (enti- echa bija़ aura echa bija़ ai ji ji) ke lie ai ji ji roga pratikarakom dvara anusarana kiya jaega|
echa bija़ e ji ke lie nakaratmaka lekina enti-echa bija़ ke lie sakaratmaka vyakti ne ya to sankramana dura kara liya hai ya pahale se pratirakshita ho chuka hai|
vyakti jo echa bija़ e ji ke lie kama se kama chhah mahinom ke lie sakaratmaka rahate haim, hipetaitisa bi ke samvahaka mane jate haim|
vayarasa ke samvahakom ko purana hipetaitisa bi ho sakata hai, jo ki uchcha sirama elenaina eminotransaphareja़ aura yakrita ke prayaha, jaisa ki bayopsi dvara pata chalata hai, dvara pratibimbita hota hai|
samvahaka jo echa bi i e ji nakaratmaka stara mem sirorupantarita ho gae haim visheshatah unako, jo jinhonne vayaska ki taraha sankramana prapta kiya hai, bahuta kama vayarala gunana hota hai aura atah lambi avadhi jatilataom ya dusarom ko sankramana hastantarita karane ke khatare kama hote haim|
abhi hala mem, pi si ara parikshana rogiyom ke namunom mem vayarala nyuklika esida ki matra ko pahachanane aura napane ke lie vikasita kie gae haim|
pi si ara parikshana vayarala lodsa kahalate haim aura eka vyakti ke sankramana stara ke mulyankana aura upachara ka nirikshana karane ke lie prayoga mem lae jate haim|
sankramita rakta ya sharira ke tarala jinamem rakta hota hai, se samparka mem ane se hastantarana parinama|
hastantarana ki sambhava svarupom mem shamila haim (lekina ina taka simita nahim haim) asurakshita sambhoga rudhira adhana karana, sankramita suiyom aura pichakari aura janma ke daurana mam se bachche me ardhvavarti hastantarana|
hastakshepa ke bina, eka mam jo hipetaitisa bi sataha pratijana ke lie sakaratmaka hai, janma ke samaya apane bachchom ko sankramana dene ke 20% khatare ko pradana karati hai|
yaha khatara 90% taka uchcha hota hai sadi mam hipetaitisa bi i entijana ke lie bhi sakaratmaka hai|
echa bi vi sambhavatah kshatigrasta tvacha ke samparka ya strava ke satha shleshmika jilli ya echa bi vi vali lara ke dvara ghara ke andara parivara ke sadasyom ke bicha bhi hastantarita ho sakata hai|
vayaskom mem hipetaitisa bi ke riporta kie gae kama se kama 30% ko eka pahachane gae khatare ke karaka se sambandhita nahim kiya ja sakata|
hastantarana ka prathama tarika eka nirdishta kshetra mem purane echa bi vi sankramana ki vyapakata ko darshata hai|
kama vyapakata vale kshetrom jaise ki dvivaya sanyukta rajya aura pashchimi yuropa, jaham 2% se kama janasankhya ko purana sankramana haim|
injekshana davai ka durupayoga aura asurakshita sambhoga prathama tarike haim yadyapi anya karaka bhi mahatvapurna ho sakate haim|
samanya vyapakata vale kshetrom mem jisamem purvi yuropa, rusa aura japana shamila haim jaham janasankhya ke 2-7% ko purana sankramana hai|
hipetaitisa bi roga bachchom mem pradhana rupa se phaila hua hai|
adhika vyapakata vale kshetrom jaise china aura dakshina purvi eshiya mem bachche ke janma ke hastantarana bahuta samanya hai|
yadyapi uchcha sthana-parimita vale anya kshetrom jaise ki aphrika mem balyavastha ke daurana hastantarana eka mahatvapurna karaka hai
uchcha sthana-parimita kshetrom mem purane echa bi vi sankramana ki vyapakata kama se kama 8% hai| 
yakrita ka sutrana roga aura yakrita kainsara hipetaitisa bi ke parinamasvarupa ho sakata hai|
hipetaitisa bi vayarasa prathamika rupa se yakrita ke karyom ke satha yakrita koshikaom mem pratikriti, hipetosaita ki taraha jana mana jane vala, ke dvara hastakshepa karata hai| 
echa bi vi sankramana ke daurana, mejabana pratirakshana pratikriya hipetosailyulara kshati aura vayarasa ki nikasi donom kara sakata hai|
yadyapi ina prakriyaom mem janmajata pratirakshana pratikriya eka mahatvapurna bhumika nahim nibhati hai|
anukulita pratiraksha pratikriya, visheshataya vayarasa jatigata saitotaॉksika ti limphosaitsa (si dhi elsa) echa bi vi sankramana se sambandhita yakrita ki adhikansha kshati ke lie sahayoga deta hai|
sankramita koshikaom ko marane ke dvara aura jivanakshama hipetosaita se echa bi vi ko sapha़ karane mem sakshama entivayarala saitokinsa utpanna karane ke dvara si ti elsa vayarasa ko nishkasita karate haim|
yadyapi yakrita kshati, si ti elsa dvara prarambha aura maghyastha ki jati hai, pratijana avishishta pradaha koshikaem si ti ela prerita pratiraksha roga vijnana ko kadatara kara sakata hai|
sankramana ke kshetra mem sakriya hue bimbanu, yakrita mem si ti elsa ke ekatrikarana ko age badha़a sakata hai|
hipetaitisa bi sankramana ko samanyatah upachara ki avashyakata nahim hoti hai kyonki adhikasham vayaska sankramana svatah hi dura karate haim|
prarambhika entivayarala upachara kevala 1% se kama rogiyom mem avashyaka ho sakata hai, jinake sankramana ko bahuta tivra upachara milata hai (phulaminenta hipetaitisa) ya jo pratirodhana samajaute vale hote haim|
dusari tarapha, purane sankramana ka upachara sutrana roga aura yakrita kainsara ke khatare ko kama karane ke lie avashyaka ho sakata hai|
sthai uchcha sirama elaina eminotransapharensa, yakrita kshati ka nishana aura echa bi vi di ena e stara ke satha purane sankramana vale vyakti upachara ke pratyashi haim| 
yadyapi upalabdha davaiyom mem se koi bhi sankramana dura nahim kara sakata, ve vayarasa ko pratikriti banane se roka sakati haim aura yakrita kshati jaise ki sutrana roga aura yakrita kainsara ko rokati hai|
upachara mem entivayarala davaiyam jaise ki laimivudaina, ediphovira aura entekevira aura pratirakshana tantra niyantrana jaise ki intapheraॉna elpha़ashamila haim|
yadyapi dusarom ki apeksha kuchha vyaktiyom mem pratikriya ki adhika sambhavana hoti hai aura yaha sankramana karane vale vayarasa ke jinotaipa ke karana ya rogi ki anuvanshikata ke karana ho sakata hai|
upachara vayarala bhara (rakta me mapi gai vayarasa kanom ki matra) ko kama karake karya karata hai, jo ki badale mem yakrita me vayarasa ki pratikriti banane ko kama karata hai| 
29 marcha 2005 ko, yu esa phuda enda draga edaministreshana (epha di e) ne hipetaitisa bi ke upachara ke lie entekevira ko svikriti di| 25 pharavari 2005 ko i yu kamishana ne pegasisa ko svikriti di| 
27 aktubara, 2006 ko telabivudaina ne epha di e ki svikriti prapta ki|
telabivudaina yu esa me ‘taijeka’ aura yu esa ke bahara ‘sebivo’ ke tredamarka nama ke antargata bechi jati hai| 
hipetaitisa bi ki samvahaka ke lie mani jane vali mataom se janmem shishuom ko hipetaitisa bi vayarasa ke lie rogapratikarakom dvara thika kiya ja sakata hai|
jaba janma ke bada baraha ghantom andara tika lagaya jata hai, hipetaitisa bi ka hone ka khatara 90% kama ho jata hai|
yaha upachara eka mam ko asake bachche ko kushalapurvaka dudha pilane ki anumati deta hai|
hipetaitisa bi vayarasa sankramana se bachava ke lie kai tike vikasita kie gae hai|
ye, vayarala enavalapa protinsa (hipetaitisa bi sarphesa entijana ya echa bi ja़ e ji) mem se eka ke upayoga para nirbhara hote haim|
tika mulabhuta rupa se rogiyom jinhem purana hipetaitisa bi vayarasa sankramana tha se prapta plajma se taiyara kiya jata hai|
hala hi mem, yaha jya़adatara di ena e takaniki ka prayoga karake banaya gaya hai, yadyapi plajma se prapta kie gae tikom ka upayoga ho raha hai, do prakara ke tike samana rupa se prabhavashali aura surakshita haim|
echa bi vi kana aura sataha pratijana ko nikalane ka sarala tarika|
hipetaitisa bi vayarasa (echa bi vi) hipedana vayarasa parivara ka sadasya hai|
vayarasa kana (viriऑna) mem bahari vasayukta parata aura protina se bani aikosahaidrala nyukliyokaipasida kora hote haim|
nyukliyokaipasida vayarala di ena e aura eka di ena e paॉlimareja़ ko dhake hue hota hai jisaki viparita pratilipi banane ki kriya hoti hai|
bahari parata mem antah sthapita protina hote haim jo ki atisamvedanashila koshikaom mem vayarasa ko bandhana aura pravesha ko shamila karata hai|
vayarasa 42 mimi vyasa vale viriऑna ke satha sabase chhote pashu vayarasom mem se eka hai, lekina kendra ke bina tantumaya aura golakara sharirom ko shamila karate hue pliyomorphika svarupa upasthita hote haim| 
yaha kana sankramaka nahim hote haim aura vasa aura protina se bane hote haim jo viriऑna ki sataha ke bhaga ko banate haim jise satahi pratijana (echa bija़ e ji) kahate haim aura yaha vayarasa ke jivana chakra ke daurana bahulata mem utpanna hote haim|
echa bi vi ke jinoma golakara di ena e se bana hota hai lekina yaha asamanya hota hai kyonki di ena e puri taraha se dvi-lada़i vala nahim hota hai|
purna lambai vali lada़i ka eka kinara vayarala di ena e paulimaraja़ se juda़a hota hai|
jinoma 3020-3320 nyukliyotaida lamba (purna lambai vali lada़i ke lie) aura 1700-2800 nyukliyotaida lamba (chhoti lambai vali lada़i ke lie) hota hai|
nishedhatmaka –bodha (a sanketa ki) vayarala ema ara ena e ka sampuraka hota hai|
koshika ke sankramana ke turanta bada vayarala di ena e kendraka mem paya jata hai|
anshika dvi-lada़iya di ena e, (+) bodha vali lada़i se sampurana aura (-) bodha vali lada़i se protina paramanu aura (+) bodha vali se ara ena e ki sankshipta krama ko hatakara purna dvi-lada़iya mem vapasa ata hai|
(-) bodha lada़i ke antima chhora se asanketika adhara hata die jate haim aura antima chhora punah juda़ jate haim|
jinoma dvara kuta banae gae chara jnata jinsa haim jo si, eksa, pi aura esa kahalate haim|
sata protina jina si (echa bija़ e ji) dvara kutabaddha kiya jata hai aura isaka prathama kodona eka upari e yu ji prathama kodona, jisase purva sat protina utpanna hota hai, dvara prarambha kiya jata hai|
echa bi i e ji pri-kora protina ki protiyolaitika karyavahi dvara utpanna hota hai|
di ena e paॉlimareja़ ji ‘pi’ dvara kuta kiya jata hai 1jina esa vaha jina hai jo satahi pratijana (echa bija़ e ji) ke lie kutabaddha karata hai|
pure vishva mem 10.6 miliyana varshika nrityuom ka 73 pratishata 5 varsha se kama ayu ka hota hai|
sabhi balaka nrityuom ka adhe se adhika kuposhana, jo ki sharira pratirodhaka kshamata ko bimara hone ke lie kama kara deta hai, se juda़i hoti hai|
karoda़om adhika jivita rahate haim kevala ghatate hue bhavishya ka samana karane ke lie, apani purna kshamata ko vikasita karane mem asamartha hote haim|
yadi abhibhavaka aura dekhabhalakarta yaha samajem ki jaba bimari ka akramana hota hai to kya kiya jae aura khatare ke chinhom ko, jo yaha batate haim ki chikitsiya sahayata ki avashyakata hai, kaise pahachane to kai nrityuom se bacha ja sakata hai|
sarala shabdom mem jivana ke satya bachchom ke jivana aura svasthya ko bachane ke vyavaharika prabhavashali aura kama kharchile tarikom ke bare mem sabase adhika pramanika janakari dete haim|
echa i ti vi ka uddeshya svasthya shaikshika karyakramom ko sthapita aura protsahita karana hai jo ki svasthya aura jivana ki gunavatta ko sudharane ke lie tivra aura dirdhakalina kshamata-gathana pradana karega aura mataom aura samudayom ko unake svasthya stara ke upara adhika niyantrana pradana karega|
surakshita pani ke satha juda़e hue svasthya labha logom ko sarala, vahana ki ja sakane vali takanikiyom jaise ki klorinikarana, chhanana, saurya nisankramana aura anake gharom mem sudhara laya hua sangrahana, dvara prapta kie ja sakate haim|
vaishvika stara ke sarvekshanom ne bataya hai ki kevala hatha-dhona 43 pratishata taka dasta hone ki sambhavanaom ko kama karata hai|
rashtriya aura rajya sarakara ki sajedari se svasthya, pani, svasthyavijnana aura sapha़ai prabandha ke lakshya kie gae kshetrom me hama mataom aura gramina svasthya karyakartaom aura vibhinna prakara ke samudayika netaom ki sahayata ke lie karya karate haim|
dasta aura nirjalana ke karana balyavastha nrityu dara aura roga dara ko kama karane ke lie eka sanghatita prastava, maharashtra, bharata 2005 -2010
pratidina 6,000 se adhika 5 varsha se kama ayu ke bharatiya bachche kuposhana ya mukhya sukshma poshaka tatva jaise ki vitamina A , loha, ayodina, jinka ya pholika esida ki kami se marate haim|
kula milakara bharata mem 57 miliyana ya vishva ke 146 miliyana kuposhita bachchom ke eka tihai se adhika haim|
bharata ke tina se kama ayu ke 45.9 pratishata bachche kama vajana ke, 39 pratishata ruki hui vriddhi ke, 20 pratishata gambhira rupa se kuposhita haim|
80 pratishata raktaalpata vale haim jabaki 67 prati 1000 shishu nrityudara hai| bharata arthika piramida ke nichale stara para 440 miliyana loga durbala haim aura prati varsha vitamina/khanija tatvom ki kami ke karana 500,000 bachche vikrita paida hote haim|
kuposhana bachche ke sanjnatmaka, sharirika aura samvegatmaka vriddhi ko gambhira rupa se kama karata hai aura vayaska varshom mem usaki prajanana kshamata mem giravata vala prabhava dalata hai|
visheshajna doharate haim ki balaka kuposhana na kevala bharata ke roga bhara ke 22 pratishata aura bharata mem 2.3 miliyana bachchom ke 50% ke lie uttaradayi hai balki eka gambhira arthika khatara bhi hai|
matri evam shishu svasthya vibhaga yunaiteta steta dipartamenta ऑpha heltha enda hyumana sarviseja़ ke heltha risorsaja enda sarviseja़ edaministreshana ka eka bhaga hai|
parivara echa ara esa e ki ora ane vali pidha़iyom ke eka svastha shuruata aura svastha bhavishya ko surakshita karane ke lie dekhate haim|
echa ara esa e ka matri evam shishu svasthya vibhaga una karyakramom ko lagu karata hai jo prativarsha 32 miliyana se adhika mahilaom, shishuom aura bachchom ki seva karate haim|
60 pratishata ke lagabhaga yu. esa mahilaem, jo janma deti haim echa ara esa e dvara samarthana prapta karyakramom dvara sevaem prapta karati haim|
adhikasham ema si echa bi nidhi, niyama adharita prakhanda anudanaom ke dvara jisaka kula yoga vitta varsha 2007 mem " 566 miliyana tha, rajyom ko bheje jate haim|
yaha prakhanda anudana, avagamana aura rogi prabandhana sevaom, jo ki parivarom ki sulabha dekhabhala dvara sahayata karate haim ke satha atyavashyaka pratirakshanom aura navashishu jancha ko bhi sahayata deti hai|
rajya prakhanda anudana punji ka prayoga, vishesha svasthya avashyakata vale bachchom aura unake parivarom ke lie samudaya adharita, parivara kendrita vyavasthaom ko vikasita aura lagu karane ke lie bhi hota hai|
isake atirikta echa ara esa e 38 rajyom mem lagabhaga 100 svastha shuruata vale sthalom ko sahayata pradana karata hai|
kolambiya aura pyurato riko ke jile unchi shishu nrityu dara aura svasthya dekhabhala dene vale samudayom mem mahilaom ke lie samudaya adharita rogi prabandhana, avasada ki jancha karana aura shaikshika kriyaem pradana karate haim,
echa ara esae yunaiteda stetsa mem bachchom ke sharirika, vyavaharika aura samvegatmaka svasthya ke sarvekshana ankada़om ko bhi sangrahita karata hai|
prativarsha prakashanom jaise ki vumansa heltha yu esa e aura chailda heltha yu esa e ke dvara mahilaom aura bachchom ke svasthya ki janakari ka prachara karata hai|
echa ara esa e karyakramom ki eka vistrita shrankhala chalata hai jo rashtra ki matri evam shishu svasthya janasankhya ki avashyakataom ko batate haim|
isa janasankhya mem garbhavati mahilaem, mataem, shishu, bachche aura unake parivara aura vishesha svasthya dekhabhala ki avashyakata vale bachche shamila haim|
karyakramom mem se sabase bada़e ‘da metaranala enda chailda heltha sarviseja़ blaॉka granta’ mem steta phaॉrmula blaॉka grantsa, speshiyala projektsa ऑpha rijanala enda neshalana signiphikennsa (esa pi ara e ena esa) aura kanyuniti integreteda sarvisa sistansa projektsa shamila haim|
matri evam bala svasthya ko sambodhita karane ke lie sanghiya vachanabaddhata ko sabase pahale childransa byuro (1912 mem sthapita) se khoja ja sakata hai|
soshiyala sikyoriti ekta (1935 mem lagu hua) ka shirshaka ‘vi’ jo ki matri evam bala svasthya sevaom para kendrita hai|
shirshaka ‘vi’ ko 1981 mem blaॉka granta karyakrama mem parivartita kiya gaya|
metaranala enda chailda heltha sarviseja़ blaॉka programa rajyom ko shishu nrityu dara kama karane ke lie anudana pradana karata hai|
rajyom ko apani punji ka 30 pratishata vishesha svasthya dekhabhala avashyakataom ke satha bachchom para kharcha karana pada़ta hai|
yaha anudana karyakrama sanghiya aura rajya sajedari ka samarthana hai jo 2.3 miliyana se adhika mahilaom ko rikta sthana bharane vali matri svasthya sevaem aura 27.8 miliyana se adhika shishuom ko prathamika aura bachavakari dekhabhala pradana karati hai|
echa ara esa e, esa pi ara e ena esa aura kanyuniti integreteda sarvisa sistama ke karyakramom ke alava matri evam bala svasthya blaॉka anudana bhi pradana karata hai|
anudana aura anubandhom ke dvara esa pi ara e ena esa pariyojanaem matri evam bala svasthya sudhara ke lie prashikshana aura anusandhana ko bhi samarthana dete haim|
esa pi ara e ena esa karyakramom ke antargata heritebala disaऑrdara aura nyubaॉrnaskrininga prayasom ko bhi anshika anudana milata hai, jisamem eka paramarsha samiti aura anuvanshika sevaom ke lie kshetriya sahayogi anudana shamila hai|
‘heldi starta’ asadharana rupa uchcha shishu nrityu dara vale samudayom mem uchcha khatare vali garbhavati mahilaom, shishuom aura mataom ki avashyakataom ko pura karane vali sevaem pradana karata hai|
37 rajyom, kolambiya ke jile aura pyuretoriko mem samudayom mem jinhem ‘heldi starta’ dvara sevaem prapta hoti haim, bada़i alpasankhyaka janasankhya, beroja़gari aura garibi aura surakshita ashraya aura chikitsa dene valom taka simita pahuncha hai|
‘heldi starta’ ina kshetrom mem samudaya dvara chalae gae karyakramom ki, khatare vale karakom ki vyapakata jo ki shishu nrityu dara mem sahayaka haim ko kama karane mem sahayata karata hai|
1991 mem isake prarambha se lekara ‘heldi starta’ ne kai sau haja़ara parivarom ko sevaem di haim|
1998 mem ‘heldi starta’ pariyojana mem pariyojana karyakartaom dvara purva prasava dekhabhala mem kevala 41.8 pratishata prathama trimaha pravishtiyam thim|
2004 taka purva prasava dekhabhala mem prathama trimaha pravishti 70 pratishata badha़ gai thi|
2002 se 2005 taka 13 heldi starta samudayom ne karyakrama bhagidarom me koi shishu nrityu darja nahim ki|
echa ara esa e rajyom ko rajya stara para shravana –shakti vikara ke satha navajata shishuom ke lie surakshatmaka, naidanika aura hastakshepa karyakramom ko sthapita karane ke lie bhi punji pradana karata hai|
echa ara esa e rajyom ko sadame vale, mastishka aghata vale vyaktiyom aura unake parivarom ke lie savasthya aura anya sevaom ki pahuncha ko sudharane ke lie bhi punji pradana karata hai|
anudana, rajakiya abhighataja mastishka aghata abhiyojana pradana karane ke lie mukhya kshamata vale ghatakom ke vikasa ko samarthana dete haim|
gambhira bimari aura chota ke karana bachchom aura yuvaom mem ayogyata aura nrityu ko kama karane ke lie bachchom ke lie ‘‘apata kalina chikitsiya sevaem’’ karyakrama bhi banaya gaya hai|
bachchom ke lie apatakalina chikitsiya sevaem vartamana apatakalina chikitsiya seva tantra ko sudharane ke lie yu.esa.rajya kshetrom aura kolambiya jile ke rajyom aura chikitsiya sansthanom ko punji ka anudana karati haim|
svasthya dekhabhala, samudayika sansadhanom aura samarthana ke bare mem parivarom dvara paryapta janakari ki prapti ko sunishchita karane ke lie parivarom dvara chalae gae sangathanom ko anudana pradana karata hai jisase ki unake bachchom ki svasthya dekhabhala ke bare mem lie gae nirnayom ke bare mem janakari prapta ho sakem|
honyopaithi bahuta logom dvara prayoga mem lai jati hai lekina upayogita bahuta khandom mem hoti hai|
eka sangathita abhiyana dvara hama honyopaithi ko pure desha mem age badha़ane vola haim, vishesha rupa se mataom aura bachchom ke svasthya pragati ke lie|
neshanala rurala heltha mishana (ena ara echa ema) ke antargata sarakara ne chikitsa ke isa svarupa ko rashtriya, rajakiya aura jila stara para age badha़ane ka nishchaya kiya hai|
sabhi aspatala aura honyopaithi chikitsakom ko karyakrama ki saphalata ko age badha़ane ke lie eka tantra ke antargata laya jaega|
desha mem mahilaom aura bachchom ka asantoshajanaka stara aura shishuom aura chhote bachchom mem uchcha roga dara aura nrityu dara|
ajakala shishu nrityu dara 58 prati 1,000 shishujanma hai jabaki matri nrityu dara 301 prati 100,000 prasava hai|
samasyaom jaise ki rakta salpata, aparyapta dugdha utpadana, mutra vahini sankramana, tvacha vikara, dama, dasta, chikinaguniya, shvasana samasyaem, nidra vikara aura kai isa taraha ki svasthya samasyaom ko honyopethi dvara aura kama kharche ke satha nivarana kiya ja sakata hai|
hamare pasa chikitsa ki paramparagata vyavastha ka nirvaha karane vale 3,000 aspatala haim|
ina aspatalom ko prathamika svasthya dekhabhala kendrom aura jila aspatalom se sammbaddhita chhote aspatalom ke satha samanyataura para logom ko aura vishesha rupa se gramina logom ko eka vikalpa pradana karane ke lie joda़a jaega|
hipetaitisa bi kuchha jane pahachane naॉna-ritrovairala vayarasom mem se eka hai jo ki 
vairasa koshika ki sataha para eka abhigrahi se juda़ne ke dvara koshika ke andara pravesha prapta karata hai aura endosaitosisa ke dvara isamem pravesha karata hai|
kyonki vayarasa eka mejabana enjaima dvara bane hue ara ena e dvara gunita hota hai, payarala jenomika di ena e ko, mejabana protina jo cheperanasa kahalate haim ke dvara koshika ke kendraka taka hastantarita hona hota hai|
anshika dvisutriya vayarasa vala di ena e phira purna dvisutriya bana jata hai aura banda golakara atyadhika kandali vale di ena e (si si si di ena e) mem badala jata hai jo ki chara vayarasa vale ema ara ena eom ke lie sanche ka kama karata hai|
sabase bada़a ema ara ena e (jo ki vayarala jinoma se labba hota hai), jenoma ki nai pratiyam banane ke aura kepasida kora protina aura vayarala di ena e paॉlimaresa banane ke kama mem ata hai|
ye chara vayarasa ki pratilipiyam aura adhika prakriyaom se guja़rati haim aura progeni viriऑnsa banane ke lie age badha़ta hai jo ki koshika dvara chhoda़e jate haim ya kendraka taka vapasa ate haim aura adhika pratilipiyam banane ke lie punah chakrita hote haim|
lamba ema ara ena e phira vapasa se saitoplajma taka bheja diya jata hai jaham viriऑna pi protina apani ulti pratilipika kriyaom se hote hue di ena ka nirmana karata hai|
apane avarana protina mem upasthita entijanika aipitopsa ke adhara para vayarasa chara mukhya sirotapsa (e di ara, e di dablu mem aura jinoma ke kula milakara nyukliyotaida anukrama parivartanom ke adhara para atha jinotapsa mem vibhajita hota hai|
jinotaipa ka eka bhinna mauli, bhaugolika vitarana hota hai aura ye vayarasa ke kramavikasa aura hastantarana ka pata lagane mem prayoga hota hai|
jinotapsa mem antara bimari ki gambhirata ko prabhavita karate haim avadhi aura sambhavanaom ki jatilataom aura upachara aura sambhatah tikakarana ko 
hipetaitisa bi ke dvara utpanna eka mahamari ka sabase pahale likhita pramana laramaina dvara 1985 mem diya gaya|
chechaka ka prakopa 1883 mem bremena mem hua aura 1,289 pota karakhane ke karyakartaom ko dusare logom se limpha़ ke dvara tika lagaya gaya|
kai saptahom bada aura bada mem antha mahinom bada taka 191 tikakarana kie hue karyakarta piliya se bimara hue aura sirama hipetaitisa se pida़ita ki taraha nidanasuchaka pae gae|
anya karyakarta jinhem limpha ke anya samuhom ke satha tika lagaya gaya tha svastha rahe|
laramainsa ka lekha, aba rajapadika roga vijnana ka eka pratishthita udaharana mana jata hai, sabita karata hai ki sankramita limpha़ prakopa ka strota tha|
1909 mem, bada ke adhastvachiya suiyom ke jinhem siphilisa ke upachara ke lie salavarasana dene ke lie prayoga mem laya gaya aura mukhya rupa se punah prayoga mem laya gaya, arambha ke bada kai isi prakara ke prakopom ke pramana die gae|
1965 taka vayarasa nahim khoja gaya tha jaba barucha blamabarga, taba neshanala instityuta ऑpha heltha (ena ai echa) mem kama kara rahe the, ne astreliyana jati vale logom ke rakta mem ऑstreliya entijana (bada mem hipetaitisa bi sarphesa entijana jana gaya) ki khoja ki|
1970 mem di. esa dena aura anya logom ne iktraॉna maikroskopi dvara vayarasa khoja
arambhika 1980 taka vayarasa ke jinoma ko shrenibaddha kiya aura prathama tikom ka parikshana kiya gaya|
roga simaom para maleriya prativarsha aphrika mem miliyana logom adhikatara pancha varsha se kama ayu ke bachche, ko prabhavita karata hai|
tivra mastishka vikasa ki avadhi ke daurana vyapaka rakta alpata aura pramastishkiya maleriya dvara sidhe mastishka adhata bhi utpanna karata hai jisake lie bachche adhika samvedanashila hote haim|
maleriya ko kama karane ka lakshya rakhane vali nitiyom sanjnanatmaka labha ho sakate haim|
yaha sujava diya gaya hai ki aphrika ka bhavi arthika aura shaikshika vikasa maleriya ke nirmulana para nirbhara karata hai|
golakarakrimi karoda़ sau logom ko sankramita karate haim|
yaha pramana hai ki antom mem krimiyom ki uchcha matra manasika padarshana ko prabhavita kara sakata hai|
aisi kai prakara ki bimariyam haim jo kendriya nada़i tantra ko prabhavita karati haim jo sanjnanatmaka kshati kara sakate haim|
kuchha samanya udaharanom mem shamila haim elja़imarsa bimari aura malti-inphrekta demanshiya|
kai bimariyam nada़itantriya ya manovikara sambandhi ho sakati haim aura vishesha rupa se mastishka ko prabhavita karati haim|
dusare anya angom ko prabhavita kara sakate haim jaise ki echa ai vi, hashimotosa thairaॉiditisa jo haipothairaidijma ya kainsara ka karana hai|
uchcha bauddhika stara vale vyaktiyom mem samanyatah vayaska roga dara aura nrityu dara kama hoti hai|
aisa isalie ho sakata hai kyonki ve chotom ko achchhi taraha bachate haim aura apane svasthya ki achchhi dekhabhala karate haim ya isake badale mem bhautika punji ke lie thoda़e badha़e hue jukava ke karana|
uchcha bauddhika stara samuha mem uttara abhighataja tanava vikara, tivra avasada aura sijophreniya kama pae jate haim,
da akaivaja़ ऑpha janarala saikietri ne nagariya aura upanagariya donom se 713 adhyayana sahabhagiyom (336 lada़ke aura 377 lada़kiyam) ke yadrichchhika chayanita namune ka lambavarti adhyayana prakashita kiya|
usa samuha ke lagabhaga 76 pratishata kama se kama eka bara sadame vali ghatana se pida़ita raha chuke the|
una sahabhagiyom ko 6 varsha ki ayu para jancha gaya aura 17 varsha ki ayu taka anugamana kiya gaya|
bachchom ke usa samuha mem, 115 se upara bauddhika stara valom mem sadame ke karana uttara-apaghataja tanava vikara hone ki kama sambhavana, vyavaharatmaka samasyaom ko pradarshita karane ki kama sambhavana aura sadama hona ki kama sambhavana spashta rupa se thi| 
uchcha bauddhika stara vale bachchom mem uttara-apaghataja tanava vikara ka ghatanaem kama hona satya tha taba bhi jabaki bachcha eka shahari vatavarana mem bada़a hua ho (jaham upashahara ki dara se sadama ausatana tina guna tha) ya vyavaharika samasyaem hom|
dusari tarapha़ uchcha bauddhika stara manograstata badhyata vikara ki sthiti valom ki uchcha dara dikhata hai|
tivra avasada uttari amarika mem janasankhya ke lagabhaga 16 pratishata ko unake jivana mem kama se kama eka avasara para to prabhavita karata hai aura ayogyata ka mukhya karana hai, pagalapana ke samana lakshana de sakata hai|
shishu ki naka se pani bahana sardi/jukama ka pahala sanketa hota hai|
yadi apake shishu ko jukama hai, isa bata ki sammbhavanaem haim ki jaba taka yaha vigada़ na jaye apako isaki janakari na ho|
akhirakara vaha apako asa pasa ghutane chalata hua dikhai nahim dega, usaki naka tishu mem sapha karate samaya|
yadyapi apa satarka haim apa usaki naka se pani girata hua paengem yadi vaha nahim bhi ro raha hai|
apane sthaniya aushadha vikreta se eka sakshana balba kharidem, yadyapi sabhi aushadha vikreta ise nahim rakhate haim, atah yaha achchha hai yadi apa apani manga jaldi bata dem akhiri kshana mem ise pane ki koshisa karane ke bajae|
yaha balba bahuta suvidhajanaka hai, kyonki yaha apake shishau ki naka se sabakuchha bahara khincha lete haim|
isake parinamasvarupa apaka shishu na kevala thoda़a sa balagama gataka lega lekina vaha adhika aramadeha ho jaega| 
isake atirikta, apako apane bachche ki naka tishu se lagatara nahim ponchhani pada़egi|
yaha apake shishu ke samvedanashila sharira ko kashta pahunchata hai aura apake shishu ki naka jaldi hi jalane lagegi yadi apa lagatara ise pochhengem| 
shishu apani naka nahim ponchha sakate aura isake parinamasvarupa ve pura shleshma nigalane lagata hai|
bahara nikalane ke dvara isase chhutakara pane ke lie shishu ka apana tarika hota hai isalie bhayabhita nahom yadi samaya samaya para apako shishu ki ultiyom mem badha़otari dekhem|
shishu ko garama aura lapeta kara rakhem, sunishchita karem ki vaha kapada़e achchhi taraha se pahane ho aura sina surakshita hai| 
shishu eka bina bahom vala svetara pahana sakata hai yadi apa sochate haim ki puri bahom ke svetara ke lie yaha kaphi garma hai| 
eka vatanukulaka ki hava vastava mem apake shishu ke samvedanashila nathunom mem jalana paida kara sakati hai|
isi prakara kendriya ushnata usaki naka ko sukha kara aura jalana paida kara sakati hai|
apane shishu ke kamare mem eka ardata upakarana rakhem yadi apa ushnata ko jalana chahate haim| 
yadi apa use eka rishtedara ya shishu rakhane vale ke pasa chhoda़ rahem haim, sunishchita kara lem ki use isa bata ki janakari haim ki apake shishu ko jukama hai aura use usake tapamana ki jancha karate rahana hai| 
hamesha, hamesha shishu rakhane vali ke pasa bukhara utarane ki davai rakhem jisase yadi apake shishu ko kabhi bhi bukhara hota hai vaha turanta karya kara sake| 
yada rakhem ki eka shishu ka bukhara bahuta jaldi teja ho jata hai|
shishu ke upara ke adhe hisse ko apane takie para sahara dem jisase ki vaha sote samaya upara utha rahe|
sunishchita kara lem ki jaba apa yaha karem apa kamare mem hom aura jage hue hom usake takie ke niche ghutane se bachane ke lie|
yada rakhem ina dinom usake lie sansa lena kathina hota hai isalie apako aura adhika savadhana hone ki avashyakata hai jaba apaka shishu kambala ya takiyom ka upayoga kara raha ho| 
apake shishu ki banda naka ko kholane ke lie apane daॉktara se bina aushadhi yukta lavaniya bundem nuskhe mem likhane ko kahem|
yadi apake pasa bhapa vala upakarana nahim hai to bathapuma mem phavvara chalu karake sabhi daravaje khida़kiyam banda kara dijie aura jaba kamara bhapa se bhara jae to apane shishu ke satha pravesha karie|
bhapa andara khinchana sankuchana ko arama deta hai aura apake shishu ki banda naka ko kholata hai| 
gusalakhane ke bahara eka chadara rakhana pahale se sunishchita kara lem jisase ki apa jaise hi bahara nikalem apa use thandi hava ke samparka mem ane se bachane ke lie use chadara mem lapeta lem| 
elarji khansi ke sabase adhika sarvajanika karanom mem se eka hai|
apake bachche ko kuchha chija़em jaise dhula, paraga, palatu janavara, paudhom se alarji ho sakati hai jo eka bahati hui naka aura sum-sum karane ka karana ho sakati hai| 
yadi elarji bahuta tivra nahim hai, shleshma usaki naka dvara bahane ya use chhinka dilane ke bajaya kevala usake gale ke niche utara sakata hai|
shleshma ka gale mem yaha lagatara girana marga mem jalana paida kara karata hai jisase apaka bachcha khansata hai|
isa taraha ki khansi ke upachara ka sabase achchha tarika isa bata ko nishchita karana hai ki apaka bachcha elarji paida karane valom ke samparka mem ane se bache|
adhikatara dhula sataha ya khule kshetrom se hatai jati hai jabaki yaha adrishya kshetrom mem chhipi hoti hai jaise ki bistara ke niche ya pharnichara ke pichhe|
sunishchita karem ki yaha chhupe hue kshetra bhi achchhi taraha sapha hai aura dhula jada़e hue haim|
bharata mem galiche pratidina sapha kie jane chahie kyonki dhula ki ina para jamane ki privatti hoti hai|
yadi apake pasa vaikyuma klinara hai ise pratidina galiche sapha karane ke lie prayoga karem jisase dhula khincha kara bahara a jaye|
isi prakara apane bachche ke sone ke kamare se paudhom ko hata dem aura sunishchita kara lem ki paudhom ki pattiyam sapha rakhi gai haim|
yadi koi bhi abhibhavaka dhunrapana karata hai yaha sambhava hai ki dusare ka dhuma apake bachche ki khansi ke lie uttaradayi hai|
apane ghara mem eka dhunrapana kaksha nirdishta kara dem jo ya to apaka sone ka kamara ya apaka padha़ne ka kamara ho sakata hai| 
yadi apake yaham eka chhajja hai, use dhunrapana kshetra hone dem aura kevala apane chhajje mem hi dhunrapana ka niyama bana lem|
isi prakara, jaba apake ghara para eka ayojana hai, apane dostom ko bataem ki unhe chhajje para jakara hi dhunrapana karana chahie jisase apaka ghara dhuma rahita rahe aura apaka bachcha svastha rahe|
chikitsaka ko bulane ke pahale apa apane apa koshisha kara sakate haim aura khansi ka upachara kara sakate haim yaham kuchha samagri apa rasoi ghara mem pa sakate haim jise khansi ka upachara karane mem madada karani chahie|
shahada gale ko arama pahunchata hai sankramana ko khatma karata hai aura isake kai anya phayade haim|
adaraka eka prakritika jalana rodhaka hai aura chhati ki ki sankulata mem arama dene mem sahayata karata hai|
siraka eka prakritika roganurodhaka hai aura isamem kai advitiya guna hote haim|
na kevala siraka tvacha ki kai sthitiyom aura balom ko arama dene ke lie lagaya ja sakata hai balki jaba khaya jata hai yaha bahuta svasthayavardhaka hai| 
uparyukta upacharom ke alava sankramana nashta karane ke ilae apane bachche hara rata garma pani, namaka aura chutaki haldi se garare karavaem|
garare usaki khansi thika karengem aura usake taॉnsilsa ko uttama svasthaya mem rakhenge isalie usaki kula pratiऱodhaka kshamata ko badha़aenge|
chikitsaka ko bulaem yadi apaka bachcha tina saptahom se jyada avadhi se khansa raha hai|
bachche mem chirakalika khansi vikasita ho sakati hai isa sthiti ke upachara ke lie davaiyam nuskhe mem likhe jane ki avashyakata hai|
lagatara khansana dame ka bhi lakshana ho sakata hai balki yadi apake bachche ko halka dama hai to khansana hi kevala eka lakshana ho sakata hai atah yaha uttama hai ki ise najara andaja na karem| 
yadi apaka bachcha adhika vajana ka hai, vaha skula mem majaka ka lakshya bana sakata hai|
bachche nirdayi ho sakate haim aura dostom ka majaka banana akasara eka pasandida samaya gujarana ho sakata hai|
yaha sambhava hai ki apaka adhika vajana vala bachcha skula mem kai majakom ka vishaya hai|
bachche akasara vapasa a kara apane abhibhavakom ko nahim batate haim jaba ve kisi ke vyavahara se ahata hue hom|
vaha adhikatara skula se bhaga jane ki koshisha karate haim atah yadi apaka bachcha skula jane memachanaka se hichakichahata dikhata hai, use apane dostom se samasya ho sakati hai|
kisi khela mem apane bachche ka nama darja karavana vajana kama karane mem usaki madada karega|
usake vajana ko kama karane ki koshisha se pahale apane bachche ko majakom ka vishaya banane ka intajara na karem| 
yadyapi sabhi bachche raiketa vale khela pasanda na karem, adhikatara bachche marshiyala arta pasanda karate haim|
atah apane bachche ka nama asa –pasa ke ke karate, kunga phu ya tekavando mem darja karavayem|
na kevala vaha vajana ghatana shuru karega balki aura adhika chusta, svastha aura atmavishvasi hoga|
taje khadya padartha aura svasthayavardhaka nashte ka sangraha karem aura ghara mem niyamita rupa se chipsa tatha anya bekara ke aura motapa badha़ane vale khane ko lane se bachem|
apane bachche ko gale se khana utarane ke lie jora na dem sirpha isalie kivo usaki tashtari mem hai|
apaka bachcha kaposhita nahim hoga agara vaha apani tashtari puri taraha sapha na kare| 
yaha apa jisamem vishvasa rakhate haim usake viruddha ho sakata hai lekina apake bachche ke lie usaki tashtari mem bina khae khana chhoda़na vastava mem papa nahim hai| 
pure vishva mem karoda़om bachche bhukhe haim, bhojana apurti ki kami ke karana nahim balki paise ki kami ke karana|
kevala itana sunishchita karem ki apa use svastha, kama vasayukta bhojana khilate haim aura baki usa para chhoda़ dem|
dusari tarapha apaka bachcha adhika bhara vala nahim ho sakata lekina yaha sambhava hai ki vaha dusare mote bachche ka jivana dukhada bana raha ho|
yaha vyavahara koi atyadhika vinashakari ho sakata hai kevala usake lie nahim balki unake lie jisaka vaha majaka banata hai|
hara koi kisi hanirahita tangakhichai mem ananda leta hai lekina lekina jaba tangakhichai majaka banane ki rekha para karati hai aura nirdayata ko badha़ati hai abhibhavaka ko apane bachchom se bata karani chahie| 
yaham taka ki apaka bachcha dusare dosta ko chidha़a sakata hai jo turanta ro sakata hai aura phira abhi taka jo hua use bhula jata hai|
yaha vaha bachche haim jo rone ke lie mana karate haim ya chunauti ke satha pichhe dekhate haim jo adhikatara ko dukha pahuncha sakata hai| 
atah apane bachche ko dusarom ki bhavanaom ke prati adhika samvedanashila hone ke lie protsahita karem| 
bachche akasara yum hi kama khate haim kyonki dina ke bada dina samana khana ubau bana jata hai|
hara mam apane kama vajana vale bachche ko adhika khane ke ke lie jora dena svabhavika rupa se chahati hai|
yadyapi apa mahasusa kara sakate haim ki jora aura anushasana ka prayoga karane ke dvara apa usake munha mem do kaura aura dala sakate haim, yaha vastava mem adhika prabhavashali tarika nahim haim|
jo bhai pakaya hai khaane ke lie apane bachchom ko jora dene ke bajaya vaha kya khana pasanda karengem puchhane ka vichara kijie|
jo paka hai usamem yadi unaki ham hai, ve lagabhaga nishchita rupa se ise khane ke lie adhika ichchhuka hongem|
malati usaki boti mona ko chapati khilane ke lie lagatara koshisha kara rahi thi|
mona ne unase napharata ki aura bada़i kathinai se eka khai|
malati ke ghara ki chapatiyam moti thi aura ye kagaja jitani patali rotiyam vaha thi jinhem mona ne pasanda kiya|
hara abhibhavaka usake bachchom ko svasthayavardhaka khilana chahate haim lekina svasthavardhaka khana hamesha svadishta nahim hota|
yadi apa apane bachche ko vo karela khilana chahate haim jisase vaha puri taraha napharata karata hai, karele ko eka vyanjana jise vaha pasanda karata hai jaise ki shayada rajama aura chavala ke satha parosane para vichara kijie|
kama se kama apake bachche ka nahim baitha jaega jaise hi vaha tebala para baithata hai aura dekhata hai dopahara ke ya rata ke khane mem kya hai|
bharata mem sabhai sambhavita bachche adhikatara vyanjanom aura shayada kuchha chainija़ vyanjanom ke samparka mem aye haim|
yadi apaka bachcha chainija़ khana pasanda karata hai koshisha kijie aura ise adhika niyamita rupa se banaem|
yaha eka achchha avasara hai apane bachche ko bahuta si sabjiyam jaise ki phaliyam, gajara, brokonli, phulagobhi, masharuma adi khilai jaem|
ina sabjiyom ko patale lambe tukada़om mem katem aura unhe nudalsa mem dale|
apa unhe eka mishrita sabjiyom vali chatani ke satha bhi parosa sakate haim|
sabjiyom, jaise ki gajara, phaliyam aura brokoli jinhe apaka bachcha shayada niyamita bhojana mem pasanda nahim karega, ko shamila karane ka nishchaya karem|
ajinomoto ya ema esa ji ko chhoda़ dem aura barika ate ya meda se bane nudalsa se bachem|
isi prakara maide ke bajaya gehum se bane vhaita saॉsa mem mishrita sabjiyam brauna breda ke satha achchhi lagati hai|
tosta para thoda़a sa lahasuna vala makkhana lagaem aura apane bachche ko eka kshana mem khana nigalate hue dekhem|
jaise ki 6 varsha ka saphara apane dostom ke satha khelane ke lie bahara bhagata hai usaki mam rekha usake phira dusare bache hue khane ko ghurati hai|
yadyapi vaha janati hai ki bahuta se bachche nakhare se rakhane vale hote haim, samira ki bhukha rekha ko chinhita karati hai kyonki vaha sochati hai ki usaka beta usaki unra ke lie bahuta kama vajana vala hai|
bachchom mem motapa ajakala batachita ka adhika vishaya hai|
apane bachchom ka svastha vajana kaise paya jae isa bata se anabhijna ve akasara jabaradasti khilane ka ashraya lete haim|
yaha samasya ka samadhana nahim karata hai balki khana khane ko eka kama banakara ise aura bura bana deta hai|
abhibhavakom ko yaha samajane ki avashyakata hai ki khana khana poshana ko pura nahim karata hai|
lekina hama apako poshana para kuchha sujava de isase pahale yaha mahatvapurna hai apa isaka sahi nidana karem|
apake bachche ka vajana aura unchai ka hisaba rakha jaega aura manaka vriddhi charta se usaki tulana ki jaegi|
apaka shishu visheshajna ya chikitsaka isa bata ka javaba dene ke lie uttama vyakti hai ki apaka balaka kama vaja़na ka hai|
apake bachche ke vaja़na aura uchai ki ganana ki jati hai aura manaka vriddhi charta se tulana ki jati hai|
apake bachche ke bi ema ai (baॉdi masa indeksa) bi ganana ki jaegi|
5 pratishata bi ema ai se kama ka artha hai ki balaka kama vaja़na ka hai|
apaka chikitsaka apako usaka poshana aura kailori antagrahana ko badha़ane ke lie bhi kaha sakata hai|
kai bachche unake vriddhi charta para dhimi pragati dikha sakate haim lekina dhire-dhire samasya ko dura kara sakate haim|
yadi uchita poshana lene ke bada bhi apake bachche ke bhara mem vriddhi nahim hoti hai to apaka chikitsaka chikitsa samasyaom ke lie parikshana karega|
apane chikitsaka ki sahayata se ahara prabandhana ka charta banaie|
apaka chikitsaka achcha kailori antagrahana ki salaha dega|
adhika kailoriyukta bhojana ka chunava karem jisamem poshana bhi ho|
gajara, palaka, charbirahita mita, purna gehum breda kuchha paushtika bhojana haim|
sampurna dudha, deyari utpada, kashtha phala, chhile alu poshaka tatvom yukta uchcha kailori kuchha bhojya padartha haim|
sirpha kailoriyukta bhojyapadartha jaise ki thande peya, alu chipsa ya janka bhojya padartha se bachana chahie|
urjayukta bhojya padarthom ki purti poshana se bhare bhojya padarthom se karem|
apane bachche ke poshana ke lie banana sheka phruta kastarda aura krima vale supa ko ajamaem|
soyabina ke tukada़e ya ubali sabjiyom jaise ki gajara, brokoli ke chhote tukada़em ko chija़ mem lapetakara ya phaॉndyu parosem|
puri/ande ke rolsa ko masale hue aluom ko matara, kasi gajara aura kati phaliyom ke satha bharakara banaem|
apane naja़uka bachche ke lie tina bhari bhojanom ke bare mem bhula jame|
chhote bhojanom ke bicha ke antarala ko supa ya halke nashte jaise ki sabjiyom se saja़e hue chipsa aura biskutsa ke satha bharem|
bhukha ki huda़ka ke lie kuchha bhune hue kaju ya mungaphali ke chabane ke lie rakhem|
nityakrama, bhojana ke samaya prasannatapurvaka batachita, ubau bhojana ko akarshaka banana kuchha tarike haim jo achchhi khane ki adatom ko badha़ava dete haim|
apane bachche ke satha sarala aura vitaminayukta vyanjana pakaem|
nashta eka bahuta mahatvapurna hai kyonki yaha pure dina ke lie balaka ka idhana hai|
taiyara kiya hua makkhana ya marjerina ke bajaya achchhi vasa jaise ki vanaspati tela, jaituna ka tela ko chune|
ahara sambandhi koi bhi apurti keva chikitsaka ki salaha para hi li jani chahie|
purvaprasava dekhabhala mahilaom ke lie garbhavastha ke purva aura daurana batai gai chikitsiya dekhabhala hai|
purvaprasava dekhabhala ka uddeshya hai kisi bhi sambhavita samasya ki jaldi pahachana karana, yadi sambhava ho unase bachava karana (paryapta poshana vyayama, vitamina antargrahana adi ke bare mem sujava denem ke dvara) aura yadi ja़ruri ho to aura mahila ko uchita visheshajna, aspatala adi taka nirdeshita karana| 
niyamita ‘purvaprasava dekhabhala’ ki uplabdhata ne matri nratyu dara aura garbhapata satha hi satha janma doshom, janma ke samaya kama vaja़na aura vikasita vishva mem hone vali anya bachavakari shishu ki samasyaom ko kama karane mem mahatvapurna bhumika ada ki hai|
yadyapi purva prasava dekhabhala ki upalabdhata ke vyaktigata aura samajika bahuta labha hote haim, samajika arthika samasyaem kai vikasita satha hi satha vikasashila deshom mem isake sarvabhaumika angikarana ko rokati haim|
kanada aura yunaiteda stetsa mem adhyayanom ne darshaya hai ki gramina kshetrom ke satha satha alpasankhyakom ke samudayom mem purvaprasava dekhabhala ki uplabdhata ki kama sambhavana hai aura unamem shishu nrityu dara aura garbhapata ki dara adhika haim|
garbhavati mataom ke lie purvaprasaviya paddhiti hai nada़iyom ke doshom ko rokane mem sahayata karane ke lie kama se kama 400 mcg phaॉlika esida ka vitaminom ke satha sevana karana|
sharirika parikshana mem samanyatah mam ke chikitsiya itihasa ka sankalana hota hai|
ddhitiya trimaha mem lagabhaga 20 saptaha ke daurana adhikatara prasavika altrasaunda kiya jata hai|
altrasaunda sapekshita rupa se surakshita mana jata hai aura 35 varshom taka garbhavastha ke prabodhaka ke lie prayoga mem laya jata hai|
eka prakashita adhyayana ‘tyuja़eda’ ke anusara aparipakva janmem shishu una shishuom ki apeksha jo purna avadhi mem janmem haim, shishu nrityu ke tivra khatare aura jaba vayaska ho jate haim kama utpadaka dara se gujarate haim|
uttari kairolina ke dakshina purvi rajya mem dyuka yunivarsiti medikala skula mem shodhakartaom ne 1967 aura 1988 ke bicha norve mem 1.16 miliyana janmom ka adhyayana kiya aura ina bachchom ka anugamana kiya|
kula sankhya mem se 60,354 ya 5.2% aparipakva janmavale the jinhem garbhadharana ke 37 saptaha bada janma lene valom ke rupa me paribhashita kiya gaya tha|
eka adhyayana jo marcha mem jarnala ऑpha da amerikana medikala asosieshana ke 26 ve prakashana mem samane aya hai thoda़e se adhika lada़ke lada़kiyom ki apeksha aparipakva paida hue (5.6 pratishata se 4.7 pratishata) jo purusha janmom ka bada़a anupata dikhata hai|
aparipakva lada़kom mem se, jo 22 se 27 saptahom ke bicha paida hue the, eka aura 5.9 varshom ki ayu ke bicha aura 6 aura 12.9 varshom ke bicha 1.1 pratishata nrityu dara thi|
lada़kiyom ke lie, 22-27 saptaha aparipakvom mem se eka aura 5.9 varsha ki ayu ke bicha nrityu dara 1.71 pratishata thi ya samanya shishuom se 9.7 guna adhika thi|
6 se 12.9 varshom ke bicha ki aparipakva lada़kiyom mem kisi ki nrityu ko darja nahim kiya gaya hai|
28 aura 32 saptahom ke bicha janmem nara shishuom mem nrityu dara –jinaki ayu eka se 5.9 thi .73 pratishata aura jinaki ayu 6 se 12.9 thi .37 pratishata thi|
una bachchom ki tulana mem jo puri avadhi mem janmem haim, yaha khatara 2.5 se 2.3 guna adhika tha|
28 aura 32 saptahom ke bicha janmi stri shishuom ne nrityu khatare mem koi badha़ottari nahim dikhai|
jaise ki vayaska purushom aura striyom mem jo 22 se 27 saptahom ke bicha janmem haim prajanana ki dara kramasha 13.9 aura 25 pratishata thi|
ina purushom aura striyam una vayaskom ki tulana mam jo purnaavadhi mem janmem haim ki tulana mem prajanana na karane ki sambhavana bhi 76 aura 67 pratishata hai|
jo 28 se 32 saptahom ke bicha janmem haim unake lie unaki prajanana dara kramashah 38.6 aura 59.2 pratishata thi|
dudha evam dugdha utpada urja, protina, kailshiyama aura vitaminom ke mukhya strota haim|
eka balaka ko pratidina dugdha utpadom ka lagabhaga ½ litara lena chahie|
pancha varshom se niche eka bachche ko dudha dena chahie aura vasarahita dudha nahim dena chahie|
aharom ke bicha mem bhari nashtom ko nahim lena chahie kyonki vaha balaka ki bhukha ko kharaba karate haim|
yaha adhika achchha hai ki use keka aura chaॉkaleta ki bajaya phala aura saindavicha dene chahie|
apako bachche ke mithai, biskuta, chaॉkaleta, keka, kola adi ke khane-pine ko niyantrita karana chahie kyonki shakkara danta ki sada़na ka mukhya karana hai|
yaha hamesha adhika achchha hai ki apake bachche ko kola ki bajaya jusa dem|
agara apaka bachcha nakharom ke satha khane-pine vala hai milka sheka, khira, airasakrima adi dene ke dvara adhika akarshaka banae|
yadi apaka bachcha kama khane vala hai, khane ke samaya kuchha dusare bachchom ko ammatrita karie jo usaki ayu ke haim aura adhika khane vale haim|
yadi apaka bachcha durusta aura svastha hai aura apeksha ke anurupa badha़ raha hai aura usake vaja़na mem vriddhi ho rahi hai, usake khane ke bare mem|
khane ke samaya anandadayaka hone chahie aura apako bhojana ke atirikta chija़o ke bare mem bata karani chahie|
yadi apaka bachcha adhika vaja़na ka hai khana pakane ke tela ka kama prayoga karem|
shakkara, namaka aura barika ate (maida ya sabhi kamom ke lie apayukta ata) ki chijom ka kama prayoga karem|
bina paॉlisha ke chavala, ya para-baॉilda chavala paॉlisha vale chavala se achchhe hote haim|
sabjiyom ko adhika na pakaem, ve apana paॉshtika mulya kho deti haim|
phalom ko na na pakaem kyonki avashyaka vitamina C nashta ho jata hai|
bahuta si sabjiyom ka chhilaka vitaminom aura khanija tatvom se bharapura hota hai isalie taja़e phalom ko bachche ko unake chhilakom ke satha dem|
jaba bhi sambhava ho chini ke sthana para guda़ ka prayoga karem kyonki isamem lauha tatva ki matra bahuta jyada hoti hai|
eka abhibhavaka ki taraha apake balaka ke ahara aura vaja़na ko niyantrita karana apake hathom mem haim|
aksara abhibhavaka yaha manate haim ki unake bachchom ko adhika se adhika khane ke lie ja़ora dena unake pyara ka pratika hai|
bahuta se abhibhavaka atyadhika chintita nahim hote haim jaba ve pate haim ki unaka bachcha mota hai|
ve aisa sochate hainki bachcha mota hai jise balaka jaldi hi kho deta hai jaba vaha eka vayaska ho jata hai|
kuchha bachche jaba bada़e ho jate haim vastava mem apane bachapana ke motape se bahara a jate haim lekina adhikatara mem aisa nahim hota|
atah usake gale ke niche jabaradasti khana na dalem, vishesha rupa jo motapa badha़ane vala ho|
yaha muje ashcharya chakita hone se kabhi nahim rokata hai jaba maim logom ko motapa badha़ane vali mithaiyom (taॉphiyom) ko khane ke lie badhya karate dekhata hum jaba ve svastha khana hi khana chahane ki koshisha kara rahe hote haim|
kuchha abhibhavaka galati se yaha manate haim ki ghi, telapurna aura chini se bhara khana paka kara ve kisi prakara prema dikha rahe haim|
apane prema ya apane gharelu kaushalom ke pramana ke satha khana parosana chhoda़ne ki koshisha karem|
yadi apaka bachcha dusare tale hue pakauda़e nahim khana chahata use na khane dem|
yadi adhika achchha hai ki apa ise kharaba hone dem isake bajaya ki ye apake bachche ke peta mem jae|
dusare abhibhavaka bhi haim jo dusare para jate haim aura apane bachchom ko, jaisa ki vo manate haim protsahana bhare shabdom ki salaha dete rahate haim jaise ve kya kha rahe haim ve dhyana rakhem, apane ahara ko niyantrita karem, adhika vyayama karem adi adhika aura adhika batane ke dvara|
bahuta jaldi hi apaki bachchi na kevala apaki salaha ke prati bahari hone vali bana jaegi balki usamem asuraksha ki bhavana bhi vikasita ho jaegi|
apa anubhava karane se bhi adhika usake atmavishvasa ko adhika nukasana pahuncha rahe haim|
apane bachche ko dubala hone ke lie apa kitani bhi koshisha karate haim use vaja़na kama karane ke lie lagatara na kahem|
apane uddeshya ko bole bina svasthya purna nashte ki suchi taiyara karane se arambha karem|
ghara para aisa krima, keka aura baja़ara ki beka ki chijom ko lana talem|
ghara para apa jitana kama sangrahana karenge utana hi kama apaka bachcha khaega|
apane bachche ko khula kuda kriyaom mem bhaga lene ke lie protsahita karem|
yadi apake parivara mem sharirika rupa se koi bhi kriyashila nahim hai to use isa disha mem bhejana asana nahim ho sakata lekina karem au usaki ruchi badha़aem|
aksara bachche nai kriyaom ko karana chahate haim aura eka adhika bada़i chunauti nahim honi chahie|
shayada apa kuchha mitrom se bhi mila sakate haim jo sharirika rupa se bahuta chusta haim ya jinake bachche eka vishesha khela ko khelate haim|
usase pahale apani bachchi se khelakuda ke vibhinna vikalpom ke bare mem batachita kara lem jisase jo khela vaha chunati hai vaha usaki ruchi ka ho|
yadi apa use eka samaya se eka se adhika khela khelane ka avasara dem jaise ki tenisa aura tairaki, apaki bachchi eka khela ki tarapha़ svabhavika abhivritti aura ruchi dikha sakati hai jisake bada apa dhyanakendrita karane mem usaki usaki sahayata kara sakate haim|
jaba vaha bahuta chhoti unra ki ho to bhi use eka khela mem shamila karane ki koshisha karem|
aisa karana apakebachche ke lie sunishchita karane ka eka tarika hai ki vaha apane jivana mem svastha rahane aura niyamita vyayama karane ko prathamikata degi|
usake bada vaha jo khana vaha khati hai usake bare mem svabhavika rupa se savadhana rahegi- yaha saba jivana shaili ke satha chalata rahata hai|
abhibhavakata jatila ho sakati hai aura yaha lagatara eka kathina kama hai|
bachche bahuta adhika rupa se bhinna ho sakate haim phira bhi kai rupom mem samana hai|
apane jaivika ya anuvanshika gunom ki paravaha na karatu hue bachche apane vatavarana ke anurupa grahana karate haim aura badha़te haim|
vatavarana bachche ke vyaktitva, manobhavom, vyavaharom aura sambandha prarupom ko bahuta adhika prabhavita karata hai|
kuchha bachche eka vishesha abhibhavaka ke lie sambhalane mem vishesha rupa se pareshani vale ho sakate haim|
eka die gaye anubhava ka ve kitane rupom mem anukulana karenge, isake lie bachche samana hone ki apeksha adhika ninna hote haim|
abhibhavaka adhikatara apane apa ko ya apane vyavahara ke prabhava ko dekhane mem asamartha hote haim kyonki anhem anajane mem apane khuda ke bachapana ki yada a jati hai|
bachche parivarika jivana mem tanava ke prati bahuta jagaruka aura samvedanashina hote haim| 
mam evam shishu svasthya ka vishaya eka bahuta mahatvapurna vishaya hai jisa para samudaya mem vyapta paramparika jnana hai|
isa artha mem yaha artha mem yaha advitiya hai ki shayada yaha kshetra hai jaham paramparika svasthya chikitsaka visheshajnom ka bada़a samuha hai arthata daiyam| 
yaha adhika se adhika yaham hai isa bata mem vishesha bhi hai ki sarakara ne (yadyapi anichchha se) sthaniya svasthya chikitsaka arthata daiyam ke astitva ko dhyana mem rakha hai|
daiyom ka prashikshana matri evam shishu svasthya karyakrama ka aba eka mahatvapurna bhaga hai na kevala bharata mem balki pachasa se adhika anya deshom mem bhi|
chikitsiya anusandhana ki bharatiya parishada ke hala ki eka riporta ke anusara vartamana matri nrityu dara 4-7 prati 1000 jivita janma hai|
shishu nrityu dara 90 prati 1000 jivita janma hai|
shishuom mem kama janma bhara ki ghatanaem 35 se 45% haim|
abhi taka kai adhyayana aura sarvekshana uplabdha haim jinhonne mam evam shishu svasthya ke kshetra mem sthaniya prathaom para janakari ikatthi ki hai|
ina adhyayanom mem se adhikatara chikitsa ki adhunika pranaliyom ke chikitsom dvara kshetrom jaise ki poshana vijnana, pusuti vijnana, stri roga vijnana, shishu roga vijnana aura samudayika svasthya mem prashikshana ke satha kie gae haim|
ina adhyayanom mem se lagabhaga sabhi simabandhom se grasta ham ki sthaniya prathaom ka ‘mulyankana’ hamesha adhunika chikitsa ki drishti se kiya gaya hai|
adhyayana, jinhonne ina prathaom ka mulyankana chikitsa ki deshi pranali (ai esa ema) ke paripekshya se kiya hai bahuta kama haim|
aisa eka udaharana nepala ki paramparagata shishu lalana-palana paddhitiyom ka adhyayana hai|
1986 mem loka svasthya parampara samvardhana samiti (ela esa pi esa esa) ‘‘paramparika chikitsa mem matri evam shishu’’ vishaya para eka sammelana ka ayojana karane ka nishchaya kiya hai|
ela esa pi esa esa ne nishchaya kiya ki matri evam shishu svasthya ke kshetra mem kai paramparika prathaom ke bare mem janakari prapta karane ke lie eka vistrita sarvekshana kiya jaega|
atah janakari jo ekatrita ki gai thi ayurveda ki drishti se mulyankita aura vishleshita ki gai|
1989 disambara mem ‘‘matri aura shishu svasthya mem chikitsa ke paramparagata paddhiti’’ vishaya para eka rashtriya sammelana ayojita kiya gaya| 
isa sammelana mem ayurveda ki drishti se aura svatantra rupa se adhunika chikitsa ki drishti se vishayom se sambandhita kai vishaya vastuom ke bare mem prastutikarana kie gae|
isa sambandha ka uddeshya ‘‘ garbhini griha’’ garbhavati mahila ki dekhabhala ke paramparika prastava ko sankshipta rupa mem prastuta karana hai|
sthaniya prathaom ki jinhem sahitya mem ela esa vi esa esa sarvekshana satha hi satha anya sarvekshanom ke daurana ankita kiya gaya, para tippani ki gai|
yaha nishchita rupa se svabhavika hai ki loka svasthya paramparaom aura ai esa ema esa ke madhya antahkriya samvada ki prakriti mem hona chahie|
eka hatha para ai esa ema esa ka nivesha loka svasthya parampara ka nirdharana evam mulyankana karane mem hamari sahayata kara sakata hai- kaunasi avaja़ sampurna hai, kya apurna hai aura kisi bhi vikriti ki pahachana ko nishchita karane ke dvara yaha ina paramparaom ko sudridha़ karane mem sahayata karega|
yadyapi, yaha samana rupa se sambhava hai ki ai esa emom svayam vibhinna sthaniya prathaom ka kramabaddha rupa se avalokana karane ke dvara labhanvita ho sakate haim inamem se kuchha ko ai esa emom ke siddhanta ke prakasha mem turanta samaja nahim sakata|
ina prathaom ko aura adhika padha़ne aura samajane ki avashyakata hai|
hama asha karate haim ki isa taraha ki antahkriya hamare ai esa emom ko adhika gatishilata bhi pradana karegi aura unaki prathaom ko adhunika banaenge|
matri evam shishu svasthya ka vishaya garbhadharana ke samaya se bahuta hi pahale hi shuru ho jata hai|
aisa isalie hai kyonki arthava (stri yauna angom ka strava) ki aura shukra (virya) ki gunavatta garbhadharana ke utpada mem jalakati hai|
chara karakom ka samanvaya –masika chakra (rithu), garbhashaya (kshetra), poshaka tarala (rasa ambu) aura svastha virya (bija shukra) aura eka uchita pathyapathya niyama ka palana garbhadharana aura eka svastha bachche ke vikasa ke lie avashyaka hai|
atah eka svastha bachche ko pane ke lie yogadana dene vale karaka jaise ki shukra aura arthava apani samanya svastha sthiti mem hone chahie|
eka samanya garbhadharana au svastha shishu ke janma ke lie uttaradayi karakom ki vyakhya karate samaya acharyom ne do binduom para ja़ora diya hai|
isaki salaha di jati hai ki purusha aura mahila vibhinna gotra ya kula ke hone chahie aura kevala tabhi unaka janma bachcha svastha aura maja़buta hoga|
sambhoga ke ulte sanketom ki vyakhya karate samaya bhavamishra ne samana gotra vali mahila se sahavasa ko shamila kiya hai|
kauna eka vaivahika sathi ho sakata hai para pratibandhom ki ganana karate samaya dharmashastra ne bhi vivarana diya hai – ‘‘asapinda cha ya mathurasagotra cha ya pithuha’’ arthata koi bhi eka vyakti se vivaha kara sakata hai jo usaki mam ka sapinda na ho ora usake pita ke gotra ke samana na ho|
purusha aura mahila ki vivaha yogya ayu kramashah 21 varsha aura 16 varsha bhi nirdharita ki gai haim, svastha santana pane ke lie|
eka svastha garbhashaya aura samanya arthava ko banae rakhane ke lie vyakti ko ritu kala (masika chakra ya masika strava ke daurana) kuchha nishchita pathyapathya niyamom ka palana karana chahie|
ritu kala ke samaya stri ko dina ke samaya sone, sharira para tela lagane, bhagane, teja़ avaja़ sunane, upavasa karane ya kisi bhari ya thakane vale kama se bachana chahie|
use apane nakhuna nahim katane chahie, rona nahim chahie ya kajala nahim lagana chahie|
masika strava rukane ke bada pahali baraha ratriyam garbhadharana ke lie (sabase achchha) uchita samaya mana jata hai|
masika strava ke shuru hone ke bada tina dinom ke lie yuva patni ko sahavasa se bachana chahie eka chatai (kusha ghasa se bani hui) para sona, halka khana jaise ki sade chavala eka chhote mitti ke bartana mem khana chahie|
chauthe dina use abhyanga snanama (tela lagakara sira dhona) aura sapha़eda paridhana lena chahie|
pati aura patni donom ko sapha़eda paridhana aura mala pahane hue, mana ki khushi aura eka dusare ke prati akarshana ke satha sahavasa me pravesha karana chahie|
eka svastha bachche ke janma ke lie eka dusare ke prati akarshana aura mana ki prasannata avashyaka haim aura isalie hamare acharyom ne ina binduom para ja़ora diya hai|
yaha sabase bahatvapurna hai ki purusha aura patni donom eka achchhi manasika sthiti mem hom arthata krodha, bhaya, irshya ya ghrina ke bina|
hama mahagranthom jaise ki mahabharata mem ghatanaom ke bare mem udaharana pa sakate haim jaham mam ya to ghrina karana ya dara ke karana mastishka ki santulita avastha mem nahim hai, vikarayukta bachchom ka janma hua jaise ki dhritarashtra (jo andha paida hua tha) aura pandu (jo kamaja़ora aura vikarayukta paida hua tha)|
dina ke samaya sahavasa ko hanikaraka bataya gaya hai|
prasanopanishada kahata hai ki ‘‘pranama va ithe prasakadanthi ye diva rathya sanyujante, bahncharyameva tadya drathru rathya sanjujyanthe’’ arthata vaha jo apani patni ke satha dina ke samaya sahavasa karate haim apani ojasvita kho dete haim aura vaha jo ise rata ke samaya taka simita rakhate haim brahnchari rahate haim|
ayurvedika darshana ke anusara manava sharira pancha mahabhutom se bana hua hai|
mam, pita, rasa aura atma ko ina mahabhutom ka strota kaha gaya hai|
kuchha manasika guna pichhale janmom se (purva jenma vasana) bhi ate haim|
eka vyakti ke kuchha guna sathnya, satva aura rajasa se bhi ate haim|
mam se lie jane vale guna haim – tvacha, rakta, mansapeshiyam, vasa, nabhi,hridaya, agnashya, yakrita, tilli, gurada, mutrashaya aura asthimajja |
pita se lie jane vale anga haim bala (shirovalka) dadha़i, munchhe, sharira ke bala, danta, haddiyam, shiraem, nasem, dhamaniyam, virya aura nakhuna|
rasajabhava hai sharira ka udagama ya vibhinna sharirika angom ka prakatikarana, sharira ki vriddhi, jivana se lagava ya sharira ke satha shakti poshana, utsaha ya umanga, shakti ya takata, svasthya asvastha avastha, ranga, sharira ka anurakshana aura lalacha ka abhava|
atma se vikasita hone vale guna haim – eka vishesha prajati mem janma, atmajnana, mana, shvasana, shvasa ka rokana aura chhoda़na aura, rupa ranga, avaja़, vishesha ranga, prasannata, vishada, lalasa aura ghrina| 
satmaya dvara prabhavita hone vale karaka haim – rogom se mukti, alasya, lalacha ka abhava, samanya avaja़, samanya ranga, samanya bija, avirala prasannata, mastishka, indriyom ka prasadatva, ojasa ki uchcha gunavatta, dirgha ayu, shakti aura urja|
purva janma se sambandhita karaka haim bhakti, achchha acharana, snriti, svachchhata, paropakara, sahasa, durbalata, sampatti, asthira mana aura irshya|
atah sabhi achchhe gunom ke satha shishu ke janma ke lie yaha avashyaka hai ki ye sabhi karaka jaise ki mam, pita, rasa (mam ke bhojana ke pachana ka utpada) atma adi achchhi avastha mem hone chahie|
jaba shukra (eka svastha purusha ka) prajanana kala ke daurana arthava (eka svastha mahila ka aprabhavita garbhashaya ke satha) ke samparka mem ata hai taba jiva (atma) mana ke satha garbha ko banane ke lie isake andara utarate haim|
yaha garbha sampurna rasa (mam ke bhojana ke pachana ka antima utpada) dvara poshita hota hai ora uchita vyavastha (mam ki) dvara samposhita hota hai|
usake bada garbha sabhi jnanendriyom aura karmendriyom (samvedi evam gatyatmaka anga) ke satha banata hai aura prasava ke samaya taka shakti, ranga aura manasika yogyataom ki shreshthata se sampanna hota hai|
athavem mahine ke pura hone ke bada bhruna prasava ke lie puri taraha se paripakva ho jata hai arthata navem mahine ke prathama dina se stri ko kisi bhi samaya prasava ke lie taiyara hona chahie|
piphaija़ra ne gramina hariyana mem 35 gamvom ko eka prathamika svasthya dekhabhala karyakrama ke lie goda liya tha|
piphaija़ra juna 2002 se tina-varsha kala ke lie prabhavashali matri evam shishu prathamika svasthya dekhabhala ko chalane ke uddeshya ke satha aura garbhavati mahilaom aura kishoriyom mem rakta alpata ko kama karane ke sutrapata ke satha gramina hariyana mem arpana risarcha enda chairitija़ trasta ko samarthana de raha hai|
pariyojana karanala jile aura hariyana mem aura asa-pasa 35 gamvom ko aura lagabhaga 60,000 logom ki janasankhya ko shamila karati hai|
karyakrama garbhavati mahilaom aura kishoriyom mem rakta alpata ko kama karane aura unako jinhem alpata se sambandhita bimariyam haim ka lakshya rakhata hai|
‘prathamika svasthya dekhabhala karyakrama’ mahilaom aura lada़kiyom (prajanana sambandhi svasthya dekhabhala aura kishoriyom ki dekhabhala ke satha) aura shishu dekhabhala sevaom ko sevaem pradana karata hai| 
‘prathamika svasthya dekhabhala karyakrama’ dasta prabandhana, prasavottara dekhabhala, bala poshana, tikakarana adi ko shamila karate hue samudayika svasthya ko protsahana deta hai|
prathamika stara – gramina svasthya karyakartah samudaya adharita bachavakari svasthya sevaem gamvom se paramparika daiyom ko shamila karate hue prathamika stara samudaya karyakartaom ke ghanishthata se juda़e dala dvara chalu kiya gaya tha|
ashakta gramina svasthya dekhabhala aura gramina vikasa kriyaom jaise ki svasthya karyakarta, daiyam, protsahanadata aura samudayika vyavasthapakom ke lie prashikshita haim|
yaha karyakarta svasthya, linga samanata, vatavaraniya svachchhata, mam evam shishu dekhabhala,vikalangom ki dekhabhala, mahilaom ki pragati aura shiksha, roja़gara aura aya utpadana ke lie logom ki badha़ti hui pahuncha mem badalati hui prathaom mem sahayata karate haim|
71 mahila samudaya adharita sangathana svasthya vikasa kriyaom mem bhaga le rahe haim|
ddhitiya stara- gatishila svasthya dalah gatishila nidanagriha-sambandhita adhara se juda़a hua, mam evam shishu ke lie svasthya sevaom ki eka vistrita shrankhala tikakarana aura parivara niyojana ghatakom ko shamila karatu hue pradana karane ke lie hara mahine pariyojana gamvom ka daura karata hai|
gatishila svasthya dala samanya bimariyom ka upachara aura gramina stariya sambandhita griha aura gramina adharita svasthya karyakartaom ke prathamika stariya dala ke lie takaniki sahara bhi pradana karata hai|
tikakarana ina lakshya gramom mem 98% se upara bachchom ko sammilita karata hai|
tritiya stara-170 bistarom vala arpana aspatala ke lie nirdishta-marija aura garbhavati mahilaem daiyom aura svasthya karyakartaom jo ki lakshita gavom mem rahate haim, dvara aspatala taka bheji jati haim|
samanyah nirdishta rogi hote haim – kathina prasava vale/jatilataem vale rogi, tivra raktaalpata, anya matri evam bala rogi jinhem nirdishta karane ki avashyakata haim, parivara niyojana shalyakriyaem|
nimoniya aura chhati ke anya sankulana, atyadhika nirjalikarana ke satha dasta, aura gurde aura usa kshetra mem pae jane vale anya patthara
rakta alpata ki sambhavana ko kama karane para dhyana kendrikarana|
adhika achchhi sarvagina svasthya dekhabhala aura garbhavati mahilaom aura pancha varsha se kama ayu ke bachchom svasthya avashyakataom ke lie jagarukata|
samudaya adharita sansthanom ki vriddhi jo samudaya ki svasthya dekhabhala au anya vikasatmaka avashyakataom ki jimmedariyom ko shuru karenge aura pura karenge|
bachavakari bimariyom (uda.dasta, maleriya visheshakara mahilaom aura bachchom mem kra niyantrana|
vishesha taura para mahilaom mem anabhijnata, andhavishvasa aura saksharata ke ninna stara ka virodha karana jo svasthya samasyaom, stanapana ki purva dharanaom aura hanikaraka prathaom ko badha़ate haim|
yauna sambandhi hastantarita bimariyom ki badha़ti sambhavana aura bhara se bachava karana|
vishesha taura para svasthya ora gramina vikasa ke lie 1980 mem arpana risarcha enda chairitija़ trasta ki sthapana ki gai|
‘arpana risarcha enda chairiti trasta’ ki vishishtata hai vibhinna rashtriyata evam dharmom vale 70 logom ka sangathana jinhonne svayam ekamatra uddeshya ‘gramina svasthya dekhabhala’ ki shapatha li hai|
prinsa charlsa ne aktubara 2003 mem apane tatkali daure ke daurana hariyana mem phija़rasa kanyuniti heltha keyara projekta ka daura kiya| 
jila raॉyala hainesa velsa ke rajakumara ne kutaila gamiri gamva ka daura kiya jo, gramina hariyana mem samudaya adharita karyakrama ‘matri evam bala prathamika svasthya dekhabhala’ ke lie phija़ra dvara goda lie gae 35 gamvom mem se eka hai|
prinsa charlsa gamva ki galiyom mem ghume, kai gharom mem gae aura mahilaom aura bachchom sahita kai logom se batachita ki| 
rajakumara gamva ki 20 daiyom se bhi mile aura dekha ki kaise arpana trasta sthaniya logom ke satha kama kara raha hai unhem kaushalom jaise ki prasuti-vidya ki shiksha dene ke lie|
navambara/disambara 2001 mem phija़ra indiya ne eka sarvekshana ka samarthana kiya jo ki volantari heltha esosieshana ऑpha indiya (vi echa e ai) dvara 35 chayanita gamvom mem sanchalita kiya gaya|
tina varshiya avadhi ki samapti para gananiya parinama shishu evam matri nrityudara mem sudharom ko darshaega|
mahilaom evam bachchom mem lauha tatva ke satha poshana apurti dvara rakta alpata aura lauha tatva apurnata mem kami, mahilaom ki adhika sankhya garbhavastha dekhabhala aura surakshita prasava grahana karegi
bachchom aura garbhavati mahilaom ke lie adhika achchhe poshana hastakshepom dvara kuposhana ka nidana|
mahatvapurna tikakarana vistara, bachchom ki vriddhi ka avalokana aura sankramaka roga, dasta, balyavastha nimoniya adi ka prabandhana|
dvitiya varsha ke anta taka prapta kie sudharom aura tritiya varsha arthata 2004 ke lie lakshyom mem sudharom ke satha mukhya parinamom mem se kuchha ninna haim|
chhote parivara jo achchhe rahana sahana ke stara ko sunishchita karenge|
bala evam matri nrityudara mem kami karyakrama ka eka lakshya hai|
abhibhavakata jatila bhi ho sakati hai aura lagatara eka kathina kama hai|
bachche bahuta bhinna hote haim phira bhi kai tarikom mem samana haim|
ve parivara mem apane sthana ke adhara para samvegotmaka rupa se aura apane svabhava mem, unaki jaiviki satha hi satha sanskriti aura anubhava jisamem ve badha़e hue haim, mem bhinna hote haim|
bachche apane jaivika ya anuvanshika gunom ke bavajuda bhi apane vatavarana ke prati anukulana aura vriddhi karate haim|
memoriyala ke childransa ke thairepi sentara mem apaka bachcha hamari pahali prathamikata hogi| 
samarpita prashikshakom ke samuha dvara achchhi svasthaya ke lie yatra ke satha apake parivara ko nirdeshita kiya jaega kyonki bachchom ki madada karana hamare utsaha mem mahatvapurna kama karata hai|
boli se vyavasayika tathaa bhoti upacharataka, pramanita chikitsaka anivaryatah bhumika ko pahachanate haim jo bachchom ki dekhabhala mem parivara nibhate haim – visheshatah vo jisamem vishesha svasthaya dekhabhala ki avashyakata hoti hai| 
isalie marija tatha parivara hamare arntavishayaka prashikshakom ke samuha ka kendra hote haim, bachche ki madada karana unaki atyadhika kshamata ko purna karata hai|
apake bachche ki achchhe svasthaya tatha atma nirbharata ki ora yatra ka hissa bana kara hama sammanita hue|
apane jivana ke kisi bindu para lagabhaga 25% bachchom ko khane ki samasya hogi|
hajarom marijom jaise ki adama MCTC vyavasayika chikitsakom ke dvara madada jindagi ke vyavasaya mem vapasa jane mem ki gai|
OT apake bachche ko hatha/ ankha ke samanvaya khane ka karya tatha samveda prakriya vikasa ko vyaktigata tatha samuha satra ke dvara sudharane mem madada karata hai|
kai bachche jaise eleksa, ghutane chalana sikhane, chalana, bhagana, kudana tatha khelane sirpha bachchom mem shaaririka chikitsaka ki madada se (PTs)|
chahe vaha mastishkiya palasi, spina biphida ka ilaja kare athava vikasa mem deri mem PTs bachchom ke satha kama karate haim tatha unake dekhabhala karane vale susangata dekhabhala pradana karane mem madada, jabaki bachche ke skula mem ghara tatha samudaya mem atmanirbharata ko protsahita karana| 
hama sabhi janate haim ki bachchom ko unake mastishka ko bolane ki avashyakata hoti haiaura unhe sikhane tatha badha़ne ke satha pure vishva se samvada karane ke kabila banane ki avashyakata hai|
vani chikitsa bachchom ki bhasha vikasa, khana, bolana, sunane tatha pravaha mem madada karata hai aise ki ve loga tatha tatha unake charom tarapha ke sansara ko jyada asani se varnana kara sake|
yaham aura bhi bahuta kuchha hai memoriyala childransa thairepi sentara mem, hamare sthala para hone vale 100 programa navare plesa tatha memoriyala childransa haॉspitala ki tulana mem|
bachchom ki chikitsa karmachari sadasya ko sabhi samudayom ke bachchom ko vishvastariya dekhabhala pradana karate haim|
sthaniya skulom tatha anya samudaya sangathanom ke satha kama karake MCTC vibhinna vyavasthaom se hajarom bachchom taka pahunchata hai unnata atmanirbharata tatha achchhi jindagi ke guna dene ka prastava deta hai|
maim mahasusa karata hum ki mere bachche ki chikitsa ke guna tatha pravinata jo ilaja vo prapta karate haim ko badha़ata hai|
chikitsaka dhyana rakhane vale, visheshajna hote haim tatha pratyeka ilaja kaushala mem visheshajna hote haim|
hara eka mere bachche ko die jane vale vyaktigata dhyana dene ki maim kadara karata hum|
yadi apa shankita haim to apake bachche ko memoriyala childrana thairepi sentara ki vishesha suvidha ki avashyakata ho sakati hai kripaya apa bachche ke balachikitsaka se milem| 
memoriyala 100 salom se bachche paida karava raha hai|
jaise hi takanika mem badalava aya aurata ko svastha bachcha pane ki sambhavana mem madada karane mem dekhabhala tatha karuna ka manaviya tatva usi rupa mem raha gaya|
ajakala memoriyala una logom ki pasanda ka aspatala hai jo chahate haim ki parivarika kendrita prasava anubhava uchcha visheshajna karmachari ki garanti tatha vishesha avashyakataom ke lie uchcha gunavatta yukta upakarana hom|
memoriyala rijanala sentara phaॉra madara tatha chailda keyara parivara ko naye bachche ke svagata karane ke lie shreshtha vyavastha pradana karata hai|
mam tatha bachche ki vyaktigata dekhabhala mem parivara ki avashyakata ke lie vishesha satkara ke lie hama prastava dete haim kai shanadara sadhana, sevaem tatha apaki prasava ka utsava ke lie vishesha sparsha ka|
aura jyada sikhane ko jari rakhie hamare vyapaka prasuti karyakrama, hamare visheshajna chikitsa adhikari tatha hamare dekhabhala karane vale narsa karmachariyom se|
naye janma ka bachche ka janma ashcharyajanaka tatha prakritika donom hota hai|
yaha mahatvapurna ghatanaparivara ke sadasyom ki jindagi mem hi nahim balki sabhi bhavishya ki pidha़iyom ki jindagi mem gahara badalava lati hai|
chaildabartha yunita ke anubhavi karmachari aurata tatha unake parivara ko purna prasava mem tatha vishishta dekhabhala vala prasava roga vishayaka nipunata yugma ke satha sahara dene ke lie vachanabaddha hai| 
hamare uchcha jokhima speshala keyara obesteriksa yunita aurata ko nipuna dekhabhala pradana karata hai jo apane prasava ke daurana jatilataom ko anubhava kara rahi hai aura 11 kauntija़ ki rijanala repharala sentara haim|
do karyakari suvidha nyuborna intesiva keyara yunita ke pasa 24 ghante enesthishiya vyakti yadi apata kala hua hai to turata pratikriya pratyuttara ki vishvasa dilata hai| 
apake aura apake naja़diki pyare dvara nae bachche ka sansara mem svagata tatha janma kamare mem satha vyatita samaya, ko bada apako madara/bebi keyara sentara mem sthanantarita kara diya jayega| 
parivarajana bachche ki dekhabhala ikai mem unaki narsa ke nirdesha ke satha kaushala vikasita karate haim jaise hi vo mam tatha bachche donom mam ke kamare ke poshita vatavarana me dekhabhala karate haim|
sharirika mulyankana, shravana skrina aura anya jancha, apako bachche tatha uttama sambhava prarambha ka vishvasa deta hai|
hama vyaktigata nirdesha tatha hamara laipha aphtara dilivari nirdesha ke satha ghara ko jana sahaja kara denge|
nai mam karmachari narsa ki prarthana kara sakati hai aspatala chhoda़ne ke dusare dina kisi bhi prashna ke uttara ke lie tatha punah ashvasana pradana karane ke lie|
memoriyala haॉspitala nyuborna intensiva keyara yunita (NICU) asamayika tatha anokhi navajata vimari apani uttama vikasita dekhabhala ke lie rashtriya stara para jana jata hai|
memoriyala ki NICU rijanala repharala sentara uttari bharata tatha dakshini pashchimi mishigana ke parivarom ko sevaem de raha hai| 
hamare nenotolojista, chikitsaka jo bimari aura aparipakva bachchom ki dekhabhala mem visheshajna haim dekhabhala karane vale peshevara jo apake bachche ke lie uttama dekhabhala pradana karane para kendrita hai|
adhunika upakarana tatha ilaja upalabdha haim|
mata pita ratabhara ke kamare, samajika sevaem, pashuchari dekhabhala tatha NICU perenta saporta grupa bhi parivarom ki madada ke lie upalabdha hai|
apake bachche ke vikasa ka pratyeka mila ka patthara vyaktigata kitaba mem kaida kara lie jisamem mata pita narsa, chikitsa tatha anya ka yogadana ho sakata hai|
eka keara kordinetara apake bachche ke gamana ke lie yojana me madada karega aura anya prabandhakom se aspatala se ghara taka sahaja paragamana ka vishvasa dilata hai|
memoriyalsa nyuborna intensiva keyara yunita (NICU) 36 bistarom ki ikai hai navajata ko chikitsa athava shalya dekhabhala vyapaka stara III pradana karata hai|
400 se jyada navajatom ka janma memoriyala hua athava anya kshetriya aspatalom se NICU parivahana samuha dvara sthanantarita kara die gae NICU prati varsha pravesha lete haim|
uttari mukhya bharatiyom ke ke lie memoriyala ka NICU kshetriya kendra hai aura mishigana ka dakshina pashchimi bhaga hai|
memoriyala ka NICU antarrashtriya rupa mem bachchom, parivara tatha karmachariyom ke lie uttama vatavarana vikasita karane ke lie jana jata hai| 
NICU ne rashtriya tatha anatarrashtriya detabesa tatha anusandhanom mem sakriya bhumika nibha kara kiya|
NICU karmachari borda pramanita menotolojista, peditrishiyana, navajata narsa abhyasi, namankita narsanarsa, prajanana chikitsaka, dudhasravana sangrahakara, chhutti sanyojaka, vikasa visheshajna, samajika karyakarta tatha abhibhavaka dekhabhala sanyojaka| 
karmachari bachchom ke tatha unake parivarom ke jivana mem sudhara karane ke prati samarpita hai|
memoriyala ka NICU ka parivara kendrita dekhabhala pradana karana uddeshya hai|
sabhi bachche vishesha rupa se apane dekhabhala karane mem madada karata hai|
bachche ke vyavahara ko janana tatha sansara ke tarike apako usaki dekhabhala uttama tarike se karane mem madada karata hai|
aise lakshana ki apaka bachcha batachita karane ke lie taiyara hai, yaha achchha samaya hai use pakada़kara bata karane, dudha pilane tatha apane bchche ke satha khelane ka|
apaka bachcha utprerana se abhibhuta hone ke lakshana, apaki parivartana ke lie avashyakata, thakana mahasusa karana, pareshani tatha darda hai| 
unhe pakada़e jaine, bata karane khelane athava dudha pine ke samaya arama ka avashkata ho sakati hai|
punah arama pane mem madada karane ke lie unhe laghu antarala, lambe arama athava anya vyavadhanom ki avashyakata ho sakati hai| 
svata- niyamana : vo lakshana jisamem apaka bachcha apane apa ko aramadayaka karane ke lie prayasa karata hai| 
pratyeka bachche ka apana athava vyaktigata sanchara ka tarika hota hai|
samanya sanketom se jana pahachana tatha unhe pahachanana tatha apane bachche ke vishesha sampreshana ka pratyuttara dena apako apane bachche ki dekhabhala mem jaba apa ghara jaenge to madada karega|
abandha sanketa – yaha khelane, pakada़ne athava apane bachche ko dudha pilane ka sahi samaya hai|
jalda hi janme bachchom se sampreshana akasara dila se tatha NICU mem shvasana maॉnitara dvara padha़a ja sakata|
NICU karmachari apaki maॉnitara ke satha jana pahachana karane mem madada karega jabapake bachche achchhe se padha़engem| 
bachche jo aramapradaka hom shanta tatha aramadayaka dikhata hai tatha vo gulabi tvacha vale hote haim| 
chhote bachche samanyataya mem banda kara dhake jate haim|
apa unake chehare ke hava bhava ko dekha sakate haim aura unhe jaise hi pakada़engem unaki gatividhiyom ko mahasusa karenge|
jaba bachche antahkriya karengem vo dekhabhala karane vale aura khilaunom para kendrita hote haim|
unake chehare ke havabhava aise dikhate haim ve jo dekate tatha sunate haim unamem ichchhuka hote haim|
unako aramadayaka sthiti mem pakada़ kara aura unake hatha tatha pamva ko sahara dene mem madada kara sakata hai| 
chehare ka eka chamakadara satarka bhava, sahaja hatha ke gati tatha khula aura athava ana chehara batata hai ki apaka bachcha khelane ke lie taiyara hai|
jaba apaka bachcha ankha kholega aura apane hatha chehare se upara uthaega (yaha usa samaya hota hai jaba khane ka samaya ho) vo apako bhukhi bhangima dikha sakata hai|
kai bara kai chija़em itani sadharana hoti haim jaisa apake bachcho ki ankhom ko pratyaksha prakasha se bachane tatha usaki ankhom ko kholane mem madada karega tatha yaha dikhata hai ki vaha vastava mem uthaa hua hai tatha dudha pine athava antahkriya ke lie satarka hai| 
sahaja shvasana (apa maॉnitara para dekha sakate haim lekina apa apane bachchom ki sansa/ mahasusa dekha sakate haim| 
narsa tatha daॉktara apase apake bachche se sambandhita sansa dara ke bare mem bata karengem|
unaki ankhom se apaki tarapha muda़na, unake sampurna (chipakana) eka tarika hai ye kahane ka ki apaka bachcha aramadayaka tatha ichchhuka hai|
shanta (gatividhi ko rokana) dikhata hai ki apaka bachcha apane dhyana ko kendrita kara raha hai – apa apane bachche ki madada kara sakate haim usako shanta rakhakara athava kebala mem chipaka kara athava sirpha use pakada़ kara athava usake hatha apane hatha mem lekara|
use arama ki avashkata ho sakati hai, prerana se antarala lijie athava niche rakhie|
tanava sanketa dila tatha shaavasana maॉnitara chetavani bhi shamila kara sakate haim|
yaha hastakshepa ki avashkata ka sanketa karati hai, apake bachche ki narsa bhi yadi avashyakata hui to apake bachche ko sahayoga degi|
bachche kai karanom se rote haim haim – prarambha mem bhukha ko vyakta karate haim, daipara badalane ki avashyakata para tatha ghabarane tatha rone para goda mem lene ki avashyakata para| 
apa apane bachchom ka rona nahim sunengem lekina apa usake hava bhava dekhakara bata sakate haim aura sharira ki gatividhiyam jo apake bachche ke arama ki avashkataem lambe samaya ke lie rote haim jaba apa apane bachche ko shanta karane mem saphala nahim hota to yaha ingita hota hai ki vaha bimara hai athava use darda hai| 
apa apane bachche ko ghara le jate haim; apakoapane daॉktara ko yadi apaka bachcha bahuta rota hai bulane ki avashyakata hogi|
ubasi lena hamesha yaha artha nahim deta ki apake bachche ko sone ki avashyakata hai – balki usaka artha hai ki vaha arama ka avashkata ko batane ka prayasa karate haim|
kai bara bachcho ko sirpha sidhe dekhana tatha batachita ki prerana se chhote antarala ki avashyakata hoti hai|
thakana athava samanyataya aramadayaka rupa mem na hona yaha ingita karata hai ki apake bachche ko jyada arama athava sthiti mem athava kuchha arama mem sahayoga ki avashyakata hai|
ungaliyom ka tirachha hona (pyara ke nama rukavata chinha) akasara bachchom ka prathama lakshana hai jaba ve harana prarambha karate haim|
ghabarana athava munha banana eka tarika hai jisase apaka bachcha batana chahata hai ki use badalava ki avashyakata hai|
gahari ninda athava nidralu tarike hai jisase bachche uttejana se dura hote haim|
teja khansi, teja sansa, epinaya (sansa rukana), khaansi munha banda hona, gala rukana tatha thukana bhi tanava ke lakshana haim|
svatah niyamana sanketa ka vyavahara tatha gatividhiyam jo apake bachche ko apane apa ko madada karane mem shaanta rahane tatha dhvani utprerana, prakasha, karyakalapa athava pareshani ki vyavastha mem prayoga hota hai| 
ungaliyom ko chusana, agutha athava samaka bachchom ka svayam ko shanta karane ka tarika hai|
chehare para hatha rakhana athava hathom mem jakada़na dono tarike bachchom ko eka tarapha khinchate haim|
bachche shanta aura atirikta gatividhiyom ko niyantrita karate haim aura dekhabhala karane vale ke hathom athava navajata ke kapada़om, mulayama hathom ko ragada़kara sahara pate haim|
bachche dhyana akarshana ke lie, ninda tutane para saikila chalate haim athava samaya samaya para batachika ke daurana ankhem banda karake arama karate haim|
bachchom ki jyada svatah shanta pravritti arama prapta karane tatha utprerana se sambandha rakhane ke lie juka athava chadha़ sakate haim|
drishya kendra eka tarika hai jisamem bachche apani gatividhiyam rakha sakata hai aura shanta ho jate hai
apake bachche se bata karem jaba vaha ghabarata hai apako bachche ko shanta tatha kendrita karane mem madada kara sakata hai

bina khilaune athava chatai usake chehare ke pasa ghara para apake bachche ko apani pitha ke bala sona chahie| 
apake bachche ki vishesha dekhabhala ke bare mem narsa ko bataem|
apane bachche ko vyaktigata rupa se janem tatha apane bachche ke sampreshana ko dekhem tatha pratikriya dem apako eka dusare ko janane tatha parivara ki vriddhi mem madada karega|
bachche ki dekhabhala ke ankada़e vistrita sarani mem prastuta haim 2004 ke sarve ऑpha inkama ke chauthe sakshatkara tatha 2005 ke pharavari marcha mem sanchalita programa partisipeshana painala se ae haim aura bachche ki dekhabhala vyavastha ko sakshatkara se pichhale mahine prayoga kiye gae ko sandarbhata karata hai| 
bachche ki dekhabhala ki SIPP se sangrahita suchana kai salom mem taiyara ki gai hai jisamem bachche ki dekhabhala prayoga para vyapaka ankada़e kimata tatha sarakara ke sahayoga ki prapti shamila haim|
1966 SIPP penala ke satha prarambha, bachche ki dekhabhala ke ankada़e 15 se kama unra ke sabhi bachchom ke lie abhibhavaka ke bala ki sabhi sthiti ke alava prabandha ke bare mem sangrahita kiye gae|
SIPP ke purane penala ne sirpha simita sankhya mem bachchom ke lie prathamika tatha dvitiyaka bachche ki dekhabhala prabandha para ankada़e sangrahita kiye, aura samanyataya sipha kamakaji abhibhavakom ke|
SIPP bachcha dekhabhala mapadanda ke samanya prativadiyom mem vo vayaska haim jo 15 sala se kama unra ke bachchom ko abhibhavaka haim|
jivana ke prathama tina sala ke daurana bachcha jyada samaya bina mam ke satha vyatita karata hai to bachche ki apani mam se kama sakaratmaka antahkriya hoti haim|
neshanala istityuta ऑpha chailda heltha tatha arli chailda ke hyumana davalapamenta stadi para adharita deshantariya tathyom se lagabhaga 1,300 bachche|
lekhaka bachche ki dekhabhala tatha kahim mam bachchom antahkriya ki matra visheshata tatha sthirata jaba bachche 6, 15, 24 tatha 36 mahine ke bachi sambandha dekhata hai|
parivara tatha unake bachche pure yunaiteda stetsa ki anusandhana saitsa se niyojita haim|
niyojita parivarom ke chaubisa pratishata alpasankhyaka manavajati ke sadasya hote haim|
upalabdhiyom ke anusara bachche jo niyamita bina mam ki dekhabhala ke samaya vyatita karate haim upani mam se kama sakaratmaka antahkriya karate haim una bachchom se jo kama athava na ke barabara bina mam ki dekhabhala mem samaya vyatita karate haim|
bachche ki dekhabhala mem ghanto ki sankhya mem vividhata donom mam ka bachche ke prati vyavahara tatha bachche ke mam se antahkriya ke sakaratmaka viniyojana se sambandhita haim|
prapti sanketa karati hai ki bachche ki dekhabhala mem lambe ghante sambandhita haim kuchha hasamana tatha mam ki kama yogyata aura eka dusare ke satha bachchom ke satha|
halanki dekhabhala ki vyavastha (ghara, kendra, rishtedara ka ghara) parinamom tatha dekhabhala ki visheshata ko parivartita nahim karata|
uchcha koti ki bachche ki dekhabhala matra samvedanashilata ki badha़ta se sabandhita haim|
uchcha koti ki dekhabhala mam se sakaratmaka bhumi ka bachche se jatila, samvedanashila antahkriya pradana karata hai|
jyada matra samvedanashilata uchcha koti bachche ki dekhabhala ke prabhava bachche ki ubharati kshamata, vyavahara svikriti tatha samajika kshamata ka phalana haim|
lekhaka ne payi gai mata-bachcha antahkriya aura kai ghanto ki dekhabhala ke bicha kuchha nakaratmaka sambandhom ko ingita kiya aura dekhabhala ki gunavatta se sambanndhita mam ke utpada jisamem bachche ki dekhabhala, dekhabhala ke prabhava ke bicha juda़ava mem kuchha sakaratmaka sambandha haim|
parinama sanketa karate haim ki mam jo apane navajata sanketom ke prati kama samvedanashila haim athava jinake bachche kama vachana baddha haim jyada ghantom ke lie bachche ki dekhabhala karate haim|
parinama yaha bhi salaha dete haim ki mam jo jyada samvedanashila haim apane bachchom ko uchcha koti ki dekhabhala chunate haim|
satha hi bachchom ki dekhabhala mem visheshata ke parinama rakhane mem tatha upayukta sandarbha mem sthirata ke lie lekhaka anya mam-bachche ki antahkriya ko dekhata hai|
eka bachche ka acharana tatha mata ke vyayita hone ke lakshana bachche ki dekhabhala unaki mata-bachcha antahkriya anumana lagane ki kshamata ke sandarbha mem gunavatta vahi rahenge|
isake bada़e pratidarsha ke karana, NICHD ankada़e anusandhanakarta ko bachche ki dekhabhala tatha bachche ke bicha sambandha ko pahachana leta hai |
vashingatana mem amerikana saikolojikala esosieshana, DC sabase bada़i vaijnanika aura vyavasayika sangathana hai yunaiteda steta mem aura vishva manovejnanikom ki sabase bada़e sangathana ko manovijnana ka pratinidhitva karati hai|
APA ki sadasyata mem 159000 se jyada shodhaka, shikshaka, klinishiyana, paramarshadata tatha chhatra haim|
halanki yaha manovijnana ke 50 bhaga vibhajita haim aura 58 rajyom se manyata prapta tatha anshakalina kenediyana pradeshika samaja, APA vijnana ki taraha utkrishta manovijnana ke lie vyavasaya ke rupa mem aura manava ke bhalai ke vikasita karane ke artha mem kama karata hai|
niyukta abhibhavaka gharom mem chune jate hai jo donom abhibhavaka pratyeka bachche ke lie bachche ki dekhabhala ki vyavastha ki riporta prastuta karate haim|
shadi shuda dampatti mem mam niyukta abhibhavaka hoti hai|
yadi sakshatkara ke lie mam upalabdha nahim haim pita athava pati se pratikriya prapta kara lete haim|
ekala-abhibhavaka parivara, nivasi abhibhavakom ko abhibhavaka niyukta kiya jata haim|
yadi koi bhi abhibhavaka ghara mem nahim rahate, sanrakshaka ko abhibhavaka niyukta kiya jata hai|
niyukta abhibhavaka jaivika charana tatha goda lie abhibhavaka athava anya sambandhi/gaira sambandhi abhibhavaka ki anupasthiti mem sanrakshaka ki bhumika nibhate haim|
bachche ki dekhabhala pradana karane vala mote taura para bachchom ke sambandhi athava gaira sambandhi vargikrata karate haim|
sambadhiyom mem mam, pita, bhai-bahana, dada-dadi aura anya sambandhi jaise ankala, anti athava kaja़na shamila haim|
gaira sambandhi mem shamila hai ghara mem bebisitara, pada़osi, mitra, tatha anya gaira sambandhi bachchom ko dekhabhala ke lie ya to ghara para athava ghara pradana karate haim|
isamem parivara ke dina mem dekhabhala pradana karane vale bhi shamila haim jo gaira sambandhi hai jo eka athava jyada gaira sambandhi bachchom ko ghara mem dekhabhala dete haim|
vyavasthita dekhabhala mem shamila hai, dina ki dekhabhala athava skula se purva tatha sangha ke heda prarambhika programa una 5 sala ke athava chhote bachchom ke lie haim|
bahuta chhote bachchom ke skula/stariya skula mam vyasthita dekhabhala kula 5 sala se chhote bachchom ke lie bhi shamila hai|
15 sala se chhote bachchom ke satha niyamita saptahita anubhavom ke vyapaka drishya ki prastuti isa riporta mem skula mem namankita bachchom ki ghatanaom aura mahatvapurna karya kalapom dikhaya hai jaise khela, patha, klaba aura skula ke bada pahale dekhabhala ka programa|
kuchha abhibhavaka vyavastha jo niyamita rupa se kama ane vale kama se kama eka saptaha ke lie ullekha karane mem asamartha hote haim aura vyavastha nahim ke rupa mem uttara dete|
chhote bachchom ke skula/stariya skula mem namankana svahdekhabhala mem bitaya samaya ko suchana ke uddeshya se saranibaddha kiya hai taba bhi jaba unhem bachchom ki dekhabhala ki vyavastha mem shamila nahim kiya gaya|
satha hi bahu-vyavastha shabda niyamita rupa se bachchom ki eka se jyada bachche dekhabhala vyavastha prapta karane vale bachchom ke lie istemala kiya jata hai|
SIPP bachcha dekhabhala ikai ki ruparekha tatha murtarupa mem nishchita sankhya unra vichalana mem netritva karati hai|
unra vichalana bachche ke janmadina para bachcha dekhabhala programa mem namankana tatha SIPP sarvekshana hone ki bicha ki samaya antarala mem hone vale dina ghatita hota hai|
udaharana svarupa pancha sala bachcha nammakita hai to chhah para branhanda se bahara hoga sarvekshana samaya para chala gaya heda starta prashna sakshatkara dinanka parinamatah heda starta mem bachchom ki sankhya kama ho jaegi|
heda starta namankana ki anya suchana hedastarta ke ऑphisa mem milegi|
heda starta eka rashtriya programa hai jo vidyalaya ki padha़ai ko samajika tatha bachchom ke sarvaganiya vikasa shiksha, sehata proshaka, samajika tatha anya sevaom dvara namankita bachchom tatha parivara ke atkarsha ko protsahita karata hai|
heda starta programa sthaniya janata tatha svayata labha athava/na labha sevaom ki arthika hani bachche tatha parivarika manyata pradana karati hai|
fy 1995 mem purva starta programa bachche ke janma se tina sala ki unra taka sevaem dete haim vyavasthayika ghatanaem jo prarambha ke sala bachche ki vriddhi tatha vikasa mahatvapurna hoti hai|
heda starta programa skuli shiksha ko bachche ki samajika tatha manasika vikasa ko utkrishta karane ke lie protsahika karata hai| namankita bachche tatha parivara ki shiksha, sehata, poshaka, samajika, tatha anya sevaem dvara|
ve abhibhavakom ko unake bachchom ke jnana mem vyasta rakhate haim aura unaki shiksha, jnana tatha rojagara uddheshyom ki vikasa dara ko banaem rakhane mem madada karate haim|
sthaniya heda starta programa ki vyavastha mem abhibhavakom ke ulajava ko sarthaka mahatva diya jata hai|
heda starta ki achchhe ke lie vachana baddhata bachchom ke svasthya, parivara tatha karmachari ke vyapaka chitrana ko angikara karata hai|
chailda heltha davalapamenta sarvisa ka uddheshya yaha nishchita karana ki parivara karmachari tatha svasthya visheshajnom tatha vikasa ko pahachanane mem samanjasya hoga|
shiksha tatha arli chailda huda vikasa ka uddeshya unaki madada ke lie sabhi bachchom ko surakshita, poshana, ulajava, ananda tatha surakshita jnana yukta vatavarana pradana hoga| 
aja ke vatavarana mem vishvasa saphalata ke lie jaruri hai aura bada skula tatha jindagi ki jimmedariyom ke lie bhi|
pratyeka bachche ko vyaktigata rupa se samaja ke mulyom adara tatha vividhata ka pratyuttara ke lie dekha jata hai|
programa ke dvara vividha anubhavom ko pradana karata hai, bachche ki vriddhi tatha vikasa ko sahara dene ke lie pratyeka bachche ka jisamem bhotika, samajika, bhavuka tatha manasika vikasa shamila hai|
heda starta abhibhavakom ko vriddhi ke lie avasara tatha sahara pradana karane ke avasara deta hai jisase ve apani kshamata, jarurata tatha ichchha ko pahachane aura unake hala nikalem|
phemili partanarashipa ke uddeshya abhibhavakom ko unaki jaruratem pahachanane aura unake uddheshyom taka pahanchane mem unake bachchom ke vikasa ko unake parivara aura sanskriti ke sandarbha mem dekhabhala karana, sabhi sanskriti samaja ke bachchom tatha parivara ki pairavi karana hai|
vishvasa, samuhika sambandha abhibhavaka tatha karmachari banate haim unhem eka dusare se sikhane tatha vichara adana pradana ki anumati pradana karate haim|
heda starta parivarom ke lie samudayom mem kama karate haim aura pahachanate haim ki kai anya sansthaem aura samuha unhim parivarom ke satha kama karate haim|
kanyuniti partanarashipa ka uddeshya yaha vishvasa dilana hai jo anudayi aura pratinidhi shakhaem apanu samudayom ke sathiyom se bachchom tatha unake parivara ko uchcha stariya sevaem pradana karane ke lie samparka karate haim|
parivarika kendrita sevaom ke vikasa ka palana aura sabhi sanskriti ke bachchom tatha parivara ke svasthya vriddhi ki jimmedari bantane vale samudayom ki pairavi|
julai 11, 2006 heda starta ka ऑphisa ghoshita hua aura arli chailda huda larninga tatha noleja sentara (ECLKC) ko anrabha kiya|
ECLKC prasangikata, samajika suchana jnana tatha heda starta programa sikhana aura bachapana ki purvavastha mem upayoga asana hai|
ECLKC vikasita hoga aura agale kai sala mem kramika vikasita hogi aura sampurna sadhana kisi ke lie purvavastha mem upayoga asana hai|
heda starta samudaya ka inaputa ECLKC ke bhavishya ke vikasa ke lie chunotipurna haim|
kripaya saita ko ankita karem aksara jaem aura unaki salaha tatha tippaniyam hamase bante|
apake sahayoga ke satha hama ECLKC ko arli chaildahuda samudaya ke lie upalabdha mahatvapurna jariya bana sakate haim|
hama apa sabake satha kama kara isa sadhana ko jo hama jisa prakara vyavasaya ko parivartita tatha badha़a sakate haim|
neshanala instityuta ऑpha chailda heltha tatha hyumana davalapamenta (ECLKC) stadi ऑpha arli chailda keyara ne sthapita kiya ki yaham ghantom ke bachchom palane mem bachchom ki dekhabhala mekam bitane mem satata sabanndha nahim hoga|
turanta prakashita vishleshana mem mam-bachche ki antahkriya vishleshana batata hai ki navajata ki dekhabhala mem vyatita ghantom ki sankhya sadharanataya mata ki bachche ke prati samvedana se usi prakara juda़i hai jaise mam ki bachche ke satha khela mem ulajava|
yaha juda़ava paryapta kama hota hai jaise yaha paristhitiyom ke bicha jisamem anya anumanakom ki mata bachche ki antahkriya ki gunavatta barabara hai athava santushtipurvaka mahatva diya jata hai|
davalapamenta saikoloji janakari navammbara prakashana mem payi gai hai|
khojakartaom ne bachche ki dekhabhala tatha matritva bhavukata ke bicha gunavatta mem halka juda़ava hai, isaka artha hai bachche ki dekhabhala mem mata ki samvedana mem uchcha stariya achchhi gunavatta|
yaha janakari usi vishleshana ki purva janakari ke samana hai dikhati hai ki dekhabhala ki uchcha gunavatta jyada pyare prabhava dekhabhala ki kama dekhabhala se sambandhita hai|
udaharana ke lie jaba bachche ki dekhabhala pradana karane vale bachchom se bata karate haim, prashna puchhane ke lie utsahita karate haim, bachchom ke pashnom ka pratyuttara, unhem padha़na, anyom ki bhavanaom ko samajane ki chunoti dete haim|
alaga prakara ki socha se bachchom ki bhasha yogyata tatha socha kshamata una paristhitiyom se achchhi hai jo kama sanriddha haim|
turanta prakashita lekha mem lekhaka ne paya ki bachchom ki dekhabhala ke ghantom ke lie parinama aura bachchom ki dekhabhala ki gunavatta prabhavita nahim hai chahe bachcha bachche ki dekhabhala ke eka prarupa mem athava anya mem|
donom ‘‘ mata samvedana ’’ tatha ‘‘bachcha ulajava’’ avalokita vyavahara tatha NICHD stadi ऑpha arli chailda keyara karmachari dvara mulyankita ko varnita kiya|
mata samvedana batati hai ki kaise mam bachche ki chahata aura avashyakata se juda़i hai|
bachche ki ulajava batata hai ki kaise bachche se juda़ava athava sambaddha samane ata he jaba vaha usaki mam se sambandhita hota hai|
bina matritva dekhabhala mem vyatita badha़a hua samaya aura mata bachche ki antahkriya ki gunavatta mem kami ke bicha juda़ava mam ki shiksha ke uchcha stariya juda़ava se kama hota hai|
isake alava bachche ki dekhabhala mem vyatita samaya aura mam bachche ki antah kriya ke bicha sambandha mam ke vyathita hone aura mata samvedana ke bicha sambandha ke barabara hai|
aise mam-bachche ki antahkriya ki visheshata ke anumanaka mem parivarika aya, mata shiksha, mam vyatha aura mam ka shadi shuda avastha (athava sathi hone ki avastha) bhi shamila hai |
jaba mam-bachche ki antahkriya ki visheshata ke anumanakom ke sandarbha mem jaba parivara eka samana nahim ho bachche ki dekhabhala me vyatita samaya aura mam-bachche ki antahkriya ke bicha sambandha pahachanana kathina hai|
purva vishleshana hai ki kya bachche dekhabhala mem vyatita samaya unaki mam se atahkriya ko prabhavita karate haim aura viparita parinama hote haim|
adhikatara purva vishleshanom ne abhasa kiya ki kevala kuchha sankhya mem bachche ke vyavahara ko kevala eka unra samuha ke lie mapa jae aura bachchom ko bina mata pita ki dekhabhala mem kuchha ghantom ke lie shamila karane mem asaphala ho gae|
NICHD stadi ऑpha arli chailda keyara sirpha 1300 se jyada bachchom ko janma ke samaya pure yunaiteda steta se 10 shodha saitom mem namankita kiya|
vishleshana mem bhagidara bachchom, ko bachchom ki vividha dekhabhala ki vyavastha mem rakha jata hai yaha mukhya aupacharika se anaupacharika taka chalati hai|
vishleshana NICHD ke janchakarta ke dvara kiya jata hai aura pure desha ke 14 vishvavidyalayom dvara kiya jata hai|
halanki vishleshana ka rashtriya pratinidhitva ho ki ruparekha nahim thi, vishleshita parivara ki ausata aya rashtra ke gharom ki ausata aya ke samana thi 
alpa jatiyom ka pratishata vishleshana mem pure rashtra ki alpa jatiyam ke pratishata ke samana tha|
mam aura bachche ki antahkriya 10-15 minata ke satra mem vidiyotepa kiya tha jaba bachche 6,15,24 tatha 36 mahine ki unra pratyeka ke 1300 bachchom ke kula eka ghanta jisake lie lekhaka ne purna suchana di|
antahkriya ki visheshata ko mam ki bachche aura usake bachche ke prati samvedana ke sandarbha mem aura bachche ke ulajava apane mam se yugma khela satra ke daurana mulyankana hota hai|
mam ka unaki bachche ke prati samvedana ke sambandha mulyankana hota hai, jaba bachche vyathita nahim ho|
paryavekshakom ne mam ke prati samvedana avastha ko antarvedhana ki avastha aura sakaratmaka sammana ki avastha jo vaha bachchom ko dikhate haim ka mulyankana karate haim|
bachchom ki viniyojana tatha sakaratmaka socha ko mam-bachchom ki antahkriya prabhava ko gati deta hai|
chunki bachchom ki dekhabhala ke anubhava bachchom ke parivara ki prishthabhumi para adharita nahim hai shodhakarta bhi parivara para mam-bachche ki antahkriya ki visheshata ke prabhava ko padha़te haim|
matri evam shishu dekhabhala karyakrama, yunaiteda charcha ina ऑstreliya ke charcha ka sathi, karyakrama hai|
isa pariyojana ka uddeshya nagapura, melaghata aura churni mem sabase adhika prithaka aura alabhakari kabile kshetrom mem se do mem matri evam shishu dekhabhala ko badha़ava dena hai| 
pariyojana mem kai bhinna bhinna prabhaga shamila haim – mataom ko mulabhuta poshana aura bachavakari svasthaya dekhabhala ke bare mem padha़ane ke lie prashikshana karyakrama|
kuposhana ke khilapha lada़ai ke lie eka poshana apurti karyakrama mem bhaga lene ke lie 5 se kama ayu ke bachchom| 
samanya svasthaya dekhabhala satha hi satha kama se kama hara chhah maha para niyamita jancha pradana karane ke lie shishu dekhabhala kendra|
melaghata aura churni maharashtra rajya mem sabase adhika prithaka aura alabhanvita do kshetra haim|
bimari aura kuposhana vyapaka rupa mem phaile haim aura bachavakari aura sadhya bimariyom pratidina mara rahe haim| 
kai sarakari vikasa upakramom ne ina kabilai kshetrom ko darakinara kara diya hai kyonki ye bahuta prithaka haim aura yatayata hamesha eka bada़i samasya hai|
loga yaham bhugata rahe haim aura UCOA vishvasa rakhata hai ki hamem apane sathi, charcha ऑpha naॉrtha indiya eka samaja jaham pratyeka vyakti ko jisamem gariba, dalita aura alpasankhyaka bhi shamila haim, jivana ki mulabhuta avashyakataem milati hai, banane ke mishana mem sahayata deni chahie| 
charcha ऑpha naॉrtha indiya ka nagapura diosija़ matri evam shishu dekhabhala pariyojana ke sanyojana ke lie uttaradayi hai|
kabile gamva mem mam evam shishu dekhabhala ko sudharane aura badha़ava dene ke kai aura tarike haim jinaka lakshya pariyojana rakhati hai|
gamvom mem dasa shishu dekhabhala kendrom ki sthapana|
yogya narsom vale eka svasthaya kendra ko jo gamva ko seva pradana karata hai, kosha pradana karana|
sabhi mahilaom aura bachchom ke lie hara chhah mahinom mem svasthaya jancha karavana|
bachava svasthaya dekhabhala batane ke lie sthaniya mahilaom ko mulabhuta svasthaya karyakartaom ki taraha prashikshana dena|
vibhinna sarakari nikayom se sambandha vikasita karane ke lie svasthaya karyakartaom aura gramina vayaskom ki sahayata karana|
pancha varsha se kama ayu ke sabhi bachchom ke lie poshana puraka karyakrama vikasita karana|
svasthaya karyakartaom ki sahayata karate hue aura shiksha evam chikitsakiya dekhabhala pradana karate hue eka narsa gamvom ke bicha ghumati hai|
UCOA ne use gamvom ki yatra karane yogya banane ke lie eka motarabaika upalabdha karai hai|
melaghata aura churni, do kshetra jaham pariyojana chala rahi hai, maharashtra rajya ke amaravati jile mem satapuda़a parvata shreniyom mem sthita hai|
amaravati jila nagapura rajya ke dusare shahara (mumbai rajadhani hai) ke lagabhaga 200 kimi pashchima mem hai|
kshetra mem kuchha gamva parvatiya paristhiyom ke karana sada़ka dvara bilkula pahuncha ke bahara hai aura anya gamva kabhi kabhi kharaba mausama ke karana bahari duniya se bilakula alaga ho jate haim|
UCOA ne matri evam shishu dekhabhala pariyojana ke lie apani sahayata 2002 mem shuru ki|
svasthaya kendra chalana aura karyakartaom bachchom ke lie poshana purakom ka bhugatana karana, gramina svasthaya karyakartaom ka prashikshana|
sada़kom para rahane vale adhikansha bachche anapadha़ haim aura unake pasa siksha bahuta kama ya bilkula nahim haim|
yaha pariyojana bachchom ko arthapurna vyavasaya pane mem sahayata karane ke lie shiksha lene ya vyavasayika kaushala prapta karane ke lie protsahita kara rahi hai|
yaha unhe garibi aura vinasha jisamem ve jakada़e hue haim, ka eka vikalpa de rahi hai|
da childrana ina da kraisisa projekta charcha ऑpha sautha indiya, bharata mem hamare sahakarmi charchom mem se eka, ki eka pariyojana hai|
pariyojana baingalora shahara ki sada़kom para rahane vale bachchom ke satha kama karati hai|
satha hi satha bachchom ke rahane ke lie surakshita jagaha aura unake khane ke lie bhojana de rahi haim|
pariyojana bachchom ko skula lautane ke lie aura shiksha prapta karane ke lie ya vyavasaya prapta karane mem sahayata dene ke lie vyavasayika prashikshana karyakrama mem bhaga lene ke lie protsahita karati hai|
gali ke bachche asani se giroha mem khincha liye jate haim aura phira hinsa aura chhoti moti chori mem phansa jate haim|
adhikansha bachche kachara binane valom, phala bechane valom, juta paॉlisha karane valom aura jeba katarom ki taraha kama karake paisa kamate haim|
yaha pariyojana bachchom ko sada़kom para isa taraha ke jivana ka vikalpa dene ka uddeshya rakhati hai|
baingalora shahara ne bharata mem pramukha audyogika aura vyaparika kendrom mem se eka ki taraha vriddhi ki hai| 
isi ke satha jompada़patti kshetrom ki matra satha hi satha sada़kom para rahane vale bachchom ki sankhya bhi badha़i hai|
ajakala baingalora mem 650 jompada़patti haim jo sarakara dvara svikrita haim|
yaha anumana lagaya gaya hai ki shahara mem lagabhaga 45,000 sada़ka para rahane vale bachche haim|
ye bachcho ube hue haim, unake jivana ka koi uddeshya nahim hai aura isalie ve kuchha karane ke lie galiyom mem yum hi ghumate haim| 
yaha akasara unamem samaya gujarane ke lie gali ke girohom aura aparadha mem phansa jane ka parinama lata hai|
sada़kom para rahane vale adhikansha bachche anapadha़ haim aura unake pasa shiksha bahuta kama ya bilakula nahim hai| 
yaha pariyojana bachchom ko arthapurna vyavasaya pane mem sahayata karane ke lie shiksha lene ya vyavasayika kaushala prapta karane ke lie protsahita kara rahi hai|
yaha unhe garibi aura vinasha jisamem ve jakada़e hue haim, ka vikalpa de rahi haim|
‘da childrana ina da kraisisa projokta’ ऑstreliya ke sahabhagi charcha, da charcha ऑpha sautha indiya, mem yunaiteda charcha ki eka gatividhi hai|
karnataka sentrala dayosija़, jisamem baingalora sthita hai, pariyojana ka samanvaya karata hai|
‘da childrana ina da kraisisa projekta’ do mukhya kendrom ke charom ora kendrita hai donom shaharom ke bicha mem sthita hai|
mukhya kendra ko ‘makkala ashraya kendra’ kahate haim jisaka artha hai ‘bachche jo sankata mem haim ke lie sharanarthiyom ka ashraya’
dusara kendra 2002 ke dusare bhaga mem sthapita kiya gaya tha aura bada़e kamare me sthita eka dainika kendra hai jise ekasthaniya charcha ne pariyojana ke prayoga mem lane ke lie anumati di hai|
2002 mem yaham vastava mem kendra para pur samaya rahane vale 40 bachche the, baki bachche kendrom para pratidina ate haim|
kendra ke samajika karyakarta aura anya karyakarta pure shahara mem nirdishta sthanom para jaise ki relave steshanom aura bajarom mem bachchom se milate haim| 
relave steshana ko malakata ke sthanom mem se eka ki taraha chuna gaya hai kyonki karmachariyom ke lie yaha dusare shaharom se baingalora ane vale bachchom se mulakata ka rasta hai|
baingalora mem sada़ka ke adhikansha bachche vyavasaya aura nai shuruata ki talasha mem ae haim|
karmachari bachchom ke satha sambandha banane ki koshisha karate haim aura unhe kendra para lane ke lie protsahita karate haim|
adhikansha bachche pahale hi hai skula chhoda़ chuke hote haim aura agara vo vapasa nahim lautana chahate to karmachari unhe dusare kaushala sikhane ke lie protsahita karate haim|
kendra para silai ka korsa haim jaham lada़ke samana jaise ki baiga aura narma khilaune banana sikha sakate haim|
kendra se bahara bhi anya korsa haim jaise ki badha़igiri, gada़i chalana aura motara mistri jinamem anya bachche shamila hote haim| 
‘da childrana ina kraisisa’ pariyojana bharata ke bharata ke dakshina mem baingaluru shahara ke madhya sthita hai| 
shahara ki apane apa mem 5.2 miliyana logom ki janasankhya hai|
shahara mem audyogika aura vyavasayika vriddhi ke satha kai samasyaem ai haim, vishesha taura para kai nai jopada़pattiyom ka utpanna hona aura sada़ka ke bachcho mem badha़otari|
2005 mem UCOA ne isa pariyojana ki sahayata ke lie baingaluru ko "26,383 bheje the|
bachchom ke lie isa pariyojana se 450 se adhika bachchom ko sahayata mili hai| 
vartamana mem 6 bachche vyavasayika prashikshana le rahe haim : 2 silai karana, 2 kara marammata karana, 1 vidyuta sambandhi aura 1 chhapai|
baingaluru mem sada़ka ke bachchom ko unake jivana mem bada़i avashyakataem haim|
unamem se adhikansha anapadha़ haim ya unake pasa bahuta kama shiksha hai|
unamem se adhikasha ke pasa sahare ke lie nirbhara hone ke lie koi parivarika jala nahim hai|
kendra nirantara milane vali sahayata ke satha ina bachchom ko protsahana de sakata hai jisaki unhe eka shaikshika aura vyavasayika kaushala prapta karane mem avashyakata hoti hai, satha hi satha unhe avashyakata vale samvegatmaka sahara bhi dete haim|
bachchom ki kuchha avashyakataem haim – bhojana aura kapada़e, kambala aura bistara, prashikshana korsa ke lie sansadhana, paramarsha sevaem| 
vartamana mem pariyojana apane satavem varsha mem hai aura 2006 mem UCOA ne vittiya sahayata mem "22,500 pradana karane ke lie vachanabaddhata di hai| 
yaha eka varsha vali shama thi jaba uda़isa ke kabile keraputa jile mem nauguda़a gamva ki madhu ko prasava vedana shuru ho gai|
vaha prasava ki antima avastha mem thi aura usamem jatilataem vikasita ho gai thim jinhe aprashikshita gramina dai (paramparagata dai) sambhala nahim pai|
madhu ki sthiti sankatapurna ho gai lekina usaka parivara use aspatala le jane ke lie anichchhuka the|
kamala khilo, usi gamva ke ‘sailpha hailpa grupa’ ki sadasya, madhu ke bachava ke lie ai aura use gamva se lagabhaga 70 kimi dura jila aspatala le gai| 
phira rajamanda nahim hone para madhu ke pati bahuta gussa hua aura kamala ko dhamaki di ki yadi madhu ya usake bachche ko kuchha hua to phira vaha use mara dega|
kamala ne shantipurvaka daॉktara ka ऑpareshana thiyetara se bahara ane ka intajara kiya|
jaba vaha anta mem bahara aye, daॉktara ne ghoshana ki ki mam aura bachcha donom achchhe the|
daॉktara ne bhai madhu ko samaya para aspatala lane ke lie kamala ko dhanyavada diya, nahim to usaka jivana bachana kathina ho jata|
sukarama, madhu ke pati ne bhi kamala se maphi mangi aura apane navajata bete ko yaha kaha kara de diya ki shishu usi ka hai kyoki yaha usi ke karana hai ki mam aura bachcha jivita bache the|
isa ghatana ka vivarana dete hue kamala ke chehare para ati prasannata jalaka uthi|
kamala khilo nagunda gamva ki chhah mahilaom mem se eka hai jisane yunisepha dvara shuru kiye gaye’ankura’ dvara ayojita ‘madara enda chailda protekshana keyara’ prashikshana prapta kiya hai| 
yaha prashaikshana potangi blaॉka mem 67 gamvom ke 60 svayam sahayata samuhom dvara mahilaom ko diya gaya hai|
chara divasiya prashikshana karyakrama ke antargata esa echa ji sadasyom ko mam ki purva aura uttara prasava prasava dekhabhala aura 0 se 6 maha ki ayu ke navajata shishu ko dekhabhala ke bare mem bataya jata hai|
ve 0 se 3 varsha ke bachchom ki dekhabhala ke lie bhi prashikshita the|
chauthe dina ka prashikshana purva skuliya shiksha, balikaom ki shiksha, skula chhoda़ne vale bachchom ko sanhalana, jala aura svachchhata ke bare mem tha|
jaise ki matri nrityu dara aura shishu nrityudara koraputa mem kaphi uchcha haim, isa prakara ki pahala age jakara parivartana lane lane mem nishchita rupa se sahayata karati hai| 
kabilai jati ko isa tathya ke bare mem jagaruka hone ki avashyakata hai ki kuchha hada taka mam aura shishu dekhabhala ke bare mem unaki anabhijnata bhi jile mem uchcha ma.nri.da. aura shi.nri.da. ke lie uttaradayi hai|
isa prashikshana karyakrama ke antargata hamane unhe na kevala mam aura shishu dekhabhala para prashikshana diya hai balki unhe isa prakriya ka prabodhana karane aura dusare graminom mem jagarukata phailane ke bare mem bhi bataya hai| 
kamala khilo, jisane madhu aura usake bachche ka jivana bachaya tha, svayam bhi isa anabhijnata ka shikara rahi hai|
usane apane pahale bachche ko kho diya kyonki aprashikshita dai ne bachche ke janma ke turanta bada nabhi nada़i ko taila ke eka teja tukada़e se kata tha|
yaha mera pahala bachcha tha aura mem navajata shishu dekhabhala se bilakula anabhijna thi|
hama socha karate the ki navajata ke lie pratirakshana hanikaraka hai isalie hama apane bachchom ko chhupa liya karate the jaba chikitsaka hamare gamva mem ae| 
una dinom hamare adhikatara bachche pratirakshita nahim the, aba yunisepha aura isake prashikshana ne hamari ankhem khola di haim|
ajakala isa gamva mem sabhi bachchom ko pratirakshita kiya gaya hai| pratirakshana se purva sansthaniya prasava koi nahim the, yadyapi sthaniya pi echa si (praimari hailtha sentara) kevala eka kimi dura hai|
aba charom garbhavati mahilaom ne apane bachche aspatala mem kie haim aura koi bhi mahila ghara para aura rasava nahi chahati hai|
maliguda़a bhi dusara gamva hai jau noguda़a ke udaharana ki barabari kara raha hai|
maliguda़a esa echa ji sadasyom ne bhi prashikshana liya hai aura apane gamva mem pathyapathya niyama ko lagu kara rahe haim|
yaham adhikansha mahilaem savadhaniyom ka palana kara rahi haim sivaya kuchha ke jo apani parivarika samasyaom ke karana isa avadhi mem arama nahim le sakati hai|
anyatha hamare gamva mem sabhi 21 bachche aba taka pratirakshita ho chuke haim aura chhah mem se chara mahilaom ke sansthanika prasava hue|
esa echa ji sadasya pratyeka ghara ka daura karate haim aura garbhavati mahilaom ko apani dekhabhala ke lie prerita karate haim satha hi satha bachchom ke lie avashyaka pratirakshana ke prati jagaruka karate haim|
ve unhe svachchhata aura mam/shishu dekhabhala ke bare mem bhi janakari de rahe haim|
yadyapi sarakara ne pratyeka gamva ko svasthaya sevaem pradana karane ka vada kiya hai lekina uchita sanchara vyavastha ke abhava mem kai kabile gamva aja bhi pahuncha se bahara haim| 
2006 mem pahali bara apane panchavem janmadina se pahale marane vale bachchom ki sankhya 10 miliyana se ghata kara 9.7 miliyana raha gai|
pancha varsha ke andara mautom mem dakshina eshiya ka yogadana hai 3.1 miliyana aura bharata ka 2.1 miliyana hai|
isa prakara vishva ki kula bala nrityu mem bharata 2.10% yogadana deta hai|
pancha varsha se kama nrityu dara 2006 mem bharata ke lie 76 anumanita ho chuki hai|
yadyapi bharata ne bala nrityu dara kama karane mem vikasa kiya hai, ausata kami pancha se kama mem 1990 aura 2006 ke bicha 2.6% rahi| 
yadi bharata 2015 taka 30 te lakshya taka pahunchata hai, to use agale salom mem prativarsha ausatana 7.6% ki kami lani hogi|
sansara mem lagabhaga 37% pancha sala se kama nrityu prasava ke karana ho jati hai|
bharata mem pancha sala se kama unra mem marane valom mem lagabhaga 75% navajata hai aura pancha sala se kama marane valom mem lagabhaga adhe janma ke chara saptaha mem hi mara jate hai|
pratyeka varsha lagabhaga 4 miliyana bachche jindagi ke pahale 28 dinom mem hi mara jate haim| prasava samaya
bharata mem yaha sankhya lagabhaga 1 miliyana hai jisaka matalaba hai ki eka chauthai paida hone vale bachchom ki nrityu sansara mem se bharata mem hoti hai| 
janma ke eka ghante mem stanapana ki jagarukata ko shamila karana eka bahuta hi sarala evam asana tarika hai paristhiti mem sudhara karane ka|
lagabhaga 38% bachchom ka vikasashila deshom mem janma ke eka ghante ke andara stanapana prarambha ho jata hai|
bharata mem 23% bachchom ka stanapana janma ke eka ghante ke andara shuru hota hai|
vikasashila vishva mem 19 miliyana shishu kama vajana ke hote haim(2500 grama se kama) bharata mem 8.3 miliyana haim|
pancha sala se kama mem nrityu mem 50% karana kuposhana hai|
sansara mem pancha sala se kama unra mem lagabhaga 50% bachche kama vajana ke haim|
bharata mem kama vajana ke bachchom ki dara 43% hai|
1990 se prativarsha ausata girakara 0.9% ho chuka hai nDG lakshya ko prapta karane ke lie tvarita vikasa ka jarurata hai|
pure vishva ke eka tihai kama vajana ke bachche bharata mem rahate haim|
kama vajana ke bachchom ki sankhya mem sabase jyada madhyapradesha, jarakhanda evam bihara haim usake bada gujarata, uda़isa, chhattisagadha़, uttara pradesha evam meghalaya haim| 
sahi tarike se shishu stanapana abhyantara bachchom ki chabi hai| 
matra stanapana pahale chhah mahinom mem, se hi 13% pancha se kama mauntom ko vikasashila deshom mem rokane ki sambhavana hai|
uchita samaya para aura upayukta puraka ahara 6% pancha se kama ki mautom ko roka sakata hai|
bharata mem 46% bachche hi stanapana para haim isaki tulana mem vikasashila deshom mem 38% aura dakshina eshiya mem 45% haim|
vikasashila deshom mem 6-9 mahinom ke bachche jinako puraka ahara diya jata hai ka pratishata 56 bharata ke 57% ke ankada़e ki tulana mem|
pancha se kama ki nrityu dara mulatah adhika bhojana sangraha ya bimariyom ka parinama hai|
24 vikasashila deshom mem apavyaya dara 10% se jyada hai|
bharata mem apavyaya samanyatah lagabhaga 20 % jisake akasmika upayom ki jarurata hai|
bharata ke pasa apratirakshita bachchom ki sabase bada़i sankhya hai, 9.4 miliyana/26 miliyana bachche jinako pratirodhaka nahim mila DPT 3 ke satha vishva stara para, 11.5 miliyana bharata mem rahate haim| 
khasara tikarodhaka bala nrityu ka eka pramukha karana hai|
vikasashila deshom mem vishvastara para khasara pratirodhana mem ullekhaniya pragati hui hai|
WFFC ne 50% nrityu dara kama karane ka lakshya prapta kara liya hai|
bharata mem khasara pratiraksha ko kshetra badha़a hai para phira bhi abhi 60% se kama hai|
nimoniya se marane vale bachche baki bimariyom se jyada haim, yaham taka ki edsa, maleriya aura khasareke kula se bhi jyada|
vishva stara para lagabhaga 2 miliyana bachche pancha sala se kama mem bimariyom se mara jate haim, lagabhaga hara pancha mem se eka bachcha|
bharata mem nimoniya se hone vali mautom ki sankhya sarvadhika hai|
lagabhaga 69% bachche jinamem nimoniya ki sambhavana hai, ko svasthaya kendrom para le jaya jata hai aura 13% ko entibayotika di jati hai|
vishva stara para dayariya dusara sabase bada़a maraka hai aura 2 miliyana bachche pancha se kama ki mauta darja hui hai|
2005-06 mem bharata mem, 39% bachche jo dayariya se pida़ita the ko ORT prapta hua|
yadyapi madhya ayavarga ke deshom ne matri nrityu dara kama karane mem kaphi vikasa kiya hai, para upa sahara aphrika ne kama vriddhi ki hai|
vikasashila deshom mem hone vali 5,00,000 matri sambandhi mautom mem se 23% bharata mem hai|
vishvastariya matri nrityu dara (2005) mem hara 1,00,000 janma para 400 anki gai|
2005 mem bharata ka anupata 450 anka gaya aura use uchcha shreni mem (UN ankada़om ke anusara) rakha gaya|
raktastrava se hone vali lagabhaga 38% ke kula matri nrityu bharata mem hai|
prasava ke daurana prashikshita upasthiti avashyaka hai, matri mautom ko kama karane ke lie|
vikasashila deshom mem lagabhaga 60% janma prashikshita svasthaya sevakom dvara dekhe jate haim|
bharata mem pratishata mem janma ke satha kushala svasthaya karmiyom dekhe jane ke pratishata 47% hai|
vikasa dara 42% se badha़ gai hai|(1998/99)
sapha aura surakshita jala bachche ke jivita rahate ke lie avashyaka hai|
vishva stara para duniya surakshita peyajala ke lie nDG ko prapta karane ke patha para hai| 
bharata bhi 84.5% gramina aura 95% shahari janasankhya ke satha dirghakalina surakshita peyajala ko prapta karane ke lie agrasara haim| 
vishvastara para svasthaya suvidhaom ki kami, saphai ki kami aura asurakshita peyajala ke karana abhi bhi 1.5 miliyana bala nrityu dayariya bimariyom se hoti hai| 
2004 mem, vishva stara para 2.4 biliyana logom ne saphai suvidhaom ka upayoga nahim kiya ja raha hai|
1990 aura 2004 ke bicha, dakshina eshiya aura bharata ne saphai suvidhaom mem dugune se jyada ki vriddhi ki|
2004 mem abhi bhi, anumanita 700 miliyana loga behatara saphai suvidhaom ka prayoga nahim kara rahe haim|
NFHS 3 ke ankada़e darshate haim ki 45% ghara mem rahane vale ne behatara saphai vyavastha ka upayoga kiya|
vishvastara para 332 miliyana loga (2.4 miliyana bachche) echa ai vi ke satha ji rahe haim|
bharata mem echa ai vi edsa ke satha jine vale logom ki sankhya 2-3.1 miliyana anki gai hai|
lagabhaga 70 hajara 15 varsha se kama ayu ke bachche echa ai vi se sankramita haim|
anumanita taura para 21 hajara bachche hara sala mata se bachche
bharata mem 23.8% mahilaom aura 32.6 purushom ne echa ai vi edsa ke vishaya mem vyapaka jnana darshaya|
lagabhaga 61% purusha aura mahilaem asurakshita yauna sambandhom ke daurana kandoma ka prayoga karate haim|
vishvastara para skula chhodane vale bachchom ki sankhya 2002 mem 115 se ghata kara 2005-06 mem 93 miliyana raha gai|
yaha dhyana mem rakhate hue ki 6-10 varsha prathamika shiksha ki unra hai, 84% bachche skula jate haim|
vishvastara para prathamika vidyalayom mem upasthiti ka linga samanata anupata prapta kiya ja sakata hai|
bharata mem prathamika shiksha ke kshetra mem samanata ko prapta karane ke patha para hai, hara 100 lada़kom para prathamika skulom mem 96 lada़kiyam haim|
yadyapi madhyamika shiksha ke kshetra mem samanata abhi bhi chunauti hai hara 100 lada़kom para madhyamika skulom mem, matra 80 lada़kiyam haim|
lagabhaga 51 miliyana bachche jo 2006 mem paida hue, ka panjikarana nahim hua, unamem se lagabhaga 9 miliyana bharata mem haim|
bharata mem pancha varsha se kama ayu ke bachchom mem 27% ke pasa janma pramana patra hai|
vishva stara para 5-14 varsha ke 158 miliyana bachche shrama mem vyasta haim|
bharata vishva ke 18% boja ka karana hai, jo lagabhaga 29 miliyana hai|
vishvabhara mem lagabhaga 60 miliyana mahilaem jo 20-24 varsha ki haim, unaki shadi 18 varsha se pahale ho gai thi| 
2005-06 mem bharata mem 20-24 varsha ki anumanita 22 miliय़na mahilaom ne mana ki unaki shadi 18 varsha se pahale ho gai thi|
bharata mem, bachchom ki atisamvedanashilata aura unako suraksha adhikarom se vanchita rakhane ke prabhava ki pravritti badha़i hui aura kaili hi hui|
ina adhikarom se vanchita rakhane ke pramanita srota shrama, bachchom ka avaidha vyapara se lekara shaaririka shoshana aura hinsa evam durupayoga ke vibhinna rupom mem hai|
anumana ke anusara 12.6 miliyana bachche jokhima bhare vyavasaya mem haim(2001 janaganana), udaharana ke lie, bharata mem vishva ke 14 varsha se kama ke sarvadhika bala shramika haim|
yadyapi balashrama ke lie garibi eka pramukha karana mana jata hai, para anya karana bhedabhava, samajika bahishkara, satha hi achchhi shiksha ki kami ya vartamana abhibhavakom ka vyavahara aura balashrama ke prati socha aura shiksha ke adarsha aura mulya adi ko bhi samajana chahie|
bihara, mijorama, rajasthana aura uttara pradesha jaise rajyom mem 60% ya usase jyada lada़kiyam pancha varsha ki prathamika shiksha puri karane se pahale skula chhoda़ deti hai|
bachchom ka avaidha vyapara abhi bhi bharatama em gambhira samasya hai|
bachchom ka avaidha vyapara ki pravritti aura vistara udyoga aura gharelu shrama se lekara balapurvaka jaldi shadi aura vyavasayi sharirika shoshana taka hai|
vartamana adhyayana dikhate haim ki 40% veshyaem 18 varsha se pahale hi veshyavritti mem a gai thim|
isake atirikta, jo bachche avaidha vyapara se chhuda़ae gae unake punarvasa ke avasara abhi bhi kama aura purnasanyojana kathina hai|
halanki bachchom ki suraksha mudde para vyavasthita ankada़e aura suchana abhi bhi upalabdha nahim hai, pramana batate haim ki bachche jinako vishesha suraksha ki avashyakata hai una samudayom se haim jo asuvidha aura samajika bahishkara jaise ninna jati, adivasi aura gariba haim|
upalabdha suvidhaom ki kami aura kanuna vyavastha ki kamajori aura purnavasa vyavasthaa abhi bhi chinta ka vishaya hai| 
pichhale tisa salom mem hone vale svasthaya sudhara ke bada bhi, prarambhika bachapana ki bimariyom, aparyapta navajata dekhabhala aura janma ke anya karanom se abhi bhi jindagi khati ja rahi hai|
20 lakha se jyada bachche hara varsha rokathama vale sankramana se mara rahe haim|
bharata mem shishu nrityudara unche stara para hara 1,000 janma para 63 mautem haim|
adhikansha shishu nrityu janma ke pahane mahine mem ho jati haim, yaham taka ki 47% pahale saptaha mem hi ho jati hai| 
halanki navajata shishu nrityu dara mem 1980 ke daurana teja avanati dekhi gai, para pichhale dashakom mem ye kami kaphi kama thi|
uchcha nrityu dara kai karana he ki kuchha mahilaom ko hi kushala karmiyom ki suvidha milati haim aura kuchha ko hi stariya akasmika prasava sambandhi suvidha hai| 
isake atirikta, sirpha 15% mahilaom ko sampurna janma purva dekhabhala milati hai aura sirpha 50% ko ayarana aura taibaleta aura sirapa mila| 
bharata mem abhi bhi bachche tikom dvara roke ja sakane vali bimariyam jaise khasara jo abhi bhi sabase bada़a maska hai, se jindagi khate ja rahe haim|
navajata shishuom mam titanasa ki samasya kama se kama pancha rajyom uttara pradesha, madhya pradesha, pashchimi bangala, rajasthana aura asama shesha hai|
bharata mem poliyo mamalom mem 1998 mem 1,934 se 2001 mem 268 ki giravata ai hai|
2002 mem badha ai thi jaba 1,600 mamale sala ke anta mem pakke ho gaye|
kintu 2003 mem poliyo ke sirpha 225 mamale darja kiye gaye|
halanki, bachcho ka anupata khasare ke viruddha tike mile 1995 mem 72% se ghata kara 1999 mem 50% ho gaya abhi bhi yaha 61% haim|
vartamana svasthaya tantra ko svasthaya karmiyom ki sankhya badha़akara majabuta kiya ja sakata hai|
janma ke samaya hone vali mautom ko ghara para chikitsakiya daurom dvara roka ja sakata hai|
bharata sarakara ke svasthaya sambandhi kadamom ne bachchom ki pratirakshana ko badha़aya hai|
adhikansha garbhavati mahilaem shishu ke surakshita janma jo jivita va svastha rahe ki asha karati haim aura unhe chchhe svasthya mem badha़te hue dekhana chahati haim|
2005 mem unake esa karane ki sambhaavanaem pahale se achhchhi haim – aisa bilakula bhi isalie nahim haim ki ve unake adhikarom ke bare mem jagaruka ho rahim haim|
aja ke jnana va takaniki ke satha, vishva bhara ki mataom tatha bachchom ko adhikansha samasyaem jo bhayabhita karati thim se bachaya ya ilaja karavaya ja sakata hai| 
adhikansha miliyana asamayika nrityu jo hoti hai ko bachaya ja sakata hai kyom ki jyadatara bimariyam vaha haim jo bure svasthya ke satha ati haim|
gahari jami hui bhavana ke karana ki prajanana ki samanya prakriya ke daurana kisi ki nrityu nahim honi chahie aura usake parivara para sarvanashakari prabhava ke karana mata ki nrityu anya se bhinna eka durghatana hai|
sabhi sanskritiyom, parivara tatha samudaya mata tatha bachchom ki dekhabhala ko samajate haim aura aisa karane mem vo apani purna yogyata lagane ki koshisha karate haim| 
vartamana varshom mem mataem,, shishuom tatha bachchom ke svasthya evam sakushalata mem sudhara mem badha़ti sankhya mem desom ne saphalata suspashta parinamom ke satha pai hai|
yadyapi 1990 ke daurana uchcha nrityu dara ke yabara tatha bure svasthya ke satha shuruata karane vale deshom ne bahuta kama pragati dikhai hai|
kuchha mem vartamana mem sthiti vastava mem aura bigada़ gai hai|
isalie pragati tukada़om mem hui hai aura jaba taka ki sarthaka rupase badha़ nahim jati taba taka 2015 mileniyama diklereshana dvara nirdharita lakshaya ki lakshita dinanka taka matri nrityudara ki tina chauthai taka aura bala nrityu dara ki do tihai taka kama hone ki asha hai|
ki sare deshom mem mataom tatha bachchom ka svasthya jitana hona chahie utani pragati nahim kara raha hai|
iseka karana klishta haim tatha eka desha mem dusare desha se bhinna hote haim|
inamem svasthya ke samanya evam sthayi shatru – garibi, asamanata, yuddha tatha nagarika upadrava shamila haim|
echaaivi/edsa ka vidhvansakari prabhava balki jana bachane vale jnana ka sarthaka vyavahara mem parinita hone mem viphalata tatha janasvasthya evam surakshita vatavarana mem yatheshtha rupa se pratishthita karata hai|
kai mataom tathaa bachchom visheshakara unamem se adhika gariba vahaniya, prabhavakari tatha prabhavaniya dekhabhala ki pahuncha jisake ki vo hakadara haim se vanchita raha jate haim|
sadiyom se bachchom ke janma tatha chhote bachchom ki dekhabhala ko mataom tatha ayaom ka kshetra gharelu karya mana gaya tha|
20 vim sadi mem mataom tatha bachchom ka svasthya purna rupa se gharelu karya se rajya ki jimmedari sahita janasvasthya prathamikata mem badala gaya tha|
21vim sadi ke arambha mem mileniyama davalapamenta gola ne ise manava adhikarom ke rupa mem garibi tatha asamanata ke khilapha lada़ai ke mularupa mem rakha tha|
vibhinna deshom ki asamana pragati ke prati vishva ki pratikriya ke tarike para isa parivartana se duragami parinama milenge|
bachchom tatha mahilaom ke svasthya sudhara ke lie jana svasthya karyakramom ke banane ka arambha unnisavi sadi ke anta mem yuropa mem hua tha|
usa samaya svastha mataem evam bachche sarakara ke dvara arthika evam rajanaitika mahatvakanksha ke rupa mem dekhe jate the|
yuropa ke kai rajanitijna isa bodha ko bhagidari haim ki rashtra ke bachche bimara sehata unaki sanskriti tatha phauji akanksha ko bhayabhita karata hai|
yaha socha jabaradasta visheshatah phransa tatha britena mem hai jo yuddha ke lie kaphi svastha sainikom ko niyukta karane mem mushkilem mahasusa karate haim| 
sarakara ne sambhava hala 1890 ke phransa pathapradarshaka ke parikshana mem dekha|
ye karyakrama sehatamanda bachchom jo utpadita karyakarta aura hrishta-pushta sainika banenge ke vaijnanika tatha vishvasotpadaka tarike prastuta karate haim| 
programa samajika sudhara tatha samaya ki paropakari gatividhiyom ke udgama ko bhi badha़ate hue sahara dete haim|
parinamasvarupa sabhi audyogika desha aura unaki thailainda tatha kai laitina amerikana deshom bhruniya mem bhi mata tatha navajata ki kama se kama eka svasthya sevaem 20vim shatabdi prarambha hui| 
aisa lagata hai kahane ke lie nrishansa vastuem lekina kuchha ko ingita karana chahie ki vishvayuddha bachchom ke lie kale hatha yukta antarala tha| 
jyada se jyada hajarom vyaktiyom ki pratyeka dina nrityu hoti hai yuddharata rashtra, sabhi jagaha nai manaviya jindagi ka dekhana prarambha jo badha़ vayaska jivana ko pashvika pratisthapana vikasita kara sakata hai bahuta rashtriya vastu mulya vala hai|
badasurata tarika tatha muje shaka hai sachchai yaha sidhe ki yaha manava jindagi upayogita mem badalava sena ke lie tha|
mam tatha bachchom ke svasthya ki dekhabhala jalda hi usake apane tarka sangata prapta karate haim, sainya tatha vittiya gananaom ke upara|
vibhinna adhikariyom ke aveshtana mem badha़ta medikala tatha jana sahayatartha tatha sarakari badha़ti akankshaom se gunja aurasamaja ki rajanitika karyakalapa| 
karmachariyom ki gatividhiyom, mataom ke samuha sahayatartha tatha vyavasayika sangha mam tatha bachche ke kai vibhinna tarike ke svastha ke karana lete haim|
intaraneshanala lebara ऑrgonaijeshana 1919 mem prasuti suraksha ke kanuni mapadanda prastuta karata hai|
nyuyaॉrka tainsa ne mata nritudara para 1930 ke prarambha mem lekha prakashita kiya|
1938 mem madarsa chartara ne yunaiteda kingadama mem 60 sthaniya sansthaom ki ghoshana ki|
kai adhikarika riporta ke dvara mata tatha bachche ke svasthaya mantralaya ke lie mahatvapurna ho gaya|
mata tatha bachche ke svasthya karyakrama jana svasthya pratimana sankramita bimari ke prati yuddha karate haim|
ye karyakrama dvitiya vishva yuddha ke bada sthapita vastava mem prarambha karata hai|
sarvabhaumika ghatanaem logom ki bhumika tatha sarakaraki aura nai bani yunaiteda neshana ke dvara 1948 mem hyumana raita ke yunivarsala diklereshana jimmedariyom mem ichchha utpanna karata hai|
vishesha dekhabhala tatha sahayoga mata tatha bachchom ke svastha mem antarrashtriya tatha naitika ayama mem vishaya joda़em haim 50 sala se rajanitika tatha vittiya vishaya mem age bada़e charana ko chitrita karata hai|
mukhya karyom mem se eka varlda hailtha ऑragenaijeshana ko 1948 ke samvidhana mem diya gaya mata tatha bachche ke ilaja tatha svasthya labha ko badha़ava dene ke lie|
1950 mem, rashtriya svasthya yojana aura vikasa sansthaom se paॉlisi dastaveja una mam tatha bachchom ko tanava dene vale bhedya samuha haim aura isalie janasvasthya karya ke lakshya ko mahatta dete haim|
mam tatha bachchom dharana abhedya samuha prathamika svasthya dekhabhala gatividhiyom alma – ata (aba alma kajakistana) arambha ki| 
prathama mukhya prayasa gramakshetra ke svasthya dekhabhala ko vrihatta rupa se mam tatha bachche ko svasthya programa ko isake rogapratirodhaka ko badha़ane ki shuruata para kendrita haim| 
samanyataya, bachche ke svasthya karyakrama adhikatara kendrita hai – akasara prarambhika svasthya dekhabhala ka prayasa hi kevala karyakrama ka prathama mudda hai|
prarambhika svasthya daekhabhala ka purva karyanvayana ka akasara sankirna kendra hota hai, lekina isake labhom mem se eka tathya ki svasthya se vikasa ko jada़ne ke lie yaha pramanita karata hai aura sara ama samaja isa asamanata para bahasa karata hai|
mam tatha bachche ki durdasha jalda hi eka jaivika dosha ki samasya se adhika dikhai dene lagati hai|
1987 ekshana phaॉra sepha madarahuda ki kaॉla, pratikula samaja sanskriti tatha samaja ke samaja ke vittiya vatavarana jada़ taka gahari spashta jami hai aura visheshatah usa vatavarana jo samaja aurata ke lie nirmita karata hai| 
isa adhika rajanitika parichaya mem aurata ke nirnaya lene ki kshamata shakti ki tulanatmaka kami aura unaki rojagara, vittiya, shiksha, adharabhuta svasthya dekhabhala mem asamanata aura anya unake bimara svasthya ki jada़ ko anya strotom para vichara kiya gaya| 
lada़kiyom mem poshana ki kami, jalda hi honevali yauna karyakalapa tatha avayaska garbhavastha, sabhi ke prabhava pada़ta hai donom mam tatha bachche para garbhavastha ke daurana tatha bada mem| 
arabom mahilaem aura unake parivara samajika parivesha mem rahate haim jo achchhe svasthya ko dhundhane tatha ananda lene ke lie kama karate haim|
mahilaem akasara shiksha suchana tatha nae vicharom se kama sarokara hota hai jo punah bachche ki dekhabhala tatha unaki jivana bachapana mem bachane mem vyatita ho jati hai|
ve nirnaya ke prati kuchha kahane ko nahim ki garbhanirodhaka ka upayoga karana hai ya athava kaham janma dena hai| 
ve svasthya sevaom ke upayoga ke anichchhuka hote haim jisase karmachariyom ke dvara dara tatha apamanita mahasusa karate haim athava unake ilaja ko prapta karane ka dabava unake apane rivaja tatha mulyom se dvandva karata hai|
garibi, sanskriti ke ritirivaja tatha kanuni avarodha unake vittiya strota ko prapta karane mem niyantrita karata hai unake athava unake bachchom ke svasthya dekhabhala dhundhana ko abhi ko abhi jyada mushkila kara deta hai|
isa paristhiti mem anyayasangata suspashta banata hai ki mam tatha bachchom ke svasthya adhikarom ki suraksha ke lie sangharsha hai|
mam tatha bachchom ke ke adhikarom se sambadhita intaraneshanala kaॉnphrensa ऑna popyuleshana tatha davalapamenta 1994 mem kairo ijipta se gatishila karata hai| 
yaha sammelana 20 sala ke karyakalapom ki yojana pradana karata hai jo branhandiya prajanana svasthya sevaom ki kranti (jo mata tatha bachche svasthya dekhabhala ko upasamuchchaya banata hai) jo vaiyaktika adhikara tatha sujava ko sambhalata hai|
yaha parivartana isa pariprekshya mem mahatvapurna,kyonki yaha mam tatha bachchom ke svasthya mem nivesha tarkasangata badalava karata hai|
aja, jyada janana ki isase pahale jo aurata tatha bachchom ke svasthya ko nirdharita karata hai aura jo badha jyada kimati sudhara lata hai|
yaha jnana nivesha ko jyada saphala tatha kama svikarya mem dekhabhala banata hai|
mam tatha bachchom ke svasthya jana svasthya mahatta ko sthapita karane ke lie pachina manadanda hai|
prathama, bachche samaja ke bhavishya hai aura unaki mataem unake bhavishya ke sanrakshaka hai|
mam dekhabhalakarta tatha ghara banane vala mahatvapurna jaise akasara bhumikaem hai se kahim jyada hai| 
jo parivara ki sanskriti ke itihasa tatha samajika ritirivaja sampradayom ke satha sancharita karata hai|
mam prarambhika vyavahara ko prabhavita karata hai tatha rahana sahana pratimana sthapita karata hai jo kevala unake bachchom ke bhavishya ke vikasa aura svastha ki samarthya, lekina samaja ke akara ko hi nirdharita nahim karata hai|
isake karana, samaja usake mam tatha bachchom ke svasthya ko apane phayade ke lie mahatva deta hai sarpha rashtra ki sampatti mem hi sahayoga nahim deta|
dusara samaja mem asamanata ke kuchha parinama aurata tatha bachchom ke svasthya tatha bachava ko nukasana karata hai|
sarakara ke lie jo asamanata ki kami ko karyakalapa aura sampatti ka punah gambhira vitarana, stara paristhitiyom mem sudhara tatha mata tatha bachchom ke svasthya dekhabhala pradana karana prathama achchha bindu hai| 
unake svasthya mem sudhara garibi mem sudhara tatha asamanata ko vishva ke dabava mukhya haim|
atha vishesha mileniyama davalapameta gola (emadiji) sthapita hote haim, pratyeka unake sankhyikiya lakshya tatha prabodhana vikasa ke lie suchaka hai|
svasthya mudda, atha uddeshyom mem se tina mem, 18 lakshyom mem se atha tatha 48 suchakom mem se 18 sunishchita karata hai|
svasthya muddom ka mahatva svasthya para vishva sarvasammati ko svasthya apane adhikarom garibi ke mahatvapurna ayama hai, bimara svasthya garibi mem sahayoga karata| mem sahayoga karata hai|
svasthya sudhara, garibi mem kami tatha vikasa ke lie sharta hai|
svasthya ka vaidha sudhara saphala garibi ki tatha asamanata mem kami para adharita hai|
yaha achanaka hone vala nahim hai ki emadiji lakshyom ka pratipadana tatha svasthya tatha aurata, mam tatha bachchom ke svasthya ko vishesha mahatva dena ka suchaka samane ata hai|
mata tathi bachche ka svasthya antarrashtriya mudda hai taba bhi vishesha mileniyama davalapameta gola ki prajanana svasthya sevaom ka vishva prapti mem anupasthiti mem |
vishvastariya, hama mata tatha bachchom ke svasthya mem emadiji ke prati vikasa kara rahe haim| 
saphalata jyada chhayadara hoti hai jabaki uchcha nrityu dara ki svikriti ki dridha़ta tatha mata evam bachchom ke svasthya mem asamanyata tatha vishva stara para svasthya dekhabhala prapta karate haim|
svastha hone ka artha sirpha bachane se jyada artha rakhata hai|
tathapi pancha sala se kama unra ke bachchom ki nrityu dara vikasa ka achchha suchaka athava dukhanta kami ko pradana karata hai| 
pancha se kama ki nrityudara 20vim shatabdi ke antima bhaga mem vishva mem phaila gai|
araba ko unmukha, jabaki puri taraha se kami ka jukava dhime ke chinha darshata hai|
1970 se 1990 ke bicha, pancha se niche nrityu dara pratyeka dashaka mem 20% se kama 1990 se 2000 ke bicha 12% se kama ho gae| 
sarvabhaumika ausata mahatvapurna kshetriya vibhinnata ko darshata hai|
1980 ke dashaka mem dablyu echa o aphrika aura vestarna pesiphika rijana mem vikasa kama ho gaya tatha 1990 ke dashaka mem istarna meditereniyana rijana mem|
aphrikana rijana uchcha stara para shuru hua, ninnatama kami dekhi (5% ke asa pasa 1980 se 2000 ke bicha kami) tatha sabase jyada ankita kami thi|
tunatmaka vikasa jari the athava amerika ke dablyu echa o rijana mem tatha dakshina purvi eshiya aura yuropiyana rijana mem badha़i|
parinama hai ki kshetrom mem vibhinnata badha़ rahi hai|
tunatmaka vikasa jari the athava amerika ke dablyu echa o rijana mem tatha dakshina purvi eshiya aura yuropiyana rijana mem badha़i|
pancha se kama mem nrityudara aba aphrika rijana mem yuropiyana rijana se sata guna jyada hai|
bachchom ki nrityu aphrikana rijana mem badha़ta rupa se kendrita thi (2003 mem kula vishva ka 43%, 1990 mem 30% taka)
dakshina purvi eshiya mem aja taka 28% bachchom ki abhi bhi nrityu hoti hai, chhah mem se do dablyu echa o kshetra aphrika tatha dakshinapurvi eshiya mem 70% se jyada bachchom ki nrityu hoti hai|
anya tarike se dekhem, 50% se jyada bachchom ki nrityu sirpha chhah deshom para kendrita hai china, janatantriya kango, ithopiya, bharata, naijiriya tatha pakistana|
vishva ke bachchom ka saubhagya inake poshana avastha bhi shamila hai|
puri taraha se, aja ke bachche jyada achchhe se poshita hote haim : 1990 se 2000 taka vishvastariya avikasita tatha ninnabhara 20% tatha 18% kramashah prachalita hai|
tathapi dakshini tatha kendriya eshiya ke charom ora uchcha stariya kuposhana se satata pidita haim|
sare upa sahariyana aphrika mem bachchom ki sankhya jo avikasita hai aura kama vajana vale haim isa kala mem badha़ gae|
bachchom ki nrityudara mem kami mem atirikta vikasa karana ho, navajata ki nrityu mem vastavika kami lane ki avashkataom ko badha़ane ke prayasa hom| 
yadi bachche ki nrityu dara kama karana agali vikasa ke lie navajata nrityu mem bhari kami ke lie prayasa badha़ane ki jarurata hai|
prathama sarvabhaumika navajata nrityudara ankalana 1983 mem hua|
aura sakhta ankalana 1995 ke lie aura 2000 ke lie upalabdha hua|
ye rashtriya janasankhyikiya sarvekshana para adharita haim satha hi sankhyikiya maॉdala para| 
naye ankalana pradarshita karate haim ki navajata ki nrityudara ka bhara kai logom ke anubhava se kaphi jyada hota hai|
pratyeka varsha lagabhaga chara lakha navajata unake chara saptaha ke hone se pahale mara jate haim|
98% vikasashila deshom mem mara jate haim 
navajata nrityu aba sarvabhaumika pancha sala se kama bachche ki nrityu mem lagabhaga 40% yogadana deti hai aura shishu nrityu dara mem adhe se jyada|
saba sahara aphrika tatha eshiya mem navajata nrityu ka dara uchchatama hai|
navajata nrityu ka do tihai aphrika 28% tatha dakshina purvi eshiya 36% ko dablyuechao kshetra mem hoti haim| 
amira aura gariba deshom ke bicha ki duri badha़ rahi hai|
navajata nrityudara aba deshom se uchchaaya deshom mem 6.5 guna kama hai|
mahila ke lie navajata bachche ko khone ka jivana ka kama jokhima eba aphrika mem 5 mem se|
upari ankada़om mem prativarsha 3.3 lakha sthira janma shamila nahim hai|
sthira janma ke ankada़e navajata nrityu se jyada bhayanaka haim|
yaha ashcharyajanaka nahim haim jaisa ki vishva mem janma ka 14% hi panjikrita hota hai|
navajata ka jivanta janma tatha nrityu kama likhe jate haim|
bhruna hatya jyada darja nahim hote haim visheshatah prarambhika bhruna hatya|
jabaki navajata nrityu aurasthira janma ka bhara bahuta adhika vastavika hai, yaham kai tarike haim kevala samasya ka hissa vahi paristhitiyam jo bhayanaka tatha akasara jindagibhara ki ayogyata mem yogadana deta hai|
lakhom bachchom jo pate haim mem prativarsha shvasarodhaka samasya bachche se gambhira pakshaghata adhigama samasya tatha anya ayogyataem| 
pratyeka navajata shishu jo mara jate haim ke lie kamase kama anya 20 janmajata chota sankramana, samayapurva janma ki jatilata tatha anya navajata halata se grasita hote haim|
unake parivara samanyataya aisi trasadi tatha gambhira prabhavom ke prati pahale se taiyara nahim hote|
svasthya tatha navajata bachchom ka jivana apani mataom se najadika se juda़a hota hai|
prathama, kyonki svastha mam ke svastha bachcha hota hai|
kyonki jaham mam ki garbhavastha ke daurana na mile athava aparyapta dekhabhala ho, bachche ka janma tatha prasavottara kala, yaha samanyatah usake navajata shishu ke vishaya mem bhi hoga| 
donom mam tatha navajata ke lie jivana ke achchhe avasara honge yadi unhe janma mem prashikshita madada mile|
garbhavastha tatha shishujanma aura unake parinama abhi taka nrityu ke mukhya karana haim|
vikasashila deshom mem auratom ki prajanana unra ki bimari tatha ayogyataem anya svasthya samasya se adhika hoti hai|
vikasashila vishva mem 30 karoda़ se jyada ajakala laghukala athava dirghakala ki bimari se grasita hote haim jo garbhavastha tatha shishu janma se ati haim|
prativarsha 529000 marate haim (68000 asurakshita garbhapata ke parinamasvarupa shamila haim) jo marane ki kagara para haim, una bachchom ko pichhe chhoda़ dete haim chunki ve mam vihina haim| 
isa paristhiti mem kuchha sarvabhaumika sudhara ke chinha haim| 
1960 tatha 1970 ke daurana kuchha deshom mem apani mata nrityudara ko 20 sala athava kama ke kala mem kama kara diye gaye haim|
kuchha deshom jaise boliviya tatha ijipta hala hi ke varshom mem jyada vyavasthita kiye gae|
mata ke svasthya hi abhi ki saphala kahaniyam bachchom ke svasthya ke kama suni jati haim|
kuchha hada taka kyom ki parinamom ko pradshana mem jyada samaya leta hai chunki mata nrityudara mem kuchha parivartana, ajakala upalabdha suchana ke srotom se jyada mapana mushkila hai|
ajakala, anumanatah adhikatara mata nrityu gariba deshom mem hoti hai|
ye nrityu aphrika tatha eshiya mem bahusankhya hoti hai|
uchcha aya deshom mem 1% se kama nrityu hoti hai|
saba sahara aphrika mem mata nrityu dara uchchatama hai, jaham mata nrityudara jivana bhara ka jokhima 16 mem se 1 hai, dhani deshom mem 2800 mem se 1 ki tulana mem|
mata nrityu dara ki suchana eka gambhira samasya ho gai hai|
1970 ke anta taka, tina mem se eka se kama vikasashila desha hi ankada़e prastuta karane ke yogya haim – aura ye samanyataya kevala aspatala ki anshika sankhyiki haim|
paristhiti mem aba sudhara hai lekina vikasashila desha mem janma tatha nrityu janasankhya ka eka chhota hissa mem hi kevala panjikrita kiye jate haim sirpha eshiya tatha laitina ameriki deshom ko chhoda़kara| 
vishva ke sirpha 100 deshom mem nrityu ka karana nityakrama se likhi jati hai, jo vishva ki janasankhya ka do tihai hai| 
vishvasa ke kabila sarvekshana ankada़e prapta karana kathina hai jo rashtriya pradarshaka hom|
62 vikasashila deshom mem uchcha stariya nrityudara vale bhi shamila haim, sirpha akarshaka ankalana sankhyiki. rupatmakata para adharita hai|
ye aura bhi bhayanaka hai una sarvekshana se natija nikalana athava anshika nrityudara panjikarana|
jo desha isa rupatmaka akalana para vishvasa karata hai vishva ke janma ka 27% ko prastuta karata hai|
karagara, 36 arabom ka bhagya ka rikaॉrda nahim chhoda़a hai lagabhaga 4 mem se 1 usa aurata ka jo prativarsha janma deti hai|
ankada़om ki upalabdhata mem dhime se sudhara, jabaki isaka araatha desha ka mata nrityudara ke detabesa mem aba vikasa hota hai|
1990 se, dablyuechao yunaiteda neshansa childra phanda (yunisepha) tatha yunaiteda neshana paॉpyuleshana phanda (yuenaephapie) ka juda़a hia karyakari samuha hai jo niyamita nirdharita kara raha hai tatha upalabdha suchana ka sanshlshana karati hai| 
yaha vishvasa ke satha parivartana ko nirdharana karana sambhava nahim hai|
yaha vishvasa ke satha parivartana ko nirdharana karana sambhava nahim hai|
mata nrityudara se sambandhita anishchitata ka ankalana mem yaha kahana mushkila hoga ki nrityudara kama hui ya badha़i|
phira bhi yaham vikasa ka abhava hai, suchakom ka padachinha jo garbhaavastha ke daurana hone vali dekhabhala mem sammanasuchaka badha़ta tatha sabhi kshetrom upa sahara aphrika ko chhoda़kara 1990 ke daurana ko ingita karata hai|
eka prashikshika pashchimi dvara janma ka anupata 1990 ke daurana 24% badha़a sijeriyana varga tiguni ho gai hai aura janma purva dekhabhala ka prayoga 21% badha़a|
prashikshita dekhabhala mata ki nrityu pherane mem nirnayaka mani jati hai satha hi mata svasthya sudhara mem, mata nrityudara anupata sabhi jagaha kama hoti pratita hoti hai sirpha una deshom ko chhoda़kara jo 1990 ke uchcha stara mem arambha hue|
sarvabhauma suchaka ke sudhara mem giravata jo chintaem, niyama nirdharaka desha ke paibandom ko chhipate haim jo raste mem hote haim, dhime vikasa ko dikhate, nishchala rahate uthava ulti tarapha jate haim| 
jaise hi desha mem adhika vikasa hua jo pahale hi apekshakrita mata aura bachche ki nrityu ninna stara para hai, balki buri taraha se bandha jata, deshom ke bicha duri avashya hi badha़ti ja rahi hai|
kula 93 deshom, jinamem adhikatara uchcha aya ke breketa mem shamila haim, apani 1990 pancha se nichali nrityudara ko 2015 mem do tihai taka athava jalda hi kama karane ke raste para haim| 
raste ke deshom mem vaha haim jo pahale hi 1990 mem ninnatara dara para the (sabase satha vo 1990 mem 59 ki dara para the)
pancha sala ki unra se kama ke bachchom ke bicha nrityu ki sankhya kama hoti ja rahi hai tathaa nrityu dara kama ho rahi hai, lekina itana teja nahim ki 2015 taka 1990 ke eka tihai taka pahuncha sake jaba taka yaha vikasa ko age ane vale 10 sala ke daurana sarthaka badha़ati nahim hai| 
ina deshom ne usa daurana hone vale ausata pancha se kama nrityu dara prati 100 ka 92 se kahim upari stara se shuruata ki| 
adhika samasyagrasta 20 desha the jaham para nrityudara nishchala thi| 
jaham nrityu ki sankhya satata badha़ti hai kyonki janma ki sankhya badha़ta se santulita kami bahuta kama rakhi jati hai|
ye vo desha haim jo jinamem 1990 mem uchcha stara tha (207 ausata)|
antatah yaham 14 rivarsala desha haim jaham pancha se kama ki nrityu dara 1990 mem ausata 111 kama hui lekina taba se badha़ gai| 
1990 ke daurana aise kai desha pichhale do dashako ko joda़kara se jyada the|
desha jo ultava athava nishchalata dikhate haim aphrikana kshetra mem adhikatara ho gae|
yaha deshom ka samuha 1990 ke daurana pancha se kama ki nrityu dara mem unnati ke anusara visheshikrita hui, mote taura para nrityu dara ke sambandha mem kya hua ko|
halanki pravaha ankada़e upalabdha nahim haim, bachcha tatha mata nrityudara pancha se kama nrityudara para ultava tatha nishchalata ke satha sabase adhika hai|
jaise hi ashaom aura akankshaom se jyada dhimi gati se paristhiti mem sudhara hua –varjaniya nrityu mem vriddhi anshika rupa se janasankhyikiya bala dvara vyartha kara diya|
mam tatha bachchom ki samaya se purva nrityu ki sankhya achchhi badha़ sakati hai kyonki aba mam ki sankhya ghategi janma aura bachche lagatara badha़engem|
vishva bhara mem jivita bachchom ki sankhya 2015 se 137 miliyana hara varsha badha़ti rahegi|
jyadatara jivita bachche upa sahara aphrika mem badha़enge aura eshiya ke pakistana aura uttari bharata ke hissom mem|
jaham janma ki sankhya lagatara achchhi badha़egi2020 mem taba bhi jaba prajanana lagatara ghate|
ye ve kshetra haim jaham kishora tatha javana mahilaom ki jalda ya avanchhita garbhavastha se bachava jyada hota hai, asurakshita garbhapata se nrityudara suspashta hai, janma dena jyagatara sanyoga hota hai aura bachapana ko bachana bahuta mushkila hai| 
prathamikata stara mem giravata ke bada isa riporta ko mam tatha bachchom ke svasthya para kendrita ki mahatta para bala dene ke lie aba taka kyom avashyaka hai| 
vikasa dhima ho gaya aura alaga alaga tarike se badha़a dhani tatha gariba deshom ke bicha duri ko badha़ata hai akasara dhani tatha gariba deshom ke andara|
apako kya karana hoga ki apane bachche ko sulane ke antima pada़ava para uddeshyapurna drishti daliye aura yadi avashyaka ho to kuchha badalava karem|
apa chahem to malisha, gale lagana, sone ke samaya sangita bajana prayoga kara sakate haim, apane bachche ko gahari ninda mem jane ke lie madada karem| 
antatah ye charana lene honge bachche ki dekhabhala athava botala pilana aura usake bada ve bhi kshina ho jaengem aura apaka bachcha lambi ninda mem chala jaega|
yaha dhyana rakhie ki apaka bachcha apane apa sone mem asamartha pratita ho raha hai to usaki kami nahim hai|
vaha isa prakara kama kara raha hai jabase vaha janma tha aura vaha chija़om ko jaise vo haim usi prakara sahi prakara se rakhane mem khusha hoga|
apaka usaki madada karane ka uddeshya achchhi tatha surakshita mahasusa karana hai jaba apake bina, apako rone se rokane ka pralobhana die bina, pyara ke kabila sone ke tarike ki khoja karata hai|
dhairya rakhie, apane bachche se sanketa lijie, aura bina samaya lie apaka bachcha so jaega|
aba apaki dilachaspi agali avastha isa shanadara, chunautipurna tatha akhirakara puraskara yukta anubhava ki ora badha़oge jise hama janakata kahate haim|
janaka ki haisiyata se, maim samajati hum bachchom ko dudha pilane, mitha khana tatha tarala bhojana khilane ke dabava ko hara deta hai| 
achchhe arthom mem mitrom tatha parivara kai bara apaki philmom ki samajane mem mushkila hogi jaba apa apane bachche ko bhojana mem hanikaraka athava kuposhita khane se rokane ki koshisha karenge| 
jaba ve mahasusa karengem ki apa apane bachche se dura haim, baki vijnana apako sahara dene ka vishvasa deta hai|
utpadaka akasara kai prakara ki chini kama mem lete haim atah vyaktigata samagri ka bhara kama karate haim|
chini sahi mem sapheda khuna ki koshikaom ki yogyataom ki antargrahana ke bada pancha ghantom ke lie kama kara deta hai atah apane bachche ke sharira ki jivanu athava vishanu jinase vaha samparka karata hai se lada़ne ki yogyata ko kama karata hai|
chini elapiela (lipoprotina laipeja) ko uttejita karata hai jo sharira ki bhaojana ko vasa ke rupa mem sangrahita karane ko utsahita karata hai| 
parishkrita anaja (sapheda breda, pasta, chavala ityadi) tatha chini mem kama athava poshana nahim hota|
transa phaita anla haidrojaneteda tela tatha tale hue bhojana mem paya jata hai jaise ki breda, kraikarsa, kukuja़, pestri, anaja bachchom anaja tatha phaॉrmula alu chipsa, pudinga, jame hue bhojana, phastaphuda ityadi| 
phastaphuda restram akasara haidrojeneteda tela gahara talane ke lie kama mem lete haim|
tiephae enjiima karyakshamata, livara ditoksiphikeshana tatha avashyaka vasiya anlom ke upapachana mem dakhala deta hai|
tiephae kainsara tatha dila ki bimariyom se juda़a hai tatha roga pratirodhaka tantra ko damana karane ke lie jana jata hai|
bachchom ke bhojana akasara sandra hote haim (udaharana ke lie bachchom ke bhojana se pani nikala dete haim) atah usamem sandra rasayana hote haim|
bachche jyada phala, jyusa phalom ke bara ityadi bahuta pasanda karate haim|
adhikatara kheti hara rasayana janche jate haim tatha 150 paundsa melsa ke lie bachchom ke lie pramanita nahim kie gae haim|
vaijnanikom ne drava sone ke srota ko dashakom se khoja nikala upalabdhata ke lie unhone vishleshita kiya aura isa ashcharyajanaka padartha ka nirmana kiya|
isa jadui drava ka matra jana gaya srota mam ki chhati hai aura yaha aba taka isake rahasya asani se nahim khule haim|
janma dene ke turanta bada, rasayanika masaja aurata ke sampupna sharira mem bheji jati hai, aura chhati ke dudha ko utpanna karane ka nirdesha deti hai|
isake utpadana mem shamila yantrika hai bachche ko pradana hone vale vibhinna labhom mem se mastishka pragati karana hai|
khana pana se hama isake guna aura isaka vishvasa ki uchcha sambhava gunavatta ko badha़a sakate haim|
teji se badha़te bachche ke lie kaphi dudha ka utpada bahuta urja la sakati hai – pratyeka dina lagabhaga 500 kailori|
itani kailori gadha़e poshaka bhojana mem pai jati haim uchcha matra ki chini mem nahim|
sunishchita kijie ki bhojana saptaha mem apako puri taraha upalabdha ho jaise ki apa yaha pana chahengem jaba apa bahuta bhukhe hom|
halanki atirikta bhojana lekara, apa apane vajana mem bina jyada sochavichara ke kaphi dekha sakate haim|
teja vajana ki kami prathama chara saptaha ke bada (prati mahine mahine 2-4 paunda se jyada) yaha chetavani deta hai ki apa paryapta kailori apa donom apake svasthaya tatha ke dudha utpanna karane ke lie kha nahim rahe hem|
apake chhati dudha mem mem paya jane vala protina asani se tatha jaldi pacha jata hai|
protina mem vikasa ke lie paripurna amino esida hote haim, jo manaviya dudha mem advitiya haim| 
postapartama ke kala mem protina ki kami ka ahara svasthaya labha samaya ko badha़a sakata hai|
kuchha tathyom ne nirdharita kiya hai ki chhati dudha mem protina ki matra bhojya padartha ke protina se badha़ai ja sakati hai, jabaki anya purna tathya isa prakara ke sambandha ko nahim batate haim|
kisi se bhi kama nahim, hama nahim jainate ki dugdha sravana mam ke bhojana mem atirikta protina ki badha़ta chahata hai|
kama se kama 4 protina ko bhojana mem pratidina shamila kijie (tophu, mungaphali, binsa, anda, machhali ityadi)
dudha mem payi vasa ki rachana uchcha vividha hai aura bhojana se bahuta adhika prabhavita hoti hai|
bhojana ke kuchha ghanto mem jisa prakara ki vasa apane grahana ki hai vaha apake dudha mem prabala vasa bana jayegi|
prasava purva aura prasava uparanta sthiti ke daurana, eka padartha vriddhi mem tantrika vijnana ka karya tatha vikasa tatha sikhana aura vyavahara mem mukhya bhumika nibhata hai jo vasa|
esenshiyala phaiti esida (mungaphali, bija, machhali, tatha hari mem pai gai) apake bhojana ke dugdha stravana tatha garbhavastha donom garbhavastha donom mem mukhya avayava haim| 
janaka EFA (linolika esida tatha livolenika esida) arakaidonika esida (AA), ekoseptanika esida (EPA) tatha dekasaheksainika esida mem badala jata hai| 
AA tatha DHA manavika mastishka tatha tantrika tantra mem adhika sandrata mem paya jata hai|
ve tarkashakti ke hisse ke rupa mem soche jate haim kyom manava mastishka mem uttama samvedi anya stanadhariyom se age yogyataem haim|
mam ka dudha EFA ka achchha srota hai, aura visheshatah bharapura hota hai yadi apa apane bhojana mem nishchita EFA bhojana ko shamila karate haim|
yadi apake bhojana mem transa phaiti esida uchchatama haim, yaha bhi navajata ko chala jata hai|
TFA jo haidrojinetida tela sansadhita bhojana mem paye jate haim| hridaya roga tantrika tantra tatha anya tajindagi bachchom mem avikasita jokhimom se juda़e haim|
halanki yadi apake bhojana mem EFA uttama matra mem haim, TFA shamila karana unake adhika urja se rokata hai aura na hi apa na apaka bachcha sambhava labha prapta kara payega|
niyamita bhojana ke srota ke satha, EFA atirikta rupa mem liya ja sakata hai|
kai utpada mam ki dekhabhala ke lie upayukta hai aura chikitsa mem upayoga kiye gaye jaba dudha prasara kama ho, rakta – sankulata samanya haim, athava navajata mem kai sankramana ke prati samvedanashila hote haim|
apake dudha se kai vitamina gujarate haim aura isalie apase prabhavita hota hai|
kai khanija nahim honge, kuchha mamalom mem chhati mem sarabora bindu hote haim jinake dvara apa sunishchita ho sakate haim ki poshana stara apake bachche ki avashyakata ko mata nahim degi|
lekina kuchha poshaka tatva jaise B6 maiganija, ayodina tatha seleniyama ki badha़ta jari rahegi|
apake dhyana ke lie thaimina, B12, pholika esida tatha vitamina mukhya puraka ho sakate haim yadi apa apane bhojana ke prati bahuta sashrita hai jisamem kaphi shamila nahim karate haim| 
thaimina, B12, pholika esida aura vitamina D navajata ke vikasa ke lie atyanta mahatvapurna hote haim aura kuchha auratom mem khataranaka tarike se kami ko dekha gaya hai|
vogana mam ko nishchita rupa se apane vitamina B12 tatha D ki matra ko dekhana chahie ye visheshataya janavariya bhojana mem paya jata hai|
bajara mem khane ki badha़ta ka agamana, kisi atirikta puraka ki avashyakataom ko kama karana chaha sakata hai yadi vo apake bhojana mem niyamita shamila hai|
spashtataya apake poshana ka uttama sthana vaha bhojana hai jo apa rakhate haim|
sunishchita karem ki apake bhojana mem uttama matra mem anaja aura taja rangina phala tatha sabji hai|
eka achchhi gunavatta ke bahu-vitamina/ khanija puraka kisi duri ko bharenge aura kai mam dvara anubhava kiye gaye thakana ko kuchha kama kara sakate haim|
pani nai mam ki tikhi vastavikata hai ki unake bachche ke satha bahara aya drava (jo kai bara jyada matra mem hota hai) unake chhati ke dudha ke dvara andara pravesha karata hai| 
atirikta drava jo apa nirmita kara rahe haim ki apane bhojana mem atirikta drava sahara deta hai|
kuchha mamem tivra pyasa mahasusa karati haim jaba unaka bachcha satha hota hai|
yaha socha jata tha ki yadi apa jyada pani lengem, jyada dudha utpanna hoga|
halanki vastava mem esa nahim hai jo hama janate haim vo hai ki jaba pani kama lete haim, mam ka sharira uttama kama nahim kara pata|
isalie lagabhaga 8-10 glasa pina , lekina isase jyada nahim pratyeka dina mahatvapurna hai|
TFA ke satha apake bhojana mem anya padartha bhi hone chahie jo apake bachche ke pasa ja sake|
kaiphina tatha sharaba, dono pashchimi bhojana mem samanya haim, jo dudha ke jarie navajata leta hai|
durbhagya se bachche ka aparipakva livara ko ina rasayanom ko pachane mem mushkila hoti hai, chahe apachana ke bahari lakshana na dikhai dem (ninda na ana, chida़na adi)| 
apake bhojana mem kama matra, dekhabhala satra ke samaya (yadi anumana anusara ho) samanyataya samasya nahim hoti| 
kai mataem shodha se chintita haim jo pradarshita karati haim ki mam ka dudha, vatavaraniya dosha ka pramukha srota hai (PCBs, DDT tatha para adi), aura kuchha apane dudha ki suraksha para sandeha karati haim|
pahachanane ke lie mahatvapurna kya hai ki jo mam ke dudha ke vikalpa mem paya jata hai vo una rasayanom se jyada khataranaka hote haim|
jnani daॉktarom ne marijom ko aushadhi ke korsa ke dvara dekhabhala jari rakhane ki salaha dena shuru kara diya kyonki ve janate haim ki vikalpa usase jyada badatara hai ki bachcha draga ki khuraka prapta kare| 
suspashta karanom se dhunrapana se bachava sahi hai, isaki jagaha ki apa dekhabhala na karem|
lekina dhunrapana karane vale bhi mam ka dudha jari rakhem jaisa ki unake bachche ke phephada़e vastava mem svastha ne jyada achchhe avasara pata hai yadi unhe dudha sahi tarike se diya jaye|
hamem nahim pata ke mam ke dudha ki pratilipi ko kaise taiyara karem aura hamane kevala abhi isaka mahatva samajana shuru kiya hai|
jaisa ki dudha pilane vali mam ko yaha mahasusa karane ke lie utsahita karana chahie ki apa apane bachche ke lie vo kara rahe hai jo usake lie uttama hai|
apana achchha dhyana rakhana aura svastha bhojana lena apako dekhabhala sambandhi avashyakataom ko pura karane mem madada karega| 
bachche kuchha bhi uthane vale aura parivartanashila hote haim isalie dhyana rakhie ki jo vo kha rahe haim vastava mem unake lie achchha ho| 
pratyeka vastu jo apaka bachcha khae poshaka tatha usaki sehata ke lie vastava mem achchha hona chahie|
isa prakara yadi unhone rata ke khane mem do tukada़e alu ke khae, apa vishvasa dilaem vo kama se kama achchha dopahara ka khana, nashta ityadi kare|
unake peta mem jyada jagaha nahim hoti, unako kharaba chijom se bharane se pahale savadhani se sochem|
akasara yaha hota hai ki abhibhavaka/dekhabhalakarta jo bachchom ko ullasita dekhane ki khushi se aposhaka khana-pana dete haim|
apaka bachcha muskana ya khushi anya jagahom se pa sakata hai – usako kharaba bhojana na dem|
khane ki samapti para dene vali disha svasthayavardhaka banaem (sirpha ananda ke lie nahim) isa taraha se jaise phala meve aura dahi bhojana ka hissa hom, sirpha khana khatma karane ka inama nahim|
sabhi achchhe bhojana davata ho sakati hai lekina hama aksara sirpha janka ke bare mem sochate haim isalie nyayapurvaka shabda prayoga karem|
samanyataya vaha dekhabhalakarta hota hai jo dejarta athava davata do sala se jyada ke bachche ko bhi dete haim jisaki shayada taba taka dekhabhala nahim hoti jaba taka unhe tina krima mem duba chaॉkaleta keka ka na mile|
bhojana arama ke samaya para dem aura jaba taka vaha na mange dobara na dem| 
jo kuchha vo nahim chahate usake lie jabaradasti na karem, aura nishchita rupa se unhe pleta khatma karane ke lie jora na dalem|
yadi apa bhojana para sangharsha karana shuru karengem to isaka lambe samaya taka hone ka jokhima hai|
bindu yaha hai ki unhe svasthayavardhaka khane ko dem, bhojana tatha niyantrana ke bicha sambandha vikasita na karem|
yaha eka karana hai ki kyom shuru se svasthayavardhaka khane ki adata mahatvapurna hai|
hama mem se kai yada kara sakate haim ki mam hamari pleta mem khana dalati hai aura hamase asha karati hai ki hama khate haim ya nahim|
vaham kuchha sangharsha bhi hota hai kyonki bachche jalda hi sitha jate haim ki jo unhe parosa ja raha hai yadi vo nahim khaenge to vo krodhita ho jaengi|
unake khuda ke dvara kuchha rata tebala para baithana jaba taka ve apana matara khatma nahim kara dete ke bada ve bina virodha ke khana sikha jaenge|
kai vayaska bachapana se galata khana, shuru kara dete haim aura kai kashta pane vale bachche ke rupa mem rata ke khane ke vakta tebala para apani ghatana yada kara sakate haim|
adara karem ki jaba bachche ka svada tatha muda apase usi taraha alaga hota hai jaise ki apake sathi athava padausi parivarika bhojana ke niyamo ko samaya samaya para toda़na sikhate haim|
apake bachche ko rachanatmaka bhojana dem jo svasthayavardhaka ho unhe pasanda ho, jaba anya parivara apane kari, stiraphrai, kesarola pa rahe hom|
jaba taka vaha svasthayavardhaka hoga, aura saptaha ki prati rata ko nahim hoga, samanyataya apaka bachcha apana bhojana chunega to apana abhibhavaka ko koi samasya nahim hogi| 
yadi jyadatara khane ke samaya svasthayavardhaka athava svasthaya rahita bhojana ki tulana mem vikasita karana jyada achchha hoga|
sabhi bachchom ke lie nashta kai bara dina ka mahatvapurna bhojana hai|
nashta chhoda़na sampurna dina ki dhuna taya karata hai aura bahuta bhukha, ninda hona khane ke lie bahuta thakana ki nichali tarapha nirmita karata hai|
kai tathya aura kai dekhabhalakarta isa tathya ko pramanita karate haim ki achchha nashta bachchom ko manasika karya achchhe se karane mem madada karate haim|
jabaki kai tathya skuli unra ke bachchom ki tarapha jukava rakhate haim, ye tathya bachchom tatha navajatom para bhi lagu hone chahiem|
kai paramparika lakshana kahate haim ki atyadhika do sala ke bachche akasara bhukhe hote haim|
nashte mem kuchha antima urja hetu protina hona chahie, bicha ki subaha ko niche ki tarapha sthapita karane mem madada karata hai|
bachava kunji hai kyonki adhikatara bachche akasara khate nahim haim aura apa unhe dudha pilane ka uddeshya pate nahim haim|
hamare bachche akasara hamem jitana jimmedari dete haim usase jyada janate haim|
kuchha peta vastava mem achchhe hote haim jo usake malika to batate haim ki kaba khaem aura kitana khaem|
bachchom ko apane sharira ko sunane dem – kai vayaska bhula chuke haim|
bachche eka jukava mem upayukta nahim hote aura anya vyakti ke niyama samanyataya apake parivara mem upayukta hone ke lie sanshodhita kiye jaem|
samaya samaya para apane bachchom ke niyamom ke anusara karem....ve apako ashcharyachakita kara dengem|
jaba abhibhavaka apane ajanme bachche ko chitrita kara pate haim, ve samanyataya mulayama, uttama nridu tatha bedaga ko chitrita karate haim|
jabaki kai matapita vastavika vriddhi apane bachchom ki tvacha ki dasha mem kami kai bara chida़chida़ahata ko sikhane mem madada karate haim|
ekjima, daipara ke nishana tatha kai bara halke jule ki kepa ki tivrata jaisi sthitiyam bhi hoti haim|
jabaki kai mata pita lakshanom ko asthai rupa se dabane ke tarike janate haim, kai aba unake bachchom ki tvacha ki dasha ko eka satha mulyankita ho kara chuke haim| 
tvacha sthiti ka jana mahatvapurna samasya hai usa prayasa se jyada jisamem lakshanom ko dabaya jata hai, lekina puraskara punah taba hai jaba apake bachche ki uttama tvacha dekhem|
yadi apaka bachcha khujali athava darda se grasita hai, yaha achchha ki pahale turanta arama prastuta karane ke prayasa karem|
sthira khujali avashya hi atirikta chida़chida़ahata utpanna karata hai aura sambhavatah baiktiriyala athava phangasa sakramana jaisi dusari samasya hai|
chamomaila, planatena tatha kelendyula bahudha tvacha ki dasha mem upayoga ki jane vali aushadhi hai|
sabhi shanta aura svastha haim aura kelendyula enti phangasa hone ka labha aura joda़ti hai|
shama Prinrose tela aura tela Borage donom sujana ko kama karane aura bechaini se rahata ke lie upayogi hote haim. 
samayika avedana shishuom ke lie bhi adarsha haim, jo anyatha kathinai ina puraka antarika le jata. 
bachava ke upaya krima kachche aura chidha़ ko turanta rahata tvacha la sakate haim. 
aise katanipa, passionflower, skullcap, kaimomaila ke rupa mem harbala nervines, aura valerian behatara madada milegi apane bachche ko arama aura darda aura khujali kama. 
bachchom ko kathinai jaba apani tvacha khujali aura pida़adayaka hai so rahi hai, so sakate haim, lekina sharira mem hi svastha madada karane mem mahatvapurna hai. 
mata pita aksara adhika upayoga aise Benedryl ke rupa mem rahata ke lie davaiyom ka mukabala karenge aura sone ke lie prerita karana. 
isake bada ke sanskarana jada़i buti ka eka sanyojana bachchom ke lie eka karagara vikalpa sabita kara di hai. 
bakha phlaॉvara upachara eka bachche ki tvacha halata ke bhavanatmaka ghataka ko sambodhita karenge. 
tanava aura chinta ama ho sake haim, aura jaba taka ke lie tvacha badha़ jari rahega sambodhita kiya. 
aise daycare ya skula shuru karane ke rupa mem jivana mem badalava, eka naya ghara hai, ya eka naya bhai ke ane jane ke satha aksara anurupa bhada़ka apa. 
upachara Walnut eka bachche ke parivartana se nibatane mem madada karata hai, aura holi irshya pata kara sakate haim. 
anya upachara bachche ki jaruratom ke adhara para upayogi ho sakati hai. 
jaba bachchom aura unake mata pita ke satha kama kara rahe hai, maim aksara unhem puchhane mem sankocha apane ahara aura paryavarana se bahuta kuchha ko nashta karane lagate haim. 
yaha bahuta hi eka bachche ko eka pratibandhita ahara khilana mushkila ho sakata hai agara ve pahale se hi eka picky khane vale haim. 
kabhi kabhi samasya asana ho sakata hai ki kuchha gaya hai anyatha ho sakata kami ke alava ke satha hala. 
avashyaka phaiti esida (EFAs) hamari tvacha ke svasthya ke lie mahatvapurna haim. 
Breastfed shishuom breastmilk ke madhyama se apane EFAs prapta hai, to mataom narsinga karana sabase achchha aisi shama halake pile ranga, borage, ya sana bija ka tela ke rupa mem atirikta khuraka lene honge. 
donom botala se sinchita aura stana tanga a shishuom shama halake pile ranga ya borage tela ko avashoshita kara sakate haim sidhe, halanki unaki tvacha. 
bada़e bachchom jaise machhali, natsa, bija, sana bija ka tela ke rupa mem khadya padarthom se unaki EFAs mila sakate haim, aura pattedara saga. 
Probiotics, samanyatah ke rupa mem acidophilus ko bheja, madada karega bachche ki pratiraksha pranali ka samarthana hai aura prabhavi eka kavaka khamira (mukabala karane mem) sankramana. 
kai mata pita ko majabuta virodhi apane bachche ki tvacha ke lie kavakiya krima lagane ke nirdesha haim. 
yadi apaka bachcha kabhi entibayotika davaom ke satha vyavahara kiya hai, apa jaba taka kavaka probiotics punah haim sankramana se hone ki sambhavana adhika samvedanashilata gavaha hoga gaya peta mem sthapana ki. 
rangina phala aura sabjiyam, sabuta anaja, sabjiyom aura protina koshika vikasa aura pratiraksha karya ke lie avashyaka poshaka tatva pradana karega. 
mam bhi ho sakata hai ke rupa mem ke rupa mem lambe samaya taka stanapana karane para vichara - chahie, khasakara agara apane bachche ki tvacha ki samasyaom ke prati samvedanashila hai. 
lahasuna ki kavaka haim una sahita sankramana ka mukabala karane mem karagara hai. apake bachche ke ahara mem bahuta shamila karem. 
, lahasuna roti Hummous, saॉsa aura dips taja lahasuna bada़e vahanom ke lie kara rahe haim. 
bhojana ki samvedanashilata aura elarji tvacha vikarom se grasta marijom ke bicha ama hai. 
deyari, gehum, makka, anda, soya, khatte phala, chaॉkaleta aura sabase ama ho sake. 
eka unmulana ahara ama taura para jo khadya padarthom ka pata lagane ke lie, yadi kisi bhi avashyaka hai, apane bachche ke lie samasyaem paida kara rahe haim. 
stanapana mam apane bhojana mem pherabadala ke rupa mem achchhi taraha se vichara karana chahie. 
deyari ke lie sabase ama aparadhi lagata hai aura isalie pahali bara hai ki jancha ki jani chahie. 
eka bara bhojana ko puri taraha samapta kara diya gaya hai, apake bachche ki tvacha mem koi parivartana nirikshana karate haim. 
kabhi kabhi eka bachche ko apane ahara aura jasusi ke kama mem eka se adhika kama ke prati samvedanashila ho sakata hai aura adhika mahatvapurna ho jata hai. 
apane bachche ke anya parivartanom ko dekhane eka bara thesa pahunchane bhojana ya khadya padartha samapta ho gaya hai hairana mata ho. 
jaise sansa ki samasyaem aura pachana gada़bada़iyom ke rupa mem lakshana bhi gayaba ho sakate haim. 
parishkrita khadya padarthom aura chini poshana anavashyaka haim aura apane bachche ke lie hanikaraka. 
asani se aise sapheda chavala, roti ke rupa mem sarala sharkara, parishkrita anaja utpadom mem parivartita kiya hai, aura pasta khamira ke eka ativriddhi mem yogadana kara sakate haim. 
ina khadya padarthom ko bhi poshaka tatvom ki matra naganya prastava aura isalie sharira ki chikitsa prakriyaom ka samarthana nahim hai. 
paryavarana samvedanashilata aura elarji se bachane ke lie kathina hai, lekina mahatvapurna pahachana. 
aise dusare ke rupa mem kuchha hatha se dhunrapana karate haim, janavara balom mem rusi, aura ghara klinara mem asani se khatma hote haim aura adhikansha gharom ke lie niyantrana. 
aise sanche mem dhalana hai aura dhula ke rupa mem anya chintaom se bachane ke lie kathina haim. 
sambhavita allergenic sabuna, shaimpu, itra ka sevana na karana, aura kapada़e dhone ditarjenta vishesha rupa se mahatvapurna hai. 
vitamina si, niyamita rupa se le liya, chikitsa ko badha़ava dene aura pratiraksha pranali ka samarthana karenge. 
vitamina si ishtatama upayoga ke lie bioflavonoids ke satha sanyojana ke rupa mem liya jana chahie. 
jinka, vitamina e, aura bita kairotina sabhi madada karega sankramana aura ghava bharane mem sahayata ke khilapha ki raksha karana. 
jinka mem amira phudsa soya sema, gehum roganu, aura kaddu ke bija shamila haim. 
vitamina e aura bita kairotina ande yolks, mithe alu, cantaloupe mem paya ja sakata hai, aura khubani. 
apake bachche ki taja hava aura jaba bhi sambhava ho dhupa ke madhyama matra mem tvacha ke samparka mem raho. 
donom hi svastha tvacha aura sharira ki chikitsa ki kshamataom ko uttejita mem madada milegi. 
apane bachche ki 'aura gaira sansa pareshana' ke lie vastra pahane. 
topiyam, vishesha rupa se, khopada़i shartom badha़ agara ve hava ke lie pravaha simita kara sakate haim ya bahuta lamba jala nami. 
supa, chaya, aura pani rakhana hoga apane bachche haidreteda. 
saba vishakta padarthom aura apashishta padartha ko nashta karane mem andara se bahara ki aura sahayata ke sharira se tvacha moisturize hoga. 
apane chhote se eka dukhi aura tuma jnana se bhara hai ki vaha eka bara phira eka aura kana sankramana hai rahe haim. 
bahuta chhote bachchom ke lie, kana sankramana avarti aura mushkila se bachane ho gaya hai. 
aisa khana elarji karakom se jyada jagaruka ho jaise kuchha samadhana,, samaya aura dridha़ta le. 
dusarom ko aura adhika tatkala rahata upalabdha karane ke sakate haim, lekina ise kama kiya jaega jaba taka rahata tha 'bada़a' tasvira ko sambodhita kiya hai. 
kana tina bhagom bahari kana mem vibhajita hai sabase adhika dikhai kshetra hai. 
dhvani kana nahara ke madhyama se yatra, kana drama atita, madhya kana ke niche. 
yaha vaha jagaha hai jaham haddiyom ki andaruni kana mem dhvani sancharita ho paya. 
bhitari kana tantrika anta ke satha amira hai ki dhvani prapta hai. 
Eustachian tyuba eka bachche ke madhya kana mem paya jata hai. 
isa marga taraha naka aura gala ko joda़ta hai, aura anumati deta hai kisi bhi madhya kana mem ekatra drava ki jala nikasi. 
shishuom aura yuva bachchom eustachian tyuba kama hai aura eka aura kshaitija sthiti mem deta hai (eka bara aura adhika giravata ka kona para vikasita hoga). 
yaha ise aura adhika mushkila ke lie atirikta thika se palayana tarala padartha hai. 
isa tyuba avaruddha hona chahie, sankramana asani se madhya kana mem vikasita kara sakate haim aura bada mem tarala padartha phansa rahe haim. 
tyuba ke karana bhida़ se avaruddha ho sakata hai eka thanda ya anya upari shvasana sankramana, ya khana ya paryavarana allergen eka pratikriya se. 
yadi apake bachche ke kana mem darda ki shikayata, baharapana, phlaita ke lie jutha karane mem asamartha hai, rata mem aksara uthata hai, bahuta hi udhama machate, eka bukhara hai, ya tarala padartha se draining usaki / usake kana, tuma eka daॉktara dekhana chahie hai. 
eka uchita nidana tuma upachara apa chunate haim aura adhika atmavishvasa dega. 
kana mem sankramana ka sabase ama karana entibayotika davaom mem se eka hai bachchom ko nirdharita kara rahe haim. 
lekina bada mem eka paksha prabhava khamira ke sharira mem prasara (Candida) ho sakata hai. 
khamira sankramana eka bachche aura kana mem sankramana ki sambhavana ke rupa mem ve apane sharira ki pratiraksha pranali ko chhoda़ kamajora aura durbala kara sakate haim kara sakate haim. 
kana mem sankramana ka ilaja abhi bhi sahitya mem kiya ja raha hai bahasa mem entibayotika davaom ke prabhava. 
yaha hai ki kai mamalom mem upachara ke bina nahim to hai khuda ko hala hota jata hai, aura kai chikitsakom samajate haim ki eka bara phira doharane nuskhe entibayotika hai dilai gai hai ama bata hai. 
kai bachchom ko darda se rahata milegi eka garma prabhavita kana para rakha diya sankshipta. 
mem eka mulayama kapada़e dubaki bahuta garma pani ya chaya aura maroda़ bahara. 
yaha bahuta jaba shanta ko chhu raha hai, yaha bachche ke kana para jagaha hai. 
ekantara se tuma eka garmi sana bija, anaja se bana hua hai, ya chavala se bhara paika istemala kara sakate haim. 
ina sabhi ko apani garmi banae rakhane ke raha rahe haim aura eka gile kapada़e se kama ganda. 
avashyaka telom laivendara, kaimomaila romana aura / ya chaya ke peda़ pani mem joda़a ja sakata hai ya garmi paika para gira diya. 
kana ke asapasa malisha Eustachian nali mem drava nali ki madada aura kuchha darda rahata pradana kara sakate haim. 
isa ke satha ya bina para agara tela malisha ka tela prayoga kiya jata hai apa avashyaka telom mem joda़ne ka socha sakate haim kiya ja sakata hai. 
kana ke pichhe se satha doharaya stroka mem jabada़a rekha se niche ki ora malisha. 
yadi apake bachche ko malisha bahuta pareshani ka karana banata hai banda. 
yadi limpha nodsa (granthiyom) sujana, halke se unhem eka hi avashyaka tela ke mishrana ka upayoga karane mem unaki madada nali aura kashta se rahata mila malisha kara rahe haim. 
agara darda jamava ke karana hota hai, khasakara ki eka thanda rakhane ke lie, shleshma apane bachche hone se thinned aise lemana bama, katanipa, eka prakara ka pudina, adaraka, aura / ya nadyapana ruta ke rupa mem bahuta garma harbala chaya pine ke satha juda़a hua hai. 
eka kamara jaham apaka bachcha sota hai, ya kisi eka bhapa se bhara batharuma mem bitae pahale bistara bhi madada kara sakata hum samaya mem prayukta vaporizer. 
tuma vaporizer, ya yukaliptusa tela laivendara avashyaka tela joda़ne para vichara yadi apake bachche ko do sala ki unra khatma ho sakata hai. 
propa eka 30 digri ke kona para apane bachche ke sira ke upara, jabaki niche jutha bola rahi. 
isa jala nikasi ko protsahita aura dabava mahasusa kara rahi hatotsahita aksara jaba phlaita jutha bola raha tha. 
bistara ke upari hisse ko itana samayojita karem ki yaha unake pairom ki ora niche gadde ya bistara ke pairom ke niche kuchha rakhakara dhalana. 
purana dhunrapana shodha dvara siddha kiya gaya hai kana sankramana ke eka bachche ke khatare ko badha़a. 
mausami elarji (ghasa ka bukha़ara) aura aisi dhula, mitti, palatu pashuom ki rusi ke rupa mem ghara elarji karakom ko samvedanashilata adi pahali do batem kara sakate haim, apake bachche ki pratiraksha kshamataom aura dusari, avaidha elarji ki pratikriya kama hai ki parinama mem sujana aura Eustachian ke avaruddha tyuba. 
paraga, mitti, palatu pashuom ki rusi, dhula ko samvedanashilata, aura khana protina kana sankramana reoccurring trigara ho sakata hai. 
pratiraksha pranali hai amataura para videshi akramanakariyom sujana bhida़ jo Eustachian tyuba blaॉka kara sakate haim mem parinama ki pratikriya. 
kyonki sharira mem itani dura '' hamalavara varda ki koshisha kara raha vyasta hai, yaha uchita eka jivanu ativriddhi ki madhya kana ki garma nama vatavarana mem ho sakata hai se nipatane mem asamartha hai. 
aksara saphai dvara apake bachche paryavarana elarji karakom ka yathochita mukta vatavarana rakhem, bedaruma mem hypoallergenic samagri ka upayoga kara, aura eka havai philtara ki sthapana. 
khana elarji aura pahachana jaba taka sharira kaphi kaphi majabuta hai anuchita eka pratiraksha pratikriya virodha se paraheja karane ki avashyakata hogi. 
yaha hai eka chikitsaka hai jo apaki madada kara sakate samasyagrasta khadya padarthom ki pahachana hai jabaki abhi bhi sunishchita karana hai ki eka svastha ahara banae rakha hai ke satha kama sabase achchha. 
ho praiktishanara haim jo bahuta pratibandhaka unmulana ahara para narsinga mataom ya chhote bachchom dala se savadhana.
prakritika aushadhiyam kana ke sankramana ko niyantrita karane mem bahuta prabhavakari hoti haim, parantu eka svastha hrishta – pushta pratirodhaka pranali usase bhi behatara hai|

kana ke sankramana apane apa ko doharane ke lie badanama haim|
kabhi – kabhi eka bachcha eka se mushkila se svastha hota dikhai deta hai ki dusare se prabhavita ho jata hai|
atah isa sankramana ko utpanna karane vale jivanu ko hatane ke uttama tarika yaha hai ki eka bachche ke pratirodhaka prakriya ko prerika kiya jaye evam prakriti ka prayoda kiya jaye|
kai sankramana jivanu sandushana ke shvasa tantra para akramana karane se avakshepita hote haim| ye socha kai adhara ki vyakhya kara sakati hai| 
nishchita karem ki bachche ka ahara parishkrita va/ya sandra sharkara(chini, anna, sharkara, shahada) se mukta ho evam prakritika sharkara jo phalom ke rasa mem pai jati hai kama ho|
sharkara sankramanom ke prati sharira ki pratirodhakata ko kama karata hai aura usa para sankramana ko prerita karane vale yista ko poshana deta hai| 
yadi apaka bachcha ekam gabhira yista sankramana se nipata raha hai jaisa ki eka yogya svasthaya karyakarta ne nirdharita kiya hai, yaha avashyaka hai ki apa apane anurakshaka ke satha gaharai se ahara mem parivartana karem| 
sabuta anaja, shaka protina jaise phaliyam, matara, meve, rangina phala aura sabjiyam evam paryapta jala, tarala padartha apake bachche ke ahara mem adhikansha matra mem hone chahie|
atha maha se chhote shishu ke kailori ka mukhya strota mam ka dudha hona chahie|
stanapana se kana se sankramana ka khatara kama hona chikitsaka pramanita karate haim|
bachche ke svayam dudha chhoda़ne jo ki do sala ya usase adhika ho sakata hai, taka stana pana karana sarvottama hai|
mam ka dudha ninnalikhita prastavita ilaja ke lie eka suvidhajanaka samvahaka hai| 
stanyada mata eka vayaska khuraka lekara usaka labha epane dudha dvara epane bachche ko pahuncha sakati hai|
ekineshiya sapheda raktakosha jo ki roga utpadakom ko hatane ke lie avashyaka hai ko, prerita karata hai|
vaha bachche jo chirakalika kana ke sankramana ki tarapha pravritta hote haim; unhe sampurna shita va sankramana ritu mem chara mem se eka saptaha chhoda़kara ekineshiya ke prayoga se labha hota hai| 
anyatha roga ke pahale sanketa para eka aura isake uparanta hara tina – chara ghante para eka khuraka se isa samasya ko age badha़ne se roka ja sakata hai|
ja़inka sampuraka ke dvara bhi sankramana ke prati pratirodhaka pratikriya ko badha़ava diya ja sakata hai|
vaha bachche jo chabane layaka bada़e haim; unhe jinka ki goliya dena atyanta suvidhanaka hai|
kuchha dusarom se adhika saralata se chabaye ja sakate haim aura kuchha prakarom mem ekineshiya va anya, pratirodhakata badha़ane vali aushadhiyam antarvishta haim|
vitamina si se na kevala pratirodhaka pratikriya badha़ti hai apitu yaha uttejaka virodhi bhi hai|
chabane .yogya va tarala vitamina vitamina si sampuraka bachchom ke lie upalabdha haim|
bada mem unake dantom ko manjana nishchita kijie kyonki yaha tejabi vitamina dantom ke enaimala ko kata deta hai|
apake bachche ko rogopachara ke daurana di gai khuraka se dasta ka anubhava bhi ho sakata hai|
khuraka kama kara dijie jaba taka dasta niyamita na hom jayem evam eka dina se adhika chalane para nirjalana ke sanketom ki ora satarka rahie| 
lahasuna ashcharyajanaka rupa se chhoti svadakalika achchhai taraha se saha leti hai yadi unhe usase jivana ke prarambha se parichita kara diya jaye|
stanaposhita bachche lahasuna ke svada ko sam ke dudha mem se sakate haim, mam ke inamem se kuchha prabhavakari roganu virodhi shaka khane ke uparanta|
kachcha lahasuna kai sampurakom se behatara hai tatha ise bachchom ko khilane ke lie rachanatmaka tarike maujuda haim| 
lahasuna ki pavaroti, hyunsa, glukamola prachalita nuskhe haim|
chhote bachche va vriddha loga jo ye akhane se mana kara sakate haim, unake pairom ke talavom para lahasuna se bhara tela mala sakate haim|
isamem ashcharyachakita na hom ki yadi apake bachche ki lahasuna sansa upachara ke kuchha samaya pashchata ho jaye kyonki lahasuna phephada़om taka vahuta tivrata se saphara karata hai|
kuchha anajane va jane karanom se kuchha bachchom mem avashyatakata se alpa antrika vanaspati hoti hai jitani ki khana pachane va rogajanakom se sharira ki raksha karane ke lie avashyaka hai|
chahe mam ne garbhavastha ke daurana upayoga mem liya ho ya bachche ke jivana ke kisi kshana mem upayoga kiya ho, pratijiva rashraka vanaspati jinaki hamem koi avashyakata nahim, ke bicha asantuna paida kara dete haim|
pathya strontom para karibi dhyana diye bina sharira ke anupasthita vanaspati ka pratisthaapana karana bahuta kathina hai|
kinvika bhojana jaise ki kephira va dahi achchhe strota haim jaise ki jaiva samarthaka sampuraka (sadharanatah anlavat ke nama se jana jata hai)
jaivasamarthakom ki kama phaphundi vardhana ke lie mancha saja dete haim jo ki kana ke sankramana ko utpanna karane ke lie jana jate haim| 
adhikatara bachche yadi sare nahim agara vo pravritta haim, sankramana se to unhe dainika sampurakom se labha hoga|
shodha ne dikhaya hai ki matrika sampurana garbhavastha ya stanapana ke daurana bachapana ki pratyurjataom ke khatare ko kama karata hai stanadudha nem rakshatmaka tatvom ki matra badha़akara|
stanapana bachche ko avashyaka pratirodhaka vriddhi vale tatvom ki apurti karata hai tatha antada़iyom ki raksha kara age bhi pratyurjataom ke khatare ko kama karata hai| (tapakati antada़iyam adhika videshi mala ko rakta pravaha mem svikrita karati haim) evam kantha – karnanali ke angika vikasa mem sahayoga deti hai|
anupuraka tatva, antrika vanaspati ke vardhana mem sahara dete haim joki prati rodhaka pranali ko badha़ate haim|
yadyapi nivarana utkrishta dava hai kana ke sankramanom ke lie, apake bachche ke darda evam kashta ke lie bahuta sare prakritika upaya bhi haim|
dhairya aura karmishthata kunji hai parantu chakra ko toda़na bhi sambhava hai|
eka pile pyaja ki moti patti ko katie aura use eka garama banda chulhe, maikroveva ya ubalate pani ke prayoga se garama kijie|
jaba vo pakada़ne yogya thanda ho jaye, to use momajame mem lapeta kara prabhavita kana para lagaiye| 
jaba vo thanda ho jae, use hata kara punah garama kijie eka bara aura upayoga ke lie|
do-tina lahasuna ke kate hue tukada़om ko eka dviguni baॉyalara ya chhoti degachi mem rakhie|
use paryapta jaituna ke tela mem dhaka kara 20-30 minata ke lie rakha dijie aura dhire se garama kijie|
banda phulom ki kaliyom ko paryapta jaituna ke tela mem rakhiye taki vo dhaka jaem|
martabana ko moharabanda kara do saptaha ke lie suraja ki roshani mem rakha dijie|
jiesai (svasthaya khadya bhandara mem upalabdha) ke 3-4 bundom ko 30 ema ela shaka glisarina mem milaiye| 
30 ema ela angura ke bija ke tela mem tina bundem romana babuna ke phula evam tina bundem chaya ke peda़ ke satva telom ki milaiye|
achchhi taraha milaiye aura eka bara mem kuchha bundem kana mem malie|
yaha vishesha rupa se labhakari haim agara kana ke parade phata gae haim aura antarika bundem uchita nahim haim|
ninnalikhita chija़om ka mishrana banaiye : nimbu, marahama, kentanipa, aura babune ke phula tatha tina chammacha sammishrana mem chara kapa ubalate pani ka dalie|
dasa minata ke lie dhakie , shishu eka chammacha chaya dina mem tina se chara bara le sakate haim|
bada़e bachche kariba ½ kapa 3-4 bara dina mem le sakate haim parantu chaya garma ho para tapati hui na ho|
yadi apaka shishu hara ghante mem jaga raha hai stanapana ke lie, shishi pine ke lie ya apane shamaka ko dhundhane ke lie, apa ye sochate honge ki aisi kya bata hai jo use bahudha uthati hai| 
sachchai yaha hai ki alpakalina ratri jagarana manushya nidra ka svabhavika anga hai kisi bhi ayuvarga mem|
apake shishu jise pratyeka eka ya do ghante mem ratri dekha bhala ki jarurata hai, mem antara yaha hai ki apako apane alpakalina jagarana avadhi mem shamila kara raha hai|
apaka shishu eka ‘ninda sahacharya’ banata hai jisamem vaha kuchha chija़om ka sambandha so jane se karata hai aura mana leta hai ki sone ke lie use ina chija़om ki jarurata hai|
mere bachche kolatana ne apane jivana ke prarambhika mahine meri bahom mem kate, usaka sira mere kampyutara ki borda ke dhunna para bhatakata rahata tha|
jaba se paida hua hai, vaha mere bagala mem soya, pratyeka japaki evam shayana samaya ke lie stanapana karate hue|
jaba taka mainem upara dekha, vaha baraha mahine ka ho gaya, dridha़ta evam purnatah stanapana ko shayana sahacharya mem sansthapita karake|
apaka bachcha mere kolatana ki taraha stanapana ka sambandha (apaka stanagra, apani botala ya shamakaapane munha mem rakhana) nidrana se karana sikha gaya hai|
maine kai shayana visheshajnom ko ise ‘nakaratmaka shayana sahacharya’ kahate suna hai| 
maim nishichita rupa se isake viruddha hum aura aisa hi mera bachcha karega| yaha shayada sabase sakaratmaka, svabhavika, sukhada shayana sahacharya hai jo ki eka bachche mem ho sakata hai| 
isa sahacharya ki samasya yaha sahacharya nahim balki hamara vyasta jivana hai|
yadi apake pasa kuchha bhi nahim hai karane ko sivaya apane bachche ka dhyana rakhane ke, yaha eka sukhada tarika hai apane rata aura dina bitane ka jaba taka ki vaha svabhavika rupa se isa jarurata se age badha़ jaye| 
apa ise eka sansya ke rupa mem dekhenge bhi nahim|
yaha saba apake bodha aura vyaktigata jaruratom ka mamala hai|
tathapi, hamari duniya mem, kuchha mata-pita ke pasa yaha sukha hota hai ki ve hara chija़ rokakara rakhem jaba taka unake bachche bada़e na ho jaem| 
ise dimaga mem rakhate hue maim, tunhe kai sujava dungi taki tuma dhire dhire evam pyara se apane shishu ki sone mem madada kara sako bina isa shaktishali shayana sahayaka ke| 
apane shishu ke shayana sahacharya ko badalane ke kadama uthane ke lie, apako ratri jagarana ko kuchha samaya ke lie kathina banana hoga para lambe samaya ke lie apako apane shishu ka shamaka, botala ya stana usake shayana sahacharya ke rupa mem chhuda़ana hoga| 
anya shabdom mem, kuchha samaya ke lie kuchha avashyaka, labhakari lambe samaya ke badalava lane ke lie, apani ninda bhanga karane ke lie taiyara rahiega|
jaba apaka shishu uthe, age badha़ie aura usaki botala ya shamaka usake munha mem dala dijie ya chhati se lagaiye| 
use vaham chhoda़kara vapisa sone ke bajaya ya use apane stana para sone dene ke bajaya, use kuchha minatom ke lie stanapana karane dijie jaba taka usaka chusana dhima na ho jaye aura vaha shithila aura nidralu ho jaye| 
phira apani ungali se mukhabandhana toda़kara halke se shamaka ya stanagra hata dijie|
akasara, khasa taura se prarambha mem, apaka shishu chaunkega aura stanagra ki ora badha़ega|
yadi shishu isake viruddha sangharsha kare ya upadrava kare ya apaki ora ya usaki botala ya shamaka ki ora badha़e, age badha़iye aura stanagra ko punah daliye parantu durikarana prakriya ko jitani bara ho sake doharai|
pratyeka shishu alaga hai parantu kariba dasa se satha sekanda durikarana ki koshisha ki shuruata ke lie|
yadi eka shishu bahuta jora se stanapana kara raha hai ya lagatara nigala raha hai, kuchha kshanom ke lie pratiksha kijie jaba taka usaki gati dhimi na ho jaye|
akasara prarambhika halachala ke bada, apaka shishu dhila va shithila ho jaega, yaha achchha samaya hai apani durikarana ki koshisha ki shuruata ke lie|
do se pancha koshishom ki avashyakata para pada़ sakati hai para anta mem apaka shishu bina shamaka ya stanagra ke so jaegi|
jaba usane aisa kai bara kara liya hoga to apa dhyana degem ki usaka durikarana bahuta asana ho gaya hai aura usaka jagarana kama ho gaya hai|
yadi apaka shishu achchhi taraha nahim japakata hai to japakane vale dinom mem durikarana takanika ka prayoga karane ka kashta na uthaem|
yada rakhie achchhi japaki ka matalaba hai behatara ratri nidra aura behatara ratri nidra ka matalaba hai achchhi japaki| 
eka bara apa apane bachche ko rata mem behatara sulane lagem, apa phira usaki japaki para karya kara sakate haim|
paintale ke komala durikarana yojana ka prayoga ka samaya rati ki pahali nidra mem uchita hai|
akasara apake shishu ke sone ka tarika usake baki ratri jagarana para asara karega|
muje aisa shayana sahacharya ke parinamom ki vajaha se lagata hai|
aisa prakata hota hai ki jisa prakara apaka bachcha puri rata rahana chahata hai, usi prakara vaha sone lagata hai|
maim ‘kabhi apane shishu ko rene mata do’ ke niyama ki palika hum aura maim use bahuta gambhirata se leti hum|
halanki jo maim nahim samaja pai vaha ye ki bachche ninda mem avajem nikalate haim| 
bachche ninda mem karahate, ghuraghurate, suda़kate pipinate aura rote bhi haim|
apane shishu ko adhika samaya ke lie sone mem madada karane ke lie agala kadama hai nidrana avajem evam jagarana avajom mem antara nishchita karana|
jaise-jaise apa usaki avajom ko dhyana se sunenge, aura use dekhenge apa nidrana phuphakara evam ‘maim utha rahi hum’ aura muje apaki jarurata hai mem antara samaja lenge|
kala rata vaha stanapana kara raha tha aura mainne use khincha liya tatha apani ungali usaki thuddi ke niche rakha di|
akasara hamara apane bachchom ke satha eka nityakrama hota hai jisaka palana apane bachche ke satha usake janma se kiya hota hai| nidra se pahale ka antima kadama stanapana ya botala lena hota hai| 
kuchha bachche isa pratirupa ko kayama kara puri rata bhi so sakate haim|
1984 mem, eka samajika karyakarta aura kathakara vale parivarom ka eka bada़a caseload jo sambandha mem badhaom ke satha kama kiya tha bairi Dickson, aura Joan Bodger, eka chikitsaka aura kathakara, eka payalata pariyojana ke rupa mem pahachana parivarom ki seva karenge yojana banai khatare mem dvara toranto bachchom ki sahayata sosayati, eka bachche ki suraksha ejensi. 
mam Goose samvardhana karyakrama hai bairi ka upayoga prayogom para adharita tha gaya jata hai aura usaka dhyana mem bachchom ke satha kahaniyam. 
nyuyaॉrka shahara ke pramukha prarambha karyakrama aura usake prayoga ke mulya mem gahara vishvasa hai Joan anubhava para gaya jata hai aura bachchom aura vayaskom ke satha maukhika rupa se kahaniyam. 
Celia Lottridge aura kaitharina Grier, donom aura sunane prashikshakom, Joan ke satha karyakrama mem sikhaya hai. 
vichara yaha hai ki mata pita aura bachche ya yuva bachche ke bicha sambandhom ke satha shuruata mem shuru ho gaya tha, aura khushi aura bijali ka upayoga karem gaya jata hai, gita aura kahaniyam sikhaya jata hai aura eka ke lie mata pita ke bachche ka rishta aura palaka ki paravarisha ki sthapana samuha mem maukhika rupa se anubhavi parivara kalyana. 
jaba payalata pariyojana ke samapta ho gaya tha, kaitharina Grier Celia Lottridge samparka kiya aura kaha ki isa karyakrama ko jari hone ki jarurata hai. 
Celia para sahamata hue, aura eka satha, kaitharina aura Celia ko samanya samudaya ke parivarom ko arthika sahayata karyakrama lane ki manga ki. 
1986 mem, janaka-bachche ki mam Goose (pi CnGP) karyakrama ke mata pita aura bachchom ke lie Etobicoke, ontariyo ke Lakeshore kshetra mem eka samuha ki peshakasha ki. 
yaha pahala karyakrama dusarom ke dvara vibhinna ilakom mem pichha kiya gaya tha, aura lambe samaya se pahale, shikshuta prashikshana ke madhyama se karyakrama ke lie eka yogyata ke satha mata pita ke the programa shikshaka ke rupa mem prashikshita kiya ja raha hai. 
jabaki autaricha parivarom para sabase jyada jarurata mem kendrita raha, karyakrama sabhi ke lie khula tha. 
bhinna digri aura jarurata ke prakara ke parivarom ke prati yaha badalava phayademanda sabita hui, mata pita ke rupa mem eka dusare ke lie sansadhanom aura maॉdala bana gai. 
ke rupa mem bachchom badha़i, do se chara ke lie eka karyakrama ke lie ki jarurata sala ke bachchom sapha hai, aura bana gaya pi ke eka dusare stara CnGP, jisamem mata pita kathaem sikha hai ki ve apane bachchom ko bata raha tha, vikasita aura joda़a karyakrama. 
nabbe ke dashaka ke madhya mem, Celia pi CnGP shikshaka prashikshana vikasita karane vyaktiyom aura darshana aura karyakrama ke tarikom mem sangathanom trena karyashalaom. 
natijatana, pi CnGP kanada aura pare bhara mem phaila gaya hai, aura kai sangathanom aura ejensiyom ke vibhinna prakara ki aba ve parivara ki seva ke lie karyakrama pesha karate haim. 
janaka-bachche ki mam Goose karyakrama toranto, ontariyo mem 1986 mem shuru kiya tha. 
mata pita ke eka samuha aura unake shishuom aura yuva bachchom ko kama amadani pada़osa mem eka charcha ke tahakhane mem ikatthe hue do shikshakom ko jo sikhaya una kavitaom aura gitom ko apane bachchom ke satha upayoga karate haim, ke satha milakara bhi unhem apani khushi ke lie kahaniyom kaha tha. 
karyakrama, Celia Lottridge aura kaitharina Grier ke sansthapakom mem eka payalata bairi Dickson aura Joan Bodger ke dvara banai gai pariyojana se prerita the. 
unhonne paya ki saja shabdom, karyom, chhaviyom aura ki bhavanaom ko gaya jata hai aura mata pita aura unake bachchom aura unhem lambe samaya diya-sthayi sansadhanom ke bicha sambandha badha़ane kahaniyam. 
janaka-bachche ki mam Goose karyakrama aura bada़a ho gaya hai eka bahupakshiya sangathana ho gaya hai. 
yaha eka rashtriya sanrachana hai, kshetriya karyalayom aura sthaniya karyakramom ke satha, aura kanada ke para se nideshakom ka borda dvara niyantrita hai. 
prabandha nideshaka ke lie jimmedara hai: prashikshana, karyakrama ke netavarka, hamari karyashala facilitators aura sadasyom ke lie vebasaita aura peshevara samarthana sahita sansadhana vikasa, sthapita karane aura banae rakhane ke manaka, vakalata, rashtriya dhana ugahane vale aura anusandhana. 
karyakrama mem sthaniya samudayom ke parivarom ke lie pratyaksha seva upalabdha karane, pratibhagiyom ke netritva kaushala, anya samudaya ke sangathanom ke satha netavarka mem badha़ne ke lie sthaniya kavita aura kahani kahane samuhom ke lie avasara paida protsahita karane, sthaniya stara para fundraise. 
shikshaka prashikshana shikshuta prashikshana ya vyavaharika anubhava ke satha karyashala prashikshana ko joda़ti hai. 
shikshaka prashikshana karyashala facilitators manyata prapta hai jo siddhantom pratibhagiyom aura janaka ke tarikom-bachche ki mam Goose karyakrama sikhane ke dvara sikhaya jata hai. 
shikshuta prashikshana hatha hai, para prashikshana ke satha eka anubhavi, manyata prapta pi CnGP shikshaka. 
vyavaharika anubhava pi CnGP shikshana anubhava hai. 
hamari karyashalaom ejensiyom aura antararashtriya stara para aura kanada ke sangathanom dvara kharida gaya hai. 
stara do karyashalaom ka gahara shikshakom ko samajane aura kshamataom. 
karyashala prashikshana shikshaka janaka ke saikada़om bachche ki mam Goose karyakrama ® kanada bhara ke samudayom mem ki peshakasha ki ja rahi ai hai. 
karyakrama ke isa netavarka se badha़ raha hai aura nyuja़letarsa ke madhyama se anubhavom ko bantane se majabuta hai, intaraneta aura sammelanom. 
pi CnG karyakrama manyata prapta shikshakom dvara sanchalita karyalaya mem niyamita rupa se riporta aura eka varshika shulka ada kara rahe haim. 
sansadhanom ki eka kisma ki kharida ke lie upalabdha haim, kavita ki kitabem aura angreji aura phrencha, storybooks, prashikshana mainualom mem sidi bhi shamila hai, aura vidiyo. 
janaka-bachche ki mam Goose karyakrama mata pita aura unake shishuom aura yuva bachchom ke lie eka samuha anubhava jo khushi aura gaya jata hai, gane ka upayoga kara ki shakti para kendrita hai, aura eka satha kathaem. 
mata pita ko pane ke kaushala aura atmavishvasa jo unhem apane bachchom ki mahatvapurna prarambhika varshom ke daurana sakaratmaka parivara ke paitarna banane ke lie sakshama kara sakate haim, aura bhasha aura sanchara ke satha apane bachchom ko dene ke jaldi svastha anubhava. 
programa setingsa ka eka vibhinna prakara mem chala ja sakata hai aura ya to eka vishesha rupa se grahaka ke samuha ke lie kara sakate haim ki peshakasha ki ho ya bada़e paimane para janata ke lie. 
pi CnGP eka samuha hai jo karyakrama mem mata pita / caregivers hissa lene ke satha satha apane bachchom ya yuva bachchom ke satha hai. 
bachchom ke bhaga ke rupa mem vikasa aura jukava ke apane stara se upayukta hai. 
gatividhi intaraektiva kavitaom, kahaniyom aura gitom para kendrita hai, saba maukhika rupa se prastuta kiya aura hava ke madhyama se. 
kavitaom aura gitom ki baithakom ki shrrinkhala ke anta mem jaba pratibhagiyom ki apurti samaya para samagri ke kuchha internalise aura use apane rojamarra ke jivana mem prayoga kiya hai rahe haim, mudrita sanskarana. 
kahani kahane kahaniyom ke satha karyakrama ka abhinna anga hai chuna aura vayaskom ke upabhoga ke lie kaha ja raha hai. 
kahaniyam jo mata pita ko janane aura unake bachchom ko bata sakate haim yaha bhi shamila ho sakata hai. 
gati dhimi aura samagri aura muddom aura savala uthata hai ki ke akasmika charcha ki punaravritti ke lie bahuta samaya ke satha arama se hai. 
vatavarana aura svikara sahayaka hai, sabhi pratibhagiyom ka vishvasa badha़ane ke irade se, aura samudaya aura samuha ke bhitara apasi sahayoga ki bhavana paida kara. 
karyakrama shikshakom ke siddhantom, samagri aura karyakrama prabhavi dhanga se chalane ki jarurata tarikom mem prashikshita kiya jata hai. 
pratyeka programa hai kama se kama do shikshaka bhi haim jo sabhi shikshana shulka ka hissa hai aura yaha samaya pratyeka satra ke samuha ke bhitara batachita ki apani tippaniyom para charcha ke bada eka satha rakhana. 
samuha kaphi chhote sunishchita karane ke lie ki eka pure ke rupa mem aura satha hi samuha ke logom ki jaruratom ko mila ja sakata hai. 
adesha mem vayaskom ke lie hai aura samuha mem arama ke lie puri taraha se bhaga lene ke, eka sahayaka ko shishu karyakrama mem bachchom ke chalane ki nigarani upalabdha hai, ho sakata hai aura chaildakaiara ke lie eka playroom mem do mem kahani kahane ke daurana karane vali chara karyakrama. 
karyakrama ke karanom mem se eka vibhinna prakara se kisi ke lie samarthana ki ja़rurata mem parivarom ka uddeshya hai. 
20 sala ke lie, janaka-bachche ki mam Goose karyakrama hai mata pita aura caregivers madada gaya hai shishuom aura yuva bachchom ke satha apane rishtom ka poshana. 
samudaya adharita karyakramom ke mata pita ko apani takata para nirmana aura nae kaushala sikhane ka avasara dete haim. 
hamare karyakrama gaya hai jokhima ki sthiti mem rahane vale mata pita ke lie vishesha rupa se prabhavi hai. 
hamari karyashalaom aura karyakramom yuva bachchom ke satha parivara ki seva ke lie apane janadesha ko pura karane mem kai sangathanom ki sahayata pradana ki hai. 
karyakrama donom shikshaka aura bhaga lene ke lie sukhada haim, aura karmachariyom ke lie hastantaraniya kaushala ki peshakasha ko samudayika seva ke anya kshetrom ke lie lagu hote haim. 
ve ke nirmana aura samudayom ko majabuta banane, aura sevaom ke kshetra mem eka surakshita pravishti bindu pradana madada karate haim. 
janaka-bachche ki mam Goose karyakrama adhyapaka prashikshana karyashala samagri, tarike aura karyakrama ke mukhya vicharom ko shamila kiya. 
hamare karyakrama sthaniya samudayika kendrom aura ejensiyom ke madhyama se diya hai, aura jokhima ki sthiti mem rahane vale mata pita ke lie eka surakshita pravishti bindu pradana kara sakate haim. 
karyakrama prashikshana aura janaka ke sansadhanom-bachche ki mam Goose karyakrama para adharita haim. 
eka karyakrama ke tredamarka nama ka upayoga sthapita ve aura pravesha janaka-bachche ki mam Goose karyakrama ke satha eka varshika samajaute mem. 
hama i mela aura teliphona paramarsha, hamari vebasaita para, programa yatraom aura sammelanom ke madhyama se chala rahe karyakrama ke lie samarthana pradana karate haim. 
kshetriya karyalayom toranto aura britisha kolambiya mem jagaha vartamana mem haim, aura anubhavi pi CnGP shikshakom aura karyashala facilitators jo sthaniya janakari aura sahayata upalabdha karane ke dvara samanvita rahe haim. 
kavitaom aura gita aura kahani kahane janaka mem bachche ki mam Goose karyakrama prapta karane ke lie, samagri aura karyakrama ke tarikom mem gahana prashikshana mem aura aura dusare hisse ko pura karane ke pi CnGP shikshakom ke satha avasara pradana karate haim. 
janaka bachche ki mam Goose karyakrama apaki gopaniyata ka sammana. 
hama vyaktigata janakari ekatrita aura hama sabhi vidhayi avashyakataom ka palana karana apaki gopaniyata ki raksha ke sambandha mem raksha karana. 
hama kiraya nahim bechate ya kisi melinga suchi sahita niji kisi bhi vyaktigata janakari vyapara. 
janakari pradana karane ke lie sevaem pradana kiya ja sakata hai, aura rakhana tuma aura bataya ki tarikha taka karyakrama, sevaom, vishesha ghatanaom sahita PCnG ki gatividhiyom ke bare mem, dhana ki jarurata hai, ke lie svechchha se ya eka dana karane ke avasarom, aura sambandhita janakari. 
janaka-bachche ki mam Goose karyakrama mata pita aura unake shishuom aura yuva bachchom ke lie eka samuha anubhava jo khushi aura gaya jata hai, gane ka upayoga kara ki shakti para kendrita hai, aura eka satha kathaem. 
mata pita ko pane ke kaushala aura atmavishvasa jo unhem apane bachchom ki mahatvapurna prarambhika varshom ke daurana sakaratmaka parivara ke paitarna banane ke lie sakshama kara sakate haim, aura bhasha aura sanchara ke satha apane bachchom ko dene ke jaldi svastha anubhava. 
programa setingsa ka eka vibhinna prakara mem chala ja sakata hai aura ya to eka vishesha rupa se grahaka ke samuha ke lie kara sakate haim ki peshakasha ki ho ya bada़e paimane para janata ke lie. 
pi CnGP eka samuha hai jo karyakrama mem mata pita / caregivers hissa lene ke satha satha apane bachchom ya yuva bachchom ke satha hai. 
bachchom ke bhaga ke rupa mem vikasa aura jukava ke apane stara se upayukta hai. 
gatividhi intaraektiva kavitaom, kahaniyom aura gitom para kendrita hai, saba maukhika rupa se prastuta kiya aura hava ke madhyama se. 
kahani kahane kahaniyom ke satha karyakrama ka abhinna anga hai chuna aura vayaskom ke upabhoga ke lie kaha ja raha hai. 
kahaniyam jo mata pita ko janane aura unake bachchom ko bata sakate haim yaha bhi shamila ho sakata hai. 
gati dhimi aura samagri aura muddom aura savala uthata hai ki ke akasmika charcha ki punaravritti ke lie bahuta samaya ke satha arama se hai. 
vatavarana aura svikara sahayaka hai, sabhi pratibhagiyom ka vishvasa badha़ane ke irade se, aura samudaya aura samuha ke bhitara apasi sahayoga ki bhavana paida kara. 
karyakrama shikshakom ke siddhantom, samagri aura karyakrama prabhavi dhanga se chalane ki jarurata tarikom mem prashikshita kiya jata hai. 
lusi matri aura shishu dekhabhala (LMCC) eka svadeshi sitambara 1993 mem sthapita sangathana hai. 
LMCC ka mukhya uddeshya kama aya vale parivarom ke bachchom ke lie samarthaka skula shiksha pradana karana hai. 
pahali pariyojana LMCC dvara chune gae kshetra mem eka adisa ababa ke audyogika kshetra mem rahane vale adisa vina phaiktari ke pichhe samudaya hai. 
yaha dekha gaya ki isa samudaya ke eka bahuta pichhada़e kshetra hai, jo bhi atyadhika pradushita hai aura kai bachchom mem sthita bahuta maili paristhitiyom mem khelane ki sada़kom para dekha jata hai. 
chayanita samudaya prathamikata mem bachchom ko jo purva mem bhaga lene ke bajaya sada़ka para apane bahumulya samaya barbada skulom ya kindergartens di. 
purva ki avadharana skula ya bala vihara ithiyopiya mem gariba samudayom mem vishesha rupa se lokapriya nahim hai, yaha shahari ya gramina kshetrom mem hai. 
bachche basa ghara mem rahana jaba taka ve skula ki unra ke haim, jaise koi bachcha sata sala ki unra mem skula mem dakhila ho jaega. 
natijatana aise bachchom ko achchhi taraha se prathamika skulom mem nahim hai pradarshana karana aura kai skula se bahara draॉpa aura sada़kom mem kuchha khatma. 
pahali LMCC purva skula sitambara 1994 mem sthapita, 25 bachchom ki eka matra ke satha tha, yaha sankhya bada ke varshom mem 150 se gulaba, jo kendra ki sharirika kshamata se upara tha. 
sitambara, 1999 khapata mem 75 bachchom taka simita thi. 
sitambara 1999 mem, eka anuvarti sarvekshana ne eka hi pada़osa mem ayojita kiya, jaham 1994 ke bada se LMCC hai ऑparetinga gaya tha. 
sarvekshana ke parinama se pata chala ki jo bachche LMCC purva se labha ke adhikansha skula aba prathamika skulom mem padha़ rahe haim. 
yaha sarvekshana se spashta tha ki dara kaphi kama kara diya gaya hai bahara draॉpa. 
1994 ke pahale ke sarvekshana ke daurana pai chhoda़ bahara samasyaom mem se eka tha. 
nae shaikshanika varsha ke lie hai, jo 1999 sitambara mem shuru hui, 75 bachche panjikrita kie gae. 
ye sarvekshana pada़osa se ae bachchom ke lie. 
aba bachchom ko pure dina rahane ke khela aura shiksha ke lie paryapta samaya kara sakate haim. 
tina shikshakom ke sabhi samudaya se bharti aba aura adhika samaya taka bachchom ki vajaha se dhyana dena hai. 
varshika samanya svasthya jancha ke marcha 1999 mem purva ke lie ayojita-Tikur Anbessa aspatala se arthat tina svayansevaka chikitsakom: diaraesa Kaleab Tesfaye, Habtamu Fekadu aura Tsinuel Girma ki madada se kiya gaya skuli bachchom. 
122 bachchom ki jancha se, 29% (36) ko koi chikitsakiya samasyaom ki pahachana ke satha svastha pae gae. 
ama svasthya ki pahachana ki samasya antom alasya, jo 36% ke lie khatom tvacha sankramana, 15%, netra sankramana, 11% kana mem sankramana, upari shvasa nalika ke sankramana, dvara kiya gaya danta cavities aura xerophthalmia khate ke kula ki shesha 12% ke lie. 
nirdharita davaom anshika rupa se mata pita ne aura kuchha hada taka LMCC dvara kharide the. 
eka sala 7 purane mahila bachche ko jabti vikara ho paya aura age mulyankana aura prabandhana ke lie Ethio-svidisha aspatala mem bheja gaya tha. 
yaha bhi dhyana dem ki sabhi bachchom ko poliyo ke khilapha svasthya mantralaya se eka medikala tima ne daura tike lagae gae sukhadayaka hai.
kenya una deshom mem se hai jaham vishva ki 97 pratishata mata evam shishu mautem hoti haim| 
eka yuena vivarana yaha bhi kahata hai ki ye desha svasthya dekhabhala mem paryapta unnati nahim kara pa rahe haim|
vishva svasthya sangathana kai aise ilakom ka ullekha karata hai jinhe garbhavati mataem evam pancha sala se niche bachchom ke svasthya se sambandhita sambodhana nahim kiya gaya hai|
chhabbisa deshom mem ne ya 38 pratishata ne bala nrityu ghatane mem aparyapta pragati ki| 
aisa vivarana hai ki 12 deshom mem pancha varsha se niche nrityu sakhya ke ausata varshika ghatava mem 1990 se aba taka koi badalava nahim aya hai| 
yaha vivarana dikhata hai ki pancha varsha se niche bachchom mem eka tihai se adhika nrityu ka pramukha karana hai aparyapta poshana|
isake alava, shishu janma ke samaya eka panchama matri nrityu ka adhahstha karana yaha bhi hai|
yuena shakha ka kahana hai ki pancha varsha se niche bachchom mem pahale varsha mem nrityu ki ganana duniya mem 40 pratishata hai ya chara lakha varshika hai| 
tathapi, yaha vivarana sanketa karata hai ki 3.2 lakha bachche pratyeka varsha janma ke pahale 12 saptaha mem marate haim|
aisa ullekha hai ki 10.6 lakha varshika nrityu mem 54% mautem pahale pancha varsha mem, nimoniya, dasta maleriya evam alpa ahara ki vajaha se hui thim|
ye dastaveja age joda़te haim ki 68 deshom mem nimoniya , dasta va maleriya ke ilaja mem visatara ki gati bahuta kama hai evam usamem sudhara nahim ho raha hai| 
aphrika mem pancha varsha se niche kariba eka lakha bachche hara varsha maleriya ki vajaha se marate haim|
vyaktika evam sansthapana sahayoga 2005 mem sthapita hua tha|
ulti ginati ka uddeshya hai hastakshepa ke lie surakshaka marga dhundhana jinaki avashyakata mileniyama devalapamenta golsa chara aura pancha aura isake satha eka, chhah evam sata ke kuchha bhaga pane ke lie|
samekaka prayatna dvara rashtriya evam antarrashtriya niti vidhayaka, yojana palaka, vikasa evam midiya sathi tatha shodha karta eka satha desha stara ki unnati ke lie utkrishta evam sabase nikatavarti suchana ka sankshepana, sanshleshana evam prachara kara rahe haim|
isa ullekha ka arambha dvitiya kaunta dauna se 2015 ke sammelana jo ki kepa tauna sautha aphrika mem apraila 17 se 19, 118vem intaraparliyamentari yuniyana asembali ke satha hua tha| 
manava avaidha vyapara ke pida़itom, jinhe khaasakara mojambika se sautha aphrika laya jata hai, ke shoshana evam durupayoga ne mukhya samacharom ko aveshita kiya hai parantu visheshajnom ka kahana hai ki yaha kevala himashaila ki noka hai|
seva da childrana – yunaiteda kingadama (esasi-yuke) mojambika ke anusara 52 sandigdha ghatanaem varsha ke prarambha se samane ai haim|
sabase navinatama ghatana tina mojambiyana bachchom ki hai, unra 14 aura 16 varsha, jinaka sautha aphrika ki rajadhani pritoriya ke vaishyalaya mem maithunika vashikarana hua tha| 
krisa melavara ke anusara, jo ki mojambika ke esasi- yuke ke desha nirdeshaka haim, ghatanaom ki badha़ti sankhya ki suchana sautha aphrika ki eka gambhira aura adhika vyapaka samasya ki ora sanketa karati hai| 
kisi bhi avaidha vyavahara ki taraha jo aise karya karan vale logom ko arthika labha pahuchata hai, yaha adhika sambhava hai kiesi bahuta sari ghatanaem haim jinhe suchita nahim kiya jata hai|
hajarom bachche mojambika, jimbavve svajilainda aura lisotho ko chhoda़kara hara varsha sima para kara sautha aphrika bina dastavejom ke jate haim|
vaiseja jikenahaina, sadarna aphrika ke yuena ऑphisa ke draga aura kraima rijanala ऑphisa ke upa visheshajna ne mana ki ye samasya alpa suchita hai|
manava avadha vyapara pur sadarna aphrika ki samasya hai halanki sautha aphrika bahuta sare deshom ka pradeshika chumbaka hai|
yadyapi aisa mana ja raha hai ki ila vyavahara me mvriddhi ho rahi hai, isa samasya ke akara ke sahi ankada़e durgrahya haim|
2003 ke eka adhyayana ke anusara jo ki intaraneshanala ऑrgenaijeshana phaॉra maigreshana dvara pranta mem manava avaidha vyapara para kiya gaya tha, aisa anumana lagaya gaya haiki kariba 1100 mojambikana auratom aura bachchom ko sautha aphrika khasataura para yauna shoshana ke lie becha gaya tha| 
yaha vishaya avaidha vyapara ki paramparagata chhavi ki isamem manushyom ko unaki ichchha ke viruddha sima para karavaya jata hai, se adhika vistrita hai|
pretoriya mem mili mojambika ki lada़kiyam jo aba mojambikaki rajadhani maputo mem, apane parivara se dubara mila gai haim ko kama aura padha़ai ka vayada kara sima ke para akarshita kiya gaya tha|
loga apani jaruratom to pura karane ke lie lobhita ho jate haim aura logom ke kiye gaye vayadom se bhavuka ho jate haim|
sima ke usa para jane ki bachchom ki aichchhika pravritti se hamari asa vishaya mem chinta ki unake satha dusare deshom mem kya hota hai kama nahim honi chahie|
unhe adarsha rupa se yadi becha bhi nahim ja raha hoga, to bhi unake kai adhikarom ullaghanna hota haiaura hamari suraksha ke yogya hai|
shongave ne kai bachchom ki madada ki hai jo mojambika jate hue usake ghara se gujare|
ina gatividhiyom ke pichhe mukhya karana mojambika ki adhika garibi hai jisaki vajaha so loga sautha aphrika eka behatara jivana ki khoja mem jate haim|
melavara ke anusara arthika aura samajika karana jo ya bheda utpanna karate haim, pranta ke kai deshom mem aura bure hote ja rahe haim|
uchcha shreni ki berojagari, behatara jivana ki khoja mem yuva, kuchha deshom mem avaidha saste shramikom ki manga, aisi sthiti utpanna karate haim jo avaidha vyapara ke lie upayukta hai|
jikenahainara ne kaha ki nikatavarti varshom mem avaidha vyapara mem aparadhika sangathanom ke aveshtana mem badha़otari hui hai kyonki inamem uncha munapha aura khatara hai| 
manushyom ka avaidha vyapara gatishila, anukulanashila, aura avasaravadi hai tatha – yuddha, lokopakari sankata evam logom ki bhedata ka phayada uthata hai|
isaka gupta svarupa avaidha vyapara ke mamale ke anusandhana tatha bhumika – khilada़iyom ki pahachana karane mem mushkilem paida karata hai|
manushyom ka avaidha vyapara gatishila, anukulanashila, aura avasaravadi hai tatha – yuddha, lokopakari sankata evam logom ki bhedata ka phayada uthata hai|
janakari ki kami aura gupta svabhava avadha vyapara ki ghatanaom ki jancha evam bhumika khilada़iyom ki pahachana karane mem mushkilem paida karata hai|
jaise jaise avaidha vyapara apani gatividhiyom ko chhupane mem aura bhi nipuna ho jate haim hamem chinta honi chahie ki nikatavarti mahino mem hamane jo dekha hai vaha kevala hilashaila ki noka hai 
jina vyapariyom ki hamane bata ki hai ve kanuna ki kami, vidhika pranali mem chheda, ka labha utha rahe haim , unhe pata hai ki pakada़e jane para yauna utpida़na evam apaharana jaise aparadhom ke lie kevala eka ansha aropita kiya jaega|
mojambikana kanuna aropita manava avaidha vyapariyom ka abhiyoga karane ki koi vyavastha nahim karata hai|
phalasvarupa kisi bhi sandigdha vyapari ki nyayika jancha nahim hui hai isa aparadha ke lie jabaki yaha vyavahara antarrashtriya kanuna ke anusara avaidha hai| 
ajakala jaham bhi avaidha vyapara aparadhika ho gaya hai, ghatanaom ko jancha nahim jata ya anusandhaana doshasiddhi mem parinamita nahim hota|
yadi inaka kanuna nahim hai to anya vidyamana kanunom ko avaidha vyapariyom para mukadama chalane ke lie kama mem lena chahie|
kai dandaka paddhati balatkara, apaharana evam dhokhadhada़i jaise aprathaom ko sammilita karata hai aura ye aparadha avaidha vyapara mem ghatate haim| 
mojambikana kaunsila ऑpha ministarsa ne eka vishishta kanuna avaidha vyapara ke viruddha banaya hai aura eka childransa ekta ka prastava rakha tha 2007 mem, parantu donom svikriti ke lie sansada mem vicharadhina pada़e haim| 
eka esasi kathana mem prastavita kanuna ki svikriti sadarna aphrika ke deshom ke bicha adhika sahayoga, avaidha vyapara ke khatarom ki adhika janakari evam uttarajiviyom ki uchita dekhabhala nishchita karane ki manga rakhi|
avaidha vyapara ki lada़ai mem rashtriya, prantiyaevam antarrashtriya stara para vistrita bahushakti evam sahayogi aparadhika pratikriya ki avashyakata hai|
arambha karane ke lie, sangathita aparadhika tantra para suchana ekatrita, adana – pradana evam vishleshana karanaeka labhadayaka drishtikona hai, manushyom mem avaidha vyapara ko sambodita karane ke lie, khasa taura se avaidha vyapariyom ke dvara prayukta marga para suchana|
mam aura bachche ki dekhabhala ka itihasa 19vim sadi ke madhya mem audyogika kranti se prarambha hua tha|
usa samaya auratom ne karakhanom mem kama karana shuru kara diya tha jisaka bachchom ki dekhabhala para asvabhavika prabhava pada़a|
auratem apane kama ki apekshaom ke karana stanapana nahim karava pati thimaura isalie khadya sampurakom ki khoja arambha ho gai|
gaya ke dudha ko chuna gaya kyonki vaha puri duniya mem asani se upalabdha tha|
logom ko tathapi isake bhinna sanyojana ki janakari nahim thi aura isake prayoga se anaraatha ho gaya : bachche mara gaye|
itane varshom mem paristhitiyam dhire dhire badala gaim haim, aura yaha aja taka jari hai, aura shishu poshana evam shishu dekhabhala visheshajna jaise balachikitsaka evam aushadha karaka ka karyakshetra ho gaya hai| 
auratem ina visheshajnom ke pasa ati haim kyonki unaki vikasa ki vritti ki samaja khatma ho gai hai aura unhe aba yaha nahim pata chalata ki kaba dudha chhuda़ana chahie|
isake lekara vishala asta vyastata hai kyonki kayadom ko eka pidha़i se dusari pidha़i taka pahunchane ki pratha banda ho gai hai aura aba midiya ke dvara sancharita kiya jata hai|
yaha janakari halanki nishpaksha nahim haim aura spashtataya vaijnanika aura apakshapati salaha chhalayojita ki jati hai – yaha juthi salaha vijnapana hai|
eviai mem hamanem hamesha juthi janakari ke khilapha sangharsha karane ki koshisha ki hai|
mansa ki avashyakata nahim hai, seliniyama ka srota nahim hai, aisa khana nahim hai jise roja khaya jae, kyonki yaha svastha bhakshana ke sandarshikaom evam yuropiyana koda phaॉra kainsara privenshana ki anushansaom ke viruddha hai aura mansa bachchom ke vikasa ke lie avashyaka nahim hai| 
yadyapi midiya hamem isake viparita bata batati hai parantu kai hara unhe apani bata badalane pe majabura kiya gaya hai hamare dabava ki vajaha se|
mota bachcha aba oka bigada़e hue rogavijnana ka ora sanketa karata hai|
bachche galata bhojana khate haim, bahuta adhika ushma vale aura bara bara bimara pada़te haim|
gaya ka dudha mam, machhali aura panira keka evam anya chija़om ke satha milate haim|
stanadhariyom ki vishishtata mem apane bachchom ke lie pyara sammilita hai, joki unake dimagi vikasa evam dudha pine se nirdharita hota hai| 
kisi anya prajati ka dudha pilana, usa sahi prajati ke jyada naja़dika jana mana jata hai jo khataranaka pratisanketom ki ora le jate haim|
manushya ke dudha mem 12% protina, mainasa 0.3% gaira protina, naitrojana, jo ki kevala 0.9% chhoda़ta hai|
avashya sharira se sokhe gae ki matra isase kama hoti hai|
gaya ke dudha mem chara guna adhika protina ki matra hoti hai aura kharagosha ke dudha mem dasa guna adhika| 
manushya aise stanadhari haim jinhe kama prochina ki avashyakata hai, kyonki hama dhire dhire badha़te haim, pachchisa varshom ki avadhi mem aura laingika paripakvata bada mem hoti hai| 
19vim shatabdi me rajah srava 18-20 varsha ki unra mem shuru hota tha, jabaki 1960 mem vaha gira ke 14 varsha para a gaya aura aba pahali bara 8-19 varsha mem hota hai| 
yaha saba ahara se sambandhita hai, sabase gambhira pahalu vriddhi mem teji hai, joki antah sravika uttejana ka parinama hai|
manushya ke dudha mem sharkara ki bahulata hai jo dimaga ko pushti deta hai, para khanija lavana kama hai|
gaira prakritika bhojana ka prayoga manushya sharira ko bara bara bimara hone ki ora pravritta karata hai, jisaki shuruata jivana ke prarambhika mahinom mem hoti hai|
dudha chhuda़ai ke hote hi, pramukha jora anna aura prakritika bijom ke sanyojana mem phalom aura sabjiyom para bhi hona chahie|
jaisa ki apa dekha sakate haim pashu ahara anavashyaka hai; bachchom ko palane ke lie do bahom aura do stanom ka avashyakata hoti hai|
bhavanatmaka ahara sabase avashyaka hai : mata pita ki samipyata ko chhuna|
hamane bachchom ki sharirika vikriti ki hai kyonki jaba unhe hamari avashyakata hoti hai hama apani duri banae rakhate haim jabaki jaba ve svatantrata ki unra taka pahunchatem hankha 14-16 ki unra mem hama una para nirbhara rahate haim|
pri vidyalaya ki dusari ghatana aidisa ebaba siti aidaministreshana ke bheje gaye visheshajnom ke dala dvara kie gaye karya ka mulyankana tha|
isa mulyankana ka uddeshya ye nirdharita karana hai ki karyakrama ne apane uddeshyom ko prapta kiya hai|
isa dala ne kai savala uthae jisake lie usane projekta ke elaemasisi sanchalaka se uttara mange| 
saba milakara yaha dala elaemasisi ke samudaya ke bachchom ke lie kie gaye kalyana ke lie atyadhika prashansaniya hai tatha unhe bhavishya mem apane prayasa tivra karane chahiem| 
isa dala ne elaemasisi ko apane bhavishya ki yojanaem aura karyakrama upayukta sanstha ko agreshita karane ko kaha| 
1999 mem kuchha prayatna pathana kaksha ki suvidhaom mem sudhara ke lie kie gae jo ki bada़e bachchom ke lie tha|
kai shanivara ko kendra mem pathana aura adhyayana ke lie ate haim|
eka shikshaka ki madada se kuchha mataom ne bunai mem ruchi dikhana yathavata rakha hai|
eka adhikrita lekhapala ne 1996-1999 mem elaemasisi ke lekhom ko akekshita kiya tha| 
ankekshakom dvara di gai prativedana va sipharishem borda dvara svikari gaim thim|
hama una sabhi dalom aura manushyom ko abhara vyakta karana chahengem jinhonne elaemasisi ko 1999 mem vittiya sahayata pradana ki|
sata varsha se kama unra ke bachche ko vidyalaya mem pravesha milega|
isa vajaha se ye bachche prarambhika skulom mem achchha pradarshana nahim kara pate aura kai skula chhoda़ dete haim aura kai sada़kom para a jate haim| 
pahala elaemasisi prarambhika skula 5 bachchom ki bharti se sitambara 1994 mem sthapita hua, yaha sankhya uttarakali varshom mem badha़kara 150 ho gai|
sitambara 1999 mem eka anuvarti sarvekshana ayojita kiya gaya tha usi prativasa mem jadam elaemasisi 1994 se prachalana kara raha hai|
isa sarvekshana ka parinama yaha darshata hai ki adhikatara vahi bachche jinhone prarambhika skula ka labha uthaya ve prathamika skula mem namankita haim|
bachchom ka prathama dala panchavim kaksha mem pahuncha gaya hai|
anya samasyaom mem se skula chhoda़ne ki samasya 1994 ke prathama sarvekshana mem samane ai|
sitambara 1999 me prarambha hone vale nae shaikshika satra mem 75 bachche namankita hue the|
aba ye bachche khelane aura padhane ke lie paryapta samaya hone ki vajaha se pura dina ruka sakate the|
hala mem skaॉtisha karyapalika ne balakom va yuva janata ke manasika svasthaya evam jo kathinai mem hem unako milane vali sovaom ko sudhaarane ki yojana ke lie aneka mahatvapurna kadama uthae haim|
balakom evam yuva janata ke manasika svasthya eka protsahana, nivarana evam anurakshana ka dhancha (ephapipisi) ke aktubara 2005 ne prakashana ke satha karyapalika ne niti prasanga nishchita kiya|
svasthaya ke lie uddhara, hala hi mem prarambha manasika svasthya ke lie uddhara (disambara2006) mem abhipurti ke lie vivarana evam eka svastha bhavishya uddhara (pharavari 2007) ke satha uddhara ke lie eka vyapaka prastava nirdharita kiya gaya|
dhanche ne kendra se manasika svasthya niti ke pratiskamadana va bhavanatmaka kalyana ka protsahana kiya aura jo kathinai mem haim unake lie shighratishighra sambhava prabhavakari hastakshepa ko uparjita karane ke lie anukula pratibaddhata pradarshita ki|
navajata manasika svasthya ka prakata hota hua kshetra ina donom ayamom ke lie spashtataya prasangika hai aura skaॉtalainda mem bhavishya mem hamari gatividhiyom ke mula tatva banane ki sambhavana hai|
haidsaapaskaॉtalainda ne adhika amusandhana mem sahayata karane ke lie navajata manasika svasthya ke sambandha mem shreshtha pramana ke sankshipta sara ki avashyakata ko pahachana hai|
glasago vishvavidyalaya ke daॉ kristina pakaringa ko eka aise parikalana ko vikasita karane ke lie eka visheshajna samuha ka ayojana karane ko kaha gaya|
yaha praleka navajata manasika svasthya : kartaom ke lie eka sandarshika daॉ. pakaringa aura unake sahakarmiyom ke dvara uttaradayitva lie gae karya ki upaja hai|
unhone hamem vartamana pramana ka sankshipta aura bahuta pathaniya sara diya hai jo balakom, abhibhaavakom va sevadarom ke satha karya karane vale usa vyapaka varga ke lie labhadayaka evam mulyavana sambhavita haim, aura maim unake lie kritajna hum|
yaha eka vyavaharika rupa se nishchita parikalana hai jo na kevala vidhika shakhaom mem sammilita svasthya, shiksha va samaja seva ki prakriya se sabandhita vichara ke lie apitu svaichchhika kshetra aura samudayika vinyasa ke lie bhi upayukta hai|
maim prasanna hum ki maim apako inase parichaya karane mem samartha hum aura asha karata hum aura asha karata hum ki skaॉtalainda mem sarvatra balakom aura yuva janata ke manasika svasthya ko sudharane ke lie hama sabase satha kama karane ke tarike mem eka mahatvapurna yogadana dega| 
haidsa apa skaॉtalainda una saba logom ko dhanyavada dena chahega jinhone isa vivarana ko ekatrita karane mem sahayoga kiya tha aura khasakara shaॉrta laipha varkiga grupa ऑna inphenta mentala hailtha aura usake pramukha daॉ. krastina pakaringa|
isake satha hi haidsaapaskaॉtalainda jena maikakyurari ke anusandhana sahayoga ka abhara vyakta karate haim|
balakom unake abhibhavakom aura sevadarom ke satha jo kama kara rahe haim aura balakom, yuva janata aura vayaska ko jo bala anurakshana aura abhibhavakom ke vishaya ke anusara shikshita karane mem vyasta haim ke lie sandarshika labhadayaka hoga|
vishishtata vali ina prakriyaom ka yaham skaॉtisha ekjekyutiva sharta laipha varkinga grupa ऑna mentala hailtha dvara navajata manasika svasthya sudharane ko protsahita karane ke lie upayukta rupa mem mulyankana kiya gaya hai|
dusare hastakshepa jinaki prabhavita ka matra paryapta rupa se kama samarthita haim, kabhi vivarana diya gaya hai evam unake kama anumodana ke karanom para bhi vichara vimarsha kiya gaya hai|
uttama abhibhaavaki balaka ke manasika svasthya va kalyana ki pragati ke lie mulatatva haim|
balakom ke prathamika anurakshakom ke rupa mem saba abhibhavakom ko samarthana evam sahayata dene ki avashyakata hai parantu khasakara taba jaba vaha kathina sthitiyom mem abhibhaavaki kara rahe hom ya apane bachchom ko palana karane ke tarike ke bare mem sunishchitata ka samana kara rahe hom|
garbhavastha va janma dene ke samaya kendribhuta kie gae hastakshepa eka balaka ke manasika svasthya samasyaom ko rokane ke lie atyanta prabhavakari haim| 
isamem vaha hastakshepa sammilita hai jo mata aura shishu ke kalyana ko sudharate va badha़ate haim evam mam navajata ke sambandha ko protsahita karate haim| 
jo garbhavastha ke manovaijnanika pahalu ka prekshana karate haim, uttama prarambhika abhibhavaka – balaka anyonyakriya, anurakti ko protsahita karate haim abhibhaavakom ki samasya nivarana kshamata ko sahayoga karate haim evam pita ki bhumika ko rekhankita karate haim|
da menntala hailtha ऑpha childrana enda yanga pipala : a phremavarka phaॉra promoshana privenshana enda keyara, skaॉtisha ekjikyutiva, 2005| 
balaka shaaririka va vikasatmaka rupa se aparipakva janma lete haim uttarajivita ke mula sadhanom se bhi rahita| 
yadyapi navajata dusare manushyom ki ora prakhara rupa se anukula hote haim va samajika sambandhana ke lie anupama rupa se tatpara hote haim|
ghrana, shravana, drishti va pravasthabaddha acharana saba janma ke prarambhika ghantom se sammukha anyonyakriya ke lie vishesharupa se anukula hote haim| 
uttarajivita ke lie adhika paripakva anurakshana ke abandha ki avashyakata hoti hai, apitu chirakalina bhavatmaka samajika evam jnanatmaka vikasa mem nirnayaka bhumika nibhate haim| 
mastishka koshika ke bicha mem sambandha banane aura tutane (sainaipatojenesisa va pruninga) pahale varsha mem advitiya gati se agrasara hoti hai jo jivana ke kisi kala mem nahim hoti|
isa prakara ke bahuta se prathamika anubhava jo chetana rupa se smarana nahim kie ja sakengem, sarthaka paravarti vikasa ke lie adhara rakhate haim|
eka snehi samvedanashila va prabhavaniya vayaska inphainta mentala hailtha : a gaida phaॉra prektishanarsa ya thoda़e se parichita vyaska uttama sambhava parinama ke ashvasana ke lie avashyaka haim| 
navajata manasika svasthya ke protsahana ke lie usa sanrachana ke sahayoga jinake dvara navajata anukulana vatavarana anubhava kara sakem ki avashyakata hoti hai|
sarvavyapaka svasthya sudhara va shiksha sansadhana isa pratibimba ka ansha hai| 
prasavika, svasthya nirikshaka/ sarvajanika svasthya upacharika ko vishala bahumata dvara parakha jata hai evam pratyeka ki balakom ko shreshtha avastha mem sahayoga dene ki va una parivarom ko janchane ki, jinake lie adhika vahana hastakshepa ki avashyakata hai, mem anupama bhumika hoti hai| 
shishu mem aparipakvata, asvasthata, janmajata va prasavika jatilataom ke rupa mem khatara vidyamana hota hai| 
shishu ka svabhava va vyaktitva bhi sambhavatah samasya utpanna kara sakata hai|
sama rupa se khatara balaka ko milane vale vatavarana ki prakriti, garibi abhibhavaka ki manasika ya sharirika asvasthataya avayaska abhibhavakom dvara utpanna kie ke vatavarana mem vasa kara sakata hai|
akasara khatare bahukramagunita hote haim, evam abhibhavaka, balaka mem vishishtataem, abhibhavaka – balaka ke madhya avesha aura vyapaka vatavarana mem karaka sammilita karate haim|
vaha abhibhavaka jo jnana prapta karane ke lie, ya svechchha se sahayata ke lie kama sambhavita hai ko vishishta dhyana dena avashyaka hai| 
parivara jinake lie shiksha eka chunauti hai ya jina ke lieangreji unaki prathama bhasha nahim hai ke lie vishishta avashyakataem ho sakati hai jo shishu ko atirikta asuvidha mem dala dete haim va sansadhana va sevaom ke sarala abhigamana mem bhi badha dalate haim|
navajata manasika svasthya ke vishayom ko sambodhana karane ke avasara abhibhavaka aura balaka ke jivana mem aneka pada़ava mem utha sakate haim|
vidyalaya pathyakrama mem garbhavastha se pahale niji va samajika vikasa ka pahala avasara hota hai evam prayah sarvatra pradana kiya jata hai|
prasava purva va parijanma suchana va sevaem parivarom se prayah apane pratyabhuta samparka ke karana sarthaka hote haim|
atirikta avashyakata vale parivarom ko pahachanane ke avasara una ko milenge jau yaha sevaem pradana karenge|
eka bara doshapurnata ka parichaya ho jata hai, dusari shakhaom ko sevaom ka vitarana hastantarita kiya ja sakata hai|
ina saba mem jina ke lie kitabem suchana grahana karane ka akarshaka madhyama nahim hoti hai ilaiktraॉnika midiya va prodyogiki se pravesha kholane ke avasara hote hai|
navajata manasika svasthya eka vishishta balaka ke jivana mem evam kathinaiyom ke sambhava vikasa kshetraom ke lie prastavana rupa mem eka shighra hastakshepa vala kshetra hai|
karyakramom ka kathorata se mulyamakana karana chahieva karyakrama prarambha hone se pahale mulyankana yojana sthapita karani chahie|
disambara 2005 va mai 2006 ke madhya svasthya prathamika shiksha, samaja seva va svayam sevaka khanda se eka laghu jivana karyakari samuha ne uttama navajata manasika svasthya ke sahayoga ke sakshya ke lie upaya evam skaॉtalainda ke lie balakom ke shreshtha nishkarsha nishchita karane ke lie sujava ke lie milem|
vaidhanika va svayam sevaka khanda niti nirdhaaraka vyavasayika shiksha, svasthya, prathamika shiksha, samajika seva khanda ke pratinidhi samuha mem sammilita the kyonki ina saba ki parivarom mem uttama sambandha protsahita karane ke lie khasakara navajatom ke lie, eka bhumika dekhi ja rahi hai|
samuha pramukha daॉ kristina pakaringa evam atirikta manovaijnanika jena maikakyuri dvara vibhinna hastakshepa ke prabhavita ke sakshya adhara ke lie atirikta anusandhana kiya gaya|
samuha ke sujavom se atirikta anusandhana taka yaha pralekha prachura hota hai|
haidsa apa skaॉtalainda – balakom aura yuva janata ke svasthya ki rashtriya pariyojana ki ora se yaha karya kiya ja raha hai|
haidsa apa skaॉtalainda, manasika svasthya sudhara va kalyana ke lie, skaॉtisha karyapalika rashtriya pariyojana ka ansha hai|
samuha ko skaॉtalainda mem uttama prayoga ke lie pratimana parakha ka, laghu khatare va adhika khatare donom parivarom mem uttama manasika svasthya ko protsahita karane ka prekshana tha| 
lakshita va gahana sevaom ko prakashita ya gre sahitya mem uchita nirupita rupa se sambhavaniya vicharita kiya gaya|
adhika sarvajanika sevaom ka bhramana kiya gaya kyonki unaki pahuncha kai bara vyaktigata hastakshepa se vyapaka thi|
balakom para sandhana ke pramanana, vo dono sarvadanika se gahana pratipadana ke nirdharana ke ilae sakshya adharita pranali para sankendrita tha| 
sakshya ki kami, sanghata ki kami pramanita nahim hoti hai apitu lambe pramanika karyakrama ke lie vishesha rupa se prabhavita ke lie sakshya prabandha karane ke lie eka dayitva hota hai, jisamem rogi ka santosha sammilita to ho sakata hai parantu usa taka simita nahim ho sakata|
disambara 2005 va mai 2006 ke madhya samuha pancha avasarom para mila, samuha sadasyom nem apani sevaom aura karyom ko prastuta kiya, samuha ne svasthya sudhara vastuom, prakashita va drishya shravya donom arupom mem ka punarikshana kiya|
jena makakyuri aura kristina pakaringa ne navajata manasika svasthya sansadhana, sevaem va upaya para sahitya ka kramabaddha pata lagaya, satha mem mantrana va nirikshana ke parinamasvarupa kuchha atirikta sahitya pramukha lekhom se ankita kiya gaya| 
vivarana, sahitya va mulyankana dusari sevaom se prapta hue jisamem enna phrayuda perenta impaikta projekta, batarakapa projekta metinga aheda evam ekaॉrna projekta shamila haim|
navajata manasika svasthya ke sandeshom ke kisi mem bhi vyapana khandashah prakara ke alava, jnana ke samanya marma aura anushasana ke vishayom ke jnana evam vaidhanika va svayam sevaka khanda ki sammilita shakhaom para nirbhara hai| 
yaha paramarsha diya gaya hai ki hara enaechaesa mandala mem eka navajata manasika svasthya kordinetara ko niyukta kiya jaye|
kordinetara ko shakhaom va anushasanom mem navajata manasika svasthya ko protsahita kiya jana chahie va rakha rakhava evam nishtha ke karyakrama ko surakshita rakhana chahie| 
nae parivarom se milane aura unaki va unake balakom ki anuraksha mem eka bhumika nibhane ke sambhavita karmachariyom ka prashikshana, samajika karya, prathamika varshom ke kartaom svasthya va shaiksha ke apa para susangata prastava ka sudridhikarana karata hai|
anurupa prashikshana vikalpa ko sulabhata ko badha़ane ke satha hi vaha sunishchita karane ke lie ki karmachariyom ke pasa upasthita rahane ke kie paryapta samaya hai va unake karyakshetra unako nirmukta kara sakate haim ke lie kadama uthane ki avashyakata hai| 
karyakshetra kisi bhi parivartana, jo isa prashikshana ke vishaya evam karmachariyom ke samaya ke nirdharana evam chala rahe nirikshana ke lie bajata aura sansadhana jinake bina prashikshana akela hi nirarthaka hai ko karyanvita karane manka samartha hona chahie|
chikitsa va balaka anurakshana karmachari ke kendra pashikshana ne paramarsha diya haiki navajata manasika svasthya ko sarvajanika svasthya paricharika, snataka chikitsaka, prasavikaom, balaka anuraksha va balaka manasika svasthya kartaom ke kendra pathyakrama ke kendra pathyacharcha ka ansha bana dena chahie| 
bahu anushasanika pariprasava manasika svasthya prashiksham ke lie manadanda va kendra yaugyataom ka vartamana mem vikasa kiya ja raha hai evam navajata manasika svasthya ke lie samantara manadandom ka bhi vikasa karana chahie|
vishishta sieemaecha kartaom ke prashikshana ka sujava diya gaya hai aura navajata manasika svasthya ke eka vishishta prashikshana mapadanda haidsa apa skaॉtalainda dvara adhikrita hona chahie vishishta sieemaecha kartaom ko vibhinna anushasana se sajjita karane ke lie|
isamem manovaijnanika, manovikriti –vijnana manahchikitsa samajika karya evam vani va bhasha ke chikitsa vijnana ko sammilita karana chahie (parantu simita nahim)|
charana 1 va 2 kartaom mem prathamika varshom ke kartaom, samajika karya va svayam sevaka sangathanom dvara prastavita ki gai sarvajanika sevaom ko chailda enda edolsenta mentala hailtha ke vishishta paramarshadataom se yadi avashyaka paramarsha, nirikshana va nirdeshana ke lie pahuncha honi chahie| 
manovaijnanika manovikriti – vaijnanika, bala manashchikitsaka va aisi vishishta sunijnata vale karta aise sansadhana prastavita kara sakate haim|
navajata manasika svasthya mem haidsa apa skaॉtalainda mapadanda dvara vishishta prashikshana prapta karana sabake dvara apekshita hai| 
prasavikaem vistrita bahumata mem hone vali mataom ko dekhati haim evam atirikta sahayoga ke bare mem janane va abhibhavakom ko joda़ne ki anupama sthiti mem hoti hai| 
svasthya nirikshaka/ sarvajanika svasthya paricharikaem bhi isa vishaya mem pramukha sthiti rakhate haim|
solihala eprocha mem udaharanasvarupa sanyukta prashikshana ko , prasavikaom, svasthya nirikshaka/sarvajanika svasthya paricharikaom ko parivarom ko atirikta sahayogi ke lie mulyankana ke lie adhika sajjita karane ke lie protsahana dena chahie evam navajata manasika svasthya ke sahayoga ke lie unaki bhumika ko adhika samaja deni chahie|
prasava purva sevaem bhi karuba hara parivara taka pahunchane ki bada़i uttama sthiti mem hoti haim|
navajata manasika svasthya para sanyukta prashikshana kartaom ko prathamika navajata sancharana ki mahatta ke bare mem uchita sandesha vitarana ke lie sajjita karata hai|
abhibhavakom ko jaise unake balaka ki badha़ti aura badalati kshamataom ko samajane evam prashansa karane mem sahayaka aura sahayoga dena chahie|
metaranala asaisamenta ऑpha da biheviyara ऑpha hara inphainta ko janma ke bada abhibhavakom ki janakari badha़ane ke lie chara saptaha taka hara saptaha taka pura karana chahie| 
aneka anushasanom va shakhaom ke para sanyukta karya abhibhaavakom ki chinta va bala samasyaom ki tivrata ko kama karane ke lie solihala eprocha mem prashikshana ke sahayoga ko asaradayaka darshaya gaya hai|
garbhavastha purva svasthya sudhara suchana, vyapaka janasankhya, satha mem kishora bhi jo age ja kara yuva abhibhavaka banenge taka pahunchane ka mulyavana avasara hai|
abhibhavakata para va usake bare mem shiksha, ko vidyalayom mem ninna stara para pradana kiya gaya hai kai bara nirdhana va anuchita sansadhanom taka hi abhigama hota hai kuchha shikshaka va adhika vidyochita chhatrom dvara apransangika rupa se dekha jata hai vishesha rupa se lada़kom dvara evam kai bara avashyaka pratibaddhata va kshamata ke bina shikshakom dvara padha़aya jata hai|
hopa enda sharalainda vivarana ki anushansa yadyapi navajata manasika svasthya ke lie vishishta nahim hoti hai, parantu abhibhaavakom se purva yuva janata dvara usa bhumika ki naitikata va apekshaem dikhane ki sahayata ke lie tarike pahachanane mem sahayaka hai| 
navajata manasika svasthya ko vyaktigata va samajika abhibhavakata ke chhatra ke bhitara avarana karane ke lie aise karyakrama ke avasara| 
shishuo ke bare mem vichara vimarsha karane ke lie bachche bhi jo shayada pahale udasina the sambandhom ke vichara vimmarsha ke adhika samanya sanrachana ke sandarbha mem akarshita ho jate haim| 
kishora garbhavastha ko ghatane se atishighra abhibhavakata ke viparita parinaama sahane vale navajatom ki sankhya bhi kama hogi|
yadyapi kishora garbhavastha ke rokathama mem BTIO ki prabhavakarita ka koi yuke sakshya nahim hai, parantu agara vyaktigata va samajika shiksha ke karyakrama ke avayava ki taraha prayoga kiya gaya to vaha garbhadhana se purva chhote balaka ki anuraksha ki apekshaom ke yatharthavadi purniveshana ke lie mulyavana srota hai|
ilaiktraॉnika khelom ke vikasa aura unaki bachchom aura yuva janata ke satha vyapaka ghanishthata bachchom aura yuva janata ko svastha shiksha samagri se parichaya karane ke nae vikalpa kholata hai|
NINTENDOGS aura aba BABYZ jaise khelom ki prasiddhi jisamem khiladi ki janavarom aura bachchom ki anuraksha karani pada़ti hai, yaha sujata hai ki khela aneka yuva balakom ke lie sulabha aura ruchipurna hai|
samana svasthya sudhara samagri utpadita ki gai hai evam dhunrapana nirodhaka karyakrama ke ansha ke rupa mem uchita mulya para vidyalayom mem vitarita kiya jae to eka mulyavana sansadhana bana sakata hai|
yadi vyaparika khela nirmata eka isase adhika parishkrita khela ka nirmana va vitarana karane lage to isaka eka vyapaka va dirghakalina prabhava hoga|
kai vyaparika sangathanom ke ऑnalaina shishu klaba haim jaham abhibhaavaka suchana prapta kara sakate haim evam samparka kara sakate haim|
NHS borda kshetrom ko sthaniya prabandhakom ke satha ina sambhavanaom ko khojani chahie va navajata manasika svasthya vishayom ko sthaniya prapya suchana ka ansha banana chahie| 
eka navajata manasika svasthya vidiyo/ dividi ka nirmana ho sakata hai nirmana vyaya uchita hona chahie satha hi pratilipiyam bahuta hi kama mulya para nirmita hona chahiem| 
prasavikaem vistrita bahumata mem hone vali mataom ko dekhati haim evam atirikta sahayoga ke bare mem janane va abhibhavakom ko joda़ne ki anupama sthiti mem hoti hai|
da kotabrija inavestinga ina inphaaintsa mulyankana vivarana varnana karata hai ki kaise samudayika prasavikaom ne saulihala eprocha ko pratikriya dikhai hai|
pahale vaha ashankita the ki unaki bhumika ko navajata manasika svasthya se kya karana hai|
vaha mam ke bhutakala va vartamana manovaijnanika kalyana evam samajika sahayoga ke bare mem prashna puchhane mem aspashta hote the isa dara se ki vaha antarvedhi haim evam una samasyaom ko ujagara kara dega jina ko sambhalana unhe nahim ata|
manosamajika suchana ke prasava purva shiksha se parichaya ki prabhavita ke sakshya ke krama mem|
antima traimasya mem sabase prakprasava kakshaem lagati haim, ye prasuti para sankendrita hoti haim evam inhe adhikatara madhyama varga ke mata pita prayoga mem lete haim|
chhote, alpashikshita evam samajika rupa se vanchita varga ki kama mem lagane ka sambhavana nyunatama hai|
prasavikaem evam svasthya nirikshana seva dvara chinhita kie gae aise matapita jinhe apane bachchom dvara kathinaiyom ka masana karane ka khatara hai, ko kama pe laga gaya hai|
mata pita prakprasava dala ke 6 satrom mem bhaga lete haim jo ki manovaijnanika samajika evam vikasashila vishayom para 20 saptaha ki garbhavadhi para chalate haim, isase pahale ki samupasthita janma sannikata evam|
jo parivara isa dala se juda़e hue haim; jo ki eka shyora starta parivara parisara mem ayojita hota haisa unhone shyora starta ke satha satha sahayaka janmottara sevaom manna badha़ti ruchi dikhai|
anupuraka sahayata ka upayukta lakshya khatare mem vale dalom ki aura karane ke lie yaha avashyaka haiki ina dalom ko pahachanane ke lie yantravali apani jagaha para ho evam prabhavi yantra unhe upalabdha karavae jaem|
intigreteda asaisamenta phremavarka ke tathya labhakari haim parantu pramanita karyavahi se juda़e hue nahim haim|
prasavikaom ke sakshatkara dvara janchana chahie jisase entinetala saikolojikala riska jaisi prashnavali ke satha madaka evam madyasara sevana gharelu hinsa evam prasuti itihasa shamila hai|
pahale parikshana mem upacharika adhika samvedanashila mahilaom se mili aura unake ahara mem sudhara kiya evam garbhavastha mem unake dhunrapana ko kama kiya evam aupacharika evam anaupacharika sahayata ke strotom ka prayoga kiya|
unaki samayapurva prasuti mem kami ai evam kishori mataom ke bhi vrihattakara shishu janme|
janmake pashchata do varshom taka ghara mem milana jari raha evam hastakshepa dalom mem chotem aura antargrahana mem kami ai|
hastakshepa dalom ke sakaratmaka parinama 15 sala bada ki karyavahi mem bhai najara aye jinamem avivahita gariba mataom ke uttaravarti garbhom mem kami ai, alpa samaya kalyana labhom para laga aura aparadhika abhishansaom evam madaka padarthom se juda़i ghatanaom mem kami ai|
garbhavastha ke parinama pratikriyaom mem kama najara ae parantu matrika jivana patha ke badalava evam behatara shishu palana mem pratyaksha the|
durbhagyapurna, nirodha ke yadrichchhika niyantrana ki koshisha auraprarambhika hastakshepa ke karyakrama samanyatah kuchha hi samaya ke lie sanketika dalom mem labhakari pramanita hue haim aura panchamela ‘khatare mem’ vale dalom alpa ya kisi mulya ke nahim haim|
rashtriya svasthya pradarshana pariyojana startinga tela ka uttaradayitva glasago mem liya gaya tha aura use oldasa maॉdala para adharita tha| 
isa adhyayana ne darshaya ki pariyojana ke pahale charana ke antam mem hastakshepa aura niyantrana kshetra mem bahuta kama antara samane ate haim joki mata pita bachchom ke rishtom ki mukhya suchakanka para adharita hai|
kshetra para adharita avashyaka suchakanka balki jyada achchha hota hai vyaktigata mulyankana aura achchhe svasthya milana sevaom ki prishthabhumi se joki parivarom ke hastakshepa aura niyantrana ke lie upayukta haim, jisane samanya dala ka prabhava kama kara diya hai| 
prasava purva sevaem pratyeka parivara taka pahunchane ki majabuta sthiti mem hai|
shishu ke manasika svasthya ko pravasikaom ke prashikshana mem shamila karane se sarvabhaumika shishu dimagi svasthya ko sakaratmaka avega milata hai|
do vishishta hastakshepom ka bada ke parinama para sarthaka prabhava dekha gaya hai|
yaha una kale alpa shaikshika kishori mataom ke manaka dala ke samane pradarshita kiya gaya hai| 
unhe phira madarsa asaisamenta ऑpha da biheviyara ऑpha inphenta skela (emaebiai) ko shishu ke janma ke pahale mahine ke eka saptaha ke antarala para bharane ke lie kaha|
eka dusare dala ne emaebiai ko usi samaya para pura kiya para enabieesa ka anupalana nahim kiya aura tisare dala ne gaira hastakshepa niyantrana dala ki bhumika nibhai|
brejalatana plasa emaebiai aura akela emaebiai donom dalom ne amane samane mam aura bachche ka chara mahine para parasparika kriya ko sampanna kiya aura sarthakatapurna achchhe dentala davalapamenta skrininga skorsa dikhaem|
baraha mahine para donom hastakshepa dalom ne sarthakatapurana behatara bele davalapamenta skorsa dikhae| 
parasparika kriya aura jnanatmaka parinama donom hi, joki isa samaya mem mam shishu ke parasparika kriya se sambandhita haim, ko sudhara gaya mataom ko bachchom ki kshamataom ki prashansa karane ka avasara deka aura isa prakara adhika prabhavashali parasparika kriya ko prerita kiya|
prasavapurva manasika svasthya ikaiyam jaham mam apane bachche ke satha bharti ho sakati hai, ke vikasa ka svagata hai|
mam aura shishu ke rishte ko bala pradana karane ka prayojana evam unake manasika svasthya evam vikasa ka surakshana karmachariyom ka mula mantra hai|
tina hastakshepom ke adhyayana jisamem sevaom aura mata pitaom ke bicha kama vyaktigata samparka hua hai, se dekha gaya hai ki unaka sarthakatapurna prabhaava matrika samvedanashilata aura lagava para hua hai|
ina karyakramom ko alpa vyavasayika samaya atah vyaya ke nivesha ki avashyakata hai|
kyonki ina karyakramom ke kharche kama haim aura prabhava bada़e haim (d=0.62), kuchha vibhinnataom ko chhoda़kara, inaki sipharisha vyaya prabhavi hastakshepom mem seva prabandhaka se adhika ho sakati hai|
phidinga yora bebi vida lava vale vidiyo ko kali kishori mataom ke sahayoga se banaya gaya hai jinhone vishaya vastu aura vyakhya ko isa prakara ye nishchita karate hue taiyara kiya ki unaki vyakhya sanskritika evam unra ki drishti se uchita hom| 
eka sadharana loka svasthya anushansa mata pita aura bachche ke amane samane purvabhimukhikarana ke lie labhakari ho sakata hai|
pichhe dekhati hui gada़iyam jo bachche ko mam ke chehare ko dekhane de, eka sadharana sa udaharana ho sakata hai|
yuropiyana arli promoshana projekta eka mulabhuta svasthya dhyana seva hai bachchom ke manasiika svasthya ko protsahita karane ke lie|
pancha yuropiyana deshom ke prathamika svasthya dhyana seva karmachariyom ko agradarshi mataom ke svasthya protsahana sakshatkara, prasuti ke eka maha ke bada ke lie prashikshana diya gaya| 
ve mataem jinhe kuchha vishishta jaruratom ke satha pahachana gaya, ko phira paramarsha adarsha ka prayoga karate hue dekha gaya| 
do varsha ki unra taka hastakshepa dalom ki mataom ko adhika aveshtita aura kama dandika dekha gaya|
chailda autakama deta ne dikhaya hai ki bachche te vyavaharika samasyaom para ya 24 maha para jnanatmaka vikasa para hastakshepa evam niyantrana dalom ka bahuta simita prabhava pada़a hai, paranchu inamem una deshom mem bhinnata dekhi gai hai jo purva vidyamana sevaom para nirbhara haim|
garbhavastha mem manochikitsaka karya ka mulya hai evam staॉkahoma mem bachchom ke svsathya kendrom mem dekha gaya hai ki ve eka manosamajika drishtikona ko dekha bhala ke lie protsahita karate haim samasya paramarsha dvara, prashikshana aura sahayata dvara jo ki manovaijnanika dvara bala karmachariyom ko di jati hai|
dagalasa evam anya ne solihala eprocha ka vikasa kiya hai una sabhi adikariyom ke lie jo bachchom ke satha kama kara rahe haim|
samaveshana, parasparikata evam vyavahara prabandhana ke mula avadharanaom ko eka bahuvibhagiya karmachariyom dvara pradana kiya jata hai evam unako sansadhana karmachariyom ka adhara hota hai| 
isa drishtikona ki abhikalpana parivarom ki avashyakataom ko pura karane ke lie hai jisaki jarurata bachche ke janma se pancha sala taka ke prarambhika vikasa ke lie hoti hai|
golja़ aura sainyualsa una kuchha patrom mem se haim jo pitaom ke yogadana aura shishu ke vikasa ka ananda ki kshamata ko sambodhita karate haim| 
eka ucharika dvara ghara mem pradana kie gae hastakshepa, srasuti purva chara saptaha para, jisamem vidiyo chepa dvara shishu snana shithilikarana takanika evam shishu malisha ka pradarshana kiya gaya hai, ne pitaom ki ane vale de mahinom mem bhagidari badha़a di|
anuvarti pe bachchom ne apane pita ka abhivadana ankhom ke samparka, muskurahata pahuncha kara evam uchcharana se kiya| 
bukastarta joki buka trasta evam skaॉtisha ekjikyutiva dvara nidhika hai, eka bahuta hi vyaya, prabhavi maॉdala hai matapita aura bachche ke bicha sampreshana ko prerita karane ka evam isaka ashcharyajanaka rupa se bahuta adhikaprabhava bhasha evam sankhya saksharata para skula jane ki unra mem hi pada़ jata hai| 
nihshulka kitabom ka gattha bachchom evam janakari mata pita ko svasthya nirikshaka /loka svasthya upacharom ko banti gai|
griha milana ko chhote bachchom vale parivarom ko sahara dene ke lie eka adarsha maॉdala mana gaya hai|
satha karyakramom ka eka kara vishleshika punaravalokana ne tathapi ye sujaya ki griha milana tina varsha ke adhka ke bachchom ke satha sankhyikiya rupa se sarthaka hai|
parinamom ka naidanika abhipraya bahuta chhota hai aura vyaya prabhavi nahim hoga, halanki bachche ke satha kie gaye durvyavahara ke khatare mem eka chhoti si kami bhi labhakari mani jaegi|
korana, baralo aura stivarta – brauna ne matri pitritva karyakramom ki 14 adhyayanom ka vishleshana kiya kishora matapita aura unake bachchom ke lie jisamem lambe satra ke griha sahayaka karyakrama shamila nahim the|
yadyapi ye vidhi tantra evam parimita parinamom mem bhinna the, matrika bhavukata tatha shishu pratikriyashilata ke parinama sakaratmaka the|
eka dala adharita karyakrama adhika prabhavakari tha jaba usaki sidhi tulana vyaktigata vyavahara karyakrama se eka eka adhyayana mem ki gai, halanki adhikatara adhyayanom ne aisi sidhi tulana nahim ki|
chhote aura anya utsahi karyakrama utane hi prabhavakari lage jitane lambe karyakrama aura 10-12 saptaha valom ko sabase vyaya prabhavi suvicharita kiya gaय़a
adhika chhantani aura sannigharshana anupata ko abhilikhita kiya gaya jabaki adhikatara namune svayansevaka rogiyom ke the atah adhika prerita hone ki sambhavana thi| 
pitao ko shamila karane vale karyakrama kama the, para inhe sakaratmaka mana gaya hai aura kishora pita jo upekshita ya tiraskrita haim ko adhika sammilita karane ki avashyakata hai| 
eka kaukarena kolaiboreshana vishleshana manasika evam shariraka svasthya svasthya ki unnti ke lie malisha ke prabhavita ko janchane ke marga para haim parantu aba taka koi parinama prakashita nahim hua hai|
philda evam anya ne malisha chikitsa ki prabhavita eka se tina maha ke shishu para jo udasa kishori mataom se hue haim, para dikhai|
ye shishu adhika sote the, kama rote the, adhika samaya sakriya chaukanni avastha mem bitate the aura unake ladamaya kortijola kama the jo yaha sujata hai ki unamem dusare dala jinhe bhulaya jata tha se kama tanava tha| 
onojava evam anya ne dikhaya ki eka shishu malisha kaksha mam aura shishu ke parasparikata ko sudharata hai udasa prasutipurna mataom ke lie|
alpa vajana vale shishuom ka kangaru dhyana rakhane se mataom ki apane shishu ke prati dharana badati dikhai pada़i tatha mam-shishu ki pratikriyashilata 41 saptaha ki garbhavastha pe badha़i| 
layansa evam anya ne darshaya ki kisibhi prashikshita karchari ya vyavasayika ke saptahika griha milana se shishu vikasa, shishu asakti, mam-shishu ke parasparikata, matrika udasi evam matrika samajika samparka mem sudhara aya| 
henika evam anya ne dikhaya ki rishtom para adharita gariba mataom ke satha hastakshepa jinhe parivara ya samudayika sahare ka abhava tha, jisamem niyantrana dala ki bajaya griha misana ka eka dala karyakrama ka prayoga karane se surakshita rupa se asakta bachchom ki sankhya evam mata pita se pratikriyashilata badha़i| 
shaiphina evam anya ne alpa aya, shishu durvyavahara ke uchcha khatare vale parivarom ke lie prastavita sevaom ka varnana kiya|
karyakrama, sampuraka chhoda़ dene valom se ya eka bara samparka hue valom se bhinna nahim the| 
karyakrama ki tivrata ya avadhika aura parinama ke bicha koi sambandha nahim tha|
ve karyakrama jo parivarom ko apani mula avaashayakataom ke pura karane tatha paramarsha prastuta kara rahe the adhika prabhavashali the matrika-pitrika aura bala vikasa karyakramom se evam chikitsalaya adharita karyakrana adhika pabhavakari the griha milana sevaom se|
bekaramensa-krenanabarga evam anya ne eka mataom ka dala chuna jo ki vayaska asakti ki shaili para adharita tha|
asurakshita rupa se asakta mataom ko chara griha milana prastuta kiye gae| 
hastakshepa vali mataem teraha mahinom para niyantrana dala vali mataom se adhika samvedanashila pai gaim|
jina mataom ko unaki asakti ke tarike para asurakshita vargikrita kiya gaya hai unhe vidiyo parasparikata margadarshana se labha hua hai|
jo mataem asurakshita thim – lavalina hokara vidiyo punarniveshana se labha pane lagim aura isake anupuraka apane bachapana ke asakti ke anubhavom para bhi vichara kiya|
yaha eka ruchikara adyayana hai, parantu edalta etaichamenta intaravyu ke parishrama gahana svabhava ko dekhate hue naidanika vyavahara ka tatkalika prayoga sambhava nahim hai jaba taka vayaska vayaska asakti ko pahachanane ke lie eka adhika mitavyayi tarika upalabdha nahim hota|
oronena evam anya ne darshaya ki 20 varsha ki anra ke kishora avastha ke loga joki uchcha khatare vale parivarom se ae haim ko 3-6 paramarsha ke adhiveshana mila jate haim|
halanki yaha prabhava sankhyikiya rupa se sarthaka haim, manasika upacharakom dvara kie gaye nivesha ke pancha varsha ka vistara ise alpa vyaya prabhavi bana dega| 
mokaloda evam nelasana ne eka bala durvyavahara nivarana karyakrama ke meta vishleshana ka vivarana diya jinamem se 32 prasava purva ya prarambhika vidyalaya ke samaya hue the|
do adhyayana jo ki prasava purva prarambha ho gae the khaasa taura para prabhavi the aura jo janma ya prasavapurva shuru hue the adhika prabhavakari the| 
badha़ti yogyata ke prabhava adhika dikhai die na ki ghatate bala durvyavahara ke, parantu hama yaha asha kara sakate haim ki pahale vale ke sakaratmaka ghatanaom ki shrrinkhala ho sakati hai|
ve karyakrama jo eka prakara se adhika hastakshepom ko sanyojita karate haim aura shakti adharita drishtikona rakhate haim, adhika shaktishali hote haim|
chhah mahine chalane vale karyakramom se adhika prabhava lambe karyakramom ka hota hai|
bahuta sari pareshaniyam ati haim tivra aupacharika hastakshepa ka meta vishleshana karane mem una parivarom ke lie jinaki jaruratem uchcha stara ki hoti hai jinamem kuchha adhyayana aupacharika prakara ke bicha parikshana ke lie hote haim ya prayoga karane ki unra aura lakshita dala ko manavikrita karane ke lie hote haim|
achchhae parinama ki paribhasha bhi durgrahya hai kyonki bahuta se mula parinama sanketaka bachche ke vikasa mem dera se pratyaksha hote haim|
tathapi matrika-paitrika samvedanashilata, jisaka eka hrishta-pushta anumanita karanatmaka sambandha surakshikta asakti se haim, aura asakti svayam mulachinhakari hai, achchhe shishu parinama ka| 
bekaramensa – krainanabarga vaina ijajenadurga aura japhara ne hastakshepom ka eka suvyavasthita punaravalokana kiya prarambhika balakapana mem samvedanashilata aura asakti ko badha़ane ke lie jisamem hastakshepom ki shreni ka avarana hai evam chhote jnanatmaka vyavaharika karyakramom se lekara lambe satra ke tivra manosamajika upacharom ka vivarana hai|
lambe tivra karyakrama jo ki samvedanashilata kalpana aura samarthana para adharita hote haim, jaisa ki lekhakom ne kaha hai nirashajanaka rupa se chhote evam nakaratmaka prabhava asakti para dikhate haim|
eka karyakrama ke satha juda़ne ke sakaratmaka prabhava ho sakate haim anya parinama karyavahi mem aura niyantrana dalom ke sannigharshana se aneka adhyayanom ki upalabdhiyom ka talochchhedana karate haim| 
molo bebija़ eka kendrita karyakrama hai jo ki jnanatmaka vyavaharika hastakshepom ka prayoga mataom ke lie karata hai aura vidiyo punarniveshana mata pita ke ilae karata hai|
vaha saptaha mem eka dina chalata hai chaudaha haphtom ke lie mataom aura eka varsha se chhote shishuom ke lie|
eka yadrichchhika pratiksha suchi niyantrita adhyayana ki, raste mem hai parantu eka aniyantrita adhyayana ne bahutaadhika matra mem upachara sampurana kiya hai, eka prasavottara udasi vale vanchita dala mem| 
mataem kama udasa haigaim aura unaki apane shishuom ke satha sakaratmaka parasparikata badha़ gai aura nakaratmaka parasparikata ghata gai|
eka vidiyom adharita hastakshepa ka prayoga kiya gaya aura asurakshita asakti ki matra uchcha khatare vale dala mem achikitsakiya niyantrana dala ki tulana mem ghata gai|
aisi sthitiyam jo eka achchhe samajika aura bhavanatmaka vikasa ko prerita karati haim, jo ki achchhe jnanatmaka aura bhasha vikasa ko prerita karate haim, ko shishuom ke lie protsahita karana chahie|
achchhi sarvajanika janakari aura svasthya sudhara janakari ko upalabdha karaya ja sakata hai bahuta asani se, ilaiktraॉnika, postara evam telivijana ke vijnapanom dvara| 
anivarya rupa se janakari aura yaha sankalpa aura sadhana anushansaom ko lagu karane ke lie, asani se una parivarom taka pahuncha sakate haim jo uttama rupa se anya tarikom se sadhana sampanna haim|
kuchha parivarom ko atirikta sahayata evam anya adhika tuvra chikitsakiya hastakshepom ki avashkata hai|
bachchom ke lie sancharana ka mahatva jivana ke prarambhika ghantom se hi sunishchita hai|
pramana ka jnana, janakari ko pratyeka parivara ko upalabdha karana aura unake prayoga ke lie sadhana upalabdha karavana eka antara shakti ki jimmedari hai|
shiksha, svasthya, samajika sevaem evam karyakarta aura sansthaem svayansevaka aura samvidhika kshetra sabhi sandesha aura sahayata shishu aura unake parivaro ko dene ki sthiti mem hai| 
hama yaha sipharisha karate haim ki prasutipurva kakshaem, khasataura para chhedya parivarom mem, di jani chahie, udaharana ke lie, phaimili larninga saintarsa ya shyaura starta kendra 20 saptaha ke sagarbhata para jo ki shishu ki kshamataom aura mata pita ke pratikriyashilata pe dhyana dem| 
rasayana janavarom ki vasa mem ekatrita ho jate haim, udaharanasvarupa vaha jo janavarom ke chare mem pae jate haim (jivanashi, kavakanashi ityadi)|
mansa upabhoga ko simita ko aura saba pratyaksha vasa ko hatao, samagra vasa deyari utpada (dudha, chija़) mem bhi uchcha avashesha antarvishta hona sambhavita hai|
marcha 1999 se 45 mem uchcha avashesha vali phasalem thim: ada़u, seba, angura, hari phaliyam, nashapati, palaka va vintara skvaisha|
britisha kolambiya ke nivasi dhyana dem : bisi mem bada़e kiye gaye utpada bahuta kama rasayana ka prayoga vastu hote haim anya padeshom aura deshom se|
bisi ko kharidom jaba bhi jaiva kharidana sambhava na ho|
uchita ayojana aura thoda़i si svashiksha ke satha abhibhavakom ko apane bachchom ko janavara utpada rahita ahara dene mem ashvasta anubhava karana chahie|
kailshiyama ke srota haim, phaliyam, hari pattedara, anjira, blaikastrepa, molasisa, pushta peya ityadi| 
lauha tatva ke srota haim, anna, sukhe meve, nariyala pani, blaikastrepa, molasisa, soyabinsa, binsa ityadi|
eka varsha ki ayu purna hone taka gehum va mevom ka parihara karana chahie, sambhava khadya samvedanashilata ke parihara ke lie|
una bachchom ke lie jo chuninda khane vale haim ke lie sampurana ki avashyakata ho sakita hai parantu adhikansha poshana avashyakataem ahara se mila sakati hai|
vitamina di aura bi12 apavada ho sakate haim evam abhibhavakom ko paushtika khadya ya saplimenta ke dvara vaikalpika apurti ko nishchita karana chahie|
apake bachche ki svastha bhojana adatom ko kayama rakhane mem sahayata ke lie samudayika sahayoga talashana bahuta mahatvapurna hai, kyonki kai bara shakahari ahara ka galata artha lagaya jata hai va kalyanakari parivarajana pada़ausi va shikshakom dvara sashankita bhi hota hai|
kai ati uttama sansadhana vidyamana hote haim jo apake bachche ke jivana ne una logom ko shikshita kara sakate haim jinako dilachaspi hoti hai va koi bhi bhrama jo unhe ho sakata hai ko sambhavatah kama kara sakata hai|
eka balaka ki shaakahari ahara para palana poshana karana aja ki haॉta daॉga aura aisakrima ki duniya mem chunautipurna ho sakata hai parantu abhibhavakom ko ashvasta hona chahie ki anusandhana unake paksha mem hai|
samanya rupa se adhyayana balakom ke lie shakahari ahara ka anumodana karate haim agara uchita bhojana ka chayana kiya jae|
vaha adhyayana jo ise sankatapurna vikalpa ingita karate haim ne samanyataya eka upajana samudaya para sankendrita hote haim jisane apane bachchom ko eka sankatapurna kama kailori ahara diya hota hai (arthata dirghajivi)|
pasta aura breda jaise karbohaidreta padartha ka bachchom ke ahara ka adhikansha bhada banane para bharosa karana|
mam ka dudha eka shakahari shishu ahara ka sabase mahatvapurna tatva hai|
mam ka dudha pahale do varshom taka dena chahie, mukhya rupa se shakahari balakom ko|
keevala nuskhom ko vikalpa mem prayukta karana chahie, soe deri va anya dugdha utpada vaha paushtika hom to bhi navajatom ke lie upayukta nahim hote| 
shakahari mataom mem paya gaya hai ki unake dudha mem kama jivanashi avashesha hote haim|
pahale chhah mahinom taka ke lie mam ka dudha ya nuskhe poshtika rupa se vaha saba prastuta karate haim jo eka balaka ke lie avashyaka hai| 
yadi parivara mem pratyurjata ka itihasa ho (hephivara, pama, dama, khadya pratyurjata) gehum aura deri ko jitana lamba ho sake varjita karana chahie aura soe makka, nimbu vansha anda jaise pratyurjata ka parihara karana chahie|
jnata rahe ki shakahari bachche apane sarvahari pratirupom se halke hote haim|
uchcha resha khadyom ko simita rakhane ka prayasa karem evam uch kailori valo ko avashya sammilita karem(branamaphina viruddha evokado)
badha़te hue bachchom ko sharira banane ke lie samagri ki avashkata hoti hai|
agara deri apake bachche ke ahara mem se varjita haim, to dusare kailshiyama prachura khadyom ko avashya sammilita karem| 
lauha bhandara 6 maha ke asa pasa kama ho jate haim aura mam ke dudha mem kama matra mem parantu atyadhika avashoshya lauha hota hai|
samanya rupa se yaha anushansa ki jati hai ki smapuraka diya jae kyonki lauha navajata ke vikasa mem bahuta nirnayaka hai| 
vitamina si khadya lauha avashoshana ka atyadhika vriddhi karate haim, isalie hara bhojana ke satha eka ko sammilita karane ka prayatna karem| 
vividhata mem prachura ahara kai bara paryapta protina pradana karata hai, para parantu vaha bachchom ke lie bahuta anivarya hai, isalie sandra protina srota ka sambharana karana eka uttama vichara hai|
padapa protinom ko sanyukta karana avashyaka hai hamare sharira ke lie avashyaka protina prophaila ko sanya rakhane ke lie| 
eka vividha ahara yaha prakritika rupa se karata hai, para yada rakhie ki shakahari protina kombabineshana kya banata hai evam yaha sunishchita karem ki apake bachche ko yaha 24 ghante ke samaya mem milata hai|
agara mam ke dudha deri ya ande ko ahara mem sammilita kiya jata hai to protina sanyojana samanyatah mahatvapurna nahim hai|
bi12 kevala jiva bhojana mem paya ja sakata hai evam agara yaha apake ahara mem se purna rupa se varjita ho to smapuraka avashyaka haim|
poshaka khadya jaise mansa tulya rupa va poshaka duggha vikalpa bi12 pradana karate haim|
agara apaka bachcha dhupa ke prabhava mem ata hai to vitamina di eka chinta nahim haim|
vaikalpika rupa se, apake bachche ki avashyakataom ko purna karane ke lie ekantaratah poshaka khadya ya smapurana chahie hote haim| 
navajata va chalane layaka shishumom ko uchita vikasa ke lie vasa ki avashyakata hoti hai|
mam ka dudha avashyaka vasiya anlom mem prachura hota hai, nuskhe nahim|
mamom ko avashyaka vasiya anla khane chahie jisase unake dudha mem prachura matra ho|
apane bachche ke ahara mem eka tela jaise phleksa ko suraksha ke lie sammilita kijie agara koi bhi dusare srota niyamita rupa se upabhoga nahim hom|
mera pahali bachchi emmo jaba 14 maha bada़i thi aura mainne eka pathyakrama mem namankana kiya tha isa manobhava kesatha ki merajivana phira se arambha ho raha hai, dudha chhuda़ana arambha kara diya tha|
yaha vaha samaya tha jaba mainne eka kitaba jo madaringa yora narsinga toddalara kahalati thi padha़i|
isa manyata ke satha, prarambha karane se ki vistarita stanapana mam aura bachche donom ke lie santoshajanaka hai, norama jena bumagaranara suraksha, atmavishvasa va svabhimana jo hama apane bachcho ko dete haim jaba hama unhe apane samayanusara dudha pine aura chhoda़ne ki anumati dete haim, ke bare mem likhati haim|
norama jena bumagaranara ne muje avashya pratyatiya kiya aura ema aura mem tina aura varshom taka stanapana karate rahe|
chhah sala bada punah padhane para unaka sandesha kama mulankura laga|
mere tina utsahi ladalom nem muje eka eka vistarita stanapana sambandha ke labha supramanita kie haim, sharirika samutthana shakti va bhavanatmaka svatantrata jo ati hai ke satha|
varshom se mainem vistarita stanapana ke lie adhika vastavika labha ke bare mem jnana prapta kiya hai|
yadyapi stanapana hamari sanskriti mem samanya hai, lagabhaga, 80% shishu stanapana se prarambha karate haim, darem shigrata se girati jati haim, 50% tadapi 1 maha uparanta stanapana kara rahe hote haim evam 24% 6 maha taka (ऑstreliyana byuro ऑpha stetistiksa) 
eka chalane yogya balaka ko stanapana karana isa prakara ki ninda ko akarshita karata hai jo kisi bhi stanapana mata ko eka pidha़i pahale jelani pada़ti thi, parinamasvarupa kuchha mataom ko kloseta narsarasa banana pada़ta hai|
isake viparita, atiparamparagata sanskritiyom mem mataem apane shishuom ko kama se kama do varsha taka stanapana karavati hai, jisa taraha se pashchima yuropa mem adhikatara mataem isa sadi se pahale karati thim (dettayalara 1995)|
madhyakala mem bhi shighra dudha chhuda़ane ke khatare samaje jate the aura asvastha navajatom, juda़vam bachchom va lada़kom ko samanyataya eka ya do varsha se adhika stanapana karavaya jata tha|
mataom ki ghara se bahara kama karane ki pravritti, bala- palana visheshajna ke pantha aura yaha salaha ki svayam mam ka dudha usake bachchom ke lie vaijnanika nuskhom se kama upayukta hai ina saba ne vistarita stanapana avanati mem yogadana diya| 
vistarita stanapana ko sahayoga karata hua vartamana rupa se majabuta sakshya hai|
dettavilara, esosieta prophesara ऑpha enthropoloji evam nyutrishansa teksasa e enda ema yunivarsiti mem, ne hala hi mem shighra hi prakashita hone vale vivarana ke chhah maha ke uparanta stanapana para anusandhana ka sankshiptikarana kiya hai| 
dettavilara likhate haim hara prakarana mem svasthya nishkarsha paravarti ke anpeksha mem jitana lamba balaka ko stanapana karavaya jata hai, utana achchha svasthaya parinama ata hai, 18 se 24 maha taka stanapana karavaye gaye bachchom ke satha sabase kama khatara hota hai| 
pro. dettvilara una kshetrom ko suchibaddha karate hainjo abhi taka adhyayana kiye gaye haim jisamem sammilita hai, dentana melaokkalasana, gestrainatestainala, dijija़, sadana impekta detha sindroma karna sankramana chailda kainsara evam chaildahuda dayabitija|
isa varsha prakashita hue nae adhyayana unake nishkarshom ka anumodana karate haim ki stanapana uchcha aikyu evam paravarti batyavastha mem vidyalaya saphalataom se sambandhita hai satha hi labha stanapana ke samaya ke satha sambhavatah badha़ jate haim| 
pro.dettavilara manavashastriya ankada़om se anumana lagate haim ki dudha chhuda़ane ki prakritika ayu 4 se 6 varsha ke madhya hai|
vistarita stanapana se mam ke lie bhi sharirika labha haim, anshika rupa se una harmona ke usake sharira mem srava se jaba vaha stanapana karati hai|
prolektina jo kai bara madaringa harmona ke rupa mem bhi jana jata hai ka mam para shithilakari prabhava va apane shishu ke satha samparka ki adhika chaha karane ka prabhava darshaya gaya hai|
ऑksitosina jo stanapana ke samaya pratikriya kama karane ke lie uttaradayi hai, prema harmona kahalata hai unmada, janma, stanapana va mitrata se apane sambandha ke karana| 
ina donom harmonom ke bare mem socha jata hai ki yaha stanapana mata mem janana kshamata ki manda vapasi mem yogadana dete haim|
stanapana endoraphinasa mam aura bachche donom ke lie sharira ke praka-tika svapaka bhi chhoda़ta hai (te, 1991), isa adbhuta va labhaprada sambandha ne donom sahabhagiyom ke sukhada paksha ko rekhankita karate hue| 
kya apa apane stanapana ko vistrita karane ke lie eka sachetana nirnaya le rahe hom ya apane sarala rupa se dudha chhuda़ane ki avashyakata nahim anubhava ki hai apako apake sthaniya narsinga madarsa grupa se sahayoga va suchana prapta hogi|
yadyapi ऑtijma va alajaimarsa roga vyapaka rupa se vibhinna ayu varga ko pida़ita karate haim vaha eka hi karana se hote haim – para|
daimithaila markari – pare ke eka jaiva rupa ki rasayanika visheshataom ka adhyayana karate hue eka dartamautha prophesara ne do bundem apane dastane pahane hue hatha para gira dim|
markari poija़ninga ke pahale lakshana chara maha bada prakata hue jaba unaki vani ka uchcharana aspashta hone laga|
chalane mem samasya va drishti ke kshaya ne isaka anusarana kiya usake bada vaha sammurchha mem chali gai aura mara gai|
prapya danta vilayana (vaha hai 35% chandi, 50% para 15% tina, jasta va anya dhatuem) mem se chandi ko galane mem prayasarata dusare vyakti ne unhe talane vale patile mem garma kiya| 
atah utpanna hue pare ke vashpa ne unhe shighrata se mara diya|
usa samaya ghara mem do aura parivarake sadasyom ki nrityu ho gai|
para sono se eka protona (nyutraॉna evam ilektraॉna) bhari hai – sone ka etaॉmika nambara 79 hai pare ka 80|
anya dhatuom se alaga para kamare ketapamana para apane tatva rupa mem drava (pighala hua) hota hai|
vaha gaisiya para paramanu ki anavarata taranga mukta karata hai jo vatavarana mem kai maha taka thahari rahati hai(antatah ajaiva rupa mem varsha ki bundom dvara prithvi va usake mahasagarom mem girati hai|
taba jaba vaha thosa avastha mem dusari dhatuom ke satha mila kara sammishrana rupa mem hota hai pare ke paramanu vayumandala mem uda़ jate haim|
tatvika para jaba danta vilayana se yukta hota hai,shvasa dvara andara liya jata hai, va phephada़om dvara avashoshita hota hai (usaka 80%) va sharira mem roka liya jata hai| 
vaiksina utpada vaiksina mem jivanu sandushana ko rokane ke lie usamem – thaimirosala dalate haim (jo adha ithaila markari hai)
yaha ajaiviya pare ka injekteda rupa mastishka va hridaya mansa peshi koshaikaom dvara sahaja hi le liya jata hai| 
mithaila markari (jo plavaka se praptahota hai vaha ise samudra se nikalane vale ajaivika pare se sanshleshita karata hai) machhali mem paya jata hai|
vartamana mem amarikiyom ke lie pare prabhavana ke do pramukha srota haim dentala amalagansa va vaiksineshana|
sanghiya sarakara ke sentara phaॉra dijija़ kantrola enda privenshana (sidisi) va phuda enda draga edaministreshana ne na samaja ane vale karanom se isa tathya ko najaraandaja karane ko chuna hai| 
yaha shakhaem va neshanala insatityuta ऑpha hailtha kevala samudri bhojana mem pare para kendribhuta karata hai, isa hada taka ki ena ai echa una adhyayanom para jora nahim dega jo amalagama va vaiksina mem pare ko sambodhita karengem|
sarakarake viruddha (edie) amerikana dentala esosieshana ka dava hai ki amalagama surakshita hote haim aura unamem para koi samasya prastuta nahim karata hai|
yadyapi isaka badha़ta hua sakshya hai ki vaiksina va amalagama mem para ऑtijma va alajaimarsa utpanna karata hai|
sidisi, ephadie evam vishesha vishaya samaja dvara prerita chikitsa sanstha, isa sakshya ko chhoda़ dete haim ya naganya samajate haim – sakshya jisamem niji rupa se nirdhibaddha maraka vijnana ke adhyayana ate haim|
ऑtijma ko ameriki bachchom mem 1943 mem khola gaya tha, kukarakhansi ke vaiksina mem ithaila markari (thai mirosala) milane ke 12 sala bada|
yuropa mem roga 1950 ta nahim dekha gaya tha jisake bada thaimirosala vaham para prastuta hone vali vaiksina mem dala gaya tha|
eka vishishta prakarana mem, apane dusare janmadina se thoda़a pahale eka samanya rupa se vikasita hota hua svastha lada़ka dusarom ke satha samparka karana chhoda़ deta hai aura anatarmukhi ho jata hai|
vaha dusarom ke satha ankhem milane se bachata hai va vichitra evam akela ho jata hai usaki drishti aspashta ho jati hai evam vaha vibhinna motara disatarbanesajaise anajane se hatha va pairom ko jatakana va panjo para chalana vikasita karata hai| 
para anya vikasatmaka vikara mem bhi bahuta sambhava eka preranarthaka karaka hai jaise vilamba se bolana evam ekagrata ki kami atisakriya vikara| 
anveshakom ne darshaya hai ki ऑtijma va vaiksinom mem badha़ti hui pare ki khuraka se sidha sambandha hai| 
1950 ke dashaka mem jaham pratirakshana anusuchi para vaiksina (dipthiriya, tetanasa, parattusisa va chechaka ke viruddha) taka simita thim, 1000 mem se balaka ko yaha roga hota tha| 
jaise anya rogom ke lie vaiksina joda़e gaye svasthya anuraksha karta bachchom mem badha़ti hui adhika matra mem pare ka tika lagane lage| 
1981 mem janma lene valom ko 135 maikrograma para (ausatana) diya gaya thaa aura usa sala paida hone vale hara 2600 balakom mem se eka mem ऑtijma ka prakarana paya gaya| 
hepitaitisa bi vaiksina (janma k samaya tike ke dvara di jane vali) va eka aura timophaila inphyuenja taipa bi, ke sankalana se prabandhaka 1996 mem janma lene vale balakom ko 246 maikrograma pare ka tika lagate the| 
ina 350 mem se eka bachche ko ऑtijma paya gaya|
aja prabandhaka sidie dvara nirmita va eepi evam eeephapi dvara anumodita pratirakshana anusuchi ka palana karate haim, jisamem di jane vali 13 vaiksina sammilita haim evam eka balaka ke do varsha ki ayu hone taka (jaba vaha mastishka ka vikasa purna kara leta hai) 33 bara parivarti sankhya ke vardhaka tike ho jate haim|
aba ऑtijma 100 mem se eka lada़ke aura 400 mem se 1 lada़ki para asara karata hai aura chikitsaka 45 mem isa roga ke 100000 nae prakarana hara sala nidana karate haim (diesaema – chara mem nidanasuchaka mapadanda ko prayukta karake jo purva diesaema – tritiya ara se adhika simita haim|)
pichhale tisa varshom mem eka lakha se adhika bachche atmavimoha se grasta hue haim, aura ajakala amerika mem pratyeka 68 parivarom mem se eka parivara mem atmavimohita bachcha hai|
mukhyadhara ke daॉktari dainiki, jaise pediyatriksa aura da nyu ingalainda jarnala ऑpha medisina, vahi adhyayana prakashita karate haim jo yaha dva dete haim ki thimorosola surakshita hai| 
aisa pratita hota hai ki ye lekha eka bada़e paimane para shodha kartaom ne tika dravya banane valom se paise lekara likhe haim; koleshana phaऑra sepha maindsa batalate hue (sensibala ekshana phaॉra endingamarakyuri indyusda nyuroloji disaऑrdara) aise eka niji sanstha ne darshaya hai| 
ina dainikom ke sampadaka aise adhyayanom ko prakashita nahim karate haim jau thimerosola aura atmavimoha mem sambandha dikhate haim jaise thaimerausela ina chaildahuda vaiksina nyurodavalapamenta disaऑrdarsa evam harta dijija़ yunaiteda stetsa mem marka aura devida geyara dvara, jo ki eka ghanishta sambandha dikhate haim tika – dravya mem antarniveshita para ki matra aura atmavimoha hai|
aise lekha vaikalpika dainiki mem svikriti prapta karate haim (joki rajanitika rupa se anuchita haim)yada aise dainiki jaise jarnala ऑpha ameriki phijishiyana enda sarjansa mem prakashita hua tha|
para ke diye gae khuraka se dimaga (aura dila bhi) ko ko pahunche nukasana ki matra unra, linga evam anuvanshika rupa se nirdharita para ko utsarjana karane ki yogyata para nirbhara karata hai|
chhote bachche jinako dimaga abhi vikasita ho raha hai, adhika bhavaprana hote haim aura purusha adhika bhavapravana hote haim diye gaye para ke khuraka ki ora kyonki testostirona tantrika vishaktata ko badha़a deta hai|
sabase avashyaka halanki hamari anuvanshika rupa se kramadesha ki gai sharira ko para se mukta karane ki yogyata hai|
dimaga mem eka ghara sapha karane vala protina hota hai joki khataranaka apashishta utpada joki tina prakara ke hote haim: APO-E2, APO-E3, evam APO-E4 ko nikala deta hai| 
APO-E2 protina para ke do paramanu ko dimaga se bahara nikala sakata hai; APO-E3 eka; aura APO-E4 eka bhi nahim|
jo jina hama prapta karate haim pratyeka abhibhavaka se ye nirdharita karata hai ki kauna se do hamare pasa haim|
jina logom mem do APO-E4 protina hote haim (atah koi APO-E2 ya E3 nahim) unamem 80% sambhavana aljaimarsa roga prapta karane ki hoti hai| 
eka adhyayana ke anusara, atmavimohita bachchom ke dimaga mem APO-E4 protina ki matra bahuta adhika hoti hai|
aljaimarsa roga 1960 mem khoja gaya tha, phira se amerika mem, jaham danta-chikitsaka para se bhare parada mishrana ka prayoga guttika ko bharate samaya ke lie karate the| (yuropa ke danta chikitsaka isaka parihara karate the|)
aja chara lakha se adhika amerikiyom ko aljaimarsa roga hai|
aljaimarsa 85 sala se upara ke adhe logom ko kashta pahunchata hai aura 20 pratishata 75 se 84 varsha ki unra valom ko| 
isa bimari ke pahale rogalakshana haim ekagrata mem kathinai mahasusa hona aura parivartaniya parivartaniya matra mem smarana shakti ka abhava, jo ki anta mem vinashakari manasika adhogati ki ora le jata hai|
alajaimarsa roga vale logom ka dimaga 25 pratishata sikuda़ jata hai aura unake suspashta rogatmaka pramana chinha hote haim; tantrika resha gutthi, amailaueda plaka, evam chau protina ka posaphaaurineshana)
mastishka koshika joki prayogashala mem vikasita ki i hai inhi tina rogatmaka khojom ko darshati hai jaba unhe pare ke naino molara khuraka ki ora anavritta kiya jata hai (3.6 × 10-10 molara), yaha matra utani hi hoti hai jitani ki una logom ke mastishka mem pai jati hai|
dentala amalagama eka vayaska ke dimaga mem pare ka mukhya srota hota hai|
eka ausata akara ke paradamishrana ki bharai mem 750,000 maikrograma para hota hai aura kariba 10 maikrograma bahara nikalata hai|
shodhakarta vikirana ankitaka para ke paradamishrana ko bheda़ ke dantom mem dalate haim aura phira skainara se yaha nirdharita karate haim ki nikala hua para kaham gaya| 
unhone dikhaya ki pare ke paramanu jo nihshvasa ke samaya naka se nikalate haim, ghranendriya snayu ke tantu dvara upara ashvamina taka chale jate haim, jo ki smarana shakti ko niyantrita karata hai, evam mastishka ke anya alochanatmaka kshetrom taka pahuncha jate haim|
eka anya adhyayana mem chuhom ko usi sandrata mem die gae para jitana ki loga apane parada mishrana se shvasa lete vakta lete haim, se dekha gaya ki ve aljaimarsa roga ke rogatmaka sanketa vikasita kara lete haim| 
aljaimarsa roga vale logom ke mastishka mem para ki matra svastha logom se 2 se 3 guna adhika hoti hai|
phlu ke tika drava mem maujuda para bhi isa bimari mem mukhya bhumika nibhata hai|
eka shodhakarta ne khoja ki ki jina logom ne phlu ke tika dravya ko pratyeka varsha 3 se 5 varshom taka lagaya, unamem alajaimarsa roga ko vikasita karane ka sambhavana una logom se dasa guna badha़ gai jinhe 0 ya 1 tika laga tha|
anya avashyaka para ke bare mem tathya jo dimaga mem ate haim, jise sidisi, ephadie ke adhikari evam aioema ke acharya nahim manate haim vaha hai, vaha hai sahakriyashilata vishaktata para ka badha़a hua prabhava jaba anya visha upasthita hom| 
eka chhoti khuraka para ki jo 100 mem se eka chuha marati hai aura eka khuraka ailyuminiyama ki jo 100 mem se 1 chuha marata hai, jaba milaye jate haim usaka eka asadharana prabhava hota hai, sare chuhe mara jate haim|
para ki khuraka jisamem 1 pratishata martyata hoti hai ki martyata dara 100 pratishata ailyuminiyama ke hone se ho jati hai|
chikitsakiya sanstha ke kuchha adhikari evam netaom ka tika dravya banane valom ke adhina karyarata hona eka karana hai|
thaimerausela aura paradamishrana ko manjuri dene ki vajaha se puchha tachha ki kali chhaya, aura ephadie dvara upayukta suraksha adhyayana una para nahim karana anya karana ho sakata hai| 
yaha parikalpana ki para atmavimoha aura aljaimarsa roga utpanna karate haim eka naya sacha hai|
1970 mem edavarda jainara ne avalokana kiya ki dudhamaliyom ke chehare para chechaka nahim the una logo ki taraha jo bada़i mata se grasta hue aura bacha gae|
chechaka grasta gayom ke dudha dohane ki vajaha se ye bada़i mata ki tarapha pratirakshita ho gaim|
edavarda jainara ne sankramita gayom ke yuyasphaotika se tarala liya, bachchom mem antarveshita kiya, aura paya ki usane unaki raksha ki, jaba unhe bada़i mata ke age anashrita chhoda़ diya gaya|
jaba anya chikitsaka evam suchita vyakti jaise thaॉmasa jaipharasana ne mana ki tika lagane se phayada pahuchata hai usaka mulya svayam suspashta ho gaya hai|
jainara ke tike ne kai lakha logom ko bachaya aura eka virupita bimari ko unmulita kiya jisaki martya dara tisa pratishata thi| 
yuesa aura yuesaesaara ki prayogashala mem bada़i mata vale vishanu ko parirakshita kiya gaya tha aura hamem aba pata hai ki soviyata ke sukshmajiva vaijnanikom ne murgi ke ando mem bada़i matra mem unhe ugaya tha taki unaka prayoga samuhika vinasha ke lie jivavaijnanika shastrom mem ho sake|
aja chikitsakiya sanstha jise AAP, AAFP, AMA, CDC aura IOM chalate haim, dusare anta taka chale gae haim|
svikrita samajadari yaha haiki tika dravya sarvarogahara hai|
svasthya dhyana prabandhaka janma ke dina se shishuom mem ise antarveshita karana shuru kara dete haim aura sarakari adhikari ise saba saba amarikiyom ke lie anivarya bana dete haim|
rohini ki vajaha se nrityu mem 1900 se 1930 taka 90 pratishata ki kami ai, behatara svachchhata aura ahara ki vajaha se, isa bimari ke lie tika banane se pahale|
isi taraha khasara dvara nrityu dara mem 1915 se 1958 taka 95 pratishata (13.3 se 0.03 nrityu pratyeka 100,000 janasankhya) giravata ai, isase pahale ki khasare ke tike ko 1963 mem prastuta kiya gaya| 
prasuti pashchata ka samaya bahuta chunautipurna hota hai kai nai mataom ke lie|
janma ke bada svastha hona, mam banana aura apana dhyana rakhana, ina saba mem bahuta shakti ki avashyakata hai|
prasuti pashchata khuna ki kami isa prakriya ko bahuta kathina bana sakati hai|
enimiya khuna mem himoglobina ki matra kama hone ki vajaha se hota hai aura eka aurata ke lie khataranaka ho sakata hai|
himoglobina eka ऑksijana yukta sanghataka hai jo ki lala rakta koshika mem paya jata hai|
chanki lala rakta koshikaem sharira ke anya koshikaom ko ऑksijana pahunchati haim, isa ऑksijana pradana karane vale tantra mem ai kisi bhi samasya ka parinama yaha hoga ki sharira ko koi bhi karya karane ke lie sangharsha karana pada़ega| 
himoglobina ka mukhya anga ayasa hai, atah yadi sharira mem ayasa ki kami hogi, vaha sharira ke ऑksijana pradana karane vale tantra ko prabhavita karega jisase sansa ki kami aura thakana jaise rogalakshana utpanna honge – enimiya ke do utkrishta sanketaka| 
bahuta se adhyayanom ne yaha darshaya hai ki prasuti purva lauha sampuraka ka parinama uchcha matrika himoglobina sandrata hota hai prasava ke do maha bada taka ke aura prasuti ke chhah mahine bada taka uchcha serama kairetina sandrata hota hai|
kairetana ki matra yaha pratibimbita karati hai ki sharira mem kitana lauha sanchita hai apitu himoglobina ki matra yaha sanketa karata hai ki lala rakta koshikaom ne kitana prayukta kara liya hai| 
mere anusara garbhavastha ke daurana ishtatama sanchaya ke labha sampurana ke alpatama nukasana se adhika prabhavashali hote haim|
ishtatama sanchaya ke bare mem kaphi anishchitata hai|
garbhavati mahilaom ke nityakrama mem lauha sampuraka lene ke bare mem kaphi vivada hai|
tathapi chunki plavika ki matra rakta ka pani ka angabhuta koshikaom ki matra se adhika badha़ jata hai himoglobina ka tanukarana ho jata hai|
atah jaba rakta ka namuna liya jata hai to himoglobina aura himetokrita ki matra mem eka prakritika giravata garbhavastha ke daurana dekhi gai hai|
kuchha sevaka yaha nishkarsha nikala sakate haim ki adhika himoglobina ke lie sharira ro raha hai aura dusare kahate haim ki hamem ise aise hi chhoda़ dena chahie||
samanya himoglobina mana jo jo garbhaavastha mem prayoga liya gaya use lauha sampurana na ki ahara dvara uchchatama sambhavita mulya sthapita karane para nirdharita kiya gaya|
aisa paya gaya ki sampurita lauha himoglobina ki matra ko ऑksijana vitarana ke lie avashyaka ishtatama matra se adhita nahim vadha़a sakate, aura tathakathita vaha adarsha matra bhai sabhi auratom ke lie|
durbhagyavasha, lauha matra jo usa nirdharita samanya prasavapurva, himoglobina ki matra taka pahunchane ke lie avashyaka hai, usa matra se bahuta adhika hai jo eka aurata apane ahara mem le sakati haim| 
atah isase hama upayukta lauha ki matra ki vastavika avashyakata ki chinta taka pahunchate haim|
garbhavastha ke daurana lauha ka avashoshana khasa taura se kushala hota hai| 
yaha saubhagya ki bata hai kyonki eka garbhavati mahila ki lauha ki avashyakata badha़ti jati hai aura shayada duguni bhai ho jati hai, jabaki usako ushma ki avashyakata utani nahim badha़ti hai (ausatana 300 kailorija़)
yaha sujava akasara diya jata hai ki eka aurata ko 3000 se adhika kailori pratidina khane ke avashyakata hai lauha ki avashyaka matra prapta karane ke lie|
lauha sampuranom ki jaiva upalabdhata eka mahatvapurna vichara kshetra hai|
sadharana sampurana jo ki phairasa salpheta se prapta kiye gaye haim, adhika mahilaem sahana nahim kara patim aura isase aisa, peta phulana aura kabja ho jati hai|
kyonki isaki chhoti matra avashoshita hoti hai (kabhi kabhi 5% jitani) atah bada़e khuraka ki avashyakata hai| 
lauha sampurana jinhe sadharanatah prakritika lauha sampurana kahate haim adhikatara phalom ke rasa aura paudhom ke arka se prapta hota hai|
unamem pradhanata pherasa glukoneta hota hai jise sharira achchhi taraha se upayoga mem leta hai|
atah kama khuraka upayukta aura prabhavi hai aura inamem kai bi vitamina vitamina aura vitamina si pae jate haim|
bara bara mainne ina sampuranom ko himoglobina ka matra akele hi prabhavashali rupa se badha़ate hue aura isake satha satha enimika mam mem jana dalate hue dekha hai|
yahi parinama kama mahange phaairasa salpheta ke prayoga se milate haim parantu lambe samaya bada aura usake anya prabhava hote haim|
lauha akela sahayaka nahim hai himoglobina ki svastha matra ko sthapita karane ke lie|
yaha una mahilaom mem adhika pratyaksha hai jo enimiya ka anubhava karati hai aura jo lauha sampurana ki ora pratikriyashila nahim hai| 
phaaulika esida bi12 aura protina sabhi eka bhumika nibhate haim himoglobina ke sanrachana mem|
vitamina e aura si ahara sambandhi lauha ke avashoshana aura upayoga mem sahayoga dete haim| 
khuna ki kami enimiya ka anya karana hai, isase bachane ke lie eka uttama ahara mahilaom ko garbhavastha mem lena chahie|
janma ke bada mahatvapurna rupa se khuna ki kami joki prasava pashchata ke raktasrava ki vajaha se hota hai, prasava pashchata ke enimiya ke khatare ko badha़a deta hai| 
janma se purva paryapta himoglobina aura lauha ka bhandara enimiya ki prachandata ko kama karata hai aura svasthaya labha jaldi pahunchata hai|
vitamina ke eka jana mana rakta skandana karta hai joki svastha antriyom mem utpanna hota hai aura yaha pattedara hari sabjiyom mem khasataura se paya jata hai|
vitamina ke kaineda ki dukanom mem sampurana ke rupa mem nahim paya jata hai lekina alaphalapha jaise ama sampurana ka prakritika anshabhuta hai| 
kuchha loga apana paryapta vitamina ko utpanna nahim kara pate (jaisa ki pratirodhaka ke prayoga ke bada) ya unaki jaivika avashyakata ke lie adhika hoti hai| 
alaphaalapha ki goliyam joki garbhavastha ke antima maha mem yadi li jaem to ve prasava pashchata ke raktastrava ke khatare ko kama kara sakata hai|
yaha usa mata ke lie khasa taura se mahatvapurna hai jisaka pichhale janma ke daurana bhari matra mem rakta ka abhava hua: tathapi kuchha satarkata ki avashyakata hai yadi vaha raktastrava aura verikoja़ vena ki tarapha samvedanashila hai| 
apane abhyasa mem maim poshana yukta ahara para jora deta hum|
maim garbhavati mahilaom ko lauha sampurana lene ke lie utsahita karata hum kyom ki mainne ina varshom mem usake sakaratmaka prabhava dekhe haim|
pratyeka garbhavati mahila ko bhari matra mem lauha ki khuraka sampurita karani chahie ki nahim, abhibhi charcha ka vishaya hai|
yaha janakari hai ki kuchha sampurana svastha mataem, svastha janma aura svastha shishu utpanna karate haim|
svastha tvacha thoda़i si anliya hoti hai anliya avarana hone ki vajaha se|
anliya avarana sebama aura pasine ka sanyoga hai aura vatavarana se tvacha ki raksha karane ke lie banaya gaya hai| 
pratidina hama 850 mili pani pasine ke dvara kho dete haim atah pine vala pani isa khoye hue tarala ki jagaha le leta hai|
parinamasvarupa, sebama ka utpada badha़ jata hai jo ki tvacha para ekatrita ho jata hai aura romakupa banda kara deta hai| 
grishma ritu mem pani ka abhava jaldi hota hai aura garmi ki vajaha se tvacha dvara pani ka abhava teja ho jata hai jaise jaise sharira apane apako thanda karane ka prayatna karata hai| 
prakritika vanaspatika sabuna ya sabuna adharita parimarjana se shuddha karana chahie| 
apani tvacha ko madyasara mukta, prakritika tonara se jalayojita karem ya eka svachchha, jivanurahita phuharane vali botala ko nisyandita pani se bharem aura apane chehare ko sapha karane ya apashalkana ke bada isase gila karem|
krima aura loshana jinamem paitroliyama adharita samagri hoti hai romakupa ko banda kara deti hai|
malisha se tvacha mem rakta ka sanchara badha़ jata hai jo prakritika navinikarana prakriya mem sote samaya madada karata hai| 
hothom ka raksha aura nami eka hotha bama, hotha chamaka ya lipastika se banae rakhie adhimanata jisamem prakritika suraja badha jaise ki taiteniyama daiऑksaida ho|
koi bhi tvacha anuraksha nibandha jisamem poshana para paricharcha sammilita na ho svastha, tvacha anuraksha ke mula siddhanta mem abhava rakhata hai|
achchha svasthaya aura sundarata paryayavachi haim| udaharanartha, eka argala va dhabbedara rupa – ranga eka santripta vasaom va sharkara se prachura ahara se sambandhita ho sakata hai|
atisamvedi tvacha kharaba pachana ya poshakom ke aparyapta avashoshana se aura bura ho sakata hai|
shushka papada़idara tvacha vasiya anla aura vitamina i mem kami vale eka ahara ko pratibimbita karati hai|
vaha tvacha jo shighra svastha nahim hoti vitamina e, bi6, si ya jinka mem durbala ho sakati hai|
eka svastha vividha ahara tvacha ko sankramana, koshika kshati evam samayapurva budha़ane se raksha karati hai khasakara grishma mahom mem|
apake taji, kachchi sabjiyom evam phalom ke dainika antargrahana ko badha़ane se apake ahara mem vitamina, entiऑksidentsa va jala ka yoga karate haim|
grishma mem taje bhojana ki badha़i hui vibhinnata ka labha uthaiye, kyonki eka achchha ahara, svastha ujvala tvacha ka avashyaka tatva hai| 
lauri strayakara purna rupa se prakritika tvacha anuraksha evam shrringara para anusandhana aura vikasa kara rahe haim, purushom va striyom ko dainika upayoga ke lie svastha angaraga prastuta karane ke prayojana se|
pratidina hamara sharira, hamare bhojana, hava va pani mem padarthom se poshita aura chunauti dono prakata karata hai|
hamare sharirom ko poshakom ko upayoga karane ke lie aura jiva vishom ke nishkasana karane ke lie bahuta nipuna banana pada़ta hai|
durbhagyavasha yaha nishkasana prakriya hamare vatavarana mem jiva visha padarthom ki adhikata se parajita ho jati hai|
yadyapi hamari sahayata se eka nirvishakari ya svastha ahara dvara apane sharira ko eka ati avashyaka avakasha de sakate haim|
nirvishakarana ki sankalpana eka bahuta puratana hai aura upavasa itihasa mem kai sadiyom se prasiddha hai|
eka sachche upavasa mem eka samaya ki avadhi taka, samanyatah kuchha dinom taka kevala jala ka upabhoga kiya jata hai|
kai svasthayakarta isa shabda ko aba angikara karate haim rasa ahara ya rasa upavasa ko bhi sammilita karane ke lie| 
parimarjana samanyatah varsha mem eka ya do bara karate haim, sadharanataya basanta mem va phira se patajada़| 
svastharupa se jite hue, bache hue aja ke adhunika jivana mem (pradushana, ganda khane se rasayana adi) yaha jiva visha andara ki ora badha़te haim bahusankhya mem roga utpanna karate hue| 
parimarjana ke sharirika va bhavanatmaka labha prativyakti vibhinna hote haim, jivavisha ke vartamana stara evam nishkasana madhyama ki kshamata para| 
samanyatah jaba koi parimarjana arambha karega karega use eka arambhika kashta ka anubhava hoga usake bada sharirika va bhavanatmaka sukhabhasa hoga|
samapti ke uparanta kai logom ko pata chalega ki unamem adhika urja hai hai, unaki tvacha va kesha adhika svastha haim, aura unaka pachana sudhara gaya hai|
eka nirikshana rahita parimarjita ahara, apekshika uttama svasthaya vale vyakti evam vaha kaisa anubhava kara rahe haim ko kayama rakhane ya sudharane ke lie shreshtha| 
vyakti jo asvastha haim ya chirakalika svasthya samasyaem anubhava karate haim svasthya kartaom se paramarsha karane evam kisi vishesha tarka para vichara vimarsha karane ke lie uttama hai|
basanta parimarjana sharira ko apane prakritika shita nishkriya avastha se bahara nikalane mem sahayata karane evam grishma ki badha़ti hui gatividhiyam, taji hava va taje phalom ki prachurata ke lie tatpara karata hai|
eka patajada़ parimarjana sharira ko ane vali shita ritu ka samana karane, jise vishishta rupa se bhitara vyatita kiya jata hai kama gatividhi va ninna stara ke bhojana ke satha, ke lie tatpara karata hai|
purusha va stri donom ke lie purvadharana kala parimarjana ke lie shreshtha atirikta samaya hai| 
vaha striyam jo garbhavastha ki yojana bana rahim haim ko svastha jiva visha rahita sharira ke mahatva ka jnana hona chahie|
svastha sharira vale purusha svastha va adhika prachura matra mem shukranu utpanna karate haim|
janana kshamataem samasyaem anubhava karate hue vyaktiyom ko pata laga sakata hai ki parimarjana unake sharira ko sapha karata hai evam garbhadharana ke lie tatpara karata hai|
masika purva samasyaem kai bara ritustrava se eka saptaha pur halke bhojana se dura ki ja sakati hai| 
pratyarjata rogisa parimarjana ka prayoga karake sambhavya bhojana pratyurjata ko nirdhaarita karake aura mausami pratikriyaom ko parabhuta kara deta hai|
tambaku, sharkara kaiphina ya sharaba ko parajita karate hue vyakti nirvishikarana parimarjana se labha prapta kara sakate haim|
sharira ke lie avashyaka puranuddhara karya ko sampadita karane ki matra kama karake pratyahara lakshana kama kara sakate haim|
jaisa ki pahale kaha gaya hai eka svastha vyakti ke lie bina kisi nirikshana ke yaha ahara lena sahi nahim hoga|
purvavarti abhava aura samanya durbalata vale kisi vyakti ko apani avastha vikriti karane ka khatara hai|
ina sthitiyom mem shakti badha़ane va nirmana karane ka drishtikona hona chahie|
vyakti jo nyunabhara haim, atyadhika thakana jelate haim, thandaka anubhava karate haim, ya shalyakriya pratyashita ya usase svasthayalabha kara rahe haim ko jaba taka unake svasthaya prakarana sambodhita na hom jaem eka parimarjana ko sthagita kara dena chahie| 
vaha striyam jo garbhavati haim ya stanapana kara rahi haim ko bhi parimarjana ke prati satarka hona chahie|
yadyapi eka halka ahara kuchha striyom ko kathina masika dharma mem sahayata kara sakata hai isa parimarjana se jo prabandha ho sakata hai usase ahariya avashyakataem adhika mahatvapurna haim|
kuchha vyakti parimarjana ke satha yatheshta anubhavi hote haim jisase jivana unake lie samanyatah anavarata chalata rahata hai, dusare vyakti (vishesha rupa se vaha jo kaiphina chhoda़ rahe haim adi) pate haim ki vaha badatarina ke lie pariyojana banana chahate haim apane lie prachura svatantrata aura aura maya svikara karake| 
apa saptahanta ya kisi aise dina jaba apako karya karane ki ya uttaradayitva ki chinta karane ki avashyakata na ho tisara charana prarambha karana chahenge|
isa charana ka tisara dina kuchha vyaktiyom mem eka prakara ka sukhabhasa lata hai aura yadyapi apako eka lambi dauda़ ki yojana nahim banani hai, apa apane apako bahuta utpadaka anubhava kara sakate haim|
yaha nishchaya kara lijie ki apako charom ora vyakti apako samajem aura apake chunava ko sammana de|
apa arambha ke do dinom dinom taka apavada svarupa chida़chida़e aura thake hue ho sakate haim aura unhe isake lie taiyara hona avashyaka hai|
hama eka aise samaja se ate haim jaham jaiva ugana va bhojana ke lie kevala utpada va pashudhana ka vardhana karana kisi samaya eka samana hi tha|
chhote parivarika kheta abhi bhi pushtom se hastantarita paramparita paddhatiyom se apane bhojana ka vardhana karate haim|
jaise vyavasayika krishi badha़a vyapara bana gaya hai phira bhi utpadana evam krishaka apana labha badha़ane ke lie va adhika badha़a pashudhana badha़ane ke vividha khojana arambha karane lage|
hama dekhenge ki eka utpada ko jaiva pukarane ke lie kisa ki avashyakata hai, jaiva bhojana va krishi kaise vatavarana va hamare svasthaya para prabhava karati hai, jaiva bhojana karane ke labha para vichara vimarsha karengem evam anusandhana jaiva vardhita utpada ke poshaka labhom ke bare mem kya kahate haim|
1995 mem yu esa neshanala ऑragenika stendarda borda ne jaiva ki parubhasha parita ki jo ऑragenika phuda prodakshana ekta ke prabhutva ko ke andara utpadika utpada ke lie suchita karata hua eka ankita shabda|
jaiva krishi eka paristhitika utpadana prabandhana tantra hai jo jaiva vividhata jaivika chakra evam nrida jaivika kriya badha़ata va unnata karata hai|
jaiva krishi, krishi vilaga nivesha evam bala prabandhana tantra prayoga jo paristhikiya samanjasya ka purnauddhara, anarakshana evam unnata karate haim|
jaiva krishi ka prathamika lakshya hai nrida jivana, padapapashu, va manushyom ke anyonyashrita samudaya ke svasthaya evam utpadakata ko sarvottama karata hai|
pashudhana ke jaiva utpadana ka tatva ko tatva jnana pashuom ki svasthaya avashyakataom va prakritika acharana ke lie paristhitiyam jutane mem haim|
jaivika pashudhana ko bahidvari, taji hava, jala surya ki roshani, ghasa va charagaha taka prakritika acharana ke lie paristhitiyam jutane mem hai| 
unako haॉrmonsa pratijivanu ya anya pashu davaem unake chare mem nahim deni ya khilani chahie|
yadi eka pashu rogagrasta ho jata hai va pratijivanu ki avashkata hoti hoti hai, use jaivika nahim mana ja sakata|
vaha romagranthi jo prakritika rupa se shakahari hote haim ko kisi bhi prakara ke pashu anga khilana varjita hai| 
kisi bhi prakara ke pashu upotpadana ka jaiva jaiva bhojana mem kisi bhi samaya samavesha nahim hota hai|
krishakom ko jaivika pramanita hone ke lie apani krishi va yojana sanchalana ke avayava ke rupa mem vistrita abhilekha rakhane chahie, pashuom ke janma se masa ke bajara taka unaki khoja vyakti hameshaa kara sakata hai|
jaba mansa ko jaivika ankitaka kiya jata hai to vaha utpada, nishchaya hi 100% jaivika hai|
yadyapi jaivika phasalem jivanashi ke upayoga ke bina utpadita ki jani chahie, yaha anumanita hai ki 10-25% jaivika phala va sabjiyam sanshleshita jivanashi ke kuchha avashesha antarvishta karate haim| 
yaha varsha, hava va pradushita jala srotom ke karana hai|
jaivika rupa se yogya kahalane ke lie phasalom ko katane se pahale tina sala taka nishiddha padarthom se rahita nrida para ugana chahie|
jaba nashaka jiva santulana se bahara ho jate haim evam paramparagatajaivika vidhiyam jivanashi para niyantrana nahim kara pati haim, krishaka neshanala ऑragenika stendarda borda se dusare utpadajo kama khatare ke mane jate haim ko upayoga karane ki anumati ka anurodha kara sakati hai| 
15 sala ke adhyayana ke anusara pharminga sistama trayala jaivika nrida mem adhika sukshmajivi ansha hota hai jo adhika svastha nrida va padapa banata hai| 
isa adhyayana se nishkarsha nikala ki jaiivaka rupa se ugaye gaye bhaojana adhika uttama sharirika sanrachana mem ugaye jate haim behatara jala nikasa ka prabandha karata hai adhika sukshmajivi kriya ko sahayoga deta hai evam akala ke varshom mem jaivika tantra,paramparagata tantra evam adhika achchha pradarshana kara sakate haim|
jaise loga adhika janakara aura avagata ho rahe haim vaha apane svasthya ko sunishchita karane ke lie kadama utha rahe haim| 
jaivika bhojana ki yu esa mem bikri 1998 mem kula 54 biliyana daॉlara thi parantu sala 2000 mem badha़ kara 7.8 biliyana ho gai thi|
2004 ke vhola phuda marketa ऑragenika phudsa trenda trekara sarvekshana mem paya gaya ki 27% amariri vaha jitana pichhale sala khate the usase adhika jaivika bhojana kha rahe haim|
kailiphorniya dipartamenta ऑpha pestisaidsa regyuleshana dvara kiya gaya eka adhyayana nivarama deta hai ki samvahana jivanashi se vishagrasta vyaktiyom ki sankhya 2000 mem 20% taka badha़ gai|
jivanashi ke bhojana mem upayo para eka badha़ti hui ruchi evam chinta 1996 ke phuda kvaliti protekshana ekta ke parana ka parinama banim, jivanashi ke upayoga va bhojana para prabhava ko punah mulyankana karane ke lie yuesa ipie ko nirdeshita kiya|
balakom evam navajatom para prabhavom para vishesha dhyana diya gaya jinaka kama sharira bhara bhojana ka uchcha upayoga prati sharira bhara jivanashi avasheshom se sambandhita kisi khatare ki ora uchcha anashrayata vidyamana karata hai| 
upabhokta sangathanom ne mai 2000 mem apani bhojana suraksha para 1999 ke prativedana ke adyana ko prakashita karate hue doharaya ki bachchom te roja khane vale bhojana mem kitanashi ke avashesha aneka bara suraksha stara se adhika hote haim|
adyatana ne vintara skvasha, adi rola, angura, nashapati, hari phaliyam, palaka, straberi va vilayati kharabujom mem javanashi avasheshom ka uchcha stara paya|
upabhokta sangathanom ne upabhogataom ko jaivika rupa se vikasita vividhata khasakara phala va sabjiyom ko kharidane para dhyana dene ka anurodha kiya|
jivanashi ka sabase ama varga yuesa mem ऑragenophaoresta hai|
anveshakom ne paya ki ausatana, paramparagata rupa se vikasita bhojana khane vale bachchom ke mutra mem, jaivika bhojana khane valom ki tulana mem opi avashesha ki 8.5 guna matra adhika thi|
adhyayana ne bhruna vikasa para bhi hanikaraka prabhava dikhae|
jivanashi kevala eka ashanka nahim hai, yadyapi pratijivanu ke 70% ka upayoga ajakala pashudhana ko mota karane ke lie kiya jata hai|
krishi pashu 24.6 miliyana paunda pratijivanu hara varsha prapta karate haim, jana svasthaya adhikari aba pashudhana mem kama matra ke pratijivanu upayoga ko adhika sankhya mem logom dvara vaha sankramana pakada़te jo samana dava se chikitsa pratirodha karate haim, se joda़te haim|
amerikana medikala asosieshana ne 2001 ke juna mem krishi mem pratijivanu ke stara para upayoga ka virodha karate hue kiya evam vishva svasthya sangathana ne apane 2001 ke prativedana ne krishakom se anurodha kiya ki vikasa protsahita karane ke lie pratijivanu ka upayoga na karem| 
adhyayana pata laga rahe haim ki upabhoktaom ki antom mem pratijivanu pratirodhi jivanu vyavasayika mansa va kukkutadi mem vikasita hone vale ke samana hai| 
abhi kuchha samaya taka isa bata ka bahuta kama sakshya tha ki jaivika rupa se ugae gaye utpadom mem poshakom ki adhikata hoti hai|
yaha bahuta pahale se jana jata haiki adhika svastha nrida, adhika poshaka gunom vale utpada utpadita karegi, para isa dharana ko samarthana dene ke lie kabhi bhi koi vijnana nahim tha| 
2001 mem poshana visheshajna varjiniya vashingatana ne 41 prakashita adhyayana ka apana punarikshana prakashita kiya jaivika paramparagata rupa se ugae gaye phala, sabjiva anaja ke poshaka tatvom ki tulana karate hue| 
usane yaha paya ki jaivika rupa se ugai gai phasala mem 17% adhika vitamina si 21% adhika lauha, 29% adhika maignishiyama, 13.6% adhka phaॉsphorasa paya jata hai paramparagata rupa se ugae ge utpadom ki tulana mem|
varjiniya vashingatana ne tippani ki, ki jaivika sabjiyom se purushom aura striyom ke lie anushansita vitamina si ka dainika antargrahana prapta ho jata hai parantu unake paramparagata rupom se nahim|
ajakala shrimati vashingatana dvara parinamita samana parinama ko dikhate hue adhika adhyayana haim|
jaivika bhojana ko khane ke svasthya labha satha hi yaha jnana ki kaise paramparagata rupase ugae aura bada़e kiye gaye bhojana griha para kya prabhava dala rahe haim yaha sochate hue aja jaivika bhojana para adhika dhyana dena paryapta hoga|
kyonki adhikatara vyakti apana bhojana sthaniya supara marketa se kharidate haim yaha achchha samachara hai ki adhika se adhika bajara apani dukana mem prakritika va jaivika bhojana muhaiya karava rahe haim|
supara marketa nyuja ke sarvekshana ke parinama ne dikhaya ki 61% upabhokta aba apana jaivika bhojana supara marketa se kharidate haim|
apane samudaya ke lie adhika krishakom ke bajara sthapita karane ke lie adhika samudaya aura svasthaya shakhaem kama kara rahi haim jo sthaniya rupa se ugaya gaya adhika jaivika bhojana upabhoktaom ke lie late haim|
agali bara jaba apa kharadadari karane jaem jaivika sangraha anveshana karane ka dhyana rakhie yaha dekhane ke lie ki yaha parivartana ke vastava mem yogya hai| 
eka bricha prastuta shishu vaha hota hai jisaka sira upara va nichala bhaga niche hota hai|
bricha yaunika janma eka bricha anubhava vale dhaireyavana chikitsaka ke satha surakshita hai| 
yonika rupa se prasava karae ge bachchom ka parinama atiuttama hota hai evam jatilata ke avasara bahuta durlabha hote haim|
yadyapi aise kuchha chikitsaka hai jo bricha bebi ke prasava ke lie taiyara hote haim adhikatara vaha sijeriyana karane ke lie adhika ichchhuka hote haim|
apane apa ko pani mem dubone se apaka emaniyotika phlyuda badhega jo bachche ko palatane mem sahayaka hoga|
tyuba ko kose pani se bhara dijie apane bachche ko kalpana karate hue va palatane ke lie shabdika rupa se protsahita karate hue|
samanya khelana, ludha़kana va tairana bachche ko palatane ke lie protsahita karata hai|
'''
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
tr4w.get_keywords(25)

ValueError: [E088] Text of length 2061600 exceeds maximum of 1000000. The v2.x parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.